In [ ]:
Explain me output of the code. I am trying to understaind feature importance table

# set the GPU
from srs.models.light_gbm import forecast_lgbm_whole_sample_LongShortTerm_w_Optuna


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **************************************
# define dates for training and evaluation 
# **************************************
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE = '2023-01-01'
FINAL_DATE_EXPERIMENTS = '2024-12-31'
n_days_test = (pd.to_datetime(FINAL_DATE_EXPERIMENTS) - pd.to_datetime(INIT_TEST_DATE)).days + (1) # additional adjustment

repo_root = Path.cwd().parents[1]
mapcodes = ["NO1"] # ["NO1","NO2","NO3","NO4","NO5"]
maps_dict = {}

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df = pd.read_csv(csv_path, parse_dates=["time_utc"])
    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz="CET",
        seed=42,
        test_days= n_days_test , # Here it should be 1*365 for 1 year of test, and so on
        dtype=torch.float64,
    )

    # fix potential problems with dates after change time zone..
    train_dates_series       = pd.DatetimeIndex(sorted(train_dates))
    id_init_exp = train_dates_series.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    id_init_test_period = train_dates_series.get_loc(pd.Timestamp(INIT_TEST_DATE))
    id_end_exp = train_dates_series.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t = data_t[id_init_exp:(id_end_exp+1), :,:]
    train_dates = pd.Series(train_dates[id_init_exp:(id_end_exp+1)])

    print(f"days for test data ..{data_t.shape[0] - id_init_test_period}")
    
    maps_dict[code] = {
        "df": df,
        "data_t": data_t,
        "train_t": train_t,
        "train_dates": train_dates,
        "price_t": price_t
    }
maps_dict.keys()

lgbm24_by_zone   = {}
rmse_by_zone  = {}



for z in mapcodes:
    print(f"\n--- {z} ---")
    price_S      = maps_dict[z]["price_t"]
    data_array   = maps_dict[z]["data_t"]
    full_dates      = maps_dict[z]["train_dates"] # <-- here i change a bit the previuous function, prepare_data_tensor, it has all dates
    feature_names= maps_dict[z]["df"].columns[1:]
    full_date_series = pd.DatetimeIndex(sorted(full_dates))

    # evaluation days (all of 2024)
    train_start_idx = full_date_series.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    id_init_eval = full_date_series.get_loc(pd.Timestamp(INIT_TEST_DATE))
    id_end_eval = full_date_series.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    eval_start_idx = id_init_eval
    eval_end_idx  = id_end_eval
    N_s            = (eval_end_idx - eval_start_idx) + 1 # additional adjustment
    full_dates = pd.to_datetime(full_dates)

    

    # new features: WD - dummy for week days, price lags for Mon, Tue and Fri, day-ahead load lag
    WD             = [1,6,7]     
    PRICE_S_LAGS   = [1,2,7]
    DA_LAG         = [0]
    S              = 24
    #D             = 730

    # tensors to collect forecasts for THIS zone
    forecasts_zone = torch.full((N_s, S, 1), float("nan"),
                                dtype=torch.float64, device=device)
    train_end_idx   = id_init_eval - 1

    for n in range(N_s):
        print(f"********************* START NS ... {n} ****************************************************")

        if n == 0:
            reduction = 0
        else:
            reduction = 60 * ((n - 1) // 100 + 1)
        # compute the window bounds
        start_idx = train_start_idx + n + reduction # rolling window (train_start_idx + n)
        end_idx   = train_end_idx  +  n   # inclusive
        
        # slice out the training data & dates
        dat_slice = data_array[start_idx : (end_idx + 2)]
        days      = pd.Series(full_dates[start_idx : (end_idx + 2) ])

        # Generate forecasts using expert_ext model and save them
        result_lgbm_iter            = forecast_lgbm_whole_sample_LongShortTerm_w_Optuna(
        dat                         = dat_slice,
        days                        = days,
        wd                          = WD,
        price_s_lags                = PRICE_S_LAGS,
        da_lag                      = DA_LAG,
        reg_names                   = feature_names,
        fuel_lags                   = [2],
        n_trials_lgbm               = 13, 
        days_for_st_model           = 7
        )

        forecasts_zone[n, :, 0] = result_lgbm_iter['forecasts'].detach().clone().to(forecasts_zone.dtype).to(forecasts_zone.device)

        # Progress tracker (as percentage)
        progress = torch.tensor((n + 1) / N_s * 100, dtype=torch.float64)
        print(f"\r-> {progress.item():.2f}% done", end="")
        print(f"\n********************* END NS.. {n} ****************************************************")
            
    #   shape: (N_s, S)
    true_vals = price_S[eval_start_idx : eval_end_idx + 1].to(device)  
    
    # compute RMSE
    diff = forecasts_zone[:, :, 0] - true_vals
    rmse = torch.sqrt((diff**2).mean()).item()
    
    print(f"Zone {z} LBGM-24h RMSE: {rmse:.4f}")

    lgbm24_by_zone[z] = forecasts_zone[:, :, 0].cpu()
    rmse_by_zone[z]  = rmse

days for test data ..731

--- NO1 ---
********************* START NS ... 0 ****************************************************
[I 2025-06-15 19:44:12,783] A new study created in memory with name: no-name-17e4ed71-5c5e-4dc8-a23e-35d74c6b86b9
[I 2025-06-15 19:44:12,905] Trial 0 finished with value: 0.039454915435123014 and parameters: {'n_estimators': 63, 'learning_rate': 0.014736837565804179, 'num_leaves': 158, 'max_depth': 6, 'min_child_samples': 23, 'subsample': 0.7061772545874583, 'colsample_bytree': 0.9130124598727951, 'reg_alpha': 7.064369216610946, 'reg_lambda': 3.798538571276203, 'min_split_gain': 0.008032072115536543}. Best is trial 0 with value: 0.039454915435123014.
[I 2025-06-15 19:44:13,051] Trial 1 finished with value: 0.027922336438825784 and parameters: {'n_estimators': 142, 'learning_rate': 0.10556057838685406, 'num_leaves': 137, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.8593348921710515, 'colsample_bytree': 0.6203828709664707, 'reg_alpha': 5.803851121692205, 'reg_lambda': 4.468461113007555, 'min_split_gain': 0.046556794282076756}. Best is trial 1 with value: 0.027922336438825784.
[Trial 0] New best: 0.03945
[Trial 1] New best: 0.02792
[I 2025-06-15 19:44:13,510] Trial 2 finished with value: 0.025894835075432613 and parameters: {'n_estimators': 272, 'learning_rate': 0.05336333624424273, 'num_leaves': 61, 'max_depth': 13, 'min_child_samples': 21, 'subsample': 0.606000504386951, 'colsample_bytree': 0.5593201392655143, 'reg_alpha': 2.139736660045739, 'reg_lambda': 2.9349705804622337, 'min_split_gain': 0.01155381087726693}. Best is trial 2 with value: 0.025894835075432613.
[I 2025-06-15 19:44:13,642] Trial 3 finished with value: 0.027187516683473526 and parameters: {'n_estimators': 193, 'learning_rate': 0.20244086191344932, 'num_leaves': 69, 'max_depth': 10, 'min_child_samples': 10, 'subsample': 0.8928255248303939, 'colsample_bytree': 0.8768323588363514, 'reg_alpha': 7.232094955544009, 'reg_lambda': 8.379656653621998, 'min_split_gain': 0.03203315506383729}. Best is trial 2 with value: 0.025894835075432613.
[Trial 2] New best: 0.02589
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:13,927] Trial 4 finished with value: 0.026355530186274973 and parameters: {'n_estimators': 233, 'learning_rate': 0.08863611839300542, 'num_leaves': 111, 'max_depth': 6, 'min_child_samples': 15, 'subsample': 0.7554115275454163, 'colsample_bytree': 0.694628183846953, 'reg_alpha': 6.437541933074943, 'reg_lambda': 6.460261378501906, 'min_split_gain': 0.04485090031434871}. Best is trial 2 with value: 0.025894835075432613.
[I 2025-06-15 19:44:14,091] Trial 5 finished with value: 0.030998187265761534 and parameters: {'n_estimators': 140, 'learning_rate': 0.0546892142276788, 'num_leaves': 97, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.9591283256789497, 'colsample_bytree': 0.5303885740671279, 'reg_alpha': 1.5299641684368948, 'reg_lambda': 0.36982004399354285, 'min_split_gain': 0.029887125994059356}. Best is trial 2 with value: 0.025894835075432613.
[I 2025-06-15 19:44:14,092] A new study created in memory with name: no-name-aee9ba39-f503-4768-89d5-0d951dad9fed
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 19:44:14,154] Trial 0 finished with value: 0.03498040251007065 and parameters: {'n_estimators': 239, 'learning_rate': 0.011163742364558436, 'num_leaves': 211, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.7133114376080227, 'colsample_bytree': 0.7426774750970757, 'reg_alpha': 2.7776226204306633, 'reg_lambda': 8.976475186559348, 'min_split_gain': 0.03331993017247747}. Best is trial 0 with value: 0.03498040251007065.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1433
1           Price_lag_2        1249
3         Load_DA_lag_0        1077
14      pct_chg_Load_DA        1030
5       WindOn_DA_lag_0         742
2           Price_lag_7         714
11           NGas_lag_2         568
15       lag168_Load_DA         544
16  roll2d_median_Price         532
10           Coal_lag_2         531
13            EUA_lag_2         464
12            Oil_lag_2         326
7                  WD_1          29
9                  WD_7          11
8                  WD_6          10
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         394
15       lag168_Load_DA          75
3         Load_DA_lag_0          52
1           Price_lag_2          46
13            EUA_lag_2           2
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 0.14% done
********************* END NS.. 0 ****************************************************
********************* START NS ... 1 ****************************************************
[I 2025-06-15 19:44:15,124] A new study created in memory with name: no-name-c4800927-c382-4b08-8420-05b7f70a6f7d
[I 2025-06-15 19:44:15,480] Trial 0 finished with value: 0.01118047856731539 and parameters: {'n_estimators': 100, 'learning_rate': 0.0856817998952099, 'num_leaves': 89, 'max_depth': 13, 'min_child_samples': 12, 'subsample': 0.5469271226150079, 'colsample_bytree': 0.9411753959091956, 'reg_alpha': 0.6992036051725714, 'reg_lambda': 7.68882437428447, 'min_split_gain': 0.01216748049310678}. Best is trial 0 with value: 0.01118047856731539.
[Trial 0] New best: 0.01118
[I 2025-06-15 19:44:15,865] Trial 1 finished with value: 0.009249990694146287 and parameters: {'n_estimators': 134, 'learning_rate': 0.08543448649628956, 'num_leaves': 246, 'max_depth': 12, 'min_child_samples': 20, 'subsample': 0.6746428150527728, 'colsample_bytree': 0.7616947012047948, 'reg_alpha': 0.031577676695693224, 'reg_lambda': 4.456830386034411, 'min_split_gain': 0.04956891054269045}. Best is trial 1 with value: 0.009249990694146287.
[Trial 1] New best: 0.00925
[I 2025-06-15 19:44:16,587] Trial 2 finished with value: 0.006738192360439846 and parameters: {'n_estimators': 173, 'learning_rate': 0.014250627530950927, 'num_leaves': 210, 'max_depth': 10, 'min_child_samples': 10, 'subsample': 0.6310311707262202, 'colsample_bytree': 0.6732226797071061, 'reg_alpha': 1.4781391437831626, 'reg_lambda': 7.519833413419579, 'min_split_gain': 0.03968759022983602}. Best is trial 2 with value: 0.006738192360439846.
[Trial 2] New best: 0.00674
[I 2025-06-15 19:44:16,840] Trial 3 finished with value: 0.008906092977263122 and parameters: {'n_estimators': 98, 'learning_rate': 0.04823404214789415, 'num_leaves': 34, 'max_depth': 10, 'min_child_samples': 15, 'subsample': 0.9137979676947685, 'colsample_bytree': 0.8459820529833402, 'reg_alpha': 1.6200010615315896, 'reg_lambda': 0.006571794232070882, 'min_split_gain': 0.015071770182359629}. Best is trial 2 with value: 0.006738192360439846.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:17,218] Trial 4 finished with value: 0.009189963249864543 and parameters: {'n_estimators': 186, 'learning_rate': 0.021223744984309758, 'num_leaves': 172, 'max_depth': 9, 'min_child_samples': 7, 'subsample': 0.5087380422768554, 'colsample_bytree': 0.9291204320876847, 'reg_alpha': 6.759948917791951, 'reg_lambda': 3.4941841350866385, 'min_split_gain': 0.031677843271717875}. Best is trial 2 with value: 0.006738192360439846.
[I 2025-06-15 19:44:17,368] Trial 5 finished with value: 0.009111710903558029 and parameters: {'n_estimators': 100, 'learning_rate': 0.14952245011841775, 'num_leaves': 222, 'max_depth': 9, 'min_child_samples': 10, 'subsample': 0.7797016898503937, 'colsample_bytree': 0.5766913942366597, 'reg_alpha': 2.91369598660679, 'reg_lambda': 3.527297668993631, 'min_split_gain': 0.039482136774970755}. Best is trial 2 with value: 0.006738192360439846.
[I 2025-06-15 19:44:17,369] A new study created in memory with name: no-name-8d92444f-f4ce-4cac-937f-77779b0b9e5a
[I 2025-06-15 19:44:17,391] Trial 0 finished with value: 0.007567735495531605 and parameters: {'n_estimators': 268, 'learning_rate': 0.10471909108953655, 'num_leaves': 107, 'max_depth': 4, 'min_child_samples': 7, 'subsample': 0.7626413436030117, 'colsample_bytree': 0.7233583675960451, 'reg_alpha': 5.3115997515413405, 'reg_lambda': 6.989690722747747, 'min_split_gain': 0.024894556388914253}. Best is trial 0 with value: 0.007567735495531605.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2370
1           Price_lag_2        1852
14      pct_chg_Load_DA         985
16  roll2d_median_Price         686
2           Price_lag_7         612
11           NGas_lag_2         415
3         Load_DA_lag_0         372
10           Coal_lag_2         372
12            Oil_lag_2         257
13            EUA_lag_2         251
15       lag168_Load_DA         212
5       WindOn_DA_lag_0          98
9                  WD_7          12
8                  WD_6           8
7                  WD_1           4
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          45
15       lag168_Load_DA          14
3         Load_DA_lag_0          10
1           Price_lag_2           2
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 0.27% done
********************* END NS.. 1 ****************************************************
********************* START NS ... 2 ****************************************************
[I 2025-06-15 19:44:18,218] A new study created in memory with name: no-name-8266870c-c4f8-442d-a42f-c9fcbfff2b4b
[I 2025-06-15 19:44:18,721] Trial 0 finished with value: 0.005832959601448413 and parameters: {'n_estimators': 248, 'learning_rate': 0.01110376072572594, 'num_leaves': 254, 'max_depth': 12, 'min_child_samples': 11, 'subsample': 0.7118119799806568, 'colsample_bytree': 0.7948645655110806, 'reg_alpha': 6.865991960620187, 'reg_lambda': 3.6353133799514294, 'min_split_gain': 0.03448582129203791}. Best is trial 0 with value: 0.005832959601448413.
[I 2025-06-15 19:44:18,909] Trial 1 finished with value: 0.013189769740336824 and parameters: {'n_estimators': 76, 'learning_rate': 0.022881611330852824, 'num_leaves': 166, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.6375795127359658, 'colsample_bytree': 0.7871603819236519, 'reg_alpha': 9.709387243711797, 'reg_lambda': 1.7897632694493004, 'min_split_gain': 0.005527330260289682}. Best is trial 0 with value: 0.005832959601448413.
[Trial 0] New best: 0.00583
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:19,048] Trial 2 finished with value: 0.004328289905740292 and parameters: {'n_estimators': 138, 'learning_rate': 0.20770325375692983, 'num_leaves': 55, 'max_depth': 11, 'min_child_samples': 25, 'subsample': 0.6625872927157681, 'colsample_bytree': 0.9095785426593788, 'reg_alpha': 6.102905271183382, 'reg_lambda': 1.5937880485616085, 'min_split_gain': 0.018520510757433595}. Best is trial 2 with value: 0.004328289905740292.
[Trial 2] New best: 0.00433
[I 2025-06-15 19:44:19,298] Trial 3 finished with value: 0.004115384550851614 and parameters: {'n_estimators': 230, 'learning_rate': 0.12717961527499475, 'num_leaves': 203, 'max_depth': 7, 'min_child_samples': 30, 'subsample': 0.8740120590870102, 'colsample_bytree': 0.9635354549612922, 'reg_alpha': 9.800560341054545, 'reg_lambda': 0.18846736670085296, 'min_split_gain': 0.04735283965843909}. Best is trial 3 with value: 0.004115384550851614.
[I 2025-06-15 19:44:19,433] Trial 4 finished with value: 0.003863995740765932 and parameters: {'n_estimators': 92, 'learning_rate': 0.16749392479378775, 'num_leaves': 230, 'max_depth': 4, 'min_child_samples': 22, 'subsample': 0.6676521039294671, 'colsample_bytree': 0.8497598771366945, 'reg_alpha': 4.196844013502911, 'reg_lambda': 7.930985939079407, 'min_split_gain': 0.0013981255544120831}. Best is trial 4 with value: 0.003863995740765932.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:44:19,679] Trial 5 finished with value: 0.004241073918759334 and parameters: {'n_estimators': 209, 'learning_rate': 0.043586976314584754, 'num_leaves': 249, 'max_depth': 4, 'min_child_samples': 18, 'subsample': 0.9068882182103888, 'colsample_bytree': 0.9344372516393554, 'reg_alpha': 1.1024293885934744, 'reg_lambda': 8.010444284831452, 'min_split_gain': 0.010866953098325428}. Best is trial 4 with value: 0.003863995740765932.
[I 2025-06-15 19:44:19,681] A new study created in memory with name: no-name-f6d017ca-3603-4966-a5b3-eef6d31b597e
[I 2025-06-15 19:44:19,706] Trial 0 finished with value: 0.04197123312133067 and parameters: {'n_estimators': 254, 'learning_rate': 0.0738883463622415, 'num_leaves': 142, 'max_depth': 4, 'min_child_samples': 23, 'subsample': 0.5819442130803499, 'colsample_bytree': 0.6854364904799772, 'reg_alpha': 4.847397791217615, 'reg_lambda': 6.0782674483168755, 'min_split_gain': 0.044284934973976814}. Best is trial 0 with value: 0.04197123312133067.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         298
14      pct_chg_Load_DA         172
1           Price_lag_2         149
3         Load_DA_lag_0         107
11           NGas_lag_2          82
2           Price_lag_7          77
13            EUA_lag_2          67
16  roll2d_median_Price          64
5       WindOn_DA_lag_0          63
15       lag168_Load_DA          58
12            Oil_lag_2          56
10           Coal_lag_2          39
9                  WD_7           6
7                  WD_1           4
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          57
15       lag168_Load_DA          12
3         Load_DA_lag_0           8
1           Price_lag_2           5
10           Coal_lag_2           3
13            EUA_lag_2           2
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 0.41% done
********************* END NS.. 2 ****************************************************
********************* START NS ... 3 ****************************************************
[I 2025-06-15 19:44:20,275] A new study created in memory with name: no-name-e4235195-ef02-4328-9d2c-e81e350b51fa
[I 2025-06-15 19:44:20,703] Trial 0 finished with value: 0.005129495093641857 and parameters: {'n_estimators': 164, 'learning_rate': 0.024387959072226826, 'num_leaves': 68, 'max_depth': 7, 'min_child_samples': 24, 'subsample': 0.7074895305742628, 'colsample_bytree': 0.5065387030102462, 'reg_alpha': 0.19526637067555086, 'reg_lambda': 3.5787157354918886, 'min_split_gain': 0.005303305929427221}. Best is trial 0 with value: 0.005129495093641857.
[Trial 0] New best: 0.00513
[I 2025-06-15 19:44:20,913] Trial 1 finished with value: 0.004017851889815741 and parameters: {'n_estimators': 83, 'learning_rate': 0.18452363636710312, 'num_leaves': 104, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.9853039221323343, 'colsample_bytree': 0.5650711946242122, 'reg_alpha': 0.919910399988183, 'reg_lambda': 4.711404079704369, 'min_split_gain': 0.024997568108786585}. Best is trial 1 with value: 0.004017851889815741.
[Trial 1] New best: 0.00402
[I 2025-06-15 19:44:21,154] Trial 2 finished with value: 0.005774865334181992 and parameters: {'n_estimators': 180, 'learning_rate': 0.019170173007605637, 'num_leaves': 165, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.5443884495680391, 'colsample_bytree': 0.9428545294119064, 'reg_alpha': 2.5072344389634815, 'reg_lambda': 4.243297100837316, 'min_split_gain': 0.026502026832391074}. Best is trial 1 with value: 0.004017851889815741.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:21,547] Trial 3 finished with value: 0.004524078286734841 and parameters: {'n_estimators': 288, 'learning_rate': 0.09613557814974692, 'num_leaves': 214, 'max_depth': 13, 'min_child_samples': 16, 'subsample': 0.9125868570195894, 'colsample_bytree': 0.7787274524543715, 'reg_alpha': 7.468051606995144, 'reg_lambda': 8.971641343566523, 'min_split_gain': 0.004258961617897511}. Best is trial 1 with value: 0.004017851889815741.
[I 2025-06-15 19:44:21,684] Trial 4 finished with value: 0.004769106580302396 and parameters: {'n_estimators': 126, 'learning_rate': 0.21166013971417966, 'num_leaves': 170, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.5745473252292499, 'colsample_bytree': 0.6427377005337347, 'reg_alpha': 4.776815321155081, 'reg_lambda': 7.000027210242398, 'min_split_gain': 0.03890768736721526}. Best is trial 1 with value: 0.004017851889815741.
[I 2025-06-15 19:44:21,686] A new study created in memory with name: no-name-09648211-97f8-47ba-bd67-66667dd401bd
[I 2025-06-15 19:44:21,718] Trial 0 finished with value: 0.11456308905147411 and parameters: {'n_estimators': 203, 'learning_rate': 0.017485834132122142, 'num_leaves': 246, 'max_depth': 11, 'min_child_samples': 20, 'subsample': 0.7298706428414536, 'colsample_bytree': 0.6433412941501369, 'reg_alpha': 2.1707920676167403, 'reg_lambda': 1.107806434880787, 'min_split_gain': 0.024434667501039856}. Best is trial 0 with value: 0.11456308905147411.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         400
0           Price_lag_1         373
3         Load_DA_lag_0         329
14      pct_chg_Load_DA         271
5       WindOn_DA_lag_0         263
2           Price_lag_7         230
16  roll2d_median_Price         189
10           Coal_lag_2         176
15       lag168_Load_DA         174
13            EUA_lag_2         171
11           NGas_lag_2         136
12            Oil_lag_2         113
7                  WD_1          10
9                  WD_7           6
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         275
15       lag168_Load_DA          72
1           Price_lag_2          45
3         Load_DA_lag_0          41
5       WindOn_DA_lag_0          27
2           Price_lag_7          25
10           Coal_lag_2          18
16  roll2d_median_Price           2
11           NGas_lag_2           1
7                  WD_1           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 0.55% done
********************* END NS.. 3 ****************************************************
********************* START NS ... 4 ****************************************************
[I 2025-06-15 19:44:22,380] A new study created in memory with name: no-name-6870d94c-9a88-41f3-ad87-9dfda5658e9d
[I 2025-06-15 19:44:22,610] Trial 0 finished with value: 0.00405557337705889 and parameters: {'n_estimators': 283, 'learning_rate': 0.2155772470952722, 'num_leaves': 88, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.7731056276300727, 'colsample_bytree': 0.9102667253823162, 'reg_alpha': 0.7434343141527944, 'reg_lambda': 9.87865329838922, 'min_split_gain': 0.014294052412481467}. Best is trial 0 with value: 0.00405557337705889.
[Trial 0] New best: 0.00406
[I 2025-06-15 19:44:23,224] Trial 1 finished with value: 0.007444677185139292 and parameters: {'n_estimators': 292, 'learning_rate': 0.0145741499312702, 'num_leaves': 106, 'max_depth': 14, 'min_child_samples': 23, 'subsample': 0.6290314843036622, 'colsample_bytree': 0.6469383827921362, 'reg_alpha': 7.975643783167571, 'reg_lambda': 4.603267071317659, 'min_split_gain': 0.017564263283141448}. Best is trial 0 with value: 0.00405557337705889.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:23,492] Trial 2 finished with value: 0.004515308225825216 and parameters: {'n_estimators': 204, 'learning_rate': 0.19857242025675773, 'num_leaves': 39, 'max_depth': 8, 'min_child_samples': 13, 'subsample': 0.5745932587062449, 'colsample_bytree': 0.8781650567937166, 'reg_alpha': 5.918277828473766, 'reg_lambda': 3.4376447638223127, 'min_split_gain': 0.00019868374593642326}. Best is trial 0 with value: 0.00405557337705889.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:44:23,706] Trial 3 finished with value: 0.005541395019309281 and parameters: {'n_estimators': 298, 'learning_rate': 0.1799507491413008, 'num_leaves': 225, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.841461687988704, 'colsample_bytree': 0.5649988893861478, 'reg_alpha': 4.39795827384366, 'reg_lambda': 1.9003229032570368, 'min_split_gain': 0.03396427539352611}. Best is trial 0 with value: 0.00405557337705889.
[I 2025-06-15 19:44:23,708] A new study created in memory with name: no-name-bad896f6-cd6e-4e23-ab48-d6c174030e5f
[I 2025-06-15 19:44:23,753] Trial 0 finished with value: 0.05593026091345535 and parameters: {'n_estimators': 231, 'learning_rate': 0.05020040088377663, 'num_leaves': 185, 'max_depth': 14, 'min_child_samples': 10, 'subsample': 0.6339236221611279, 'colsample_bytree': 0.8628625964985731, 'reg_alpha': 4.272535398589831, 'reg_lambda': 5.779992833637646, 'min_split_gain': 0.015015937027945582}. Best is trial 0 with value: 0.05593026091345535.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         441
14      pct_chg_Load_DA         399
1           Price_lag_2         320
3         Load_DA_lag_0         309
5       WindOn_DA_lag_0         213
2           Price_lag_7         200
16  roll2d_median_Price         191
15       lag168_Load_DA         151
11           NGas_lag_2         136
10           Coal_lag_2         133
12            Oil_lag_2         119
13            EUA_lag_2         101
9                  WD_7           8
7                  WD_1           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         129
15       lag168_Load_DA          37
3         Load_DA_lag_0          20
5       WindOn_DA_lag_0          16
1           Price_lag_2          10
2           Price_lag_7           5
10           Coal_lag_2           3
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 0.68% done
********************* END NS.. 4 ****************************************************
********************* START NS ... 5 ****************************************************
[I 2025-06-15 19:44:24,565] A new study created in memory with name: no-name-6a6900fa-cd81-4c95-963f-ad69175e05ae
[I 2025-06-15 19:44:24,839] Trial 0 finished with value: 0.009324284912943537 and parameters: {'n_estimators': 66, 'learning_rate': 0.05465984511145704, 'num_leaves': 81, 'max_depth': 8, 'min_child_samples': 13, 'subsample': 0.7168376861099148, 'colsample_bytree': 0.5283078804187759, 'reg_alpha': 5.757621130843303, 'reg_lambda': 8.17929803879873, 'min_split_gain': 0.0147559677623957}. Best is trial 0 with value: 0.009324284912943537.
[Trial 0] New best: 0.00932
[I 2025-06-15 19:44:25,208] Trial 1 finished with value: 0.006587225389119465 and parameters: {'n_estimators': 177, 'learning_rate': 0.03644404277135564, 'num_leaves': 46, 'max_depth': 14, 'min_child_samples': 20, 'subsample': 0.8637349640996647, 'colsample_bytree': 0.596956486706925, 'reg_alpha': 8.184203115013789, 'reg_lambda': 9.69681513392591, 'min_split_gain': 0.023592658272589842}. Best is trial 1 with value: 0.006587225389119465.
[Trial 1] New best: 0.00659
[I 2025-06-15 19:44:25,469] Trial 2 finished with value: 0.005078960499537814 and parameters: {'n_estimators': 233, 'learning_rate': 0.07598760069683948, 'num_leaves': 99, 'max_depth': 7, 'min_child_samples': 16, 'subsample': 0.8917193924669898, 'colsample_bytree': 0.6692744308615435, 'reg_alpha': 8.910461325418657, 'reg_lambda': 1.9772088493915152, 'min_split_gain': 0.045059059165613945}. Best is trial 2 with value: 0.005078960499537814.
[I 2025-06-15 19:44:25,637] Trial 3 finished with value: 0.00580632811907223 and parameters: {'n_estimators': 97, 'learning_rate': 0.2801736850783864, 'num_leaves': 155, 'max_depth': 6, 'min_child_samples': 7, 'subsample': 0.8977331504461081, 'colsample_bytree': 0.7492967301920322, 'reg_alpha': 0.008800886595444757, 'reg_lambda': 4.601193313486575, 'min_split_gain': 0.032986480647528696}. Best is trial 2 with value: 0.005078960499537814.
[Trial 2] New best: 0.00508
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:25,966] Trial 4 finished with value: 0.014477706846797174 and parameters: {'n_estimators': 103, 'learning_rate': 0.019469556317724034, 'num_leaves': 195, 'max_depth': 4, 'min_child_samples': 7, 'subsample': 0.8283046074281921, 'colsample_bytree': 0.6426009485506836, 'reg_alpha': 2.9992835553748565, 'reg_lambda': 8.92147915348839, 'min_split_gain': 0.015560856447952654}. Best is trial 2 with value: 0.005078960499537814.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:44:26,538] Trial 5 finished with value: 0.004877305694190879 and parameters: {'n_estimators': 104, 'learning_rate': 0.12308042312777888, 'num_leaves': 75, 'max_depth': 7, 'min_child_samples': 23, 'subsample': 0.5820047822025898, 'colsample_bytree': 0.7564151201535491, 'reg_alpha': 7.483213586230405, 'reg_lambda': 5.889951744450857, 'min_split_gain': 0.015433307505237498}. Best is trial 5 with value: 0.004877305694190879.
[I 2025-06-15 19:44:26,540] A new study created in memory with name: no-name-a27ff905-d1fc-4f5b-97d5-851b7fbcf526
[I 2025-06-15 19:44:26,621] Trial 0 finished with value: 0.0377256359463794 and parameters: {'n_estimators': 269, 'learning_rate': 0.011391652877551652, 'num_leaves': 124, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.727706378117662, 'colsample_bytree': 0.6214304850069621, 'reg_alpha': 4.6158880603371895, 'reg_lambda': 4.650002095678493, 'min_split_gain': 0.02054479552323875}. Best is trial 0 with value: 0.0377256359463794.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         365
1           Price_lag_2         251
14      pct_chg_Load_DA         213
3         Load_DA_lag_0         144
13            EUA_lag_2         110
11           NGas_lag_2         106
2           Price_lag_7          96
15       lag168_Load_DA          87
5       WindOn_DA_lag_0          81
16  roll2d_median_Price          80
10           Coal_lag_2          74
12            Oil_lag_2          71
9                  WD_7           5
7                  WD_1           2
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         355
1           Price_lag_2          68
3         Load_DA_lag_0          62
15       lag168_Load_DA          41
5       WindOn_DA_lag_0          23
16  roll2d_median_Price           4
10           Coal_lag_2           3
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 0.82% done
********************* END NS.. 5 ****************************************************
********************* START NS ... 6 ****************************************************
[I 2025-06-15 19:44:27,684] A new study created in memory with name: no-name-b342a4e3-70a2-4dfe-9c4e-e5e4907abae3
[I 2025-06-15 19:44:28,012] Trial 0 finished with value: 0.005233813212077324 and parameters: {'n_estimators': 99, 'learning_rate': 0.07729330348553697, 'num_leaves': 150, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.9364899417139565, 'colsample_bytree': 0.9006727138248516, 'reg_alpha': 4.640916301635114, 'reg_lambda': 1.1312531053045305, 'min_split_gain': 0.024742429152225}. Best is trial 0 with value: 0.005233813212077324.
[Trial 0] New best: 0.00523
[I 2025-06-15 19:44:28,536] Trial 1 finished with value: 0.007376348920308448 and parameters: {'n_estimators': 266, 'learning_rate': 0.028262839910655462, 'num_leaves': 130, 'max_depth': 4, 'min_child_samples': 28, 'subsample': 0.6828591642105315, 'colsample_bytree': 0.6946900343115976, 'reg_alpha': 9.82169614738805, 'reg_lambda': 0.7314098200014807, 'min_split_gain': 0.031444505019256185}. Best is trial 0 with value: 0.005233813212077324.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:28,843] Trial 2 finished with value: 0.0057638435702120835 and parameters: {'n_estimators': 266, 'learning_rate': 0.2026944441093077, 'num_leaves': 81, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.7388501344787617, 'colsample_bytree': 0.6587207449191382, 'reg_alpha': 3.8473227142600575, 'reg_lambda': 2.2350883498281213, 'min_split_gain': 0.03573108316994993}. Best is trial 0 with value: 0.005233813212077324.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:44:29,214] Trial 3 finished with value: 0.006615052252520966 and parameters: {'n_estimators': 109, 'learning_rate': 0.06486073668965228, 'num_leaves': 79, 'max_depth': 4, 'min_child_samples': 10, 'subsample': 0.8022522301241289, 'colsample_bytree': 0.7929118271441014, 'reg_alpha': 9.208025580239704, 'reg_lambda': 4.839869657989659, 'min_split_gain': 0.00759997806512791}. Best is trial 0 with value: 0.005233813212077324.
[I 2025-06-15 19:44:29,217] A new study created in memory with name: no-name-ee63a3bf-b1db-4104-8841-f99ee4828e0c
[I 2025-06-15 19:44:29,239] Trial 0 finished with value: 0.0552101884948471 and parameters: {'n_estimators': 84, 'learning_rate': 0.07891913054247168, 'num_leaves': 157, 'max_depth': 14, 'min_child_samples': 18, 'subsample': 0.9071207166875925, 'colsample_bytree': 0.9809330561018008, 'reg_alpha': 8.562930709283222, 'reg_lambda': 0.4153028761233568, 'min_split_gain': 0.030118463460710926}. Best is trial 0 with value: 0.0552101884948471.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         694
14      pct_chg_Load_DA         428
1           Price_lag_2         331
3         Load_DA_lag_0         271
11           NGas_lag_2         201
2           Price_lag_7         179
16  roll2d_median_Price         166
5       WindOn_DA_lag_0         142
15       lag168_Load_DA         141
13            EUA_lag_2         134
12            Oil_lag_2         119
10           Coal_lag_2         100
7                  WD_1           7
9                  WD_7           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          47
3         Load_DA_lag_0           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 0.96% done
********************* END NS.. 6 ****************************************************
********************* START NS ... 7 ****************************************************
[I 2025-06-15 19:44:30,188] A new study created in memory with name: no-name-da29ad1b-9bf6-41ac-af26-39ab5227c1cf
[I 2025-06-15 19:44:31,030] Trial 0 finished with value: 0.0070694913607234665 and parameters: {'n_estimators': 186, 'learning_rate': 0.026773619976404678, 'num_leaves': 103, 'max_depth': 7, 'min_child_samples': 20, 'subsample': 0.8456212006445083, 'colsample_bytree': 0.6565269184433513, 'reg_alpha': 0.47303607989908625, 'reg_lambda': 6.600906120088546, 'min_split_gain': 0.03270404970464339}. Best is trial 0 with value: 0.0070694913607234665.
[Trial 0] New best: 0.00707
[I 2025-06-15 19:44:31,466] Trial 1 finished with value: 0.005139795133763929 and parameters: {'n_estimators': 132, 'learning_rate': 0.19632056825007296, 'num_leaves': 221, 'max_depth': 12, 'min_child_samples': 26, 'subsample': 0.7781769225908142, 'colsample_bytree': 0.9802446011939101, 'reg_alpha': 7.147165220030712, 'reg_lambda': 2.1362203913688527, 'min_split_gain': 8.577533077589461e-05}. Best is trial 1 with value: 0.005139795133763929.
[Trial 1] New best: 0.00514
[I 2025-06-15 19:44:31,846] Trial 2 finished with value: 0.006940883013801917 and parameters: {'n_estimators': 237, 'learning_rate': 0.16642075433704034, 'num_leaves': 136, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.9784265546946926, 'colsample_bytree': 0.6185485487482576, 'reg_alpha': 5.339148163580298, 'reg_lambda': 1.4278087229458702, 'min_split_gain': 0.025480244526775414}. Best is trial 1 with value: 0.005139795133763929.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:32,480] Trial 3 finished with value: 0.006171205385530565 and parameters: {'n_estimators': 130, 'learning_rate': 0.07808621185901221, 'num_leaves': 167, 'max_depth': 13, 'min_child_samples': 1, 'subsample': 0.7088071319651463, 'colsample_bytree': 0.7047932156152011, 'reg_alpha': 3.6777198261262543, 'reg_lambda': 5.225657335984017, 'min_split_gain': 0.041615552125096225}. Best is trial 1 with value: 0.005139795133763929.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:44:33,153] Trial 4 finished with value: 0.005890834813713434 and parameters: {'n_estimators': 138, 'learning_rate': 0.01764522363807616, 'num_leaves': 168, 'max_depth': 7, 'min_child_samples': 16, 'subsample': 0.887321576873261, 'colsample_bytree': 0.8405109135504639, 'reg_alpha': 1.4027218392866536, 'reg_lambda': 1.677142345594762, 'min_split_gain': 0.001160180819948309}. Best is trial 1 with value: 0.005139795133763929.
[I 2025-06-15 19:44:33,156] A new study created in memory with name: no-name-73f8e4f9-40c2-4cd1-b460-fb7c60a1dfc0
[I 2025-06-15 19:44:33,184] Trial 0 finished with value: 0.05382496913044571 and parameters: {'n_estimators': 87, 'learning_rate': 0.14746060889081572, 'num_leaves': 193, 'max_depth': 6, 'min_child_samples': 26, 'subsample': 0.7599742552921964, 'colsample_bytree': 0.8746258808669025, 'reg_alpha': 6.674548703137409, 'reg_lambda': 2.6800567142098086, 'min_split_gain': 0.01345811688506049}. Best is trial 0 with value: 0.05382496913044571.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         507
14      pct_chg_Load_DA         440
3         Load_DA_lag_0         387
5       WindOn_DA_lag_0         335
1           Price_lag_2         298
15       lag168_Load_DA         269
2           Price_lag_7         247
13            EUA_lag_2         224
12            Oil_lag_2         221
16  roll2d_median_Price         196
11           NGas_lag_2         193
10           Coal_lag_2         185
9                  WD_7          11
7                  WD_1           8
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          29
3         Load_DA_lag_0          10
1           Price_lag_2           3
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 1.09% done
********************* END NS.. 7 ****************************************************
********************* START NS ... 8 ****************************************************
[I 2025-06-15 19:44:34,159] A new study created in memory with name: no-name-c576838f-b054-47ce-bf0c-7f1988aebf98
[I 2025-06-15 19:44:34,570] Trial 0 finished with value: 0.009195312010100718 and parameters: {'n_estimators': 261, 'learning_rate': 0.0701333477897768, 'num_leaves': 102, 'max_depth': 15, 'min_child_samples': 14, 'subsample': 0.6615898501257901, 'colsample_bytree': 0.8339440136257728, 'reg_alpha': 4.145968347911907, 'reg_lambda': 5.480519957260097, 'min_split_gain': 0.014617002069321084}. Best is trial 0 with value: 0.009195312010100718.
[Trial 0] New best: 0.00920
[I 2025-06-15 19:44:35,255] Trial 1 finished with value: 0.010828304420942096 and parameters: {'n_estimators': 192, 'learning_rate': 0.012192320276545255, 'num_leaves': 106, 'max_depth': 7, 'min_child_samples': 14, 'subsample': 0.6354421789481457, 'colsample_bytree': 0.9461763046420499, 'reg_alpha': 6.21593766675683, 'reg_lambda': 3.8821889761292274, 'min_split_gain': 0.0045803776366603825}. Best is trial 0 with value: 0.009195312010100718.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:36,485] Trial 2 finished with value: 0.008559365657691002 and parameters: {'n_estimators': 285, 'learning_rate': 0.025635930640981523, 'num_leaves': 248, 'max_depth': 9, 'min_child_samples': 11, 'subsample': 0.8743882976120549, 'colsample_bytree': 0.9384712071562373, 'reg_alpha': 4.987967598027165, 'reg_lambda': 7.694846199233076, 'min_split_gain': 0.0168787675008339}. Best is trial 2 with value: 0.008559365657691002.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:44:37,535] Trial 3 finished with value: 0.026492129179097792 and parameters: {'n_estimators': 154, 'learning_rate': 0.010390732532054005, 'num_leaves': 253, 'max_depth': 15, 'min_child_samples': 1, 'subsample': 0.9811216520806478, 'colsample_bytree': 0.5950880795616846, 'reg_alpha': 2.664200798328107, 'reg_lambda': 1.7993111074721857, 'min_split_gain': 0.023134808884257032}. Best is trial 2 with value: 0.008559365657691002.
[I 2025-06-15 19:44:37,537] A new study created in memory with name: no-name-2a12fec1-eab6-4aa1-a6b8-0d73e281c3ae
[I 2025-06-15 19:44:37,559] Trial 0 finished with value: 0.23279980117647028 and parameters: {'n_estimators': 50, 'learning_rate': 0.025394998869937728, 'num_leaves': 154, 'max_depth': 8, 'min_child_samples': 24, 'subsample': 0.9477933449292342, 'colsample_bytree': 0.8035331400711212, 'reg_alpha': 0.9013309466312858, 'reg_lambda': 4.28030325934728, 'min_split_gain': 0.015018858241503215}. Best is trial 0 with value: 0.23279980117647028.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2309
14      pct_chg_Load_DA        1408
1           Price_lag_2        1228
3         Load_DA_lag_0         966
11           NGas_lag_2         725
2           Price_lag_7         705
16  roll2d_median_Price         660
5       WindOn_DA_lag_0         541
15       lag168_Load_DA         481
12            Oil_lag_2         476
13            EUA_lag_2         425
10           Coal_lag_2         404
7                  WD_1          38
9                  WD_7          32
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         120
3         Load_DA_lag_0          23
1           Price_lag_2          14
15       lag168_Load_DA           9
16  roll2d_median_Price           5
5       WindOn_DA_lag_0           4
11           NGas_lag_2           2
14      pct_chg_Load_DA           2
2           Price_lag_7           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
8                  WD_6           0
-> 1.23% done
********************* END NS.. 8 ****************************************************
********************* START NS ... 9 ****************************************************
[I 2025-06-15 19:44:38,953] A new study created in memory with name: no-name-42978fc9-a95f-4e3b-8738-8078d1c5fc4c
[I 2025-06-15 19:44:39,181] Trial 0 finished with value: 0.012005636027182333 and parameters: {'n_estimators': 215, 'learning_rate': 0.14843729856963145, 'num_leaves': 123, 'max_depth': 12, 'min_child_samples': 5, 'subsample': 0.7658607607461312, 'colsample_bytree': 0.6953816391357572, 'reg_alpha': 2.4535062276188615, 'reg_lambda': 2.3701119387720326, 'min_split_gain': 0.018833089839865765}. Best is trial 0 with value: 0.012005636027182333.
[I 2025-06-15 19:44:39,311] Trial 1 finished with value: 0.012392699682363036 and parameters: {'n_estimators': 81, 'learning_rate': 0.16311535979078967, 'num_leaves': 82, 'max_depth': 6, 'min_child_samples': 28, 'subsample': 0.5099573112905129, 'colsample_bytree': 0.816393228727315, 'reg_alpha': 9.46447591148586, 'reg_lambda': 2.4776645407797004, 'min_split_gain': 0.04806340155870579}. Best is trial 0 with value: 0.012005636027182333.
[Trial 0] New best: 0.01201
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:39,630] Trial 2 finished with value: 0.02303718728922514 and parameters: {'n_estimators': 142, 'learning_rate': 0.02295247568377602, 'num_leaves': 252, 'max_depth': 5, 'min_child_samples': 26, 'subsample': 0.7570672870873254, 'colsample_bytree': 0.5867902237585023, 'reg_alpha': 3.115324356799085, 'reg_lambda': 1.859868128016855, 'min_split_gain': 0.004218693362718728}. Best is trial 0 with value: 0.012005636027182333.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:44:39,858] Trial 3 finished with value: 0.02297885882044044 and parameters: {'n_estimators': 66, 'learning_rate': 0.025727700047259165, 'num_leaves': 161, 'max_depth': 12, 'min_child_samples': 29, 'subsample': 0.8384981902349693, 'colsample_bytree': 0.9630397761815152, 'reg_alpha': 8.165829079838558, 'reg_lambda': 6.039645050958637, 'min_split_gain': 0.048058438850134444}. Best is trial 0 with value: 0.012005636027182333.
[I 2025-06-15 19:44:39,861] A new study created in memory with name: no-name-18eaef8c-ec7c-4b4d-b255-f3b632f3a416
[I 2025-06-15 19:44:39,891] Trial 0 finished with value: 0.22288602503195631 and parameters: {'n_estimators': 95, 'learning_rate': 0.04108590175776806, 'num_leaves': 52, 'max_depth': 11, 'min_child_samples': 11, 'subsample': 0.512001031795412, 'colsample_bytree': 0.7547300062957716, 'reg_alpha': 5.97446107054871, 'reg_lambda': 9.645853311146784, 'min_split_gain': 0.040116121103072305}. Best is trial 0 with value: 0.22288602503195631.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         461
1           Price_lag_2         355
3         Load_DA_lag_0         348
14      pct_chg_Load_DA         328
5       WindOn_DA_lag_0         217
2           Price_lag_7         214
11           NGas_lag_2         197
15       lag168_Load_DA         194
10           Coal_lag_2         182
16  roll2d_median_Price         165
12            Oil_lag_2         136
13            EUA_lag_2         135
7                  WD_1           7
9                  WD_7           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         132
15       lag168_Load_DA          42
1           Price_lag_2          16
3         Load_DA_lag_0          15
11           NGas_lag_2           5
13            EUA_lag_2           2
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 1.37% done
********************* END NS.. 9 ****************************************************
********************* START NS ... 10 ****************************************************
[I 2025-06-15 19:44:40,789] A new study created in memory with name: no-name-ab127aa9-28a5-4456-8126-c9cb2ea17cb3
[I 2025-06-15 19:44:41,128] Trial 0 finished with value: 0.010571188322926797 and parameters: {'n_estimators': 170, 'learning_rate': 0.04191360503537781, 'num_leaves': 94, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.888251859388325, 'colsample_bytree': 0.8679221772320189, 'reg_alpha': 7.332572142625054, 'reg_lambda': 2.913339798913909, 'min_split_gain': 0.04593729902328179}. Best is trial 0 with value: 0.010571188322926797.
[Trial 0] New best: 0.01057
[I 2025-06-15 19:44:41,348] Trial 1 finished with value: 0.010697268320965653 and parameters: {'n_estimators': 98, 'learning_rate': 0.08714784968342885, 'num_leaves': 226, 'max_depth': 15, 'min_child_samples': 21, 'subsample': 0.7518882844906636, 'colsample_bytree': 0.9562407736103518, 'reg_alpha': 9.039044620078133, 'reg_lambda': 4.528060009044282, 'min_split_gain': 0.03596970824448409}. Best is trial 0 with value: 0.010571188322926797.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:42,016] Trial 2 finished with value: 0.023420343068958502 and parameters: {'n_estimators': 171, 'learning_rate': 0.01429720977817519, 'num_leaves': 138, 'max_depth': 16, 'min_child_samples': 15, 'subsample': 0.5001308486825655, 'colsample_bytree': 0.5048253656024948, 'reg_alpha': 3.640896070971759, 'reg_lambda': 1.687301354437546, 'min_split_gain': 0.031130519734821907}. Best is trial 0 with value: 0.010571188322926797.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:44:42,309] Trial 3 finished with value: 0.011289823323943224 and parameters: {'n_estimators': 290, 'learning_rate': 0.08611453272885687, 'num_leaves': 153, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.830676781024908, 'colsample_bytree': 0.8505561827819473, 'reg_alpha': 9.947975423712359, 'reg_lambda': 0.44331644363864164, 'min_split_gain': 0.04987740968102777}. Best is trial 0 with value: 0.010571188322926797.
[I 2025-06-15 19:44:42,311] A new study created in memory with name: no-name-bdaabc23-ca4a-40a6-b4ae-9888685df42a
[I 2025-06-15 19:44:42,354] Trial 0 finished with value: 0.10937175774513125 and parameters: {'n_estimators': 214, 'learning_rate': 0.10798769092199939, 'num_leaves': 107, 'max_depth': 10, 'min_child_samples': 1, 'subsample': 0.6862115504206946, 'colsample_bytree': 0.7861474770815664, 'reg_alpha': 9.142285938375043, 'reg_lambda': 5.290782639994115, 'min_split_gain': 0.017396661545940186}. Best is trial 0 with value: 0.10937175774513125.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1082
14      pct_chg_Load_DA         526
1           Price_lag_2         522
3         Load_DA_lag_0         315
11           NGas_lag_2         271
2           Price_lag_7         233
16  roll2d_median_Price         191
13            EUA_lag_2         154
10           Coal_lag_2         143
12            Oil_lag_2         126
15       lag168_Load_DA         104
5       WindOn_DA_lag_0          40
9                  WD_7           9
7                  WD_1           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          52
15       lag168_Load_DA          14
3         Load_DA_lag_0          12
1           Price_lag_2           5
10           Coal_lag_2           2
9                  WD_7           1
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 1.50% done
********************* END NS.. 10 ****************************************************
********************* START NS ... 11 ****************************************************
[I 2025-06-15 19:44:43,207] A new study created in memory with name: no-name-1fc549e2-cc4c-49d8-91ec-41ab165133dc
[I 2025-06-15 19:44:43,404] Trial 0 finished with value: 0.009349214514351268 and parameters: {'n_estimators': 212, 'learning_rate': 0.29976916827032274, 'num_leaves': 82, 'max_depth': 5, 'min_child_samples': 25, 'subsample': 0.548310140380048, 'colsample_bytree': 0.5340634624275651, 'reg_alpha': 1.035775608624887, 'reg_lambda': 3.040937032925312, 'min_split_gain': 0.016493961954524572}. Best is trial 0 with value: 0.009349214514351268.
[Trial 0] New best: 0.00935
[I 2025-06-15 19:44:43,677] Trial 1 finished with value: 0.013314429584752726 and parameters: {'n_estimators': 246, 'learning_rate': 0.0938477533457577, 'num_leaves': 229, 'max_depth': 8, 'min_child_samples': 16, 'subsample': 0.8820643618955795, 'colsample_bytree': 0.7587050860142138, 'reg_alpha': 9.578678527450418, 'reg_lambda': 5.187489227972831, 'min_split_gain': 0.03226156132732252}. Best is trial 0 with value: 0.009349214514351268.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:44,529] Trial 2 finished with value: 0.014058190991371048 and parameters: {'n_estimators': 299, 'learning_rate': 0.03073155297908923, 'num_leaves': 229, 'max_depth': 12, 'min_child_samples': 9, 'subsample': 0.9492837409229892, 'colsample_bytree': 0.6655584249924271, 'reg_alpha': 7.284789145997226, 'reg_lambda': 9.17211104855945, 'min_split_gain': 0.031208498175649114}. Best is trial 0 with value: 0.009349214514351268.
[I 2025-06-15 19:44:44,712] Trial 3 finished with value: 0.013689934520468244 and parameters: {'n_estimators': 51, 'learning_rate': 0.10607379141586362, 'num_leaves': 177, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.8269651194612215, 'colsample_bytree': 0.6480603930251745, 'reg_alpha': 6.4702002278057025, 'reg_lambda': 6.90538799376977, 'min_split_gain': 0.03815515494592012}. Best is trial 0 with value: 0.009349214514351268.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:44:44,714] A new study created in memory with name: no-name-0719afc4-64cc-4dc7-b901-da23c71b9b43
[I 2025-06-15 19:44:44,746] Trial 0 finished with value: 0.09488077483486344 and parameters: {'n_estimators': 135, 'learning_rate': 0.010882258587682324, 'num_leaves': 253, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.8920593410844757, 'colsample_bytree': 0.9553499166601943, 'reg_alpha': 7.847526796236805, 'reg_lambda': 8.886690609734021, 'min_split_gain': 0.03414998511208147}. Best is trial 0 with value: 0.09488077483486344.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         259
1           Price_lag_2         215
16  roll2d_median_Price         166
3         Load_DA_lag_0         152
2           Price_lag_7         107
10           Coal_lag_2          96
11           NGas_lag_2          91
5       WindOn_DA_lag_0          84
13            EUA_lag_2          84
14      pct_chg_Load_DA          82
15       lag168_Load_DA          77
12            Oil_lag_2          76
9                  WD_7           6
7                  WD_1           5
8                  WD_6           3
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         275
3         Load_DA_lag_0          21
1           Price_lag_2          15
15       lag168_Load_DA           4
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 1.64% done
********************* END NS.. 11 ****************************************************
********************* START NS ... 12 ****************************************************
[I 2025-06-15 19:44:45,883] A new study created in memory with name: no-name-98e9d49a-ff99-4a42-8b26-ad31d2fe6ce8
[I 2025-06-15 19:44:46,094] Trial 0 finished with value: 0.026667239684943292 and parameters: {'n_estimators': 62, 'learning_rate': 0.26866993050555926, 'num_leaves': 41, 'max_depth': 9, 'min_child_samples': 26, 'subsample': 0.5539914933526107, 'colsample_bytree': 0.7118592596895932, 'reg_alpha': 0.01610980708793286, 'reg_lambda': 3.635509801442929, 'min_split_gain': 0.005615894409392808}. Best is trial 0 with value: 0.026667239684943292.
[Trial 0] New best: 0.02667
[I 2025-06-15 19:44:46,631] Trial 1 finished with value: 0.03027124134033113 and parameters: {'n_estimators': 214, 'learning_rate': 0.023516352190628052, 'num_leaves': 106, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.8826416329681057, 'colsample_bytree': 0.8524436261203254, 'reg_alpha': 5.1805225750228265, 'reg_lambda': 9.385835185103144, 'min_split_gain': 0.04550985016566578}. Best is trial 0 with value: 0.026667239684943292.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:47,330] Trial 2 finished with value: 0.032023275142853795 and parameters: {'n_estimators': 220, 'learning_rate': 0.0815317061883276, 'num_leaves': 41, 'max_depth': 11, 'min_child_samples': 8, 'subsample': 0.5910452077257742, 'colsample_bytree': 0.5366859470456047, 'reg_alpha': 1.6458454707047443, 'reg_lambda': 9.693843749750922, 'min_split_gain': 0.014302399276243511}. Best is trial 0 with value: 0.026667239684943292.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:44:47,659] Trial 3 finished with value: 0.03340897423923309 and parameters: {'n_estimators': 196, 'learning_rate': 0.012465352581284705, 'num_leaves': 110, 'max_depth': 4, 'min_child_samples': 20, 'subsample': 0.6797318767752533, 'colsample_bytree': 0.8870401049346122, 'reg_alpha': 5.943023206651315, 'reg_lambda': 0.9266907014079184, 'min_split_gain': 0.00238579690792225}. Best is trial 0 with value: 0.026667239684943292.
[I 2025-06-15 19:44:47,662] A new study created in memory with name: no-name-b20d94b6-8b4b-4ebc-b0c3-b3fc1cfdd28a
[I 2025-06-15 19:44:47,685] Trial 0 finished with value: 0.08315077959468309 and parameters: {'n_estimators': 53, 'learning_rate': 0.04075753866521108, 'num_leaves': 129, 'max_depth': 12, 'min_child_samples': 25, 'subsample': 0.723904534414463, 'colsample_bytree': 0.641396868382777, 'reg_alpha': 1.927998880896612, 'reg_lambda': 1.622186430692626, 'min_split_gain': 0.014619336829927942}. Best is trial 0 with value: 0.08315077959468309.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         372
1           Price_lag_2         261
14      pct_chg_Load_DA         255
3         Load_DA_lag_0         253
5       WindOn_DA_lag_0         228
15       lag168_Load_DA         207
2           Price_lag_7         201
16  roll2d_median_Price         160
11           NGas_lag_2         134
13            EUA_lag_2         134
12            Oil_lag_2         114
10           Coal_lag_2         107
7                  WD_1          10
9                  WD_7          10
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          93
3         Load_DA_lag_0          18
14      pct_chg_Load_DA          17
15       lag168_Load_DA          10
1           Price_lag_2          10
11           NGas_lag_2           3
5       WindOn_DA_lag_0           2
10           Coal_lag_2           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 1.78% done
********************* END NS.. 12 ****************************************************
********************* START NS ... 13 ****************************************************
[I 2025-06-15 19:44:48,445] A new study created in memory with name: no-name-dd249c8e-ddf9-471b-9952-adf6cedc59a8
[I 2025-06-15 19:44:49,019] Trial 0 finished with value: 0.02438611630435399 and parameters: {'n_estimators': 209, 'learning_rate': 0.03258827825984652, 'num_leaves': 222, 'max_depth': 16, 'min_child_samples': 17, 'subsample': 0.6491533446320412, 'colsample_bytree': 0.7929844094847898, 'reg_alpha': 2.9882716959884146, 'reg_lambda': 6.569546133540069, 'min_split_gain': 0.04686836789633188}. Best is trial 0 with value: 0.02438611630435399.
[Trial 0] New best: 0.02439
[I 2025-06-15 19:44:49,648] Trial 1 finished with value: 0.022716655966175198 and parameters: {'n_estimators': 206, 'learning_rate': 0.03999588031427406, 'num_leaves': 60, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.9085724683186647, 'colsample_bytree': 0.9711588048150922, 'reg_alpha': 2.7080041367811325, 'reg_lambda': 3.395846400607941, 'min_split_gain': 0.031427946982348624}. Best is trial 1 with value: 0.022716655966175198.
[I 2025-06-15 19:44:49,850] Trial 2 finished with value: 0.026904600065925082 and parameters: {'n_estimators': 233, 'learning_rate': 0.27960417029679463, 'num_leaves': 72, 'max_depth': 8, 'min_child_samples': 2, 'subsample': 0.5304736206388361, 'colsample_bytree': 0.8991036480199014, 'reg_alpha': 8.587876544452804, 'reg_lambda': 3.898060578408531, 'min_split_gain': 0.029036602997438845}. Best is trial 1 with value: 0.022716655966175198.
[Trial 1] New best: 0.02272
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:50,312] Trial 3 finished with value: 0.031129964937544108 and parameters: {'n_estimators': 259, 'learning_rate': 0.037958671399971194, 'num_leaves': 191, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.6436193112370657, 'colsample_bytree': 0.5510168805477405, 'reg_alpha': 7.373626806638728, 'reg_lambda': 9.733814349600893, 'min_split_gain': 0.0015809080404257216}. Best is trial 1 with value: 0.022716655966175198.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:44:50,575] Trial 4 finished with value: 0.024505963538983228 and parameters: {'n_estimators': 113, 'learning_rate': 0.09283543803218505, 'num_leaves': 42, 'max_depth': 6, 'min_child_samples': 10, 'subsample': 0.5127420206788377, 'colsample_bytree': 0.6656761629659553, 'reg_alpha': 3.1235338753529165, 'reg_lambda': 8.334625182922089, 'min_split_gain': 0.02059177042237413}. Best is trial 1 with value: 0.022716655966175198.
[I 2025-06-15 19:44:50,578] A new study created in memory with name: no-name-3cb62b46-acb3-4363-8f23-1e5039ec7371
[I 2025-06-15 19:44:50,643] Trial 0 finished with value: 0.05725259177097355 and parameters: {'n_estimators': 284, 'learning_rate': 0.027021978606822976, 'num_leaves': 194, 'max_depth': 4, 'min_child_samples': 2, 'subsample': 0.7614691345169017, 'colsample_bytree': 0.7683016899040394, 'reg_alpha': 4.924735737741388, 'reg_lambda': 9.113160300529913, 'min_split_gain': 0.022153171285885776}. Best is trial 0 with value: 0.05725259177097355.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1557
14      pct_chg_Load_DA        1153
1           Price_lag_2         686
3         Load_DA_lag_0         677
16  roll2d_median_Price         495
11           NGas_lag_2         470
15       lag168_Load_DA         431
2           Price_lag_7         424
5       WindOn_DA_lag_0         324
12            Oil_lag_2         298
10           Coal_lag_2         295
13            EUA_lag_2         250
7                  WD_1          13
9                  WD_7          13
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         237
3         Load_DA_lag_0          92
15       lag168_Load_DA          60
11           NGas_lag_2          38
14      pct_chg_Load_DA          25
1           Price_lag_2          21
5       WindOn_DA_lag_0          13
7                  WD_1          11
13            EUA_lag_2           7
2           Price_lag_7           5
10           Coal_lag_2           1
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 1.92% done
********************* END NS.. 13 ****************************************************
********************* START NS ... 14 ****************************************************
[I 2025-06-15 19:44:51,831] A new study created in memory with name: no-name-6dede702-a253-4295-9926-1c345338c30b
[I 2025-06-15 19:44:52,070] Trial 0 finished with value: 0.021559257585981637 and parameters: {'n_estimators': 71, 'learning_rate': 0.022186364135438674, 'num_leaves': 217, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.6791674409492181, 'colsample_bytree': 0.5654901723076398, 'reg_alpha': 1.9393972767205714, 'reg_lambda': 5.521892851265341, 'min_split_gain': 0.011905001238002434}. Best is trial 0 with value: 0.021559257585981637.
[Trial 0] New best: 0.02156
[I 2025-06-15 19:44:52,422] Trial 1 finished with value: 0.00958195616354293 and parameters: {'n_estimators': 284, 'learning_rate': 0.16464863892883194, 'num_leaves': 82, 'max_depth': 6, 'min_child_samples': 28, 'subsample': 0.6898972702356915, 'colsample_bytree': 0.5229878861919093, 'reg_alpha': 2.418118656306917, 'reg_lambda': 1.6581704966955513, 'min_split_gain': 0.00242087280573437}. Best is trial 1 with value: 0.00958195616354293.
[Trial 1] New best: 0.00958
[I 2025-06-15 19:44:52,702] Trial 2 finished with value: 0.01565808379053006 and parameters: {'n_estimators': 66, 'learning_rate': 0.023714243818908323, 'num_leaves': 116, 'max_depth': 10, 'min_child_samples': 30, 'subsample': 0.6869443927633203, 'colsample_bytree': 0.7120404641021765, 'reg_alpha': 1.2796670196052573, 'reg_lambda': 6.849958527202105, 'min_split_gain': 0.02926260825197559}. Best is trial 1 with value: 0.00958195616354293.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:52,973] Trial 3 finished with value: 0.007670436111252759 and parameters: {'n_estimators': 80, 'learning_rate': 0.205518019804506, 'num_leaves': 162, 'max_depth': 7, 'min_child_samples': 24, 'subsample': 0.7366614668067628, 'colsample_bytree': 0.7859728419145015, 'reg_alpha': 0.8630830951435886, 'reg_lambda': 5.476672674072107, 'min_split_gain': 0.00494425625695103}. Best is trial 3 with value: 0.007670436111252759.
[Trial 3] New best: 0.00767
[I 2025-06-15 19:44:53,253] Trial 4 finished with value: 0.021642743181117852 and parameters: {'n_estimators': 63, 'learning_rate': 0.01493147481131699, 'num_leaves': 75, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.9857930892700741, 'colsample_bytree': 0.8380154893655183, 'reg_alpha': 1.1186850963487838, 'reg_lambda': 9.499817122719259, 'min_split_gain': 0.04877210845718653}. Best is trial 3 with value: 0.007670436111252759.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:54,252] Trial 5 finished with value: 0.007023446488365036 and parameters: {'n_estimators': 220, 'learning_rate': 0.12456132000711115, 'num_leaves': 204, 'max_depth': 10, 'min_child_samples': 17, 'subsample': 0.9723957197453822, 'colsample_bytree': 0.7200780557463042, 'reg_alpha': 1.176035244792366, 'reg_lambda': 4.703996242922103, 'min_split_gain': 0.0008810289420500107}. Best is trial 5 with value: 0.007023446488365036.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:44:54,859] Trial 6 finished with value: 0.0102172198057745 and parameters: {'n_estimators': 252, 'learning_rate': 0.02412867483189287, 'num_leaves': 144, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.9383496946465878, 'colsample_bytree': 0.9326921126814652, 'reg_alpha': 0.8354597813379694, 'reg_lambda': 4.956416704857079, 'min_split_gain': 0.04232853427934216}. Best is trial 5 with value: 0.007023446488365036.
[I 2025-06-15 19:44:54,861] A new study created in memory with name: no-name-b50dbf9f-daf5-48d5-a26f-e6e9380c140e
[I 2025-06-15 19:44:54,905] Trial 0 finished with value: 0.01319227924326393 and parameters: {'n_estimators': 124, 'learning_rate': 0.032750980225323884, 'num_leaves': 63, 'max_depth': 15, 'min_child_samples': 13, 'subsample': 0.629122153605582, 'colsample_bytree': 0.8491484930389306, 'reg_alpha': 1.6221978219991318, 'reg_lambda': 3.8505366063094493, 'min_split_gain': 0.03414049848020476}. Best is trial 0 with value: 0.01319227924326393.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
3         Load_DA_lag_0        1616
14      pct_chg_Load_DA        1533
0           Price_lag_1        1528
1           Price_lag_2        1306
15       lag168_Load_DA        1260
5       WindOn_DA_lag_0        1172
2           Price_lag_7        1087
16  roll2d_median_Price         795
13            EUA_lag_2         708
12            Oil_lag_2         687
11           NGas_lag_2         618
10           Coal_lag_2         605
9                  WD_7          27
7                  WD_1          22
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         178
3         Load_DA_lag_0          56
14      pct_chg_Load_DA          54
11           NGas_lag_2          30
1           Price_lag_2          11
16  roll2d_median_Price           7
12            Oil_lag_2           4
5       WindOn_DA_lag_0           1
13            EUA_lag_2           1
15       lag168_Load_DA           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
8                  WD_6           0
-> 2.05% done
********************* END NS.. 14 ****************************************************
********************* START NS ... 15 ****************************************************
[I 2025-06-15 19:44:56,078] A new study created in memory with name: no-name-b4b87693-20ca-4455-a299-879bc90bc8fa
[I 2025-06-15 19:44:56,271] Trial 0 finished with value: 0.013162712188003932 and parameters: {'n_estimators': 56, 'learning_rate': 0.19196084458142895, 'num_leaves': 199, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.5794734768199412, 'colsample_bytree': 0.6671991803657169, 'reg_alpha': 3.926054861442058, 'reg_lambda': 8.614254803716678, 'min_split_gain': 0.03308629619972299}. Best is trial 0 with value: 0.013162712188003932.
[Trial 0] New best: 0.01316
[I 2025-06-15 19:44:57,637] Trial 1 finished with value: 0.016952520466888724 and parameters: {'n_estimators': 296, 'learning_rate': 0.017118500377620933, 'num_leaves': 214, 'max_depth': 14, 'min_child_samples': 10, 'subsample': 0.841925015812391, 'colsample_bytree': 0.5443250166185016, 'reg_alpha': 3.178407423131173, 'reg_lambda': 4.4652815146341, 'min_split_gain': 0.0010783416962326653}. Best is trial 0 with value: 0.013162712188003932.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:57,956] Trial 2 finished with value: 0.012091793540607522 and parameters: {'n_estimators': 106, 'learning_rate': 0.26592650973296617, 'num_leaves': 69, 'max_depth': 15, 'min_child_samples': 1, 'subsample': 0.7961852591340448, 'colsample_bytree': 0.9162365472867036, 'reg_alpha': 5.172698161864609, 'reg_lambda': 9.361767437511816, 'min_split_gain': 0.02823866575636005}. Best is trial 2 with value: 0.012091793540607522.
[Trial 2] New best: 0.01209
[I 2025-06-15 19:44:58,874] Trial 3 finished with value: 0.018787906495411708 and parameters: {'n_estimators': 293, 'learning_rate': 0.026790580599786343, 'num_leaves': 168, 'max_depth': 8, 'min_child_samples': 14, 'subsample': 0.5517437913473336, 'colsample_bytree': 0.577803551870821, 'reg_alpha': 8.43933214727206, 'reg_lambda': 4.5629913230508325, 'min_split_gain': 0.0334084611651747}. Best is trial 2 with value: 0.012091793540607522.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:44:59,159] Trial 4 finished with value: 0.012170026323180755 and parameters: {'n_estimators': 194, 'learning_rate': 0.16214483449207334, 'num_leaves': 119, 'max_depth': 16, 'min_child_samples': 9, 'subsample': 0.83636898651294, 'colsample_bytree': 0.8355650342770327, 'reg_alpha': 3.318585766193005, 'reg_lambda': 1.252462502318299, 'min_split_gain': 0.030437611267225224}. Best is trial 2 with value: 0.012091793540607522.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:44:59,635] Trial 5 finished with value: 0.01886114537270925 and parameters: {'n_estimators': 111, 'learning_rate': 0.04597195650842023, 'num_leaves': 184, 'max_depth': 9, 'min_child_samples': 12, 'subsample': 0.6464030013214881, 'colsample_bytree': 0.5342235049319564, 'reg_alpha': 3.853226874510872, 'reg_lambda': 0.09450105155418442, 'min_split_gain': 0.024251705763992572}. Best is trial 2 with value: 0.012091793540607522.
[I 2025-06-15 19:44:59,637] A new study created in memory with name: no-name-703f75ef-2941-46f1-a34e-c9c42c83ccf1
[I 2025-06-15 19:44:59,690] Trial 0 finished with value: 0.08068027536723184 and parameters: {'n_estimators': 300, 'learning_rate': 0.06624767127688477, 'num_leaves': 181, 'max_depth': 8, 'min_child_samples': 12, 'subsample': 0.839051336760473, 'colsample_bytree': 0.811715590161848, 'reg_alpha': 7.946988972395895, 'reg_lambda': 6.044113381644031, 'min_split_gain': 0.03428477719335922}. Best is trial 0 with value: 0.08068027536723184.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         208
14      pct_chg_Load_DA         147
1           Price_lag_2         140
3         Load_DA_lag_0          84
2           Price_lag_7          78
11           NGas_lag_2          71
16  roll2d_median_Price          68
10           Coal_lag_2          51
5       WindOn_DA_lag_0          51
15       lag168_Load_DA          49
13            EUA_lag_2          48
12            Oil_lag_2          32
7                  WD_1           3
9                  WD_7           3
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          65
3         Load_DA_lag_0          33
1           Price_lag_2           5
11           NGas_lag_2           3
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 2.19% done
********************* END NS.. 15 ****************************************************
********************* START NS ... 16 ****************************************************
[I 2025-06-15 19:45:00,492] A new study created in memory with name: no-name-3def4915-7d68-4256-b516-b46d8d84a5d4
[I 2025-06-15 19:45:01,167] Trial 0 finished with value: 0.012395590271671043 and parameters: {'n_estimators': 263, 'learning_rate': 0.027206561362787512, 'num_leaves': 195, 'max_depth': 15, 'min_child_samples': 25, 'subsample': 0.6586128989017828, 'colsample_bytree': 0.6603753485974724, 'reg_alpha': 9.753144108324431, 'reg_lambda': 5.698463390669879, 'min_split_gain': 0.014362672219671524}. Best is trial 0 with value: 0.012395590271671043.
[Trial 0] New best: 0.01240
[I 2025-06-15 19:45:01,810] Trial 1 finished with value: 0.01261215326931634 and parameters: {'n_estimators': 225, 'learning_rate': 0.033065836401859366, 'num_leaves': 141, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.566245894501473, 'colsample_bytree': 0.57773567409902, 'reg_alpha': 3.8012830778819726, 'reg_lambda': 5.519609854706406, 'min_split_gain': 0.007360902799423874}. Best is trial 0 with value: 0.012395590271671043.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:02,185] Trial 2 finished with value: 0.008621833421486329 and parameters: {'n_estimators': 172, 'learning_rate': 0.0772084284685432, 'num_leaves': 116, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.9240611200895164, 'colsample_bytree': 0.8939574065748828, 'reg_alpha': 7.8599817009750295, 'reg_lambda': 8.913198927583597, 'min_split_gain': 0.026841916182916675}. Best is trial 2 with value: 0.008621833421486329.
[Trial 2] New best: 0.00862
[I 2025-06-15 19:45:02,606] Trial 3 finished with value: 0.03970858014807881 and parameters: {'n_estimators': 95, 'learning_rate': 0.01124123804679557, 'num_leaves': 214, 'max_depth': 11, 'min_child_samples': 4, 'subsample': 0.8325901519101753, 'colsample_bytree': 0.9297528391030502, 'reg_alpha': 7.6553443962456305, 'reg_lambda': 4.403819555704741, 'min_split_gain': 0.00682323256919381}. Best is trial 2 with value: 0.008621833421486329.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:02,872] Trial 4 finished with value: 0.06637282039001498 and parameters: {'n_estimators': 90, 'learning_rate': 0.011309752692816458, 'num_leaves': 127, 'max_depth': 5, 'min_child_samples': 11, 'subsample': 0.9428863572661286, 'colsample_bytree': 0.5328143788517736, 'reg_alpha': 9.995480791349294, 'reg_lambda': 8.699531491843626, 'min_split_gain': 0.042078973876447406}. Best is trial 2 with value: 0.008621833421486329.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:03,159] Trial 5 finished with value: 0.011566571593739616 and parameters: {'n_estimators': 264, 'learning_rate': 0.2092623104290232, 'num_leaves': 217, 'max_depth': 14, 'min_child_samples': 4, 'subsample': 0.910666573251713, 'colsample_bytree': 0.5726593332128949, 'reg_alpha': 4.97081779363871, 'reg_lambda': 3.3403462456655566, 'min_split_gain': 0.04718275032859162}. Best is trial 2 with value: 0.008621833421486329.
[I 2025-06-15 19:45:03,161] A new study created in memory with name: no-name-965e9077-bfbd-49b5-90c4-a00bc2cee1b1
[I 2025-06-15 19:45:03,221] Trial 0 finished with value: 0.05558103218989918 and parameters: {'n_estimators': 290, 'learning_rate': 0.09724971724343795, 'num_leaves': 111, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.9702936108132141, 'colsample_bytree': 0.6165157018091334, 'reg_alpha': 2.266996258172713, 'reg_lambda': 0.1055646810191635, 'min_split_gain': 0.04543615587267928}. Best is trial 0 with value: 0.05558103218989918.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         630
14      pct_chg_Load_DA         355
1           Price_lag_2         333
3         Load_DA_lag_0         213
11           NGas_lag_2         185
2           Price_lag_7         175
13            EUA_lag_2         154
16  roll2d_median_Price         151
10           Coal_lag_2         149
15       lag168_Load_DA         100
12            Oil_lag_2          89
5       WindOn_DA_lag_0          83
7                  WD_1           6
9                  WD_7           6
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          48
3         Load_DA_lag_0          20
1           Price_lag_2           7
14      pct_chg_Load_DA           5
5       WindOn_DA_lag_0           4
12            Oil_lag_2           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 2.33% done
********************* END NS.. 16 ****************************************************
********************* START NS ... 17 ****************************************************
[I 2025-06-15 19:45:04,078] A new study created in memory with name: no-name-5ed8ed92-3a4d-44ef-9ff2-6b4be551ca91
[I 2025-06-15 19:45:04,249] Trial 0 finished with value: 0.005981727479795533 and parameters: {'n_estimators': 258, 'learning_rate': 0.24217863337440856, 'num_leaves': 208, 'max_depth': 5, 'min_child_samples': 4, 'subsample': 0.5368288813116171, 'colsample_bytree': 0.5263120126056999, 'reg_alpha': 1.8641873074327053, 'reg_lambda': 6.024735404003708, 'min_split_gain': 0.026142727018146225}. Best is trial 0 with value: 0.005981727479795533.
[Trial 0] New best: 0.00598
[I 2025-06-15 19:45:04,454] Trial 1 finished with value: 0.0062282937661055715 and parameters: {'n_estimators': 70, 'learning_rate': 0.05647174179922606, 'num_leaves': 167, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.6510815472841714, 'colsample_bytree': 0.8164690656241388, 'reg_alpha': 7.236130750504631, 'reg_lambda': 1.1603364061303145, 'min_split_gain': 0.021628124387594556}. Best is trial 0 with value: 0.005981727479795533.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:04,703] Trial 2 finished with value: 0.004487546127470156 and parameters: {'n_estimators': 218, 'learning_rate': 0.05896398490881924, 'num_leaves': 129, 'max_depth': 4, 'min_child_samples': 16, 'subsample': 0.9273808837891855, 'colsample_bytree': 0.8467461128898583, 'reg_alpha': 4.614200590697376, 'reg_lambda': 5.422465891445537, 'min_split_gain': 0.042962674915872676}. Best is trial 2 with value: 0.004487546127470156.
[Trial 2] New best: 0.00449
[I 2025-06-15 19:45:05,020] Trial 3 finished with value: 0.006159627916034055 and parameters: {'n_estimators': 218, 'learning_rate': 0.18751687949108065, 'num_leaves': 63, 'max_depth': 13, 'min_child_samples': 1, 'subsample': 0.7814936043334599, 'colsample_bytree': 0.7401359823496051, 'reg_alpha': 3.0352352292789075, 'reg_lambda': 2.3575069244315285, 'min_split_gain': 0.011607639793864439}. Best is trial 2 with value: 0.004487546127470156.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:05,487] Trial 4 finished with value: 0.006250862169432387 and parameters: {'n_estimators': 197, 'learning_rate': 0.040203512198290554, 'num_leaves': 143, 'max_depth': 9, 'min_child_samples': 23, 'subsample': 0.9476924704402983, 'colsample_bytree': 0.5274148772536331, 'reg_alpha': 7.991922830217253, 'reg_lambda': 3.131222380483937, 'min_split_gain': 0.022866049484586756}. Best is trial 2 with value: 0.004487546127470156.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:05,892] Trial 5 finished with value: 0.007802539799411843 and parameters: {'n_estimators': 96, 'learning_rate': 0.0432912538085643, 'num_leaves': 163, 'max_depth': 12, 'min_child_samples': 16, 'subsample': 0.8169140418483265, 'colsample_bytree': 0.591390805886391, 'reg_alpha': 3.6734938983543195, 'reg_lambda': 5.7930881742534615, 'min_split_gain': 0.03756133886344662}. Best is trial 2 with value: 0.004487546127470156.
[I 2025-06-15 19:45:05,895] A new study created in memory with name: no-name-e901b854-6d7a-4e2c-8e19-cf345555901e
[I 2025-06-15 19:45:05,938] Trial 0 finished with value: 0.1617004269090413 and parameters: {'n_estimators': 202, 'learning_rate': 0.23121482397664558, 'num_leaves': 153, 'max_depth': 5, 'min_child_samples': 21, 'subsample': 0.9524363820879127, 'colsample_bytree': 0.6365946975458792, 'reg_alpha': 6.77682969943124, 'reg_lambda': 9.398506173377337, 'min_split_gain': 0.0027609009936621523}. Best is trial 0 with value: 0.1617004269090413.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         652
14      pct_chg_Load_DA         297
1           Price_lag_2         200
3         Load_DA_lag_0         165
11           NGas_lag_2         127
2           Price_lag_7          95
13            EUA_lag_2          95
16  roll2d_median_Price          94
15       lag168_Load_DA          61
5       WindOn_DA_lag_0          57
12            Oil_lag_2          54
10           Coal_lag_2          35
7                  WD_1           4
9                  WD_7           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          20
3         Load_DA_lag_0          17
11           NGas_lag_2           4
13            EUA_lag_2           2
1           Price_lag_2           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 2.46% done
********************* END NS.. 17 ****************************************************
********************* START NS ... 18 ****************************************************
[I 2025-06-15 19:45:06,706] A new study created in memory with name: no-name-f2e4ead4-7ad3-467f-9636-18143cf982c9
[I 2025-06-15 19:45:06,909] Trial 0 finished with value: 0.004132909713812631 and parameters: {'n_estimators': 250, 'learning_rate': 0.07661412667690391, 'num_leaves': 134, 'max_depth': 4, 'min_child_samples': 17, 'subsample': 0.9031309889344912, 'colsample_bytree': 0.6875663212766314, 'reg_alpha': 9.44267517939882, 'reg_lambda': 8.048842509288779, 'min_split_gain': 0.03164180926164064}. Best is trial 0 with value: 0.004132909713812631.
[Trial 0] New best: 0.00413
[I 2025-06-15 19:45:07,445] Trial 1 finished with value: 0.004417446268888565 and parameters: {'n_estimators': 251, 'learning_rate': 0.038367373454289075, 'num_leaves': 236, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.7914321386998532, 'colsample_bytree': 0.922748760251717, 'reg_alpha': 3.573126687402569, 'reg_lambda': 7.652191409371691, 'min_split_gain': 0.03357059161949326}. Best is trial 0 with value: 0.004132909713812631.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:08,306] Trial 2 finished with value: 0.005077943505719439 and parameters: {'n_estimators': 241, 'learning_rate': 0.0137201075132788, 'num_leaves': 192, 'max_depth': 16, 'min_child_samples': 6, 'subsample': 0.8711115082522246, 'colsample_bytree': 0.8801541136783808, 'reg_alpha': 3.104485451727551, 'reg_lambda': 2.8085791410438077, 'min_split_gain': 0.042834521814730867}. Best is trial 0 with value: 0.004132909713812631.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:08,938] Trial 3 finished with value: 0.004640614763664219 and parameters: {'n_estimators': 220, 'learning_rate': 0.05387736899033658, 'num_leaves': 69, 'max_depth': 12, 'min_child_samples': 11, 'subsample': 0.7863069696294105, 'colsample_bytree': 0.8599213303001552, 'reg_alpha': 0.9586533089156757, 'reg_lambda': 5.074103493159068, 'min_split_gain': 0.027137336907720203}. Best is trial 0 with value: 0.004132909713812631.
[I 2025-06-15 19:45:08,941] A new study created in memory with name: no-name-9be89225-5736-4e8f-9861-05881b51196b
[I 2025-06-15 19:45:09,008] Trial 0 finished with value: 0.08910164965129734 and parameters: {'n_estimators': 154, 'learning_rate': 0.013514364167311261, 'num_leaves': 228, 'max_depth': 13, 'min_child_samples': 1, 'subsample': 0.6731146531554648, 'colsample_bytree': 0.8718725677653378, 'reg_alpha': 3.565946042530086, 'reg_lambda': 5.220550261940659, 'min_split_gain': 0.018319003744534434}. Best is trial 0 with value: 0.08910164965129734.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         445
14      pct_chg_Load_DA         187
1           Price_lag_2         163
3         Load_DA_lag_0         117
11           NGas_lag_2          98
16  roll2d_median_Price          75
2           Price_lag_7          72
13            EUA_lag_2          67
15       lag168_Load_DA          54
5       WindOn_DA_lag_0          34
10           Coal_lag_2          31
12            Oil_lag_2          28
7                  WD_1           7
9                  WD_7           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         317
15       lag168_Load_DA         122
3         Load_DA_lag_0          96
1           Price_lag_2          37
11           NGas_lag_2          33
2           Price_lag_7          33
7                  WD_1          33
13            EUA_lag_2           2
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 2.60% done
********************* END NS.. 18 ****************************************************
********************* START NS ... 19 ****************************************************
[I 2025-06-15 19:45:09,780] A new study created in memory with name: no-name-7cc1b4a9-e0e8-40bf-8c04-b4ba39b5e5ba
[I 2025-06-15 19:45:10,274] Trial 0 finished with value: 0.01938594493799253 and parameters: {'n_estimators': 139, 'learning_rate': 0.012260604692459499, 'num_leaves': 121, 'max_depth': 15, 'min_child_samples': 10, 'subsample': 0.6387006834690748, 'colsample_bytree': 0.6292451337367038, 'reg_alpha': 2.9306712701650994, 'reg_lambda': 3.3919341103122616, 'min_split_gain': 0.007591075636988654}. Best is trial 0 with value: 0.01938594493799253.
[Trial 0] New best: 0.01939
[I 2025-06-15 19:45:10,866] Trial 1 finished with value: 0.0056542643784349466 and parameters: {'n_estimators': 256, 'learning_rate': 0.04214556368626568, 'num_leaves': 59, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.7081590411475973, 'colsample_bytree': 0.6440994721725074, 'reg_alpha': 5.004282839259959, 'reg_lambda': 2.449558941646849, 'min_split_gain': 0.021683947097980674}. Best is trial 1 with value: 0.0056542643784349466.
[Trial 1] New best: 0.00565
[I 2025-06-15 19:45:11,091] Trial 2 finished with value: 0.005473975208090687 and parameters: {'n_estimators': 153, 'learning_rate': 0.16661596403075463, 'num_leaves': 147, 'max_depth': 11, 'min_child_samples': 2, 'subsample': 0.6180597449719444, 'colsample_bytree': 0.5627281555237191, 'reg_alpha': 4.35207160061407, 'reg_lambda': 9.720202942762436, 'min_split_gain': 0.042889497270414224}. Best is trial 2 with value: 0.005473975208090687.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:11,380] Trial 3 finished with value: 0.005729948362902061 and parameters: {'n_estimators': 199, 'learning_rate': 0.14177670049819457, 'num_leaves': 116, 'max_depth': 13, 'min_child_samples': 7, 'subsample': 0.5691373672622274, 'colsample_bytree': 0.7527853033817187, 'reg_alpha': 1.9631014421664683, 'reg_lambda': 5.648896140270168, 'min_split_gain': 0.037929064411597956}. Best is trial 2 with value: 0.005473975208090687.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:11,611] Trial 4 finished with value: 0.015453815073138354 and parameters: {'n_estimators': 70, 'learning_rate': 0.027868319580266497, 'num_leaves': 148, 'max_depth': 11, 'min_child_samples': 17, 'subsample': 0.9571351358248148, 'colsample_bytree': 0.6272364876196048, 'reg_alpha': 3.971488319891856, 'reg_lambda': 6.56066867871003, 'min_split_gain': 0.042401268667746475}. Best is trial 2 with value: 0.005473975208090687.
[I 2025-06-15 19:45:11,613] A new study created in memory with name: no-name-5a27f755-4f12-4557-b193-e6d81664a877
[I 2025-06-15 19:45:11,641] Trial 0 finished with value: 0.033954688389143284 and parameters: {'n_estimators': 169, 'learning_rate': 0.29575852295967703, 'num_leaves': 106, 'max_depth': 10, 'min_child_samples': 13, 'subsample': 0.7953382513151364, 'colsample_bytree': 0.6783443940419365, 'reg_alpha': 1.6802231061324613, 'reg_lambda': 9.220716293828957, 'min_split_gain': 0.047267429709444655}. Best is trial 0 with value: 0.033954688389143284.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         318
1           Price_lag_2         282
3         Load_DA_lag_0         187
14      pct_chg_Load_DA         165
2           Price_lag_7         153
16  roll2d_median_Price         148
5       WindOn_DA_lag_0         136
13            EUA_lag_2         132
10           Coal_lag_2         127
15       lag168_Load_DA          97
11           NGas_lag_2          94
12            Oil_lag_2          71
7                  WD_1           8
9                  WD_7           8
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          24
3         Load_DA_lag_0           5
14      pct_chg_Load_DA           3
1           Price_lag_2           2
15       lag168_Load_DA           1
11           NGas_lag_2           1
10           Coal_lag_2           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 2.74% done
********************* END NS.. 19 ****************************************************
********************* START NS ... 20 ****************************************************
[I 2025-06-15 19:45:12,424] A new study created in memory with name: no-name-357b884e-0bd7-40bc-a1c1-e7524f042b63
[I 2025-06-15 19:45:12,905] Trial 0 finished with value: 0.012788239815312385 and parameters: {'n_estimators': 278, 'learning_rate': 0.018918410920121385, 'num_leaves': 181, 'max_depth': 5, 'min_child_samples': 3, 'subsample': 0.7361171155358768, 'colsample_bytree': 0.6256145774016441, 'reg_alpha': 5.265049808543716, 'reg_lambda': 1.0379938269001943, 'min_split_gain': 0.03076246147783439}. Best is trial 0 with value: 0.012788239815312385.
[Trial 0] New best: 0.01279
[I 2025-06-15 19:45:13,128] Trial 1 finished with value: 0.016231825750922264 and parameters: {'n_estimators': 70, 'learning_rate': 0.04205646062104442, 'num_leaves': 224, 'max_depth': 7, 'min_child_samples': 13, 'subsample': 0.658172332399036, 'colsample_bytree': 0.5593944502347841, 'reg_alpha': 5.407735339699654, 'reg_lambda': 2.3935280752763175, 'min_split_gain': 0.0402284248814902}. Best is trial 0 with value: 0.012788239815312385.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:13,655] Trial 2 finished with value: 0.01166944821767174 and parameters: {'n_estimators': 190, 'learning_rate': 0.036382171986882025, 'num_leaves': 72, 'max_depth': 10, 'min_child_samples': 11, 'subsample': 0.7472874820827762, 'colsample_bytree': 0.9843708361082599, 'reg_alpha': 4.198027132653692, 'reg_lambda': 9.826731831991395, 'min_split_gain': 0.04649337990905686}. Best is trial 2 with value: 0.01166944821767174.
[I 2025-06-15 19:45:13,861] Trial 3 finished with value: 0.011811186847272186 and parameters: {'n_estimators': 130, 'learning_rate': 0.20572008112583812, 'num_leaves': 108, 'max_depth': 14, 'min_child_samples': 30, 'subsample': 0.5358798745794702, 'colsample_bytree': 0.937255317961779, 'reg_alpha': 4.381020209696342, 'reg_lambda': 6.17424515950607, 'min_split_gain': 0.017862342417143014}. Best is trial 2 with value: 0.01166944821767174.
[Trial 2] New best: 0.01167
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:14,372] Trial 4 finished with value: 0.014356601067843486 and parameters: {'n_estimators': 217, 'learning_rate': 0.013507756913974588, 'num_leaves': 42, 'max_depth': 5, 'min_child_samples': 21, 'subsample': 0.6267798584648681, 'colsample_bytree': 0.8005244057855747, 'reg_alpha': 5.274426742724836, 'reg_lambda': 8.34700085275552, 'min_split_gain': 0.002023306819290971}. Best is trial 2 with value: 0.01166944821767174.
[I 2025-06-15 19:45:14,585] Trial 5 finished with value: 0.011409742424810332 and parameters: {'n_estimators': 70, 'learning_rate': 0.22997259027605016, 'num_leaves': 52, 'max_depth': 7, 'min_child_samples': 30, 'subsample': 0.5821996441558255, 'colsample_bytree': 0.8666302404672037, 'reg_alpha': 0.715155077678229, 'reg_lambda': 2.173207707408056, 'min_split_gain': 0.0003678921538470381}. Best is trial 5 with value: 0.011409742424810332.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:14,588] A new study created in memory with name: no-name-2a5882df-f211-41b2-be5b-abd0d7288eb2
[I 2025-06-15 19:45:14,607] Trial 0 finished with value: 0.07837994953613592 and parameters: {'n_estimators': 102, 'learning_rate': 0.1099478274655677, 'num_leaves': 162, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.5015988957635819, 'colsample_bytree': 0.9814931257481241, 'reg_alpha': 4.346975982275336, 'reg_lambda': 2.7138993465019534, 'min_split_gain': 0.04850332245787707}. Best is trial 0 with value: 0.07837994953613592.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         477
14      pct_chg_Load_DA         354
1           Price_lag_2         336
3         Load_DA_lag_0         269
5       WindOn_DA_lag_0         209
15       lag168_Load_DA         190
2           Price_lag_7         182
16  roll2d_median_Price         166
11           NGas_lag_2         164
13            EUA_lag_2         123
10           Coal_lag_2         118
12            Oil_lag_2         118
7                  WD_1          10
9                  WD_7           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          46
3         Load_DA_lag_0          12
16  roll2d_median_Price           5
5       WindOn_DA_lag_0           4
15       lag168_Load_DA           1
2           Price_lag_7           0
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 2.87% done
********************* END NS.. 20 ****************************************************
********************* START NS ... 21 ****************************************************
[I 2025-06-15 19:45:15,361] A new study created in memory with name: no-name-befffe76-c9aa-473c-8e3a-5101568a1f32
[I 2025-06-15 19:45:15,574] Trial 0 finished with value: 0.01424930137300247 and parameters: {'n_estimators': 285, 'learning_rate': 0.1568299467424368, 'num_leaves': 238, 'max_depth': 8, 'min_child_samples': 9, 'subsample': 0.894103385876001, 'colsample_bytree': 0.7426940024547591, 'reg_alpha': 6.614665217912425, 'reg_lambda': 1.976059157565172, 'min_split_gain': 0.045973103014872736}. Best is trial 0 with value: 0.01424930137300247.
[Trial 0] New best: 0.01425
[I 2025-06-15 19:45:16,218] Trial 1 finished with value: 0.013939604125606159 and parameters: {'n_estimators': 256, 'learning_rate': 0.0173519536444288, 'num_leaves': 116, 'max_depth': 14, 'min_child_samples': 14, 'subsample': 0.7458326608980055, 'colsample_bytree': 0.742440686075996, 'reg_alpha': 9.390244822182817, 'reg_lambda': 8.090205515133055, 'min_split_gain': 0.02898929802897131}. Best is trial 1 with value: 0.013939604125606159.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:16,779] Trial 2 finished with value: 0.020902858370662525 and parameters: {'n_estimators': 153, 'learning_rate': 0.014131527907155744, 'num_leaves': 122, 'max_depth': 10, 'min_child_samples': 26, 'subsample': 0.7822842972823879, 'colsample_bytree': 0.7619989336587452, 'reg_alpha': 1.6996865765898594, 'reg_lambda': 8.968974941874702, 'min_split_gain': 0.042033774334485796}. Best is trial 1 with value: 0.013939604125606159.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:17,319] Trial 3 finished with value: 0.015077084765252172 and parameters: {'n_estimators': 148, 'learning_rate': 0.06706507866856833, 'num_leaves': 100, 'max_depth': 14, 'min_child_samples': 8, 'subsample': 0.6412860297004515, 'colsample_bytree': 0.936504618729461, 'reg_alpha': 5.30941460025257, 'reg_lambda': 4.309381197920404, 'min_split_gain': 0.014591807315558586}. Best is trial 1 with value: 0.013939604125606159.
[I 2025-06-15 19:45:17,321] A new study created in memory with name: no-name-b51ab2c8-8b20-4877-b05f-bd858eaf3760
[I 2025-06-15 19:45:17,341] Trial 0 finished with value: 0.08487497991559025 and parameters: {'n_estimators': 58, 'learning_rate': 0.021635675313491705, 'num_leaves': 109, 'max_depth': 10, 'min_child_samples': 20, 'subsample': 0.6105109818209842, 'colsample_bytree': 0.6206947254255627, 'reg_alpha': 9.782106527366782, 'reg_lambda': 6.018266417134747, 'min_split_gain': 0.0450750564392471}. Best is trial 0 with value: 0.08487497991559025.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1951
1           Price_lag_2        1215
14      pct_chg_Load_DA         800
16  roll2d_median_Price         440
2           Price_lag_7         352
3         Load_DA_lag_0         323
11           NGas_lag_2         297
10           Coal_lag_2         251
13            EUA_lag_2         194
12            Oil_lag_2         152
15       lag168_Load_DA         150
5       WindOn_DA_lag_0         127
9                  WD_7          26
8                  WD_6           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1          62
3         Load_DA_lag_0          19
1           Price_lag_2          15
16  roll2d_median_Price           9
15       lag168_Load_DA           4
12            Oil_lag_2           2
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
2           Price_lag_7           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 3.01% done
********************* END NS.. 21 ****************************************************
********************* START NS ... 22 ****************************************************
[I 2025-06-15 19:45:18,366] A new study created in memory with name: no-name-7d2c102d-43d9-49cb-ac0b-849a17276e57
[I 2025-06-15 19:45:18,555] Trial 0 finished with value: 0.01742248946440008 and parameters: {'n_estimators': 175, 'learning_rate': 0.01610000307800882, 'num_leaves': 82, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.6917464284058104, 'colsample_bytree': 0.5154785616602022, 'reg_alpha': 6.133746098494408, 'reg_lambda': 0.9064973190257686, 'min_split_gain': 0.0070473015241423855}. Best is trial 0 with value: 0.01742248946440008.
[Trial 0] New best: 0.01742
[I 2025-06-15 19:45:18,799] Trial 1 finished with value: 0.011927677140502838 and parameters: {'n_estimators': 109, 'learning_rate': 0.046430648727570974, 'num_leaves': 34, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.5997646774827249, 'colsample_bytree': 0.6082040642780877, 'reg_alpha': 8.520156486947126, 'reg_lambda': 0.9740969308968817, 'min_split_gain': 0.032290179378216786}. Best is trial 1 with value: 0.011927677140502838.
[Trial 1] New best: 0.01193
[I 2025-06-15 19:45:19,225] Trial 2 finished with value: 0.013046029101276202 and parameters: {'n_estimators': 171, 'learning_rate': 0.020008106889970036, 'num_leaves': 73, 'max_depth': 5, 'min_child_samples': 9, 'subsample': 0.9936729018607193, 'colsample_bytree': 0.8705417260062759, 'reg_alpha': 0.19262711214983463, 'reg_lambda': 1.8218669965963596, 'min_split_gain': 0.04114334676662918}. Best is trial 1 with value: 0.011927677140502838.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:19,450] Trial 3 finished with value: 0.013594953059729903 and parameters: {'n_estimators': 133, 'learning_rate': 0.03111625203933078, 'num_leaves': 62, 'max_depth': 4, 'min_child_samples': 10, 'subsample': 0.7301982052965027, 'colsample_bytree': 0.779558019416628, 'reg_alpha': 9.695415342268216, 'reg_lambda': 5.866401363762066, 'min_split_gain': 0.004216603953600423}. Best is trial 1 with value: 0.011927677140502838.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:19,935] Trial 4 finished with value: 0.014253868881312746 and parameters: {'n_estimators': 293, 'learning_rate': 0.01242008119883012, 'num_leaves': 161, 'max_depth': 4, 'min_child_samples': 23, 'subsample': 0.7270086196121447, 'colsample_bytree': 0.8371288082353243, 'reg_alpha': 1.7835649408347931, 'reg_lambda': 8.362400488899244, 'min_split_gain': 0.003327676412200087}. Best is trial 1 with value: 0.011927677140502838.
[I 2025-06-15 19:45:19,937] A new study created in memory with name: no-name-fab70464-a71e-465f-ac5c-597e97e191e1
[I 2025-06-15 19:45:19,998] Trial 0 finished with value: 0.02016095587477348 and parameters: {'n_estimators': 232, 'learning_rate': 0.017531584614727615, 'num_leaves': 93, 'max_depth': 7, 'min_child_samples': 26, 'subsample': 0.6436580208649886, 'colsample_bytree': 0.7212032798657613, 'reg_alpha': 1.6379329332985948, 'reg_lambda': 7.698229604460466, 'min_split_gain': 0.0035848089118714955}. Best is trial 0 with value: 0.02016095587477348.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         641
1           Price_lag_2         497
14      pct_chg_Load_DA         290
3         Load_DA_lag_0         197
16  roll2d_median_Price         170
2           Price_lag_7         162
11           NGas_lag_2         142
13            EUA_lag_2          98
10           Coal_lag_2          90
15       lag168_Load_DA          88
5       WindOn_DA_lag_0          62
12            Oil_lag_2          60
9                  WD_7           6
8                  WD_6           4
7                  WD_1           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         303
15       lag168_Load_DA         120
14      pct_chg_Load_DA          99
3         Load_DA_lag_0          72
16  roll2d_median_Price          45
2           Price_lag_7          38
10           Coal_lag_2          26
1           Price_lag_2          19
12            Oil_lag_2           6
13            EUA_lag_2           2
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
11           NGas_lag_2           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 3.15% done
********************* END NS.. 22 ****************************************************
********************* START NS ... 23 ****************************************************
[I 2025-06-15 19:45:20,796] A new study created in memory with name: no-name-1b4f8e74-8273-413c-982e-4c924ef3672a
[I 2025-06-15 19:45:21,431] Trial 0 finished with value: 0.040864517689646905 and parameters: {'n_estimators': 232, 'learning_rate': 0.02648411561247806, 'num_leaves': 239, 'max_depth': 14, 'min_child_samples': 20, 'subsample': 0.8375825453735315, 'colsample_bytree': 0.5501824386034326, 'reg_alpha': 2.914188583073205, 'reg_lambda': 7.041043563030746, 'min_split_gain': 0.031272420046984765}. Best is trial 0 with value: 0.040864517689646905.
[I 2025-06-15 19:45:21,592] Trial 1 finished with value: 0.033752472070306676 and parameters: {'n_estimators': 63, 'learning_rate': 0.269984602856552, 'num_leaves': 85, 'max_depth': 14, 'min_child_samples': 26, 'subsample': 0.5185670047006674, 'colsample_bytree': 0.5948874867548078, 'reg_alpha': 5.511519034272759, 'reg_lambda': 3.215350742000982, 'min_split_gain': 0.005776431090916196}. Best is trial 1 with value: 0.033752472070306676.
[Trial 0] New best: 0.04086
[Trial 1] New best: 0.03375
[I 2025-06-15 19:45:21,798] Trial 2 finished with value: 0.05511832766064303 and parameters: {'n_estimators': 69, 'learning_rate': 0.03349196252880374, 'num_leaves': 92, 'max_depth': 5, 'min_child_samples': 9, 'subsample': 0.7730983475445538, 'colsample_bytree': 0.7560308041781653, 'reg_alpha': 6.381121312855658, 'reg_lambda': 3.2353706885569746, 'min_split_gain': 0.04041968104933289}. Best is trial 1 with value: 0.033752472070306676.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:22,462] Trial 3 finished with value: 0.056003612453739304 and parameters: {'n_estimators': 263, 'learning_rate': 0.010868169816152911, 'num_leaves': 225, 'max_depth': 16, 'min_child_samples': 24, 'subsample': 0.797345429638423, 'colsample_bytree': 0.6700068103621958, 'reg_alpha': 8.325482182827987, 'reg_lambda': 7.435490988742158, 'min_split_gain': 0.011389664742849788}. Best is trial 1 with value: 0.033752472070306676.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:23,065] Trial 4 finished with value: 0.05830002542953078 and parameters: {'n_estimators': 132, 'learning_rate': 0.014760433844641815, 'num_leaves': 57, 'max_depth': 15, 'min_child_samples': 3, 'subsample': 0.8445557174903672, 'colsample_bytree': 0.9936370453772323, 'reg_alpha': 2.1543913419525484, 'reg_lambda': 4.713864354227449, 'min_split_gain': 0.021503731370232745}. Best is trial 1 with value: 0.033752472070306676.
[I 2025-06-15 19:45:23,067] A new study created in memory with name: no-name-e3ca4e56-8137-477c-beaf-5f34295efd2d
[I 2025-06-15 19:45:23,084] Trial 0 finished with value: 0.279201174896128 and parameters: {'n_estimators': 163, 'learning_rate': 0.023317165786293656, 'num_leaves': 62, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.5188767686722962, 'colsample_bytree': 0.6397447306455752, 'reg_alpha': 8.263227862526746, 'reg_lambda': 8.302216975372483, 'min_split_gain': 0.03807937132693342}. Best is trial 0 with value: 0.279201174896128.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         245
1           Price_lag_2         196
3         Load_DA_lag_0         176
5       WindOn_DA_lag_0         132
14      pct_chg_Load_DA         120
15       lag168_Load_DA         100
2           Price_lag_7          96
13            EUA_lag_2          94
16  roll2d_median_Price          92
11           NGas_lag_2          91
12            Oil_lag_2          75
10           Coal_lag_2          66
7                  WD_1           6
9                  WD_7           2
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         110
3         Load_DA_lag_0          56
15       lag168_Load_DA          48
1           Price_lag_2          12
12            Oil_lag_2           8
16  roll2d_median_Price           4
6      WindOff_DA_lag_0           0
7                  WD_1           0
5       WindOn_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
4        Solar_DA_lag_0           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 3.28% done
********************* END NS.. 23 ****************************************************
********************* START NS ... 24 ****************************************************
[I 2025-06-15 19:45:23,738] A new study created in memory with name: no-name-f0ff5cbe-b4c8-4a3f-a455-a34c71dd93b8
[I 2025-06-15 19:45:24,146] Trial 0 finished with value: 0.07137892498634933 and parameters: {'n_estimators': 282, 'learning_rate': 0.04795105856425522, 'num_leaves': 254, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.6064041581669888, 'colsample_bytree': 0.9313020960405174, 'reg_alpha': 5.4412050013928965, 'reg_lambda': 0.7897333296261744, 'min_split_gain': 0.02708178193519324}. Best is trial 0 with value: 0.07137892498634933.
[I 2025-06-15 19:45:24,287] Trial 1 finished with value: 0.06337867002252047 and parameters: {'n_estimators': 105, 'learning_rate': 0.17524785628813955, 'num_leaves': 206, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.6962213350637687, 'colsample_bytree': 0.7558510994133262, 'reg_alpha': 3.6953451981543903, 'reg_lambda': 8.123651898725116, 'min_split_gain': 0.03425725099561375}. Best is trial 1 with value: 0.06337867002252047.
[Trial 0] New best: 0.07138
[Trial 1] New best: 0.06338
[I 2025-06-15 19:45:26,670] Trial 2 finished with value: 0.08623756474528443 and parameters: {'n_estimators': 293, 'learning_rate': 0.047654338705760003, 'num_leaves': 254, 'max_depth': 10, 'min_child_samples': 4, 'subsample': 0.5048025412238226, 'colsample_bytree': 0.7615573371836943, 'reg_alpha': 1.778656241133968, 'reg_lambda': 2.845815341076948, 'min_split_gain': 0.0002738849470293792}. Best is trial 1 with value: 0.06337867002252047.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:27,227] Trial 3 finished with value: 0.08328616282353651 and parameters: {'n_estimators': 195, 'learning_rate': 0.04244084225617602, 'num_leaves': 254, 'max_depth': 8, 'min_child_samples': 16, 'subsample': 0.7569000666528977, 'colsample_bytree': 0.9558685839538588, 'reg_alpha': 2.914884952021662, 'reg_lambda': 3.0902713159626414, 'min_split_gain': 0.04690168243107529}. Best is trial 1 with value: 0.06337867002252047.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:27,941] Trial 4 finished with value: 0.10088807060170164 and parameters: {'n_estimators': 280, 'learning_rate': 0.014187696971951732, 'num_leaves': 255, 'max_depth': 9, 'min_child_samples': 7, 'subsample': 0.76495745747948, 'colsample_bytree': 0.536049251852226, 'reg_alpha': 8.681856405343689, 'reg_lambda': 3.1190997903983586, 'min_split_gain': 0.04544180454115175}. Best is trial 1 with value: 0.06337867002252047.
[I 2025-06-15 19:45:27,942] A new study created in memory with name: no-name-af886507-f5cd-481b-8fb6-a5195bd57398
[I 2025-06-15 19:45:27,965] Trial 0 finished with value: 0.2207580421112911 and parameters: {'n_estimators': 246, 'learning_rate': 0.1962814478091394, 'num_leaves': 89, 'max_depth': 9, 'min_child_samples': 13, 'subsample': 0.5379275069304965, 'colsample_bytree': 0.6105062853253622, 'reg_alpha': 0.4020520048548315, 'reg_lambda': 8.313132721822234, 'min_split_gain': 0.040739853371604205}. Best is trial 0 with value: 0.2207580421112911.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         257
1           Price_lag_2         158
14      pct_chg_Load_DA         131
3         Load_DA_lag_0          88
2           Price_lag_7          80
11           NGas_lag_2          60
16  roll2d_median_Price          56
13            EUA_lag_2          49
5       WindOn_DA_lag_0          44
15       lag168_Load_DA          41
10           Coal_lag_2          35
12            Oil_lag_2          35
7                  WD_1           5
9                  WD_7           3
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          26
15       lag168_Load_DA          16
3         Load_DA_lag_0           8
14      pct_chg_Load_DA           7
16  roll2d_median_Price           2
1           Price_lag_2           1
10           Coal_lag_2           1
2           Price_lag_7           1
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 3.42% done
********************* END NS.. 24 ****************************************************
********************* START NS ... 25 ****************************************************
[I 2025-06-15 19:45:28,621] A new study created in memory with name: no-name-833110d3-242d-46f4-98d2-d1b246becc0c
[I 2025-06-15 19:45:29,001] Trial 0 finished with value: 0.043785389716978464 and parameters: {'n_estimators': 167, 'learning_rate': 0.058680124925271415, 'num_leaves': 42, 'max_depth': 14, 'min_child_samples': 2, 'subsample': 0.6836942854297478, 'colsample_bytree': 0.8750678028151182, 'reg_alpha': 6.131089608952877, 'reg_lambda': 1.14884928845584, 'min_split_gain': 0.017295265810705302}. Best is trial 0 with value: 0.043785389716978464.
[Trial 0] New best: 0.04379
[I 2025-06-15 19:45:30,356] Trial 1 finished with value: 0.046306633193845474 and parameters: {'n_estimators': 269, 'learning_rate': 0.01679224493280968, 'num_leaves': 203, 'max_depth': 7, 'min_child_samples': 1, 'subsample': 0.8827157297088131, 'colsample_bytree': 0.9371462749694284, 'reg_alpha': 0.0539798044795714, 'reg_lambda': 5.135051942179425, 'min_split_gain': 0.02323019273408446}. Best is trial 0 with value: 0.043785389716978464.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:30,618] Trial 2 finished with value: 0.04856593593408707 and parameters: {'n_estimators': 268, 'learning_rate': 0.227108118092486, 'num_leaves': 114, 'max_depth': 6, 'min_child_samples': 13, 'subsample': 0.7747726016376967, 'colsample_bytree': 0.9364956778616951, 'reg_alpha': 9.477833645922903, 'reg_lambda': 1.4354369199814199, 'min_split_gain': 0.021827668810632368}. Best is trial 0 with value: 0.043785389716978464.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:31,384] Trial 3 finished with value: 0.04613655197589494 and parameters: {'n_estimators': 294, 'learning_rate': 0.020726292668811225, 'num_leaves': 192, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.5660332959592829, 'colsample_bytree': 0.6714715211622719, 'reg_alpha': 9.69043493547858, 'reg_lambda': 2.310711272555853, 'min_split_gain': 0.029857476274341474}. Best is trial 0 with value: 0.043785389716978464.
[I 2025-06-15 19:45:31,386] A new study created in memory with name: no-name-075440d9-95a7-4ec6-9513-5415b898f520
[I 2025-06-15 19:45:31,413] Trial 0 finished with value: 0.12489612791344308 and parameters: {'n_estimators': 135, 'learning_rate': 0.06634975877580611, 'num_leaves': 204, 'max_depth': 14, 'min_child_samples': 5, 'subsample': 0.7123128335748137, 'colsample_bytree': 0.9887392995989116, 'reg_alpha': 6.84938768859732, 'reg_lambda': 4.065809822606497, 'min_split_gain': 0.011033876420727852}. Best is trial 0 with value: 0.12489612791344308.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         985
1           Price_lag_2         667
14      pct_chg_Load_DA         613
3         Load_DA_lag_0         370
2           Price_lag_7         354
11           NGas_lag_2         310
5       WindOn_DA_lag_0         266
15       lag168_Load_DA         257
10           Coal_lag_2         253
16  roll2d_median_Price         244
12            Oil_lag_2         198
13            EUA_lag_2         190
9                  WD_7          19
7                  WD_1          15
8                  WD_6          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          58
15       lag168_Load_DA          45
3         Load_DA_lag_0          32
5       WindOn_DA_lag_0           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 3.56% done
********************* END NS.. 25 ****************************************************
********************* START NS ... 26 ****************************************************
[I 2025-06-15 19:45:32,281] A new study created in memory with name: no-name-b831a1f5-caf4-4d74-b87e-793024fae210
[I 2025-06-15 19:45:32,667] Trial 0 finished with value: 0.004481104394935636 and parameters: {'n_estimators': 249, 'learning_rate': 0.050417524833160726, 'num_leaves': 236, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.7653021792936248, 'colsample_bytree': 0.6195466076786884, 'reg_alpha': 8.393493485466085, 'reg_lambda': 7.306175609159387, 'min_split_gain': 0.02408061455919737}. Best is trial 0 with value: 0.004481104394935636.
[Trial 0] New best: 0.00448
[I 2025-06-15 19:45:32,952] Trial 1 finished with value: 0.00468463352865803 and parameters: {'n_estimators': 290, 'learning_rate': 0.19636687594300542, 'num_leaves': 59, 'max_depth': 10, 'min_child_samples': 16, 'subsample': 0.7777953511476114, 'colsample_bytree': 0.7777702424792757, 'reg_alpha': 2.287439881139793, 'reg_lambda': 8.207850855681382, 'min_split_gain': 0.01929273726046887}. Best is trial 0 with value: 0.004481104394935636.
[I 2025-06-15 19:45:33,117] Trial 2 finished with value: 0.005099678724853057 and parameters: {'n_estimators': 156, 'learning_rate': 0.25641706830627425, 'num_leaves': 247, 'max_depth': 7, 'min_child_samples': 6, 'subsample': 0.7911239141125783, 'colsample_bytree': 0.6331654193280313, 'reg_alpha': 8.162816131389564, 'reg_lambda': 5.739757045313717, 'min_split_gain': 0.039456108545912465}. Best is trial 0 with value: 0.004481104394935636.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:33,666] Trial 3 finished with value: 0.004098847370043456 and parameters: {'n_estimators': 288, 'learning_rate': 0.034825427088834356, 'num_leaves': 165, 'max_depth': 5, 'min_child_samples': 12, 'subsample': 0.5741727929168334, 'colsample_bytree': 0.8007640875937063, 'reg_alpha': 1.2207608184328367, 'reg_lambda': 8.292788380380738, 'min_split_gain': 0.012455461798005496}. Best is trial 3 with value: 0.004098847370043456.
[I 2025-06-15 19:45:33,669] A new study created in memory with name: no-name-4c762c1b-6b69-406a-b392-bbc257565753
[I 2025-06-15 19:45:33,695] Trial 0 finished with value: 0.16201482918199858 and parameters: {'n_estimators': 132, 'learning_rate': 0.060400348359155205, 'num_leaves': 240, 'max_depth': 13, 'min_child_samples': 23, 'subsample': 0.8780355914379028, 'colsample_bytree': 0.6410471596186924, 'reg_alpha': 6.085474342666622, 'reg_lambda': 4.620165931192378, 'min_split_gain': 0.021144785291123597}. Best is trial 0 with value: 0.16201482918199858.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1646
1           Price_lag_2        1039
14      pct_chg_Load_DA         859
3         Load_DA_lag_0         613
11           NGas_lag_2         421
15       lag168_Load_DA         381
5       WindOn_DA_lag_0         381
16  roll2d_median_Price         360
2           Price_lag_7         358
12            Oil_lag_2         324
13            EUA_lag_2         322
10           Coal_lag_2         249
9                  WD_7          25
7                  WD_1          21
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          64
15       lag168_Load_DA          20
3         Load_DA_lag_0          17
1           Price_lag_2           4
10           Coal_lag_2           1
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 3.69% done
********************* END NS.. 26 ****************************************************
********************* START NS ... 27 ****************************************************
[I 2025-06-15 19:45:34,717] A new study created in memory with name: no-name-1d3ef967-215d-4915-ac09-91be5c6948f3
[I 2025-06-15 19:45:34,880] Trial 0 finished with value: 0.018368511493361064 and parameters: {'n_estimators': 90, 'learning_rate': 0.1494004331943047, 'num_leaves': 133, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.941412540312928, 'colsample_bytree': 0.6136516413368235, 'reg_alpha': 4.305323266490234, 'reg_lambda': 3.0279690312627094, 'min_split_gain': 0.043457369857944544}. Best is trial 0 with value: 0.018368511493361064.
[I 2025-06-15 19:45:35,005] Trial 1 finished with value: 0.019180415253066872 and parameters: {'n_estimators': 146, 'learning_rate': 0.24042203187633462, 'num_leaves': 178, 'max_depth': 12, 'min_child_samples': 27, 'subsample': 0.8504850126446952, 'colsample_bytree': 0.9621505153219561, 'reg_alpha': 8.177477129421122, 'reg_lambda': 6.219214653356813, 'min_split_gain': 0.044965207586286166}. Best is trial 0 with value: 0.018368511493361064.
[Trial 0] New best: 0.01837
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:35,348] Trial 2 finished with value: 0.01841740405740555 and parameters: {'n_estimators': 195, 'learning_rate': 0.08850534506073085, 'num_leaves': 59, 'max_depth': 9, 'min_child_samples': 24, 'subsample': 0.6567241212738113, 'colsample_bytree': 0.9973898116520422, 'reg_alpha': 2.9624853005313376, 'reg_lambda': 1.2934633308234633, 'min_split_gain': 0.018339110255344633}. Best is trial 0 with value: 0.018368511493361064.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:35,666] Trial 3 finished with value: 0.017347065045536358 and parameters: {'n_estimators': 169, 'learning_rate': 0.25646132905111796, 'num_leaves': 132, 'max_depth': 16, 'min_child_samples': 3, 'subsample': 0.6848909720690928, 'colsample_bytree': 0.5851922434557304, 'reg_alpha': 6.654103992660071, 'reg_lambda': 2.67793964663145, 'min_split_gain': 0.0020132852023071247}. Best is trial 3 with value: 0.017347065045536358.
[Trial 3] New best: 0.01735
[I 2025-06-15 19:45:35,900] Trial 4 finished with value: 0.019892867658305544 and parameters: {'n_estimators': 273, 'learning_rate': 0.28240354034061255, 'num_leaves': 242, 'max_depth': 9, 'min_child_samples': 6, 'subsample': 0.52137858143619, 'colsample_bytree': 0.7569008099427248, 'reg_alpha': 8.928528313496857, 'reg_lambda': 5.903998899606881, 'min_split_gain': 0.04287986313740629}. Best is trial 3 with value: 0.017347065045536358.
[I 2025-06-15 19:45:36,110] Trial 5 finished with value: 0.018999810439148 and parameters: {'n_estimators': 199, 'learning_rate': 0.2585357139564384, 'num_leaves': 256, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.7701301247816601, 'colsample_bytree': 0.9565296539535573, 'reg_alpha': 3.618238358557242, 'reg_lambda': 9.794427678701794, 'min_split_gain': 0.030233553345078174}. Best is trial 3 with value: 0.017347065045536358.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:36,323] Trial 6 finished with value: 0.0366833884759187 and parameters: {'n_estimators': 113, 'learning_rate': 0.01441174163465036, 'num_leaves': 229, 'max_depth': 4, 'min_child_samples': 24, 'subsample': 0.9377901525958516, 'colsample_bytree': 0.6623069575854973, 'reg_alpha': 8.614733725218988, 'reg_lambda': 6.277461268846972, 'min_split_gain': 0.04990502955569312}. Best is trial 3 with value: 0.017347065045536358.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:36,326] A new study created in memory with name: no-name-2108094a-5299-4457-a462-0c6e4ad5fd46
[I 2025-06-15 19:45:36,349] Trial 0 finished with value: 0.08570052064373383 and parameters: {'n_estimators': 123, 'learning_rate': 0.08758798167665631, 'num_leaves': 39, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.7152877414438874, 'colsample_bytree': 0.7445794718463445, 'reg_alpha': 2.3161888191521927, 'reg_lambda': 9.113873384719682, 'min_split_gain': 0.04813198938691086}. Best is trial 0 with value: 0.08570052064373383.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         346
0           Price_lag_1         319
3         Load_DA_lag_0         223
5       WindOn_DA_lag_0         220
2           Price_lag_7         214
14      pct_chg_Load_DA         213
13            EUA_lag_2         190
15       lag168_Load_DA         173
10           Coal_lag_2         160
11           NGas_lag_2         154
12            Oil_lag_2         145
16  roll2d_median_Price         124
7                  WD_1          12
9                  WD_7           9
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          49
15       lag168_Load_DA          25
3         Load_DA_lag_0          20
5       WindOn_DA_lag_0          11
10           Coal_lag_2          10
11           NGas_lag_2           3
16  roll2d_median_Price           2
1           Price_lag_2           1
7                  WD_1           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 3.83% done
********************* END NS.. 27 ****************************************************
********************* START NS ... 28 ****************************************************
[I 2025-06-15 19:45:37,320] A new study created in memory with name: no-name-4b982e27-4c82-41fc-bef8-6197903207ec
[I 2025-06-15 19:45:37,579] Trial 0 finished with value: 0.025761920835268487 and parameters: {'n_estimators': 223, 'learning_rate': 0.01116779241864362, 'num_leaves': 183, 'max_depth': 4, 'min_child_samples': 27, 'subsample': 0.9131538250658375, 'colsample_bytree': 0.5739392463131332, 'reg_alpha': 6.39713478345167, 'reg_lambda': 9.715810369978183, 'min_split_gain': 0.017239088314697493}. Best is trial 0 with value: 0.025761920835268487.
[Trial 0] New best: 0.02576
[I 2025-06-15 19:45:38,009] Trial 1 finished with value: 0.020428050788770347 and parameters: {'n_estimators': 284, 'learning_rate': 0.017837325207782875, 'num_leaves': 75, 'max_depth': 4, 'min_child_samples': 9, 'subsample': 0.868598077410444, 'colsample_bytree': 0.8526332469019675, 'reg_alpha': 9.385890614995537, 'reg_lambda': 4.711211489507452, 'min_split_gain': 0.01797065007363301}. Best is trial 1 with value: 0.020428050788770347.
[Trial 1] New best: 0.02043
[I 2025-06-15 19:45:38,311] Trial 2 finished with value: 0.018596495549736237 and parameters: {'n_estimators': 229, 'learning_rate': 0.0927788651877541, 'num_leaves': 115, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.5858774452003495, 'colsample_bytree': 0.8901764396786325, 'reg_alpha': 5.1786206390904, 'reg_lambda': 5.340144729099285, 'min_split_gain': 0.03833906964045937}. Best is trial 2 with value: 0.018596495549736237.
[Trial 2] New best: 0.01860
[I 2025-06-15 19:45:38,531] Trial 3 finished with value: 0.020024144093125393 and parameters: {'n_estimators': 85, 'learning_rate': 0.07174058414590595, 'num_leaves': 246, 'max_depth': 5, 'min_child_samples': 28, 'subsample': 0.852754438550585, 'colsample_bytree': 0.8711949368795375, 'reg_alpha': 6.537622768534463, 'reg_lambda': 1.9964396085568203, 'min_split_gain': 0.04905950989440227}. Best is trial 2 with value: 0.018596495549736237.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:39,054] Trial 4 finished with value: 0.01972664803163409 and parameters: {'n_estimators': 221, 'learning_rate': 0.048525862679248424, 'num_leaves': 79, 'max_depth': 6, 'min_child_samples': 29, 'subsample': 0.7487409371487993, 'colsample_bytree': 0.8442132581923334, 'reg_alpha': 9.431529818450196, 'reg_lambda': 0.21559141918632063, 'min_split_gain': 0.00343874202908247}. Best is trial 2 with value: 0.018596495549736237.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:39,678] Trial 5 finished with value: 0.025667225054015055 and parameters: {'n_estimators': 164, 'learning_rate': 0.013726353178600836, 'num_leaves': 115, 'max_depth': 16, 'min_child_samples': 19, 'subsample': 0.9717399792314019, 'colsample_bytree': 0.6435913785213812, 'reg_alpha': 8.736522477891704, 'reg_lambda': 4.949232475664186, 'min_split_gain': 0.003693166181593727}. Best is trial 2 with value: 0.018596495549736237.
[I 2025-06-15 19:45:39,681] A new study created in memory with name: no-name-7b3f9d87-972f-4275-8312-674359becfa3
[I 2025-06-15 19:45:39,718] Trial 0 finished with value: 0.05715315660697306 and parameters: {'n_estimators': 185, 'learning_rate': 0.12130291670556624, 'num_leaves': 60, 'max_depth': 10, 'min_child_samples': 8, 'subsample': 0.6087425724921094, 'colsample_bytree': 0.9370753762483898, 'reg_alpha': 2.9174602066418953, 'reg_lambda': 6.034472767132826, 'min_split_gain': 0.04497416202670392}. Best is trial 0 with value: 0.05715315660697306.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         537
14      pct_chg_Load_DA         318
1           Price_lag_2         291
3         Load_DA_lag_0         194
16  roll2d_median_Price         170
2           Price_lag_7         163
11           NGas_lag_2         162
10           Coal_lag_2         113
15       lag168_Load_DA         107
13            EUA_lag_2         103
12            Oil_lag_2          94
5       WindOn_DA_lag_0          50
7                  WD_1           6
9                  WD_7           4
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          37
3         Load_DA_lag_0          12
15       lag168_Load_DA          10
5       WindOn_DA_lag_0           8
10           Coal_lag_2           7
11           NGas_lag_2           1
16  roll2d_median_Price           1
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 3.97% done
********************* END NS.. 28 ****************************************************
********************* START NS ... 29 ****************************************************
[I 2025-06-15 19:45:40,545] A new study created in memory with name: no-name-a1180d37-b043-4f68-ae43-7b13a9749388
[I 2025-06-15 19:45:41,052] Trial 0 finished with value: 0.0014269583583545777 and parameters: {'n_estimators': 269, 'learning_rate': 0.029588580619435093, 'num_leaves': 225, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.8655930684433856, 'colsample_bytree': 0.6709552245524532, 'reg_alpha': 9.831557613748133, 'reg_lambda': 9.35360879662564, 'min_split_gain': 0.036788395394734545}. Best is trial 0 with value: 0.0014269583583545777.
[I 2025-06-15 19:45:41,231] Trial 1 finished with value: 0.0014483234871788483 and parameters: {'n_estimators': 140, 'learning_rate': 0.2211155363793338, 'num_leaves': 240, 'max_depth': 15, 'min_child_samples': 15, 'subsample': 0.7341116720610548, 'colsample_bytree': 0.8303804159332406, 'reg_alpha': 6.964095677281101, 'reg_lambda': 1.8170303471147786, 'min_split_gain': 0.011294737394210625}. Best is trial 0 with value: 0.0014269583583545777.
[Trial 0] New best: 0.00143
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:41,462] Trial 2 finished with value: 0.004731027017787695 and parameters: {'n_estimators': 59, 'learning_rate': 0.018131981908018943, 'num_leaves': 89, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.5350082666878511, 'colsample_bytree': 0.5798088807454046, 'reg_alpha': 2.977850021397621, 'reg_lambda': 3.979307948499815, 'min_split_gain': 0.04765880588803446}. Best is trial 0 with value: 0.0014269583583545777.
[I 2025-06-15 19:45:41,619] Trial 3 finished with value: 0.0019918378069628925 and parameters: {'n_estimators': 113, 'learning_rate': 0.11233478171728396, 'num_leaves': 33, 'max_depth': 4, 'min_child_samples': 13, 'subsample': 0.8631503522624202, 'colsample_bytree': 0.5504118456039923, 'reg_alpha': 5.147987886065396, 'reg_lambda': 1.6311165430110408, 'min_split_gain': 0.046674486531969595}. Best is trial 0 with value: 0.0014269583583545777.
[I 2025-06-15 19:45:41,620] A new study created in memory with name: no-name-8c08c989-7be5-4b49-8873-a0c763e67d76
[I 2025-06-15 19:45:41,669] Trial 0 finished with value: 0.13549288354121614 and parameters: {'n_estimators': 270, 'learning_rate': 0.026194037797410603, 'num_leaves': 113, 'max_depth': 11, 'min_child_samples': 11, 'subsample': 0.9452125176518327, 'colsample_bytree': 0.6402603274514929, 'reg_alpha': 7.125649320244536, 'reg_lambda': 9.122785650098532, 'min_split_gain': 0.038504846089057736}. Best is trial 0 with value: 0.13549288354121614.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1345
1           Price_lag_2         918
14      pct_chg_Load_DA         665
3         Load_DA_lag_0         448
16  roll2d_median_Price         389
2           Price_lag_7         354
11           NGas_lag_2         304
13            EUA_lag_2         255
5       WindOn_DA_lag_0         249
15       lag168_Load_DA         237
10           Coal_lag_2         215
12            Oil_lag_2         185
9                  WD_7          32
8                  WD_6          21
7                  WD_1          16
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         139
3         Load_DA_lag_0          58
5       WindOn_DA_lag_0          55
16  roll2d_median_Price          37
1           Price_lag_2          14
10           Coal_lag_2           4
11           NGas_lag_2           1
13            EUA_lag_2           1
4        Solar_DA_lag_0           0
2           Price_lag_7           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
12            Oil_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 4.10% done
********************* END NS.. 29 ****************************************************
********************* START NS ... 30 ****************************************************
[I 2025-06-15 19:45:42,703] A new study created in memory with name: no-name-79486b1a-22f0-4b9b-841d-d1282f5d5548
[I 2025-06-15 19:45:42,995] Trial 0 finished with value: 0.003697157867904486 and parameters: {'n_estimators': 274, 'learning_rate': 0.08651319453679257, 'num_leaves': 127, 'max_depth': 16, 'min_child_samples': 17, 'subsample': 0.5052667942176208, 'colsample_bytree': 0.5116411346689704, 'reg_alpha': 4.732887140985657, 'reg_lambda': 0.6014344453797138, 'min_split_gain': 0.03549691475712235}. Best is trial 0 with value: 0.003697157867904486.
[Trial 0] New best: 0.00370
[I 2025-06-15 19:45:43,503] Trial 1 finished with value: 0.003276457357440135 and parameters: {'n_estimators': 222, 'learning_rate': 0.02851504824168628, 'num_leaves': 181, 'max_depth': 11, 'min_child_samples': 18, 'subsample': 0.559554279607916, 'colsample_bytree': 0.6507782272684168, 'reg_alpha': 9.66109466911272, 'reg_lambda': 0.7569537980849006, 'min_split_gain': 0.030784518327535065}. Best is trial 1 with value: 0.003276457357440135.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:44,162] Trial 2 finished with value: 0.002605198075246246 and parameters: {'n_estimators': 114, 'learning_rate': 0.01944370121711396, 'num_leaves': 86, 'max_depth': 14, 'min_child_samples': 15, 'subsample': 0.6802619803712322, 'colsample_bytree': 0.6868250402283609, 'reg_alpha': 0.011608575500144935, 'reg_lambda': 8.106514607073272, 'min_split_gain': 0.026577657329189427}. Best is trial 2 with value: 0.002605198075246246.
[I 2025-06-15 19:45:44,296] Trial 3 finished with value: 0.0036000345281553235 and parameters: {'n_estimators': 109, 'learning_rate': 0.1532376879188605, 'num_leaves': 79, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.5450487905144867, 'colsample_bytree': 0.8113643091235803, 'reg_alpha': 9.81167375462766, 'reg_lambda': 0.791309001895405, 'min_split_gain': 0.02493142280316111}. Best is trial 2 with value: 0.002605198075246246.
[Trial 2] New best: 0.00261
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:44,595] Trial 4 finished with value: 0.002148771647260414 and parameters: {'n_estimators': 52, 'learning_rate': 0.08434027947902989, 'num_leaves': 247, 'max_depth': 11, 'min_child_samples': 8, 'subsample': 0.8717483621907105, 'colsample_bytree': 0.657058600355287, 'reg_alpha': 0.6074470711275992, 'reg_lambda': 8.270569425941423, 'min_split_gain': 0.04199905392096552}. Best is trial 4 with value: 0.002148771647260414.
[I 2025-06-15 19:45:44,768] Trial 5 finished with value: 0.0030276422705102523 and parameters: {'n_estimators': 159, 'learning_rate': 0.23478205524077594, 'num_leaves': 156, 'max_depth': 5, 'min_child_samples': 24, 'subsample': 0.8875339991370461, 'colsample_bytree': 0.6071212679417932, 'reg_alpha': 5.966751920643436, 'reg_lambda': 2.2873196681882515, 'min_split_gain': 0.03951323524431809}. Best is trial 4 with value: 0.002148771647260414.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:44,771] A new study created in memory with name: no-name-43d6acb9-a681-41db-b428-e1f4c76bdb88
[I 2025-06-15 19:45:44,831] Trial 0 finished with value: 0.1853157113022028 and parameters: {'n_estimators': 135, 'learning_rate': 0.030088401992856492, 'num_leaves': 194, 'max_depth': 5, 'min_child_samples': 29, 'subsample': 0.92180484203665, 'colsample_bytree': 0.9241822707701786, 'reg_alpha': 8.113931859967309, 'reg_lambda': 1.0393920824286684, 'min_split_gain': 0.0077060186028248935}. Best is trial 0 with value: 0.1853157113022028.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         662
1           Price_lag_2         617
14      pct_chg_Load_DA         428
3         Load_DA_lag_0         384
16  roll2d_median_Price         305
2           Price_lag_7         279
15       lag168_Load_DA         227
11           NGas_lag_2         221
10           Coal_lag_2         214
5       WindOn_DA_lag_0         194
12            Oil_lag_2         190
13            EUA_lag_2         186
9                  WD_7          16
7                  WD_1           9
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         175
3         Load_DA_lag_0          44
16  roll2d_median_Price          28
5       WindOn_DA_lag_0          25
1           Price_lag_2           7
2           Price_lag_7           0
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 4.24% done
********************* END NS.. 30 ****************************************************
********************* START NS ... 31 ****************************************************
[I 2025-06-15 19:45:45,595] A new study created in memory with name: no-name-f4cea506-775d-497e-8be8-0812c24861eb
[I 2025-06-15 19:45:45,743] Trial 0 finished with value: 0.002320091838294704 and parameters: {'n_estimators': 274, 'learning_rate': 0.27133630465834746, 'num_leaves': 109, 'max_depth': 16, 'min_child_samples': 23, 'subsample': 0.5434959843855041, 'colsample_bytree': 0.692094661841856, 'reg_alpha': 9.361501485129303, 'reg_lambda': 7.446705445559216, 'min_split_gain': 0.034120300829183624}. Best is trial 0 with value: 0.002320091838294704.
[Trial 0] New best: 0.00232
[I 2025-06-15 19:45:46,107] Trial 1 finished with value: 0.0025287311684473 and parameters: {'n_estimators': 291, 'learning_rate': 0.08049338087865406, 'num_leaves': 238, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.5628483081433431, 'colsample_bytree': 0.8114213714722178, 'reg_alpha': 5.430499779013464, 'reg_lambda': 6.944945563224786, 'min_split_gain': 0.02556523059606382}. Best is trial 0 with value: 0.002320091838294704.
[I 2025-06-15 19:45:46,290] Trial 2 finished with value: 0.004285229277050812 and parameters: {'n_estimators': 96, 'learning_rate': 0.1458362147053827, 'num_leaves': 255, 'max_depth': 5, 'min_child_samples': 26, 'subsample': 0.5501232619344976, 'colsample_bytree': 0.5538388965102239, 'reg_alpha': 3.1599937262145428, 'reg_lambda': 6.43308244707602, 'min_split_gain': 0.013514416236910944}. Best is trial 0 with value: 0.002320091838294704.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:47,174] Trial 3 finished with value: 0.002378838261310758 and parameters: {'n_estimators': 292, 'learning_rate': 0.02409474895635317, 'num_leaves': 113, 'max_depth': 12, 'min_child_samples': 9, 'subsample': 0.5010497510560177, 'colsample_bytree': 0.7894151639800091, 'reg_alpha': 5.851853013207787, 'reg_lambda': 0.40614631587534866, 'min_split_gain': 0.017494356035273002}. Best is trial 0 with value: 0.002320091838294704.
[I 2025-06-15 19:45:47,177] A new study created in memory with name: no-name-4b8beb86-86b1-40a9-bb43-c288282d28a5
[I 2025-06-15 19:45:47,215] Trial 0 finished with value: 0.10131900905951609 and parameters: {'n_estimators': 206, 'learning_rate': 0.016094583903088547, 'num_leaves': 209, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.7093280493906488, 'colsample_bytree': 0.9257621676759791, 'reg_alpha': 7.871547123667787, 'reg_lambda': 1.0902978898247817, 'min_split_gain': 0.004048024041770482}. Best is trial 0 with value: 0.10131900905951609.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         154
1           Price_lag_2          65
14      pct_chg_Load_DA          52
3         Load_DA_lag_0          46
2           Price_lag_7          38
11           NGas_lag_2          38
16  roll2d_median_Price          33
13            EUA_lag_2          32
15       lag168_Load_DA          26
10           Coal_lag_2          22
12            Oil_lag_2          17
5       WindOn_DA_lag_0          13
9                  WD_7           2
8                  WD_6           2
7                  WD_1           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         296
16  roll2d_median_Price          79
3         Load_DA_lag_0          21
1           Price_lag_2           2
2           Price_lag_7           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 4.38% done
********************* END NS.. 31 ****************************************************
********************* START NS ... 32 ****************************************************
[I 2025-06-15 19:45:47,992] A new study created in memory with name: no-name-fe718f7d-c431-4c85-9ee7-ccd99791a721
[I 2025-06-15 19:45:48,113] Trial 0 finished with value: 0.007731495318401806 and parameters: {'n_estimators': 57, 'learning_rate': 0.015063751985910983, 'num_leaves': 142, 'max_depth': 5, 'min_child_samples': 22, 'subsample': 0.6496632449489338, 'colsample_bytree': 0.8886767837442144, 'reg_alpha': 7.444030510166193, 'reg_lambda': 0.37295891848134977, 'min_split_gain': 0.046161021735286474}. Best is trial 0 with value: 0.007731495318401806.
[Trial 0] New best: 0.00773
[I 2025-06-15 19:45:48,426] Trial 1 finished with value: 0.0018659020197397084 and parameters: {'n_estimators': 150, 'learning_rate': 0.021932944847847226, 'num_leaves': 42, 'max_depth': 5, 'min_child_samples': 13, 'subsample': 0.6086952859847182, 'colsample_bytree': 0.7061661743842436, 'reg_alpha': 3.2432083257404454, 'reg_lambda': 7.060719911490619, 'min_split_gain': 0.04053013187550821}. Best is trial 1 with value: 0.0018659020197397084.
[Trial 1] New best: 0.00187
[I 2025-06-15 19:45:49,417] Trial 2 finished with value: 0.001472656062892152 and parameters: {'n_estimators': 269, 'learning_rate': 0.01733406368178148, 'num_leaves': 186, 'max_depth': 7, 'min_child_samples': 4, 'subsample': 0.8687340622912312, 'colsample_bytree': 0.9221139105528355, 'reg_alpha': 3.1663394419518687, 'reg_lambda': 5.415181629216445, 'min_split_gain': 0.011633589304182686}. Best is trial 2 with value: 0.001472656062892152.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:49,894] Trial 3 finished with value: 0.0015078578247190286 and parameters: {'n_estimators': 201, 'learning_rate': 0.09298895326458285, 'num_leaves': 213, 'max_depth': 7, 'min_child_samples': 15, 'subsample': 0.7391120951473029, 'colsample_bytree': 0.76619659780011, 'reg_alpha': 4.683812847979041, 'reg_lambda': 7.652510613748616, 'min_split_gain': 0.013653999511824522}. Best is trial 2 with value: 0.001472656062892152.
[I 2025-06-15 19:45:50,076] Trial 4 finished with value: 0.001668306334390659 and parameters: {'n_estimators': 86, 'learning_rate': 0.05049720069817687, 'num_leaves': 253, 'max_depth': 4, 'min_child_samples': 16, 'subsample': 0.8139791006600943, 'colsample_bytree': 0.8049848398963837, 'reg_alpha': 0.2993379373968974, 'reg_lambda': 0.6838753310472501, 'min_split_gain': 0.009152181058320259}. Best is trial 2 with value: 0.001472656062892152.
[I 2025-06-15 19:45:50,078] A new study created in memory with name: no-name-303c1ee5-db6c-453e-9b2d-a6d0e3a46bfb
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 19:45:50,138] Trial 0 finished with value: 0.010826241264875268 and parameters: {'n_estimators': 258, 'learning_rate': 0.018139420051303867, 'num_leaves': 213, 'max_depth': 11, 'min_child_samples': 12, 'subsample': 0.7924810527807568, 'colsample_bytree': 0.5776022461962069, 'reg_alpha': 3.0515163424570724, 'reg_lambda': 0.7543598413928709, 'min_split_gain': 0.04933289383659693}. Best is trial 0 with value: 0.010826241264875268.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3260
1           Price_lag_2        2098
14      pct_chg_Load_DA        1783
16  roll2d_median_Price        1046
3         Load_DA_lag_0        1045
2           Price_lag_7         932
11           NGas_lag_2         831
10           Coal_lag_2         763
13            EUA_lag_2         586
12            Oil_lag_2         557
5       WindOn_DA_lag_0         506
15       lag168_Load_DA         461
9                  WD_7          25
7                  WD_1           9
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         187
1           Price_lag_2          41
16  roll2d_median_Price          39
3         Load_DA_lag_0          34
5       WindOn_DA_lag_0           8
2           Price_lag_7           8
15       lag168_Load_DA           4
12            Oil_lag_2           2
10           Coal_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
11           NGas_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 4.51% done
********************* END NS.. 32 ****************************************************
********************* START NS ... 33 ****************************************************
[I 2025-06-15 19:45:51,475] A new study created in memory with name: no-name-858dced4-9596-4f25-a8cb-0145f804051e
[I 2025-06-15 19:45:51,691] Trial 0 finished with value: 0.009125947524531225 and parameters: {'n_estimators': 275, 'learning_rate': 0.14632937363148552, 'num_leaves': 89, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.6411868138868916, 'colsample_bytree': 0.6906234029819376, 'reg_alpha': 3.6108895907204817, 'reg_lambda': 5.281758725277129, 'min_split_gain': 0.02168234135116053}. Best is trial 0 with value: 0.009125947524531225.
[Trial 0] New best: 0.00913
[I 2025-06-15 19:45:52,282] Trial 1 finished with value: 0.009149461972363913 and parameters: {'n_estimators': 262, 'learning_rate': 0.04826907279566994, 'num_leaves': 186, 'max_depth': 14, 'min_child_samples': 20, 'subsample': 0.8564386104146134, 'colsample_bytree': 0.5746722636183021, 'reg_alpha': 4.64801378155472, 'reg_lambda': 1.1424607049580504, 'min_split_gain': 0.04157864708471164}. Best is trial 0 with value: 0.009125947524531225.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:53,012] Trial 2 finished with value: 0.010571976176920938 and parameters: {'n_estimators': 287, 'learning_rate': 0.016137834146319516, 'num_leaves': 48, 'max_depth': 16, 'min_child_samples': 25, 'subsample': 0.9721300505262578, 'colsample_bytree': 0.9600911740463712, 'reg_alpha': 9.14019839895026, 'reg_lambda': 8.42133198020675, 'min_split_gain': 0.027011233753676995}. Best is trial 0 with value: 0.009125947524531225.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:53,330] Trial 3 finished with value: 0.010501860384548853 and parameters: {'n_estimators': 96, 'learning_rate': 0.0422866074915843, 'num_leaves': 169, 'max_depth': 15, 'min_child_samples': 20, 'subsample': 0.7393772953180754, 'colsample_bytree': 0.6540919116290763, 'reg_alpha': 4.723747972629242, 'reg_lambda': 5.548948905525194, 'min_split_gain': 0.035532322965226425}. Best is trial 0 with value: 0.009125947524531225.
[I 2025-06-15 19:45:53,333] A new study created in memory with name: no-name-6daff7d0-6b79-403d-a941-fb224ad67118
[I 2025-06-15 19:45:53,376] Trial 0 finished with value: 0.05738444036131052 and parameters: {'n_estimators': 219, 'learning_rate': 0.03152654521962904, 'num_leaves': 140, 'max_depth': 8, 'min_child_samples': 28, 'subsample': 0.7879281404586558, 'colsample_bytree': 0.8178889101830692, 'reg_alpha': 1.8231290176414117, 'reg_lambda': 5.42869097206818, 'min_split_gain': 0.017626142579802533}. Best is trial 0 with value: 0.05738444036131052.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         391
1           Price_lag_2         268
14      pct_chg_Load_DA         227
3         Load_DA_lag_0         196
15       lag168_Load_DA         155
16  roll2d_median_Price         137
11           NGas_lag_2         134
2           Price_lag_7         128
10           Coal_lag_2         111
5       WindOn_DA_lag_0          98
13            EUA_lag_2          96
12            Oil_lag_2          85
9                  WD_7           6
7                  WD_1           5
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         158
3         Load_DA_lag_0          33
16  roll2d_median_Price          16
1           Price_lag_2          12
14      pct_chg_Load_DA           4
5       WindOn_DA_lag_0           2
2           Price_lag_7           0
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 4.65% done
********************* END NS.. 33 ****************************************************
********************* START NS ... 34 ****************************************************
[I 2025-06-15 19:45:54,156] A new study created in memory with name: no-name-8e797569-dc0b-42c3-9648-fbe9cbffbda3
[I 2025-06-15 19:45:54,367] Trial 0 finished with value: 0.00827297927455877 and parameters: {'n_estimators': 288, 'learning_rate': 0.20386242957509057, 'num_leaves': 109, 'max_depth': 9, 'min_child_samples': 7, 'subsample': 0.7527885401248287, 'colsample_bytree': 0.9395832862342003, 'reg_alpha': 4.41458636497095, 'reg_lambda': 0.15589328512914613, 'min_split_gain': 0.006774189249404594}. Best is trial 0 with value: 0.00827297927455877.
[I 2025-06-15 19:45:54,566] Trial 1 finished with value: 0.008904854841566385 and parameters: {'n_estimators': 232, 'learning_rate': 0.1750224726992833, 'num_leaves': 177, 'max_depth': 5, 'min_child_samples': 26, 'subsample': 0.6129548844509036, 'colsample_bytree': 0.7584469624193922, 'reg_alpha': 4.66723858516792, 'reg_lambda': 6.519823447965535, 'min_split_gain': 0.01237753112495752}. Best is trial 0 with value: 0.00827297927455877.
[Trial 0] New best: 0.00827
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:55,013] Trial 2 finished with value: 0.008768829234152236 and parameters: {'n_estimators': 290, 'learning_rate': 0.08716094478972007, 'num_leaves': 189, 'max_depth': 9, 'min_child_samples': 3, 'subsample': 0.9586554608659483, 'colsample_bytree': 0.6860220649758877, 'reg_alpha': 7.142294483816301, 'reg_lambda': 6.717136434290446, 'min_split_gain': 0.014099854453868955}. Best is trial 0 with value: 0.00827297927455877.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:55,295] Trial 3 finished with value: 0.009371373422601883 and parameters: {'n_estimators': 277, 'learning_rate': 0.1512953124895731, 'num_leaves': 111, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.9061780617135722, 'colsample_bytree': 0.9201367966933891, 'reg_alpha': 4.898270814394086, 'reg_lambda': 1.2270200677842202, 'min_split_gain': 0.019850865106031734}. Best is trial 0 with value: 0.00827297927455877.
[I 2025-06-15 19:45:55,297] A new study created in memory with name: no-name-3128da28-1100-4d91-8aba-ac824b60d404
[I 2025-06-15 19:45:55,325] Trial 0 finished with value: 0.14532927181846067 and parameters: {'n_estimators': 187, 'learning_rate': 0.010658030280637791, 'num_leaves': 63, 'max_depth': 8, 'min_child_samples': 22, 'subsample': 0.5915849278241447, 'colsample_bytree': 0.633757476068578, 'reg_alpha': 7.762210572779894, 'reg_lambda': 8.058680226057366, 'min_split_gain': 0.03487455083743695}. Best is trial 0 with value: 0.14532927181846067.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         371
14      pct_chg_Load_DA         271
3         Load_DA_lag_0         231
1           Price_lag_2         214
5       WindOn_DA_lag_0         184
16  roll2d_median_Price         151
2           Price_lag_7         147
15       lag168_Load_DA         143
11           NGas_lag_2         120
13            EUA_lag_2         120
12            Oil_lag_2         112
10           Coal_lag_2         108
7                  WD_1           6
9                  WD_7           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         119
3         Load_DA_lag_0          26
1           Price_lag_2          22
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 4.79% done
********************* END NS.. 34 ****************************************************
********************* START NS ... 35 ****************************************************
[I 2025-06-15 19:45:56,205] A new study created in memory with name: no-name-7b838315-bcb3-42ee-ad93-e0281d908880
[I 2025-06-15 19:45:56,855] Trial 0 finished with value: 0.004253102345567941 and parameters: {'n_estimators': 226, 'learning_rate': 0.04658131215003313, 'num_leaves': 243, 'max_depth': 16, 'min_child_samples': 20, 'subsample': 0.5971357104058616, 'colsample_bytree': 0.9990302071113817, 'reg_alpha': 2.1972971106180115, 'reg_lambda': 3.2749328040861547, 'min_split_gain': 0.019613746691010805}. Best is trial 0 with value: 0.004253102345567941.
[Trial 0] New best: 0.00425
[I 2025-06-15 19:45:57,277] Trial 1 finished with value: 0.003564718411822278 and parameters: {'n_estimators': 171, 'learning_rate': 0.0939946290562352, 'num_leaves': 201, 'max_depth': 13, 'min_child_samples': 18, 'subsample': 0.7881146834200701, 'colsample_bytree': 0.9582185707443365, 'reg_alpha': 5.364632671709417, 'reg_lambda': 9.99576114820339, 'min_split_gain': 0.004527920222916565}. Best is trial 1 with value: 0.003564718411822278.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:45:57,509] Trial 2 finished with value: 0.004256485442940127 and parameters: {'n_estimators': 139, 'learning_rate': 0.02262225212933272, 'num_leaves': 216, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.5658010381973823, 'colsample_bytree': 0.9523893005450642, 'reg_alpha': 5.293930341476249, 'reg_lambda': 9.070003410574333, 'min_split_gain': 0.0021810316158339784}. Best is trial 1 with value: 0.003564718411822278.
[I 2025-06-15 19:45:57,709] Trial 3 finished with value: 0.003521906740380136 and parameters: {'n_estimators': 129, 'learning_rate': 0.17615360825274723, 'num_leaves': 115, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.9289979736755374, 'colsample_bytree': 0.9231532353422963, 'reg_alpha': 6.001837244733626, 'reg_lambda': 1.2370591873566328, 'min_split_gain': 0.01320118947714527}. Best is trial 3 with value: 0.003521906740380136.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:45:57,712] A new study created in memory with name: no-name-9f4af481-6366-4c27-98fb-a4a0ccd330d2
[I 2025-06-15 19:45:57,746] Trial 0 finished with value: 0.052456348038239974 and parameters: {'n_estimators': 296, 'learning_rate': 0.04571736230273309, 'num_leaves': 142, 'max_depth': 10, 'min_child_samples': 14, 'subsample': 0.7294126113651502, 'colsample_bytree': 0.8920152905730511, 'reg_alpha': 8.954792258555429, 'reg_lambda': 8.359759248033926, 'min_split_gain': 0.04824367292509416}. Best is trial 0 with value: 0.052456348038239974.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         312
14      pct_chg_Load_DA         177
3         Load_DA_lag_0         148
1           Price_lag_2         137
5       WindOn_DA_lag_0          88
16  roll2d_median_Price          84
11           NGas_lag_2          82
2           Price_lag_7          80
13            EUA_lag_2          79
15       lag168_Load_DA          76
12            Oil_lag_2          66
10           Coal_lag_2          55
9                  WD_7           5
7                  WD_1           3
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          39
1           Price_lag_2           3
3         Load_DA_lag_0           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 4.92% done
********************* END NS.. 35 ****************************************************
********************* START NS ... 36 ****************************************************
[I 2025-06-15 19:45:58,487] A new study created in memory with name: no-name-0e93d525-72f2-45b0-bc49-14d106b798e0
[I 2025-06-15 19:45:58,714] Trial 0 finished with value: 0.0275458807682437 and parameters: {'n_estimators': 51, 'learning_rate': 0.019707889338350222, 'num_leaves': 74, 'max_depth': 11, 'min_child_samples': 24, 'subsample': 0.535510597921359, 'colsample_bytree': 0.66402637622606, 'reg_alpha': 5.605025404134832, 'reg_lambda': 4.109841060194355, 'min_split_gain': 0.020970100316865616}. Best is trial 0 with value: 0.0275458807682437.
[Trial 0] New best: 0.02755
[I 2025-06-15 19:45:59,023] Trial 1 finished with value: 0.0034588646123064047 and parameters: {'n_estimators': 289, 'learning_rate': 0.17697704281352816, 'num_leaves': 109, 'max_depth': 11, 'min_child_samples': 17, 'subsample': 0.9473404376257475, 'colsample_bytree': 0.746841395847353, 'reg_alpha': 5.750371898388206, 'reg_lambda': 8.769878549670613, 'min_split_gain': 0.0060089612930757045}. Best is trial 1 with value: 0.0034588646123064047.
[Trial 1] New best: 0.00346
[I 2025-06-15 19:45:59,879] Trial 2 finished with value: 0.004485908518651101 and parameters: {'n_estimators': 216, 'learning_rate': 0.01177375766278525, 'num_leaves': 143, 'max_depth': 9, 'min_child_samples': 23, 'subsample': 0.8409347597258771, 'colsample_bytree': 0.966688791382049, 'reg_alpha': 1.5797958884540708, 'reg_lambda': 4.2596847071668, 'min_split_gain': 0.02503944511468051}. Best is trial 1 with value: 0.0034588646123064047.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:00,106] Trial 3 finished with value: 0.0034591773343842975 and parameters: {'n_estimators': 123, 'learning_rate': 0.09780038821505527, 'num_leaves': 63, 'max_depth': 7, 'min_child_samples': 24, 'subsample': 0.7669222576230177, 'colsample_bytree': 0.779272505583714, 'reg_alpha': 8.866049753948257, 'reg_lambda': 1.743940892539283, 'min_split_gain': 0.04029293071085052}. Best is trial 1 with value: 0.0034588646123064047.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:00,460] Trial 4 finished with value: 0.003651160702552116 and parameters: {'n_estimators': 126, 'learning_rate': 0.056851706051742355, 'num_leaves': 79, 'max_depth': 12, 'min_child_samples': 3, 'subsample': 0.7975860628872149, 'colsample_bytree': 0.8040976190131396, 'reg_alpha': 7.675575831664505, 'reg_lambda': 7.050653280171678, 'min_split_gain': 0.04439978390828367}. Best is trial 1 with value: 0.0034588646123064047.
[I 2025-06-15 19:46:00,462] A new study created in memory with name: no-name-6c4c3da2-12ca-4584-8e1e-eb2173f48df5
[I 2025-06-15 19:46:00,529] Trial 0 finished with value: 0.016297181155900214 and parameters: {'n_estimators': 289, 'learning_rate': 0.028615063895089297, 'num_leaves': 167, 'max_depth': 12, 'min_child_samples': 25, 'subsample': 0.5450926750088125, 'colsample_bytree': 0.7007873661268058, 'reg_alpha': 0.42871724992444404, 'reg_lambda': 1.3198718329918901, 'min_split_gain': 0.007140078465796701}. Best is trial 0 with value: 0.016297181155900214.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         352
14      pct_chg_Load_DA         274
1           Price_lag_2         267
3         Load_DA_lag_0         217
5       WindOn_DA_lag_0         158
15       lag168_Load_DA         154
13            EUA_lag_2         143
10           Coal_lag_2         132
2           Price_lag_7         130
16  roll2d_median_Price         119
11           NGas_lag_2         114
12            Oil_lag_2          76
7                  WD_1           9
9                  WD_7           6
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         199
16  roll2d_median_Price         142
3         Load_DA_lag_0         109
14      pct_chg_Load_DA          87
5       WindOn_DA_lag_0          40
12            Oil_lag_2          38
11           NGas_lag_2          22
1           Price_lag_2          18
10           Coal_lag_2           4
2           Price_lag_7           3
13            EUA_lag_2           2
15       lag168_Load_DA           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
8                  WD_6           0
-> 5.06% done
********************* END NS.. 36 ****************************************************
********************* START NS ... 37 ****************************************************
[I 2025-06-15 19:46:01,426] A new study created in memory with name: no-name-64a6e09c-7c96-4945-9b5f-f749ff5478ad
[I 2025-06-15 19:46:01,584] Trial 0 finished with value: 0.018152256898947283 and parameters: {'n_estimators': 61, 'learning_rate': 0.045582457425256635, 'num_leaves': 72, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.6038691340809059, 'colsample_bytree': 0.970019204718986, 'reg_alpha': 5.42388387860946, 'reg_lambda': 7.686036049648161, 'min_split_gain': 0.04738417145567587}. Best is trial 0 with value: 0.018152256898947283.
[Trial 0] New best: 0.01815
[I 2025-06-15 19:46:02,043] Trial 1 finished with value: 0.012525277498957202 and parameters: {'n_estimators': 222, 'learning_rate': 0.05156840727085879, 'num_leaves': 242, 'max_depth': 13, 'min_child_samples': 21, 'subsample': 0.9494787845527628, 'colsample_bytree': 0.9559575480899074, 'reg_alpha': 5.127258923418906, 'reg_lambda': 2.0902202435889894, 'min_split_gain': 0.02549143586591028}. Best is trial 1 with value: 0.012525277498957202.
[Trial 1] New best: 0.01253
[I 2025-06-15 19:46:02,400] Trial 2 finished with value: 0.01468299691617804 and parameters: {'n_estimators': 113, 'learning_rate': 0.04705332637554843, 'num_leaves': 46, 'max_depth': 15, 'min_child_samples': 24, 'subsample': 0.7296706234508985, 'colsample_bytree': 0.6119691367726603, 'reg_alpha': 7.147896882848446, 'reg_lambda': 6.446534050923272, 'min_split_gain': 0.0077934907962135536}. Best is trial 1 with value: 0.012525277498957202.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:02,820] Trial 3 finished with value: 0.010008048049771468 and parameters: {'n_estimators': 293, 'learning_rate': 0.141545684351053, 'num_leaves': 235, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.9500495001532485, 'colsample_bytree': 0.5401683865258018, 'reg_alpha': 2.5042514614685563, 'reg_lambda': 2.4743563605411456, 'min_split_gain': 0.01215446505042559}. Best is trial 3 with value: 0.010008048049771468.
[Trial 3] New best: 0.01001
[I 2025-06-15 19:46:03,179] Trial 4 finished with value: 0.012794414086044875 and parameters: {'n_estimators': 88, 'learning_rate': 0.07673650838359168, 'num_leaves': 72, 'max_depth': 11, 'min_child_samples': 2, 'subsample': 0.6438015667436906, 'colsample_bytree': 0.5467646698301667, 'reg_alpha': 5.206406240472145, 'reg_lambda': 3.4733349467817165, 'min_split_gain': 0.008341550236393857}. Best is trial 3 with value: 0.010008048049771468.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:03,464] Trial 5 finished with value: 0.010212513528467068 and parameters: {'n_estimators': 181, 'learning_rate': 0.16710988019128725, 'num_leaves': 217, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.9462006221367663, 'colsample_bytree': 0.7973893442967492, 'reg_alpha': 0.9800833931807917, 'reg_lambda': 1.9078620277496816, 'min_split_gain': 0.03607558149421537}. Best is trial 3 with value: 0.010008048049771468.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:04,259] Trial 6 finished with value: 0.015038795839595406 and parameters: {'n_estimators': 260, 'learning_rate': 0.020932876122869367, 'num_leaves': 212, 'max_depth': 8, 'min_child_samples': 5, 'subsample': 0.6317587931261731, 'colsample_bytree': 0.7691556672664734, 'reg_alpha': 8.20061864725228, 'reg_lambda': 2.835691788650948, 'min_split_gain': 0.026536866150229894}. Best is trial 3 with value: 0.010008048049771468.
[I 2025-06-15 19:46:04,261] A new study created in memory with name: no-name-c4d0705f-b641-4aec-b8a9-2831ebb262ea
[I 2025-06-15 19:46:04,314] Trial 0 finished with value: 0.11620244916528788 and parameters: {'n_estimators': 162, 'learning_rate': 0.10898035306327428, 'num_leaves': 60, 'max_depth': 12, 'min_child_samples': 14, 'subsample': 0.9181374273297781, 'colsample_bytree': 0.5197028799476546, 'reg_alpha': 9.600159187210044, 'reg_lambda': 8.212529316556994, 'min_split_gain': 0.018858235549519983}. Best is trial 0 with value: 0.11620244916528788.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         596
1           Price_lag_2         528
14      pct_chg_Load_DA         370
3         Load_DA_lag_0         365
2           Price_lag_7         315
15       lag168_Load_DA         280
16  roll2d_median_Price         259
5       WindOn_DA_lag_0         253
11           NGas_lag_2         212
13            EUA_lag_2         192
10           Coal_lag_2         175
12            Oil_lag_2         175
7                  WD_1          22
9                  WD_7          13
8                  WD_6          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          27
1           Price_lag_2           8
3         Load_DA_lag_0           2
11           NGas_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 5.20% done
********************* END NS.. 37 ****************************************************
********************* START NS ... 38 ****************************************************
[I 2025-06-15 19:46:05,358] A new study created in memory with name: no-name-73f461fa-bf14-4053-9f5d-29605ba61d20
[I 2025-06-15 19:46:05,542] Trial 0 finished with value: 0.01214206972426165 and parameters: {'n_estimators': 246, 'learning_rate': 0.27725911967742, 'num_leaves': 235, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.6162450202287615, 'colsample_bytree': 0.7164809293864929, 'reg_alpha': 6.161663340409128, 'reg_lambda': 0.5414456573402837, 'min_split_gain': 0.005412805311010022}. Best is trial 0 with value: 0.01214206972426165.
[Trial 0] New best: 0.01214
[I 2025-06-15 19:46:05,923] Trial 1 finished with value: 0.013428994446680076 and parameters: {'n_estimators': 240, 'learning_rate': 0.19327136034774262, 'num_leaves': 89, 'max_depth': 12, 'min_child_samples': 4, 'subsample': 0.7596560305378248, 'colsample_bytree': 0.7882972737916445, 'reg_alpha': 1.9681362017444992, 'reg_lambda': 8.17270113905121, 'min_split_gain': 0.017921774078819848}. Best is trial 0 with value: 0.01214206972426165.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:06,827] Trial 2 finished with value: 0.013881736448568154 and parameters: {'n_estimators': 251, 'learning_rate': 0.01363447186875382, 'num_leaves': 31, 'max_depth': 16, 'min_child_samples': 10, 'subsample': 0.7445969500986731, 'colsample_bytree': 0.7662241592955396, 'reg_alpha': 0.616415028178332, 'reg_lambda': 5.0935809699007715, 'min_split_gain': 0.004126539723862138}. Best is trial 0 with value: 0.01214206972426165.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:07,398] Trial 3 finished with value: 0.012319343067684651 and parameters: {'n_estimators': 167, 'learning_rate': 0.09510832247524043, 'num_leaves': 236, 'max_depth': 9, 'min_child_samples': 29, 'subsample': 0.8680572593059517, 'colsample_bytree': 0.6147677795588471, 'reg_alpha': 5.626577311982155, 'reg_lambda': 3.649091364759033, 'min_split_gain': 0.007724997192266426}. Best is trial 0 with value: 0.01214206972426165.
[I 2025-06-15 19:46:07,401] A new study created in memory with name: no-name-b1625def-4898-4c1f-8b5b-3d3957296306
[I 2025-06-15 19:46:07,435] Trial 0 finished with value: 0.08091271601377066 and parameters: {'n_estimators': 65, 'learning_rate': 0.025226346176000152, 'num_leaves': 94, 'max_depth': 6, 'min_child_samples': 24, 'subsample': 0.645170463648906, 'colsample_bytree': 0.5710297910345143, 'reg_alpha': 5.617485742657782, 'reg_lambda': 0.3749917104614142, 'min_split_gain': 0.01295460934879656}. Best is trial 0 with value: 0.08091271601377066.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         187
14      pct_chg_Load_DA         107
1           Price_lag_2         103
3         Load_DA_lag_0          82
16  roll2d_median_Price          56
11           NGas_lag_2          53
13            EUA_lag_2          50
15       lag168_Load_DA          46
5       WindOn_DA_lag_0          44
2           Price_lag_7          44
10           Coal_lag_2          40
12            Oil_lag_2          21
7                  WD_1           5
9                  WD_7           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          68
1           Price_lag_2          34
3         Load_DA_lag_0          22
11           NGas_lag_2           6
13            EUA_lag_2           4
15       lag168_Load_DA           2
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 5.34% done
********************* END NS.. 38 ****************************************************
********************* START NS ... 39 ****************************************************
[I 2025-06-15 19:46:08,213] A new study created in memory with name: no-name-40b327f2-598f-4342-a308-fb506780127a
[I 2025-06-15 19:46:10,350] Trial 0 finished with value: 0.0009051941312179981 and parameters: {'n_estimators': 266, 'learning_rate': 0.024640114976587857, 'num_leaves': 82, 'max_depth': 13, 'min_child_samples': 7, 'subsample': 0.5561608015689978, 'colsample_bytree': 0.7993439133840279, 'reg_alpha': 1.0678707371699347, 'reg_lambda': 2.6250692092309302, 'min_split_gain': 0.006768560796509726}. Best is trial 0 with value: 0.0009051941312179981.
[Trial 0] New best: 0.00091
[I 2025-06-15 19:46:10,661] Trial 1 finished with value: 0.0021461189491306307 and parameters: {'n_estimators': 266, 'learning_rate': 0.1934317468402894, 'num_leaves': 164, 'max_depth': 14, 'min_child_samples': 27, 'subsample': 0.7532415038858288, 'colsample_bytree': 0.5650980479694483, 'reg_alpha': 5.042358430413463, 'reg_lambda': 0.5060115241193719, 'min_split_gain': 0.028563426699094088}. Best is trial 0 with value: 0.0009051941312179981.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:10,993] Trial 2 finished with value: 0.0013761664272751658 and parameters: {'n_estimators': 102, 'learning_rate': 0.18558146842721676, 'num_leaves': 199, 'max_depth': 15, 'min_child_samples': 27, 'subsample': 0.9389307059617424, 'colsample_bytree': 0.7435702092623684, 'reg_alpha': 3.2427298703676852, 'reg_lambda': 5.877990982548652, 'min_split_gain': 0.005618087695037743}. Best is trial 0 with value: 0.0009051941312179981.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:11,431] Trial 3 finished with value: 0.002740530783005688 and parameters: {'n_estimators': 210, 'learning_rate': 0.14551761278345182, 'num_leaves': 191, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.599662533287985, 'colsample_bytree': 0.521348294055602, 'reg_alpha': 3.340671585513161, 'reg_lambda': 4.913434691060553, 'min_split_gain': 0.0226921277773881}. Best is trial 0 with value: 0.0009051941312179981.
[I 2025-06-15 19:46:11,434] A new study created in memory with name: no-name-2d3664e5-c692-4b71-8af2-750761a3a06e
[I 2025-06-15 19:46:11,501] Trial 0 finished with value: 0.009624114865462106 and parameters: {'n_estimators': 169, 'learning_rate': 0.043196558718024455, 'num_leaves': 73, 'max_depth': 16, 'min_child_samples': 10, 'subsample': 0.7542973962595252, 'colsample_bytree': 0.540324493667078, 'reg_alpha': 6.516858160848134, 'reg_lambda': 0.3782266853022975, 'min_split_gain': 0.005094515606534589}. Best is trial 0 with value: 0.009624114865462106.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3402
1           Price_lag_2        3015
14      pct_chg_Load_DA        2721
3         Load_DA_lag_0        1947
5       WindOn_DA_lag_0        1521
15       lag168_Load_DA        1457
2           Price_lag_7        1434
16  roll2d_median_Price        1343
11           NGas_lag_2        1201
10           Coal_lag_2        1030
12            Oil_lag_2         898
13            EUA_lag_2         794
9                  WD_7          59
7                  WD_1          50
8                  WD_6          10
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          93
1           Price_lag_2          29
3         Load_DA_lag_0          19
5       WindOn_DA_lag_0           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 5.47% done
********************* END NS.. 39 ****************************************************
********************* START NS ... 40 ****************************************************
[I 2025-06-15 19:46:14,428] A new study created in memory with name: no-name-4416a7c5-18bb-40a1-975c-8816a96550ec
[I 2025-06-15 19:46:14,752] Trial 0 finished with value: 0.0008823341469157204 and parameters: {'n_estimators': 149, 'learning_rate': 0.01688952581352868, 'num_leaves': 82, 'max_depth': 4, 'min_child_samples': 1, 'subsample': 0.8607655192944846, 'colsample_bytree': 0.7918332353577044, 'reg_alpha': 9.677394700683182, 'reg_lambda': 5.2108516937065374, 'min_split_gain': 0.019654866363647452}. Best is trial 0 with value: 0.0008823341469157204.
[Trial 0] New best: 0.00088
[I 2025-06-15 19:46:15,257] Trial 1 finished with value: 0.000921897056306242 and parameters: {'n_estimators': 159, 'learning_rate': 0.0215123359319531, 'num_leaves': 82, 'max_depth': 15, 'min_child_samples': 15, 'subsample': 0.6714884451297847, 'colsample_bytree': 0.8926678730884687, 'reg_alpha': 8.846983623641245, 'reg_lambda': 0.5050951846896479, 'min_split_gain': 0.044513703076888456}. Best is trial 0 with value: 0.0008823341469157204.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:15,635] Trial 2 finished with value: 0.00133901424573545 and parameters: {'n_estimators': 274, 'learning_rate': 0.08247475657041785, 'num_leaves': 60, 'max_depth': 6, 'min_child_samples': 24, 'subsample': 0.8624320667147647, 'colsample_bytree': 0.9840816701494131, 'reg_alpha': 8.265220583035456, 'reg_lambda': 7.289437349670158, 'min_split_gain': 0.022189132295161743}. Best is trial 0 with value: 0.0008823341469157204.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:16,374] Trial 3 finished with value: 0.0007929583314244415 and parameters: {'n_estimators': 174, 'learning_rate': 0.014668203772814113, 'num_leaves': 227, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.7246859185540178, 'colsample_bytree': 0.979830440040392, 'reg_alpha': 8.502481271811718, 'reg_lambda': 6.225400285723669, 'min_split_gain': 0.02424229431511626}. Best is trial 3 with value: 0.0007929583314244415.
[I 2025-06-15 19:46:16,377] A new study created in memory with name: no-name-e3626c5e-204b-4fe8-a12c-e7a60151b907
[I 2025-06-15 19:46:16,398] Trial 0 finished with value: 0.043095955702004286 and parameters: {'n_estimators': 54, 'learning_rate': 0.159632189125975, 'num_leaves': 101, 'max_depth': 11, 'min_child_samples': 8, 'subsample': 0.8390105242445122, 'colsample_bytree': 0.8271061920215654, 'reg_alpha': 3.7547460549339307, 'reg_lambda': 6.508472167625273, 'min_split_gain': 0.013932973997137077}. Best is trial 0 with value: 0.043095955702004286.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2200
1           Price_lag_2         682
16  roll2d_median_Price         481
14      pct_chg_Load_DA         470
2           Price_lag_7         327
10           Coal_lag_2         295
13            EUA_lag_2         220
12            Oil_lag_2         204
11           NGas_lag_2         186
3         Load_DA_lag_0         115
15       lag168_Load_DA          23
5       WindOn_DA_lag_0          17
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          35
5       WindOn_DA_lag_0           7
3         Load_DA_lag_0           5
1           Price_lag_2           4
16  roll2d_median_Price           4
10           Coal_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
2           Price_lag_7           0
9                  WD_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 5.61% done
********************* END NS.. 40 ****************************************************
********************* START NS ... 41 ****************************************************
[I 2025-06-15 19:46:18,023] A new study created in memory with name: no-name-ab905690-74f2-4001-a4d1-316fbaa289a0
[I 2025-06-15 19:46:18,151] Trial 0 finished with value: 0.0016781298707908756 and parameters: {'n_estimators': 70, 'learning_rate': 0.11112445679711994, 'num_leaves': 100, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.8385744882347598, 'colsample_bytree': 0.7855420485288265, 'reg_alpha': 0.0018135836396782068, 'reg_lambda': 6.210573453642043, 'min_split_gain': 0.043693225405406794}. Best is trial 0 with value: 0.0016781298707908756.
[I 2025-06-15 19:46:18,332] Trial 1 finished with value: 0.0015050904884617792 and parameters: {'n_estimators': 143, 'learning_rate': 0.1538594548371958, 'num_leaves': 246, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.9345566754697008, 'colsample_bytree': 0.8948926383121647, 'reg_alpha': 2.36535001711263, 'reg_lambda': 6.369540694900443, 'min_split_gain': 0.01804237715983494}. Best is trial 1 with value: 0.0015050904884617792.
[Trial 0] New best: 0.00168
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:18,963] Trial 2 finished with value: 0.0016863783746222272 and parameters: {'n_estimators': 240, 'learning_rate': 0.01041794814413201, 'num_leaves': 120, 'max_depth': 7, 'min_child_samples': 20, 'subsample': 0.8051040532019085, 'colsample_bytree': 0.7466934353302874, 'reg_alpha': 8.526635388987858, 'reg_lambda': 8.260551417097687, 'min_split_gain': 0.04080843560581491}. Best is trial 1 with value: 0.0015050904884617792.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:19,576] Trial 3 finished with value: 0.0020763711048174486 and parameters: {'n_estimators': 266, 'learning_rate': 0.08435812562605984, 'num_leaves': 180, 'max_depth': 12, 'min_child_samples': 14, 'subsample': 0.7286074835240846, 'colsample_bytree': 0.6307708200987372, 'reg_alpha': 6.213347013702525, 'reg_lambda': 1.5881589536347307, 'min_split_gain': 0.007767241686455456}. Best is trial 1 with value: 0.0015050904884617792.
[I 2025-06-15 19:46:19,579] A new study created in memory with name: no-name-5c54fb42-385a-4b77-ba63-c507b6268575
[I 2025-06-15 19:46:19,604] Trial 0 finished with value: 0.1346981281990683 and parameters: {'n_estimators': 154, 'learning_rate': 0.02733604635908989, 'num_leaves': 64, 'max_depth': 14, 'min_child_samples': 30, 'subsample': 0.5980516939802927, 'colsample_bytree': 0.6502032371779585, 'reg_alpha': 9.382369345787685, 'reg_lambda': 5.476052076119303, 'min_split_gain': 0.025346851815155114}. Best is trial 0 with value: 0.1346981281990683.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         420
1           Price_lag_2         259
14      pct_chg_Load_DA         235
3         Load_DA_lag_0         199
5       WindOn_DA_lag_0         133
16  roll2d_median_Price         116
11           NGas_lag_2         111
2           Price_lag_7         111
15       lag168_Load_DA          88
13            EUA_lag_2          84
10           Coal_lag_2          70
12            Oil_lag_2          65
7                  WD_1           5
9                  WD_7           3
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          66
1           Price_lag_2          12
10           Coal_lag_2           4
3         Load_DA_lag_0           2
12            Oil_lag_2           1
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 5.75% done
********************* END NS.. 41 ****************************************************
********************* START NS ... 42 ****************************************************
[I 2025-06-15 19:46:21,345] A new study created in memory with name: no-name-7f5caa30-a456-4db1-b198-34d5dd0d9c0c
[I 2025-06-15 19:46:21,472] Trial 0 finished with value: 0.0024987070583302635 and parameters: {'n_estimators': 62, 'learning_rate': 0.2878521989665134, 'num_leaves': 153, 'max_depth': 5, 'min_child_samples': 4, 'subsample': 0.9962146946022349, 'colsample_bytree': 0.517328952148076, 'reg_alpha': 9.832965208315773, 'reg_lambda': 5.894616062990041, 'min_split_gain': 0.03914509157565469}. Best is trial 0 with value: 0.0024987070583302635.
[Trial 0] New best: 0.00250
[I 2025-06-15 19:46:22,408] Trial 1 finished with value: 0.002631458839226117 and parameters: {'n_estimators': 132, 'learning_rate': 0.054414822592480774, 'num_leaves': 123, 'max_depth': 12, 'min_child_samples': 9, 'subsample': 0.7106154163603243, 'colsample_bytree': 0.5794482524284645, 'reg_alpha': 0.2042339434639151, 'reg_lambda': 0.2695133697069785, 'min_split_gain': 0.01600220607121905}. Best is trial 0 with value: 0.0024987070583302635.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:22,696] Trial 2 finished with value: 0.0017907616283887328 and parameters: {'n_estimators': 158, 'learning_rate': 0.13361988253203547, 'num_leaves': 179, 'max_depth': 11, 'min_child_samples': 8, 'subsample': 0.7203909311522986, 'colsample_bytree': 0.9728456794104754, 'reg_alpha': 5.5928763762701275, 'reg_lambda': 3.6617039084740743, 'min_split_gain': 0.014374103067267496}. Best is trial 2 with value: 0.0017907616283887328.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:23,359] Trial 3 finished with value: 0.0020618875868603555 and parameters: {'n_estimators': 274, 'learning_rate': 0.07308850045719721, 'num_leaves': 138, 'max_depth': 7, 'min_child_samples': 20, 'subsample': 0.9482672312343664, 'colsample_bytree': 0.5671844368031734, 'reg_alpha': 7.64201902168927, 'reg_lambda': 4.008814252285722, 'min_split_gain': 0.018984162724272183}. Best is trial 2 with value: 0.0017907616283887328.
[I 2025-06-15 19:46:23,362] A new study created in memory with name: no-name-6e775a81-50c6-40af-95ef-af98d6dc1760
[I 2025-06-15 19:46:23,399] Trial 0 finished with value: 0.0656437808061485 and parameters: {'n_estimators': 200, 'learning_rate': 0.08834124922754283, 'num_leaves': 95, 'max_depth': 7, 'min_child_samples': 27, 'subsample': 0.6397389050816193, 'colsample_bytree': 0.5769804241792584, 'reg_alpha': 5.788040427554586, 'reg_lambda': 2.6685323110590375, 'min_split_gain': 0.040082778842398935}. Best is trial 0 with value: 0.0656437808061485.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         469
14      pct_chg_Load_DA         301
1           Price_lag_2         225
3         Load_DA_lag_0         178
2           Price_lag_7         169
16  roll2d_median_Price         159
11           NGas_lag_2         141
5       WindOn_DA_lag_0         132
13            EUA_lag_2         132
10           Coal_lag_2         128
15       lag168_Load_DA         113
12            Oil_lag_2          96
7                  WD_1           8
9                  WD_7           6
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          33
10           Coal_lag_2           4
1           Price_lag_2           3
3         Load_DA_lag_0           2
16  roll2d_median_Price           1
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
2           Price_lag_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 5.88% done
********************* END NS.. 42 ****************************************************
********************* START NS ... 43 ****************************************************
[I 2025-06-15 19:46:25,828] A new study created in memory with name: no-name-3482c75f-1d9c-4ef0-8e20-e0bb6e7d3156
[I 2025-06-15 19:46:27,626] Trial 0 finished with value: 0.0022701828541102725 and parameters: {'n_estimators': 231, 'learning_rate': 0.011185065149024091, 'num_leaves': 33, 'max_depth': 14, 'min_child_samples': 22, 'subsample': 0.5689325777677681, 'colsample_bytree': 0.566432170132172, 'reg_alpha': 5.585772426943713, 'reg_lambda': 8.382097780248033, 'min_split_gain': 0.021255703248736036}. Best is trial 0 with value: 0.0022701828541102725.
[Trial 0] New best: 0.00227
[I 2025-06-15 19:46:28,125] Trial 1 finished with value: 0.0015135234040088187 and parameters: {'n_estimators': 167, 'learning_rate': 0.18886518618038922, 'num_leaves': 107, 'max_depth': 9, 'min_child_samples': 4, 'subsample': 0.5631120821481229, 'colsample_bytree': 0.7847886149870557, 'reg_alpha': 0.4512313584674399, 'reg_lambda': 2.005640053501195, 'min_split_gain': 0.0420221520152616}. Best is trial 1 with value: 0.0015135234040088187.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:29,095] Trial 2 finished with value: 0.001792193568695943 and parameters: {'n_estimators': 186, 'learning_rate': 0.011119705302740809, 'num_leaves': 160, 'max_depth': 15, 'min_child_samples': 20, 'subsample': 0.9418448617425415, 'colsample_bytree': 0.8312749054539972, 'reg_alpha': 6.88146303153117, 'reg_lambda': 7.817139678978011, 'min_split_gain': 0.04705524252386325}. Best is trial 1 with value: 0.0015135234040088187.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:29,885] Trial 3 finished with value: 0.00270169434747086 and parameters: {'n_estimators': 172, 'learning_rate': 0.012963224806380699, 'num_leaves': 34, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.955879299102344, 'colsample_bytree': 0.5431626270896471, 'reg_alpha': 7.530417661863308, 'reg_lambda': 9.122311653943743, 'min_split_gain': 0.0328011242362036}. Best is trial 1 with value: 0.0015135234040088187.
[I 2025-06-15 19:46:29,888] A new study created in memory with name: no-name-ed0413ba-2669-463b-9889-cb40dac4a437
[I 2025-06-15 19:46:29,981] Trial 0 finished with value: 0.005761789439974303 and parameters: {'n_estimators': 97, 'learning_rate': 0.022402919612627283, 'num_leaves': 253, 'max_depth': 7, 'min_child_samples': 8, 'subsample': 0.617413174826154, 'colsample_bytree': 0.5829670171857384, 'reg_alpha': 0.362336783580095, 'reg_lambda': 4.192818681056237, 'min_split_gain': 0.04657016663334881}. Best is trial 0 with value: 0.005761789439974303.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         445
14      pct_chg_Load_DA         444
1           Price_lag_2         412
3         Load_DA_lag_0         273
2           Price_lag_7         206
16  roll2d_median_Price         183
11           NGas_lag_2         158
5       WindOn_DA_lag_0         148
13            EUA_lag_2         135
15       lag168_Load_DA         131
10           Coal_lag_2         128
12            Oil_lag_2         103
9                  WD_7           7
7                  WD_1           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         145
1           Price_lag_2          50
16  roll2d_median_Price          27
3         Load_DA_lag_0          25
5       WindOn_DA_lag_0          16
14      pct_chg_Load_DA          12
7                  WD_1           7
10           Coal_lag_2           7
2           Price_lag_7           2
15       lag168_Load_DA           2
12            Oil_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
9                  WD_7           0
11           NGas_lag_2           0
13            EUA_lag_2           0
8                  WD_6           0
-> 6.02% done
********************* END NS.. 43 ****************************************************
********************* START NS ... 44 ****************************************************
[I 2025-06-15 19:46:31,168] A new study created in memory with name: no-name-74e8d7b5-ce66-4e65-9bda-afef4f293b41
[I 2025-06-15 19:46:31,362] Trial 0 finished with value: 0.0011863225253030153 and parameters: {'n_estimators': 69, 'learning_rate': 0.19545699154241916, 'num_leaves': 208, 'max_depth': 11, 'min_child_samples': 15, 'subsample': 0.6977910313809775, 'colsample_bytree': 0.6972058443357085, 'reg_alpha': 3.6253222627646284, 'reg_lambda': 0.4841898093087693, 'min_split_gain': 0.011759726739022858}. Best is trial 0 with value: 0.0011863225253030153.
[Trial 0] New best: 0.00119
[I 2025-06-15 19:46:31,637] Trial 1 finished with value: 0.0055308745538692515 and parameters: {'n_estimators': 64, 'learning_rate': 0.022639795457049087, 'num_leaves': 212, 'max_depth': 8, 'min_child_samples': 5, 'subsample': 0.9907178422599462, 'colsample_bytree': 0.518087342618734, 'reg_alpha': 7.921760095669876, 'reg_lambda': 1.2906027219710003, 'min_split_gain': 0.011190065244913333}. Best is trial 0 with value: 0.0011863225253030153.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:32,023] Trial 2 finished with value: 0.0013071300089773526 and parameters: {'n_estimators': 181, 'learning_rate': 0.12981995644928665, 'num_leaves': 227, 'max_depth': 6, 'min_child_samples': 3, 'subsample': 0.8020824221637868, 'colsample_bytree': 0.6360421136368639, 'reg_alpha': 0.8326244615567846, 'reg_lambda': 2.262859213010354, 'min_split_gain': 0.01852679781151983}. Best is trial 0 with value: 0.0011863225253030153.
[I 2025-06-15 19:46:32,230] Trial 3 finished with value: 0.001506532275670407 and parameters: {'n_estimators': 214, 'learning_rate': 0.2267061718017668, 'num_leaves': 118, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.6962845811139258, 'colsample_bytree': 0.7896767768487685, 'reg_alpha': 8.52139777264968, 'reg_lambda': 1.8690118708103032, 'min_split_gain': 0.049154453118794794}. Best is trial 0 with value: 0.0011863225253030153.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:32,233] A new study created in memory with name: no-name-3a882255-ec53-4ace-aa25-e714d41d01cc
[I 2025-06-15 19:46:32,264] Trial 0 finished with value: 0.0028792436615385684 and parameters: {'n_estimators': 137, 'learning_rate': 0.03369699822969873, 'num_leaves': 94, 'max_depth': 12, 'min_child_samples': 18, 'subsample': 0.5502652752937254, 'colsample_bytree': 0.7496480543465363, 'reg_alpha': 5.768233199448682, 'reg_lambda': 8.878242687113207, 'min_split_gain': 0.009663049794638334}. Best is trial 0 with value: 0.0028792436615385684.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         337
1           Price_lag_2         262
3         Load_DA_lag_0         237
14      pct_chg_Load_DA         186
2           Price_lag_7         148
15       lag168_Load_DA         143
13            EUA_lag_2         120
16  roll2d_median_Price         117
11           NGas_lag_2         115
5       WindOn_DA_lag_0         111
10           Coal_lag_2         107
12            Oil_lag_2          94
9                  WD_7           8
7                  WD_1           7
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          72
1           Price_lag_2          20
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 6.16% done
********************* END NS.. 44 ****************************************************
********************* START NS ... 45 ****************************************************
[I 2025-06-15 19:46:33,082] A new study created in memory with name: no-name-2cd0f294-196a-4719-a028-2b5bd9fd0680
[I 2025-06-15 19:46:33,367] Trial 0 finished with value: 0.02746195940131857 and parameters: {'n_estimators': 182, 'learning_rate': 0.10059016151597096, 'num_leaves': 98, 'max_depth': 15, 'min_child_samples': 11, 'subsample': 0.8280618892508316, 'colsample_bytree': 0.6839590108674516, 'reg_alpha': 5.268933265382395, 'reg_lambda': 4.286499783070949, 'min_split_gain': 0.020232663697406957}. Best is trial 0 with value: 0.02746195940131857.
[Trial 0] New best: 0.02746
[I 2025-06-15 19:46:33,580] Trial 1 finished with value: 0.02673320454652131 and parameters: {'n_estimators': 76, 'learning_rate': 0.09309173681922714, 'num_leaves': 180, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.8947296447716924, 'colsample_bytree': 0.9633590740761849, 'reg_alpha': 3.3789047217911508, 'reg_lambda': 7.9535502994333775, 'min_split_gain': 0.026131115890705443}. Best is trial 1 with value: 0.02673320454652131.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:33,825] Trial 2 finished with value: 0.02844912103440017 and parameters: {'n_estimators': 166, 'learning_rate': 0.23674605238914168, 'num_leaves': 55, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.8022513099695212, 'colsample_bytree': 0.7159731064664625, 'reg_alpha': 8.532011126578675, 'reg_lambda': 1.7462277588218666, 'min_split_gain': 0.040824018758824965}. Best is trial 1 with value: 0.02673320454652131.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:34,262] Trial 3 finished with value: 0.05119916355229439 and parameters: {'n_estimators': 119, 'learning_rate': 0.010167277080821881, 'num_leaves': 158, 'max_depth': 8, 'min_child_samples': 23, 'subsample': 0.6406182673263852, 'colsample_bytree': 0.7379976321545249, 'reg_alpha': 6.974167103156467, 'reg_lambda': 3.1720713272493386, 'min_split_gain': 0.026316043910909748}. Best is trial 1 with value: 0.02673320454652131.
[I 2025-06-15 19:46:34,265] A new study created in memory with name: no-name-5d774278-9b2c-4965-959c-0ada426fab3c
[I 2025-06-15 19:46:34,292] Trial 0 finished with value: 0.12086533552107577 and parameters: {'n_estimators': 205, 'learning_rate': 0.2732866105810179, 'num_leaves': 221, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.8574170551708616, 'colsample_bytree': 0.9546230772822697, 'reg_alpha': 4.441085795622226, 'reg_lambda': 5.908084655549403, 'min_split_gain': 0.026147297447725523}. Best is trial 0 with value: 0.12086533552107577.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         513
14      pct_chg_Load_DA         278
1           Price_lag_2         203
3         Load_DA_lag_0         128
11           NGas_lag_2         117
2           Price_lag_7         103
16  roll2d_median_Price         101
13            EUA_lag_2          84
12            Oil_lag_2          68
15       lag168_Load_DA          61
5       WindOn_DA_lag_0          54
10           Coal_lag_2          37
7                  WD_1           7
9                  WD_7           5
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          15
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 6.29% done
********************* END NS.. 45 ****************************************************
********************* START NS ... 46 ****************************************************
[I 2025-06-15 19:46:35,145] A new study created in memory with name: no-name-8346c36b-28c9-4e61-b75f-1e715fcdae35
[I 2025-06-15 19:46:35,295] Trial 0 finished with value: 0.039502931206932435 and parameters: {'n_estimators': 185, 'learning_rate': 0.21400700893113883, 'num_leaves': 221, 'max_depth': 6, 'min_child_samples': 19, 'subsample': 0.5349753462836411, 'colsample_bytree': 0.7639593248823893, 'reg_alpha': 0.8542617169969169, 'reg_lambda': 1.7249966025216967, 'min_split_gain': 0.04692050480407552}. Best is trial 0 with value: 0.039502931206932435.
[Trial 0] New best: 0.03950
[I 2025-06-15 19:46:35,813] Trial 1 finished with value: 0.06031392316801456 and parameters: {'n_estimators': 145, 'learning_rate': 0.011509166250901428, 'num_leaves': 221, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.9316438446876572, 'colsample_bytree': 0.603606270268838, 'reg_alpha': 8.428913781337302, 'reg_lambda': 1.9541326903613243, 'min_split_gain': 0.025712679426302017}. Best is trial 0 with value: 0.039502931206932435.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:36,388] Trial 2 finished with value: 0.04031225586811231 and parameters: {'n_estimators': 169, 'learning_rate': 0.14588994066138236, 'num_leaves': 232, 'max_depth': 10, 'min_child_samples': 22, 'subsample': 0.7605476106998066, 'colsample_bytree': 0.5687270557007027, 'reg_alpha': 0.7522961800163341, 'reg_lambda': 4.48071963531126, 'min_split_gain': 0.008110225878935324}. Best is trial 0 with value: 0.039502931206932435.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:37,603] Trial 3 finished with value: 0.0459315781113056 and parameters: {'n_estimators': 173, 'learning_rate': 0.014364649231280315, 'num_leaves': 106, 'max_depth': 14, 'min_child_samples': 2, 'subsample': 0.9175347239947056, 'colsample_bytree': 0.6345169376690399, 'reg_alpha': 0.7643499225956785, 'reg_lambda': 6.352155054796706, 'min_split_gain': 0.02075230618149093}. Best is trial 0 with value: 0.039502931206932435.
[I 2025-06-15 19:46:37,606] A new study created in memory with name: no-name-bf8a136a-34d1-4619-b80e-66fc8aab3c8a
[I 2025-06-15 19:46:37,643] Trial 0 finished with value: 0.1460349272394227 and parameters: {'n_estimators': 233, 'learning_rate': 0.020720634625105934, 'num_leaves': 32, 'max_depth': 8, 'min_child_samples': 16, 'subsample': 0.8341356850379085, 'colsample_bytree': 0.7174010417266042, 'reg_alpha': 3.6768412594957067, 'reg_lambda': 0.9104304523014484, 'min_split_gain': 0.016698377175858366}. Best is trial 0 with value: 0.1460349272394227.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         309
14      pct_chg_Load_DA         180
1           Price_lag_2         176
3         Load_DA_lag_0         117
16  roll2d_median_Price          78
11           NGas_lag_2          75
2           Price_lag_7          64
10           Coal_lag_2          53
13            EUA_lag_2          52
15       lag168_Load_DA          47
5       WindOn_DA_lag_0          31
12            Oil_lag_2          31
7                  WD_1           1
9                  WD_7           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         200
1           Price_lag_2          35
5       WindOn_DA_lag_0          26
10           Coal_lag_2          24
3         Load_DA_lag_0          21
16  roll2d_median_Price           3
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
2           Price_lag_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 6.43% done
********************* END NS.. 46 ****************************************************
********************* START NS ... 47 ****************************************************
[I 2025-06-15 19:46:38,666] A new study created in memory with name: no-name-f01bdd2e-d5bd-48fe-b10d-26dc068a765b
[I 2025-06-15 19:46:38,940] Trial 0 finished with value: 0.02342270884858258 and parameters: {'n_estimators': 188, 'learning_rate': 0.14052898651649273, 'num_leaves': 202, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.627040268656436, 'colsample_bytree': 0.8507834539462961, 'reg_alpha': 1.59000019054651, 'reg_lambda': 0.9806357030867352, 'min_split_gain': 0.016699752494541944}. Best is trial 0 with value: 0.02342270884858258.
[Trial 0] New best: 0.02342
[I 2025-06-15 19:46:39,395] Trial 1 finished with value: 0.02372745952055542 and parameters: {'n_estimators': 236, 'learning_rate': 0.020944150840423312, 'num_leaves': 157, 'max_depth': 4, 'min_child_samples': 30, 'subsample': 0.7885312697873272, 'colsample_bytree': 0.8921541819999417, 'reg_alpha': 0.17383043445778346, 'reg_lambda': 4.730945864522407, 'min_split_gain': 0.04414574664113388}. Best is trial 0 with value: 0.02342270884858258.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:40,396] Trial 2 finished with value: 0.021016834170927215 and parameters: {'n_estimators': 179, 'learning_rate': 0.025650883987177294, 'num_leaves': 163, 'max_depth': 14, 'min_child_samples': 3, 'subsample': 0.7145764358943594, 'colsample_bytree': 0.537215023728757, 'reg_alpha': 1.725386146157205, 'reg_lambda': 6.216040108942913, 'min_split_gain': 0.030736617973026892}. Best is trial 2 with value: 0.021016834170927215.
[Trial 2] New best: 0.02102
[I 2025-06-15 19:46:40,624] Trial 3 finished with value: 0.02248945735461004 and parameters: {'n_estimators': 260, 'learning_rate': 0.1838776323236668, 'num_leaves': 207, 'max_depth': 5, 'min_child_samples': 28, 'subsample': 0.5817245975721761, 'colsample_bytree': 0.8923018685417752, 'reg_alpha': 5.222138606297974, 'reg_lambda': 0.30437855058850105, 'min_split_gain': 0.03888643240974377}. Best is trial 2 with value: 0.021016834170927215.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:40,870] Trial 4 finished with value: 0.021981716061586064 and parameters: {'n_estimators': 99, 'learning_rate': 0.02777946024517516, 'num_leaves': 122, 'max_depth': 4, 'min_child_samples': 23, 'subsample': 0.7329693834319608, 'colsample_bytree': 0.7380579868447654, 'reg_alpha': 5.212575331705104, 'reg_lambda': 1.6738660655570181, 'min_split_gain': 0.016327644499429455}. Best is trial 2 with value: 0.021016834170927215.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:41,498] Trial 5 finished with value: 0.02207191990120296 and parameters: {'n_estimators': 230, 'learning_rate': 0.03328317120158756, 'num_leaves': 162, 'max_depth': 11, 'min_child_samples': 13, 'subsample': 0.6673997541369758, 'colsample_bytree': 0.7887210771986772, 'reg_alpha': 7.136400717299674, 'reg_lambda': 6.540282122185932, 'min_split_gain': 0.029415804749967658}. Best is trial 2 with value: 0.021016834170927215.
[I 2025-06-15 19:46:41,500] A new study created in memory with name: no-name-554d1b44-cec4-430b-a0f3-04b6b6342f62
[I 2025-06-15 19:46:41,529] Trial 0 finished with value: 0.04190617000348723 and parameters: {'n_estimators': 166, 'learning_rate': 0.029253214168621304, 'num_leaves': 233, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.5284832090155216, 'colsample_bytree': 0.9602698739133166, 'reg_alpha': 9.182519506436925, 'reg_lambda': 8.950588571072682, 'min_split_gain': 0.02829731998518778}. Best is trial 0 with value: 0.04190617000348723.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1931
0           Price_lag_1        1894
14      pct_chg_Load_DA        1459
3         Load_DA_lag_0        1297
2           Price_lag_7        1217
16  roll2d_median_Price        1030
10           Coal_lag_2         987
15       lag168_Load_DA         919
11           NGas_lag_2         754
5       WindOn_DA_lag_0         749
13            EUA_lag_2         706
12            Oil_lag_2         528
9                  WD_7          53
8                  WD_6          25
7                  WD_1          22
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          27
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 6.57% done
********************* END NS.. 47 ****************************************************
********************* START NS ... 48 ****************************************************
[I 2025-06-15 19:46:42,970] A new study created in memory with name: no-name-71705dd5-aef5-4dd8-8d36-f7c3d39e1f5e
[I 2025-06-15 19:46:43,210] Trial 0 finished with value: 0.009329333132863202 and parameters: {'n_estimators': 76, 'learning_rate': 0.07684962965576443, 'num_leaves': 45, 'max_depth': 11, 'min_child_samples': 15, 'subsample': 0.83918331005731, 'colsample_bytree': 0.6640929380721134, 'reg_alpha': 0.2861476214034264, 'reg_lambda': 6.311085803319337, 'min_split_gain': 0.006422981640077419}. Best is trial 0 with value: 0.009329333132863202.
[Trial 0] New best: 0.00933
[I 2025-06-15 19:46:44,235] Trial 1 finished with value: 0.009132823733333314 and parameters: {'n_estimators': 288, 'learning_rate': 0.02648451983649527, 'num_leaves': 151, 'max_depth': 10, 'min_child_samples': 8, 'subsample': 0.7594187778704251, 'colsample_bytree': 0.8600761155105443, 'reg_alpha': 5.292245169559057, 'reg_lambda': 7.680479613411368, 'min_split_gain': 0.008817740854781464}. Best is trial 1 with value: 0.009132823733333314.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:44,523] Trial 2 finished with value: 0.014728270175232122 and parameters: {'n_estimators': 115, 'learning_rate': 0.010651893568069077, 'num_leaves': 253, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.9775341128317204, 'colsample_bytree': 0.5447857288394388, 'reg_alpha': 3.2579896073521164, 'reg_lambda': 7.082295290434763, 'min_split_gain': 0.041634719832353534}. Best is trial 1 with value: 0.009132823733333314.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:45,086] Trial 3 finished with value: 0.00915997048875364 and parameters: {'n_estimators': 183, 'learning_rate': 0.03298390567123629, 'num_leaves': 114, 'max_depth': 9, 'min_child_samples': 12, 'subsample': 0.6794400961271561, 'colsample_bytree': 0.8222826053498666, 'reg_alpha': 9.463226690947618, 'reg_lambda': 1.7374240640109162, 'min_split_gain': 0.012486203395190255}. Best is trial 1 with value: 0.009132823733333314.
[I 2025-06-15 19:46:45,087] A new study created in memory with name: no-name-703e9dc1-c702-4d19-9174-67bb0efd9cce
[I 2025-06-15 19:46:45,148] Trial 0 finished with value: 0.04072938406371924 and parameters: {'n_estimators': 275, 'learning_rate': 0.020740742934930095, 'num_leaves': 192, 'max_depth': 12, 'min_child_samples': 1, 'subsample': 0.6782213450341908, 'colsample_bytree': 0.5067418323339445, 'reg_alpha': 5.476648318870799, 'reg_lambda': 9.193541780275575, 'min_split_gain': 0.011924401579114391}. Best is trial 0 with value: 0.04072938406371924.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2376
1           Price_lag_2        1681
14      pct_chg_Load_DA        1548
3         Load_DA_lag_0        1082
2           Price_lag_7         776
16  roll2d_median_Price         743
5       WindOn_DA_lag_0         731
11           NGas_lag_2         726
15       lag168_Load_DA         718
13            EUA_lag_2         717
10           Coal_lag_2         633
12            Oil_lag_2         625
9                  WD_7          40
7                  WD_1          33
8                  WD_6          20
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         175
1           Price_lag_2          56
3         Load_DA_lag_0          49
16  roll2d_median_Price          24
13            EUA_lag_2           6
5       WindOn_DA_lag_0           3
2           Price_lag_7           3
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 6.70% done
********************* END NS.. 48 ****************************************************
********************* START NS ... 49 ****************************************************
[I 2025-06-15 19:46:46,877] A new study created in memory with name: no-name-bb0e64fb-a597-4eeb-ad4f-40d6a6643c13
[I 2025-06-15 19:46:47,240] Trial 0 finished with value: 0.0038993148254143043 and parameters: {'n_estimators': 174, 'learning_rate': 0.04693143131542957, 'num_leaves': 74, 'max_depth': 6, 'min_child_samples': 24, 'subsample': 0.9143611144472141, 'colsample_bytree': 0.5508529407892089, 'reg_alpha': 7.459433113581496, 'reg_lambda': 2.6786055944045972, 'min_split_gain': 0.047308308908817126}. Best is trial 0 with value: 0.0038993148254143043.
[Trial 0] New best: 0.00390
[I 2025-06-15 19:46:47,825] Trial 1 finished with value: 0.003586651383330543 and parameters: {'n_estimators': 192, 'learning_rate': 0.036366100986390815, 'num_leaves': 236, 'max_depth': 12, 'min_child_samples': 26, 'subsample': 0.9424929768264836, 'colsample_bytree': 0.522003272431034, 'reg_alpha': 9.904684616957482, 'reg_lambda': 7.5246542963969745, 'min_split_gain': 0.025935228948760665}. Best is trial 1 with value: 0.003586651383330543.
[I 2025-06-15 19:46:48,024] Trial 2 finished with value: 0.003684397044484385 and parameters: {'n_estimators': 102, 'learning_rate': 0.05365349130467409, 'num_leaves': 230, 'max_depth': 4, 'min_child_samples': 18, 'subsample': 0.9944263521412893, 'colsample_bytree': 0.5802035877658799, 'reg_alpha': 3.396766078509146, 'reg_lambda': 7.062838021046, 'min_split_gain': 0.03414793639256058}. Best is trial 1 with value: 0.003586651383330543.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:48,353] Trial 3 finished with value: 0.004140082938859002 and parameters: {'n_estimators': 151, 'learning_rate': 0.186746918719223, 'num_leaves': 68, 'max_depth': 10, 'min_child_samples': 17, 'subsample': 0.5265930940612724, 'colsample_bytree': 0.5639810566035168, 'reg_alpha': 3.506643134592445, 'reg_lambda': 3.894751101895623, 'min_split_gain': 0.007978569003675618}. Best is trial 1 with value: 0.003586651383330543.
[I 2025-06-15 19:46:48,356] A new study created in memory with name: no-name-1fa5e2e8-4336-461f-a9a4-a39c8fb7caaa
[I 2025-06-15 19:46:48,380] Trial 0 finished with value: 0.0939802171568825 and parameters: {'n_estimators': 298, 'learning_rate': 0.041627297372111696, 'num_leaves': 217, 'max_depth': 15, 'min_child_samples': 15, 'subsample': 0.7958189177032339, 'colsample_bytree': 0.929952730845621, 'reg_alpha': 7.912515210114518, 'reg_lambda': 6.891988663401978, 'min_split_gain': 0.022015323217652455}. Best is trial 0 with value: 0.0939802171568825.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         960
1           Price_lag_2         790
14      pct_chg_Load_DA         449
3         Load_DA_lag_0         387
2           Price_lag_7         349
16  roll2d_median_Price         337
11           NGas_lag_2         233
15       lag168_Load_DA         233
13            EUA_lag_2         206
5       WindOn_DA_lag_0         186
10           Coal_lag_2         184
12            Oil_lag_2         144
9                  WD_7          31
8                  WD_6          24
7                  WD_1           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          40
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 6.84% done
********************* END NS.. 49 ****************************************************
********************* START NS ... 50 ****************************************************
[I 2025-06-15 19:46:49,482] A new study created in memory with name: no-name-e4776c49-b429-42d6-9ed3-5d38376d9dcb
[I 2025-06-15 19:46:49,649] Trial 0 finished with value: 0.0019501813144912148 and parameters: {'n_estimators': 78, 'learning_rate': 0.13678360204957316, 'num_leaves': 203, 'max_depth': 12, 'min_child_samples': 16, 'subsample': 0.6221288063813186, 'colsample_bytree': 0.7947489516018225, 'reg_alpha': 8.8624881646067, 'reg_lambda': 6.6035674695150695, 'min_split_gain': 0.024607757447871643}. Best is trial 0 with value: 0.0019501813144912148.
[Trial 0] New best: 0.00195
[I 2025-06-15 19:46:49,867] Trial 1 finished with value: 0.0019976629708047746 and parameters: {'n_estimators': 56, 'learning_rate': 0.11861001580659919, 'num_leaves': 223, 'max_depth': 8, 'min_child_samples': 7, 'subsample': 0.632291770771717, 'colsample_bytree': 0.8188077481258542, 'reg_alpha': 6.138259896108731, 'reg_lambda': 9.532088030115004, 'min_split_gain': 0.022239542155072203}. Best is trial 0 with value: 0.0019501813144912148.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:50,232] Trial 2 finished with value: 0.001994741221886825 and parameters: {'n_estimators': 223, 'learning_rate': 0.06421714279890743, 'num_leaves': 153, 'max_depth': 13, 'min_child_samples': 19, 'subsample': 0.6451516807876125, 'colsample_bytree': 0.7782613306568482, 'reg_alpha': 6.542303694987695, 'reg_lambda': 2.677000469511742, 'min_split_gain': 0.04933278881808739}. Best is trial 0 with value: 0.0019501813144912148.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:50,620] Trial 3 finished with value: 0.001950331954138852 and parameters: {'n_estimators': 278, 'learning_rate': 0.10586123130469641, 'num_leaves': 64, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.6700556448614559, 'colsample_bytree': 0.6297708187225428, 'reg_alpha': 8.216222489800225, 'reg_lambda': 2.046880809766769, 'min_split_gain': 0.014705130089080948}. Best is trial 0 with value: 0.0019501813144912148.
[I 2025-06-15 19:46:50,622] A new study created in memory with name: no-name-edbea359-1f76-467b-a362-889907a832f7
[I 2025-06-15 19:46:50,641] Trial 0 finished with value: 0.045116840717200846 and parameters: {'n_estimators': 96, 'learning_rate': 0.07179315460234625, 'num_leaves': 226, 'max_depth': 10, 'min_child_samples': 28, 'subsample': 0.6044538770774113, 'colsample_bytree': 0.9391451437769704, 'reg_alpha': 8.264995259621358, 'reg_lambda': 9.772604491185076, 'min_split_gain': 0.049689252182848226}. Best is trial 0 with value: 0.045116840717200846.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         320
1           Price_lag_2         196
14      pct_chg_Load_DA         176
3         Load_DA_lag_0          98
2           Price_lag_7          94
11           NGas_lag_2          82
13            EUA_lag_2          77
12            Oil_lag_2          56
16  roll2d_median_Price          52
15       lag168_Load_DA          50
5       WindOn_DA_lag_0          41
10           Coal_lag_2          35
9                  WD_7           5
8                  WD_6           3
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          15
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 6.98% done
********************* END NS.. 50 ****************************************************
********************* START NS ... 51 ****************************************************
[I 2025-06-15 19:46:51,413] A new study created in memory with name: no-name-6b62ef78-ad49-470e-948e-8479531d1a09
[I 2025-06-15 19:46:51,690] Trial 0 finished with value: 0.008054004467563467 and parameters: {'n_estimators': 204, 'learning_rate': 0.06419039225086044, 'num_leaves': 113, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.7413282552802266, 'colsample_bytree': 0.629527308827985, 'reg_alpha': 1.470653120573524, 'reg_lambda': 5.5622861169983615, 'min_split_gain': 0.03945142997538918}. Best is trial 0 with value: 0.008054004467563467.
[Trial 0] New best: 0.00805
[I 2025-06-15 19:46:51,915] Trial 1 finished with value: 0.007011558272652987 and parameters: {'n_estimators': 72, 'learning_rate': 0.2718474261374586, 'num_leaves': 145, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.8530385249200986, 'colsample_bytree': 0.6817257030751336, 'reg_alpha': 6.739316994215972, 'reg_lambda': 6.510223401187528, 'min_split_gain': 0.002429683278398537}. Best is trial 1 with value: 0.007011558272652987.
[Trial 1] New best: 0.00701
[I 2025-06-15 19:46:52,503] Trial 2 finished with value: 0.009220366331760004 and parameters: {'n_estimators': 273, 'learning_rate': 0.043948818520091644, 'num_leaves': 97, 'max_depth': 13, 'min_child_samples': 11, 'subsample': 0.9442192675172012, 'colsample_bytree': 0.5153099321263199, 'reg_alpha': 5.742221587138358, 'reg_lambda': 5.945346001265567, 'min_split_gain': 0.027253805851528798}. Best is trial 1 with value: 0.007011558272652987.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:52,908] Trial 3 finished with value: 0.007849622938077361 and parameters: {'n_estimators': 250, 'learning_rate': 0.10526029964952661, 'num_leaves': 35, 'max_depth': 15, 'min_child_samples': 29, 'subsample': 0.6216291571439295, 'colsample_bytree': 0.5705255725960432, 'reg_alpha': 9.463178011110848, 'reg_lambda': 7.267938863400704, 'min_split_gain': 0.010133943621879811}. Best is trial 1 with value: 0.007011558272652987.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:53,558] Trial 4 finished with value: 0.00910737032337715 and parameters: {'n_estimators': 102, 'learning_rate': 0.03479352135939641, 'num_leaves': 158, 'max_depth': 12, 'min_child_samples': 2, 'subsample': 0.8143910289362375, 'colsample_bytree': 0.5310604672572531, 'reg_alpha': 1.1520825240650634, 'reg_lambda': 3.262902970722913, 'min_split_gain': 0.033693360630114806}. Best is trial 1 with value: 0.007011558272652987.
[I 2025-06-15 19:46:53,561] A new study created in memory with name: no-name-5f7ee5f7-c035-46e7-92e2-b6b34e3267c9
[I 2025-06-15 19:46:53,638] Trial 0 finished with value: 0.03042358688752819 and parameters: {'n_estimators': 222, 'learning_rate': 0.011665143229980891, 'num_leaves': 102, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.971789048147333, 'colsample_bytree': 0.6689270561400167, 'reg_alpha': 0.9426652724993922, 'reg_lambda': 4.910339717072617, 'min_split_gain': 0.031314991814342795}. Best is trial 0 with value: 0.03042358688752819.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         262
1           Price_lag_2         190
3         Load_DA_lag_0         159
14      pct_chg_Load_DA         158
13            EUA_lag_2         137
15       lag168_Load_DA         122
5       WindOn_DA_lag_0         119
2           Price_lag_7         113
11           NGas_lag_2         111
10           Coal_lag_2         102
12            Oil_lag_2          98
16  roll2d_median_Price          85
9                  WD_7           9
7                  WD_1           5
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         340
3         Load_DA_lag_0          75
1           Price_lag_2          58
15       lag168_Load_DA          53
16  roll2d_median_Price          27
11           NGas_lag_2          12
5       WindOn_DA_lag_0          10
2           Price_lag_7           4
12            Oil_lag_2           3
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 7.11% done
********************* END NS.. 51 ****************************************************
********************* START NS ... 52 ****************************************************
[I 2025-06-15 19:46:54,432] A new study created in memory with name: no-name-31d14637-5e6e-4af2-8cfc-e49423ec304d
[I 2025-06-15 19:46:54,647] Trial 0 finished with value: 0.013388623707930855 and parameters: {'n_estimators': 57, 'learning_rate': 0.04499254145281984, 'num_leaves': 149, 'max_depth': 7, 'min_child_samples': 8, 'subsample': 0.5502371154707485, 'colsample_bytree': 0.62898438108406, 'reg_alpha': 0.4787029793495945, 'reg_lambda': 5.6893799325768875, 'min_split_gain': 0.006483781583481386}. Best is trial 0 with value: 0.013388623707930855.
[Trial 0] New best: 0.01339
[I 2025-06-15 19:46:55,128] Trial 1 finished with value: 0.012744226307121154 and parameters: {'n_estimators': 104, 'learning_rate': 0.0853665446003681, 'num_leaves': 70, 'max_depth': 10, 'min_child_samples': 10, 'subsample': 0.6835661523789271, 'colsample_bytree': 0.5076146949570458, 'reg_alpha': 0.5643876400701464, 'reg_lambda': 8.271620862241791, 'min_split_gain': 0.016987934722535188}. Best is trial 1 with value: 0.012744226307121154.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:55,583] Trial 2 finished with value: 0.010473738518039339 and parameters: {'n_estimators': 125, 'learning_rate': 0.07993628252301666, 'num_leaves': 156, 'max_depth': 9, 'min_child_samples': 19, 'subsample': 0.5819040594284064, 'colsample_bytree': 0.6584068942011565, 'reg_alpha': 0.6037901094292919, 'reg_lambda': 8.8783208195837, 'min_split_gain': 0.027415358530147045}. Best is trial 2 with value: 0.010473738518039339.
[Trial 2] New best: 0.01047
[I 2025-06-15 19:46:55,813] Trial 3 finished with value: 0.023030976727866855 and parameters: {'n_estimators': 97, 'learning_rate': 0.010905338258927142, 'num_leaves': 75, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.5970479803498173, 'colsample_bytree': 0.6391488019218646, 'reg_alpha': 9.123211806389984, 'reg_lambda': 9.26438641081665, 'min_split_gain': 0.03137705128637034}. Best is trial 2 with value: 0.010473738518039339.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:56,277] Trial 4 finished with value: 0.010899392929754342 and parameters: {'n_estimators': 296, 'learning_rate': 0.18758992632108307, 'num_leaves': 145, 'max_depth': 10, 'min_child_samples': 26, 'subsample': 0.6869038557446994, 'colsample_bytree': 0.7811484214654167, 'reg_alpha': 0.8542248306788436, 'reg_lambda': 8.689201534070046, 'min_split_gain': 0.01769713003570998}. Best is trial 2 with value: 0.010473738518039339.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:56,571] Trial 5 finished with value: 0.021060976693553723 and parameters: {'n_estimators': 80, 'learning_rate': 0.016729432767707192, 'num_leaves': 53, 'max_depth': 8, 'min_child_samples': 25, 'subsample': 0.846074277656484, 'colsample_bytree': 0.5886520019801587, 'reg_alpha': 4.586925634771374, 'reg_lambda': 9.232226780181378, 'min_split_gain': 0.02769382344512125}. Best is trial 2 with value: 0.010473738518039339.
[I 2025-06-15 19:46:56,573] A new study created in memory with name: no-name-981911de-1557-43e1-a5dd-3692bd262dbb
[I 2025-06-15 19:46:56,612] Trial 0 finished with value: 0.04234702751166558 and parameters: {'n_estimators': 199, 'learning_rate': 0.28132318527838457, 'num_leaves': 251, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.8906304561363877, 'colsample_bytree': 0.9260338619250805, 'reg_alpha': 1.8017893004647778, 'reg_lambda': 0.842729283994561, 'min_split_gain': 0.018886247994950502}. Best is trial 0 with value: 0.04234702751166558.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         956
1           Price_lag_2         842
14      pct_chg_Load_DA         702
3         Load_DA_lag_0         592
11           NGas_lag_2         420
2           Price_lag_7         409
5       WindOn_DA_lag_0         405
15       lag168_Load_DA         404
16  roll2d_median_Price         398
10           Coal_lag_2         310
13            EUA_lag_2         308
12            Oil_lag_2         243
7                  WD_1          16
9                  WD_7           9
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          20
2           Price_lag_7           3
3         Load_DA_lag_0           1
15       lag168_Load_DA           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 7.25% done
********************* END NS.. 52 ****************************************************
********************* START NS ... 53 ****************************************************
[I 2025-06-15 19:46:57,552] A new study created in memory with name: no-name-17bac679-1970-49e6-bf41-7056c37ff8c0
[I 2025-06-15 19:46:57,849] Trial 0 finished with value: 0.006149078412084019 and parameters: {'n_estimators': 153, 'learning_rate': 0.12101367401245534, 'num_leaves': 156, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.9149686427001789, 'colsample_bytree': 0.5288814914239985, 'reg_alpha': 3.556778677275746, 'reg_lambda': 2.998489552367878, 'min_split_gain': 0.022909879513817982}. Best is trial 0 with value: 0.006149078412084019.
[Trial 0] New best: 0.00615
[I 2025-06-15 19:46:58,208] Trial 1 finished with value: 0.004538395521386803 and parameters: {'n_estimators': 112, 'learning_rate': 0.08489799513985162, 'num_leaves': 120, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.5111405337786699, 'colsample_bytree': 0.9201861038549899, 'reg_alpha': 6.855497330505066, 'reg_lambda': 7.247134888256822, 'min_split_gain': 0.00830862226260295}. Best is trial 1 with value: 0.004538395521386803.
[Trial 1] New best: 0.00454
[I 2025-06-15 19:46:59,212] Trial 2 finished with value: 0.0048364479555515925 and parameters: {'n_estimators': 195, 'learning_rate': 0.03488255696973636, 'num_leaves': 166, 'max_depth': 10, 'min_child_samples': 17, 'subsample': 0.8081381433728766, 'colsample_bytree': 0.7450604177280238, 'reg_alpha': 3.2727000139753626, 'reg_lambda': 7.0172922231600365, 'min_split_gain': 0.001250689343520134}. Best is trial 1 with value: 0.004538395521386803.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:46:59,513] Trial 3 finished with value: 0.004615300946895241 and parameters: {'n_estimators': 283, 'learning_rate': 0.0847125638711359, 'num_leaves': 131, 'max_depth': 5, 'min_child_samples': 25, 'subsample': 0.7377402824151281, 'colsample_bytree': 0.9448590139946413, 'reg_alpha': 7.546727275846271, 'reg_lambda': 3.459818065488119, 'min_split_gain': 0.0348431749024583}. Best is trial 1 with value: 0.004538395521386803.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:46:59,943] Trial 4 finished with value: 0.005142623016792007 and parameters: {'n_estimators': 125, 'learning_rate': 0.020105956257136678, 'num_leaves': 162, 'max_depth': 10, 'min_child_samples': 16, 'subsample': 0.5284422003223661, 'colsample_bytree': 0.8777305382586058, 'reg_alpha': 8.36261658403455, 'reg_lambda': 0.33530673004438016, 'min_split_gain': 0.0451856974072542}. Best is trial 1 with value: 0.004538395521386803.
[I 2025-06-15 19:46:59,945] A new study created in memory with name: no-name-0602c3c2-5f3d-4e21-9346-cf01e1f20a63
[I 2025-06-15 19:46:59,968] Trial 0 finished with value: 0.039862279599268594 and parameters: {'n_estimators': 128, 'learning_rate': 0.28306149909690653, 'num_leaves': 132, 'max_depth': 10, 'min_child_samples': 16, 'subsample': 0.5629688937319106, 'colsample_bytree': 0.7209946677726194, 'reg_alpha': 5.588733204579719, 'reg_lambda': 0.9590098488265775, 'min_split_gain': 0.02333135619400997}. Best is trial 0 with value: 0.039862279599268594.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         692
14      pct_chg_Load_DA         476
1           Price_lag_2         405
3         Load_DA_lag_0         328
5       WindOn_DA_lag_0         269
13            EUA_lag_2         259
2           Price_lag_7         236
11           NGas_lag_2         234
16  roll2d_median_Price         217
15       lag168_Load_DA         207
10           Coal_lag_2         198
12            Oil_lag_2         181
9                  WD_7          19
7                  WD_1          10
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          11
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 7.39% done
********************* END NS.. 53 ****************************************************
********************* START NS ... 54 ****************************************************
[I 2025-06-15 19:47:00,840] A new study created in memory with name: no-name-7705b874-d0ef-40cf-8fc9-7b2336d723b4
[I 2025-06-15 19:47:01,427] Trial 0 finished with value: 0.0009711831752509444 and parameters: {'n_estimators': 175, 'learning_rate': 0.015943177309334713, 'num_leaves': 143, 'max_depth': 7, 'min_child_samples': 5, 'subsample': 0.5812857438230571, 'colsample_bytree': 0.655633717079828, 'reg_alpha': 1.990056988312754, 'reg_lambda': 4.548218721496743, 'min_split_gain': 0.011402506414111491}. Best is trial 0 with value: 0.0009711831752509444.
[Trial 0] New best: 0.00097
[I 2025-06-15 19:47:01,780] Trial 1 finished with value: 0.001110443947663642 and parameters: {'n_estimators': 95, 'learning_rate': 0.02940612125370426, 'num_leaves': 173, 'max_depth': 7, 'min_child_samples': 3, 'subsample': 0.6997925228529691, 'colsample_bytree': 0.5729091537710026, 'reg_alpha': 5.814397456723162, 'reg_lambda': 1.8643397953570695, 'min_split_gain': 0.046293331113800615}. Best is trial 0 with value: 0.0009711831752509444.
[I 2025-06-15 19:47:01,962] Trial 2 finished with value: 0.0013521939557239878 and parameters: {'n_estimators': 120, 'learning_rate': 0.1830858449974019, 'num_leaves': 97, 'max_depth': 5, 'min_child_samples': 29, 'subsample': 0.6645348032962882, 'colsample_bytree': 0.5062579720970153, 'reg_alpha': 8.297202142767011, 'reg_lambda': 6.5439678268006505, 'min_split_gain': 0.02640476582817787}. Best is trial 0 with value: 0.0009711831752509444.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:02,605] Trial 3 finished with value: 0.0009418524623960302 and parameters: {'n_estimators': 224, 'learning_rate': 0.04359274328142993, 'num_leaves': 146, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.9968418861957202, 'colsample_bytree': 0.8681143355223626, 'reg_alpha': 0.04401457851343382, 'reg_lambda': 3.506378007439909, 'min_split_gain': 0.03295092750981236}. Best is trial 3 with value: 0.0009418524623960302.
[I 2025-06-15 19:47:02,608] A new study created in memory with name: no-name-874ed05f-9788-4465-a2f7-2bd9b3d591d6
[I 2025-06-15 19:47:02,647] Trial 0 finished with value: 0.026458419181487596 and parameters: {'n_estimators': 242, 'learning_rate': 0.0492655195148136, 'num_leaves': 97, 'max_depth': 7, 'min_child_samples': 12, 'subsample': 0.7276036520247535, 'colsample_bytree': 0.5815056807443344, 'reg_alpha': 5.193452753247607, 'reg_lambda': 3.6959547634657195, 'min_split_gain': 0.022726505422801625}. Best is trial 0 with value: 0.026458419181487596.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1725
1           Price_lag_2        1202
14      pct_chg_Load_DA         939
3         Load_DA_lag_0         602
2           Price_lag_7         575
16  roll2d_median_Price         463
11           NGas_lag_2         451
5       WindOn_DA_lag_0         387
15       lag168_Load_DA         353
10           Coal_lag_2         318
12            Oil_lag_2         288
13            EUA_lag_2         252
7                  WD_1          21
9                  WD_7          18
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          50
1           Price_lag_2          16
2           Price_lag_7           3
3         Load_DA_lag_0           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 7.52% done
********************* END NS.. 54 ****************************************************
********************* START NS ... 55 ****************************************************
[I 2025-06-15 19:47:03,765] A new study created in memory with name: no-name-97851048-444c-4255-9041-5aa701683673
[I 2025-06-15 19:47:04,431] Trial 0 finished with value: 0.0009926584291073196 and parameters: {'n_estimators': 229, 'learning_rate': 0.03281106979792553, 'num_leaves': 132, 'max_depth': 7, 'min_child_samples': 23, 'subsample': 0.8965088687256757, 'colsample_bytree': 0.8815836059816897, 'reg_alpha': 2.2192759705405396, 'reg_lambda': 5.13983759299681, 'min_split_gain': 0.01694367316527323}. Best is trial 0 with value: 0.0009926584291073196.
[Trial 0] New best: 0.00099
[I 2025-06-15 19:47:05,072] Trial 1 finished with value: 0.0010294897912201367 and parameters: {'n_estimators': 184, 'learning_rate': 0.011978972392972584, 'num_leaves': 207, 'max_depth': 9, 'min_child_samples': 7, 'subsample': 0.7250293241434396, 'colsample_bytree': 0.8912953161030804, 'reg_alpha': 7.937673630990712, 'reg_lambda': 3.112825274213301, 'min_split_gain': 0.016947431981251045}. Best is trial 0 with value: 0.0009926584291073196.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:05,465] Trial 2 finished with value: 0.0015883766149047396 and parameters: {'n_estimators': 169, 'learning_rate': 0.12539401176681492, 'num_leaves': 149, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.8726872192623122, 'colsample_bytree': 0.5075733659634512, 'reg_alpha': 0.8980149817475602, 'reg_lambda': 6.53934861795686, 'min_split_gain': 0.043995181869793135}. Best is trial 0 with value: 0.0009926584291073196.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:05,689] Trial 3 finished with value: 0.005663613626129535 and parameters: {'n_estimators': 63, 'learning_rate': 0.015722560436508548, 'num_leaves': 185, 'max_depth': 10, 'min_child_samples': 22, 'subsample': 0.9027915703342873, 'colsample_bytree': 0.9975747605525145, 'reg_alpha': 9.282582356165342, 'reg_lambda': 2.4762701524215425, 'min_split_gain': 0.01680317996742751}. Best is trial 0 with value: 0.0009926584291073196.
[I 2025-06-15 19:47:05,691] A new study created in memory with name: no-name-2e5bfcbf-c6aa-45ab-8af7-1e703c72f879
[I 2025-06-15 19:47:05,794] Trial 0 finished with value: 0.0015061829721566052 and parameters: {'n_estimators': 240, 'learning_rate': 0.02077564447940133, 'num_leaves': 111, 'max_depth': 13, 'min_child_samples': 7, 'subsample': 0.5046050191666462, 'colsample_bytree': 0.5918983954033816, 'reg_alpha': 0.9080234345988314, 'reg_lambda': 4.519949805979488, 'min_split_gain': 0.03842203655076165}. Best is trial 0 with value: 0.0015061829721566052.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2025
1           Price_lag_2        1460
14      pct_chg_Load_DA        1295
3         Load_DA_lag_0         887
11           NGas_lag_2         606
2           Price_lag_7         594
16  roll2d_median_Price         566
5       WindOn_DA_lag_0         479
13            EUA_lag_2         456
15       lag168_Load_DA         446
12            Oil_lag_2         395
10           Coal_lag_2         370
7                  WD_1          26
9                  WD_7          16
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         158
1           Price_lag_2          54
15       lag168_Load_DA          27
2           Price_lag_7          25
3         Load_DA_lag_0          15
12            Oil_lag_2          10
5       WindOn_DA_lag_0           3
7                  WD_1           1
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 7.66% done
********************* END NS.. 55 ****************************************************
********************* START NS ... 56 ****************************************************
[I 2025-06-15 19:47:06,957] A new study created in memory with name: no-name-bc72070e-dfaa-4081-be19-fd95e80f77de
[I 2025-06-15 19:47:07,128] Trial 0 finished with value: 0.0015277875711610855 and parameters: {'n_estimators': 170, 'learning_rate': 0.283713727594308, 'num_leaves': 159, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.7810680181628153, 'colsample_bytree': 0.631726220817736, 'reg_alpha': 9.577904114374558, 'reg_lambda': 0.048469500967580625, 'min_split_gain': 0.0027040327758978966}. Best is trial 0 with value: 0.0015277875711610855.
[Trial 0] New best: 0.00153
[I 2025-06-15 19:47:08,197] Trial 1 finished with value: 0.0012336975510526544 and parameters: {'n_estimators': 248, 'learning_rate': 0.01787529384198921, 'num_leaves': 198, 'max_depth': 16, 'min_child_samples': 15, 'subsample': 0.9698788441582051, 'colsample_bytree': 0.7857517358834716, 'reg_alpha': 3.887711248646589, 'reg_lambda': 1.0202072217330993, 'min_split_gain': 0.003250864796025477}. Best is trial 1 with value: 0.0012336975510526544.
[I 2025-06-15 19:47:08,355] Trial 2 finished with value: 0.0008936454687505834 and parameters: {'n_estimators': 70, 'learning_rate': 0.032956737102467035, 'num_leaves': 230, 'max_depth': 4, 'min_child_samples': 10, 'subsample': 0.5893608724675674, 'colsample_bytree': 0.6677258519174378, 'reg_alpha': 7.582955982254573, 'reg_lambda': 8.64012744505598, 'min_split_gain': 0.014233601656523176}. Best is trial 2 with value: 0.0008936454687505834.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:08,588] Trial 3 finished with value: 0.001533635683547846 and parameters: {'n_estimators': 200, 'learning_rate': 0.15465735344919285, 'num_leaves': 46, 'max_depth': 8, 'min_child_samples': 22, 'subsample': 0.7396417668402115, 'colsample_bytree': 0.7834722495493496, 'reg_alpha': 4.619322730304438, 'reg_lambda': 4.645717485926459, 'min_split_gain': 0.04765502763280183}. Best is trial 2 with value: 0.0008936454687505834.
[I 2025-06-15 19:47:08,591] A new study created in memory with name: no-name-710dd816-44a9-42db-b9b5-56543ba797ce
[I 2025-06-15 19:47:08,625] Trial 0 finished with value: 0.0025010908855176185 and parameters: {'n_estimators': 225, 'learning_rate': 0.111362855935145, 'num_leaves': 199, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.5324261437208093, 'colsample_bytree': 0.7147607098822973, 'reg_alpha': 2.6677450079705354, 'reg_lambda': 3.959607134633635, 'min_split_gain': 0.017978057345560943}. Best is trial 0 with value: 0.0025010908855176185.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         500
1           Price_lag_2         198
16  roll2d_median_Price          98
14      pct_chg_Load_DA          79
2           Price_lag_7          36
11           NGas_lag_2          21
12            Oil_lag_2           8
13            EUA_lag_2           7
3         Load_DA_lag_0           6
10           Coal_lag_2           5
15       lag168_Load_DA           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          33
1           Price_lag_2           6
3         Load_DA_lag_0           4
12            Oil_lag_2           3
2           Price_lag_7           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 7.80% done
********************* END NS.. 56 ****************************************************
********************* START NS ... 57 ****************************************************
[I 2025-06-15 19:47:09,367] A new study created in memory with name: no-name-65e5ca5e-c48f-42ba-82d7-b348ff9981df
[I 2025-06-15 19:47:10,221] Trial 0 finished with value: 0.0018292990842729232 and parameters: {'n_estimators': 291, 'learning_rate': 0.010176953173737813, 'num_leaves': 48, 'max_depth': 9, 'min_child_samples': 11, 'subsample': 0.9305047167759662, 'colsample_bytree': 0.9837892179120236, 'reg_alpha': 7.415065498109596, 'reg_lambda': 1.0503411427998632, 'min_split_gain': 0.034779483598917825}. Best is trial 0 with value: 0.0018292990842729232.
[Trial 0] New best: 0.00183
[I 2025-06-15 19:47:10,556] Trial 1 finished with value: 0.0019281341704287895 and parameters: {'n_estimators': 270, 'learning_rate': 0.07992802034412408, 'num_leaves': 240, 'max_depth': 5, 'min_child_samples': 9, 'subsample': 0.8578510099103192, 'colsample_bytree': 0.8446751690009329, 'reg_alpha': 5.983909029312258, 'reg_lambda': 1.9273208501826544, 'min_split_gain': 0.029444126777135572}. Best is trial 0 with value: 0.0018292990842729232.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:11,010] Trial 2 finished with value: 0.0019382978168785692 and parameters: {'n_estimators': 266, 'learning_rate': 0.13720011560359646, 'num_leaves': 246, 'max_depth': 8, 'min_child_samples': 13, 'subsample': 0.7213869673192607, 'colsample_bytree': 0.7008599367970392, 'reg_alpha': 1.276925364407967, 'reg_lambda': 6.8719949309547035, 'min_split_gain': 0.03815841258517954}. Best is trial 0 with value: 0.0018292990842729232.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:11,300] Trial 3 finished with value: 0.002424616509395659 and parameters: {'n_estimators': 94, 'learning_rate': 0.12512444404711112, 'num_leaves': 39, 'max_depth': 7, 'min_child_samples': 21, 'subsample': 0.9563326763553999, 'colsample_bytree': 0.5397341272961631, 'reg_alpha': 5.045491707981849, 'reg_lambda': 4.98923770226199, 'min_split_gain': 0.019749211536491302}. Best is trial 0 with value: 0.0018292990842729232.
[I 2025-06-15 19:47:11,303] A new study created in memory with name: no-name-ca7914a7-254a-494d-ae9e-37cd3519bb94
[I 2025-06-15 19:47:11,339] Trial 0 finished with value: 0.003950965438803932 and parameters: {'n_estimators': 183, 'learning_rate': 0.040317111976527965, 'num_leaves': 179, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.946658485418904, 'colsample_bytree': 0.8793434653041902, 'reg_alpha': 4.812139992431187, 'reg_lambda': 8.537577055409228, 'min_split_gain': 0.005316989281311069}. Best is trial 0 with value: 0.003950965438803932.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3634
14      pct_chg_Load_DA        1055
1           Price_lag_2        1038
16  roll2d_median_Price         683
2           Price_lag_7         574
11           NGas_lag_2         407
10           Coal_lag_2         286
12            Oil_lag_2         224
13            EUA_lag_2         222
3         Load_DA_lag_0         205
15       lag168_Load_DA          45
5       WindOn_DA_lag_0          35
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         123
1           Price_lag_2           9
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 7.93% done
********************* END NS.. 57 ****************************************************
********************* START NS ... 58 ****************************************************
[I 2025-06-15 19:47:12,782] A new study created in memory with name: no-name-f808ea55-f508-4314-97ec-46c3a2755732
[I 2025-06-15 19:47:13,031] Trial 0 finished with value: 0.014703116720719306 and parameters: {'n_estimators': 188, 'learning_rate': 0.11024150800860741, 'num_leaves': 64, 'max_depth': 6, 'min_child_samples': 21, 'subsample': 0.778952824865411, 'colsample_bytree': 0.6720028705313709, 'reg_alpha': 3.285662159169674, 'reg_lambda': 9.824861524014658, 'min_split_gain': 0.033866387220374015}. Best is trial 0 with value: 0.014703116720719306.
[Trial 0] New best: 0.01470
[I 2025-06-15 19:47:13,237] Trial 1 finished with value: 0.016224763331218087 and parameters: {'n_estimators': 261, 'learning_rate': 0.1882522459839614, 'num_leaves': 152, 'max_depth': 16, 'min_child_samples': 4, 'subsample': 0.7888208300755337, 'colsample_bytree': 0.8210062214026386, 'reg_alpha': 9.113502520728346, 'reg_lambda': 3.259947338089791, 'min_split_gain': 0.040368911580889816}. Best is trial 0 with value: 0.014703116720719306.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:14,421] Trial 2 finished with value: 0.01459819074105371 and parameters: {'n_estimators': 262, 'learning_rate': 0.019334511615902644, 'num_leaves': 166, 'max_depth': 9, 'min_child_samples': 16, 'subsample': 0.7658556760665272, 'colsample_bytree': 0.7317317369114615, 'reg_alpha': 2.6679660265119134, 'reg_lambda': 9.382452289264764, 'min_split_gain': 0.025636850669340818}. Best is trial 2 with value: 0.01459819074105371.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:15,613] Trial 3 finished with value: 0.014968681654375779 and parameters: {'n_estimators': 297, 'learning_rate': 0.03726485216008292, 'num_leaves': 76, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.6789464518859799, 'colsample_bytree': 0.7264348367188489, 'reg_alpha': 1.3638888030712848, 'reg_lambda': 9.227381950426444, 'min_split_gain': 0.011020322896017794}. Best is trial 2 with value: 0.01459819074105371.
[I 2025-06-15 19:47:15,616] A new study created in memory with name: no-name-9bc22657-cf8f-4b59-856c-aa462afd73bd
[I 2025-06-15 19:47:15,651] Trial 0 finished with value: 0.019987968645930013 and parameters: {'n_estimators': 135, 'learning_rate': 0.012346950044034528, 'num_leaves': 82, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.9917822190161321, 'colsample_bytree': 0.8599166409823742, 'reg_alpha': 1.6544310946010121, 'reg_lambda': 9.669364954210542, 'min_split_gain': 0.02692484846987212}. Best is trial 0 with value: 0.019987968645930013.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2708
1           Price_lag_2        2246
14      pct_chg_Load_DA        1537
3         Load_DA_lag_0         959
16  roll2d_median_Price         769
2           Price_lag_7         736
11           NGas_lag_2         687
15       lag168_Load_DA         530
5       WindOn_DA_lag_0         478
10           Coal_lag_2         446
12            Oil_lag_2         410
13            EUA_lag_2         391
9                  WD_7          44
7                  WD_1          18
8                  WD_6          17
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         254
1           Price_lag_2          19
15       lag168_Load_DA           7
2           Price_lag_7           1
5       WindOn_DA_lag_0           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
16  roll2d_median_Price           0
-> 8.07% done
********************* END NS.. 58 ****************************************************
********************* START NS ... 59 ****************************************************
[I 2025-06-15 19:47:17,520] A new study created in memory with name: no-name-250d2bea-e3f4-412a-b162-93681d630b17
[I 2025-06-15 19:47:17,959] Trial 0 finished with value: 0.0149983500511496 and parameters: {'n_estimators': 206, 'learning_rate': 0.0176484551764774, 'num_leaves': 46, 'max_depth': 5, 'min_child_samples': 17, 'subsample': 0.609890253984125, 'colsample_bytree': 0.7570465101098371, 'reg_alpha': 0.4019182455325587, 'reg_lambda': 2.8549641412121742, 'min_split_gain': 0.003303391381249332}. Best is trial 0 with value: 0.0149983500511496.
[Trial 0] New best: 0.01500
[I 2025-06-15 19:47:18,202] Trial 1 finished with value: 0.014307274583065242 and parameters: {'n_estimators': 183, 'learning_rate': 0.12918524727491448, 'num_leaves': 133, 'max_depth': 7, 'min_child_samples': 21, 'subsample': 0.6938373131466551, 'colsample_bytree': 0.8166711409858578, 'reg_alpha': 2.8557437586733037, 'reg_lambda': 0.9935029429278897, 'min_split_gain': 0.04545823038396421}. Best is trial 1 with value: 0.014307274583065242.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:18,617] Trial 2 finished with value: 0.013979157051065554 and parameters: {'n_estimators': 135, 'learning_rate': 0.0913316437808764, 'num_leaves': 53, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.6035576049002953, 'colsample_bytree': 0.549596972296059, 'reg_alpha': 3.982729241620695, 'reg_lambda': 4.707666352544707, 'min_split_gain': 0.0056936562718555685}. Best is trial 2 with value: 0.013979157051065554.
[Trial 2] New best: 0.01398
[I 2025-06-15 19:47:19,059] Trial 3 finished with value: 0.01406064727854977 and parameters: {'n_estimators': 239, 'learning_rate': 0.01944243702320253, 'num_leaves': 75, 'max_depth': 4, 'min_child_samples': 30, 'subsample': 0.7908534021757134, 'colsample_bytree': 0.6465651487563665, 'reg_alpha': 7.035787862297964, 'reg_lambda': 5.353947347204451, 'min_split_gain': 0.0011464211651712741}. Best is trial 2 with value: 0.013979157051065554.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:19,441] Trial 4 finished with value: 0.015170774404690725 and parameters: {'n_estimators': 110, 'learning_rate': 0.043819066601917735, 'num_leaves': 87, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.8784581517424688, 'colsample_bytree': 0.6719808392021409, 'reg_alpha': 4.423526199898133, 'reg_lambda': 1.9046250440243417, 'min_split_gain': 0.0457254666319574}. Best is trial 2 with value: 0.013979157051065554.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:19,712] Trial 5 finished with value: 0.01434927164979443 and parameters: {'n_estimators': 154, 'learning_rate': 0.04218561961170899, 'num_leaves': 82, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.8824917104090217, 'colsample_bytree': 0.5674591206178113, 'reg_alpha': 0.6917795002246641, 'reg_lambda': 8.10606536641918, 'min_split_gain': 0.040885840309454566}. Best is trial 2 with value: 0.013979157051065554.
[I 2025-06-15 19:47:19,715] A new study created in memory with name: no-name-0fc98591-fa77-492a-beda-082fc17dc4d0
[I 2025-06-15 19:47:19,747] Trial 0 finished with value: 0.03370686518761352 and parameters: {'n_estimators': 90, 'learning_rate': 0.14958702630125376, 'num_leaves': 134, 'max_depth': 14, 'min_child_samples': 18, 'subsample': 0.6763348520283632, 'colsample_bytree': 0.9686050728121401, 'reg_alpha': 8.53391938977007, 'reg_lambda': 6.1501146019679265, 'min_split_gain': 0.026439139460034553}. Best is trial 0 with value: 0.03370686518761352.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         788
1           Price_lag_2         701
3         Load_DA_lag_0         496
14      pct_chg_Load_DA         481
5       WindOn_DA_lag_0         435
2           Price_lag_7         431
13            EUA_lag_2         386
15       lag168_Load_DA         381
10           Coal_lag_2         354
11           NGas_lag_2         317
16  roll2d_median_Price         285
12            Oil_lag_2         276
9                  WD_7          32
7                  WD_1          19
8                  WD_6          17
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 8.21% done
********************* END NS.. 59 ****************************************************
********************* START NS ... 60 ****************************************************
[I 2025-06-15 19:47:20,855] A new study created in memory with name: no-name-b2f98c7a-aa93-4b86-bdea-647067aeb7c3
[I 2025-06-15 19:47:21,039] Trial 0 finished with value: 0.009053598438601667 and parameters: {'n_estimators': 69, 'learning_rate': 0.11470422241665346, 'num_leaves': 139, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.5971338919398506, 'colsample_bytree': 0.5186523089349973, 'reg_alpha': 4.8326369795239055, 'reg_lambda': 0.33852264813340693, 'min_split_gain': 0.014914103460000178}. Best is trial 0 with value: 0.009053598438601667.
[I 2025-06-15 19:47:21,185] Trial 1 finished with value: 0.007065552856165796 and parameters: {'n_estimators': 154, 'learning_rate': 0.2499048405332192, 'num_leaves': 104, 'max_depth': 8, 'min_child_samples': 30, 'subsample': 0.6754819684566238, 'colsample_bytree': 0.7140290557543598, 'reg_alpha': 7.6767182797548825, 'reg_lambda': 6.108779191797563, 'min_split_gain': 0.01926620935432945}. Best is trial 1 with value: 0.007065552856165796.
[Trial 0] New best: 0.00905
[Trial 1] New best: 0.00707
[I 2025-06-15 19:47:21,773] Trial 2 finished with value: 0.009134461042993348 and parameters: {'n_estimators': 254, 'learning_rate': 0.035791091621445205, 'num_leaves': 238, 'max_depth': 8, 'min_child_samples': 3, 'subsample': 0.8881075320258964, 'colsample_bytree': 0.6368808923793015, 'reg_alpha': 8.70118570126873, 'reg_lambda': 0.7493376438179344, 'min_split_gain': 0.03644515350281829}. Best is trial 1 with value: 0.007065552856165796.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:22,075] Trial 3 finished with value: 0.008728891220023424 and parameters: {'n_estimators': 168, 'learning_rate': 0.10475390456076453, 'num_leaves': 206, 'max_depth': 6, 'min_child_samples': 12, 'subsample': 0.6571293226141934, 'colsample_bytree': 0.8264153384101361, 'reg_alpha': 3.8665544385691075, 'reg_lambda': 7.588005559277882, 'min_split_gain': 0.0114488196191248}. Best is trial 1 with value: 0.007065552856165796.
[I 2025-06-15 19:47:22,237] Trial 4 finished with value: 0.02929434361162822 and parameters: {'n_estimators': 78, 'learning_rate': 0.011355615579112157, 'num_leaves': 190, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.9780559237206619, 'colsample_bytree': 0.83659911053368, 'reg_alpha': 1.0333310217050773, 'reg_lambda': 6.4315000248838565, 'min_split_gain': 0.02345061736093387}. Best is trial 1 with value: 0.007065552856165796.
[I 2025-06-15 19:47:22,240] A new study created in memory with name: no-name-7b56d5a0-ef3d-4dd2-bf8a-cf12591265a3
[I 2025-06-15 19:47:22,262] Trial 0 finished with value: 0.02793847763590648 and parameters: {'n_estimators': 196, 'learning_rate': 0.18184321240444018, 'num_leaves': 216, 'max_depth': 11, 'min_child_samples': 18, 'subsample': 0.6126161808920514, 'colsample_bytree': 0.6617813354265143, 'reg_alpha': 2.8772557541714026, 'reg_lambda': 4.623473654534182, 'min_split_gain': 0.0009429891606613727}. Best is trial 0 with value: 0.02793847763590648.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         182
14      pct_chg_Load_DA         101
1           Price_lag_2          93
3         Load_DA_lag_0          64
13            EUA_lag_2          49
11           NGas_lag_2          45
2           Price_lag_7          43
16  roll2d_median_Price          40
5       WindOn_DA_lag_0          34
12            Oil_lag_2          31
10           Coal_lag_2          29
15       lag168_Load_DA          27
7                  WD_1           2
9                  WD_7           1
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          34
15       lag168_Load_DA           5
10           Coal_lag_2           2
1           Price_lag_2           2
11           NGas_lag_2           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 8.34% done
********************* END NS.. 60 ****************************************************
********************* START NS ... 61 ****************************************************
[I 2025-06-15 19:47:23,092] A new study created in memory with name: no-name-4a2bb1b1-22e4-44f7-b30d-fb0bd36b0072
[I 2025-06-15 19:47:23,561] Trial 0 finished with value: 0.010274832500866594 and parameters: {'n_estimators': 264, 'learning_rate': 0.021248379008591684, 'num_leaves': 193, 'max_depth': 4, 'min_child_samples': 20, 'subsample': 0.5327689638108041, 'colsample_bytree': 0.6821250940059334, 'reg_alpha': 2.609073551537433, 'reg_lambda': 0.5162024717770441, 'min_split_gain': 0.00908373657372502}. Best is trial 0 with value: 0.010274832500866594.
[Trial 0] New best: 0.01027
[I 2025-06-15 19:47:24,276] Trial 1 finished with value: 0.007887907191545212 and parameters: {'n_estimators': 161, 'learning_rate': 0.03513760783285714, 'num_leaves': 99, 'max_depth': 7, 'min_child_samples': 24, 'subsample': 0.9847253278838268, 'colsample_bytree': 0.8468780048452168, 'reg_alpha': 8.463199362057567, 'reg_lambda': 9.546142386762352, 'min_split_gain': 0.005657026695552703}. Best is trial 1 with value: 0.007887907191545212.
[Trial 1] New best: 0.00789
[I 2025-06-15 19:47:24,610] Trial 2 finished with value: 0.024164729620770892 and parameters: {'n_estimators': 105, 'learning_rate': 0.010550523950452516, 'num_leaves': 181, 'max_depth': 14, 'min_child_samples': 20, 'subsample': 0.9935170949798454, 'colsample_bytree': 0.659503515321961, 'reg_alpha': 9.02868896895921, 'reg_lambda': 0.969357369581284, 'min_split_gain': 0.01922730247562586}. Best is trial 1 with value: 0.007887907191545212.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:25,151] Trial 3 finished with value: 0.007312389734708692 and parameters: {'n_estimators': 157, 'learning_rate': 0.06933217088224869, 'num_leaves': 226, 'max_depth': 10, 'min_child_samples': 4, 'subsample': 0.9749183686673532, 'colsample_bytree': 0.5215177881328623, 'reg_alpha': 3.916863282549272, 'reg_lambda': 8.926816172240597, 'min_split_gain': 0.01581609504901425}. Best is trial 3 with value: 0.007312389734708692.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:25,457] Trial 4 finished with value: 0.009508327890847953 and parameters: {'n_estimators': 252, 'learning_rate': 0.09587371901596764, 'num_leaves': 43, 'max_depth': 9, 'min_child_samples': 28, 'subsample': 0.679746368509063, 'colsample_bytree': 0.7410040204060121, 'reg_alpha': 8.523806488483007, 'reg_lambda': 2.6712509162719513, 'min_split_gain': 0.01907349652363302}. Best is trial 3 with value: 0.007312389734708692.
[I 2025-06-15 19:47:25,460] A new study created in memory with name: no-name-5313fd94-f499-48fd-b823-8932ce5a2780
[I 2025-06-15 19:47:25,486] Trial 0 finished with value: 0.03248443386259322 and parameters: {'n_estimators': 231, 'learning_rate': 0.04881987910269799, 'num_leaves': 182, 'max_depth': 4, 'min_child_samples': 17, 'subsample': 0.7089353773556011, 'colsample_bytree': 0.5809816487343564, 'reg_alpha': 9.45978065477341, 'reg_lambda': 4.296992026610338, 'min_split_gain': 0.02851410379676906}. Best is trial 0 with value: 0.03248443386259322.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         881
1           Price_lag_2         867
3         Load_DA_lag_0         620
14      pct_chg_Load_DA         602
2           Price_lag_7         521
5       WindOn_DA_lag_0         435
15       lag168_Load_DA         412
10           Coal_lag_2         389
16  roll2d_median_Price         385
11           NGas_lag_2         367
13            EUA_lag_2         366
12            Oil_lag_2         212
9                  WD_7          26
8                  WD_6          26
7                  WD_1          18
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 8.48% done
********************* END NS.. 61 ****************************************************
********************* START NS ... 62 ****************************************************
[I 2025-06-15 19:47:26,458] A new study created in memory with name: no-name-689da7e0-b117-4a17-bf20-2f7fa317816d
[I 2025-06-15 19:47:26,972] Trial 0 finished with value: 0.0007713315820595222 and parameters: {'n_estimators': 164, 'learning_rate': 0.03831718550425316, 'num_leaves': 239, 'max_depth': 10, 'min_child_samples': 15, 'subsample': 0.9292088081597089, 'colsample_bytree': 0.7512128995215032, 'reg_alpha': 4.045403527481572, 'reg_lambda': 1.596994890957415, 'min_split_gain': 0.038122943102262136}. Best is trial 0 with value: 0.0007713315820595222.
[Trial 0] New best: 0.00077
[I 2025-06-15 19:47:27,170] Trial 1 finished with value: 0.0009061873290653796 and parameters: {'n_estimators': 204, 'learning_rate': 0.2646954041908525, 'num_leaves': 207, 'max_depth': 10, 'min_child_samples': 5, 'subsample': 0.5686674651232195, 'colsample_bytree': 0.6635614663695419, 'reg_alpha': 1.7433256737060299, 'reg_lambda': 1.9577115244062382, 'min_split_gain': 0.04675909036458155}. Best is trial 0 with value: 0.0007713315820595222.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:27,440] Trial 2 finished with value: 0.0011300406260166696 and parameters: {'n_estimators': 119, 'learning_rate': 0.10562342277048102, 'num_leaves': 217, 'max_depth': 6, 'min_child_samples': 29, 'subsample': 0.5746494775257176, 'colsample_bytree': 0.5420294978984304, 'reg_alpha': 6.9111526524645335, 'reg_lambda': 3.163399451901756, 'min_split_gain': 0.0198025885323174}. Best is trial 0 with value: 0.0007713315820595222.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:28,666] Trial 3 finished with value: 0.0005428321467822301 and parameters: {'n_estimators': 237, 'learning_rate': 0.014868448791078672, 'num_leaves': 86, 'max_depth': 16, 'min_child_samples': 9, 'subsample': 0.6829807437546148, 'colsample_bytree': 0.9588041685221056, 'reg_alpha': 2.081296815577265, 'reg_lambda': 9.064086741133307, 'min_split_gain': 0.0030808963529401546}. Best is trial 3 with value: 0.0005428321467822301.
[I 2025-06-15 19:47:28,668] A new study created in memory with name: no-name-12a1383d-f94a-40de-8d24-a46618548f0a
[I 2025-06-15 19:47:28,685] Trial 0 finished with value: 0.0036443259851052915 and parameters: {'n_estimators': 70, 'learning_rate': 0.1873777516809229, 'num_leaves': 202, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.6386474001902662, 'colsample_bytree': 0.693564569048386, 'reg_alpha': 4.992390777012147, 'reg_lambda': 1.1390297422121631, 'min_split_gain': 0.04140541067045786}. Best is trial 0 with value: 0.0036443259851052915.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3842
1           Price_lag_2        2300
14      pct_chg_Load_DA        2295
2           Price_lag_7        1299
16  roll2d_median_Price        1197
3         Load_DA_lag_0         975
11           NGas_lag_2         836
5       WindOn_DA_lag_0         693
10           Coal_lag_2         692
13            EUA_lag_2         579
12            Oil_lag_2         500
15       lag168_Load_DA         451
9                  WD_7           7
8                  WD_6           2
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 8.62% done
********************* END NS.. 62 ****************************************************
********************* START NS ... 63 ****************************************************
[I 2025-06-15 19:47:30,872] A new study created in memory with name: no-name-e4e5699c-e357-45e2-b261-54b3b56eb9a8
[I 2025-06-15 19:47:31,223] Trial 0 finished with value: 0.0008341936392263679 and parameters: {'n_estimators': 182, 'learning_rate': 0.14089612902542867, 'num_leaves': 114, 'max_depth': 10, 'min_child_samples': 29, 'subsample': 0.8965787036664672, 'colsample_bytree': 0.8665569102670639, 'reg_alpha': 2.233941506632954, 'reg_lambda': 9.067747543902888, 'min_split_gain': 0.0034727690063510565}. Best is trial 0 with value: 0.0008341936392263679.
[Trial 0] New best: 0.00083
[I 2025-06-15 19:47:31,455] Trial 1 finished with value: 0.0011342793727391037 and parameters: {'n_estimators': 143, 'learning_rate': 0.0912370049759818, 'num_leaves': 63, 'max_depth': 5, 'min_child_samples': 25, 'subsample': 0.6737195571610894, 'colsample_bytree': 0.7172318034287193, 'reg_alpha': 1.5533225100119918, 'reg_lambda': 1.6473786005500968, 'min_split_gain': 0.034921355794757025}. Best is trial 0 with value: 0.0008341936392263679.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:31,855] Trial 2 finished with value: 0.0012573632852610245 and parameters: {'n_estimators': 171, 'learning_rate': 0.02398972599134944, 'num_leaves': 214, 'max_depth': 5, 'min_child_samples': 17, 'subsample': 0.7166282099675193, 'colsample_bytree': 0.552012191188911, 'reg_alpha': 2.3126314834459425, 'reg_lambda': 8.882268096564982, 'min_split_gain': 0.02185298439331005}. Best is trial 0 with value: 0.0008341936392263679.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:32,295] Trial 3 finished with value: 0.001579767509341372 and parameters: {'n_estimators': 171, 'learning_rate': 0.014805395504899562, 'num_leaves': 133, 'max_depth': 6, 'min_child_samples': 20, 'subsample': 0.5521613839166235, 'colsample_bytree': 0.6600995916246817, 'reg_alpha': 6.909074346910279, 'reg_lambda': 3.38920349150315, 'min_split_gain': 0.03485465901467931}. Best is trial 0 with value: 0.0008341936392263679.
[I 2025-06-15 19:47:32,297] A new study created in memory with name: no-name-8ded8fcc-e651-40a5-b3b0-1d458f3f3e86
[I 2025-06-15 19:47:32,320] Trial 0 finished with value: 0.014390029329102058 and parameters: {'n_estimators': 194, 'learning_rate': 0.1553570006465766, 'num_leaves': 138, 'max_depth': 13, 'min_child_samples': 12, 'subsample': 0.9171127517356789, 'colsample_bytree': 0.5874603062507031, 'reg_alpha': 8.562989499947722, 'reg_lambda': 8.769243601726846, 'min_split_gain': 0.040858066929310266}. Best is trial 0 with value: 0.014390029329102058.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         673
14      pct_chg_Load_DA         606
3         Load_DA_lag_0         599
1           Price_lag_2         560
5       WindOn_DA_lag_0         491
15       lag168_Load_DA         394
2           Price_lag_7         315
11           NGas_lag_2         275
16  roll2d_median_Price         261
13            EUA_lag_2         240
10           Coal_lag_2         239
12            Oil_lag_2         239
9                  WD_7          16
7                  WD_1          11
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 8.76% done
********************* END NS.. 63 ****************************************************
********************* START NS ... 64 ****************************************************
[I 2025-06-15 19:47:33,171] A new study created in memory with name: no-name-e70d5667-eab3-4616-a246-c47c8c4acb79
[I 2025-06-15 19:47:33,603] Trial 0 finished with value: 0.003527315399165979 and parameters: {'n_estimators': 144, 'learning_rate': 0.06844538739644926, 'num_leaves': 196, 'max_depth': 16, 'min_child_samples': 20, 'subsample': 0.5623172926735822, 'colsample_bytree': 0.7034486475277157, 'reg_alpha': 2.4957611334773153, 'reg_lambda': 6.053798427303154, 'min_split_gain': 0.014474003101334744}. Best is trial 0 with value: 0.003527315399165979.
[Trial 0] New best: 0.00353
[I 2025-06-15 19:47:34,225] Trial 1 finished with value: 0.004036053446052206 and parameters: {'n_estimators': 157, 'learning_rate': 0.046601906844765735, 'num_leaves': 125, 'max_depth': 8, 'min_child_samples': 15, 'subsample': 0.9508908766919499, 'colsample_bytree': 0.6159516196064132, 'reg_alpha': 3.232628628379416, 'reg_lambda': 7.959463337380269, 'min_split_gain': 0.0008883993030521453}. Best is trial 0 with value: 0.003527315399165979.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:34,470] Trial 2 finished with value: 0.004578295431661551 and parameters: {'n_estimators': 278, 'learning_rate': 0.15758563745162119, 'num_leaves': 177, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.5298385972155222, 'colsample_bytree': 0.6223834823146852, 'reg_alpha': 7.295206781559493, 'reg_lambda': 9.404115070007926, 'min_split_gain': 0.02034936331385367}. Best is trial 0 with value: 0.003527315399165979.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:34,993] Trial 3 finished with value: 0.005131104967344894 and parameters: {'n_estimators': 187, 'learning_rate': 0.01635625618468268, 'num_leaves': 172, 'max_depth': 6, 'min_child_samples': 23, 'subsample': 0.7697267099753046, 'colsample_bytree': 0.7651118018914482, 'reg_alpha': 8.875174385026511, 'reg_lambda': 2.382510452264274, 'min_split_gain': 0.025049231773562476}. Best is trial 0 with value: 0.003527315399165979.
[I 2025-06-15 19:47:34,997] A new study created in memory with name: no-name-54b232ab-1bb4-4ccc-a046-fdd3057896b6
[I 2025-06-15 19:47:35,022] Trial 0 finished with value: 0.031797818206207286 and parameters: {'n_estimators': 140, 'learning_rate': 0.028620596852985455, 'num_leaves': 254, 'max_depth': 15, 'min_child_samples': 7, 'subsample': 0.8281883018834393, 'colsample_bytree': 0.5215847288583322, 'reg_alpha': 7.452569295599348, 'reg_lambda': 8.049222999750901, 'min_split_gain': 0.015643925047359637}. Best is trial 0 with value: 0.031797818206207286.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1018
1           Price_lag_2         849
14      pct_chg_Load_DA         844
3         Load_DA_lag_0         720
5       WindOn_DA_lag_0         501
11           NGas_lag_2         415
15       lag168_Load_DA         414
2           Price_lag_7         371
10           Coal_lag_2         336
16  roll2d_median_Price         327
13            EUA_lag_2         313
12            Oil_lag_2         219
7                  WD_1          17
9                  WD_7          15
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 8.89% done
********************* END NS.. 64 ****************************************************
********************* START NS ... 65 ****************************************************
[I 2025-06-15 19:47:35,998] A new study created in memory with name: no-name-b4b4020f-7328-4354-b975-27d10867da19
[I 2025-06-15 19:47:36,368] Trial 0 finished with value: 0.023322047096277488 and parameters: {'n_estimators': 115, 'learning_rate': 0.04404795099355739, 'num_leaves': 215, 'max_depth': 14, 'min_child_samples': 5, 'subsample': 0.7380585767487607, 'colsample_bytree': 0.9340435666965484, 'reg_alpha': 7.33391866998113, 'reg_lambda': 1.7264143216771632, 'min_split_gain': 0.02452567610861784}. Best is trial 0 with value: 0.023322047096277488.
[Trial 0] New best: 0.02332
[I 2025-06-15 19:47:36,645] Trial 1 finished with value: 0.026625861926270224 and parameters: {'n_estimators': 217, 'learning_rate': 0.08719439043487524, 'num_leaves': 51, 'max_depth': 10, 'min_child_samples': 8, 'subsample': 0.9595395392668982, 'colsample_bytree': 0.6942755096736684, 'reg_alpha': 9.558049119609512, 'reg_lambda': 0.39362220483302957, 'min_split_gain': 0.04870242260382751}. Best is trial 0 with value: 0.023322047096277488.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:36,885] Trial 2 finished with value: 0.021889732786223175 and parameters: {'n_estimators': 114, 'learning_rate': 0.12548788419337073, 'num_leaves': 39, 'max_depth': 13, 'min_child_samples': 20, 'subsample': 0.5667705884539247, 'colsample_bytree': 0.530828467558976, 'reg_alpha': 8.338665503816392, 'reg_lambda': 3.135011928528253, 'min_split_gain': 0.04768684486163888}. Best is trial 2 with value: 0.021889732786223175.
[Trial 2] New best: 0.02189
[I 2025-06-15 19:47:37,438] Trial 3 finished with value: 0.03217487656197502 and parameters: {'n_estimators': 174, 'learning_rate': 0.01684375404826995, 'num_leaves': 231, 'max_depth': 11, 'min_child_samples': 5, 'subsample': 0.563556200967174, 'colsample_bytree': 0.6208219604267368, 'reg_alpha': 7.944061549822534, 'reg_lambda': 2.646293233474922, 'min_split_gain': 0.032749529667056736}. Best is trial 2 with value: 0.021889732786223175.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:37,930] Trial 4 finished with value: 0.020637924592502993 and parameters: {'n_estimators': 236, 'learning_rate': 0.057734874251035365, 'num_leaves': 196, 'max_depth': 15, 'min_child_samples': 27, 'subsample': 0.7324029076202636, 'colsample_bytree': 0.5423726597528185, 'reg_alpha': 2.7104831741758915, 'reg_lambda': 8.307693588125865, 'min_split_gain': 0.0419118491504239}. Best is trial 4 with value: 0.020637924592502993.
[Trial 4] New best: 0.02064
[I 2025-06-15 19:47:38,376] Trial 5 finished with value: 0.021955785135681527 and parameters: {'n_estimators': 213, 'learning_rate': 0.044883118857565615, 'num_leaves': 232, 'max_depth': 16, 'min_child_samples': 20, 'subsample': 0.630398332677192, 'colsample_bytree': 0.6098745419073837, 'reg_alpha': 7.486505098518047, 'reg_lambda': 6.663019406649459, 'min_split_gain': 0.04999688402098567}. Best is trial 4 with value: 0.020637924592502993.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:38,913] Trial 6 finished with value: 0.04609071482293564 and parameters: {'n_estimators': 195, 'learning_rate': 0.01094472645891858, 'num_leaves': 105, 'max_depth': 7, 'min_child_samples': 11, 'subsample': 0.7053801844017847, 'colsample_bytree': 0.5054050854050088, 'reg_alpha': 8.206457061590177, 'reg_lambda': 7.063519320748285, 'min_split_gain': 0.002392251659729239}. Best is trial 4 with value: 0.020637924592502993.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:39,438] Trial 7 finished with value: 0.021668084259948536 and parameters: {'n_estimators': 256, 'learning_rate': 0.12330762413179312, 'num_leaves': 194, 'max_depth': 13, 'min_child_samples': 4, 'subsample': 0.7234624584407179, 'colsample_bytree': 0.8322633286339735, 'reg_alpha': 4.511358972134244, 'reg_lambda': 1.009815222751711, 'min_split_gain': 0.002113767778667303}. Best is trial 4 with value: 0.020637924592502993.
[I 2025-06-15 19:47:39,441] A new study created in memory with name: no-name-3944df5e-4dd3-4737-8c16-105a6636521b
[I 2025-06-15 19:47:39,467] Trial 0 finished with value: 0.10942745533474126 and parameters: {'n_estimators': 209, 'learning_rate': 0.21810673069368525, 'num_leaves': 247, 'max_depth': 8, 'min_child_samples': 7, 'subsample': 0.5075826639465745, 'colsample_bytree': 0.7267383258142387, 'reg_alpha': 3.648976741872989, 'reg_lambda': 2.0755334038869586, 'min_split_gain': 0.005719429639513446}. Best is trial 0 with value: 0.10942745533474126.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         965
1           Price_lag_2         879
14      pct_chg_Load_DA         560
3         Load_DA_lag_0         513
2           Price_lag_7         354
15       lag168_Load_DA         311
5       WindOn_DA_lag_0         295
16  roll2d_median_Price         286
11           NGas_lag_2         276
13            EUA_lag_2         251
10           Coal_lag_2         237
12            Oil_lag_2         130
7                  WD_1          21
9                  WD_7          13
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          19
5       WindOn_DA_lag_0           4
3         Load_DA_lag_0           2
1           Price_lag_2           2
12            Oil_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 9.03% done
********************* END NS.. 65 ****************************************************
********************* START NS ... 66 ****************************************************
[I 2025-06-15 19:47:40,534] A new study created in memory with name: no-name-098c650c-f788-48b4-b9ed-5120eaed9f05
[I 2025-06-15 19:47:40,696] Trial 0 finished with value: 0.024211742506529448 and parameters: {'n_estimators': 299, 'learning_rate': 0.25091432974842076, 'num_leaves': 87, 'max_depth': 13, 'min_child_samples': 19, 'subsample': 0.5868617337479323, 'colsample_bytree': 0.6037971743603532, 'reg_alpha': 4.276968799221948, 'reg_lambda': 2.523488035040745, 'min_split_gain': 0.04050323354305073}. Best is trial 0 with value: 0.024211742506529448.
[Trial 0] New best: 0.02421
[I 2025-06-15 19:47:41,040] Trial 1 finished with value: 0.01977062572477552 and parameters: {'n_estimators': 245, 'learning_rate': 0.2032912293689978, 'num_leaves': 209, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.6114429990534959, 'colsample_bytree': 0.7109786383098667, 'reg_alpha': 0.1998134685172881, 'reg_lambda': 5.813777051391723, 'min_split_gain': 0.01301698586581785}. Best is trial 1 with value: 0.01977062572477552.
[Trial 1] New best: 0.01977
[I 2025-06-15 19:47:41,721] Trial 2 finished with value: 0.02459416960598181 and parameters: {'n_estimators': 215, 'learning_rate': 0.012349499698397961, 'num_leaves': 251, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.7045044835016766, 'colsample_bytree': 0.7022612253738735, 'reg_alpha': 1.086149069168888, 'reg_lambda': 4.188746998776649, 'min_split_gain': 0.011758572098000165}. Best is trial 1 with value: 0.01977062572477552.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:42,024] Trial 3 finished with value: 0.01980402731459969 and parameters: {'n_estimators': 135, 'learning_rate': 0.05530800207109415, 'num_leaves': 229, 'max_depth': 5, 'min_child_samples': 26, 'subsample': 0.5357992366488828, 'colsample_bytree': 0.607721775236512, 'reg_alpha': 2.474251089701837, 'reg_lambda': 3.86742233862146, 'min_split_gain': 0.014620444080689389}. Best is trial 1 with value: 0.01977062572477552.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:42,821] Trial 4 finished with value: 0.023868039143077658 and parameters: {'n_estimators': 262, 'learning_rate': 0.01584114080897358, 'num_leaves': 172, 'max_depth': 10, 'min_child_samples': 17, 'subsample': 0.8450790628006086, 'colsample_bytree': 0.9082754685170465, 'reg_alpha': 8.00453985438647, 'reg_lambda': 2.6965570543836357, 'min_split_gain': 0.01987650542251066}. Best is trial 1 with value: 0.01977062572477552.
[I 2025-06-15 19:47:42,824] A new study created in memory with name: no-name-41435a0a-3f2a-4d28-8877-ae6096135e29
[I 2025-06-15 19:47:42,849] Trial 0 finished with value: 0.09109394436823211 and parameters: {'n_estimators': 77, 'learning_rate': 0.02399645013253976, 'num_leaves': 53, 'max_depth': 13, 'min_child_samples': 5, 'subsample': 0.7607215967531431, 'colsample_bytree': 0.5757850372276588, 'reg_alpha': 4.685318533833048, 'reg_lambda': 8.006644850710291, 'min_split_gain': 0.004915572115652267}. Best is trial 0 with value: 0.09109394436823211.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         589
1           Price_lag_2         511
14      pct_chg_Load_DA         506
3         Load_DA_lag_0         392
16  roll2d_median_Price         302
5       WindOn_DA_lag_0         279
2           Price_lag_7         262
15       lag168_Load_DA         212
11           NGas_lag_2         205
13            EUA_lag_2         169
10           Coal_lag_2         161
12            Oil_lag_2         134
7                  WD_1           6
9                  WD_7           5
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          86
1           Price_lag_2          44
5       WindOn_DA_lag_0          22
3         Load_DA_lag_0          14
7                  WD_1           1
12            Oil_lag_2           1
11           NGas_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 9.17% done
********************* END NS.. 66 ****************************************************
********************* START NS ... 67 ****************************************************
[I 2025-06-15 19:47:43,821] A new study created in memory with name: no-name-e5043734-77c4-45c7-95b2-932ab43da830
[I 2025-06-15 19:47:44,096] Trial 0 finished with value: 0.03731245030907487 and parameters: {'n_estimators': 83, 'learning_rate': 0.012499253700292378, 'num_leaves': 220, 'max_depth': 15, 'min_child_samples': 27, 'subsample': 0.8045887743811861, 'colsample_bytree': 0.5418471820153217, 'reg_alpha': 2.367181918120822, 'reg_lambda': 2.0973262134550916, 'min_split_gain': 0.03420788799765565}. Best is trial 0 with value: 0.03731245030907487.
[Trial 0] New best: 0.03731
[I 2025-06-15 19:47:44,385] Trial 1 finished with value: 0.007287394672617464 and parameters: {'n_estimators': 106, 'learning_rate': 0.03982052880521152, 'num_leaves': 32, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.6285165082894859, 'colsample_bytree': 0.7899974062410746, 'reg_alpha': 7.360715482618479, 'reg_lambda': 4.500332467550164, 'min_split_gain': 0.037728678341170066}. Best is trial 1 with value: 0.007287394672617464.
[Trial 1] New best: 0.00729
[I 2025-06-15 19:47:44,602] Trial 2 finished with value: 0.006984285208741549 and parameters: {'n_estimators': 232, 'learning_rate': 0.16810963481890487, 'num_leaves': 236, 'max_depth': 10, 'min_child_samples': 13, 'subsample': 0.9285514441083951, 'colsample_bytree': 0.8841968790177476, 'reg_alpha': 8.240685200008814, 'reg_lambda': 3.6891488622077007, 'min_split_gain': 0.047617410936633374}. Best is trial 2 with value: 0.006984285208741549.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:44,922] Trial 3 finished with value: 0.006042431970185669 and parameters: {'n_estimators': 96, 'learning_rate': 0.14322119110364523, 'num_leaves': 106, 'max_depth': 13, 'min_child_samples': 11, 'subsample': 0.5659896369722779, 'colsample_bytree': 0.6990072399603884, 'reg_alpha': 7.294912597180939, 'reg_lambda': 6.410294196208101, 'min_split_gain': 0.0022525438106927587}. Best is trial 3 with value: 0.006042431970185669.
[Trial 3] New best: 0.00604
[I 2025-06-15 19:47:45,248] Trial 4 finished with value: 0.006310612493916776 and parameters: {'n_estimators': 271, 'learning_rate': 0.06506754086501348, 'num_leaves': 72, 'max_depth': 6, 'min_child_samples': 30, 'subsample': 0.6335955590893854, 'colsample_bytree': 0.8575508981309805, 'reg_alpha': 9.125088011753222, 'reg_lambda': 0.6748388664033089, 'min_split_gain': 0.029437312686423713}. Best is trial 3 with value: 0.006042431970185669.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:45,545] Trial 5 finished with value: 0.005874945267669365 and parameters: {'n_estimators': 78, 'learning_rate': 0.158039413230604, 'num_leaves': 70, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.5581848988380019, 'colsample_bytree': 0.8244768875182767, 'reg_alpha': 0.890121826197563, 'reg_lambda': 6.889890078662373, 'min_split_gain': 0.015889157908585454}. Best is trial 5 with value: 0.005874945267669365.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:46,560] Trial 6 finished with value: 0.006600315635318532 and parameters: {'n_estimators': 287, 'learning_rate': 0.0161323097786224, 'num_leaves': 90, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.8978373885723241, 'colsample_bytree': 0.9509681436948174, 'reg_alpha': 7.515993829266929, 'reg_lambda': 2.286439683585387, 'min_split_gain': 0.006228527005389806}. Best is trial 5 with value: 0.005874945267669365.
[I 2025-06-15 19:47:46,564] A new study created in memory with name: no-name-21ec81c8-58cc-42e3-a87e-6d705732a9b8
[I 2025-06-15 19:47:46,599] Trial 0 finished with value: 0.0588961069485839 and parameters: {'n_estimators': 287, 'learning_rate': 0.054261493680820914, 'num_leaves': 126, 'max_depth': 5, 'min_child_samples': 2, 'subsample': 0.9234068249906999, 'colsample_bytree': 0.78278706039145, 'reg_alpha': 9.101944491764154, 'reg_lambda': 0.6974484139029336, 'min_split_gain': 0.01517480650329573}. Best is trial 0 with value: 0.0588961069485839.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         537
1           Price_lag_2         432
14      pct_chg_Load_DA         429
3         Load_DA_lag_0         356
15       lag168_Load_DA         242
2           Price_lag_7         234
5       WindOn_DA_lag_0         221
16  roll2d_median_Price         209
11           NGas_lag_2         173
10           Coal_lag_2         151
12            Oil_lag_2         134
13            EUA_lag_2         131
7                  WD_1           9
9                  WD_7           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          35
1           Price_lag_2           3
11           NGas_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 9.30% done
********************* END NS.. 67 ****************************************************
********************* START NS ... 68 ****************************************************
[I 2025-06-15 19:47:47,375] A new study created in memory with name: no-name-9c2a5046-8860-42c8-afaa-1d6050b9d279
[I 2025-06-15 19:47:47,537] Trial 0 finished with value: 0.004091730342538004 and parameters: {'n_estimators': 188, 'learning_rate': 0.23107519314112013, 'num_leaves': 194, 'max_depth': 9, 'min_child_samples': 7, 'subsample': 0.7915222643844135, 'colsample_bytree': 0.8452486278547202, 'reg_alpha': 6.276460737849475, 'reg_lambda': 5.867574359056423, 'min_split_gain': 0.008992569373779518}. Best is trial 0 with value: 0.004091730342538004.
[Trial 0] New best: 0.00409
[I 2025-06-15 19:47:47,941] Trial 1 finished with value: 0.006105193649753621 and parameters: {'n_estimators': 220, 'learning_rate': 0.09489565094245904, 'num_leaves': 80, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.9051714894545819, 'colsample_bytree': 0.5544403742108233, 'reg_alpha': 6.524138595701943, 'reg_lambda': 8.375883567495219, 'min_split_gain': 0.009127441112961465}. Best is trial 0 with value: 0.004091730342538004.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:48,237] Trial 2 finished with value: 0.006944317913858629 and parameters: {'n_estimators': 213, 'learning_rate': 0.2067259116085415, 'num_leaves': 70, 'max_depth': 10, 'min_child_samples': 7, 'subsample': 0.6726540005988833, 'colsample_bytree': 0.5426410920139604, 'reg_alpha': 5.590969231312572, 'reg_lambda': 1.9074411992371687, 'min_split_gain': 0.0063033500423797855}. Best is trial 0 with value: 0.004091730342538004.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:48,653] Trial 3 finished with value: 0.004232704635963449 and parameters: {'n_estimators': 98, 'learning_rate': 0.04193555814749264, 'num_leaves': 169, 'max_depth': 13, 'min_child_samples': 21, 'subsample': 0.5714102156584797, 'colsample_bytree': 0.9743043129555564, 'reg_alpha': 2.1713898877459936, 'reg_lambda': 3.517017966826569, 'min_split_gain': 0.04219932381415918}. Best is trial 0 with value: 0.004091730342538004.
[I 2025-06-15 19:47:48,656] A new study created in memory with name: no-name-c92f55c5-af48-457b-b998-4783540c8f96
[I 2025-06-15 19:47:48,676] Trial 0 finished with value: 0.03498936458217728 and parameters: {'n_estimators': 72, 'learning_rate': 0.08843045468610597, 'num_leaves': 135, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.533838116931499, 'colsample_bytree': 0.6183249846354454, 'reg_alpha': 4.689052189202778, 'reg_lambda': 6.464980230432404, 'min_split_gain': 0.040591967619374614}. Best is trial 0 with value: 0.03498936458217728.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         285
14      pct_chg_Load_DA         218
1           Price_lag_2         184
3         Load_DA_lag_0         132
13            EUA_lag_2         105
2           Price_lag_7         103
16  roll2d_median_Price          95
10           Coal_lag_2          94
5       WindOn_DA_lag_0          92
15       lag168_Load_DA          75
11           NGas_lag_2          72
12            Oil_lag_2          50
9                  WD_7           6
7                  WD_1           4
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          33
1           Price_lag_2           9
5       WindOn_DA_lag_0           2
3         Load_DA_lag_0           1
13            EUA_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 9.44% done
********************* END NS.. 68 ****************************************************
********************* START NS ... 69 ****************************************************
[I 2025-06-15 19:47:49,414] A new study created in memory with name: no-name-a36c41d4-40e4-48eb-a440-ded8c54c2c5f
[I 2025-06-15 19:47:49,783] Trial 0 finished with value: 0.01655046289694047 and parameters: {'n_estimators': 124, 'learning_rate': 0.01043460094434313, 'num_leaves': 75, 'max_depth': 11, 'min_child_samples': 13, 'subsample': 0.515668115446394, 'colsample_bytree': 0.6386957412859231, 'reg_alpha': 4.839241059196856, 'reg_lambda': 5.188859080379316, 'min_split_gain': 0.006417417294722217}. Best is trial 0 with value: 0.01655046289694047.
[Trial 0] New best: 0.01655
[I 2025-06-15 19:47:50,046] Trial 1 finished with value: 0.010916794409468916 and parameters: {'n_estimators': 157, 'learning_rate': 0.011448272406624254, 'num_leaves': 170, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.6013072283486323, 'colsample_bytree': 0.5946962065028123, 'reg_alpha': 9.329214843110506, 'reg_lambda': 5.154730919056448, 'min_split_gain': 0.03633905441260729}. Best is trial 1 with value: 0.010916794409468916.
[Trial 1] New best: 0.01092
[I 2025-06-15 19:47:50,531] Trial 2 finished with value: 0.003905820548100663 and parameters: {'n_estimators': 260, 'learning_rate': 0.043675696800486295, 'num_leaves': 130, 'max_depth': 5, 'min_child_samples': 13, 'subsample': 0.9032431893361924, 'colsample_bytree': 0.7632841876004621, 'reg_alpha': 9.588274946623658, 'reg_lambda': 8.700293564557946, 'min_split_gain': 0.0002479773497667648}. Best is trial 2 with value: 0.003905820548100663.
[Trial 2] New best: 0.00391
[I 2025-06-15 19:47:50,840] Trial 3 finished with value: 0.006970214441725396 and parameters: {'n_estimators': 55, 'learning_rate': 0.03436296774804762, 'num_leaves': 47, 'max_depth': 13, 'min_child_samples': 27, 'subsample': 0.7229844065901316, 'colsample_bytree': 0.5712732687961602, 'reg_alpha': 0.12327270939270885, 'reg_lambda': 0.7260404657554864, 'min_split_gain': 0.024691771786562185}. Best is trial 2 with value: 0.003905820548100663.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:51,303] Trial 4 finished with value: 0.0041308161042923125 and parameters: {'n_estimators': 135, 'learning_rate': 0.033053142717873206, 'num_leaves': 237, 'max_depth': 15, 'min_child_samples': 14, 'subsample': 0.8602023591533601, 'colsample_bytree': 0.7256150244840585, 'reg_alpha': 4.0683036006707605, 'reg_lambda': 7.002722706737412, 'min_split_gain': 0.04028784927350867}. Best is trial 2 with value: 0.003905820548100663.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:51,697] Trial 5 finished with value: 0.003797871987883854 and parameters: {'n_estimators': 138, 'learning_rate': 0.05965068569723287, 'num_leaves': 46, 'max_depth': 13, 'min_child_samples': 21, 'subsample': 0.6689427495904541, 'colsample_bytree': 0.7535689190707711, 'reg_alpha': 8.782596820151024, 'reg_lambda': 7.4824560870236585, 'min_split_gain': 0.010567815362091493}. Best is trial 5 with value: 0.003797871987883854.
[I 2025-06-15 19:47:51,700] A new study created in memory with name: no-name-6202556b-e8ee-4acb-b616-7c20e1234c7d
[I 2025-06-15 19:47:51,738] Trial 0 finished with value: 0.010603148280810668 and parameters: {'n_estimators': 297, 'learning_rate': 0.07881744746911781, 'num_leaves': 113, 'max_depth': 7, 'min_child_samples': 10, 'subsample': 0.8308363441694286, 'colsample_bytree': 0.7932030815656684, 'reg_alpha': 5.0786889253531085, 'reg_lambda': 5.450341260735424, 'min_split_gain': 0.024174029413983524}. Best is trial 0 with value: 0.010603148280810668.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         778
1           Price_lag_2         560
14      pct_chg_Load_DA         420
3         Load_DA_lag_0         319
13            EUA_lag_2         245
2           Price_lag_7         205
5       WindOn_DA_lag_0         203
16  roll2d_median_Price         199
11           NGas_lag_2         187
15       lag168_Load_DA         170
10           Coal_lag_2         160
12            Oil_lag_2         139
8                  WD_6          16
7                  WD_1          13
9                  WD_7          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          44
15       lag168_Load_DA           6
1           Price_lag_2           6
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 9.58% done
********************* END NS.. 69 ****************************************************
********************* START NS ... 70 ****************************************************
[I 2025-06-15 19:47:52,617] A new study created in memory with name: no-name-f954bad0-b62b-4389-8756-76c9b105a358
[I 2025-06-15 19:47:52,864] Trial 0 finished with value: 0.0029758027307318995 and parameters: {'n_estimators': 265, 'learning_rate': 0.15895939096974318, 'num_leaves': 168, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.9963923162296187, 'colsample_bytree': 0.7817144006780542, 'reg_alpha': 1.5973683660967186, 'reg_lambda': 4.584380268009465, 'min_split_gain': 0.03086547304358312}. Best is trial 0 with value: 0.0029758027307318995.
[Trial 0] New best: 0.00298
[I 2025-06-15 19:47:53,411] Trial 1 finished with value: 0.0036182587634926862 and parameters: {'n_estimators': 180, 'learning_rate': 0.021050291208412897, 'num_leaves': 54, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.9923771764443116, 'colsample_bytree': 0.6743704060068479, 'reg_alpha': 0.23748859627927055, 'reg_lambda': 3.3324125925822, 'min_split_gain': 0.02651213722227512}. Best is trial 0 with value: 0.0029758027307318995.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:54,423] Trial 2 finished with value: 0.0028656962168472602 and parameters: {'n_estimators': 153, 'learning_rate': 0.02970240837930357, 'num_leaves': 173, 'max_depth': 15, 'min_child_samples': 7, 'subsample': 0.5103095605226927, 'colsample_bytree': 0.8878710336836947, 'reg_alpha': 2.434992798645521, 'reg_lambda': 3.236946323716011, 'min_split_gain': 0.005921332735460971}. Best is trial 2 with value: 0.0028656962168472602.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:54,895] Trial 3 finished with value: 0.00424964903967268 and parameters: {'n_estimators': 138, 'learning_rate': 0.01613235164197848, 'num_leaves': 63, 'max_depth': 9, 'min_child_samples': 21, 'subsample': 0.8375053832447039, 'colsample_bytree': 0.6793254179353138, 'reg_alpha': 8.03431704538361, 'reg_lambda': 3.5415485683768755, 'min_split_gain': 0.03597565411079228}. Best is trial 2 with value: 0.0028656962168472602.
[I 2025-06-15 19:47:54,898] A new study created in memory with name: no-name-1e83f4b1-52e6-45bb-a168-1b1f27d9e59d
[I 2025-06-15 19:47:54,926] Trial 0 finished with value: 0.03381721639630606 and parameters: {'n_estimators': 146, 'learning_rate': 0.1480446130757282, 'num_leaves': 147, 'max_depth': 13, 'min_child_samples': 3, 'subsample': 0.8178571253224962, 'colsample_bytree': 0.667192427895423, 'reg_alpha': 5.5985541513804815, 'reg_lambda': 0.7021125479063073, 'min_split_gain': 0.02154807623641819}. Best is trial 0 with value: 0.03381721639630606.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2207
1           Price_lag_2        1764
14      pct_chg_Load_DA        1595
3         Load_DA_lag_0        1057
2           Price_lag_7         983
5       WindOn_DA_lag_0         800
16  roll2d_median_Price         788
11           NGas_lag_2         761
10           Coal_lag_2         625
13            EUA_lag_2         575
12            Oil_lag_2         498
15       lag168_Load_DA         464
9                  WD_7          23
8                  WD_6          11
7                  WD_1           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          23
1           Price_lag_2           2
15       lag168_Load_DA           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 9.71% done
********************* END NS.. 70 ****************************************************
********************* START NS ... 71 ****************************************************
[I 2025-06-15 19:47:56,252] A new study created in memory with name: no-name-d3808d56-d821-4862-9dd8-5009643bde2b
[I 2025-06-15 19:47:56,685] Trial 0 finished with value: 0.0029417022471331425 and parameters: {'n_estimators': 231, 'learning_rate': 0.043006985817984573, 'num_leaves': 176, 'max_depth': 6, 'min_child_samples': 28, 'subsample': 0.5035335059345417, 'colsample_bytree': 0.7960911437748512, 'reg_alpha': 7.384184237704908, 'reg_lambda': 6.086073888988034, 'min_split_gain': 0.024290185872609865}. Best is trial 0 with value: 0.0029417022471331425.
[Trial 0] New best: 0.00294
[I 2025-06-15 19:47:57,346] Trial 1 finished with value: 0.002817342476230595 and parameters: {'n_estimators': 150, 'learning_rate': 0.011250653389049157, 'num_leaves': 215, 'max_depth': 15, 'min_child_samples': 5, 'subsample': 0.7628213013470799, 'colsample_bytree': 0.7345703098020806, 'reg_alpha': 3.2338698935826757, 'reg_lambda': 2.020603214232924, 'min_split_gain': 0.013861433980268163}. Best is trial 1 with value: 0.002817342476230595.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:57,587] Trial 2 finished with value: 0.0029273364925117666 and parameters: {'n_estimators': 134, 'learning_rate': 0.1453351944918882, 'num_leaves': 213, 'max_depth': 7, 'min_child_samples': 29, 'subsample': 0.8162666653276882, 'colsample_bytree': 0.6307540210939526, 'reg_alpha': 0.8390980844990348, 'reg_lambda': 4.278637123541868, 'min_split_gain': 0.026865802560213165}. Best is trial 1 with value: 0.002817342476230595.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:47:57,870] Trial 3 finished with value: 0.002425974930619908 and parameters: {'n_estimators': 77, 'learning_rate': 0.025766110391537818, 'num_leaves': 187, 'max_depth': 9, 'min_child_samples': 11, 'subsample': 0.9709265171686414, 'colsample_bytree': 0.9394826120113705, 'reg_alpha': 9.77826441427032, 'reg_lambda': 5.590594692425489, 'min_split_gain': 0.03704829595867385}. Best is trial 3 with value: 0.002425974930619908.
[I 2025-06-15 19:47:57,873] A new study created in memory with name: no-name-d82f847c-1d39-44bd-a4b9-159ea03e1b55
[I 2025-06-15 19:47:57,903] Trial 0 finished with value: 0.07046148165546867 and parameters: {'n_estimators': 189, 'learning_rate': 0.05898248698954079, 'num_leaves': 84, 'max_depth': 13, 'min_child_samples': 26, 'subsample': 0.6715603047437084, 'colsample_bytree': 0.6185149974515833, 'reg_alpha': 7.59381737792144, 'reg_lambda': 3.9879822251646835, 'min_split_gain': 0.0005012134678573033}. Best is trial 0 with value: 0.07046148165546867.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         952
1           Price_lag_2         336
14      pct_chg_Load_DA         153
16  roll2d_median_Price         139
2           Price_lag_7          80
11           NGas_lag_2          70
10           Coal_lag_2          61
13            EUA_lag_2          57
12            Oil_lag_2          50
3         Load_DA_lag_0          38
15       lag168_Load_DA          10
5       WindOn_DA_lag_0           5
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         114
11           NGas_lag_2           4
1           Price_lag_2           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 9.85% done
********************* END NS.. 71 ****************************************************
********************* START NS ... 72 ****************************************************
[I 2025-06-15 19:47:58,690] A new study created in memory with name: no-name-3e37d203-fe31-4e26-8e79-26e8da8402f1
[I 2025-06-15 19:47:59,140] Trial 0 finished with value: 0.003937827349773469 and parameters: {'n_estimators': 176, 'learning_rate': 0.010190254415554757, 'num_leaves': 158, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.745382732631017, 'colsample_bytree': 0.5744842282016502, 'reg_alpha': 7.08152547634932, 'reg_lambda': 8.233779452146614, 'min_split_gain': 0.009929445464996101}. Best is trial 0 with value: 0.003937827349773469.
[Trial 0] New best: 0.00394
[I 2025-06-15 19:47:59,360] Trial 1 finished with value: 0.003319691999320797 and parameters: {'n_estimators': 233, 'learning_rate': 0.13702791971822886, 'num_leaves': 104, 'max_depth': 11, 'min_child_samples': 7, 'subsample': 0.653399864957053, 'colsample_bytree': 0.6471499971148299, 'reg_alpha': 8.730809023322546, 'reg_lambda': 4.738219936649552, 'min_split_gain': 0.027218609898953895}. Best is trial 1 with value: 0.003319691999320797.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:47:59,639] Trial 2 finished with value: 0.004964604736542631 and parameters: {'n_estimators': 72, 'learning_rate': 0.011421813829378194, 'num_leaves': 205, 'max_depth': 11, 'min_child_samples': 1, 'subsample': 0.806230993223735, 'colsample_bytree': 0.5715277796476632, 'reg_alpha': 8.646598710285765, 'reg_lambda': 4.172019986835918, 'min_split_gain': 0.02948060155230834}. Best is trial 1 with value: 0.003319691999320797.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:00,171] Trial 3 finished with value: 0.002923218411023269 and parameters: {'n_estimators': 166, 'learning_rate': 0.014665184777356812, 'num_leaves': 212, 'max_depth': 13, 'min_child_samples': 24, 'subsample': 0.8988267296618634, 'colsample_bytree': 0.8464047319096819, 'reg_alpha': 8.91849103213296, 'reg_lambda': 0.3335027703116, 'min_split_gain': 0.018796714794151948}. Best is trial 3 with value: 0.002923218411023269.
[I 2025-06-15 19:48:00,330] Trial 4 finished with value: 0.0028536470631444193 and parameters: {'n_estimators': 106, 'learning_rate': 0.25616701302874556, 'num_leaves': 114, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.7262790425710285, 'colsample_bytree': 0.8114399154165357, 'reg_alpha': 5.729844308158894, 'reg_lambda': 5.329126212375881, 'min_split_gain': 0.028251918415665673}. Best is trial 4 with value: 0.0028536470631444193.
[Trial 3] New best: 0.00292
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:01,068] Trial 5 finished with value: 0.003728471499904584 and parameters: {'n_estimators': 276, 'learning_rate': 0.03871474216845633, 'num_leaves': 179, 'max_depth': 11, 'min_child_samples': 2, 'subsample': 0.6397987061620076, 'colsample_bytree': 0.6172814185567479, 'reg_alpha': 5.997896798652614, 'reg_lambda': 5.978805059882934, 'min_split_gain': 0.029724435197146723}. Best is trial 4 with value: 0.0028536470631444193.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:01,331] Trial 6 finished with value: 0.003087760231627551 and parameters: {'n_estimators': 179, 'learning_rate': 0.09192434910664617, 'num_leaves': 110, 'max_depth': 15, 'min_child_samples': 17, 'subsample': 0.9135626264412247, 'colsample_bytree': 0.8965781608434357, 'reg_alpha': 9.67607760392177, 'reg_lambda': 4.463217398747065, 'min_split_gain': 0.030986988713781388}. Best is trial 4 with value: 0.0028536470631444193.
[I 2025-06-15 19:48:01,333] A new study created in memory with name: no-name-db9c72a5-c633-4d68-a3f8-ca14ecb6e313
[I 2025-06-15 19:48:01,353] Trial 0 finished with value: 0.08322126791215056 and parameters: {'n_estimators': 131, 'learning_rate': 0.010195411423137955, 'num_leaves': 55, 'max_depth': 10, 'min_child_samples': 26, 'subsample': 0.8234490722592689, 'colsample_bytree': 0.8365873687840745, 'reg_alpha': 7.883629786023578, 'reg_lambda': 5.498681364308956, 'min_split_gain': 0.037786536343277694}. Best is trial 0 with value: 0.08322126791215056.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         181
14      pct_chg_Load_DA         114
1           Price_lag_2          95
3         Load_DA_lag_0          69
2           Price_lag_7          53
16  roll2d_median_Price          51
11           NGas_lag_2          45
13            EUA_lag_2          44
5       WindOn_DA_lag_0          36
15       lag168_Load_DA          32
10           Coal_lag_2          25
12            Oil_lag_2          15
9                  WD_7           4
7                  WD_1           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         140
1           Price_lag_2          16
16  roll2d_median_Price           3
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
2           Price_lag_7           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 9.99% done
********************* END NS.. 72 ****************************************************
********************* START NS ... 73 ****************************************************
[I 2025-06-15 19:48:02,130] A new study created in memory with name: no-name-c50868b8-1815-42b0-b20f-ede3283120a3
[I 2025-06-15 19:48:03,799] Trial 0 finished with value: 0.0020564819192286417 and parameters: {'n_estimators': 238, 'learning_rate': 0.015982616170224513, 'num_leaves': 180, 'max_depth': 14, 'min_child_samples': 24, 'subsample': 0.5607594442856348, 'colsample_bytree': 0.661755831607642, 'reg_alpha': 2.877451889970292, 'reg_lambda': 2.713427533605539, 'min_split_gain': 0.0003900379717797331}. Best is trial 0 with value: 0.0020564819192286417.
[Trial 0] New best: 0.00206
[I 2025-06-15 19:48:04,119] Trial 1 finished with value: 0.0016244454552602969 and parameters: {'n_estimators': 178, 'learning_rate': 0.24450791050304088, 'num_leaves': 41, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.9490975977601166, 'colsample_bytree': 0.74340853960291, 'reg_alpha': 3.17193563228824, 'reg_lambda': 1.6132728882586056, 'min_split_gain': 0.0023756621373808686}. Best is trial 1 with value: 0.0016244454552602969.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:04,438] Trial 2 finished with value: 0.002454708978213328 and parameters: {'n_estimators': 158, 'learning_rate': 0.04789958084388708, 'num_leaves': 254, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.976773053028699, 'colsample_bytree': 0.6545673610586583, 'reg_alpha': 9.919829664968443, 'reg_lambda': 3.030202786224987, 'min_split_gain': 0.04353253830226792}. Best is trial 1 with value: 0.0016244454552602969.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:04,930] Trial 3 finished with value: 0.002311681124077768 and parameters: {'n_estimators': 269, 'learning_rate': 0.06995247875685884, 'num_leaves': 100, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.7201689754985312, 'colsample_bytree': 0.6945766339165215, 'reg_alpha': 9.623430096830432, 'reg_lambda': 7.5255573276553855, 'min_split_gain': 0.01222306007395534}. Best is trial 1 with value: 0.0016244454552602969.
[I 2025-06-15 19:48:04,933] A new study created in memory with name: no-name-d4adfe1f-f0d7-4f35-8792-5a3caaa87f7b
[I 2025-06-15 19:48:04,963] Trial 0 finished with value: 0.07965571408687654 and parameters: {'n_estimators': 93, 'learning_rate': 0.01092991934935021, 'num_leaves': 97, 'max_depth': 15, 'min_child_samples': 12, 'subsample': 0.7648250922299482, 'colsample_bytree': 0.5320331260181759, 'reg_alpha': 7.93397186175734, 'reg_lambda': 3.2922471873130066, 'min_split_gain': 0.006457278059975847}. Best is trial 0 with value: 0.07965571408687654.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         343
1           Price_lag_2         265
14      pct_chg_Load_DA         246
3         Load_DA_lag_0         224
5       WindOn_DA_lag_0         194
15       lag168_Load_DA         163
2           Price_lag_7         150
13            EUA_lag_2         137
12            Oil_lag_2         125
16  roll2d_median_Price         121
11           NGas_lag_2         119
10           Coal_lag_2         109
7                  WD_1           8
9                  WD_7           6
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         112
1           Price_lag_2          31
16  roll2d_median_Price          10
10           Coal_lag_2           5
2           Price_lag_7           3
3         Load_DA_lag_0           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 10.12% done
********************* END NS.. 73 ****************************************************
********************* START NS ... 74 ****************************************************
[I 2025-06-15 19:48:05,896] A new study created in memory with name: no-name-61832dc4-a06b-4728-b303-b02d20dad310
[I 2025-06-15 19:48:06,172] Trial 0 finished with value: 0.004888804646763348 and parameters: {'n_estimators': 135, 'learning_rate': 0.12142377047530183, 'num_leaves': 112, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.7715518903836212, 'colsample_bytree': 0.9504575020011918, 'reg_alpha': 1.955739886874569, 'reg_lambda': 1.6864343267653148, 'min_split_gain': 0.032491756905293184}. Best is trial 0 with value: 0.004888804646763348.
[Trial 0] New best: 0.00489
[I 2025-06-15 19:48:06,796] Trial 1 finished with value: 0.005393131272330306 and parameters: {'n_estimators': 99, 'learning_rate': 0.03620722939208573, 'num_leaves': 250, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.9553132435823675, 'colsample_bytree': 0.6788234512030433, 'reg_alpha': 1.9693510607804354, 'reg_lambda': 5.099660894983779, 'min_split_gain': 0.036109082333626995}. Best is trial 0 with value: 0.004888804646763348.
[I 2025-06-15 19:48:06,960] Trial 2 finished with value: 0.011509014920121102 and parameters: {'n_estimators': 67, 'learning_rate': 0.015098621662504304, 'num_leaves': 207, 'max_depth': 4, 'min_child_samples': 17, 'subsample': 0.6312794753044055, 'colsample_bytree': 0.5603569883596256, 'reg_alpha': 0.40411547314794904, 'reg_lambda': 9.362880365268165, 'min_split_gain': 0.030030511304988412}. Best is trial 0 with value: 0.004888804646763348.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:07,132] Trial 3 finished with value: 0.010333683054795833 and parameters: {'n_estimators': 65, 'learning_rate': 0.016460493422545082, 'num_leaves': 254, 'max_depth': 4, 'min_child_samples': 8, 'subsample': 0.7945510243979489, 'colsample_bytree': 0.7439277882702522, 'reg_alpha': 4.765242210629468, 'reg_lambda': 4.188889251143624, 'min_split_gain': 0.041525842720853176}. Best is trial 0 with value: 0.004888804646763348.
[I 2025-06-15 19:48:07,134] A new study created in memory with name: no-name-f5a8c919-4e99-42e6-a163-224d31a40439
[I 2025-06-15 19:48:07,151] Trial 0 finished with value: 0.022820856403531792 and parameters: {'n_estimators': 65, 'learning_rate': 0.03531285752511278, 'num_leaves': 76, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.714801653436383, 'colsample_bytree': 0.6721862828941223, 'reg_alpha': 8.598759591691433, 'reg_lambda': 7.003805147830841, 'min_split_gain': 0.04123113804741456}. Best is trial 0 with value: 0.022820856403531792.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         562
14      pct_chg_Load_DA         440
3         Load_DA_lag_0         313
1           Price_lag_2         310
2           Price_lag_7         213
11           NGas_lag_2         204
16  roll2d_median_Price         197
15       lag168_Load_DA         168
10           Coal_lag_2         103
13            EUA_lag_2         101
5       WindOn_DA_lag_0          97
12            Oil_lag_2          93
9                  WD_7           6
7                  WD_1           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          19
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 10.26% done
********************* END NS.. 74 ****************************************************
********************* START NS ... 75 ****************************************************
[I 2025-06-15 19:48:08,127] A new study created in memory with name: no-name-1bd1dcd5-c234-4723-a878-1903dd08dfe9
[I 2025-06-15 19:48:08,338] Trial 0 finished with value: 0.015246579309659123 and parameters: {'n_estimators': 69, 'learning_rate': 0.014215230368011658, 'num_leaves': 67, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.9155449126738191, 'colsample_bytree': 0.5649445720669917, 'reg_alpha': 2.89098876829889, 'reg_lambda': 0.3497123437855121, 'min_split_gain': 0.018661277945283484}. Best is trial 0 with value: 0.015246579309659123.
[Trial 0] New best: 0.01525
[I 2025-06-15 19:48:08,661] Trial 1 finished with value: 0.007457470234606179 and parameters: {'n_estimators': 200, 'learning_rate': 0.12210922165276877, 'num_leaves': 176, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.553507796195466, 'colsample_bytree': 0.8307611467777611, 'reg_alpha': 4.217733319931995, 'reg_lambda': 3.2244432727056105, 'min_split_gain': 0.0024500970214543284}. Best is trial 1 with value: 0.007457470234606179.
[Trial 1] New best: 0.00746
[I 2025-06-15 19:48:09,458] Trial 2 finished with value: 0.00812839421989795 and parameters: {'n_estimators': 266, 'learning_rate': 0.01383051004495179, 'num_leaves': 136, 'max_depth': 5, 'min_child_samples': 13, 'subsample': 0.9692735065511231, 'colsample_bytree': 0.6628200589994127, 'reg_alpha': 6.321932108626864, 'reg_lambda': 7.824687156292008, 'min_split_gain': 0.03266952355718923}. Best is trial 1 with value: 0.007457470234606179.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:10,577] Trial 3 finished with value: 0.00792840717119082 and parameters: {'n_estimators': 223, 'learning_rate': 0.011869973884675639, 'num_leaves': 145, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.7889302576082564, 'colsample_bytree': 0.8320506486909284, 'reg_alpha': 6.366608110672401, 'reg_lambda': 0.839904087635196, 'min_split_gain': 0.03782269703785072}. Best is trial 1 with value: 0.007457470234606179.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:11,475] Trial 4 finished with value: 0.007702001845864499 and parameters: {'n_estimators': 260, 'learning_rate': 0.012923074946920455, 'num_leaves': 96, 'max_depth': 16, 'min_child_samples': 20, 'subsample': 0.6009374005284268, 'colsample_bytree': 0.8930592765851597, 'reg_alpha': 9.302215349288122, 'reg_lambda': 4.324221094839526, 'min_split_gain': 0.02719683583908188}. Best is trial 1 with value: 0.007457470234606179.
[I 2025-06-15 19:48:11,478] A new study created in memory with name: no-name-1ac5c08f-afc3-4bd9-856b-37f014172dce
[I 2025-06-15 19:48:11,505] Trial 0 finished with value: 0.025600975848973483 and parameters: {'n_estimators': 124, 'learning_rate': 0.10333818795118803, 'num_leaves': 242, 'max_depth': 6, 'min_child_samples': 19, 'subsample': 0.7243171899823238, 'colsample_bytree': 0.6043470851069641, 'reg_alpha': 5.84025792306953, 'reg_lambda': 3.1662354617058885, 'min_split_gain': 0.020346476182563463}. Best is trial 0 with value: 0.025600975848973483.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         556
1           Price_lag_2         342
14      pct_chg_Load_DA         262
3         Load_DA_lag_0         248
11           NGas_lag_2         197
5       WindOn_DA_lag_0         193
2           Price_lag_7         146
10           Coal_lag_2         123
16  roll2d_median_Price         118
13            EUA_lag_2         117
12            Oil_lag_2         100
15       lag168_Load_DA          87
7                  WD_1          13
9                  WD_7          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          12
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 10.40% done
********************* END NS.. 75 ****************************************************
********************* START NS ... 76 ****************************************************
[I 2025-06-15 19:48:12,645] A new study created in memory with name: no-name-b3779e05-6bdb-45ff-9074-19d5a58d3402
[I 2025-06-15 19:48:13,146] Trial 0 finished with value: 0.0027465989317668822 and parameters: {'n_estimators': 136, 'learning_rate': 0.035783588006718284, 'num_leaves': 41, 'max_depth': 11, 'min_child_samples': 14, 'subsample': 0.7531211764416743, 'colsample_bytree': 0.9589327267826853, 'reg_alpha': 5.335926830843951, 'reg_lambda': 7.4316400528567925, 'min_split_gain': 0.015831574888405944}. Best is trial 0 with value: 0.0027465989317668822.
[Trial 0] New best: 0.00275
[I 2025-06-15 19:48:13,670] Trial 1 finished with value: 0.0023712206372722357 and parameters: {'n_estimators': 109, 'learning_rate': 0.12824916549184717, 'num_leaves': 165, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.808438660516711, 'colsample_bytree': 0.9933183617521189, 'reg_alpha': 3.600199045891106, 'reg_lambda': 2.0842160835880144, 'min_split_gain': 0.003224066449924984}. Best is trial 1 with value: 0.0023712206372722357.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:14,343] Trial 2 finished with value: 0.004202480472200268 and parameters: {'n_estimators': 145, 'learning_rate': 0.011683424397323816, 'num_leaves': 142, 'max_depth': 15, 'min_child_samples': 1, 'subsample': 0.8206850275267448, 'colsample_bytree': 0.9493421383415024, 'reg_alpha': 7.600977182401466, 'reg_lambda': 1.3119216943586753, 'min_split_gain': 0.005339417433810312}. Best is trial 1 with value: 0.0023712206372722357.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:14,675] Trial 3 finished with value: 0.0026322541430922907 and parameters: {'n_estimators': 240, 'learning_rate': 0.26946340538310815, 'num_leaves': 143, 'max_depth': 11, 'min_child_samples': 14, 'subsample': 0.7624275283151642, 'colsample_bytree': 0.7171180766105425, 'reg_alpha': 9.127013191240824, 'reg_lambda': 5.42487476354105, 'min_split_gain': 0.016110877559062067}. Best is trial 1 with value: 0.0023712206372722357.
[I 2025-06-15 19:48:14,678] A new study created in memory with name: no-name-55666456-86ac-485c-9804-26d50bf3f391
[I 2025-06-15 19:48:14,751] Trial 0 finished with value: 0.010847143566656364 and parameters: {'n_estimators': 270, 'learning_rate': 0.012667631164252399, 'num_leaves': 57, 'max_depth': 7, 'min_child_samples': 8, 'subsample': 0.534851434011581, 'colsample_bytree': 0.992010522209648, 'reg_alpha': 2.571971320668489, 'reg_lambda': 4.272783063313979, 'min_split_gain': 0.04924511355250549}. Best is trial 0 with value: 0.010847143566656364.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA         889
0           Price_lag_1         857
3         Load_DA_lag_0         710
1           Price_lag_2         709
5       WindOn_DA_lag_0         672
2           Price_lag_7         549
10           Coal_lag_2         441
16  roll2d_median_Price         431
13            EUA_lag_2         394
11           NGas_lag_2         369
15       lag168_Load_DA         361
12            Oil_lag_2         301
9                  WD_7          20
7                  WD_1          13
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         127
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 10.53% done
********************* END NS.. 76 ****************************************************
********************* START NS ... 77 ****************************************************
[I 2025-06-15 19:48:16,468] A new study created in memory with name: no-name-80e408c7-3b3f-4262-86f7-f4a53b44d195
[I 2025-06-15 19:48:16,783] Trial 0 finished with value: 0.0011220604659727725 and parameters: {'n_estimators': 233, 'learning_rate': 0.2910866662696997, 'num_leaves': 167, 'max_depth': 5, 'min_child_samples': 18, 'subsample': 0.9552395482036702, 'colsample_bytree': 0.8148609977678027, 'reg_alpha': 8.864469939910409, 'reg_lambda': 3.3258188370972217, 'min_split_gain': 0.0008027312138410071}. Best is trial 0 with value: 0.0011220604659727725.
[Trial 0] New best: 0.00112
[I 2025-06-15 19:48:17,511] Trial 1 finished with value: 0.0007097035280778214 and parameters: {'n_estimators': 176, 'learning_rate': 0.08001694057364368, 'num_leaves': 208, 'max_depth': 13, 'min_child_samples': 2, 'subsample': 0.801508783934305, 'colsample_bytree': 0.9867136332623463, 'reg_alpha': 3.4060352466684076, 'reg_lambda': 2.2966603801080723, 'min_split_gain': 0.02359904930664411}. Best is trial 1 with value: 0.0007097035280778214.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:18,312] Trial 2 finished with value: 0.0015229394014833963 and parameters: {'n_estimators': 156, 'learning_rate': 0.01266366349551524, 'num_leaves': 140, 'max_depth': 13, 'min_child_samples': 23, 'subsample': 0.8414903100646791, 'colsample_bytree': 0.553613106625863, 'reg_alpha': 8.563578930515588, 'reg_lambda': 9.306957534267713, 'min_split_gain': 0.01812542012268049}. Best is trial 1 with value: 0.0007097035280778214.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:18,849] Trial 3 finished with value: 0.0008691872135466321 and parameters: {'n_estimators': 82, 'learning_rate': 0.028482078320366106, 'num_leaves': 127, 'max_depth': 11, 'min_child_samples': 13, 'subsample': 0.8078066887953764, 'colsample_bytree': 0.9322291347060075, 'reg_alpha': 5.574792530457269, 'reg_lambda': 6.551485944957696, 'min_split_gain': 0.004206870104131045}. Best is trial 1 with value: 0.0007097035280778214.
[I 2025-06-15 19:48:18,851] A new study created in memory with name: no-name-b7525deb-6c8f-4b9b-b407-418dffdf6732
[I 2025-06-15 19:48:18,873] Trial 0 finished with value: 0.005665084376784329 and parameters: {'n_estimators': 70, 'learning_rate': 0.13706425838310576, 'num_leaves': 132, 'max_depth': 10, 'min_child_samples': 5, 'subsample': 0.5904929307736684, 'colsample_bytree': 0.6745628652355266, 'reg_alpha': 8.862126071902095, 'reg_lambda': 6.784199499667251, 'min_split_gain': 0.0368489740989973}. Best is trial 0 with value: 0.005665084376784329.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         862
14      pct_chg_Load_DA         667
1           Price_lag_2         567
3         Load_DA_lag_0         459
2           Price_lag_7         369
16  roll2d_median_Price         345
11           NGas_lag_2         327
5       WindOn_DA_lag_0         276
10           Coal_lag_2         257
15       lag168_Load_DA         254
13            EUA_lag_2         245
12            Oil_lag_2         163
9                  WD_7           8
7                  WD_1           7
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 10.67% done
********************* END NS.. 77 ****************************************************
********************* START NS ... 78 ****************************************************
[I 2025-06-15 19:48:20,977] A new study created in memory with name: no-name-d5cfa14c-1e45-4647-82be-ee972f5b5a04
[I 2025-06-15 19:48:21,104] Trial 0 finished with value: 0.0010590565234652835 and parameters: {'n_estimators': 84, 'learning_rate': 0.1772387009176518, 'num_leaves': 48, 'max_depth': 5, 'min_child_samples': 17, 'subsample': 0.8031586428135684, 'colsample_bytree': 0.5557799850727132, 'reg_alpha': 8.996967739978906, 'reg_lambda': 3.638020105463439, 'min_split_gain': 0.03619911842005191}. Best is trial 0 with value: 0.0010590565234652835.
[Trial 0] New best: 0.00106
[I 2025-06-15 19:48:22,295] Trial 1 finished with value: 0.0006987124384494137 and parameters: {'n_estimators': 217, 'learning_rate': 0.015276298076096215, 'num_leaves': 152, 'max_depth': 16, 'min_child_samples': 2, 'subsample': 0.5141261380472296, 'colsample_bytree': 0.8586885556641051, 'reg_alpha': 5.5275020064788825, 'reg_lambda': 9.480971600142231, 'min_split_gain': 0.02267164414233859}. Best is trial 1 with value: 0.0006987124384494137.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:23,277] Trial 2 finished with value: 0.0005989580793487447 and parameters: {'n_estimators': 233, 'learning_rate': 0.01763403991336371, 'num_leaves': 62, 'max_depth': 11, 'min_child_samples': 13, 'subsample': 0.8895079542430512, 'colsample_bytree': 0.94710846636895, 'reg_alpha': 3.7512977776745537, 'reg_lambda': 5.772797597166646, 'min_split_gain': 0.026794313623931205}. Best is trial 2 with value: 0.0005989580793487447.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:24,128] Trial 3 finished with value: 0.0007439979878467027 and parameters: {'n_estimators': 175, 'learning_rate': 0.049010999319575435, 'num_leaves': 70, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.675508552182192, 'colsample_bytree': 0.7789032728527299, 'reg_alpha': 3.1016354720336246, 'reg_lambda': 0.07959091963808063, 'min_split_gain': 0.024032512244510147}. Best is trial 2 with value: 0.0005989580793487447.
[I 2025-06-15 19:48:24,131] A new study created in memory with name: no-name-cb8c0b64-63cc-4c69-9bef-fa9edf05ddea
[I 2025-06-15 19:48:24,174] Trial 0 finished with value: 0.0008275924244166083 and parameters: {'n_estimators': 260, 'learning_rate': 0.017227774493684907, 'num_leaves': 130, 'max_depth': 16, 'min_child_samples': 6, 'subsample': 0.9346283154141086, 'colsample_bytree': 0.7794999294777597, 'reg_alpha': 1.0915771372012695, 'reg_lambda': 9.297973255470932, 'min_split_gain': 0.04635852015616706}. Best is trial 0 with value: 0.0008275924244166083.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2864
1           Price_lag_2        1408
14      pct_chg_Load_DA        1339
16  roll2d_median_Price         663
2           Price_lag_7         649
3         Load_DA_lag_0         583
11           NGas_lag_2         567
13            EUA_lag_2         321
10           Coal_lag_2         317
5       WindOn_DA_lag_0         290
12            Oil_lag_2         267
15       lag168_Load_DA         263
9                  WD_7          26
8                  WD_6           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1         173
1           Price_lag_2          15
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 10.81% done
********************* END NS.. 78 ****************************************************
********************* START NS ... 79 ****************************************************
[I 2025-06-15 19:48:26,753] A new study created in memory with name: no-name-da65d70a-1306-4a28-8f2b-d1242c8bae3b
[I 2025-06-15 19:48:27,028] Trial 0 finished with value: 0.0016907456078140409 and parameters: {'n_estimators': 205, 'learning_rate': 0.10332165609180621, 'num_leaves': 210, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.9011236248836358, 'colsample_bytree': 0.7774440818591943, 'reg_alpha': 8.425742630429095, 'reg_lambda': 9.899973372172479, 'min_split_gain': 0.019076675566376253}. Best is trial 0 with value: 0.0016907456078140409.
[Trial 0] New best: 0.00169
[I 2025-06-15 19:48:28,123] Trial 1 finished with value: 0.0015629823111848 and parameters: {'n_estimators': 174, 'learning_rate': 0.02692641340967613, 'num_leaves': 78, 'max_depth': 10, 'min_child_samples': 23, 'subsample': 0.7526178039879481, 'colsample_bytree': 0.9401251858192827, 'reg_alpha': 0.6264277473974067, 'reg_lambda': 8.151684781404331, 'min_split_gain': 0.00815988217299352}. Best is trial 1 with value: 0.0015629823111848.
[I 2025-06-15 19:48:28,326] Trial 2 finished with value: 0.005255519423939932 and parameters: {'n_estimators': 52, 'learning_rate': 0.017088916492587203, 'num_leaves': 68, 'max_depth': 12, 'min_child_samples': 18, 'subsample': 0.8784182145226072, 'colsample_bytree': 0.728996928539376, 'reg_alpha': 5.698221101058519, 'reg_lambda': 1.626006369045575, 'min_split_gain': 0.029691779727529916}. Best is trial 1 with value: 0.0015629823111848.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:28,591] Trial 3 finished with value: 0.0015027060077421623 and parameters: {'n_estimators': 173, 'learning_rate': 0.030100912120045342, 'num_leaves': 79, 'max_depth': 4, 'min_child_samples': 28, 'subsample': 0.9125719080556562, 'colsample_bytree': 0.5878809432242187, 'reg_alpha': 9.555650360383153, 'reg_lambda': 7.066304386071351, 'min_split_gain': 0.00887218431509243}. Best is trial 3 with value: 0.0015027060077421623.
[I 2025-06-15 19:48:28,593] A new study created in memory with name: no-name-b825ad42-9453-435d-8793-652a2593fcfc
[I 2025-06-15 19:48:28,624] Trial 0 finished with value: 0.0034849655070506875 and parameters: {'n_estimators': 216, 'learning_rate': 0.1098510203625745, 'num_leaves': 159, 'max_depth': 5, 'min_child_samples': 18, 'subsample': 0.6814883442236159, 'colsample_bytree': 0.6062142029460947, 'reg_alpha': 5.645379837554351, 'reg_lambda': 1.9522871355728066, 'min_split_gain': 0.0068599610324887985}. Best is trial 0 with value: 0.0034849655070506875.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         785
1           Price_lag_2         425
14      pct_chg_Load_DA         276
16  roll2d_median_Price         175
3         Load_DA_lag_0         158
2           Price_lag_7         108
11           NGas_lag_2          99
15       lag168_Load_DA          61
13            EUA_lag_2          57
12            Oil_lag_2          42
10           Coal_lag_2          37
5       WindOn_DA_lag_0          26
9                  WD_7           8
8                  WD_6           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 10.94% done
********************* END NS.. 79 ****************************************************
********************* START NS ... 80 ****************************************************
[I 2025-06-15 19:48:29,858] A new study created in memory with name: no-name-7dbf69e3-25a4-4c39-a1e4-846ab47d58e6
[I 2025-06-15 19:48:30,137] Trial 0 finished with value: 0.001844223757845506 and parameters: {'n_estimators': 258, 'learning_rate': 0.11564098807904954, 'num_leaves': 156, 'max_depth': 15, 'min_child_samples': 23, 'subsample': 0.6535250625864881, 'colsample_bytree': 0.6078960479497861, 'reg_alpha': 2.8335530161366664, 'reg_lambda': 0.19254786920146483, 'min_split_gain': 0.026574295104809766}. Best is trial 0 with value: 0.001844223757845506.
[Trial 0] New best: 0.00184
[I 2025-06-15 19:48:31,088] Trial 1 finished with value: 0.0014320005344109564 and parameters: {'n_estimators': 219, 'learning_rate': 0.03418151314371644, 'num_leaves': 82, 'max_depth': 6, 'min_child_samples': 5, 'subsample': 0.6526003030580007, 'colsample_bytree': 0.7211526070431717, 'reg_alpha': 1.2424369703365346, 'reg_lambda': 0.41734712503337934, 'min_split_gain': 0.0042762918639751455}. Best is trial 1 with value: 0.0014320005344109564.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:31,727] Trial 2 finished with value: 0.0016794075534085965 and parameters: {'n_estimators': 89, 'learning_rate': 0.04670541212452745, 'num_leaves': 228, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.5495832822756762, 'colsample_bytree': 0.9252544055350249, 'reg_alpha': 3.3046440980473557, 'reg_lambda': 7.737092441204078, 'min_split_gain': 0.042567201571822755}. Best is trial 1 with value: 0.0014320005344109564.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:32,747] Trial 3 finished with value: 0.0014981136575587994 and parameters: {'n_estimators': 174, 'learning_rate': 0.018104356263808092, 'num_leaves': 61, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.697808338494839, 'colsample_bytree': 0.9948570524679914, 'reg_alpha': 2.0289117847238645, 'reg_lambda': 4.941880695728492, 'min_split_gain': 0.023588715470199167}. Best is trial 1 with value: 0.0014320005344109564.
[I 2025-06-15 19:48:32,749] A new study created in memory with name: no-name-dabbce64-42f5-4fc3-a127-026a5899ce96
[I 2025-06-15 19:48:32,838] Trial 0 finished with value: 0.0017617108824215834 and parameters: {'n_estimators': 177, 'learning_rate': 0.010775514911626313, 'num_leaves': 181, 'max_depth': 11, 'min_child_samples': 29, 'subsample': 0.656936786043516, 'colsample_bytree': 0.7085546092625439, 'reg_alpha': 0.2261939273360325, 'reg_lambda': 8.34442043039188, 'min_split_gain': 0.03301702974862673}. Best is trial 0 with value: 0.0017617108824215834.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1944
1           Price_lag_2        1694
14      pct_chg_Load_DA        1211
3         Load_DA_lag_0         866
2           Price_lag_7         687
16  roll2d_median_Price         650
11           NGas_lag_2         619
10           Coal_lag_2         542
15       lag168_Load_DA         539
5       WindOn_DA_lag_0         531
13            EUA_lag_2         489
12            Oil_lag_2         383
9                  WD_7          33
7                  WD_1          30
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         234
15       lag168_Load_DA          24
1           Price_lag_2          20
3         Load_DA_lag_0           5
13            EUA_lag_2           3
5       WindOn_DA_lag_0           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 11.08% done
********************* END NS.. 80 ****************************************************
********************* START NS ... 81 ****************************************************
[I 2025-06-15 19:48:35,092] A new study created in memory with name: no-name-a971798a-8638-4fe1-8e7c-eba87a5e4d31
[I 2025-06-15 19:48:36,782] Trial 0 finished with value: 0.0017816559689320048 and parameters: {'n_estimators': 277, 'learning_rate': 0.019322440880780094, 'num_leaves': 247, 'max_depth': 10, 'min_child_samples': 2, 'subsample': 0.701457981917341, 'colsample_bytree': 0.5985475388782667, 'reg_alpha': 3.92346104365111, 'reg_lambda': 3.5682746516920174, 'min_split_gain': 0.01194332493346345}. Best is trial 0 with value: 0.0017816559689320048.
[Trial 0] New best: 0.00178
[I 2025-06-15 19:48:36,997] Trial 1 finished with value: 0.0017754927784515383 and parameters: {'n_estimators': 228, 'learning_rate': 0.25330130854680866, 'num_leaves': 256, 'max_depth': 8, 'min_child_samples': 23, 'subsample': 0.8068465560540212, 'colsample_bytree': 0.6558619515911945, 'reg_alpha': 9.51297693289715, 'reg_lambda': 5.217650440794604, 'min_split_gain': 0.0469152360036084}. Best is trial 1 with value: 0.0017754927784515383.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:37,329] Trial 2 finished with value: 0.002150851875070465 and parameters: {'n_estimators': 140, 'learning_rate': 0.07638949444028582, 'num_leaves': 82, 'max_depth': 7, 'min_child_samples': 17, 'subsample': 0.9658167657814969, 'colsample_bytree': 0.5444413494806838, 'reg_alpha': 3.466503826088724, 'reg_lambda': 4.77874904603186, 'min_split_gain': 0.038029981391251834}. Best is trial 1 with value: 0.0017754927784515383.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:37,538] Trial 3 finished with value: 0.0012206305625801952 and parameters: {'n_estimators': 67, 'learning_rate': 0.027049437878022643, 'num_leaves': 37, 'max_depth': 7, 'min_child_samples': 30, 'subsample': 0.5519173200432507, 'colsample_bytree': 0.9797868984453499, 'reg_alpha': 8.565280452309905, 'reg_lambda': 0.9699785494932944, 'min_split_gain': 0.012181470341493535}. Best is trial 3 with value: 0.0012206305625801952.
[I 2025-06-15 19:48:37,541] A new study created in memory with name: no-name-e0a415eb-21e6-42f2-9ff5-45b5ad675d09
[I 2025-06-15 19:48:37,568] Trial 0 finished with value: 0.016898362986093807 and parameters: {'n_estimators': 157, 'learning_rate': 0.02743788105268054, 'num_leaves': 233, 'max_depth': 16, 'min_child_samples': 10, 'subsample': 0.8289022665897285, 'colsample_bytree': 0.5856688317758032, 'reg_alpha': 5.025237848783146, 'reg_lambda': 4.105865384902639, 'min_split_gain': 0.005469969299473848}. Best is trial 0 with value: 0.016898362986093807.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1006
1           Price_lag_2         193
14      pct_chg_Load_DA         182
16  roll2d_median_Price          61
2           Price_lag_7          58
11           NGas_lag_2          55
10           Coal_lag_2          38
13            EUA_lag_2          38
3         Load_DA_lag_0          27
12            Oil_lag_2          21
5       WindOn_DA_lag_0           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
15       lag168_Load_DA           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          18
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 11.22% done
********************* END NS.. 81 ****************************************************
********************* START NS ... 82 ****************************************************
[I 2025-06-15 19:48:39,038] A new study created in memory with name: no-name-d2f78c5a-afcc-4e9b-ae5a-1ba96b953d0d
[I 2025-06-15 19:48:39,431] Trial 0 finished with value: 0.00668549079474721 and parameters: {'n_estimators': 163, 'learning_rate': 0.019133464503710697, 'num_leaves': 197, 'max_depth': 7, 'min_child_samples': 24, 'subsample': 0.7901248761439039, 'colsample_bytree': 0.5174909510210193, 'reg_alpha': 4.728163659623389, 'reg_lambda': 1.8187963663784878, 'min_split_gain': 0.007201862084341759}. Best is trial 0 with value: 0.00668549079474721.
[Trial 0] New best: 0.00669
[I 2025-06-15 19:48:39,660] Trial 1 finished with value: 0.005943252555795423 and parameters: {'n_estimators': 131, 'learning_rate': 0.18173298382224984, 'num_leaves': 202, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.6226924611015763, 'colsample_bytree': 0.5096922807484456, 'reg_alpha': 1.037575436759962, 'reg_lambda': 7.460833509617425, 'min_split_gain': 0.029415684664574417}. Best is trial 1 with value: 0.005943252555795423.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:40,913] Trial 2 finished with value: 0.005166614053544434 and parameters: {'n_estimators': 283, 'learning_rate': 0.015404311377335026, 'num_leaves': 155, 'max_depth': 16, 'min_child_samples': 23, 'subsample': 0.9390324688507362, 'colsample_bytree': 0.7598324019434355, 'reg_alpha': 1.7579138539573536, 'reg_lambda': 1.6852060518571432, 'min_split_gain': 0.03619137606390598}. Best is trial 2 with value: 0.005166614053544434.
[Trial 2] New best: 0.00517
[I 2025-06-15 19:48:41,185] Trial 3 finished with value: 0.006006134647841765 and parameters: {'n_estimators': 148, 'learning_rate': 0.17386197299077966, 'num_leaves': 237, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.5632550594707353, 'colsample_bytree': 0.9791797669392905, 'reg_alpha': 6.200489079467905, 'reg_lambda': 1.3596336175090729, 'min_split_gain': 0.005960836471215009}. Best is trial 2 with value: 0.005166614053544434.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:42,023] Trial 4 finished with value: 0.005795379925717307 and parameters: {'n_estimators': 183, 'learning_rate': 0.012476791587287305, 'num_leaves': 204, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.9615834228265612, 'colsample_bytree': 0.8505003554562116, 'reg_alpha': 8.834026608504276, 'reg_lambda': 7.520277747848975, 'min_split_gain': 0.008159843846633796}. Best is trial 2 with value: 0.005166614053544434.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:42,639] Trial 5 finished with value: 0.00614202702360309 and parameters: {'n_estimators': 119, 'learning_rate': 0.010629931202338133, 'num_leaves': 47, 'max_depth': 9, 'min_child_samples': 20, 'subsample': 0.8618908349771839, 'colsample_bytree': 0.9187073444160037, 'reg_alpha': 0.1771834598632993, 'reg_lambda': 2.156247912439256, 'min_split_gain': 0.04264245217523653}. Best is trial 2 with value: 0.005166614053544434.
[I 2025-06-15 19:48:42,640] A new study created in memory with name: no-name-6a96a9fa-33f0-4616-8d8e-cba7de57922f
[I 2025-06-15 19:48:42,667] Trial 0 finished with value: 0.05030713749854895 and parameters: {'n_estimators': 224, 'learning_rate': 0.12061071576001617, 'num_leaves': 72, 'max_depth': 6, 'min_child_samples': 28, 'subsample': 0.5177730461921208, 'colsample_bytree': 0.5935846293978166, 'reg_alpha': 5.169661858313214, 'reg_lambda': 7.713192310436226, 'min_split_gain': 0.0257688100439417}. Best is trial 0 with value: 0.05030713749854895.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3502
1           Price_lag_2        3024
14      pct_chg_Load_DA        2028
3         Load_DA_lag_0        1021
16  roll2d_median_Price         923
2           Price_lag_7         777
11           NGas_lag_2         768
15       lag168_Load_DA         619
5       WindOn_DA_lag_0         525
13            EUA_lag_2         511
10           Coal_lag_2         447
12            Oil_lag_2         430
9                  WD_7          27
7                  WD_1          11
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           7
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 11.35% done
********************* END NS.. 82 ****************************************************
********************* START NS ... 83 ****************************************************
[I 2025-06-15 19:48:44,878] A new study created in memory with name: no-name-cb555c21-95c8-424b-a6a4-7529a7a497ca
[I 2025-06-15 19:48:45,069] Trial 0 finished with value: 0.005707496091466862 and parameters: {'n_estimators': 96, 'learning_rate': 0.0849733039236391, 'num_leaves': 155, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.7089182276930752, 'colsample_bytree': 0.7492845004312703, 'reg_alpha': 4.432900824917257, 'reg_lambda': 7.8817974167298175, 'min_split_gain': 0.040775995281678336}. Best is trial 0 with value: 0.005707496091466862.
[Trial 0] New best: 0.00571
[I 2025-06-15 19:48:45,386] Trial 1 finished with value: 0.005536141175506913 and parameters: {'n_estimators': 239, 'learning_rate': 0.2252970469109648, 'num_leaves': 117, 'max_depth': 16, 'min_child_samples': 4, 'subsample': 0.7388065053418871, 'colsample_bytree': 0.5026671962649575, 'reg_alpha': 7.339915267089282, 'reg_lambda': 5.718863155782044, 'min_split_gain': 0.004398834108569389}. Best is trial 1 with value: 0.005536141175506913.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:46,213] Trial 2 finished with value: 0.005760246495826256 and parameters: {'n_estimators': 122, 'learning_rate': 0.020064169301428538, 'num_leaves': 213, 'max_depth': 13, 'min_child_samples': 19, 'subsample': 0.6657377698707669, 'colsample_bytree': 0.5691111106801281, 'reg_alpha': 0.15305849424662066, 'reg_lambda': 9.088243072922022, 'min_split_gain': 0.010430185878127197}. Best is trial 1 with value: 0.005536141175506913.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:46,688] Trial 3 finished with value: 0.0057884022930505645 and parameters: {'n_estimators': 192, 'learning_rate': 0.05919766915469137, 'num_leaves': 206, 'max_depth': 12, 'min_child_samples': 14, 'subsample': 0.8843389852854291, 'colsample_bytree': 0.970544459397235, 'reg_alpha': 2.5092332983803303, 'reg_lambda': 2.7075868531942806, 'min_split_gain': 0.037534605902765174}. Best is trial 1 with value: 0.005536141175506913.
[I 2025-06-15 19:48:46,690] A new study created in memory with name: no-name-8f32c299-367b-4387-99b8-276b7715a917
[I 2025-06-15 19:48:46,715] Trial 0 finished with value: 0.058756542584408766 and parameters: {'n_estimators': 176, 'learning_rate': 0.24626559669371006, 'num_leaves': 168, 'max_depth': 14, 'min_child_samples': 10, 'subsample': 0.5446309356722108, 'colsample_bytree': 0.97383742052832, 'reg_alpha': 7.918382687818263, 'reg_lambda': 8.005054601965135, 'min_split_gain': 0.00947749208812599}. Best is trial 0 with value: 0.058756542584408766.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         435
1           Price_lag_2         361
3         Load_DA_lag_0         249
16  roll2d_median_Price         235
2           Price_lag_7         192
13            EUA_lag_2         188
14      pct_chg_Load_DA         182
12            Oil_lag_2         168
5       WindOn_DA_lag_0         165
11           NGas_lag_2         146
15       lag168_Load_DA         144
10           Coal_lag_2         125
9                  WD_7          17
8                  WD_6          14
7                  WD_1           9
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           3
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 11.49% done
********************* END NS.. 83 ****************************************************
********************* START NS ... 84 ****************************************************
[I 2025-06-15 19:48:48,603] A new study created in memory with name: no-name-74b1be3f-1d14-4aa4-8758-994c8d7e0afb
[I 2025-06-15 19:48:49,402] Trial 0 finished with value: 0.011748326546844726 and parameters: {'n_estimators': 222, 'learning_rate': 0.03077925911084259, 'num_leaves': 104, 'max_depth': 8, 'min_child_samples': 9, 'subsample': 0.6102964387826455, 'colsample_bytree': 0.5984850017091832, 'reg_alpha': 1.695163170116406, 'reg_lambda': 5.014951689844183, 'min_split_gain': 0.01974841039114841}. Best is trial 0 with value: 0.011748326546844726.
[Trial 0] New best: 0.01175
[I 2025-06-15 19:48:49,658] Trial 1 finished with value: 0.011517365364693487 and parameters: {'n_estimators': 223, 'learning_rate': 0.1463678714090896, 'num_leaves': 241, 'max_depth': 12, 'min_child_samples': 4, 'subsample': 0.5980277082729023, 'colsample_bytree': 0.7662223966310293, 'reg_alpha': 9.46181437511727, 'reg_lambda': 6.0212222231976265, 'min_split_gain': 0.0166883792701606}. Best is trial 1 with value: 0.011517365364693487.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:50,175] Trial 2 finished with value: 0.012961417822257916 and parameters: {'n_estimators': 156, 'learning_rate': 0.016058118015400812, 'num_leaves': 245, 'max_depth': 15, 'min_child_samples': 20, 'subsample': 0.9794591354555393, 'colsample_bytree': 0.7460875324504225, 'reg_alpha': 6.831630669441825, 'reg_lambda': 8.720287308607707, 'min_split_gain': 0.015448963566686952}. Best is trial 1 with value: 0.011517365364693487.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:50,644] Trial 3 finished with value: 0.010899025620791564 and parameters: {'n_estimators': 294, 'learning_rate': 0.09511200034477249, 'num_leaves': 51, 'max_depth': 7, 'min_child_samples': 1, 'subsample': 0.733793500667408, 'colsample_bytree': 0.9971425522415154, 'reg_alpha': 9.076567311167516, 'reg_lambda': 1.3588098414865413, 'min_split_gain': 0.0023076431737805328}. Best is trial 3 with value: 0.010899025620791564.
[I 2025-06-15 19:48:50,646] A new study created in memory with name: no-name-95225b61-5731-4d50-8cd4-80ebe154e55b
[I 2025-06-15 19:48:50,697] Trial 0 finished with value: 0.08288177740843343 and parameters: {'n_estimators': 256, 'learning_rate': 0.10417389339613561, 'num_leaves': 166, 'max_depth': 14, 'min_child_samples': 23, 'subsample': 0.9551631280854777, 'colsample_bytree': 0.5177781681565898, 'reg_alpha': 8.149375910917692, 'reg_lambda': 5.429764663507507, 'min_split_gain': 0.025230497530036797}. Best is trial 0 with value: 0.08288177740843343.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         683
14      pct_chg_Load_DA         455
1           Price_lag_2         437
2           Price_lag_7         310
3         Load_DA_lag_0         284
13            EUA_lag_2         270
11           NGas_lag_2         259
10           Coal_lag_2         254
5       WindOn_DA_lag_0         243
16  roll2d_median_Price         238
12            Oil_lag_2         224
15       lag168_Load_DA         180
7                  WD_1          16
9                  WD_7          10
8                  WD_6          10
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           9
1           Price_lag_2           3
16  roll2d_median_Price           1
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
2           Price_lag_7           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 11.63% done
********************* END NS.. 84 ****************************************************
********************* START NS ... 85 ****************************************************
[I 2025-06-15 19:48:51,969] A new study created in memory with name: no-name-cda3645a-73a9-4ded-8aa5-898d376b2896
[I 2025-06-15 19:48:52,244] Trial 0 finished with value: 0.010202513270048091 and parameters: {'n_estimators': 213, 'learning_rate': 0.1711619083528401, 'num_leaves': 236, 'max_depth': 10, 'min_child_samples': 25, 'subsample': 0.930732809889775, 'colsample_bytree': 0.6541328269750719, 'reg_alpha': 6.3985777839321125, 'reg_lambda': 5.573209814581998, 'min_split_gain': 0.0007695019853469154}. Best is trial 0 with value: 0.010202513270048091.
[Trial 0] New best: 0.01020
[I 2025-06-15 19:48:52,847] Trial 1 finished with value: 0.011304992165185407 and parameters: {'n_estimators': 210, 'learning_rate': 0.0261167340676598, 'num_leaves': 203, 'max_depth': 8, 'min_child_samples': 22, 'subsample': 0.9415338725984013, 'colsample_bytree': 0.7339435661533737, 'reg_alpha': 4.951981028211577, 'reg_lambda': 5.310806294096762, 'min_split_gain': 0.03620723742923842}. Best is trial 0 with value: 0.010202513270048091.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:53,268] Trial 2 finished with value: 0.013536357488224397 and parameters: {'n_estimators': 108, 'learning_rate': 0.012057346380400536, 'num_leaves': 107, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.959401053707418, 'colsample_bytree': 0.7854884250697525, 'reg_alpha': 1.0607465911442737, 'reg_lambda': 8.863260156675048, 'min_split_gain': 0.025750887676989898}. Best is trial 0 with value: 0.010202513270048091.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:53,598] Trial 3 finished with value: 0.013791663372140125 and parameters: {'n_estimators': 133, 'learning_rate': 0.04664079454360437, 'num_leaves': 249, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.9152015243225864, 'colsample_bytree': 0.5324965841262372, 'reg_alpha': 9.25444746316679, 'reg_lambda': 0.08431949265747196, 'min_split_gain': 0.00890305155798543}. Best is trial 0 with value: 0.010202513270048091.
[I 2025-06-15 19:48:53,600] A new study created in memory with name: no-name-0ef3565e-30d8-46c7-859e-9bae3d25123e
[I 2025-06-15 19:48:53,623] Trial 0 finished with value: 0.03728774913716424 and parameters: {'n_estimators': 129, 'learning_rate': 0.10075852642589919, 'num_leaves': 118, 'max_depth': 13, 'min_child_samples': 11, 'subsample': 0.9810298978999545, 'colsample_bytree': 0.5506049821113432, 'reg_alpha': 2.306099014950197, 'reg_lambda': 2.5305022032138105, 'min_split_gain': 0.036450250404985556}. Best is trial 0 with value: 0.03728774913716424.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         535
1           Price_lag_2         428
3         Load_DA_lag_0         327
14      pct_chg_Load_DA         322
13            EUA_lag_2         301
5       WindOn_DA_lag_0         249
15       lag168_Load_DA         238
2           Price_lag_7         209
10           Coal_lag_2         205
11           NGas_lag_2         184
12            Oil_lag_2         180
16  roll2d_median_Price         170
7                  WD_1          11
9                  WD_7           8
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          24
3         Load_DA_lag_0           8
1           Price_lag_2           7
16  roll2d_median_Price           1
2           Price_lag_7           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 11.76% done
********************* END NS.. 85 ****************************************************
********************* START NS ... 86 ****************************************************
[I 2025-06-15 19:48:55,519] A new study created in memory with name: no-name-3bb5f651-0863-47b1-8cbf-368d764b2314
[I 2025-06-15 19:48:55,982] Trial 0 finished with value: 0.0024956318705376002 and parameters: {'n_estimators': 165, 'learning_rate': 0.011831429701930253, 'num_leaves': 116, 'max_depth': 10, 'min_child_samples': 1, 'subsample': 0.6128823942299084, 'colsample_bytree': 0.990779748141283, 'reg_alpha': 9.672191844226917, 'reg_lambda': 1.5950532749574298, 'min_split_gain': 0.014385257426880904}. Best is trial 0 with value: 0.0024956318705376002.
[Trial 0] New best: 0.00250
[I 2025-06-15 19:48:56,547] Trial 1 finished with value: 0.0023686008023196064 and parameters: {'n_estimators': 214, 'learning_rate': 0.03356363807538611, 'num_leaves': 162, 'max_depth': 13, 'min_child_samples': 4, 'subsample': 0.7524898516471383, 'colsample_bytree': 0.9528178694969293, 'reg_alpha': 5.350322819599446, 'reg_lambda': 3.149340125689748, 'min_split_gain': 0.041014966546886404}. Best is trial 1 with value: 0.0023686008023196064.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:48:56,821] Trial 2 finished with value: 0.002939624867281105 and parameters: {'n_estimators': 277, 'learning_rate': 0.15588730708496323, 'num_leaves': 82, 'max_depth': 5, 'min_child_samples': 23, 'subsample': 0.8419784693179702, 'colsample_bytree': 0.8255695434510808, 'reg_alpha': 0.5890109685525535, 'reg_lambda': 7.93987573838989, 'min_split_gain': 0.021404673711615307}. Best is trial 1 with value: 0.0023686008023196064.
[I 2025-06-15 19:48:57,005] Trial 3 finished with value: 0.003709780252282746 and parameters: {'n_estimators': 92, 'learning_rate': 0.02187736344238888, 'num_leaves': 181, 'max_depth': 4, 'min_child_samples': 23, 'subsample': 0.5199400164651128, 'colsample_bytree': 0.6357160386970784, 'reg_alpha': 1.1680577056206154, 'reg_lambda': 7.823697654097278, 'min_split_gain': 0.03565335996260915}. Best is trial 1 with value: 0.0023686008023196064.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:48:57,008] A new study created in memory with name: no-name-37c0aa48-6234-46c6-a3cc-de2812745d3e
[I 2025-06-15 19:48:57,030] Trial 0 finished with value: 0.006303824906443913 and parameters: {'n_estimators': 91, 'learning_rate': 0.19838487773307306, 'num_leaves': 57, 'max_depth': 11, 'min_child_samples': 7, 'subsample': 0.9014516763907192, 'colsample_bytree': 0.5966186563565208, 'reg_alpha': 1.0568263549479928, 'reg_lambda': 5.771504151296336, 'min_split_gain': 0.037581826684125214}. Best is trial 0 with value: 0.006303824906443913.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1594
14      pct_chg_Load_DA         953
1           Price_lag_2         883
3         Load_DA_lag_0         536
2           Price_lag_7         483
11           NGas_lag_2         464
16  roll2d_median_Price         437
13            EUA_lag_2         324
10           Coal_lag_2         319
12            Oil_lag_2         283
15       lag168_Load_DA         231
5       WindOn_DA_lag_0         129
9                  WD_7          12
7                  WD_1          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          18
3         Load_DA_lag_0           7
1           Price_lag_2           3
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 11.90% done
********************* END NS.. 86 ****************************************************
********************* START NS ... 87 ****************************************************
[I 2025-06-15 19:48:58,389] A new study created in memory with name: no-name-e2138863-3267-436c-8eb8-f9aaf1ef9608
[I 2025-06-15 19:48:58,736] Trial 0 finished with value: 0.012012092370180862 and parameters: {'n_estimators': 215, 'learning_rate': 0.09566070638760929, 'num_leaves': 154, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.957643118234603, 'colsample_bytree': 0.9127591679931268, 'reg_alpha': 0.3661800862993303, 'reg_lambda': 7.739515563960434, 'min_split_gain': 0.012745805226595082}. Best is trial 0 with value: 0.012012092370180862.
[Trial 0] New best: 0.01201
[I 2025-06-15 19:48:59,586] Trial 1 finished with value: 0.015992562918805037 and parameters: {'n_estimators': 181, 'learning_rate': 0.01233456021122246, 'num_leaves': 39, 'max_depth': 14, 'min_child_samples': 5, 'subsample': 0.7426092250030065, 'colsample_bytree': 0.7902166166392217, 'reg_alpha': 4.474313866486252, 'reg_lambda': 4.6424698309636545, 'min_split_gain': 0.03448994770565418}. Best is trial 0 with value: 0.012012092370180862.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:49:00,094] Trial 2 finished with value: 0.014809126899998257 and parameters: {'n_estimators': 250, 'learning_rate': 0.06154879597701225, 'num_leaves': 68, 'max_depth': 9, 'min_child_samples': 29, 'subsample': 0.5514442589618962, 'colsample_bytree': 0.633236326079651, 'reg_alpha': 8.974721707275119, 'reg_lambda': 3.40685692182526, 'min_split_gain': 0.04855708430434149}. Best is trial 0 with value: 0.012012092370180862.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:00,539] Trial 3 finished with value: 0.018444984273748282 and parameters: {'n_estimators': 113, 'learning_rate': 0.02657149975409567, 'num_leaves': 232, 'max_depth': 8, 'min_child_samples': 30, 'subsample': 0.6474656024479744, 'colsample_bytree': 0.5505038057471665, 'reg_alpha': 4.583359429397012, 'reg_lambda': 7.36187631818507, 'min_split_gain': 0.03997385437630599}. Best is trial 0 with value: 0.012012092370180862.
[I 2025-06-15 19:49:00,542] A new study created in memory with name: no-name-d5eb8199-e9c3-4484-bd9c-dc35c7096655
[I 2025-06-15 19:49:00,568] Trial 0 finished with value: 0.11121825537184937 and parameters: {'n_estimators': 212, 'learning_rate': 0.26039269601220905, 'num_leaves': 114, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.7491600232057218, 'colsample_bytree': 0.5732628300627336, 'reg_alpha': 9.647063248425336, 'reg_lambda': 1.7077902762118613, 'min_split_gain': 0.006292222179625173}. Best is trial 0 with value: 0.11121825537184937.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1095
14      pct_chg_Load_DA         997
1           Price_lag_2         851
3         Load_DA_lag_0         703
2           Price_lag_7         515
16  roll2d_median_Price         508
5       WindOn_DA_lag_0         496
15       lag168_Load_DA         471
11           NGas_lag_2         447
10           Coal_lag_2         378
13            EUA_lag_2         296
12            Oil_lag_2         281
7                  WD_1          18
9                  WD_7          17
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 12.04% done
********************* END NS.. 87 ****************************************************
********************* START NS ... 88 ****************************************************
[I 2025-06-15 19:49:02,416] A new study created in memory with name: no-name-0375b1ad-d65a-485e-93af-f87a4bd8b8e8
[I 2025-06-15 19:49:02,676] Trial 0 finished with value: 0.017439854092771715 and parameters: {'n_estimators': 149, 'learning_rate': 0.16965945335445165, 'num_leaves': 55, 'max_depth': 13, 'min_child_samples': 18, 'subsample': 0.7513539731308725, 'colsample_bytree': 0.6522501910338824, 'reg_alpha': 1.7511567904238956, 'reg_lambda': 2.716306659265195, 'min_split_gain': 0.010217502660973838}. Best is trial 0 with value: 0.017439854092771715.
[Trial 0] New best: 0.01744
[I 2025-06-15 19:49:03,427] Trial 1 finished with value: 0.01570550131903845 and parameters: {'n_estimators': 281, 'learning_rate': 0.058577894662359284, 'num_leaves': 130, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.6311754026395322, 'colsample_bytree': 0.9538175699516982, 'reg_alpha': 1.5450178351085808, 'reg_lambda': 1.957365308057455, 'min_split_gain': 0.021855119852247862}. Best is trial 1 with value: 0.01570550131903845.
[Trial 1] New best: 0.01571
[I 2025-06-15 19:49:03,858] Trial 2 finished with value: 0.02537670519682113 and parameters: {'n_estimators': 123, 'learning_rate': 0.02366568359973189, 'num_leaves': 112, 'max_depth': 12, 'min_child_samples': 20, 'subsample': 0.6350734874547229, 'colsample_bytree': 0.5891416865013355, 'reg_alpha': 7.942071476550289, 'reg_lambda': 4.240034917214639, 'min_split_gain': 0.040452729055419745}. Best is trial 1 with value: 0.01570550131903845.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:49:05,255] Trial 3 finished with value: 0.01581258703233409 and parameters: {'n_estimators': 178, 'learning_rate': 0.034943845023983656, 'num_leaves': 254, 'max_depth': 12, 'min_child_samples': 5, 'subsample': 0.8318564794363152, 'colsample_bytree': 0.9186945451194058, 'reg_alpha': 0.27853000482777723, 'reg_lambda': 7.878619899120896, 'min_split_gain': 0.04963978617690047}. Best is trial 1 with value: 0.01570550131903845.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:05,668] Trial 4 finished with value: 0.018841561079042685 and parameters: {'n_estimators': 142, 'learning_rate': 0.12544845237512703, 'num_leaves': 123, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.6028738918191632, 'colsample_bytree': 0.9186934354543954, 'reg_alpha': 2.619720647478896, 'reg_lambda': 2.989192822789594, 'min_split_gain': 0.03264595133269606}. Best is trial 1 with value: 0.01570550131903845.
[I 2025-06-15 19:49:05,672] A new study created in memory with name: no-name-7f08e300-3ecf-4a79-adbd-36a46d5304e0
[I 2025-06-15 19:49:05,718] Trial 0 finished with value: 0.1360310369774479 and parameters: {'n_estimators': 293, 'learning_rate': 0.10007564617808302, 'num_leaves': 189, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.5887282844131791, 'colsample_bytree': 0.887276455611999, 'reg_alpha': 2.6563724676247844, 'reg_lambda': 0.5290733810972714, 'min_split_gain': 0.020325870813767757}. Best is trial 0 with value: 0.1360310369774479.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1372
14      pct_chg_Load_DA        1127
1           Price_lag_2         908
3         Load_DA_lag_0         826
2           Price_lag_7         491
11           NGas_lag_2         465
15       lag168_Load_DA         413
16  roll2d_median_Price         412
5       WindOn_DA_lag_0         315
13            EUA_lag_2         303
10           Coal_lag_2         271
12            Oil_lag_2         258
9                  WD_7          15
7                  WD_1          13
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          52
3         Load_DA_lag_0           7
15       lag168_Load_DA           6
10           Coal_lag_2           4
16  roll2d_median_Price           1
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
4        Solar_DA_lag_0           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 12.18% done
********************* END NS.. 88 ****************************************************
********************* START NS ... 89 ****************************************************
[I 2025-06-15 19:49:07,586] A new study created in memory with name: no-name-e6c5d082-fdee-46ca-8379-9f23d938694d
[I 2025-06-15 19:49:08,514] Trial 0 finished with value: 0.010809105339625258 and parameters: {'n_estimators': 208, 'learning_rate': 0.011934212877588616, 'num_leaves': 196, 'max_depth': 8, 'min_child_samples': 12, 'subsample': 0.609481095616909, 'colsample_bytree': 0.878053804716999, 'reg_alpha': 3.40251484492311, 'reg_lambda': 4.696068972932403, 'min_split_gain': 0.0006528209160470756}. Best is trial 0 with value: 0.010809105339625258.
[Trial 0] New best: 0.01081
[I 2025-06-15 19:49:09,518] Trial 1 finished with value: 0.012203840611442243 and parameters: {'n_estimators': 134, 'learning_rate': 0.015330143349549954, 'num_leaves': 113, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.659940914298925, 'colsample_bytree': 0.7563537895028811, 'reg_alpha': 2.3225151755015205, 'reg_lambda': 3.9748419963182746, 'min_split_gain': 0.0033408082865481917}. Best is trial 0 with value: 0.010809105339625258.
[I 2025-06-15 19:49:09,711] Trial 2 finished with value: 0.011954041408480623 and parameters: {'n_estimators': 74, 'learning_rate': 0.053335178939445664, 'num_leaves': 195, 'max_depth': 4, 'min_child_samples': 9, 'subsample': 0.9927183459106373, 'colsample_bytree': 0.8195049789549294, 'reg_alpha': 7.120532506781951, 'reg_lambda': 5.787203621482503, 'min_split_gain': 0.04821281054282659}. Best is trial 0 with value: 0.010809105339625258.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:10,898] Trial 3 finished with value: 0.009183635556791796 and parameters: {'n_estimators': 196, 'learning_rate': 0.016232103969318814, 'num_leaves': 74, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.8432339934271167, 'colsample_bytree': 0.9414108290170591, 'reg_alpha': 1.627220902101728, 'reg_lambda': 5.531080094277744, 'min_split_gain': 0.04511469911448382}. Best is trial 3 with value: 0.009183635556791796.
[Trial 3] New best: 0.00918
[I 2025-06-15 19:49:11,227] Trial 4 finished with value: 0.009919465589399863 and parameters: {'n_estimators': 234, 'learning_rate': 0.05617241014049232, 'num_leaves': 171, 'max_depth': 4, 'min_child_samples': 22, 'subsample': 0.8548253837398967, 'colsample_bytree': 0.9233728658027172, 'reg_alpha': 1.208312931334633, 'reg_lambda': 3.4323214220883145, 'min_split_gain': 0.04424436116264485}. Best is trial 3 with value: 0.009183635556791796.
[I 2025-06-15 19:49:11,377] Trial 5 finished with value: 0.009835965845468649 and parameters: {'n_estimators': 76, 'learning_rate': 0.19172477639938665, 'num_leaves': 32, 'max_depth': 14, 'min_child_samples': 17, 'subsample': 0.8462611775218513, 'colsample_bytree': 0.802754003837791, 'reg_alpha': 4.6912174415744765, 'reg_lambda': 9.816716420732192, 'min_split_gain': 0.042306939258995174}. Best is trial 3 with value: 0.009183635556791796.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:12,156] Trial 6 finished with value: 0.009086699414629239 and parameters: {'n_estimators': 182, 'learning_rate': 0.045342441472116475, 'num_leaves': 76, 'max_depth': 13, 'min_child_samples': 3, 'subsample': 0.7336551267704534, 'colsample_bytree': 0.8199294811375185, 'reg_alpha': 5.596969909366976, 'reg_lambda': 2.737775428115208, 'min_split_gain': 0.003960613602864061}. Best is trial 6 with value: 0.009086699414629239.
[I 2025-06-15 19:49:12,159] A new study created in memory with name: no-name-ed5ce2a9-f272-462e-adc0-19839a667897
[I 2025-06-15 19:49:12,188] Trial 0 finished with value: 0.03735993504075584 and parameters: {'n_estimators': 216, 'learning_rate': 0.2253446806672113, 'num_leaves': 224, 'max_depth': 4, 'min_child_samples': 23, 'subsample': 0.5833860399564081, 'colsample_bytree': 0.635748463757503, 'reg_alpha': 3.0684993089878443, 'reg_lambda': 2.3817494429684762, 'min_split_gain': 0.043861416254722424}. Best is trial 0 with value: 0.03735993504075584.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1654
1           Price_lag_2        1391
14      pct_chg_Load_DA        1209
3         Load_DA_lag_0         904
5       WindOn_DA_lag_0         758
2           Price_lag_7         721
11           NGas_lag_2         683
15       lag168_Load_DA         637
13            EUA_lag_2         634
16  roll2d_median_Price         563
10           Coal_lag_2         514
12            Oil_lag_2         488
9                  WD_7          30
8                  WD_6          27
7                  WD_1          19
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          16
15       lag168_Load_DA           2
10           Coal_lag_2           1
16  roll2d_median_Price           1
3         Load_DA_lag_0           1
2           Price_lag_7           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 12.31% done
********************* END NS.. 89 ****************************************************
********************* START NS ... 90 ****************************************************
[I 2025-06-15 19:49:13,949] A new study created in memory with name: no-name-4594dca9-78ed-4327-93b6-a8f06a782505
[I 2025-06-15 19:49:14,376] Trial 0 finished with value: 0.006038138103992428 and parameters: {'n_estimators': 163, 'learning_rate': 0.01706292014525861, 'num_leaves': 228, 'max_depth': 11, 'min_child_samples': 19, 'subsample': 0.5659594195380105, 'colsample_bytree': 0.6695216569375648, 'reg_alpha': 4.056872519406247, 'reg_lambda': 5.0671411208884045, 'min_split_gain': 0.0022082921223457396}. Best is trial 0 with value: 0.006038138103992428.
[Trial 0] New best: 0.00604
[I 2025-06-15 19:49:15,091] Trial 1 finished with value: 0.0036568510556198794 and parameters: {'n_estimators': 235, 'learning_rate': 0.08569748210185842, 'num_leaves': 222, 'max_depth': 12, 'min_child_samples': 19, 'subsample': 0.948471801508799, 'colsample_bytree': 0.850802530286451, 'reg_alpha': 2.3520820059522505, 'reg_lambda': 7.801832931307702, 'min_split_gain': 0.007689998132175963}. Best is trial 1 with value: 0.0036568510556198794.
[Trial 1] New best: 0.00366
[I 2025-06-15 19:49:15,322] Trial 2 finished with value: 0.00912371669994667 and parameters: {'n_estimators': 55, 'learning_rate': 0.024887989323210683, 'num_leaves': 126, 'max_depth': 9, 'min_child_samples': 4, 'subsample': 0.8635595438676938, 'colsample_bytree': 0.5899812693280104, 'reg_alpha': 8.479175570218622, 'reg_lambda': 2.8453338417320815, 'min_split_gain': 0.016409986444567815}. Best is trial 1 with value: 0.0036568510556198794.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:49:16,277] Trial 3 finished with value: 0.004636610560722357 and parameters: {'n_estimators': 295, 'learning_rate': 0.04130745936640801, 'num_leaves': 113, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.6835604740415275, 'colsample_bytree': 0.7542392415153528, 'reg_alpha': 2.25683639028869, 'reg_lambda': 1.350644878601771, 'min_split_gain': 0.02663744062309143}. Best is trial 1 with value: 0.0036568510556198794.
[I 2025-06-15 19:49:16,487] Trial 4 finished with value: 0.003963498736505005 and parameters: {'n_estimators': 135, 'learning_rate': 0.2051019926980345, 'num_leaves': 155, 'max_depth': 14, 'min_child_samples': 7, 'subsample': 0.6021546640646791, 'colsample_bytree': 0.8511210864735999, 'reg_alpha': 4.12375240526182, 'reg_lambda': 2.7164409016466173, 'min_split_gain': 0.018687113812883554}. Best is trial 1 with value: 0.0036568510556198794.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:16,488] A new study created in memory with name: no-name-576826be-e422-48ef-a73a-82371122bb23
[I 2025-06-15 19:49:16,520] Trial 0 finished with value: 0.012144913220664341 and parameters: {'n_estimators': 153, 'learning_rate': 0.01708381900710092, 'num_leaves': 225, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.539306425699081, 'colsample_bytree': 0.75144362325383, 'reg_alpha': 6.438677045681725, 'reg_lambda': 1.3578281833505013, 'min_split_gain': 0.0365834120859908}. Best is trial 0 with value: 0.012144913220664341.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1022
14      pct_chg_Load_DA        1019
3         Load_DA_lag_0         925
1           Price_lag_2         819
5       WindOn_DA_lag_0         560
2           Price_lag_7         525
15       lag168_Load_DA         486
11           NGas_lag_2         431
16  roll2d_median_Price         388
13            EUA_lag_2         327
12            Oil_lag_2         326
10           Coal_lag_2         302
7                  WD_1          20
9                  WD_7          17
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         137
15       lag168_Load_DA          34
1           Price_lag_2           8
3         Load_DA_lag_0           5
16  roll2d_median_Price           2
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 12.45% done
********************* END NS.. 90 ****************************************************
********************* START NS ... 91 ****************************************************
[I 2025-06-15 19:49:18,258] A new study created in memory with name: no-name-f4392238-1707-4116-bbbf-a5e4a8af9381
[I 2025-06-15 19:49:18,738] Trial 0 finished with value: 0.001974560221185124 and parameters: {'n_estimators': 195, 'learning_rate': 0.04053573999597538, 'num_leaves': 59, 'max_depth': 12, 'min_child_samples': 5, 'subsample': 0.7573981024337636, 'colsample_bytree': 0.6674409472395567, 'reg_alpha': 0.13716096321405913, 'reg_lambda': 0.2677641288639765, 'min_split_gain': 0.04516097832250371}. Best is trial 0 with value: 0.001974560221185124.
[Trial 0] New best: 0.00197
[I 2025-06-15 19:49:19,060] Trial 1 finished with value: 0.0022075741831339276 and parameters: {'n_estimators': 227, 'learning_rate': 0.16475727339692062, 'num_leaves': 156, 'max_depth': 6, 'min_child_samples': 6, 'subsample': 0.6917381336935693, 'colsample_bytree': 0.5468889725041677, 'reg_alpha': 5.105356943598976, 'reg_lambda': 6.682591300367411, 'min_split_gain': 0.0041428493991939154}. Best is trial 0 with value: 0.001974560221185124.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:49:19,308] Trial 2 finished with value: 0.0030100576991540333 and parameters: {'n_estimators': 106, 'learning_rate': 0.032605599807341686, 'num_leaves': 70, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.8055785256095542, 'colsample_bytree': 0.5306329107104125, 'reg_alpha': 4.643675079650582, 'reg_lambda': 4.881613395632644, 'min_split_gain': 0.048185544549258295}. Best is trial 0 with value: 0.001974560221185124.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:19,551] Trial 3 finished with value: 0.001877986768108043 and parameters: {'n_estimators': 283, 'learning_rate': 0.2272738790356042, 'num_leaves': 196, 'max_depth': 15, 'min_child_samples': 15, 'subsample': 0.6347145109208756, 'colsample_bytree': 0.9270904198386783, 'reg_alpha': 4.800872932055516, 'reg_lambda': 9.035747645426158, 'min_split_gain': 0.019477494968589838}. Best is trial 3 with value: 0.001877986768108043.
[I 2025-06-15 19:49:19,553] A new study created in memory with name: no-name-9b9fabc1-7098-451c-8833-bce7471fec0b
[I 2025-06-15 19:49:19,589] Trial 0 finished with value: 0.01305971473413417 and parameters: {'n_estimators': 268, 'learning_rate': 0.03866392925301157, 'num_leaves': 197, 'max_depth': 4, 'min_child_samples': 6, 'subsample': 0.9933478928329111, 'colsample_bytree': 0.9300632863330146, 'reg_alpha': 6.290680748885926, 'reg_lambda': 3.682685095103931, 'min_split_gain': 0.03218472968860379}. Best is trial 0 with value: 0.01305971473413417.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         249
14      pct_chg_Load_DA         201
1           Price_lag_2         141
3         Load_DA_lag_0         128
16  roll2d_median_Price          84
2           Price_lag_7          82
5       WindOn_DA_lag_0          78
11           NGas_lag_2          75
13            EUA_lag_2          74
12            Oil_lag_2          60
15       lag168_Load_DA          60
10           Coal_lag_2          49
7                  WD_1           4
9                  WD_7           2
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          45
15       lag168_Load_DA          13
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 12.59% done
********************* END NS.. 91 ****************************************************
********************* START NS ... 92 ****************************************************
[I 2025-06-15 19:49:21,216] A new study created in memory with name: no-name-aa5a2b25-ba4d-4db4-9e34-100776679ce9
[I 2025-06-15 19:49:21,590] Trial 0 finished with value: 0.001188682636788036 and parameters: {'n_estimators': 245, 'learning_rate': 0.2732984500884701, 'num_leaves': 162, 'max_depth': 14, 'min_child_samples': 11, 'subsample': 0.9586464174044795, 'colsample_bytree': 0.7245882363109722, 'reg_alpha': 7.299054428390647, 'reg_lambda': 4.931205878281402, 'min_split_gain': 0.03526960701198031}. Best is trial 0 with value: 0.001188682636788036.
[Trial 0] New best: 0.00119
[I 2025-06-15 19:49:23,665] Trial 1 finished with value: 0.0008464190958242245 and parameters: {'n_estimators': 180, 'learning_rate': 0.019160155718740073, 'num_leaves': 192, 'max_depth': 10, 'min_child_samples': 23, 'subsample': 0.7365104982816191, 'colsample_bytree': 0.8330249253535298, 'reg_alpha': 1.1486651808057746, 'reg_lambda': 8.9554662616663, 'min_split_gain': 0.002394966460033232}. Best is trial 1 with value: 0.0008464190958242245.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:49:24,595] Trial 2 finished with value: 0.0008636541261636656 and parameters: {'n_estimators': 101, 'learning_rate': 0.025864012172828327, 'num_leaves': 43, 'max_depth': 16, 'min_child_samples': 24, 'subsample': 0.6509536353189024, 'colsample_bytree': 0.8736224395165251, 'reg_alpha': 7.523584563626229, 'reg_lambda': 5.157764637848494, 'min_split_gain': 0.02885443392465827}. Best is trial 1 with value: 0.0008464190958242245.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:25,091] Trial 3 finished with value: 0.001528158348881489 and parameters: {'n_estimators': 232, 'learning_rate': 0.16108563577642082, 'num_leaves': 158, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.5170801503159872, 'colsample_bytree': 0.5526394450494062, 'reg_alpha': 8.025858718959961, 'reg_lambda': 5.000926148338646, 'min_split_gain': 0.025569257653098327}. Best is trial 1 with value: 0.0008464190958242245.
[I 2025-06-15 19:49:25,095] A new study created in memory with name: no-name-02d3576d-a68a-43e8-b4d0-0f3f799ddfba
[I 2025-06-15 19:49:25,177] Trial 0 finished with value: 0.011667632472052753 and parameters: {'n_estimators': 294, 'learning_rate': 0.01329225730080289, 'num_leaves': 128, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.7259251625466854, 'colsample_bytree': 0.7957990417039975, 'reg_alpha': 5.194198957389839, 'reg_lambda': 0.6165514864743582, 'min_split_gain': 0.009738378119638286}. Best is trial 0 with value: 0.011667632472052753.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3170
1           Price_lag_2        2669
14      pct_chg_Load_DA        1790
2           Price_lag_7         830
3         Load_DA_lag_0         800
16  roll2d_median_Price         778
11           NGas_lag_2         652
13            EUA_lag_2         564
5       WindOn_DA_lag_0         470
10           Coal_lag_2         446
12            Oil_lag_2         374
15       lag168_Load_DA         371
9                  WD_7          16
7                  WD_1           7
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         208
15       lag168_Load_DA          61
1           Price_lag_2          17
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 12.72% done
********************* END NS.. 92 ****************************************************
********************* START NS ... 93 ****************************************************
[I 2025-06-15 19:49:28,471] A new study created in memory with name: no-name-a8ff467e-8fc2-44cd-8497-0b8d3a6f2ecf
[I 2025-06-15 19:49:29,201] Trial 0 finished with value: 0.01419647696184074 and parameters: {'n_estimators': 273, 'learning_rate': 0.01429242130868277, 'num_leaves': 94, 'max_depth': 14, 'min_child_samples': 21, 'subsample': 0.6263243184068376, 'colsample_bytree': 0.6941686057626711, 'reg_alpha': 9.342519678604909, 'reg_lambda': 6.283548317082595, 'min_split_gain': 0.0263420431908529}. Best is trial 0 with value: 0.01419647696184074.
[Trial 0] New best: 0.01420
[I 2025-06-15 19:49:29,843] Trial 1 finished with value: 0.01147227968715004 and parameters: {'n_estimators': 170, 'learning_rate': 0.04213647415715048, 'num_leaves': 173, 'max_depth': 10, 'min_child_samples': 8, 'subsample': 0.5772705759443333, 'colsample_bytree': 0.7112400329813884, 'reg_alpha': 6.345098851656122, 'reg_lambda': 7.874344264818865, 'min_split_gain': 0.012644501561191596}. Best is trial 1 with value: 0.01147227968715004.
[Trial 1] New best: 0.01147
[I 2025-06-15 19:49:30,188] Trial 2 finished with value: 0.012417352896488409 and parameters: {'n_estimators': 130, 'learning_rate': 0.19285082987985686, 'num_leaves': 135, 'max_depth': 15, 'min_child_samples': 11, 'subsample': 0.8406187274379703, 'colsample_bytree': 0.6008554545279737, 'reg_alpha': 4.647482199825788, 'reg_lambda': 9.994486068950332, 'min_split_gain': 0.023417632085807036}. Best is trial 1 with value: 0.01147227968715004.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:49:30,486] Trial 3 finished with value: 0.014135602859013158 and parameters: {'n_estimators': 95, 'learning_rate': 0.03263732725200489, 'num_leaves': 173, 'max_depth': 10, 'min_child_samples': 26, 'subsample': 0.8400107689056888, 'colsample_bytree': 0.66340773752239, 'reg_alpha': 9.689697617158588, 'reg_lambda': 8.674328153851933, 'min_split_gain': 0.03887064110662262}. Best is trial 1 with value: 0.01147227968715004.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:30,910] Trial 4 finished with value: 0.01015735992914008 and parameters: {'n_estimators': 279, 'learning_rate': 0.23617847472738981, 'num_leaves': 66, 'max_depth': 13, 'min_child_samples': 27, 'subsample': 0.9040472849919731, 'colsample_bytree': 0.8023858145423036, 'reg_alpha': 0.8916339970614962, 'reg_lambda': 8.897683165891522, 'min_split_gain': 0.0043487749131750075}. Best is trial 4 with value: 0.01015735992914008.
[Trial 4] New best: 0.01016
[I 2025-06-15 19:49:31,453] Trial 5 finished with value: 0.014678933743067502 and parameters: {'n_estimators': 174, 'learning_rate': 0.012805862693499434, 'num_leaves': 202, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.7210259599949653, 'colsample_bytree': 0.9322090047862601, 'reg_alpha': 9.374821343300637, 'reg_lambda': 8.665314490679291, 'min_split_gain': 0.01402055398942767}. Best is trial 4 with value: 0.01015735992914008.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:49:32,092] Trial 6 finished with value: 0.010727830396226372 and parameters: {'n_estimators': 295, 'learning_rate': 0.06543423483160554, 'num_leaves': 121, 'max_depth': 13, 'min_child_samples': 22, 'subsample': 0.738350699558632, 'colsample_bytree': 0.6630719077158562, 'reg_alpha': 2.19876370897596, 'reg_lambda': 8.853016738682655, 'min_split_gain': 0.020220002073759593}. Best is trial 4 with value: 0.01015735992914008.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:32,791] Trial 7 finished with value: 0.01235948433905309 and parameters: {'n_estimators': 239, 'learning_rate': 0.035739366828529816, 'num_leaves': 71, 'max_depth': 14, 'min_child_samples': 13, 'subsample': 0.750782443561491, 'colsample_bytree': 0.6286466631562375, 'reg_alpha': 5.803903422626748, 'reg_lambda': 4.428671995983184, 'min_split_gain': 0.0358274927368262}. Best is trial 4 with value: 0.01015735992914008.
[I 2025-06-15 19:49:32,794] A new study created in memory with name: no-name-e3512ef1-c2d7-43be-94ed-1536d293e081
[I 2025-06-15 19:49:32,837] Trial 0 finished with value: 0.04626108897593028 and parameters: {'n_estimators': 191, 'learning_rate': 0.06655285149423752, 'num_leaves': 152, 'max_depth': 12, 'min_child_samples': 5, 'subsample': 0.5595881638114168, 'colsample_bytree': 0.5844705198848363, 'reg_alpha': 9.951138757430883, 'reg_lambda': 3.2294152639452456, 'min_split_gain': 0.005092643501889033}. Best is trial 0 with value: 0.04626108897593028.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         570
14      pct_chg_Load_DA         459
5       WindOn_DA_lag_0         412
1           Price_lag_2         387
3         Load_DA_lag_0         380
15       lag168_Load_DA         291
2           Price_lag_7         257
16  roll2d_median_Price         217
11           NGas_lag_2         199
13            EUA_lag_2         189
10           Coal_lag_2         178
12            Oil_lag_2         159
9                  WD_7          11
7                  WD_1           8
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          23
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 12.86% done
********************* END NS.. 93 ****************************************************
********************* START NS ... 94 ****************************************************
[I 2025-06-15 19:49:34,501] A new study created in memory with name: no-name-6dc49e1b-f0f3-42f5-9c54-c4864b73a6b0
[I 2025-06-15 19:49:34,917] Trial 0 finished with value: 0.018045291574273845 and parameters: {'n_estimators': 127, 'learning_rate': 0.08067132568000758, 'num_leaves': 74, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.8974367887212154, 'colsample_bytree': 0.9145193746358603, 'reg_alpha': 1.9121697084134581, 'reg_lambda': 6.234923053982716, 'min_split_gain': 0.022344587213720624}. Best is trial 0 with value: 0.018045291574273845.
[Trial 0] New best: 0.01805
[I 2025-06-15 19:49:35,247] Trial 1 finished with value: 0.018709894473660973 and parameters: {'n_estimators': 279, 'learning_rate': 0.28463019083404967, 'num_leaves': 250, 'max_depth': 7, 'min_child_samples': 25, 'subsample': 0.9043186050489106, 'colsample_bytree': 0.6752591265384864, 'reg_alpha': 1.6221165591508047, 'reg_lambda': 5.472087153046099, 'min_split_gain': 0.0031586746333965576}. Best is trial 0 with value: 0.018045291574273845.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:49:35,585] Trial 2 finished with value: 0.021322931854373617 and parameters: {'n_estimators': 84, 'learning_rate': 0.08644523215888904, 'num_leaves': 47, 'max_depth': 12, 'min_child_samples': 29, 'subsample': 0.768992029512207, 'colsample_bytree': 0.6236180014676935, 'reg_alpha': 6.585879229801984, 'reg_lambda': 4.02017412928692, 'min_split_gain': 0.018260820114475353}. Best is trial 0 with value: 0.018045291574273845.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:35,932] Trial 3 finished with value: 0.020868614183096056 and parameters: {'n_estimators': 239, 'learning_rate': 0.13102648448695908, 'num_leaves': 166, 'max_depth': 12, 'min_child_samples': 14, 'subsample': 0.7060427451320731, 'colsample_bytree': 0.9952805052968912, 'reg_alpha': 8.71112046969711, 'reg_lambda': 2.928389937619018, 'min_split_gain': 0.023220398525921084}. Best is trial 0 with value: 0.018045291574273845.
[I 2025-06-15 19:49:35,935] A new study created in memory with name: no-name-4b34fd0b-ba1a-401e-82d9-88be68ec60c7
[I 2025-06-15 19:49:35,968] Trial 0 finished with value: 0.05469662472501575 and parameters: {'n_estimators': 223, 'learning_rate': 0.11098876349073979, 'num_leaves': 80, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.6867925249715807, 'colsample_bytree': 0.9701288065425118, 'reg_alpha': 2.312189275655582, 'reg_lambda': 3.481999637678286, 'min_split_gain': 0.04744081007499137}. Best is trial 0 with value: 0.05469662472501575.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         924
14      pct_chg_Load_DA         798
1           Price_lag_2         645
3         Load_DA_lag_0         502
2           Price_lag_7         360
16  roll2d_median_Price         349
11           NGas_lag_2         338
15       lag168_Load_DA         278
10           Coal_lag_2         267
5       WindOn_DA_lag_0         265
13            EUA_lag_2         244
12            Oil_lag_2         204
7                  WD_1          17
9                  WD_7           8
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          34
15       lag168_Load_DA           7
3         Load_DA_lag_0           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 13.00% done
********************* END NS.. 94 ****************************************************
********************* START NS ... 95 ****************************************************
[I 2025-06-15 19:49:37,648] A new study created in memory with name: no-name-45fbd606-d141-4259-a7c2-95bc684afbef
[I 2025-06-15 19:49:37,827] Trial 0 finished with value: 0.018336385449514474 and parameters: {'n_estimators': 70, 'learning_rate': 0.03632124526348775, 'num_leaves': 65, 'max_depth': 10, 'min_child_samples': 4, 'subsample': 0.8880182706437583, 'colsample_bytree': 0.6018128429963174, 'reg_alpha': 8.072830411083661, 'reg_lambda': 7.689734595508664, 'min_split_gain': 0.03724677100379786}. Best is trial 0 with value: 0.018336385449514474.
[Trial 0] New best: 0.01834
[I 2025-06-15 19:49:38,206] Trial 1 finished with value: 0.019387567117664893 and parameters: {'n_estimators': 127, 'learning_rate': 0.014766470358405627, 'num_leaves': 126, 'max_depth': 12, 'min_child_samples': 3, 'subsample': 0.5291250673609076, 'colsample_bytree': 0.8578327081010972, 'reg_alpha': 5.3571088545173415, 'reg_lambda': 4.635929613793905, 'min_split_gain': 0.034861631690997784}. Best is trial 0 with value: 0.018336385449514474.
[I 2025-06-15 19:49:38,348] Trial 2 finished with value: 0.01585384687881042 and parameters: {'n_estimators': 88, 'learning_rate': 0.22753956597399466, 'num_leaves': 65, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.9804398566079965, 'colsample_bytree': 0.6559722696727411, 'reg_alpha': 7.641179439606805, 'reg_lambda': 8.983257864173428, 'min_split_gain': 0.01844601628689672}. Best is trial 2 with value: 0.01585384687881042.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] New best: 0.01585
[I 2025-06-15 19:49:38,847] Trial 3 finished with value: 0.016057362075984755 and parameters: {'n_estimators': 216, 'learning_rate': 0.05660111682157093, 'num_leaves': 154, 'max_depth': 15, 'min_child_samples': 6, 'subsample': 0.854053415838844, 'colsample_bytree': 0.8711270778728282, 'reg_alpha': 7.585211677422884, 'reg_lambda': 0.4068730319255043, 'min_split_gain': 0.02218401197291526}. Best is trial 2 with value: 0.01585384687881042.
[I 2025-06-15 19:49:39,023] Trial 4 finished with value: 0.01570769240199001 and parameters: {'n_estimators': 70, 'learning_rate': 0.2896500482162969, 'num_leaves': 254, 'max_depth': 10, 'min_child_samples': 16, 'subsample': 0.7192054386811828, 'colsample_bytree': 0.7136932511571112, 'reg_alpha': 5.1507136236017175, 'reg_lambda': 2.7419372558152877, 'min_split_gain': 0.012468148753846608}. Best is trial 4 with value: 0.01570769240199001.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:40,263] Trial 5 finished with value: 0.014951414827848652 and parameters: {'n_estimators': 272, 'learning_rate': 0.025638124035854398, 'num_leaves': 201, 'max_depth': 15, 'min_child_samples': 7, 'subsample': 0.7429607198269005, 'colsample_bytree': 0.551071570218896, 'reg_alpha': 5.660707046004677, 'reg_lambda': 3.454972181729593, 'min_split_gain': 0.0030407325602262504}. Best is trial 5 with value: 0.014951414827848652.
[I 2025-06-15 19:49:40,266] A new study created in memory with name: no-name-252ad47d-43e0-4ed1-9050-f597ce97d81a
[I 2025-06-15 19:49:40,307] Trial 0 finished with value: 0.02995040169845753 and parameters: {'n_estimators': 251, 'learning_rate': 0.03398898484778191, 'num_leaves': 142, 'max_depth': 8, 'min_child_samples': 10, 'subsample': 0.8849673295291954, 'colsample_bytree': 0.7790014295330456, 'reg_alpha': 0.5529355967139371, 'reg_lambda': 1.0594486770723777, 'min_split_gain': 0.032139117891578764}. Best is trial 0 with value: 0.02995040169845753.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2226
1           Price_lag_2        2026
14      pct_chg_Load_DA        1573
2           Price_lag_7        1230
3         Load_DA_lag_0        1208
16  roll2d_median_Price        1138
13            EUA_lag_2        1025
5       WindOn_DA_lag_0         975
15       lag168_Load_DA         920
10           Coal_lag_2         908
11           NGas_lag_2         832
12            Oil_lag_2         674
9                  WD_7          77
8                  WD_6          62
7                  WD_1          44
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         141
15       lag168_Load_DA          49
14      pct_chg_Load_DA          36
1           Price_lag_2          22
2           Price_lag_7           6
10           Coal_lag_2           3
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
16  roll2d_median_Price           0
-> 13.13% done
********************* END NS.. 95 ****************************************************
********************* START NS ... 96 ****************************************************
[I 2025-06-15 19:49:42,328] A new study created in memory with name: no-name-450a13d8-cd68-46cb-a651-84da8c037cec
[I 2025-06-15 19:49:42,762] Trial 0 finished with value: 0.00655584219467084 and parameters: {'n_estimators': 156, 'learning_rate': 0.021174363104097753, 'num_leaves': 201, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.9620269158235206, 'colsample_bytree': 0.8170924034289967, 'reg_alpha': 1.8598400764765977, 'reg_lambda': 4.465167588887372, 'min_split_gain': 0.046592831562330725}. Best is trial 0 with value: 0.00655584219467084.
[I 2025-06-15 19:49:42,882] Trial 1 finished with value: 0.008980941329498572 and parameters: {'n_estimators': 51, 'learning_rate': 0.145642398299999, 'num_leaves': 242, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.5961129166610769, 'colsample_bytree': 0.5060611832998405, 'reg_alpha': 0.15115739880056878, 'reg_lambda': 5.796315366499383, 'min_split_gain': 0.013842711096126743}. Best is trial 0 with value: 0.00655584219467084.
[Trial 0] New best: 0.00656
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:49:43,082] Trial 2 finished with value: 0.0060385497012991835 and parameters: {'n_estimators': 97, 'learning_rate': 0.14466432176828337, 'num_leaves': 109, 'max_depth': 11, 'min_child_samples': 15, 'subsample': 0.5092054691468147, 'colsample_bytree': 0.6290995202413532, 'reg_alpha': 3.5927946065438077, 'reg_lambda': 3.169344035206044, 'min_split_gain': 0.04715918255619414}. Best is trial 2 with value: 0.0060385497012991835.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:43,410] Trial 3 finished with value: 0.0064259423790907775 and parameters: {'n_estimators': 73, 'learning_rate': 0.13731998564617817, 'num_leaves': 109, 'max_depth': 7, 'min_child_samples': 3, 'subsample': 0.5239277327631686, 'colsample_bytree': 0.8074469086789482, 'reg_alpha': 0.9717969268673221, 'reg_lambda': 3.3637468638766963, 'min_split_gain': 0.0031906140427836963}. Best is trial 2 with value: 0.0060385497012991835.
[I 2025-06-15 19:49:43,412] A new study created in memory with name: no-name-82777b6e-b884-455f-98a1-4e6c15b06f34
[I 2025-06-15 19:49:43,435] Trial 0 finished with value: 0.009890470926480606 and parameters: {'n_estimators': 82, 'learning_rate': 0.05595289666882052, 'num_leaves': 98, 'max_depth': 7, 'min_child_samples': 15, 'subsample': 0.9895403261681857, 'colsample_bytree': 0.7419737247710543, 'reg_alpha': 0.8778509951681157, 'reg_lambda': 1.3639734463800524, 'min_split_gain': 0.010436085045399264}. Best is trial 0 with value: 0.009890470926480606.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         336
1           Price_lag_2         241
3         Load_DA_lag_0         187
14      pct_chg_Load_DA         170
2           Price_lag_7         129
11           NGas_lag_2         119
15       lag168_Load_DA          94
5       WindOn_DA_lag_0          80
10           Coal_lag_2          78
16  roll2d_median_Price          75
13            EUA_lag_2          71
12            Oil_lag_2          55
9                  WD_7           6
7                  WD_1           5
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          87
15       lag168_Load_DA          27
14      pct_chg_Load_DA          25
1           Price_lag_2          18
2           Price_lag_7          13
10           Coal_lag_2           9
3         Load_DA_lag_0           4
13            EUA_lag_2           1
12            Oil_lag_2           1
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 13.27% done
********************* END NS.. 96 ****************************************************
********************* START NS ... 97 ****************************************************
[I 2025-06-15 19:49:44,437] A new study created in memory with name: no-name-19cc3b14-2c07-49df-b357-6d61aff4f589
[I 2025-06-15 19:49:45,002] Trial 0 finished with value: 0.000989986599512821 and parameters: {'n_estimators': 207, 'learning_rate': 0.019004645352091006, 'num_leaves': 45, 'max_depth': 8, 'min_child_samples': 20, 'subsample': 0.8867181346840888, 'colsample_bytree': 0.8010422267953294, 'reg_alpha': 2.9922196877683094, 'reg_lambda': 0.17641732384692022, 'min_split_gain': 0.0463707814030557}. Best is trial 0 with value: 0.000989986599512821.
[I 2025-06-15 19:49:45,186] Trial 1 finished with value: 0.0010437827463948847 and parameters: {'n_estimators': 135, 'learning_rate': 0.12345576030868423, 'num_leaves': 175, 'max_depth': 5, 'min_child_samples': 22, 'subsample': 0.9170183089881809, 'colsample_bytree': 0.721194720378493, 'reg_alpha': 4.398655108402671, 'reg_lambda': 0.24317233992192433, 'min_split_gain': 0.04106768230591737}. Best is trial 0 with value: 0.000989986599512821.
[Trial 0] New best: 0.00099
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:49:45,414] Trial 2 finished with value: 0.001032327122369699 and parameters: {'n_estimators': 211, 'learning_rate': 0.29339874329266724, 'num_leaves': 64, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.6860189501894126, 'colsample_bytree': 0.6556570058182263, 'reg_alpha': 2.726150647146067, 'reg_lambda': 1.9418514142354693, 'min_split_gain': 0.01786644290102889}. Best is trial 0 with value: 0.000989986599512821.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:45,952] Trial 3 finished with value: 0.0010150674227586479 and parameters: {'n_estimators': 131, 'learning_rate': 0.07493334734079571, 'num_leaves': 92, 'max_depth': 9, 'min_child_samples': 15, 'subsample': 0.6889030388806027, 'colsample_bytree': 0.6993518997741175, 'reg_alpha': 0.32678525081409115, 'reg_lambda': 5.572699584222127, 'min_split_gain': 0.029266727401392786}. Best is trial 0 with value: 0.000989986599512821.
[I 2025-06-15 19:49:45,954] A new study created in memory with name: no-name-b179bbf8-7f6c-4864-b083-1cdc6e4601e7
[I 2025-06-15 19:49:45,981] Trial 0 finished with value: 0.005877961619299034 and parameters: {'n_estimators': 164, 'learning_rate': 0.062381306905450254, 'num_leaves': 244, 'max_depth': 8, 'min_child_samples': 25, 'subsample': 0.5111297101064967, 'colsample_bytree': 0.8518321397699495, 'reg_alpha': 8.663601984385865, 'reg_lambda': 4.779725454407839, 'min_split_gain': 0.0070034614270942045}. Best is trial 0 with value: 0.005877961619299034.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2548
1           Price_lag_2        1653
14      pct_chg_Load_DA        1150
16  roll2d_median_Price         478
11           NGas_lag_2         441
2           Price_lag_7         438
3         Load_DA_lag_0         416
12            Oil_lag_2         169
15       lag168_Load_DA         169
5       WindOn_DA_lag_0         162
13            EUA_lag_2         158
10           Coal_lag_2         135
9                  WD_7           4
8                  WD_6           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1          22
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 13.41% done
********************* END NS.. 97 ****************************************************
********************* START NS ... 98 ****************************************************
[I 2025-06-15 19:49:47,343] A new study created in memory with name: no-name-d839db4d-3a91-43ab-9094-f120895e450b
[I 2025-06-15 19:49:47,514] Trial 0 finished with value: 0.0012825811441875155 and parameters: {'n_estimators': 148, 'learning_rate': 0.2479772300856234, 'num_leaves': 239, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.6312991465085346, 'colsample_bytree': 0.544149705691374, 'reg_alpha': 9.001440441409965, 'reg_lambda': 7.39673007079532, 'min_split_gain': 0.0473910080342361}. Best is trial 0 with value: 0.0012825811441875155.
[I 2025-06-15 19:49:47,655] Trial 1 finished with value: 0.0008186129768646068 and parameters: {'n_estimators': 66, 'learning_rate': 0.09620876461796612, 'num_leaves': 134, 'max_depth': 6, 'min_child_samples': 13, 'subsample': 0.9570269140647789, 'colsample_bytree': 0.7446890697624252, 'reg_alpha': 0.8762629734495275, 'reg_lambda': 6.145744030380249, 'min_split_gain': 0.0039634151697936935}. Best is trial 1 with value: 0.0008186129768646068.
[Trial 0] New best: 0.00128
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:49:48,192] Trial 2 finished with value: 0.0006990123529492866 and parameters: {'n_estimators': 111, 'learning_rate': 0.042590077524863185, 'num_leaves': 89, 'max_depth': 11, 'min_child_samples': 9, 'subsample': 0.6591062326500277, 'colsample_bytree': 0.9022089607226556, 'reg_alpha': 0.8101239948771477, 'reg_lambda': 9.966510127312212, 'min_split_gain': 0.029508302667403837}. Best is trial 2 with value: 0.0006990123529492866.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:48,423] Trial 3 finished with value: 0.0010263669356042717 and parameters: {'n_estimators': 154, 'learning_rate': 0.20754214671759244, 'num_leaves': 75, 'max_depth': 13, 'min_child_samples': 19, 'subsample': 0.8057478456404267, 'colsample_bytree': 0.5852934670807439, 'reg_alpha': 4.490103524453035, 'reg_lambda': 3.251069951293204, 'min_split_gain': 0.03619418850545661}. Best is trial 2 with value: 0.0006990123529492866.
[I 2025-06-15 19:49:48,426] A new study created in memory with name: no-name-a5c4e066-a99a-471a-93bd-dc2e41d6f71d
[I 2025-06-15 19:49:48,480] Trial 0 finished with value: 0.007275701984695013 and parameters: {'n_estimators': 163, 'learning_rate': 0.012482660522544724, 'num_leaves': 47, 'max_depth': 7, 'min_child_samples': 30, 'subsample': 0.6447479613864913, 'colsample_bytree': 0.7578301327101018, 'reg_alpha': 5.414300594920213, 'reg_lambda': 2.281644304196453, 'min_split_gain': 0.015033238153106143}. Best is trial 0 with value: 0.007275701984695013.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1636
1           Price_lag_2        1256
14      pct_chg_Load_DA        1212
3         Load_DA_lag_0         656
16  roll2d_median_Price         587
2           Price_lag_7         586
11           NGas_lag_2         518
15       lag168_Load_DA         439
5       WindOn_DA_lag_0         403
10           Coal_lag_2         334
13            EUA_lag_2         315
12            Oil_lag_2         276
9                  WD_7          14
7                  WD_1           8
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         112
1           Price_lag_2           6
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 13.54% done
********************* END NS.. 98 ****************************************************
********************* START NS ... 99 ****************************************************
[I 2025-06-15 19:49:50,588] A new study created in memory with name: no-name-565ea08d-2861-42a9-848f-edd0cf20e6de
[I 2025-06-15 19:49:50,869] Trial 0 finished with value: 0.003112540747675821 and parameters: {'n_estimators': 181, 'learning_rate': 0.10344943523365227, 'num_leaves': 222, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.8461608913930813, 'colsample_bytree': 0.9189432311811887, 'reg_alpha': 9.171595200475668, 'reg_lambda': 3.825530319967191, 'min_split_gain': 0.03299932058768383}. Best is trial 0 with value: 0.003112540747675821.
[Trial 0] New best: 0.00311
[I 2025-06-15 19:49:51,383] Trial 1 finished with value: 0.005005726589299567 and parameters: {'n_estimators': 247, 'learning_rate': 0.013268778610685468, 'num_leaves': 72, 'max_depth': 4, 'min_child_samples': 9, 'subsample': 0.5606961373455126, 'colsample_bytree': 0.628336264901145, 'reg_alpha': 3.2809407639287924, 'reg_lambda': 6.352755399736071, 'min_split_gain': 0.02889437468873936}. Best is trial 0 with value: 0.003112540747675821.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:49:52,686] Trial 2 finished with value: 0.0026362108373160207 and parameters: {'n_estimators': 243, 'learning_rate': 0.03659558149455267, 'num_leaves': 68, 'max_depth': 8, 'min_child_samples': 30, 'subsample': 0.8588998220297073, 'colsample_bytree': 0.8415853550797172, 'reg_alpha': 0.33287915710380367, 'reg_lambda': 7.180630032050907, 'min_split_gain': 0.03942336543313933}. Best is trial 2 with value: 0.0026362108373160207.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:53,476] Trial 3 finished with value: 0.0038825340520106367 and parameters: {'n_estimators': 300, 'learning_rate': 0.05942722272473041, 'num_leaves': 154, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.7681029678628523, 'colsample_bytree': 0.6361709511048544, 'reg_alpha': 4.465119067809859, 'reg_lambda': 5.8138775162761345, 'min_split_gain': 0.029809591880231297}. Best is trial 2 with value: 0.0026362108373160207.
[I 2025-06-15 19:49:53,479] A new study created in memory with name: no-name-be42b040-4e38-40dd-ab69-ca594d1e044c
[I 2025-06-15 19:49:53,538] Trial 0 finished with value: 0.029278740653489843 and parameters: {'n_estimators': 266, 'learning_rate': 0.18690283328253532, 'num_leaves': 103, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.5565160480825544, 'colsample_bytree': 0.8339354886194497, 'reg_alpha': 3.2389527903680704, 'reg_lambda': 8.254945003685265, 'min_split_gain': 0.046293344886642836}. Best is trial 0 with value: 0.029278740653489843.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2253
1           Price_lag_2        1814
14      pct_chg_Load_DA        1455
3         Load_DA_lag_0         925
2           Price_lag_7         648
16  roll2d_median_Price         635
11           NGas_lag_2         627
15       lag168_Load_DA         552
12            Oil_lag_2         411
10           Coal_lag_2         364
13            EUA_lag_2         356
5       WindOn_DA_lag_0         298
7                  WD_1          28
9                  WD_7          12
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          12
3         Load_DA_lag_0           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 13.68% done
********************* END NS.. 99 ****************************************************
********************* START NS ... 100 ****************************************************
[I 2025-06-15 19:49:55,439] A new study created in memory with name: no-name-e6b21220-44cf-4ea9-9d95-9f852068513b
[I 2025-06-15 19:49:55,823] Trial 0 finished with value: 0.013194844873859966 and parameters: {'n_estimators': 168, 'learning_rate': 0.05527162028838692, 'num_leaves': 221, 'max_depth': 4, 'min_child_samples': 11, 'subsample': 0.8104611806088526, 'colsample_bytree': 0.9757287064349102, 'reg_alpha': 8.56181857639415, 'reg_lambda': 3.002254401289762, 'min_split_gain': 0.0044318927497747134}. Best is trial 0 with value: 0.013194844873859966.
[Trial 0] New best: 0.01319
[I 2025-06-15 19:49:56,107] Trial 1 finished with value: 0.03222327106639412 and parameters: {'n_estimators': 97, 'learning_rate': 0.015222885742300402, 'num_leaves': 209, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.5291874758593242, 'colsample_bytree': 0.6298111361091203, 'reg_alpha': 5.607454302409179, 'reg_lambda': 4.063787750319886, 'min_split_gain': 0.026473753984013068}. Best is trial 0 with value: 0.013194844873859966.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:49:56,748] Trial 2 finished with value: 0.01768455916519863 and parameters: {'n_estimators': 182, 'learning_rate': 0.12161322979727475, 'num_leaves': 201, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.8383378547038225, 'colsample_bytree': 0.8886645400258554, 'reg_alpha': 3.2588850668370792, 'reg_lambda': 9.72681559992803, 'min_split_gain': 0.006241964642201847}. Best is trial 0 with value: 0.013194844873859966.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:49:57,086] Trial 3 finished with value: 0.020683789338070028 and parameters: {'n_estimators': 207, 'learning_rate': 0.1371516528363598, 'num_leaves': 39, 'max_depth': 7, 'min_child_samples': 22, 'subsample': 0.5523975631214917, 'colsample_bytree': 0.5526074035811817, 'reg_alpha': 4.903839567039378, 'reg_lambda': 8.667853987214972, 'min_split_gain': 0.02080292077073827}. Best is trial 0 with value: 0.013194844873859966.
[I 2025-06-15 19:49:57,089] A new study created in memory with name: no-name-211acdb4-da74-4252-bc5e-5055edfc9452
[I 2025-06-15 19:49:57,106] Trial 0 finished with value: 0.2763142332871259 and parameters: {'n_estimators': 62, 'learning_rate': 0.0827460772901838, 'num_leaves': 159, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.6683053170077753, 'colsample_bytree': 0.6870799460004098, 'reg_alpha': 6.281702599607371, 'reg_lambda': 4.179697421495674, 'min_split_gain': 0.0032960778330021093}. Best is trial 0 with value: 0.2763142332871259.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         716
14      pct_chg_Load_DA         352
1           Price_lag_2         172
3         Load_DA_lag_0         169
11           NGas_lag_2         166
16  roll2d_median_Price         120
2           Price_lag_7         118
13            EUA_lag_2         112
15       lag168_Load_DA          92
5       WindOn_DA_lag_0          84
12            Oil_lag_2          77
10           Coal_lag_2          59
7                  WD_1           5
9                  WD_7           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          61
1           Price_lag_2          10
10           Coal_lag_2           3
5       WindOn_DA_lag_0           1
16  roll2d_median_Price           1
3         Load_DA_lag_0           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
2           Price_lag_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 13.82% done
********************* END NS.. 100 ****************************************************
********************* START NS ... 101 ****************************************************
[I 2025-06-15 19:49:58,762] A new study created in memory with name: no-name-6683b78f-5d04-4c4b-9b04-04e773a578fa
[I 2025-06-15 19:49:59,176] Trial 0 finished with value: 0.019288663467579743 and parameters: {'n_estimators': 211, 'learning_rate': 0.06591913821302098, 'num_leaves': 140, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.867924236835257, 'colsample_bytree': 0.8695498553406136, 'reg_alpha': 9.767425697128374, 'reg_lambda': 0.23180021598112832, 'min_split_gain': 0.041988343849865946}. Best is trial 0 with value: 0.019288663467579743.
[Trial 0] New best: 0.01929
[I 2025-06-15 19:50:00,107] Trial 1 finished with value: 0.02894031884742893 and parameters: {'n_estimators': 144, 'learning_rate': 0.02337359117762215, 'num_leaves': 69, 'max_depth': 16, 'min_child_samples': 1, 'subsample': 0.6092278597066825, 'colsample_bytree': 0.9388968422931372, 'reg_alpha': 5.248857831337412, 'reg_lambda': 1.7719441600461017, 'min_split_gain': 0.006990638275846789}. Best is trial 0 with value: 0.019288663467579743.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:00,625] Trial 2 finished with value: 0.0283318257400333 and parameters: {'n_estimators': 66, 'learning_rate': 0.06901300220278883, 'num_leaves': 204, 'max_depth': 11, 'min_child_samples': 1, 'subsample': 0.9972074445270527, 'colsample_bytree': 0.6895065437591718, 'reg_alpha': 2.334614110790704, 'reg_lambda': 9.542727081024745, 'min_split_gain': 0.048511030356595104}. Best is trial 0 with value: 0.019288663467579743.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:01,062] Trial 3 finished with value: 0.022175092933938195 and parameters: {'n_estimators': 96, 'learning_rate': 0.025281826633114388, 'num_leaves': 238, 'max_depth': 5, 'min_child_samples': 13, 'subsample': 0.6227454170111533, 'colsample_bytree': 0.927911928496534, 'reg_alpha': 1.5276287363577357, 'reg_lambda': 0.28074800383520615, 'min_split_gain': 0.030575066210461327}. Best is trial 0 with value: 0.019288663467579743.
[I 2025-06-15 19:50:01,064] A new study created in memory with name: no-name-121b1013-946f-4e9f-990a-044cc66ceb25
[I 2025-06-15 19:50:01,126] Trial 0 finished with value: 0.3062825775836473 and parameters: {'n_estimators': 248, 'learning_rate': 0.03520487813125285, 'num_leaves': 56, 'max_depth': 15, 'min_child_samples': 24, 'subsample': 0.5174169847821335, 'colsample_bytree': 0.9195195971106453, 'reg_alpha': 0.9670926425120041, 'reg_lambda': 8.980793055201493, 'min_split_gain': 0.032535712964593975}. Best is trial 0 with value: 0.3062825775836473.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         640
14      pct_chg_Load_DA         328
1           Price_lag_2         272
11           NGas_lag_2         164
3         Load_DA_lag_0         158
2           Price_lag_7         138
16  roll2d_median_Price         104
12            Oil_lag_2          79
13            EUA_lag_2          78
10           Coal_lag_2          74
15       lag168_Load_DA          56
5       WindOn_DA_lag_0          23
9                  WD_7           3
7                  WD_1           1
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         102
3         Load_DA_lag_0          45
16  roll2d_median_Price          43
2           Price_lag_7          22
12            Oil_lag_2          14
15       lag168_Load_DA           3
1           Price_lag_2           2
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 13.95% done
********************* END NS.. 101 ****************************************************
********************* START NS ... 102 ****************************************************
[I 2025-06-15 19:50:03,099] A new study created in memory with name: no-name-bbd7719d-eaa5-4c82-a5a2-d5bd3edf8633
[I 2025-06-15 19:50:04,320] Trial 0 finished with value: 0.010629875251157212 and parameters: {'n_estimators': 260, 'learning_rate': 0.017279340908448983, 'num_leaves': 67, 'max_depth': 10, 'min_child_samples': 23, 'subsample': 0.8611031013190388, 'colsample_bytree': 0.9367278216319466, 'reg_alpha': 2.287541645362643, 'reg_lambda': 7.022270962257515, 'min_split_gain': 0.002551331267385576}. Best is trial 0 with value: 0.010629875251157212.
[Trial 0] New best: 0.01063
[I 2025-06-15 19:50:05,056] Trial 1 finished with value: 0.010598243609832534 and parameters: {'n_estimators': 144, 'learning_rate': 0.03202140095773428, 'num_leaves': 118, 'max_depth': 12, 'min_child_samples': 1, 'subsample': 0.8457886524715936, 'colsample_bytree': 0.7849415338019787, 'reg_alpha': 4.105315963143835, 'reg_lambda': 7.99000562846679, 'min_split_gain': 0.018625864274360943}. Best is trial 1 with value: 0.010598243609832534.
[I 2025-06-15 19:50:05,256] Trial 2 finished with value: 0.010766923780972418 and parameters: {'n_estimators': 82, 'learning_rate': 0.07458623515678801, 'num_leaves': 124, 'max_depth': 5, 'min_child_samples': 26, 'subsample': 0.6038519013307545, 'colsample_bytree': 0.9856581188195863, 'reg_alpha': 7.231330704412575, 'reg_lambda': 9.092172714906429, 'min_split_gain': 0.004746540526316185}. Best is trial 1 with value: 0.010598243609832534.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:06,376] Trial 3 finished with value: 0.014532432802488218 and parameters: {'n_estimators': 274, 'learning_rate': 0.02095501557363245, 'num_leaves': 236, 'max_depth': 13, 'min_child_samples': 5, 'subsample': 0.7016563085699434, 'colsample_bytree': 0.5923914207986436, 'reg_alpha': 3.483415997651027, 'reg_lambda': 6.146870382349009, 'min_split_gain': 0.0475601063916663}. Best is trial 1 with value: 0.010598243609832534.
[I 2025-06-15 19:50:06,379] A new study created in memory with name: no-name-98376c6f-db6a-4cee-9dcb-8b4ff940e057
[I 2025-06-15 19:50:06,428] Trial 0 finished with value: 0.061529552691189295 and parameters: {'n_estimators': 299, 'learning_rate': 0.03689559538759539, 'num_leaves': 236, 'max_depth': 4, 'min_child_samples': 28, 'subsample': 0.870877814655989, 'colsample_bytree': 0.6964965419048534, 'reg_alpha': 1.8736231738303188, 'reg_lambda': 2.3841056422051254, 'min_split_gain': 0.02437179842055546}. Best is trial 0 with value: 0.061529552691189295.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1591
1           Price_lag_2        1443
14      pct_chg_Load_DA         963
2           Price_lag_7         703
11           NGas_lag_2         597
3         Load_DA_lag_0         536
16  roll2d_median_Price         523
10           Coal_lag_2         505
5       WindOn_DA_lag_0         364
15       lag168_Load_DA         339
13            EUA_lag_2         329
12            Oil_lag_2         261
9                  WD_7          22
7                  WD_1           1
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          82
3         Load_DA_lag_0          39
2           Price_lag_7          28
16  roll2d_median_Price          18
1           Price_lag_2          11
12            Oil_lag_2           5
10           Coal_lag_2           1
15       lag168_Load_DA           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
11           NGas_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 14.09% done
********************* END NS.. 102 ****************************************************
********************* START NS ... 103 ****************************************************
[I 2025-06-15 19:50:08,199] A new study created in memory with name: no-name-9f571832-493c-4ae0-9208-5eab185917d8
[I 2025-06-15 19:50:08,701] Trial 0 finished with value: 0.002471786829819434 and parameters: {'n_estimators': 202, 'learning_rate': 0.0899431945244832, 'num_leaves': 32, 'max_depth': 14, 'min_child_samples': 23, 'subsample': 0.6582941942744934, 'colsample_bytree': 0.6836399513221827, 'reg_alpha': 5.361100066162049, 'reg_lambda': 7.5540969909745295, 'min_split_gain': 0.002377463851916145}. Best is trial 0 with value: 0.002471786829819434.
[Trial 0] New best: 0.00247
[I 2025-06-15 19:50:09,196] Trial 1 finished with value: 0.0021937629043043715 and parameters: {'n_estimators': 94, 'learning_rate': 0.045893428128811996, 'num_leaves': 218, 'max_depth': 13, 'min_child_samples': 7, 'subsample': 0.6139979321947431, 'colsample_bytree': 0.9389564251846125, 'reg_alpha': 1.8477175728258233, 'reg_lambda': 3.7632290008289493, 'min_split_gain': 0.02862302520987031}. Best is trial 1 with value: 0.0021937629043043715.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:09,496] Trial 2 finished with value: 0.00238649358843864 and parameters: {'n_estimators': 171, 'learning_rate': 0.07629036966143027, 'num_leaves': 78, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.5858858337919058, 'colsample_bytree': 0.9155265924944777, 'reg_alpha': 6.013541956739124, 'reg_lambda': 7.0685638871816225, 'min_split_gain': 0.023028777467677647}. Best is trial 1 with value: 0.0021937629043043715.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:09,983] Trial 3 finished with value: 0.0027181722467804407 and parameters: {'n_estimators': 261, 'learning_rate': 0.044934633391033124, 'num_leaves': 150, 'max_depth': 6, 'min_child_samples': 14, 'subsample': 0.7526988081698914, 'colsample_bytree': 0.775573034264468, 'reg_alpha': 9.120171640999818, 'reg_lambda': 8.017321719533653, 'min_split_gain': 0.020944122519571902}. Best is trial 1 with value: 0.0021937629043043715.
[I 2025-06-15 19:50:09,986] A new study created in memory with name: no-name-0841067b-3d76-492a-94d5-8967585890c6
[I 2025-06-15 19:50:10,009] Trial 0 finished with value: 0.007739888707403071 and parameters: {'n_estimators': 107, 'learning_rate': 0.04530605274492675, 'num_leaves': 128, 'max_depth': 14, 'min_child_samples': 28, 'subsample': 0.8310462554284728, 'colsample_bytree': 0.9707976972595258, 'reg_alpha': 7.108474279528058, 'reg_lambda': 1.966260836067918, 'min_split_gain': 0.04408256850884818}. Best is trial 0 with value: 0.007739888707403071.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1349
1           Price_lag_2         999
14      pct_chg_Load_DA         899
3         Load_DA_lag_0         502
2           Price_lag_7         465
16  roll2d_median_Price         406
11           NGas_lag_2         355
15       lag168_Load_DA         273
5       WindOn_DA_lag_0         251
10           Coal_lag_2         240
12            Oil_lag_2         210
13            EUA_lag_2         204
9                  WD_7          13
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          73
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 14.23% done
********************* END NS.. 103 ****************************************************
********************* START NS ... 104 ****************************************************
[I 2025-06-15 19:50:11,242] A new study created in memory with name: no-name-96add9a0-24eb-4473-9d40-b23254bad9bf
[I 2025-06-15 19:50:11,438] Trial 0 finished with value: 0.0015741554482724268 and parameters: {'n_estimators': 107, 'learning_rate': 0.08590453043805339, 'num_leaves': 100, 'max_depth': 9, 'min_child_samples': 12, 'subsample': 0.6482226820384503, 'colsample_bytree': 0.8520515340451327, 'reg_alpha': 4.193052633433486, 'reg_lambda': 9.565583032235319, 'min_split_gain': 0.04912170719457307}. Best is trial 0 with value: 0.0015741554482724268.
[Trial 0] New best: 0.00157
[I 2025-06-15 19:50:12,381] Trial 1 finished with value: 0.001379172305911281 and parameters: {'n_estimators': 199, 'learning_rate': 0.017001565542897536, 'num_leaves': 75, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.9216640041976729, 'colsample_bytree': 0.8178144467766069, 'reg_alpha': 0.6015556348327278, 'reg_lambda': 8.163039085929444, 'min_split_gain': 0.037646647550959686}. Best is trial 1 with value: 0.001379172305911281.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:12,601] Trial 2 finished with value: 0.0017364931433192208 and parameters: {'n_estimators': 183, 'learning_rate': 0.2018983896179026, 'num_leaves': 224, 'max_depth': 15, 'min_child_samples': 12, 'subsample': 0.5564145155880322, 'colsample_bytree': 0.9216622256814795, 'reg_alpha': 9.935639064938664, 'reg_lambda': 8.080275815401466, 'min_split_gain': 0.005725801870901443}. Best is trial 1 with value: 0.001379172305911281.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:13,144] Trial 3 finished with value: 0.001800132034204314 and parameters: {'n_estimators': 148, 'learning_rate': 0.02832716056843995, 'num_leaves': 49, 'max_depth': 16, 'min_child_samples': 2, 'subsample': 0.7119371697238366, 'colsample_bytree': 0.5404392235888011, 'reg_alpha': 5.119783224420623, 'reg_lambda': 0.18706356284217351, 'min_split_gain': 0.010800607652897753}. Best is trial 1 with value: 0.001379172305911281.
[I 2025-06-15 19:50:13,146] A new study created in memory with name: no-name-c6f13fca-9bc2-40fc-bf6b-3b81e4939840
[I 2025-06-15 19:50:13,178] Trial 0 finished with value: 0.0066951885099212965 and parameters: {'n_estimators': 263, 'learning_rate': 0.23120391542974372, 'num_leaves': 171, 'max_depth': 7, 'min_child_samples': 20, 'subsample': 0.9276742317925866, 'colsample_bytree': 0.8487796043984213, 'reg_alpha': 5.272582283811431, 'reg_lambda': 4.033875540986768, 'min_split_gain': 0.04578957168099382}. Best is trial 0 with value: 0.0066951885099212965.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3477
1           Price_lag_2        2924
14      pct_chg_Load_DA        1731
16  roll2d_median_Price         876
11           NGas_lag_2         800
2           Price_lag_7         790
3         Load_DA_lag_0         713
10           Coal_lag_2         600
12            Oil_lag_2         391
15       lag168_Load_DA         391
5       WindOn_DA_lag_0         353
13            EUA_lag_2         310
9                  WD_7           8
7                  WD_1           3
8                  WD_6           3
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          17
12            Oil_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 14.36% done
********************* END NS.. 104 ****************************************************
********************* START NS ... 105 ****************************************************
[I 2025-06-15 19:50:14,849] A new study created in memory with name: no-name-2c140a2a-085b-4b75-8abc-766c722f689d
[I 2025-06-15 19:50:15,072] Trial 0 finished with value: 0.0010530916799668818 and parameters: {'n_estimators': 69, 'learning_rate': 0.03463123669922312, 'num_leaves': 190, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.7091018826460374, 'colsample_bytree': 0.8616982110483354, 'reg_alpha': 2.2306066376576106, 'reg_lambda': 4.952127071962259, 'min_split_gain': 0.004745520439234341}. Best is trial 0 with value: 0.0010530916799668818.
[Trial 0] New best: 0.00105
[I 2025-06-15 19:50:15,444] Trial 1 finished with value: 0.0012609089792303154 and parameters: {'n_estimators': 162, 'learning_rate': 0.05952713850639601, 'num_leaves': 250, 'max_depth': 6, 'min_child_samples': 5, 'subsample': 0.7527747097540456, 'colsample_bytree': 0.7786029332978825, 'reg_alpha': 0.7142185675930002, 'reg_lambda': 5.170789262336051, 'min_split_gain': 0.033473270802374326}. Best is trial 0 with value: 0.0010530916799668818.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:15,693] Trial 2 finished with value: 0.0028841760917019082 and parameters: {'n_estimators': 99, 'learning_rate': 0.011551582851647106, 'num_leaves': 240, 'max_depth': 5, 'min_child_samples': 19, 'subsample': 0.9300605248569571, 'colsample_bytree': 0.9319260300709351, 'reg_alpha': 6.356052828761991, 'reg_lambda': 9.024814370651793, 'min_split_gain': 0.023867327400783622}. Best is trial 0 with value: 0.0010530916799668818.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:16,700] Trial 3 finished with value: 0.0012686300186848431 and parameters: {'n_estimators': 299, 'learning_rate': 0.04480151881304473, 'num_leaves': 131, 'max_depth': 13, 'min_child_samples': 21, 'subsample': 0.535206438529755, 'colsample_bytree': 0.9143423999597337, 'reg_alpha': 7.486292767376335, 'reg_lambda': 5.661279158197211, 'min_split_gain': 0.0052233813516409}. Best is trial 0 with value: 0.0010530916799668818.
[I 2025-06-15 19:50:16,702] A new study created in memory with name: no-name-a16ad658-e80d-4deb-a596-e0ba0d809517
[I 2025-06-15 19:50:16,729] Trial 0 finished with value: 0.019190784274464318 and parameters: {'n_estimators': 63, 'learning_rate': 0.04619420603072317, 'num_leaves': 201, 'max_depth': 6, 'min_child_samples': 7, 'subsample': 0.7692779923617687, 'colsample_bytree': 0.824622602234744, 'reg_alpha': 6.982570338795088, 'reg_lambda': 9.389040942349746, 'min_split_gain': 0.03958149197177465}. Best is trial 0 with value: 0.019190784274464318.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1156
1           Price_lag_2         816
14      pct_chg_Load_DA         521
16  roll2d_median_Price         323
2           Price_lag_7         287
11           NGas_lag_2         224
10           Coal_lag_2         172
3         Load_DA_lag_0         133
13            EUA_lag_2         117
12            Oil_lag_2          75
15       lag168_Load_DA          66
5       WindOn_DA_lag_0          65
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          77
1           Price_lag_2           9
3         Load_DA_lag_0           5
12            Oil_lag_2           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 14.50% done
********************* END NS.. 105 ****************************************************
********************* START NS ... 106 ****************************************************
[I 2025-06-15 19:50:18,348] A new study created in memory with name: no-name-5b9a61c1-e878-4445-88fe-e2b070600020
[I 2025-06-15 19:50:18,931] Trial 0 finished with value: 0.0033691961231432483 and parameters: {'n_estimators': 94, 'learning_rate': 0.011192189797461127, 'num_leaves': 194, 'max_depth': 9, 'min_child_samples': 15, 'subsample': 0.865492224498616, 'colsample_bytree': 0.5458708776600032, 'reg_alpha': 1.8364696977482498, 'reg_lambda': 4.694850199190056, 'min_split_gain': 0.038839972805690876}. Best is trial 0 with value: 0.0033691961231432483.
[Trial 0] New best: 0.00337
[I 2025-06-15 19:50:19,203] Trial 1 finished with value: 0.0013192762883049922 and parameters: {'n_estimators': 145, 'learning_rate': 0.18459711088097258, 'num_leaves': 127, 'max_depth': 15, 'min_child_samples': 17, 'subsample': 0.9918715739207538, 'colsample_bytree': 0.9857587860753683, 'reg_alpha': 7.786100490676235, 'reg_lambda': 3.743530649324579, 'min_split_gain': 0.01473925276316045}. Best is trial 1 with value: 0.0013192762883049922.
[Trial 1] New best: 0.00132
[I 2025-06-15 19:50:20,633] Trial 2 finished with value: 0.0010772812550332487 and parameters: {'n_estimators': 264, 'learning_rate': 0.013995525432019785, 'num_leaves': 81, 'max_depth': 10, 'min_child_samples': 5, 'subsample': 0.5237131247234176, 'colsample_bytree': 0.9084452489228638, 'reg_alpha': 5.076267071200299, 'reg_lambda': 6.460100180117919, 'min_split_gain': 0.017110844574565808}. Best is trial 2 with value: 0.0010772812550332487.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:21,403] Trial 3 finished with value: 0.0013572708533040286 and parameters: {'n_estimators': 118, 'learning_rate': 0.02158829057985215, 'num_leaves': 179, 'max_depth': 8, 'min_child_samples': 20, 'subsample': 0.5481776349665437, 'colsample_bytree': 0.7097943481841914, 'reg_alpha': 7.078059586271848, 'reg_lambda': 2.5085402332944087, 'min_split_gain': 0.029248496244402802}. Best is trial 2 with value: 0.0010772812550332487.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:22,342] Trial 4 finished with value: 0.0011794753613385994 and parameters: {'n_estimators': 111, 'learning_rate': 0.042391177801146575, 'num_leaves': 122, 'max_depth': 8, 'min_child_samples': 18, 'subsample': 0.8317349671652061, 'colsample_bytree': 0.6100099987710814, 'reg_alpha': 1.7117203607633458, 'reg_lambda': 6.0486650849755765, 'min_split_gain': 0.04419128704960973}. Best is trial 2 with value: 0.0010772812550332487.
[I 2025-06-15 19:50:22,346] A new study created in memory with name: no-name-376dc85a-3283-49cf-8dc7-375c209b383d
[I 2025-06-15 19:50:22,382] Trial 0 finished with value: 0.013741040464359316 and parameters: {'n_estimators': 190, 'learning_rate': 0.27590161852212647, 'num_leaves': 37, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.9650180685123746, 'colsample_bytree': 0.5584551414545671, 'reg_alpha': 7.408047789947826, 'reg_lambda': 3.6525129271613723, 'min_split_gain': 0.02300288863551427}. Best is trial 0 with value: 0.013741040464359316.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3239
1           Price_lag_2        1666
14      pct_chg_Load_DA        1474
2           Price_lag_7         904
16  roll2d_median_Price         834
11           NGas_lag_2         753
10           Coal_lag_2         609
3         Load_DA_lag_0         491
13            EUA_lag_2         400
12            Oil_lag_2         391
5       WindOn_DA_lag_0         241
15       lag168_Load_DA         206
9                  WD_7          19
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          10
3         Load_DA_lag_0           4
1           Price_lag_2           2
11           NGas_lag_2           2
10           Coal_lag_2           1
2           Price_lag_7           1
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 14.64% done
********************* END NS.. 106 ****************************************************
********************* START NS ... 107 ****************************************************
[I 2025-06-15 19:50:24,927] A new study created in memory with name: no-name-829bb5ea-9731-42b2-84ed-7f7311752a42
[I 2025-06-15 19:50:26,147] Trial 0 finished with value: 0.00834217935384316 and parameters: {'n_estimators': 257, 'learning_rate': 0.014184304137449522, 'num_leaves': 66, 'max_depth': 12, 'min_child_samples': 9, 'subsample': 0.7814725457928873, 'colsample_bytree': 0.8426680626260951, 'reg_alpha': 7.445738502716863, 'reg_lambda': 0.8212763865011397, 'min_split_gain': 0.02734081060017629}. Best is trial 0 with value: 0.00834217935384316.
[Trial 0] New best: 0.00834
[I 2025-06-15 19:50:26,484] Trial 1 finished with value: 0.014555922903140525 and parameters: {'n_estimators': 122, 'learning_rate': 0.011093469855557585, 'num_leaves': 124, 'max_depth': 4, 'min_child_samples': 28, 'subsample': 0.5829342970242187, 'colsample_bytree': 0.5570598759031304, 'reg_alpha': 0.2077242068741847, 'reg_lambda': 0.24506232425965946, 'min_split_gain': 0.02554543323913837}. Best is trial 0 with value: 0.00834217935384316.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:26,953] Trial 2 finished with value: 0.008730452889585833 and parameters: {'n_estimators': 196, 'learning_rate': 0.16229238935983795, 'num_leaves': 71, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.7726922809965079, 'colsample_bytree': 0.8923292756682559, 'reg_alpha': 4.822647347504861, 'reg_lambda': 0.5336450302753559, 'min_split_gain': 0.032227565768707576}. Best is trial 0 with value: 0.00834217935384316.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:27,706] Trial 3 finished with value: 0.009846442213969025 and parameters: {'n_estimators': 197, 'learning_rate': 0.011326582152723181, 'num_leaves': 51, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.9885644385782626, 'colsample_bytree': 0.6558457634032326, 'reg_alpha': 7.503715962608833, 'reg_lambda': 6.854461769431164, 'min_split_gain': 0.02598149685121559}. Best is trial 0 with value: 0.00834217935384316.
[I 2025-06-15 19:50:27,707] A new study created in memory with name: no-name-e5ed7bae-e115-4c65-bf6f-e4f13cdedcb0
[I 2025-06-15 19:50:27,750] Trial 0 finished with value: 0.027759566007816375 and parameters: {'n_estimators': 177, 'learning_rate': 0.020177699959316103, 'num_leaves': 100, 'max_depth': 6, 'min_child_samples': 15, 'subsample': 0.5842564307616089, 'colsample_bytree': 0.7735514747791451, 'reg_alpha': 2.11171483768544, 'reg_lambda': 1.6380528950462836, 'min_split_gain': 0.012830988891308865}. Best is trial 0 with value: 0.027759566007816375.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2772
1           Price_lag_2        1431
14      pct_chg_Load_DA        1022
16  roll2d_median_Price         566
2           Price_lag_7         513
11           NGas_lag_2         418
10           Coal_lag_2         313
3         Load_DA_lag_0         266
13            EUA_lag_2         239
12            Oil_lag_2         220
5       WindOn_DA_lag_0         116
15       lag168_Load_DA         116
9                  WD_7          13
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         222
3         Load_DA_lag_0          67
2           Price_lag_7          30
1           Price_lag_2          21
5       WindOn_DA_lag_0           5
16  roll2d_median_Price           2
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 14.77% done
********************* END NS.. 107 ****************************************************
********************* START NS ... 108 ****************************************************
[I 2025-06-15 19:50:29,417] A new study created in memory with name: no-name-ced11b1d-1a01-4271-adf7-80b37c07e288
[I 2025-06-15 19:50:29,742] Trial 0 finished with value: 0.010083636487874552 and parameters: {'n_estimators': 60, 'learning_rate': 0.061559139044901304, 'num_leaves': 212, 'max_depth': 11, 'min_child_samples': 17, 'subsample': 0.7489193667361832, 'colsample_bytree': 0.7884407486427546, 'reg_alpha': 0.3853656918185244, 'reg_lambda': 5.735757469877556, 'min_split_gain': 0.04708659805979054}. Best is trial 0 with value: 0.010083636487874552.
[Trial 0] New best: 0.01008
[I 2025-06-15 19:50:29,954] Trial 1 finished with value: 0.011423688446681615 and parameters: {'n_estimators': 243, 'learning_rate': 0.2387592873679479, 'num_leaves': 223, 'max_depth': 7, 'min_child_samples': 23, 'subsample': 0.6675094784089577, 'colsample_bytree': 0.8994631106799309, 'reg_alpha': 9.019586630206613, 'reg_lambda': 5.544531987335551, 'min_split_gain': 0.04177388876457}. Best is trial 0 with value: 0.010083636487874552.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:30,229] Trial 2 finished with value: 0.010372617030907189 and parameters: {'n_estimators': 87, 'learning_rate': 0.08392168483160654, 'num_leaves': 49, 'max_depth': 13, 'min_child_samples': 19, 'subsample': 0.5001533422517933, 'colsample_bytree': 0.9404997314867738, 'reg_alpha': 6.142993668014157, 'reg_lambda': 0.44335570111588063, 'min_split_gain': 0.047680659269527616}. Best is trial 0 with value: 0.010083636487874552.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:30,669] Trial 3 finished with value: 0.013120965215663619 and parameters: {'n_estimators': 157, 'learning_rate': 0.013772307630168176, 'num_leaves': 202, 'max_depth': 5, 'min_child_samples': 25, 'subsample': 0.7127397662595373, 'colsample_bytree': 0.9110680902400119, 'reg_alpha': 7.184490351187916, 'reg_lambda': 5.182551143707813, 'min_split_gain': 0.04206961569091506}. Best is trial 0 with value: 0.010083636487874552.
[I 2025-06-15 19:50:30,671] A new study created in memory with name: no-name-1efc6ac2-478a-4e52-9d28-6a56ee425fc3
[I 2025-06-15 19:50:30,707] Trial 0 finished with value: 0.04188184820927381 and parameters: {'n_estimators': 136, 'learning_rate': 0.013270117544384785, 'num_leaves': 102, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.615143169316956, 'colsample_bytree': 0.7579050415423345, 'reg_alpha': 4.342172241000108, 'reg_lambda': 2.573364983960208, 'min_split_gain': 0.044802967458785814}. Best is trial 0 with value: 0.04188184820927381.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1015
1           Price_lag_2         989
14      pct_chg_Load_DA         619
3         Load_DA_lag_0         308
2           Price_lag_7         293
16  roll2d_median_Price         287
11           NGas_lag_2         285
13            EUA_lag_2         163
15       lag168_Load_DA         162
10           Coal_lag_2         140
12            Oil_lag_2         120
5       WindOn_DA_lag_0         106
9                  WD_7           5
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           8
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 14.91% done
********************* END NS.. 108 ****************************************************
********************* START NS ... 109 ****************************************************
[I 2025-06-15 19:50:32,425] A new study created in memory with name: no-name-6f80d89b-aa20-4c80-9ce5-86557da52972
[I 2025-06-15 19:50:32,685] Trial 0 finished with value: 0.006563328240923652 and parameters: {'n_estimators': 200, 'learning_rate': 0.044521640783121146, 'num_leaves': 35, 'max_depth': 4, 'min_child_samples': 27, 'subsample': 0.5043526714553426, 'colsample_bytree': 0.8831610969274228, 'reg_alpha': 1.4192321358574123, 'reg_lambda': 3.407863075751145, 'min_split_gain': 0.048825153779096145}. Best is trial 0 with value: 0.006563328240923652.
[Trial 0] New best: 0.00656
[I 2025-06-15 19:50:33,637] Trial 1 finished with value: 0.0072347357375658295 and parameters: {'n_estimators': 252, 'learning_rate': 0.04059430936, 'num_leaves': 45, 'max_depth': 12, 'min_child_samples': 21, 'subsample': 0.7316066726835821, 'colsample_bytree': 0.6085146149337775, 'reg_alpha': 9.145872478769785, 'reg_lambda': 1.5344082304668372, 'min_split_gain': 0.0020385478724056216}. Best is trial 0 with value: 0.006563328240923652.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:33,896] Trial 2 finished with value: 0.006044122499090202 and parameters: {'n_estimators': 195, 'learning_rate': 0.26854274315268456, 'num_leaves': 211, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.9050260969855157, 'colsample_bytree': 0.8898247120348979, 'reg_alpha': 1.0842261667774278, 'reg_lambda': 8.478379570949869, 'min_split_gain': 0.01732619591174774}. Best is trial 2 with value: 0.006044122499090202.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:34,226] Trial 3 finished with value: 0.005198136893366272 and parameters: {'n_estimators': 246, 'learning_rate': 0.12228038308739987, 'num_leaves': 195, 'max_depth': 10, 'min_child_samples': 10, 'subsample': 0.7616719818520092, 'colsample_bytree': 0.8372632013716563, 'reg_alpha': 3.3125722405699864, 'reg_lambda': 4.850128556211298, 'min_split_gain': 0.0397472560577633}. Best is trial 3 with value: 0.005198136893366272.
[Trial 3] New best: 0.00520
[I 2025-06-15 19:50:34,442] Trial 4 finished with value: 0.006476419088824092 and parameters: {'n_estimators': 244, 'learning_rate': 0.2069243346661966, 'num_leaves': 148, 'max_depth': 15, 'min_child_samples': 15, 'subsample': 0.5183104036355132, 'colsample_bytree': 0.8870532353971092, 'reg_alpha': 6.0493173078215525, 'reg_lambda': 3.031457893623306, 'min_split_gain': 0.045961079059272575}. Best is trial 3 with value: 0.005198136893366272.
[I 2025-06-15 19:50:34,631] Trial 5 finished with value: 0.006659928289001623 and parameters: {'n_estimators': 85, 'learning_rate': 0.19629049066840396, 'num_leaves': 64, 'max_depth': 14, 'min_child_samples': 15, 'subsample': 0.9536650367075401, 'colsample_bytree': 0.8665065450198668, 'reg_alpha': 6.054003720821093, 'reg_lambda': 2.9788005415006458, 'min_split_gain': 0.049861769259959283}. Best is trial 3 with value: 0.005198136893366272.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:34,846] Trial 6 finished with value: 0.008111068942923723 and parameters: {'n_estimators': 67, 'learning_rate': 0.09754387891188622, 'num_leaves': 241, 'max_depth': 4, 'min_child_samples': 1, 'subsample': 0.9630569093849195, 'colsample_bytree': 0.6038181196188259, 'reg_alpha': 8.300968075015179, 'reg_lambda': 6.128126795861012, 'min_split_gain': 0.03200569403225847}. Best is trial 3 with value: 0.005198136893366272.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:34,849] A new study created in memory with name: no-name-36ec0d14-d3ae-4e47-a371-71a49afdf5d2
[I 2025-06-15 19:50:34,894] Trial 0 finished with value: 0.01585293310388294 and parameters: {'n_estimators': 138, 'learning_rate': 0.026209593812954148, 'num_leaves': 94, 'max_depth': 4, 'min_child_samples': 7, 'subsample': 0.982731781366453, 'colsample_bytree': 0.7027189011246877, 'reg_alpha': 0.7876279946793407, 'reg_lambda': 3.6978197419079106, 'min_split_gain': 0.040765436162069346}. Best is trial 0 with value: 0.01585293310388294.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         459
14      pct_chg_Load_DA         305
1           Price_lag_2         290
3         Load_DA_lag_0         201
11           NGas_lag_2         169
2           Price_lag_7         151
16  roll2d_median_Price         123
15       lag168_Load_DA         110
10           Coal_lag_2          81
13            EUA_lag_2          81
12            Oil_lag_2          79
5       WindOn_DA_lag_0          66
7                  WD_1           7
9                  WD_7           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         134
3         Load_DA_lag_0          32
1           Price_lag_2          21
14      pct_chg_Load_DA          10
2           Price_lag_7           3
10           Coal_lag_2           0
15       lag168_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 15.05% done
********************* END NS.. 109 ****************************************************
********************* START NS ... 110 ****************************************************
[I 2025-06-15 19:50:36,599] A new study created in memory with name: no-name-3c655991-f871-40e8-9432-8d1dda9b007a
[I 2025-06-15 19:50:37,079] Trial 0 finished with value: 0.0035934393500547924 and parameters: {'n_estimators': 64, 'learning_rate': 0.06709930737900978, 'num_leaves': 143, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.6596208274045476, 'colsample_bytree': 0.5643491503529328, 'reg_alpha': 1.8306105384345939, 'reg_lambda': 0.33637985827759787, 'min_split_gain': 0.007380228124547212}. Best is trial 0 with value: 0.0035934393500547924.
[Trial 0] New best: 0.00359
[I 2025-06-15 19:50:37,390] Trial 1 finished with value: 0.003445106090892367 and parameters: {'n_estimators': 218, 'learning_rate': 0.18090506162257367, 'num_leaves': 232, 'max_depth': 13, 'min_child_samples': 20, 'subsample': 0.9778556078056788, 'colsample_bytree': 0.7110746174659579, 'reg_alpha': 0.7927789002553309, 'reg_lambda': 5.777458003995247, 'min_split_gain': 0.03492699322515063}. Best is trial 1 with value: 0.003445106090892367.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:37,771] Trial 2 finished with value: 0.0031482013307587205 and parameters: {'n_estimators': 77, 'learning_rate': 0.0313345972538685, 'num_leaves': 112, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.9035309712348791, 'colsample_bytree': 0.7970169443008281, 'reg_alpha': 3.586226192502935, 'reg_lambda': 0.2140563115597227, 'min_split_gain': 0.033532375113312876}. Best is trial 2 with value: 0.0031482013307587205.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:38,007] Trial 3 finished with value: 0.0033777792518319472 and parameters: {'n_estimators': 159, 'learning_rate': 0.26931527908529757, 'num_leaves': 177, 'max_depth': 11, 'min_child_samples': 23, 'subsample': 0.6333541591099043, 'colsample_bytree': 0.9005096447088778, 'reg_alpha': 6.6983302208918, 'reg_lambda': 1.7044820221959, 'min_split_gain': 0.003169993359948975}. Best is trial 2 with value: 0.0031482013307587205.
[I 2025-06-15 19:50:38,010] A new study created in memory with name: no-name-c36e6f99-a8d0-43de-87cb-bb852c65d9bc
[I 2025-06-15 19:50:38,042] Trial 0 finished with value: 0.012194107583058757 and parameters: {'n_estimators': 189, 'learning_rate': 0.04073685218323654, 'num_leaves': 235, 'max_depth': 10, 'min_child_samples': 29, 'subsample': 0.6181537607085444, 'colsample_bytree': 0.5798212969431575, 'reg_alpha': 0.9800638525041339, 'reg_lambda': 5.249440616343511, 'min_split_gain': 0.03734726822323229}. Best is trial 0 with value: 0.012194107583058757.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1113
1           Price_lag_2         678
14      pct_chg_Load_DA         407
16  roll2d_median_Price         215
11           NGas_lag_2         181
2           Price_lag_7         176
3         Load_DA_lag_0          87
10           Coal_lag_2          85
13            EUA_lag_2          75
12            Oil_lag_2          64
15       lag168_Load_DA          39
5       WindOn_DA_lag_0          26
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          70
3         Load_DA_lag_0          12
14      pct_chg_Load_DA           6
5       WindOn_DA_lag_0           3
1           Price_lag_2           2
16  roll2d_median_Price           2
10           Coal_lag_2           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
2           Price_lag_7           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 15.18% done
********************* END NS.. 110 ****************************************************
********************* START NS ... 111 ****************************************************
[I 2025-06-15 19:50:39,479] A new study created in memory with name: no-name-f3a4cd6d-504a-40ae-b490-f412ad40688b
[I 2025-06-15 19:50:39,954] Trial 0 finished with value: 0.0025862489638003206 and parameters: {'n_estimators': 251, 'learning_rate': 0.03939741794197447, 'num_leaves': 164, 'max_depth': 5, 'min_child_samples': 7, 'subsample': 0.9096215479565625, 'colsample_bytree': 0.7384529007448705, 'reg_alpha': 3.5051074484572418, 'reg_lambda': 3.5964869648602593, 'min_split_gain': 0.017184775312959456}. Best is trial 0 with value: 0.0025862489638003206.
[Trial 0] New best: 0.00259
[I 2025-06-15 19:50:41,013] Trial 1 finished with value: 0.0023744788131838854 and parameters: {'n_estimators': 222, 'learning_rate': 0.04803280331359132, 'num_leaves': 40, 'max_depth': 10, 'min_child_samples': 29, 'subsample': 0.696715654958445, 'colsample_bytree': 0.9561809106410537, 'reg_alpha': 9.33509626450227, 'reg_lambda': 7.216069661134618, 'min_split_gain': 0.004497125325077511}. Best is trial 1 with value: 0.0023744788131838854.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:41,631] Trial 2 finished with value: 0.0021551760712632314 and parameters: {'n_estimators': 175, 'learning_rate': 0.15768252597242038, 'num_leaves': 91, 'max_depth': 13, 'min_child_samples': 4, 'subsample': 0.8776532215070183, 'colsample_bytree': 0.7765208128563386, 'reg_alpha': 3.6487062897851965, 'reg_lambda': 3.431166610842279, 'min_split_gain': 0.029019795046583682}. Best is trial 2 with value: 0.0021551760712632314.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:41,922] Trial 3 finished with value: 0.0027305834827691844 and parameters: {'n_estimators': 127, 'learning_rate': 0.16552737959116803, 'num_leaves': 203, 'max_depth': 5, 'min_child_samples': 4, 'subsample': 0.8546243403416609, 'colsample_bytree': 0.6932937876927856, 'reg_alpha': 8.010291576053802, 'reg_lambda': 8.029252557286224, 'min_split_gain': 0.037485602165442555}. Best is trial 2 with value: 0.0021551760712632314.
[I 2025-06-15 19:50:41,925] A new study created in memory with name: no-name-8f3cf69d-4715-4c65-ae98-1b30677314fe
[I 2025-06-15 19:50:41,975] Trial 0 finished with value: 0.014450310670233406 and parameters: {'n_estimators': 175, 'learning_rate': 0.09752150462361962, 'num_leaves': 67, 'max_depth': 11, 'min_child_samples': 20, 'subsample': 0.730092279174203, 'colsample_bytree': 0.529520659692984, 'reg_alpha': 3.2246341649684416, 'reg_lambda': 0.47563136800190975, 'min_split_gain': 0.028436734241507746}. Best is trial 0 with value: 0.014450310670233406.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         399
1           Price_lag_2         275
14      pct_chg_Load_DA         270
3         Load_DA_lag_0         180
2           Price_lag_7         152
11           NGas_lag_2         130
16  roll2d_median_Price         120
15       lag168_Load_DA         113
5       WindOn_DA_lag_0          93
10           Coal_lag_2          89
12            Oil_lag_2          78
13            EUA_lag_2          75
9                  WD_7           8
7                  WD_1           6
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          18
3         Load_DA_lag_0           2
1           Price_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 15.32% done
********************* END NS.. 111 ****************************************************
********************* START NS ... 112 ****************************************************
[I 2025-06-15 19:50:43,880] A new study created in memory with name: no-name-147c3f1d-a49a-4d1c-a1d4-0489b7de1e20
[I 2025-06-15 19:50:44,694] Trial 0 finished with value: 0.005916242098396775 and parameters: {'n_estimators': 86, 'learning_rate': 0.028645270583747644, 'num_leaves': 233, 'max_depth': 14, 'min_child_samples': 9, 'subsample': 0.853544382483598, 'colsample_bytree': 0.8945488224085594, 'reg_alpha': 1.9707779277604986, 'reg_lambda': 1.7461004240552125, 'min_split_gain': 0.012976415117242047}. Best is trial 0 with value: 0.005916242098396775.
[Trial 0] New best: 0.00592
[I 2025-06-15 19:50:45,086] Trial 1 finished with value: 0.005480329622176323 and parameters: {'n_estimators': 77, 'learning_rate': 0.19694520090397255, 'num_leaves': 229, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.5399684260595885, 'colsample_bytree': 0.9103421201696218, 'reg_alpha': 7.938573709501599, 'reg_lambda': 7.226747109134743, 'min_split_gain': 0.0020901161353975906}. Best is trial 1 with value: 0.005480329622176323.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:45,549] Trial 2 finished with value: 0.00515154749162679 and parameters: {'n_estimators': 224, 'learning_rate': 0.16894964308100796, 'num_leaves': 149, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.9623383842439955, 'colsample_bytree': 0.9376400807045577, 'reg_alpha': 0.7134341131939759, 'reg_lambda': 3.4102854501164837, 'min_split_gain': 0.01239748780399428}. Best is trial 2 with value: 0.00515154749162679.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:46,058] Trial 3 finished with value: 0.005426563144291457 and parameters: {'n_estimators': 237, 'learning_rate': 0.07737905249381767, 'num_leaves': 181, 'max_depth': 13, 'min_child_samples': 1, 'subsample': 0.88528090709585, 'colsample_bytree': 0.9900278516260075, 'reg_alpha': 5.347183169897265, 'reg_lambda': 1.8768520758908658, 'min_split_gain': 0.03651192937975375}. Best is trial 2 with value: 0.00515154749162679.
[I 2025-06-15 19:50:46,060] A new study created in memory with name: no-name-62f36690-b146-45be-aadb-27a38865ca3f
[I 2025-06-15 19:50:46,224] Trial 0 finished with value: 0.04826960819335946 and parameters: {'n_estimators': 272, 'learning_rate': 0.01723059198769078, 'num_leaves': 213, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.577960500552813, 'colsample_bytree': 0.7959954187527569, 'reg_alpha': 9.974227155025343, 'reg_lambda': 3.615317367460463, 'min_split_gain': 0.04128871920648104}. Best is trial 0 with value: 0.04826960819335946.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         633
1           Price_lag_2         451
14      pct_chg_Load_DA         440
3         Load_DA_lag_0         334
2           Price_lag_7         247
16  roll2d_median_Price         236
5       WindOn_DA_lag_0         228
11           NGas_lag_2         183
15       lag168_Load_DA         179
13            EUA_lag_2         163
10           Coal_lag_2         147
12            Oil_lag_2         123
7                  WD_1          10
9                  WD_7           5
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 15.46% done
********************* END NS.. 112 ****************************************************
********************* START NS ... 113 ****************************************************
[I 2025-06-15 19:50:47,853] A new study created in memory with name: no-name-c9242b69-98cc-477c-af45-378ffa4d1ccb
[I 2025-06-15 19:50:48,662] Trial 0 finished with value: 0.012816994625668746 and parameters: {'n_estimators': 234, 'learning_rate': 0.021895917439429084, 'num_leaves': 254, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.5551778459187786, 'colsample_bytree': 0.8809107374383236, 'reg_alpha': 9.672945577212973, 'reg_lambda': 2.2191922647664217, 'min_split_gain': 0.02158641363503293}. Best is trial 0 with value: 0.012816994625668746.
[Trial 0] New best: 0.01282
[I 2025-06-15 19:50:49,045] Trial 1 finished with value: 0.014830661463870914 and parameters: {'n_estimators': 130, 'learning_rate': 0.10343545441252239, 'num_leaves': 230, 'max_depth': 12, 'min_child_samples': 5, 'subsample': 0.8428554142601445, 'colsample_bytree': 0.501763402316104, 'reg_alpha': 4.997474725615573, 'reg_lambda': 8.970352306240134, 'min_split_gain': 0.03570931406016585}. Best is trial 0 with value: 0.012816994625668746.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:49,567] Trial 2 finished with value: 0.0154209268689204 and parameters: {'n_estimators': 122, 'learning_rate': 0.051889072204315975, 'num_leaves': 72, 'max_depth': 15, 'min_child_samples': 25, 'subsample': 0.5391779219013916, 'colsample_bytree': 0.5488258939209635, 'reg_alpha': 6.96812155213963, 'reg_lambda': 3.3819581190079995, 'min_split_gain': 0.0017849482449450893}. Best is trial 0 with value: 0.012816994625668746.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:49,837] Trial 3 finished with value: 0.011036761952963195 and parameters: {'n_estimators': 106, 'learning_rate': 0.215702317979211, 'num_leaves': 182, 'max_depth': 15, 'min_child_samples': 3, 'subsample': 0.6090132987066992, 'colsample_bytree': 0.6304501093290331, 'reg_alpha': 2.378362278452922, 'reg_lambda': 6.652014551986943, 'min_split_gain': 0.03529299264856606}. Best is trial 3 with value: 0.011036761952963195.
[Trial 3] New best: 0.01104
[I 2025-06-15 19:50:50,913] Trial 4 finished with value: 0.01485733618559392 and parameters: {'n_estimators': 229, 'learning_rate': 0.025117946459758848, 'num_leaves': 177, 'max_depth': 15, 'min_child_samples': 5, 'subsample': 0.9682697455897833, 'colsample_bytree': 0.5386165080190168, 'reg_alpha': 4.800509166809083, 'reg_lambda': 8.946886026944407, 'min_split_gain': 0.01835086115803763}. Best is trial 3 with value: 0.011036761952963195.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:51,186] Trial 5 finished with value: 0.010543170258678623 and parameters: {'n_estimators': 138, 'learning_rate': 0.12380349554744323, 'num_leaves': 211, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.8987648813737622, 'colsample_bytree': 0.8970539904071424, 'reg_alpha': 2.1843292651491373, 'reg_lambda': 6.631648047033139, 'min_split_gain': 0.03901904313893773}. Best is trial 5 with value: 0.010543170258678623.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:51,587] Trial 6 finished with value: 0.017615526345136568 and parameters: {'n_estimators': 292, 'learning_rate': 0.04456985965633562, 'num_leaves': 129, 'max_depth': 4, 'min_child_samples': 27, 'subsample': 0.7074475927772674, 'colsample_bytree': 0.5537959481643892, 'reg_alpha': 0.6778002026207075, 'reg_lambda': 6.797206673155981, 'min_split_gain': 0.03124502780858219}. Best is trial 5 with value: 0.010543170258678623.
[I 2025-06-15 19:50:51,590] A new study created in memory with name: no-name-c9fad6f2-431b-4e47-8258-dc794660ce8a
[I 2025-06-15 19:50:51,612] Trial 0 finished with value: 0.0772464713120556 and parameters: {'n_estimators': 139, 'learning_rate': 0.05211694994427687, 'num_leaves': 105, 'max_depth': 11, 'min_child_samples': 14, 'subsample': 0.7647639763462684, 'colsample_bytree': 0.6060542253805162, 'reg_alpha': 8.39294135135066, 'reg_lambda': 3.866864612473698, 'min_split_gain': 0.04551650653749051}. Best is trial 0 with value: 0.0772464713120556.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         498
14      pct_chg_Load_DA         351
1           Price_lag_2         319
3         Load_DA_lag_0         195
11           NGas_lag_2         175
16  roll2d_median_Price         157
15       lag168_Load_DA         147
2           Price_lag_7         142
12            Oil_lag_2          95
5       WindOn_DA_lag_0          90
10           Coal_lag_2          87
13            EUA_lag_2          68
7                  WD_1           7
9                  WD_7           5
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           5
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 15.60% done
********************* END NS.. 113 ****************************************************
********************* START NS ... 114 ****************************************************
[I 2025-06-15 19:50:52,431] A new study created in memory with name: no-name-8c40d6fd-6561-43be-99bf-80fef100c49b
[I 2025-06-15 19:50:52,885] Trial 0 finished with value: 0.008515675885445552 and parameters: {'n_estimators': 228, 'learning_rate': 0.06622468160204442, 'num_leaves': 118, 'max_depth': 10, 'min_child_samples': 22, 'subsample': 0.6022697562358763, 'colsample_bytree': 0.7968669356080755, 'reg_alpha': 2.163350992618116, 'reg_lambda': 2.0219647046936062, 'min_split_gain': 0.0234888120889353}. Best is trial 0 with value: 0.008515675885445552.
[Trial 0] New best: 0.00852
[I 2025-06-15 19:50:53,320] Trial 1 finished with value: 0.010192910590805845 and parameters: {'n_estimators': 225, 'learning_rate': 0.05144509888053948, 'num_leaves': 132, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.7639793146762166, 'colsample_bytree': 0.8102976000802993, 'reg_alpha': 6.740868147264648, 'reg_lambda': 1.9010108471099463, 'min_split_gain': 0.03525009534226099}. Best is trial 0 with value: 0.008515675885445552.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:53,983] Trial 2 finished with value: 0.007815925541634702 and parameters: {'n_estimators': 248, 'learning_rate': 0.07013520320725947, 'num_leaves': 199, 'max_depth': 14, 'min_child_samples': 14, 'subsample': 0.9493537448572271, 'colsample_bytree': 0.7368979529206917, 'reg_alpha': 0.3449103047783608, 'reg_lambda': 6.123483433519058, 'min_split_gain': 0.037934705139544195}. Best is trial 2 with value: 0.007815925541634702.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:54,910] Trial 3 finished with value: 0.011003027936516313 and parameters: {'n_estimators': 121, 'learning_rate': 0.011105972511711339, 'num_leaves': 97, 'max_depth': 14, 'min_child_samples': 9, 'subsample': 0.9641761221362639, 'colsample_bytree': 0.8656414343171823, 'reg_alpha': 0.15731619784527529, 'reg_lambda': 2.0360325759592457, 'min_split_gain': 0.04648549037493472}. Best is trial 2 with value: 0.007815925541634702.
[I 2025-06-15 19:50:54,913] A new study created in memory with name: no-name-a55f6985-b3b9-43fc-94ca-7e764f4f827d
[I 2025-06-15 19:50:54,935] Trial 0 finished with value: 0.04400466915610026 and parameters: {'n_estimators': 152, 'learning_rate': 0.20793762855407436, 'num_leaves': 138, 'max_depth': 13, 'min_child_samples': 16, 'subsample': 0.7614493174317463, 'colsample_bytree': 0.8587960041538456, 'reg_alpha': 3.9202819537264744, 'reg_lambda': 4.211080669749208, 'min_split_gain': 0.0485104774038303}. Best is trial 0 with value: 0.04400466915610026.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1219
14      pct_chg_Load_DA        1161
0           Price_lag_1        1119
3         Load_DA_lag_0         912
15       lag168_Load_DA         666
2           Price_lag_7         578
11           NGas_lag_2         534
16  roll2d_median_Price         505
5       WindOn_DA_lag_0         433
10           Coal_lag_2         386
13            EUA_lag_2         297
12            Oil_lag_2         295
9                  WD_7          17
7                  WD_1          15
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          11
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 15.73% done
********************* END NS.. 114 ****************************************************
********************* START NS ... 115 ****************************************************
[I 2025-06-15 19:50:55,961] A new study created in memory with name: no-name-7d01baed-6172-4b92-aa59-b4ccc915b85a
[I 2025-06-15 19:50:56,183] Trial 0 finished with value: 0.0043409889889769035 and parameters: {'n_estimators': 72, 'learning_rate': 0.06321257671637488, 'num_leaves': 244, 'max_depth': 12, 'min_child_samples': 1, 'subsample': 0.8520313929462231, 'colsample_bytree': 0.6994035862526597, 'reg_alpha': 8.621236097392313, 'reg_lambda': 0.8964430183348449, 'min_split_gain': 0.01737505166423265}. Best is trial 0 with value: 0.0043409889889769035.
[Trial 0] New best: 0.00434
[I 2025-06-15 19:50:56,557] Trial 1 finished with value: 0.007251502574425212 and parameters: {'n_estimators': 52, 'learning_rate': 0.028478237002111443, 'num_leaves': 168, 'max_depth': 13, 'min_child_samples': 12, 'subsample': 0.5579351432615529, 'colsample_bytree': 0.5191679723936875, 'reg_alpha': 0.9522196181976439, 'reg_lambda': 0.03285199546019468, 'min_split_gain': 0.0018970460668169876}. Best is trial 0 with value: 0.0043409889889769035.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:50:56,966] Trial 2 finished with value: 0.003916966902273433 and parameters: {'n_estimators': 123, 'learning_rate': 0.027833653982327, 'num_leaves': 202, 'max_depth': 7, 'min_child_samples': 23, 'subsample': 0.7291683050903753, 'colsample_bytree': 0.9841480501420059, 'reg_alpha': 7.221325558216488, 'reg_lambda': 5.218785381502064, 'min_split_gain': 0.031630935294978096}. Best is trial 2 with value: 0.003916966902273433.
[I 2025-06-15 19:50:57,121] Trial 3 finished with value: 0.003313383440232095 and parameters: {'n_estimators': 113, 'learning_rate': 0.2507075298303831, 'num_leaves': 72, 'max_depth': 8, 'min_child_samples': 14, 'subsample': 0.9200014363176026, 'colsample_bytree': 0.7264890979978516, 'reg_alpha': 3.352633530435498, 'reg_lambda': 1.6259946391613267, 'min_split_gain': 0.035660255610333}. Best is trial 3 with value: 0.003313383440232095.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] New best: 0.00331
[I 2025-06-15 19:50:57,409] Trial 4 finished with value: 0.004268208361423863 and parameters: {'n_estimators': 113, 'learning_rate': 0.03339660745327004, 'num_leaves': 201, 'max_depth': 5, 'min_child_samples': 19, 'subsample': 0.9161040350766518, 'colsample_bytree': 0.8257507758879878, 'reg_alpha': 2.3318844558864136, 'reg_lambda': 9.895682182122199, 'min_split_gain': 0.0405168527722397}. Best is trial 3 with value: 0.003313383440232095.
[I 2025-06-15 19:50:57,625] Trial 5 finished with value: 0.00360641786142922 and parameters: {'n_estimators': 194, 'learning_rate': 0.16210946657388253, 'num_leaves': 144, 'max_depth': 8, 'min_child_samples': 10, 'subsample': 0.8446516284166606, 'colsample_bytree': 0.8590470961973391, 'reg_alpha': 7.28361799294937, 'reg_lambda': 5.12744325982019, 'min_split_gain': 0.02728994415817404}. Best is trial 3 with value: 0.003313383440232095.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:50:58,148] Trial 6 finished with value: 0.003494232882396647 and parameters: {'n_estimators': 166, 'learning_rate': 0.05086059171344753, 'num_leaves': 252, 'max_depth': 6, 'min_child_samples': 8, 'subsample': 0.9057208897462077, 'colsample_bytree': 0.8062547601224832, 'reg_alpha': 0.6007763695001789, 'reg_lambda': 8.653698644135533, 'min_split_gain': 0.010129672054650807}. Best is trial 3 with value: 0.003313383440232095.
[I 2025-06-15 19:50:58,151] A new study created in memory with name: no-name-06ae59f0-8819-4df1-8217-2acd1abd2940
[I 2025-06-15 19:50:58,178] Trial 0 finished with value: 0.01205209904315852 and parameters: {'n_estimators': 148, 'learning_rate': 0.09574802370533694, 'num_leaves': 197, 'max_depth': 8, 'min_child_samples': 5, 'subsample': 0.7629525316686229, 'colsample_bytree': 0.8283436996517115, 'reg_alpha': 2.3120524267995424, 'reg_lambda': 0.6978537222042092, 'min_split_gain': 0.03165450825018379}. Best is trial 0 with value: 0.01205209904315852.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         228
1           Price_lag_2         128
14      pct_chg_Load_DA         114
3         Load_DA_lag_0          88
11           NGas_lag_2          69
2           Price_lag_7          63
16  roll2d_median_Price          53
5       WindOn_DA_lag_0          48
15       lag168_Load_DA          43
10           Coal_lag_2          36
13            EUA_lag_2          35
12            Oil_lag_2          27
7                  WD_1           2
9                  WD_7           1
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          34
1           Price_lag_2           2
2           Price_lag_7           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
16  roll2d_median_Price           0
-> 15.87% done
********************* END NS.. 115 ****************************************************
********************* START NS ... 116 ****************************************************
[I 2025-06-15 19:50:58,945] A new study created in memory with name: no-name-20ae12f6-f01e-4dfe-9adb-20913bdaa95f
[I 2025-06-15 19:50:59,167] Trial 0 finished with value: 0.0019275881847853132 and parameters: {'n_estimators': 124, 'learning_rate': 0.07888935756621192, 'num_leaves': 131, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.5959540709412054, 'colsample_bytree': 0.8485941729929415, 'reg_alpha': 7.500383113999244, 'reg_lambda': 0.5298367173474616, 'min_split_gain': 0.048655187661681115}. Best is trial 0 with value: 0.0019275881847853132.
[Trial 0] New best: 0.00193
[I 2025-06-15 19:50:59,518] Trial 1 finished with value: 0.001912577751750531 and parameters: {'n_estimators': 273, 'learning_rate': 0.06955504555664582, 'num_leaves': 206, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.7588740244552756, 'colsample_bytree': 0.672584401796156, 'reg_alpha': 6.959340477181927, 'reg_lambda': 9.7526147047532, 'min_split_gain': 0.04981981910254768}. Best is trial 1 with value: 0.001912577751750531.
[I 2025-06-15 19:50:59,650] Trial 2 finished with value: 0.0019069114375312066 and parameters: {'n_estimators': 52, 'learning_rate': 0.29031188896576593, 'num_leaves': 83, 'max_depth': 9, 'min_child_samples': 21, 'subsample': 0.7659099707193073, 'colsample_bytree': 0.7713685647476696, 'reg_alpha': 6.271875890054124, 'reg_lambda': 9.40545234245474, 'min_split_gain': 0.02227779774226378}. Best is trial 2 with value: 0.0019069114375312066.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:00,695] Trial 3 finished with value: 0.0016668780044021194 and parameters: {'n_estimators': 290, 'learning_rate': 0.0450160737020859, 'num_leaves': 177, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.8645148883854208, 'colsample_bytree': 0.8163030983796153, 'reg_alpha': 5.883368140700026, 'reg_lambda': 6.027223982958599, 'min_split_gain': 0.004265699913135773}. Best is trial 3 with value: 0.0016668780044021194.
[I 2025-06-15 19:51:00,698] A new study created in memory with name: no-name-50843db7-febb-4297-a7b2-8cadc8b58dc5
[I 2025-06-15 19:51:00,716] Trial 0 finished with value: 0.007685136150246935 and parameters: {'n_estimators': 77, 'learning_rate': 0.09062623216049233, 'num_leaves': 245, 'max_depth': 11, 'min_child_samples': 25, 'subsample': 0.807271824984499, 'colsample_bytree': 0.9785854059739045, 'reg_alpha': 5.100519872880795, 'reg_lambda': 1.0182722907589281, 'min_split_gain': 0.0493887266746144}. Best is trial 0 with value: 0.007685136150246935.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1664
1           Price_lag_2        1485
14      pct_chg_Load_DA        1226
3         Load_DA_lag_0         953
5       WindOn_DA_lag_0         887
13            EUA_lag_2         707
2           Price_lag_7         674
15       lag168_Load_DA         633
11           NGas_lag_2         612
10           Coal_lag_2         551
12            Oil_lag_2         525
16  roll2d_median_Price         464
9                  WD_7          43
7                  WD_1          24
8                  WD_6          18
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          14
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 16.01% done
********************* END NS.. 116 ****************************************************
********************* START NS ... 117 ****************************************************
[I 2025-06-15 19:51:01,944] A new study created in memory with name: no-name-6a8a7a8c-2055-431e-b8f3-5322d4f8e574
[I 2025-06-15 19:51:02,082] Trial 0 finished with value: 0.014190956538709255 and parameters: {'n_estimators': 57, 'learning_rate': 0.01753443960371227, 'num_leaves': 79, 'max_depth': 5, 'min_child_samples': 22, 'subsample': 0.6442762977843202, 'colsample_bytree': 0.7497471921914715, 'reg_alpha': 5.3633461058942435, 'reg_lambda': 8.31811885904527, 'min_split_gain': 0.025491453743175924}. Best is trial 0 with value: 0.014190956538709255.
[Trial 0] New best: 0.01419
[I 2025-06-15 19:51:02,302] Trial 1 finished with value: 0.008913122849678014 and parameters: {'n_estimators': 236, 'learning_rate': 0.16986318494058386, 'num_leaves': 35, 'max_depth': 15, 'min_child_samples': 3, 'subsample': 0.966086631337308, 'colsample_bytree': 0.7539049618813078, 'reg_alpha': 6.4360192018399225, 'reg_lambda': 5.043111314158679, 'min_split_gain': 0.04574472574969235}. Best is trial 1 with value: 0.008913122849678014.
[Trial 1] New best: 0.00891
[I 2025-06-15 19:51:02,612] Trial 2 finished with value: 0.009785987709559192 and parameters: {'n_estimators': 61, 'learning_rate': 0.06169933088420527, 'num_leaves': 109, 'max_depth': 8, 'min_child_samples': 10, 'subsample': 0.9408618580614477, 'colsample_bytree': 0.6931038366418648, 'reg_alpha': 1.0754518082477171, 'reg_lambda': 4.195192667434913, 'min_split_gain': 0.043822258182312626}. Best is trial 1 with value: 0.008913122849678014.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:02,910] Trial 3 finished with value: 0.009717760936769126 and parameters: {'n_estimators': 94, 'learning_rate': 0.029292281155879077, 'num_leaves': 203, 'max_depth': 5, 'min_child_samples': 25, 'subsample': 0.8614123808840342, 'colsample_bytree': 0.8232532827795547, 'reg_alpha': 2.2516147747386217, 'reg_lambda': 5.293275731755632, 'min_split_gain': 0.008716834295290982}. Best is trial 1 with value: 0.008913122849678014.
[I 2025-06-15 19:51:03,076] Trial 4 finished with value: 0.008973787421701697 and parameters: {'n_estimators': 52, 'learning_rate': 0.2890101564647277, 'num_leaves': 105, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.6522667811935123, 'colsample_bytree': 0.5643690382214176, 'reg_alpha': 0.7468586036809133, 'reg_lambda': 2.4309682256424656, 'min_split_gain': 0.040179137387754374}. Best is trial 1 with value: 0.008913122849678014.
[I 2025-06-15 19:51:03,078] A new study created in memory with name: no-name-12c1b5b0-0b82-43a2-a40d-5b96e65287fb
[I 2025-06-15 19:51:03,107] Trial 0 finished with value: 0.025231765094814108 and parameters: {'n_estimators': 79, 'learning_rate': 0.02527084997420945, 'num_leaves': 33, 'max_depth': 11, 'min_child_samples': 5, 'subsample': 0.9213750340001843, 'colsample_bytree': 0.5125053081908272, 'reg_alpha': 3.0486538706678443, 'reg_lambda': 7.4773933169791515, 'min_split_gain': 0.002178322939766725}. Best is trial 0 with value: 0.025231765094814108.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         251
14      pct_chg_Load_DA         152
1           Price_lag_2         151
3         Load_DA_lag_0          85
11           NGas_lag_2          81
2           Price_lag_7          79
16  roll2d_median_Price          74
10           Coal_lag_2          47
12            Oil_lag_2          47
15       lag168_Load_DA          47
13            EUA_lag_2          38
5       WindOn_DA_lag_0          30
9                  WD_7           5
8                  WD_6           3
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         115
1           Price_lag_2          53
3         Load_DA_lag_0          49
11           NGas_lag_2          12
10           Coal_lag_2          12
12            Oil_lag_2           4
13            EUA_lag_2           3
16  roll2d_median_Price           1
7                  WD_1           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 16.14% done
********************* END NS.. 117 ****************************************************
********************* START NS ... 118 ****************************************************
[I 2025-06-15 19:51:03,824] A new study created in memory with name: no-name-f9a1d52e-4496-4d1a-ba03-14c381a9abb8
[I 2025-06-15 19:51:04,152] Trial 0 finished with value: 0.010339234490466085 and parameters: {'n_estimators': 201, 'learning_rate': 0.08033414680692973, 'num_leaves': 39, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.7130570389719385, 'colsample_bytree': 0.6739758996939806, 'reg_alpha': 6.226387954032554, 'reg_lambda': 4.582970173498808, 'min_split_gain': 0.015874061915377553}. Best is trial 0 with value: 0.010339234490466085.
[I 2025-06-15 19:51:04,309] Trial 1 finished with value: 0.01313303844135201 and parameters: {'n_estimators': 54, 'learning_rate': 0.04886701203307499, 'num_leaves': 246, 'max_depth': 5, 'min_child_samples': 11, 'subsample': 0.68610420696592, 'colsample_bytree': 0.5271534550095291, 'reg_alpha': 3.010043412516711, 'reg_lambda': 9.90546226748866, 'min_split_gain': 0.013164115785708292}. Best is trial 0 with value: 0.010339234490466085.
[Trial 0] New best: 0.01034
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:05,088] Trial 2 finished with value: 0.009643185849438777 and parameters: {'n_estimators': 286, 'learning_rate': 0.024049470749705943, 'num_leaves': 221, 'max_depth': 6, 'min_child_samples': 7, 'subsample': 0.91493815715828, 'colsample_bytree': 0.960450468342116, 'reg_alpha': 6.873140627341283, 'reg_lambda': 6.256890609885243, 'min_split_gain': 0.014212394138669055}. Best is trial 2 with value: 0.009643185849438777.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:06,344] Trial 3 finished with value: 0.009818324316014206 and parameters: {'n_estimators': 297, 'learning_rate': 0.017583569333322537, 'num_leaves': 110, 'max_depth': 8, 'min_child_samples': 13, 'subsample': 0.7235692017631721, 'colsample_bytree': 0.8752645779883992, 'reg_alpha': 3.916252564903048, 'reg_lambda': 0.25309162650142936, 'min_split_gain': 0.005114676487806247}. Best is trial 2 with value: 0.009643185849438777.
[I 2025-06-15 19:51:06,346] A new study created in memory with name: no-name-e369ba8d-f973-4e89-8854-e4cefaa2dc35
[I 2025-06-15 19:51:06,374] Trial 0 finished with value: 0.0442776372472401 and parameters: {'n_estimators': 167, 'learning_rate': 0.050060142362989606, 'num_leaves': 186, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.6470204836275972, 'colsample_bytree': 0.563302936589092, 'reg_alpha': 7.152994277356418, 'reg_lambda': 0.19621598178512167, 'min_split_gain': 0.02660815770414729}. Best is trial 0 with value: 0.0442776372472401.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2108
14      pct_chg_Load_DA        1189
1           Price_lag_2        1161
3         Load_DA_lag_0         553
11           NGas_lag_2         545
2           Price_lag_7         539
16  roll2d_median_Price         503
13            EUA_lag_2         413
10           Coal_lag_2         339
12            Oil_lag_2         318
5       WindOn_DA_lag_0         270
15       lag168_Load_DA         264
8                  WD_6          14
9                  WD_7          11
7                  WD_1           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          17
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 16.28% done
********************* END NS.. 118 ****************************************************
********************* START NS ... 119 ****************************************************
[I 2025-06-15 19:51:07,704] A new study created in memory with name: no-name-ef1c3a99-3dde-487c-9f83-d452d67a9b76
[I 2025-06-15 19:51:07,880] Trial 0 finished with value: 0.0027453155741234865 and parameters: {'n_estimators': 138, 'learning_rate': 0.17677030554499898, 'num_leaves': 66, 'max_depth': 12, 'min_child_samples': 25, 'subsample': 0.6334194848124146, 'colsample_bytree': 0.6578288592349502, 'reg_alpha': 8.355681004179324, 'reg_lambda': 0.19753293444132125, 'min_split_gain': 0.01590261685507718}. Best is trial 0 with value: 0.0027453155741234865.
[I 2025-06-15 19:51:08,040] Trial 1 finished with value: 0.0027528441019808547 and parameters: {'n_estimators': 54, 'learning_rate': 0.1619999132893901, 'num_leaves': 220, 'max_depth': 6, 'min_child_samples': 7, 'subsample': 0.9151761049523065, 'colsample_bytree': 0.6949890755739712, 'reg_alpha': 0.02376753600506598, 'reg_lambda': 5.808603289446244, 'min_split_gain': 0.03879158368943116}. Best is trial 0 with value: 0.0027453155741234865.
[Trial 0] New best: 0.00275
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:08,619] Trial 2 finished with value: 0.0031141565242135314 and parameters: {'n_estimators': 197, 'learning_rate': 0.012662910259816895, 'num_leaves': 199, 'max_depth': 10, 'min_child_samples': 11, 'subsample': 0.9198361621024858, 'colsample_bytree': 0.9444585701475869, 'reg_alpha': 8.301225341837181, 'reg_lambda': 8.827737686184893, 'min_split_gain': 0.026233317776469964}. Best is trial 0 with value: 0.0027453155741234865.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:09,273] Trial 3 finished with value: 0.0028384196982609374 and parameters: {'n_estimators': 216, 'learning_rate': 0.016271058550233045, 'num_leaves': 208, 'max_depth': 6, 'min_child_samples': 14, 'subsample': 0.806003968720439, 'colsample_bytree': 0.8140281891365232, 'reg_alpha': 4.992188776886296, 'reg_lambda': 3.9817929803602725, 'min_split_gain': 0.02465491981452137}. Best is trial 0 with value: 0.0027453155741234865.
[I 2025-06-15 19:51:09,275] A new study created in memory with name: no-name-54085506-229d-4e3c-819d-2b9ce888d375
[I 2025-06-15 19:51:09,307] Trial 0 finished with value: 0.018736505425584995 and parameters: {'n_estimators': 156, 'learning_rate': 0.07676988486353276, 'num_leaves': 228, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.7384756653760229, 'colsample_bytree': 0.9750785552260315, 'reg_alpha': 7.34741560216384, 'reg_lambda': 4.179017564391286, 'min_split_gain': 0.03520456896352946}. Best is trial 0 with value: 0.018736505425584995.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         275
1           Price_lag_2         159
3         Load_DA_lag_0         118
14      pct_chg_Load_DA         101
13            EUA_lag_2          91
11           NGas_lag_2          91
2           Price_lag_7          63
16  roll2d_median_Price          58
5       WindOn_DA_lag_0          55
10           Coal_lag_2          52
12            Oil_lag_2          47
15       lag168_Load_DA          45
9                  WD_7           3
8                  WD_6           2
7                  WD_1           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 16.42% done
********************* END NS.. 119 ****************************************************
********************* START NS ... 120 ****************************************************
[I 2025-06-15 19:51:10,012] A new study created in memory with name: no-name-7c5d496d-e949-4cd4-81e3-330e7dad1211
[I 2025-06-15 19:51:10,339] Trial 0 finished with value: 0.009757307434139644 and parameters: {'n_estimators': 145, 'learning_rate': 0.06244885851494458, 'num_leaves': 92, 'max_depth': 14, 'min_child_samples': 12, 'subsample': 0.6674818743867805, 'colsample_bytree': 0.6194585254528908, 'reg_alpha': 3.6777476209016315, 'reg_lambda': 0.7967940875082213, 'min_split_gain': 0.047796939040731984}. Best is trial 0 with value: 0.009757307434139644.
[Trial 0] New best: 0.00976
[I 2025-06-15 19:51:10,611] Trial 1 finished with value: 0.00937540833670051 and parameters: {'n_estimators': 159, 'learning_rate': 0.027225848639889406, 'num_leaves': 60, 'max_depth': 4, 'min_child_samples': 13, 'subsample': 0.6476315318650542, 'colsample_bytree': 0.9799849397648981, 'reg_alpha': 2.495462124726149, 'reg_lambda': 2.863168127888609, 'min_split_gain': 0.047319650142506914}. Best is trial 1 with value: 0.00937540833670051.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:11,165] Trial 2 finished with value: 0.009462649229364288 and parameters: {'n_estimators': 124, 'learning_rate': 0.06561937792256516, 'num_leaves': 172, 'max_depth': 7, 'min_child_samples': 20, 'subsample': 0.6580742474136614, 'colsample_bytree': 0.7688518893374638, 'reg_alpha': 0.8408784788506285, 'reg_lambda': 7.690429488268653, 'min_split_gain': 0.02121662839279618}. Best is trial 1 with value: 0.00937540833670051.
[I 2025-06-15 19:51:11,367] Trial 3 finished with value: 0.009840028578753641 and parameters: {'n_estimators': 79, 'learning_rate': 0.11633110091726835, 'num_leaves': 59, 'max_depth': 6, 'min_child_samples': 7, 'subsample': 0.9603724710743875, 'colsample_bytree': 0.923387344202254, 'reg_alpha': 8.644064636829413, 'reg_lambda': 2.3634628085473586, 'min_split_gain': 0.04693279098671403}. Best is trial 1 with value: 0.00937540833670051.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:11,369] A new study created in memory with name: no-name-d1a6e008-389e-46eb-bf5b-72c5c4bad232
[I 2025-06-15 19:51:11,387] Trial 0 finished with value: 0.040083155420519705 and parameters: {'n_estimators': 69, 'learning_rate': 0.010296617749966045, 'num_leaves': 237, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.5874781402693449, 'colsample_bytree': 0.5828946694620669, 'reg_alpha': 8.210399133340015, 'reg_lambda': 4.176326145912461, 'min_split_gain': 0.005368799997412965}. Best is trial 0 with value: 0.040083155420519705.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1252
14      pct_chg_Load_DA         420
16  roll2d_median_Price         163
2           Price_lag_7         112
1           Price_lag_2         102
11           NGas_lag_2          96
3         Load_DA_lag_0          69
13            EUA_lag_2          42
15       lag168_Load_DA          38
12            Oil_lag_2          29
5       WindOn_DA_lag_0          11
10           Coal_lag_2           9
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          17
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 16.55% done
********************* END NS.. 120 ****************************************************
********************* START NS ... 121 ****************************************************
[I 2025-06-15 19:51:12,175] A new study created in memory with name: no-name-3af2ade1-8682-474d-9ea9-63b43c3d94a0
[I 2025-06-15 19:51:12,358] Trial 0 finished with value: 0.010345647561725431 and parameters: {'n_estimators': 128, 'learning_rate': 0.01211450716582934, 'num_leaves': 62, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.6309028933661052, 'colsample_bytree': 0.8530833649622165, 'reg_alpha': 1.3976654848277037, 'reg_lambda': 5.016757375903281, 'min_split_gain': 0.03917418747774787}. Best is trial 0 with value: 0.010345647561725431.
[Trial 0] New best: 0.01035
[I 2025-06-15 19:51:13,030] Trial 1 finished with value: 0.010233009982793811 and parameters: {'n_estimators': 181, 'learning_rate': 0.02153630448230001, 'num_leaves': 214, 'max_depth': 13, 'min_child_samples': 4, 'subsample': 0.5327180428820151, 'colsample_bytree': 0.7148629893326814, 'reg_alpha': 6.905801055714377, 'reg_lambda': 4.107306623642481, 'min_split_gain': 0.02522588413618337}. Best is trial 1 with value: 0.010233009982793811.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:13,700] Trial 2 finished with value: 0.009859896895019133 and parameters: {'n_estimators': 227, 'learning_rate': 0.028988867830244576, 'num_leaves': 188, 'max_depth': 6, 'min_child_samples': 13, 'subsample': 0.6009892532725384, 'colsample_bytree': 0.8519489901922086, 'reg_alpha': 2.3203092594181687, 'reg_lambda': 3.830296441093771, 'min_split_gain': 0.04572586526641009}. Best is trial 2 with value: 0.009859896895019133.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:14,278] Trial 3 finished with value: 0.010160100196889293 and parameters: {'n_estimators': 212, 'learning_rate': 0.08874443051324668, 'num_leaves': 189, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.6420695390715334, 'colsample_bytree': 0.5138610682589507, 'reg_alpha': 0.02978930197860441, 'reg_lambda': 1.5564272631703502, 'min_split_gain': 0.03443479976980589}. Best is trial 2 with value: 0.009859896895019133.
[I 2025-06-15 19:51:14,280] A new study created in memory with name: no-name-08326309-f23e-4074-98d3-13eb66d6af73
[I 2025-06-15 19:51:14,323] Trial 0 finished with value: 0.03424580053110703 and parameters: {'n_estimators': 207, 'learning_rate': 0.010970658693172946, 'num_leaves': 118, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.9696482226487579, 'colsample_bytree': 0.5630264731321541, 'reg_alpha': 2.6454774715743037, 'reg_lambda': 6.991314734911018, 'min_split_gain': 0.013912317803529495}. Best is trial 0 with value: 0.03424580053110703.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2017
1           Price_lag_2        1289
14      pct_chg_Load_DA        1069
3         Load_DA_lag_0         633
11           NGas_lag_2         504
2           Price_lag_7         451
16  roll2d_median_Price         432
15       lag168_Load_DA         280
13            EUA_lag_2         240
12            Oil_lag_2         197
10           Coal_lag_2         187
5       WindOn_DA_lag_0         177
7                  WD_1          20
9                  WD_7           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         216
1           Price_lag_2          88
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 16.69% done
********************* END NS.. 121 ****************************************************
********************* START NS ... 122 ****************************************************
[I 2025-06-15 19:51:15,387] A new study created in memory with name: no-name-d1b4f53a-c99c-41c0-8952-3558a451d947
[I 2025-06-15 19:51:15,714] Trial 0 finished with value: 0.0048783149316680785 and parameters: {'n_estimators': 166, 'learning_rate': 0.11966988046598437, 'num_leaves': 202, 'max_depth': 14, 'min_child_samples': 15, 'subsample': 0.76572542123748, 'colsample_bytree': 0.8626705408890494, 'reg_alpha': 6.644159666955317, 'reg_lambda': 2.5883401967271027, 'min_split_gain': 0.0024121054121870877}. Best is trial 0 with value: 0.0048783149316680785.
[Trial 0] New best: 0.00488
[I 2025-06-15 19:51:15,982] Trial 1 finished with value: 0.00551114744072535 and parameters: {'n_estimators': 139, 'learning_rate': 0.27811062200837455, 'num_leaves': 256, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.9314829555784532, 'colsample_bytree': 0.5687250816123743, 'reg_alpha': 0.177841209745645, 'reg_lambda': 0.6930501162581904, 'min_split_gain': 0.015925158620978753}. Best is trial 0 with value: 0.0048783149316680785.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:16,585] Trial 2 finished with value: 0.008250109344544121 and parameters: {'n_estimators': 199, 'learning_rate': 0.010958509627466148, 'num_leaves': 245, 'max_depth': 10, 'min_child_samples': 11, 'subsample': 0.5096665973749981, 'colsample_bytree': 0.7406554476012064, 'reg_alpha': 9.928062463515953, 'reg_lambda': 5.634039999213577, 'min_split_gain': 0.021584698705036078}. Best is trial 0 with value: 0.0048783149316680785.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:17,313] Trial 3 finished with value: 0.0063593511257314635 and parameters: {'n_estimators': 231, 'learning_rate': 0.01180798598594658, 'num_leaves': 158, 'max_depth': 8, 'min_child_samples': 5, 'subsample': 0.8352024286040831, 'colsample_bytree': 0.7890942360150035, 'reg_alpha': 9.486098916330448, 'reg_lambda': 3.198924498622071, 'min_split_gain': 0.016575035483625124}. Best is trial 0 with value: 0.0048783149316680785.
[I 2025-06-15 19:51:17,315] A new study created in memory with name: no-name-9fca8928-ac28-4855-90f3-50a85531557f
[I 2025-06-15 19:51:17,336] Trial 0 finished with value: 0.02637983917143907 and parameters: {'n_estimators': 105, 'learning_rate': 0.14217842803055553, 'num_leaves': 55, 'max_depth': 11, 'min_child_samples': 29, 'subsample': 0.646993487508228, 'colsample_bytree': 0.528909979011152, 'reg_alpha': 8.729938342855249, 'reg_lambda': 4.421782221946719, 'min_split_gain': 0.045184244579660915}. Best is trial 0 with value: 0.02637983917143907.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         634
1           Price_lag_2         492
14      pct_chg_Load_DA         453
3         Load_DA_lag_0         367
5       WindOn_DA_lag_0         327
2           Price_lag_7         266
13            EUA_lag_2         262
15       lag168_Load_DA         248
11           NGas_lag_2         228
10           Coal_lag_2         215
12            Oil_lag_2         210
16  roll2d_median_Price         194
9                  WD_7          13
7                  WD_1          12
8                  WD_6          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 16.83% done
********************* END NS.. 122 ****************************************************
********************* START NS ... 123 ****************************************************
[I 2025-06-15 19:51:18,191] A new study created in memory with name: no-name-10ae59eb-13b0-4610-be69-23760ea384c0
[I 2025-06-15 19:51:18,632] Trial 0 finished with value: 0.00778304330227603 and parameters: {'n_estimators': 189, 'learning_rate': 0.018447076567213748, 'num_leaves': 245, 'max_depth': 9, 'min_child_samples': 7, 'subsample': 0.6188335456828323, 'colsample_bytree': 0.5039920856580465, 'reg_alpha': 8.768775709390432, 'reg_lambda': 7.795916737105606, 'min_split_gain': 0.04215224723540034}. Best is trial 0 with value: 0.00778304330227603.
[I 2025-06-15 19:51:18,818] Trial 1 finished with value: 0.005243383761955762 and parameters: {'n_estimators': 266, 'learning_rate': 0.29821296569842753, 'num_leaves': 144, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.9543468426242514, 'colsample_bytree': 0.8450657425491062, 'reg_alpha': 4.509775680674121, 'reg_lambda': 0.44143174956099984, 'min_split_gain': 0.03626444803288038}. Best is trial 1 with value: 0.005243383761955762.
[Trial 0] New best: 0.00778
[Trial 1] New best: 0.00524
[I 2025-06-15 19:51:19,366] Trial 2 finished with value: 0.005998157486644579 and parameters: {'n_estimators': 249, 'learning_rate': 0.05316255224406441, 'num_leaves': 36, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.7116554475485986, 'colsample_bytree': 0.5351890944690982, 'reg_alpha': 6.640899084416341, 'reg_lambda': 0.5932869754899694, 'min_split_gain': 0.04050807011830308}. Best is trial 1 with value: 0.005243383761955762.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:19,595] Trial 3 finished with value: 0.006262658827252907 and parameters: {'n_estimators': 65, 'learning_rate': 0.0540326377568499, 'num_leaves': 230, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.6996861706712126, 'colsample_bytree': 0.6053474863594381, 'reg_alpha': 0.9379688434693645, 'reg_lambda': 7.842883029411814, 'min_split_gain': 0.004222210329684556}. Best is trial 1 with value: 0.005243383761955762.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:19,896] Trial 4 finished with value: 0.0044613040968275244 and parameters: {'n_estimators': 208, 'learning_rate': 0.13218278898296212, 'num_leaves': 40, 'max_depth': 7, 'min_child_samples': 20, 'subsample': 0.8774736405049404, 'colsample_bytree': 0.6582454840063905, 'reg_alpha': 1.8500334373305105, 'reg_lambda': 6.6737754186874545, 'min_split_gain': 0.048302315626294466}. Best is trial 4 with value: 0.0044613040968275244.
[I 2025-06-15 19:51:19,899] A new study created in memory with name: no-name-b516b166-3639-4c49-bbaf-f795a3626678
[I 2025-06-15 19:51:19,919] Trial 0 finished with value: 0.02647929917598552 and parameters: {'n_estimators': 104, 'learning_rate': 0.04299977420143164, 'num_leaves': 48, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.7506934317217011, 'colsample_bytree': 0.9971001522805697, 'reg_alpha': 8.133591711942296, 'reg_lambda': 7.479139828589945, 'min_split_gain': 0.03951705851008174}. Best is trial 0 with value: 0.02647929917598552.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         412
1           Price_lag_2         296
14      pct_chg_Load_DA         220
3         Load_DA_lag_0         172
11           NGas_lag_2         116
16  roll2d_median_Price         109
2           Price_lag_7         104
15       lag168_Load_DA          94
10           Coal_lag_2          73
13            EUA_lag_2          73
5       WindOn_DA_lag_0          55
12            Oil_lag_2          47
7                  WD_1           8
9                  WD_7           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          14
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 16.96% done
********************* END NS.. 123 ****************************************************
********************* START NS ... 124 ****************************************************
[I 2025-06-15 19:51:20,670] A new study created in memory with name: no-name-7db3ebf7-7417-4a91-a8ec-35f4a76cc7a7
[I 2025-06-15 19:51:20,967] Trial 0 finished with value: 0.006622613281743783 and parameters: {'n_estimators': 175, 'learning_rate': 0.1260231569533006, 'num_leaves': 241, 'max_depth': 8, 'min_child_samples': 28, 'subsample': 0.9811830786510501, 'colsample_bytree': 0.5310719759823492, 'reg_alpha': 8.045957249128175, 'reg_lambda': 2.4296263035587975, 'min_split_gain': 0.027806626900088272}. Best is trial 0 with value: 0.006622613281743783.
[Trial 0] New best: 0.00662
[I 2025-06-15 19:51:21,537] Trial 1 finished with value: 0.006355111158160488 and parameters: {'n_estimators': 163, 'learning_rate': 0.027139267727313517, 'num_leaves': 173, 'max_depth': 7, 'min_child_samples': 13, 'subsample': 0.5137850298056186, 'colsample_bytree': 0.7030601758412892, 'reg_alpha': 7.276625863221852, 'reg_lambda': 6.5129953283318525, 'min_split_gain': 0.013949396281315585}. Best is trial 1 with value: 0.006355111158160488.
[I 2025-06-15 19:51:21,731] Trial 2 finished with value: 0.007121073122032272 and parameters: {'n_estimators': 67, 'learning_rate': 0.12377739688758707, 'num_leaves': 243, 'max_depth': 16, 'min_child_samples': 6, 'subsample': 0.7468274552224999, 'colsample_bytree': 0.5546223268556233, 'reg_alpha': 8.823491758612803, 'reg_lambda': 6.511993059525286, 'min_split_gain': 0.036816188939408405}. Best is trial 1 with value: 0.006355111158160488.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:22,524] Trial 3 finished with value: 0.008808431933341099 and parameters: {'n_estimators': 240, 'learning_rate': 0.01297247663841895, 'num_leaves': 176, 'max_depth': 14, 'min_child_samples': 10, 'subsample': 0.5262131585503864, 'colsample_bytree': 0.541700077271609, 'reg_alpha': 7.419332156042594, 'reg_lambda': 2.5883489784221703, 'min_split_gain': 0.03850121369358407}. Best is trial 1 with value: 0.006355111158160488.
[I 2025-06-15 19:51:22,527] A new study created in memory with name: no-name-9d868fc0-fe05-4b2e-8b8e-376230d6e5fc
[I 2025-06-15 19:51:22,544] Trial 0 finished with value: 0.02804698721249399 and parameters: {'n_estimators': 91, 'learning_rate': 0.298048147245022, 'num_leaves': 124, 'max_depth': 12, 'min_child_samples': 6, 'subsample': 0.8892223511521536, 'colsample_bytree': 0.8269608624558955, 'reg_alpha': 9.369497606368874, 'reg_lambda': 3.39286393020363, 'min_split_gain': 0.006622300395633213}. Best is trial 0 with value: 0.02804698721249399.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1396
1           Price_lag_2        1009
14      pct_chg_Load_DA         636
3         Load_DA_lag_0         370
16  roll2d_median_Price         336
11           NGas_lag_2         326
2           Price_lag_7         306
13            EUA_lag_2         227
10           Coal_lag_2         176
5       WindOn_DA_lag_0         148
12            Oil_lag_2         126
15       lag168_Load_DA          93
9                  WD_7          19
8                  WD_6          11
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           3
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 17.10% done
********************* END NS.. 124 ****************************************************
********************* START NS ... 125 ****************************************************
[I 2025-06-15 19:51:23,507] A new study created in memory with name: no-name-c8ad550a-18c5-41ae-a61a-46ca6df6467a
[I 2025-06-15 19:51:23,641] Trial 0 finished with value: 0.004810777753050695 and parameters: {'n_estimators': 206, 'learning_rate': 0.2563816330126169, 'num_leaves': 82, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.7232572623043887, 'colsample_bytree': 0.6395722167340818, 'reg_alpha': 6.565379315894527, 'reg_lambda': 8.429098008779862, 'min_split_gain': 0.019349558969969313}. Best is trial 0 with value: 0.004810777753050695.
[Trial 0] New best: 0.00481
[I 2025-06-15 19:51:24,023] Trial 1 finished with value: 0.005085862274910402 and parameters: {'n_estimators': 90, 'learning_rate': 0.07689966985388011, 'num_leaves': 89, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.7759655469597908, 'colsample_bytree': 0.5385971984143181, 'reg_alpha': 6.205121518970755, 'reg_lambda': 9.356260919306367, 'min_split_gain': 0.018922177872590953}. Best is trial 0 with value: 0.004810777753050695.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:24,418] Trial 2 finished with value: 0.004659111595870189 and parameters: {'n_estimators': 203, 'learning_rate': 0.10023015561849304, 'num_leaves': 54, 'max_depth': 5, 'min_child_samples': 11, 'subsample': 0.8258099464324653, 'colsample_bytree': 0.7295245989396383, 'reg_alpha': 3.4368794056915575, 'reg_lambda': 4.324381577626351, 'min_split_gain': 0.007330047905847165}. Best is trial 2 with value: 0.004659111595870189.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:25,368] Trial 3 finished with value: 0.00498260915645263 and parameters: {'n_estimators': 254, 'learning_rate': 0.016741332257245542, 'num_leaves': 150, 'max_depth': 9, 'min_child_samples': 15, 'subsample': 0.9949939093443614, 'colsample_bytree': 0.8558591142862834, 'reg_alpha': 8.595470494345799, 'reg_lambda': 6.951470493586415, 'min_split_gain': 0.038303810483869966}. Best is trial 2 with value: 0.004659111595870189.
[I 2025-06-15 19:51:25,370] A new study created in memory with name: no-name-a1267676-4c0a-4fd7-8a5d-709ea6f51b16
[I 2025-06-15 19:51:25,412] Trial 0 finished with value: 0.010450639980552087 and parameters: {'n_estimators': 141, 'learning_rate': 0.03318519921067737, 'num_leaves': 163, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.6779437036097105, 'colsample_bytree': 0.5191325478056591, 'reg_alpha': 0.397021746801135, 'reg_lambda': 3.6999858578674316, 'min_split_gain': 0.01468382637425657}. Best is trial 0 with value: 0.010450639980552087.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         600
1           Price_lag_2         406
14      pct_chg_Load_DA         353
3         Load_DA_lag_0         328
11           NGas_lag_2         245
5       WindOn_DA_lag_0         174
2           Price_lag_7         161
16  roll2d_median_Price         160
15       lag168_Load_DA         143
13            EUA_lag_2         132
10           Coal_lag_2         117
12            Oil_lag_2          76
7                  WD_1          11
9                  WD_7           7
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         150
15       lag168_Load_DA          38
14      pct_chg_Load_DA          36
1           Price_lag_2          35
5       WindOn_DA_lag_0          22
2           Price_lag_7          14
3         Load_DA_lag_0           5
16  roll2d_median_Price           3
12            Oil_lag_2           2
13            EUA_lag_2           1
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 17.24% done
********************* END NS.. 125 ****************************************************
********************* START NS ... 126 ****************************************************
[I 2025-06-15 19:51:26,296] A new study created in memory with name: no-name-873caa59-3985-4a7b-805e-383dac4d5f98
[I 2025-06-15 19:51:26,918] Trial 0 finished with value: 0.002942261262680868 and parameters: {'n_estimators': 250, 'learning_rate': 0.022306594815304188, 'num_leaves': 146, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.8236381772298194, 'colsample_bytree': 0.908044408106909, 'reg_alpha': 8.188905302646136, 'reg_lambda': 8.68344055586157, 'min_split_gain': 0.03039557673960132}. Best is trial 0 with value: 0.002942261262680868.
[Trial 0] New best: 0.00294
[I 2025-06-15 19:51:28,572] Trial 1 finished with value: 0.002661622528774514 and parameters: {'n_estimators': 177, 'learning_rate': 0.026496347471000076, 'num_leaves': 214, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.8067655169332277, 'colsample_bytree': 0.5018974516927063, 'reg_alpha': 0.08213432027749379, 'reg_lambda': 5.880974590280897, 'min_split_gain': 0.02235480618929897}. Best is trial 1 with value: 0.002661622528774514.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:29,623] Trial 2 finished with value: 0.002871258971500213 and parameters: {'n_estimators': 246, 'learning_rate': 0.015580439079650033, 'num_leaves': 63, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.6084382858125851, 'colsample_bytree': 0.6050743668884153, 'reg_alpha': 2.9917119662140057, 'reg_lambda': 4.7480565200148375, 'min_split_gain': 0.03599373246613145}. Best is trial 1 with value: 0.002661622528774514.
[I 2025-06-15 19:51:29,821] Trial 3 finished with value: 0.002616089896683471 and parameters: {'n_estimators': 280, 'learning_rate': 0.23567132316778144, 'num_leaves': 187, 'max_depth': 4, 'min_child_samples': 17, 'subsample': 0.7155150354033417, 'colsample_bytree': 0.979623228519779, 'reg_alpha': 9.751543155594458, 'reg_lambda': 9.083805524184497, 'min_split_gain': 0.03350702635925822}. Best is trial 3 with value: 0.002616089896683471.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:29,824] A new study created in memory with name: no-name-43b1d642-e663-4dfc-8982-e0293133860c
[I 2025-06-15 19:51:29,853] Trial 0 finished with value: 0.009301253388507496 and parameters: {'n_estimators': 124, 'learning_rate': 0.06291206977398152, 'num_leaves': 239, 'max_depth': 11, 'min_child_samples': 10, 'subsample': 0.6302758432154105, 'colsample_bytree': 0.9185894753324727, 'reg_alpha': 1.9629144922729425, 'reg_lambda': 7.310891321372528, 'min_split_gain': 0.0485464515945401}. Best is trial 0 with value: 0.009301253388507496.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         145
14      pct_chg_Load_DA          69
11           NGas_lag_2          31
3         Load_DA_lag_0          27
2           Price_lag_7          25
16  roll2d_median_Price          22
1           Price_lag_2          17
13            EUA_lag_2          16
15       lag168_Load_DA          16
10           Coal_lag_2          13
12            Oil_lag_2          12
5       WindOn_DA_lag_0           5
7                  WD_1           1
9                  WD_7           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          61
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 17.37% done
********************* END NS.. 126 ****************************************************
********************* START NS ... 127 ****************************************************
[I 2025-06-15 19:51:30,557] A new study created in memory with name: no-name-c023f30a-25cd-4e21-bbea-45bcbd099c59
[I 2025-06-15 19:51:31,007] Trial 0 finished with value: 0.004224567089824406 and parameters: {'n_estimators': 104, 'learning_rate': 0.04387906360862983, 'num_leaves': 128, 'max_depth': 7, 'min_child_samples': 18, 'subsample': 0.9778076186269233, 'colsample_bytree': 0.8658606158736477, 'reg_alpha': 0.038006703735364855, 'reg_lambda': 3.451383238511545, 'min_split_gain': 0.03452918217661032}. Best is trial 0 with value: 0.004224567089824406.
[Trial 0] New best: 0.00422
[I 2025-06-15 19:51:31,289] Trial 1 finished with value: 0.0049638330305474465 and parameters: {'n_estimators': 248, 'learning_rate': 0.19317820810952377, 'num_leaves': 140, 'max_depth': 15, 'min_child_samples': 19, 'subsample': 0.5910083496762157, 'colsample_bytree': 0.5485841120108421, 'reg_alpha': 5.100337375892824, 'reg_lambda': 5.426719106744628, 'min_split_gain': 0.0173985104815861}. Best is trial 0 with value: 0.004224567089824406.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:31,743] Trial 2 finished with value: 0.004285662281923195 and parameters: {'n_estimators': 288, 'learning_rate': 0.05544997601749821, 'num_leaves': 171, 'max_depth': 10, 'min_child_samples': 12, 'subsample': 0.7147890103121861, 'colsample_bytree': 0.9135348519907094, 'reg_alpha': 6.315298837051689, 'reg_lambda': 7.170500193243976, 'min_split_gain': 0.029533656787756154}. Best is trial 0 with value: 0.004224567089824406.
[I 2025-06-15 19:51:31,917] Trial 3 finished with value: 0.004504512536521654 and parameters: {'n_estimators': 110, 'learning_rate': 0.2887509357251277, 'num_leaves': 148, 'max_depth': 6, 'min_child_samples': 19, 'subsample': 0.5790196119485655, 'colsample_bytree': 0.6244864616579366, 'reg_alpha': 5.517465639684219, 'reg_lambda': 2.0433364341792206, 'min_split_gain': 0.020125745630368997}. Best is trial 0 with value: 0.004224567089824406.
[I 2025-06-15 19:51:31,919] A new study created in memory with name: no-name-7f42b2fe-0910-40da-af34-3abb5926c9e9
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 19:51:31,957] Trial 0 finished with value: 0.03279725175354652 and parameters: {'n_estimators': 235, 'learning_rate': 0.019854989838274575, 'num_leaves': 238, 'max_depth': 15, 'min_child_samples': 13, 'subsample': 0.7645226820449875, 'colsample_bytree': 0.9331991685084333, 'reg_alpha': 2.448354081304427, 'reg_lambda': 8.224373904383794, 'min_split_gain': 0.0388725108496173}. Best is trial 0 with value: 0.03279725175354652.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1634
1           Price_lag_2        1308
14      pct_chg_Load_DA         933
16  roll2d_median_Price         454
2           Price_lag_7         439
3         Load_DA_lag_0         402
11           NGas_lag_2         381
15       lag168_Load_DA         256
10           Coal_lag_2         246
13            EUA_lag_2         244
5       WindOn_DA_lag_0         192
12            Oil_lag_2         188
7                  WD_1           9
9                  WD_7           3
8                  WD_6           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         211
15       lag168_Load_DA          11
1           Price_lag_2           7
5       WindOn_DA_lag_0           2
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 17.51% done
********************* END NS.. 127 ****************************************************
********************* START NS ... 128 ****************************************************
[I 2025-06-15 19:51:33,005] A new study created in memory with name: no-name-8c651ef7-6f6c-4c08-8670-41c64844e768
[I 2025-06-15 19:51:33,274] Trial 0 finished with value: 0.005147450579364034 and parameters: {'n_estimators': 102, 'learning_rate': 0.0385484783685966, 'num_leaves': 214, 'max_depth': 4, 'min_child_samples': 20, 'subsample': 0.5398905259330368, 'colsample_bytree': 0.8182570949764223, 'reg_alpha': 6.680392670056411, 'reg_lambda': 4.404346855177112, 'min_split_gain': 0.02363382593387558}. Best is trial 0 with value: 0.005147450579364034.
[Trial 0] New best: 0.00515
[I 2025-06-15 19:51:33,651] Trial 1 finished with value: 0.003424143502277184 and parameters: {'n_estimators': 265, 'learning_rate': 0.19842688717023219, 'num_leaves': 59, 'max_depth': 10, 'min_child_samples': 17, 'subsample': 0.6931555117497097, 'colsample_bytree': 0.7605520836398159, 'reg_alpha': 3.906752272077135, 'reg_lambda': 7.815420179063537, 'min_split_gain': 0.020722450783127566}. Best is trial 1 with value: 0.003424143502277184.
[Trial 1] New best: 0.00342
[I 2025-06-15 19:51:34,392] Trial 2 finished with value: 0.003840210999866965 and parameters: {'n_estimators': 226, 'learning_rate': 0.06489918641463803, 'num_leaves': 44, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.5783631561277214, 'colsample_bytree': 0.7408210078723665, 'reg_alpha': 4.188537574221059, 'reg_lambda': 2.986634637783453, 'min_split_gain': 0.01518481943925703}. Best is trial 1 with value: 0.003424143502277184.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:35,130] Trial 3 finished with value: 0.003959082599010081 and parameters: {'n_estimators': 167, 'learning_rate': 0.041820064561963285, 'num_leaves': 81, 'max_depth': 7, 'min_child_samples': 30, 'subsample': 0.9183644420922554, 'colsample_bytree': 0.85576947762077, 'reg_alpha': 9.26576268231177, 'reg_lambda': 6.777135549775771, 'min_split_gain': 0.017639258080853315}. Best is trial 1 with value: 0.003424143502277184.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:35,374] Trial 4 finished with value: 0.011292763140134878 and parameters: {'n_estimators': 95, 'learning_rate': 0.011366394189167881, 'num_leaves': 193, 'max_depth': 4, 'min_child_samples': 24, 'subsample': 0.511455885189614, 'colsample_bytree': 0.613092748379275, 'reg_alpha': 2.578685826341083, 'reg_lambda': 9.570428658190677, 'min_split_gain': 0.0003100015742421858}. Best is trial 1 with value: 0.003424143502277184.
[I 2025-06-15 19:51:35,377] A new study created in memory with name: no-name-0046e215-c019-455f-b774-1e82687e1fa2
[I 2025-06-15 19:51:35,415] Trial 0 finished with value: 0.03814635226984423 and parameters: {'n_estimators': 244, 'learning_rate': 0.16550424840877628, 'num_leaves': 234, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.9481694506068913, 'colsample_bytree': 0.9946461151070465, 'reg_alpha': 6.8806711926633035, 'reg_lambda': 2.8635564129633106, 'min_split_gain': 0.04960810559008419}. Best is trial 0 with value: 0.03814635226984423.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         304
1           Price_lag_2         215
14      pct_chg_Load_DA         184
3         Load_DA_lag_0         167
5       WindOn_DA_lag_0         107
11           NGas_lag_2         103
13            EUA_lag_2          94
2           Price_lag_7          90
15       lag168_Load_DA          88
16  roll2d_median_Price          82
10           Coal_lag_2          55
12            Oil_lag_2          48
7                  WD_1           4
9                  WD_7           3
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           6
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 17.65% done
********************* END NS.. 128 ****************************************************
********************* START NS ... 129 ****************************************************
[I 2025-06-15 19:51:36,361] A new study created in memory with name: no-name-c4f58cfe-923a-4120-be94-60319184a7db
[I 2025-06-15 19:51:37,694] Trial 0 finished with value: 0.002562171942133723 and parameters: {'n_estimators': 255, 'learning_rate': 0.03165635616207376, 'num_leaves': 71, 'max_depth': 14, 'min_child_samples': 10, 'subsample': 0.5730916825189335, 'colsample_bytree': 0.8269859103203479, 'reg_alpha': 5.913727836154309, 'reg_lambda': 8.516401672902083, 'min_split_gain': 0.02456679698906372}. Best is trial 0 with value: 0.002562171942133723.
[Trial 0] New best: 0.00256
[I 2025-06-15 19:51:38,155] Trial 1 finished with value: 0.003813192041508623 and parameters: {'n_estimators': 76, 'learning_rate': 0.25549906381578846, 'num_leaves': 184, 'max_depth': 11, 'min_child_samples': 14, 'subsample': 0.9576151202334994, 'colsample_bytree': 0.5452106151529219, 'reg_alpha': 4.8381791097269735, 'reg_lambda': 2.02120729831155, 'min_split_gain': 0.010387643352716486}. Best is trial 0 with value: 0.002562171942133723.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:38,631] Trial 2 finished with value: 0.0028484118980969674 and parameters: {'n_estimators': 138, 'learning_rate': 0.11093007979805829, 'num_leaves': 119, 'max_depth': 8, 'min_child_samples': 13, 'subsample': 0.5353093929503006, 'colsample_bytree': 0.885392498925299, 'reg_alpha': 7.776114176272255, 'reg_lambda': 5.179184910731818, 'min_split_gain': 0.025378181550272962}. Best is trial 0 with value: 0.002562171942133723.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:39,038] Trial 3 finished with value: 0.0023974635327439872 and parameters: {'n_estimators': 144, 'learning_rate': 0.1570904979545406, 'num_leaves': 70, 'max_depth': 7, 'min_child_samples': 27, 'subsample': 0.9539305791659864, 'colsample_bytree': 0.708420909051974, 'reg_alpha': 2.8697311112554678, 'reg_lambda': 3.802956045929764, 'min_split_gain': 0.004061377139604622}. Best is trial 3 with value: 0.0023974635327439872.
[I 2025-06-15 19:51:39,040] A new study created in memory with name: no-name-2c0e80c3-2c05-4f4d-9452-46cfdb85fc09
[I 2025-06-15 19:51:39,072] Trial 0 finished with value: 0.007170287252733702 and parameters: {'n_estimators': 153, 'learning_rate': 0.010904058527773899, 'num_leaves': 198, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.6618668834872343, 'colsample_bytree': 0.5926795987649675, 'reg_alpha': 4.7246466796437, 'reg_lambda': 8.085429193818658, 'min_split_gain': 0.03106216365369771}. Best is trial 0 with value: 0.007170287252733702.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         531
1           Price_lag_2         412
3         Load_DA_lag_0         340
14      pct_chg_Load_DA         323
13            EUA_lag_2         231
2           Price_lag_7         216
5       WindOn_DA_lag_0         213
10           Coal_lag_2         207
11           NGas_lag_2         201
15       lag168_Load_DA         194
16  roll2d_median_Price         136
12            Oil_lag_2          99
9                  WD_7           8
7                  WD_1           7
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          96
1           Price_lag_2          22
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 17.78% done
********************* END NS.. 129 ****************************************************
********************* START NS ... 130 ****************************************************
[I 2025-06-15 19:51:40,006] A new study created in memory with name: no-name-cd07005f-0f98-4159-8058-38aaa6bd8a77
[I 2025-06-15 19:51:40,357] Trial 0 finished with value: 0.0024119209991874113 and parameters: {'n_estimators': 270, 'learning_rate': 0.10778306697158996, 'num_leaves': 38, 'max_depth': 7, 'min_child_samples': 14, 'subsample': 0.9733563793774203, 'colsample_bytree': 0.5482680841993084, 'reg_alpha': 9.749593751239246, 'reg_lambda': 0.29732082165953244, 'min_split_gain': 0.03408634356467113}. Best is trial 0 with value: 0.0024119209991874113.
[I 2025-06-15 19:51:40,554] Trial 1 finished with value: 0.0015944203393064146 and parameters: {'n_estimators': 136, 'learning_rate': 0.18705415219534255, 'num_leaves': 40, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.9902443304859547, 'colsample_bytree': 0.8067951567665402, 'reg_alpha': 6.85927376946591, 'reg_lambda': 1.0963331588359415, 'min_split_gain': 0.016009734714238857}. Best is trial 1 with value: 0.0015944203393064146.
[Trial 0] New best: 0.00241
[I 2025-06-15 19:51:40,710] Trial 2 finished with value: 0.0051181008664578755 and parameters: {'n_estimators': 59, 'learning_rate': 0.013275836183031402, 'num_leaves': 100, 'max_depth': 4, 'min_child_samples': 17, 'subsample': 0.5020250106683057, 'colsample_bytree': 0.8214569079722156, 'reg_alpha': 0.8708988095448766, 'reg_lambda': 4.684826761233879, 'min_split_gain': 0.0015301564631538423}. Best is trial 1 with value: 0.0015944203393064146.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:40,998] Trial 3 finished with value: 0.001697068198566784 and parameters: {'n_estimators': 105, 'learning_rate': 0.18304864838840185, 'num_leaves': 189, 'max_depth': 14, 'min_child_samples': 22, 'subsample': 0.8289293922459957, 'colsample_bytree': 0.6882855099665786, 'reg_alpha': 9.922614436198845, 'reg_lambda': 6.608591652384694, 'min_split_gain': 0.0376745606179241}. Best is trial 1 with value: 0.0015944203393064146.
[I 2025-06-15 19:51:41,002] A new study created in memory with name: no-name-9724aa17-d7b4-45ee-a923-6d21ade2d8e9
[I 2025-06-15 19:51:41,062] Trial 0 finished with value: 0.0022927854402726637 and parameters: {'n_estimators': 157, 'learning_rate': 0.056430779528138686, 'num_leaves': 140, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.8303847163737216, 'colsample_bytree': 0.8156592352567857, 'reg_alpha': 0.5501463454362088, 'reg_lambda': 5.864539861910913, 'min_split_gain': 0.011316861099758165}. Best is trial 0 with value: 0.0022927854402726637.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         209
14      pct_chg_Load_DA         111
1           Price_lag_2         111
3         Load_DA_lag_0          63
11           NGas_lag_2          60
2           Price_lag_7          51
13            EUA_lag_2          48
15       lag168_Load_DA          37
10           Coal_lag_2          31
16  roll2d_median_Price          30
5       WindOn_DA_lag_0          27
12            Oil_lag_2          23
7                  WD_1           4
9                  WD_7           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         107
14      pct_chg_Load_DA          42
1           Price_lag_2           8
15       lag168_Load_DA           6
10           Coal_lag_2           5
16  roll2d_median_Price           4
3         Load_DA_lag_0           2
6      WindOff_DA_lag_0           0
7                  WD_1           0
5       WindOn_DA_lag_0           0
9                  WD_7           0
4        Solar_DA_lag_0           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
2           Price_lag_7           0
8                  WD_6           0
-> 17.92% done
********************* END NS.. 130 ****************************************************
********************* START NS ... 131 ****************************************************
[I 2025-06-15 19:51:41,941] A new study created in memory with name: no-name-23efb4f7-3ebc-4ad9-8299-aa99df717173
[I 2025-06-15 19:51:42,119] Trial 0 finished with value: 0.0010725941805496038 and parameters: {'n_estimators': 206, 'learning_rate': 0.27903849295588223, 'num_leaves': 118, 'max_depth': 10, 'min_child_samples': 11, 'subsample': 0.886827930175803, 'colsample_bytree': 0.8571430878502511, 'reg_alpha': 9.306255934495796, 'reg_lambda': 0.05955345088260122, 'min_split_gain': 0.039629225154333876}. Best is trial 0 with value: 0.0010725941805496038.
[Trial 0] New best: 0.00107
[I 2025-06-15 19:51:42,974] Trial 1 finished with value: 0.001050280499004232 and parameters: {'n_estimators': 222, 'learning_rate': 0.017286655574197805, 'num_leaves': 33, 'max_depth': 7, 'min_child_samples': 18, 'subsample': 0.8435250558824647, 'colsample_bytree': 0.8779206215206821, 'reg_alpha': 1.5247268386891655, 'reg_lambda': 7.51091484117169, 'min_split_gain': 0.020079239104182484}. Best is trial 1 with value: 0.001050280499004232.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:43,327] Trial 2 finished with value: 0.0010990912537800158 and parameters: {'n_estimators': 95, 'learning_rate': 0.12076015722965298, 'num_leaves': 116, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.8904625815941242, 'colsample_bytree': 0.6858374118191672, 'reg_alpha': 9.166071861009923, 'reg_lambda': 6.413464476563606, 'min_split_gain': 0.02929059357194133}. Best is trial 1 with value: 0.001050280499004232.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:44,633] Trial 3 finished with value: 0.0013876871583458747 and parameters: {'n_estimators': 131, 'learning_rate': 0.013634022833836327, 'num_leaves': 194, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.6341231124371609, 'colsample_bytree': 0.63449380524616, 'reg_alpha': 0.26822491664199366, 'reg_lambda': 6.851216007764522, 'min_split_gain': 0.01085412689511538}. Best is trial 1 with value: 0.001050280499004232.
[I 2025-06-15 19:51:44,637] A new study created in memory with name: no-name-c138172d-8831-4aa6-b564-4c70b1d31529
[I 2025-06-15 19:51:44,723] Trial 0 finished with value: 0.007377535711595283 and parameters: {'n_estimators': 62, 'learning_rate': 0.04057165689698059, 'num_leaves': 208, 'max_depth': 6, 'min_child_samples': 26, 'subsample': 0.5553148761963125, 'colsample_bytree': 0.7657976439152744, 'reg_alpha': 5.2047742062626545, 'reg_lambda': 6.141027409856275, 'min_split_gain': 0.024441903077510138}. Best is trial 0 with value: 0.007377535711595283.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2683
1           Price_lag_2        1272
14      pct_chg_Load_DA        1023
11           NGas_lag_2         396
16  roll2d_median_Price         380
2           Price_lag_7         344
3         Load_DA_lag_0         303
15       lag168_Load_DA         194
12            Oil_lag_2         149
13            EUA_lag_2         139
10           Coal_lag_2         108
5       WindOn_DA_lag_0         100
9                  WD_7           9
8                  WD_6           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 18.06% done
********************* END NS.. 131 ****************************************************
********************* START NS ... 132 ****************************************************
[I 2025-06-15 19:51:46,500] A new study created in memory with name: no-name-075ed948-9c3c-46bf-aca3-7919c3f64151
[I 2025-06-15 19:51:47,461] Trial 0 finished with value: 0.0021740591562948385 and parameters: {'n_estimators': 69, 'learning_rate': 0.01248628625153769, 'num_leaves': 163, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.6369801200488414, 'colsample_bytree': 0.5645760091309284, 'reg_alpha': 1.6995439263238044, 'reg_lambda': 8.41969724783506, 'min_split_gain': 0.008187020505061727}. Best is trial 0 with value: 0.0021740591562948385.
[Trial 0] New best: 0.00217
[I 2025-06-15 19:51:49,479] Trial 1 finished with value: 0.0009918990236190705 and parameters: {'n_estimators': 222, 'learning_rate': 0.030365002376977958, 'num_leaves': 220, 'max_depth': 12, 'min_child_samples': 26, 'subsample': 0.5160828254022486, 'colsample_bytree': 0.6480937952123712, 'reg_alpha': 1.61637398938288, 'reg_lambda': 2.203388761803713, 'min_split_gain': 0.006869071232761126}. Best is trial 1 with value: 0.0009918990236190705.
[Trial 1] New best: 0.00099
[I 2025-06-15 19:51:49,880] Trial 2 finished with value: 0.001077910791444163 and parameters: {'n_estimators': 214, 'learning_rate': 0.11641818120908581, 'num_leaves': 98, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.5227133524778028, 'colsample_bytree': 0.9199059163508297, 'reg_alpha': 4.329932076119459, 'reg_lambda': 6.856077571091939, 'min_split_gain': 0.019100653532085607}. Best is trial 1 with value: 0.0009918990236190705.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:50,148] Trial 3 finished with value: 0.0015225530221798303 and parameters: {'n_estimators': 240, 'learning_rate': 0.2582465155785314, 'num_leaves': 256, 'max_depth': 4, 'min_child_samples': 24, 'subsample': 0.6072107767252852, 'colsample_bytree': 0.6193888104078116, 'reg_alpha': 1.0688981273362974, 'reg_lambda': 1.7076360434657067, 'min_split_gain': 0.0456847289576397}. Best is trial 1 with value: 0.0009918990236190705.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:50,803] Trial 4 finished with value: 0.0010595005167895047 and parameters: {'n_estimators': 128, 'learning_rate': 0.01942643189707447, 'num_leaves': 175, 'max_depth': 11, 'min_child_samples': 28, 'subsample': 0.568891726194549, 'colsample_bytree': 0.6490327594605296, 'reg_alpha': 6.177977533402618, 'reg_lambda': 7.886009997590414, 'min_split_gain': 0.03289395777424388}. Best is trial 1 with value: 0.0009918990236190705.
[I 2025-06-15 19:51:50,806] A new study created in memory with name: no-name-d1bc9dbf-1ed1-4a90-bf73-ea4cd42c3229
[I 2025-06-15 19:51:50,858] Trial 0 finished with value: 0.004648232408850777 and parameters: {'n_estimators': 256, 'learning_rate': 0.1421597722924274, 'num_leaves': 104, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.5360931234977568, 'colsample_bytree': 0.6737103561760878, 'reg_alpha': 8.682978202073093, 'reg_lambda': 8.689730225912871, 'min_split_gain': 0.03826167512224787}. Best is trial 0 with value: 0.004648232408850777.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2790
1           Price_lag_2        2672
14      pct_chg_Load_DA        2066
3         Load_DA_lag_0        1540
15       lag168_Load_DA        1208
5       WindOn_DA_lag_0        1173
2           Price_lag_7        1156
16  roll2d_median_Price        1147
11           NGas_lag_2         967
13            EUA_lag_2         932
10           Coal_lag_2         801
12            Oil_lag_2         677
9                  WD_7          42
7                  WD_1          37
8                  WD_6          14
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 18.19% done
********************* END NS.. 132 ****************************************************
********************* START NS ... 133 ****************************************************
[I 2025-06-15 19:51:52,604] A new study created in memory with name: no-name-00a0d464-db91-4c98-8523-57dd862cf3d9
[I 2025-06-15 19:51:52,955] Trial 0 finished with value: 0.012365014299096583 and parameters: {'n_estimators': 217, 'learning_rate': 0.03128402746974789, 'num_leaves': 143, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.5865828351148473, 'colsample_bytree': 0.5098192510261985, 'reg_alpha': 0.8313187208172668, 'reg_lambda': 9.912858653110792, 'min_split_gain': 0.009543591856521483}. Best is trial 0 with value: 0.012365014299096583.
[I 2025-06-15 19:51:53,110] Trial 1 finished with value: 0.008222245368963934 and parameters: {'n_estimators': 172, 'learning_rate': 0.2664425411214521, 'num_leaves': 136, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.9886865520335455, 'colsample_bytree': 0.8273138392790491, 'reg_alpha': 6.1387652001243715, 'reg_lambda': 0.6790853801468744, 'min_split_gain': 0.016531693598837695}. Best is trial 1 with value: 0.008222245368963934.
[Trial 0] New best: 0.01237
[Trial 1] New best: 0.00822
[I 2025-06-15 19:51:53,578] Trial 2 finished with value: 0.011513024547662802 and parameters: {'n_estimators': 147, 'learning_rate': 0.05317779685392002, 'num_leaves': 55, 'max_depth': 9, 'min_child_samples': 8, 'subsample': 0.7890967747151916, 'colsample_bytree': 0.5803395930905941, 'reg_alpha': 8.752399243266094, 'reg_lambda': 3.641597017708885, 'min_split_gain': 0.027857174328453468}. Best is trial 1 with value: 0.008222245368963934.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:53,931] Trial 3 finished with value: 0.009665128476193767 and parameters: {'n_estimators': 206, 'learning_rate': 0.15660243528690082, 'num_leaves': 121, 'max_depth': 15, 'min_child_samples': 21, 'subsample': 0.9317457789611638, 'colsample_bytree': 0.747937658461999, 'reg_alpha': 9.50504389903781, 'reg_lambda': 4.462615641546019, 'min_split_gain': 0.03996563598173185}. Best is trial 1 with value: 0.008222245368963934.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:54,382] Trial 4 finished with value: 0.008225272377914256 and parameters: {'n_estimators': 218, 'learning_rate': 0.0708398072307845, 'num_leaves': 46, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.7024405915428436, 'colsample_bytree': 0.8166554523254914, 'reg_alpha': 6.066402882650779, 'reg_lambda': 5.90363816367014, 'min_split_gain': 0.006581430235065688}. Best is trial 1 with value: 0.008222245368963934.
[I 2025-06-15 19:51:54,385] A new study created in memory with name: no-name-e93cef22-668f-4716-8660-bfe603e6b56f
[I 2025-06-15 19:51:54,401] Trial 0 finished with value: 0.06576214973504092 and parameters: {'n_estimators': 54, 'learning_rate': 0.0725562313209285, 'num_leaves': 158, 'max_depth': 6, 'min_child_samples': 19, 'subsample': 0.9164984557685649, 'colsample_bytree': 0.7681067185752894, 'reg_alpha': 6.545372386939709, 'reg_lambda': 6.520382977119953, 'min_split_gain': 0.029899582294658285}. Best is trial 0 with value: 0.06576214973504092.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         192
14      pct_chg_Load_DA         128
1           Price_lag_2         104
3         Load_DA_lag_0          68
2           Price_lag_7          64
11           NGas_lag_2          61
16  roll2d_median_Price          58
5       WindOn_DA_lag_0          48
13            EUA_lag_2          46
15       lag168_Load_DA          35
10           Coal_lag_2          33
12            Oil_lag_2          32
7                  WD_1           3
9                  WD_7           1
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           8
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 18.33% done
********************* END NS.. 133 ****************************************************
********************* START NS ... 134 ****************************************************
[I 2025-06-15 19:51:55,048] A new study created in memory with name: no-name-e6923d33-fa37-4b9c-8edc-1e6a859bec4b
[I 2025-06-15 19:51:55,157] Trial 0 finished with value: 0.018399993234448533 and parameters: {'n_estimators': 134, 'learning_rate': 0.1972469004083817, 'num_leaves': 110, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.5959518783967361, 'colsample_bytree': 0.7928909521954557, 'reg_alpha': 5.816408698975886, 'reg_lambda': 1.4882289071883248, 'min_split_gain': 0.044712969126799555}. Best is trial 0 with value: 0.018399993234448533.
[I 2025-06-15 19:51:55,311] Trial 1 finished with value: 0.01770441302091073 and parameters: {'n_estimators': 188, 'learning_rate': 0.1815886415686448, 'num_leaves': 60, 'max_depth': 16, 'min_child_samples': 21, 'subsample': 0.5256773820273974, 'colsample_bytree': 0.8911833667935536, 'reg_alpha': 5.005642943125118, 'reg_lambda': 3.7341746145501755, 'min_split_gain': 0.04849729274376866}. Best is trial 1 with value: 0.01770441302091073.
[Trial 0] New best: 0.01840
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:55,833] Trial 2 finished with value: 0.02127993204617681 and parameters: {'n_estimators': 149, 'learning_rate': 0.013116291677745485, 'num_leaves': 181, 'max_depth': 9, 'min_child_samples': 1, 'subsample': 0.8976424434999951, 'colsample_bytree': 0.9579144587511412, 'reg_alpha': 7.1201640962321955, 'reg_lambda': 6.276600101444077, 'min_split_gain': 0.03591851546482005}. Best is trial 1 with value: 0.01770441302091073.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:56,219] Trial 3 finished with value: 0.019688782012301297 and parameters: {'n_estimators': 199, 'learning_rate': 0.049069525748423146, 'num_leaves': 119, 'max_depth': 11, 'min_child_samples': 18, 'subsample': 0.6522826776179744, 'colsample_bytree': 0.7799566408927914, 'reg_alpha': 9.642184370993446, 'reg_lambda': 7.144926661033303, 'min_split_gain': 0.04804771407619033}. Best is trial 1 with value: 0.01770441302091073.
[I 2025-06-15 19:51:56,222] A new study created in memory with name: no-name-89c1a891-f86c-48f7-b3b3-7d5471b78da0
[I 2025-06-15 19:51:56,252] Trial 0 finished with value: 0.1503266706175105 and parameters: {'n_estimators': 207, 'learning_rate': 0.08767417814936398, 'num_leaves': 124, 'max_depth': 11, 'min_child_samples': 25, 'subsample': 0.6023687912821626, 'colsample_bytree': 0.8309303968674477, 'reg_alpha': 7.714878447845029, 'reg_lambda': 4.541910369947533, 'min_split_gain': 0.03349334792651853}. Best is trial 0 with value: 0.1503266706175105.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         272
14      pct_chg_Load_DA         157
1           Price_lag_2         135
3         Load_DA_lag_0          76
11           NGas_lag_2          74
16  roll2d_median_Price          68
2           Price_lag_7          59
15       lag168_Load_DA          47
10           Coal_lag_2          42
13            EUA_lag_2          39
12            Oil_lag_2          30
5       WindOn_DA_lag_0          19
7                  WD_1           2
9                  WD_7           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          11
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 18.47% done
********************* END NS.. 134 ****************************************************
********************* START NS ... 135 ****************************************************
[I 2025-06-15 19:51:56,936] A new study created in memory with name: no-name-e0883f1d-ec9e-41d5-9b89-0f6edfd3d03b
[I 2025-06-15 19:51:57,534] Trial 0 finished with value: 0.011003564167803714 and parameters: {'n_estimators': 189, 'learning_rate': 0.1283037860968568, 'num_leaves': 147, 'max_depth': 8, 'min_child_samples': 25, 'subsample': 0.7207040276294676, 'colsample_bytree': 0.5289275518045453, 'reg_alpha': 0.0817787933563019, 'reg_lambda': 8.861525930074253, 'min_split_gain': 0.018759553501968253}. Best is trial 0 with value: 0.011003564167803714.
[Trial 0] New best: 0.01100
[I 2025-06-15 19:51:58,186] Trial 1 finished with value: 0.012807386661516242 and parameters: {'n_estimators': 203, 'learning_rate': 0.03260305842476681, 'num_leaves': 57, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.6426221294938326, 'colsample_bytree': 0.7641884394655751, 'reg_alpha': 7.550932035960357, 'reg_lambda': 0.7375111271167201, 'min_split_gain': 0.045902486767895656}. Best is trial 0 with value: 0.011003564167803714.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:51:58,642] Trial 2 finished with value: 0.012597462317400781 and parameters: {'n_estimators': 118, 'learning_rate': 0.020699287146969707, 'num_leaves': 179, 'max_depth': 12, 'min_child_samples': 14, 'subsample': 0.9037551763829241, 'colsample_bytree': 0.9450995068248447, 'reg_alpha': 5.481545722672281, 'reg_lambda': 3.900354323225701, 'min_split_gain': 0.005332237526080114}. Best is trial 0 with value: 0.011003564167803714.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:51:59,164] Trial 3 finished with value: 0.011901895403598215 and parameters: {'n_estimators': 189, 'learning_rate': 0.04474026112518221, 'num_leaves': 75, 'max_depth': 15, 'min_child_samples': 24, 'subsample': 0.5191923255361979, 'colsample_bytree': 0.7696211962638975, 'reg_alpha': 8.67810397218115, 'reg_lambda': 6.89113559476856, 'min_split_gain': 0.01989567245179179}. Best is trial 0 with value: 0.011003564167803714.
[I 2025-06-15 19:51:59,166] A new study created in memory with name: no-name-e62f0ab9-e39d-442e-997b-b780bd53a200
[I 2025-06-15 19:51:59,218] Trial 0 finished with value: 0.03076947676975872 and parameters: {'n_estimators': 290, 'learning_rate': 0.17082240670033735, 'num_leaves': 226, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.7551007253036843, 'colsample_bytree': 0.8567183044215103, 'reg_alpha': 0.1904851166401733, 'reg_lambda': 3.8570441853394533, 'min_split_gain': 0.031502821440315264}. Best is trial 0 with value: 0.03076947676975872.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         730
0           Price_lag_1         724
14      pct_chg_Load_DA         496
3         Load_DA_lag_0         452
2           Price_lag_7         445
11           NGas_lag_2         373
16  roll2d_median_Price         354
5       WindOn_DA_lag_0         303
13            EUA_lag_2         290
15       lag168_Load_DA         273
10           Coal_lag_2         261
12            Oil_lag_2         256
7                  WD_1          21
9                  WD_7           7
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          35
2           Price_lag_7          10
1           Price_lag_2           5
3         Load_DA_lag_0           2
15       lag168_Load_DA           1
5       WindOn_DA_lag_0           1
12            Oil_lag_2           1
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 18.60% done
********************* END NS.. 135 ****************************************************
********************* START NS ... 136 ****************************************************
[I 2025-06-15 19:52:00,191] A new study created in memory with name: no-name-d53ad5cc-65bb-4ab9-ad95-170986cf41c7
[I 2025-06-15 19:52:00,326] Trial 0 finished with value: 0.008548115352994446 and parameters: {'n_estimators': 60, 'learning_rate': 0.14859090298526767, 'num_leaves': 76, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.6125833470667943, 'colsample_bytree': 0.5943542202868339, 'reg_alpha': 7.257065569996933, 'reg_lambda': 5.013996700422449, 'min_split_gain': 0.02283624029429436}. Best is trial 0 with value: 0.008548115352994446.
[Trial 0] New best: 0.00855
[I 2025-06-15 19:52:00,878] Trial 1 finished with value: 0.011548149040834588 and parameters: {'n_estimators': 115, 'learning_rate': 0.01759204937407435, 'num_leaves': 103, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.5752950514606568, 'colsample_bytree': 0.7793972728238399, 'reg_alpha': 6.089644615638039, 'reg_lambda': 2.7019761191533833, 'min_split_gain': 0.04460084706392925}. Best is trial 0 with value: 0.008548115352994446.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:01,478] Trial 2 finished with value: 0.009974337723519237 and parameters: {'n_estimators': 172, 'learning_rate': 0.03051478949601299, 'num_leaves': 53, 'max_depth': 14, 'min_child_samples': 19, 'subsample': 0.5986668785684406, 'colsample_bytree': 0.5700876667447272, 'reg_alpha': 7.718907442311532, 'reg_lambda': 4.5864604706499215, 'min_split_gain': 0.046075527104819256}. Best is trial 0 with value: 0.008548115352994446.
[I 2025-06-15 19:52:01,667] Trial 3 finished with value: 0.007026321733528881 and parameters: {'n_estimators': 111, 'learning_rate': 0.13195757821468934, 'num_leaves': 100, 'max_depth': 5, 'min_child_samples': 29, 'subsample': 0.5304347004346615, 'colsample_bytree': 0.9325451499807362, 'reg_alpha': 5.629826583855699, 'reg_lambda': 5.437136462766842, 'min_split_gain': 0.04669718178163416}. Best is trial 3 with value: 0.007026321733528881.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] New best: 0.00703
[I 2025-06-15 19:52:02,347] Trial 4 finished with value: 0.010615279281207632 and parameters: {'n_estimators': 107, 'learning_rate': 0.04420130657192835, 'num_leaves': 202, 'max_depth': 7, 'min_child_samples': 1, 'subsample': 0.5133611119799548, 'colsample_bytree': 0.5193143562470883, 'reg_alpha': 2.70248776989666, 'reg_lambda': 3.3430471653604945, 'min_split_gain': 0.012990397758451245}. Best is trial 3 with value: 0.007026321733528881.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:02,880] Trial 5 finished with value: 0.007831549306739485 and parameters: {'n_estimators': 170, 'learning_rate': 0.1341486549320104, 'num_leaves': 53, 'max_depth': 12, 'min_child_samples': 7, 'subsample': 0.9228185919132532, 'colsample_bytree': 0.7984172859157396, 'reg_alpha': 2.541193807879212, 'reg_lambda': 1.16892458153389, 'min_split_gain': 0.011255051838359748}. Best is trial 3 with value: 0.007026321733528881.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:52:04,059] Trial 6 finished with value: 0.008937665716908245 and parameters: {'n_estimators': 226, 'learning_rate': 0.012646524619681195, 'num_leaves': 158, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.6692090191197416, 'colsample_bytree': 0.9125706576829806, 'reg_alpha': 3.701235733629933, 'reg_lambda': 0.04704840065776539, 'min_split_gain': 0.005999091370224436}. Best is trial 3 with value: 0.007026321733528881.
[I 2025-06-15 19:52:04,063] A new study created in memory with name: no-name-9b05f841-4f5b-4c20-92ad-ed5765583085
[I 2025-06-15 19:52:04,137] Trial 0 finished with value: 0.021369675758936557 and parameters: {'n_estimators': 263, 'learning_rate': 0.04455469717021187, 'num_leaves': 105, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.7249719597552746, 'colsample_bytree': 0.7143790559149594, 'reg_alpha': 0.8386160087240668, 'reg_lambda': 7.934180581045101, 'min_split_gain': 0.00599169502280289}. Best is trial 0 with value: 0.021369675758936557.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         331
14      pct_chg_Load_DA         179
1           Price_lag_2         110
11           NGas_lag_2          83
3         Load_DA_lag_0          69
2           Price_lag_7          65
16  roll2d_median_Price          64
15       lag168_Load_DA          41
12            Oil_lag_2          40
13            EUA_lag_2          39
10           Coal_lag_2          35
5       WindOn_DA_lag_0           5
7                  WD_1           3
8                  WD_6           1
6      WindOff_DA_lag_0           0
9                  WD_7           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         119
2           Price_lag_7          36
3         Load_DA_lag_0          30
5       WindOn_DA_lag_0          26
1           Price_lag_2          17
15       lag168_Load_DA          11
12            Oil_lag_2           4
16  roll2d_median_Price           3
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
6      WindOff_DA_lag_0           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 18.74% done
********************* END NS.. 136 ****************************************************
********************* START NS ... 137 ****************************************************
[I 2025-06-15 19:52:05,047] A new study created in memory with name: no-name-38cd89b2-8437-461f-95fb-82e6788857a7
[I 2025-06-15 19:52:05,875] Trial 0 finished with value: 0.06892667693674871 and parameters: {'n_estimators': 65, 'learning_rate': 0.011946165274367319, 'num_leaves': 175, 'max_depth': 14, 'min_child_samples': 15, 'subsample': 0.9297888646076856, 'colsample_bytree': 0.6734507611968504, 'reg_alpha': 1.7472502421116543, 'reg_lambda': 6.708058744896277, 'min_split_gain': 0.035314475052502445}. Best is trial 0 with value: 0.06892667693674871.
[Trial 0] New best: 0.06893
[I 2025-06-15 19:52:06,482] Trial 1 finished with value: 0.014823016680213415 and parameters: {'n_estimators': 196, 'learning_rate': 0.1383507789486249, 'num_leaves': 81, 'max_depth': 8, 'min_child_samples': 5, 'subsample': 0.7790492657140875, 'colsample_bytree': 0.7033670579892233, 'reg_alpha': 7.2075742678768755, 'reg_lambda': 3.133863899972412, 'min_split_gain': 0.023641464914956025}. Best is trial 1 with value: 0.014823016680213415.
[Trial 1] New best: 0.01482
[I 2025-06-15 19:52:07,638] Trial 2 finished with value: 0.01907041767999208 and parameters: {'n_estimators': 122, 'learning_rate': 0.03279066710657916, 'num_leaves': 78, 'max_depth': 7, 'min_child_samples': 29, 'subsample': 0.7124784556269332, 'colsample_bytree': 0.5364753028418106, 'reg_alpha': 3.1763126356860716, 'reg_lambda': 9.404860002295333, 'min_split_gain': 0.0401843585545464}. Best is trial 1 with value: 0.014823016680213415.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:08,128] Trial 3 finished with value: 0.013377797109534392 and parameters: {'n_estimators': 229, 'learning_rate': 0.15137152199241866, 'num_leaves': 183, 'max_depth': 16, 'min_child_samples': 6, 'subsample': 0.8734414651197319, 'colsample_bytree': 0.7687007346657295, 'reg_alpha': 9.2684200702915, 'reg_lambda': 4.844954576139763, 'min_split_gain': 0.0198980944182041}. Best is trial 3 with value: 0.013377797109534392.
[Trial 3] New best: 0.01338
[I 2025-06-15 19:52:08,377] Trial 4 finished with value: 0.01582816471364552 and parameters: {'n_estimators': 50, 'learning_rate': 0.08161158350757974, 'num_leaves': 57, 'max_depth': 6, 'min_child_samples': 23, 'subsample': 0.7945892358740085, 'colsample_bytree': 0.7573411313050711, 'reg_alpha': 2.0645710012646603, 'reg_lambda': 5.558479930954515, 'min_split_gain': 0.026901193999963477}. Best is trial 3 with value: 0.013377797109534392.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:08,860] Trial 5 finished with value: 0.027540144714160542 and parameters: {'n_estimators': 123, 'learning_rate': 0.015895159359922455, 'num_leaves': 173, 'max_depth': 5, 'min_child_samples': 26, 'subsample': 0.7200542420893359, 'colsample_bytree': 0.6213148348352272, 'reg_alpha': 9.894568747321674, 'reg_lambda': 0.37014950739839403, 'min_split_gain': 0.04638455963182425}. Best is trial 3 with value: 0.013377797109534392.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:52:09,398] Trial 6 finished with value: 0.01530111943156841 and parameters: {'n_estimators': 113, 'learning_rate': 0.13750876696570796, 'num_leaves': 182, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.5543455468415066, 'colsample_bytree': 0.7426431609868528, 'reg_alpha': 2.670357529259374, 'reg_lambda': 5.503684806141569, 'min_split_gain': 0.012632587228999216}. Best is trial 3 with value: 0.013377797109534392.
[I 2025-06-15 19:52:09,401] A new study created in memory with name: no-name-0d876f90-a3d5-43f6-b549-880bea67763b
[I 2025-06-15 19:52:09,458] Trial 0 finished with value: 0.11606699458457766 and parameters: {'n_estimators': 242, 'learning_rate': 0.017473619126104158, 'num_leaves': 206, 'max_depth': 10, 'min_child_samples': 30, 'subsample': 0.7499037986150867, 'colsample_bytree': 0.9432930880074832, 'reg_alpha': 7.842959570054978, 'reg_lambda': 5.981706761076641, 'min_split_gain': 0.038869788421307845}. Best is trial 0 with value: 0.11606699458457766.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         315
1           Price_lag_2         205
14      pct_chg_Load_DA         170
2           Price_lag_7         101
16  roll2d_median_Price          97
11           NGas_lag_2          96
3         Load_DA_lag_0          90
10           Coal_lag_2          75
13            EUA_lag_2          73
15       lag168_Load_DA          72
12            Oil_lag_2          61
5       WindOn_DA_lag_0          43
7                  WD_1           4
9                  WD_7           3
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         142
1           Price_lag_2          10
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 18.88% done
********************* END NS.. 137 ****************************************************
********************* START NS ... 138 ****************************************************
[I 2025-06-15 19:52:11,151] A new study created in memory with name: no-name-c49337a6-bed0-4cc9-b5df-b93ca7f4b089
[I 2025-06-15 19:52:12,173] Trial 0 finished with value: 0.01218995936570935 and parameters: {'n_estimators': 136, 'learning_rate': 0.23520968643440548, 'num_leaves': 90, 'max_depth': 8, 'min_child_samples': 1, 'subsample': 0.6876760709608944, 'colsample_bytree': 0.6161818171184038, 'reg_alpha': 6.990538372761424, 'reg_lambda': 9.310948166862618, 'min_split_gain': 0.0163309465236904}. Best is trial 0 with value: 0.01218995936570935.
[Trial 0] New best: 0.01219
[I 2025-06-15 19:52:14,902] Trial 1 finished with value: 0.011699372822408648 and parameters: {'n_estimators': 277, 'learning_rate': 0.04473528319988966, 'num_leaves': 193, 'max_depth': 12, 'min_child_samples': 7, 'subsample': 0.7863179758095773, 'colsample_bytree': 0.8694505320857924, 'reg_alpha': 4.84805190361811, 'reg_lambda': 5.290312480386844, 'min_split_gain': 0.048927347255817884}. Best is trial 1 with value: 0.011699372822408648.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:15,919] Trial 2 finished with value: 0.020011340002583224 and parameters: {'n_estimators': 132, 'learning_rate': 0.011484175548273303, 'num_leaves': 234, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.5524380759458314, 'colsample_bytree': 0.7577684217164524, 'reg_alpha': 6.537778168884581, 'reg_lambda': 0.7388178346683727, 'min_split_gain': 0.04399220998379419}. Best is trial 1 with value: 0.011699372822408648.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:52:17,069] Trial 3 finished with value: 0.0127182035485641 and parameters: {'n_estimators': 145, 'learning_rate': 0.04525037724095438, 'num_leaves': 192, 'max_depth': 14, 'min_child_samples': 27, 'subsample': 0.7028949028234885, 'colsample_bytree': 0.6200669699778403, 'reg_alpha': 4.231684353462516, 'reg_lambda': 6.8639751715477715, 'min_split_gain': 0.03220811961962797}. Best is trial 1 with value: 0.011699372822408648.
[I 2025-06-15 19:52:17,072] A new study created in memory with name: no-name-0688d14c-6d4e-4755-919a-98e6d079eb81
[I 2025-06-15 19:52:17,144] Trial 0 finished with value: 0.06683433664077287 and parameters: {'n_estimators': 205, 'learning_rate': 0.10167116103047963, 'num_leaves': 50, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.6304715995983083, 'colsample_bytree': 0.5185370612146881, 'reg_alpha': 9.229300354330558, 'reg_lambda': 1.5533824498656323, 'min_split_gain': 0.0032879118545060208}. Best is trial 0 with value: 0.06683433664077287.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1175
1           Price_lag_2         736
14      pct_chg_Load_DA         701
3         Load_DA_lag_0         389
11           NGas_lag_2         387
2           Price_lag_7         351
16  roll2d_median_Price         252
10           Coal_lag_2         222
12            Oil_lag_2         193
15       lag168_Load_DA         188
13            EUA_lag_2         154
5       WindOn_DA_lag_0          88
7                  WD_1          11
9                  WD_7           9
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          38
1           Price_lag_2           6
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 19.02% done
********************* END NS.. 138 ****************************************************
********************* START NS ... 139 ****************************************************
[I 2025-06-15 19:52:19,343] A new study created in memory with name: no-name-4a9d358f-48b8-4499-bd2c-10de903798f2
[I 2025-06-15 19:52:20,251] Trial 0 finished with value: 0.006076229456112181 and parameters: {'n_estimators': 95, 'learning_rate': 0.13530570938471048, 'num_leaves': 228, 'max_depth': 12, 'min_child_samples': 1, 'subsample': 0.8929316192328784, 'colsample_bytree': 0.7526588367738665, 'reg_alpha': 7.204906564672616, 'reg_lambda': 2.780401327465075, 'min_split_gain': 0.0014007057718826877}. Best is trial 0 with value: 0.006076229456112181.
[Trial 0] New best: 0.00608
[I 2025-06-15 19:52:20,814] Trial 1 finished with value: 0.006729912787282605 and parameters: {'n_estimators': 75, 'learning_rate': 0.05927335332321093, 'num_leaves': 223, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.9887899732115963, 'colsample_bytree': 0.7734827951785946, 'reg_alpha': 2.7824045404627986, 'reg_lambda': 3.890051434574353, 'min_split_gain': 0.005765178285520984}. Best is trial 0 with value: 0.006076229456112181.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:21,450] Trial 2 finished with value: 0.00582842289128601 and parameters: {'n_estimators': 98, 'learning_rate': 0.15292274254410496, 'num_leaves': 247, 'max_depth': 15, 'min_child_samples': 10, 'subsample': 0.6703497709866441, 'colsample_bytree': 0.7283780117355105, 'reg_alpha': 4.216384847708438, 'reg_lambda': 9.90222854281959, 'min_split_gain': 0.006195617110814966}. Best is trial 2 with value: 0.00582842289128601.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:52:23,002] Trial 3 finished with value: 0.006540178664354424 and parameters: {'n_estimators': 238, 'learning_rate': 0.03687186976536018, 'num_leaves': 221, 'max_depth': 12, 'min_child_samples': 19, 'subsample': 0.9270725885249333, 'colsample_bytree': 0.8866822931252549, 'reg_alpha': 9.026150819958026, 'reg_lambda': 8.246098271940449, 'min_split_gain': 0.00837418254601599}. Best is trial 2 with value: 0.00582842289128601.
[I 2025-06-15 19:52:23,007] A new study created in memory with name: no-name-ea4965dd-a93c-42bf-bd1e-b13e6fbb0158
[I 2025-06-15 19:52:23,109] Trial 0 finished with value: 0.04127886929358466 and parameters: {'n_estimators': 246, 'learning_rate': 0.10866894046981714, 'num_leaves': 228, 'max_depth': 15, 'min_child_samples': 22, 'subsample': 0.868020138144211, 'colsample_bytree': 0.5364079020045951, 'reg_alpha': 9.81262660230084, 'reg_lambda': 5.644396176607552, 'min_split_gain': 0.025937202933620142}. Best is trial 0 with value: 0.04127886929358466.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         497
1           Price_lag_2         436
14      pct_chg_Load_DA         412
3         Load_DA_lag_0         353
5       WindOn_DA_lag_0         314
2           Price_lag_7         248
11           NGas_lag_2         227
15       lag168_Load_DA         226
16  roll2d_median_Price         206
13            EUA_lag_2         200
10           Coal_lag_2         192
12            Oil_lag_2         143
9                  WD_7           7
7                  WD_1           7
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          18
1           Price_lag_2           3
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 19.15% done
********************* END NS.. 139 ****************************************************
********************* START NS ... 140 ****************************************************
[I 2025-06-15 19:52:24,724] A new study created in memory with name: no-name-2ac5e597-90d5-469f-9823-12609b2fe177
[I 2025-06-15 19:52:25,131] Trial 0 finished with value: 0.011647848193911023 and parameters: {'n_estimators': 53, 'learning_rate': 0.10125419994180788, 'num_leaves': 131, 'max_depth': 7, 'min_child_samples': 6, 'subsample': 0.833349104612869, 'colsample_bytree': 0.9436172285492874, 'reg_alpha': 7.6494280895907805, 'reg_lambda': 6.918355583705927, 'min_split_gain': 0.041562755352014674}. Best is trial 0 with value: 0.011647848193911023.
[Trial 0] New best: 0.01165
[I 2025-06-15 19:52:26,508] Trial 1 finished with value: 0.01180086011234797 and parameters: {'n_estimators': 136, 'learning_rate': 0.04265672165498629, 'num_leaves': 133, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.5807546343288215, 'colsample_bytree': 0.7090629576356144, 'reg_alpha': 2.8062455245760107, 'reg_lambda': 7.85054122650806, 'min_split_gain': 0.011678677564070256}. Best is trial 0 with value: 0.011647848193911023.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:28,691] Trial 2 finished with value: 0.011858025964799632 and parameters: {'n_estimators': 274, 'learning_rate': 0.019950397662271076, 'num_leaves': 245, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.9132196072536675, 'colsample_bytree': 0.6492008161717386, 'reg_alpha': 0.03520787371249723, 'reg_lambda': 2.247696143675072, 'min_split_gain': 0.0074500354730422606}. Best is trial 0 with value: 0.011647848193911023.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:52:29,519] Trial 3 finished with value: 0.012463996328447921 and parameters: {'n_estimators': 200, 'learning_rate': 0.04711886214180147, 'num_leaves': 95, 'max_depth': 14, 'min_child_samples': 8, 'subsample': 0.8039035034167357, 'colsample_bytree': 0.6654401412815929, 'reg_alpha': 9.08787566764759, 'reg_lambda': 7.497790770433718, 'min_split_gain': 0.008180655618288518}. Best is trial 0 with value: 0.011647848193911023.
[I 2025-06-15 19:52:29,522] A new study created in memory with name: no-name-18148cf7-6be9-4b32-8029-1673a39238a1
[I 2025-06-15 19:52:29,558] Trial 0 finished with value: 0.02813396008640844 and parameters: {'n_estimators': 89, 'learning_rate': 0.07177365207949922, 'num_leaves': 102, 'max_depth': 5, 'min_child_samples': 29, 'subsample': 0.5797906990068671, 'colsample_bytree': 0.8250961845993755, 'reg_alpha': 0.8252372899455795, 'reg_lambda': 3.275700634406886, 'min_split_gain': 0.043162598986167505}. Best is trial 0 with value: 0.02813396008640844.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         426
14      pct_chg_Load_DA         220
1           Price_lag_2         195
16  roll2d_median_Price         110
2           Price_lag_7         108
11           NGas_lag_2         107
3         Load_DA_lag_0          80
10           Coal_lag_2          65
13            EUA_lag_2          49
12            Oil_lag_2          47
15       lag168_Load_DA          38
5       WindOn_DA_lag_0          19
9                  WD_7           4
8                  WD_6           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1          64
5       WindOn_DA_lag_0           7
15       lag168_Load_DA           6
1           Price_lag_2           6
16  roll2d_median_Price           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
3         Load_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 19.29% done
********************* END NS.. 140 ****************************************************
********************* START NS ... 141 ****************************************************
[I 2025-06-15 19:52:30,402] A new study created in memory with name: no-name-ccf11c57-2547-4f09-a5f2-f5084bc479a4
[I 2025-06-15 19:52:31,314] Trial 0 finished with value: 0.020581505764979628 and parameters: {'n_estimators': 260, 'learning_rate': 0.019337667105765018, 'num_leaves': 228, 'max_depth': 9, 'min_child_samples': 27, 'subsample': 0.6094538888244603, 'colsample_bytree': 0.746337250875964, 'reg_alpha': 8.44700698629666, 'reg_lambda': 3.386618141396603, 'min_split_gain': 0.02722049983533642}. Best is trial 0 with value: 0.020581505764979628.
[Trial 0] New best: 0.02058
[I 2025-06-15 19:52:31,796] Trial 1 finished with value: 0.016427083634201724 and parameters: {'n_estimators': 110, 'learning_rate': 0.08805063131232563, 'num_leaves': 156, 'max_depth': 10, 'min_child_samples': 17, 'subsample': 0.8552227389085438, 'colsample_bytree': 0.982421121578279, 'reg_alpha': 2.8668362269487293, 'reg_lambda': 8.072826404363449, 'min_split_gain': 0.029687438575213304}. Best is trial 1 with value: 0.016427083634201724.
[Trial 1] New best: 0.01643
[I 2025-06-15 19:52:32,196] Trial 2 finished with value: 0.03662521891025769 and parameters: {'n_estimators': 93, 'learning_rate': 0.01721118113878341, 'num_leaves': 44, 'max_depth': 8, 'min_child_samples': 24, 'subsample': 0.866935512697814, 'colsample_bytree': 0.9706654627404985, 'reg_alpha': 7.686404476764276, 'reg_lambda': 6.03239584268774, 'min_split_gain': 0.006928950894064734}. Best is trial 1 with value: 0.016427083634201724.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:32,495] Trial 3 finished with value: 0.0175382899305422 and parameters: {'n_estimators': 212, 'learning_rate': 0.22901755237187, 'num_leaves': 181, 'max_depth': 10, 'min_child_samples': 22, 'subsample': 0.9980044593299048, 'colsample_bytree': 0.9580732262977367, 'reg_alpha': 7.000223452076657, 'reg_lambda': 0.7328485745961155, 'min_split_gain': 0.008047921644827889}. Best is trial 1 with value: 0.016427083634201724.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:52:33,010] Trial 4 finished with value: 0.01633551831506069 and parameters: {'n_estimators': 122, 'learning_rate': 0.0789976377329314, 'num_leaves': 188, 'max_depth': 6, 'min_child_samples': 9, 'subsample': 0.7259137206600774, 'colsample_bytree': 0.6582807983227925, 'reg_alpha': 2.1192195818209125, 'reg_lambda': 8.13244594243365, 'min_split_gain': 0.012938750643432084}. Best is trial 4 with value: 0.01633551831506069.
[I 2025-06-15 19:52:33,013] A new study created in memory with name: no-name-d5288d99-9606-4f72-966b-2c35c2e96877
[I 2025-06-15 19:52:33,072] Trial 0 finished with value: 0.050757961337548285 and parameters: {'n_estimators': 226, 'learning_rate': 0.12020915892288185, 'num_leaves': 210, 'max_depth': 12, 'min_child_samples': 10, 'subsample': 0.6957383637094082, 'colsample_bytree': 0.6556131858081173, 'reg_alpha': 0.2668796487546854, 'reg_lambda': 2.663920936493361, 'min_split_gain': 0.013399940017809043}. Best is trial 0 with value: 0.050757961337548285.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         842
1           Price_lag_2         563
14      pct_chg_Load_DA         483
3         Load_DA_lag_0         482
11           NGas_lag_2         322
2           Price_lag_7         274
5       WindOn_DA_lag_0         251
15       lag168_Load_DA         232
16  roll2d_median_Price         229
13            EUA_lag_2         203
10           Coal_lag_2         176
12            Oil_lag_2         144
9                  WD_7          19
7                  WD_1          18
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          61
2           Price_lag_7          20
15       lag168_Load_DA          19
5       WindOn_DA_lag_0          18
1           Price_lag_2           9
16  roll2d_median_Price           5
3         Load_DA_lag_0           3
10           Coal_lag_2           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 19.43% done
********************* END NS.. 141 ****************************************************
********************* START NS ... 142 ****************************************************
[I 2025-06-15 19:52:34,088] A new study created in memory with name: no-name-b1579c52-ad77-48bd-9dd1-6f2d2bb5b743
[I 2025-06-15 19:52:34,529] Trial 0 finished with value: 0.009662602381505096 and parameters: {'n_estimators': 204, 'learning_rate': 0.06781337882045635, 'num_leaves': 250, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.7063847847862642, 'colsample_bytree': 0.9337946199992633, 'reg_alpha': 4.268578025328776, 'reg_lambda': 2.713743077293096, 'min_split_gain': 0.033670815645357746}. Best is trial 0 with value: 0.009662602381505096.
[Trial 0] New best: 0.00966
[I 2025-06-15 19:52:34,962] Trial 1 finished with value: 0.010071115297068268 and parameters: {'n_estimators': 298, 'learning_rate': 0.22457787132825324, 'num_leaves': 132, 'max_depth': 15, 'min_child_samples': 22, 'subsample': 0.5532372212177004, 'colsample_bytree': 0.9271117890444454, 'reg_alpha': 5.901075109706843, 'reg_lambda': 4.769390522114071, 'min_split_gain': 0.005558583621950531}. Best is trial 0 with value: 0.009662602381505096.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:35,899] Trial 2 finished with value: 0.01112055373335536 and parameters: {'n_estimators': 185, 'learning_rate': 0.03400775980534089, 'num_leaves': 149, 'max_depth': 14, 'min_child_samples': 13, 'subsample': 0.6851730373371502, 'colsample_bytree': 0.7474537964212851, 'reg_alpha': 7.943417415513203, 'reg_lambda': 7.757291737440955, 'min_split_gain': 0.012871829298379434}. Best is trial 0 with value: 0.009662602381505096.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:52:36,641] Trial 3 finished with value: 0.008705793437102106 and parameters: {'n_estimators': 118, 'learning_rate': 0.05511944479220134, 'num_leaves': 49, 'max_depth': 16, 'min_child_samples': 12, 'subsample': 0.856303665946879, 'colsample_bytree': 0.7762626384759383, 'reg_alpha': 2.3054111408951785, 'reg_lambda': 0.06337852974952818, 'min_split_gain': 0.030912388220165756}. Best is trial 3 with value: 0.008705793437102106.
[I 2025-06-15 19:52:36,645] A new study created in memory with name: no-name-c8fc1604-5cc0-4289-a8e4-cfe780bf3ab4
[I 2025-06-15 19:52:36,729] Trial 0 finished with value: 0.07307762836702116 and parameters: {'n_estimators': 288, 'learning_rate': 0.023220100437505813, 'num_leaves': 238, 'max_depth': 8, 'min_child_samples': 30, 'subsample': 0.6815544431780205, 'colsample_bytree': 0.6138692539827795, 'reg_alpha': 7.763084085260919, 'reg_lambda': 0.4815284586658075, 'min_split_gain': 0.04736607246131625}. Best is trial 0 with value: 0.07307762836702116.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1153
1           Price_lag_2         864
14      pct_chg_Load_DA         780
3         Load_DA_lag_0         509
11           NGas_lag_2         382
16  roll2d_median_Price         321
2           Price_lag_7         317
15       lag168_Load_DA         277
5       WindOn_DA_lag_0         223
10           Coal_lag_2         192
12            Oil_lag_2         189
13            EUA_lag_2         165
9                  WD_7          15
7                  WD_1           5
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         107
1           Price_lag_2          14
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 19.56% done
********************* END NS.. 142 ****************************************************
********************* START NS ... 143 ****************************************************
[I 2025-06-15 19:52:38,058] A new study created in memory with name: no-name-f433acd5-737c-4b9e-8a12-81d0f5d56891
[I 2025-06-15 19:52:38,721] Trial 0 finished with value: 0.008301333760617697 and parameters: {'n_estimators': 116, 'learning_rate': 0.01954283052829451, 'num_leaves': 116, 'max_depth': 15, 'min_child_samples': 24, 'subsample': 0.8239060514313494, 'colsample_bytree': 0.7105156457022959, 'reg_alpha': 1.2759309483442183, 'reg_lambda': 9.339538229874154, 'min_split_gain': 0.012137472704971381}. Best is trial 0 with value: 0.008301333760617697.
[Trial 0] New best: 0.00830
[I 2025-06-15 19:52:40,059] Trial 1 finished with value: 0.006399514748192251 and parameters: {'n_estimators': 246, 'learning_rate': 0.02101413389786857, 'num_leaves': 57, 'max_depth': 11, 'min_child_samples': 2, 'subsample': 0.90663124082352, 'colsample_bytree': 0.9407846846660115, 'reg_alpha': 9.986153213386267, 'reg_lambda': 6.9096480913784095, 'min_split_gain': 0.04450790662759083}. Best is trial 1 with value: 0.006399514748192251.
[Trial 1] New best: 0.00640
[I 2025-06-15 19:52:41,560] Trial 2 finished with value: 0.008656532545410496 and parameters: {'n_estimators': 286, 'learning_rate': 0.028734958786547995, 'num_leaves': 69, 'max_depth': 8, 'min_child_samples': 20, 'subsample': 0.9320854986750804, 'colsample_bytree': 0.5485084986468751, 'reg_alpha': 6.519947794955493, 'reg_lambda': 7.973015245032849, 'min_split_gain': 0.04879192756700132}. Best is trial 1 with value: 0.006399514748192251.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:42,194] Trial 3 finished with value: 0.006607100286987219 and parameters: {'n_estimators': 299, 'learning_rate': 0.10473304754529719, 'num_leaves': 245, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.6875779714765704, 'colsample_bytree': 0.8025511023828399, 'reg_alpha': 9.55842975930767, 'reg_lambda': 9.25727068560874, 'min_split_gain': 0.009674801447556891}. Best is trial 1 with value: 0.006399514748192251.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:52:42,907] Trial 4 finished with value: 0.008812039955012183 and parameters: {'n_estimators': 95, 'learning_rate': 0.02319806465330038, 'num_leaves': 87, 'max_depth': 12, 'min_child_samples': 18, 'subsample': 0.6575795611050212, 'colsample_bytree': 0.7182663659416351, 'reg_alpha': 5.913901092469935, 'reg_lambda': 3.32985127229388, 'min_split_gain': 0.022039232623954755}. Best is trial 1 with value: 0.006399514748192251.
[I 2025-06-15 19:52:42,910] A new study created in memory with name: no-name-c7429e6f-29f2-4e87-ab4c-f938abd02053
[I 2025-06-15 19:52:42,975] Trial 0 finished with value: 0.014739276210436242 and parameters: {'n_estimators': 287, 'learning_rate': 0.03937024036304926, 'num_leaves': 101, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.9726397016200625, 'colsample_bytree': 0.5578732148552674, 'reg_alpha': 7.546882841915724, 'reg_lambda': 8.16859939019404, 'min_split_gain': 0.027848863965558364}. Best is trial 0 with value: 0.014739276210436242.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1955
1           Price_lag_2        1055
14      pct_chg_Load_DA         938
2           Price_lag_7         537
11           NGas_lag_2         480
16  roll2d_median_Price         468
10           Coal_lag_2         366
3         Load_DA_lag_0         309
13            EUA_lag_2         247
12            Oil_lag_2         231
5       WindOn_DA_lag_0         178
15       lag168_Load_DA         140
9                  WD_7          12
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          69
1           Price_lag_2          26
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 19.70% done
********************* END NS.. 143 ****************************************************
********************* START NS ... 144 ****************************************************
[I 2025-06-15 19:52:44,699] A new study created in memory with name: no-name-d7c3929b-0889-4154-a60f-d42f9649f570
[I 2025-06-15 19:52:45,335] Trial 0 finished with value: 0.006669278793199587 and parameters: {'n_estimators': 104, 'learning_rate': 0.02779779618496057, 'num_leaves': 200, 'max_depth': 14, 'min_child_samples': 16, 'subsample': 0.626526688631697, 'colsample_bytree': 0.9742534689618096, 'reg_alpha': 6.33215828941178, 'reg_lambda': 6.2413914082724675, 'min_split_gain': 0.0011110355466500688}. Best is trial 0 with value: 0.006669278793199587.
[Trial 0] New best: 0.00667
[I 2025-06-15 19:52:45,648] Trial 1 finished with value: 0.007197824878241183 and parameters: {'n_estimators': 50, 'learning_rate': 0.12899245086147595, 'num_leaves': 247, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.5453095225843945, 'colsample_bytree': 0.7754740508916707, 'reg_alpha': 7.166310457486668, 'reg_lambda': 4.614515805983888, 'min_split_gain': 0.0065427855596923105}. Best is trial 0 with value: 0.006669278793199587.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:45,942] Trial 2 finished with value: 0.007273683822220367 and parameters: {'n_estimators': 151, 'learning_rate': 0.24690264483433005, 'num_leaves': 92, 'max_depth': 12, 'min_child_samples': 25, 'subsample': 0.6721362501432306, 'colsample_bytree': 0.6866457529086106, 'reg_alpha': 9.422265293643562, 'reg_lambda': 1.7514083042226103, 'min_split_gain': 0.03664704490672612}. Best is trial 0 with value: 0.006669278793199587.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:52:49,129] Trial 3 finished with value: 0.005189230328448531 and parameters: {'n_estimators': 261, 'learning_rate': 0.027878799584226194, 'num_leaves': 187, 'max_depth': 15, 'min_child_samples': 27, 'subsample': 0.854028004478681, 'colsample_bytree': 0.9511859104457048, 'reg_alpha': 1.163712574641319, 'reg_lambda': 5.589627997449631, 'min_split_gain': 0.003142490636233053}. Best is trial 3 with value: 0.005189230328448531.
[Trial 3] New best: 0.00519
[I 2025-06-15 19:52:49,853] Trial 4 finished with value: 0.006298976526179033 and parameters: {'n_estimators': 165, 'learning_rate': 0.04932858270769817, 'num_leaves': 76, 'max_depth': 7, 'min_child_samples': 25, 'subsample': 0.6453208443362579, 'colsample_bytree': 0.7497235487321006, 'reg_alpha': 0.9927036097790742, 'reg_lambda': 7.4485122578246425, 'min_split_gain': 0.040635640668217204}. Best is trial 3 with value: 0.005189230328448531.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:50,074] Trial 5 finished with value: 0.007600323867913798 and parameters: {'n_estimators': 97, 'learning_rate': 0.2971717428044253, 'num_leaves': 38, 'max_depth': 12, 'min_child_samples': 10, 'subsample': 0.9018376111692319, 'colsample_bytree': 0.6646658142453692, 'reg_alpha': 7.6033908389886005, 'reg_lambda': 3.731602935388226, 'min_split_gain': 0.005444468951896886}. Best is trial 3 with value: 0.005189230328448531.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:52:50,280] Trial 6 finished with value: 0.012522810443570865 and parameters: {'n_estimators': 78, 'learning_rate': 0.027317383448395717, 'num_leaves': 117, 'max_depth': 4, 'min_child_samples': 2, 'subsample': 0.9180385539418267, 'colsample_bytree': 0.6217933846729704, 'reg_alpha': 6.329824834713486, 'reg_lambda': 6.363377912123001, 'min_split_gain': 0.0396744836429303}. Best is trial 3 with value: 0.005189230328448531.
[I 2025-06-15 19:52:50,282] A new study created in memory with name: no-name-1b9dbbe3-d805-4056-bb19-a2f006729596
[I 2025-06-15 19:52:50,345] Trial 0 finished with value: 0.013310662302550602 and parameters: {'n_estimators': 149, 'learning_rate': 0.04133266189192102, 'num_leaves': 233, 'max_depth': 12, 'min_child_samples': 19, 'subsample': 0.9425966610504095, 'colsample_bytree': 0.8142383146719268, 'reg_alpha': 6.55782945335598, 'reg_lambda': 1.3245678522512383, 'min_split_gain': 0.008748586687355403}. Best is trial 0 with value: 0.013310662302550602.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        4547
14      pct_chg_Load_DA        3850
1           Price_lag_2        3578
3         Load_DA_lag_0        2619
15       lag168_Load_DA        2000
5       WindOn_DA_lag_0        1972
2           Price_lag_7        1963
16  roll2d_median_Price        1820
11           NGas_lag_2        1678
13            EUA_lag_2        1528
12            Oil_lag_2        1298
10           Coal_lag_2        1186
7                  WD_1          66
9                  WD_7          49
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         106
1           Price_lag_2          16
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 19.84% done
********************* END NS.. 144 ****************************************************
********************* START NS ... 145 ****************************************************
[I 2025-06-15 19:52:53,253] A new study created in memory with name: no-name-983aabdc-f912-4c1f-b08b-aeb62ab3fb7c
[I 2025-06-15 19:52:53,913] Trial 0 finished with value: 0.006053199748856122 and parameters: {'n_estimators': 287, 'learning_rate': 0.015075019581426949, 'num_leaves': 35, 'max_depth': 5, 'min_child_samples': 2, 'subsample': 0.5062479819423977, 'colsample_bytree': 0.7478854927550582, 'reg_alpha': 6.8358184971631175, 'reg_lambda': 7.468133980947869, 'min_split_gain': 0.049730241825612316}. Best is trial 0 with value: 0.006053199748856122.
[Trial 0] New best: 0.00605
[I 2025-06-15 19:52:54,316] Trial 1 finished with value: 0.005458798160049708 and parameters: {'n_estimators': 297, 'learning_rate': 0.08801068525557916, 'num_leaves': 36, 'max_depth': 7, 'min_child_samples': 12, 'subsample': 0.7381018058836251, 'colsample_bytree': 0.6426256786521434, 'reg_alpha': 6.394764080603501, 'reg_lambda': 2.7073505413277066, 'min_split_gain': 0.015683457672779233}. Best is trial 1 with value: 0.005458798160049708.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:54,708] Trial 2 finished with value: 0.0051766083339063644 and parameters: {'n_estimators': 176, 'learning_rate': 0.07638640257339475, 'num_leaves': 135, 'max_depth': 7, 'min_child_samples': 13, 'subsample': 0.6831495508236647, 'colsample_bytree': 0.6577319949964012, 'reg_alpha': 5.42599165958605, 'reg_lambda': 7.856437020866998, 'min_split_gain': 0.019803605182308344}. Best is trial 2 with value: 0.0051766083339063644.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:52:55,217] Trial 3 finished with value: 0.00722907127406978 and parameters: {'n_estimators': 265, 'learning_rate': 0.09790665715559023, 'num_leaves': 141, 'max_depth': 16, 'min_child_samples': 7, 'subsample': 0.641167584736633, 'colsample_bytree': 0.5477820903973067, 'reg_alpha': 8.98008588566053, 'reg_lambda': 2.9000757637027164, 'min_split_gain': 0.04078504317890351}. Best is trial 2 with value: 0.0051766083339063644.
[I 2025-06-15 19:52:55,220] A new study created in memory with name: no-name-2ce69e28-9fc6-41c3-9320-8dfb6da0503e
[I 2025-06-15 19:52:55,261] Trial 0 finished with value: 0.00972176954885677 and parameters: {'n_estimators': 114, 'learning_rate': 0.10715746899953725, 'num_leaves': 168, 'max_depth': 14, 'min_child_samples': 21, 'subsample': 0.9670021187618372, 'colsample_bytree': 0.6489942081931062, 'reg_alpha': 0.4970711415926843, 'reg_lambda': 6.539366087793233, 'min_split_gain': 0.037698050794297555}. Best is trial 0 with value: 0.00972176954885677.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         674
1           Price_lag_2         505
14      pct_chg_Load_DA         381
3         Load_DA_lag_0         295
11           NGas_lag_2         258
2           Price_lag_7         197
16  roll2d_median_Price         181
5       WindOn_DA_lag_0         179
15       lag168_Load_DA         166
13            EUA_lag_2         152
10           Coal_lag_2         130
12            Oil_lag_2          91
9                  WD_7          15
7                  WD_1          13
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          50
1           Price_lag_2           8
5       WindOn_DA_lag_0           5
15       lag168_Load_DA           2
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 19.97% done
********************* END NS.. 145 ****************************************************
********************* START NS ... 146 ****************************************************
[I 2025-06-15 19:52:56,152] A new study created in memory with name: no-name-ee84548a-75cc-4f9e-95f3-0eca7ae0cb16
[I 2025-06-15 19:52:56,310] Trial 0 finished with value: 0.00997962380105604 and parameters: {'n_estimators': 135, 'learning_rate': 0.26024043908434946, 'num_leaves': 239, 'max_depth': 8, 'min_child_samples': 19, 'subsample': 0.7121607814413333, 'colsample_bytree': 0.7824631901193015, 'reg_alpha': 3.8803299070161423, 'reg_lambda': 0.11799632780939873, 'min_split_gain': 0.03845388974120215}. Best is trial 0 with value: 0.00997962380105604.
[I 2025-06-15 19:52:56,500] Trial 1 finished with value: 0.008181195678504031 and parameters: {'n_estimators': 220, 'learning_rate': 0.19352571279374714, 'num_leaves': 131, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.866777390241004, 'colsample_bytree': 0.8631171009763154, 'reg_alpha': 6.525187004799275, 'reg_lambda': 0.5116976481003688, 'min_split_gain': 0.04070484197413671}. Best is trial 1 with value: 0.008181195678504031.
[Trial 0] New best: 0.00998
[I 2025-06-15 19:52:56,678] Trial 2 finished with value: 0.007928444084886258 and parameters: {'n_estimators': 115, 'learning_rate': 0.1894880768649665, 'num_leaves': 101, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.9506392585472129, 'colsample_bytree': 0.9001291734667811, 'reg_alpha': 7.594879452063854, 'reg_lambda': 7.228914280809659, 'min_split_gain': 0.03776509161762395}. Best is trial 2 with value: 0.007928444084886258.
[Trial 1] New best: 0.00818
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:52:57,014] Trial 3 finished with value: 0.008375100277811682 and parameters: {'n_estimators': 100, 'learning_rate': 0.07524883340881293, 'num_leaves': 85, 'max_depth': 6, 'min_child_samples': 5, 'subsample': 0.9559519043528946, 'colsample_bytree': 0.646722660273179, 'reg_alpha': 7.545459643822109, 'reg_lambda': 9.885830517051422, 'min_split_gain': 0.023992843660269915}. Best is trial 2 with value: 0.007928444084886258.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:52:57,453] Trial 4 finished with value: 0.008723346563451828 and parameters: {'n_estimators': 99, 'learning_rate': 0.08697728711753994, 'num_leaves': 195, 'max_depth': 14, 'min_child_samples': 24, 'subsample': 0.9072190640276561, 'colsample_bytree': 0.7570052024993015, 'reg_alpha': 2.659467872651422, 'reg_lambda': 5.292166082261069, 'min_split_gain': 0.017479723977761274}. Best is trial 2 with value: 0.007928444084886258.
[I 2025-06-15 19:52:57,455] A new study created in memory with name: no-name-cee6360c-3a83-4ceb-b24f-cbc035681627
[I 2025-06-15 19:52:57,522] Trial 0 finished with value: 0.020181443183311523 and parameters: {'n_estimators': 228, 'learning_rate': 0.016870405473531823, 'num_leaves': 234, 'max_depth': 6, 'min_child_samples': 29, 'subsample': 0.6724949200247308, 'colsample_bytree': 0.5818857095575474, 'reg_alpha': 1.7997240124400116, 'reg_lambda': 8.625536659210617, 'min_split_gain': 0.028250207153414454}. Best is trial 0 with value: 0.020181443183311523.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         242
14      pct_chg_Load_DA         152
1           Price_lag_2         104
16  roll2d_median_Price          70
11           NGas_lag_2          68
2           Price_lag_7          67
3         Load_DA_lag_0          65
13            EUA_lag_2          47
10           Coal_lag_2          45
12            Oil_lag_2          34
15       lag168_Load_DA          32
5       WindOn_DA_lag_0          14
9                  WD_7           2
7                  WD_1           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         208
1           Price_lag_2          42
5       WindOn_DA_lag_0          15
12            Oil_lag_2           8
11           NGas_lag_2           7
3         Load_DA_lag_0           7
15       lag168_Load_DA           4
2           Price_lag_7           1
13            EUA_lag_2           1
14      pct_chg_Load_DA           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 20.11% done
********************* END NS.. 146 ****************************************************
********************* START NS ... 147 ****************************************************
[I 2025-06-15 19:52:58,264] A new study created in memory with name: no-name-ac6fa212-5b3e-4182-a68f-a91f8338b6d4
[I 2025-06-15 19:52:58,552] Trial 0 finished with value: 0.007836496700030395 and parameters: {'n_estimators': 281, 'learning_rate': 0.06438371688308363, 'num_leaves': 199, 'max_depth': 5, 'min_child_samples': 4, 'subsample': 0.5855305046200241, 'colsample_bytree': 0.9942985238797724, 'reg_alpha': 2.639100380020938, 'reg_lambda': 1.171686300688085, 'min_split_gain': 0.03939003808727455}. Best is trial 0 with value: 0.007836496700030395.
[Trial 0] New best: 0.00784
[I 2025-06-15 19:52:59,724] Trial 1 finished with value: 0.012946457781238874 and parameters: {'n_estimators': 254, 'learning_rate': 0.010210670316539876, 'num_leaves': 76, 'max_depth': 7, 'min_child_samples': 6, 'subsample': 0.8497027267931231, 'colsample_bytree': 0.7032379466029535, 'reg_alpha': 1.016591092930036, 'reg_lambda': 8.155968146522053, 'min_split_gain': 0.029174490160661024}. Best is trial 0 with value: 0.007836496700030395.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:00,246] Trial 2 finished with value: 0.009217663063541449 and parameters: {'n_estimators': 279, 'learning_rate': 0.10033058534156014, 'num_leaves': 43, 'max_depth': 16, 'min_child_samples': 20, 'subsample': 0.8173634156223042, 'colsample_bytree': 0.9297630204516284, 'reg_alpha': 4.364714956195559, 'reg_lambda': 6.281581703277839, 'min_split_gain': 0.02756266141429603}. Best is trial 0 with value: 0.007836496700030395.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:00,837] Trial 3 finished with value: 0.010022637225671236 and parameters: {'n_estimators': 135, 'learning_rate': 0.1267054770767991, 'num_leaves': 160, 'max_depth': 6, 'min_child_samples': 23, 'subsample': 0.983142445836953, 'colsample_bytree': 0.9071672281348729, 'reg_alpha': 7.239878013331258, 'reg_lambda': 6.261152011214891, 'min_split_gain': 0.000413180605313479}. Best is trial 0 with value: 0.007836496700030395.
[I 2025-06-15 19:53:00,840] A new study created in memory with name: no-name-57611919-aa46-422e-a6fa-e08ad08f2955
[I 2025-06-15 19:53:00,903] Trial 0 finished with value: 0.03980268614834298 and parameters: {'n_estimators': 246, 'learning_rate': 0.11899231633266355, 'num_leaves': 133, 'max_depth': 16, 'min_child_samples': 7, 'subsample': 0.5771280015518785, 'colsample_bytree': 0.7078552154588977, 'reg_alpha': 6.099596866873664, 'reg_lambda': 3.9295531789179194, 'min_split_gain': 0.03660072548217053}. Best is trial 0 with value: 0.03980268614834298.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         832
14      pct_chg_Load_DA         441
1           Price_lag_2         303
3         Load_DA_lag_0         276
11           NGas_lag_2         208
2           Price_lag_7         187
16  roll2d_median_Price         177
13            EUA_lag_2         102
15       lag168_Load_DA         100
10           Coal_lag_2          86
12            Oil_lag_2          80
5       WindOn_DA_lag_0          48
7                  WD_1           9
9                  WD_7           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          25
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 20.25% done
********************* END NS.. 147 ****************************************************
********************* START NS ... 148 ****************************************************
[I 2025-06-15 19:53:01,816] A new study created in memory with name: no-name-64017225-a769-4c25-9307-9d891aabe810
[I 2025-06-15 19:53:02,463] Trial 0 finished with value: 0.01803129003980464 and parameters: {'n_estimators': 235, 'learning_rate': 0.02454471520410943, 'num_leaves': 188, 'max_depth': 6, 'min_child_samples': 26, 'subsample': 0.8800672225778936, 'colsample_bytree': 0.6505019350720637, 'reg_alpha': 1.9208535082948086, 'reg_lambda': 5.2968883377288245, 'min_split_gain': 0.040843119994062295}. Best is trial 0 with value: 0.01803129003980464.
[Trial 0] New best: 0.01803
[I 2025-06-15 19:53:02,695] Trial 1 finished with value: 0.017889451665308642 and parameters: {'n_estimators': 131, 'learning_rate': 0.11059641319751995, 'num_leaves': 254, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.8437639235234966, 'colsample_bytree': 0.5674470386715491, 'reg_alpha': 0.2295327272513159, 'reg_lambda': 7.128217407606452, 'min_split_gain': 0.01342205876004814}. Best is trial 1 with value: 0.017889451665308642.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:03,299] Trial 2 finished with value: 0.016179671909926256 and parameters: {'n_estimators': 62, 'learning_rate': 0.07296476255364082, 'num_leaves': 150, 'max_depth': 14, 'min_child_samples': 10, 'subsample': 0.519569472760359, 'colsample_bytree': 0.7036287334594037, 'reg_alpha': 1.9329266195888073, 'reg_lambda': 2.973504301899829, 'min_split_gain': 0.0003028800196004267}. Best is trial 2 with value: 0.016179671909926256.
[Trial 2] New best: 0.01618
[I 2025-06-15 19:53:03,637] Trial 3 finished with value: 0.015570929372899568 and parameters: {'n_estimators': 81, 'learning_rate': 0.15047384810064104, 'num_leaves': 181, 'max_depth': 5, 'min_child_samples': 11, 'subsample': 0.8855981558086814, 'colsample_bytree': 0.9449349350259753, 'reg_alpha': 3.21904307982658, 'reg_lambda': 6.931780906952698, 'min_split_gain': 0.045004530557117346}. Best is trial 3 with value: 0.015570929372899568.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:04,016] Trial 4 finished with value: 0.061388056656163925 and parameters: {'n_estimators': 62, 'learning_rate': 0.022904223462146912, 'num_leaves': 56, 'max_depth': 9, 'min_child_samples': 29, 'subsample': 0.8117504369028927, 'colsample_bytree': 0.5358743165832192, 'reg_alpha': 2.9176799988383006, 'reg_lambda': 9.604117719148658, 'min_split_gain': 0.02353276383542713}. Best is trial 3 with value: 0.015570929372899568.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:04,805] Trial 5 finished with value: 0.04834424775862848 and parameters: {'n_estimators': 157, 'learning_rate': 0.010997828552868474, 'num_leaves': 103, 'max_depth': 7, 'min_child_samples': 20, 'subsample': 0.7530258846010274, 'colsample_bytree': 0.5621991884259743, 'reg_alpha': 2.3118613541016373, 'reg_lambda': 9.156127869090703, 'min_split_gain': 0.028505460752017625}. Best is trial 3 with value: 0.015570929372899568.
[I 2025-06-15 19:53:04,809] A new study created in memory with name: no-name-c8665205-293c-49ba-b982-1cffbbe5634b
[I 2025-06-15 19:53:04,868] Trial 0 finished with value: 0.08624571663200013 and parameters: {'n_estimators': 122, 'learning_rate': 0.026503610117316015, 'num_leaves': 72, 'max_depth': 4, 'min_child_samples': 28, 'subsample': 0.7559839368017673, 'colsample_bytree': 0.9764917051152966, 'reg_alpha': 2.9723949677391914, 'reg_lambda': 1.1992405879959267, 'min_split_gain': 0.016025694205665537}. Best is trial 0 with value: 0.08624571663200013.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         316
14      pct_chg_Load_DA         182
1           Price_lag_2         107
3         Load_DA_lag_0          90
11           NGas_lag_2          72
16  roll2d_median_Price          69
2           Price_lag_7          68
15       lag168_Load_DA          55
10           Coal_lag_2          33
5       WindOn_DA_lag_0          32
12            Oil_lag_2          31
13            EUA_lag_2          31
7                  WD_1           2
9                  WD_7           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         187
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 20.38% done
********************* END NS.. 148 ****************************************************
********************* START NS ... 149 ****************************************************
[I 2025-06-15 19:53:05,634] A new study created in memory with name: no-name-5928e6b2-6a3f-4e33-8a2e-6309aa495e4e
[I 2025-06-15 19:53:05,827] Trial 0 finished with value: 0.01777135959687588 and parameters: {'n_estimators': 293, 'learning_rate': 0.24868148782839392, 'num_leaves': 129, 'max_depth': 4, 'min_child_samples': 6, 'subsample': 0.5149860314365899, 'colsample_bytree': 0.8693059373754862, 'reg_alpha': 7.015024536582011, 'reg_lambda': 1.7583572310858797, 'min_split_gain': 0.021791316280992}. Best is trial 0 with value: 0.01777135959687588.
[Trial 0] New best: 0.01777
[I 2025-06-15 19:53:06,145] Trial 1 finished with value: 0.04849960180384219 and parameters: {'n_estimators': 62, 'learning_rate': 0.02538953454013188, 'num_leaves': 95, 'max_depth': 13, 'min_child_samples': 4, 'subsample': 0.7568848085570693, 'colsample_bytree': 0.7114274909255693, 'reg_alpha': 7.864374772316478, 'reg_lambda': 1.4329642509283802, 'min_split_gain': 0.04970835047434268}. Best is trial 0 with value: 0.01777135959687588.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:06,465] Trial 2 finished with value: 0.017449089473032666 and parameters: {'n_estimators': 232, 'learning_rate': 0.23408900893726226, 'num_leaves': 100, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.7428930741439295, 'colsample_bytree': 0.6278894018866115, 'reg_alpha': 5.901757453632389, 'reg_lambda': 7.451369238219188, 'min_split_gain': 0.024205248590250524}. Best is trial 2 with value: 0.017449089473032666.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:07,058] Trial 3 finished with value: 0.017773317910685846 and parameters: {'n_estimators': 144, 'learning_rate': 0.03096945315917742, 'num_leaves': 211, 'max_depth': 16, 'min_child_samples': 10, 'subsample': 0.648749487588776, 'colsample_bytree': 0.8172251443566572, 'reg_alpha': 9.980464940552729, 'reg_lambda': 6.098599547947106, 'min_split_gain': 0.034468813373771855}. Best is trial 2 with value: 0.017449089473032666.
[I 2025-06-15 19:53:07,061] A new study created in memory with name: no-name-6b660a36-3900-4d7e-b330-7c2d6bd49b24
[I 2025-06-15 19:53:07,097] Trial 0 finished with value: 0.02418433186435893 and parameters: {'n_estimators': 169, 'learning_rate': 0.06194843009388313, 'num_leaves': 124, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.7016450105481638, 'colsample_bytree': 0.8496797174689361, 'reg_alpha': 0.5588171082241244, 'reg_lambda': 4.276817532985552, 'min_split_gain': 0.006781312803268247}. Best is trial 0 with value: 0.02418433186435893.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         225
1           Price_lag_2         177
3         Load_DA_lag_0         110
14      pct_chg_Load_DA         104
2           Price_lag_7          97
11           NGas_lag_2          92
5       WindOn_DA_lag_0          53
16  roll2d_median_Price          49
10           Coal_lag_2          47
13            EUA_lag_2          45
15       lag168_Load_DA          45
12            Oil_lag_2          41
7                  WD_1           4
9                  WD_7           4
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         128
2           Price_lag_7          27
5       WindOn_DA_lag_0          21
16  roll2d_median_Price          21
1           Price_lag_2          18
3         Load_DA_lag_0          12
15       lag168_Load_DA           7
14      pct_chg_Load_DA           4
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
8                  WD_6           0
-> 20.52% done
********************* END NS.. 149 ****************************************************
********************* START NS ... 150 ****************************************************
[I 2025-06-15 19:53:07,895] A new study created in memory with name: no-name-f7d07e51-82c1-47fa-84b6-6d1329b39b5f
[I 2025-06-15 19:53:08,198] Trial 0 finished with value: 0.00921500089564809 and parameters: {'n_estimators': 234, 'learning_rate': 0.06270397279184692, 'num_leaves': 32, 'max_depth': 4, 'min_child_samples': 23, 'subsample': 0.766814264718266, 'colsample_bytree': 0.9855203461713676, 'reg_alpha': 5.047859832230545, 'reg_lambda': 5.509836306589784, 'min_split_gain': 0.03326372862132623}. Best is trial 0 with value: 0.00921500089564809.
[Trial 0] New best: 0.00922
[I 2025-06-15 19:53:09,059] Trial 1 finished with value: 0.012777229188455154 and parameters: {'n_estimators': 252, 'learning_rate': 0.01069029893692456, 'num_leaves': 192, 'max_depth': 10, 'min_child_samples': 20, 'subsample': 0.6750301850481872, 'colsample_bytree': 0.6808983926571224, 'reg_alpha': 9.202642209266495, 'reg_lambda': 4.979119332640366, 'min_split_gain': 0.049368379146715026}. Best is trial 0 with value: 0.00921500089564809.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:09,400] Trial 2 finished with value: 0.008823606942891653 and parameters: {'n_estimators': 286, 'learning_rate': 0.1268723524707721, 'num_leaves': 220, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.5607014689262415, 'colsample_bytree': 0.8693789801677962, 'reg_alpha': 9.679851667849302, 'reg_lambda': 1.4508306493032264, 'min_split_gain': 0.02694906792980601}. Best is trial 2 with value: 0.008823606942891653.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:09,635] Trial 3 finished with value: 0.008433029717605109 and parameters: {'n_estimators': 163, 'learning_rate': 0.21623286644346335, 'num_leaves': 240, 'max_depth': 16, 'min_child_samples': 4, 'subsample': 0.9752833966082, 'colsample_bytree': 0.7802251251823316, 'reg_alpha': 1.710331788308549, 'reg_lambda': 3.5299725416199945, 'min_split_gain': 0.03667753342444222}. Best is trial 3 with value: 0.008433029717605109.
[I 2025-06-15 19:53:09,638] A new study created in memory with name: no-name-b984b739-1157-4b41-b965-23cf43f2052b
[I 2025-06-15 19:53:09,657] Trial 0 finished with value: 0.03166536655513421 and parameters: {'n_estimators': 61, 'learning_rate': 0.18035480488680392, 'num_leaves': 155, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.88773403057846, 'colsample_bytree': 0.7643341284536314, 'reg_alpha': 6.766523466932379, 'reg_lambda': 3.2518418189176126, 'min_split_gain': 0.006695745697519545}. Best is trial 0 with value: 0.03166536655513421.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         306
14      pct_chg_Load_DA         301
1           Price_lag_2         279
3         Load_DA_lag_0         175
2           Price_lag_7         148
11           NGas_lag_2         136
15       lag168_Load_DA         118
16  roll2d_median_Price         108
10           Coal_lag_2          81
5       WindOn_DA_lag_0          77
12            Oil_lag_2          69
13            EUA_lag_2          62
9                  WD_7           7
7                  WD_1           6
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          21
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 20.66% done
********************* END NS.. 150 ****************************************************
********************* START NS ... 151 ****************************************************
[I 2025-06-15 19:53:10,413] A new study created in memory with name: no-name-dfd72031-1533-42d1-a398-ef66b3d47de4
[I 2025-06-15 19:53:10,999] Trial 0 finished with value: 0.009035942016001 and parameters: {'n_estimators': 150, 'learning_rate': 0.029759930341043024, 'num_leaves': 138, 'max_depth': 8, 'min_child_samples': 30, 'subsample': 0.8458613249161563, 'colsample_bytree': 0.5348253588651595, 'reg_alpha': 1.7938359877045251, 'reg_lambda': 0.5717040648595972, 'min_split_gain': 0.03248346110509846}. Best is trial 0 with value: 0.009035942016001.
[Trial 0] New best: 0.00904
[I 2025-06-15 19:53:11,367] Trial 1 finished with value: 0.008356968731602 and parameters: {'n_estimators': 213, 'learning_rate': 0.08031485343716717, 'num_leaves': 154, 'max_depth': 10, 'min_child_samples': 20, 'subsample': 0.838231693629554, 'colsample_bytree': 0.8867063937018593, 'reg_alpha': 8.456921433895767, 'reg_lambda': 5.3992280386886495, 'min_split_gain': 0.027581553225771285}. Best is trial 1 with value: 0.008356968731602.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:11,754] Trial 2 finished with value: 0.00696994345020734 and parameters: {'n_estimators': 118, 'learning_rate': 0.18663563969545302, 'num_leaves': 132, 'max_depth': 13, 'min_child_samples': 27, 'subsample': 0.6179043687785739, 'colsample_bytree': 0.7451673994929746, 'reg_alpha': 0.34080142922172985, 'reg_lambda': 7.463468243304154, 'min_split_gain': 0.013151569792616787}. Best is trial 2 with value: 0.00696994345020734.
[Trial 2] New best: 0.00697
[I 2025-06-15 19:53:12,213] Trial 3 finished with value: 0.0072594211041378766 and parameters: {'n_estimators': 102, 'learning_rate': 0.18837197545748183, 'num_leaves': 95, 'max_depth': 16, 'min_child_samples': 18, 'subsample': 0.883322005682579, 'colsample_bytree': 0.6112262028511798, 'reg_alpha': 0.6236818588915605, 'reg_lambda': 2.1160674041079517, 'min_split_gain': 0.031357488329235496}. Best is trial 2 with value: 0.00696994345020734.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:12,519] Trial 4 finished with value: 0.009799050465121896 and parameters: {'n_estimators': 297, 'learning_rate': 0.17995216751629833, 'num_leaves': 198, 'max_depth': 7, 'min_child_samples': 14, 'subsample': 0.9149199373893805, 'colsample_bytree': 0.6018009729070442, 'reg_alpha': 5.604953910327826, 'reg_lambda': 9.79075576436363, 'min_split_gain': 0.020963634239364932}. Best is trial 2 with value: 0.00696994345020734.
[I 2025-06-15 19:53:12,667] Trial 5 finished with value: 0.008502947012774181 and parameters: {'n_estimators': 76, 'learning_rate': 0.193651534113583, 'num_leaves': 223, 'max_depth': 5, 'min_child_samples': 25, 'subsample': 0.5325567415466947, 'colsample_bytree': 0.9427090443413113, 'reg_alpha': 8.87736356401765, 'reg_lambda': 2.3598859457934553, 'min_split_gain': 0.03555252114798157}. Best is trial 2 with value: 0.00696994345020734.
[I 2025-06-15 19:53:12,670] A new study created in memory with name: no-name-a30a8bda-ab50-4242-8289-dd16ec84979b
[I 2025-06-15 19:53:12,712] Trial 0 finished with value: 0.015419757494463115 and parameters: {'n_estimators': 285, 'learning_rate': 0.03801032847632695, 'num_leaves': 238, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.9960432197304012, 'colsample_bytree': 0.6822134140426519, 'reg_alpha': 0.8388872651658918, 'reg_lambda': 8.296930540629695, 'min_split_gain': 0.00812858887751391}. Best is trial 0 with value: 0.015419757494463115.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         602
1           Price_lag_2         555
14      pct_chg_Load_DA         527
3         Load_DA_lag_0         416
15       lag168_Load_DA         323
5       WindOn_DA_lag_0         298
16  roll2d_median_Price         277
2           Price_lag_7         275
11           NGas_lag_2         236
13            EUA_lag_2         201
10           Coal_lag_2         185
12            Oil_lag_2         113
7                  WD_1          12
9                  WD_7           8
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         152
2           Price_lag_7          50
5       WindOn_DA_lag_0          16
1           Price_lag_2          12
3         Load_DA_lag_0          11
11           NGas_lag_2           3
15       lag168_Load_DA           2
12            Oil_lag_2           2
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 20.79% done
********************* END NS.. 151 ****************************************************
********************* START NS ... 152 ****************************************************
[I 2025-06-15 19:53:13,615] A new study created in memory with name: no-name-a684cc81-04ef-4a6b-ac5c-573e9bab3cf7
[I 2025-06-15 19:53:14,450] Trial 0 finished with value: 0.01049630360163478 and parameters: {'n_estimators': 235, 'learning_rate': 0.016645844805059085, 'num_leaves': 137, 'max_depth': 9, 'min_child_samples': 28, 'subsample': 0.7291106369204923, 'colsample_bytree': 0.7762419144887374, 'reg_alpha': 4.321688341369855, 'reg_lambda': 1.0364453401402307, 'min_split_gain': 0.028987186321335547}. Best is trial 0 with value: 0.01049630360163478.
[I 2025-06-15 19:53:14,653] Trial 1 finished with value: 0.011900652982765984 and parameters: {'n_estimators': 96, 'learning_rate': 0.13405217898077926, 'num_leaves': 44, 'max_depth': 10, 'min_child_samples': 17, 'subsample': 0.586839193780996, 'colsample_bytree': 0.5874443740987992, 'reg_alpha': 5.2474858405781095, 'reg_lambda': 3.9125314053696867, 'min_split_gain': 0.04335456807513394}. Best is trial 0 with value: 0.01049630360163478.
[Trial 0] New best: 0.01050
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:15,083] Trial 2 finished with value: 0.008767363723525053 and parameters: {'n_estimators': 105, 'learning_rate': 0.08148918884270166, 'num_leaves': 49, 'max_depth': 10, 'min_child_samples': 25, 'subsample': 0.8782956438268907, 'colsample_bytree': 0.8043506766999504, 'reg_alpha': 0.8863362691951016, 'reg_lambda': 8.584554298995844, 'min_split_gain': 0.015974811890567958}. Best is trial 2 with value: 0.008767363723525053.
[Trial 2] New best: 0.00877
[I 2025-06-15 19:53:15,478] Trial 3 finished with value: 0.008740526224252018 and parameters: {'n_estimators': 167, 'learning_rate': 0.15206696154488747, 'num_leaves': 99, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.586302928971006, 'colsample_bytree': 0.6635946533137976, 'reg_alpha': 1.0409992349512842, 'reg_lambda': 4.692316530385668, 'min_split_gain': 0.024478371637697196}. Best is trial 3 with value: 0.008740526224252018.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:15,954] Trial 4 finished with value: 0.009264626881885977 and parameters: {'n_estimators': 274, 'learning_rate': 0.22166224789101446, 'num_leaves': 238, 'max_depth': 9, 'min_child_samples': 14, 'subsample': 0.8377472881576794, 'colsample_bytree': 0.9981650527006432, 'reg_alpha': 3.88385184376834, 'reg_lambda': 2.486736138136254, 'min_split_gain': 0.005437103499571944}. Best is trial 3 with value: 0.008740526224252018.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:16,197] Trial 5 finished with value: 0.011906575985299617 and parameters: {'n_estimators': 270, 'learning_rate': 0.19552388535666787, 'num_leaves': 225, 'max_depth': 4, 'min_child_samples': 10, 'subsample': 0.6814480509140144, 'colsample_bytree': 0.8667128387537986, 'reg_alpha': 8.046404320360844, 'reg_lambda': 9.094749756025411, 'min_split_gain': 0.023385470374362494}. Best is trial 3 with value: 0.008740526224252018.
[I 2025-06-15 19:53:16,199] A new study created in memory with name: no-name-a5ce8e86-b707-4b28-b9ff-cd19ebdb93c8
[I 2025-06-15 19:53:16,235] Trial 0 finished with value: 0.02555277486517642 and parameters: {'n_estimators': 205, 'learning_rate': 0.14498424687643563, 'num_leaves': 121, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.5863063611818983, 'colsample_bytree': 0.6368801465710503, 'reg_alpha': 7.314864798819942, 'reg_lambda': 3.3770801100954726, 'min_split_gain': 0.03563817069587673}. Best is trial 0 with value: 0.02555277486517642.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         546
1           Price_lag_2         406
14      pct_chg_Load_DA         326
3         Load_DA_lag_0         287
15       lag168_Load_DA         212
16  roll2d_median_Price         187
2           Price_lag_7         176
11           NGas_lag_2         169
10           Coal_lag_2         148
13            EUA_lag_2         143
5       WindOn_DA_lag_0         124
12            Oil_lag_2         108
9                  WD_7           8
7                  WD_1           5
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          15
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 20.93% done
********************* END NS.. 152 ****************************************************
********************* START NS ... 153 ****************************************************
[I 2025-06-15 19:53:17,142] A new study created in memory with name: no-name-25bfaf73-ea26-4818-b434-a48b9fbc5d61
[I 2025-06-15 19:53:17,388] Trial 0 finished with value: 0.0055076268485480155 and parameters: {'n_estimators': 111, 'learning_rate': 0.17202170558915017, 'num_leaves': 195, 'max_depth': 7, 'min_child_samples': 27, 'subsample': 0.6305208502322212, 'colsample_bytree': 0.7206639580550807, 'reg_alpha': 5.074033142902148, 'reg_lambda': 1.6375904525670115, 'min_split_gain': 0.021553519235776628}. Best is trial 0 with value: 0.0055076268485480155.
[Trial 0] New best: 0.00551
[I 2025-06-15 19:53:18,802] Trial 1 finished with value: 0.006378843051997139 and parameters: {'n_estimators': 233, 'learning_rate': 0.011500579634844075, 'num_leaves': 40, 'max_depth': 8, 'min_child_samples': 24, 'subsample': 0.7156791576094024, 'colsample_bytree': 0.7610816402725495, 'reg_alpha': 4.7328637467451635, 'reg_lambda': 9.143590194678902, 'min_split_gain': 0.018074975655399995}. Best is trial 0 with value: 0.0055076268485480155.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:19,420] Trial 2 finished with value: 0.006808849185867203 and parameters: {'n_estimators': 78, 'learning_rate': 0.03173234592464706, 'num_leaves': 125, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.7324566659836016, 'colsample_bytree': 0.9294776446592403, 'reg_alpha': 5.483910612984712, 'reg_lambda': 6.779868323268992, 'min_split_gain': 0.015086686203629013}. Best is trial 0 with value: 0.0055076268485480155.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:19,700] Trial 3 finished with value: 0.021463929715507257 and parameters: {'n_estimators': 55, 'learning_rate': 0.021529222161913532, 'num_leaves': 209, 'max_depth': 13, 'min_child_samples': 24, 'subsample': 0.9734419754429244, 'colsample_bytree': 0.5489611261262937, 'reg_alpha': 5.060133768716134, 'reg_lambda': 6.374161249478432, 'min_split_gain': 0.010918759292967928}. Best is trial 0 with value: 0.0055076268485480155.
[I 2025-06-15 19:53:19,702] A new study created in memory with name: no-name-b5c258c7-5dc5-4a98-9129-8586f6daf80e
[I 2025-06-15 19:53:19,731] Trial 0 finished with value: 0.009942126646808292 and parameters: {'n_estimators': 94, 'learning_rate': 0.18997371547306047, 'num_leaves': 256, 'max_depth': 16, 'min_child_samples': 1, 'subsample': 0.6333996378780993, 'colsample_bytree': 0.687659400204511, 'reg_alpha': 1.1005853983950387, 'reg_lambda': 3.0657616905379093, 'min_split_gain': 0.006450067442530405}. Best is trial 0 with value: 0.009942126646808292.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         300
1           Price_lag_2         200
14      pct_chg_Load_DA         192
3         Load_DA_lag_0         118
11           NGas_lag_2          93
16  roll2d_median_Price          71
2           Price_lag_7          69
13            EUA_lag_2          65
15       lag168_Load_DA          64
10           Coal_lag_2          61
5       WindOn_DA_lag_0          53
12            Oil_lag_2          41
7                  WD_1           3
9                  WD_7           1
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          28
2           Price_lag_7           9
5       WindOn_DA_lag_0           6
16  roll2d_median_Price           6
3         Load_DA_lag_0           6
10           Coal_lag_2           3
1           Price_lag_2           2
14      pct_chg_Load_DA           1
15       lag168_Load_DA           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
8                  WD_6           0
-> 21.07% done
********************* END NS.. 153 ****************************************************
********************* START NS ... 154 ****************************************************
[I 2025-06-15 19:53:20,533] A new study created in memory with name: no-name-85dae3ad-0a6d-4923-8e29-37a8a15214b3
[I 2025-06-15 19:53:20,747] Trial 0 finished with value: 0.006878093588820455 and parameters: {'n_estimators': 143, 'learning_rate': 0.021183984152475778, 'num_leaves': 122, 'max_depth': 4, 'min_child_samples': 16, 'subsample': 0.8900143706312198, 'colsample_bytree': 0.8417390491816865, 'reg_alpha': 4.513666760913436, 'reg_lambda': 7.49718178453743, 'min_split_gain': 0.03938219416783365}. Best is trial 0 with value: 0.006878093588820455.
[Trial 0] New best: 0.00688
[I 2025-06-15 19:53:20,976] Trial 1 finished with value: 0.005178623772164668 and parameters: {'n_estimators': 113, 'learning_rate': 0.23633116388412806, 'num_leaves': 156, 'max_depth': 10, 'min_child_samples': 16, 'subsample': 0.9753886550739732, 'colsample_bytree': 0.5728064541737696, 'reg_alpha': 1.9740839259735399, 'reg_lambda': 8.621734672005282, 'min_split_gain': 0.01785739062489022}. Best is trial 1 with value: 0.005178623772164668.
[Trial 1] New best: 0.00518
[I 2025-06-15 19:53:21,425] Trial 2 finished with value: 0.009193472892320224 and parameters: {'n_estimators': 106, 'learning_rate': 0.019728440174281773, 'num_leaves': 215, 'max_depth': 5, 'min_child_samples': 29, 'subsample': 0.5551716731997326, 'colsample_bytree': 0.8432792005959382, 'reg_alpha': 1.5364416910879874, 'reg_lambda': 8.227209528430581, 'min_split_gain': 0.006447883790187948}. Best is trial 1 with value: 0.005178623772164668.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:22,396] Trial 3 finished with value: 0.005970325675619045 and parameters: {'n_estimators': 248, 'learning_rate': 0.02147910651343258, 'num_leaves': 110, 'max_depth': 15, 'min_child_samples': 10, 'subsample': 0.8199586295546832, 'colsample_bytree': 0.9832934533583195, 'reg_alpha': 4.51142882262379, 'reg_lambda': 3.5093157127089656, 'min_split_gain': 0.04716575725733287}. Best is trial 1 with value: 0.005178623772164668.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:22,896] Trial 4 finished with value: 0.007611208638243328 and parameters: {'n_estimators': 106, 'learning_rate': 0.023571744130091915, 'num_leaves': 92, 'max_depth': 9, 'min_child_samples': 6, 'subsample': 0.5424442630515218, 'colsample_bytree': 0.7706638191405131, 'reg_alpha': 5.025571971332242, 'reg_lambda': 2.259167303096005, 'min_split_gain': 0.018150825764809336}. Best is trial 1 with value: 0.005178623772164668.
[I 2025-06-15 19:53:22,899] A new study created in memory with name: no-name-aeacfa21-7b51-419e-9e73-aa1ca8e7bdf3
[I 2025-06-15 19:53:22,932] Trial 0 finished with value: 0.01193352729573575 and parameters: {'n_estimators': 78, 'learning_rate': 0.01566267426676485, 'num_leaves': 102, 'max_depth': 5, 'min_child_samples': 10, 'subsample': 0.6783021939601678, 'colsample_bytree': 0.7003967888987273, 'reg_alpha': 0.520059442467874, 'reg_lambda': 8.918783987960722, 'min_split_gain': 0.0321571981637366}. Best is trial 0 with value: 0.01193352729573575.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         331
1           Price_lag_2         304
3         Load_DA_lag_0         257
14      pct_chg_Load_DA         181
2           Price_lag_7         158
5       WindOn_DA_lag_0         134
16  roll2d_median_Price         132
15       lag168_Load_DA         131
10           Coal_lag_2         120
11           NGas_lag_2         118
13            EUA_lag_2          97
12            Oil_lag_2          81
9                  WD_7           8
7                  WD_1           7
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          79
1           Price_lag_2          44
2           Price_lag_7          14
15       lag168_Load_DA          14
3         Load_DA_lag_0           9
16  roll2d_median_Price           3
5       WindOn_DA_lag_0           3
13            EUA_lag_2           2
12            Oil_lag_2           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 21.20% done
********************* END NS.. 154 ****************************************************
********************* START NS ... 155 ****************************************************
[I 2025-06-15 19:53:23,771] A new study created in memory with name: no-name-6a886264-5cb9-4c48-a448-2d0c1efb67b8
[I 2025-06-15 19:53:24,308] Trial 0 finished with value: 0.04663285861415686 and parameters: {'n_estimators': 103, 'learning_rate': 0.010836677681151768, 'num_leaves': 56, 'max_depth': 12, 'min_child_samples': 7, 'subsample': 0.59200062028179, 'colsample_bytree': 0.7792481949267618, 'reg_alpha': 0.7094318379512221, 'reg_lambda': 1.221782503652472, 'min_split_gain': 0.033135010896222464}. Best is trial 0 with value: 0.04663285861415686.
[Trial 0] New best: 0.04663
[I 2025-06-15 19:53:24,813] Trial 1 finished with value: 0.02274406121245152 and parameters: {'n_estimators': 101, 'learning_rate': 0.017612477623423866, 'num_leaves': 202, 'max_depth': 7, 'min_child_samples': 17, 'subsample': 0.549897969702536, 'colsample_bytree': 0.7010392435340147, 'reg_alpha': 1.5402842764760627, 'reg_lambda': 6.366884332739689, 'min_split_gain': 0.008625914561866633}. Best is trial 1 with value: 0.02274406121245152.
[I 2025-06-15 19:53:25,028] Trial 2 finished with value: 0.011532754106489905 and parameters: {'n_estimators': 108, 'learning_rate': 0.25081272117873954, 'num_leaves': 89, 'max_depth': 11, 'min_child_samples': 30, 'subsample': 0.9610302862918692, 'colsample_bytree': 0.7062512600712763, 'reg_alpha': 6.993260040863004, 'reg_lambda': 0.30199095484556127, 'min_split_gain': 0.0067874895663171345}. Best is trial 2 with value: 0.011532754106489905.
[Trial 1] New best: 0.02274
[Trial 2] New best: 0.01153
[I 2025-06-15 19:53:25,332] Trial 3 finished with value: 0.010922363883594079 and parameters: {'n_estimators': 58, 'learning_rate': 0.10677890534276384, 'num_leaves': 146, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.9901251724821032, 'colsample_bytree': 0.632622744534578, 'reg_alpha': 3.723680176067278, 'reg_lambda': 7.371384833004347, 'min_split_gain': 0.002548104023370762}. Best is trial 3 with value: 0.010922363883594079.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:25,720] Trial 4 finished with value: 0.011766592675906759 and parameters: {'n_estimators': 286, 'learning_rate': 0.12372772864835835, 'num_leaves': 165, 'max_depth': 16, 'min_child_samples': 20, 'subsample': 0.9969208172731414, 'colsample_bytree': 0.8657782930335651, 'reg_alpha': 9.984540897244345, 'reg_lambda': 8.147621880320838, 'min_split_gain': 0.03530707017716341}. Best is trial 3 with value: 0.010922363883594079.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:26,554] Trial 5 finished with value: 0.010506782745773007 and parameters: {'n_estimators': 255, 'learning_rate': 0.0845848863871966, 'num_leaves': 63, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.7061209688462022, 'colsample_bytree': 0.7002053044028916, 'reg_alpha': 5.966603100506936, 'reg_lambda': 5.080268987851374, 'min_split_gain': 0.007849753362204392}. Best is trial 5 with value: 0.010506782745773007.
[Trial 5] New best: 0.01051
[I 2025-06-15 19:53:27,234] Trial 6 finished with value: 0.012279121570204056 and parameters: {'n_estimators': 262, 'learning_rate': 0.04068959557175012, 'num_leaves': 189, 'max_depth': 6, 'min_child_samples': 30, 'subsample': 0.8674132581046095, 'colsample_bytree': 0.6883763500883122, 'reg_alpha': 9.537093561518537, 'reg_lambda': 2.8535462027920278, 'min_split_gain': 0.027186758838840964}. Best is trial 5 with value: 0.010506782745773007.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:27,895] Trial 7 finished with value: 0.012511358685124263 and parameters: {'n_estimators': 193, 'learning_rate': 0.10110034766979605, 'num_leaves': 108, 'max_depth': 9, 'min_child_samples': 2, 'subsample': 0.8452339182000375, 'colsample_bytree': 0.9563371364549837, 'reg_alpha': 1.0918033164741991, 'reg_lambda': 2.507696508562196, 'min_split_gain': 0.0423581243391842}. Best is trial 5 with value: 0.010506782745773007.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:28,403] Trial 8 finished with value: 0.012428846577966536 and parameters: {'n_estimators': 145, 'learning_rate': 0.027994566319367843, 'num_leaves': 212, 'max_depth': 5, 'min_child_samples': 16, 'subsample': 0.8043839331980838, 'colsample_bytree': 0.8114976225182442, 'reg_alpha': 8.52795663143698, 'reg_lambda': 3.7598616614028426, 'min_split_gain': 0.03473672982234488}. Best is trial 5 with value: 0.010506782745773007.
[I 2025-06-15 19:53:28,405] A new study created in memory with name: no-name-8b109ba7-469d-47f7-ae1f-176f8c29a3c5
[I 2025-06-15 19:53:28,462] Trial 0 finished with value: 0.04682032341815312 and parameters: {'n_estimators': 254, 'learning_rate': 0.04587284016858465, 'num_leaves': 241, 'max_depth': 10, 'min_child_samples': 26, 'subsample': 0.7206336342192687, 'colsample_bytree': 0.882454514076633, 'reg_alpha': 7.029157300184021, 'reg_lambda': 6.502918776817195, 'min_split_gain': 0.03861334193210947}. Best is trial 0 with value: 0.04682032341815312.
[Trial 8] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         683
1           Price_lag_2         585
14      pct_chg_Load_DA         491
3         Load_DA_lag_0         382
11           NGas_lag_2         298
5       WindOn_DA_lag_0         265
10           Coal_lag_2         247
13            EUA_lag_2         247
15       lag168_Load_DA         243
2           Price_lag_7         231
16  roll2d_median_Price         223
12            Oil_lag_2         167
9                  WD_7          15
7                  WD_1          10
8                  WD_6          10
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          36
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 21.34% done
********************* END NS.. 155 ****************************************************
********************* START NS ... 156 ****************************************************
[I 2025-06-15 19:53:29,668] A new study created in memory with name: no-name-d8a5060a-a478-4be5-b34e-f95f7b4fa6ee
[I 2025-06-15 19:53:29,861] Trial 0 finished with value: 0.006883635112377208 and parameters: {'n_estimators': 82, 'learning_rate': 0.24923757298438343, 'num_leaves': 209, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.6258721667711525, 'colsample_bytree': 0.8247662408277008, 'reg_alpha': 6.31473030955796, 'reg_lambda': 4.274150206577731, 'min_split_gain': 0.028511824571340934}. Best is trial 0 with value: 0.006883635112377208.
[I 2025-06-15 19:53:30,052] Trial 1 finished with value: 0.0067659872983181525 and parameters: {'n_estimators': 126, 'learning_rate': 0.16190670070857588, 'num_leaves': 134, 'max_depth': 4, 'min_child_samples': 27, 'subsample': 0.6743535604759747, 'colsample_bytree': 0.7942801867497071, 'reg_alpha': 9.080477656851242, 'reg_lambda': 5.3438218731496425, 'min_split_gain': 0.008281602867113769}. Best is trial 1 with value: 0.0067659872983181525.
[Trial 0] New best: 0.00688
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:30,266] Trial 2 finished with value: 0.0065759955297711075 and parameters: {'n_estimators': 156, 'learning_rate': 0.2910020320465543, 'num_leaves': 103, 'max_depth': 4, 'min_child_samples': 21, 'subsample': 0.5780750844017121, 'colsample_bytree': 0.7069675111118556, 'reg_alpha': 0.3257024006656861, 'reg_lambda': 1.3691945315319687, 'min_split_gain': 0.02010100347628335}. Best is trial 2 with value: 0.0065759955297711075.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:30,743] Trial 3 finished with value: 0.006301204881516886 and parameters: {'n_estimators': 235, 'learning_rate': 0.08602616476511185, 'num_leaves': 91, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.7399658630653414, 'colsample_bytree': 0.6358459417707587, 'reg_alpha': 2.685881078467255, 'reg_lambda': 0.6580002872407698, 'min_split_gain': 0.009785317120601445}. Best is trial 3 with value: 0.006301204881516886.
[I 2025-06-15 19:53:30,746] A new study created in memory with name: no-name-c650d312-6404-4326-9949-531e89003ea0
[I 2025-06-15 19:53:30,822] Trial 0 finished with value: 0.00926263688809359 and parameters: {'n_estimators': 52, 'learning_rate': 0.05918428445338297, 'num_leaves': 91, 'max_depth': 4, 'min_child_samples': 2, 'subsample': 0.991525347349308, 'colsample_bytree': 0.505324716437161, 'reg_alpha': 2.1212223203795944, 'reg_lambda': 7.822260059360472, 'min_split_gain': 0.013791275161803325}. Best is trial 0 with value: 0.00926263688809359.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         630
1           Price_lag_2         399
3         Load_DA_lag_0         272
14      pct_chg_Load_DA         272
11           NGas_lag_2         270
2           Price_lag_7         168
15       lag168_Load_DA         145
10           Coal_lag_2         142
13            EUA_lag_2         137
5       WindOn_DA_lag_0         116
16  roll2d_median_Price         104
12            Oil_lag_2          69
7                  WD_1          13
9                  WD_7           7
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          30
1           Price_lag_2          15
5       WindOn_DA_lag_0           7
14      pct_chg_Load_DA           4
10           Coal_lag_2           2
15       lag168_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 21.48% done
********************* END NS.. 156 ****************************************************
********************* START NS ... 157 ****************************************************
[I 2025-06-15 19:53:31,866] A new study created in memory with name: no-name-e74ece3c-3499-4d9e-a87c-d8ad8ac8cfb9
[I 2025-06-15 19:53:32,132] Trial 0 finished with value: 0.00020442006443455963 and parameters: {'n_estimators': 100, 'learning_rate': 0.048331338986938824, 'num_leaves': 156, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.992829183933226, 'colsample_bytree': 0.5322100842204807, 'reg_alpha': 1.149060959003223, 'reg_lambda': 4.534401206300044, 'min_split_gain': 0.030454210478761054}. Best is trial 0 with value: 0.00020442006443455963.
[Trial 0] New best: 0.00020
[I 2025-06-15 19:53:32,734] Trial 1 finished with value: 0.00021194934467993183 and parameters: {'n_estimators': 207, 'learning_rate': 0.10593729219158078, 'num_leaves': 171, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.8220263073692082, 'colsample_bytree': 0.7995432846558048, 'reg_alpha': 0.5655216838457411, 'reg_lambda': 1.5015231200836066, 'min_split_gain': 0.03030371030203854}. Best is trial 0 with value: 0.00020442006443455963.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:33,555] Trial 2 finished with value: 0.00046498086447616213 and parameters: {'n_estimators': 131, 'learning_rate': 0.020534596011555665, 'num_leaves': 62, 'max_depth': 14, 'min_child_samples': 25, 'subsample': 0.7260647741891071, 'colsample_bytree': 0.5969475597833174, 'reg_alpha': 6.09250525813105, 'reg_lambda': 4.904281573867881, 'min_split_gain': 0.02461628914941351}. Best is trial 0 with value: 0.00020442006443455963.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:34,539] Trial 3 finished with value: 0.006443373421351095 and parameters: {'n_estimators': 133, 'learning_rate': 0.011878325078088758, 'num_leaves': 72, 'max_depth': 9, 'min_child_samples': 26, 'subsample': 0.7935300458101151, 'colsample_bytree': 0.6295888965718592, 'reg_alpha': 1.310113934426852, 'reg_lambda': 4.783388184159195, 'min_split_gain': 0.002561918032466265}. Best is trial 0 with value: 0.00020442006443455963.
[I 2025-06-15 19:53:34,542] A new study created in memory with name: no-name-3d405bbe-e9d8-4b26-a6d1-a9a8e123cb8a
[I 2025-06-15 19:53:34,559] Trial 0 finished with value: 0.0012047194869822192 and parameters: {'n_estimators': 54, 'learning_rate': 0.22224115755397372, 'num_leaves': 203, 'max_depth': 10, 'min_child_samples': 8, 'subsample': 0.741709326613628, 'colsample_bytree': 0.6488508503542756, 'reg_alpha': 7.066544940563638, 'reg_lambda': 5.984548264450681, 'min_split_gain': 0.03448835914293282}. Best is trial 0 with value: 0.0012047194869822192.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         664
1           Price_lag_2         506
14      pct_chg_Load_DA         300
2           Price_lag_7         185
3         Load_DA_lag_0         175
16  roll2d_median_Price         168
11           NGas_lag_2         150
10           Coal_lag_2         115
15       lag168_Load_DA         113
13            EUA_lag_2         105
5       WindOn_DA_lag_0          62
12            Oil_lag_2          54
7                  WD_1           7
9                  WD_7           3
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 21.61% done
********************* END NS.. 157 ****************************************************
********************* START NS ... 158 ****************************************************
[I 2025-06-15 19:53:35,490] A new study created in memory with name: no-name-f403523b-5c47-4c7c-9a3a-633fdb2eb4e1
[I 2025-06-15 19:53:36,323] Trial 0 finished with value: 0.0052061018056445375 and parameters: {'n_estimators': 110, 'learning_rate': 0.01569906862962585, 'num_leaves': 244, 'max_depth': 14, 'min_child_samples': 6, 'subsample': 0.5439482165334873, 'colsample_bytree': 0.6138779664505121, 'reg_alpha': 2.1217656319770883, 'reg_lambda': 3.3382355095858163, 'min_split_gain': 0.009026495294283954}. Best is trial 0 with value: 0.0052061018056445375.
[Trial 0] New best: 0.00521
[I 2025-06-15 19:53:36,810] Trial 1 finished with value: 0.0018234674986136717 and parameters: {'n_estimators': 90, 'learning_rate': 0.0449110851005588, 'num_leaves': 47, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.711201584823995, 'colsample_bytree': 0.632243305551074, 'reg_alpha': 3.060942188969642, 'reg_lambda': 5.293094654161767, 'min_split_gain': 0.038613827883292894}. Best is trial 1 with value: 0.0018234674986136717.
[Trial 1] New best: 0.00182
[I 2025-06-15 19:53:37,330] Trial 2 finished with value: 0.001817795318334115 and parameters: {'n_estimators': 238, 'learning_rate': 0.0802795731015258, 'num_leaves': 199, 'max_depth': 8, 'min_child_samples': 13, 'subsample': 0.9666851255247668, 'colsample_bytree': 0.6350544832714984, 'reg_alpha': 7.07434307304537, 'reg_lambda': 1.0267314877987133, 'min_split_gain': 0.01723387000781055}. Best is trial 2 with value: 0.001817795318334115.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:37,838] Trial 3 finished with value: 0.0018671494135180284 and parameters: {'n_estimators': 184, 'learning_rate': 0.07108466063544658, 'num_leaves': 135, 'max_depth': 6, 'min_child_samples': 13, 'subsample': 0.837201893850116, 'colsample_bytree': 0.6148785799002643, 'reg_alpha': 4.837300637800598, 'reg_lambda': 0.5353127955901882, 'min_split_gain': 0.020305756918107654}. Best is trial 2 with value: 0.001817795318334115.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:38,104] Trial 4 finished with value: 0.001824282823074841 and parameters: {'n_estimators': 130, 'learning_rate': 0.07452296993995014, 'num_leaves': 142, 'max_depth': 4, 'min_child_samples': 13, 'subsample': 0.6292822190775438, 'colsample_bytree': 0.9140489951437045, 'reg_alpha': 9.665240095845254, 'reg_lambda': 1.5891598795889772, 'min_split_gain': 0.0303927349818813}. Best is trial 2 with value: 0.001817795318334115.
[I 2025-06-15 19:53:38,107] A new study created in memory with name: no-name-7d5af239-28eb-4144-b9c9-1343bc220529
[I 2025-06-15 19:53:38,135] Trial 0 finished with value: 0.007021178252878406 and parameters: {'n_estimators': 89, 'learning_rate': 0.2959953864179605, 'num_leaves': 179, 'max_depth': 12, 'min_child_samples': 10, 'subsample': 0.9417343317851264, 'colsample_bytree': 0.6920613918175014, 'reg_alpha': 1.0652914598990848, 'reg_lambda': 0.8906134621108008, 'min_split_gain': 0.0038253504325378666}. Best is trial 0 with value: 0.007021178252878406.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         590
1           Price_lag_2         420
14      pct_chg_Load_DA         345
3         Load_DA_lag_0         274
11           NGas_lag_2         230
5       WindOn_DA_lag_0         174
13            EUA_lag_2         173
2           Price_lag_7         160
16  roll2d_median_Price         148
15       lag168_Load_DA         127
10           Coal_lag_2         121
12            Oil_lag_2         111
9                  WD_7          17
8                  WD_6          16
7                  WD_1          10
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          17
3         Load_DA_lag_0           3
12            Oil_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 21.75% done
********************* END NS.. 158 ****************************************************
********************* START NS ... 159 ****************************************************
[I 2025-06-15 19:53:39,155] A new study created in memory with name: no-name-cbc6c0b4-d513-4ab7-893d-c8704ca642c4
[I 2025-06-15 19:53:39,479] Trial 0 finished with value: 0.002581662054277768 and parameters: {'n_estimators': 66, 'learning_rate': 0.031210635035581694, 'num_leaves': 237, 'max_depth': 16, 'min_child_samples': 27, 'subsample': 0.8100589834028693, 'colsample_bytree': 0.5422475843417626, 'reg_alpha': 3.365584512067903, 'reg_lambda': 1.7365696274149411, 'min_split_gain': 0.006603361643414352}. Best is trial 0 with value: 0.002581662054277768.
[Trial 0] New best: 0.00258
[I 2025-06-15 19:53:39,944] Trial 1 finished with value: 0.0019855564974822462 and parameters: {'n_estimators': 135, 'learning_rate': 0.07649423138521874, 'num_leaves': 203, 'max_depth': 16, 'min_child_samples': 19, 'subsample': 0.5403933833086438, 'colsample_bytree': 0.7904533964883715, 'reg_alpha': 4.062279213451693, 'reg_lambda': 6.688071162087591, 'min_split_gain': 0.042704191480147174}. Best is trial 1 with value: 0.0019855564974822462.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:40,385] Trial 2 finished with value: 0.0019942295634078453 and parameters: {'n_estimators': 172, 'learning_rate': 0.10594359536531148, 'num_leaves': 106, 'max_depth': 6, 'min_child_samples': 8, 'subsample': 0.6920755577045543, 'colsample_bytree': 0.5260336112955976, 'reg_alpha': 0.12146115518519407, 'reg_lambda': 9.378200429375372, 'min_split_gain': 0.04195194614396339}. Best is trial 1 with value: 0.0019855564974822462.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:40,945] Trial 3 finished with value: 0.001813872911310272 and parameters: {'n_estimators': 66, 'learning_rate': 0.05600266523324434, 'num_leaves': 145, 'max_depth': 12, 'min_child_samples': 5, 'subsample': 0.6872956579467268, 'colsample_bytree': 0.64497949809029, 'reg_alpha': 4.375038444323036, 'reg_lambda': 1.9067013707511216, 'min_split_gain': 0.0125876650900141}. Best is trial 3 with value: 0.001813872911310272.
[I 2025-06-15 19:53:40,948] A new study created in memory with name: no-name-9c156b8c-f623-467c-9d63-a6b824b6b87f
[I 2025-06-15 19:53:40,986] Trial 0 finished with value: 0.01783610189301707 and parameters: {'n_estimators': 138, 'learning_rate': 0.04506988445136732, 'num_leaves': 249, 'max_depth': 6, 'min_child_samples': 28, 'subsample': 0.7895292623946412, 'colsample_bytree': 0.7606644105763752, 'reg_alpha': 7.124654512226629, 'reg_lambda': 4.332134345507219, 'min_split_gain': 0.0013288715107634575}. Best is trial 0 with value: 0.01783610189301707.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         685
1           Price_lag_2         550
14      pct_chg_Load_DA         378
16  roll2d_median_Price         272
2           Price_lag_7         265
11           NGas_lag_2         240
3         Load_DA_lag_0         195
10           Coal_lag_2         179
5       WindOn_DA_lag_0         168
13            EUA_lag_2         151
12            Oil_lag_2         134
15       lag168_Load_DA         130
9                  WD_7          10
8                  WD_6           5
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 21.89% done
********************* END NS.. 159 ****************************************************
********************* START NS ... 160 ****************************************************
[I 2025-06-15 19:53:42,237] A new study created in memory with name: no-name-62232b04-ad38-4a1f-8a40-4d7be7f59b93
[I 2025-06-15 19:53:43,183] Trial 0 finished with value: 0.00022285064635703988 and parameters: {'n_estimators': 233, 'learning_rate': 0.01909619644727047, 'num_leaves': 242, 'max_depth': 6, 'min_child_samples': 10, 'subsample': 0.6169237330762591, 'colsample_bytree': 0.8325836777899478, 'reg_alpha': 4.461139364087822, 'reg_lambda': 1.7770036878251672, 'min_split_gain': 0.0038548701700669543}. Best is trial 0 with value: 0.00022285064635703988.
[I 2025-06-15 19:53:43,343] Trial 1 finished with value: 0.0007749099204507574 and parameters: {'n_estimators': 52, 'learning_rate': 0.04143645514279984, 'num_leaves': 140, 'max_depth': 4, 'min_child_samples': 16, 'subsample': 0.7611936091769635, 'colsample_bytree': 0.7618671332911435, 'reg_alpha': 6.383781015911934, 'reg_lambda': 7.938390833864486, 'min_split_gain': 0.03754418193912969}. Best is trial 0 with value: 0.00022285064635703988.
[Trial 0] New best: 0.00022
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:43,615] Trial 2 finished with value: 0.0003823694107334393 and parameters: {'n_estimators': 116, 'learning_rate': 0.17977117262354483, 'num_leaves': 68, 'max_depth': 11, 'min_child_samples': 8, 'subsample': 0.9100156663306596, 'colsample_bytree': 0.6560964634225421, 'reg_alpha': 9.592918384227012, 'reg_lambda': 8.333273225414086, 'min_split_gain': 0.009594818221065816}. Best is trial 0 with value: 0.00022285064635703988.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:44,012] Trial 3 finished with value: 0.00029923491135721366 and parameters: {'n_estimators': 87, 'learning_rate': 0.1623791175538196, 'num_leaves': 192, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.6062530943545248, 'colsample_bytree': 0.7618000886275335, 'reg_alpha': 5.3354676263315906, 'reg_lambda': 8.714729393983866, 'min_split_gain': 0.00741604681250041}. Best is trial 0 with value: 0.00022285064635703988.
[I 2025-06-15 19:53:44,015] A new study created in memory with name: no-name-10c50b4c-c6d3-4e70-8b76-11529fd73777
[I 2025-06-15 19:53:44,055] Trial 0 finished with value: 0.008176303911866398 and parameters: {'n_estimators': 233, 'learning_rate': 0.06594530161977888, 'num_leaves': 127, 'max_depth': 6, 'min_child_samples': 13, 'subsample': 0.7906341928385282, 'colsample_bytree': 0.9715257186393682, 'reg_alpha': 8.520523015845743, 'reg_lambda': 7.592698440682315, 'min_split_gain': 0.029367787172228522}. Best is trial 0 with value: 0.008176303911866398.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2514
1           Price_lag_2        1569
14      pct_chg_Load_DA        1141
3         Load_DA_lag_0         532
11           NGas_lag_2         508
16  roll2d_median_Price         489
2           Price_lag_7         438
13            EUA_lag_2         267
10           Coal_lag_2         263
5       WindOn_DA_lag_0         233
15       lag168_Load_DA         222
12            Oil_lag_2         220
9                  WD_7          10
8                  WD_6           4
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 22.02% done
********************* END NS.. 160 ****************************************************
********************* START NS ... 161 ****************************************************
[I 2025-06-15 19:53:46,234] A new study created in memory with name: no-name-7f82fefe-7f66-49d0-9e59-beb5382322c3
[I 2025-06-15 19:53:47,592] Trial 0 finished with value: 0.005827048600195515 and parameters: {'n_estimators': 203, 'learning_rate': 0.017244940478949724, 'num_leaves': 219, 'max_depth': 7, 'min_child_samples': 26, 'subsample': 0.6359163396841182, 'colsample_bytree': 0.9868111822264105, 'reg_alpha': 1.80326934877871, 'reg_lambda': 0.31912356488717486, 'min_split_gain': 0.04270343127814767}. Best is trial 0 with value: 0.005827048600195515.
[Trial 0] New best: 0.00583
[I 2025-06-15 19:53:48,393] Trial 1 finished with value: 0.007241561170902529 and parameters: {'n_estimators': 86, 'learning_rate': 0.029668041045825797, 'num_leaves': 84, 'max_depth': 12, 'min_child_samples': 7, 'subsample': 0.5402784050101631, 'colsample_bytree': 0.9580523162569108, 'reg_alpha': 6.636163211831888, 'reg_lambda': 0.5085715121280865, 'min_split_gain': 0.02863539594462626}. Best is trial 0 with value: 0.005827048600195515.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:49,481] Trial 2 finished with value: 0.006556011859018474 and parameters: {'n_estimators': 220, 'learning_rate': 0.01575232479262239, 'num_leaves': 134, 'max_depth': 12, 'min_child_samples': 22, 'subsample': 0.5263062479788365, 'colsample_bytree': 0.8164887576834525, 'reg_alpha': 9.30473771138599, 'reg_lambda': 3.5090130130183184, 'min_split_gain': 0.0035477094198148097}. Best is trial 0 with value: 0.005827048600195515.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:50,207] Trial 3 finished with value: 0.005340759309861252 and parameters: {'n_estimators': 119, 'learning_rate': 0.04445776116050734, 'num_leaves': 207, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.5891867783028062, 'colsample_bytree': 0.8146953250750999, 'reg_alpha': 1.573806140617604, 'reg_lambda': 6.513101738349572, 'min_split_gain': 0.007273619721158225}. Best is trial 3 with value: 0.005340759309861252.
[I 2025-06-15 19:53:50,211] A new study created in memory with name: no-name-261674bc-e557-41b6-9c84-c17829053a74
[I 2025-06-15 19:53:50,240] Trial 0 finished with value: 0.02941112328568594 and parameters: {'n_estimators': 151, 'learning_rate': 0.10581652408929017, 'num_leaves': 244, 'max_depth': 16, 'min_child_samples': 9, 'subsample': 0.8892074606998654, 'colsample_bytree': 0.6819590705572989, 'reg_alpha': 9.193956724954802, 'reg_lambda': 3.1718732989943565, 'min_split_gain': 0.022531994091230717}. Best is trial 0 with value: 0.02941112328568594.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1475
1           Price_lag_2        1118
14      pct_chg_Load_DA         777
3         Load_DA_lag_0         398
11           NGas_lag_2         355
2           Price_lag_7         331
16  roll2d_median_Price         314
15       lag168_Load_DA         228
13            EUA_lag_2         225
5       WindOn_DA_lag_0         217
10           Coal_lag_2         163
12            Oil_lag_2         147
7                  WD_1          12
8                  WD_6           8
9                  WD_7           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 22.16% done
********************* END NS.. 161 ****************************************************
********************* START NS ... 162 ****************************************************
[I 2025-06-15 19:53:51,480] A new study created in memory with name: no-name-24672086-5c66-4617-a2ea-1dafc865ca12
[I 2025-06-15 19:53:51,825] Trial 0 finished with value: 0.012517987920578101 and parameters: {'n_estimators': 206, 'learning_rate': 0.17295322873498034, 'num_leaves': 63, 'max_depth': 10, 'min_child_samples': 14, 'subsample': 0.9400635049905344, 'colsample_bytree': 0.6356406311451696, 'reg_alpha': 1.5782642647684553, 'reg_lambda': 8.299515982453537, 'min_split_gain': 0.03800265938793341}. Best is trial 0 with value: 0.012517987920578101.
[Trial 0] New best: 0.01252
[I 2025-06-15 19:53:53,050] Trial 1 finished with value: 0.01363315369925639 and parameters: {'n_estimators': 201, 'learning_rate': 0.023717156475111724, 'num_leaves': 179, 'max_depth': 7, 'min_child_samples': 10, 'subsample': 0.6369457066404838, 'colsample_bytree': 0.6424473312820435, 'reg_alpha': 0.9318520645837891, 'reg_lambda': 5.540788066799152, 'min_split_gain': 0.024957720008692858}. Best is trial 0 with value: 0.012517987920578101.
[I 2025-06-15 19:53:53,265] Trial 2 finished with value: 0.013895555350907045 and parameters: {'n_estimators': 70, 'learning_rate': 0.24840015963281892, 'num_leaves': 180, 'max_depth': 12, 'min_child_samples': 9, 'subsample': 0.8755552478796486, 'colsample_bytree': 0.7049162686363579, 'reg_alpha': 5.592992854095181, 'reg_lambda': 2.525972362099369, 'min_split_gain': 0.048996568577079486}. Best is trial 0 with value: 0.012517987920578101.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:53:55,194] Trial 3 finished with value: 0.01378133827599535 and parameters: {'n_estimators': 244, 'learning_rate': 0.022545642169110872, 'num_leaves': 142, 'max_depth': 13, 'min_child_samples': 19, 'subsample': 0.8230096114014291, 'colsample_bytree': 0.6991547263826816, 'reg_alpha': 3.291254661490104, 'reg_lambda': 3.0725873975108744, 'min_split_gain': 0.02032890224730105}. Best is trial 0 with value: 0.012517987920578101.
[I 2025-06-15 19:53:55,198] A new study created in memory with name: no-name-ea6cd03f-f6ef-45fc-acc1-f23cfcbcafb7
[I 2025-06-15 19:53:55,234] Trial 0 finished with value: 0.09252661974738986 and parameters: {'n_estimators': 130, 'learning_rate': 0.20651930564062868, 'num_leaves': 34, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.5053219463488028, 'colsample_bytree': 0.9047024606848001, 'reg_alpha': 9.934827348803307, 'reg_lambda': 3.399584704591294, 'min_split_gain': 0.03291545930061436}. Best is trial 0 with value: 0.09252661974738986.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         382
1           Price_lag_2         280
14      pct_chg_Load_DA         222
3         Load_DA_lag_0         173
11           NGas_lag_2         142
16  roll2d_median_Price         138
2           Price_lag_7         112
15       lag168_Load_DA         104
10           Coal_lag_2          99
13            EUA_lag_2          95
5       WindOn_DA_lag_0          87
12            Oil_lag_2          59
7                  WD_1           5
9                  WD_7           3
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 22.30% done
********************* END NS.. 162 ****************************************************
********************* START NS ... 163 ****************************************************
[I 2025-06-15 19:53:56,213] A new study created in memory with name: no-name-daa4e8a6-be74-4a4e-afa9-6d064cba6b70
[I 2025-06-15 19:53:56,441] Trial 0 finished with value: 0.007903483458177834 and parameters: {'n_estimators': 140, 'learning_rate': 0.09568654612692601, 'num_leaves': 194, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.9426911797085081, 'colsample_bytree': 0.9000162829613703, 'reg_alpha': 3.0569981989816397, 'reg_lambda': 4.0140014574237615, 'min_split_gain': 0.03322043205789732}. Best is trial 0 with value: 0.007903483458177834.
[Trial 0] New best: 0.00790
[I 2025-06-15 19:53:57,315] Trial 1 finished with value: 0.008503677628171809 and parameters: {'n_estimators': 254, 'learning_rate': 0.021137101959575574, 'num_leaves': 142, 'max_depth': 5, 'min_child_samples': 14, 'subsample': 0.5939106114900004, 'colsample_bytree': 0.8842506314138421, 'reg_alpha': 5.877483189533537, 'reg_lambda': 1.1037267343365242, 'min_split_gain': 0.027366832889264992}. Best is trial 0 with value: 0.007903483458177834.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:53:59,939] Trial 2 finished with value: 0.008017154618493318 and parameters: {'n_estimators': 294, 'learning_rate': 0.021770846947679428, 'num_leaves': 59, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.7866011643606959, 'colsample_bytree': 0.5981025295727367, 'reg_alpha': 0.31412245112805937, 'reg_lambda': 8.179418245793563, 'min_split_gain': 0.02257492130454998}. Best is trial 0 with value: 0.007903483458177834.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:00,988] Trial 3 finished with value: 0.009722231765623379 and parameters: {'n_estimators': 165, 'learning_rate': 0.01722148912745837, 'num_leaves': 175, 'max_depth': 8, 'min_child_samples': 17, 'subsample': 0.7487157439675467, 'colsample_bytree': 0.9030188114889307, 'reg_alpha': 4.029677282174363, 'reg_lambda': 6.8725293769612765, 'min_split_gain': 0.004866537442605984}. Best is trial 0 with value: 0.007903483458177834.
[I 2025-06-15 19:54:00,991] A new study created in memory with name: no-name-022cbcbb-0882-4bd1-8443-83ab56888699
[I 2025-06-15 19:54:01,024] Trial 0 finished with value: 0.06562327081026473 and parameters: {'n_estimators': 96, 'learning_rate': 0.03517087757332024, 'num_leaves': 231, 'max_depth': 10, 'min_child_samples': 1, 'subsample': 0.8771435636078081, 'colsample_bytree': 0.5571007392296494, 'reg_alpha': 5.475466792083203, 'reg_lambda': 8.233258348631544, 'min_split_gain': 0.015575368879918894}. Best is trial 0 with value: 0.06562327081026473.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         410
14      pct_chg_Load_DA         184
1           Price_lag_2         137
3         Load_DA_lag_0         128
11           NGas_lag_2         103
16  roll2d_median_Price          77
2           Price_lag_7          67
15       lag168_Load_DA          54
13            EUA_lag_2          51
12            Oil_lag_2          37
5       WindOn_DA_lag_0          36
10           Coal_lag_2          35
7                  WD_1           4
9                  WD_7           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          58
1           Price_lag_2          14
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 22.44% done
********************* END NS.. 163 ****************************************************
********************* START NS ... 164 ****************************************************
[I 2025-06-15 19:54:01,903] A new study created in memory with name: no-name-1d8de198-5a7e-4bd3-8d81-efbca256414b
[I 2025-06-15 19:54:02,863] Trial 0 finished with value: 0.0011393377239237725 and parameters: {'n_estimators': 165, 'learning_rate': 0.020978997861804797, 'num_leaves': 237, 'max_depth': 11, 'min_child_samples': 9, 'subsample': 0.8495957903772721, 'colsample_bytree': 0.9263797519728147, 'reg_alpha': 4.011010667581905, 'reg_lambda': 1.542208063957714, 'min_split_gain': 0.015210162493725587}. Best is trial 0 with value: 0.0011393377239237725.
[Trial 0] New best: 0.00114
[I 2025-06-15 19:54:03,767] Trial 1 finished with value: 0.0012161192039986488 and parameters: {'n_estimators': 244, 'learning_rate': 0.040628382078576074, 'num_leaves': 123, 'max_depth': 16, 'min_child_samples': 18, 'subsample': 0.978624492950064, 'colsample_bytree': 0.9718699964012916, 'reg_alpha': 4.175539758758151, 'reg_lambda': 1.43090567071666, 'min_split_gain': 0.02969088743988703}. Best is trial 0 with value: 0.0011393377239237725.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:54:04,278] Trial 2 finished with value: 0.0013095672826663317 and parameters: {'n_estimators': 160, 'learning_rate': 0.015842493115327604, 'num_leaves': 218, 'max_depth': 5, 'min_child_samples': 11, 'subsample': 0.5710019323520947, 'colsample_bytree': 0.7203747798571604, 'reg_alpha': 9.300397888306092, 'reg_lambda': 6.9055318395632925, 'min_split_gain': 0.03916200326286942}. Best is trial 0 with value: 0.0011393377239237725.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:05,090] Trial 3 finished with value: 0.0020901392352084495 and parameters: {'n_estimators': 150, 'learning_rate': 0.013367095794856718, 'num_leaves': 73, 'max_depth': 10, 'min_child_samples': 15, 'subsample': 0.5776759237124144, 'colsample_bytree': 0.9464569118115778, 'reg_alpha': 4.856174506148182, 'reg_lambda': 7.155406958557176, 'min_split_gain': 0.008783600899429278}. Best is trial 0 with value: 0.0011393377239237725.
[I 2025-06-15 19:54:05,092] A new study created in memory with name: no-name-c2d054b4-1eb6-44cf-9a3f-3080ccb3b46d
[I 2025-06-15 19:54:05,121] Trial 0 finished with value: 0.018116449349809872 and parameters: {'n_estimators': 127, 'learning_rate': 0.20196137893107102, 'num_leaves': 139, 'max_depth': 14, 'min_child_samples': 18, 'subsample': 0.894511562679472, 'colsample_bytree': 0.9988732938659046, 'reg_alpha': 6.4371781859778, 'reg_lambda': 8.889888463219846, 'min_split_gain': 0.027686885187476654}. Best is trial 0 with value: 0.018116449349809872.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2340
1           Price_lag_2        1313
14      pct_chg_Load_DA        1083
16  roll2d_median_Price         557
2           Price_lag_7         557
11           NGas_lag_2         539
3         Load_DA_lag_0         366
10           Coal_lag_2         267
13            EUA_lag_2         227
12            Oil_lag_2         222
5       WindOn_DA_lag_0         207
15       lag168_Load_DA         123
9                  WD_7           3
8                  WD_6           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1           5
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 22.57% done
********************* END NS.. 164 ****************************************************
********************* START NS ... 165 ****************************************************
[I 2025-06-15 19:54:07,068] A new study created in memory with name: no-name-59391988-c4b4-4f0f-b680-f41a75844ce6
[I 2025-06-15 19:54:08,090] Trial 0 finished with value: 0.0010436124354864286 and parameters: {'n_estimators': 127, 'learning_rate': 0.11060201826745637, 'num_leaves': 207, 'max_depth': 10, 'min_child_samples': 29, 'subsample': 0.6910359501692109, 'colsample_bytree': 0.7582316817931347, 'reg_alpha': 0.15046660341630047, 'reg_lambda': 1.3578493526210655, 'min_split_gain': 0.034278905077518566}. Best is trial 0 with value: 0.0010436124354864286.
[Trial 0] New best: 0.00104
[I 2025-06-15 19:54:09,898] Trial 1 finished with value: 0.0013117120045908082 and parameters: {'n_estimators': 198, 'learning_rate': 0.0143082652895957, 'num_leaves': 202, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.9169432557391988, 'colsample_bytree': 0.6214457783216858, 'reg_alpha': 7.875462706830532, 'reg_lambda': 5.403628250071045, 'min_split_gain': 0.028771236563095572}. Best is trial 0 with value: 0.0010436124354864286.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:54:11,543] Trial 2 finished with value: 0.001513874754443757 and parameters: {'n_estimators': 300, 'learning_rate': 0.03046961168177852, 'num_leaves': 116, 'max_depth': 15, 'min_child_samples': 11, 'subsample': 0.9556014759899445, 'colsample_bytree': 0.9821506881780824, 'reg_alpha': 6.510837107322766, 'reg_lambda': 9.249941555131297, 'min_split_gain': 0.04146573511842405}. Best is trial 0 with value: 0.0010436124354864286.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:12,938] Trial 3 finished with value: 0.0014354526152815326 and parameters: {'n_estimators': 212, 'learning_rate': 0.04295918104105886, 'num_leaves': 45, 'max_depth': 16, 'min_child_samples': 25, 'subsample': 0.5581408025620404, 'colsample_bytree': 0.8725799235962233, 'reg_alpha': 4.441332026574768, 'reg_lambda': 7.66709943775451, 'min_split_gain': 0.002122127318551387}. Best is trial 0 with value: 0.0010436124354864286.
[I 2025-06-15 19:54:12,941] A new study created in memory with name: no-name-432e501a-a4c7-48e8-a901-e014fa754053
[I 2025-06-15 19:54:12,983] Trial 0 finished with value: 0.024321334723711886 and parameters: {'n_estimators': 187, 'learning_rate': 0.08489234307671888, 'num_leaves': 160, 'max_depth': 6, 'min_child_samples': 6, 'subsample': 0.5473831180495274, 'colsample_bytree': 0.8194538934914444, 'reg_alpha': 5.800712487302717, 'reg_lambda': 4.481936190232236, 'min_split_gain': 0.03602615384565699}. Best is trial 0 with value: 0.024321334723711886.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         871
1           Price_lag_2         856
14      pct_chg_Load_DA         661
3         Load_DA_lag_0         372
2           Price_lag_7         337
16  roll2d_median_Price         327
11           NGas_lag_2         270
15       lag168_Load_DA         269
13            EUA_lag_2         202
10           Coal_lag_2         191
5       WindOn_DA_lag_0         180
12            Oil_lag_2         147
9                  WD_7          10
7                  WD_1           8
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          16
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 22.71% done
********************* END NS.. 165 ****************************************************
********************* START NS ... 166 ****************************************************
[I 2025-06-15 19:54:14,292] A new study created in memory with name: no-name-c6354c91-600d-4acf-9767-40b78a7e7298
[I 2025-06-15 19:54:16,089] Trial 0 finished with value: 0.000996841861544291 and parameters: {'n_estimators': 238, 'learning_rate': 0.03920862357248923, 'num_leaves': 168, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.694270576525146, 'colsample_bytree': 0.6171775829684647, 'reg_alpha': 0.44839817400691095, 'reg_lambda': 6.398585648429908, 'min_split_gain': 0.025407152287662407}. Best is trial 0 with value: 0.000996841861544291.
[Trial 0] New best: 0.00100
[I 2025-06-15 19:54:17,226] Trial 1 finished with value: 0.0014616972860417448 and parameters: {'n_estimators': 168, 'learning_rate': 0.05704087508803035, 'num_leaves': 118, 'max_depth': 14, 'min_child_samples': 4, 'subsample': 0.9331073395780369, 'colsample_bytree': 0.5006376269840991, 'reg_alpha': 5.685167643402381, 'reg_lambda': 9.04498088387512, 'min_split_gain': 0.004188159480932235}. Best is trial 0 with value: 0.000996841861544291.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:54:17,657] Trial 2 finished with value: 0.0010552608455812193 and parameters: {'n_estimators': 247, 'learning_rate': 0.13321485292472018, 'num_leaves': 61, 'max_depth': 10, 'min_child_samples': 15, 'subsample': 0.7368635793144739, 'colsample_bytree': 0.5997968938364282, 'reg_alpha': 8.731106373722493, 'reg_lambda': 0.43738541265515907, 'min_split_gain': 0.010460223226735184}. Best is trial 0 with value: 0.000996841861544291.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:18,480] Trial 3 finished with value: 0.0009353009914182266 and parameters: {'n_estimators': 295, 'learning_rate': 0.05246602612127178, 'num_leaves': 70, 'max_depth': 11, 'min_child_samples': 24, 'subsample': 0.7177940106865464, 'colsample_bytree': 0.7616149386870047, 'reg_alpha': 0.9499948848637607, 'reg_lambda': 7.384415812292562, 'min_split_gain': 0.039047931543651604}. Best is trial 3 with value: 0.0009353009914182266.
[I 2025-06-15 19:54:18,483] A new study created in memory with name: no-name-e52f4f9a-4af8-4b4c-b568-512c79191701
[I 2025-06-15 19:54:18,521] Trial 0 finished with value: 0.05927422709103684 and parameters: {'n_estimators': 124, 'learning_rate': 0.03171160863599907, 'num_leaves': 252, 'max_depth': 16, 'min_child_samples': 26, 'subsample': 0.8952480995752766, 'colsample_bytree': 0.7840426668840135, 'reg_alpha': 8.807669333714509, 'reg_lambda': 3.678519982364766, 'min_split_gain': 0.03869835345662717}. Best is trial 0 with value: 0.05927422709103684.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1344
1           Price_lag_2        1261
14      pct_chg_Load_DA        1017
3         Load_DA_lag_0         738
11           NGas_lag_2         538
15       lag168_Load_DA         506
2           Price_lag_7         452
5       WindOn_DA_lag_0         423
16  roll2d_median_Price         400
10           Coal_lag_2         324
12            Oil_lag_2         285
13            EUA_lag_2         231
7                  WD_1          16
9                  WD_7          15
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          11
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 22.85% done
********************* END NS.. 166 ****************************************************
********************* START NS ... 167 ****************************************************
[I 2025-06-15 19:54:19,754] A new study created in memory with name: no-name-f866a3e2-a39c-44b1-b4c6-4462fb3c0345
[I 2025-06-15 19:54:20,742] Trial 0 finished with value: 0.0004914647578806683 and parameters: {'n_estimators': 275, 'learning_rate': 0.04388716264535567, 'num_leaves': 173, 'max_depth': 16, 'min_child_samples': 19, 'subsample': 0.8270306982338048, 'colsample_bytree': 0.6425176603326805, 'reg_alpha': 8.910731027626403, 'reg_lambda': 9.343401513344208, 'min_split_gain': 0.0055831189286634055}. Best is trial 0 with value: 0.0004914647578806683.
[Trial 0] New best: 0.00049
[I 2025-06-15 19:54:21,739] Trial 1 finished with value: 0.0005294616711181691 and parameters: {'n_estimators': 259, 'learning_rate': 0.025737520883640612, 'num_leaves': 186, 'max_depth': 5, 'min_child_samples': 17, 'subsample': 0.8964200592808897, 'colsample_bytree': 0.7883578016349378, 'reg_alpha': 7.720167760680922, 'reg_lambda': 5.392471394485408, 'min_split_gain': 0.027582110395970034}. Best is trial 0 with value: 0.0004914647578806683.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:54:24,221] Trial 2 finished with value: 0.000472963693585985 and parameters: {'n_estimators': 294, 'learning_rate': 0.015924985789208566, 'num_leaves': 172, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.5828837903187506, 'colsample_bytree': 0.7319549412781581, 'reg_alpha': 2.5487879723200413, 'reg_lambda': 2.7813923772966564, 'min_split_gain': 0.04640381579322245}. Best is trial 2 with value: 0.000472963693585985.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:25,132] Trial 3 finished with value: 0.0002692242210869968 and parameters: {'n_estimators': 149, 'learning_rate': 0.021542625301052987, 'num_leaves': 129, 'max_depth': 6, 'min_child_samples': 3, 'subsample': 0.7823207313554387, 'colsample_bytree': 0.8110095609333775, 'reg_alpha': 0.658058445640326, 'reg_lambda': 0.11049045057447304, 'min_split_gain': 0.04344736577510193}. Best is trial 3 with value: 0.0002692242210869968.
[I 2025-06-15 19:54:25,134] A new study created in memory with name: no-name-1de382b6-6666-4520-8e93-f0e9881d0c37
[I 2025-06-15 19:54:25,159] Trial 0 finished with value: 0.07180023970156639 and parameters: {'n_estimators': 124, 'learning_rate': 0.09607693590723486, 'num_leaves': 57, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.796644074718405, 'colsample_bytree': 0.862787600950137, 'reg_alpha': 8.783803210275945, 'reg_lambda': 0.7675104066953997, 'min_split_gain': 0.029291314778429785}. Best is trial 0 with value: 0.07180023970156639.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2443
1           Price_lag_2        1761
14      pct_chg_Load_DA        1066
16  roll2d_median_Price         493
2           Price_lag_7         475
11           NGas_lag_2         424
3         Load_DA_lag_0         380
10           Coal_lag_2         266
12            Oil_lag_2         231
13            EUA_lag_2         169
15       lag168_Load_DA         166
5       WindOn_DA_lag_0         139
7                  WD_1           3
9                  WD_7           3
8                  WD_6           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          12
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 22.98% done
********************* END NS.. 167 ****************************************************
********************* START NS ... 168 ****************************************************
[I 2025-06-15 19:54:26,312] A new study created in memory with name: no-name-ff4cc94e-1db2-491c-b1a9-e1a0a28f1654
[I 2025-06-15 19:54:27,104] Trial 0 finished with value: 0.0031575338918137645 and parameters: {'n_estimators': 256, 'learning_rate': 0.014967352664318746, 'num_leaves': 151, 'max_depth': 15, 'min_child_samples': 19, 'subsample': 0.6415017488515253, 'colsample_bytree': 0.8911007156558113, 'reg_alpha': 8.3632137411732, 'reg_lambda': 1.5415175338561393, 'min_split_gain': 0.04115963210370007}. Best is trial 0 with value: 0.0031575338918137645.
[Trial 0] New best: 0.00316
[I 2025-06-15 19:54:27,378] Trial 1 finished with value: 0.0036606810759906272 and parameters: {'n_estimators': 117, 'learning_rate': 0.01461233051695734, 'num_leaves': 198, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.9019097139620036, 'colsample_bytree': 0.8503816678314382, 'reg_alpha': 7.476080039429749, 'reg_lambda': 1.4964283949495472, 'min_split_gain': 0.03988322117189702}. Best is trial 0 with value: 0.0031575338918137645.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:54:27,981] Trial 2 finished with value: 0.002744524963184243 and parameters: {'n_estimators': 211, 'learning_rate': 0.10751509031540377, 'num_leaves': 187, 'max_depth': 15, 'min_child_samples': 24, 'subsample': 0.5053687891527707, 'colsample_bytree': 0.8698399801145472, 'reg_alpha': 0.4385650148963782, 'reg_lambda': 2.9385483159985535, 'min_split_gain': 0.030227697890492674}. Best is trial 2 with value: 0.002744524963184243.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:29,212] Trial 3 finished with value: 0.002958660565131447 and parameters: {'n_estimators': 276, 'learning_rate': 0.027432400216563065, 'num_leaves': 241, 'max_depth': 12, 'min_child_samples': 16, 'subsample': 0.6566964751333393, 'colsample_bytree': 0.7858176760370019, 'reg_alpha': 4.154060066249869, 'reg_lambda': 9.896833200745666, 'min_split_gain': 0.017334703366934292}. Best is trial 2 with value: 0.002744524963184243.
[I 2025-06-15 19:54:29,215] A new study created in memory with name: no-name-abbcaeb6-94a7-48ee-bafa-c2f2460fb386
[I 2025-06-15 19:54:29,271] Trial 0 finished with value: 0.01210887510746257 and parameters: {'n_estimators': 272, 'learning_rate': 0.05497445396861491, 'num_leaves': 69, 'max_depth': 12, 'min_child_samples': 11, 'subsample': 0.6903509736589338, 'colsample_bytree': 0.5603081695718681, 'reg_alpha': 0.8514865695920981, 'reg_lambda': 1.311423472435761, 'min_split_gain': 0.04249043815431927}. Best is trial 0 with value: 0.01210887510746257.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA         861
0           Price_lag_1         850
1           Price_lag_2         815
3         Load_DA_lag_0         492
2           Price_lag_7         395
15       lag168_Load_DA         369
16  roll2d_median_Price         368
11           NGas_lag_2         305
5       WindOn_DA_lag_0         278
10           Coal_lag_2         188
12            Oil_lag_2         175
13            EUA_lag_2         141
7                  WD_1          10
9                  WD_7          10
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          43
1           Price_lag_2          19
16  roll2d_median_Price          15
5       WindOn_DA_lag_0          10
9                  WD_7           5
13            EUA_lag_2           5
3         Load_DA_lag_0           3
12            Oil_lag_2           3
2           Price_lag_7           2
11           NGas_lag_2           2
14      pct_chg_Load_DA           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
10           Coal_lag_2           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 23.12% done
********************* END NS.. 168 ****************************************************
********************* START NS ... 169 ****************************************************
[I 2025-06-15 19:54:30,544] A new study created in memory with name: no-name-8c08f122-26e4-49e3-ae44-bd1787bf1ca0
[I 2025-06-15 19:54:31,113] Trial 0 finished with value: 0.00687464483809658 and parameters: {'n_estimators': 178, 'learning_rate': 0.04519046986605761, 'num_leaves': 239, 'max_depth': 8, 'min_child_samples': 29, 'subsample': 0.5158969064567911, 'colsample_bytree': 0.577299751377321, 'reg_alpha': 4.292137462890686, 'reg_lambda': 0.9279939198360521, 'min_split_gain': 0.03317785116460523}. Best is trial 0 with value: 0.00687464483809658.
[Trial 0] New best: 0.00687
[I 2025-06-15 19:54:31,346] Trial 1 finished with value: 0.007722552571977961 and parameters: {'n_estimators': 197, 'learning_rate': 0.08661105137312823, 'num_leaves': 228, 'max_depth': 4, 'min_child_samples': 16, 'subsample': 0.8231990297858782, 'colsample_bytree': 0.7824077633650743, 'reg_alpha': 8.42474270419412, 'reg_lambda': 3.52775389851562, 'min_split_gain': 0.021151354386842056}. Best is trial 0 with value: 0.00687464483809658.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:54:31,722] Trial 2 finished with value: 0.0068147647813029885 and parameters: {'n_estimators': 238, 'learning_rate': 0.07186363513462607, 'num_leaves': 222, 'max_depth': 5, 'min_child_samples': 2, 'subsample': 0.9183495349989411, 'colsample_bytree': 0.8096037033311198, 'reg_alpha': 6.844417444982422, 'reg_lambda': 0.4376529222596226, 'min_split_gain': 0.03481411666361632}. Best is trial 2 with value: 0.0068147647813029885.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:32,180] Trial 3 finished with value: 0.006083532517901627 and parameters: {'n_estimators': 152, 'learning_rate': 0.054193485459476096, 'num_leaves': 110, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.5906414550760386, 'colsample_bytree': 0.9815453453151983, 'reg_alpha': 1.5578599220019396, 'reg_lambda': 5.1369282800153275, 'min_split_gain': 0.025577609063082277}. Best is trial 3 with value: 0.006083532517901627.
[I 2025-06-15 19:54:32,183] A new study created in memory with name: no-name-26458fc0-3ad6-400e-8eca-c6b25060515a
[I 2025-06-15 19:54:32,202] Trial 0 finished with value: 0.021521408005991036 and parameters: {'n_estimators': 160, 'learning_rate': 0.03798755437104606, 'num_leaves': 231, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.9353039173516156, 'colsample_bytree': 0.7598060561909661, 'reg_alpha': 8.673982599131975, 'reg_lambda': 2.3759113747254537, 'min_split_gain': 0.028729833866108924}. Best is trial 0 with value: 0.021521408005991036.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1285
14      pct_chg_Load_DA         788
1           Price_lag_2         598
3         Load_DA_lag_0         485
11           NGas_lag_2         364
16  roll2d_median_Price         356
2           Price_lag_7         283
15       lag168_Load_DA         260
5       WindOn_DA_lag_0         199
13            EUA_lag_2         193
10           Coal_lag_2         177
12            Oil_lag_2         159
7                  WD_1          11
9                  WD_7           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 23.26% done
********************* END NS.. 169 ****************************************************
********************* START NS ... 170 ****************************************************
[I 2025-06-15 19:54:33,159] A new study created in memory with name: no-name-f509482c-8c04-4080-854a-3c69ad3bb09d
[I 2025-06-15 19:54:33,292] Trial 0 finished with value: 0.012392148551073694 and parameters: {'n_estimators': 76, 'learning_rate': 0.01051627846488469, 'num_leaves': 142, 'max_depth': 4, 'min_child_samples': 3, 'subsample': 0.5726398931508085, 'colsample_bytree': 0.5710048043533156, 'reg_alpha': 9.337165889560849, 'reg_lambda': 6.986360880395093, 'min_split_gain': 0.015497682652143914}. Best is trial 0 with value: 0.012392148551073694.
[Trial 0] New best: 0.01239
[I 2025-06-15 19:54:34,036] Trial 1 finished with value: 0.0043793246579401585 and parameters: {'n_estimators': 202, 'learning_rate': 0.010594090029815366, 'num_leaves': 157, 'max_depth': 9, 'min_child_samples': 15, 'subsample': 0.6023421141745706, 'colsample_bytree': 0.88756140048748, 'reg_alpha': 5.044510773454103, 'reg_lambda': 1.4778450509216545, 'min_split_gain': 0.03904318305533991}. Best is trial 1 with value: 0.0043793246579401585.
[Trial 1] New best: 0.00438
[I 2025-06-15 19:54:35,125] Trial 2 finished with value: 0.004194489814513058 and parameters: {'n_estimators': 297, 'learning_rate': 0.012931157635869004, 'num_leaves': 126, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.8686379173042751, 'colsample_bytree': 0.8580867567573822, 'reg_alpha': 6.524733508301972, 'reg_lambda': 4.524995984237593, 'min_split_gain': 0.0007533903049032065}. Best is trial 2 with value: 0.004194489814513058.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:54:35,326] Trial 3 finished with value: 0.004751751960588971 and parameters: {'n_estimators': 140, 'learning_rate': 0.18751589382932865, 'num_leaves': 107, 'max_depth': 9, 'min_child_samples': 4, 'subsample': 0.8115715087011912, 'colsample_bytree': 0.6310530680229629, 'reg_alpha': 3.693831669608766, 'reg_lambda': 8.545984580347604, 'min_split_gain': 0.04296178172548843}. Best is trial 2 with value: 0.004194489814513058.
[I 2025-06-15 19:54:35,466] Trial 4 finished with value: 0.010098447875048303 and parameters: {'n_estimators': 70, 'learning_rate': 0.011664662710327943, 'num_leaves': 146, 'max_depth': 4, 'min_child_samples': 13, 'subsample': 0.8114217779620293, 'colsample_bytree': 0.9305901927505862, 'reg_alpha': 5.466330869880698, 'reg_lambda': 8.2172328800764, 'min_split_gain': 0.005446225189105936}. Best is trial 2 with value: 0.004194489814513058.
[I 2025-06-15 19:54:35,468] A new study created in memory with name: no-name-8229c979-1115-44c3-a2c8-3babe0dfb6ce
[I 2025-06-15 19:54:35,498] Trial 0 finished with value: 0.012564653337222434 and parameters: {'n_estimators': 283, 'learning_rate': 0.13793495159810448, 'num_leaves': 114, 'max_depth': 10, 'min_child_samples': 12, 'subsample': 0.5886347303422602, 'colsample_bytree': 0.7280112240187491, 'reg_alpha': 0.46820671697469063, 'reg_lambda': 4.51388457393559, 'min_split_gain': 0.028446661648946426}. Best is trial 0 with value: 0.012564653337222434.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3431
1           Price_lag_2        2036
14      pct_chg_Load_DA        1506
16  roll2d_median_Price         836
2           Price_lag_7         730
11           NGas_lag_2         710
3         Load_DA_lag_0         610
13            EUA_lag_2         473
10           Coal_lag_2         447
5       WindOn_DA_lag_0         361
12            Oil_lag_2         322
15       lag168_Load_DA         306
9                  WD_7          13
8                  WD_6           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1          25
15       lag168_Load_DA           2
1           Price_lag_2           2
3         Load_DA_lag_0           1
13            EUA_lag_2           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 23.39% done
********************* END NS.. 170 ****************************************************
********************* START NS ... 171 ****************************************************
[I 2025-06-15 19:54:36,782] A new study created in memory with name: no-name-0c01a6f9-7a4b-4aa6-9da0-9aaf2951b61b
[I 2025-06-15 19:54:37,159] Trial 0 finished with value: 0.0009292158868379504 and parameters: {'n_estimators': 162, 'learning_rate': 0.031600825258050406, 'num_leaves': 117, 'max_depth': 9, 'min_child_samples': 4, 'subsample': 0.7123575024888855, 'colsample_bytree': 0.5273927089772088, 'reg_alpha': 8.294620649100056, 'reg_lambda': 9.129931370467009, 'min_split_gain': 0.04493574200028441}. Best is trial 0 with value: 0.0009292158868379504.
[Trial 0] New best: 0.00093
[I 2025-06-15 19:54:37,566] Trial 1 finished with value: 0.000673204146071948 and parameters: {'n_estimators': 287, 'learning_rate': 0.0585451998269086, 'num_leaves': 232, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.7692168952465324, 'colsample_bytree': 0.7363674060383496, 'reg_alpha': 8.981898425925237, 'reg_lambda': 4.937075510950947, 'min_split_gain': 0.035930058462356765}. Best is trial 1 with value: 0.000673204146071948.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:54:38,010] Trial 2 finished with value: 0.0005246015128306272 and parameters: {'n_estimators': 285, 'learning_rate': 0.1069042327422787, 'num_leaves': 60, 'max_depth': 15, 'min_child_samples': 23, 'subsample': 0.6639547081639554, 'colsample_bytree': 0.7417282032849122, 'reg_alpha': 0.3935457722561697, 'reg_lambda': 8.605124730087763, 'min_split_gain': 0.02645332675222188}. Best is trial 2 with value: 0.0005246015128306272.
[I 2025-06-15 19:54:38,206] Trial 3 finished with value: 0.0007255553750346108 and parameters: {'n_estimators': 71, 'learning_rate': 0.12966331496674424, 'num_leaves': 117, 'max_depth': 11, 'min_child_samples': 29, 'subsample': 0.9998607435057049, 'colsample_bytree': 0.912893286249989, 'reg_alpha': 5.682071415478695, 'reg_lambda': 2.6925393253986973, 'min_split_gain': 0.0327212109369317}. Best is trial 2 with value: 0.0005246015128306272.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:38,209] A new study created in memory with name: no-name-48bbe2e2-165b-491a-b2cd-8af29c044bde
[I 2025-06-15 19:54:38,233] Trial 0 finished with value: 0.006289064401920723 and parameters: {'n_estimators': 189, 'learning_rate': 0.25882686402426386, 'num_leaves': 98, 'max_depth': 12, 'min_child_samples': 3, 'subsample': 0.985777431823344, 'colsample_bytree': 0.8607930817218843, 'reg_alpha': 5.869685099674488, 'reg_lambda': 4.954986100845611, 'min_split_gain': 0.04193793860732904}. Best is trial 0 with value: 0.006289064401920723.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         749
1           Price_lag_2         596
14      pct_chg_Load_DA         552
3         Load_DA_lag_0         390
5       WindOn_DA_lag_0         280
2           Price_lag_7         272
15       lag168_Load_DA         272
16  roll2d_median_Price         264
11           NGas_lag_2         260
12            Oil_lag_2         199
10           Coal_lag_2         145
13            EUA_lag_2         145
7                  WD_1           9
9                  WD_7           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 23.53% done
********************* END NS.. 171 ****************************************************
********************* START NS ... 172 ****************************************************
[I 2025-06-15 19:54:39,114] A new study created in memory with name: no-name-e04a7b0d-d70e-4238-ad12-0dce0ec0ad1c
[I 2025-06-15 19:54:39,542] Trial 0 finished with value: 0.00012223769900167948 and parameters: {'n_estimators': 104, 'learning_rate': 0.03365271122754117, 'num_leaves': 151, 'max_depth': 9, 'min_child_samples': 4, 'subsample': 0.6919136241488495, 'colsample_bytree': 0.6054493962788781, 'reg_alpha': 2.1828268476496406, 'reg_lambda': 0.9795562255163537, 'min_split_gain': 0.034929618099563214}. Best is trial 0 with value: 0.00012223769900167948.
[Trial 0] New best: 0.00012
[I 2025-06-15 19:54:40,092] Trial 1 finished with value: 0.0001595656177986868 and parameters: {'n_estimators': 191, 'learning_rate': 0.05543813162055055, 'num_leaves': 187, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.817584641471945, 'colsample_bytree': 0.7094155299866006, 'reg_alpha': 2.7434129771997484, 'reg_lambda': 7.68459489665285, 'min_split_gain': 0.004247796629359807}. Best is trial 0 with value: 0.00012223769900167948.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:54:40,394] Trial 2 finished with value: 0.00024419798075527537 and parameters: {'n_estimators': 182, 'learning_rate': 0.0703111878044714, 'num_leaves': 98, 'max_depth': 6, 'min_child_samples': 28, 'subsample': 0.5577998972625966, 'colsample_bytree': 0.67331326317225, 'reg_alpha': 4.663880716959406, 'reg_lambda': 9.09137703054068, 'min_split_gain': 0.045066809892060536}. Best is trial 0 with value: 0.00012223769900167948.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:40,938] Trial 3 finished with value: 0.0003514246020251633 and parameters: {'n_estimators': 136, 'learning_rate': 0.034188773606385864, 'num_leaves': 219, 'max_depth': 12, 'min_child_samples': 26, 'subsample': 0.9766263749586255, 'colsample_bytree': 0.6850769753742679, 'reg_alpha': 8.123340996434308, 'reg_lambda': 1.6779945926901552, 'min_split_gain': 0.04605899440381899}. Best is trial 0 with value: 0.00012223769900167948.
[I 2025-06-15 19:54:40,939] A new study created in memory with name: no-name-85398de1-3688-4a46-aa0d-1e1031b9ea0d
[I 2025-06-15 19:54:40,962] Trial 0 finished with value: 0.0003361077599373096 and parameters: {'n_estimators': 79, 'learning_rate': 0.03662995517878085, 'num_leaves': 196, 'max_depth': 4, 'min_child_samples': 23, 'subsample': 0.8572146164124337, 'colsample_bytree': 0.512255924461513, 'reg_alpha': 3.2845222602675173, 'reg_lambda': 7.724280152685531, 'min_split_gain': 0.022698258197721706}. Best is trial 0 with value: 0.0003361077599373096.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1211
1           Price_lag_2        1187
14      pct_chg_Load_DA         755
2           Price_lag_7         527
3         Load_DA_lag_0         505
16  roll2d_median_Price         498
11           NGas_lag_2         479
10           Coal_lag_2         386
15       lag168_Load_DA         346
13            EUA_lag_2         299
5       WindOn_DA_lag_0         239
12            Oil_lag_2         239
9                  WD_7          16
8                  WD_6           9
7                  WD_1           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 23.67% done
********************* END NS.. 172 ****************************************************
********************* START NS ... 173 ****************************************************
[I 2025-06-15 19:54:42,023] A new study created in memory with name: no-name-19c6ef66-be02-4d86-9480-cb00c8f8f1ba
[I 2025-06-15 19:54:42,280] Trial 0 finished with value: 0.0003646657293043937 and parameters: {'n_estimators': 230, 'learning_rate': 0.07701351514641697, 'num_leaves': 96, 'max_depth': 14, 'min_child_samples': 2, 'subsample': 0.6795818249006441, 'colsample_bytree': 0.827569121507147, 'reg_alpha': 6.947106383077864, 'reg_lambda': 3.1241156607961083, 'min_split_gain': 0.04564225139031582}. Best is trial 0 with value: 0.0003646657293043937.
[I 2025-06-15 19:54:42,446] Trial 1 finished with value: 0.00022311166403897885 and parameters: {'n_estimators': 53, 'learning_rate': 0.06249037290686595, 'num_leaves': 163, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.9560377554674871, 'colsample_bytree': 0.5846948024169207, 'reg_alpha': 2.243301196541858, 'reg_lambda': 2.275532405666433, 'min_split_gain': 7.0735420902801055e-06}. Best is trial 1 with value: 0.00022311166403897885.
[Trial 0] New best: 0.00036
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:54:42,693] Trial 2 finished with value: 0.000788641269200765 and parameters: {'n_estimators': 65, 'learning_rate': 0.02621724557079018, 'num_leaves': 71, 'max_depth': 7, 'min_child_samples': 12, 'subsample': 0.6503733782680214, 'colsample_bytree': 0.7740519977089089, 'reg_alpha': 5.291436236930464, 'reg_lambda': 5.45473275654555, 'min_split_gain': 0.032247550573318594}. Best is trial 1 with value: 0.00022311166403897885.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:43,006] Trial 3 finished with value: 0.00029302705413057536 and parameters: {'n_estimators': 117, 'learning_rate': 0.1344534233837031, 'num_leaves': 96, 'max_depth': 6, 'min_child_samples': 1, 'subsample': 0.8404638014392036, 'colsample_bytree': 0.8410362180098336, 'reg_alpha': 5.983529721165204, 'reg_lambda': 5.842836215005978, 'min_split_gain': 0.04282598350738251}. Best is trial 1 with value: 0.00022311166403897885.
[I 2025-06-15 19:54:43,008] A new study created in memory with name: no-name-4dc7df77-2bbe-4b38-bd03-6ddeaf0c1f5c
[I 2025-06-15 19:54:43,035] Trial 0 finished with value: 0.001300261545555175 and parameters: {'n_estimators': 183, 'learning_rate': 0.02822354844399883, 'num_leaves': 53, 'max_depth': 10, 'min_child_samples': 12, 'subsample': 0.6207617609399232, 'colsample_bytree': 0.842005695767623, 'reg_alpha': 2.9539606634743896, 'reg_lambda': 4.261374144372315, 'min_split_gain': 0.015650105429564715}. Best is trial 0 with value: 0.001300261545555175.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         536
1           Price_lag_2         415
14      pct_chg_Load_DA         243
3         Load_DA_lag_0         154
11           NGas_lag_2         141
16  roll2d_median_Price         136
2           Price_lag_7         132
15       lag168_Load_DA          77
13            EUA_lag_2          72
10           Coal_lag_2          63
12            Oil_lag_2          50
5       WindOn_DA_lag_0          29
8                  WD_6           2
7                  WD_1           1
6      WindOff_DA_lag_0           0
9                  WD_7           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           7
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 23.80% done
********************* END NS.. 173 ****************************************************
********************* START NS ... 174 ****************************************************
[I 2025-06-15 19:54:43,826] A new study created in memory with name: no-name-7e289c3b-1758-4924-b63d-50c7bb4e3de3
[I 2025-06-15 19:54:45,174] Trial 0 finished with value: 0.0005892519299695457 and parameters: {'n_estimators': 289, 'learning_rate': 0.010856512903009654, 'num_leaves': 223, 'max_depth': 10, 'min_child_samples': 13, 'subsample': 0.9602234966644028, 'colsample_bytree': 0.9695309880101283, 'reg_alpha': 3.198192268616454, 'reg_lambda': 5.458994350049741, 'min_split_gain': 0.03068910327163254}. Best is trial 0 with value: 0.0005892519299695457.
[Trial 0] New best: 0.00059
[I 2025-06-15 19:54:45,957] Trial 1 finished with value: 0.0022009727230731906 and parameters: {'n_estimators': 151, 'learning_rate': 0.010683920720411121, 'num_leaves': 177, 'max_depth': 11, 'min_child_samples': 7, 'subsample': 0.5806755354189058, 'colsample_bytree': 0.7936472193879394, 'reg_alpha': 5.252560298608455, 'reg_lambda': 9.04754039047008, 'min_split_gain': 0.033816431981764365}. Best is trial 0 with value: 0.0005892519299695457.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:54:46,225] Trial 2 finished with value: 0.010505498875242512 and parameters: {'n_estimators': 59, 'learning_rate': 0.01088563222672846, 'num_leaves': 199, 'max_depth': 10, 'min_child_samples': 15, 'subsample': 0.7185071072509961, 'colsample_bytree': 0.723380171015837, 'reg_alpha': 4.756024866087199, 'reg_lambda': 4.331031391569425, 'min_split_gain': 0.03498722303209934}. Best is trial 0 with value: 0.0005892519299695457.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:46,549] Trial 3 finished with value: 0.000792192356336251 and parameters: {'n_estimators': 285, 'learning_rate': 0.24012154985286738, 'num_leaves': 205, 'max_depth': 13, 'min_child_samples': 2, 'subsample': 0.9192935161966445, 'colsample_bytree': 0.5482921916290404, 'reg_alpha': 7.0470956952138835, 'reg_lambda': 8.025120625407864, 'min_split_gain': 0.046121791831751704}. Best is trial 0 with value: 0.0005892519299695457.
[I 2025-06-15 19:54:46,552] A new study created in memory with name: no-name-f65ccc1d-eaae-4a15-9009-b3a93591a9bc
[I 2025-06-15 19:54:46,574] Trial 0 finished with value: 0.006181788761123182 and parameters: {'n_estimators': 73, 'learning_rate': 0.1958875067606958, 'num_leaves': 112, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.5083498023017163, 'colsample_bytree': 0.7815403407050185, 'reg_alpha': 3.3059286838528212, 'reg_lambda': 4.827425015943554, 'min_split_gain': 0.03320876329817483}. Best is trial 0 with value: 0.006181788761123182.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        4497
14      pct_chg_Load_DA        1873
1           Price_lag_2        1826
16  roll2d_median_Price         900
11           NGas_lag_2         873
2           Price_lag_7         760
3         Load_DA_lag_0         430
12            Oil_lag_2         313
10           Coal_lag_2         243
13            EUA_lag_2         191
15       lag168_Load_DA         186
5       WindOn_DA_lag_0         147
9                  WD_7           3
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 23.94% done
********************* END NS.. 174 ****************************************************
********************* START NS ... 175 ****************************************************
[I 2025-06-15 19:54:48,276] A new study created in memory with name: no-name-dddf84e7-6c37-451e-ac82-73c7e494ec1b
[I 2025-06-15 19:54:48,419] Trial 0 finished with value: 0.00463841395367479 and parameters: {'n_estimators': 228, 'learning_rate': 0.2950180056575315, 'num_leaves': 151, 'max_depth': 5, 'min_child_samples': 21, 'subsample': 0.6946216345634737, 'colsample_bytree': 0.9035903535647942, 'reg_alpha': 4.137187131027567, 'reg_lambda': 8.058390860634926, 'min_split_gain': 0.029718900341768197}. Best is trial 0 with value: 0.00463841395367479.
[Trial 0] New best: 0.00464
[I 2025-06-15 19:54:48,958] Trial 1 finished with value: 0.007965669759098404 and parameters: {'n_estimators': 113, 'learning_rate': 0.026614512119651594, 'num_leaves': 231, 'max_depth': 15, 'min_child_samples': 25, 'subsample': 0.7853478495741788, 'colsample_bytree': 0.6507508463953269, 'reg_alpha': 2.280075488039931, 'reg_lambda': 5.668158670833389, 'min_split_gain': 0.013184628516234871}. Best is trial 0 with value: 0.00463841395367479.
[I 2025-06-15 19:54:49,161] Trial 2 finished with value: 0.005673811751109926 and parameters: {'n_estimators': 92, 'learning_rate': 0.11471231028139879, 'num_leaves': 122, 'max_depth': 15, 'min_child_samples': 6, 'subsample': 0.6146755943203364, 'colsample_bytree': 0.602272793627799, 'reg_alpha': 7.616781024051401, 'reg_lambda': 7.125970721825525, 'min_split_gain': 0.04849454998900108}. Best is trial 0 with value: 0.00463841395367479.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:49,472] Trial 3 finished with value: 0.006032392062655548 and parameters: {'n_estimators': 202, 'learning_rate': 0.1119462429578386, 'num_leaves': 114, 'max_depth': 10, 'min_child_samples': 13, 'subsample': 0.8362620676776344, 'colsample_bytree': 0.7862797722015357, 'reg_alpha': 7.129760150885734, 'reg_lambda': 5.2137553937781735, 'min_split_gain': 0.04254224096699763}. Best is trial 0 with value: 0.00463841395367479.
[I 2025-06-15 19:54:49,474] A new study created in memory with name: no-name-6b9f5ad7-045f-42bf-975e-d73bde837686
[I 2025-06-15 19:54:49,511] Trial 0 finished with value: 0.08450812202458087 and parameters: {'n_estimators': 262, 'learning_rate': 0.01500726993622144, 'num_leaves': 94, 'max_depth': 14, 'min_child_samples': 5, 'subsample': 0.7110735596250235, 'colsample_bytree': 0.8388851174705674, 'reg_alpha': 8.284049624190782, 'reg_lambda': 0.28924811010187623, 'min_split_gain': 0.0056536476333067355}. Best is trial 0 with value: 0.08450812202458087.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         173
14      pct_chg_Load_DA          85
1           Price_lag_2          61
3         Load_DA_lag_0          52
11           NGas_lag_2          51
2           Price_lag_7          34
16  roll2d_median_Price          28
15       lag168_Load_DA          27
10           Coal_lag_2          25
5       WindOn_DA_lag_0          20
12            Oil_lag_2          18
13            EUA_lag_2          18
7                  WD_1           2
6      WindOff_DA_lag_0           0
9                  WD_7           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          53
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 24.08% done
********************* END NS.. 175 ****************************************************
********************* START NS ... 176 ****************************************************
[I 2025-06-15 19:54:50,263] A new study created in memory with name: no-name-568db5d4-5211-44c7-b8b5-faa2a940b57a
[I 2025-06-15 19:54:50,525] Trial 0 finished with value: 0.022866977561011446 and parameters: {'n_estimators': 117, 'learning_rate': 0.014197137481761216, 'num_leaves': 167, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.5827331287088111, 'colsample_bytree': 0.8888179824724587, 'reg_alpha': 3.353545443062141, 'reg_lambda': 9.51162611375287, 'min_split_gain': 0.024798952052116675}. Best is trial 0 with value: 0.022866977561011446.
[Trial 0] New best: 0.02287
[I 2025-06-15 19:54:51,005] Trial 1 finished with value: 0.010585505634582456 and parameters: {'n_estimators': 194, 'learning_rate': 0.05137933459106951, 'num_leaves': 177, 'max_depth': 5, 'min_child_samples': 14, 'subsample': 0.6184896071132033, 'colsample_bytree': 0.7484871353445824, 'reg_alpha': 8.268596988695819, 'reg_lambda': 7.380913723858784, 'min_split_gain': 0.007087822435380165}. Best is trial 1 with value: 0.010585505634582456.
[I 2025-06-15 19:54:51,208] Trial 2 finished with value: 0.011252487722870644 and parameters: {'n_estimators': 136, 'learning_rate': 0.19075156289723055, 'num_leaves': 250, 'max_depth': 10, 'min_child_samples': 26, 'subsample': 0.569005052994614, 'colsample_bytree': 0.975368733394096, 'reg_alpha': 9.457097646505156, 'reg_lambda': 4.751895118095502, 'min_split_gain': 0.03342869556231045}. Best is trial 1 with value: 0.010585505634582456.
[Trial 1] New best: 0.01059
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:54:53,458] Trial 3 finished with value: 0.010956372134045197 and parameters: {'n_estimators': 268, 'learning_rate': 0.012477527993733372, 'num_leaves': 209, 'max_depth': 16, 'min_child_samples': 26, 'subsample': 0.923675488743107, 'colsample_bytree': 0.6541565644023806, 'reg_alpha': 0.33963394510599043, 'reg_lambda': 8.03468145505081, 'min_split_gain': 0.030450806213522176}. Best is trial 1 with value: 0.010585505634582456.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:54,083] Trial 4 finished with value: 0.01790790325586874 and parameters: {'n_estimators': 59, 'learning_rate': 0.03342830042486927, 'num_leaves': 136, 'max_depth': 15, 'min_child_samples': 12, 'subsample': 0.5441629879946782, 'colsample_bytree': 0.8174812519681758, 'reg_alpha': 0.6618267866039473, 'reg_lambda': 5.142304700878936, 'min_split_gain': 0.01301259190708251}. Best is trial 1 with value: 0.010585505634582456.
[I 2025-06-15 19:54:54,086] A new study created in memory with name: no-name-14c6c11f-61b1-4657-aa96-0dfcf1f691a1
[I 2025-06-15 19:54:54,126] Trial 0 finished with value: 0.18912691017335312 and parameters: {'n_estimators': 138, 'learning_rate': 0.042736414274155504, 'num_leaves': 112, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.8257824618174437, 'colsample_bytree': 0.6333271155169045, 'reg_alpha': 5.965872207179434, 'reg_lambda': 0.40958399224469866, 'min_split_gain': 0.0021810932250835914}. Best is trial 0 with value: 0.18912691017335312.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         920
1           Price_lag_2         583
14      pct_chg_Load_DA         465
3         Load_DA_lag_0         351
11           NGas_lag_2         256
13            EUA_lag_2         180
15       lag168_Load_DA         179
5       WindOn_DA_lag_0         171
2           Price_lag_7         164
16  roll2d_median_Price         150
10           Coal_lag_2         129
12            Oil_lag_2         112
9                  WD_7          17
7                  WD_1          15
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          69
16  roll2d_median_Price          22
1           Price_lag_2          15
12            Oil_lag_2           3
15       lag168_Load_DA           2
2           Price_lag_7           0
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 24.21% done
********************* END NS.. 176 ****************************************************
********************* START NS ... 177 ****************************************************
[I 2025-06-15 19:54:55,205] A new study created in memory with name: no-name-916fcf1c-5cb1-4b13-ae9a-a837f11753dd
[I 2025-06-15 19:54:55,869] Trial 0 finished with value: 0.006037917317745201 and parameters: {'n_estimators': 201, 'learning_rate': 0.03196556242554147, 'num_leaves': 124, 'max_depth': 7, 'min_child_samples': 12, 'subsample': 0.7510141008123815, 'colsample_bytree': 0.5945282752299815, 'reg_alpha': 0.8379210955359739, 'reg_lambda': 2.9661892767492604, 'min_split_gain': 0.032543052895528654}. Best is trial 0 with value: 0.006037917317745201.
[I 2025-06-15 19:54:56,069] Trial 1 finished with value: 0.007010214702665932 and parameters: {'n_estimators': 189, 'learning_rate': 0.18144790204309622, 'num_leaves': 184, 'max_depth': 14, 'min_child_samples': 17, 'subsample': 0.8605504276293101, 'colsample_bytree': 0.6916146907057603, 'reg_alpha': 9.774830191909151, 'reg_lambda': 2.4948814517787, 'min_split_gain': 0.03110087338346508}. Best is trial 0 with value: 0.006037917317745201.
[Trial 0] New best: 0.00604
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:54:56,528] Trial 2 finished with value: 0.008829812810130672 and parameters: {'n_estimators': 148, 'learning_rate': 0.023605456559673662, 'num_leaves': 129, 'max_depth': 7, 'min_child_samples': 23, 'subsample': 0.9789250379502783, 'colsample_bytree': 0.6860018159198074, 'reg_alpha': 8.050814977074909, 'reg_lambda': 5.186006494454743, 'min_split_gain': 0.008277215520471354}. Best is trial 0 with value: 0.006037917317745201.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:54:57,042] Trial 3 finished with value: 0.00846588341636792 and parameters: {'n_estimators': 251, 'learning_rate': 0.11070761819678827, 'num_leaves': 134, 'max_depth': 8, 'min_child_samples': 23, 'subsample': 0.8539290125918335, 'colsample_bytree': 0.5730472066371513, 'reg_alpha': 7.192312000696839, 'reg_lambda': 7.842471109399405, 'min_split_gain': 0.02737607121176451}. Best is trial 0 with value: 0.006037917317745201.
[I 2025-06-15 19:54:57,046] A new study created in memory with name: no-name-dc24432f-9f81-4bef-9646-0ab1f1ce1f70
[I 2025-06-15 19:54:57,071] Trial 0 finished with value: 0.0840129815267308 and parameters: {'n_estimators': 109, 'learning_rate': 0.21330211947415542, 'num_leaves': 223, 'max_depth': 15, 'min_child_samples': 25, 'subsample': 0.8694819134066445, 'colsample_bytree': 0.6763999756383798, 'reg_alpha': 6.4876911268346715, 'reg_lambda': 6.612841981745702, 'min_split_gain': 0.03445583652620425}. Best is trial 0 with value: 0.0840129815267308.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2000
1           Price_lag_2        1897
14      pct_chg_Load_DA        1246
3         Load_DA_lag_0         986
2           Price_lag_7         790
16  roll2d_median_Price         764
15       lag168_Load_DA         741
11           NGas_lag_2         717
10           Coal_lag_2         609
13            EUA_lag_2         546
5       WindOn_DA_lag_0         446
12            Oil_lag_2         387
7                  WD_1          33
9                  WD_7          29
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           7
1           Price_lag_2           1
16  roll2d_median_Price           1
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
2           Price_lag_7           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 24.35% done
********************* END NS.. 177 ****************************************************
********************* START NS ... 178 ****************************************************
[I 2025-06-15 19:54:58,778] A new study created in memory with name: no-name-7062b003-af2a-424a-a936-cd8b0b5ebca4
[I 2025-06-15 19:54:59,178] Trial 0 finished with value: 0.002355377209284974 and parameters: {'n_estimators': 116, 'learning_rate': 0.12249011287167769, 'num_leaves': 182, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.8728094405230489, 'colsample_bytree': 0.7871773792171657, 'reg_alpha': 1.598275613544996, 'reg_lambda': 2.6618154387617943, 'min_split_gain': 0.020199125502108384}. Best is trial 0 with value: 0.002355377209284974.
[Trial 0] New best: 0.00236
[I 2025-06-15 19:54:59,548] Trial 1 finished with value: 0.0023177756608816647 and parameters: {'n_estimators': 159, 'learning_rate': 0.11937491788486064, 'num_leaves': 82, 'max_depth': 10, 'min_child_samples': 5, 'subsample': 0.7756294294560486, 'colsample_bytree': 0.914410775610913, 'reg_alpha': 1.3572320631231827, 'reg_lambda': 1.6743386972908092, 'min_split_gain': 0.0348603489925405}. Best is trial 1 with value: 0.0023177756608816647.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:00,098] Trial 2 finished with value: 0.0031863567633882424 and parameters: {'n_estimators': 283, 'learning_rate': 0.08933181433980947, 'num_leaves': 145, 'max_depth': 13, 'min_child_samples': 11, 'subsample': 0.9355159364184108, 'colsample_bytree': 0.5030089520851957, 'reg_alpha': 2.793226129634261, 'reg_lambda': 4.930067369760854, 'min_split_gain': 0.0317713367642071}. Best is trial 1 with value: 0.0023177756608816647.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:00,432] Trial 3 finished with value: 0.002352716113411213 and parameters: {'n_estimators': 299, 'learning_rate': 0.17843532912134094, 'num_leaves': 180, 'max_depth': 8, 'min_child_samples': 7, 'subsample': 0.7924844967616458, 'colsample_bytree': 0.8413667233100364, 'reg_alpha': 7.979178455175624, 'reg_lambda': 0.8632374343634419, 'min_split_gain': 0.015872374368021535}. Best is trial 1 with value: 0.0023177756608816647.
[I 2025-06-15 19:55:00,434] A new study created in memory with name: no-name-71208dd3-bd70-4af9-be00-af5d14552869
[I 2025-06-15 19:55:00,456] Trial 0 finished with value: 0.011188618526408666 and parameters: {'n_estimators': 101, 'learning_rate': 0.09378328024540078, 'num_leaves': 41, 'max_depth': 16, 'min_child_samples': 7, 'subsample': 0.55556354352459, 'colsample_bytree': 0.6290962581699054, 'reg_alpha': 6.035448995787894, 'reg_lambda': 1.6053327864133027, 'min_split_gain': 0.008769929982479802}. Best is trial 0 with value: 0.011188618526408666.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         651
14      pct_chg_Load_DA         584
1           Price_lag_2         513
3         Load_DA_lag_0         366
2           Price_lag_7         290
11           NGas_lag_2         258
16  roll2d_median_Price         246
15       lag168_Load_DA         223
5       WindOn_DA_lag_0         172
10           Coal_lag_2         158
13            EUA_lag_2         134
12            Oil_lag_2         133
7                  WD_1          11
9                  WD_7          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          26
1           Price_lag_2          11
2           Price_lag_7           5
15       lag168_Load_DA           1
11           NGas_lag_2           1
10           Coal_lag_2           1
16  roll2d_median_Price           1
7                  WD_1           0
6      WindOff_DA_lag_0           0
9                  WD_7           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
3         Load_DA_lag_0           0
8                  WD_6           0
-> 24.49% done
********************* END NS.. 178 ****************************************************
********************* START NS ... 179 ****************************************************
[I 2025-06-15 19:55:01,417] A new study created in memory with name: no-name-0711281c-8521-4e74-b8b3-d2d53ecf76e0
[I 2025-06-15 19:55:01,711] Trial 0 finished with value: 0.0023592742485045726 and parameters: {'n_estimators': 89, 'learning_rate': 0.02323979671083121, 'num_leaves': 35, 'max_depth': 11, 'min_child_samples': 30, 'subsample': 0.9411976438870657, 'colsample_bytree': 0.7310095546112245, 'reg_alpha': 1.9487711270974362, 'reg_lambda': 5.181669064599886, 'min_split_gain': 0.03830066559852636}. Best is trial 0 with value: 0.0023592742485045726.
[Trial 0] New best: 0.00236
[I 2025-06-15 19:55:02,243] Trial 1 finished with value: 0.0021303473350214454 and parameters: {'n_estimators': 193, 'learning_rate': 0.013093433074866564, 'num_leaves': 82, 'max_depth': 5, 'min_child_samples': 8, 'subsample': 0.8073375897870736, 'colsample_bytree': 0.9769307794376971, 'reg_alpha': 7.828350887413491, 'reg_lambda': 1.8969584064940437, 'min_split_gain': 0.0099927740035762}. Best is trial 1 with value: 0.0021303473350214454.
[I 2025-06-15 19:55:02,421] Trial 2 finished with value: 0.00250360614325981 and parameters: {'n_estimators': 87, 'learning_rate': 0.06589619373500588, 'num_leaves': 209, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.657093208324825, 'colsample_bytree': 0.6324838163853359, 'reg_alpha': 5.811519113506781, 'reg_lambda': 7.466578878630049, 'min_split_gain': 0.019980678430347083}. Best is trial 1 with value: 0.0021303473350214454.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:02,619] Trial 3 finished with value: 0.0024425335196662655 and parameters: {'n_estimators': 58, 'learning_rate': 0.14437809281246736, 'num_leaves': 51, 'max_depth': 15, 'min_child_samples': 5, 'subsample': 0.5476086013738362, 'colsample_bytree': 0.7226141793636445, 'reg_alpha': 6.8990314673929305, 'reg_lambda': 4.873003269099665, 'min_split_gain': 0.032210756267646426}. Best is trial 1 with value: 0.0021303473350214454.
[I 2025-06-15 19:55:02,621] A new study created in memory with name: no-name-515f0516-373c-4ae4-bb0c-39e59014c633
[I 2025-06-15 19:55:02,649] Trial 0 finished with value: 0.01828086901472785 and parameters: {'n_estimators': 197, 'learning_rate': 0.07642889984178647, 'num_leaves': 126, 'max_depth': 10, 'min_child_samples': 23, 'subsample': 0.7190071131455041, 'colsample_bytree': 0.9476734755342491, 'reg_alpha': 4.843417223614371, 'reg_lambda': 4.9873216782541165, 'min_split_gain': 0.03135229549121415}. Best is trial 0 with value: 0.01828086901472785.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2381
14      pct_chg_Load_DA         545
1           Price_lag_2         539
16  roll2d_median_Price         322
2           Price_lag_7         266
11           NGas_lag_2         233
10           Coal_lag_2          95
3         Load_DA_lag_0          88
12            Oil_lag_2          51
13            EUA_lag_2          33
5       WindOn_DA_lag_0           6
15       lag168_Load_DA           6
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          36
2           Price_lag_7           1
1           Price_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
16  roll2d_median_Price           0
-> 24.62% done
********************* END NS.. 179 ****************************************************
********************* START NS ... 180 ****************************************************
[I 2025-06-15 19:55:03,669] A new study created in memory with name: no-name-b57c2d0d-5109-481d-ab7c-2a541b80d04f
[I 2025-06-15 19:55:04,508] Trial 0 finished with value: 0.002543762802038754 and parameters: {'n_estimators': 234, 'learning_rate': 0.0184756004706244, 'num_leaves': 234, 'max_depth': 13, 'min_child_samples': 17, 'subsample': 0.8717784671732738, 'colsample_bytree': 0.7546875593139593, 'reg_alpha': 6.80513269099082, 'reg_lambda': 9.805840822723566, 'min_split_gain': 0.03595551885651565}. Best is trial 0 with value: 0.002543762802038754.
[Trial 0] New best: 0.00254
[I 2025-06-15 19:55:04,821] Trial 1 finished with value: 0.002377882308923464 and parameters: {'n_estimators': 282, 'learning_rate': 0.13527842749106395, 'num_leaves': 104, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.7854826783487232, 'colsample_bytree': 0.660614350373018, 'reg_alpha': 7.508483819884965, 'reg_lambda': 6.310141808120409, 'min_split_gain': 0.030996133806895365}. Best is trial 1 with value: 0.002377882308923464.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:05,194] Trial 2 finished with value: 0.002234408544273684 and parameters: {'n_estimators': 169, 'learning_rate': 0.1322672080868666, 'num_leaves': 80, 'max_depth': 10, 'min_child_samples': 20, 'subsample': 0.9307827639197694, 'colsample_bytree': 0.6916560149849936, 'reg_alpha': 4.592764551089967, 'reg_lambda': 9.232422779599032, 'min_split_gain': 0.014217413941111168}. Best is trial 2 with value: 0.002234408544273684.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:06,806] Trial 3 finished with value: 0.0026654497909724957 and parameters: {'n_estimators': 249, 'learning_rate': 0.018229212318438087, 'num_leaves': 110, 'max_depth': 14, 'min_child_samples': 5, 'subsample': 0.862993105345754, 'colsample_bytree': 0.5070551444038977, 'reg_alpha': 2.514783568635087, 'reg_lambda': 6.453113905267554, 'min_split_gain': 0.029310552191922114}. Best is trial 2 with value: 0.002234408544273684.
[I 2025-06-15 19:55:06,809] A new study created in memory with name: no-name-ff52d551-f5dd-4bf3-bd08-48a9f6b65abf
[I 2025-06-15 19:55:06,870] Trial 0 finished with value: 0.014477323753110567 and parameters: {'n_estimators': 182, 'learning_rate': 0.1355224170682644, 'num_leaves': 38, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.6134980176081354, 'colsample_bytree': 0.5842589572825089, 'reg_alpha': 0.9962154221370212, 'reg_lambda': 7.059761047264842, 'min_split_gain': 0.04024256434448125}. Best is trial 0 with value: 0.014477323753110567.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         446
1           Price_lag_2         352
14      pct_chg_Load_DA         257
3         Load_DA_lag_0         249
11           NGas_lag_2         181
2           Price_lag_7         146
15       lag168_Load_DA         139
13            EUA_lag_2         131
16  roll2d_median_Price         130
5       WindOn_DA_lag_0         110
10           Coal_lag_2         103
12            Oil_lag_2         103
9                  WD_7           7
7                  WD_1           7
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          20
3         Load_DA_lag_0           7
2           Price_lag_7           6
15       lag168_Load_DA           3
1           Price_lag_2           2
5       WindOn_DA_lag_0           1
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 24.76% done
********************* END NS.. 180 ****************************************************
********************* START NS ... 181 ****************************************************
[I 2025-06-15 19:55:07,794] A new study created in memory with name: no-name-6743695d-cdc6-463b-b820-cbd83e75878c
[I 2025-06-15 19:55:07,922] Trial 0 finished with value: 0.0013909710801376215 and parameters: {'n_estimators': 90, 'learning_rate': 0.07902747346271154, 'num_leaves': 148, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.9511284906251563, 'colsample_bytree': 0.7886881361246092, 'reg_alpha': 1.6062997502196696, 'reg_lambda': 0.7391091588509724, 'min_split_gain': 0.004610935185677567}. Best is trial 0 with value: 0.0013909710801376215.
[Trial 0] New best: 0.00139
[I 2025-06-15 19:55:08,406] Trial 1 finished with value: 0.0013140218964301043 and parameters: {'n_estimators': 254, 'learning_rate': 0.04052423611311951, 'num_leaves': 116, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.5602727783015335, 'colsample_bytree': 0.7742530676466062, 'reg_alpha': 2.21687536092783, 'reg_lambda': 6.241408134496212, 'min_split_gain': 0.03713541511401419}. Best is trial 1 with value: 0.0013140218964301043.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:08,651] Trial 2 finished with value: 0.0013620368665417614 and parameters: {'n_estimators': 92, 'learning_rate': 0.2926120261029277, 'num_leaves': 124, 'max_depth': 14, 'min_child_samples': 3, 'subsample': 0.5555416994424319, 'colsample_bytree': 0.9387879393599345, 'reg_alpha': 8.627851513639976, 'reg_lambda': 5.200445572745956, 'min_split_gain': 0.004613884651739758}. Best is trial 1 with value: 0.0013140218964301043.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:08,906] Trial 3 finished with value: 0.0013855181812844981 and parameters: {'n_estimators': 59, 'learning_rate': 0.2917265292662577, 'num_leaves': 167, 'max_depth': 10, 'min_child_samples': 13, 'subsample': 0.8445033311005373, 'colsample_bytree': 0.8965308098117266, 'reg_alpha': 9.499675966589479, 'reg_lambda': 7.33785038594097, 'min_split_gain': 0.011550670466768243}. Best is trial 1 with value: 0.0013140218964301043.
[I 2025-06-15 19:55:08,909] A new study created in memory with name: no-name-45b748cd-1b82-4eac-854a-b39f0870fd88
[I 2025-06-15 19:55:08,960] Trial 0 finished with value: 0.010519098147833824 and parameters: {'n_estimators': 134, 'learning_rate': 0.03765752807832588, 'num_leaves': 41, 'max_depth': 10, 'min_child_samples': 13, 'subsample': 0.840131722047604, 'colsample_bytree': 0.8805480071518599, 'reg_alpha': 3.4472704520606934, 'reg_lambda': 8.319077794331928, 'min_split_gain': 0.028739372376039414}. Best is trial 0 with value: 0.010519098147833824.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1306
1           Price_lag_2         769
14      pct_chg_Load_DA         660
3         Load_DA_lag_0         415
11           NGas_lag_2         356
15       lag168_Load_DA         235
2           Price_lag_7         229
16  roll2d_median_Price         208
13            EUA_lag_2         202
5       WindOn_DA_lag_0         144
12            Oil_lag_2         139
10           Coal_lag_2         130
7                  WD_1          12
9                  WD_7           4
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          91
10           Coal_lag_2          18
2           Price_lag_7           5
3         Load_DA_lag_0           3
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 24.90% done
********************* END NS.. 181 ****************************************************
********************* START NS ... 182 ****************************************************
[I 2025-06-15 19:55:09,880] A new study created in memory with name: no-name-c38ca6c4-d9b3-4c0b-9f23-f25d0c9fcc68
[I 2025-06-15 19:55:10,229] Trial 0 finished with value: 0.0074743751309022605 and parameters: {'n_estimators': 128, 'learning_rate': 0.031916131410440404, 'num_leaves': 228, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.9531441461749821, 'colsample_bytree': 0.6729271834687743, 'reg_alpha': 8.278612840447677, 'reg_lambda': 6.243019530458175, 'min_split_gain': 0.011757089872913268}. Best is trial 0 with value: 0.0074743751309022605.
[Trial 0] New best: 0.00747
[I 2025-06-15 19:55:10,486] Trial 1 finished with value: 0.004333977092806927 and parameters: {'n_estimators': 138, 'learning_rate': 0.2654605074106481, 'num_leaves': 148, 'max_depth': 8, 'min_child_samples': 3, 'subsample': 0.6023624765985786, 'colsample_bytree': 0.628154236891092, 'reg_alpha': 5.138110176516358, 'reg_lambda': 7.104915828258927, 'min_split_gain': 0.002878862754037598}. Best is trial 1 with value: 0.004333977092806927.
[I 2025-06-15 19:55:10,664] Trial 2 finished with value: 0.0060991065785896735 and parameters: {'n_estimators': 114, 'learning_rate': 0.21060922121444653, 'num_leaves': 256, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.8305490419231134, 'colsample_bytree': 0.9384528730797359, 'reg_alpha': 6.028638914321983, 'reg_lambda': 8.327416051916574, 'min_split_gain': 0.04152140429124593}. Best is trial 1 with value: 0.004333977092806927.
[Trial 1] New best: 0.00433
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:11,046] Trial 3 finished with value: 0.006194494862136691 and parameters: {'n_estimators': 186, 'learning_rate': 0.11108099077883038, 'num_leaves': 218, 'max_depth': 16, 'min_child_samples': 17, 'subsample': 0.9119362835318927, 'colsample_bytree': 0.7278901389433634, 'reg_alpha': 2.6979424870866153, 'reg_lambda': 9.035661849456272, 'min_split_gain': 0.04395494004941922}. Best is trial 1 with value: 0.004333977092806927.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:11,400] Trial 4 finished with value: 0.005793963254282347 and parameters: {'n_estimators': 148, 'learning_rate': 0.11129533614598065, 'num_leaves': 185, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.9163258077775246, 'colsample_bytree': 0.6856922783892351, 'reg_alpha': 5.519305350020883, 'reg_lambda': 1.5104896674468482, 'min_split_gain': 0.008248796835437545}. Best is trial 1 with value: 0.004333977092806927.
[I 2025-06-15 19:55:11,403] A new study created in memory with name: no-name-dbec67f6-0635-4460-8ea3-ff0d95487019
[I 2025-06-15 19:55:11,429] Trial 0 finished with value: 0.04592596347562505 and parameters: {'n_estimators': 277, 'learning_rate': 0.01718323004987306, 'num_leaves': 65, 'max_depth': 8, 'min_child_samples': 18, 'subsample': 0.6784503617743571, 'colsample_bytree': 0.9992195039598534, 'reg_alpha': 8.768485816063084, 'reg_lambda': 1.6617601564607931, 'min_split_gain': 0.041974215344863175}. Best is trial 0 with value: 0.04592596347562505.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         361
1           Price_lag_2         341
2           Price_lag_7         227
3         Load_DA_lag_0         221
14      pct_chg_Load_DA         197
13            EUA_lag_2         158
5       WindOn_DA_lag_0         152
15       lag168_Load_DA         142
11           NGas_lag_2         138
10           Coal_lag_2         128
16  roll2d_median_Price         118
12            Oil_lag_2         106
9                  WD_7          13
8                  WD_6           8
7                  WD_1           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          10
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 25.03% done
********************* END NS.. 182 ****************************************************
********************* START NS ... 183 ****************************************************
[I 2025-06-15 19:55:12,272] A new study created in memory with name: no-name-7cdcb9cf-a230-43de-b9e8-9f675f85ec4b
[I 2025-06-15 19:55:13,367] Trial 0 finished with value: 0.022798288677920452 and parameters: {'n_estimators': 295, 'learning_rate': 0.01799193435275121, 'num_leaves': 116, 'max_depth': 13, 'min_child_samples': 5, 'subsample': 0.5589077404080682, 'colsample_bytree': 0.6325776874444122, 'reg_alpha': 5.748195994654629, 'reg_lambda': 3.7459854455582606, 'min_split_gain': 0.010324024327790516}. Best is trial 0 with value: 0.022798288677920452.
[Trial 0] New best: 0.02280
[I 2025-06-15 19:55:13,921] Trial 1 finished with value: 0.02234578617279166 and parameters: {'n_estimators': 168, 'learning_rate': 0.07919577280141912, 'num_leaves': 192, 'max_depth': 13, 'min_child_samples': 11, 'subsample': 0.72022498348004, 'colsample_bytree': 0.5521473244193674, 'reg_alpha': 5.868818150559464, 'reg_lambda': 7.155674352822491, 'min_split_gain': 0.04635959955084003}. Best is trial 1 with value: 0.02234578617279166.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:14,709] Trial 2 finished with value: 0.01751894384716732 and parameters: {'n_estimators': 175, 'learning_rate': 0.023937985954602933, 'num_leaves': 248, 'max_depth': 12, 'min_child_samples': 23, 'subsample': 0.9233820351424114, 'colsample_bytree': 0.9014030071643849, 'reg_alpha': 6.332251903633338, 'reg_lambda': 7.097192118285172, 'min_split_gain': 0.01677375612554374}. Best is trial 2 with value: 0.01751894384716732.
[Trial 2] New best: 0.01752
[I 2025-06-15 19:55:15,492] Trial 3 finished with value: 0.021849177589021412 and parameters: {'n_estimators': 188, 'learning_rate': 0.021292981918681474, 'num_leaves': 163, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.7518170013237724, 'colsample_bytree': 0.6500593889518791, 'reg_alpha': 3.6167822175426, 'reg_lambda': 5.851830530712844, 'min_split_gain': 0.012734490547195748}. Best is trial 2 with value: 0.01751894384716732.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:15,734] Trial 4 finished with value: 0.011504222739961822 and parameters: {'n_estimators': 280, 'learning_rate': 0.29683840735089234, 'num_leaves': 43, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.985971127389782, 'colsample_bytree': 0.6351608550977841, 'reg_alpha': 1.1585270385019408, 'reg_lambda': 8.898206445525396, 'min_split_gain': 0.02758151247934383}. Best is trial 4 with value: 0.011504222739961822.
[Trial 4] New best: 0.01150
[I 2025-06-15 19:55:17,317] Trial 5 finished with value: 0.014711479194163045 and parameters: {'n_estimators': 275, 'learning_rate': 0.015813183624473193, 'num_leaves': 133, 'max_depth': 15, 'min_child_samples': 23, 'subsample': 0.8480227417782309, 'colsample_bytree': 0.897468091069646, 'reg_alpha': 1.4945453254845797, 'reg_lambda': 1.523092051374153, 'min_split_gain': 0.04901818719705847}. Best is trial 4 with value: 0.011504222739961822.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:17,914] Trial 6 finished with value: 0.04515327292120832 and parameters: {'n_estimators': 123, 'learning_rate': 0.013840978747811095, 'num_leaves': 106, 'max_depth': 14, 'min_child_samples': 26, 'subsample': 0.9291053690707818, 'colsample_bytree': 0.8991009986442924, 'reg_alpha': 7.199721263018777, 'reg_lambda': 5.790725729011936, 'min_split_gain': 0.009704593711304794}. Best is trial 4 with value: 0.011504222739961822.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:18,359] Trial 7 finished with value: 0.016961488324082348 and parameters: {'n_estimators': 190, 'learning_rate': 0.224325721772198, 'num_leaves': 167, 'max_depth': 14, 'min_child_samples': 30, 'subsample': 0.5017949594134149, 'colsample_bytree': 0.5509110800819732, 'reg_alpha': 2.368941925203255, 'reg_lambda': 3.6876262078445246, 'min_split_gain': 0.0025888198373010653}. Best is trial 4 with value: 0.011504222739961822.
[I 2025-06-15 19:55:18,363] A new study created in memory with name: no-name-588a91b1-ac9f-4f3d-8fa0-7e092d362124
[I 2025-06-15 19:55:18,381] Trial 0 finished with value: 0.31377383501238404 and parameters: {'n_estimators': 135, 'learning_rate': 0.014771376802994583, 'num_leaves': 175, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.8337613442023504, 'colsample_bytree': 0.903546479601032, 'reg_alpha': 5.782857703718584, 'reg_lambda': 6.364929501304064, 'min_split_gain': 0.016056033208332966}. Best is trial 0 with value: 0.31377383501238404.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         276
1           Price_lag_2         123
3         Load_DA_lag_0          94
14      pct_chg_Load_DA          92
11           NGas_lag_2          78
16  roll2d_median_Price          70
2           Price_lag_7          69
15       lag168_Load_DA          66
5       WindOn_DA_lag_0          63
13            EUA_lag_2          57
10           Coal_lag_2          42
12            Oil_lag_2          25
9                  WD_7           7
7                  WD_1           4
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         176
1           Price_lag_2           8
3         Load_DA_lag_0           7
2           Price_lag_7           1
11           NGas_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 25.17% done
********************* END NS.. 183 ****************************************************
********************* START NS ... 184 ****************************************************
[I 2025-06-15 19:55:19,128] A new study created in memory with name: no-name-22e16246-adfc-48c0-a1e0-b95e28485cbb
[I 2025-06-15 19:55:19,653] Trial 0 finished with value: 0.0161163642357938 and parameters: {'n_estimators': 242, 'learning_rate': 0.03447859883669312, 'num_leaves': 103, 'max_depth': 9, 'min_child_samples': 21, 'subsample': 0.8456428527023789, 'colsample_bytree': 0.9829831064251491, 'reg_alpha': 5.629883161304262, 'reg_lambda': 2.4149594144973707, 'min_split_gain': 0.03895570621838506}. Best is trial 0 with value: 0.0161163642357938.
[Trial 0] New best: 0.01612
[I 2025-06-15 19:55:20,878] Trial 1 finished with value: 0.01675841762547856 and parameters: {'n_estimators': 206, 'learning_rate': 0.025404866665043473, 'num_leaves': 165, 'max_depth': 14, 'min_child_samples': 3, 'subsample': 0.7282972743855585, 'colsample_bytree': 0.9958352510214212, 'reg_alpha': 1.8919531969983339, 'reg_lambda': 2.2478347133051013, 'min_split_gain': 0.026594200504939488}. Best is trial 0 with value: 0.0161163642357938.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:21,433] Trial 2 finished with value: 0.018665649176796986 and parameters: {'n_estimators': 87, 'learning_rate': 0.10063788112315454, 'num_leaves': 181, 'max_depth': 10, 'min_child_samples': 11, 'subsample': 0.50731436540338, 'colsample_bytree': 0.6263909382252641, 'reg_alpha': 1.0098460454233993, 'reg_lambda': 8.009634920734165, 'min_split_gain': 0.03980799985500729}. Best is trial 0 with value: 0.0161163642357938.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:21,722] Trial 3 finished with value: 0.017383761263361955 and parameters: {'n_estimators': 55, 'learning_rate': 0.1782234104188075, 'num_leaves': 186, 'max_depth': 6, 'min_child_samples': 1, 'subsample': 0.5278824824082268, 'colsample_bytree': 0.926706336552839, 'reg_alpha': 6.832818290098971, 'reg_lambda': 0.7835219558722817, 'min_split_gain': 0.015127061483869442}. Best is trial 0 with value: 0.0161163642357938.
[I 2025-06-15 19:55:21,726] A new study created in memory with name: no-name-4093b380-8841-44ac-877c-f34c9e8d4b7d
[I 2025-06-15 19:55:21,779] Trial 0 finished with value: 0.3152389785344831 and parameters: {'n_estimators': 178, 'learning_rate': 0.012972335387236305, 'num_leaves': 93, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.9542307115311668, 'colsample_bytree': 0.6794913261124451, 'reg_alpha': 6.878403809752367, 'reg_lambda': 8.12801333650013, 'min_split_gain': 0.037027126046971386}. Best is trial 0 with value: 0.3152389785344831.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1537
14      pct_chg_Load_DA         904
1           Price_lag_2         646
11           NGas_lag_2         508
3         Load_DA_lag_0         463
16  roll2d_median_Price         362
2           Price_lag_7         357
15       lag168_Load_DA         229
13            EUA_lag_2         226
10           Coal_lag_2         209
12            Oil_lag_2         183
5       WindOn_DA_lag_0          60
9                  WD_7          17
7                  WD_1          15
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         134
1           Price_lag_2          33
12            Oil_lag_2          16
16  roll2d_median_Price           6
11           NGas_lag_2           1
2           Price_lag_7           1
15       lag168_Load_DA           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
3         Load_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 25.31% done
********************* END NS.. 184 ****************************************************
********************* START NS ... 185 ****************************************************
[I 2025-06-15 19:55:22,840] A new study created in memory with name: no-name-a9c4eb20-2ca8-45dd-a8e9-96949ba07212
[I 2025-06-15 19:55:23,012] Trial 0 finished with value: 0.0104022674661437 and parameters: {'n_estimators': 152, 'learning_rate': 0.17816622843654176, 'num_leaves': 57, 'max_depth': 13, 'min_child_samples': 21, 'subsample': 0.7814174677838153, 'colsample_bytree': 0.8899715789123388, 'reg_alpha': 4.966618085255561, 'reg_lambda': 1.7148752240600895, 'min_split_gain': 0.03305174924403732}. Best is trial 0 with value: 0.0104022674661437.
[I 2025-06-15 19:55:23,157] Trial 1 finished with value: 0.011523843045433338 and parameters: {'n_estimators': 96, 'learning_rate': 0.17211884296817231, 'num_leaves': 100, 'max_depth': 12, 'min_child_samples': 14, 'subsample': 0.751191319970554, 'colsample_bytree': 0.6845878461107222, 'reg_alpha': 8.730395982283182, 'reg_lambda': 1.944341867378383, 'min_split_gain': 0.031956950243979994}. Best is trial 0 with value: 0.0104022674661437.
[Trial 0] New best: 0.01040
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:23,686] Trial 2 finished with value: 0.016446630038694354 and parameters: {'n_estimators': 132, 'learning_rate': 0.013900435313251346, 'num_leaves': 101, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.8821352155102153, 'colsample_bytree': 0.7287956641245492, 'reg_alpha': 3.1507052379500964, 'reg_lambda': 2.8248019741683574, 'min_split_gain': 0.010634160648145298}. Best is trial 0 with value: 0.0104022674661437.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:24,226] Trial 3 finished with value: 0.011266744775286929 and parameters: {'n_estimators': 220, 'learning_rate': 0.0742427156320126, 'num_leaves': 166, 'max_depth': 13, 'min_child_samples': 20, 'subsample': 0.5736390079951275, 'colsample_bytree': 0.7836694321218891, 'reg_alpha': 6.1620472898341285, 'reg_lambda': 5.8521926468262, 'min_split_gain': 0.02327704583785318}. Best is trial 0 with value: 0.0104022674661437.
[I 2025-06-15 19:55:24,228] A new study created in memory with name: no-name-8bd04039-7c08-44de-9d6b-510bf5557ac2
[I 2025-06-15 19:55:24,266] Trial 0 finished with value: 0.017975445793211257 and parameters: {'n_estimators': 143, 'learning_rate': 0.1546032629427744, 'num_leaves': 59, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.596278328499133, 'colsample_bytree': 0.5274935565035095, 'reg_alpha': 5.346799215214539, 'reg_lambda': 7.881828965049198, 'min_split_gain': 0.00939229289555913}. Best is trial 0 with value: 0.017975445793211257.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         299
14      pct_chg_Load_DA         176
1           Price_lag_2         166
3         Load_DA_lag_0         110
11           NGas_lag_2          92
2           Price_lag_7          81
15       lag168_Load_DA          64
16  roll2d_median_Price          63
10           Coal_lag_2          56
13            EUA_lag_2          50
12            Oil_lag_2          43
5       WindOn_DA_lag_0          28
7                  WD_1           3
9                  WD_7           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          24
16  roll2d_median_Price           3
1           Price_lag_2           2
12            Oil_lag_2           2
10           Coal_lag_2           1
11           NGas_lag_2           1
15       lag168_Load_DA           1
2           Price_lag_7           0
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 25.44% done
********************* END NS.. 185 ****************************************************
********************* START NS ... 186 ****************************************************
[I 2025-06-15 19:55:25,056] A new study created in memory with name: no-name-0c756e36-45f0-4d6f-bbea-90d19cce3ce6
[I 2025-06-15 19:55:25,259] Trial 0 finished with value: 0.011345308841850003 and parameters: {'n_estimators': 70, 'learning_rate': 0.021971525267393172, 'num_leaves': 51, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.7299497409672064, 'colsample_bytree': 0.9974505446182754, 'reg_alpha': 6.15085932352172, 'reg_lambda': 3.510429035358543, 'min_split_gain': 0.03521801871434157}. Best is trial 0 with value: 0.011345308841850003.
[I 2025-06-15 19:55:25,386] Trial 1 finished with value: 0.0050423516427736135 and parameters: {'n_estimators': 55, 'learning_rate': 0.06225848158500924, 'num_leaves': 196, 'max_depth': 4, 'min_child_samples': 21, 'subsample': 0.6983106193078263, 'colsample_bytree': 0.828299438515901, 'reg_alpha': 3.8884362705480258, 'reg_lambda': 3.123011561484862, 'min_split_gain': 0.006376774814617948}. Best is trial 1 with value: 0.0050423516427736135.
[Trial 0] New best: 0.01135
[Trial 1] New best: 0.00504
[I 2025-06-15 19:55:25,499] Trial 2 finished with value: 0.006235424396471882 and parameters: {'n_estimators': 55, 'learning_rate': 0.18618168794935894, 'num_leaves': 206, 'max_depth': 4, 'min_child_samples': 8, 'subsample': 0.6893178693748345, 'colsample_bytree': 0.5329558063656699, 'reg_alpha': 5.077997015658472, 'reg_lambda': 8.162282177580703, 'min_split_gain': 0.022942163715887906}. Best is trial 1 with value: 0.0050423516427736135.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:26,909] Trial 3 finished with value: 0.004911242227615063 and parameters: {'n_estimators': 227, 'learning_rate': 0.020548498880612484, 'num_leaves': 218, 'max_depth': 11, 'min_child_samples': 5, 'subsample': 0.5656270436863224, 'colsample_bytree': 0.5063065903637233, 'reg_alpha': 5.382086215192711, 'reg_lambda': 0.7550081365412764, 'min_split_gain': 0.012168499118479198}. Best is trial 3 with value: 0.004911242227615063.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:27,274] Trial 4 finished with value: 0.005877128135026246 and parameters: {'n_estimators': 196, 'learning_rate': 0.2710622884823222, 'num_leaves': 168, 'max_depth': 12, 'min_child_samples': 23, 'subsample': 0.9582255407145928, 'colsample_bytree': 0.5158150860527633, 'reg_alpha': 9.555742562238738, 'reg_lambda': 7.365169789849309, 'min_split_gain': 0.017044174151781732}. Best is trial 3 with value: 0.004911242227615063.
[I 2025-06-15 19:55:27,276] A new study created in memory with name: no-name-0d3563d1-063c-4dfb-9b4a-e48f2f9885eb
[I 2025-06-15 19:55:27,323] Trial 0 finished with value: 0.009732497968844065 and parameters: {'n_estimators': 123, 'learning_rate': 0.02942288395669939, 'num_leaves': 227, 'max_depth': 4, 'min_child_samples': 1, 'subsample': 0.9299451755635417, 'colsample_bytree': 0.979344637060553, 'reg_alpha': 1.9749569515534127, 'reg_lambda': 4.710614929937681, 'min_split_gain': 0.02426391914041296}. Best is trial 0 with value: 0.009732497968844065.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1889
0           Price_lag_1        1884
14      pct_chg_Load_DA        1181
2           Price_lag_7         944
3         Load_DA_lag_0         904
16  roll2d_median_Price         881
11           NGas_lag_2         712
10           Coal_lag_2         703
15       lag168_Load_DA         626
13            EUA_lag_2         540
5       WindOn_DA_lag_0         535
12            Oil_lag_2         445
9                  WD_7          54
8                  WD_6          36
7                  WD_1          14
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         175
9                  WD_7          65
5       WindOn_DA_lag_0          23
10           Coal_lag_2          18
16  roll2d_median_Price          18
14      pct_chg_Load_DA          13
15       lag168_Load_DA           7
3         Load_DA_lag_0           3
11           NGas_lag_2           3
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
2           Price_lag_7           0
12            Oil_lag_2           0
13            EUA_lag_2           0
8                  WD_6           0
-> 25.58% done
********************* END NS.. 186 ****************************************************
********************* START NS ... 187 ****************************************************
[I 2025-06-15 19:55:29,140] A new study created in memory with name: no-name-c6cd6b07-54ec-4232-a28c-102be2efca68
[I 2025-06-15 19:55:29,601] Trial 0 finished with value: 0.004152324287392029 and parameters: {'n_estimators': 141, 'learning_rate': 0.014637848095281663, 'num_leaves': 31, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.6177110959949726, 'colsample_bytree': 0.7867163984563523, 'reg_alpha': 0.6741132627678548, 'reg_lambda': 5.87526370102313, 'min_split_gain': 0.01686009371923628}. Best is trial 0 with value: 0.004152324287392029.
[Trial 0] New best: 0.00415
[I 2025-06-15 19:55:30,524] Trial 1 finished with value: 0.0019056694476825367 and parameters: {'n_estimators': 239, 'learning_rate': 0.01684515074669691, 'num_leaves': 46, 'max_depth': 7, 'min_child_samples': 16, 'subsample': 0.8213943116600755, 'colsample_bytree': 0.829812907739126, 'reg_alpha': 6.821424373462515, 'reg_lambda': 7.270537976688791, 'min_split_gain': 0.013484354812228384}. Best is trial 1 with value: 0.0019056694476825367.
[Trial 1] New best: 0.00191
[I 2025-06-15 19:55:31,566] Trial 2 finished with value: 0.0017426521743376642 and parameters: {'n_estimators': 166, 'learning_rate': 0.023555848977731355, 'num_leaves': 243, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.5430585686778905, 'colsample_bytree': 0.7903095929300574, 'reg_alpha': 1.7633172119811558, 'reg_lambda': 0.6318089490676448, 'min_split_gain': 0.037602017312249505}. Best is trial 2 with value: 0.0017426521743376642.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:33,083] Trial 3 finished with value: 0.00212074398712478 and parameters: {'n_estimators': 265, 'learning_rate': 0.016100337976555384, 'num_leaves': 178, 'max_depth': 12, 'min_child_samples': 6, 'subsample': 0.7820580216429547, 'colsample_bytree': 0.5824706200440666, 'reg_alpha': 4.3930535381080205, 'reg_lambda': 8.240961191082405, 'min_split_gain': 0.002056746224500239}. Best is trial 2 with value: 0.0017426521743376642.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:33,317] Trial 4 finished with value: 0.002197165625925006 and parameters: {'n_estimators': 59, 'learning_rate': 0.14461185873003354, 'num_leaves': 119, 'max_depth': 11, 'min_child_samples': 15, 'subsample': 0.8506090085646958, 'colsample_bytree': 0.602046269317024, 'reg_alpha': 7.621862845774886, 'reg_lambda': 3.6084245912457047, 'min_split_gain': 0.039867262012940365}. Best is trial 2 with value: 0.0017426521743376642.
[I 2025-06-15 19:55:33,319] A new study created in memory with name: no-name-839334db-6f67-47b4-8a79-678782baedf5
[I 2025-06-15 19:55:33,483] Trial 0 finished with value: 0.006091196853784254 and parameters: {'n_estimators': 260, 'learning_rate': 0.11606899630567533, 'num_leaves': 141, 'max_depth': 5, 'min_child_samples': 16, 'subsample': 0.6464535061880237, 'colsample_bytree': 0.7917163766508108, 'reg_alpha': 5.083589146812187, 'reg_lambda': 6.1166560934195235, 'min_split_gain': 0.002311305110116052}. Best is trial 0 with value: 0.006091196853784254.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2450
1           Price_lag_2        2066
14      pct_chg_Load_DA        1249
11           NGas_lag_2         500
3         Load_DA_lag_0         475
16  roll2d_median_Price         468
2           Price_lag_7         465
15       lag168_Load_DA         243
5       WindOn_DA_lag_0         182
13            EUA_lag_2         163
12            Oil_lag_2         153
10           Coal_lag_2         136
9                  WD_7          11
7                  WD_1           4
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          43
1           Price_lag_2           5
9                  WD_7           3
5       WindOn_DA_lag_0           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 25.72% done
********************* END NS.. 187 ****************************************************
********************* START NS ... 188 ****************************************************
[I 2025-06-15 19:55:34,711] A new study created in memory with name: no-name-6f26833f-6f9d-48f8-8648-767139469d67
[I 2025-06-15 19:55:35,013] Trial 0 finished with value: 0.001110638024586688 and parameters: {'n_estimators': 252, 'learning_rate': 0.14055661211071485, 'num_leaves': 117, 'max_depth': 10, 'min_child_samples': 7, 'subsample': 0.5932378873370324, 'colsample_bytree': 0.7960480466138504, 'reg_alpha': 6.022223962899647, 'reg_lambda': 2.3015631184577403, 'min_split_gain': 0.008248583633526369}. Best is trial 0 with value: 0.001110638024586688.
[I 2025-06-15 19:55:35,166] Trial 1 finished with value: 0.0015073291440673286 and parameters: {'n_estimators': 102, 'learning_rate': 0.18349813656340297, 'num_leaves': 47, 'max_depth': 4, 'min_child_samples': 16, 'subsample': 0.8776051796063868, 'colsample_bytree': 0.5602993201550488, 'reg_alpha': 7.759473964998786, 'reg_lambda': 5.495289710892962, 'min_split_gain': 0.049860596644271254}. Best is trial 0 with value: 0.001110638024586688.
[Trial 0] New best: 0.00111
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:35,433] Trial 2 finished with value: 0.0011135244183261344 and parameters: {'n_estimators': 89, 'learning_rate': 0.14574692064463896, 'num_leaves': 229, 'max_depth': 13, 'min_child_samples': 22, 'subsample': 0.9734795637240021, 'colsample_bytree': 0.7147198593438708, 'reg_alpha': 3.0418591544096083, 'reg_lambda': 6.366298374343845, 'min_split_gain': 0.025673616751318736}. Best is trial 0 with value: 0.001110638024586688.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:35,719] Trial 3 finished with value: 0.0012720156693397568 and parameters: {'n_estimators': 195, 'learning_rate': 0.14818226681526125, 'num_leaves': 191, 'max_depth': 15, 'min_child_samples': 6, 'subsample': 0.9266544332528373, 'colsample_bytree': 0.6196450174089303, 'reg_alpha': 7.185303748756247, 'reg_lambda': 0.529026524066365, 'min_split_gain': 0.016841102472839393}. Best is trial 0 with value: 0.001110638024586688.
[I 2025-06-15 19:55:35,723] A new study created in memory with name: no-name-01bd45f2-f838-47f2-8c5c-1f652691b143
[I 2025-06-15 19:55:35,821] Trial 0 finished with value: 0.009014443303814429 and parameters: {'n_estimators': 286, 'learning_rate': 0.013758895592621617, 'num_leaves': 110, 'max_depth': 16, 'min_child_samples': 2, 'subsample': 0.5975664089053976, 'colsample_bytree': 0.5341084643543139, 'reg_alpha': 0.7994113583044327, 'reg_lambda': 3.734841449993417, 'min_split_gain': 0.0484414847615438}. Best is trial 0 with value: 0.009014443303814429.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         420
1           Price_lag_2         355
14      pct_chg_Load_DA         316
3         Load_DA_lag_0         220
13            EUA_lag_2         201
2           Price_lag_7         172
11           NGas_lag_2         170
15       lag168_Load_DA         167
5       WindOn_DA_lag_0         157
16  roll2d_median_Price         154
10           Coal_lag_2         133
12            Oil_lag_2         104
9                  WD_7          10
7                  WD_1           9
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         227
1           Price_lag_2          67
9                  WD_7          67
16  roll2d_median_Price          37
3         Load_DA_lag_0          26
10           Coal_lag_2          17
11           NGas_lag_2           9
13            EUA_lag_2           9
12            Oil_lag_2           6
15       lag168_Load_DA           6
14      pct_chg_Load_DA           5
5       WindOn_DA_lag_0           4
7                  WD_1           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
2           Price_lag_7           0
8                  WD_6           0
-> 25.85% done
********************* END NS.. 188 ****************************************************
********************* START NS ... 189 ****************************************************
[I 2025-06-15 19:55:36,720] A new study created in memory with name: no-name-05e2d330-0ab9-4031-a779-0e74c5559972
[I 2025-06-15 19:55:36,926] Trial 0 finished with value: 0.0026635198240858513 and parameters: {'n_estimators': 205, 'learning_rate': 0.28266711586450355, 'num_leaves': 93, 'max_depth': 13, 'min_child_samples': 17, 'subsample': 0.5733596273293562, 'colsample_bytree': 0.6393313978371091, 'reg_alpha': 1.8442553886188684, 'reg_lambda': 3.983219151160051, 'min_split_gain': 0.021338801214066394}. Best is trial 0 with value: 0.0026635198240858513.
[Trial 0] New best: 0.00266
[I 2025-06-15 19:55:37,223] Trial 1 finished with value: 0.0026178441581297704 and parameters: {'n_estimators': 239, 'learning_rate': 0.1548892587457709, 'num_leaves': 178, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.5687962589677236, 'colsample_bytree': 0.7282135106170978, 'reg_alpha': 3.0644585894136966, 'reg_lambda': 9.30990057412781, 'min_split_gain': 0.026677989582940626}. Best is trial 1 with value: 0.0026178441581297704.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:37,456] Trial 2 finished with value: 0.04697173711003411 and parameters: {'n_estimators': 54, 'learning_rate': 0.010519653493589198, 'num_leaves': 145, 'max_depth': 13, 'min_child_samples': 21, 'subsample': 0.8077710799811959, 'colsample_bytree': 0.9258260309151791, 'reg_alpha': 7.221835681638222, 'reg_lambda': 6.420546345877343, 'min_split_gain': 0.010476208259514197}. Best is trial 1 with value: 0.0026178441581297704.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:38,046] Trial 3 finished with value: 0.0030479461942705672 and parameters: {'n_estimators': 279, 'learning_rate': 0.08359097099028555, 'num_leaves': 68, 'max_depth': 15, 'min_child_samples': 30, 'subsample': 0.8702799919904081, 'colsample_bytree': 0.5245089721145502, 'reg_alpha': 4.567280925909572, 'reg_lambda': 1.8896410454266266, 'min_split_gain': 0.015711511587720772}. Best is trial 1 with value: 0.0026178441581297704.
[I 2025-06-15 19:55:38,048] A new study created in memory with name: no-name-af033e79-14dc-4721-b407-f72d98f1b0e6
[I 2025-06-15 19:55:38,071] Trial 0 finished with value: 0.013038305275158951 and parameters: {'n_estimators': 104, 'learning_rate': 0.055513864349852525, 'num_leaves': 115, 'max_depth': 6, 'min_child_samples': 8, 'subsample': 0.6648221159844772, 'colsample_bytree': 0.5272895947557172, 'reg_alpha': 3.635017815674587, 'reg_lambda': 5.263565269655933, 'min_split_gain': 0.042063882589723565}. Best is trial 0 with value: 0.013038305275158951.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         389
1           Price_lag_2         291
14      pct_chg_Load_DA         268
3         Load_DA_lag_0         219
11           NGas_lag_2         154
2           Price_lag_7         144
13            EUA_lag_2         135
16  roll2d_median_Price         131
15       lag168_Load_DA         122
10           Coal_lag_2          99
12            Oil_lag_2          80
5       WindOn_DA_lag_0          60
7                  WD_1           8
9                  WD_7           5
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          45
10           Coal_lag_2           9
1           Price_lag_2           8
11           NGas_lag_2           6
13            EUA_lag_2           4
12            Oil_lag_2           3
16  roll2d_median_Price           3
7                  WD_1           0
6      WindOff_DA_lag_0           0
9                  WD_7           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 25.99% done
********************* END NS.. 189 ****************************************************
********************* START NS ... 190 ****************************************************
[I 2025-06-15 19:55:38,846] A new study created in memory with name: no-name-be5c83c7-3db8-42ee-aa5a-dac492482d73
[I 2025-06-15 19:55:39,291] Trial 0 finished with value: 0.004592609524808784 and parameters: {'n_estimators': 130, 'learning_rate': 0.01975851467220015, 'num_leaves': 193, 'max_depth': 8, 'min_child_samples': 14, 'subsample': 0.6263009198682992, 'colsample_bytree': 0.6624578602777959, 'reg_alpha': 2.6739024392118393, 'reg_lambda': 5.948838497358632, 'min_split_gain': 0.045923289082411105}. Best is trial 0 with value: 0.004592609524808784.
[Trial 0] New best: 0.00459
[I 2025-06-15 19:55:39,536] Trial 1 finished with value: 0.003697697161742416 and parameters: {'n_estimators': 235, 'learning_rate': 0.21334074266235947, 'num_leaves': 170, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.7594706802197708, 'colsample_bytree': 0.6175121257640033, 'reg_alpha': 1.1574677439924286, 'reg_lambda': 9.282090574399437, 'min_split_gain': 0.028310057437286353}. Best is trial 1 with value: 0.003697697161742416.
[I 2025-06-15 19:55:39,744] Trial 2 finished with value: 0.003146578743694336 and parameters: {'n_estimators': 121, 'learning_rate': 0.15585075388745323, 'num_leaves': 184, 'max_depth': 6, 'min_child_samples': 24, 'subsample': 0.6935774347543842, 'colsample_bytree': 0.9983122466090575, 'reg_alpha': 4.15864162100772, 'reg_lambda': 6.124817330168117, 'min_split_gain': 0.015521395655472104}. Best is trial 2 with value: 0.003146578743694336.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] New best: 0.00315
[I 2025-06-15 19:55:40,168] Trial 3 finished with value: 0.003198163052608173 and parameters: {'n_estimators': 209, 'learning_rate': 0.21431669033539769, 'num_leaves': 150, 'max_depth': 8, 'min_child_samples': 9, 'subsample': 0.6997753845806487, 'colsample_bytree': 0.9635300012445888, 'reg_alpha': 2.6080029121965307, 'reg_lambda': 9.898902309120277, 'min_split_gain': 0.001060458785500923}. Best is trial 2 with value: 0.003146578743694336.
[I 2025-06-15 19:55:40,329] Trial 4 finished with value: 0.003612959301152338 and parameters: {'n_estimators': 77, 'learning_rate': 0.09157586375572599, 'num_leaves': 146, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.6793843985083605, 'colsample_bytree': 0.9736963798169093, 'reg_alpha': 8.483944000888226, 'reg_lambda': 1.5899244894214093, 'min_split_gain': 0.04873062956825609}. Best is trial 2 with value: 0.003146578743694336.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:41,017] Trial 5 finished with value: 0.0034905547436337747 and parameters: {'n_estimators': 241, 'learning_rate': 0.015623899247790519, 'num_leaves': 122, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.6236050935771535, 'colsample_bytree': 0.9223054609843833, 'reg_alpha': 8.531741049874908, 'reg_lambda': 5.3530089467316335, 'min_split_gain': 0.046640579280668504}. Best is trial 2 with value: 0.003146578743694336.
[I 2025-06-15 19:55:41,019] A new study created in memory with name: no-name-0aaf0c55-cf6a-4b7d-8dc8-9384218e9cec
[I 2025-06-15 19:55:41,040] Trial 0 finished with value: 0.005270071950083073 and parameters: {'n_estimators': 109, 'learning_rate': 0.07077297145085522, 'num_leaves': 63, 'max_depth': 16, 'min_child_samples': 27, 'subsample': 0.6562668199219932, 'colsample_bytree': 0.8778892450848154, 'reg_alpha': 2.14220369358884, 'reg_lambda': 7.082307360997062, 'min_split_gain': 0.027701110923966245}. Best is trial 0 with value: 0.005270071950083073.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         360
14      pct_chg_Load_DA         251
1           Price_lag_2         174
3         Load_DA_lag_0         174
11           NGas_lag_2         123
2           Price_lag_7          93
15       lag168_Load_DA          90
16  roll2d_median_Price          86
5       WindOn_DA_lag_0          84
13            EUA_lag_2          81
10           Coal_lag_2          61
12            Oil_lag_2          53
7                  WD_1           7
9                  WD_7           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          46
1           Price_lag_2           2
16  roll2d_median_Price           2
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
2           Price_lag_7           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 26.13% done
********************* END NS.. 190 ****************************************************
********************* START NS ... 191 ****************************************************
[I 2025-06-15 19:55:41,751] A new study created in memory with name: no-name-25fc4b00-18c0-4ad5-acd7-bae5881b699b
[I 2025-06-15 19:55:42,095] Trial 0 finished with value: 0.00097972812582139 and parameters: {'n_estimators': 217, 'learning_rate': 0.05837237228498478, 'num_leaves': 46, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.790680231605025, 'colsample_bytree': 0.6510868648857793, 'reg_alpha': 5.392015052591702, 'reg_lambda': 1.7036874922625644, 'min_split_gain': 0.03714535469027591}. Best is trial 0 with value: 0.00097972812582139.
[Trial 0] New best: 0.00098
[I 2025-06-15 19:55:42,961] Trial 1 finished with value: 0.0009366124939415039 and parameters: {'n_estimators': 168, 'learning_rate': 0.07712711286034579, 'num_leaves': 211, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.807159519722962, 'colsample_bytree': 0.7348817860624567, 'reg_alpha': 0.11052983655206239, 'reg_lambda': 8.379063338986395, 'min_split_gain': 0.018127517685067398}. Best is trial 1 with value: 0.0009366124939415039.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:43,381] Trial 2 finished with value: 0.0009429406900798787 and parameters: {'n_estimators': 218, 'learning_rate': 0.07386539591056478, 'num_leaves': 180, 'max_depth': 13, 'min_child_samples': 3, 'subsample': 0.8853456010016949, 'colsample_bytree': 0.8633140207574928, 'reg_alpha': 4.153542625206146, 'reg_lambda': 8.70200226211569, 'min_split_gain': 0.03091481114271723}. Best is trial 1 with value: 0.0009366124939415039.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:43,627] Trial 3 finished with value: 0.03153585910048357 and parameters: {'n_estimators': 58, 'learning_rate': 0.011695982173606819, 'num_leaves': 216, 'max_depth': 12, 'min_child_samples': 1, 'subsample': 0.5003444050149807, 'colsample_bytree': 0.9765872084840527, 'reg_alpha': 9.61869603535172, 'reg_lambda': 0.04601534426756815, 'min_split_gain': 0.045975537240545226}. Best is trial 1 with value: 0.0009366124939415039.
[I 2025-06-15 19:55:43,629] A new study created in memory with name: no-name-0e04810a-62ec-4875-9db6-c07a04e1ffe8
[I 2025-06-15 19:55:43,656] Trial 0 finished with value: 0.0038173520229136753 and parameters: {'n_estimators': 174, 'learning_rate': 0.17431386746362854, 'num_leaves': 165, 'max_depth': 8, 'min_child_samples': 23, 'subsample': 0.8047123312622201, 'colsample_bytree': 0.5039679942650601, 'reg_alpha': 7.164542802193802, 'reg_lambda': 1.7073529820702804, 'min_split_gain': 0.02060289369953455}. Best is trial 0 with value: 0.0038173520229136753.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1580
0           Price_lag_1        1434
14      pct_chg_Load_DA        1287
3         Load_DA_lag_0         992
2           Price_lag_7         786
15       lag168_Load_DA         738
11           NGas_lag_2         709
5       WindOn_DA_lag_0         701
16  roll2d_median_Price         685
13            EUA_lag_2         520
10           Coal_lag_2         475
12            Oil_lag_2         438
9                  WD_7          22
7                  WD_1          19
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 26.27% done
********************* END NS.. 191 ****************************************************
********************* START NS ... 192 ****************************************************
[I 2025-06-15 19:55:44,724] A new study created in memory with name: no-name-91044792-9611-4786-877f-96b40d747b7a
[I 2025-06-15 19:55:45,794] Trial 0 finished with value: 0.000729299690735207 and parameters: {'n_estimators': 274, 'learning_rate': 0.018051917002191376, 'num_leaves': 116, 'max_depth': 8, 'min_child_samples': 19, 'subsample': 0.5029430690680552, 'colsample_bytree': 0.8731267224325512, 'reg_alpha': 1.5259101742277148, 'reg_lambda': 8.730977867318897, 'min_split_gain': 0.012100058002551518}. Best is trial 0 with value: 0.000729299690735207.
[Trial 0] New best: 0.00073
[I 2025-06-15 19:55:46,671] Trial 1 finished with value: 0.003806518257656523 and parameters: {'n_estimators': 155, 'learning_rate': 0.011080020303701223, 'num_leaves': 162, 'max_depth': 16, 'min_child_samples': 3, 'subsample': 0.8128413685679867, 'colsample_bytree': 0.8645531361735079, 'reg_alpha': 5.491775139034173, 'reg_lambda': 0.8827320386290638, 'min_split_gain': 0.00019991431269806849}. Best is trial 0 with value: 0.000729299690735207.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:46,905] Trial 2 finished with value: 0.0008576440117074952 and parameters: {'n_estimators': 71, 'learning_rate': 0.14586952427190766, 'num_leaves': 183, 'max_depth': 12, 'min_child_samples': 30, 'subsample': 0.8858426018897396, 'colsample_bytree': 0.5660448872699662, 'reg_alpha': 4.274569961891583, 'reg_lambda': 7.768816461557545, 'min_split_gain': 0.04382034835730262}. Best is trial 0 with value: 0.000729299690735207.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:47,682] Trial 3 finished with value: 0.0006553061865232842 and parameters: {'n_estimators': 300, 'learning_rate': 0.012108177901226943, 'num_leaves': 228, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.6191044704187902, 'colsample_bytree': 0.8451861073506688, 'reg_alpha': 9.94763932562202, 'reg_lambda': 2.021421848039978, 'min_split_gain': 0.02017653730453796}. Best is trial 3 with value: 0.0006553061865232842.
[I 2025-06-15 19:55:47,686] A new study created in memory with name: no-name-80718c48-5c0b-491a-b696-130d51c0c793
[I 2025-06-15 19:55:47,728] Trial 0 finished with value: 0.0008718110983299124 and parameters: {'n_estimators': 270, 'learning_rate': 0.050418662555799194, 'num_leaves': 211, 'max_depth': 10, 'min_child_samples': 4, 'subsample': 0.5748831718293053, 'colsample_bytree': 0.8322700541526479, 'reg_alpha': 0.6449916290883362, 'reg_lambda': 1.4963247026897497, 'min_split_gain': 0.01884932299601554}. Best is trial 0 with value: 0.0008718110983299124.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3052
1           Price_lag_2        1405
14      pct_chg_Load_DA        1028
16  roll2d_median_Price         436
11           NGas_lag_2         421
2           Price_lag_7         352
3         Load_DA_lag_0         251
10           Coal_lag_2         202
13            EUA_lag_2         134
12            Oil_lag_2         118
5       WindOn_DA_lag_0          86
15       lag168_Load_DA          77
9                  WD_7           9
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          53
15       lag168_Load_DA           6
1           Price_lag_2           3
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 26.40% done
********************* END NS.. 192 ****************************************************
********************* START NS ... 193 ****************************************************
[I 2025-06-15 19:55:48,910] A new study created in memory with name: no-name-16e39e97-4986-493a-9cdf-ca1425c7ee4b
[I 2025-06-15 19:55:49,091] Trial 0 finished with value: 0.0031833306431553045 and parameters: {'n_estimators': 197, 'learning_rate': 0.1501714521888213, 'num_leaves': 248, 'max_depth': 16, 'min_child_samples': 5, 'subsample': 0.7642841502584852, 'colsample_bytree': 0.6064144587480107, 'reg_alpha': 5.103768403612375, 'reg_lambda': 2.9176429688612804, 'min_split_gain': 0.03920364500659304}. Best is trial 0 with value: 0.0031833306431553045.
[Trial 0] New best: 0.00318
[I 2025-06-15 19:55:49,384] Trial 1 finished with value: 0.0027376055840313105 and parameters: {'n_estimators': 83, 'learning_rate': 0.0712389382701137, 'num_leaves': 200, 'max_depth': 14, 'min_child_samples': 4, 'subsample': 0.514678003511029, 'colsample_bytree': 0.9768167300649397, 'reg_alpha': 5.810785111144802, 'reg_lambda': 6.370480039616081, 'min_split_gain': 0.019100495797069468}. Best is trial 1 with value: 0.0027376055840313105.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:49,605] Trial 2 finished with value: 0.0030272430205627032 and parameters: {'n_estimators': 55, 'learning_rate': 0.06778808293076073, 'num_leaves': 205, 'max_depth': 16, 'min_child_samples': 9, 'subsample': 0.968074168990397, 'colsample_bytree': 0.95338639438583, 'reg_alpha': 6.265979682026135, 'reg_lambda': 9.296936188279856, 'min_split_gain': 0.025181927238058694}. Best is trial 1 with value: 0.0027376055840313105.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:50,184] Trial 3 finished with value: 0.0032772158981422667 and parameters: {'n_estimators': 290, 'learning_rate': 0.05385480802420688, 'num_leaves': 82, 'max_depth': 16, 'min_child_samples': 13, 'subsample': 0.738153725829038, 'colsample_bytree': 0.6073946241841672, 'reg_alpha': 6.559654757907692, 'reg_lambda': 5.060972776700804, 'min_split_gain': 0.016565758744433102}. Best is trial 1 with value: 0.0027376055840313105.
[I 2025-06-15 19:55:50,186] A new study created in memory with name: no-name-c2340525-f9c4-43be-863e-070e3c212c27
[I 2025-06-15 19:55:50,209] Trial 0 finished with value: 0.009717678039038556 and parameters: {'n_estimators': 139, 'learning_rate': 0.1967041340233684, 'num_leaves': 168, 'max_depth': 4, 'min_child_samples': 15, 'subsample': 0.8066039791995727, 'colsample_bytree': 0.5244586882259568, 'reg_alpha': 0.3481300035574286, 'reg_lambda': 7.045376671000421, 'min_split_gain': 0.030810040642991518}. Best is trial 0 with value: 0.009717678039038556.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         774
14      pct_chg_Load_DA         485
1           Price_lag_2         451
16  roll2d_median_Price         270
2           Price_lag_7         267
11           NGas_lag_2         254
3         Load_DA_lag_0         245
10           Coal_lag_2         203
13            EUA_lag_2         137
15       lag168_Load_DA         131
5       WindOn_DA_lag_0         127
12            Oil_lag_2         123
9                  WD_7          16
8                  WD_6           6
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1          13
1           Price_lag_2           2
3         Load_DA_lag_0           1
11           NGas_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 26.54% done
********************* END NS.. 193 ****************************************************
********************* START NS ... 194 ****************************************************
[I 2025-06-15 19:55:51,004] A new study created in memory with name: no-name-32eac62c-84f4-4fda-9e39-8f9e76ce2366
[I 2025-06-15 19:55:51,669] Trial 0 finished with value: 0.00362107973796954 and parameters: {'n_estimators': 285, 'learning_rate': 0.012412801139354036, 'num_leaves': 166, 'max_depth': 7, 'min_child_samples': 11, 'subsample': 0.9147044879502519, 'colsample_bytree': 0.5717746696503089, 'reg_alpha': 8.55350352605122, 'reg_lambda': 0.28224341397921515, 'min_split_gain': 0.01713974276604587}. Best is trial 0 with value: 0.00362107973796954.
[Trial 0] New best: 0.00362
[I 2025-06-15 19:55:52,016] Trial 1 finished with value: 0.0028345689466533187 and parameters: {'n_estimators': 172, 'learning_rate': 0.1255985953798468, 'num_leaves': 166, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.5551612736093845, 'colsample_bytree': 0.5221757913087574, 'reg_alpha': 0.8975836661779513, 'reg_lambda': 9.576307502282008, 'min_split_gain': 0.04816395892962259}. Best is trial 1 with value: 0.0028345689466533187.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:52,229] Trial 2 finished with value: 0.002901886089183023 and parameters: {'n_estimators': 199, 'learning_rate': 0.2661390091551144, 'num_leaves': 170, 'max_depth': 15, 'min_child_samples': 17, 'subsample': 0.551160438905688, 'colsample_bytree': 0.5986526842831448, 'reg_alpha': 3.863417022958523, 'reg_lambda': 6.285373086268724, 'min_split_gain': 0.01557943910034087}. Best is trial 1 with value: 0.0028345689466533187.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:52,458] Trial 3 finished with value: 0.0024442412892694393 and parameters: {'n_estimators': 67, 'learning_rate': 0.10879749998696313, 'num_leaves': 203, 'max_depth': 16, 'min_child_samples': 5, 'subsample': 0.6058181091017099, 'colsample_bytree': 0.9860801575115683, 'reg_alpha': 8.161095454281668, 'reg_lambda': 4.216118290568649, 'min_split_gain': 0.017463413477869883}. Best is trial 3 with value: 0.0024442412892694393.
[I 2025-06-15 19:55:52,586] Trial 4 finished with value: 0.0026783593601372156 and parameters: {'n_estimators': 82, 'learning_rate': 0.26188727820781, 'num_leaves': 138, 'max_depth': 4, 'min_child_samples': 15, 'subsample': 0.5510441271289857, 'colsample_bytree': 0.623824677502434, 'reg_alpha': 0.5022635588615487, 'reg_lambda': 4.361746692548163, 'min_split_gain': 0.04573507956151374}. Best is trial 3 with value: 0.0024442412892694393.
[Trial 3] New best: 0.00244
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:53,054] Trial 5 finished with value: 0.01970372944890782 and parameters: {'n_estimators': 126, 'learning_rate': 0.010821553192608135, 'num_leaves': 181, 'max_depth': 10, 'min_child_samples': 23, 'subsample': 0.6114838586176117, 'colsample_bytree': 0.6656661653696729, 'reg_alpha': 5.544540199608264, 'reg_lambda': 6.333903906624839, 'min_split_gain': 0.02780141539968796}. Best is trial 3 with value: 0.0024442412892694393.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:53,709] Trial 6 finished with value: 0.007834567586462704 and parameters: {'n_estimators': 127, 'learning_rate': 0.015582475852323254, 'num_leaves': 240, 'max_depth': 14, 'min_child_samples': 24, 'subsample': 0.5622615156781315, 'colsample_bytree': 0.7560239000103961, 'reg_alpha': 1.426031455324469, 'reg_lambda': 1.6621889006055035, 'min_split_gain': 0.04057186611247622}. Best is trial 3 with value: 0.0024442412892694393.
[I 2025-06-15 19:55:53,711] A new study created in memory with name: no-name-006740ed-9aa4-4e43-9889-1231fc39c641
[I 2025-06-15 19:55:53,861] Trial 0 finished with value: 0.013374247217246947 and parameters: {'n_estimators': 137, 'learning_rate': 0.011075700423286618, 'num_leaves': 97, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.5153440808741944, 'colsample_bytree': 0.6191317654815105, 'reg_alpha': 0.11551402735947902, 'reg_lambda': 5.623417751545264, 'min_split_gain': 0.01701169457798023}. Best is trial 0 with value: 0.013374247217246947.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         483
14      pct_chg_Load_DA         291
1           Price_lag_2         269
11           NGas_lag_2         172
3         Load_DA_lag_0         155
2           Price_lag_7         154
16  roll2d_median_Price         146
10           Coal_lag_2         116
13            EUA_lag_2          97
12            Oil_lag_2          93
15       lag168_Load_DA          92
5       WindOn_DA_lag_0          71
9                  WD_7          13
7                  WD_1           5
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         186
1           Price_lag_2          47
15       lag168_Load_DA          29
10           Coal_lag_2          24
2           Price_lag_7          24
14      pct_chg_Load_DA          14
12            Oil_lag_2           6
3         Load_DA_lag_0           5
13            EUA_lag_2           3
16  roll2d_median_Price           2
5       WindOn_DA_lag_0           1
9                  WD_7           0
11           NGas_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 26.68% done
********************* END NS.. 194 ****************************************************
********************* START NS ... 195 ****************************************************
[I 2025-06-15 19:55:54,703] A new study created in memory with name: no-name-9b5c5649-3c02-45fc-aee4-47fdca57a27b
[I 2025-06-15 19:55:55,029] Trial 0 finished with value: 0.0027895161003366003 and parameters: {'n_estimators': 89, 'learning_rate': 0.02626690139030373, 'num_leaves': 170, 'max_depth': 15, 'min_child_samples': 23, 'subsample': 0.5514121975600872, 'colsample_bytree': 0.9173640102251028, 'reg_alpha': 5.995988414161278, 'reg_lambda': 5.243913442724869, 'min_split_gain': 0.011391614511444416}. Best is trial 0 with value: 0.0027895161003366003.
[Trial 0] New best: 0.00279
[I 2025-06-15 19:55:55,968] Trial 1 finished with value: 0.0011465595722854785 and parameters: {'n_estimators': 238, 'learning_rate': 0.01264471762167031, 'num_leaves': 179, 'max_depth': 16, 'min_child_samples': 13, 'subsample': 0.5992132940985846, 'colsample_bytree': 0.9391381915870445, 'reg_alpha': 5.368409452806668, 'reg_lambda': 8.622512664551843, 'min_split_gain': 0.015990490357853528}. Best is trial 1 with value: 0.0011465595722854785.
[Trial 1] New best: 0.00115
[I 2025-06-15 19:55:56,351] Trial 2 finished with value: 0.000510248263291639 and parameters: {'n_estimators': 286, 'learning_rate': 0.17112656934304243, 'num_leaves': 152, 'max_depth': 11, 'min_child_samples': 3, 'subsample': 0.6071780498986901, 'colsample_bytree': 0.5085575173338042, 'reg_alpha': 5.188218167510323, 'reg_lambda': 4.069213556676793, 'min_split_gain': 0.037823011221540144}. Best is trial 2 with value: 0.000510248263291639.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:55:56,631] Trial 3 finished with value: 0.005871310860231224 and parameters: {'n_estimators': 60, 'learning_rate': 0.03350993917332323, 'num_leaves': 174, 'max_depth': 9, 'min_child_samples': 15, 'subsample': 0.9495461746610911, 'colsample_bytree': 0.5850027380331606, 'reg_alpha': 9.717464227403166, 'reg_lambda': 6.352188344706919, 'min_split_gain': 0.020294351864424628}. Best is trial 2 with value: 0.000510248263291639.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:55:57,122] Trial 4 finished with value: 0.0005140997402164159 and parameters: {'n_estimators': 153, 'learning_rate': 0.09156013774709683, 'num_leaves': 237, 'max_depth': 15, 'min_child_samples': 30, 'subsample': 0.5816197654904935, 'colsample_bytree': 0.5539776923929035, 'reg_alpha': 6.261277650961618, 'reg_lambda': 8.355789257727915, 'min_split_gain': 0.01736165691787104}. Best is trial 2 with value: 0.000510248263291639.
[I 2025-06-15 19:55:57,124] A new study created in memory with name: no-name-5aface57-c82b-4e38-b15d-4f621e86fc39
[I 2025-06-15 19:55:57,177] Trial 0 finished with value: 0.010254913101150108 and parameters: {'n_estimators': 224, 'learning_rate': 0.02194810341505269, 'num_leaves': 42, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.8237044065555569, 'colsample_bytree': 0.9758224275877023, 'reg_alpha': 2.9286148571321213, 'reg_lambda': 6.14716758001057, 'min_split_gain': 0.044996087925619425}. Best is trial 0 with value: 0.010254913101150108.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         327
1           Price_lag_2         274
16  roll2d_median_Price         184
14      pct_chg_Load_DA         155
3         Load_DA_lag_0         141
2           Price_lag_7         136
11           NGas_lag_2         129
15       lag168_Load_DA         103
13            EUA_lag_2          90
5       WindOn_DA_lag_0          85
10           Coal_lag_2          77
12            Oil_lag_2          67
8                  WD_6          13
7                  WD_1           9
9                  WD_7           9
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          34
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 26.81% done
********************* END NS.. 195 ****************************************************
********************* START NS ... 196 ****************************************************
[I 2025-06-15 19:55:58,084] A new study created in memory with name: no-name-c0848023-b9e8-4174-8aaa-c446c34079c2
[I 2025-06-15 19:55:58,636] Trial 0 finished with value: 0.0057595857379826865 and parameters: {'n_estimators': 166, 'learning_rate': 0.019390288472744856, 'num_leaves': 154, 'max_depth': 12, 'min_child_samples': 5, 'subsample': 0.7501520891834239, 'colsample_bytree': 0.9067775398815439, 'reg_alpha': 9.230016032808122, 'reg_lambda': 8.965196926548751, 'min_split_gain': 0.03406997363014218}. Best is trial 0 with value: 0.0057595857379826865.
[Trial 0] New best: 0.00576
[I 2025-06-15 19:55:59,159] Trial 1 finished with value: 0.004636210093831544 and parameters: {'n_estimators': 152, 'learning_rate': 0.05652084123154568, 'num_leaves': 143, 'max_depth': 14, 'min_child_samples': 18, 'subsample': 0.8576513180183809, 'colsample_bytree': 0.6257801145504965, 'reg_alpha': 5.128025395561032, 'reg_lambda': 5.257031428926183, 'min_split_gain': 0.04072257454739589}. Best is trial 1 with value: 0.004636210093831544.
[Trial 1] New best: 0.00464
[I 2025-06-15 19:55:59,438] Trial 2 finished with value: 0.006455235321647985 and parameters: {'n_estimators': 83, 'learning_rate': 0.225979743899373, 'num_leaves': 175, 'max_depth': 7, 'min_child_samples': 1, 'subsample': 0.9124253877375622, 'colsample_bytree': 0.5893528987390244, 'reg_alpha': 3.9482163285266756, 'reg_lambda': 9.71746954315403, 'min_split_gain': 0.005647184444419967}. Best is trial 1 with value: 0.004636210093831544.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:00,437] Trial 3 finished with value: 0.007804078193487327 and parameters: {'n_estimators': 214, 'learning_rate': 0.012406669265908521, 'num_leaves': 123, 'max_depth': 8, 'min_child_samples': 30, 'subsample': 0.7348371115523944, 'colsample_bytree': 0.8106107245655957, 'reg_alpha': 2.5780966079057577, 'reg_lambda': 8.149341295173974, 'min_split_gain': 0.037483518345551185}. Best is trial 1 with value: 0.004636210093831544.
[I 2025-06-15 19:56:00,608] Trial 4 finished with value: 0.0031779081507095244 and parameters: {'n_estimators': 169, 'learning_rate': 0.15150282205454524, 'num_leaves': 92, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.6859849286297378, 'colsample_bytree': 0.7792624325866331, 'reg_alpha': 2.4182451882882114, 'reg_lambda': 7.884827564881256, 'min_split_gain': 0.04991775748939101}. Best is trial 4 with value: 0.0031779081507095244.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 4] New best: 0.00318
[I 2025-06-15 19:56:01,110] Trial 5 finished with value: 0.0039373033734900215 and parameters: {'n_estimators': 186, 'learning_rate': 0.056569692743305325, 'num_leaves': 169, 'max_depth': 10, 'min_child_samples': 27, 'subsample': 0.8029590011141315, 'colsample_bytree': 0.6664678513019301, 'reg_alpha': 8.265409450542334, 'reg_lambda': 7.902610300406982, 'min_split_gain': 0.007844192923576632}. Best is trial 4 with value: 0.0031779081507095244.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:01,456] Trial 6 finished with value: 0.0041822945537554855 and parameters: {'n_estimators': 244, 'learning_rate': 0.08574065970633063, 'num_leaves': 127, 'max_depth': 7, 'min_child_samples': 5, 'subsample': 0.5245924729524253, 'colsample_bytree': 0.8819190292233791, 'reg_alpha': 3.4203475961729013, 'reg_lambda': 4.116562748412797, 'min_split_gain': 0.03763903760337258}. Best is trial 4 with value: 0.0031779081507095244.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:01,844] Trial 7 finished with value: 0.004654927661534581 and parameters: {'n_estimators': 107, 'learning_rate': 0.06533799200243857, 'num_leaves': 40, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.6620497020490825, 'colsample_bytree': 0.5198812874814086, 'reg_alpha': 2.6052069026512417, 'reg_lambda': 6.549959879854816, 'min_split_gain': 0.025644282222417738}. Best is trial 4 with value: 0.0031779081507095244.
[I 2025-06-15 19:56:01,847] A new study created in memory with name: no-name-990abe9a-3183-4f6a-8615-e2a66270398e
[I 2025-06-15 19:56:01,878] Trial 0 finished with value: 0.07671810492413524 and parameters: {'n_estimators': 282, 'learning_rate': 0.03835190741333077, 'num_leaves': 233, 'max_depth': 16, 'min_child_samples': 27, 'subsample': 0.6049021477748469, 'colsample_bytree': 0.9328933352515024, 'reg_alpha': 7.12773783978739, 'reg_lambda': 5.056341080582137, 'min_split_gain': 0.028368843900712433}. Best is trial 0 with value: 0.07671810492413524.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         328
1           Price_lag_2         166
14      pct_chg_Load_DA         161
3         Load_DA_lag_0          99
11           NGas_lag_2          79
2           Price_lag_7          57
15       lag168_Load_DA          56
16  roll2d_median_Price          56
13            EUA_lag_2          52
12            Oil_lag_2          29
10           Coal_lag_2          26
5       WindOn_DA_lag_0          14
7                  WD_1           4
9                  WD_7           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          14
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 26.95% done
********************* END NS.. 196 ****************************************************
********************* START NS ... 197 ****************************************************
[I 2025-06-15 19:56:02,691] A new study created in memory with name: no-name-f7e839ab-35af-4536-9832-87f4d3612d62
[I 2025-06-15 19:56:02,919] Trial 0 finished with value: 0.034859573421276234 and parameters: {'n_estimators': 271, 'learning_rate': 0.2714594515585921, 'num_leaves': 86, 'max_depth': 14, 'min_child_samples': 12, 'subsample': 0.8695084621085014, 'colsample_bytree': 0.9251253223996603, 'reg_alpha': 1.730274342229653, 'reg_lambda': 0.44401031310970396, 'min_split_gain': 0.0448576772528601}. Best is trial 0 with value: 0.034859573421276234.
[Trial 0] New best: 0.03486
[I 2025-06-15 19:56:03,881] Trial 1 finished with value: 0.05007062563078029 and parameters: {'n_estimators': 268, 'learning_rate': 0.010980961174171347, 'num_leaves': 153, 'max_depth': 15, 'min_child_samples': 5, 'subsample': 0.6575212656907596, 'colsample_bytree': 0.7725723757991572, 'reg_alpha': 9.522140089693693, 'reg_lambda': 4.24284688648295, 'min_split_gain': 0.009688889115288302}. Best is trial 0 with value: 0.034859573421276234.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:04,467] Trial 2 finished with value: 0.10752935885096211 and parameters: {'n_estimators': 137, 'learning_rate': 0.013155644589897111, 'num_leaves': 154, 'max_depth': 13, 'min_child_samples': 1, 'subsample': 0.7987715747285173, 'colsample_bytree': 0.5361962546876825, 'reg_alpha': 4.100316456547832, 'reg_lambda': 7.432352750592438, 'min_split_gain': 0.043041696247493565}. Best is trial 0 with value: 0.034859573421276234.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:05,158] Trial 3 finished with value: 0.05050236367436286 and parameters: {'n_estimators': 215, 'learning_rate': 0.013241940472949739, 'num_leaves': 188, 'max_depth': 14, 'min_child_samples': 6, 'subsample': 0.8817930611590725, 'colsample_bytree': 0.875448642951429, 'reg_alpha': 7.002200088739478, 'reg_lambda': 6.733986719594358, 'min_split_gain': 0.040523715931157965}. Best is trial 0 with value: 0.034859573421276234.
[I 2025-06-15 19:56:05,161] A new study created in memory with name: no-name-34957922-be43-4a55-a1ef-aa674dfa2696
[I 2025-06-15 19:56:05,184] Trial 0 finished with value: 0.26643108803758325 and parameters: {'n_estimators': 149, 'learning_rate': 0.031766409379204996, 'num_leaves': 162, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.9425828671793289, 'colsample_bytree': 0.7371203527388994, 'reg_alpha': 4.197737515587, 'reg_lambda': 9.247261016958046, 'min_split_gain': 0.022742667002878886}. Best is trial 0 with value: 0.26643108803758325.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         260
14      pct_chg_Load_DA         180
1           Price_lag_2         172
3         Load_DA_lag_0         138
2           Price_lag_7          92
16  roll2d_median_Price          87
11           NGas_lag_2          73
15       lag168_Load_DA          68
10           Coal_lag_2          41
12            Oil_lag_2          34
5       WindOn_DA_lag_0          33
13            EUA_lag_2          29
7                  WD_1           3
9                  WD_7           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         129
16  roll2d_median_Price          27
1           Price_lag_2           9
15       lag168_Load_DA           9
2           Price_lag_7           8
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 27.09% done
********************* END NS.. 197 ****************************************************
********************* START NS ... 198 ****************************************************
[I 2025-06-15 19:56:05,928] A new study created in memory with name: no-name-c4f712df-7843-4ad8-b090-46f9db680e36
[I 2025-06-15 19:56:06,473] Trial 0 finished with value: 0.03809837498536805 and parameters: {'n_estimators': 276, 'learning_rate': 0.04100274872860023, 'num_leaves': 195, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.9128085746840311, 'colsample_bytree': 0.6854444437964602, 'reg_alpha': 9.237443743209752, 'reg_lambda': 8.098683646144575, 'min_split_gain': 0.00906519781823481}. Best is trial 0 with value: 0.03809837498536805.
[I 2025-06-15 19:56:06,618] Trial 1 finished with value: 0.03503696545466476 and parameters: {'n_estimators': 59, 'learning_rate': 0.15983612465019484, 'num_leaves': 147, 'max_depth': 5, 'min_child_samples': 30, 'subsample': 0.5948718134442699, 'colsample_bytree': 0.6522041874542428, 'reg_alpha': 0.8195446984855237, 'reg_lambda': 3.321331828796895, 'min_split_gain': 0.018532079626849197}. Best is trial 1 with value: 0.03503696545466476.
[Trial 0] New best: 0.03810
[Trial 1] New best: 0.03504
[I 2025-06-15 19:56:06,875] Trial 2 finished with value: 0.039398558564168275 and parameters: {'n_estimators': 110, 'learning_rate': 0.2996222206964777, 'num_leaves': 166, 'max_depth': 16, 'min_child_samples': 4, 'subsample': 0.7930570704065543, 'colsample_bytree': 0.628765215481047, 'reg_alpha': 6.572385137901032, 'reg_lambda': 3.3523869143618246, 'min_split_gain': 0.0024447181941817334}. Best is trial 1 with value: 0.03503696545466476.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:07,506] Trial 3 finished with value: 0.04207140243557061 and parameters: {'n_estimators': 278, 'learning_rate': 0.049439683050894204, 'num_leaves': 156, 'max_depth': 15, 'min_child_samples': 15, 'subsample': 0.8991276686896558, 'colsample_bytree': 0.5729043290679379, 'reg_alpha': 5.369350292226178, 'reg_lambda': 9.117106268365541, 'min_split_gain': 0.014623124504707702}. Best is trial 1 with value: 0.03503696545466476.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:08,114] Trial 4 finished with value: 0.03882228957862236 and parameters: {'n_estimators': 167, 'learning_rate': 0.03598843819021879, 'num_leaves': 180, 'max_depth': 16, 'min_child_samples': 7, 'subsample': 0.6581808927673982, 'colsample_bytree': 0.7098859945330255, 'reg_alpha': 7.08518287498862, 'reg_lambda': 4.4847234705301355, 'min_split_gain': 0.016841365677932537}. Best is trial 1 with value: 0.03503696545466476.
[I 2025-06-15 19:56:08,117] A new study created in memory with name: no-name-41f61ccd-7e45-45d1-bab4-2cc4da7b88f2
[I 2025-06-15 19:56:08,152] Trial 0 finished with value: 0.18220377193646964 and parameters: {'n_estimators': 253, 'learning_rate': 0.2866289805670924, 'num_leaves': 199, 'max_depth': 15, 'min_child_samples': 23, 'subsample': 0.9862395434942387, 'colsample_bytree': 0.7516196074000252, 'reg_alpha': 5.921637492972246, 'reg_lambda': 9.93351261287058, 'min_split_gain': 7.410942358410111e-05}. Best is trial 0 with value: 0.18220377193646964.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         346
1           Price_lag_2         195
14      pct_chg_Load_DA         134
3         Load_DA_lag_0         112
16  roll2d_median_Price          78
11           NGas_lag_2          67
13            EUA_lag_2          66
2           Price_lag_7          66
12            Oil_lag_2          59
5       WindOn_DA_lag_0          56
15       lag168_Load_DA          56
10           Coal_lag_2          50
7                  WD_1           5
9                  WD_7           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          28
16  roll2d_median_Price           9
2           Price_lag_7           1
10           Coal_lag_2           1
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 27.22% done
********************* END NS.. 198 ****************************************************
********************* START NS ... 199 ****************************************************
[I 2025-06-15 19:56:08,838] A new study created in memory with name: no-name-184bffb1-ab40-43fe-b568-566091ad7196
[I 2025-06-15 19:56:09,691] Trial 0 finished with value: 0.009794887509316124 and parameters: {'n_estimators': 233, 'learning_rate': 0.010511721270757569, 'num_leaves': 211, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.8625408605205769, 'colsample_bytree': 0.7336977769026163, 'reg_alpha': 3.2026634725956296, 'reg_lambda': 2.602965516065586, 'min_split_gain': 0.04030407693108898}. Best is trial 0 with value: 0.009794887509316124.
[Trial 0] New best: 0.00979
[I 2025-06-15 19:56:10,040] Trial 1 finished with value: 0.004587755532520056 and parameters: {'n_estimators': 134, 'learning_rate': 0.06634788660370279, 'num_leaves': 172, 'max_depth': 6, 'min_child_samples': 23, 'subsample': 0.6307717473692016, 'colsample_bytree': 0.5548759901096293, 'reg_alpha': 6.106742829428828, 'reg_lambda': 3.958772531749104, 'min_split_gain': 0.010860637864730367}. Best is trial 1 with value: 0.004587755532520056.
[Trial 1] New best: 0.00459
[I 2025-06-15 19:56:10,351] Trial 2 finished with value: 0.008571347400257273 and parameters: {'n_estimators': 72, 'learning_rate': 0.036673836409082096, 'num_leaves': 229, 'max_depth': 12, 'min_child_samples': 20, 'subsample': 0.5187437250765241, 'colsample_bytree': 0.7498250570269018, 'reg_alpha': 3.0070613054480253, 'reg_lambda': 8.023021681606286, 'min_split_gain': 0.03018642120921221}. Best is trial 1 with value: 0.004587755532520056.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:10,770] Trial 3 finished with value: 0.0058000310882857465 and parameters: {'n_estimators': 254, 'learning_rate': 0.04439304287831066, 'num_leaves': 53, 'max_depth': 5, 'min_child_samples': 16, 'subsample': 0.7026297003072832, 'colsample_bytree': 0.9419139535913255, 'reg_alpha': 7.341035447414982, 'reg_lambda': 5.879109532181815, 'min_split_gain': 0.01684507645337755}. Best is trial 1 with value: 0.004587755532520056.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:11,085] Trial 4 finished with value: 0.12599380746766928 and parameters: {'n_estimators': 59, 'learning_rate': 0.013002080803914767, 'num_leaves': 121, 'max_depth': 5, 'min_child_samples': 16, 'subsample': 0.7348416524071335, 'colsample_bytree': 0.5773286263569848, 'reg_alpha': 6.86921121826375, 'reg_lambda': 2.6722610576449526, 'min_split_gain': 0.014839737296962774}. Best is trial 1 with value: 0.004587755532520056.
[I 2025-06-15 19:56:11,088] A new study created in memory with name: no-name-278341fb-9ceb-4673-acc9-a19eb87bf1f1
[I 2025-06-15 19:56:11,138] Trial 0 finished with value: 0.004363145628235977 and parameters: {'n_estimators': 281, 'learning_rate': 0.08645661708337103, 'num_leaves': 32, 'max_depth': 8, 'min_child_samples': 12, 'subsample': 0.5301509878122761, 'colsample_bytree': 0.995626414452943, 'reg_alpha': 5.32290724894812, 'reg_lambda': 4.908791807816889, 'min_split_gain': 0.0021472403022772426}. Best is trial 0 with value: 0.004363145628235977.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         708
1           Price_lag_2         582
14      pct_chg_Load_DA         364
3         Load_DA_lag_0         294
11           NGas_lag_2         282
2           Price_lag_7         262
15       lag168_Load_DA         211
13            EUA_lag_2         185
16  roll2d_median_Price         176
10           Coal_lag_2         167
5       WindOn_DA_lag_0         156
12            Oil_lag_2         117
9                  WD_7          17
7                  WD_1          15
8                  WD_6          14
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          59
16  roll2d_median_Price          24
11           NGas_lag_2           1
15       lag168_Load_DA           1
2           Price_lag_7           0
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 27.36% done
********************* END NS.. 199 ****************************************************
********************* START NS ... 200 ****************************************************
[I 2025-06-15 19:56:12,047] A new study created in memory with name: no-name-ec52de29-01ff-4428-b559-c19b0140b505
[I 2025-06-15 19:56:12,396] Trial 0 finished with value: 0.0002492754602643491 and parameters: {'n_estimators': 218, 'learning_rate': 0.04642591032638798, 'num_leaves': 155, 'max_depth': 5, 'min_child_samples': 29, 'subsample': 0.8654161809597654, 'colsample_bytree': 0.5289667140411922, 'reg_alpha': 2.75703850721432, 'reg_lambda': 0.8807291879169954, 'min_split_gain': 0.018513848917386923}. Best is trial 0 with value: 0.0002492754602643491.
[Trial 0] New best: 0.00025
[I 2025-06-15 19:56:13,150] Trial 1 finished with value: 0.012254661966280658 and parameters: {'n_estimators': 149, 'learning_rate': 0.012534753768855053, 'num_leaves': 234, 'max_depth': 13, 'min_child_samples': 26, 'subsample': 0.5446612385010341, 'colsample_bytree': 0.9148009009868009, 'reg_alpha': 2.0062284984347722, 'reg_lambda': 1.155116392330734, 'min_split_gain': 0.007409353929957303}. Best is trial 0 with value: 0.0002492754602643491.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:13,780] Trial 2 finished with value: 0.00039002720946285574 and parameters: {'n_estimators': 260, 'learning_rate': 0.03051828955644032, 'num_leaves': 107, 'max_depth': 14, 'min_child_samples': 30, 'subsample': 0.5509223386049055, 'colsample_bytree': 0.567710652769218, 'reg_alpha': 6.552226488720788, 'reg_lambda': 6.348053086684345, 'min_split_gain': 0.03352260314884499}. Best is trial 0 with value: 0.0002492754602643491.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:14,071] Trial 3 finished with value: 0.11675013030168623 and parameters: {'n_estimators': 75, 'learning_rate': 0.010016838024753102, 'num_leaves': 89, 'max_depth': 13, 'min_child_samples': 5, 'subsample': 0.5992967556698915, 'colsample_bytree': 0.7885797206201172, 'reg_alpha': 7.210173239012462, 'reg_lambda': 3.1388809213181954, 'min_split_gain': 0.04837627013444634}. Best is trial 0 with value: 0.0002492754602643491.
[I 2025-06-15 19:56:14,073] A new study created in memory with name: no-name-0827f84f-8eb9-4da4-ac7d-16345eb5972a
[I 2025-06-15 19:56:14,094] Trial 0 finished with value: 0.0005159452380008952 and parameters: {'n_estimators': 95, 'learning_rate': 0.05266805516609812, 'num_leaves': 227, 'max_depth': 8, 'min_child_samples': 7, 'subsample': 0.960232356809313, 'colsample_bytree': 0.5207709148727457, 'reg_alpha': 8.77666078515137, 'reg_lambda': 9.669397846573013, 'min_split_gain': 0.011790855084077501}. Best is trial 0 with value: 0.0005159452380008952.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1005
1           Price_lag_2         738
14      pct_chg_Load_DA         518
3         Load_DA_lag_0         414
11           NGas_lag_2         320
16  roll2d_median_Price         297
15       lag168_Load_DA         289
2           Price_lag_7         271
5       WindOn_DA_lag_0         258
13            EUA_lag_2         223
10           Coal_lag_2         190
12            Oil_lag_2         114
7                  WD_1          21
9                  WD_7          19
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          35
1           Price_lag_2           9
16  roll2d_median_Price           2
2           Price_lag_7           1
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 27.50% done
********************* END NS.. 200 ****************************************************
********************* START NS ... 201 ****************************************************
[I 2025-06-15 19:56:14,985] A new study created in memory with name: no-name-5b810424-1c2d-4557-8a3c-21d574c98497
[I 2025-06-15 19:56:15,346] Trial 0 finished with value: 0.00011398154416911486 and parameters: {'n_estimators': 168, 'learning_rate': 0.03779193607538796, 'num_leaves': 240, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.5491631682722551, 'colsample_bytree': 0.6485909972409439, 'reg_alpha': 6.013516503102636, 'reg_lambda': 5.5255920652088255, 'min_split_gain': 0.019065456149925622}. Best is trial 0 with value: 0.00011398154416911486.
[Trial 0] New best: 0.00011
[I 2025-06-15 19:56:15,560] Trial 1 finished with value: 0.00010241030907567107 and parameters: {'n_estimators': 165, 'learning_rate': 0.1288986869126909, 'num_leaves': 239, 'max_depth': 4, 'min_child_samples': 15, 'subsample': 0.5848350056783155, 'colsample_bytree': 0.6421292230618907, 'reg_alpha': 5.712022605315195, 'reg_lambda': 9.274871800405661, 'min_split_gain': 0.006666527032846897}. Best is trial 1 with value: 0.00010241030907567107.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:15,792] Trial 2 finished with value: 0.0007957557239133155 and parameters: {'n_estimators': 66, 'learning_rate': 0.0539829201073298, 'num_leaves': 183, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.9146652836405249, 'colsample_bytree': 0.8986856970255181, 'reg_alpha': 8.043547613119289, 'reg_lambda': 2.6474058754484875, 'min_split_gain': 0.0288212518820837}. Best is trial 1 with value: 0.00010241030907567107.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:16,826] Trial 3 finished with value: 0.0004241706435209206 and parameters: {'n_estimators': 234, 'learning_rate': 0.019363242827422567, 'num_leaves': 185, 'max_depth': 13, 'min_child_samples': 18, 'subsample': 0.5199239849969177, 'colsample_bytree': 0.5724389918428845, 'reg_alpha': 2.693539099091949, 'reg_lambda': 3.86544597719808, 'min_split_gain': 0.01734876332696546}. Best is trial 1 with value: 0.00010241030907567107.
[I 2025-06-15 19:56:16,828] A new study created in memory with name: no-name-e0633d1f-311a-4536-936d-f452770473a2
[I 2025-06-15 19:56:16,854] Trial 0 finished with value: 0.00015146551704258652 and parameters: {'n_estimators': 129, 'learning_rate': 0.29180162567142803, 'num_leaves': 148, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.7003966331914215, 'colsample_bytree': 0.554943146725005, 'reg_alpha': 7.007707833584127, 'reg_lambda': 0.7447162893131165, 'min_split_gain': 0.0009166019518403013}. Best is trial 0 with value: 0.00015146551704258652.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         420
1           Price_lag_2         245
14      pct_chg_Load_DA         158
3         Load_DA_lag_0         134
11           NGas_lag_2         130
10           Coal_lag_2          91
5       WindOn_DA_lag_0          88
16  roll2d_median_Price          82
13            EUA_lag_2          78
15       lag168_Load_DA          76
2           Price_lag_7          73
12            Oil_lag_2          31
9                  WD_7          15
7                  WD_1           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           9
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 27.63% done
********************* END NS.. 201 ****************************************************
********************* START NS ... 202 ****************************************************
[I 2025-06-15 19:56:17,548] A new study created in memory with name: no-name-c7e2987e-f33e-4e3e-a110-1cb449d9f78c
[I 2025-06-15 19:56:17,750] Trial 0 finished with value: 0.0013276243088788154 and parameters: {'n_estimators': 188, 'learning_rate': 0.14748596601829608, 'num_leaves': 254, 'max_depth': 13, 'min_child_samples': 1, 'subsample': 0.6588033387090739, 'colsample_bytree': 0.8924489687679054, 'reg_alpha': 7.52933368384239, 'reg_lambda': 9.478092496654707, 'min_split_gain': 0.014987376853040546}. Best is trial 0 with value: 0.0013276243088788154.
[I 2025-06-15 19:56:17,940] Trial 1 finished with value: 0.0007398278258820595 and parameters: {'n_estimators': 109, 'learning_rate': 0.0355817498861603, 'num_leaves': 216, 'max_depth': 4, 'min_child_samples': 22, 'subsample': 0.8133886385054412, 'colsample_bytree': 0.7882621053663644, 'reg_alpha': 8.807918577173531, 'reg_lambda': 0.19350724237628403, 'min_split_gain': 0.02678108038016755}. Best is trial 1 with value: 0.0007398278258820595.
[Trial 0] New best: 0.00133
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:18,165] Trial 2 finished with value: 0.0020777954979966603 and parameters: {'n_estimators': 69, 'learning_rate': 0.04881024445957004, 'num_leaves': 236, 'max_depth': 13, 'min_child_samples': 19, 'subsample': 0.7745509956355905, 'colsample_bytree': 0.8855019976204616, 'reg_alpha': 8.016169383557664, 'reg_lambda': 3.749594299524075, 'min_split_gain': 0.044838770461036646}. Best is trial 1 with value: 0.0007398278258820595.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:19,031] Trial 3 finished with value: 0.00042345689630687204 and parameters: {'n_estimators': 168, 'learning_rate': 0.025258618235868526, 'num_leaves': 71, 'max_depth': 16, 'min_child_samples': 11, 'subsample': 0.6439689071792392, 'colsample_bytree': 0.6976715712824296, 'reg_alpha': 0.033770063068850886, 'reg_lambda': 6.3870714729041875, 'min_split_gain': 0.014151059037055314}. Best is trial 3 with value: 0.00042345689630687204.
[I 2025-06-15 19:56:19,034] A new study created in memory with name: no-name-5d3ee6f6-784c-407e-a75b-378f65f34af5
[I 2025-06-15 19:56:19,053] Trial 0 finished with value: 0.000116454125865945 and parameters: {'n_estimators': 105, 'learning_rate': 0.05930134469300717, 'num_leaves': 179, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.796059977464839, 'colsample_bytree': 0.5002168905213267, 'reg_alpha': 1.6111385905809938, 'reg_lambda': 6.044728737512867, 'min_split_gain': 0.03725443100487174}. Best is trial 0 with value: 0.000116454125865945.
[I 2025-06-15 19:56:19,103] Trial 1 finished with value: 0.0001072499283269717 and parameters: {'n_estimators': 255, 'learning_rate': 0.01103173030600458, 'num_leaves': 174, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.7802692919457206, 'colsample_bytree': 0.8875249826704268, 'reg_alpha': 5.001439265347543, 'reg_lambda': 2.233405881221594, 'min_split_gain': 0.0269609138993627}. Best is trial 1 with value: 0.0001072499283269717.
[I 2025-06-15 19:56:19,133] Trial 2 finished with value: 0.0002669057006130427 and parameters: {'n_estimators': 247, 'learning_rate': 0.056835515741333165, 'num_leaves': 250, 'max_depth': 15, 'min_child_samples': 25, 'subsample': 0.719675188500609, 'colsample_bytree': 0.9769643671650516, 'reg_alpha': 6.9825324836291305, 'reg_lambda': 0.6223065835843888, 'min_split_gain': 0.022197341055196874}. Best is trial 1 with value: 0.0001072499283269717.
[I 2025-06-15 19:56:19,157] Trial 3 finished with value: 6.216503840643772e-05 and parameters: {'n_estimators': 197, 'learning_rate': 0.138773277437596, 'num_leaves': 167, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.9113241695578125, 'colsample_bytree': 0.6362691304332289, 'reg_alpha': 3.1470491292707017, 'reg_lambda': 9.43217921049444, 'min_split_gain': 0.01566702116691347}. Best is trial 3 with value: 6.216503840643772e-05.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] New best: 0.00012
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        2464
0           Price_lag_1        2329
14      pct_chg_Load_DA        1252
3         Load_DA_lag_0         957
16  roll2d_median_Price         753
11           NGas_lag_2         731
2           Price_lag_7         695
15       lag168_Load_DA         564
10           Coal_lag_2         525
5       WindOn_DA_lag_0         486
13            EUA_lag_2         452
12            Oil_lag_2         419
9                  WD_7          37
7                  WD_1          23
8                  WD_6          20
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          21
9                  WD_7           1
1           Price_lag_2           1
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 27.77% done
********************* END NS.. 202 ****************************************************
********************* START NS ... 203 ****************************************************
[I 2025-06-15 19:56:20,467] A new study created in memory with name: no-name-a3ca8ef4-45ae-4f6e-85ed-81ca28a7ffb6
[I 2025-06-15 19:56:20,869] Trial 0 finished with value: 7.671188994814479e-05 and parameters: {'n_estimators': 112, 'learning_rate': 0.07664682613455838, 'num_leaves': 55, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.7427621038467658, 'colsample_bytree': 0.9768973149520174, 'reg_alpha': 0.5105495087967382, 'reg_lambda': 4.338219624700013, 'min_split_gain': 0.044818404974622736}. Best is trial 0 with value: 7.671188994814479e-05.
[Trial 0] New best: 0.00008
[I 2025-06-15 19:56:21,528] Trial 1 finished with value: 0.0023134888973181072 and parameters: {'n_estimators': 226, 'learning_rate': 0.014077331816136279, 'num_leaves': 36, 'max_depth': 14, 'min_child_samples': 16, 'subsample': 0.6182306681698228, 'colsample_bytree': 0.5566663320360649, 'reg_alpha': 5.684105147398157, 'reg_lambda': 6.5677684623311805, 'min_split_gain': 0.02738526444949302}. Best is trial 0 with value: 7.671188994814479e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:21,942] Trial 2 finished with value: 0.00026386040876459014 and parameters: {'n_estimators': 292, 'learning_rate': 0.07171832971430905, 'num_leaves': 60, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.9273024799958536, 'colsample_bytree': 0.8241613653904679, 'reg_alpha': 6.1897433335596475, 'reg_lambda': 0.9881096894059394, 'min_split_gain': 0.027103338336931255}. Best is trial 0 with value: 7.671188994814479e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:22,329] Trial 3 finished with value: 0.00032030503515879157 and parameters: {'n_estimators': 290, 'learning_rate': 0.09706303410413251, 'num_leaves': 129, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.6417658439418131, 'colsample_bytree': 0.9574620341779191, 'reg_alpha': 2.9433243122565997, 'reg_lambda': 9.919194991806718, 'min_split_gain': 0.021533366814587562}. Best is trial 0 with value: 7.671188994814479e-05.
[I 2025-06-15 19:56:22,332] A new study created in memory with name: no-name-8f0c2aea-381c-4e68-bdaf-57ac7a0b86a5
[I 2025-06-15 19:56:22,349] Trial 0 finished with value: 5.296363782257631e-05 and parameters: {'n_estimators': 139, 'learning_rate': 0.015531902239660084, 'num_leaves': 84, 'max_depth': 12, 'min_child_samples': 4, 'subsample': 0.9235476827721345, 'colsample_bytree': 0.8136298540075706, 'reg_alpha': 6.714711885965935, 'reg_lambda': 3.764736334131559, 'min_split_gain': 0.0235952599576395}. Best is trial 0 with value: 5.296363782257631e-05.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         902
14      pct_chg_Load_DA         519
1           Price_lag_2         497
3         Load_DA_lag_0         303
11           NGas_lag_2         231
16  roll2d_median_Price         227
2           Price_lag_7         211
15       lag168_Load_DA         163
12            Oil_lag_2         111
10           Coal_lag_2         101
13            EUA_lag_2          89
5       WindOn_DA_lag_0          66
7                  WD_1          14
9                  WD_7           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 27.91% done
********************* END NS.. 203 ****************************************************
********************* START NS ... 204 ****************************************************
[I 2025-06-15 19:56:23,149] A new study created in memory with name: no-name-25cf0c58-82be-4453-8437-168fd64e2d7e
[I 2025-06-15 19:56:23,699] Trial 0 finished with value: 0.001369437266268059 and parameters: {'n_estimators': 233, 'learning_rate': 0.017235098076773522, 'num_leaves': 73, 'max_depth': 12, 'min_child_samples': 18, 'subsample': 0.5168736061006105, 'colsample_bytree': 0.9393709536202917, 'reg_alpha': 9.740832021156683, 'reg_lambda': 7.379816975877568, 'min_split_gain': 0.02086602508569181}. Best is trial 0 with value: 0.001369437266268059.
[Trial 0] New best: 0.00137
[I 2025-06-15 19:56:24,025] Trial 1 finished with value: 0.001131679274494083 and parameters: {'n_estimators': 257, 'learning_rate': 0.19765030807546713, 'num_leaves': 250, 'max_depth': 12, 'min_child_samples': 19, 'subsample': 0.6219961396923032, 'colsample_bytree': 0.5252332904656268, 'reg_alpha': 5.940673604597055, 'reg_lambda': 2.376423633364891, 'min_split_gain': 0.01180611212151887}. Best is trial 1 with value: 0.001131679274494083.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:24,497] Trial 2 finished with value: 0.0009020099229760577 and parameters: {'n_estimators': 209, 'learning_rate': 0.06609699024763148, 'num_leaves': 145, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.6404006722748355, 'colsample_bytree': 0.5128706175933629, 'reg_alpha': 5.296767054228916, 'reg_lambda': 0.15623249812842466, 'min_split_gain': 0.017901472302105398}. Best is trial 2 with value: 0.0009020099229760577.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:24,790] Trial 3 finished with value: 0.0017455753549854675 and parameters: {'n_estimators': 253, 'learning_rate': 0.2636686721420665, 'num_leaves': 134, 'max_depth': 6, 'min_child_samples': 14, 'subsample': 0.8698662035988645, 'colsample_bytree': 0.8743181363133787, 'reg_alpha': 6.247917614213073, 'reg_lambda': 0.4220234344224316, 'min_split_gain': 0.00030177191809814823}. Best is trial 2 with value: 0.0009020099229760577.
[I 2025-06-15 19:56:24,792] A new study created in memory with name: no-name-59ec48cb-346f-4705-84b0-7c35f339b2a7
[I 2025-06-15 19:56:24,825] Trial 0 finished with value: 0.005298700353281647 and parameters: {'n_estimators': 234, 'learning_rate': 0.036044685005952946, 'num_leaves': 36, 'max_depth': 8, 'min_child_samples': 12, 'subsample': 0.8733055596532806, 'colsample_bytree': 0.8731266162146496, 'reg_alpha': 7.88908618387339, 'reg_lambda': 1.1516912981116534, 'min_split_gain': 0.040583292944174676}. Best is trial 0 with value: 0.005298700353281647.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         904
1           Price_lag_2         801
14      pct_chg_Load_DA         536
2           Price_lag_7         408
3         Load_DA_lag_0         404
13            EUA_lag_2         367
11           NGas_lag_2         366
15       lag168_Load_DA         342
16  roll2d_median_Price         327
5       WindOn_DA_lag_0         289
10           Coal_lag_2         272
12            Oil_lag_2         172
9                  WD_7          41
8                  WD_6          23
7                  WD_1          22
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 28.04% done
********************* END NS.. 204 ****************************************************
********************* START NS ... 205 ****************************************************
[I 2025-06-15 19:56:25,839] A new study created in memory with name: no-name-9f219a22-063c-4e94-a168-719e884ea4c6
[I 2025-06-15 19:56:26,628] Trial 0 finished with value: 0.0011535764119181143 and parameters: {'n_estimators': 191, 'learning_rate': 0.042233111220491344, 'num_leaves': 190, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.5901061355839377, 'colsample_bytree': 0.5121235088623752, 'reg_alpha': 6.183892761933395, 'reg_lambda': 6.314676006150659, 'min_split_gain': 0.008422448882665813}. Best is trial 0 with value: 0.0011535764119181143.
[Trial 0] New best: 0.00115
[I 2025-06-15 19:56:27,148] Trial 1 finished with value: 0.016046034391091427 and parameters: {'n_estimators': 157, 'learning_rate': 0.012071510051600575, 'num_leaves': 45, 'max_depth': 5, 'min_child_samples': 7, 'subsample': 0.6833077522997011, 'colsample_bytree': 0.7702852244942486, 'reg_alpha': 0.6303979950348226, 'reg_lambda': 2.0421875117660946, 'min_split_gain': 0.009349759926487523}. Best is trial 0 with value: 0.0011535764119181143.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:27,998] Trial 2 finished with value: 0.0011450553626759438 and parameters: {'n_estimators': 206, 'learning_rate': 0.023681002617025913, 'num_leaves': 194, 'max_depth': 16, 'min_child_samples': 25, 'subsample': 0.8422275740634555, 'colsample_bytree': 0.87957766158472, 'reg_alpha': 1.9753039886423174, 'reg_lambda': 3.2969735764464225, 'min_split_gain': 0.03656033482623102}. Best is trial 2 with value: 0.0011450553626759438.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:28,347] Trial 3 finished with value: 0.0013508682879625814 and parameters: {'n_estimators': 135, 'learning_rate': 0.09811142892631235, 'num_leaves': 166, 'max_depth': 9, 'min_child_samples': 10, 'subsample': 0.8335512737049965, 'colsample_bytree': 0.7638345342536684, 'reg_alpha': 6.112002136061036, 'reg_lambda': 0.7048821230894486, 'min_split_gain': 0.03072225206960736}. Best is trial 2 with value: 0.0011450553626759438.
[I 2025-06-15 19:56:28,350] A new study created in memory with name: no-name-de37e4e5-71d9-4ae4-b3a2-d1a42b50729e
[I 2025-06-15 19:56:28,392] Trial 0 finished with value: 0.007952928890915196 and parameters: {'n_estimators': 110, 'learning_rate': 0.08591333096378774, 'num_leaves': 58, 'max_depth': 14, 'min_child_samples': 4, 'subsample': 0.7034841588725462, 'colsample_bytree': 0.9515856487817318, 'reg_alpha': 4.983903456331786, 'reg_lambda': 5.854471281149742, 'min_split_gain': 0.00839731425434066}. Best is trial 0 with value: 0.007952928890915196.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2577
1           Price_lag_2        2032
14      pct_chg_Load_DA        1451
3         Load_DA_lag_0         726
11           NGas_lag_2         656
16  roll2d_median_Price         593
2           Price_lag_7         587
15       lag168_Load_DA         424
12            Oil_lag_2         300
10           Coal_lag_2         268
5       WindOn_DA_lag_0         236
13            EUA_lag_2         184
9                  WD_7          29
7                  WD_1           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 28.18% done
********************* END NS.. 205 ****************************************************
********************* START NS ... 206 ****************************************************
[I 2025-06-15 19:56:29,555] A new study created in memory with name: no-name-7d01e4d4-d200-468d-93f0-eb5012d839e9
[I 2025-06-15 19:56:30,010] Trial 0 finished with value: 0.0006640150096611282 and parameters: {'n_estimators': 233, 'learning_rate': 0.019539647298039007, 'num_leaves': 110, 'max_depth': 5, 'min_child_samples': 26, 'subsample': 0.5766878334170186, 'colsample_bytree': 0.8441928044679273, 'reg_alpha': 1.1554796569775982, 'reg_lambda': 7.227811306717975, 'min_split_gain': 0.04775557399367706}. Best is trial 0 with value: 0.0006640150096611282.
[Trial 0] New best: 0.00066
[I 2025-06-15 19:56:30,251] Trial 1 finished with value: 0.0006587372822387059 and parameters: {'n_estimators': 271, 'learning_rate': 0.1518910519015934, 'num_leaves': 43, 'max_depth': 15, 'min_child_samples': 14, 'subsample': 0.8746499146555087, 'colsample_bytree': 0.8897239398858587, 'reg_alpha': 9.341990276020042, 'reg_lambda': 6.09215485063514, 'min_split_gain': 0.012585706085686217}. Best is trial 1 with value: 0.0006587372822387059.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:30,719] Trial 2 finished with value: 0.0019222404552837375 and parameters: {'n_estimators': 142, 'learning_rate': 0.021328013170371175, 'num_leaves': 153, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.5145845645604261, 'colsample_bytree': 0.8370925510975393, 'reg_alpha': 5.032658448383124, 'reg_lambda': 5.174640630900024, 'min_split_gain': 0.043244802502626314}. Best is trial 1 with value: 0.0006587372822387059.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:31,223] Trial 3 finished with value: 0.0016900159523922053 and parameters: {'n_estimators': 258, 'learning_rate': 0.13493909835935727, 'num_leaves': 227, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.52364838064403, 'colsample_bytree': 0.7658984410370604, 'reg_alpha': 1.676817386590388, 'reg_lambda': 7.001343992082853, 'min_split_gain': 0.00874646615310854}. Best is trial 1 with value: 0.0006587372822387059.
[I 2025-06-15 19:56:31,226] A new study created in memory with name: no-name-2df9529f-4d8f-4cef-922c-9095de562d2b
[I 2025-06-15 19:56:31,244] Trial 0 finished with value: 0.011895105001929555 and parameters: {'n_estimators': 117, 'learning_rate': 0.2801907657735243, 'num_leaves': 83, 'max_depth': 7, 'min_child_samples': 30, 'subsample': 0.6651531723771268, 'colsample_bytree': 0.6602490216774519, 'reg_alpha': 7.095813127739126, 'reg_lambda': 7.334959049208454, 'min_split_gain': 0.02153369336929591}. Best is trial 0 with value: 0.011895105001929555.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         315
1           Price_lag_2         193
14      pct_chg_Load_DA         182
3         Load_DA_lag_0         130
11           NGas_lag_2         126
2           Price_lag_7         101
5       WindOn_DA_lag_0          89
13            EUA_lag_2          88
16  roll2d_median_Price          86
12            Oil_lag_2          74
15       lag168_Load_DA          70
10           Coal_lag_2          66
9                  WD_7          13
7                  WD_1           5
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 28.32% done
********************* END NS.. 206 ****************************************************
********************* START NS ... 207 ****************************************************
[I 2025-06-15 19:56:31,994] A new study created in memory with name: no-name-641a5d7f-00d4-4c19-930a-4de61478bfe0
[I 2025-06-15 19:56:32,296] Trial 0 finished with value: 0.03507131140179517 and parameters: {'n_estimators': 94, 'learning_rate': 0.014310671111454214, 'num_leaves': 224, 'max_depth': 16, 'min_child_samples': 24, 'subsample': 0.5160794582263049, 'colsample_bytree': 0.8688928226917738, 'reg_alpha': 9.753507492678073, 'reg_lambda': 4.860430305278097, 'min_split_gain': 0.017789765229007343}. Best is trial 0 with value: 0.03507131140179517.
[Trial 0] New best: 0.03507
[I 2025-06-15 19:56:33,076] Trial 1 finished with value: 0.0014273828752770516 and parameters: {'n_estimators': 152, 'learning_rate': 0.021549555913824535, 'num_leaves': 153, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.7976209558056271, 'colsample_bytree': 0.7096649189940303, 'reg_alpha': 7.844685036380071, 'reg_lambda': 4.37704116726737, 'min_split_gain': 0.040303389265644}. Best is trial 1 with value: 0.0014273828752770516.
[Trial 1] New best: 0.00143
[I 2025-06-15 19:56:33,708] Trial 2 finished with value: 0.06196933657948165 and parameters: {'n_estimators': 91, 'learning_rate': 0.011554796321128553, 'num_leaves': 185, 'max_depth': 12, 'min_child_samples': 27, 'subsample': 0.6203951729986812, 'colsample_bytree': 0.6048693814994923, 'reg_alpha': 2.544700176424801, 'reg_lambda': 1.9704830217625802, 'min_split_gain': 0.035366369626614345}. Best is trial 1 with value: 0.0014273828752770516.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:34,537] Trial 3 finished with value: 0.006075257484250256 and parameters: {'n_estimators': 199, 'learning_rate': 0.012079954050979535, 'num_leaves': 130, 'max_depth': 7, 'min_child_samples': 4, 'subsample': 0.9287375812204282, 'colsample_bytree': 0.7185198756504053, 'reg_alpha': 4.402672307461353, 'reg_lambda': 9.631386895517846, 'min_split_gain': 0.028038790714376757}. Best is trial 1 with value: 0.0014273828752770516.
[I 2025-06-15 19:56:34,701] Trial 4 finished with value: 0.000723956548533644 and parameters: {'n_estimators': 164, 'learning_rate': 0.2904189924968814, 'num_leaves': 61, 'max_depth': 11, 'min_child_samples': 19, 'subsample': 0.5573682385461879, 'colsample_bytree': 0.7897250205500824, 'reg_alpha': 5.499597005673488, 'reg_lambda': 0.2427732381364056, 'min_split_gain': 0.04597861760860755}. Best is trial 4 with value: 0.000723956548533644.
[I 2025-06-15 19:56:34,703] A new study created in memory with name: no-name-4f5e6788-fe95-4ea1-bb9b-5597d0e62341
[I 2025-06-15 19:56:34,735] Trial 0 finished with value: 0.010375526015628108 and parameters: {'n_estimators': 50, 'learning_rate': 0.010946306019542667, 'num_leaves': 157, 'max_depth': 8, 'min_child_samples': 1, 'subsample': 0.6743428496053012, 'colsample_bytree': 0.8515243768759531, 'reg_alpha': 0.6474312988944386, 'reg_lambda': 5.223591230170487, 'min_split_gain': 0.02343354698182323}. Best is trial 0 with value: 0.010375526015628108.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         146
1           Price_lag_2          90
14      pct_chg_Load_DA          89
2           Price_lag_7          51
11           NGas_lag_2          47
3         Load_DA_lag_0          42
15       lag168_Load_DA          32
16  roll2d_median_Price          30
13            EUA_lag_2          29
12            Oil_lag_2          23
10           Coal_lag_2          20
5       WindOn_DA_lag_0          20
9                  WD_7           4
7                  WD_1           3
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          40
12            Oil_lag_2          26
11           NGas_lag_2           3
1           Price_lag_2           2
5       WindOn_DA_lag_0           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 28.45% done
********************* END NS.. 207 ****************************************************
********************* START NS ... 208 ****************************************************
[I 2025-06-15 19:56:35,470] A new study created in memory with name: no-name-10eaf929-96e2-4e79-9697-099011ba0015
[I 2025-06-15 19:56:35,888] Trial 0 finished with value: 0.005371890725871535 and parameters: {'n_estimators': 128, 'learning_rate': 0.01946219206759263, 'num_leaves': 37, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.5293347670166695, 'colsample_bytree': 0.5923741219541445, 'reg_alpha': 5.910035289901085, 'reg_lambda': 5.167866914118862, 'min_split_gain': 0.007574712090435837}. Best is trial 0 with value: 0.005371890725871535.
[Trial 0] New best: 0.00537
[I 2025-06-15 19:56:38,350] Trial 1 finished with value: 0.0009317246124228806 and parameters: {'n_estimators': 273, 'learning_rate': 0.012606981367816367, 'num_leaves': 98, 'max_depth': 13, 'min_child_samples': 11, 'subsample': 0.7767785095978623, 'colsample_bytree': 0.8982407080754302, 'reg_alpha': 0.2093435330653981, 'reg_lambda': 4.865304061859296, 'min_split_gain': 0.013187618357263299}. Best is trial 1 with value: 0.0009317246124228806.
[Trial 1] New best: 0.00093
[I 2025-06-15 19:56:38,589] Trial 2 finished with value: 0.00021036805584911068 and parameters: {'n_estimators': 261, 'learning_rate': 0.26671251980369076, 'num_leaves': 46, 'max_depth': 14, 'min_child_samples': 16, 'subsample': 0.5657071631034192, 'colsample_bytree': 0.8001619721977307, 'reg_alpha': 7.371022410533783, 'reg_lambda': 6.084224956720865, 'min_split_gain': 0.03989646500615171}. Best is trial 2 with value: 0.00021036805584911068.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:39,399] Trial 3 finished with value: 0.00015847427053464417 and parameters: {'n_estimators': 221, 'learning_rate': 0.04334675503741306, 'num_leaves': 251, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.9411036503843064, 'colsample_bytree': 0.842946625025356, 'reg_alpha': 9.7655947567209, 'reg_lambda': 0.6931245132550767, 'min_split_gain': 0.014604137477557712}. Best is trial 3 with value: 0.00015847427053464417.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:40,051] Trial 4 finished with value: 0.00013381448793143257 and parameters: {'n_estimators': 171, 'learning_rate': 0.07868654994082726, 'num_leaves': 109, 'max_depth': 8, 'min_child_samples': 17, 'subsample': 0.7180891486947233, 'colsample_bytree': 0.5729256811807786, 'reg_alpha': 2.185474592525476, 'reg_lambda': 4.200932159521098, 'min_split_gain': 0.008722192030785975}. Best is trial 4 with value: 0.00013381448793143257.
[I 2025-06-15 19:56:40,054] A new study created in memory with name: no-name-348c9aa4-6b6e-4514-a1b7-39cbf54cfad7
[I 2025-06-15 19:56:40,078] Trial 0 finished with value: 0.01206955334584836 and parameters: {'n_estimators': 114, 'learning_rate': 0.05439877909014706, 'num_leaves': 67, 'max_depth': 8, 'min_child_samples': 30, 'subsample': 0.6971608819235469, 'colsample_bytree': 0.732811366637399, 'reg_alpha': 2.0563250487396822, 'reg_lambda': 8.317201067474297, 'min_split_gain': 0.03419974598439118}. Best is trial 0 with value: 0.01206955334584836.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1090
1           Price_lag_2         974
14      pct_chg_Load_DA         686
3         Load_DA_lag_0         515
11           NGas_lag_2         515
16  roll2d_median_Price         457
5       WindOn_DA_lag_0         450
2           Price_lag_7         423
15       lag168_Load_DA         418
13            EUA_lag_2         325
10           Coal_lag_2         321
12            Oil_lag_2         221
9                  WD_7          31
7                  WD_1          18
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          14
12            Oil_lag_2           8
1           Price_lag_2           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 28.59% done
********************* END NS.. 208 ****************************************************
********************* START NS ... 209 ****************************************************
[I 2025-06-15 19:56:41,119] A new study created in memory with name: no-name-ab1ec810-ee65-425a-8593-c7a54c3f9ebe
[I 2025-06-15 19:56:41,395] Trial 0 finished with value: 0.0002031228253310867 and parameters: {'n_estimators': 209, 'learning_rate': 0.04809859127973736, 'num_leaves': 247, 'max_depth': 4, 'min_child_samples': 27, 'subsample': 0.5760890966734509, 'colsample_bytree': 0.7254865826742123, 'reg_alpha': 4.140791150433336, 'reg_lambda': 3.679350648221594, 'min_split_gain': 0.015432691109763053}. Best is trial 0 with value: 0.0002031228253310867.
[Trial 0] New best: 0.00020
[I 2025-06-15 19:56:41,925] Trial 1 finished with value: 0.00020567088399448238 and parameters: {'n_estimators': 187, 'learning_rate': 0.051193536892875396, 'num_leaves': 252, 'max_depth': 14, 'min_child_samples': 25, 'subsample': 0.9905383290670104, 'colsample_bytree': 0.9272643164489169, 'reg_alpha': 4.202173240175249, 'reg_lambda': 4.842321388680128, 'min_split_gain': 0.04348141776743158}. Best is trial 0 with value: 0.0002031228253310867.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:44,126] Trial 2 finished with value: 0.0021319389179820965 and parameters: {'n_estimators': 223, 'learning_rate': 0.012914782678430723, 'num_leaves': 190, 'max_depth': 10, 'min_child_samples': 2, 'subsample': 0.8255112562388857, 'colsample_bytree': 0.910997304951323, 'reg_alpha': 0.8789359228138083, 'reg_lambda': 7.079526290356476, 'min_split_gain': 0.02057359207884917}. Best is trial 0 with value: 0.0002031228253310867.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:44,433] Trial 3 finished with value: 0.0002892519065145564 and parameters: {'n_estimators': 215, 'learning_rate': 0.08773896776237886, 'num_leaves': 106, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.5282697684357297, 'colsample_bytree': 0.7859889957736994, 'reg_alpha': 8.232988240219475, 'reg_lambda': 7.076972195476028, 'min_split_gain': 0.03318638631957722}. Best is trial 0 with value: 0.0002031228253310867.
[I 2025-06-15 19:56:44,436] A new study created in memory with name: no-name-1352f517-9b53-4ae1-b999-24339e6280f4
[I 2025-06-15 19:56:44,473] Trial 0 finished with value: 0.0026625337919982573 and parameters: {'n_estimators': 228, 'learning_rate': 0.14785779751806796, 'num_leaves': 183, 'max_depth': 12, 'min_child_samples': 23, 'subsample': 0.7347874887826684, 'colsample_bytree': 0.5475970433046307, 'reg_alpha': 0.4214118644104836, 'reg_lambda': 6.9849732012686445, 'min_split_gain': 0.020344965997926723}. Best is trial 0 with value: 0.0026625337919982573.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         818
1           Price_lag_2         403
14      pct_chg_Load_DA         349
3         Load_DA_lag_0         241
11           NGas_lag_2         183
16  roll2d_median_Price         133
13            EUA_lag_2         121
2           Price_lag_7         115
15       lag168_Load_DA         113
5       WindOn_DA_lag_0          95
12            Oil_lag_2          86
10           Coal_lag_2          79
9                  WD_7          18
7                  WD_1          14
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          11
16  roll2d_median_Price           5
1           Price_lag_2           2
12            Oil_lag_2           1
13            EUA_lag_2           1
2           Price_lag_7           0
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 28.73% done
********************* END NS.. 209 ****************************************************
********************* START NS ... 210 ****************************************************
[I 2025-06-15 19:56:45,284] A new study created in memory with name: no-name-955393b6-469f-41f7-b3fb-3513ca35e7d5
[I 2025-06-15 19:56:45,437] Trial 0 finished with value: 0.07530130820561008 and parameters: {'n_estimators': 62, 'learning_rate': 0.016053197740891124, 'num_leaves': 33, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.6527446009359477, 'colsample_bytree': 0.5732178057963442, 'reg_alpha': 9.106961337059762, 'reg_lambda': 8.182479188263489, 'min_split_gain': 0.033275110897187966}. Best is trial 0 with value: 0.07530130820561008.
[Trial 0] New best: 0.07530
[I 2025-06-15 19:56:45,679] Trial 1 finished with value: 0.0006638664268431726 and parameters: {'n_estimators': 163, 'learning_rate': 0.02877323477873881, 'num_leaves': 173, 'max_depth': 4, 'min_child_samples': 6, 'subsample': 0.6541708861620323, 'colsample_bytree': 0.861962857189787, 'reg_alpha': 3.2119602678705217, 'reg_lambda': 6.791165759009013, 'min_split_gain': 0.03390546845951744}. Best is trial 1 with value: 0.0006638664268431726.
[Trial 1] New best: 0.00066
[I 2025-06-15 19:56:46,864] Trial 2 finished with value: 0.0007417586946724306 and parameters: {'n_estimators': 173, 'learning_rate': 0.026507020414169143, 'num_leaves': 162, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.6295011996710815, 'colsample_bytree': 0.5009243351430992, 'reg_alpha': 0.20777027541376114, 'reg_lambda': 9.48987599306264, 'min_split_gain': 0.049475636592958616}. Best is trial 1 with value: 0.0006638664268431726.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:47,203] Trial 3 finished with value: 0.0006185707697634809 and parameters: {'n_estimators': 277, 'learning_rate': 0.061899864820068554, 'num_leaves': 247, 'max_depth': 5, 'min_child_samples': 3, 'subsample': 0.9486082026706123, 'colsample_bytree': 0.7679936212876275, 'reg_alpha': 3.2345535482736554, 'reg_lambda': 4.405618432887169, 'min_split_gain': 0.041345906713754876}. Best is trial 3 with value: 0.0006185707697634809.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:47,410] Trial 4 finished with value: 0.0011107270343581855 and parameters: {'n_estimators': 74, 'learning_rate': 0.053337846338854626, 'num_leaves': 216, 'max_depth': 6, 'min_child_samples': 19, 'subsample': 0.6372968417015258, 'colsample_bytree': 0.5676080650234284, 'reg_alpha': 8.40712440636738, 'reg_lambda': 6.968870258125147, 'min_split_gain': 0.035838527450569715}. Best is trial 3 with value: 0.0006185707697634809.
[I 2025-06-15 19:56:47,413] A new study created in memory with name: no-name-eddea6e8-a9c4-40ba-a767-746297e3d07f
[I 2025-06-15 19:56:47,441] Trial 0 finished with value: 0.003888208537908777 and parameters: {'n_estimators': 180, 'learning_rate': 0.02593575742805399, 'num_leaves': 175, 'max_depth': 16, 'min_child_samples': 17, 'subsample': 0.6608025985516268, 'colsample_bytree': 0.7402474981929582, 'reg_alpha': 7.38452606217308, 'reg_lambda': 5.855010890677333, 'min_split_gain': 0.018481927533744554}. Best is trial 0 with value: 0.003888208537908777.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         782
1           Price_lag_2         463
14      pct_chg_Load_DA         416
3         Load_DA_lag_0         259
11           NGas_lag_2         239
2           Price_lag_7         174
16  roll2d_median_Price         160
15       lag168_Load_DA         129
10           Coal_lag_2         110
13            EUA_lag_2          89
12            Oil_lag_2          83
5       WindOn_DA_lag_0          77
9                  WD_7          14
7                  WD_1           9
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 28.86% done
********************* END NS.. 210 ****************************************************
********************* START NS ... 211 ****************************************************
[I 2025-06-15 19:56:48,244] A new study created in memory with name: no-name-18ddaa51-e504-45ec-8542-3c3e9dfa17d5
[I 2025-06-15 19:56:48,725] Trial 0 finished with value: 0.007177387465823454 and parameters: {'n_estimators': 119, 'learning_rate': 0.022042716573745225, 'num_leaves': 140, 'max_depth': 10, 'min_child_samples': 3, 'subsample': 0.8944462903769765, 'colsample_bytree': 0.65762145110418, 'reg_alpha': 2.698162284858242, 'reg_lambda': 4.344265769287157, 'min_split_gain': 0.038793346538747525}. Best is trial 0 with value: 0.007177387465823454.
[Trial 0] New best: 0.00718
[I 2025-06-15 19:56:49,115] Trial 1 finished with value: 0.0686223476689689 and parameters: {'n_estimators': 83, 'learning_rate': 0.013575440170334687, 'num_leaves': 193, 'max_depth': 15, 'min_child_samples': 22, 'subsample': 0.7129154718599349, 'colsample_bytree': 0.7170576725793683, 'reg_alpha': 1.845959822732196, 'reg_lambda': 8.470299783177849, 'min_split_gain': 0.01058671558297612}. Best is trial 0 with value: 0.007177387465823454.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:49,511] Trial 2 finished with value: 0.0026010510046980866 and parameters: {'n_estimators': 130, 'learning_rate': 0.05676415223404962, 'num_leaves': 32, 'max_depth': 11, 'min_child_samples': 25, 'subsample': 0.868911744050308, 'colsample_bytree': 0.5887793216122921, 'reg_alpha': 4.098787757488331, 'reg_lambda': 5.952637419004096, 'min_split_gain': 0.0006057844397109879}. Best is trial 2 with value: 0.0026010510046980866.
[Trial 2] New best: 0.00260
[I 2025-06-15 19:56:50,100] Trial 3 finished with value: 0.006800962367696804 and parameters: {'n_estimators': 208, 'learning_rate': 0.014840154288666849, 'num_leaves': 249, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.7211903461854093, 'colsample_bytree': 0.511793198629068, 'reg_alpha': 7.93069142373059, 'reg_lambda': 4.833155177938889, 'min_split_gain': 0.03905652256798401}. Best is trial 2 with value: 0.0026010510046980866.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:50,342] Trial 4 finished with value: 0.0021995049651592396 and parameters: {'n_estimators': 227, 'learning_rate': 0.16332587222146444, 'num_leaves': 59, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.5008093911147371, 'colsample_bytree': 0.6472424970789789, 'reg_alpha': 4.248428114507444, 'reg_lambda': 5.939803068541249, 'min_split_gain': 0.025979724945557}. Best is trial 4 with value: 0.0021995049651592396.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:50,714] Trial 5 finished with value: 0.0023559713723676624 and parameters: {'n_estimators': 74, 'learning_rate': 0.12177521093817995, 'num_leaves': 166, 'max_depth': 15, 'min_child_samples': 12, 'subsample': 0.7292240566271776, 'colsample_bytree': 0.6046364719357952, 'reg_alpha': 5.713160415345545, 'reg_lambda': 4.514693487300699, 'min_split_gain': 0.025202314647033375}. Best is trial 4 with value: 0.0021995049651592396.
[I 2025-06-15 19:56:50,717] A new study created in memory with name: no-name-1fb585f5-017c-413c-8ec6-ad7bcdcf5684
[I 2025-06-15 19:56:50,786] Trial 0 finished with value: 0.017140109095708886 and parameters: {'n_estimators': 272, 'learning_rate': 0.11346057271819754, 'num_leaves': 121, 'max_depth': 4, 'min_child_samples': 27, 'subsample': 0.580220095439562, 'colsample_bytree': 0.5285797628085638, 'reg_alpha': 7.337030993064218, 'reg_lambda': 7.537384127667421, 'min_split_gain': 0.0026939582725360347}. Best is trial 0 with value: 0.017140109095708886.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         334
1           Price_lag_2         234
14      pct_chg_Load_DA         163
11           NGas_lag_2         125
3         Load_DA_lag_0         115
2           Price_lag_7         110
15       lag168_Load_DA         104
16  roll2d_median_Price          93
13            EUA_lag_2          85
5       WindOn_DA_lag_0          81
10           Coal_lag_2          70
12            Oil_lag_2          48
9                  WD_7           9
7                  WD_1           7
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 29.00% done
********************* END NS.. 211 ****************************************************
********************* START NS ... 212 ****************************************************
[I 2025-06-15 19:56:51,488] A new study created in memory with name: no-name-5d50266b-ca48-4247-b402-c98c9b4f5d60
[I 2025-06-15 19:56:51,993] Trial 0 finished with value: 0.0015604823475178292 and parameters: {'n_estimators': 194, 'learning_rate': 0.1316391766043896, 'num_leaves': 149, 'max_depth': 12, 'min_child_samples': 2, 'subsample': 0.8774668898661908, 'colsample_bytree': 0.7200220154949097, 'reg_alpha': 0.6375883907861624, 'reg_lambda': 5.602224931201426, 'min_split_gain': 0.009604704325720954}. Best is trial 0 with value: 0.0015604823475178292.
[Trial 0] New best: 0.00156
[I 2025-06-15 19:56:52,214] Trial 1 finished with value: 0.004784474430934448 and parameters: {'n_estimators': 75, 'learning_rate': 0.037737244430325514, 'num_leaves': 95, 'max_depth': 7, 'min_child_samples': 17, 'subsample': 0.5586618247814958, 'colsample_bytree': 0.6386376452444793, 'reg_alpha': 9.803794041196877, 'reg_lambda': 4.76433302471742, 'min_split_gain': 0.017062391362843488}. Best is trial 0 with value: 0.0015604823475178292.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:52,563] Trial 2 finished with value: 0.0038916255481370628 and parameters: {'n_estimators': 175, 'learning_rate': 0.02087425771058247, 'num_leaves': 89, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.969016405024287, 'colsample_bytree': 0.5266085878161435, 'reg_alpha': 8.951448062083406, 'reg_lambda': 5.311448327943758, 'min_split_gain': 0.03715198968573239}. Best is trial 0 with value: 0.0015604823475178292.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:56:53,103] Trial 3 finished with value: 0.002956632617760184 and parameters: {'n_estimators': 196, 'learning_rate': 0.022102426889115425, 'num_leaves': 113, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.853376760478313, 'colsample_bytree': 0.5669112679857762, 'reg_alpha': 7.684518979559228, 'reg_lambda': 6.459006199188979, 'min_split_gain': 0.033027623408513083}. Best is trial 0 with value: 0.0015604823475178292.
[I 2025-06-15 19:56:53,105] A new study created in memory with name: no-name-2975371a-c2a1-42b8-a371-2d72820ae092
[I 2025-06-15 19:56:53,138] Trial 0 finished with value: 0.01478636165294342 and parameters: {'n_estimators': 250, 'learning_rate': 0.036234558291890956, 'num_leaves': 154, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.928745531516652, 'colsample_bytree': 0.567754840590733, 'reg_alpha': 6.64182869365599, 'reg_lambda': 6.7658381021635, 'min_split_gain': 0.002413282525527283}. Best is trial 0 with value: 0.01478636165294342.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA        1099
3         Load_DA_lag_0        1053
1           Price_lag_2         998
0           Price_lag_1         851
5       WindOn_DA_lag_0         717
2           Price_lag_7         666
15       lag168_Load_DA         644
16  roll2d_median_Price         504
11           NGas_lag_2         447
10           Coal_lag_2         435
12            Oil_lag_2         414
13            EUA_lag_2         408
9                  WD_7          34
7                  WD_1          24
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 29.14% done
********************* END NS.. 212 ****************************************************
********************* START NS ... 213 ****************************************************
[I 2025-06-15 19:56:54,094] A new study created in memory with name: no-name-99e1233a-2dad-4122-ae57-44e193564777
[I 2025-06-15 19:56:54,361] Trial 0 finished with value: 0.018905868127812902 and parameters: {'n_estimators': 68, 'learning_rate': 0.023413059250407098, 'num_leaves': 68, 'max_depth': 16, 'min_child_samples': 26, 'subsample': 0.576413034470874, 'colsample_bytree': 0.9367133136817397, 'reg_alpha': 1.4530623964002476, 'reg_lambda': 9.988225506331599, 'min_split_gain': 0.04526189507358932}. Best is trial 0 with value: 0.018905868127812902.
[I 2025-06-15 19:56:54,558] Trial 1 finished with value: 0.00015873113058842677 and parameters: {'n_estimators': 251, 'learning_rate': 0.09474806871997225, 'num_leaves': 192, 'max_depth': 4, 'min_child_samples': 28, 'subsample': 0.689994080450871, 'colsample_bytree': 0.877306427227826, 'reg_alpha': 6.096254545394668, 'reg_lambda': 1.2241765534130011, 'min_split_gain': 0.046756379493491165}. Best is trial 1 with value: 0.00015873113058842677.
[Trial 0] New best: 0.01891
[I 2025-06-15 19:56:54,776] Trial 2 finished with value: 0.0001750111165476532 and parameters: {'n_estimators': 65, 'learning_rate': 0.11945621786221655, 'num_leaves': 68, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.9563932129778568, 'colsample_bytree': 0.8814035945487773, 'reg_alpha': 7.046842050607055, 'reg_lambda': 6.969784582678305, 'min_split_gain': 0.02352329793004105}. Best is trial 1 with value: 0.00015873113058842677.
[Trial 1] New best: 0.00016
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:55,012] Trial 3 finished with value: 0.00018537546509929335 and parameters: {'n_estimators': 181, 'learning_rate': 0.10547939402282192, 'num_leaves': 82, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.7458525505443798, 'colsample_bytree': 0.8731682916967585, 'reg_alpha': 4.676179289711669, 'reg_lambda': 2.8783045648600614, 'min_split_gain': 0.02938132321505024}. Best is trial 1 with value: 0.00015873113058842677.
[I 2025-06-15 19:56:55,178] Trial 4 finished with value: 0.041382240656074244 and parameters: {'n_estimators': 54, 'learning_rate': 0.02221514642459365, 'num_leaves': 71, 'max_depth': 5, 'min_child_samples': 14, 'subsample': 0.5789361780077147, 'colsample_bytree': 0.8506510974080324, 'reg_alpha': 7.9700549367507065, 'reg_lambda': 1.5260431837871313, 'min_split_gain': 0.00119512057753583}. Best is trial 1 with value: 0.00015873113058842677.
[I 2025-06-15 19:56:55,181] A new study created in memory with name: no-name-fb438d5b-da8f-4d9c-9ce3-ad29ee4ad4b8
[I 2025-06-15 19:56:55,198] Trial 0 finished with value: 0.005002623229429755 and parameters: {'n_estimators': 56, 'learning_rate': 0.012201675843167701, 'num_leaves': 117, 'max_depth': 15, 'min_child_samples': 28, 'subsample': 0.5836017340564027, 'colsample_bytree': 0.5426686379673957, 'reg_alpha': 8.115092891466823, 'reg_lambda': 8.63433175178696, 'min_split_gain': 0.009593957427723749}. Best is trial 0 with value: 0.005002623229429755.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         375
14      pct_chg_Load_DA         178
1           Price_lag_2         114
3         Load_DA_lag_0          97
11           NGas_lag_2          81
2           Price_lag_7          59
16  roll2d_median_Price          47
15       lag168_Load_DA          36
12            Oil_lag_2          34
13            EUA_lag_2          32
10           Coal_lag_2          29
5       WindOn_DA_lag_0           7
7                  WD_1           3
9                  WD_7           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 29.27% done
********************* END NS.. 213 ****************************************************
********************* START NS ... 214 ****************************************************
[I 2025-06-15 19:56:55,891] A new study created in memory with name: no-name-ce6798be-e71b-40a6-be5c-1eb8e54cbe8a
[I 2025-06-15 19:56:56,283] Trial 0 finished with value: 0.0022839909214761177 and parameters: {'n_estimators': 202, 'learning_rate': 0.014116790838665306, 'num_leaves': 201, 'max_depth': 5, 'min_child_samples': 9, 'subsample': 0.5064094621178197, 'colsample_bytree': 0.6533770076350116, 'reg_alpha': 0.3248200345391805, 'reg_lambda': 6.8640835242230205, 'min_split_gain': 0.04737370449798743}. Best is trial 0 with value: 0.0022839909214761177.
[Trial 0] New best: 0.00228
[I 2025-06-15 19:56:56,632] Trial 1 finished with value: 0.012705390009856031 and parameters: {'n_estimators': 88, 'learning_rate': 0.020698215802224362, 'num_leaves': 190, 'max_depth': 12, 'min_child_samples': 6, 'subsample': 0.5669546629797737, 'colsample_bytree': 0.9910338771144734, 'reg_alpha': 3.2810813060550004, 'reg_lambda': 8.199953430368403, 'min_split_gain': 0.031809290471030725}. Best is trial 0 with value: 0.0022839909214761177.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:57,934] Trial 2 finished with value: 0.00071957832684607 and parameters: {'n_estimators': 278, 'learning_rate': 0.013164213936291747, 'num_leaves': 166, 'max_depth': 15, 'min_child_samples': 11, 'subsample': 0.8610178889913302, 'colsample_bytree': 0.7091999931748076, 'reg_alpha': 4.7740632808071926, 'reg_lambda': 5.6373692459244555, 'min_split_gain': 0.019566933638232767}. Best is trial 2 with value: 0.00071957832684607.
[Trial 2] New best: 0.00072
[I 2025-06-15 19:56:58,274] Trial 3 finished with value: 0.004887212438803157 and parameters: {'n_estimators': 88, 'learning_rate': 0.026238667883430815, 'num_leaves': 179, 'max_depth': 7, 'min_child_samples': 3, 'subsample': 0.5650757342703105, 'colsample_bytree': 0.7282498038401154, 'reg_alpha': 9.116773268096289, 'reg_lambda': 7.995508452802853, 'min_split_gain': 0.031744140733430626}. Best is trial 2 with value: 0.00071957832684607.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:56:59,250] Trial 4 finished with value: 0.0003120459813171556 and parameters: {'n_estimators': 237, 'learning_rate': 0.02200878122172642, 'num_leaves': 47, 'max_depth': 10, 'min_child_samples': 3, 'subsample': 0.9496470271541368, 'colsample_bytree': 0.555047588318254, 'reg_alpha': 0.06816411045577953, 'reg_lambda': 6.852167020632562, 'min_split_gain': 0.040464570876085126}. Best is trial 4 with value: 0.0003120459813171556.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:00,005] Trial 5 finished with value: 0.00045356974703665397 and parameters: {'n_estimators': 247, 'learning_rate': 0.01674865552566589, 'num_leaves': 50, 'max_depth': 16, 'min_child_samples': 15, 'subsample': 0.9386718405216791, 'colsample_bytree': 0.6494280071279595, 'reg_alpha': 9.182113495711723, 'reg_lambda': 0.47328268623855085, 'min_split_gain': 0.011831178806908744}. Best is trial 4 with value: 0.0003120459813171556.
[I 2025-06-15 19:57:00,007] A new study created in memory with name: no-name-c5d4e111-939c-4fbb-bba6-8de2ce9b0734
[I 2025-06-15 19:57:00,054] Trial 0 finished with value: 0.002099711198543453 and parameters: {'n_estimators': 271, 'learning_rate': 0.01832161181168328, 'num_leaves': 80, 'max_depth': 10, 'min_child_samples': 28, 'subsample': 0.9040845592156543, 'colsample_bytree': 0.6942640417610872, 'reg_alpha': 6.752076843075832, 'reg_lambda': 4.268346250261271, 'min_split_gain': 0.008221846471484667}. Best is trial 0 with value: 0.002099711198543453.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        2103
0           Price_lag_1        2069
14      pct_chg_Load_DA        1030
3         Load_DA_lag_0         938
2           Price_lag_7         763
16  roll2d_median_Price         704
11           NGas_lag_2         625
15       lag168_Load_DA         595
10           Coal_lag_2         583
5       WindOn_DA_lag_0         495
13            EUA_lag_2         465
12            Oil_lag_2         421
9                  WD_7          55
7                  WD_1          42
8                  WD_6          14
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 29.41% done
********************* END NS.. 214 ****************************************************
********************* START NS ... 215 ****************************************************
[I 2025-06-15 19:57:01,384] A new study created in memory with name: no-name-7812e6a6-148a-44a2-b68a-6aa568676ecd
[I 2025-06-15 19:57:01,723] Trial 0 finished with value: 0.0024570354752907154 and parameters: {'n_estimators': 163, 'learning_rate': 0.13168250948577256, 'num_leaves': 115, 'max_depth': 15, 'min_child_samples': 29, 'subsample': 0.5009616783954441, 'colsample_bytree': 0.6355506191969837, 'reg_alpha': 0.2466306768941151, 'reg_lambda': 3.9464193663027314, 'min_split_gain': 0.028719155003756536}. Best is trial 0 with value: 0.0024570354752907154.
[Trial 0] New best: 0.00246
[I 2025-06-15 19:57:02,283] Trial 1 finished with value: 0.0020523200207346846 and parameters: {'n_estimators': 219, 'learning_rate': 0.15825884145563904, 'num_leaves': 57, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.789909052850684, 'colsample_bytree': 0.7456605686787856, 'reg_alpha': 0.10513483888069719, 'reg_lambda': 9.513623355812978, 'min_split_gain': 0.006531632217941014}. Best is trial 1 with value: 0.0020523200207346846.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:03,819] Trial 2 finished with value: 0.0023791952334629948 and parameters: {'n_estimators': 172, 'learning_rate': 0.031055343999470485, 'num_leaves': 255, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.8986571626858928, 'colsample_bytree': 0.7358646463492523, 'reg_alpha': 1.105415501184902, 'reg_lambda': 5.0121600139933165, 'min_split_gain': 0.021765496340064194}. Best is trial 1 with value: 0.0020523200207346846.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:04,066] Trial 3 finished with value: 0.019219570798266767 and parameters: {'n_estimators': 50, 'learning_rate': 0.03457190078467392, 'num_leaves': 134, 'max_depth': 5, 'min_child_samples': 3, 'subsample': 0.8891957778995265, 'colsample_bytree': 0.970772271218007, 'reg_alpha': 9.71098841817499, 'reg_lambda': 8.391859971568982, 'min_split_gain': 0.04304331455269467}. Best is trial 1 with value: 0.0020523200207346846.
[I 2025-06-15 19:57:04,068] A new study created in memory with name: no-name-b67d41f6-4dc7-4fd8-86b3-481b6f8d4c9c
[I 2025-06-15 19:57:04,116] Trial 0 finished with value: 0.004867218119029787 and parameters: {'n_estimators': 239, 'learning_rate': 0.035349481033685376, 'num_leaves': 108, 'max_depth': 12, 'min_child_samples': 3, 'subsample': 0.5253717034592922, 'colsample_bytree': 0.7827658400868551, 'reg_alpha': 3.6788319510616962, 'reg_lambda': 8.188779158296926, 'min_split_gain': 0.04931664735505384}. Best is trial 0 with value: 0.004867218119029787.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         762
1           Price_lag_2         680
14      pct_chg_Load_DA         677
3         Load_DA_lag_0         583
5       WindOn_DA_lag_0         493
15       lag168_Load_DA         467
2           Price_lag_7         411
16  roll2d_median_Price         327
11           NGas_lag_2         324
13            EUA_lag_2         268
10           Coal_lag_2         221
12            Oil_lag_2         220
9                  WD_7          22
7                  WD_1          16
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 29.55% done
********************* END NS.. 215 ****************************************************
********************* START NS ... 216 ****************************************************
[I 2025-06-15 19:57:05,221] A new study created in memory with name: no-name-21edde40-4b63-42bc-9962-ffbb62416184
[I 2025-06-15 19:57:06,326] Trial 0 finished with value: 0.004475100975793866 and parameters: {'n_estimators': 291, 'learning_rate': 0.010478074208756329, 'num_leaves': 242, 'max_depth': 11, 'min_child_samples': 17, 'subsample': 0.9520357560499699, 'colsample_bytree': 0.7571031888104687, 'reg_alpha': 9.555534694248347, 'reg_lambda': 6.881266304250683, 'min_split_gain': 0.046946950841059015}. Best is trial 0 with value: 0.004475100975793866.
[Trial 0] New best: 0.00448
[I 2025-06-15 19:57:06,607] Trial 1 finished with value: 0.002073927959936337 and parameters: {'n_estimators': 56, 'learning_rate': 0.12675223494887874, 'num_leaves': 95, 'max_depth': 12, 'min_child_samples': 28, 'subsample': 0.6280617387069711, 'colsample_bytree': 0.6493724740417768, 'reg_alpha': 6.885553297249326, 'reg_lambda': 2.6989595030544624, 'min_split_gain': 0.0027727778783287805}. Best is trial 1 with value: 0.002073927959936337.
[I 2025-06-15 19:57:06,793] Trial 2 finished with value: 0.0022956645926105117 and parameters: {'n_estimators': 100, 'learning_rate': 0.2608956999747921, 'num_leaves': 113, 'max_depth': 10, 'min_child_samples': 2, 'subsample': 0.723904350965239, 'colsample_bytree': 0.7283407745718438, 'reg_alpha': 4.293763808351406, 'reg_lambda': 7.4351417809742575, 'min_split_gain': 0.04888257098928548}. Best is trial 1 with value: 0.002073927959936337.
[Trial 1] New best: 0.00207
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:07,179] Trial 3 finished with value: 0.01829678426002866 and parameters: {'n_estimators': 161, 'learning_rate': 0.01159519729467698, 'num_leaves': 58, 'max_depth': 4, 'min_child_samples': 14, 'subsample': 0.9436906973673513, 'colsample_bytree': 0.6883100605735061, 'reg_alpha': 2.5588019177439203, 'reg_lambda': 1.4510218310823386, 'min_split_gain': 0.020164372526416764}. Best is trial 1 with value: 0.002073927959936337.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:07,599] Trial 4 finished with value: 0.0023712735696891185 and parameters: {'n_estimators': 133, 'learning_rate': 0.04202057502278764, 'num_leaves': 250, 'max_depth': 12, 'min_child_samples': 21, 'subsample': 0.5556799882615076, 'colsample_bytree': 0.7599222692243142, 'reg_alpha': 8.8213729265596, 'reg_lambda': 6.32802438416349, 'min_split_gain': 0.03495052243512681}. Best is trial 1 with value: 0.002073927959936337.
[I 2025-06-15 19:57:07,602] A new study created in memory with name: no-name-4024b751-638e-49c5-a7c2-2a06cc02d200
[I 2025-06-15 19:57:07,628] Trial 0 finished with value: 0.0024992337940124075 and parameters: {'n_estimators': 156, 'learning_rate': 0.09033098131463527, 'num_leaves': 41, 'max_depth': 11, 'min_child_samples': 18, 'subsample': 0.8782494369365088, 'colsample_bytree': 0.6011699926001969, 'reg_alpha': 0.46173915663101583, 'reg_lambda': 4.149167229813395, 'min_split_gain': 0.03702966599213655}. Best is trial 0 with value: 0.0024992337940124075.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         404
1           Price_lag_2         300
14      pct_chg_Load_DA         256
3         Load_DA_lag_0         204
2           Price_lag_7         134
13            EUA_lag_2         133
11           NGas_lag_2         122
15       lag168_Load_DA         119
16  roll2d_median_Price         110
5       WindOn_DA_lag_0         106
10           Coal_lag_2         102
12            Oil_lag_2          82
9                  WD_7          14
7                  WD_1           3
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          25
1           Price_lag_2           5
15       lag168_Load_DA           4
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 29.69% done
********************* END NS.. 216 ****************************************************
********************* START NS ... 217 ****************************************************
[I 2025-06-15 19:57:08,379] A new study created in memory with name: no-name-c2ad5cff-a91d-4c10-9009-ca36b934537e
[I 2025-06-15 19:57:08,821] Trial 0 finished with value: 0.0007164431972885541 and parameters: {'n_estimators': 154, 'learning_rate': 0.029123302548037733, 'num_leaves': 167, 'max_depth': 9, 'min_child_samples': 14, 'subsample': 0.8824737479564029, 'colsample_bytree': 0.5982853298145179, 'reg_alpha': 5.604948752275256, 'reg_lambda': 7.947221865730635, 'min_split_gain': 0.047418661125328644}. Best is trial 0 with value: 0.0007164431972885541.
[Trial 0] New best: 0.00072
[I 2025-06-15 19:57:09,452] Trial 1 finished with value: 0.0006510418313087114 and parameters: {'n_estimators': 270, 'learning_rate': 0.020337207890027565, 'num_leaves': 185, 'max_depth': 6, 'min_child_samples': 3, 'subsample': 0.902108437372672, 'colsample_bytree': 0.5635340180713369, 'reg_alpha': 9.85013127421314, 'reg_lambda': 2.063277272864136, 'min_split_gain': 0.029565130331123665}. Best is trial 1 with value: 0.0006510418313087114.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:09,701] Trial 2 finished with value: 0.0007900364059585606 and parameters: {'n_estimators': 113, 'learning_rate': 0.24081734231613042, 'num_leaves': 158, 'max_depth': 13, 'min_child_samples': 2, 'subsample': 0.5224229630395678, 'colsample_bytree': 0.5386692440385499, 'reg_alpha': 0.901101980564103, 'reg_lambda': 7.372755070719043, 'min_split_gain': 0.046803890161089184}. Best is trial 1 with value: 0.0006510418313087114.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:10,290] Trial 3 finished with value: 0.0003300271238605207 and parameters: {'n_estimators': 122, 'learning_rate': 0.053061825250716226, 'num_leaves': 177, 'max_depth': 13, 'min_child_samples': 26, 'subsample': 0.8311380147013044, 'colsample_bytree': 0.8712652347317433, 'reg_alpha': 3.022697904263701, 'reg_lambda': 1.4197113300391428, 'min_split_gain': 0.022275940987237244}. Best is trial 3 with value: 0.0003300271238605207.
[I 2025-06-15 19:57:10,293] A new study created in memory with name: no-name-99cfdb9d-9cde-4a4e-ae73-cbdbe5a72978
[I 2025-06-15 19:57:10,355] Trial 0 finished with value: 0.0007646330616309871 and parameters: {'n_estimators': 290, 'learning_rate': 0.01642518382878304, 'num_leaves': 125, 'max_depth': 6, 'min_child_samples': 24, 'subsample': 0.5693429598423814, 'colsample_bytree': 0.9512579365118381, 'reg_alpha': 0.5264216959252155, 'reg_lambda': 4.9531999829391005, 'min_split_gain': 0.005282763702611182}. Best is trial 0 with value: 0.0007646330616309871.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1162
1           Price_lag_2         839
14      pct_chg_Load_DA         773
3         Load_DA_lag_0         567
11           NGas_lag_2         396
2           Price_lag_7         333
16  roll2d_median_Price         291
15       lag168_Load_DA         287
10           Coal_lag_2         182
5       WindOn_DA_lag_0         179
12            Oil_lag_2         148
13            EUA_lag_2         141
9                  WD_7          20
7                  WD_1          15
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         303
15       lag168_Load_DA          55
16  roll2d_median_Price          11
11           NGas_lag_2           8
1           Price_lag_2           7
12            Oil_lag_2           5
5       WindOn_DA_lag_0           2
13            EUA_lag_2           2
3         Load_DA_lag_0           1
9                  WD_7           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 29.82% done
********************* END NS.. 217 ****************************************************
********************* START NS ... 218 ****************************************************
[I 2025-06-15 19:57:11,438] A new study created in memory with name: no-name-c127915f-2c1b-489e-a656-9d1f8898d790
[I 2025-06-15 19:57:11,649] Trial 0 finished with value: 0.00045618579092540987 and parameters: {'n_estimators': 196, 'learning_rate': 0.14101381081981437, 'num_leaves': 204, 'max_depth': 9, 'min_child_samples': 6, 'subsample': 0.8930470555232164, 'colsample_bytree': 0.6197263172926185, 'reg_alpha': 8.899481284439734, 'reg_lambda': 0.2889606748684792, 'min_split_gain': 0.027948458067331185}. Best is trial 0 with value: 0.00045618579092540987.
[Trial 0] New best: 0.00046
[I 2025-06-15 19:57:12,974] Trial 1 finished with value: 0.000450041570790973 and parameters: {'n_estimators': 256, 'learning_rate': 0.018732078400624293, 'num_leaves': 184, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.9659685538917252, 'colsample_bytree': 0.7325103862174712, 'reg_alpha': 1.5447334379968969, 'reg_lambda': 4.6349002925539295, 'min_split_gain': 0.025390271259060317}. Best is trial 1 with value: 0.000450041570790973.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:14,120] Trial 2 finished with value: 0.002379177202870007 and parameters: {'n_estimators': 235, 'learning_rate': 0.012551562893271017, 'num_leaves': 86, 'max_depth': 8, 'min_child_samples': 1, 'subsample': 0.5719128511552038, 'colsample_bytree': 0.578439467575147, 'reg_alpha': 5.163575308367817, 'reg_lambda': 6.9349988972865715, 'min_split_gain': 0.036874139363579854}. Best is trial 1 with value: 0.000450041570790973.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:14,358] Trial 3 finished with value: 0.0781480796714192 and parameters: {'n_estimators': 66, 'learning_rate': 0.015400990359132366, 'num_leaves': 85, 'max_depth': 15, 'min_child_samples': 16, 'subsample': 0.7043392399871278, 'colsample_bytree': 0.9896086615706319, 'reg_alpha': 9.27003779032591, 'reg_lambda': 9.677700399686467, 'min_split_gain': 0.010035907171977766}. Best is trial 1 with value: 0.000450041570790973.
[I 2025-06-15 19:57:14,361] A new study created in memory with name: no-name-ccd806fd-e2ed-4d98-8e17-a7c42d00196f
[I 2025-06-15 19:57:14,377] Trial 0 finished with value: 0.011464603587699218 and parameters: {'n_estimators': 68, 'learning_rate': 0.2973832477733436, 'num_leaves': 118, 'max_depth': 7, 'min_child_samples': 17, 'subsample': 0.5300103293121723, 'colsample_bytree': 0.9669445454770578, 'reg_alpha': 2.3017663442837764, 'reg_lambda': 7.6063166097657415, 'min_split_gain': 0.04303624489782441}. Best is trial 0 with value: 0.011464603587699218.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3337
1           Price_lag_2        3190
14      pct_chg_Load_DA        1962
3         Load_DA_lag_0        1077
11           NGas_lag_2         903
16  roll2d_median_Price         874
2           Price_lag_7         846
15       lag168_Load_DA         676
5       WindOn_DA_lag_0         514
10           Coal_lag_2         505
12            Oil_lag_2         417
13            EUA_lag_2         344
9                  WD_7          41
7                  WD_1          18
8                  WD_6          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           3
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 29.96% done
********************* END NS.. 218 ****************************************************
********************* START NS ... 219 ****************************************************
[I 2025-06-15 19:57:15,689] A new study created in memory with name: no-name-b1021608-078b-4606-83ab-1d589d6bf551
[I 2025-06-15 19:57:15,930] Trial 0 finished with value: 0.003944288517450516 and parameters: {'n_estimators': 164, 'learning_rate': 0.11758681975287874, 'num_leaves': 119, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.8455042663991655, 'colsample_bytree': 0.7808018980413022, 'reg_alpha': 1.1549766235107095, 'reg_lambda': 7.339373126410162, 'min_split_gain': 0.048218552572106055}. Best is trial 0 with value: 0.003944288517450516.
[Trial 0] New best: 0.00394
[I 2025-06-15 19:57:16,611] Trial 1 finished with value: 0.0026975636987594995 and parameters: {'n_estimators': 212, 'learning_rate': 0.05688334834995091, 'num_leaves': 135, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.8098663758021192, 'colsample_bytree': 0.5086583800106155, 'reg_alpha': 6.1008146658998, 'reg_lambda': 0.13728973595907745, 'min_split_gain': 0.012956696446531753}. Best is trial 1 with value: 0.0026975636987594995.
[Trial 1] New best: 0.00270
[I 2025-06-15 19:57:17,093] Trial 2 finished with value: 0.0046534390422430846 and parameters: {'n_estimators': 159, 'learning_rate': 0.02823258322772581, 'num_leaves': 64, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.6247999353052425, 'colsample_bytree': 0.8035302152507642, 'reg_alpha': 6.408498158561028, 'reg_lambda': 9.184125858895452, 'min_split_gain': 0.039254875561150156}. Best is trial 1 with value: 0.0026975636987594995.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:17,707] Trial 3 finished with value: 0.004265114983660432 and parameters: {'n_estimators': 244, 'learning_rate': 0.01762018923043144, 'num_leaves': 175, 'max_depth': 10, 'min_child_samples': 15, 'subsample': 0.9747103147993146, 'colsample_bytree': 0.5628177852815699, 'reg_alpha': 8.673184911933879, 'reg_lambda': 2.839890078751306, 'min_split_gain': 0.047675296735526035}. Best is trial 1 with value: 0.0026975636987594995.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:18,157] Trial 4 finished with value: 0.012672149208311787 and parameters: {'n_estimators': 103, 'learning_rate': 0.023976264964009373, 'num_leaves': 185, 'max_depth': 8, 'min_child_samples': 2, 'subsample': 0.8389876315550383, 'colsample_bytree': 0.8150101542063419, 'reg_alpha': 4.624332224538504, 'reg_lambda': 3.4372403134538896, 'min_split_gain': 0.000742207068314843}. Best is trial 1 with value: 0.0026975636987594995.
[I 2025-06-15 19:57:18,159] A new study created in memory with name: no-name-84bf2f2c-7b97-4dc9-aa28-d1d16beb768c
[I 2025-06-15 19:57:18,189] Trial 0 finished with value: 0.02409834474233183 and parameters: {'n_estimators': 297, 'learning_rate': 0.0847677846825381, 'num_leaves': 152, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.5454197161819689, 'colsample_bytree': 0.9059047279734369, 'reg_alpha': 6.155067601544277, 'reg_lambda': 0.30904820415023626, 'min_split_gain': 0.0020836254856307023}. Best is trial 0 with value: 0.02409834474233183.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1012
1           Price_lag_2         866
14      pct_chg_Load_DA         565
2           Price_lag_7         408
3         Load_DA_lag_0         390
11           NGas_lag_2         357
13            EUA_lag_2         331
15       lag168_Load_DA         309
5       WindOn_DA_lag_0         277
16  roll2d_median_Price         266
10           Coal_lag_2         216
12            Oil_lag_2         160
9                  WD_7          29
7                  WD_1          27
8                  WD_6          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 30.10% done
********************* END NS.. 219 ****************************************************
********************* START NS ... 220 ****************************************************
[I 2025-06-15 19:57:19,142] A new study created in memory with name: no-name-284309d6-fbea-4eb4-a3eb-7e8d4addc474
[I 2025-06-15 19:57:19,474] Trial 0 finished with value: 0.005118124367425107 and parameters: {'n_estimators': 271, 'learning_rate': 0.047113526113177866, 'num_leaves': 202, 'max_depth': 5, 'min_child_samples': 25, 'subsample': 0.6972239722238716, 'colsample_bytree': 0.5554525882484147, 'reg_alpha': 9.99902753358993, 'reg_lambda': 4.583231657067226, 'min_split_gain': 0.014963997803351793}. Best is trial 0 with value: 0.005118124367425107.
[I 2025-06-15 19:57:19,646] Trial 1 finished with value: 0.004970080876511182 and parameters: {'n_estimators': 105, 'learning_rate': 0.13537894465849873, 'num_leaves': 53, 'max_depth': 9, 'min_child_samples': 23, 'subsample': 0.9000438924817076, 'colsample_bytree': 0.8250601397936224, 'reg_alpha': 4.165630749368875, 'reg_lambda': 1.2910391737317317, 'min_split_gain': 0.04844530474713957}. Best is trial 1 with value: 0.004970080876511182.
[Trial 0] New best: 0.00512
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:20,086] Trial 2 finished with value: 0.00998767285085768 and parameters: {'n_estimators': 96, 'learning_rate': 0.03187032080540149, 'num_leaves': 163, 'max_depth': 10, 'min_child_samples': 27, 'subsample': 0.781202382495336, 'colsample_bytree': 0.5490891529684114, 'reg_alpha': 2.021012171715096, 'reg_lambda': 4.836072253593491, 'min_split_gain': 0.0038612187041827174}. Best is trial 1 with value: 0.004970080876511182.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:20,561] Trial 3 finished with value: 0.005923437013303918 and parameters: {'n_estimators': 266, 'learning_rate': 0.0386211852393429, 'num_leaves': 123, 'max_depth': 11, 'min_child_samples': 25, 'subsample': 0.8443041615571911, 'colsample_bytree': 0.9776585918653918, 'reg_alpha': 9.056258849476968, 'reg_lambda': 4.012656218489312, 'min_split_gain': 0.035623747030345224}. Best is trial 1 with value: 0.004970080876511182.
[I 2025-06-15 19:57:20,564] A new study created in memory with name: no-name-3d98a757-0e3e-491a-b803-ce5126a1ef59
[I 2025-06-15 19:57:20,581] Trial 0 finished with value: 0.05140096340128608 and parameters: {'n_estimators': 103, 'learning_rate': 0.2702183863100348, 'num_leaves': 230, 'max_depth': 11, 'min_child_samples': 28, 'subsample': 0.841383883572602, 'colsample_bytree': 0.9891619969293421, 'reg_alpha': 4.891745551683641, 'reg_lambda': 9.240346122622647, 'min_split_gain': 0.028477706450024665}. Best is trial 0 with value: 0.05140096340128608.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         368
1           Price_lag_2         246
14      pct_chg_Load_DA         210
3         Load_DA_lag_0         143
11           NGas_lag_2         112
16  roll2d_median_Price          82
2           Price_lag_7          79
15       lag168_Load_DA          77
10           Coal_lag_2          54
12            Oil_lag_2          36
13            EUA_lag_2          35
5       WindOn_DA_lag_0          21
9                  WD_7           6
7                  WD_1           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 30.23% done
********************* END NS.. 220 ****************************************************
********************* START NS ... 221 ****************************************************
[I 2025-06-15 19:57:21,213] A new study created in memory with name: no-name-e15b33f4-bdea-4c2b-86f6-632cc5433a1d
[I 2025-06-15 19:57:22,457] Trial 0 finished with value: 0.0008577486535328157 and parameters: {'n_estimators': 249, 'learning_rate': 0.02529054482544747, 'num_leaves': 107, 'max_depth': 12, 'min_child_samples': 16, 'subsample': 0.7598701812948366, 'colsample_bytree': 0.8317274346241836, 'reg_alpha': 2.3216290358184466, 'reg_lambda': 3.5129582031497133, 'min_split_gain': 0.027155059320046828}. Best is trial 0 with value: 0.0008577486535328157.
[I 2025-06-15 19:57:22,639] Trial 1 finished with value: 0.0007319499642770027 and parameters: {'n_estimators': 103, 'learning_rate': 0.2465586594215962, 'num_leaves': 103, 'max_depth': 4, 'min_child_samples': 13, 'subsample': 0.552938979368738, 'colsample_bytree': 0.6736339214845617, 'reg_alpha': 1.1865050105794572, 'reg_lambda': 3.8689578502064306, 'min_split_gain': 0.03180693513088957}. Best is trial 1 with value: 0.0007319499642770027.
[Trial 0] New best: 0.00086
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:23,625] Trial 2 finished with value: 0.0005318876986330724 and parameters: {'n_estimators': 265, 'learning_rate': 0.06399012507546468, 'num_leaves': 213, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.9684309402528137, 'colsample_bytree': 0.7412645957846761, 'reg_alpha': 1.3640957136779064, 'reg_lambda': 9.62600306613232, 'min_split_gain': 0.046841919473739134}. Best is trial 2 with value: 0.0005318876986330724.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:24,042] Trial 3 finished with value: 0.001244514845100854 and parameters: {'n_estimators': 258, 'learning_rate': 0.19025700511600005, 'num_leaves': 53, 'max_depth': 14, 'min_child_samples': 20, 'subsample': 0.5235083890441818, 'colsample_bytree': 0.8664695260102658, 'reg_alpha': 3.391754868646947, 'reg_lambda': 5.1699516082731956, 'min_split_gain': 0.025527761378715232}. Best is trial 2 with value: 0.0005318876986330724.
[I 2025-06-15 19:57:24,045] A new study created in memory with name: no-name-8341f5e5-5cdb-43ec-b480-988cfae0d185
[I 2025-06-15 19:57:24,087] Trial 0 finished with value: 0.03517047895011572 and parameters: {'n_estimators': 195, 'learning_rate': 0.023119998742624805, 'num_leaves': 151, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.8711878618964255, 'colsample_bytree': 0.9804780320017938, 'reg_alpha': 6.754917666494784, 'reg_lambda': 1.6222394398848605, 'min_split_gain': 0.005563164055938891}. Best is trial 0 with value: 0.03517047895011572.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         962
1           Price_lag_2         876
14      pct_chg_Load_DA         731
3         Load_DA_lag_0         505
11           NGas_lag_2         390
15       lag168_Load_DA         371
2           Price_lag_7         311
16  roll2d_median_Price         293
10           Coal_lag_2         209
5       WindOn_DA_lag_0         175
12            Oil_lag_2         137
13            EUA_lag_2         122
9                  WD_7          24
7                  WD_1          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          44
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 30.37% done
********************* END NS.. 221 ****************************************************
********************* START NS ... 222 ****************************************************
[I 2025-06-15 19:57:25,349] A new study created in memory with name: no-name-f88aff5b-06d8-4eab-bbec-27e9f699ba7c
[I 2025-06-15 19:57:25,647] Trial 0 finished with value: 0.08403390162405967 and parameters: {'n_estimators': 63, 'learning_rate': 0.01723280691742498, 'num_leaves': 255, 'max_depth': 11, 'min_child_samples': 19, 'subsample': 0.5599228972228449, 'colsample_bytree': 0.699841507562918, 'reg_alpha': 0.07789720398427291, 'reg_lambda': 7.672332133413754, 'min_split_gain': 0.04650410969050571}. Best is trial 0 with value: 0.08403390162405967.
[Trial 0] New best: 0.08403
[I 2025-06-15 19:57:26,280] Trial 1 finished with value: 0.0008283263351860293 and parameters: {'n_estimators': 200, 'learning_rate': 0.056133169878428354, 'num_leaves': 145, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.7990330234216969, 'colsample_bytree': 0.5008199372399695, 'reg_alpha': 0.24872439341727226, 'reg_lambda': 5.903556723259954, 'min_split_gain': 0.0388965575574833}. Best is trial 1 with value: 0.0008283263351860293.
[I 2025-06-15 19:57:26,488] Trial 2 finished with value: 0.0005540051897362222 and parameters: {'n_estimators': 199, 'learning_rate': 0.12894412887551465, 'num_leaves': 62, 'max_depth': 11, 'min_child_samples': 25, 'subsample': 0.7854434014012414, 'colsample_bytree': 0.7464622045418783, 'reg_alpha': 7.438934869359562, 'reg_lambda': 5.207888526881229, 'min_split_gain': 0.04850641879020732}. Best is trial 2 with value: 0.0005540051897362222.
[Trial 1] New best: 0.00083
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:27,576] Trial 3 finished with value: 0.0009973653911138716 and parameters: {'n_estimators': 158, 'learning_rate': 0.024949137503325648, 'num_leaves': 163, 'max_depth': 10, 'min_child_samples': 23, 'subsample': 0.6667576888348421, 'colsample_bytree': 0.96589555952834, 'reg_alpha': 0.25805125156899367, 'reg_lambda': 8.750758803083011, 'min_split_gain': 0.0010119531235610046}. Best is trial 2 with value: 0.0005540051897362222.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:28,048] Trial 4 finished with value: 0.0004609086986767412 and parameters: {'n_estimators': 238, 'learning_rate': 0.07851448979394626, 'num_leaves': 94, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.9670305905159096, 'colsample_bytree': 0.793364672260044, 'reg_alpha': 5.00029338425004, 'reg_lambda': 4.657310239779599, 'min_split_gain': 0.017813228942201536}. Best is trial 4 with value: 0.0004609086986767412.
[I 2025-06-15 19:57:28,051] A new study created in memory with name: no-name-75fdbf03-b568-44a9-ab26-87e81218591b
[I 2025-06-15 19:57:28,076] Trial 0 finished with value: 0.002531054062053778 and parameters: {'n_estimators': 94, 'learning_rate': 0.03281718013481565, 'num_leaves': 44, 'max_depth': 9, 'min_child_samples': 28, 'subsample': 0.6528044788221685, 'colsample_bytree': 0.6820032648872232, 'reg_alpha': 0.911474942017666, 'reg_lambda': 4.459622115656206, 'min_split_gain': 0.021403548018969872}. Best is trial 0 with value: 0.002531054062053778.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         727
1           Price_lag_2         525
14      pct_chg_Load_DA         480
3         Load_DA_lag_0         308
11           NGas_lag_2         256
2           Price_lag_7         212
15       lag168_Load_DA         212
5       WindOn_DA_lag_0         192
16  roll2d_median_Price         183
10           Coal_lag_2         139
13            EUA_lag_2         129
12            Oil_lag_2         106
7                  WD_1          12
9                  WD_7          11
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          49
1           Price_lag_2          18
2           Price_lag_7           5
10           Coal_lag_2           4
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
16  roll2d_median_Price           0
-> 30.51% done
********************* END NS.. 222 ****************************************************
********************* START NS ... 223 ****************************************************
[I 2025-06-15 19:57:28,944] A new study created in memory with name: no-name-b46e6b0a-b130-4306-ba62-7654a6d03664
[I 2025-06-15 19:57:29,484] Trial 0 finished with value: 0.000629508477180138 and parameters: {'n_estimators': 273, 'learning_rate': 0.02044867263360281, 'num_leaves': 95, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.7254607341382737, 'colsample_bytree': 0.9154600393744172, 'reg_alpha': 0.08692686732810317, 'reg_lambda': 9.7706275804379, 'min_split_gain': 0.04181202024838882}. Best is trial 0 with value: 0.000629508477180138.
[Trial 0] New best: 0.00063
[I 2025-06-15 19:57:30,036] Trial 1 finished with value: 0.0006862110225423143 and parameters: {'n_estimators': 197, 'learning_rate': 0.03997365348612493, 'num_leaves': 193, 'max_depth': 10, 'min_child_samples': 30, 'subsample': 0.8157924955181755, 'colsample_bytree': 0.6517019466991044, 'reg_alpha': 5.725358294826397, 'reg_lambda': 5.057333067175783, 'min_split_gain': 0.045236868014724645}. Best is trial 0 with value: 0.000629508477180138.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:30,388] Trial 2 finished with value: 0.0005940647699174292 and parameters: {'n_estimators': 156, 'learning_rate': 0.10814095734573817, 'num_leaves': 183, 'max_depth': 12, 'min_child_samples': 4, 'subsample': 0.6292702901699956, 'colsample_bytree': 0.8924118973461286, 'reg_alpha': 8.169014902400722, 'reg_lambda': 8.600153243028416, 'min_split_gain': 0.006739166486089504}. Best is trial 2 with value: 0.0005940647699174292.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:30,704] Trial 3 finished with value: 0.0008022202435135926 and parameters: {'n_estimators': 195, 'learning_rate': 0.05743591924025143, 'num_leaves': 93, 'max_depth': 4, 'min_child_samples': 24, 'subsample': 0.6860413851891642, 'colsample_bytree': 0.5361030875958591, 'reg_alpha': 5.487797483176231, 'reg_lambda': 8.895250904141982, 'min_split_gain': 0.026324766883294104}. Best is trial 2 with value: 0.0005940647699174292.
[I 2025-06-15 19:57:30,708] A new study created in memory with name: no-name-b0363e42-480b-43d1-8ceb-6856730e305d
[I 2025-06-15 19:57:30,744] Trial 0 finished with value: 0.00947441972371908 and parameters: {'n_estimators': 253, 'learning_rate': 0.14931262820440383, 'num_leaves': 177, 'max_depth': 13, 'min_child_samples': 5, 'subsample': 0.9491838401755268, 'colsample_bytree': 0.8354828432775201, 'reg_alpha': 7.350077647251288, 'reg_lambda': 7.742492873892583, 'min_split_gain': 0.01828163229602997}. Best is trial 0 with value: 0.00947441972371908.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         531
1           Price_lag_2         468
14      pct_chg_Load_DA         397
3         Load_DA_lag_0         294
2           Price_lag_7         233
11           NGas_lag_2         226
16  roll2d_median_Price         213
5       WindOn_DA_lag_0         202
13            EUA_lag_2         191
10           Coal_lag_2         189
15       lag168_Load_DA         184
12            Oil_lag_2         150
9                  WD_7          17
8                  WD_6           9
7                  WD_1           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 30.64% done
********************* END NS.. 223 ****************************************************
********************* START NS ... 224 ****************************************************
[I 2025-06-15 19:57:31,627] A new study created in memory with name: no-name-a223d7a0-66a4-42dd-abbf-1409af7e3caa
[I 2025-06-15 19:57:32,131] Trial 0 finished with value: 0.0008131032515669902 and parameters: {'n_estimators': 211, 'learning_rate': 0.020304776081131733, 'num_leaves': 157, 'max_depth': 7, 'min_child_samples': 12, 'subsample': 0.8861610896659685, 'colsample_bytree': 0.8168799323242892, 'reg_alpha': 9.499405277900241, 'reg_lambda': 3.257879087138571, 'min_split_gain': 0.001900624846612542}. Best is trial 0 with value: 0.0008131032515669902.
[I 2025-06-15 19:57:32,328] Trial 1 finished with value: 0.0012139166181415965 and parameters: {'n_estimators': 123, 'learning_rate': 0.029278993270147886, 'num_leaves': 139, 'max_depth': 4, 'min_child_samples': 20, 'subsample': 0.8449943169187708, 'colsample_bytree': 0.6864723945269604, 'reg_alpha': 5.577385973199334, 'reg_lambda': 3.6544893278775756, 'min_split_gain': 0.02902900007454023}. Best is trial 0 with value: 0.0008131032515669902.
[Trial 0] New best: 0.00081
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:32,965] Trial 2 finished with value: 0.0013232006990451619 and parameters: {'n_estimators': 137, 'learning_rate': 0.032533806833900725, 'num_leaves': 254, 'max_depth': 11, 'min_child_samples': 30, 'subsample': 0.574616432589081, 'colsample_bytree': 0.5116400823634812, 'reg_alpha': 1.1958858825515317, 'reg_lambda': 7.282436237871207, 'min_split_gain': 0.022368255467303446}. Best is trial 0 with value: 0.0008131032515669902.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:33,663] Trial 3 finished with value: 0.0006579623997183802 and parameters: {'n_estimators': 264, 'learning_rate': 0.04946888831448305, 'num_leaves': 247, 'max_depth': 11, 'min_child_samples': 4, 'subsample': 0.9384729866927588, 'colsample_bytree': 0.9063956680846713, 'reg_alpha': 2.3939505321149945, 'reg_lambda': 0.016752299524200875, 'min_split_gain': 0.026763574509057593}. Best is trial 3 with value: 0.0006579623997183802.
[I 2025-06-15 19:57:33,666] A new study created in memory with name: no-name-42adbf86-5a06-4218-b275-48674b2a21d5
[I 2025-06-15 19:57:33,695] Trial 0 finished with value: 0.007812573346929778 and parameters: {'n_estimators': 289, 'learning_rate': 0.20222361302636743, 'num_leaves': 256, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.6900480805712501, 'colsample_bytree': 0.9974037456506519, 'reg_alpha': 6.661252772597116, 'reg_lambda': 3.3301548976207895, 'min_split_gain': 0.009087895578063438}. Best is trial 0 with value: 0.007812573346929778.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1440
14      pct_chg_Load_DA        1201
1           Price_lag_2        1173
3         Load_DA_lag_0         852
11           NGas_lag_2         651
2           Price_lag_7         617
15       lag168_Load_DA         555
16  roll2d_median_Price         555
5       WindOn_DA_lag_0         383
10           Coal_lag_2         366
13            EUA_lag_2         257
12            Oil_lag_2         252
9                  WD_7          33
7                  WD_1          22
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 30.78% done
********************* END NS.. 224 ****************************************************
********************* START NS ... 225 ****************************************************
[I 2025-06-15 19:57:34,703] A new study created in memory with name: no-name-c0bf18a7-c866-4e73-b71b-00ce6c3218f2
[I 2025-06-15 19:57:34,863] Trial 0 finished with value: 0.0010220633956689272 and parameters: {'n_estimators': 214, 'learning_rate': 0.1858280658009014, 'num_leaves': 156, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.8130387201495279, 'colsample_bytree': 0.6987931479084928, 'reg_alpha': 2.0084470695302423, 'reg_lambda': 4.385118264957562, 'min_split_gain': 0.026860317729054197}. Best is trial 0 with value: 0.0010220633956689272.
[Trial 0] New best: 0.00102
[I 2025-06-15 19:57:35,119] Trial 1 finished with value: 0.0010992659140604255 and parameters: {'n_estimators': 90, 'learning_rate': 0.1411072230596577, 'num_leaves': 149, 'max_depth': 13, 'min_child_samples': 23, 'subsample': 0.9707240996404995, 'colsample_bytree': 0.884245391656973, 'reg_alpha': 0.5331199019218491, 'reg_lambda': 1.4810183989286874, 'min_split_gain': 0.027828210249384353}. Best is trial 0 with value: 0.0010220633956689272.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:35,336] Trial 2 finished with value: 0.003563540429588528 and parameters: {'n_estimators': 139, 'learning_rate': 0.021026678588009405, 'num_leaves': 117, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.5485856910893698, 'colsample_bytree': 0.727574135244142, 'reg_alpha': 1.2927412915371483, 'reg_lambda': 8.107241129320359, 'min_split_gain': 0.026860124040805695}. Best is trial 0 with value: 0.0010220633956689272.
[I 2025-06-15 19:57:35,473] Trial 3 finished with value: 0.0011145314195914909 and parameters: {'n_estimators': 164, 'learning_rate': 0.29490242412479606, 'num_leaves': 175, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.5995363740687727, 'colsample_bytree': 0.7868455765765263, 'reg_alpha': 7.929287740135848, 'reg_lambda': 1.2987351313249573, 'min_split_gain': 0.03344797217968261}. Best is trial 0 with value: 0.0010220633956689272.
[I 2025-06-15 19:57:35,476] A new study created in memory with name: no-name-8e8c3cc8-e305-4df7-85d2-9a609fcfc946
[I 2025-06-15 19:57:35,498] Trial 0 finished with value: 0.002061567691234377 and parameters: {'n_estimators': 51, 'learning_rate': 0.013938867170226172, 'num_leaves': 156, 'max_depth': 8, 'min_child_samples': 15, 'subsample': 0.7222301587120291, 'colsample_bytree': 0.8622270388062849, 'reg_alpha': 0.8534923575284903, 'reg_lambda': 5.664363601794817, 'min_split_gain': 0.022293088276170315}. Best is trial 0 with value: 0.002061567691234377.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         292
1           Price_lag_2         163
14      pct_chg_Load_DA         109
3         Load_DA_lag_0         103
11           NGas_lag_2          87
2           Price_lag_7          74
15       lag168_Load_DA          74
5       WindOn_DA_lag_0          70
16  roll2d_median_Price          67
10           Coal_lag_2          54
13            EUA_lag_2          54
12            Oil_lag_2          46
9                  WD_7           7
7                  WD_1           3
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          78
1           Price_lag_2           9
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 30.92% done
********************* END NS.. 225 ****************************************************
********************* START NS ... 226 ****************************************************
[I 2025-06-15 19:57:36,172] A new study created in memory with name: no-name-580ebf14-af78-463c-98d6-9a5a96f08906
[I 2025-06-15 19:57:36,351] Trial 0 finished with value: 0.0008259147552823545 and parameters: {'n_estimators': 198, 'learning_rate': 0.12366941934468158, 'num_leaves': 111, 'max_depth': 10, 'min_child_samples': 19, 'subsample': 0.9690447003146867, 'colsample_bytree': 0.6700891074443093, 'reg_alpha': 5.195374254013682, 'reg_lambda': 1.2907573528126204, 'min_split_gain': 0.03644142305762785}. Best is trial 0 with value: 0.0008259147552823545.
[Trial 0] New best: 0.00083
[I 2025-06-15 19:57:36,763] Trial 1 finished with value: 0.0010867456273094404 and parameters: {'n_estimators': 165, 'learning_rate': 0.10344760482106467, 'num_leaves': 150, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.59975904794513, 'colsample_bytree': 0.624206406101385, 'reg_alpha': 6.215960312639227, 'reg_lambda': 3.7728364404192782, 'min_split_gain': 0.0015422925019874024}. Best is trial 0 with value: 0.0008259147552823545.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:37,137] Trial 2 finished with value: 0.005919726258619772 and parameters: {'n_estimators': 108, 'learning_rate': 0.02212272189075395, 'num_leaves': 228, 'max_depth': 12, 'min_child_samples': 28, 'subsample': 0.8086570561057336, 'colsample_bytree': 0.7525725345139376, 'reg_alpha': 5.866475408549043, 'reg_lambda': 5.971286978672423, 'min_split_gain': 0.002170378618129881}. Best is trial 0 with value: 0.0008259147552823545.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:37,382] Trial 3 finished with value: 0.0009182342495986438 and parameters: {'n_estimators': 237, 'learning_rate': 0.14522311525124484, 'num_leaves': 36, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.7457865104362738, 'colsample_bytree': 0.9126750793545884, 'reg_alpha': 6.389202834558843, 'reg_lambda': 4.566912224026204, 'min_split_gain': 0.0018253849952533875}. Best is trial 0 with value: 0.0008259147552823545.
[I 2025-06-15 19:57:37,384] A new study created in memory with name: no-name-3a18efc9-5f68-4758-9a4c-b6d88b8767ba
[I 2025-06-15 19:57:37,416] Trial 0 finished with value: 0.006502361711753273 and parameters: {'n_estimators': 260, 'learning_rate': 0.09561224303048452, 'num_leaves': 104, 'max_depth': 11, 'min_child_samples': 20, 'subsample': 0.502400438749782, 'colsample_bytree': 0.6994160069416274, 'reg_alpha': 2.139504177755067, 'reg_lambda': 1.7040795249107166, 'min_split_gain': 0.017971495367373674}. Best is trial 0 with value: 0.006502361711753273.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         393
1           Price_lag_2         231
14      pct_chg_Load_DA         210
3         Load_DA_lag_0         175
11           NGas_lag_2         124
2           Price_lag_7          95
16  roll2d_median_Price          89
10           Coal_lag_2          80
15       lag168_Load_DA          63
13            EUA_lag_2          53
12            Oil_lag_2          49
5       WindOn_DA_lag_0          29
9                  WD_7           7
7                  WD_1           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          23
1           Price_lag_2           2
13            EUA_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 31.05% done
********************* END NS.. 226 ****************************************************
********************* START NS ... 227 ****************************************************
[I 2025-06-15 19:57:38,142] A new study created in memory with name: no-name-9766b71a-4b79-4721-9c27-36ca4f241734
[I 2025-06-15 19:57:38,525] Trial 0 finished with value: 0.0007806827071041482 and parameters: {'n_estimators': 200, 'learning_rate': 0.052388574286339494, 'num_leaves': 196, 'max_depth': 13, 'min_child_samples': 5, 'subsample': 0.7827043501070442, 'colsample_bytree': 0.6216959387408534, 'reg_alpha': 7.2188510133672965, 'reg_lambda': 5.220684650491502, 'min_split_gain': 0.026612189373542677}. Best is trial 0 with value: 0.0007806827071041482.
[Trial 0] New best: 0.00078
[I 2025-06-15 19:57:39,088] Trial 1 finished with value: 0.0005571852758281141 and parameters: {'n_estimators': 201, 'learning_rate': 0.07552041377339459, 'num_leaves': 63, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.6849557445231823, 'colsample_bytree': 0.792395505172628, 'reg_alpha': 2.1702337968289696, 'reg_lambda': 1.3962993042747995, 'min_split_gain': 0.007171848179405283}. Best is trial 1 with value: 0.0005571852758281141.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:39,741] Trial 2 finished with value: 0.012866222591776194 and parameters: {'n_estimators': 174, 'learning_rate': 0.01089750264501207, 'num_leaves': 153, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.6338152274246804, 'colsample_bytree': 0.5975879469608676, 'reg_alpha': 7.178018914075558, 'reg_lambda': 0.611232837597464, 'min_split_gain': 0.026224784815694555}. Best is trial 1 with value: 0.0005571852758281141.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:40,174] Trial 3 finished with value: 0.001016212132588311 and parameters: {'n_estimators': 114, 'learning_rate': 0.04180097425662097, 'num_leaves': 37, 'max_depth': 16, 'min_child_samples': 6, 'subsample': 0.6223535932385985, 'colsample_bytree': 0.5168241238696629, 'reg_alpha': 1.745943637972962, 'reg_lambda': 8.561814496639185, 'min_split_gain': 0.019134086893867492}. Best is trial 1 with value: 0.0005571852758281141.
[I 2025-06-15 19:57:40,176] A new study created in memory with name: no-name-e773c9a0-96df-400e-a3ce-25807905f28f
[I 2025-06-15 19:57:40,197] Trial 0 finished with value: 0.0070462008904169345 and parameters: {'n_estimators': 52, 'learning_rate': 0.01969469491693018, 'num_leaves': 52, 'max_depth': 12, 'min_child_samples': 7, 'subsample': 0.709652010009663, 'colsample_bytree': 0.9127629310995626, 'reg_alpha': 1.0418390741293893, 'reg_lambda': 7.696635746711726, 'min_split_gain': 0.0393919123722151}. Best is trial 0 with value: 0.0070462008904169345.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1134
14      pct_chg_Load_DA        1030
1           Price_lag_2         985
3         Load_DA_lag_0         879
5       WindOn_DA_lag_0         632
15       lag168_Load_DA         574
2           Price_lag_7         557
11           NGas_lag_2         429
16  roll2d_median_Price         427
13            EUA_lag_2         375
10           Coal_lag_2         358
12            Oil_lag_2         337
9                  WD_7          46
7                  WD_1          22
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          56
1           Price_lag_2           5
15       lag168_Load_DA           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 31.19% done
********************* END NS.. 227 ****************************************************
********************* START NS ... 228 ****************************************************
[I 2025-06-15 19:57:41,146] A new study created in memory with name: no-name-a9504312-7351-44f3-ab62-8ceb12cbf44b
[I 2025-06-15 19:57:41,568] Trial 0 finished with value: 0.0004172332823037801 and parameters: {'n_estimators': 273, 'learning_rate': 0.06342246755404143, 'num_leaves': 252, 'max_depth': 7, 'min_child_samples': 1, 'subsample': 0.5513984488668977, 'colsample_bytree': 0.6337086330414674, 'reg_alpha': 0.4426904043079427, 'reg_lambda': 1.6456211356505979, 'min_split_gain': 0.03084027941177856}. Best is trial 0 with value: 0.0004172332823037801.
[Trial 0] New best: 0.00042
[I 2025-06-15 19:57:42,155] Trial 1 finished with value: 0.0007798407512974562 and parameters: {'n_estimators': 114, 'learning_rate': 0.03167648081303884, 'num_leaves': 253, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.7940024598963059, 'colsample_bytree': 0.9391510199277044, 'reg_alpha': 0.6044349477717337, 'reg_lambda': 4.677835248544322, 'min_split_gain': 0.040190568675215026}. Best is trial 0 with value: 0.0004172332823037801.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:42,554] Trial 2 finished with value: 0.0005336954820811746 and parameters: {'n_estimators': 287, 'learning_rate': 0.029598843377683495, 'num_leaves': 230, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.9250707198831718, 'colsample_bytree': 0.5826246951871954, 'reg_alpha': 1.1958975669678706, 'reg_lambda': 8.825675789959282, 'min_split_gain': 0.03723878625573084}. Best is trial 0 with value: 0.0004172332823037801.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:43,695] Trial 3 finished with value: 0.0018893999428804085 and parameters: {'n_estimators': 245, 'learning_rate': 0.012305810134533185, 'num_leaves': 102, 'max_depth': 12, 'min_child_samples': 26, 'subsample': 0.5539251906918122, 'colsample_bytree': 0.8059825932204217, 'reg_alpha': 1.2749168329260352, 'reg_lambda': 7.50935945421589, 'min_split_gain': 0.008283638813328931}. Best is trial 0 with value: 0.0004172332823037801.
[I 2025-06-15 19:57:43,697] A new study created in memory with name: no-name-1077710e-8a9c-4691-a930-793e148c2cc4
[I 2025-06-15 19:57:43,728] Trial 0 finished with value: 0.002889600211954066 and parameters: {'n_estimators': 223, 'learning_rate': 0.08084904756099533, 'num_leaves': 223, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.8814731185894433, 'colsample_bytree': 0.5540192051003232, 'reg_alpha': 3.3087262349428572, 'reg_lambda': 9.094104875540781, 'min_split_gain': 0.011643820873520533}. Best is trial 0 with value: 0.002889600211954066.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1219
1           Price_lag_2        1199
14      pct_chg_Load_DA         969
3         Load_DA_lag_0         759
2           Price_lag_7         677
11           NGas_lag_2         596
16  roll2d_median_Price         528
15       lag168_Load_DA         489
10           Coal_lag_2         443
5       WindOn_DA_lag_0         369
13            EUA_lag_2         323
12            Oil_lag_2         295
9                  WD_7          34
7                  WD_1          21
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 31.33% done
********************* END NS.. 228 ****************************************************
********************* START NS ... 229 ****************************************************
[I 2025-06-15 19:57:44,684] A new study created in memory with name: no-name-7c0cf4af-bd9a-4445-91fb-1f84fe7e6da5
[I 2025-06-15 19:57:45,027] Trial 0 finished with value: 0.0005296952452103547 and parameters: {'n_estimators': 292, 'learning_rate': 0.054912231270246005, 'num_leaves': 234, 'max_depth': 16, 'min_child_samples': 6, 'subsample': 0.6802965231320983, 'colsample_bytree': 0.8214454915183587, 'reg_alpha': 9.360345760010846, 'reg_lambda': 4.918779304414106, 'min_split_gain': 0.0392183959711317}. Best is trial 0 with value: 0.0005296952452103547.
[Trial 0] New best: 0.00053
[I 2025-06-15 19:57:46,982] Trial 1 finished with value: 0.0007872368844081901 and parameters: {'n_estimators': 252, 'learning_rate': 0.016813977591193584, 'num_leaves': 135, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.6517363479597009, 'colsample_bytree': 0.7302950678403526, 'reg_alpha': 0.6088002372726797, 'reg_lambda': 5.780722823962197, 'min_split_gain': 0.01403933542150358}. Best is trial 0 with value: 0.0005296952452103547.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:47,342] Trial 2 finished with value: 0.036584834127095914 and parameters: {'n_estimators': 114, 'learning_rate': 0.012704572094894171, 'num_leaves': 105, 'max_depth': 11, 'min_child_samples': 13, 'subsample': 0.9103656178764468, 'colsample_bytree': 0.5444558619673912, 'reg_alpha': 5.4765747458738145, 'reg_lambda': 4.019428054396763, 'min_split_gain': 0.039177701176719315}. Best is trial 0 with value: 0.0005296952452103547.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:47,614] Trial 3 finished with value: 0.008631464239551476 and parameters: {'n_estimators': 91, 'learning_rate': 0.023606763618704077, 'num_leaves': 219, 'max_depth': 16, 'min_child_samples': 10, 'subsample': 0.9542255093049437, 'colsample_bytree': 0.682542902197775, 'reg_alpha': 8.617406221518616, 'reg_lambda': 5.5690861873258655, 'min_split_gain': 0.045036796361904136}. Best is trial 0 with value: 0.0005296952452103547.
[I 2025-06-15 19:57:47,617] A new study created in memory with name: no-name-8af112f8-ae90-4013-9a2e-02368fd06773
[I 2025-06-15 19:57:47,641] Trial 0 finished with value: 0.003125828689321285 and parameters: {'n_estimators': 182, 'learning_rate': 0.04661453828876549, 'num_leaves': 60, 'max_depth': 9, 'min_child_samples': 16, 'subsample': 0.9180998692360134, 'colsample_bytree': 0.5858909133183683, 'reg_alpha': 5.046772590891878, 'reg_lambda': 8.127400227728485, 'min_split_gain': 0.01295517681651965}. Best is trial 0 with value: 0.003125828689321285.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         823
1           Price_lag_2         489
14      pct_chg_Load_DA         435
3         Load_DA_lag_0         262
11           NGas_lag_2         237
2           Price_lag_7         204
16  roll2d_median_Price         170
10           Coal_lag_2         121
13            EUA_lag_2         113
15       lag168_Load_DA         102
12            Oil_lag_2          94
5       WindOn_DA_lag_0          42
9                  WD_7          21
7                  WD_1           7
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 31.46% done
********************* END NS.. 229 ****************************************************
********************* START NS ... 230 ****************************************************
[I 2025-06-15 19:57:48,445] A new study created in memory with name: no-name-3dd5d29b-d457-4360-8795-0de66ac1a70d
[I 2025-06-15 19:57:48,578] Trial 0 finished with value: 0.0005246202048258017 and parameters: {'n_estimators': 263, 'learning_rate': 0.1319874348412283, 'num_leaves': 239, 'max_depth': 4, 'min_child_samples': 1, 'subsample': 0.6901532727198514, 'colsample_bytree': 0.8021793545349396, 'reg_alpha': 3.3449837905328375, 'reg_lambda': 2.1053492405565635, 'min_split_gain': 0.04760324299410024}. Best is trial 0 with value: 0.0005246202048258017.
[Trial 0] New best: 0.00052
[I 2025-06-15 19:57:48,830] Trial 1 finished with value: 0.047031367665225955 and parameters: {'n_estimators': 78, 'learning_rate': 0.01628719342278103, 'num_leaves': 93, 'max_depth': 11, 'min_child_samples': 18, 'subsample': 0.725352537955578, 'colsample_bytree': 0.6755163133713908, 'reg_alpha': 6.514836636584919, 'reg_lambda': 2.477789059925821, 'min_split_gain': 0.015280570422847578}. Best is trial 0 with value: 0.0005246202048258017.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:49,479] Trial 2 finished with value: 0.0009613105686224014 and parameters: {'n_estimators': 208, 'learning_rate': 0.02308565632454546, 'num_leaves': 55, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.7237654739232349, 'colsample_bytree': 0.5453267978815857, 'reg_alpha': 7.346580194044783, 'reg_lambda': 1.4522379161637777, 'min_split_gain': 0.03486153437470692}. Best is trial 0 with value: 0.0005246202048258017.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:51,577] Trial 3 finished with value: 0.0006998640364815137 and parameters: {'n_estimators': 253, 'learning_rate': 0.033044101383778834, 'num_leaves': 168, 'max_depth': 11, 'min_child_samples': 1, 'subsample': 0.8737617362400059, 'colsample_bytree': 0.5781670175484448, 'reg_alpha': 0.8235933467054435, 'reg_lambda': 7.349091977314531, 'min_split_gain': 0.008619176681662405}. Best is trial 0 with value: 0.0005246202048258017.
[I 2025-06-15 19:57:51,578] A new study created in memory with name: no-name-16dfec37-7eca-4426-845f-49a0dfca001b
[I 2025-06-15 19:57:51,599] Trial 0 finished with value: 0.005565763438848383 and parameters: {'n_estimators': 157, 'learning_rate': 0.010071950893438784, 'num_leaves': 163, 'max_depth': 9, 'min_child_samples': 24, 'subsample': 0.6641980786487154, 'colsample_bytree': 0.9708630866455339, 'reg_alpha': 2.998401089918543, 'reg_lambda': 5.375435871576384, 'min_split_gain': 0.03296299917708887}. Best is trial 0 with value: 0.005565763438848383.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         281
1           Price_lag_2         135
14      pct_chg_Load_DA         114
3         Load_DA_lag_0          76
11           NGas_lag_2          58
2           Price_lag_7          57
16  roll2d_median_Price          52
15       lag168_Load_DA          40
12            Oil_lag_2          32
13            EUA_lag_2          27
10           Coal_lag_2          23
9                  WD_7           7
5       WindOn_DA_lag_0           7
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 31.60% done
********************* END NS.. 230 ****************************************************
********************* START NS ... 231 ****************************************************
[I 2025-06-15 19:57:52,376] A new study created in memory with name: no-name-913883b1-8560-48ff-b08c-1bcd6b79d7fd
[I 2025-06-15 19:57:52,542] Trial 0 finished with value: 0.0004466047803973326 and parameters: {'n_estimators': 201, 'learning_rate': 0.245632198468159, 'num_leaves': 177, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.8122902923715776, 'colsample_bytree': 0.922368478505742, 'reg_alpha': 2.7111299231784436, 'reg_lambda': 5.363651462651193, 'min_split_gain': 0.020972609276487787}. Best is trial 0 with value: 0.0004466047803973326.
[Trial 0] New best: 0.00045
[I 2025-06-15 19:57:52,876] Trial 1 finished with value: 0.0005596048385737824 and parameters: {'n_estimators': 190, 'learning_rate': 0.08682129676595944, 'num_leaves': 187, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.8625255131233545, 'colsample_bytree': 0.5489594651731915, 'reg_alpha': 8.408009583933685, 'reg_lambda': 1.305347912086472, 'min_split_gain': 0.023180279068463545}. Best is trial 0 with value: 0.0004466047803973326.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:53,465] Trial 2 finished with value: 0.0006998549075423007 and parameters: {'n_estimators': 186, 'learning_rate': 0.021085452502316655, 'num_leaves': 249, 'max_depth': 14, 'min_child_samples': 6, 'subsample': 0.973536892692182, 'colsample_bytree': 0.7722814508194826, 'reg_alpha': 8.028979566054655, 'reg_lambda': 2.702991845213841, 'min_split_gain': 0.00940533424540922}. Best is trial 0 with value: 0.0004466047803973326.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:53,874] Trial 3 finished with value: 0.000521764892009963 and parameters: {'n_estimators': 298, 'learning_rate': 0.0306646559469955, 'num_leaves': 194, 'max_depth': 4, 'min_child_samples': 22, 'subsample': 0.992223853324249, 'colsample_bytree': 0.5437761339824863, 'reg_alpha': 4.812844254410789, 'reg_lambda': 0.011782695577514657, 'min_split_gain': 0.035583202213737807}. Best is trial 0 with value: 0.0004466047803973326.
[I 2025-06-15 19:57:53,876] A new study created in memory with name: no-name-d701fd0f-cb56-437d-88d3-20da1b992d8a
[I 2025-06-15 19:57:53,907] Trial 0 finished with value: 0.0030465687895713882 and parameters: {'n_estimators': 272, 'learning_rate': 0.05243162443329933, 'num_leaves': 43, 'max_depth': 16, 'min_child_samples': 12, 'subsample': 0.9316081462339594, 'colsample_bytree': 0.6952772531511681, 'reg_alpha': 9.665002534688199, 'reg_lambda': 1.9827812467151396, 'min_split_gain': 0.04465180408128824}. Best is trial 0 with value: 0.0030465687895713882.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         300
14      pct_chg_Load_DA         241
1           Price_lag_2         195
3         Load_DA_lag_0         188
2           Price_lag_7         119
15       lag168_Load_DA         102
11           NGas_lag_2          93
16  roll2d_median_Price          90
12            Oil_lag_2          76
10           Coal_lag_2          69
13            EUA_lag_2          67
5       WindOn_DA_lag_0          58
7                  WD_1           5
9                  WD_7           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 31.74% done
********************* END NS.. 231 ****************************************************
********************* START NS ... 232 ****************************************************
[I 2025-06-15 19:57:54,575] A new study created in memory with name: no-name-36894407-3a10-4ed7-9940-68e424d837e4
[I 2025-06-15 19:57:54,714] Trial 0 finished with value: 0.02028384154480446 and parameters: {'n_estimators': 51, 'learning_rate': 0.03347704106437577, 'num_leaves': 155, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.6738984406456722, 'colsample_bytree': 0.858062650572617, 'reg_alpha': 6.036823711735436, 'reg_lambda': 8.963936750072396, 'min_split_gain': 0.04826494591416887}. Best is trial 0 with value: 0.02028384154480446.
[Trial 0] New best: 0.02028
[I 2025-06-15 19:57:54,916] Trial 1 finished with value: 0.02661106075186087 and parameters: {'n_estimators': 131, 'learning_rate': 0.012468034244190072, 'num_leaves': 182, 'max_depth': 4, 'min_child_samples': 23, 'subsample': 0.5565068278160843, 'colsample_bytree': 0.6088362439934467, 'reg_alpha': 6.206308633931988, 'reg_lambda': 9.670861694712476, 'min_split_gain': 0.007546987168999309}. Best is trial 0 with value: 0.02028384154480446.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:55,255] Trial 2 finished with value: 0.00028786359236798444 and parameters: {'n_estimators': 232, 'learning_rate': 0.05632073161058482, 'num_leaves': 189, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.5517759220254111, 'colsample_bytree': 0.7539401584199594, 'reg_alpha': 7.625442851871824, 'reg_lambda': 0.4361794935862995, 'min_split_gain': 0.04883606477947154}. Best is trial 2 with value: 0.00028786359236798444.
[Trial 2] New best: 0.00029
[I 2025-06-15 19:57:55,934] Trial 3 finished with value: 0.0005232672829814 and parameters: {'n_estimators': 242, 'learning_rate': 0.015969746091595485, 'num_leaves': 144, 'max_depth': 12, 'min_child_samples': 19, 'subsample': 0.5529311884805657, 'colsample_bytree': 0.9406901752495708, 'reg_alpha': 6.764802518789105, 'reg_lambda': 6.587039133056441, 'min_split_gain': 0.030803285720534014}. Best is trial 2 with value: 0.00028786359236798444.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:56,219] Trial 4 finished with value: 0.00040012170445486974 and parameters: {'n_estimators': 118, 'learning_rate': 0.18343486162012212, 'num_leaves': 212, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.7105253019665337, 'colsample_bytree': 0.5255699880643829, 'reg_alpha': 1.6606726064140986, 'reg_lambda': 0.136665900811207, 'min_split_gain': 0.009532725017579053}. Best is trial 2 with value: 0.00028786359236798444.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:57:56,571] Trial 5 finished with value: 0.0003227215260097731 and parameters: {'n_estimators': 270, 'learning_rate': 0.0321850124820622, 'num_leaves': 120, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.5863471613379879, 'colsample_bytree': 0.7429417364869526, 'reg_alpha': 7.848053651191149, 'reg_lambda': 0.8887868516751052, 'min_split_gain': 0.018105176776792686}. Best is trial 2 with value: 0.00028786359236798444.
[I 2025-06-15 19:57:56,574] A new study created in memory with name: no-name-27173a46-7f97-45d4-a565-6792ca510208
[I 2025-06-15 19:57:56,591] Trial 0 finished with value: 0.0018611145929769 and parameters: {'n_estimators': 99, 'learning_rate': 0.04651479629767525, 'num_leaves': 76, 'max_depth': 5, 'min_child_samples': 22, 'subsample': 0.877982454381072, 'colsample_bytree': 0.6192771018625004, 'reg_alpha': 4.176328314400829, 'reg_lambda': 5.885914516910432, 'min_split_gain': 0.04689854152687903}. Best is trial 0 with value: 0.0018611145929769.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         752
1           Price_lag_2         431
14      pct_chg_Load_DA         419
3         Load_DA_lag_0         233
11           NGas_lag_2         199
2           Price_lag_7         156
16  roll2d_median_Price         124
10           Coal_lag_2         101
15       lag168_Load_DA         101
12            Oil_lag_2          75
5       WindOn_DA_lag_0          63
13            EUA_lag_2          60
9                  WD_7          14
7                  WD_1           6
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 31.87% done
********************* END NS.. 232 ****************************************************
********************* START NS ... 233 ****************************************************
[I 2025-06-15 19:57:57,369] A new study created in memory with name: no-name-58c422bd-57bb-4ad8-ab38-4f0f328e3eec
[I 2025-06-15 19:57:58,116] Trial 0 finished with value: 0.0027756825297426263 and parameters: {'n_estimators': 223, 'learning_rate': 0.011694474600452653, 'num_leaves': 90, 'max_depth': 9, 'min_child_samples': 1, 'subsample': 0.5069259340914638, 'colsample_bytree': 0.9804446637944124, 'reg_alpha': 7.501086228897311, 'reg_lambda': 5.8326079403919895, 'min_split_gain': 0.041881250501427866}. Best is trial 0 with value: 0.0027756825297426263.
[Trial 0] New best: 0.00278
[I 2025-06-15 19:57:58,374] Trial 1 finished with value: 0.00018528909995159533 and parameters: {'n_estimators': 84, 'learning_rate': 0.06661604812486957, 'num_leaves': 255, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.5020645359936826, 'colsample_bytree': 0.6752919055413682, 'reg_alpha': 7.641036354279912, 'reg_lambda': 9.28329893141594, 'min_split_gain': 0.02418733233134346}. Best is trial 1 with value: 0.00018528909995159533.
[Trial 1] New best: 0.00019
[I 2025-06-15 19:57:59,240] Trial 2 finished with value: 0.0002093287233369738 and parameters: {'n_estimators': 290, 'learning_rate': 0.019607437619680663, 'num_leaves': 42, 'max_depth': 14, 'min_child_samples': 24, 'subsample': 0.742841801729575, 'colsample_bytree': 0.5050681054053708, 'reg_alpha': 2.2300204662019487, 'reg_lambda': 7.701993491334504, 'min_split_gain': 0.00822950271604569}. Best is trial 1 with value: 0.00018528909995159533.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:57:59,922] Trial 3 finished with value: 0.00022399228148908755 and parameters: {'n_estimators': 159, 'learning_rate': 0.07397580199321444, 'num_leaves': 172, 'max_depth': 16, 'min_child_samples': 27, 'subsample': 0.9080822240484159, 'colsample_bytree': 0.5323401142550152, 'reg_alpha': 1.6033319040577365, 'reg_lambda': 6.314566591184127, 'min_split_gain': 0.007077658271982501}. Best is trial 1 with value: 0.00018528909995159533.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:00,230] Trial 4 finished with value: 0.000246538260598544 and parameters: {'n_estimators': 279, 'learning_rate': 0.15122469983067072, 'num_leaves': 246, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.6161414218749401, 'colsample_bytree': 0.7696660487418647, 'reg_alpha': 1.3108677191202789, 'reg_lambda': 6.276811017170215, 'min_split_gain': 0.03115403190445012}. Best is trial 1 with value: 0.00018528909995159533.
[I 2025-06-15 19:58:00,232] A new study created in memory with name: no-name-7a29f5e5-bc23-404d-a203-fae5cf90da01
[I 2025-06-15 19:58:00,254] Trial 0 finished with value: 0.0008554356508685979 and parameters: {'n_estimators': 127, 'learning_rate': 0.10080085144554736, 'num_leaves': 198, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.6914213963302318, 'colsample_bytree': 0.9841420989121763, 'reg_alpha': 8.142481388710767, 'reg_lambda': 1.3664434426326255, 'min_split_gain': 0.0208753737782108}. Best is trial 0 with value: 0.0008554356508685979.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         557
1           Price_lag_2         424
14      pct_chg_Load_DA         300
3         Load_DA_lag_0         202
2           Price_lag_7         188
11           NGas_lag_2         188
16  roll2d_median_Price         172
10           Coal_lag_2         139
15       lag168_Load_DA         120
5       WindOn_DA_lag_0         119
13            EUA_lag_2         108
12            Oil_lag_2          92
9                  WD_7          23
8                  WD_6          14
7                  WD_1           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 32.01% done
********************* END NS.. 233 ****************************************************
********************* START NS ... 234 ****************************************************
[I 2025-06-15 19:58:00,995] A new study created in memory with name: no-name-9b4dfaa2-ab3a-44c5-9ab7-41f99f822b66
[I 2025-06-15 19:58:01,169] Trial 0 finished with value: 7.221978725818365e-05 and parameters: {'n_estimators': 143, 'learning_rate': 0.13293842516210871, 'num_leaves': 168, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.7806673501618284, 'colsample_bytree': 0.9758451509914099, 'reg_alpha': 3.854504318673965, 'reg_lambda': 5.5546550855263614, 'min_split_gain': 0.02762920232531766}. Best is trial 0 with value: 7.221978725818365e-05.
[Trial 0] New best: 0.00007
[I 2025-06-15 19:58:01,476] Trial 1 finished with value: 0.00029938255742643205 and parameters: {'n_estimators': 107, 'learning_rate': 0.034677550634018846, 'num_leaves': 192, 'max_depth': 16, 'min_child_samples': 17, 'subsample': 0.5190808824371222, 'colsample_bytree': 0.6085346505558502, 'reg_alpha': 8.388732217875342, 'reg_lambda': 7.731540535163384, 'min_split_gain': 0.0073657122059408225}. Best is trial 0 with value: 7.221978725818365e-05.
[I 2025-06-15 19:58:01,620] Trial 2 finished with value: 0.05521252077436813 and parameters: {'n_estimators': 62, 'learning_rate': 0.018477569332475758, 'num_leaves': 180, 'max_depth': 5, 'min_child_samples': 13, 'subsample': 0.5737597755321768, 'colsample_bytree': 0.5157289466856898, 'reg_alpha': 2.1793802511501092, 'reg_lambda': 8.385110442029745, 'min_split_gain': 0.02537096351182043}. Best is trial 0 with value: 7.221978725818365e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:02,230] Trial 3 finished with value: 7.89271996844711e-05 and parameters: {'n_estimators': 165, 'learning_rate': 0.026543458428297195, 'num_leaves': 132, 'max_depth': 15, 'min_child_samples': 11, 'subsample': 0.797724758649438, 'colsample_bytree': 0.9896059502073965, 'reg_alpha': 4.502036088308872, 'reg_lambda': 8.226117222872345, 'min_split_gain': 0.016758213353540937}. Best is trial 0 with value: 7.221978725818365e-05.
[I 2025-06-15 19:58:02,233] A new study created in memory with name: no-name-5db296fc-871b-4fe7-977c-2eb51aedcee2
[I 2025-06-15 19:58:02,266] Trial 0 finished with value: 0.0009203364660905039 and parameters: {'n_estimators': 297, 'learning_rate': 0.2246924047056125, 'num_leaves': 95, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.8201486635676019, 'colsample_bytree': 0.7054027501750882, 'reg_alpha': 2.8069884680719093, 'reg_lambda': 2.3085863119412964, 'min_split_gain': 0.04669283045598799}. Best is trial 0 with value: 0.0009203364660905039.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         440
14      pct_chg_Load_DA         314
1           Price_lag_2         220
3         Load_DA_lag_0         192
11           NGas_lag_2         143
2           Price_lag_7         122
16  roll2d_median_Price          98
10           Coal_lag_2          96
15       lag168_Load_DA          85
13            EUA_lag_2          77
12            Oil_lag_2          59
5       WindOn_DA_lag_0          48
9                  WD_7           9
7                  WD_1           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 32.15% done
********************* END NS.. 234 ****************************************************
********************* START NS ... 235 ****************************************************
[I 2025-06-15 19:58:02,951] A new study created in memory with name: no-name-a944ad1f-a784-4080-8f13-e1a570662c25
[I 2025-06-15 19:58:03,180] Trial 0 finished with value: 0.006360216143280453 and parameters: {'n_estimators': 109, 'learning_rate': 0.020189635500836328, 'num_leaves': 150, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.8442638327339689, 'colsample_bytree': 0.566457891228245, 'reg_alpha': 7.94649141293413, 'reg_lambda': 9.55591837918927, 'min_split_gain': 0.047838460282153356}. Best is trial 0 with value: 0.006360216143280453.
[I 2025-06-15 19:58:03,377] Trial 1 finished with value: 0.00012142468288801694 and parameters: {'n_estimators': 249, 'learning_rate': 0.22379727070416625, 'num_leaves': 168, 'max_depth': 16, 'min_child_samples': 23, 'subsample': 0.980655814178615, 'colsample_bytree': 0.8560789619937827, 'reg_alpha': 6.136634162459157, 'reg_lambda': 7.524340785054133, 'min_split_gain': 0.010494407645195403}. Best is trial 1 with value: 0.00012142468288801694.
[Trial 0] New best: 0.00636
[Trial 1] New best: 0.00012
[I 2025-06-15 19:58:03,676] Trial 2 finished with value: 6.0330690040336144e-05 and parameters: {'n_estimators': 142, 'learning_rate': 0.14113242569411152, 'num_leaves': 34, 'max_depth': 7, 'min_child_samples': 17, 'subsample': 0.7038846490446786, 'colsample_bytree': 0.8935301628544341, 'reg_alpha': 2.257132956740385, 'reg_lambda': 9.488941602259684, 'min_split_gain': 0.0036426208100457606}. Best is trial 2 with value: 6.0330690040336144e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:04,059] Trial 3 finished with value: 0.007323356515630329 and parameters: {'n_estimators': 97, 'learning_rate': 0.021769356845576154, 'num_leaves': 85, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.895075538964432, 'colsample_bytree': 0.5968696459781782, 'reg_alpha': 4.669289267281615, 'reg_lambda': 4.794325560125826, 'min_split_gain': 0.007223373293303032}. Best is trial 2 with value: 6.0330690040336144e-05.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:04,289] Trial 4 finished with value: 0.00011126905313435032 and parameters: {'n_estimators': 241, 'learning_rate': 0.18573604531425486, 'num_leaves': 110, 'max_depth': 16, 'min_child_samples': 19, 'subsample': 0.9507786506507634, 'colsample_bytree': 0.5267903337247235, 'reg_alpha': 4.868193982432237, 'reg_lambda': 8.703874002680616, 'min_split_gain': 0.03038508731945985}. Best is trial 2 with value: 6.0330690040336144e-05.
[I 2025-06-15 19:58:04,293] A new study created in memory with name: no-name-6fa6cccd-2b37-404d-9788-ed2a1f290a95
[I 2025-06-15 19:58:04,328] Trial 0 finished with value: 0.00041756929613162574 and parameters: {'n_estimators': 176, 'learning_rate': 0.010658176691046729, 'num_leaves': 200, 'max_depth': 11, 'min_child_samples': 14, 'subsample': 0.9187572938332595, 'colsample_bytree': 0.9982119804578143, 'reg_alpha': 2.603226713045199, 'reg_lambda': 6.054074224231881, 'min_split_gain': 0.02579675471711573}. Best is trial 0 with value: 0.00041756929613162574.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         654
14      pct_chg_Load_DA         511
3         Load_DA_lag_0         408
1           Price_lag_2         396
15       lag168_Load_DA         281
5       WindOn_DA_lag_0         263
11           NGas_lag_2         245
2           Price_lag_7         218
16  roll2d_median_Price         189
13            EUA_lag_2         187
12            Oil_lag_2         175
10           Coal_lag_2         127
9                  WD_7          18
7                  WD_1          12
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 32.28% done
********************* END NS.. 235 ****************************************************
********************* START NS ... 236 ****************************************************
[I 2025-06-15 19:58:05,228] A new study created in memory with name: no-name-b397d593-e61b-4f8c-9845-69d00a70dad7
[I 2025-06-15 19:58:05,510] Trial 0 finished with value: 0.0025956243315563145 and parameters: {'n_estimators': 118, 'learning_rate': 0.022447994021318623, 'num_leaves': 96, 'max_depth': 9, 'min_child_samples': 13, 'subsample': 0.5702744866226109, 'colsample_bytree': 0.926336251917559, 'reg_alpha': 7.369764590160522, 'reg_lambda': 1.3144169803456351, 'min_split_gain': 0.02196635248499178}. Best is trial 0 with value: 0.0025956243315563145.
[Trial 0] New best: 0.00260
[I 2025-06-15 19:58:06,116] Trial 1 finished with value: 0.00010790436123011504 and parameters: {'n_estimators': 269, 'learning_rate': 0.04894512923128691, 'num_leaves': 121, 'max_depth': 8, 'min_child_samples': 3, 'subsample': 0.8202318414101866, 'colsample_bytree': 0.5430742047388111, 'reg_alpha': 1.726086290226121, 'reg_lambda': 9.208259653213887, 'min_split_gain': 0.04042873323068508}. Best is trial 1 with value: 0.00010790436123011504.
[Trial 1] New best: 0.00011
[I 2025-06-15 19:58:06,401] Trial 2 finished with value: 0.00010104531966611016 and parameters: {'n_estimators': 188, 'learning_rate': 0.10092973465098029, 'num_leaves': 232, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.8403970482514966, 'colsample_bytree': 0.8941656910074346, 'reg_alpha': 5.656224995616225, 'reg_lambda': 1.4932931230033408, 'min_split_gain': 0.0145062322448266}. Best is trial 2 with value: 0.00010104531966611016.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:06,660] Trial 3 finished with value: 0.00010924740781926145 and parameters: {'n_estimators': 200, 'learning_rate': 0.15780492853417408, 'num_leaves': 49, 'max_depth': 7, 'min_child_samples': 13, 'subsample': 0.5045692849375717, 'colsample_bytree': 0.7007052879646231, 'reg_alpha': 7.331002939498742, 'reg_lambda': 7.660813540435642, 'min_split_gain': 0.003109242048301192}. Best is trial 2 with value: 0.00010104531966611016.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:07,242] Trial 4 finished with value: 0.00022077904774825942 and parameters: {'n_estimators': 181, 'learning_rate': 0.023701882880043223, 'num_leaves': 47, 'max_depth': 7, 'min_child_samples': 29, 'subsample': 0.7507755654318976, 'colsample_bytree': 0.632404742631451, 'reg_alpha': 5.780149295143568, 'reg_lambda': 9.675972872489334, 'min_split_gain': 0.02075665472259062}. Best is trial 2 with value: 0.00010104531966611016.
[I 2025-06-15 19:58:07,245] A new study created in memory with name: no-name-4e86ebb8-3ea6-4395-a9b4-7d5e3604ebbf
[I 2025-06-15 19:58:07,262] Trial 0 finished with value: 0.0002613006187090893 and parameters: {'n_estimators': 50, 'learning_rate': 0.08361534178682102, 'num_leaves': 65, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.7739525426458184, 'colsample_bytree': 0.93014382193085, 'reg_alpha': 1.5240718186156788, 'reg_lambda': 0.33893362818748995, 'min_split_gain': 0.02636003136888795}. Best is trial 0 with value: 0.0002613006187090893.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         549
14      pct_chg_Load_DA         378
1           Price_lag_2         360
3         Load_DA_lag_0         244
11           NGas_lag_2         208
2           Price_lag_7         180
16  roll2d_median_Price         152
5       WindOn_DA_lag_0         148
10           Coal_lag_2         128
15       lag168_Load_DA         124
13            EUA_lag_2          93
12            Oil_lag_2          83
9                  WD_7          13
7                  WD_1          10
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 32.42% done
********************* END NS.. 236 ****************************************************
********************* START NS ... 237 ****************************************************
[I 2025-06-15 19:58:08,200] A new study created in memory with name: no-name-8ccf5656-218e-4a4d-a609-543006b1f498
[I 2025-06-15 19:58:08,467] Trial 0 finished with value: 0.0009287413545005912 and parameters: {'n_estimators': 106, 'learning_rate': 0.063357999861545, 'num_leaves': 222, 'max_depth': 10, 'min_child_samples': 5, 'subsample': 0.6107742809769676, 'colsample_bytree': 0.7000557133619629, 'reg_alpha': 9.803271120838295, 'reg_lambda': 4.335927053535294, 'min_split_gain': 0.03827242621580262}. Best is trial 0 with value: 0.0009287413545005912.
[Trial 0] New best: 0.00093
[I 2025-06-15 19:58:08,759] Trial 1 finished with value: 0.001095160334154446 and parameters: {'n_estimators': 216, 'learning_rate': 0.1524476983259426, 'num_leaves': 127, 'max_depth': 5, 'min_child_samples': 17, 'subsample': 0.7166664953754687, 'colsample_bytree': 0.6496851032309066, 'reg_alpha': 0.09112620176245922, 'reg_lambda': 4.561246742338335, 'min_split_gain': 0.016976186563674268}. Best is trial 0 with value: 0.0009287413545005912.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:09,728] Trial 2 finished with value: 0.0009170761251808555 and parameters: {'n_estimators': 259, 'learning_rate': 0.0266979543069398, 'num_leaves': 171, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.6058288804609508, 'colsample_bytree': 0.6981155416942608, 'reg_alpha': 1.7693957378556757, 'reg_lambda': 0.9572355532551191, 'min_split_gain': 0.020362826671950808}. Best is trial 2 with value: 0.0009170761251808555.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:09,982] Trial 3 finished with value: 0.0015851975005562464 and parameters: {'n_estimators': 126, 'learning_rate': 0.027438842725829127, 'num_leaves': 215, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.6889835630935075, 'colsample_bytree': 0.9923623838460311, 'reg_alpha': 7.909989844388534, 'reg_lambda': 2.6513198760278414, 'min_split_gain': 0.029095223258052125}. Best is trial 2 with value: 0.0009170761251808555.
[I 2025-06-15 19:58:09,984] A new study created in memory with name: no-name-a1791bac-b418-467c-816f-682ef26aed62
[I 2025-06-15 19:58:10,020] Trial 0 finished with value: 0.0010791446106469193 and parameters: {'n_estimators': 175, 'learning_rate': 0.03241532205174503, 'num_leaves': 138, 'max_depth': 12, 'min_child_samples': 20, 'subsample': 0.7123459645299453, 'colsample_bytree': 0.9091134734443094, 'reg_alpha': 8.429377081351767, 'reg_lambda': 9.053606312673272, 'min_split_gain': 0.010538203330722485}. Best is trial 0 with value: 0.0010791446106469193.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2171
1           Price_lag_2        1676
14      pct_chg_Load_DA        1249
3         Load_DA_lag_0         827
16  roll2d_median_Price         622
11           NGas_lag_2         591
2           Price_lag_7         549
15       lag168_Load_DA         503
13            EUA_lag_2         390
10           Coal_lag_2         386
5       WindOn_DA_lag_0         358
12            Oil_lag_2         317
7                  WD_1          42
9                  WD_7          32
8                  WD_6           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 32.56% done
********************* END NS.. 237 ****************************************************
********************* START NS ... 238 ****************************************************
[I 2025-06-15 19:58:11,137] A new study created in memory with name: no-name-6aace2e3-4a02-4a41-971a-853ef03540ef
[I 2025-06-15 19:58:11,421] Trial 0 finished with value: 0.002929473715718587 and parameters: {'n_estimators': 141, 'learning_rate': 0.021998927317166395, 'num_leaves': 126, 'max_depth': 5, 'min_child_samples': 13, 'subsample': 0.6251833188414622, 'colsample_bytree': 0.7885707666976216, 'reg_alpha': 7.851089695576846, 'reg_lambda': 3.157925748218764, 'min_split_gain': 0.019797728215553947}. Best is trial 0 with value: 0.002929473715718587.
[I 2025-06-15 19:58:11,611] Trial 1 finished with value: 0.022220168862151942 and parameters: {'n_estimators': 93, 'learning_rate': 0.01911922187310328, 'num_leaves': 80, 'max_depth': 4, 'min_child_samples': 1, 'subsample': 0.9272656363199909, 'colsample_bytree': 0.6210419391211193, 'reg_alpha': 1.99543818870334, 'reg_lambda': 8.974105914460804, 'min_split_gain': 0.04323724317207528}. Best is trial 0 with value: 0.002929473715718587.
[Trial 0] New best: 0.00293
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:11,927] Trial 2 finished with value: 0.0014143384802343475 and parameters: {'n_estimators': 104, 'learning_rate': 0.2999912488159743, 'num_leaves': 92, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.8926882087753889, 'colsample_bytree': 0.6198326473245814, 'reg_alpha': 7.9450224977895445, 'reg_lambda': 8.034164431139803, 'min_split_gain': 0.00012973583212363415}. Best is trial 2 with value: 0.0014143384802343475.
[Trial 2] New best: 0.00141
[I 2025-06-15 19:58:12,208] Trial 3 finished with value: 0.001423790705761304 and parameters: {'n_estimators': 185, 'learning_rate': 0.023634656351922866, 'num_leaves': 149, 'max_depth': 4, 'min_child_samples': 21, 'subsample': 0.9329730008184709, 'colsample_bytree': 0.6844761874451286, 'reg_alpha': 3.313010567730653, 'reg_lambda': 3.9499694532953877, 'min_split_gain': 0.025354329834068226}. Best is trial 2 with value: 0.0014143384802343475.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:12,504] Trial 4 finished with value: 0.0015025210641641557 and parameters: {'n_estimators': 182, 'learning_rate': 0.14411129497078823, 'num_leaves': 99, 'max_depth': 12, 'min_child_samples': 9, 'subsample': 0.9312610083203998, 'colsample_bytree': 0.9813947604419908, 'reg_alpha': 2.353921768649206, 'reg_lambda': 5.281444697751296, 'min_split_gain': 0.02333536620914367}. Best is trial 2 with value: 0.0014143384802343475.
[I 2025-06-15 19:58:12,686] Trial 5 finished with value: 0.0015131236696765147 and parameters: {'n_estimators': 157, 'learning_rate': 0.17163972163894342, 'num_leaves': 215, 'max_depth': 12, 'min_child_samples': 6, 'subsample': 0.7695225755493544, 'colsample_bytree': 0.7208060460036134, 'reg_alpha': 4.88290933962224, 'reg_lambda': 2.3745046054436902, 'min_split_gain': 0.04963357148083507}. Best is trial 2 with value: 0.0014143384802343475.
[I 2025-06-15 19:58:12,689] A new study created in memory with name: no-name-b0ca87ae-2cf1-4293-99c4-ace34b8ffa2c
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 19:58:12,713] Trial 0 finished with value: 0.00299309030828233 and parameters: {'n_estimators': 260, 'learning_rate': 0.022513064011413727, 'num_leaves': 180, 'max_depth': 13, 'min_child_samples': 19, 'subsample': 0.6095169947020622, 'colsample_bytree': 0.7557615918615128, 'reg_alpha': 2.1577754404213625, 'reg_lambda': 6.2132941070005865, 'min_split_gain': 0.02366420974457811}. Best is trial 0 with value: 0.00299309030828233.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         405
1           Price_lag_2         372
2           Price_lag_7         222
14      pct_chg_Load_DA         221
3         Load_DA_lag_0         192
10           Coal_lag_2         170
13            EUA_lag_2         170
5       WindOn_DA_lag_0         165
15       lag168_Load_DA         161
11           NGas_lag_2         140
16  roll2d_median_Price         138
12            Oil_lag_2         111
9                  WD_7          14
7                  WD_1           7
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 32.69% done
********************* END NS.. 238 ****************************************************
********************* START NS ... 239 ****************************************************
[I 2025-06-15 19:58:13,452] A new study created in memory with name: no-name-7bb51314-4ec7-4e6d-9b87-ace1740375fe
[I 2025-06-15 19:58:14,365] Trial 0 finished with value: 0.0007460970617116642 and parameters: {'n_estimators': 134, 'learning_rate': 0.08416855265873216, 'num_leaves': 183, 'max_depth': 16, 'min_child_samples': 17, 'subsample': 0.6408220101315628, 'colsample_bytree': 0.8046756471741063, 'reg_alpha': 0.5600437134234426, 'reg_lambda': 9.375543830429002, 'min_split_gain': 0.004016327699034356}. Best is trial 0 with value: 0.0007460970617116642.
[I 2025-06-15 19:58:14,547] Trial 1 finished with value: 0.0008754015841236496 and parameters: {'n_estimators': 272, 'learning_rate': 0.2794196731288273, 'num_leaves': 228, 'max_depth': 5, 'min_child_samples': 24, 'subsample': 0.5658005895083873, 'colsample_bytree': 0.8555464436376483, 'reg_alpha': 1.3482902565323096, 'reg_lambda': 4.68262296695727, 'min_split_gain': 0.02733120179280676}. Best is trial 0 with value: 0.0007460970617116642.
[Trial 0] New best: 0.00075
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:14,776] Trial 2 finished with value: 0.0012471467823865622 and parameters: {'n_estimators': 175, 'learning_rate': 0.29622167673696204, 'num_leaves': 53, 'max_depth': 14, 'min_child_samples': 11, 'subsample': 0.620081588050396, 'colsample_bytree': 0.578426544909115, 'reg_alpha': 3.1323980404779705, 'reg_lambda': 1.202885490998088, 'min_split_gain': 0.00810785315276526}. Best is trial 0 with value: 0.0007460970617116642.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:15,141] Trial 3 finished with value: 0.0014463122118342359 and parameters: {'n_estimators': 103, 'learning_rate': 0.03327999494052776, 'num_leaves': 226, 'max_depth': 13, 'min_child_samples': 1, 'subsample': 0.974913485685151, 'colsample_bytree': 0.845190404635778, 'reg_alpha': 8.331048652234527, 'reg_lambda': 0.043553877172274325, 'min_split_gain': 0.040696934555596434}. Best is trial 0 with value: 0.0007460970617116642.
[I 2025-06-15 19:58:15,144] A new study created in memory with name: no-name-34d1e0b3-5de9-496f-82ff-b67730663802
[I 2025-06-15 19:58:15,181] Trial 0 finished with value: 0.003833848586428948 and parameters: {'n_estimators': 243, 'learning_rate': 0.03327506653347782, 'num_leaves': 200, 'max_depth': 11, 'min_child_samples': 30, 'subsample': 0.5094180206531379, 'colsample_bytree': 0.5019966712337147, 'reg_alpha': 0.06502740935231954, 'reg_lambda': 1.4243007087243698, 'min_split_gain': 0.04069239846634151}. Best is trial 0 with value: 0.003833848586428948.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA        2283
0           Price_lag_1        2014
1           Price_lag_2        1957
3         Load_DA_lag_0        1860
15       lag168_Load_DA        1571
5       WindOn_DA_lag_0        1540
2           Price_lag_7        1422
16  roll2d_median_Price        1107
11           NGas_lag_2        1049
12            Oil_lag_2         790
13            EUA_lag_2         787
10           Coal_lag_2         753
9                  WD_7          49
7                  WD_1          41
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          15
15       lag168_Load_DA           2
10           Coal_lag_2           2
1           Price_lag_2           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 32.83% done
********************* END NS.. 239 ****************************************************
********************* START NS ... 240 ****************************************************
[I 2025-06-15 19:58:16,617] A new study created in memory with name: no-name-c99ca126-9253-4b8f-a09b-cdb4fa682061
[I 2025-06-15 19:58:17,372] Trial 0 finished with value: 0.0008824145725827912 and parameters: {'n_estimators': 169, 'learning_rate': 0.05567896575510295, 'num_leaves': 161, 'max_depth': 14, 'min_child_samples': 2, 'subsample': 0.7248518937061759, 'colsample_bytree': 0.6324287215389885, 'reg_alpha': 4.594292047857939, 'reg_lambda': 7.700853176169609, 'min_split_gain': 0.023613105816137077}. Best is trial 0 with value: 0.0008824145725827912.
[Trial 0] New best: 0.00088
[I 2025-06-15 19:58:17,863] Trial 1 finished with value: 0.0009818778556662832 and parameters: {'n_estimators': 256, 'learning_rate': 0.02968936602137379, 'num_leaves': 209, 'max_depth': 4, 'min_child_samples': 22, 'subsample': 0.5763357462818355, 'colsample_bytree': 0.7868337781610348, 'reg_alpha': 5.884599920637383, 'reg_lambda': 5.711746469697277, 'min_split_gain': 0.011900214464567661}. Best is trial 0 with value: 0.0008824145725827912.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:18,478] Trial 2 finished with value: 0.0009467205551009474 and parameters: {'n_estimators': 231, 'learning_rate': 0.037353112856301336, 'num_leaves': 37, 'max_depth': 15, 'min_child_samples': 19, 'subsample': 0.7569974448186338, 'colsample_bytree': 0.5206679988240441, 'reg_alpha': 8.389738576494404, 'reg_lambda': 1.9873981223795245, 'min_split_gain': 0.009266613401633312}. Best is trial 0 with value: 0.0008824145725827912.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:19,164] Trial 3 finished with value: 0.0012161512849125837 and parameters: {'n_estimators': 186, 'learning_rate': 0.07003015724173199, 'num_leaves': 229, 'max_depth': 14, 'min_child_samples': 17, 'subsample': 0.6345149538100636, 'colsample_bytree': 0.9552176193072909, 'reg_alpha': 5.8698969796620055, 'reg_lambda': 1.2796912599325372, 'min_split_gain': 0.0049573445226098415}. Best is trial 0 with value: 0.0008824145725827912.
[I 2025-06-15 19:58:19,167] A new study created in memory with name: no-name-94691074-385a-4089-88e5-525f756b9b7d
[I 2025-06-15 19:58:19,185] Trial 0 finished with value: 0.0055000600841260084 and parameters: {'n_estimators': 75, 'learning_rate': 0.24558111424815704, 'num_leaves': 251, 'max_depth': 8, 'min_child_samples': 24, 'subsample': 0.6273846504232793, 'colsample_bytree': 0.8137469143014946, 'reg_alpha': 5.203535882416649, 'reg_lambda': 4.774979882748472, 'min_split_gain': 0.013761214073558809}. Best is trial 0 with value: 0.0055000600841260084.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1036
1           Price_lag_2         941
14      pct_chg_Load_DA         643
2           Price_lag_7         532
3         Load_DA_lag_0         514
11           NGas_lag_2         475
15       lag168_Load_DA         437
5       WindOn_DA_lag_0         394
16  roll2d_median_Price         369
10           Coal_lag_2         346
13            EUA_lag_2         323
12            Oil_lag_2         209
9                  WD_7          37
7                  WD_1          19
8                  WD_6          16
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 32.97% done
********************* END NS.. 240 ****************************************************
********************* START NS ... 241 ****************************************************
[I 2025-06-15 19:58:20,289] A new study created in memory with name: no-name-2c18fb3c-3f0d-4b6c-b13b-02881a26f6a0
[I 2025-06-15 19:58:20,509] Trial 0 finished with value: 0.0031426271877442054 and parameters: {'n_estimators': 157, 'learning_rate': 0.017424683314973023, 'num_leaves': 157, 'max_depth': 4, 'min_child_samples': 20, 'subsample': 0.6336302835056786, 'colsample_bytree': 0.6961130296903857, 'reg_alpha': 6.92888438711904, 'reg_lambda': 9.194400050985463, 'min_split_gain': 0.0347088126434858}. Best is trial 0 with value: 0.0031426271877442054.
[Trial 0] New best: 0.00314
[I 2025-06-15 19:58:20,771] Trial 1 finished with value: 0.0013040687914629362 and parameters: {'n_estimators': 100, 'learning_rate': 0.06391965267138547, 'num_leaves': 240, 'max_depth': 5, 'min_child_samples': 30, 'subsample': 0.5870871378827613, 'colsample_bytree': 0.6673695559537176, 'reg_alpha': 8.296248931329137, 'reg_lambda': 7.954438109812292, 'min_split_gain': 0.02453797357492545}. Best is trial 1 with value: 0.0013040687914629362.
[Trial 1] New best: 0.00130
[I 2025-06-15 19:58:20,986] Trial 2 finished with value: 0.0017469554265337516 and parameters: {'n_estimators': 200, 'learning_rate': 0.29032670917095077, 'num_leaves': 82, 'max_depth': 15, 'min_child_samples': 23, 'subsample': 0.8104884898500745, 'colsample_bytree': 0.835450377274836, 'reg_alpha': 5.958818243430594, 'reg_lambda': 8.151835470714836, 'min_split_gain': 0.020693326624704585}. Best is trial 1 with value: 0.0013040687914629362.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:21,387] Trial 3 finished with value: 0.0013774627243002722 and parameters: {'n_estimators': 191, 'learning_rate': 0.10628812314669152, 'num_leaves': 194, 'max_depth': 9, 'min_child_samples': 10, 'subsample': 0.7641434834399108, 'colsample_bytree': 0.5411528954618963, 'reg_alpha': 3.37229151215673, 'reg_lambda': 3.5754628692969637, 'min_split_gain': 0.011395286376504405}. Best is trial 1 with value: 0.0013040687914629362.
[I 2025-06-15 19:58:21,545] Trial 4 finished with value: 0.0013682089853557084 and parameters: {'n_estimators': 255, 'learning_rate': 0.23319248801056874, 'num_leaves': 119, 'max_depth': 4, 'min_child_samples': 3, 'subsample': 0.7086524187334446, 'colsample_bytree': 0.9960552561172096, 'reg_alpha': 5.698683254969675, 'reg_lambda': 4.75807425154454, 'min_split_gain': 0.03142443334958186}. Best is trial 1 with value: 0.0013040687914629362.
[I 2025-06-15 19:58:21,547] A new study created in memory with name: no-name-9f06b7e9-dcc1-4c1a-911b-88a8817a93ab
[I 2025-06-15 19:58:21,569] Trial 0 finished with value: 0.007032509665980837 and parameters: {'n_estimators': 246, 'learning_rate': 0.2685421741560436, 'num_leaves': 36, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.9480642274717352, 'colsample_bytree': 0.5110921318982942, 'reg_alpha': 7.663992769072098, 'reg_lambda': 9.31511030641516, 'min_split_gain': 0.037682859360687854}. Best is trial 0 with value: 0.007032509665980837.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         543
1           Price_lag_2         289
14      pct_chg_Load_DA         247
3         Load_DA_lag_0         141
11           NGas_lag_2         120
16  roll2d_median_Price         105
2           Price_lag_7          92
15       lag168_Load_DA          72
13            EUA_lag_2          65
10           Coal_lag_2          50
5       WindOn_DA_lag_0          45
12            Oil_lag_2          40
9                  WD_7          10
7                  WD_1           2
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 33.11% done
********************* END NS.. 241 ****************************************************
********************* START NS ... 242 ****************************************************
[I 2025-06-15 19:58:22,248] A new study created in memory with name: no-name-fa5aa485-9c01-4291-9d76-c98817125c8b
[I 2025-06-15 19:58:22,637] Trial 0 finished with value: 0.0011386039057437972 and parameters: {'n_estimators': 295, 'learning_rate': 0.03970820830653616, 'num_leaves': 158, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.9696339245143591, 'colsample_bytree': 0.6285140059509947, 'reg_alpha': 9.197249349712429, 'reg_lambda': 2.256260170556814, 'min_split_gain': 0.03203745055436904}. Best is trial 0 with value: 0.0011386039057437972.
[Trial 0] New best: 0.00114
[I 2025-06-15 19:58:23,276] Trial 1 finished with value: 0.0010783377810502533 and parameters: {'n_estimators': 236, 'learning_rate': 0.06176236274145567, 'num_leaves': 71, 'max_depth': 8, 'min_child_samples': 22, 'subsample': 0.9572344746375394, 'colsample_bytree': 0.6167130762452748, 'reg_alpha': 0.08494975356241108, 'reg_lambda': 3.6486202887353247, 'min_split_gain': 0.02352790545723531}. Best is trial 1 with value: 0.0010783377810502533.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:23,964] Trial 2 finished with value: 0.0010721788683168453 and parameters: {'n_estimators': 267, 'learning_rate': 0.02379786588073006, 'num_leaves': 161, 'max_depth': 16, 'min_child_samples': 24, 'subsample': 0.9525849620864549, 'colsample_bytree': 0.8627575820587916, 'reg_alpha': 8.155270403215564, 'reg_lambda': 6.370433578826384, 'min_split_gain': 0.02657308200271858}. Best is trial 2 with value: 0.0010721788683168453.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:24,618] Trial 3 finished with value: 0.0010185710311025854 and parameters: {'n_estimators': 180, 'learning_rate': 0.06989025772457375, 'num_leaves': 162, 'max_depth': 14, 'min_child_samples': 7, 'subsample': 0.8566480744139704, 'colsample_bytree': 0.9714946800645134, 'reg_alpha': 9.75295620945603, 'reg_lambda': 8.002941245084696, 'min_split_gain': 0.0015657625899773487}. Best is trial 3 with value: 0.0010185710311025854.
[I 2025-06-15 19:58:24,623] A new study created in memory with name: no-name-7bb11005-f1b2-44c3-afa8-5b0fa398f8f8
[I 2025-06-15 19:58:24,693] Trial 0 finished with value: 0.0027154218024999567 and parameters: {'n_estimators': 236, 'learning_rate': 0.19852632251578217, 'num_leaves': 223, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.5494801488746739, 'colsample_bytree': 0.9575743329239436, 'reg_alpha': 5.0622201934580024, 'reg_lambda': 9.281043103992811, 'min_split_gain': 0.017336021838662157}. Best is trial 0 with value: 0.0027154218024999567.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1005
14      pct_chg_Load_DA         805
1           Price_lag_2         629
3         Load_DA_lag_0         564
5       WindOn_DA_lag_0         505
2           Price_lag_7         474
13            EUA_lag_2         421
11           NGas_lag_2         402
10           Coal_lag_2         399
16  roll2d_median_Price         399
15       lag168_Load_DA         389
12            Oil_lag_2         287
9                  WD_7          42
8                  WD_6          28
7                  WD_1           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 33.24% done
********************* END NS.. 242 ****************************************************
********************* START NS ... 243 ****************************************************
[I 2025-06-15 19:58:28,814] A new study created in memory with name: no-name-c6d30d67-42e0-4efe-b36f-186ce50ff6ee
[I 2025-06-15 19:58:29,110] Trial 0 finished with value: 0.0007316770976610247 and parameters: {'n_estimators': 244, 'learning_rate': 0.18960187995336114, 'num_leaves': 114, 'max_depth': 8, 'min_child_samples': 4, 'subsample': 0.6961673105379111, 'colsample_bytree': 0.6119060014501188, 'reg_alpha': 7.239829104428557, 'reg_lambda': 2.231725554266971, 'min_split_gain': 0.046881632938534645}. Best is trial 0 with value: 0.0007316770976610247.
[Trial 0] New best: 0.00073
[I 2025-06-15 19:58:29,323] Trial 1 finished with value: 0.0007648060227056537 and parameters: {'n_estimators': 94, 'learning_rate': 0.08542945687863741, 'num_leaves': 152, 'max_depth': 4, 'min_child_samples': 7, 'subsample': 0.5641973165820753, 'colsample_bytree': 0.8484786071271015, 'reg_alpha': 0.3796883986563393, 'reg_lambda': 3.307201343465654, 'min_split_gain': 0.017360927818292328}. Best is trial 0 with value: 0.0007316770976610247.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:30,152] Trial 2 finished with value: 0.0008010291726897886 and parameters: {'n_estimators': 210, 'learning_rate': 0.025485065772550247, 'num_leaves': 248, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.8829483534929845, 'colsample_bytree': 0.6353228836371325, 'reg_alpha': 6.614680331760949, 'reg_lambda': 3.6742094600104735, 'min_split_gain': 0.006244026868328029}. Best is trial 0 with value: 0.0007316770976610247.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:31,116] Trial 3 finished with value: 0.0008980468775242864 and parameters: {'n_estimators': 174, 'learning_rate': 0.0264492410649849, 'num_leaves': 192, 'max_depth': 8, 'min_child_samples': 1, 'subsample': 0.8879328911731252, 'colsample_bytree': 0.6306130014096136, 'reg_alpha': 4.628000857208177, 'reg_lambda': 3.930150306096434, 'min_split_gain': 0.049947977010720374}. Best is trial 0 with value: 0.0007316770976610247.
[I 2025-06-15 19:58:31,118] A new study created in memory with name: no-name-1f5ab6ff-fc66-4ec9-bcea-4e10962f07b6
[I 2025-06-15 19:58:31,177] Trial 0 finished with value: 0.0012833452876752397 and parameters: {'n_estimators': 255, 'learning_rate': 0.021559640637590707, 'num_leaves': 232, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.579805796335465, 'colsample_bytree': 0.6329058614587211, 'reg_alpha': 9.589143341736184, 'reg_lambda': 6.861936407339181, 'min_split_gain': 0.010945640577752064}. Best is trial 0 with value: 0.0012833452876752397.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         226
1           Price_lag_2         149
14      pct_chg_Load_DA          92
2           Price_lag_7          88
11           NGas_lag_2          74
3         Load_DA_lag_0          71
16  roll2d_median_Price          65
15       lag168_Load_DA          59
10           Coal_lag_2          45
5       WindOn_DA_lag_0          37
12            Oil_lag_2          29
13            EUA_lag_2          28
9                  WD_7          10
7                  WD_1           3
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 33.38% done
********************* END NS.. 243 ****************************************************
********************* START NS ... 244 ****************************************************
[I 2025-06-15 19:58:32,043] A new study created in memory with name: no-name-89585ebd-bc26-4e02-8d36-a1305be54091
[I 2025-06-15 19:58:32,598] Trial 0 finished with value: 0.01056102000907115 and parameters: {'n_estimators': 195, 'learning_rate': 0.011178163563519443, 'num_leaves': 231, 'max_depth': 11, 'min_child_samples': 15, 'subsample': 0.9051230841282936, 'colsample_bytree': 0.8009974525443216, 'reg_alpha': 8.777596887714056, 'reg_lambda': 4.368487833757467, 'min_split_gain': 0.002345114649865854}. Best is trial 0 with value: 0.01056102000907115.
[Trial 0] New best: 0.01056
[I 2025-06-15 19:58:33,175] Trial 1 finished with value: 0.0011686800512455912 and parameters: {'n_estimators': 137, 'learning_rate': 0.036749011607027586, 'num_leaves': 53, 'max_depth': 9, 'min_child_samples': 3, 'subsample': 0.8912097351621722, 'colsample_bytree': 0.8745230499374042, 'reg_alpha': 6.773748199078885, 'reg_lambda': 7.611295497487205, 'min_split_gain': 0.01150078832677985}. Best is trial 1 with value: 0.0011686800512455912.
[Trial 1] New best: 0.00117
[I 2025-06-15 19:58:33,907] Trial 2 finished with value: 0.001126633341054383 and parameters: {'n_estimators': 258, 'learning_rate': 0.02042503873005145, 'num_leaves': 174, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.5549275359618921, 'colsample_bytree': 0.5150629703791632, 'reg_alpha': 2.1124753372617153, 'reg_lambda': 8.699105092285901, 'min_split_gain': 0.0476771796021391}. Best is trial 2 with value: 0.001126633341054383.
[I 2025-06-15 19:58:34,084] Trial 3 finished with value: 0.18761086898740445 and parameters: {'n_estimators': 58, 'learning_rate': 0.011334232353254594, 'num_leaves': 142, 'max_depth': 5, 'min_child_samples': 24, 'subsample': 0.8728717421763283, 'colsample_bytree': 0.9388943426048617, 'reg_alpha': 6.614888348058382, 'reg_lambda': 4.674597877788907, 'min_split_gain': 0.010887341313870437}. Best is trial 2 with value: 0.001126633341054383.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:34,721] Trial 4 finished with value: 0.001078113031898095 and parameters: {'n_estimators': 180, 'learning_rate': 0.0477620517612547, 'num_leaves': 54, 'max_depth': 10, 'min_child_samples': 27, 'subsample': 0.7992321085329559, 'colsample_bytree': 0.9751327951174813, 'reg_alpha': 7.511171026238337, 'reg_lambda': 7.828038542180842, 'min_split_gain': 0.003227594781735366}. Best is trial 4 with value: 0.001078113031898095.
[I 2025-06-15 19:58:34,723] A new study created in memory with name: no-name-a786e498-64ab-442f-9089-6324dc4590a5
[I 2025-06-15 19:58:34,756] Trial 0 finished with value: 0.007440562043188048 and parameters: {'n_estimators': 125, 'learning_rate': 0.04788432265195965, 'num_leaves': 229, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.9677974759930659, 'colsample_bytree': 0.7267236673526807, 'reg_alpha': 2.35235410972923, 'reg_lambda': 2.8732128017521, 'min_split_gain': 0.005986884374321983}. Best is trial 0 with value: 0.007440562043188048.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1267
14      pct_chg_Load_DA         915
1           Price_lag_2         702
3         Load_DA_lag_0         569
2           Price_lag_7         440
11           NGas_lag_2         420
10           Coal_lag_2         395
5       WindOn_DA_lag_0         384
16  roll2d_median_Price         373
13            EUA_lag_2         361
12            Oil_lag_2         315
15       lag168_Load_DA         274
9                  WD_7          31
8                  WD_6          22
7                  WD_1          15
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          27
10           Coal_lag_2           5
3         Load_DA_lag_0           2
1           Price_lag_2           1
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 33.52% done
********************* END NS.. 244 ****************************************************
********************* START NS ... 245 ****************************************************
[I 2025-06-15 19:58:35,908] A new study created in memory with name: no-name-13c91464-15fb-4f5d-a781-04fc4536fd45
[I 2025-06-15 19:58:36,185] Trial 0 finished with value: 0.0014890759391571254 and parameters: {'n_estimators': 275, 'learning_rate': 0.0767152075625074, 'num_leaves': 222, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.6708795064981465, 'colsample_bytree': 0.8351991598532571, 'reg_alpha': 5.3742595478999675, 'reg_lambda': 8.076083378684686, 'min_split_gain': 0.022959129912477144}. Best is trial 0 with value: 0.0014890759391571254.
[Trial 0] New best: 0.00149
[I 2025-06-15 19:58:36,401] Trial 1 finished with value: 0.0011579486530381197 and parameters: {'n_estimators': 124, 'learning_rate': 0.14244690060193954, 'num_leaves': 88, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.8138707604887597, 'colsample_bytree': 0.9260096398501678, 'reg_alpha': 3.0237697942643837, 'reg_lambda': 3.6122634520269847, 'min_split_gain': 0.04585490672336452}. Best is trial 1 with value: 0.0011579486530381197.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:36,885] Trial 2 finished with value: 0.0015530399049423248 and parameters: {'n_estimators': 245, 'learning_rate': 0.08355316327039448, 'num_leaves': 98, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.8338099223822932, 'colsample_bytree': 0.8950505254291143, 'reg_alpha': 0.8728381224352244, 'reg_lambda': 2.8290712061300582, 'min_split_gain': 0.007648784184015107}. Best is trial 1 with value: 0.0011579486530381197.
[I 2025-06-15 19:58:37,063] Trial 3 finished with value: 0.0006146177231126409 and parameters: {'n_estimators': 83, 'learning_rate': 0.0893778471660871, 'num_leaves': 224, 'max_depth': 4, 'min_child_samples': 3, 'subsample': 0.8653529380319753, 'colsample_bytree': 0.8942386074178138, 'reg_alpha': 4.526874972258126, 'reg_lambda': 8.56644935690711, 'min_split_gain': 0.016049121181019234}. Best is trial 3 with value: 0.0006146177231126409.
[I 2025-06-15 19:58:37,065] A new study created in memory with name: no-name-2d0d8e51-8e81-48bf-9132-785d840f3227
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 19:58:37,082] Trial 0 finished with value: 0.026197416847549327 and parameters: {'n_estimators': 62, 'learning_rate': 0.013920333868661284, 'num_leaves': 249, 'max_depth': 13, 'min_child_samples': 2, 'subsample': 0.9132436274411644, 'colsample_bytree': 0.6574747664329472, 'reg_alpha': 5.02975464663538, 'reg_lambda': 8.923805371926248, 'min_split_gain': 0.02761231682818362}. Best is trial 0 with value: 0.026197416847549327.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         393
14      pct_chg_Load_DA         175
1           Price_lag_2         120
3         Load_DA_lag_0          80
11           NGas_lag_2          72
2           Price_lag_7          67
16  roll2d_median_Price          56
12            Oil_lag_2          41
13            EUA_lag_2          39
10           Coal_lag_2          38
15       lag168_Load_DA          37
5       WindOn_DA_lag_0          22
7                  WD_1           5
9                  WD_7           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 33.65% done
********************* END NS.. 245 ****************************************************
********************* START NS ... 246 ****************************************************
[I 2025-06-15 19:58:37,847] A new study created in memory with name: no-name-013e905f-5722-4dbc-b6a5-0865d907cd6a
[I 2025-06-15 19:58:38,167] Trial 0 finished with value: 0.0002819599211217342 and parameters: {'n_estimators': 240, 'learning_rate': 0.11689885870849918, 'num_leaves': 100, 'max_depth': 13, 'min_child_samples': 2, 'subsample': 0.6268255362045285, 'colsample_bytree': 0.5201407596796782, 'reg_alpha': 3.023815107880824, 'reg_lambda': 9.774879717670423, 'min_split_gain': 0.03420836237173742}. Best is trial 0 with value: 0.0002819599211217342.
[Trial 0] New best: 0.00028
[I 2025-06-15 19:58:38,559] Trial 1 finished with value: 0.18792914754404785 and parameters: {'n_estimators': 65, 'learning_rate': 0.012629324414383287, 'num_leaves': 199, 'max_depth': 10, 'min_child_samples': 19, 'subsample': 0.9983264383526057, 'colsample_bytree': 0.976798766974255, 'reg_alpha': 5.4747878627155675, 'reg_lambda': 0.7693146912377125, 'min_split_gain': 0.004705369897681073}. Best is trial 0 with value: 0.0002819599211217342.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:38,882] Trial 2 finished with value: 0.19130232414802814 and parameters: {'n_estimators': 72, 'learning_rate': 0.011603469787245642, 'num_leaves': 77, 'max_depth': 13, 'min_child_samples': 23, 'subsample': 0.7092489923714727, 'colsample_bytree': 0.5485138145835565, 'reg_alpha': 7.977974938717507, 'reg_lambda': 9.189465343604601, 'min_split_gain': 0.03989103758333329}. Best is trial 0 with value: 0.0002819599211217342.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:39,653] Trial 3 finished with value: 0.00028890051409462345 and parameters: {'n_estimators': 226, 'learning_rate': 0.02667878733245449, 'num_leaves': 155, 'max_depth': 12, 'min_child_samples': 7, 'subsample': 0.7506162437624004, 'colsample_bytree': 0.6884911463532271, 'reg_alpha': 5.245013598773443, 'reg_lambda': 9.217991290279082, 'min_split_gain': 0.033564478390689885}. Best is trial 0 with value: 0.0002819599211217342.
[I 2025-06-15 19:58:39,656] A new study created in memory with name: no-name-f2650b02-71a6-442d-8270-274738d2c216
[I 2025-06-15 19:58:39,695] Trial 0 finished with value: 0.032834985250723814 and parameters: {'n_estimators': 263, 'learning_rate': 0.08458018663517966, 'num_leaves': 64, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.7029688442668524, 'colsample_bytree': 0.510832695466854, 'reg_alpha': 6.165743624366583, 'reg_lambda': 4.6580101065961355, 'min_split_gain': 0.01564886031382558}. Best is trial 0 with value: 0.032834985250723814.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         547
1           Price_lag_2         519
14      pct_chg_Load_DA         337
3         Load_DA_lag_0         307
2           Price_lag_7         289
16  roll2d_median_Price         265
15       lag168_Load_DA         220
5       WindOn_DA_lag_0         219
11           NGas_lag_2         218
10           Coal_lag_2         195
13            EUA_lag_2         168
12            Oil_lag_2         129
7                  WD_1          18
8                  WD_6          17
9                  WD_7          13
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           4
10           Coal_lag_2           1
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 33.79% done
********************* END NS.. 246 ****************************************************
********************* START NS ... 247 ****************************************************
[I 2025-06-15 19:58:40,757] A new study created in memory with name: no-name-4c172651-5795-4ba6-8a48-12aa5e1f891c
[I 2025-06-15 19:58:40,915] Trial 0 finished with value: 0.0007183549820239974 and parameters: {'n_estimators': 186, 'learning_rate': 0.19133800695609152, 'num_leaves': 179, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.8074478867933853, 'colsample_bytree': 0.6977590660754838, 'reg_alpha': 9.290540931655823, 'reg_lambda': 1.6350227920081684, 'min_split_gain': 0.03323867593062877}. Best is trial 0 with value: 0.0007183549820239974.
[Trial 0] New best: 0.00072
[I 2025-06-15 19:58:41,144] Trial 1 finished with value: 0.00016728705192362709 and parameters: {'n_estimators': 279, 'learning_rate': 0.2657892759349952, 'num_leaves': 116, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.6093610395023769, 'colsample_bytree': 0.6223589266702881, 'reg_alpha': 5.524084907817139, 'reg_lambda': 3.4049832435819214, 'min_split_gain': 0.024209222373226903}. Best is trial 1 with value: 0.00016728705192362709.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:41,486] Trial 2 finished with value: 0.003659544605917567 and parameters: {'n_estimators': 172, 'learning_rate': 0.01921725866529145, 'num_leaves': 75, 'max_depth': 4, 'min_child_samples': 2, 'subsample': 0.7736665565816818, 'colsample_bytree': 0.9953304374331615, 'reg_alpha': 4.704041323612619, 'reg_lambda': 2.620055953447137, 'min_split_gain': 0.026923166185077674}. Best is trial 1 with value: 0.00016728705192362709.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:41,773] Trial 3 finished with value: 0.0006644420993024208 and parameters: {'n_estimators': 261, 'learning_rate': 0.06902813253545113, 'num_leaves': 42, 'max_depth': 4, 'min_child_samples': 20, 'subsample': 0.5782400851415386, 'colsample_bytree': 0.929591971916945, 'reg_alpha': 9.945444062305704, 'reg_lambda': 0.391792364969209, 'min_split_gain': 0.03426900150705987}. Best is trial 1 with value: 0.00016728705192362709.
[I 2025-06-15 19:58:41,776] A new study created in memory with name: no-name-d13acfd7-7840-429a-8fae-4635196a5668
[I 2025-06-15 19:58:41,822] Trial 0 finished with value: 0.022179009170062852 and parameters: {'n_estimators': 214, 'learning_rate': 0.06809106706982782, 'num_leaves': 222, 'max_depth': 10, 'min_child_samples': 4, 'subsample': 0.5395498591312433, 'colsample_bytree': 0.9867167101041528, 'reg_alpha': 4.75531327191638, 'reg_lambda': 3.458536092481135, 'min_split_gain': 0.045724346354232906}. Best is trial 0 with value: 0.022179009170062852.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         209
1           Price_lag_2         148
2           Price_lag_7          92
3         Load_DA_lag_0          91
14      pct_chg_Load_DA          85
11           NGas_lag_2          74
15       lag168_Load_DA          70
16  roll2d_median_Price          58
5       WindOn_DA_lag_0          58
13            EUA_lag_2          53
10           Coal_lag_2          49
12            Oil_lag_2          33
9                  WD_7           5
7                  WD_1           4
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           9
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 33.93% done
********************* END NS.. 247 ****************************************************
********************* START NS ... 248 ****************************************************
[I 2025-06-15 19:58:42,724] A new study created in memory with name: no-name-9e7fdcd7-7c69-4af0-ba3c-9ea7749b09a1
[I 2025-06-15 19:58:43,146] Trial 0 finished with value: 0.046298574030503414 and parameters: {'n_estimators': 116, 'learning_rate': 0.013950181216042062, 'num_leaves': 162, 'max_depth': 11, 'min_child_samples': 13, 'subsample': 0.5306769584562201, 'colsample_bytree': 0.7161617249446681, 'reg_alpha': 2.999993693626765, 'reg_lambda': 3.467590596367361, 'min_split_gain': 0.024040570702736713}. Best is trial 0 with value: 0.046298574030503414.
[Trial 0] New best: 0.04630
[I 2025-06-15 19:58:43,495] Trial 1 finished with value: 0.028583471345142452 and parameters: {'n_estimators': 64, 'learning_rate': 0.02943315781115038, 'num_leaves': 197, 'max_depth': 15, 'min_child_samples': 29, 'subsample': 0.5297995324765669, 'colsample_bytree': 0.8425709246627431, 'reg_alpha': 3.7221830979532387, 'reg_lambda': 0.3598268612449518, 'min_split_gain': 0.01665177158532573}. Best is trial 1 with value: 0.028583471345142452.
[Trial 1] New best: 0.02858
[I 2025-06-15 19:58:44,678] Trial 2 finished with value: 6.479018704955951e-05 and parameters: {'n_estimators': 197, 'learning_rate': 0.030567454563460467, 'num_leaves': 66, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.7271815536979882, 'colsample_bytree': 0.6827241294779516, 'reg_alpha': 2.5975970170750653, 'reg_lambda': 8.713108557657147, 'min_split_gain': 0.0016760730373159283}. Best is trial 2 with value: 6.479018704955951e-05.
[I 2025-06-15 19:58:44,880] Trial 3 finished with value: 0.22393120880000353 and parameters: {'n_estimators': 61, 'learning_rate': 0.012582262469648336, 'num_leaves': 60, 'max_depth': 5, 'min_child_samples': 9, 'subsample': 0.880760003221474, 'colsample_bytree': 0.8025598305438395, 'reg_alpha': 7.8915950446832195, 'reg_lambda': 3.8114524115319037, 'min_split_gain': 0.002893430789396623}. Best is trial 2 with value: 6.479018704955951e-05.
[Trial 2] New best: 0.00006
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:45,319] Trial 4 finished with value: 0.001272722657983255 and parameters: {'n_estimators': 257, 'learning_rate': 0.019510383935241956, 'num_leaves': 124, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.7051890279587055, 'colsample_bytree': 0.9899116128337789, 'reg_alpha': 8.874060595325751, 'reg_lambda': 3.461484273601082, 'min_split_gain': 0.038451303899514885}. Best is trial 2 with value: 6.479018704955951e-05.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:46,428] Trial 5 finished with value: 0.00014261008343486478 and parameters: {'n_estimators': 260, 'learning_rate': 0.044010123657847726, 'num_leaves': 209, 'max_depth': 14, 'min_child_samples': 4, 'subsample': 0.5079648631626006, 'colsample_bytree': 0.5417942791532209, 'reg_alpha': 6.359616222029154, 'reg_lambda': 4.079375897907243, 'min_split_gain': 0.0037509218862306127}. Best is trial 2 with value: 6.479018704955951e-05.
[I 2025-06-15 19:58:46,430] A new study created in memory with name: no-name-532d1d4c-5146-452a-b60d-39459d5783e5
[I 2025-06-15 19:58:46,466] Trial 0 finished with value: 0.01139250979049053 and parameters: {'n_estimators': 179, 'learning_rate': 0.03286100971123283, 'num_leaves': 197, 'max_depth': 11, 'min_child_samples': 3, 'subsample': 0.5183191577484085, 'colsample_bytree': 0.5359394164966295, 'reg_alpha': 7.718114185875038, 'reg_lambda': 9.755993199094414, 'min_split_gain': 0.03578335128951321}. Best is trial 0 with value: 0.01139250979049053.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2204
1           Price_lag_2        1976
14      pct_chg_Load_DA        1273
3         Load_DA_lag_0        1020
2           Price_lag_7         817
16  roll2d_median_Price         776
5       WindOn_DA_lag_0         713
11           NGas_lag_2         711
15       lag168_Load_DA         702
10           Coal_lag_2         544
13            EUA_lag_2         495
12            Oil_lag_2         461
9                  WD_7          43
8                  WD_6          25
7                  WD_1          21
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 34.06% done
********************* END NS.. 248 ****************************************************
********************* START NS ... 249 ****************************************************
[I 2025-06-15 19:58:50,936] A new study created in memory with name: no-name-a6e7fc84-48e8-42e6-b637-4d0ab92490f4
[I 2025-06-15 19:58:51,665] Trial 0 finished with value: 3.917997086499085e-05 and parameters: {'n_estimators': 131, 'learning_rate': 0.12218551059847718, 'num_leaves': 202, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.6731690550927161, 'colsample_bytree': 0.8307869497126497, 'reg_alpha': 3.327807885751456, 'reg_lambda': 5.845002568883634, 'min_split_gain': 0.04287277491016644}. Best is trial 0 with value: 3.917997086499085e-05.
[Trial 0] New best: 0.00004
[I 2025-06-15 19:58:52,476] Trial 1 finished with value: 0.000178834377287424 and parameters: {'n_estimators': 197, 'learning_rate': 0.22438209932379954, 'num_leaves': 244, 'max_depth': 14, 'min_child_samples': 28, 'subsample': 0.6405331884014164, 'colsample_bytree': 0.9771359087477969, 'reg_alpha': 0.6817966401007358, 'reg_lambda': 5.5523583460861605, 'min_split_gain': 0.010188722492770075}. Best is trial 0 with value: 3.917997086499085e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:58:53,911] Trial 2 finished with value: 9.803254975846759e-05 and parameters: {'n_estimators': 156, 'learning_rate': 0.15480984145761034, 'num_leaves': 185, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.983629646096714, 'colsample_bytree': 0.9410777410164116, 'reg_alpha': 0.10845623497656831, 'reg_lambda': 1.046564604283463, 'min_split_gain': 0.007328818098506451}. Best is trial 0 with value: 3.917997086499085e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:58:54,700] Trial 3 finished with value: 0.0010064680889532202 and parameters: {'n_estimators': 135, 'learning_rate': 0.0891720982822093, 'num_leaves': 103, 'max_depth': 16, 'min_child_samples': 9, 'subsample': 0.5946807524238105, 'colsample_bytree': 0.5807006746363133, 'reg_alpha': 6.848069579629002, 'reg_lambda': 6.839585011075922, 'min_split_gain': 0.03323270685611458}. Best is trial 0 with value: 3.917997086499085e-05.
[I 2025-06-15 19:58:54,703] A new study created in memory with name: no-name-bd7715a6-a62e-49c8-83c3-796665caa85a
[I 2025-06-15 19:58:54,726] Trial 0 finished with value: 0.001558947638002685 and parameters: {'n_estimators': 75, 'learning_rate': 0.030335121443405857, 'num_leaves': 194, 'max_depth': 16, 'min_child_samples': 4, 'subsample': 0.6444853303045559, 'colsample_bytree': 0.8362570479442684, 'reg_alpha': 7.484934300653746, 'reg_lambda': 8.049150219090986, 'min_split_gain': 0.03677515118369603}. Best is trial 0 with value: 0.001558947638002685.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         412
1           Price_lag_2         278
14      pct_chg_Load_DA         269
3         Load_DA_lag_0         146
11           NGas_lag_2         130
2           Price_lag_7          98
16  roll2d_median_Price          94
15       lag168_Load_DA          83
5       WindOn_DA_lag_0          50
13            EUA_lag_2          50
12            Oil_lag_2          49
10           Coal_lag_2          43
9                  WD_7          12
7                  WD_1           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 34.20% done
********************* END NS.. 249 ****************************************************
********************* START NS ... 250 ****************************************************
[I 2025-06-15 19:58:56,836] A new study created in memory with name: no-name-d1c3be13-7e9f-4a1a-8237-552e92ee173f
[I 2025-06-15 19:58:57,271] Trial 0 finished with value: 2.051424380286276e-05 and parameters: {'n_estimators': 263, 'learning_rate': 0.05620959577033746, 'num_leaves': 167, 'max_depth': 4, 'min_child_samples': 6, 'subsample': 0.9203055455740021, 'colsample_bytree': 0.7799430149371259, 'reg_alpha': 7.3253579409723955, 'reg_lambda': 4.347284775777474, 'min_split_gain': 0.03270220447576726}. Best is trial 0 with value: 2.051424380286276e-05.
[Trial 0] New best: 0.00002
[I 2025-06-15 19:58:58,507] Trial 1 finished with value: 0.01677786097794573 and parameters: {'n_estimators': 168, 'learning_rate': 0.012130267228320148, 'num_leaves': 119, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.9757242150471046, 'colsample_bytree': 0.5645521217913302, 'reg_alpha': 8.962291445571369, 'reg_lambda': 9.612109218484985, 'min_split_gain': 0.04774109430068527}. Best is trial 0 with value: 2.051424380286276e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:59:00,390] Trial 2 finished with value: 0.0008869971239652582 and parameters: {'n_estimators': 211, 'learning_rate': 0.016549947084782662, 'num_leaves': 167, 'max_depth': 11, 'min_child_samples': 17, 'subsample': 0.5284781794029743, 'colsample_bytree': 0.6833308003658853, 'reg_alpha': 5.133128011270708, 'reg_lambda': 4.010714204939578, 'min_split_gain': 0.04240148796319692}. Best is trial 0 with value: 2.051424380286276e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:59:01,680] Trial 3 finished with value: 1.951201458217683e-05 and parameters: {'n_estimators': 214, 'learning_rate': 0.046405140022353755, 'num_leaves': 201, 'max_depth': 9, 'min_child_samples': 29, 'subsample': 0.7724728208016571, 'colsample_bytree': 0.9647433229847593, 'reg_alpha': 4.55501853331964, 'reg_lambda': 5.599335153508244, 'min_split_gain': 0.02398313234918652}. Best is trial 3 with value: 1.951201458217683e-05.
[I 2025-06-15 19:59:01,682] A new study created in memory with name: no-name-2eaafef0-8829-4d59-a39b-aaafb5b94608
[I 2025-06-15 19:59:01,710] Trial 0 finished with value: 0.0005504394686456808 and parameters: {'n_estimators': 83, 'learning_rate': 0.033538954168030384, 'num_leaves': 82, 'max_depth': 15, 'min_child_samples': 17, 'subsample': 0.6163948309842435, 'colsample_bytree': 0.8240726989725478, 'reg_alpha': 7.250909842841948, 'reg_lambda': 9.533886482353939, 'min_split_gain': 0.006537656891379479}. Best is trial 0 with value: 0.0005504394686456808.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1251
14      pct_chg_Load_DA         857
1           Price_lag_2         728
3         Load_DA_lag_0         456
11           NGas_lag_2         423
2           Price_lag_7         299
16  roll2d_median_Price         290
15       lag168_Load_DA         252
10           Coal_lag_2         247
5       WindOn_DA_lag_0         224
12            Oil_lag_2         213
13            EUA_lag_2         179
9                  WD_7          25
7                  WD_1          19
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 34.34% done
********************* END NS.. 250 ****************************************************
********************* START NS ... 251 ****************************************************
[I 2025-06-15 19:59:04,856] A new study created in memory with name: no-name-825ad38f-c050-4152-b155-93bb864015b1
[I 2025-06-15 19:59:05,500] Trial 0 finished with value: 0.12334228468600422 and parameters: {'n_estimators': 91, 'learning_rate': 0.010489306756863832, 'num_leaves': 121, 'max_depth': 11, 'min_child_samples': 7, 'subsample': 0.6713943372780318, 'colsample_bytree': 0.7148502023902727, 'reg_alpha': 2.8760689567789086, 'reg_lambda': 9.01525310885811, 'min_split_gain': 0.04070382640626993}. Best is trial 0 with value: 0.12334228468600422.
[Trial 0] New best: 0.12334
[I 2025-06-15 19:59:08,276] Trial 1 finished with value: 0.00016250160113435164 and parameters: {'n_estimators': 298, 'learning_rate': 0.01734196467772296, 'num_leaves': 69, 'max_depth': 13, 'min_child_samples': 19, 'subsample': 0.5829448340866197, 'colsample_bytree': 0.5081072356490289, 'reg_alpha': 3.7520265620607383, 'reg_lambda': 3.9402429442458695, 'min_split_gain': 0.043943232665585585}. Best is trial 1 with value: 0.00016250160113435164.
[Trial 1] New best: 0.00016
[I 2025-06-15 19:59:08,763] Trial 2 finished with value: 1.2657432663679689e-05 and parameters: {'n_estimators': 131, 'learning_rate': 0.24432455453475718, 'num_leaves': 200, 'max_depth': 15, 'min_child_samples': 16, 'subsample': 0.535147452822611, 'colsample_bytree': 0.949681095831806, 'reg_alpha': 2.9803943319813153, 'reg_lambda': 8.767953679499247, 'min_split_gain': 0.00926177862957298}. Best is trial 2 with value: 1.2657432663679689e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:59:09,899] Trial 3 finished with value: 0.0018890270991980564 and parameters: {'n_estimators': 247, 'learning_rate': 0.012367113254801285, 'num_leaves': 38, 'max_depth': 5, 'min_child_samples': 13, 'subsample': 0.6822969648571827, 'colsample_bytree': 0.8282883661453249, 'reg_alpha': 5.525343958974219, 'reg_lambda': 1.6749006982227754, 'min_split_gain': 0.037209968066932955}. Best is trial 2 with value: 1.2657432663679689e-05.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:59:12,359] Trial 4 finished with value: 0.000611652531415916 and parameters: {'n_estimators': 184, 'learning_rate': 0.019435479081366, 'num_leaves': 204, 'max_depth': 10, 'min_child_samples': 13, 'subsample': 0.5956207005146863, 'colsample_bytree': 0.7002895427848583, 'reg_alpha': 1.149249805741962, 'reg_lambda': 6.055287011411817, 'min_split_gain': 0.03519768072780629}. Best is trial 2 with value: 1.2657432663679689e-05.
[I 2025-06-15 19:59:12,361] A new study created in memory with name: no-name-fefd3404-a3aa-4164-b793-dce67893103b
[I 2025-06-15 19:59:12,399] Trial 0 finished with value: 0.001118822509993122 and parameters: {'n_estimators': 133, 'learning_rate': 0.13736600301100765, 'num_leaves': 165, 'max_depth': 8, 'min_child_samples': 13, 'subsample': 0.5572636642217129, 'colsample_bytree': 0.7487675296076559, 'reg_alpha': 9.981608909877098, 'reg_lambda': 0.3746153917697914, 'min_split_gain': 0.0004814858872377037}. Best is trial 0 with value: 0.001118822509993122.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         339
14      pct_chg_Load_DA         310
1           Price_lag_2         219
3         Load_DA_lag_0         190
5       WindOn_DA_lag_0         174
15       lag168_Load_DA         147
2           Price_lag_7         138
11           NGas_lag_2         133
16  roll2d_median_Price         130
13            EUA_lag_2         108
10           Coal_lag_2          91
12            Oil_lag_2          74
9                  WD_7          11
7                  WD_1           6
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 34.47% done
********************* END NS.. 251 ****************************************************
********************* START NS ... 252 ****************************************************
[I 2025-06-15 19:59:14,613] A new study created in memory with name: no-name-6f84bb43-a0d6-439d-8395-550120cde17c
[I 2025-06-15 19:59:15,695] Trial 0 finished with value: 9.991742401076397e-05 and parameters: {'n_estimators': 152, 'learning_rate': 0.03237718212254671, 'num_leaves': 128, 'max_depth': 7, 'min_child_samples': 6, 'subsample': 0.9546829504546044, 'colsample_bytree': 0.8829293732150024, 'reg_alpha': 3.162694326325246, 'reg_lambda': 8.478238511122322, 'min_split_gain': 0.03930805966219647}. Best is trial 0 with value: 9.991742401076397e-05.
[Trial 0] New best: 0.00010
[I 2025-06-15 19:59:18,847] Trial 1 finished with value: 1.9991565836108195e-05 and parameters: {'n_estimators': 268, 'learning_rate': 0.04641592471278508, 'num_leaves': 224, 'max_depth': 12, 'min_child_samples': 6, 'subsample': 0.7290818800913109, 'colsample_bytree': 0.8876963355941233, 'reg_alpha': 4.143898504767078, 'reg_lambda': 5.422163144402395, 'min_split_gain': 0.0047163119558016}. Best is trial 1 with value: 1.9991565836108195e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:59:19,279] Trial 2 finished with value: 0.004172726113714042 and parameters: {'n_estimators': 56, 'learning_rate': 0.04771178906520093, 'num_leaves': 186, 'max_depth': 10, 'min_child_samples': 11, 'subsample': 0.5409474431531496, 'colsample_bytree': 0.7936420139410965, 'reg_alpha': 7.590059661745462, 'reg_lambda': 0.06918420380225498, 'min_split_gain': 0.03935071497748405}. Best is trial 1 with value: 1.9991565836108195e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:59:19,650] Trial 3 finished with value: 3.902968216073689e-05 and parameters: {'n_estimators': 192, 'learning_rate': 0.22499563122178867, 'num_leaves': 188, 'max_depth': 8, 'min_child_samples': 19, 'subsample': 0.6072666212320759, 'colsample_bytree': 0.7453719452412086, 'reg_alpha': 4.303979418201934, 'reg_lambda': 5.537869686962022, 'min_split_gain': 0.01815444523847456}. Best is trial 1 with value: 1.9991565836108195e-05.
[I 2025-06-15 19:59:19,652] A new study created in memory with name: no-name-f9e69906-22ba-454e-9d6a-04b96d07ad5a
[I 2025-06-15 19:59:19,686] Trial 0 finished with value: 0.00040327156203264473 and parameters: {'n_estimators': 116, 'learning_rate': 0.011957537369618331, 'num_leaves': 216, 'max_depth': 10, 'min_child_samples': 28, 'subsample': 0.6077319458906559, 'colsample_bytree': 0.9555525999331536, 'reg_alpha': 6.302345157914414, 'reg_lambda': 9.370698386812961, 'min_split_gain': 0.016090323770739098}. Best is trial 0 with value: 0.00040327156203264473.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1897
14      pct_chg_Load_DA        1743
1           Price_lag_2        1621
3         Load_DA_lag_0        1274
5       WindOn_DA_lag_0        1090
2           Price_lag_7         911
15       lag168_Load_DA         883
11           NGas_lag_2         857
16  roll2d_median_Price         757
10           Coal_lag_2         739
13            EUA_lag_2         738
12            Oil_lag_2         584
9                  WD_7          66
7                  WD_1          42
8                  WD_6          35
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 34.61% done
********************* END NS.. 252 ****************************************************
********************* START NS ... 253 ****************************************************
[I 2025-06-15 19:59:23,629] A new study created in memory with name: no-name-aacc4234-a97e-4ef5-b13c-f97f1e7e0705
[I 2025-06-15 19:59:24,875] Trial 0 finished with value: 0.05251088045998703 and parameters: {'n_estimators': 108, 'learning_rate': 0.013136317927316366, 'num_leaves': 180, 'max_depth': 15, 'min_child_samples': 19, 'subsample': 0.6595929770934006, 'colsample_bytree': 0.6477538730180905, 'reg_alpha': 2.0401627772747197, 'reg_lambda': 2.6730037630687162, 'min_split_gain': 0.03604411997880702}. Best is trial 0 with value: 0.05251088045998703.
[Trial 0] New best: 0.05251
[I 2025-06-15 19:59:25,921] Trial 1 finished with value: 0.0005921479314050316 and parameters: {'n_estimators': 254, 'learning_rate': 0.015239098873591353, 'num_leaves': 107, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.5993885306792296, 'colsample_bytree': 0.9498313189811516, 'reg_alpha': 2.208489355753623, 'reg_lambda': 8.990863324344042, 'min_split_gain': 0.03239949441382012}. Best is trial 1 with value: 0.0005921479314050316.
[Trial 1] New best: 0.00059
[I 2025-06-15 19:59:26,194] Trial 2 finished with value: 0.0017830741258497238 and parameters: {'n_estimators': 54, 'learning_rate': 0.06020874160689154, 'num_leaves': 151, 'max_depth': 4, 'min_child_samples': 21, 'subsample': 0.9917320947143229, 'colsample_bytree': 0.5798699863452501, 'reg_alpha': 9.127113049060291, 'reg_lambda': 3.7212695173215327, 'min_split_gain': 0.006637026622293824}. Best is trial 1 with value: 0.0005921479314050316.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:59:26,633] Trial 3 finished with value: 0.00022348732252947312 and parameters: {'n_estimators': 50, 'learning_rate': 0.09184162239833864, 'num_leaves': 165, 'max_depth': 12, 'min_child_samples': 10, 'subsample': 0.5091173547652077, 'colsample_bytree': 0.922159223903392, 'reg_alpha': 8.629852135223379, 'reg_lambda': 4.7800032627699265, 'min_split_gain': 0.02430922274918155}. Best is trial 3 with value: 0.00022348732252947312.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:59:27,956] Trial 4 finished with value: 0.026837712700539567 and parameters: {'n_estimators': 162, 'learning_rate': 0.01095483629949467, 'num_leaves': 164, 'max_depth': 10, 'min_child_samples': 14, 'subsample': 0.9253103792851888, 'colsample_bytree': 0.950337380577565, 'reg_alpha': 4.287755570256818, 'reg_lambda': 3.5945713328329276, 'min_split_gain': 0.03230617030617413}. Best is trial 3 with value: 0.00022348732252947312.
[I 2025-06-15 19:59:27,958] A new study created in memory with name: no-name-8ac9b12a-2fbc-4fb1-ab36-846437fa8fc4
[I 2025-06-15 19:59:28,011] Trial 0 finished with value: 0.00016304342596350197 and parameters: {'n_estimators': 231, 'learning_rate': 0.042031393233747834, 'num_leaves': 155, 'max_depth': 15, 'min_child_samples': 20, 'subsample': 0.8130884072529079, 'colsample_bytree': 0.777044107604798, 'reg_alpha': 7.210209178921001, 'reg_lambda': 6.43072882071861, 'min_split_gain': 0.03996644369814377}. Best is trial 0 with value: 0.00016304342596350197.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         447
14      pct_chg_Load_DA         218
1           Price_lag_2         210
16  roll2d_median_Price         113
2           Price_lag_7         105
3         Load_DA_lag_0          93
11           NGas_lag_2          86
10           Coal_lag_2          58
13            EUA_lag_2          47
12            Oil_lag_2          38
5       WindOn_DA_lag_0          37
15       lag168_Load_DA          24
9                  WD_7          14
8                  WD_6           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 34.75% done
********************* END NS.. 253 ****************************************************
********************* START NS ... 254 ****************************************************
[I 2025-06-15 19:59:29,845] A new study created in memory with name: no-name-0147ab92-97d9-4255-abf9-131f0480b055
[I 2025-06-15 19:59:31,312] Trial 0 finished with value: 0.00012230646032156215 and parameters: {'n_estimators': 113, 'learning_rate': 0.07732089553722316, 'num_leaves': 178, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.7339427114189252, 'colsample_bytree': 0.7470972000490096, 'reg_alpha': 0.0194070527791268, 'reg_lambda': 1.8785267803995764, 'min_split_gain': 0.037850821803179904}. Best is trial 0 with value: 0.00012230646032156215.
[Trial 0] New best: 0.00012
[I 2025-06-15 19:59:33,415] Trial 1 finished with value: 0.02131177205734708 and parameters: {'n_estimators': 169, 'learning_rate': 0.011179062054619203, 'num_leaves': 208, 'max_depth': 13, 'min_child_samples': 26, 'subsample': 0.8826374655422938, 'colsample_bytree': 0.5420703607771236, 'reg_alpha': 1.8234062229229109, 'reg_lambda': 9.522749767452877, 'min_split_gain': 0.028479969635633165}. Best is trial 0 with value: 0.00012230646032156215.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:59:34,582] Trial 2 finished with value: 0.0008360708836311518 and parameters: {'n_estimators': 171, 'learning_rate': 0.021066346874100736, 'num_leaves': 152, 'max_depth': 6, 'min_child_samples': 1, 'subsample': 0.9012166234039773, 'colsample_bytree': 0.874261291318603, 'reg_alpha': 6.985139669665227, 'reg_lambda': 3.8655264197401373, 'min_split_gain': 0.013528447620019791}. Best is trial 0 with value: 0.00012230646032156215.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:59:35,162] Trial 3 finished with value: 4.7930302157764795e-05 and parameters: {'n_estimators': 198, 'learning_rate': 0.11151860626835898, 'num_leaves': 105, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.8351646424908563, 'colsample_bytree': 0.6550864483618957, 'reg_alpha': 3.6336497815302913, 'reg_lambda': 9.91446594861112, 'min_split_gain': 0.03900005937231108}. Best is trial 3 with value: 4.7930302157764795e-05.
[I 2025-06-15 19:59:35,164] A new study created in memory with name: no-name-83e55f81-5eb2-4910-9a0a-12400caa14d8
[I 2025-06-15 19:59:35,210] Trial 0 finished with value: 0.00014942910872832045 and parameters: {'n_estimators': 143, 'learning_rate': 0.16607686916512868, 'num_leaves': 36, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.9948033716499873, 'colsample_bytree': 0.5428669482928716, 'reg_alpha': 2.021133217760467, 'reg_lambda': 1.3320880710784855, 'min_split_gain': 0.023640476057099513}. Best is trial 0 with value: 0.00014942910872832045.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         450
1           Price_lag_2         320
14      pct_chg_Load_DA         261
3         Load_DA_lag_0         200
11           NGas_lag_2         146
16  roll2d_median_Price         115
15       lag168_Load_DA         106
2           Price_lag_7          97
5       WindOn_DA_lag_0          91
10           Coal_lag_2          86
12            Oil_lag_2          68
13            EUA_lag_2          53
9                  WD_7           9
7                  WD_1           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 34.88% done
********************* END NS.. 254 ****************************************************
********************* START NS ... 255 ****************************************************
[I 2025-06-15 19:59:37,100] A new study created in memory with name: no-name-bbd81bdf-5ecb-4eb6-80b9-d1681d9984c0
[I 2025-06-15 19:59:37,789] Trial 0 finished with value: 8.97549569298065e-05 and parameters: {'n_estimators': 271, 'learning_rate': 0.16288968563675044, 'num_leaves': 206, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.7297348768081368, 'colsample_bytree': 0.5764029959824257, 'reg_alpha': 1.1371387455798965, 'reg_lambda': 5.868966197640711, 'min_split_gain': 0.031234045419915358}. Best is trial 0 with value: 8.97549569298065e-05.
[Trial 0] New best: 0.00009
[I 2025-06-15 19:59:41,039] Trial 1 finished with value: 8.010554828767536e-05 and parameters: {'n_estimators': 282, 'learning_rate': 0.02846773480420827, 'num_leaves': 158, 'max_depth': 9, 'min_child_samples': 20, 'subsample': 0.6534115174223911, 'colsample_bytree': 0.7999938641749142, 'reg_alpha': 1.311351751118035, 'reg_lambda': 6.7584878162689215, 'min_split_gain': 0.015130279788001466}. Best is trial 1 with value: 8.010554828767536e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:59:42,459] Trial 2 finished with value: 0.0038681913143078883 and parameters: {'n_estimators': 165, 'learning_rate': 0.016601327384703473, 'num_leaves': 206, 'max_depth': 15, 'min_child_samples': 22, 'subsample': 0.5907204606488525, 'colsample_bytree': 0.7758180074997358, 'reg_alpha': 3.2361800010344863, 'reg_lambda': 9.838860474724516, 'min_split_gain': 0.010944528940874743}. Best is trial 1 with value: 8.010554828767536e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:59:43,443] Trial 3 finished with value: 0.017955299986884582 and parameters: {'n_estimators': 100, 'learning_rate': 0.019533279058605694, 'num_leaves': 123, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.8592802261420123, 'colsample_bytree': 0.9635688419779266, 'reg_alpha': 3.592110463504208, 'reg_lambda': 9.471150160249714, 'min_split_gain': 0.019687307450845716}. Best is trial 1 with value: 8.010554828767536e-05.
[I 2025-06-15 19:59:43,445] A new study created in memory with name: no-name-f1e9855e-b152-4f5a-8ed4-2d814b1f93ce
[I 2025-06-15 19:59:43,502] Trial 0 finished with value: 0.00016225766029665783 and parameters: {'n_estimators': 262, 'learning_rate': 0.07539422159140165, 'num_leaves': 165, 'max_depth': 16, 'min_child_samples': 30, 'subsample': 0.7697434878132892, 'colsample_bytree': 0.6309506869203689, 'reg_alpha': 1.2293016286613945, 'reg_lambda': 3.4240924816148164, 'min_split_gain': 0.027325145661504582}. Best is trial 0 with value: 0.00016225766029665783.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3101
1           Price_lag_2        2684
14      pct_chg_Load_DA        2386
3         Load_DA_lag_0        1651
15       lag168_Load_DA        1240
16  roll2d_median_Price        1218
11           NGas_lag_2        1181
2           Price_lag_7        1093
5       WindOn_DA_lag_0        1007
10           Coal_lag_2         721
12            Oil_lag_2         708
13            EUA_lag_2         535
9                  WD_7          65
7                  WD_1          51
8                  WD_6          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 35.02% done
********************* END NS.. 255 ****************************************************
********************* START NS ... 256 ****************************************************
[I 2025-06-15 19:59:47,709] A new study created in memory with name: no-name-220c3b7a-4584-49c3-8649-fd33a19a72f0
[I 2025-06-15 19:59:47,951] Trial 0 finished with value: 4.462125175811354e-05 and parameters: {'n_estimators': 195, 'learning_rate': 0.16124679630249655, 'num_leaves': 254, 'max_depth': 5, 'min_child_samples': 16, 'subsample': 0.6117102633614115, 'colsample_bytree': 0.6695770038813392, 'reg_alpha': 7.639762230456725, 'reg_lambda': 8.996194887043258, 'min_split_gain': 0.03689400749212762}. Best is trial 0 with value: 4.462125175811354e-05.
[Trial 0] New best: 0.00004
[I 2025-06-15 19:59:48,657] Trial 1 finished with value: 0.06840839895893037 and parameters: {'n_estimators': 95, 'learning_rate': 0.013635692314665762, 'num_leaves': 91, 'max_depth': 7, 'min_child_samples': 22, 'subsample': 0.9137049633816372, 'colsample_bytree': 0.6583744091395753, 'reg_alpha': 7.956702913768218, 'reg_lambda': 1.3578330539365202, 'min_split_gain': 0.013587445356305761}. Best is trial 0 with value: 4.462125175811354e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:59:50,893] Trial 2 finished with value: 0.0005118542450944008 and parameters: {'n_estimators': 288, 'learning_rate': 0.01668873441059728, 'num_leaves': 164, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.8257191257690712, 'colsample_bytree': 0.536360074496945, 'reg_alpha': 7.184294146623555, 'reg_lambda': 5.542537017627172, 'min_split_gain': 0.02141241577470427}. Best is trial 0 with value: 4.462125175811354e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:59:51,656] Trial 3 finished with value: 0.0009976036850985151 and parameters: {'n_estimators': 299, 'learning_rate': 0.1211162080690189, 'num_leaves': 244, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.84374561284475, 'colsample_bytree': 0.5770762371445255, 'reg_alpha': 2.3500204646595515, 'reg_lambda': 7.676561117105864, 'min_split_gain': 0.04015939842148023}. Best is trial 0 with value: 4.462125175811354e-05.
[I 2025-06-15 19:59:51,659] A new study created in memory with name: no-name-e23de9a9-5967-4023-bcdc-f112744487ba
[I 2025-06-15 19:59:51,713] Trial 0 finished with value: 0.00017127028365059572 and parameters: {'n_estimators': 179, 'learning_rate': 0.17913208916668158, 'num_leaves': 211, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.5491554409187114, 'colsample_bytree': 0.5282984719471605, 'reg_alpha': 6.8530770425295975, 'reg_lambda': 6.151049416834682, 'min_split_gain': 0.03995816392105714}. Best is trial 0 with value: 0.00017127028365059572.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         250
1           Price_lag_2         118
14      pct_chg_Load_DA          98
3         Load_DA_lag_0          71
11           NGas_lag_2          54
2           Price_lag_7          52
16  roll2d_median_Price          41
13            EUA_lag_2          34
15       lag168_Load_DA          33
10           Coal_lag_2          30
12            Oil_lag_2          21
5       WindOn_DA_lag_0          20
9                  WD_7           4
8                  WD_6           2
7                  WD_1           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 35.16% done
********************* END NS.. 256 ****************************************************
********************* START NS ... 257 ****************************************************
[I 2025-06-15 19:59:53,688] A new study created in memory with name: no-name-428dbf81-6e9a-4d44-a2e5-4c9881a54277
[I 2025-06-15 19:59:54,220] Trial 0 finished with value: 4.899179048611297e-05 and parameters: {'n_estimators': 116, 'learning_rate': 0.11209093781520915, 'num_leaves': 94, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.7325695697304412, 'colsample_bytree': 0.6841873476250855, 'reg_alpha': 2.6143894957341915, 'reg_lambda': 3.020331539827138, 'min_split_gain': 0.040540111970153996}. Best is trial 0 with value: 4.899179048611297e-05.
[Trial 0] New best: 0.00005
[I 2025-06-15 19:59:56,186] Trial 1 finished with value: 0.01828957396875275 and parameters: {'n_estimators': 178, 'learning_rate': 0.01117953022502772, 'num_leaves': 55, 'max_depth': 11, 'min_child_samples': 28, 'subsample': 0.6745367163464314, 'colsample_bytree': 0.5992981661312826, 'reg_alpha': 1.6600750773341322, 'reg_lambda': 1.379316589046885, 'min_split_gain': 0.019905499459552035}. Best is trial 0 with value: 4.899179048611297e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 19:59:57,130] Trial 2 finished with value: 2.5959628436754092e-05 and parameters: {'n_estimators': 167, 'learning_rate': 0.09052481908281966, 'num_leaves': 226, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.6566840220339101, 'colsample_bytree': 0.6661015482678582, 'reg_alpha': 2.2394953234945514, 'reg_lambda': 5.7489262768743945, 'min_split_gain': 0.018534307439334177}. Best is trial 2 with value: 2.5959628436754092e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 19:59:57,809] Trial 3 finished with value: 0.17058548077786326 and parameters: {'n_estimators': 72, 'learning_rate': 0.011517031975310836, 'num_leaves': 88, 'max_depth': 11, 'min_child_samples': 5, 'subsample': 0.7669697495019641, 'colsample_bytree': 0.7539047184220298, 'reg_alpha': 7.220198079301841, 'reg_lambda': 6.897271492516541, 'min_split_gain': 0.029121443903361355}. Best is trial 2 with value: 2.5959628436754092e-05.
[I 2025-06-15 19:59:57,812] A new study created in memory with name: no-name-8cfad6dd-ef54-4bd2-a4eb-41e79d4c1174
[I 2025-06-15 19:59:57,849] Trial 0 finished with value: 3.852840908783627e-05 and parameters: {'n_estimators': 137, 'learning_rate': 0.010890066165297273, 'num_leaves': 59, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.6724205942529373, 'colsample_bytree': 0.7562585435240908, 'reg_alpha': 5.825635136485791, 'reg_lambda': 0.20010121336514364, 'min_split_gain': 0.036994921597730886}. Best is trial 0 with value: 3.852840908783627e-05.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         741
1           Price_lag_2         626
14      pct_chg_Load_DA         548
3         Load_DA_lag_0         516
15       lag168_Load_DA         322
11           NGas_lag_2         301
5       WindOn_DA_lag_0         293
2           Price_lag_7         285
16  roll2d_median_Price         256
10           Coal_lag_2         193
12            Oil_lag_2         180
13            EUA_lag_2         154
9                  WD_7          16
7                  WD_1          16
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 35.29% done
********************* END NS.. 257 ****************************************************
********************* START NS ... 258 ****************************************************
[I 2025-06-15 20:00:00,281] A new study created in memory with name: no-name-0ce14cf0-b947-491b-b2ee-03218546ff00
[I 2025-06-15 20:00:01,670] Trial 0 finished with value: 0.0003949724224778834 and parameters: {'n_estimators': 214, 'learning_rate': 0.05251313113497635, 'num_leaves': 181, 'max_depth': 10, 'min_child_samples': 27, 'subsample': 0.6073501123244516, 'colsample_bytree': 0.5435358623269099, 'reg_alpha': 3.4053979123287412, 'reg_lambda': 5.531683037822463, 'min_split_gain': 0.045320163564158765}. Best is trial 0 with value: 0.0003949724224778834.
[Trial 0] New best: 0.00039
[I 2025-06-15 20:00:02,412] Trial 1 finished with value: 0.00010792034380063976 and parameters: {'n_estimators': 88, 'learning_rate': 0.18369692886689223, 'num_leaves': 152, 'max_depth': 8, 'min_child_samples': 5, 'subsample': 0.861094123237452, 'colsample_bytree': 0.5813288111445951, 'reg_alpha': 9.324342291714762, 'reg_lambda': 0.7807571879526487, 'min_split_gain': 0.0026787429640466156}. Best is trial 1 with value: 0.00010792034380063976.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:00:03,357] Trial 2 finished with value: 0.002188553233587412 and parameters: {'n_estimators': 198, 'learning_rate': 0.017060309677360122, 'num_leaves': 111, 'max_depth': 5, 'min_child_samples': 9, 'subsample': 0.9065621944542821, 'colsample_bytree': 0.5999865023821382, 'reg_alpha': 5.661220753384612, 'reg_lambda': 2.820617369857085, 'min_split_gain': 0.04666086998685258}. Best is trial 1 with value: 0.00010792034380063976.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:00:07,482] Trial 3 finished with value: 2.27350866950477e-05 and parameters: {'n_estimators': 231, 'learning_rate': 0.029659679634267227, 'num_leaves': 195, 'max_depth': 16, 'min_child_samples': 3, 'subsample': 0.8375364943591996, 'colsample_bytree': 0.7444751462054147, 'reg_alpha': 0.3506067164180604, 'reg_lambda': 6.951037698741186, 'min_split_gain': 0.045569652695292236}. Best is trial 3 with value: 2.27350866950477e-05.
[I 2025-06-15 20:00:07,484] A new study created in memory with name: no-name-e948c268-4143-42bc-9abc-fc838e846eac
[I 2025-06-15 20:00:07,512] Trial 0 finished with value: 6.197645678623194e-05 and parameters: {'n_estimators': 92, 'learning_rate': 0.011280667890549026, 'num_leaves': 144, 'max_depth': 14, 'min_child_samples': 11, 'subsample': 0.9883263650965235, 'colsample_bytree': 0.9207613083010315, 'reg_alpha': 5.694208546278095, 'reg_lambda': 1.9694556301716137, 'min_split_gain': 0.048272807156049924}. Best is trial 0 with value: 6.197645678623194e-05.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        3245
14      pct_chg_Load_DA        2849
0           Price_lag_1        2752
3         Load_DA_lag_0        2563
2           Price_lag_7        1690
15       lag168_Load_DA        1663
11           NGas_lag_2        1420
16  roll2d_median_Price        1348
10           Coal_lag_2        1164
12            Oil_lag_2         931
5       WindOn_DA_lag_0         759
13            EUA_lag_2         742
9                  WD_7          72
7                  WD_1          43
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 35.43% done
********************* END NS.. 258 ****************************************************
********************* START NS ... 259 ****************************************************
[I 2025-06-15 20:00:12,554] A new study created in memory with name: no-name-6f96575b-6f10-4880-a6bb-6780dfba16ef
[I 2025-06-15 20:00:13,736] Trial 0 finished with value: 0.014807714031053408 and parameters: {'n_estimators': 168, 'learning_rate': 0.01423600957651038, 'num_leaves': 211, 'max_depth': 6, 'min_child_samples': 12, 'subsample': 0.5999882572928843, 'colsample_bytree': 0.502492907846096, 'reg_alpha': 1.866362267018431, 'reg_lambda': 2.4975363498935597, 'min_split_gain': 0.04558371089173588}. Best is trial 0 with value: 0.014807714031053408.
[Trial 0] New best: 0.01481
[I 2025-06-15 20:00:14,417] Trial 1 finished with value: 0.07903946323505205 and parameters: {'n_estimators': 57, 'learning_rate': 0.02317752054820167, 'num_leaves': 213, 'max_depth': 11, 'min_child_samples': 15, 'subsample': 0.8208959103153022, 'colsample_bytree': 0.7708429483312356, 'reg_alpha': 4.549331926709515, 'reg_lambda': 0.07849666115152476, 'min_split_gain': 0.005806753521244857}. Best is trial 0 with value: 0.014807714031053408.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:00:15,157] Trial 2 finished with value: 0.0008391834851223563 and parameters: {'n_estimators': 97, 'learning_rate': 0.0795675235675945, 'num_leaves': 139, 'max_depth': 16, 'min_child_samples': 3, 'subsample': 0.8318520132549425, 'colsample_bytree': 0.6207081933921534, 'reg_alpha': 4.579363620037378, 'reg_lambda': 0.42059963728038885, 'min_split_gain': 0.049165182911572645}. Best is trial 2 with value: 0.0008391834851223563.
[Trial 2] New best: 0.00084
[I 2025-06-15 20:00:16,334] Trial 3 finished with value: 0.0010106276205513345 and parameters: {'n_estimators': 295, 'learning_rate': 0.036963694572520044, 'num_leaves': 238, 'max_depth': 8, 'min_child_samples': 12, 'subsample': 0.5572444336148971, 'colsample_bytree': 0.9629952776927695, 'reg_alpha': 7.4711799976539774, 'reg_lambda': 3.2170417571281895, 'min_split_gain': 0.03923076975072426}. Best is trial 2 with value: 0.0008391834851223563.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:00:16,871] Trial 4 finished with value: 0.0012642517008012843 and parameters: {'n_estimators': 54, 'learning_rate': 0.09469143107137966, 'num_leaves': 60, 'max_depth': 10, 'min_child_samples': 23, 'subsample': 0.9453184964561531, 'colsample_bytree': 0.6513451246687848, 'reg_alpha': 4.236882367337881, 'reg_lambda': 7.785538245412026, 'min_split_gain': 0.010292860805337945}. Best is trial 2 with value: 0.0008391834851223563.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:00:17,647] Trial 5 finished with value: 0.00010996475182707233 and parameters: {'n_estimators': 216, 'learning_rate': 0.09155881758791241, 'num_leaves': 79, 'max_depth': 10, 'min_child_samples': 29, 'subsample': 0.8494544536108477, 'colsample_bytree': 0.7762308032609426, 'reg_alpha': 1.4037110927612928, 'reg_lambda': 1.3834296703286386, 'min_split_gain': 0.04960768556513884}. Best is trial 5 with value: 0.00010996475182707233.
[I 2025-06-15 20:00:17,650] A new study created in memory with name: no-name-63bd6e20-40aa-48a2-949b-40e336300db5
[I 2025-06-15 20:00:17,682] Trial 0 finished with value: 0.0017305602478994379 and parameters: {'n_estimators': 106, 'learning_rate': 0.2992325812228156, 'num_leaves': 247, 'max_depth': 9, 'min_child_samples': 29, 'subsample': 0.8015266914338699, 'colsample_bytree': 0.8292710843899487, 'reg_alpha': 3.9559938211778434, 'reg_lambda': 1.1425594018899798, 'min_split_gain': 0.041929862062641474}. Best is trial 0 with value: 0.0017305602478994379.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         739
1           Price_lag_2         616
14      pct_chg_Load_DA         510
3         Load_DA_lag_0         324
11           NGas_lag_2         236
15       lag168_Load_DA         195
2           Price_lag_7         192
16  roll2d_median_Price         174
12            Oil_lag_2          87
10           Coal_lag_2          75
13            EUA_lag_2          71
5       WindOn_DA_lag_0          65
9                  WD_7          11
7                  WD_1           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 35.57% done
********************* END NS.. 259 ****************************************************
********************* START NS ... 260 ****************************************************
[I 2025-06-15 20:00:19,977] A new study created in memory with name: no-name-13266c47-0d19-49ec-9d0c-088b26685e94
[I 2025-06-15 20:00:21,824] Trial 0 finished with value: 0.0035352340109460006 and parameters: {'n_estimators': 278, 'learning_rate': 0.01578539005838948, 'num_leaves': 82, 'max_depth': 6, 'min_child_samples': 12, 'subsample': 0.8379142362617806, 'colsample_bytree': 0.6141587656696372, 'reg_alpha': 7.661752284930455, 'reg_lambda': 2.986401054047607, 'min_split_gain': 0.003584210597619414}. Best is trial 0 with value: 0.0035352340109460006.
[Trial 0] New best: 0.00354
[I 2025-06-15 20:00:23,109] Trial 1 finished with value: 0.00035077175003886494 and parameters: {'n_estimators': 260, 'learning_rate': 0.0448423019829154, 'num_leaves': 122, 'max_depth': 7, 'min_child_samples': 22, 'subsample': 0.6996334038117756, 'colsample_bytree': 0.671887569555363, 'reg_alpha': 4.189262192306563, 'reg_lambda': 7.161778128069346, 'min_split_gain': 0.022759053771648875}. Best is trial 1 with value: 0.00035077175003886494.
[Trial 1] New best: 0.00035
[I 2025-06-15 20:00:24,125] Trial 2 finished with value: 0.00771441824920673 and parameters: {'n_estimators': 82, 'learning_rate': 0.03846600054299791, 'num_leaves': 187, 'max_depth': 15, 'min_child_samples': 30, 'subsample': 0.8261602690616338, 'colsample_bytree': 0.5328361329315467, 'reg_alpha': 5.651123820153517, 'reg_lambda': 3.7107632686272063, 'min_split_gain': 0.03323394276100299}. Best is trial 1 with value: 0.00035077175003886494.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:00:24,449] Trial 3 finished with value: 0.0017494299159001002 and parameters: {'n_estimators': 185, 'learning_rate': 0.1777811686334828, 'num_leaves': 167, 'max_depth': 12, 'min_child_samples': 27, 'subsample': 0.9837982251379169, 'colsample_bytree': 0.8359718771451349, 'reg_alpha': 8.511677004504575, 'reg_lambda': 3.3708291746943253, 'min_split_gain': 0.04273887878790103}. Best is trial 1 with value: 0.00035077175003886494.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:00:25,345] Trial 4 finished with value: 0.048315805580326304 and parameters: {'n_estimators': 143, 'learning_rate': 0.012104824608304879, 'num_leaves': 59, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.5313138930243073, 'colsample_bytree': 0.8732132679775222, 'reg_alpha': 8.741401667202933, 'reg_lambda': 8.816986321834811, 'min_split_gain': 0.041408788271981196}. Best is trial 1 with value: 0.00035077175003886494.
[I 2025-06-15 20:00:25,347] A new study created in memory with name: no-name-1944788f-a919-446e-8fbf-9d83069ca603
[I 2025-06-15 20:00:25,376] Trial 0 finished with value: 0.002596956070924291 and parameters: {'n_estimators': 107, 'learning_rate': 0.17776288950853528, 'num_leaves': 37, 'max_depth': 4, 'min_child_samples': 16, 'subsample': 0.9965614784354409, 'colsample_bytree': 0.9499511833173797, 'reg_alpha': 5.164392469209547, 'reg_lambda': 5.554123660656755, 'min_split_gain': 0.03674788464178709}. Best is trial 0 with value: 0.002596956070924291.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1229
1           Price_lag_2         949
14      pct_chg_Load_DA         748
3         Load_DA_lag_0         577
11           NGas_lag_2         442
15       lag168_Load_DA         368
5       WindOn_DA_lag_0         354
2           Price_lag_7         340
16  roll2d_median_Price         296
10           Coal_lag_2         229
13            EUA_lag_2         204
12            Oil_lag_2         176
9                  WD_7          32
7                  WD_1          28
8                  WD_6          18
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 35.70% done
********************* END NS.. 260 ****************************************************
********************* START NS ... 261 ****************************************************
[I 2025-06-15 20:00:27,951] A new study created in memory with name: no-name-1e1be311-5a5c-4e47-ab3f-7db020fce2f0
[I 2025-06-15 20:00:29,021] Trial 0 finished with value: 0.0003510387169727137 and parameters: {'n_estimators': 194, 'learning_rate': 0.03588972769404678, 'num_leaves': 181, 'max_depth': 5, 'min_child_samples': 29, 'subsample': 0.7837607641627502, 'colsample_bytree': 0.6695949190309778, 'reg_alpha': 7.990666877815022, 'reg_lambda': 2.6765070582590123, 'min_split_gain': 0.014170068220924448}. Best is trial 0 with value: 0.0003510387169727137.
[Trial 0] New best: 0.00035
[I 2025-06-15 20:00:29,875] Trial 1 finished with value: 0.0006887264548456124 and parameters: {'n_estimators': 93, 'learning_rate': 0.04767377974950124, 'num_leaves': 179, 'max_depth': 9, 'min_child_samples': 27, 'subsample': 0.9243862743946312, 'colsample_bytree': 0.9821578300122825, 'reg_alpha': 3.1226465146867888, 'reg_lambda': 8.453028789624828, 'min_split_gain': 0.025468140976573135}. Best is trial 0 with value: 0.0003510387169727137.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:00:32,105] Trial 2 finished with value: 0.0002045757071800792 and parameters: {'n_estimators': 153, 'learning_rate': 0.0355036138919412, 'num_leaves': 187, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.8341237799644718, 'colsample_bytree': 0.5842687969217562, 'reg_alpha': 1.2208520947947388, 'reg_lambda': 3.3780813213296144, 'min_split_gain': 0.024180212030396043}. Best is trial 2 with value: 0.0002045757071800792.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:00:32,781] Trial 3 finished with value: 9.970752076528652e-05 and parameters: {'n_estimators': 85, 'learning_rate': 0.15348116713412727, 'num_leaves': 47, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.701985177948149, 'colsample_bytree': 0.8148185220122213, 'reg_alpha': 4.317279198139511, 'reg_lambda': 8.60519033053442, 'min_split_gain': 0.005364930931171086}. Best is trial 3 with value: 9.970752076528652e-05.
[I 2025-06-15 20:00:32,784] A new study created in memory with name: no-name-bfb30d9a-8a52-4961-b772-b0ecf8981fb3
[I 2025-06-15 20:00:32,818] Trial 0 finished with value: 0.0006114439070508005 and parameters: {'n_estimators': 70, 'learning_rate': 0.27361381820004627, 'num_leaves': 200, 'max_depth': 14, 'min_child_samples': 27, 'subsample': 0.614850384982386, 'colsample_bytree': 0.5568826935028617, 'reg_alpha': 3.4693252579555676, 'reg_lambda': 5.181674457414917, 'min_split_gain': 0.010661610721777693}. Best is trial 0 with value: 0.0006114439070508005.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         449
1           Price_lag_2         356
14      pct_chg_Load_DA         331
3         Load_DA_lag_0         276
5       WindOn_DA_lag_0         237
15       lag168_Load_DA         234
2           Price_lag_7         195
16  roll2d_median_Price         164
13            EUA_lag_2         161
11           NGas_lag_2         157
10           Coal_lag_2         123
12            Oil_lag_2         122
9                  WD_7          20
7                  WD_1           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 35.84% done
********************* END NS.. 261 ****************************************************
********************* START NS ... 262 ****************************************************
[I 2025-06-15 20:00:34,942] A new study created in memory with name: no-name-8658452c-935b-4aa2-bea3-580388585534
[I 2025-06-15 20:00:35,432] Trial 0 finished with value: 0.000749305173481158 and parameters: {'n_estimators': 59, 'learning_rate': 0.14380016783640334, 'num_leaves': 60, 'max_depth': 16, 'min_child_samples': 2, 'subsample': 0.5201229722174137, 'colsample_bytree': 0.5123460692484817, 'reg_alpha': 8.79504999046327, 'reg_lambda': 8.132488468779863, 'min_split_gain': 0.020276008070714138}. Best is trial 0 with value: 0.000749305173481158.
[Trial 0] New best: 0.00075
[I 2025-06-15 20:00:36,068] Trial 1 finished with value: 0.00040124451452693157 and parameters: {'n_estimators': 162, 'learning_rate': 0.11735961026046539, 'num_leaves': 195, 'max_depth': 8, 'min_child_samples': 15, 'subsample': 0.9591253729215211, 'colsample_bytree': 0.5848181758236695, 'reg_alpha': 5.522939176247279, 'reg_lambda': 4.542631130560143, 'min_split_gain': 0.015550663387247582}. Best is trial 1 with value: 0.00040124451452693157.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:00:37,574] Trial 2 finished with value: 0.0006761019423425881 and parameters: {'n_estimators': 276, 'learning_rate': 0.06389625313979327, 'num_leaves': 89, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.5252495305693525, 'colsample_bytree': 0.8746900351790382, 'reg_alpha': 6.637423556874338, 'reg_lambda': 6.66397462067997, 'min_split_gain': 0.002211583773556414}. Best is trial 1 with value: 0.00040124451452693157.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:00:38,582] Trial 3 finished with value: 0.0008490330430156924 and parameters: {'n_estimators': 281, 'learning_rate': 0.027539055651359678, 'num_leaves': 209, 'max_depth': 4, 'min_child_samples': 24, 'subsample': 0.5588431274861532, 'colsample_bytree': 0.7808022685706775, 'reg_alpha': 4.102398504346871, 'reg_lambda': 0.9920267897957646, 'min_split_gain': 0.030966482309869387}. Best is trial 1 with value: 0.00040124451452693157.
[I 2025-06-15 20:00:38,585] A new study created in memory with name: no-name-438da2d0-c7f6-43e1-86d1-5a1fb1c0200b
[I 2025-06-15 20:00:38,622] Trial 0 finished with value: 0.0011376261222583463 and parameters: {'n_estimators': 115, 'learning_rate': 0.2438982038333206, 'num_leaves': 170, 'max_depth': 8, 'min_child_samples': 14, 'subsample': 0.6768304788185966, 'colsample_bytree': 0.6076355803282381, 'reg_alpha': 5.743688684240375, 'reg_lambda': 7.261567920998817, 'min_split_gain': 0.023417693804883816}. Best is trial 0 with value: 0.0011376261222583463.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         522
1           Price_lag_2         390
14      pct_chg_Load_DA         257
3         Load_DA_lag_0         217
11           NGas_lag_2         189
2           Price_lag_7         164
5       WindOn_DA_lag_0         158
16  roll2d_median_Price         128
13            EUA_lag_2         120
15       lag168_Load_DA         119
10           Coal_lag_2         110
12            Oil_lag_2          90
9                  WD_7          16
7                  WD_1          12
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 35.98% done
********************* END NS.. 262 ****************************************************
********************* START NS ... 263 ****************************************************
[I 2025-06-15 20:00:40,653] A new study created in memory with name: no-name-42fbe2a5-d4f4-4a2d-a4f8-5463dc5b028c
[I 2025-06-15 20:00:41,531] Trial 0 finished with value: 0.003952705677621134 and parameters: {'n_estimators': 91, 'learning_rate': 0.058682937329610343, 'num_leaves': 84, 'max_depth': 10, 'min_child_samples': 9, 'subsample': 0.7659500584857453, 'colsample_bytree': 0.930918725992075, 'reg_alpha': 6.869670800057072, 'reg_lambda': 8.5061790963178, 'min_split_gain': 0.02488269886137965}. Best is trial 0 with value: 0.003952705677621134.
[Trial 0] New best: 0.00395
[I 2025-06-15 20:00:42,173] Trial 1 finished with value: 0.0029958125865720883 and parameters: {'n_estimators': 84, 'learning_rate': 0.08952228532095001, 'num_leaves': 224, 'max_depth': 8, 'min_child_samples': 12, 'subsample': 0.5977238984456368, 'colsample_bytree': 0.7447666040392866, 'reg_alpha': 8.263227270380408, 'reg_lambda': 4.38688522469579, 'min_split_gain': 0.00913326696049201}. Best is trial 1 with value: 0.0029958125865720883.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:00:43,790] Trial 2 finished with value: 0.004916876438872156 and parameters: {'n_estimators': 216, 'learning_rate': 0.020338122632064727, 'num_leaves': 107, 'max_depth': 13, 'min_child_samples': 18, 'subsample': 0.8214575122644903, 'colsample_bytree': 0.9081031738126458, 'reg_alpha': 5.077754677774364, 'reg_lambda': 0.08114426527427554, 'min_split_gain': 0.04350425395018746}. Best is trial 1 with value: 0.0029958125865720883.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:00:44,807] Trial 3 finished with value: 0.04944023883187001 and parameters: {'n_estimators': 122, 'learning_rate': 0.014861849259940618, 'num_leaves': 141, 'max_depth': 16, 'min_child_samples': 21, 'subsample': 0.6034112575079, 'colsample_bytree': 0.8885500449115424, 'reg_alpha': 5.926954104847786, 'reg_lambda': 4.53089171685806, 'min_split_gain': 0.027974095799262744}. Best is trial 1 with value: 0.0029958125865720883.
[I 2025-06-15 20:00:44,810] A new study created in memory with name: no-name-631ea51c-14fc-4cb5-80c7-6ba2364561cb
[I 2025-06-15 20:00:44,833] Trial 0 finished with value: 0.0024096233159935578 and parameters: {'n_estimators': 70, 'learning_rate': 0.059754569807056394, 'num_leaves': 109, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.7908161498377664, 'colsample_bytree': 0.855121375455345, 'reg_alpha': 1.716266675717758, 'reg_lambda': 2.295606001548851, 'min_split_gain': 0.04428248028958942}. Best is trial 0 with value: 0.0024096233159935578.
[I 2025-06-15 20:00:44,859] Trial 1 finished with value: 0.0024096233159935578 and parameters: {'n_estimators': 76, 'learning_rate': 0.06160639021377857, 'num_leaves': 96, 'max_depth': 10, 'min_child_samples': 30, 'subsample': 0.9689592590027394, 'colsample_bytree': 0.6347808176091307, 'reg_alpha': 5.326626362654666, 'reg_lambda': 6.392003765941173, 'min_split_gain': 0.028567536274426476}. Best is trial 0 with value: 0.0024096233159935578.
[I 2025-06-15 20:00:44,897] Trial 2 finished with value: 0.0024096233159935578 and parameters: {'n_estimators': 143, 'learning_rate': 0.04152567240108376, 'num_leaves': 167, 'max_depth': 11, 'min_child_samples': 20, 'subsample': 0.6777398558976904, 'colsample_bytree': 0.7478628179814473, 'reg_alpha': 5.495254699119428, 'reg_lambda': 9.94866456197471, 'min_split_gain': 0.004603820068456416}. Best is trial 0 with value: 0.0024096233159935578.
[I 2025-06-15 20:00:44,943] Trial 3 finished with value: 0.0024096233159935578 and parameters: {'n_estimators': 180, 'learning_rate': 0.05164008565728463, 'num_leaves': 77, 'max_depth': 9, 'min_child_samples': 16, 'subsample': 0.6932383263953572, 'colsample_bytree': 0.9475778868731095, 'reg_alpha': 0.5492817281286089, 'reg_lambda': 0.4479943879942361, 'min_split_gain': 0.02029034217674765}. Best is trial 0 with value: 0.0024096233159935578.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] New best: 0.00241
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         530
1           Price_lag_2         369
14      pct_chg_Load_DA         341
3         Load_DA_lag_0         219
11           NGas_lag_2         177
2           Price_lag_7         157
16  roll2d_median_Price         138
5       WindOn_DA_lag_0         134
13            EUA_lag_2         121
10           Coal_lag_2         119
15       lag168_Load_DA         118
12            Oil_lag_2          91
9                  WD_7          15
8                  WD_6          11
7                  WD_1           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 36.11% done
********************* END NS.. 263 ****************************************************
********************* START NS ... 264 ****************************************************
[I 2025-06-15 20:00:47,226] A new study created in memory with name: no-name-bf46e84e-d2c7-4e2a-b85d-a671b69e0cbb
[I 2025-06-15 20:00:47,614] Trial 0 finished with value: 0.0016226438042277348 and parameters: {'n_estimators': 215, 'learning_rate': 0.15400453774571302, 'num_leaves': 179, 'max_depth': 13, 'min_child_samples': 21, 'subsample': 0.623205463649978, 'colsample_bytree': 0.652302411718748, 'reg_alpha': 5.67798889491753, 'reg_lambda': 9.820379536436775, 'min_split_gain': 0.04975565859993393}. Best is trial 0 with value: 0.0016226438042277348.
[Trial 0] New best: 0.00162
[I 2025-06-15 20:00:48,143] Trial 1 finished with value: 0.00036356218409510857 and parameters: {'n_estimators': 219, 'learning_rate': 0.06917145216887152, 'num_leaves': 204, 'max_depth': 5, 'min_child_samples': 30, 'subsample': 0.989587458472645, 'colsample_bytree': 0.9165790403789242, 'reg_alpha': 2.448376104269203, 'reg_lambda': 3.7259446126080373, 'min_split_gain': 0.01517089715849495}. Best is trial 1 with value: 0.00036356218409510857.
[Trial 1] New best: 0.00036
[I 2025-06-15 20:00:49,582] Trial 2 finished with value: 0.005250615513622682 and parameters: {'n_estimators': 260, 'learning_rate': 0.01575742452837964, 'num_leaves': 254, 'max_depth': 7, 'min_child_samples': 17, 'subsample': 0.615551422971953, 'colsample_bytree': 0.7069043384267839, 'reg_alpha': 9.96046998932956, 'reg_lambda': 7.34338576579364, 'min_split_gain': 0.027861296336827986}. Best is trial 1 with value: 0.00036356218409510857.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:00:51,059] Trial 3 finished with value: 0.0178793643184228 and parameters: {'n_estimators': 165, 'learning_rate': 0.015099980045677938, 'num_leaves': 58, 'max_depth': 14, 'min_child_samples': 17, 'subsample': 0.664186819413609, 'colsample_bytree': 0.8028203912064406, 'reg_alpha': 6.441635813269003, 'reg_lambda': 0.6328024421543155, 'min_split_gain': 0.021002140253945145}. Best is trial 1 with value: 0.00036356218409510857.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:00:54,946] Trial 4 finished with value: 0.00035719494930678544 and parameters: {'n_estimators': 256, 'learning_rate': 0.05507160295855369, 'num_leaves': 245, 'max_depth': 12, 'min_child_samples': 9, 'subsample': 0.8832968931430545, 'colsample_bytree': 0.6316057559989098, 'reg_alpha': 3.173887060874554, 'reg_lambda': 4.401867437488881, 'min_split_gain': 0.0012809555335245026}. Best is trial 4 with value: 0.00035719494930678544.
[I 2025-06-15 20:00:54,948] A new study created in memory with name: no-name-b24b5551-8fcf-42dc-ab0a-a574d701be06
[I 2025-06-15 20:00:54,991] Trial 0 finished with value: 0.0005751991500077228 and parameters: {'n_estimators': 170, 'learning_rate': 0.1951113670976235, 'num_leaves': 227, 'max_depth': 10, 'min_child_samples': 15, 'subsample': 0.6377282249247995, 'colsample_bytree': 0.9523507929661459, 'reg_alpha': 0.2880430963468217, 'reg_lambda': 9.69187319928698, 'min_split_gain': 0.0020585650415581858}. Best is trial 0 with value: 0.0005751991500077228.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2301
1           Price_lag_2        2197
14      pct_chg_Load_DA        2016
5       WindOn_DA_lag_0        1826
3         Load_DA_lag_0        1692
15       lag168_Load_DA        1566
2           Price_lag_7        1547
13            EUA_lag_2        1208
12            Oil_lag_2        1165
11           NGas_lag_2        1150
10           Coal_lag_2        1087
16  roll2d_median_Price        1086
9                  WD_7          90
7                  WD_1          50
8                  WD_6          42
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          16
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 36.25% done
********************* END NS.. 264 ****************************************************
********************* START NS ... 265 ****************************************************
[I 2025-06-15 20:00:59,651] A new study created in memory with name: no-name-b4fc6551-d780-4782-bdee-cadf9786b540
[I 2025-06-15 20:01:00,215] Trial 0 finished with value: 0.0009684602375461456 and parameters: {'n_estimators': 300, 'learning_rate': 0.07153720209086366, 'num_leaves': 166, 'max_depth': 6, 'min_child_samples': 10, 'subsample': 0.8139450843516389, 'colsample_bytree': 0.8041740755252299, 'reg_alpha': 9.67629009364033, 'reg_lambda': 4.873715667393878, 'min_split_gain': 0.03590540088054249}. Best is trial 0 with value: 0.0009684602375461456.
[Trial 0] New best: 0.00097
[I 2025-06-15 20:01:01,082] Trial 1 finished with value: 0.005474013231836383 and parameters: {'n_estimators': 114, 'learning_rate': 0.027464470072834788, 'num_leaves': 126, 'max_depth': 14, 'min_child_samples': 16, 'subsample': 0.9261415772016421, 'colsample_bytree': 0.8441014228870627, 'reg_alpha': 4.337130848180726, 'reg_lambda': 7.438228515936693, 'min_split_gain': 0.04782477798413728}. Best is trial 0 with value: 0.0009684602375461456.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:01:02,384] Trial 2 finished with value: 0.02065228769009454 and parameters: {'n_estimators': 169, 'learning_rate': 0.013109183502153433, 'num_leaves': 168, 'max_depth': 11, 'min_child_samples': 15, 'subsample': 0.5603538904047477, 'colsample_bytree': 0.5273786310892616, 'reg_alpha': 7.722314675917667, 'reg_lambda': 3.4355779700061415, 'min_split_gain': 0.03754641989510906}. Best is trial 0 with value: 0.0009684602375461456.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:01:05,407] Trial 3 finished with value: 0.0015632760359639258 and parameters: {'n_estimators': 300, 'learning_rate': 0.01251559640255889, 'num_leaves': 46, 'max_depth': 9, 'min_child_samples': 3, 'subsample': 0.6326120871960446, 'colsample_bytree': 0.8144066525818328, 'reg_alpha': 1.2798524826743418, 'reg_lambda': 2.69619131178359, 'min_split_gain': 0.02014093643379694}. Best is trial 0 with value: 0.0009684602375461456.
[I 2025-06-15 20:01:05,410] A new study created in memory with name: no-name-ab65a45d-33ef-48fd-8692-0fc012f1ecd1
[I 2025-06-15 20:01:05,433] Trial 0 finished with value: 0.0007048672355761138 and parameters: {'n_estimators': 60, 'learning_rate': 0.09894034604296542, 'num_leaves': 37, 'max_depth': 14, 'min_child_samples': 10, 'subsample': 0.8726836408072272, 'colsample_bytree': 0.7416250286302752, 'reg_alpha': 1.587029992660286, 'reg_lambda': 9.048890327468062, 'min_split_gain': 0.013893228607444592}. Best is trial 0 with value: 0.0007048672355761138.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         620
1           Price_lag_2         324
14      pct_chg_Load_DA         323
3         Load_DA_lag_0         181
11           NGas_lag_2         161
2           Price_lag_7         133
16  roll2d_median_Price         111
10           Coal_lag_2          93
13            EUA_lag_2          75
15       lag168_Load_DA          75
5       WindOn_DA_lag_0          48
12            Oil_lag_2          47
9                  WD_7          15
7                  WD_1           5
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 36.39% done
********************* END NS.. 265 ****************************************************
********************* START NS ... 266 ****************************************************
[I 2025-06-15 20:01:07,664] A new study created in memory with name: no-name-d9c302aa-7803-48dd-a6fc-77ef94da72a2
[I 2025-06-15 20:01:10,343] Trial 0 finished with value: 0.0007228249560750702 and parameters: {'n_estimators': 236, 'learning_rate': 0.0160408035425446, 'num_leaves': 187, 'max_depth': 11, 'min_child_samples': 28, 'subsample': 0.6221436598459127, 'colsample_bytree': 0.7930797106912006, 'reg_alpha': 1.947254150617439, 'reg_lambda': 1.6836775176444985, 'min_split_gain': 0.014891212548130623}. Best is trial 0 with value: 0.0007228249560750702.
[Trial 0] New best: 0.00072
[I 2025-06-15 20:01:11,411] Trial 1 finished with value: 7.652496435081708e-05 and parameters: {'n_estimators': 265, 'learning_rate': 0.14174078387225658, 'num_leaves': 211, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.8587769256912854, 'colsample_bytree': 0.8789261079146864, 'reg_alpha': 0.1140531656449062, 'reg_lambda': 8.361161455086188, 'min_split_gain': 0.029680120689947805}. Best is trial 1 with value: 7.652496435081708e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:01:12,068] Trial 2 finished with value: 0.0004466031262091383 and parameters: {'n_estimators': 244, 'learning_rate': 0.08583555245576957, 'num_leaves': 161, 'max_depth': 9, 'min_child_samples': 12, 'subsample': 0.5183625090042714, 'colsample_bytree': 0.8351089195149215, 'reg_alpha': 9.562141887075228, 'reg_lambda': 3.639907666559324, 'min_split_gain': 0.04078128085077398}. Best is trial 1 with value: 7.652496435081708e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:01:12,360] Trial 3 finished with value: 0.0003291078071938561 and parameters: {'n_estimators': 138, 'learning_rate': 0.16797897607718584, 'num_leaves': 125, 'max_depth': 5, 'min_child_samples': 23, 'subsample': 0.6393878536868951, 'colsample_bytree': 0.6877947161309372, 'reg_alpha': 8.497251787693768, 'reg_lambda': 4.668049724741882, 'min_split_gain': 0.017941870119201755}. Best is trial 1 with value: 7.652496435081708e-05.
[I 2025-06-15 20:01:12,362] A new study created in memory with name: no-name-c4fe3ca1-48db-4af7-86d3-0674a3c9b160
[I 2025-06-15 20:01:12,429] Trial 0 finished with value: 0.0008010525245785568 and parameters: {'n_estimators': 285, 'learning_rate': 0.025445125358842288, 'num_leaves': 256, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.5381015745299338, 'colsample_bytree': 0.8507167300416787, 'reg_alpha': 0.4449056861406664, 'reg_lambda': 6.754131365936683, 'min_split_gain': 0.024041804634367826}. Best is trial 0 with value: 0.0008010525245785568.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA         776
0           Price_lag_1         773
1           Price_lag_2         746
3         Load_DA_lag_0         566
16  roll2d_median_Price         403
2           Price_lag_7         397
15       lag168_Load_DA         376
11           NGas_lag_2         337
5       WindOn_DA_lag_0         286
10           Coal_lag_2         260
12            Oil_lag_2         185
13            EUA_lag_2         129
9                  WD_7          14
7                  WD_1           9
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          19
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 36.53% done
********************* END NS.. 266 ****************************************************
********************* START NS ... 267 ****************************************************
[I 2025-06-15 20:01:14,981] A new study created in memory with name: no-name-a0db0bce-4a30-4e1f-92f1-149d3e3b8f58
[I 2025-06-15 20:01:15,465] Trial 0 finished with value: 9.985658849184716e-05 and parameters: {'n_estimators': 157, 'learning_rate': 0.21146445766396113, 'num_leaves': 156, 'max_depth': 15, 'min_child_samples': 9, 'subsample': 0.7788341746966473, 'colsample_bytree': 0.9606086812098464, 'reg_alpha': 2.162464200151314, 'reg_lambda': 1.461074979311231, 'min_split_gain': 0.03477333486973783}. Best is trial 0 with value: 9.985658849184716e-05.
[Trial 0] New best: 0.00010
[I 2025-06-15 20:01:16,099] Trial 1 finished with value: 0.00018425772265972463 and parameters: {'n_estimators': 145, 'learning_rate': 0.1474564976356686, 'num_leaves': 237, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.8475800273137521, 'colsample_bytree': 0.502406840810723, 'reg_alpha': 9.465195843059606, 'reg_lambda': 5.112007881985995, 'min_split_gain': 0.007713266968443999}. Best is trial 0 with value: 9.985658849184716e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:01:17,298] Trial 2 finished with value: 0.0001543174490060725 and parameters: {'n_estimators': 232, 'learning_rate': 0.04665992122794975, 'num_leaves': 136, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.8568725069128501, 'colsample_bytree': 0.9412220460031291, 'reg_alpha': 1.3103416199308826, 'reg_lambda': 7.233835256460149, 'min_split_gain': 0.04185257109941736}. Best is trial 0 with value: 9.985658849184716e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:01:17,833] Trial 3 finished with value: 0.25390508877324514 and parameters: {'n_estimators': 64, 'learning_rate': 0.010040256516482304, 'num_leaves': 52, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.8810545995667454, 'colsample_bytree': 0.911767527349156, 'reg_alpha': 1.1016650835975372, 'reg_lambda': 5.170073998613677, 'min_split_gain': 0.0362089593495929}. Best is trial 0 with value: 9.985658849184716e-05.
[I 2025-06-15 20:01:17,836] A new study created in memory with name: no-name-3a04e711-8fdb-40dc-af7f-1533083dbdd4
[I 2025-06-15 20:01:17,864] Trial 0 finished with value: 0.001237451179198084 and parameters: {'n_estimators': 76, 'learning_rate': 0.22576303873799983, 'num_leaves': 64, 'max_depth': 12, 'min_child_samples': 4, 'subsample': 0.7503474297952847, 'colsample_bytree': 0.6864390399203779, 'reg_alpha': 1.153623678212552, 'reg_lambda': 3.3660437671966514, 'min_split_gain': 0.03204158355873329}. Best is trial 0 with value: 0.001237451179198084.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         312
14      pct_chg_Load_DA         257
1           Price_lag_2         210
3         Load_DA_lag_0         196
2           Price_lag_7         118
16  roll2d_median_Price         114
11           NGas_lag_2         107
5       WindOn_DA_lag_0         101
15       lag168_Load_DA          75
10           Coal_lag_2          68
12            Oil_lag_2          58
13            EUA_lag_2          48
9                  WD_7          11
7                  WD_1           6
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 36.66% done
********************* END NS.. 267 ****************************************************
********************* START NS ... 268 ****************************************************
[I 2025-06-15 20:01:20,133] A new study created in memory with name: no-name-2f90a6f9-230b-48e9-b52b-a6b44e8d4803
[I 2025-06-15 20:01:20,495] Trial 0 finished with value: 8.339068574098647e-05 and parameters: {'n_estimators': 244, 'learning_rate': 0.16531836089869853, 'num_leaves': 149, 'max_depth': 8, 'min_child_samples': 17, 'subsample': 0.7556625385476506, 'colsample_bytree': 0.898119601104028, 'reg_alpha': 4.391941774133944, 'reg_lambda': 9.813888121777566, 'min_split_gain': 0.046810089548372985}. Best is trial 0 with value: 8.339068574098647e-05.
[Trial 0] New best: 0.00008
[I 2025-06-15 20:01:21,928] Trial 1 finished with value: 0.00012945443315695481 and parameters: {'n_estimators': 238, 'learning_rate': 0.05673964548296458, 'num_leaves': 227, 'max_depth': 15, 'min_child_samples': 22, 'subsample': 0.5482643565002013, 'colsample_bytree': 0.5450164764832174, 'reg_alpha': 3.3762059136711198, 'reg_lambda': 0.4276715825176225, 'min_split_gain': 0.018836848088786606}. Best is trial 0 with value: 8.339068574098647e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:01:22,615] Trial 2 finished with value: 0.00015493749683519493 and parameters: {'n_estimators': 191, 'learning_rate': 0.06612298696884059, 'num_leaves': 213, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.8179618237875745, 'colsample_bytree': 0.5630092232353276, 'reg_alpha': 0.530661256538264, 'reg_lambda': 8.96049334896073, 'min_split_gain': 0.023605908214746892}. Best is trial 0 with value: 8.339068574098647e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:01:23,553] Trial 3 finished with value: 0.0005002520288109379 and parameters: {'n_estimators': 74, 'learning_rate': 0.057437775901495916, 'num_leaves': 98, 'max_depth': 10, 'min_child_samples': 12, 'subsample': 0.9085244626831205, 'colsample_bytree': 0.8387076395823504, 'reg_alpha': 4.587017462989954, 'reg_lambda': 5.968916917912215, 'min_split_gain': 0.0008528794214629876}. Best is trial 0 with value: 8.339068574098647e-05.
[I 2025-06-15 20:01:23,555] A new study created in memory with name: no-name-25997d98-0c78-4874-a1e2-e8872ba7e822
[I 2025-06-15 20:01:23,611] Trial 0 finished with value: 0.0007941200104780755 and parameters: {'n_estimators': 246, 'learning_rate': 0.18609819365346375, 'num_leaves': 179, 'max_depth': 7, 'min_child_samples': 25, 'subsample': 0.7090601891763938, 'colsample_bytree': 0.9333666207611856, 'reg_alpha': 8.352966013241465, 'reg_lambda': 3.2421355975891366, 'min_split_gain': 0.00758115837188288}. Best is trial 0 with value: 0.0007941200104780755.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         310
14      pct_chg_Load_DA         175
1           Price_lag_2         157
3         Load_DA_lag_0         105
11           NGas_lag_2          94
2           Price_lag_7          73
16  roll2d_median_Price          72
15       lag168_Load_DA          68
10           Coal_lag_2          53
13            EUA_lag_2          36
5       WindOn_DA_lag_0          28
12            Oil_lag_2          25
9                  WD_7           6
7                  WD_1           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 36.80% done
********************* END NS.. 268 ****************************************************
********************* START NS ... 269 ****************************************************
[I 2025-06-15 20:01:25,677] A new study created in memory with name: no-name-c13898fc-ef55-4c25-98bc-527f703eaad4
[I 2025-06-15 20:01:26,464] Trial 0 finished with value: 0.0005731486890913583 and parameters: {'n_estimators': 164, 'learning_rate': 0.038653461213034986, 'num_leaves': 84, 'max_depth': 5, 'min_child_samples': 24, 'subsample': 0.5272994488412039, 'colsample_bytree': 0.5174044666283181, 'reg_alpha': 7.390651391743229, 'reg_lambda': 5.892155105959883, 'min_split_gain': 0.00399144023013856}. Best is trial 0 with value: 0.0005731486890913583.
[Trial 0] New best: 0.00057
[I 2025-06-15 20:01:26,920] Trial 1 finished with value: 0.00011345824930560625 and parameters: {'n_estimators': 150, 'learning_rate': 0.211416221870646, 'num_leaves': 255, 'max_depth': 14, 'min_child_samples': 14, 'subsample': 0.6353603945762755, 'colsample_bytree': 0.8153979651962591, 'reg_alpha': 2.903066330439057, 'reg_lambda': 0.8677011081694919, 'min_split_gain': 0.01866753105056747}. Best is trial 1 with value: 0.00011345824930560625.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:01:28,165] Trial 2 finished with value: 0.0025220067730561996 and parameters: {'n_estimators': 177, 'learning_rate': 0.021961011314420097, 'num_leaves': 113, 'max_depth': 8, 'min_child_samples': 9, 'subsample': 0.8172073394774544, 'colsample_bytree': 0.9825263342568347, 'reg_alpha': 8.77547240342925, 'reg_lambda': 7.986882130433031, 'min_split_gain': 0.029607583378570185}. Best is trial 1 with value: 0.00011345824930560625.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:01:28,910] Trial 3 finished with value: 0.00041831179102391714 and parameters: {'n_estimators': 294, 'learning_rate': 0.10456491712078103, 'num_leaves': 120, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.837300201811505, 'colsample_bytree': 0.6121792648498201, 'reg_alpha': 8.379320242166337, 'reg_lambda': 8.045454759005356, 'min_split_gain': 0.04508965799404499}. Best is trial 1 with value: 0.00011345824930560625.
[I 2025-06-15 20:01:28,913] A new study created in memory with name: no-name-2589919f-4154-4d16-ba18-2d5b5666b2f3
[I 2025-06-15 20:01:28,974] Trial 0 finished with value: 0.0003648424777545198 and parameters: {'n_estimators': 246, 'learning_rate': 0.013902676403322699, 'num_leaves': 64, 'max_depth': 6, 'min_child_samples': 7, 'subsample': 0.575494385965314, 'colsample_bytree': 0.7746711698979367, 'reg_alpha': 5.9255655538763925, 'reg_lambda': 5.490073452740488, 'min_split_gain': 0.029281097692944603}. Best is trial 0 with value: 0.0003648424777545198.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         325
1           Price_lag_2         263
14      pct_chg_Load_DA         258
3         Load_DA_lag_0         186
15       lag168_Load_DA         134
11           NGas_lag_2         124
2           Price_lag_7         120
5       WindOn_DA_lag_0         106
16  roll2d_median_Price         103
12            Oil_lag_2          72
10           Coal_lag_2          65
13            EUA_lag_2          54
9                  WD_7           5
7                  WD_1           4
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 36.94% done
********************* END NS.. 269 ****************************************************
********************* START NS ... 270 ****************************************************
[I 2025-06-15 20:01:30,983] A new study created in memory with name: no-name-a6841062-adff-4b62-953b-ce7d56f7555c
[I 2025-06-15 20:01:31,514] Trial 0 finished with value: 0.03660694129700917 and parameters: {'n_estimators': 100, 'learning_rate': 0.01744034853213938, 'num_leaves': 66, 'max_depth': 5, 'min_child_samples': 2, 'subsample': 0.8308706110517101, 'colsample_bytree': 0.5266984885646662, 'reg_alpha': 1.0600484427962797, 'reg_lambda': 0.13573262812260012, 'min_split_gain': 0.011444415630838957}. Best is trial 0 with value: 0.03660694129700917.
[Trial 0] New best: 0.03661
[I 2025-06-15 20:01:31,935] Trial 1 finished with value: 0.0002432457028349929 and parameters: {'n_estimators': 87, 'learning_rate': 0.1400479954601813, 'num_leaves': 131, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.552908741793851, 'colsample_bytree': 0.8813891077224665, 'reg_alpha': 3.5108429780493724, 'reg_lambda': 4.9176173358505935, 'min_split_gain': 0.015611859956459424}. Best is trial 1 with value: 0.0002432457028349929.
[Trial 1] New best: 0.00024
[I 2025-06-15 20:01:32,311] Trial 2 finished with value: 0.0008282101837663809 and parameters: {'n_estimators': 233, 'learning_rate': 0.18171763771550392, 'num_leaves': 51, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.9893722662083135, 'colsample_bytree': 0.8880193265769796, 'reg_alpha': 7.074782392817615, 'reg_lambda': 8.297618261151234, 'min_split_gain': 0.025439448181693875}. Best is trial 1 with value: 0.0002432457028349929.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:01:33,009] Trial 3 finished with value: 0.000552994224429069 and parameters: {'n_estimators': 137, 'learning_rate': 0.07525555613334013, 'num_leaves': 181, 'max_depth': 9, 'min_child_samples': 19, 'subsample': 0.5548262686850354, 'colsample_bytree': 0.6096865656492436, 'reg_alpha': 7.7212529677509565, 'reg_lambda': 5.464178512931525, 'min_split_gain': 0.03026900791725536}. Best is trial 1 with value: 0.0002432457028349929.
[I 2025-06-15 20:01:33,210] Trial 4 finished with value: 0.0012860604998618695 and parameters: {'n_estimators': 55, 'learning_rate': 0.07834203686118402, 'num_leaves': 151, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.52531584642644, 'colsample_bytree': 0.9107898791853405, 'reg_alpha': 2.6816549439004564, 'reg_lambda': 1.466999233963212, 'min_split_gain': 0.006874733115040727}. Best is trial 1 with value: 0.0002432457028349929.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:01:33,213] A new study created in memory with name: no-name-97f9c1f3-ff12-4dab-89bb-d543b1fde80e
[I 2025-06-15 20:01:33,267] Trial 0 finished with value: 0.0009067273237784002 and parameters: {'n_estimators': 238, 'learning_rate': 0.012218748641247982, 'num_leaves': 105, 'max_depth': 5, 'min_child_samples': 22, 'subsample': 0.733726885066706, 'colsample_bytree': 0.7227430789724847, 'reg_alpha': 3.9682003528060683, 'reg_lambda': 7.435345384702533, 'min_split_gain': 0.023601567280727744}. Best is trial 0 with value: 0.0009067273237784002.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         433
1           Price_lag_2         290
14      pct_chg_Load_DA         288
3         Load_DA_lag_0         200
2           Price_lag_7         164
11           NGas_lag_2         139
5       WindOn_DA_lag_0         131
16  roll2d_median_Price         118
15       lag168_Load_DA         112
10           Coal_lag_2          99
12            Oil_lag_2          84
13            EUA_lag_2          65
9                  WD_7          12
7                  WD_1           7
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 37.07% done
********************* END NS.. 270 ****************************************************
********************* START NS ... 271 ****************************************************
[I 2025-06-15 20:01:35,339] A new study created in memory with name: no-name-02f946f8-4a4d-43a5-99d6-5a8f419c462b
[I 2025-06-15 20:01:36,537] Trial 0 finished with value: 0.0007367425697445143 and parameters: {'n_estimators': 277, 'learning_rate': 0.04188137356729834, 'num_leaves': 156, 'max_depth': 9, 'min_child_samples': 3, 'subsample': 0.6043107650753581, 'colsample_bytree': 0.9784977598423323, 'reg_alpha': 9.080700266628678, 'reg_lambda': 3.09332528111446, 'min_split_gain': 0.02243687107324785}. Best is trial 0 with value: 0.0007367425697445143.
[Trial 0] New best: 0.00074
[I 2025-06-15 20:01:38,291] Trial 1 finished with value: 0.016473780152481365 and parameters: {'n_estimators': 163, 'learning_rate': 0.014172210295900881, 'num_leaves': 86, 'max_depth': 11, 'min_child_samples': 4, 'subsample': 0.5797863546788579, 'colsample_bytree': 0.8192116240649556, 'reg_alpha': 3.6743345112619465, 'reg_lambda': 9.496655020165917, 'min_split_gain': 0.035689227812026124}. Best is trial 0 with value: 0.0007367425697445143.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:01:39,645] Trial 2 finished with value: 0.0005834028076929905 and parameters: {'n_estimators': 102, 'learning_rate': 0.07641804884928208, 'num_leaves': 162, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.5612741150072318, 'colsample_bytree': 0.6580080661116957, 'reg_alpha': 7.98735047005921, 'reg_lambda': 5.068837315307534, 'min_split_gain': 0.002470026213312954}. Best is trial 2 with value: 0.0005834028076929905.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:01:40,140] Trial 3 finished with value: 0.00034438013346370805 and parameters: {'n_estimators': 277, 'learning_rate': 0.1335957870016341, 'num_leaves': 231, 'max_depth': 4, 'min_child_samples': 13, 'subsample': 0.8052195889162823, 'colsample_bytree': 0.775386412737898, 'reg_alpha': 4.885165826785949, 'reg_lambda': 1.6876314491304167, 'min_split_gain': 0.031051234534747375}. Best is trial 3 with value: 0.00034438013346370805.
[I 2025-06-15 20:01:40,143] A new study created in memory with name: no-name-903d89dc-ec5a-4f76-93ac-5aa9ee7c7a97
[I 2025-06-15 20:01:40,169] Trial 0 finished with value: 0.0006596017137151354 and parameters: {'n_estimators': 56, 'learning_rate': 0.21573645966089464, 'num_leaves': 223, 'max_depth': 9, 'min_child_samples': 6, 'subsample': 0.8671435007446301, 'colsample_bytree': 0.9811827990899695, 'reg_alpha': 2.1974240838489214, 'reg_lambda': 2.9448582282751468, 'min_split_gain': 0.005492324658202874}. Best is trial 0 with value: 0.0006596017137151354.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         291
14      pct_chg_Load_DA         127
1           Price_lag_2         120
3         Load_DA_lag_0          93
11           NGas_lag_2          65
16  roll2d_median_Price          61
15       lag168_Load_DA          51
10           Coal_lag_2          48
2           Price_lag_7          45
12            Oil_lag_2          31
13            EUA_lag_2          27
5       WindOn_DA_lag_0          21
7                  WD_1           6
9                  WD_7           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 37.21% done
********************* END NS.. 271 ****************************************************
********************* START NS ... 272 ****************************************************
[I 2025-06-15 20:01:42,256] A new study created in memory with name: no-name-9d21ec51-4160-4ee6-a786-e6785123e6bd
[I 2025-06-15 20:01:42,687] Trial 0 finished with value: 0.0002468116982460016 and parameters: {'n_estimators': 92, 'learning_rate': 0.10201494923018171, 'num_leaves': 194, 'max_depth': 7, 'min_child_samples': 12, 'subsample': 0.9559300429458851, 'colsample_bytree': 0.5734144037116959, 'reg_alpha': 3.6897739918303483, 'reg_lambda': 6.078196542563486, 'min_split_gain': 0.012696915318021779}. Best is trial 0 with value: 0.0002468116982460016.
[Trial 0] New best: 0.00025
[I 2025-06-15 20:01:43,455] Trial 1 finished with value: 0.0002021673070485645 and parameters: {'n_estimators': 88, 'learning_rate': 0.059816701550290445, 'num_leaves': 175, 'max_depth': 9, 'min_child_samples': 21, 'subsample': 0.6812805145165796, 'colsample_bytree': 0.6004985291958767, 'reg_alpha': 6.32628078496113, 'reg_lambda': 8.532289847024098, 'min_split_gain': 0.0001597632412393635}. Best is trial 1 with value: 0.0002021673070485645.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:01:46,399] Trial 2 finished with value: 0.0038301724783175958 and parameters: {'n_estimators': 95, 'learning_rate': 0.029794628353669694, 'num_leaves': 187, 'max_depth': 13, 'min_child_samples': 1, 'subsample': 0.8590504928364107, 'colsample_bytree': 0.8899242775993101, 'reg_alpha': 1.4514083403576161, 'reg_lambda': 9.474080711490808, 'min_split_gain': 3.9277459666547945e-05}. Best is trial 1 with value: 0.0002021673070485645.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:01:46,882] Trial 3 finished with value: 0.01028582682756575 and parameters: {'n_estimators': 78, 'learning_rate': 0.031258198241709334, 'num_leaves': 198, 'max_depth': 13, 'min_child_samples': 23, 'subsample': 0.5638367800056028, 'colsample_bytree': 0.8610020458868136, 'reg_alpha': 9.3551837423004, 'reg_lambda': 6.2489885873739865, 'min_split_gain': 0.04993725101897284}. Best is trial 1 with value: 0.0002021673070485645.
[I 2025-06-15 20:01:46,884] A new study created in memory with name: no-name-268c7f58-5714-4184-b337-18b6419de76f
[I 2025-06-15 20:01:46,926] Trial 0 finished with value: 0.00036963103819211634 and parameters: {'n_estimators': 182, 'learning_rate': 0.07043652943540869, 'num_leaves': 67, 'max_depth': 10, 'min_child_samples': 17, 'subsample': 0.8503762938393731, 'colsample_bytree': 0.7104828857514827, 'reg_alpha': 8.84406202204913, 'reg_lambda': 6.354572596650226, 'min_split_gain': 0.025272463415198183}. Best is trial 0 with value: 0.00036963103819211634.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         709
0           Price_lag_1         690
14      pct_chg_Load_DA         454
2           Price_lag_7         303
3         Load_DA_lag_0         283
15       lag168_Load_DA         278
11           NGas_lag_2         262
16  roll2d_median_Price         238
13            EUA_lag_2         225
10           Coal_lag_2         214
5       WindOn_DA_lag_0         176
12            Oil_lag_2         153
9                  WD_7          25
8                  WD_6          10
7                  WD_1           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 37.35% done
********************* END NS.. 272 ****************************************************
********************* START NS ... 273 ****************************************************
[I 2025-06-15 20:01:49,111] A new study created in memory with name: no-name-c470a477-d81c-46e7-b427-3dba59d63e53
[I 2025-06-15 20:01:50,002] Trial 0 finished with value: 0.00016171944165890083 and parameters: {'n_estimators': 234, 'learning_rate': 0.10020875539530838, 'num_leaves': 167, 'max_depth': 16, 'min_child_samples': 7, 'subsample': 0.7820064501111568, 'colsample_bytree': 0.55102739017933, 'reg_alpha': 5.822683666977299, 'reg_lambda': 4.153703662804666, 'min_split_gain': 0.014850878494055126}. Best is trial 0 with value: 0.00016171944165890083.
[Trial 0] New best: 0.00016
[I 2025-06-15 20:01:50,576] Trial 1 finished with value: 0.0669124941049529 and parameters: {'n_estimators': 88, 'learning_rate': 0.015995911271693787, 'num_leaves': 97, 'max_depth': 9, 'min_child_samples': 24, 'subsample': 0.8973685713792268, 'colsample_bytree': 0.9246885483588696, 'reg_alpha': 7.888055457517384, 'reg_lambda': 1.270701258002439, 'min_split_gain': 0.048476500554885935}. Best is trial 0 with value: 0.00016171944165890083.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:01:50,914] Trial 2 finished with value: 0.0704447983445367 and parameters: {'n_estimators': 102, 'learning_rate': 0.013673568010580063, 'num_leaves': 37, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.8969035831224963, 'colsample_bytree': 0.6556300979943821, 'reg_alpha': 9.181033375236042, 'reg_lambda': 9.047776793195151, 'min_split_gain': 0.033488149822580034}. Best is trial 0 with value: 0.00016171944165890083.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:01:51,685] Trial 3 finished with value: 6.488605411010752e-05 and parameters: {'n_estimators': 82, 'learning_rate': 0.1088340809225434, 'num_leaves': 104, 'max_depth': 14, 'min_child_samples': 5, 'subsample': 0.8991420793976871, 'colsample_bytree': 0.7454285008873451, 'reg_alpha': 8.118270223940263, 'reg_lambda': 1.7749373584034356, 'min_split_gain': 0.00200083184193211}. Best is trial 3 with value: 6.488605411010752e-05.
[I 2025-06-15 20:01:51,687] A new study created in memory with name: no-name-6ba7f01a-d59e-411b-99f9-5f270027bf2d
[I 2025-06-15 20:01:51,770] Trial 0 finished with value: 0.000266127998790499 and parameters: {'n_estimators': 296, 'learning_rate': 0.0303659303607225, 'num_leaves': 45, 'max_depth': 14, 'min_child_samples': 16, 'subsample': 0.8447344090766007, 'colsample_bytree': 0.5473166739170574, 'reg_alpha': 0.6723112377617357, 'reg_lambda': 4.237218936439852, 'min_split_gain': 0.003547821626785297}. Best is trial 0 with value: 0.000266127998790499.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         543
14      pct_chg_Load_DA         444
1           Price_lag_2         440
3         Load_DA_lag_0         327
5       WindOn_DA_lag_0         322
2           Price_lag_7         252
13            EUA_lag_2         245
11           NGas_lag_2         229
16  roll2d_median_Price         229
15       lag168_Load_DA         224
10           Coal_lag_2         209
12            Oil_lag_2         183
9                  WD_7          20
8                  WD_6          12
7                  WD_1           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          11
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 37.48% done
********************* END NS.. 273 ****************************************************
********************* START NS ... 274 ****************************************************
[I 2025-06-15 20:01:54,162] A new study created in memory with name: no-name-71fa7e3f-aba0-469d-82a2-d4c6d442f32d
[I 2025-06-15 20:01:55,426] Trial 0 finished with value: 9.02536364023496e-05 and parameters: {'n_estimators': 228, 'learning_rate': 0.0377673163117565, 'num_leaves': 146, 'max_depth': 15, 'min_child_samples': 9, 'subsample': 0.6499206852589678, 'colsample_bytree': 0.6268922293124306, 'reg_alpha': 9.309487463887384, 'reg_lambda': 8.216079019120405, 'min_split_gain': 0.01817596822865147}. Best is trial 0 with value: 9.02536364023496e-05.
[Trial 0] New best: 0.00009
[I 2025-06-15 20:01:56,463] Trial 1 finished with value: 0.0055129482176536305 and parameters: {'n_estimators': 188, 'learning_rate': 0.016491538225700555, 'num_leaves': 243, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.6649105940366487, 'colsample_bytree': 0.7501380546125975, 'reg_alpha': 9.822023144849808, 'reg_lambda': 8.39264843635735, 'min_split_gain': 0.04797432889719111}. Best is trial 0 with value: 9.02536364023496e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:01:57,992] Trial 2 finished with value: 4.8203583459211886e-05 and parameters: {'n_estimators': 253, 'learning_rate': 0.048751487463936524, 'num_leaves': 93, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.7537987689648245, 'colsample_bytree': 0.5660067314038231, 'reg_alpha': 8.489593147254594, 'reg_lambda': 1.9018406170584856, 'min_split_gain': 0.017348979128164656}. Best is trial 2 with value: 4.8203583459211886e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:01:59,071] Trial 3 finished with value: 0.0016914471055467907 and parameters: {'n_estimators': 190, 'learning_rate': 0.022866859144246236, 'num_leaves': 230, 'max_depth': 12, 'min_child_samples': 28, 'subsample': 0.815519300855353, 'colsample_bytree': 0.9114722824397761, 'reg_alpha': 9.242873544275225, 'reg_lambda': 3.6945891121380834, 'min_split_gain': 0.047181877516881104}. Best is trial 2 with value: 4.8203583459211886e-05.
[I 2025-06-15 20:01:59,073] A new study created in memory with name: no-name-c1962346-2d86-47e8-a1f0-7d6e40bd544b
[I 2025-06-15 20:01:59,131] Trial 0 finished with value: 0.00010460013638780262 and parameters: {'n_estimators': 282, 'learning_rate': 0.03900829850105665, 'num_leaves': 195, 'max_depth': 6, 'min_child_samples': 21, 'subsample': 0.7583349169651934, 'colsample_bytree': 0.7262952377129659, 'reg_alpha': 8.530785179371748, 'reg_lambda': 0.43401563315571434, 'min_split_gain': 0.032424473008144}. Best is trial 0 with value: 0.00010460013638780262.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1016
1           Price_lag_2         868
14      pct_chg_Load_DA         597
2           Price_lag_7         480
3         Load_DA_lag_0         455
11           NGas_lag_2         374
16  roll2d_median_Price         340
15       lag168_Load_DA         333
5       WindOn_DA_lag_0         311
10           Coal_lag_2         269
13            EUA_lag_2         241
12            Oil_lag_2         151
9                  WD_7          40
7                  WD_1          31
8                  WD_6          31
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 37.62% done
********************* END NS.. 274 ****************************************************
********************* START NS ... 275 ****************************************************
[I 2025-06-15 20:02:01,679] A new study created in memory with name: no-name-2748697d-5468-40f3-9df7-8911ba25add2
[I 2025-06-15 20:02:02,897] Trial 0 finished with value: 8.92187377936453e-05 and parameters: {'n_estimators': 78, 'learning_rate': 0.17999980699261833, 'num_leaves': 206, 'max_depth': 11, 'min_child_samples': 1, 'subsample': 0.9268365403519679, 'colsample_bytree': 0.7827391278622282, 'reg_alpha': 5.168011805292068, 'reg_lambda': 1.8401692977783457, 'min_split_gain': 0.0006071528376654479}. Best is trial 0 with value: 8.92187377936453e-05.
[Trial 0] New best: 0.00009
[I 2025-06-15 20:02:03,403] Trial 1 finished with value: 0.00047567376601421926 and parameters: {'n_estimators': 270, 'learning_rate': 0.210734907851809, 'num_leaves': 226, 'max_depth': 4, 'min_child_samples': 8, 'subsample': 0.6082385319150738, 'colsample_bytree': 0.5867678889528344, 'reg_alpha': 1.68364759899166, 'reg_lambda': 3.0881414311279167, 'min_split_gain': 0.007930586494261238}. Best is trial 0 with value: 8.92187377936453e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:02:03,842] Trial 2 finished with value: 0.0001919532321052156 and parameters: {'n_estimators': 280, 'learning_rate': 0.22590839274237753, 'num_leaves': 139, 'max_depth': 16, 'min_child_samples': 25, 'subsample': 0.9742180517867668, 'colsample_bytree': 0.9973464919703303, 'reg_alpha': 8.352121416310624, 'reg_lambda': 5.326494761261792, 'min_split_gain': 0.008742366887493335}. Best is trial 0 with value: 8.92187377936453e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:02:04,094] Trial 3 finished with value: 0.00018892776248040873 and parameters: {'n_estimators': 58, 'learning_rate': 0.21912100631267675, 'num_leaves': 149, 'max_depth': 14, 'min_child_samples': 11, 'subsample': 0.5611546114377168, 'colsample_bytree': 0.9827792896773747, 'reg_alpha': 9.927745468876848, 'reg_lambda': 2.571228074294444, 'min_split_gain': 0.024081406909871983}. Best is trial 0 with value: 8.92187377936453e-05.
[I 2025-06-15 20:02:04,097] A new study created in memory with name: no-name-500d21ba-e20d-4b2c-8f22-10302dca7ce2
[I 2025-06-15 20:02:04,150] Trial 0 finished with value: 0.0009411633372762568 and parameters: {'n_estimators': 226, 'learning_rate': 0.04868349061957645, 'num_leaves': 171, 'max_depth': 5, 'min_child_samples': 10, 'subsample': 0.9494140495859911, 'colsample_bytree': 0.8614118735524627, 'reg_alpha': 9.336908062064278, 'reg_lambda': 3.554152917894643, 'min_split_gain': 0.01296527237975556}. Best is trial 0 with value: 0.0009411633372762568.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         691
1           Price_lag_2         652
14      pct_chg_Load_DA         566
2           Price_lag_7         536
3         Load_DA_lag_0         494
5       WindOn_DA_lag_0         474
15       lag168_Load_DA         472
13            EUA_lag_2         382
16  roll2d_median_Price         347
11           NGas_lag_2         343
10           Coal_lag_2         334
12            Oil_lag_2         258
9                  WD_7          26
7                  WD_1          10
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 37.76% done
********************* END NS.. 275 ****************************************************
********************* START NS ... 276 ****************************************************
[I 2025-06-15 20:02:06,629] A new study created in memory with name: no-name-7f7dd635-1d5b-4540-b240-520389fc46fa
[I 2025-06-15 20:02:09,359] Trial 0 finished with value: 0.001064133039919825 and parameters: {'n_estimators': 183, 'learning_rate': 0.031615276080099765, 'num_leaves': 115, 'max_depth': 16, 'min_child_samples': 25, 'subsample': 0.6307242614969361, 'colsample_bytree': 0.8224651865195147, 'reg_alpha': 0.5165157964161882, 'reg_lambda': 0.44189417222175953, 'min_split_gain': 0.041465872201325554}. Best is trial 0 with value: 0.001064133039919825.
[Trial 0] New best: 0.00106
[I 2025-06-15 20:02:10,302] Trial 1 finished with value: 0.00040743696986622266 and parameters: {'n_estimators': 218, 'learning_rate': 0.04344962822214887, 'num_leaves': 239, 'max_depth': 5, 'min_child_samples': 12, 'subsample': 0.8181505804674634, 'colsample_bytree': 0.8030206296539055, 'reg_alpha': 0.08988089607333327, 'reg_lambda': 7.923169147997044, 'min_split_gain': 0.03650263020702101}. Best is trial 1 with value: 0.00040743696986622266.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:02:11,925] Trial 2 finished with value: 0.0006965799815523388 and parameters: {'n_estimators': 266, 'learning_rate': 0.019719074021395316, 'num_leaves': 208, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.7099037398149806, 'colsample_bytree': 0.9797585853852872, 'reg_alpha': 5.55847475792463, 'reg_lambda': 6.99690454365459, 'min_split_gain': 0.0147943100999604}. Best is trial 1 with value: 0.00040743696986622266.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:02:12,849] Trial 3 finished with value: 0.0003676930943607355 and parameters: {'n_estimators': 167, 'learning_rate': 0.24514257757749325, 'num_leaves': 157, 'max_depth': 16, 'min_child_samples': 1, 'subsample': 0.5365767956480444, 'colsample_bytree': 0.5153599675188528, 'reg_alpha': 7.88642054577486, 'reg_lambda': 1.4448637758971061, 'min_split_gain': 0.0033239715885601097}. Best is trial 3 with value: 0.0003676930943607355.
[I 2025-06-15 20:02:12,851] A new study created in memory with name: no-name-b1134c8c-c6c4-4cd2-8edb-d086d83106b9
[I 2025-06-15 20:02:12,915] Trial 0 finished with value: 0.001447965783947911 and parameters: {'n_estimators': 249, 'learning_rate': 0.029372232946036473, 'num_leaves': 129, 'max_depth': 10, 'min_child_samples': 5, 'subsample': 0.7426807128157957, 'colsample_bytree': 0.6629856050744505, 'reg_alpha': 8.117306502374639, 'reg_lambda': 2.911629886573036, 'min_split_gain': 0.024426227126610495}. Best is trial 0 with value: 0.001447965783947911.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         439
0           Price_lag_1         435
16  roll2d_median_Price         297
3         Load_DA_lag_0         243
2           Price_lag_7         230
14      pct_chg_Load_DA         211
13            EUA_lag_2         201
5       WindOn_DA_lag_0         188
11           NGas_lag_2         176
12            Oil_lag_2         171
15       lag168_Load_DA         167
10           Coal_lag_2         163
9                  WD_7          25
8                  WD_6          11
7                  WD_1          10
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 37.89% done
********************* END NS.. 276 ****************************************************
********************* START NS ... 277 ****************************************************
[I 2025-06-15 20:02:15,060] A new study created in memory with name: no-name-9b0bacef-566b-4ae1-a4d3-00efd7266f92
[I 2025-06-15 20:02:15,780] Trial 0 finished with value: 0.042523084074667004 and parameters: {'n_estimators': 89, 'learning_rate': 0.01788231178178295, 'num_leaves': 254, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.9028675586763891, 'colsample_bytree': 0.5798462825726153, 'reg_alpha': 4.686701049874711, 'reg_lambda': 2.855257006280251, 'min_split_gain': 0.0372421386328284}. Best is trial 0 with value: 0.042523084074667004.
[Trial 0] New best: 0.04252
[I 2025-06-15 20:02:17,207] Trial 1 finished with value: 0.0007826409216447884 and parameters: {'n_estimators': 253, 'learning_rate': 0.016592168723958738, 'num_leaves': 73, 'max_depth': 6, 'min_child_samples': 29, 'subsample': 0.6604246511534494, 'colsample_bytree': 0.5471700713252214, 'reg_alpha': 8.716007847757748, 'reg_lambda': 1.1163334281480985, 'min_split_gain': 0.009439897628997208}. Best is trial 1 with value: 0.0007826409216447884.
[Trial 1] New best: 0.00078
[I 2025-06-15 20:02:19,285] Trial 2 finished with value: 0.00031464703507170654 and parameters: {'n_estimators': 291, 'learning_rate': 0.038005508922088274, 'num_leaves': 143, 'max_depth': 10, 'min_child_samples': 4, 'subsample': 0.6921426111604073, 'colsample_bytree': 0.5958851214621818, 'reg_alpha': 3.6534425531893944, 'reg_lambda': 9.08467226462736, 'min_split_gain': 0.04746413227700055}. Best is trial 2 with value: 0.00031464703507170654.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:02:19,703] Trial 3 finished with value: 0.0003345636794997483 and parameters: {'n_estimators': 150, 'learning_rate': 0.19686773378149552, 'num_leaves': 144, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.7543621399095205, 'colsample_bytree': 0.8719518126935022, 'reg_alpha': 2.5603850892909588, 'reg_lambda': 5.302777510251935, 'min_split_gain': 0.04440909498181651}. Best is trial 2 with value: 0.00031464703507170654.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:02:20,450] Trial 4 finished with value: 0.015251070281229994 and parameters: {'n_estimators': 115, 'learning_rate': 0.018571068490619746, 'num_leaves': 145, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.8786025675256162, 'colsample_bytree': 0.7462396128557212, 'reg_alpha': 6.747705734109352, 'reg_lambda': 2.1689890845654625, 'min_split_gain': 0.003808321728253461}. Best is trial 2 with value: 0.00031464703507170654.
[I 2025-06-15 20:02:20,452] A new study created in memory with name: no-name-9ff19d10-9cc0-415e-a9c8-083aeba070c2
[I 2025-06-15 20:02:20,492] Trial 0 finished with value: 0.0005909118413646432 and parameters: {'n_estimators': 167, 'learning_rate': 0.07174710105531869, 'num_leaves': 155, 'max_depth': 7, 'min_child_samples': 17, 'subsample': 0.8788815074877249, 'colsample_bytree': 0.6454326171408653, 'reg_alpha': 5.992121519702479, 'reg_lambda': 3.74210642259409, 'min_split_gain': 0.021556715857189754}. Best is trial 0 with value: 0.0005909118413646432.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1373
1           Price_lag_2        1221
14      pct_chg_Load_DA         855
3         Load_DA_lag_0         709
2           Price_lag_7         543
15       lag168_Load_DA         514
11           NGas_lag_2         500
16  roll2d_median_Price         468
10           Coal_lag_2         396
5       WindOn_DA_lag_0         362
13            EUA_lag_2         279
12            Oil_lag_2         219
9                  WD_7          51
8                  WD_6          23
7                  WD_1          20
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 38.03% done
********************* END NS.. 277 ****************************************************
********************* START NS ... 278 ****************************************************
[I 2025-06-15 20:02:23,680] A new study created in memory with name: no-name-4ba52d80-d6ae-44cd-b003-c494a8a7927d
[I 2025-06-15 20:02:25,608] Trial 0 finished with value: 9.261484182762206e-05 and parameters: {'n_estimators': 160, 'learning_rate': 0.0479599664582798, 'num_leaves': 68, 'max_depth': 11, 'min_child_samples': 12, 'subsample': 0.5306076728133521, 'colsample_bytree': 0.9916033255369194, 'reg_alpha': 0.27656737645431195, 'reg_lambda': 7.182395196540781, 'min_split_gain': 0.03765813284395545}. Best is trial 0 with value: 9.261484182762206e-05.
[Trial 0] New best: 0.00009
[I 2025-06-15 20:02:26,237] Trial 1 finished with value: 8.534501107437552e-05 and parameters: {'n_estimators': 200, 'learning_rate': 0.23627868287843243, 'num_leaves': 58, 'max_depth': 16, 'min_child_samples': 11, 'subsample': 0.6356364846757474, 'colsample_bytree': 0.8872378610749789, 'reg_alpha': 0.5378388325834293, 'reg_lambda': 6.9922204083495645, 'min_split_gain': 0.013438739022623837}. Best is trial 1 with value: 8.534501107437552e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:02:26,809] Trial 2 finished with value: 0.00027968287426207935 and parameters: {'n_estimators': 74, 'learning_rate': 0.06067709917226168, 'num_leaves': 46, 'max_depth': 7, 'min_child_samples': 14, 'subsample': 0.7754341000694427, 'colsample_bytree': 0.6394500863192617, 'reg_alpha': 6.909710065848186, 'reg_lambda': 4.242447110604822, 'min_split_gain': 0.008126964884447524}. Best is trial 1 with value: 8.534501107437552e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:02:27,567] Trial 3 finished with value: 7.313316593914278e-05 and parameters: {'n_estimators': 163, 'learning_rate': 0.10313445127798689, 'num_leaves': 49, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.802671577928455, 'colsample_bytree': 0.9155478238229331, 'reg_alpha': 7.439553421086441, 'reg_lambda': 7.82993309393447, 'min_split_gain': 0.006544089855919405}. Best is trial 3 with value: 7.313316593914278e-05.
[I 2025-06-15 20:02:27,570] A new study created in memory with name: no-name-598d64a1-dc90-4b34-862f-bfa3de1582b4
[I 2025-06-15 20:02:27,626] Trial 0 finished with value: 0.0008431341637827392 and parameters: {'n_estimators': 183, 'learning_rate': 0.14131141667057165, 'num_leaves': 111, 'max_depth': 6, 'min_child_samples': 28, 'subsample': 0.6164279799721955, 'colsample_bytree': 0.5825521821114416, 'reg_alpha': 0.4258610428056042, 'reg_lambda': 8.325574544663915, 'min_split_gain': 0.0230898484704194}. Best is trial 0 with value: 0.0008431341637827392.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         597
14      pct_chg_Load_DA         430
1           Price_lag_2         361
3         Load_DA_lag_0         301
5       WindOn_DA_lag_0         248
11           NGas_lag_2         226
2           Price_lag_7         225
16  roll2d_median_Price         202
13            EUA_lag_2         176
15       lag168_Load_DA         165
10           Coal_lag_2         161
12            Oil_lag_2         128
9                  WD_7          24
7                  WD_1          13
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 38.17% done
********************* END NS.. 278 ****************************************************
********************* START NS ... 279 ****************************************************
[I 2025-06-15 20:02:30,017] A new study created in memory with name: no-name-7d42590c-deb6-432e-9dab-4621c92c2761
[I 2025-06-15 20:02:30,597] Trial 0 finished with value: 0.002952320695881939 and parameters: {'n_estimators': 199, 'learning_rate': 0.01505587745702832, 'num_leaves': 175, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.856657364670532, 'colsample_bytree': 0.58019837645111, 'reg_alpha': 0.38838690700614964, 'reg_lambda': 3.4858310213000987, 'min_split_gain': 0.029551685181961624}. Best is trial 0 with value: 0.002952320695881939.
[Trial 0] New best: 0.00295
[I 2025-06-15 20:02:32,281] Trial 1 finished with value: 0.0001255078995725521 and parameters: {'n_estimators': 224, 'learning_rate': 0.03966812721925403, 'num_leaves': 45, 'max_depth': 10, 'min_child_samples': 29, 'subsample': 0.5106039868069614, 'colsample_bytree': 0.5352303933695288, 'reg_alpha': 2.398872106017227, 'reg_lambda': 7.011233576110028, 'min_split_gain': 0.021471634002020748}. Best is trial 1 with value: 0.0001255078995725521.
[Trial 1] New best: 0.00013
[I 2025-06-15 20:02:33,822] Trial 2 finished with value: 0.00015688987573889723 and parameters: {'n_estimators': 211, 'learning_rate': 0.0595226664975814, 'num_leaves': 65, 'max_depth': 8, 'min_child_samples': 18, 'subsample': 0.6895687770386422, 'colsample_bytree': 0.542762458678576, 'reg_alpha': 1.073937256202696, 'reg_lambda': 8.548319239131732, 'min_split_gain': 0.035447653811939636}. Best is trial 1 with value: 0.0001255078995725521.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:02:34,897] Trial 3 finished with value: 0.00010695064575272558 and parameters: {'n_estimators': 94, 'learning_rate': 0.0977233626384963, 'num_leaves': 244, 'max_depth': 16, 'min_child_samples': 5, 'subsample': 0.6966562052089234, 'colsample_bytree': 0.9277147706649418, 'reg_alpha': 1.0990435708916102, 'reg_lambda': 8.344524762383154, 'min_split_gain': 0.04443765319927492}. Best is trial 3 with value: 0.00010695064575272558.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:02:35,815] Trial 4 finished with value: 0.0001516549672462879 and parameters: {'n_estimators': 107, 'learning_rate': 0.05723460634625641, 'num_leaves': 90, 'max_depth': 10, 'min_child_samples': 19, 'subsample': 0.7717245290917296, 'colsample_bytree': 0.5007259645692115, 'reg_alpha': 9.53176845909455, 'reg_lambda': 0.9608776262764585, 'min_split_gain': 0.002752304519239868}. Best is trial 3 with value: 0.00010695064575272558.
[I 2025-06-15 20:02:35,818] A new study created in memory with name: no-name-929c9cc1-5d11-4d54-a52d-b765f900bb43
[I 2025-06-15 20:02:35,845] Trial 0 finished with value: 0.0011395107403951799 and parameters: {'n_estimators': 86, 'learning_rate': 0.022191934083683886, 'num_leaves': 212, 'max_depth': 10, 'min_child_samples': 12, 'subsample': 0.7958926666149082, 'colsample_bytree': 0.8882957206974603, 'reg_alpha': 6.177998424582629, 'reg_lambda': 4.661550376732028, 'min_split_gain': 0.028440400548940904}. Best is trial 0 with value: 0.0011395107403951799.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         761
14      pct_chg_Load_DA         723
1           Price_lag_2         641
3         Load_DA_lag_0         546
2           Price_lag_7         343
11           NGas_lag_2         339
16  roll2d_median_Price         321
15       lag168_Load_DA         301
10           Coal_lag_2         182
5       WindOn_DA_lag_0         178
12            Oil_lag_2         162
13            EUA_lag_2         106
9                  WD_7          18
7                  WD_1          12
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 38.30% done
********************* END NS.. 279 ****************************************************
********************* START NS ... 280 ****************************************************
[I 2025-06-15 20:02:38,424] A new study created in memory with name: no-name-d3c63a53-1977-4a60-9d25-8ad0c422ea7a
[I 2025-06-15 20:02:39,267] Trial 0 finished with value: 9.763335553232402e-05 and parameters: {'n_estimators': 205, 'learning_rate': 0.08560305845473008, 'num_leaves': 60, 'max_depth': 14, 'min_child_samples': 23, 'subsample': 0.6839515610561251, 'colsample_bytree': 0.6356715344895625, 'reg_alpha': 2.458232180362012, 'reg_lambda': 8.556602414939022, 'min_split_gain': 0.04538939684092999}. Best is trial 0 with value: 9.763335553232402e-05.
[Trial 0] New best: 0.00010
[I 2025-06-15 20:02:39,610] Trial 1 finished with value: 0.00011217539886016609 and parameters: {'n_estimators': 225, 'learning_rate': 0.22029621682609, 'num_leaves': 198, 'max_depth': 14, 'min_child_samples': 15, 'subsample': 0.6726431931233943, 'colsample_bytree': 0.6742034419561079, 'reg_alpha': 5.913094487366686, 'reg_lambda': 6.392568140002722, 'min_split_gain': 0.038787226134451354}. Best is trial 0 with value: 9.763335553232402e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:02:41,618] Trial 2 finished with value: 0.00011374971565365423 and parameters: {'n_estimators': 267, 'learning_rate': 0.0263459268563046, 'num_leaves': 133, 'max_depth': 10, 'min_child_samples': 27, 'subsample': 0.8745151781873622, 'colsample_bytree': 0.571387782669966, 'reg_alpha': 7.539754142978178, 'reg_lambda': 6.901987568150511, 'min_split_gain': 0.016043188589176062}. Best is trial 0 with value: 9.763335553232402e-05.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:02:44,375] Trial 3 finished with value: 8.392788096776735e-05 and parameters: {'n_estimators': 273, 'learning_rate': 0.03127313568638755, 'num_leaves': 100, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.6902402861229271, 'colsample_bytree': 0.786129715342452, 'reg_alpha': 6.418975553494723, 'reg_lambda': 8.353056444764459, 'min_split_gain': 0.005379400040723176}. Best is trial 3 with value: 8.392788096776735e-05.
[I 2025-06-15 20:02:44,377] A new study created in memory with name: no-name-43593755-eed3-4c3f-819b-6e23abae4e98
[I 2025-06-15 20:02:44,425] Trial 0 finished with value: 0.0005569066835520402 and parameters: {'n_estimators': 153, 'learning_rate': 0.014565750418833514, 'num_leaves': 115, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.8146676830911781, 'colsample_bytree': 0.5393938768646456, 'reg_alpha': 1.3740822501199446, 'reg_lambda': 7.698882209733235, 'min_split_gain': 0.03006651890488368}. Best is trial 0 with value: 0.0005569066835520402.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2102
1           Price_lag_2        1719
14      pct_chg_Load_DA        1492
3         Load_DA_lag_0        1078
5       WindOn_DA_lag_0         933
2           Price_lag_7         816
15       lag168_Load_DA         773
13            EUA_lag_2         748
16  roll2d_median_Price         733
11           NGas_lag_2         714
10           Coal_lag_2         683
12            Oil_lag_2         538
9                  WD_7          63
8                  WD_6          42
7                  WD_1          23
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 38.44% done
********************* END NS.. 280 ****************************************************
********************* START NS ... 281 ****************************************************
[I 2025-06-15 20:02:48,335] A new study created in memory with name: no-name-4e7b0ffa-b7e9-474a-826c-2510fbf22a9c
[I 2025-06-15 20:02:50,104] Trial 0 finished with value: 0.0007671943035469772 and parameters: {'n_estimators': 223, 'learning_rate': 0.017936032383767505, 'num_leaves': 40, 'max_depth': 9, 'min_child_samples': 1, 'subsample': 0.6841726951572981, 'colsample_bytree': 0.5734862161783264, 'reg_alpha': 9.678903881531035, 'reg_lambda': 1.6244988993180098, 'min_split_gain': 0.027597783814336543}. Best is trial 0 with value: 0.0007671943035469772.
[Trial 0] New best: 0.00077
[I 2025-06-15 20:02:50,908] Trial 1 finished with value: 0.0001415633994094363 and parameters: {'n_estimators': 273, 'learning_rate': 0.07566347815008685, 'num_leaves': 150, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.6856624901580815, 'colsample_bytree': 0.7521032380881971, 'reg_alpha': 7.948082149435823, 'reg_lambda': 6.915032791507962, 'min_split_gain': 0.011744168304869696}. Best is trial 1 with value: 0.0001415633994094363.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:02:52,823] Trial 2 finished with value: 0.0001280144297505748 and parameters: {'n_estimators': 269, 'learning_rate': 0.020636657371386393, 'num_leaves': 206, 'max_depth': 11, 'min_child_samples': 30, 'subsample': 0.6908058888624363, 'colsample_bytree': 0.7701691783819893, 'reg_alpha': 5.029642369674918, 'reg_lambda': 1.7219677181009474, 'min_split_gain': 0.03376616164709139}. Best is trial 2 with value: 0.0001280144297505748.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:02:53,478] Trial 3 finished with value: 0.00012655734150041084 and parameters: {'n_estimators': 199, 'learning_rate': 0.14332192368987814, 'num_leaves': 125, 'max_depth': 16, 'min_child_samples': 17, 'subsample': 0.8499824391093838, 'colsample_bytree': 0.8119689438274386, 'reg_alpha': 7.686527061908073, 'reg_lambda': 7.93712178361452, 'min_split_gain': 0.006479681630642504}. Best is trial 3 with value: 0.00012655734150041084.
[I 2025-06-15 20:02:53,481] A new study created in memory with name: no-name-7fc84c8b-c9a3-4525-9aa2-ecf64f23d6f2
[I 2025-06-15 20:02:53,517] Trial 0 finished with value: 0.0008067246781335479 and parameters: {'n_estimators': 136, 'learning_rate': 0.27436203528822123, 'num_leaves': 205, 'max_depth': 16, 'min_child_samples': 2, 'subsample': 0.8178345732521759, 'colsample_bytree': 0.7653286567402888, 'reg_alpha': 6.168135797858481, 'reg_lambda': 7.699092108384944, 'min_split_gain': 0.044505888126153234}. Best is trial 0 with value: 0.0008067246781335479.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         416
1           Price_lag_2         310
14      pct_chg_Load_DA         262
3         Load_DA_lag_0         184
13            EUA_lag_2         160
5       WindOn_DA_lag_0         153
11           NGas_lag_2         141
16  roll2d_median_Price         126
2           Price_lag_7         122
15       lag168_Load_DA         121
10           Coal_lag_2          90
12            Oil_lag_2          75
9                  WD_7          16
7                  WD_1           7
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 38.58% done
********************* END NS.. 281 ****************************************************
********************* START NS ... 282 ****************************************************
[I 2025-06-15 20:02:55,541] A new study created in memory with name: no-name-e7de91e3-0439-45b5-beea-1b41bb80a3d1
[I 2025-06-15 20:02:57,864] Trial 0 finished with value: 0.012424366738459258 and parameters: {'n_estimators': 148, 'learning_rate': 0.05753886385764622, 'num_leaves': 253, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.5233208270959147, 'colsample_bytree': 0.5034529267595236, 'reg_alpha': 3.5671572066777646, 'reg_lambda': 2.6391015273389518, 'min_split_gain': 0.005376959453621728}. Best is trial 0 with value: 0.012424366738459258.
[Trial 0] New best: 0.01242
[I 2025-06-15 20:02:58,608] Trial 1 finished with value: 0.01328164609949733 and parameters: {'n_estimators': 196, 'learning_rate': 0.07693220666405454, 'num_leaves': 214, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.5617513395947739, 'colsample_bytree': 0.5042155648719444, 'reg_alpha': 6.9867912215561265, 'reg_lambda': 4.89012179391239, 'min_split_gain': 0.03998768483155904}. Best is trial 0 with value: 0.012424366738459258.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:02:59,762] Trial 2 finished with value: 0.01178710380767782 and parameters: {'n_estimators': 158, 'learning_rate': 0.041103111597742194, 'num_leaves': 53, 'max_depth': 12, 'min_child_samples': 26, 'subsample': 0.518346293843037, 'colsample_bytree': 0.8961246388519005, 'reg_alpha': 6.651056928405411, 'reg_lambda': 0.4987200468734565, 'min_split_gain': 0.017364812665999176}. Best is trial 2 with value: 0.01178710380767782.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:03:00,519] Trial 3 finished with value: 0.013429193859539923 and parameters: {'n_estimators': 275, 'learning_rate': 0.1823310542359933, 'num_leaves': 224, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.733267056278857, 'colsample_bytree': 0.5266363227952526, 'reg_alpha': 5.42807153028698, 'reg_lambda': 0.3562294028793933, 'min_split_gain': 0.011552865388853174}. Best is trial 2 with value: 0.01178710380767782.
[I 2025-06-15 20:03:00,522] A new study created in memory with name: no-name-8d96d9a2-938c-4be8-bf97-181f86a4b0f9
[I 2025-06-15 20:03:00,599] Trial 0 finished with value: 0.15111403220356534 and parameters: {'n_estimators': 276, 'learning_rate': 0.14795740778058625, 'num_leaves': 46, 'max_depth': 8, 'min_child_samples': 20, 'subsample': 0.7975683065583402, 'colsample_bytree': 0.6066027400696686, 'reg_alpha': 1.6250965721843669, 'reg_lambda': 7.982521034044392, 'min_split_gain': 0.025019904036993987}. Best is trial 0 with value: 0.15111403220356534.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1247
1           Price_lag_2         746
14      pct_chg_Load_DA         675
3         Load_DA_lag_0         402
11           NGas_lag_2         345
2           Price_lag_7         304
16  roll2d_median_Price         242
10           Coal_lag_2         208
5       WindOn_DA_lag_0         199
13            EUA_lag_2         173
12            Oil_lag_2         161
15       lag168_Load_DA         154
9                  WD_7          19
7                  WD_1           7
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
3         Load_DA_lag_0           7
0           Price_lag_1           5
10           Coal_lag_2           5
1           Price_lag_2           3
11           NGas_lag_2           2
13            EUA_lag_2           1
7                  WD_1           1
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 38.71% done
********************* END NS.. 282 ****************************************************
********************* START NS ... 283 ****************************************************
[I 2025-06-15 20:03:03,284] A new study created in memory with name: no-name-7769e558-d6b0-47b9-a215-604007bdfa56
[I 2025-06-15 20:03:03,635] Trial 0 finished with value: 0.016478356498932215 and parameters: {'n_estimators': 161, 'learning_rate': 0.21742860659127144, 'num_leaves': 168, 'max_depth': 5, 'min_child_samples': 18, 'subsample': 0.6633679476099537, 'colsample_bytree': 0.5922682909345173, 'reg_alpha': 9.447084430392614, 'reg_lambda': 1.8905351161230388, 'min_split_gain': 0.04159980279867469}. Best is trial 0 with value: 0.016478356498932215.
[Trial 0] New best: 0.01648
[I 2025-06-15 20:03:05,722] Trial 1 finished with value: 0.016019411526471895 and parameters: {'n_estimators': 135, 'learning_rate': 0.04899335516380037, 'num_leaves': 124, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.5822724729311163, 'colsample_bytree': 0.6664181394235168, 'reg_alpha': 1.5080287630228073, 'reg_lambda': 3.0410391855787577, 'min_split_gain': 0.009268140513931355}. Best is trial 1 with value: 0.016019411526471895.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:03:06,515] Trial 2 finished with value: 0.019131698799739527 and parameters: {'n_estimators': 202, 'learning_rate': 0.061825280411207835, 'num_leaves': 188, 'max_depth': 16, 'min_child_samples': 25, 'subsample': 0.8216861759774134, 'colsample_bytree': 0.5077749875867683, 'reg_alpha': 9.958656445575762, 'reg_lambda': 0.8911943603821759, 'min_split_gain': 0.04409977963098152}. Best is trial 1 with value: 0.016019411526471895.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:03:07,039] Trial 3 finished with value: 0.015553843537422707 and parameters: {'n_estimators': 240, 'learning_rate': 0.159057983824458, 'num_leaves': 101, 'max_depth': 11, 'min_child_samples': 25, 'subsample': 0.9684795521647027, 'colsample_bytree': 0.660748679867121, 'reg_alpha': 6.662874375591662, 'reg_lambda': 3.825738633756526, 'min_split_gain': 0.011350878831319673}. Best is trial 3 with value: 0.015553843537422707.
[I 2025-06-15 20:03:07,041] A new study created in memory with name: no-name-385c4a7e-37bc-4289-b10f-4865b1d222ee
[I 2025-06-15 20:03:07,084] Trial 0 finished with value: 0.24640028921369742 and parameters: {'n_estimators': 160, 'learning_rate': 0.015588830516585451, 'num_leaves': 89, 'max_depth': 6, 'min_child_samples': 23, 'subsample': 0.6146673683480746, 'colsample_bytree': 0.7281491421957467, 'reg_alpha': 1.2048814972412947, 'reg_lambda': 6.423174309634392, 'min_split_gain': 0.03906839667634203}. Best is trial 0 with value: 0.24640028921369742.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         369
1           Price_lag_2         229
14      pct_chg_Load_DA         191
3         Load_DA_lag_0         152
13            EUA_lag_2         112
2           Price_lag_7         104
11           NGas_lag_2         100
10           Coal_lag_2          97
16  roll2d_median_Price          94
15       lag168_Load_DA          88
5       WindOn_DA_lag_0          87
12            Oil_lag_2          54
9                  WD_7           8
7                  WD_1           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         152
3         Load_DA_lag_0          26
10           Coal_lag_2          14
11           NGas_lag_2           6
1           Price_lag_2           2
13            EUA_lag_2           1
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 38.85% done
********************* END NS.. 283 ****************************************************
********************* START NS ... 284 ****************************************************
[I 2025-06-15 20:03:09,558] A new study created in memory with name: no-name-6c7c497a-928d-45bb-91ec-0f59cff9759f
[I 2025-06-15 20:03:10,332] Trial 0 finished with value: 0.018195507816707167 and parameters: {'n_estimators': 110, 'learning_rate': 0.018423648279649162, 'num_leaves': 36, 'max_depth': 11, 'min_child_samples': 2, 'subsample': 0.9683059320480479, 'colsample_bytree': 0.8625449751017045, 'reg_alpha': 7.26856353312225, 'reg_lambda': 4.960054922145622, 'min_split_gain': 0.031222857111583076}. Best is trial 0 with value: 0.018195507816707167.
[Trial 0] New best: 0.01820
[I 2025-06-15 20:03:11,069] Trial 1 finished with value: 0.00985031071912349 and parameters: {'n_estimators': 83, 'learning_rate': 0.037684381154261565, 'num_leaves': 94, 'max_depth': 7, 'min_child_samples': 4, 'subsample': 0.7353300027200573, 'colsample_bytree': 0.5510554848204339, 'reg_alpha': 4.521578114119799, 'reg_lambda': 2.491759093900993, 'min_split_gain': 0.0480575072978007}. Best is trial 1 with value: 0.00985031071912349.
[Trial 1] New best: 0.00985
[I 2025-06-15 20:03:11,532] Trial 2 finished with value: 0.005147643113619709 and parameters: {'n_estimators': 106, 'learning_rate': 0.13963344642491446, 'num_leaves': 78, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.7552688793291176, 'colsample_bytree': 0.9268856112617945, 'reg_alpha': 7.200304603187653, 'reg_lambda': 8.157215983605582, 'min_split_gain': 0.013951065629537402}. Best is trial 2 with value: 0.005147643113619709.
[Trial 2] New best: 0.00515
[I 2025-06-15 20:03:11,958] Trial 3 finished with value: 0.008870758955722228 and parameters: {'n_estimators': 77, 'learning_rate': 0.03448442789509334, 'num_leaves': 60, 'max_depth': 5, 'min_child_samples': 11, 'subsample': 0.5607950838435496, 'colsample_bytree': 0.8214619682952643, 'reg_alpha': 8.263955616963592, 'reg_lambda': 1.7392404667199202, 'min_split_gain': 0.010013338218578628}. Best is trial 2 with value: 0.005147643113619709.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:03:13,068] Trial 4 finished with value: 0.004861698318033946 and parameters: {'n_estimators': 156, 'learning_rate': 0.05000179337488712, 'num_leaves': 42, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.8191956502389452, 'colsample_bytree': 0.7659846529829015, 'reg_alpha': 8.889568995828562, 'reg_lambda': 2.0089373312251846, 'min_split_gain': 0.015441647160962963}. Best is trial 4 with value: 0.004861698318033946.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:03:14,094] Trial 5 finished with value: 0.0175911473106353 and parameters: {'n_estimators': 151, 'learning_rate': 0.013318857855150041, 'num_leaves': 122, 'max_depth': 11, 'min_child_samples': 9, 'subsample': 0.6943375651509445, 'colsample_bytree': 0.9031463660865164, 'reg_alpha': 8.551448225404604, 'reg_lambda': 9.530256964269286, 'min_split_gain': 0.03068290097662146}. Best is trial 4 with value: 0.004861698318033946.
[I 2025-06-15 20:03:14,096] A new study created in memory with name: no-name-1f2bbe6d-513e-4870-813a-319d6e2b58d4
[I 2025-06-15 20:03:14,133] Trial 0 finished with value: 0.03309729020278268 and parameters: {'n_estimators': 88, 'learning_rate': 0.08233475647150969, 'num_leaves': 204, 'max_depth': 7, 'min_child_samples': 8, 'subsample': 0.6282937143104591, 'colsample_bytree': 0.7401372532038424, 'reg_alpha': 4.274431562569649, 'reg_lambda': 6.02043458917591, 'min_split_gain': 0.0333028229466158}. Best is trial 0 with value: 0.03309729020278268.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         948
1           Price_lag_2         693
14      pct_chg_Load_DA         586
3         Load_DA_lag_0         383
2           Price_lag_7         316
11           NGas_lag_2         283
16  roll2d_median_Price         266
5       WindOn_DA_lag_0         242
10           Coal_lag_2         223
13            EUA_lag_2         198
15       lag168_Load_DA         195
12            Oil_lag_2         143
9                  WD_7          32
8                  WD_6          28
7                  WD_1          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          32
3         Load_DA_lag_0          14
1           Price_lag_2           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 38.99% done
********************* END NS.. 284 ****************************************************
********************* START NS ... 285 ****************************************************
[I 2025-06-15 20:03:16,978] A new study created in memory with name: no-name-84948626-8264-4eff-b160-efe70d99537b
[I 2025-06-15 20:03:17,390] Trial 0 finished with value: 0.0013331175454503834 and parameters: {'n_estimators': 83, 'learning_rate': 0.08306037011889081, 'num_leaves': 185, 'max_depth': 5, 'min_child_samples': 9, 'subsample': 0.8031118735191238, 'colsample_bytree': 0.7048486873039603, 'reg_alpha': 7.799467533939842, 'reg_lambda': 4.3383364643479405, 'min_split_gain': 0.004886778983184592}. Best is trial 0 with value: 0.0013331175454503834.
[Trial 0] New best: 0.00133
[I 2025-06-15 20:03:18,153] Trial 1 finished with value: 0.0017310910643629183 and parameters: {'n_estimators': 295, 'learning_rate': 0.0837354834632714, 'num_leaves': 182, 'max_depth': 12, 'min_child_samples': 10, 'subsample': 0.5682929268405807, 'colsample_bytree': 0.8024339075984974, 'reg_alpha': 5.756349421937453, 'reg_lambda': 0.7111146977969784, 'min_split_gain': 0.024224113638426154}. Best is trial 0 with value: 0.0013331175454503834.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:03:19,207] Trial 2 finished with value: 0.0014820906333034665 and parameters: {'n_estimators': 249, 'learning_rate': 0.07313779202019362, 'num_leaves': 203, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.6134798642772155, 'colsample_bytree': 0.9347891112399949, 'reg_alpha': 0.6129203034807351, 'reg_lambda': 8.512648215875924, 'min_split_gain': 0.007431355686703784}. Best is trial 0 with value: 0.0013331175454503834.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:03:21,308] Trial 3 finished with value: 0.0029341292161501145 and parameters: {'n_estimators': 122, 'learning_rate': 0.037061075744472684, 'num_leaves': 161, 'max_depth': 16, 'min_child_samples': 14, 'subsample': 0.9672047244749624, 'colsample_bytree': 0.6013615770583278, 'reg_alpha': 1.8664661751376987, 'reg_lambda': 0.4347012097795089, 'min_split_gain': 0.014726600634619358}. Best is trial 0 with value: 0.0013331175454503834.
[I 2025-06-15 20:03:21,311] A new study created in memory with name: no-name-16354c71-ecae-4799-9d15-a8e3064bcf17
[I 2025-06-15 20:03:21,399] Trial 0 finished with value: 0.007157868408290194 and parameters: {'n_estimators': 146, 'learning_rate': 0.027247261964400384, 'num_leaves': 159, 'max_depth': 8, 'min_child_samples': 2, 'subsample': 0.5621646453586115, 'colsample_bytree': 0.8856151998312055, 'reg_alpha': 2.947466447405369, 'reg_lambda': 9.487912617542847, 'min_split_gain': 0.019498485784657595}. Best is trial 0 with value: 0.007157868408290194.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         483
1           Price_lag_2         252
14      pct_chg_Load_DA         241
3         Load_DA_lag_0         158
11           NGas_lag_2         117
2           Price_lag_7          95
16  roll2d_median_Price          94
13            EUA_lag_2          87
10           Coal_lag_2          79
15       lag168_Load_DA          73
5       WindOn_DA_lag_0          54
12            Oil_lag_2          50
9                  WD_7          11
8                  WD_6           6
7                  WD_1           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         159
3         Load_DA_lag_0          67
5       WindOn_DA_lag_0          30
12            Oil_lag_2          22
1           Price_lag_2          15
15       lag168_Load_DA           7
10           Coal_lag_2           1
16  roll2d_median_Price           1
2           Price_lag_7           1
7                  WD_1           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
11           NGas_lag_2           0
4        Solar_DA_lag_0           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 39.12% done
********************* END NS.. 285 ****************************************************
********************* START NS ... 286 ****************************************************
[I 2025-06-15 20:03:23,192] A new study created in memory with name: no-name-56ecb18a-0611-4f27-93ee-bd5b7d988087
[I 2025-06-15 20:03:25,527] Trial 0 finished with value: 0.0057748611643515324 and parameters: {'n_estimators': 237, 'learning_rate': 0.01908660614416993, 'num_leaves': 239, 'max_depth': 14, 'min_child_samples': 3, 'subsample': 0.7064207626028567, 'colsample_bytree': 0.5216215163346575, 'reg_alpha': 5.297973761138107, 'reg_lambda': 8.100348145758801, 'min_split_gain': 0.041511312764662966}. Best is trial 0 with value: 0.0057748611643515324.
[Trial 0] New best: 0.00577
[I 2025-06-15 20:03:27,644] Trial 1 finished with value: 0.004070542637746743 and parameters: {'n_estimators': 276, 'learning_rate': 0.014900139031423282, 'num_leaves': 187, 'max_depth': 10, 'min_child_samples': 3, 'subsample': 0.6332092642033852, 'colsample_bytree': 0.8276675577835503, 'reg_alpha': 8.37734838033314, 'reg_lambda': 7.901729614448831, 'min_split_gain': 0.025535695827398626}. Best is trial 1 with value: 0.004070542637746743.
[Trial 1] New best: 0.00407
[I 2025-06-15 20:03:28,618] Trial 2 finished with value: 0.004468838885767185 and parameters: {'n_estimators': 181, 'learning_rate': 0.024973379133094442, 'num_leaves': 164, 'max_depth': 6, 'min_child_samples': 21, 'subsample': 0.5501908035773486, 'colsample_bytree': 0.6908413846867324, 'reg_alpha': 9.367169988072193, 'reg_lambda': 8.038804758699197, 'min_split_gain': 0.005277491250895778}. Best is trial 1 with value: 0.004070542637746743.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:03:29,390] Trial 3 finished with value: 0.0029822363229558974 and parameters: {'n_estimators': 272, 'learning_rate': 0.18372607720246692, 'num_leaves': 72, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.6297731990037263, 'colsample_bytree': 0.683807261707772, 'reg_alpha': 0.2698873937947044, 'reg_lambda': 0.2590592895701205, 'min_split_gain': 0.020897552505158665}. Best is trial 3 with value: 0.0029822363229558974.
[Trial 3] New best: 0.00298
[I 2025-06-15 20:03:29,689] Trial 4 finished with value: 0.0034383281618681964 and parameters: {'n_estimators': 225, 'learning_rate': 0.24417107875895286, 'num_leaves': 189, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.9589939125339466, 'colsample_bytree': 0.7784489315202558, 'reg_alpha': 7.552083313368482, 'reg_lambda': 4.125642879491965, 'min_split_gain': 0.046139181381631415}. Best is trial 3 with value: 0.0029822363229558974.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:03:30,697] Trial 5 finished with value: 0.0043257556199792066 and parameters: {'n_estimators': 155, 'learning_rate': 0.031418247472480905, 'num_leaves': 134, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.6974633973006017, 'colsample_bytree': 0.699095911405283, 'reg_alpha': 9.046929849655, 'reg_lambda': 5.111495977443465, 'min_split_gain': 0.025067788048955}. Best is trial 3 with value: 0.0029822363229558974.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:03:31,753] Trial 6 finished with value: 0.0035396385826873926 and parameters: {'n_estimators': 146, 'learning_rate': 0.07320472204213545, 'num_leaves': 158, 'max_depth': 11, 'min_child_samples': 4, 'subsample': 0.7382794434024686, 'colsample_bytree': 0.7258259508690721, 'reg_alpha': 5.135529663502229, 'reg_lambda': 2.4607366174203604, 'min_split_gain': 0.01537723191035727}. Best is trial 3 with value: 0.0029822363229558974.
[I 2025-06-15 20:03:31,756] A new study created in memory with name: no-name-5a6e36b1-74cb-405d-a688-04f58ced2287
[I 2025-06-15 20:03:31,850] Trial 0 finished with value: 0.009385648144342574 and parameters: {'n_estimators': 236, 'learning_rate': 0.03033812872749065, 'num_leaves': 242, 'max_depth': 6, 'min_child_samples': 5, 'subsample': 0.6511455661038217, 'colsample_bytree': 0.8436062408921017, 'reg_alpha': 5.258668804482012, 'reg_lambda': 5.698766242532335, 'min_split_gain': 0.011149782179476153}. Best is trial 0 with value: 0.009385648144342574.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         508
1           Price_lag_2         350
14      pct_chg_Load_DA         325
3         Load_DA_lag_0         259
5       WindOn_DA_lag_0         195
15       lag168_Load_DA         192
2           Price_lag_7         190
11           NGas_lag_2         155
16  roll2d_median_Price         152
12            Oil_lag_2         137
10           Coal_lag_2         115
13            EUA_lag_2         103
9                  WD_7          10
7                  WD_1           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         141
3         Load_DA_lag_0          43
1           Price_lag_2          13
12            Oil_lag_2           4
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 39.26% done
********************* END NS.. 286 ****************************************************
********************* START NS ... 287 ****************************************************
[I 2025-06-15 20:03:33,937] A new study created in memory with name: no-name-8f733a88-677e-45b5-b5e2-6009744b47bc
[I 2025-06-15 20:03:35,466] Trial 0 finished with value: 0.00447165047568765 and parameters: {'n_estimators': 111, 'learning_rate': 0.03662126307803911, 'num_leaves': 135, 'max_depth': 15, 'min_child_samples': 20, 'subsample': 0.8867913136365577, 'colsample_bytree': 0.8989350263342908, 'reg_alpha': 3.4787826232028527, 'reg_lambda': 6.434158616331823, 'min_split_gain': 0.001459230923339594}. Best is trial 0 with value: 0.00447165047568765.
[Trial 0] New best: 0.00447
[I 2025-06-15 20:03:36,974] Trial 1 finished with value: 0.003683495715452291 and parameters: {'n_estimators': 196, 'learning_rate': 0.04116587177086068, 'num_leaves': 216, 'max_depth': 9, 'min_child_samples': 13, 'subsample': 0.5451385903561232, 'colsample_bytree': 0.9523339686294695, 'reg_alpha': 8.111853300771601, 'reg_lambda': 6.765935897261458, 'min_split_gain': 0.008197588117192884}. Best is trial 1 with value: 0.003683495715452291.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:03:37,611] Trial 2 finished with value: 0.003815587938750731 and parameters: {'n_estimators': 115, 'learning_rate': 0.2173105411326548, 'num_leaves': 210, 'max_depth': 10, 'min_child_samples': 25, 'subsample': 0.827303114013815, 'colsample_bytree': 0.8828559316799574, 'reg_alpha': 3.538453150942895, 'reg_lambda': 0.7131307832035039, 'min_split_gain': 0.008093347687527957}. Best is trial 1 with value: 0.003683495715452291.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:03:38,766] Trial 3 finished with value: 0.008420409102116643 and parameters: {'n_estimators': 143, 'learning_rate': 0.020909952528332235, 'num_leaves': 132, 'max_depth': 8, 'min_child_samples': 22, 'subsample': 0.9153090987600279, 'colsample_bytree': 0.6400029650172485, 'reg_alpha': 3.9135380074323542, 'reg_lambda': 0.16208258796713948, 'min_split_gain': 0.03726423367348183}. Best is trial 1 with value: 0.003683495715452291.
[I 2025-06-15 20:03:38,768] A new study created in memory with name: no-name-a51e6356-5450-4d3b-920b-489f2d461852
[I 2025-06-15 20:03:38,825] Trial 0 finished with value: 0.010721886293945056 and parameters: {'n_estimators': 61, 'learning_rate': 0.02230649091471807, 'num_leaves': 221, 'max_depth': 16, 'min_child_samples': 15, 'subsample': 0.5779772767552074, 'colsample_bytree': 0.6414667223571834, 'reg_alpha': 1.0907023470665378, 'reg_lambda': 0.5468215338698279, 'min_split_gain': 0.007792533293393323}. Best is trial 0 with value: 0.010721886293945056.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1308
14      pct_chg_Load_DA         887
1           Price_lag_2         819
3         Load_DA_lag_0         558
11           NGas_lag_2         428
2           Price_lag_7         422
5       WindOn_DA_lag_0         386
16  roll2d_median_Price         371
13            EUA_lag_2         305
10           Coal_lag_2         288
12            Oil_lag_2         278
15       lag168_Load_DA         268
9                  WD_7          39
8                  WD_6          21
7                  WD_1          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          93
3         Load_DA_lag_0          48
1           Price_lag_2          25
5       WindOn_DA_lag_0          12
16  roll2d_median_Price           7
14      pct_chg_Load_DA           6
2           Price_lag_7           4
15       lag168_Load_DA           2
12            Oil_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
13            EUA_lag_2           0
8                  WD_6           0
-> 39.40% done
********************* END NS.. 287 ****************************************************
********************* START NS ... 288 ****************************************************
[I 2025-06-15 20:03:41,907] A new study created in memory with name: no-name-afa2a26c-fa3a-4c99-aace-262dac1f02ba
[I 2025-06-15 20:03:42,940] Trial 0 finished with value: 0.0036600765939426394 and parameters: {'n_estimators': 155, 'learning_rate': 0.025265390847075426, 'num_leaves': 133, 'max_depth': 14, 'min_child_samples': 4, 'subsample': 0.6645037383531699, 'colsample_bytree': 0.8426919177820501, 'reg_alpha': 8.500862283420998, 'reg_lambda': 5.263998247450829, 'min_split_gain': 0.012669651075749423}. Best is trial 0 with value: 0.0036600765939426394.
[Trial 0] New best: 0.00366
[I 2025-06-15 20:03:43,949] Trial 1 finished with value: 0.004262954354055306 and parameters: {'n_estimators': 121, 'learning_rate': 0.02753257987278869, 'num_leaves': 123, 'max_depth': 14, 'min_child_samples': 25, 'subsample': 0.8219007381964731, 'colsample_bytree': 0.7429899678070255, 'reg_alpha': 5.375045114732188, 'reg_lambda': 5.569085749740164, 'min_split_gain': 0.0032120192661672154}. Best is trial 0 with value: 0.0036600765939426394.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:03:46,114] Trial 2 finished with value: 0.002637320157994271 and parameters: {'n_estimators': 196, 'learning_rate': 0.06277584196424935, 'num_leaves': 98, 'max_depth': 16, 'min_child_samples': 5, 'subsample': 0.7710133838443982, 'colsample_bytree': 0.6068989015127689, 'reg_alpha': 2.020488057173454, 'reg_lambda': 4.733646079552356, 'min_split_gain': 0.015182443296633192}. Best is trial 2 with value: 0.002637320157994271.
[Trial 2] New best: 0.00264
[I 2025-06-15 20:03:46,423] Trial 3 finished with value: 0.0023276802444852367 and parameters: {'n_estimators': 122, 'learning_rate': 0.25191187684811617, 'num_leaves': 207, 'max_depth': 12, 'min_child_samples': 30, 'subsample': 0.6235244088319832, 'colsample_bytree': 0.6834296093059342, 'reg_alpha': 2.1947528435216124, 'reg_lambda': 4.138879316722375, 'min_split_gain': 0.04836106013619413}. Best is trial 3 with value: 0.0023276802444852367.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:03:46,869] Trial 4 finished with value: 0.21363471431655082 and parameters: {'n_estimators': 50, 'learning_rate': 0.013152697366155654, 'num_leaves': 149, 'max_depth': 8, 'min_child_samples': 29, 'subsample': 0.8690268375954704, 'colsample_bytree': 0.6088741382403209, 'reg_alpha': 3.227860331565121, 'reg_lambda': 8.031229624657698, 'min_split_gain': 0.03967643432017662}. Best is trial 3 with value: 0.0023276802444852367.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:03:47,756] Trial 5 finished with value: 0.002323804362016167 and parameters: {'n_estimators': 243, 'learning_rate': 0.05436885869458335, 'num_leaves': 105, 'max_depth': 9, 'min_child_samples': 24, 'subsample': 0.8039180799430301, 'colsample_bytree': 0.9013422548183438, 'reg_alpha': 7.748040098331604, 'reg_lambda': 2.2243978454082014, 'min_split_gain': 0.023722002456556947}. Best is trial 5 with value: 0.002323804362016167.
[I 2025-06-15 20:03:47,759] A new study created in memory with name: no-name-2b8561d7-58d4-4cb3-86f1-84b19477258f
[I 2025-06-15 20:03:47,822] Trial 0 finished with value: 0.047136293747150976 and parameters: {'n_estimators': 289, 'learning_rate': 0.057585369943507464, 'num_leaves': 213, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.5684525535357976, 'colsample_bytree': 0.9849655052090531, 'reg_alpha': 9.075963278083258, 'reg_lambda': 9.579586450069556, 'min_split_gain': 0.03491171700426528}. Best is trial 0 with value: 0.047136293747150976.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         917
14      pct_chg_Load_DA         553
1           Price_lag_2         413
3         Load_DA_lag_0         294
11           NGas_lag_2         273
2           Price_lag_7         215
16  roll2d_median_Price         190
10           Coal_lag_2         158
15       lag168_Load_DA         142
5       WindOn_DA_lag_0         113
13            EUA_lag_2          98
12            Oil_lag_2          97
9                  WD_7          14
7                  WD_1          11
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          38
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 39.53% done
********************* END NS.. 288 ****************************************************
********************* START NS ... 289 ****************************************************
[I 2025-06-15 20:03:50,335] A new study created in memory with name: no-name-6c8caf35-4840-4dde-b768-2d80a59c3e95
[I 2025-06-15 20:03:50,616] Trial 0 finished with value: 0.0024991066641781644 and parameters: {'n_estimators': 58, 'learning_rate': 0.15689682627856075, 'num_leaves': 194, 'max_depth': 4, 'min_child_samples': 30, 'subsample': 0.5749888201258921, 'colsample_bytree': 0.9121228779719268, 'reg_alpha': 9.707988470742002, 'reg_lambda': 7.085714641642871, 'min_split_gain': 0.04855038497202747}. Best is trial 0 with value: 0.0024991066641781644.
[Trial 0] New best: 0.00250
[I 2025-06-15 20:03:52,183] Trial 1 finished with value: 0.0018671179571410357 and parameters: {'n_estimators': 286, 'learning_rate': 0.02059964631373708, 'num_leaves': 193, 'max_depth': 7, 'min_child_samples': 21, 'subsample': 0.7998293248507602, 'colsample_bytree': 0.8586120458332642, 'reg_alpha': 7.0549188381320445, 'reg_lambda': 5.3731672179748, 'min_split_gain': 0.018727766039119215}. Best is trial 1 with value: 0.0018671179571410357.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:03:52,600] Trial 2 finished with value: 0.002101363200427227 and parameters: {'n_estimators': 143, 'learning_rate': 0.21305487042112523, 'num_leaves': 161, 'max_depth': 13, 'min_child_samples': 20, 'subsample': 0.9392357657461163, 'colsample_bytree': 0.9516148742186075, 'reg_alpha': 9.65472613808093, 'reg_lambda': 9.310909714373809, 'min_split_gain': 0.028688200776987246}. Best is trial 1 with value: 0.0018671179571410357.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:03:53,262] Trial 3 finished with value: 0.002026044656154049 and parameters: {'n_estimators': 97, 'learning_rate': 0.0679632318430762, 'num_leaves': 171, 'max_depth': 6, 'min_child_samples': 15, 'subsample': 0.6923887687568002, 'colsample_bytree': 0.653616844880566, 'reg_alpha': 1.7482093502122276, 'reg_lambda': 7.705141092928559, 'min_split_gain': 0.04187654325517098}. Best is trial 1 with value: 0.0018671179571410357.
[I 2025-06-15 20:03:53,265] A new study created in memory with name: no-name-b5613f15-8715-4047-b7d4-db0c175bcb32
[I 2025-06-15 20:03:53,312] Trial 0 finished with value: 0.06628925518927747 and parameters: {'n_estimators': 102, 'learning_rate': 0.020438748582122315, 'num_leaves': 140, 'max_depth': 7, 'min_child_samples': 14, 'subsample': 0.7161994117151502, 'colsample_bytree': 0.5995663723421977, 'reg_alpha': 8.45547892873546, 'reg_lambda': 8.587575397986006, 'min_split_gain': 0.011533185225326887}. Best is trial 0 with value: 0.06628925518927747.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2355
1           Price_lag_2        1451
14      pct_chg_Load_DA        1225
3         Load_DA_lag_0         620
11           NGas_lag_2         531
2           Price_lag_7         510
16  roll2d_median_Price         468
15       lag168_Load_DA         300
10           Coal_lag_2         295
13            EUA_lag_2         273
5       WindOn_DA_lag_0         238
12            Oil_lag_2         210
9                  WD_7          30
7                  WD_1           5
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          96
1           Price_lag_2          37
2           Price_lag_7           3
3         Load_DA_lag_0           3
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 39.67% done
********************* END NS.. 289 ****************************************************
********************* START NS ... 290 ****************************************************
[I 2025-06-15 20:03:56,624] A new study created in memory with name: no-name-587031f4-f3ba-4fdb-b91c-20843c27383a
[I 2025-06-15 20:03:57,055] Trial 0 finished with value: 0.0008343013560989021 and parameters: {'n_estimators': 93, 'learning_rate': 0.12083813497990956, 'num_leaves': 128, 'max_depth': 14, 'min_child_samples': 9, 'subsample': 0.8264439743142873, 'colsample_bytree': 0.9840446344187336, 'reg_alpha': 6.439656484248651, 'reg_lambda': 2.2402445994525086, 'min_split_gain': 0.021755847070243174}. Best is trial 0 with value: 0.0008343013560989021.
[Trial 0] New best: 0.00083
[I 2025-06-15 20:03:57,909] Trial 1 finished with value: 0.0007245531172839006 and parameters: {'n_estimators': 126, 'learning_rate': 0.03290585987801756, 'num_leaves': 60, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.7282708963278803, 'colsample_bytree': 0.779044578573476, 'reg_alpha': 7.322160021793054, 'reg_lambda': 5.419502320398437, 'min_split_gain': 0.0053117020956604645}. Best is trial 1 with value: 0.0007245531172839006.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:03:59,184] Trial 2 finished with value: 0.0007806418204769093 and parameters: {'n_estimators': 166, 'learning_rate': 0.03378061354598477, 'num_leaves': 97, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.8871516719916313, 'colsample_bytree': 0.9506454389625242, 'reg_alpha': 3.1424134032339834, 'reg_lambda': 6.400156171442259, 'min_split_gain': 0.04434173951815337}. Best is trial 1 with value: 0.0007245531172839006.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:04:00,847] Trial 3 finished with value: 0.0010214012932366137 and parameters: {'n_estimators': 153, 'learning_rate': 0.024070234981634703, 'num_leaves': 103, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.9897085534641068, 'colsample_bytree': 0.5863957657432755, 'reg_alpha': 6.077868058225592, 'reg_lambda': 8.969654479670755, 'min_split_gain': 0.004840255119298609}. Best is trial 1 with value: 0.0007245531172839006.
[I 2025-06-15 20:04:00,850] A new study created in memory with name: no-name-24148510-7934-4b7b-83b8-8bdbdb6a0e35
[I 2025-06-15 20:04:01,040] Trial 0 finished with value: 0.1316652448107922 and parameters: {'n_estimators': 168, 'learning_rate': 0.01200325299686644, 'num_leaves': 162, 'max_depth': 4, 'min_child_samples': 16, 'subsample': 0.6949761291739108, 'colsample_bytree': 0.652506668764618, 'reg_alpha': 5.018533403145396, 'reg_lambda': 8.279106017558249, 'min_split_gain': 0.039227809360563075}. Best is trial 0 with value: 0.1316652448107922.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1271
1           Price_lag_2         834
14      pct_chg_Load_DA         583
3         Load_DA_lag_0         283
2           Price_lag_7         269
16  roll2d_median_Price         255
11           NGas_lag_2         248
10           Coal_lag_2         128
12            Oil_lag_2         115
13            EUA_lag_2         104
5       WindOn_DA_lag_0         102
15       lag168_Load_DA         100
9                  WD_7          10
8                  WD_6           2
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         178
1           Price_lag_2          41
10           Coal_lag_2          12
2           Price_lag_7           7
15       lag168_Load_DA           7
13            EUA_lag_2           4
11           NGas_lag_2           1
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
16  roll2d_median_Price           0
-> 39.81% done
********************* END NS.. 290 ****************************************************
********************* START NS ... 291 ****************************************************
[I 2025-06-15 20:04:03,658] A new study created in memory with name: no-name-3a5e36fd-d626-4f95-8c4d-e992c8f78901
[I 2025-06-15 20:04:04,531] Trial 0 finished with value: 0.0019310337989891355 and parameters: {'n_estimators': 152, 'learning_rate': 0.051574805593096566, 'num_leaves': 94, 'max_depth': 9, 'min_child_samples': 19, 'subsample': 0.5748648013205384, 'colsample_bytree': 0.9368243284344296, 'reg_alpha': 3.9877749413090946, 'reg_lambda': 7.225528680778486, 'min_split_gain': 0.048568773627675595}. Best is trial 0 with value: 0.0019310337989891355.
[Trial 0] New best: 0.00193
[I 2025-06-15 20:04:05,566] Trial 1 finished with value: 0.0017949649230998722 and parameters: {'n_estimators': 213, 'learning_rate': 0.06341066470571179, 'num_leaves': 133, 'max_depth': 11, 'min_child_samples': 29, 'subsample': 0.5408788246445564, 'colsample_bytree': 0.7029958393409214, 'reg_alpha': 5.913344807446056, 'reg_lambda': 8.513271248935654, 'min_split_gain': 0.01612313790848034}. Best is trial 1 with value: 0.0017949649230998722.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:04:06,049] Trial 2 finished with value: 0.012502266728392447 and parameters: {'n_estimators': 141, 'learning_rate': 0.010278290761763696, 'num_leaves': 79, 'max_depth': 4, 'min_child_samples': 7, 'subsample': 0.912795853476444, 'colsample_bytree': 0.7894676894409194, 'reg_alpha': 6.2771513212818695, 'reg_lambda': 3.1347253419247165, 'min_split_gain': 0.025326366293529242}. Best is trial 1 with value: 0.0017949649230998722.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:04:06,887] Trial 3 finished with value: 0.001975878369961587 and parameters: {'n_estimators': 52, 'learning_rate': 0.24329640532994815, 'num_leaves': 255, 'max_depth': 12, 'min_child_samples': 22, 'subsample': 0.895502300555125, 'colsample_bytree': 0.619386998380454, 'reg_alpha': 3.911255776770549, 'reg_lambda': 5.701485897251982, 'min_split_gain': 0.006028598527855866}. Best is trial 1 with value: 0.0017949649230998722.
[I 2025-06-15 20:04:06,892] A new study created in memory with name: no-name-e5ed7f8f-0e48-4247-b657-1debcf0479c7
[I 2025-06-15 20:04:06,997] Trial 0 finished with value: 0.08355009391828594 and parameters: {'n_estimators': 291, 'learning_rate': 0.07634576893678503, 'num_leaves': 248, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.5559456442133288, 'colsample_bytree': 0.8689152295023772, 'reg_alpha': 8.810336802107555, 'reg_lambda': 0.7744507904826936, 'min_split_gain': 0.016398279545066407}. Best is trial 0 with value: 0.08355009391828594.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         885
1           Price_lag_2         656
14      pct_chg_Load_DA         528
3         Load_DA_lag_0         341
11           NGas_lag_2         301
2           Price_lag_7         256
5       WindOn_DA_lag_0         229
16  roll2d_median_Price         198
15       lag168_Load_DA         196
10           Coal_lag_2         189
13            EUA_lag_2         174
12            Oil_lag_2         125
9                  WD_7          23
7                  WD_1          19
8                  WD_6           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          30
1           Price_lag_2           5
10           Coal_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 39.95% done
********************* END NS.. 291 ****************************************************
********************* START NS ... 292 ****************************************************
[I 2025-06-15 20:04:09,918] A new study created in memory with name: no-name-b0a2d729-b72f-4703-9125-559b68b64e2d
[I 2025-06-15 20:04:11,009] Trial 0 finished with value: 0.010951067267372097 and parameters: {'n_estimators': 94, 'learning_rate': 0.020825349449135996, 'num_leaves': 52, 'max_depth': 11, 'min_child_samples': 18, 'subsample': 0.9862613000244501, 'colsample_bytree': 0.7151124414361518, 'reg_alpha': 3.262034717850116, 'reg_lambda': 3.617777366235566, 'min_split_gain': 0.019032546591866584}. Best is trial 0 with value: 0.010951067267372097.
[Trial 0] New best: 0.01095
[I 2025-06-15 20:04:11,589] Trial 1 finished with value: 0.002487475128512192 and parameters: {'n_estimators': 163, 'learning_rate': 0.10124314258493963, 'num_leaves': 68, 'max_depth': 6, 'min_child_samples': 26, 'subsample': 0.63012405494878, 'colsample_bytree': 0.9909470225183528, 'reg_alpha': 2.000805992960879, 'reg_lambda': 6.1592621953064235, 'min_split_gain': 0.042020184027796635}. Best is trial 1 with value: 0.002487475128512192.
[Trial 1] New best: 0.00249
[I 2025-06-15 20:04:12,612] Trial 2 finished with value: 0.024115701028641936 and parameters: {'n_estimators': 61, 'learning_rate': 0.02314307063573263, 'num_leaves': 176, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.5502615053540671, 'colsample_bytree': 0.8849310378893723, 'reg_alpha': 5.414649275841986, 'reg_lambda': 7.256522657507568, 'min_split_gain': 0.0014245960782853341}. Best is trial 1 with value: 0.002487475128512192.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:04:13,422] Trial 3 finished with value: 0.0024966994188690642 and parameters: {'n_estimators': 202, 'learning_rate': 0.17892032424440654, 'num_leaves': 105, 'max_depth': 16, 'min_child_samples': 21, 'subsample': 0.8108421957358466, 'colsample_bytree': 0.8711235498145133, 'reg_alpha': 3.676898411087132, 'reg_lambda': 5.853518366828222, 'min_split_gain': 0.0068859343880011085}. Best is trial 1 with value: 0.002487475128512192.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:04:14,835] Trial 4 finished with value: 0.01307641244441661 and parameters: {'n_estimators': 180, 'learning_rate': 0.010324875515231676, 'num_leaves': 111, 'max_depth': 6, 'min_child_samples': 23, 'subsample': 0.9905425415252671, 'colsample_bytree': 0.7007153728036442, 'reg_alpha': 1.0088743358832764, 'reg_lambda': 6.195411199190546, 'min_split_gain': 0.011403907935120479}. Best is trial 1 with value: 0.002487475128512192.
[I 2025-06-15 20:04:14,838] A new study created in memory with name: no-name-61cd04cf-eec5-4a5f-a7e4-0b7738b6fb07
[I 2025-06-15 20:04:14,924] Trial 0 finished with value: 0.010492736230826472 and parameters: {'n_estimators': 292, 'learning_rate': 0.11706865101072002, 'num_leaves': 192, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.7028457942869308, 'colsample_bytree': 0.6425052880064864, 'reg_alpha': 3.4177953528866367, 'reg_lambda': 9.235126383571886, 'min_split_gain': 0.0004119364421661109}. Best is trial 0 with value: 0.010492736230826472.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         602
14      pct_chg_Load_DA         408
1           Price_lag_2         272
3         Load_DA_lag_0         214
16  roll2d_median_Price         162
11           NGas_lag_2         148
2           Price_lag_7         139
15       lag168_Load_DA          90
10           Coal_lag_2          89
5       WindOn_DA_lag_0          87
12            Oil_lag_2          59
13            EUA_lag_2          48
7                  WD_1           8
9                  WD_7           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          62
15       lag168_Load_DA          10
1           Price_lag_2          10
3         Load_DA_lag_0           8
2           Price_lag_7           7
10           Coal_lag_2           4
13            EUA_lag_2           2
11           NGas_lag_2           1
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 40.08% done
********************* END NS.. 292 ****************************************************
********************* START NS ... 293 ****************************************************
[I 2025-06-15 20:04:17,704] A new study created in memory with name: no-name-a5fe860b-0fb3-4668-8589-6c3653f74302
[I 2025-06-15 20:04:18,616] Trial 0 finished with value: 0.001325521326543886 and parameters: {'n_estimators': 167, 'learning_rate': 0.07117192055307767, 'num_leaves': 249, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.5170619086725532, 'colsample_bytree': 0.8622432321577773, 'reg_alpha': 7.586852441493183, 'reg_lambda': 8.765096232905178, 'min_split_gain': 0.03709999995100332}. Best is trial 0 with value: 0.001325521326543886.
[Trial 0] New best: 0.00133
[I 2025-06-15 20:04:19,194] Trial 1 finished with value: 0.0028128846884723064 and parameters: {'n_estimators': 294, 'learning_rate': 0.21317245770842858, 'num_leaves': 232, 'max_depth': 6, 'min_child_samples': 26, 'subsample': 0.6320997108380246, 'colsample_bytree': 0.5511947605617866, 'reg_alpha': 8.117971851031134, 'reg_lambda': 5.321986961295221, 'min_split_gain': 0.037354911370111696}. Best is trial 0 with value: 0.001325521326543886.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:04:20,273] Trial 2 finished with value: 0.002977388342182606 and parameters: {'n_estimators': 207, 'learning_rate': 0.01540873262667932, 'num_leaves': 254, 'max_depth': 5, 'min_child_samples': 18, 'subsample': 0.7474385438921389, 'colsample_bytree': 0.7347332604906633, 'reg_alpha': 2.650816826414617, 'reg_lambda': 2.5133885365860364, 'min_split_gain': 0.04675241396931991}. Best is trial 0 with value: 0.001325521326543886.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:04:21,914] Trial 3 finished with value: 0.002758564012691224 and parameters: {'n_estimators': 90, 'learning_rate': 0.04072463920200563, 'num_leaves': 146, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.6474027455144569, 'colsample_bytree': 0.6050315220260836, 'reg_alpha': 1.6792490303956509, 'reg_lambda': 0.07969775656359923, 'min_split_gain': 0.010665094796000514}. Best is trial 0 with value: 0.001325521326543886.
[I 2025-06-15 20:04:21,917] A new study created in memory with name: no-name-87a99dde-c08b-4b01-a88f-ad93723ba1b4
[I 2025-06-15 20:04:22,011] Trial 0 finished with value: 0.003817570037598649 and parameters: {'n_estimators': 234, 'learning_rate': 0.02394351546515417, 'num_leaves': 196, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.7330197527597175, 'colsample_bytree': 0.7925674673252745, 'reg_alpha': 8.367014287747747, 'reg_lambda': 3.928897360550876, 'min_split_gain': 0.00356641548226202}. Best is trial 0 with value: 0.003817570037598649.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         655
14      pct_chg_Load_DA         387
1           Price_lag_2         353
11           NGas_lag_2         214
3         Load_DA_lag_0         203
2           Price_lag_7         188
16  roll2d_median_Price         150
10           Coal_lag_2         123
5       WindOn_DA_lag_0         113
15       lag168_Load_DA         108
12            Oil_lag_2          76
13            EUA_lag_2          68
9                  WD_7          19
7                  WD_1          11
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         188
1           Price_lag_2          33
3         Load_DA_lag_0           7
10           Coal_lag_2           1
13            EUA_lag_2           1
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 40.22% done
********************* END NS.. 293 ****************************************************
********************* START NS ... 294 ****************************************************
[I 2025-06-15 20:04:24,853] A new study created in memory with name: no-name-e26ebe74-a508-459f-bcc8-dee1df90c881
[I 2025-06-15 20:04:27,130] Trial 0 finished with value: 0.002222455150217588 and parameters: {'n_estimators': 136, 'learning_rate': 0.18908833720920243, 'num_leaves': 235, 'max_depth': 16, 'min_child_samples': 3, 'subsample': 0.6416289912532755, 'colsample_bytree': 0.5525894678598668, 'reg_alpha': 4.015822064732001, 'reg_lambda': 0.77636809980967, 'min_split_gain': 0.021245563164795346}. Best is trial 0 with value: 0.002222455150217588.
[Trial 0] New best: 0.00222
[I 2025-06-15 20:04:28,226] Trial 1 finished with value: 0.0019618921464336267 and parameters: {'n_estimators': 152, 'learning_rate': 0.07520629209719919, 'num_leaves': 167, 'max_depth': 6, 'min_child_samples': 5, 'subsample': 0.8579207746156805, 'colsample_bytree': 0.6057223518233705, 'reg_alpha': 6.095609438532828, 'reg_lambda': 7.869348935190605, 'min_split_gain': 0.02590064543059145}. Best is trial 1 with value: 0.0019618921464336267.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:04:30,915] Trial 2 finished with value: 0.001904195756852829 and parameters: {'n_estimators': 259, 'learning_rate': 0.04403126824044548, 'num_leaves': 70, 'max_depth': 14, 'min_child_samples': 8, 'subsample': 0.8915599640689047, 'colsample_bytree': 0.6784357087672127, 'reg_alpha': 9.034372686847533, 'reg_lambda': 9.443497360948594, 'min_split_gain': 0.04880278682160055}. Best is trial 2 with value: 0.001904195756852829.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:04:32,310] Trial 3 finished with value: 0.04852324285489007 and parameters: {'n_estimators': 56, 'learning_rate': 0.01758241314997909, 'num_leaves': 88, 'max_depth': 15, 'min_child_samples': 11, 'subsample': 0.8602943377251446, 'colsample_bytree': 0.6721179469294092, 'reg_alpha': 0.8339682704850748, 'reg_lambda': 0.19735718197085972, 'min_split_gain': 0.012554024335205817}. Best is trial 2 with value: 0.001904195756852829.
[I 2025-06-15 20:04:32,314] A new study created in memory with name: no-name-479e026d-560f-4add-b0cb-71dc4a4a5760
[I 2025-06-15 20:04:32,346] Trial 0 finished with value: 0.007577328576855055 and parameters: {'n_estimators': 83, 'learning_rate': 0.011896968990382499, 'num_leaves': 71, 'max_depth': 4, 'min_child_samples': 2, 'subsample': 0.8404963284547756, 'colsample_bytree': 0.7645903180993171, 'reg_alpha': 9.901953449541349, 'reg_lambda': 1.517978556197136, 'min_split_gain': 0.005621497837587053}. Best is trial 0 with value: 0.007577328576855055.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         925
1           Price_lag_2         573
14      pct_chg_Load_DA         472
3         Load_DA_lag_0         315
2           Price_lag_7         257
11           NGas_lag_2         255
16  roll2d_median_Price         231
15       lag168_Load_DA         177
5       WindOn_DA_lag_0         172
10           Coal_lag_2         168
13            EUA_lag_2         103
12            Oil_lag_2         101
9                  WD_7          34
8                  WD_6          18
7                  WD_1          15
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          55
1           Price_lag_2          11
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 40.36% done
********************* END NS.. 294 ****************************************************
********************* START NS ... 295 ****************************************************
[I 2025-06-15 20:04:35,931] A new study created in memory with name: no-name-aea1e5b9-b37a-40ea-b44d-5e82abb090fb
[I 2025-06-15 20:04:36,635] Trial 0 finished with value: 0.029497878617376166 and parameters: {'n_estimators': 55, 'learning_rate': 0.02130506095025098, 'num_leaves': 133, 'max_depth': 6, 'min_child_samples': 14, 'subsample': 0.6055882747796613, 'colsample_bytree': 0.541854119706661, 'reg_alpha': 0.9078631133549897, 'reg_lambda': 8.330638959929397, 'min_split_gain': 0.03884057544535771}. Best is trial 0 with value: 0.029497878617376166.
[Trial 0] New best: 0.02950
[I 2025-06-15 20:04:37,825] Trial 1 finished with value: 0.003669340783408769 and parameters: {'n_estimators': 228, 'learning_rate': 0.027157107379598745, 'num_leaves': 96, 'max_depth': 5, 'min_child_samples': 17, 'subsample': 0.8030072352262705, 'colsample_bytree': 0.8553363271219474, 'reg_alpha': 7.506548281552872, 'reg_lambda': 7.27566420127975, 'min_split_gain': 0.023894091825757587}. Best is trial 1 with value: 0.003669340783408769.
[Trial 1] New best: 0.00367
[I 2025-06-15 20:04:39,402] Trial 2 finished with value: 0.004184046748771934 and parameters: {'n_estimators': 273, 'learning_rate': 0.029946891748632572, 'num_leaves': 241, 'max_depth': 7, 'min_child_samples': 21, 'subsample': 0.8114889327749755, 'colsample_bytree': 0.5915537920430708, 'reg_alpha': 6.506056406517966, 'reg_lambda': 7.879189474758622, 'min_split_gain': 0.03820077889632753}. Best is trial 1 with value: 0.003669340783408769.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:04:42,252] Trial 3 finished with value: 0.0036649723778786783 and parameters: {'n_estimators': 239, 'learning_rate': 0.01455830755396079, 'num_leaves': 48, 'max_depth': 9, 'min_child_samples': 8, 'subsample': 0.5314896967818716, 'colsample_bytree': 0.9339050303044011, 'reg_alpha': 1.105166369367222, 'reg_lambda': 9.453939140697772, 'min_split_gain': 0.013770376172002314}. Best is trial 3 with value: 0.0036649723778786783.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:04:43,057] Trial 4 finished with value: 0.003397207743425379 and parameters: {'n_estimators': 122, 'learning_rate': 0.040067796185626246, 'num_leaves': 233, 'max_depth': 7, 'min_child_samples': 12, 'subsample': 0.5859460697365293, 'colsample_bytree': 0.9450439713927834, 'reg_alpha': 7.299385898099793, 'reg_lambda': 9.767859775193026, 'min_split_gain': 0.04280439321551865}. Best is trial 4 with value: 0.003397207743425379.
[I 2025-06-15 20:04:43,060] A new study created in memory with name: no-name-b6f037c2-336a-4a0d-8a16-3b9bbc84b1f1
[I 2025-06-15 20:04:43,100] Trial 0 finished with value: 0.011886124364869721 and parameters: {'n_estimators': 154, 'learning_rate': 0.09058940250454833, 'num_leaves': 191, 'max_depth': 8, 'min_child_samples': 13, 'subsample': 0.5452732189912324, 'colsample_bytree': 0.8098643681577069, 'reg_alpha': 7.64871601122999, 'reg_lambda': 8.238197186589224, 'min_split_gain': 0.020545521866759023}. Best is trial 0 with value: 0.011886124364869721.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1062
1           Price_lag_2         513
14      pct_chg_Load_DA         512
2           Price_lag_7         232
11           NGas_lag_2         230
3         Load_DA_lag_0         215
16  roll2d_median_Price         191
10           Coal_lag_2          88
12            Oil_lag_2          86
13            EUA_lag_2          83
15       lag168_Load_DA          45
5       WindOn_DA_lag_0          36
9                  WD_7          28
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           5
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 40.49% done
********************* END NS.. 295 ****************************************************
********************* START NS ... 296 ****************************************************
[I 2025-06-15 20:04:45,187] A new study created in memory with name: no-name-7274c795-2f3c-4d33-9f0d-26a9d86d3e53
[I 2025-06-15 20:04:46,125] Trial 0 finished with value: 0.038685022600635326 and parameters: {'n_estimators': 145, 'learning_rate': 0.13622555203883913, 'num_leaves': 108, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.697219875560694, 'colsample_bytree': 0.5575443108431994, 'reg_alpha': 1.5655605747509205, 'reg_lambda': 0.9787717463456147, 'min_split_gain': 0.03226479310255009}. Best is trial 0 with value: 0.038685022600635326.
[Trial 0] New best: 0.03869
[I 2025-06-15 20:04:47,637] Trial 1 finished with value: 0.03546547462268091 and parameters: {'n_estimators': 178, 'learning_rate': 0.040306339986115894, 'num_leaves': 129, 'max_depth': 8, 'min_child_samples': 12, 'subsample': 0.7702843490761719, 'colsample_bytree': 0.8904904980686641, 'reg_alpha': 4.1032960544293395, 'reg_lambda': 6.541762358557197, 'min_split_gain': 0.018755849954346066}. Best is trial 1 with value: 0.03546547462268091.
[Trial 1] New best: 0.03547
[I 2025-06-15 20:04:48,276] Trial 2 finished with value: 0.038266281562926004 and parameters: {'n_estimators': 242, 'learning_rate': 0.20886419332152323, 'num_leaves': 54, 'max_depth': 16, 'min_child_samples': 25, 'subsample': 0.9157997768671824, 'colsample_bytree': 0.6182902505573579, 'reg_alpha': 3.516722639495905, 'reg_lambda': 8.536080883397242, 'min_split_gain': 0.03293278738012705}. Best is trial 1 with value: 0.03546547462268091.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:04:49,362] Trial 3 finished with value: 0.04084831635250074 and parameters: {'n_estimators': 94, 'learning_rate': 0.01969157205242344, 'num_leaves': 114, 'max_depth': 8, 'min_child_samples': 4, 'subsample': 0.8837492348021805, 'colsample_bytree': 0.9190355429269135, 'reg_alpha': 1.837955501639229, 'reg_lambda': 8.883804320925826, 'min_split_gain': 0.04622505123892999}. Best is trial 1 with value: 0.03546547462268091.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:04:50,312] Trial 4 finished with value: 0.039456230849154325 and parameters: {'n_estimators': 117, 'learning_rate': 0.049071749382185295, 'num_leaves': 232, 'max_depth': 7, 'min_child_samples': 17, 'subsample': 0.7867947203775203, 'colsample_bytree': 0.576406753834372, 'reg_alpha': 2.938308977836337, 'reg_lambda': 6.12054777081566, 'min_split_gain': 0.03591322322193262}. Best is trial 1 with value: 0.03546547462268091.
[I 2025-06-15 20:04:50,315] A new study created in memory with name: no-name-14ffbea3-f20f-4079-9cfd-f725102957ec
[I 2025-06-15 20:04:50,378] Trial 0 finished with value: 0.2155106152978307 and parameters: {'n_estimators': 161, 'learning_rate': 0.06290664030357933, 'num_leaves': 243, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.9873146354722377, 'colsample_bytree': 0.5070252305576374, 'reg_alpha': 5.264944954214882, 'reg_lambda': 7.177258501166035, 'min_split_gain': 0.02230471056356705}. Best is trial 0 with value: 0.2155106152978307.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1531
1           Price_lag_2        1152
14      pct_chg_Load_DA         990
3         Load_DA_lag_0         710
11           NGas_lag_2         501
2           Price_lag_7         486
16  roll2d_median_Price         398
5       WindOn_DA_lag_0         383
15       lag168_Load_DA         298
12            Oil_lag_2         270
13            EUA_lag_2         268
10           Coal_lag_2         247
9                  WD_7          32
7                  WD_1          22
8                  WD_6          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          53
1           Price_lag_2          17
5       WindOn_DA_lag_0          13
7                  WD_1           6
11           NGas_lag_2           3
15       lag168_Load_DA           2
13            EUA_lag_2           2
10           Coal_lag_2           1
16  roll2d_median_Price           1
9                  WD_7           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
12            Oil_lag_2           0
3         Load_DA_lag_0           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 40.63% done
********************* END NS.. 296 ****************************************************
********************* START NS ... 297 ****************************************************
[I 2025-06-15 20:04:53,309] A new study created in memory with name: no-name-06b64c00-4d6a-47c5-bfcf-29ab771de4bc
[I 2025-06-15 20:04:53,574] Trial 0 finished with value: 0.03580627861475345 and parameters: {'n_estimators': 230, 'learning_rate': 0.18225974336627088, 'num_leaves': 217, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.8976809585838774, 'colsample_bytree': 0.9789959778161739, 'reg_alpha': 4.699102868328541, 'reg_lambda': 7.531546459594721, 'min_split_gain': 0.025362726297784008}. Best is trial 0 with value: 0.03580627861475345.
[Trial 0] New best: 0.03581
[I 2025-06-15 20:04:56,045] Trial 1 finished with value: 0.03496108405762922 and parameters: {'n_estimators': 229, 'learning_rate': 0.02510584326861431, 'num_leaves': 76, 'max_depth': 10, 'min_child_samples': 15, 'subsample': 0.7630742157562521, 'colsample_bytree': 0.7871227729687477, 'reg_alpha': 2.1395756631538454, 'reg_lambda': 4.645743714778434, 'min_split_gain': 0.04914259683881239}. Best is trial 1 with value: 0.03496108405762922.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:04:57,785] Trial 2 finished with value: 0.03407509708195281 and parameters: {'n_estimators': 197, 'learning_rate': 0.01468173689125046, 'num_leaves': 159, 'max_depth': 11, 'min_child_samples': 11, 'subsample': 0.5434140519536416, 'colsample_bytree': 0.8574489379967394, 'reg_alpha': 3.0789951595390685, 'reg_lambda': 9.058044457874, 'min_split_gain': 0.04076362074289863}. Best is trial 2 with value: 0.03407509708195281.
[Trial 2] New best: 0.03408
[I 2025-06-15 20:04:59,078] Trial 3 finished with value: 0.03312007677472931 and parameters: {'n_estimators': 190, 'learning_rate': 0.05556103176486528, 'num_leaves': 126, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.8732548036712868, 'colsample_bytree': 0.9885357156193808, 'reg_alpha': 6.835316511153788, 'reg_lambda': 0.4779881363635652, 'min_split_gain': 0.013766887555837626}. Best is trial 3 with value: 0.03312007677472931.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:04:59,578] Trial 4 finished with value: 0.03474671047251637 and parameters: {'n_estimators': 267, 'learning_rate': 0.12592364457598984, 'num_leaves': 190, 'max_depth': 6, 'min_child_samples': 10, 'subsample': 0.6795473468448705, 'colsample_bytree': 0.677253339625463, 'reg_alpha': 6.630545996394135, 'reg_lambda': 1.5337814703758934, 'min_split_gain': 0.03959517183880204}. Best is trial 3 with value: 0.03312007677472931.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:05:00,831] Trial 5 finished with value: 0.035770125243107125 and parameters: {'n_estimators': 160, 'learning_rate': 0.030857095453948363, 'num_leaves': 64, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.5762830405921141, 'colsample_bytree': 0.6992556094764129, 'reg_alpha': 3.4709470234044026, 'reg_lambda': 7.410448531735908, 'min_split_gain': 0.007710299714126218}. Best is trial 3 with value: 0.03312007677472931.
[I 2025-06-15 20:05:00,834] A new study created in memory with name: no-name-4ad49ca7-4128-4d41-8704-f8f6fa6827ee
[I 2025-06-15 20:05:00,887] Trial 0 finished with value: 0.24901310940019597 and parameters: {'n_estimators': 246, 'learning_rate': 0.010991011295925116, 'num_leaves': 52, 'max_depth': 7, 'min_child_samples': 12, 'subsample': 0.9280497608293568, 'colsample_bytree': 0.6993183815431572, 'reg_alpha': 6.904429323352527, 'reg_lambda': 4.466668644094252, 'min_split_gain': 0.012738254655017479}. Best is trial 0 with value: 0.24901310940019597.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1081
14      pct_chg_Load_DA         697
1           Price_lag_2         531
3         Load_DA_lag_0         461
11           NGas_lag_2         372
2           Price_lag_7         371
16  roll2d_median_Price         324
5       WindOn_DA_lag_0         280
10           Coal_lag_2         272
13            EUA_lag_2         226
15       lag168_Load_DA         201
12            Oil_lag_2         164
9                  WD_7          33
7                  WD_1          22
8                  WD_6          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         296
1           Price_lag_2          58
5       WindOn_DA_lag_0          26
7                  WD_1          20
16  roll2d_median_Price          19
13            EUA_lag_2           5
12            Oil_lag_2           4
15       lag168_Load_DA           4
11           NGas_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
9                  WD_7           0
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 40.77% done
********************* END NS.. 297 ****************************************************
********************* START NS ... 298 ****************************************************
[I 2025-06-15 20:05:03,676] A new study created in memory with name: no-name-de2f5cd5-048b-4cff-93df-be277d962490
[I 2025-06-15 20:05:04,707] Trial 0 finished with value: 0.0002818169041847128 and parameters: {'n_estimators': 276, 'learning_rate': 0.015423625440728042, 'num_leaves': 243, 'max_depth': 4, 'min_child_samples': 11, 'subsample': 0.8626445780890845, 'colsample_bytree': 0.9799482262612786, 'reg_alpha': 8.370515217321572, 'reg_lambda': 1.5096532602479384, 'min_split_gain': 0.006905342214360094}. Best is trial 0 with value: 0.0002818169041847128.
[Trial 0] New best: 0.00028
[I 2025-06-15 20:05:05,877] Trial 1 finished with value: 0.000402508617525671 and parameters: {'n_estimators': 201, 'learning_rate': 0.04540398303477936, 'num_leaves': 73, 'max_depth': 7, 'min_child_samples': 13, 'subsample': 0.7904344013583825, 'colsample_bytree': 0.7462004371951915, 'reg_alpha': 5.23454191815844, 'reg_lambda': 4.943878020499936, 'min_split_gain': 0.04420061809348863}. Best is trial 0 with value: 0.0002818169041847128.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:05:07,830] Trial 2 finished with value: 0.00032942080043894443 and parameters: {'n_estimators': 142, 'learning_rate': 0.03637021109568163, 'num_leaves': 78, 'max_depth': 12, 'min_child_samples': 22, 'subsample': 0.8774341150960334, 'colsample_bytree': 0.6218921084240656, 'reg_alpha': 0.9380507831058127, 'reg_lambda': 8.82372333175471, 'min_split_gain': 0.04954569816010479}. Best is trial 0 with value: 0.0002818169041847128.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:05:09,075] Trial 3 finished with value: 0.0018167346547665376 and parameters: {'n_estimators': 124, 'learning_rate': 0.015908099120441388, 'num_leaves': 41, 'max_depth': 16, 'min_child_samples': 4, 'subsample': 0.5783660340982892, 'colsample_bytree': 0.8419192886122251, 'reg_alpha': 4.0482242476388075, 'reg_lambda': 0.5343505729474929, 'min_split_gain': 0.024860023494923264}. Best is trial 0 with value: 0.0002818169041847128.
[I 2025-06-15 20:05:09,078] A new study created in memory with name: no-name-c8ff0b9e-0ceb-4432-b41f-bda00ef8cf50
[I 2025-06-15 20:05:09,218] Trial 0 finished with value: 0.012603332779377877 and parameters: {'n_estimators': 263, 'learning_rate': 0.04338745197912571, 'num_leaves': 125, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.6620585013726976, 'colsample_bytree': 0.7484545259764581, 'reg_alpha': 2.4962389172067203, 'reg_lambda': 7.120653462225114, 'min_split_gain': 0.0018825342251021504}. Best is trial 0 with value: 0.012603332779377877.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2071
14      pct_chg_Load_DA         692
2           Price_lag_7         273
16  roll2d_median_Price         259
3         Load_DA_lag_0         150
1           Price_lag_2         134
11           NGas_lag_2         125
12            Oil_lag_2          77
10           Coal_lag_2          55
15       lag168_Load_DA          36
13            EUA_lag_2          33
5       WindOn_DA_lag_0          28
9                  WD_7          18
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         149
5       WindOn_DA_lag_0         136
3         Load_DA_lag_0          58
15       lag168_Load_DA          41
16  roll2d_median_Price          19
1           Price_lag_2          17
12            Oil_lag_2          14
2           Price_lag_7          12
13            EUA_lag_2           8
11           NGas_lag_2           6
7                  WD_1           4
9                  WD_7           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 40.90% done
********************* END NS.. 298 ****************************************************
********************* START NS ... 299 ****************************************************
[I 2025-06-15 20:05:11,937] A new study created in memory with name: no-name-8e9ddca7-fca7-4c25-babb-9c872e82ac3b
[I 2025-06-15 20:05:13,891] Trial 0 finished with value: 0.0007312436214718904 and parameters: {'n_estimators': 275, 'learning_rate': 0.013579339515880353, 'num_leaves': 232, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.9103218230509692, 'colsample_bytree': 0.682366094833003, 'reg_alpha': 9.50659347308125, 'reg_lambda': 2.387807513474084, 'min_split_gain': 0.029558298532677857}. Best is trial 0 with value: 0.0007312436214718904.
[Trial 0] New best: 0.00073
[I 2025-06-15 20:05:15,449] Trial 1 finished with value: 0.0007655754442133965 and parameters: {'n_estimators': 236, 'learning_rate': 0.03924177899941318, 'num_leaves': 240, 'max_depth': 8, 'min_child_samples': 15, 'subsample': 0.6106624625259893, 'colsample_bytree': 0.763359846590701, 'reg_alpha': 2.7455656732950895, 'reg_lambda': 4.033970223652061, 'min_split_gain': 0.038285148365762973}. Best is trial 0 with value: 0.0007312436214718904.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:05:16,623] Trial 2 finished with value: 0.0007097001549131277 and parameters: {'n_estimators': 142, 'learning_rate': 0.02246975305277903, 'num_leaves': 202, 'max_depth': 12, 'min_child_samples': 29, 'subsample': 0.9708717309934567, 'colsample_bytree': 0.5813748530524543, 'reg_alpha': 8.665426653848007, 'reg_lambda': 9.100369125262905, 'min_split_gain': 0.029191621332819346}. Best is trial 2 with value: 0.0007097001549131277.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:05:17,941] Trial 3 finished with value: 0.0006210331294470081 and parameters: {'n_estimators': 119, 'learning_rate': 0.026606954733407004, 'num_leaves': 205, 'max_depth': 12, 'min_child_samples': 23, 'subsample': 0.8084560811677111, 'colsample_bytree': 0.6554660763188993, 'reg_alpha': 1.6409584731380211, 'reg_lambda': 7.328236462082762, 'min_split_gain': 0.042174715417767225}. Best is trial 3 with value: 0.0006210331294470081.
[I 2025-06-15 20:05:17,944] A new study created in memory with name: no-name-884c456e-5c85-4600-bc8a-fd578936f7be
[I 2025-06-15 20:05:18,000] Trial 0 finished with value: 0.02166776383036766 and parameters: {'n_estimators': 91, 'learning_rate': 0.09919749938566638, 'num_leaves': 45, 'max_depth': 15, 'min_child_samples': 27, 'subsample': 0.7620614756127604, 'colsample_bytree': 0.7412949051579061, 'reg_alpha': 0.7309063477288369, 'reg_lambda': 3.038129636501772, 'min_split_gain': 0.008464804567099532}. Best is trial 0 with value: 0.02166776383036766.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1537
1           Price_lag_2        1359
14      pct_chg_Load_DA         764
16  roll2d_median_Price         418
11           NGas_lag_2         395
3         Load_DA_lag_0         348
2           Price_lag_7         339
15       lag168_Load_DA         203
10           Coal_lag_2         131
5       WindOn_DA_lag_0         122
12            Oil_lag_2         116
13            EUA_lag_2          97
9                  WD_7          14
7                  WD_1           8
8                  WD_6           5
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          42
15       lag168_Load_DA          22
16  roll2d_median_Price          22
5       WindOn_DA_lag_0          17
2           Price_lag_7          15
14      pct_chg_Load_DA          14
3         Load_DA_lag_0          13
11           NGas_lag_2          11
12            Oil_lag_2           6
1           Price_lag_2           1
13            EUA_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
8                  WD_6           0
-> 41.04% done
********************* END NS.. 299 ****************************************************
********************* START NS ... 300 ****************************************************
[I 2025-06-15 20:05:20,797] A new study created in memory with name: no-name-d44e03b6-7b97-4517-8a6a-beaa90bbfef3
[I 2025-06-15 20:05:21,381] Trial 0 finished with value: 0.018435004281223816 and parameters: {'n_estimators': 206, 'learning_rate': 0.05404841305234829, 'num_leaves': 96, 'max_depth': 4, 'min_child_samples': 14, 'subsample': 0.7065988705322723, 'colsample_bytree': 0.668773549008505, 'reg_alpha': 1.4292609274380752, 'reg_lambda': 0.6931864060342452, 'min_split_gain': 0.035488297341842494}. Best is trial 0 with value: 0.018435004281223816.
[Trial 0] New best: 0.01844
[I 2025-06-15 20:05:21,957] Trial 1 finished with value: 0.016144837352805277 and parameters: {'n_estimators': 272, 'learning_rate': 0.08972966568763233, 'num_leaves': 145, 'max_depth': 5, 'min_child_samples': 22, 'subsample': 0.6040103780330321, 'colsample_bytree': 0.9540172137982064, 'reg_alpha': 1.29096870241438, 'reg_lambda': 8.294445392483677, 'min_split_gain': 0.027321348063116632}. Best is trial 1 with value: 0.016144837352805277.
[Trial 1] New best: 0.01614
[I 2025-06-15 20:05:22,382] Trial 2 finished with value: 0.02533919940149131 and parameters: {'n_estimators': 118, 'learning_rate': 0.27475193721836044, 'num_leaves': 194, 'max_depth': 7, 'min_child_samples': 25, 'subsample': 0.5118731727380418, 'colsample_bytree': 0.5517533021305265, 'reg_alpha': 1.3263191458096857, 'reg_lambda': 7.483707017895571, 'min_split_gain': 0.03144572299220906}. Best is trial 1 with value: 0.016144837352805277.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:05:26,810] Trial 3 finished with value: 0.014391744679883267 and parameters: {'n_estimators': 257, 'learning_rate': 0.03831226342935398, 'num_leaves': 164, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.6521760582789164, 'colsample_bytree': 0.5799500207941753, 'reg_alpha': 0.6902828998963306, 'reg_lambda': 6.723749981316054, 'min_split_gain': 0.015746106353841084}. Best is trial 3 with value: 0.014391744679883267.
[Trial 3] New best: 0.01439
[I 2025-06-15 20:05:27,781] Trial 4 finished with value: 0.019305532026956968 and parameters: {'n_estimators': 151, 'learning_rate': 0.022941272092537154, 'num_leaves': 138, 'max_depth': 9, 'min_child_samples': 14, 'subsample': 0.88335861759544, 'colsample_bytree': 0.7968537504548545, 'reg_alpha': 8.906119773929026, 'reg_lambda': 6.554395901650673, 'min_split_gain': 0.03561065831646342}. Best is trial 3 with value: 0.014391744679883267.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:05:31,402] Trial 5 finished with value: 0.015972707670228146 and parameters: {'n_estimators': 273, 'learning_rate': 0.0649804270085316, 'num_leaves': 118, 'max_depth': 9, 'min_child_samples': 11, 'subsample': 0.8079230398302152, 'colsample_bytree': 0.5062410291338042, 'reg_alpha': 3.4380785519734536, 'reg_lambda': 1.5834202642342021, 'min_split_gain': 0.0005749691002436664}. Best is trial 3 with value: 0.014391744679883267.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:05:32,806] Trial 6 finished with value: 0.019779851519082417 and parameters: {'n_estimators': 192, 'learning_rate': 0.030976025328217513, 'num_leaves': 212, 'max_depth': 10, 'min_child_samples': 3, 'subsample': 0.5456852875199387, 'colsample_bytree': 0.6326378886506374, 'reg_alpha': 9.042590627492581, 'reg_lambda': 2.2143706604705837, 'min_split_gain': 0.04354273345495363}. Best is trial 3 with value: 0.014391744679883267.
[I 2025-06-15 20:05:32,808] A new study created in memory with name: no-name-5b32ee42-a173-4701-bc34-25ee3f94d2fc
[I 2025-06-15 20:05:32,964] Trial 0 finished with value: 0.08695343102505985 and parameters: {'n_estimators': 249, 'learning_rate': 0.014891366375205384, 'num_leaves': 100, 'max_depth': 14, 'min_child_samples': 18, 'subsample': 0.8664297130415827, 'colsample_bytree': 0.5478501860262232, 'reg_alpha': 3.556211037237257, 'reg_lambda': 0.2689760346851733, 'min_split_gain': 0.03568970618644062}. Best is trial 0 with value: 0.08695343102505985.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        2971
0           Price_lag_1        2708
14      pct_chg_Load_DA        2519
3         Load_DA_lag_0        2312
5       WindOn_DA_lag_0        1727
2           Price_lag_7        1669
15       lag168_Load_DA        1547
16  roll2d_median_Price        1417
11           NGas_lag_2        1310
10           Coal_lag_2        1262
13            EUA_lag_2         949
12            Oil_lag_2         914
9                  WD_7          76
7                  WD_1          64
8                  WD_6          24
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          99
15       lag168_Load_DA          89
11           NGas_lag_2          74
5       WindOn_DA_lag_0          53
3         Load_DA_lag_0          33
12            Oil_lag_2          16
1           Price_lag_2          13
16  roll2d_median_Price          10
10           Coal_lag_2           4
13            EUA_lag_2           3
2           Price_lag_7           2
7                  WD_1           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 41.18% done
********************* END NS.. 300 ****************************************************
********************* START NS ... 301 ****************************************************
[I 2025-06-15 20:05:39,085] A new study created in memory with name: no-name-7b820827-9b06-4790-a23c-81ab2faeec7c
[I 2025-06-15 20:05:40,329] Trial 0 finished with value: 0.02676902042407579 and parameters: {'n_estimators': 193, 'learning_rate': 0.02316064142787121, 'num_leaves': 51, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.9692166754212554, 'colsample_bytree': 0.9916045114319654, 'reg_alpha': 9.781515521431581, 'reg_lambda': 1.7129217963400667, 'min_split_gain': 0.02907656782481685}. Best is trial 0 with value: 0.02676902042407579.
[Trial 0] New best: 0.02677
[I 2025-06-15 20:05:41,320] Trial 1 finished with value: 0.03221390391572429 and parameters: {'n_estimators': 83, 'learning_rate': 0.012610197078247773, 'num_leaves': 228, 'max_depth': 14, 'min_child_samples': 25, 'subsample': 0.6160942103921121, 'colsample_bytree': 0.874589071725864, 'reg_alpha': 2.045717094802959, 'reg_lambda': 6.972846780239838, 'min_split_gain': 0.0008921846554456659}. Best is trial 0 with value: 0.02676902042407579.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:05:41,675] Trial 2 finished with value: 0.026522486228821678 and parameters: {'n_estimators': 74, 'learning_rate': 0.04927248104108911, 'num_leaves': 200, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.5597997300680523, 'colsample_bytree': 0.9870530463841992, 'reg_alpha': 7.063004426324487, 'reg_lambda': 9.9809737989884, 'min_split_gain': 0.0346868992456968}. Best is trial 2 with value: 0.026522486228821678.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:05:42,419] Trial 3 finished with value: 0.02610340586784995 and parameters: {'n_estimators': 204, 'learning_rate': 0.14193900696217265, 'num_leaves': 230, 'max_depth': 4, 'min_child_samples': 9, 'subsample': 0.5171188396285115, 'colsample_bytree': 0.5090811132621041, 'reg_alpha': 5.662002804997689, 'reg_lambda': 1.7410339317510615, 'min_split_gain': 0.0031370814193840304}. Best is trial 3 with value: 0.02610340586784995.
[I 2025-06-15 20:05:42,420] A new study created in memory with name: no-name-88e14eab-02c9-4297-887a-35949b6b1fa8
[I 2025-06-15 20:05:42,516] Trial 0 finished with value: 0.10532172172477683 and parameters: {'n_estimators': 135, 'learning_rate': 0.011530926709903641, 'num_leaves': 226, 'max_depth': 7, 'min_child_samples': 1, 'subsample': 0.5970495658346431, 'colsample_bytree': 0.6307681369960803, 'reg_alpha': 6.01113075153186, 'reg_lambda': 9.730255014522722, 'min_split_gain': 0.01374220490025293}. Best is trial 0 with value: 0.10532172172477683.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         508
1           Price_lag_2         395
14      pct_chg_Load_DA         228
16  roll2d_median_Price         184
2           Price_lag_7         155
3         Load_DA_lag_0         152
11           NGas_lag_2         139
13            EUA_lag_2         104
10           Coal_lag_2         103
5       WindOn_DA_lag_0          92
12            Oil_lag_2          74
15       lag168_Load_DA          72
9                  WD_7          13
7                  WD_1          11
8                  WD_6           7
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         108
5       WindOn_DA_lag_0          78
11           NGas_lag_2          70
15       lag168_Load_DA          69
3         Load_DA_lag_0          58
16  roll2d_median_Price          35
1           Price_lag_2          21
13            EUA_lag_2          10
12            Oil_lag_2           3
2           Price_lag_7           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
4        Solar_DA_lag_0           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 41.31% done
********************* END NS.. 301 ****************************************************
********************* START NS ... 302 ****************************************************
[I 2025-06-15 20:05:44,787] A new study created in memory with name: no-name-0cf9a550-2075-4b1f-80d5-68cbcd34dc51
[I 2025-06-15 20:05:45,312] Trial 0 finished with value: 0.011547178565564734 and parameters: {'n_estimators': 293, 'learning_rate': 0.08559835895194688, 'num_leaves': 130, 'max_depth': 15, 'min_child_samples': 17, 'subsample': 0.6261306646447532, 'colsample_bytree': 0.7428058206237644, 'reg_alpha': 8.021861065364963, 'reg_lambda': 8.990551147321927, 'min_split_gain': 0.027571448714567584}. Best is trial 0 with value: 0.011547178565564734.
[Trial 0] New best: 0.01155
[I 2025-06-15 20:05:45,872] Trial 1 finished with value: 0.013246194254272876 and parameters: {'n_estimators': 58, 'learning_rate': 0.1797428639812284, 'num_leaves': 232, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.5556411181595232, 'colsample_bytree': 0.5365997830671098, 'reg_alpha': 6.800107643199351, 'reg_lambda': 2.9559813692594803, 'min_split_gain': 0.003043337534803803}. Best is trial 0 with value: 0.011547178565564734.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:05:46,659] Trial 2 finished with value: 0.009671883711893906 and parameters: {'n_estimators': 171, 'learning_rate': 0.1433471517187402, 'num_leaves': 169, 'max_depth': 15, 'min_child_samples': 27, 'subsample': 0.745904171566044, 'colsample_bytree': 0.829843904097828, 'reg_alpha': 2.1235451536764427, 'reg_lambda': 4.152274010096738, 'min_split_gain': 0.00972437039947417}. Best is trial 2 with value: 0.009671883711893906.
[Trial 2] New best: 0.00967
[I 2025-06-15 20:05:49,145] Trial 3 finished with value: 0.013469925274941333 and parameters: {'n_estimators': 231, 'learning_rate': 0.018573963363675526, 'num_leaves': 235, 'max_depth': 8, 'min_child_samples': 28, 'subsample': 0.8493732842711397, 'colsample_bytree': 0.504012949511986, 'reg_alpha': 2.4114894097649806, 'reg_lambda': 7.619398670907583, 'min_split_gain': 0.032999792700453896}. Best is trial 2 with value: 0.009671883711893906.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:05:49,838] Trial 4 finished with value: 0.011729762623095996 and parameters: {'n_estimators': 265, 'learning_rate': 0.09646184903143652, 'num_leaves': 129, 'max_depth': 7, 'min_child_samples': 17, 'subsample': 0.8192652901892868, 'colsample_bytree': 0.6342236725144599, 'reg_alpha': 5.73255743748671, 'reg_lambda': 5.739189352511356, 'min_split_gain': 0.01873211297771183}. Best is trial 2 with value: 0.009671883711893906.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:05:50,509] Trial 5 finished with value: 0.05532489072132718 and parameters: {'n_estimators': 77, 'learning_rate': 0.011168991735394275, 'num_leaves': 235, 'max_depth': 9, 'min_child_samples': 14, 'subsample': 0.5652821380789568, 'colsample_bytree': 0.6309344342950753, 'reg_alpha': 7.293129089553082, 'reg_lambda': 3.530497731394796, 'min_split_gain': 0.028113930924503656}. Best is trial 2 with value: 0.009671883711893906.
[I 2025-06-15 20:05:50,512] A new study created in memory with name: no-name-446d2b8b-7b33-4610-a82d-ecc1a23aa57e
[I 2025-06-15 20:05:50,591] Trial 0 finished with value: 0.0644310969898463 and parameters: {'n_estimators': 120, 'learning_rate': 0.013134822382268223, 'num_leaves': 252, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.9824322083962551, 'colsample_bytree': 0.7723104186149785, 'reg_alpha': 3.668685097749207, 'reg_lambda': 2.283173217582074, 'min_split_gain': 0.00582184684065849}. Best is trial 0 with value: 0.0644310969898463.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         601
14      pct_chg_Load_DA         545
1           Price_lag_2         510
3         Load_DA_lag_0         344
15       lag168_Load_DA         279
5       WindOn_DA_lag_0         278
16  roll2d_median_Price         237
2           Price_lag_7         224
11           NGas_lag_2         205
10           Coal_lag_2         169
13            EUA_lag_2         153
12            Oil_lag_2         140
9                  WD_7          11
7                  WD_1          10
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         190
15       lag168_Load_DA          94
1           Price_lag_2          20
5       WindOn_DA_lag_0           3
10           Coal_lag_2           1
3         Load_DA_lag_0           1
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 41.45% done
********************* END NS.. 302 ****************************************************
********************* START NS ... 303 ****************************************************
[I 2025-06-15 20:05:52,890] A new study created in memory with name: no-name-f26222dc-2699-44f9-bcfa-bc3740428ff3
[I 2025-06-15 20:05:56,141] Trial 0 finished with value: 0.008241705900301656 and parameters: {'n_estimators': 205, 'learning_rate': 0.015990581067468826, 'num_leaves': 163, 'max_depth': 9, 'min_child_samples': 4, 'subsample': 0.9537752840323948, 'colsample_bytree': 0.6750156455348547, 'reg_alpha': 1.9125646041500288, 'reg_lambda': 3.419740326682945, 'min_split_gain': 0.023349984362524008}. Best is trial 0 with value: 0.008241705900301656.
[Trial 0] New best: 0.00824
[I 2025-06-15 20:05:56,731] Trial 1 finished with value: 0.008061568695502558 and parameters: {'n_estimators': 198, 'learning_rate': 0.06661181789158768, 'num_leaves': 103, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.7117057054939824, 'colsample_bytree': 0.5967759739660383, 'reg_alpha': 2.980832452924073, 'reg_lambda': 9.642213269751297, 'min_split_gain': 0.011929421627320103}. Best is trial 1 with value: 0.008061568695502558.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:05:57,515] Trial 2 finished with value: 0.005473625870796764 and parameters: {'n_estimators': 210, 'learning_rate': 0.14441318094144792, 'num_leaves': 55, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.65751606577373, 'colsample_bytree': 0.7057403240078619, 'reg_alpha': 1.027795148012497, 'reg_lambda': 3.1851779539172878, 'min_split_gain': 0.025908199772713497}. Best is trial 2 with value: 0.005473625870796764.
[Trial 2] New best: 0.00547
[I 2025-06-15 20:06:01,232] Trial 3 finished with value: 0.007986895317637297 and parameters: {'n_estimators': 163, 'learning_rate': 0.01326650822915044, 'num_leaves': 115, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.515717020084997, 'colsample_bytree': 0.9794138027798845, 'reg_alpha': 0.4728788341217893, 'reg_lambda': 0.31852923612544104, 'min_split_gain': 0.040499779451452333}. Best is trial 2 with value: 0.005473625870796764.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:06:03,404] Trial 4 finished with value: 0.005647606211496695 and parameters: {'n_estimators': 245, 'learning_rate': 0.02764321226589265, 'num_leaves': 181, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.6139543883170693, 'colsample_bytree': 0.9994476367285885, 'reg_alpha': 3.7357888577908183, 'reg_lambda': 9.908533030920978, 'min_split_gain': 0.02023426220768286}. Best is trial 2 with value: 0.005473625870796764.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:06:03,912] Trial 5 finished with value: 0.00911972837853982 and parameters: {'n_estimators': 184, 'learning_rate': 0.16714507469096607, 'num_leaves': 120, 'max_depth': 9, 'min_child_samples': 12, 'subsample': 0.6955274422569598, 'colsample_bytree': 0.565051672015249, 'reg_alpha': 7.007913520576298, 'reg_lambda': 5.6964584883021345, 'min_split_gain': 0.032827655231927294}. Best is trial 2 with value: 0.005473625870796764.
[I 2025-06-15 20:06:03,914] A new study created in memory with name: no-name-2578096d-a23b-4557-855c-b5af57ca7ed6
[I 2025-06-15 20:06:04,072] Trial 0 finished with value: 0.07787938993217695 and parameters: {'n_estimators': 240, 'learning_rate': 0.013726170200091238, 'num_leaves': 63, 'max_depth': 12, 'min_child_samples': 9, 'subsample': 0.7251545327481723, 'colsample_bytree': 0.993097715851989, 'reg_alpha': 4.085080494558307, 'reg_lambda': 8.282376196646092, 'min_split_gain': 0.0008579513626023272}. Best is trial 0 with value: 0.07787938993217695.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         494
1           Price_lag_2         363
14      pct_chg_Load_DA         359
3         Load_DA_lag_0         304
5       WindOn_DA_lag_0         219
2           Price_lag_7         182
16  roll2d_median_Price         170
15       lag168_Load_DA         163
11           NGas_lag_2         160
10           Coal_lag_2         108
13            EUA_lag_2         105
12            Oil_lag_2          84
9                  WD_7          13
7                  WD_1          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         597
3         Load_DA_lag_0         117
15       lag168_Load_DA          91
2           Price_lag_7          54
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 41.59% done
********************* END NS.. 303 ****************************************************
********************* START NS ... 304 ****************************************************
[I 2025-06-15 20:06:06,668] A new study created in memory with name: no-name-b6ac7d24-a19c-47e8-9bf1-4d18fef14996
[I 2025-06-15 20:06:07,339] Trial 0 finished with value: 0.021497968878491074 and parameters: {'n_estimators': 245, 'learning_rate': 0.26598090042154326, 'num_leaves': 31, 'max_depth': 16, 'min_child_samples': 7, 'subsample': 0.5377664763923615, 'colsample_bytree': 0.9936242886894819, 'reg_alpha': 4.512266784045925, 'reg_lambda': 9.03261426271592, 'min_split_gain': 0.002188593331736488}. Best is trial 0 with value: 0.021497968878491074.
[Trial 0] New best: 0.02150
[I 2025-06-15 20:06:07,830] Trial 1 finished with value: 0.019448145460240516 and parameters: {'n_estimators': 193, 'learning_rate': 0.12050874671414474, 'num_leaves': 156, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.5284026114332843, 'colsample_bytree': 0.8457237440289341, 'reg_alpha': 4.034820427195762, 'reg_lambda': 7.207894089430244, 'min_split_gain': 0.041702789958728476}. Best is trial 1 with value: 0.019448145460240516.
[Trial 1] New best: 0.01945
[I 2025-06-15 20:06:09,246] Trial 2 finished with value: 0.018542819643816367 and parameters: {'n_estimators': 222, 'learning_rate': 0.01823696448080213, 'num_leaves': 191, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.9369194830061551, 'colsample_bytree': 0.7087392798045078, 'reg_alpha': 3.736204082606264, 'reg_lambda': 1.0274959197370115, 'min_split_gain': 0.0478670301432364}. Best is trial 2 with value: 0.018542819643816367.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:06:11,011] Trial 3 finished with value: 0.018558790389951584 and parameters: {'n_estimators': 252, 'learning_rate': 0.041397735166954455, 'num_leaves': 199, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.8937069289188304, 'colsample_bytree': 0.7937681064835453, 'reg_alpha': 9.587889404367811, 'reg_lambda': 6.930800437879885, 'min_split_gain': 0.0078080776144721755}. Best is trial 2 with value: 0.018542819643816367.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:06:11,770] Trial 4 finished with value: 0.01903151084469964 and parameters: {'n_estimators': 82, 'learning_rate': 0.09408206908359526, 'num_leaves': 134, 'max_depth': 12, 'min_child_samples': 28, 'subsample': 0.7265250484439887, 'colsample_bytree': 0.7510277627388435, 'reg_alpha': 7.639633320850758, 'reg_lambda': 5.051547041244589, 'min_split_gain': 0.00933873810364615}. Best is trial 2 with value: 0.018542819643816367.
[I 2025-06-15 20:06:11,772] A new study created in memory with name: no-name-d6f519da-c1f7-4605-914a-ee340ad538ec
[I 2025-06-15 20:06:11,814] Trial 0 finished with value: 0.12246821710663704 and parameters: {'n_estimators': 136, 'learning_rate': 0.02579731019941575, 'num_leaves': 174, 'max_depth': 9, 'min_child_samples': 2, 'subsample': 0.7122523465747146, 'colsample_bytree': 0.6246369277058084, 'reg_alpha': 9.284446463668125, 'reg_lambda': 8.157375851784552, 'min_split_gain': 0.004407722614153265}. Best is trial 0 with value: 0.12246821710663704.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2238
1           Price_lag_2        1720
14      pct_chg_Load_DA        1055
16  roll2d_median_Price         448
11           NGas_lag_2         440
3         Load_DA_lag_0         420
2           Price_lag_7         371
15       lag168_Load_DA         201
10           Coal_lag_2         196
5       WindOn_DA_lag_0         122
13            EUA_lag_2         122
12            Oil_lag_2         108
9                  WD_7          31
8                  WD_6           8
7                  WD_1           3
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         120
3         Load_DA_lag_0          83
1           Price_lag_2          32
15       lag168_Load_DA          30
2           Price_lag_7          13
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 41.72% done
********************* END NS.. 304 ****************************************************
********************* START NS ... 305 ****************************************************
[I 2025-06-15 20:06:14,639] A new study created in memory with name: no-name-51b21402-a2a7-44e9-af62-17c79a6db60f
[I 2025-06-15 20:06:16,636] Trial 0 finished with value: 0.016454750375158856 and parameters: {'n_estimators': 229, 'learning_rate': 0.0332143990377983, 'num_leaves': 61, 'max_depth': 9, 'min_child_samples': 24, 'subsample': 0.6935293905412552, 'colsample_bytree': 0.9630765085706312, 'reg_alpha': 2.850614793919013, 'reg_lambda': 8.040178075438654, 'min_split_gain': 0.04929424703342419}. Best is trial 0 with value: 0.016454750375158856.
[Trial 0] New best: 0.01645
[I 2025-06-15 20:06:18,459] Trial 1 finished with value: 0.016118398662347575 and parameters: {'n_estimators': 172, 'learning_rate': 0.015195232497537457, 'num_leaves': 129, 'max_depth': 16, 'min_child_samples': 17, 'subsample': 0.5917811626929255, 'colsample_bytree': 0.6876147151563277, 'reg_alpha': 3.7682846683716353, 'reg_lambda': 8.70825681863867, 'min_split_gain': 0.04083338909831402}. Best is trial 1 with value: 0.016118398662347575.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:06:19,586] Trial 2 finished with value: 0.022729295018445256 and parameters: {'n_estimators': 176, 'learning_rate': 0.010127882425332347, 'num_leaves': 187, 'max_depth': 5, 'min_child_samples': 24, 'subsample': 0.6154427221803479, 'colsample_bytree': 0.604081828521905, 'reg_alpha': 2.5433196773473368, 'reg_lambda': 6.322387464405669, 'min_split_gain': 0.0010802588280060478}. Best is trial 1 with value: 0.016118398662347575.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:06:20,353] Trial 3 finished with value: 0.015219676404887077 and parameters: {'n_estimators': 99, 'learning_rate': 0.06791190530692873, 'num_leaves': 170, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.6640251859205282, 'colsample_bytree': 0.6211762561753209, 'reg_alpha': 8.660430036665753, 'reg_lambda': 7.989418980475456, 'min_split_gain': 0.020851022264516173}. Best is trial 3 with value: 0.015219676404887077.
[Trial 3] New best: 0.01522
[I 2025-06-15 20:06:21,572] Trial 4 finished with value: 0.018626429874271586 and parameters: {'n_estimators': 170, 'learning_rate': 0.014610163259922569, 'num_leaves': 61, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.8258862207756364, 'colsample_bytree': 0.5772815097419303, 'reg_alpha': 7.9904488473100335, 'reg_lambda': 1.0090431425996393, 'min_split_gain': 0.04469191491805599}. Best is trial 3 with value: 0.015219676404887077.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:06:22,045] Trial 5 finished with value: 0.014459162871841574 and parameters: {'n_estimators': 55, 'learning_rate': 0.05705223703283667, 'num_leaves': 64, 'max_depth': 6, 'min_child_samples': 24, 'subsample': 0.7886483038441352, 'colsample_bytree': 0.7004212054652783, 'reg_alpha': 0.9969088495054201, 'reg_lambda': 6.673785174323275, 'min_split_gain': 0.04385329540072161}. Best is trial 5 with value: 0.014459162871841574.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:06:22,475] Trial 6 finished with value: 0.014113992739623165 and parameters: {'n_estimators': 156, 'learning_rate': 0.16077547986687604, 'num_leaves': 82, 'max_depth': 10, 'min_child_samples': 27, 'subsample': 0.9660998827647829, 'colsample_bytree': 0.8094424185575341, 'reg_alpha': 2.932634365609034, 'reg_lambda': 9.255711743458889, 'min_split_gain': 0.047406123758361854}. Best is trial 6 with value: 0.014113992739623165.
[Trial 6] New best: 0.01411
[I 2025-06-15 20:06:23,313] Trial 7 finished with value: 0.017877959597471366 and parameters: {'n_estimators': 235, 'learning_rate': 0.024463153322983596, 'num_leaves': 256, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.9494177274139133, 'colsample_bytree': 0.7118392707908021, 'reg_alpha': 4.248773498700672, 'reg_lambda': 4.355068683305504, 'min_split_gain': 0.04086515078694833}. Best is trial 6 with value: 0.014113992739623165.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:06:25,442] Trial 8 finished with value: 0.016442204858059865 and parameters: {'n_estimators': 168, 'learning_rate': 0.02729928363481998, 'num_leaves': 57, 'max_depth': 16, 'min_child_samples': 3, 'subsample': 0.5381882025244149, 'colsample_bytree': 0.9897655626304448, 'reg_alpha': 6.160464337695725, 'reg_lambda': 6.825609061758145, 'min_split_gain': 0.00980668789291605}. Best is trial 6 with value: 0.014113992739623165.
[Trial 8] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:06:25,806] Trial 9 finished with value: 0.017393608330129755 and parameters: {'n_estimators': 54, 'learning_rate': 0.293907451062207, 'num_leaves': 222, 'max_depth': 10, 'min_child_samples': 11, 'subsample': 0.7051331750012346, 'colsample_bytree': 0.5721166499261121, 'reg_alpha': 4.936877276469456, 'reg_lambda': 7.303887367033504, 'min_split_gain': 0.015478580202906834}. Best is trial 6 with value: 0.014113992739623165.
[I 2025-06-15 20:06:25,809] A new study created in memory with name: no-name-75a235df-aa03-4dbe-8025-4d0e0ee67863
[I 2025-06-15 20:06:26,003] Trial 0 finished with value: 0.03697795527314342 and parameters: {'n_estimators': 254, 'learning_rate': 0.013064530252207536, 'num_leaves': 41, 'max_depth': 13, 'min_child_samples': 4, 'subsample': 0.8218695838840722, 'colsample_bytree': 0.7815140195598878, 'reg_alpha': 3.697127084631624, 'reg_lambda': 9.782032383957977, 'min_split_gain': 0.010143902731035909}. Best is trial 0 with value: 0.03697795527314342.
[Trial 9] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         347
1           Price_lag_2         253
14      pct_chg_Load_DA         220
3         Load_DA_lag_0         128
11           NGas_lag_2         104
16  roll2d_median_Price          91
2           Price_lag_7          90
15       lag168_Load_DA          80
5       WindOn_DA_lag_0          68
10           Coal_lag_2          52
13            EUA_lag_2          38
12            Oil_lag_2          29
7                  WD_1           5
9                  WD_7           4
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         473
3         Load_DA_lag_0         167
2           Price_lag_7         119
1           Price_lag_2          69
15       lag168_Load_DA          66
5       WindOn_DA_lag_0           8
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 41.86% done
********************* END NS.. 305 ****************************************************
********************* START NS ... 306 ****************************************************
[I 2025-06-15 20:06:28,113] A new study created in memory with name: no-name-5694e4d1-bf7b-40f1-a99d-d3d96068ed8a
[I 2025-06-15 20:06:31,050] Trial 0 finished with value: 0.003948770143704247 and parameters: {'n_estimators': 231, 'learning_rate': 0.04033435048626518, 'num_leaves': 206, 'max_depth': 14, 'min_child_samples': 8, 'subsample': 0.7946110589523318, 'colsample_bytree': 0.7832838397297225, 'reg_alpha': 2.786248138833849, 'reg_lambda': 1.1444054930388148, 'min_split_gain': 0.012315689852587514}. Best is trial 0 with value: 0.003948770143704247.
[Trial 0] New best: 0.00395
[I 2025-06-15 20:06:31,583] Trial 1 finished with value: 0.0038029168209485786 and parameters: {'n_estimators': 146, 'learning_rate': 0.25572913305589995, 'num_leaves': 187, 'max_depth': 11, 'min_child_samples': 8, 'subsample': 0.9410857948358358, 'colsample_bytree': 0.7473423465604208, 'reg_alpha': 4.857145783466491, 'reg_lambda': 8.192976951700468, 'min_split_gain': 0.0072528699731567725}. Best is trial 1 with value: 0.0038029168209485786.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:06:33,072] Trial 2 finished with value: 0.004902922225865765 and parameters: {'n_estimators': 222, 'learning_rate': 0.03133484227923949, 'num_leaves': 86, 'max_depth': 13, 'min_child_samples': 4, 'subsample': 0.9140191427735789, 'colsample_bytree': 0.5939165809747139, 'reg_alpha': 8.999864817255771, 'reg_lambda': 3.846559054136729, 'min_split_gain': 0.04990910434158141}. Best is trial 1 with value: 0.0038029168209485786.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:06:36,475] Trial 3 finished with value: 0.005445151782289458 and parameters: {'n_estimators': 154, 'learning_rate': 0.04954301013061751, 'num_leaves': 107, 'max_depth': 12, 'min_child_samples': 16, 'subsample': 0.7145722724361229, 'colsample_bytree': 0.5414894102994836, 'reg_alpha': 0.31554364494005394, 'reg_lambda': 4.0659079835034975, 'min_split_gain': 0.011862295717465927}. Best is trial 1 with value: 0.0038029168209485786.
[I 2025-06-15 20:06:36,478] A new study created in memory with name: no-name-1276f696-f3d6-4b6f-a138-8096bf0c1c2d
[I 2025-06-15 20:06:36,611] Trial 0 finished with value: 0.010633826168354066 and parameters: {'n_estimators': 105, 'learning_rate': 0.012731361851890473, 'num_leaves': 156, 'max_depth': 15, 'min_child_samples': 10, 'subsample': 0.9039586482161126, 'colsample_bytree': 0.9081965220917918, 'reg_alpha': 0.49785137958337167, 'reg_lambda': 0.5280254675351537, 'min_split_gain': 0.009028183382704674}. Best is trial 0 with value: 0.010633826168354066.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         292
1           Price_lag_2         243
14      pct_chg_Load_DA         222
3         Load_DA_lag_0         191
2           Price_lag_7         138
5       WindOn_DA_lag_0         134
11           NGas_lag_2         129
15       lag168_Load_DA         125
13            EUA_lag_2         120
16  roll2d_median_Price         118
10           Coal_lag_2          85
12            Oil_lag_2          59
9                  WD_7           7
7                  WD_1           7
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         332
3         Load_DA_lag_0          92
14      pct_chg_Load_DA          83
2           Price_lag_7          52
15       lag168_Load_DA          51
13            EUA_lag_2          17
1           Price_lag_2          15
5       WindOn_DA_lag_0           1
11           NGas_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 42.00% done
********************* END NS.. 306 ****************************************************
********************* START NS ... 307 ****************************************************
[I 2025-06-15 20:06:38,808] A new study created in memory with name: no-name-ad41acf0-6216-4f30-9a85-ef27108fe023
[I 2025-06-15 20:06:39,341] Trial 0 finished with value: 0.006700389082480542 and parameters: {'n_estimators': 138, 'learning_rate': 0.07707353140640541, 'num_leaves': 172, 'max_depth': 14, 'min_child_samples': 19, 'subsample': 0.7274486486057654, 'colsample_bytree': 0.8027588553055747, 'reg_alpha': 8.949564469251056, 'reg_lambda': 1.084201217778915, 'min_split_gain': 0.0373798771458596}. Best is trial 0 with value: 0.006700389082480542.
[Trial 0] New best: 0.00670
[I 2025-06-15 20:06:41,338] Trial 1 finished with value: 0.006008170673777902 and parameters: {'n_estimators': 198, 'learning_rate': 0.02633006647869907, 'num_leaves': 207, 'max_depth': 8, 'min_child_samples': 25, 'subsample': 0.6471775660810618, 'colsample_bytree': 0.8944573254034627, 'reg_alpha': 1.6430692736031838, 'reg_lambda': 6.842030777129792, 'min_split_gain': 0.047827106356677154}. Best is trial 1 with value: 0.006008170673777902.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:06:42,870] Trial 2 finished with value: 0.012303285588950963 and parameters: {'n_estimators': 174, 'learning_rate': 0.01122006789256352, 'num_leaves': 189, 'max_depth': 8, 'min_child_samples': 17, 'subsample': 0.7846428172429251, 'colsample_bytree': 0.901997742862414, 'reg_alpha': 4.531777577573549, 'reg_lambda': 1.72550780739811, 'min_split_gain': 0.022122794617655213}. Best is trial 1 with value: 0.006008170673777902.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:06:44,655] Trial 3 finished with value: 0.007699768657141355 and parameters: {'n_estimators': 253, 'learning_rate': 0.012166460783732406, 'num_leaves': 40, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.6073380247732961, 'colsample_bytree': 0.9571114728644314, 'reg_alpha': 9.976466846653038, 'reg_lambda': 3.061700078933879, 'min_split_gain': 0.044000650011948785}. Best is trial 1 with value: 0.006008170673777902.
[I 2025-06-15 20:06:44,662] A new study created in memory with name: no-name-77e56187-62ba-4ff4-8545-ec5e66fb0438
[I 2025-06-15 20:06:44,848] Trial 0 finished with value: 0.05653992913678074 and parameters: {'n_estimators': 65, 'learning_rate': 0.01993759977561038, 'num_leaves': 240, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.778575168153326, 'colsample_bytree': 0.5846172582800484, 'reg_alpha': 9.9641547387657, 'reg_lambda': 0.6328910530965204, 'min_split_gain': 0.04421149795443796}. Best is trial 0 with value: 0.05653992913678074.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2421
1           Price_lag_2        1853
14      pct_chg_Load_DA        1290
3         Load_DA_lag_0         653
11           NGas_lag_2         596
2           Price_lag_7         568
16  roll2d_median_Price         531
15       lag168_Load_DA         351
5       WindOn_DA_lag_0         256
12            Oil_lag_2         232
10           Coal_lag_2         209
13            EUA_lag_2         176
7                  WD_1          17
9                  WD_7          13
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          36
1           Price_lag_2          18
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 42.13% done
********************* END NS.. 307 ****************************************************
********************* START NS ... 308 ****************************************************
[I 2025-06-15 20:06:48,181] A new study created in memory with name: no-name-7152239c-ba48-40ab-a83d-c63b7d29b862
[I 2025-06-15 20:06:48,484] Trial 0 finished with value: 0.005302333274198943 and parameters: {'n_estimators': 117, 'learning_rate': 0.21432829391676617, 'num_leaves': 75, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.6266600791589972, 'colsample_bytree': 0.9158812931752962, 'reg_alpha': 8.85638250037374, 'reg_lambda': 3.2467127284181387, 'min_split_gain': 0.03244890641415175}. Best is trial 0 with value: 0.005302333274198943.
[Trial 0] New best: 0.00530
[I 2025-06-15 20:06:49,503] Trial 1 finished with value: 0.004038845288705594 and parameters: {'n_estimators': 53, 'learning_rate': 0.2624455793590941, 'num_leaves': 112, 'max_depth': 15, 'min_child_samples': 3, 'subsample': 0.769719486548085, 'colsample_bytree': 0.9720049851832948, 'reg_alpha': 1.3904582090682205, 'reg_lambda': 6.928352645249155, 'min_split_gain': 0.0029304586935727074}. Best is trial 1 with value: 0.004038845288705594.
[Trial 1] New best: 0.00404
[I 2025-06-15 20:06:50,550] Trial 2 finished with value: 0.005072597744483819 and parameters: {'n_estimators': 106, 'learning_rate': 0.0625488020287969, 'num_leaves': 230, 'max_depth': 7, 'min_child_samples': 10, 'subsample': 0.6354065800449141, 'colsample_bytree': 0.6502325847048853, 'reg_alpha': 3.4352336952796625, 'reg_lambda': 8.762660638237099, 'min_split_gain': 0.009197220567020386}. Best is trial 1 with value: 0.004038845288705594.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:06:51,452] Trial 3 finished with value: 0.009426351236852086 and parameters: {'n_estimators': 56, 'learning_rate': 0.044586306507545026, 'num_leaves': 166, 'max_depth': 9, 'min_child_samples': 27, 'subsample': 0.7738974895450974, 'colsample_bytree': 0.6122289325519281, 'reg_alpha': 0.6571451905315306, 'reg_lambda': 9.177568074595698, 'min_split_gain': 0.012975350935348197}. Best is trial 1 with value: 0.004038845288705594.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:06:53,579] Trial 4 finished with value: 0.004786092689190612 and parameters: {'n_estimators': 228, 'learning_rate': 0.03517100188555229, 'num_leaves': 172, 'max_depth': 14, 'min_child_samples': 16, 'subsample': 0.7209443508170751, 'colsample_bytree': 0.8681947863263122, 'reg_alpha': 3.9967222555708393, 'reg_lambda': 4.302561203502716, 'min_split_gain': 0.03606535659253673}. Best is trial 1 with value: 0.004038845288705594.
[I 2025-06-15 20:06:53,581] A new study created in memory with name: no-name-ae9496cd-ee00-40ad-b1a1-69c4e993eeb9
[I 2025-06-15 20:06:53,711] Trial 0 finished with value: 0.029884355497373432 and parameters: {'n_estimators': 286, 'learning_rate': 0.017774175377810543, 'num_leaves': 191, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.7884225232320533, 'colsample_bytree': 0.9449724683200582, 'reg_alpha': 4.414017507774063, 'reg_lambda': 5.335925444837431, 'min_split_gain': 0.017782852702361265}. Best is trial 0 with value: 0.029884355497373432.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         697
14      pct_chg_Load_DA         670
5       WindOn_DA_lag_0         537
3         Load_DA_lag_0         485
1           Price_lag_2         475
2           Price_lag_7         442
15       lag168_Load_DA         408
16  roll2d_median_Price         312
11           NGas_lag_2         311
12            Oil_lag_2         244
13            EUA_lag_2         230
10           Coal_lag_2         185
9                  WD_7          13
7                  WD_1           8
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         256
3         Load_DA_lag_0          91
15       lag168_Load_DA          33
2           Price_lag_7          15
5       WindOn_DA_lag_0           8
1           Price_lag_2           7
12            Oil_lag_2           6
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 42.27% done
********************* END NS.. 308 ****************************************************
********************* START NS ... 309 ****************************************************
[I 2025-06-15 20:06:56,459] A new study created in memory with name: no-name-d808240a-a2b7-4278-a097-9cc1c799ed5c
[I 2025-06-15 20:06:57,562] Trial 0 finished with value: 0.0021663076640367454 and parameters: {'n_estimators': 101, 'learning_rate': 0.16338359655460302, 'num_leaves': 75, 'max_depth': 14, 'min_child_samples': 6, 'subsample': 0.8935892503946156, 'colsample_bytree': 0.8177147691882363, 'reg_alpha': 3.601736535072594, 'reg_lambda': 4.733089527960316, 'min_split_gain': 0.010008670712439644}. Best is trial 0 with value: 0.0021663076640367454.
[Trial 0] New best: 0.00217
[I 2025-06-15 20:06:58,936] Trial 1 finished with value: 0.003297929072177233 and parameters: {'n_estimators': 110, 'learning_rate': 0.026008646440077603, 'num_leaves': 186, 'max_depth': 15, 'min_child_samples': 20, 'subsample': 0.9815757480579378, 'colsample_bytree': 0.8597970801062988, 'reg_alpha': 4.793213028744071, 'reg_lambda': 2.23786394460064, 'min_split_gain': 0.034864556140220014}. Best is trial 0 with value: 0.0021663076640367454.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:06:59,678] Trial 2 finished with value: 0.0025302748066574894 and parameters: {'n_estimators': 278, 'learning_rate': 0.26210148423265694, 'num_leaves': 208, 'max_depth': 12, 'min_child_samples': 16, 'subsample': 0.8355800918256655, 'colsample_bytree': 0.6293281627031038, 'reg_alpha': 9.770052730510315, 'reg_lambda': 3.5958853876713848, 'min_split_gain': 0.004488279879939905}. Best is trial 0 with value: 0.0021663076640367454.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:07:00,559] Trial 3 finished with value: 0.002322860069042573 and parameters: {'n_estimators': 99, 'learning_rate': 0.08907467487009528, 'num_leaves': 252, 'max_depth': 14, 'min_child_samples': 13, 'subsample': 0.8471942025347415, 'colsample_bytree': 0.6489886693568501, 'reg_alpha': 7.049821658010595, 'reg_lambda': 6.4227300076272344, 'min_split_gain': 0.008656999174362147}. Best is trial 0 with value: 0.0021663076640367454.
[I 2025-06-15 20:07:00,561] A new study created in memory with name: no-name-9da1fccb-d113-4a54-81bc-64f62ee3236f
[I 2025-06-15 20:07:00,604] Trial 0 finished with value: 0.011346637830033151 and parameters: {'n_estimators': 64, 'learning_rate': 0.11406536254710499, 'num_leaves': 153, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.8861136466240971, 'colsample_bytree': 0.9569503688189958, 'reg_alpha': 2.810092057252349, 'reg_lambda': 1.9196021735406588, 'min_split_gain': 0.01838748114495879}. Best is trial 0 with value: 0.011346637830033151.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         489
14      pct_chg_Load_DA         425
1           Price_lag_2         391
3         Load_DA_lag_0         331
5       WindOn_DA_lag_0         243
2           Price_lag_7         223
11           NGas_lag_2         197
15       lag168_Load_DA         184
16  roll2d_median_Price         175
10           Coal_lag_2         144
13            EUA_lag_2         110
12            Oil_lag_2          89
9                  WD_7          18
7                  WD_1           9
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          47
3         Load_DA_lag_0          15
15       lag168_Load_DA          12
2           Price_lag_7           3
12            Oil_lag_2           2
16  roll2d_median_Price           2
1           Price_lag_2           1
5       WindOn_DA_lag_0           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 42.41% done
********************* END NS.. 309 ****************************************************
********************* START NS ... 310 ****************************************************
[I 2025-06-15 20:07:02,877] A new study created in memory with name: no-name-5104d96d-724f-4048-8a3e-44fab7204b8a
[I 2025-06-15 20:07:03,455] Trial 0 finished with value: 0.013910910173704459 and parameters: {'n_estimators': 70, 'learning_rate': 0.026372353235074444, 'num_leaves': 199, 'max_depth': 8, 'min_child_samples': 7, 'subsample': 0.5560937968268842, 'colsample_bytree': 0.9717566176462722, 'reg_alpha': 4.504564217727288, 'reg_lambda': 0.9118710506322258, 'min_split_gain': 0.03212108191683432}. Best is trial 0 with value: 0.013910910173704459.
[Trial 0] New best: 0.01391
[I 2025-06-15 20:07:04,138] Trial 1 finished with value: 0.010374603410024823 and parameters: {'n_estimators': 136, 'learning_rate': 0.06110618737758033, 'num_leaves': 142, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.721680517844188, 'colsample_bytree': 0.8099496320089075, 'reg_alpha': 7.858977711267478, 'reg_lambda': 2.9674899336601737, 'min_split_gain': 0.039220153553745495}. Best is trial 1 with value: 0.010374603410024823.
[Trial 1] New best: 0.01037
[I 2025-06-15 20:07:04,851] Trial 2 finished with value: 0.01043676974042348 and parameters: {'n_estimators': 139, 'learning_rate': 0.12612034545133186, 'num_leaves': 246, 'max_depth': 5, 'min_child_samples': 13, 'subsample': 0.9536110059631078, 'colsample_bytree': 0.5207110939023089, 'reg_alpha': 3.467270994927223, 'reg_lambda': 1.2552756535070642, 'min_split_gain': 0.0033036121108712627}. Best is trial 1 with value: 0.010374603410024823.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:07:05,433] Trial 3 finished with value: 0.009816170521659973 and parameters: {'n_estimators': 182, 'learning_rate': 0.11522056373964341, 'num_leaves': 83, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.5758588994303195, 'colsample_bytree': 0.9286552782458418, 'reg_alpha': 7.336947058014044, 'reg_lambda': 3.191135633648262, 'min_split_gain': 0.028256298583972963}. Best is trial 3 with value: 0.009816170521659973.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:07:05,944] Trial 4 finished with value: 0.06632545240747995 and parameters: {'n_estimators': 62, 'learning_rate': 0.01005339526928001, 'num_leaves': 187, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.9927643332642215, 'colsample_bytree': 0.6782437908765175, 'reg_alpha': 4.853671929262468, 'reg_lambda': 9.56556052349464, 'min_split_gain': 0.039614515640477264}. Best is trial 3 with value: 0.009816170521659973.
[I 2025-06-15 20:07:05,947] A new study created in memory with name: no-name-08354c07-dae2-4a74-9978-bb2409aff56f
[I 2025-06-15 20:07:06,025] Trial 0 finished with value: 0.05254193653804352 and parameters: {'n_estimators': 227, 'learning_rate': 0.02097507267197197, 'num_leaves': 33, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.8385294085723283, 'colsample_bytree': 0.9724715940157169, 'reg_alpha': 8.372656097717204, 'reg_lambda': 8.071997234316292, 'min_split_gain': 0.03156840146617908}. Best is trial 0 with value: 0.05254193653804352.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         435
14      pct_chg_Load_DA         256
1           Price_lag_2         188
3         Load_DA_lag_0         139
11           NGas_lag_2         120
2           Price_lag_7          98
16  roll2d_median_Price          94
15       lag168_Load_DA          77
10           Coal_lag_2          61
5       WindOn_DA_lag_0          59
13            EUA_lag_2          56
12            Oil_lag_2          35
9                  WD_7           6
7                  WD_1           4
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         159
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 42.54% done
********************* END NS.. 310 ****************************************************
********************* START NS ... 311 ****************************************************
[I 2025-06-15 20:07:08,159] A new study created in memory with name: no-name-7e181478-ef5e-4c4e-b04b-f644539d3117
[I 2025-06-15 20:07:08,977] Trial 0 finished with value: 0.017567907527161235 and parameters: {'n_estimators': 264, 'learning_rate': 0.09344798341586386, 'num_leaves': 228, 'max_depth': 10, 'min_child_samples': 27, 'subsample': 0.6706139243656456, 'colsample_bytree': 0.7860210349777295, 'reg_alpha': 9.178760900684953, 'reg_lambda': 8.711071277116805, 'min_split_gain': 0.010660427668186418}. Best is trial 0 with value: 0.017567907527161235.
[Trial 0] New best: 0.01757
[I 2025-06-15 20:07:10,081] Trial 1 finished with value: 0.024399510149217957 and parameters: {'n_estimators': 156, 'learning_rate': 0.02309489809712414, 'num_leaves': 73, 'max_depth': 10, 'min_child_samples': 28, 'subsample': 0.9649333386116185, 'colsample_bytree': 0.5434231376438967, 'reg_alpha': 9.5389884414546, 'reg_lambda': 1.1236402539997847, 'min_split_gain': 0.04354741048190781}. Best is trial 0 with value: 0.017567907527161235.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:07:10,493] Trial 2 finished with value: 0.016976627828998832 and parameters: {'n_estimators': 222, 'learning_rate': 0.24692470748585477, 'num_leaves': 203, 'max_depth': 13, 'min_child_samples': 2, 'subsample': 0.6366491163047738, 'colsample_bytree': 0.7692302317212921, 'reg_alpha': 7.299649151073673, 'reg_lambda': 0.11831704363346685, 'min_split_gain': 0.011558718907171129}. Best is trial 2 with value: 0.016976627828998832.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:07:12,844] Trial 3 finished with value: 0.019260974078505824 and parameters: {'n_estimators': 288, 'learning_rate': 0.01746076677894029, 'num_leaves': 32, 'max_depth': 14, 'min_child_samples': 18, 'subsample': 0.5024341667664999, 'colsample_bytree': 0.7486106046630059, 'reg_alpha': 5.02919925786324, 'reg_lambda': 0.9106219104381008, 'min_split_gain': 0.03700813036525396}. Best is trial 2 with value: 0.016976627828998832.
[I 2025-06-15 20:07:12,847] A new study created in memory with name: no-name-36d299e9-b83f-4463-b209-0be4fb836ddb
[I 2025-06-15 20:07:12,901] Trial 0 finished with value: 0.1910734538989327 and parameters: {'n_estimators': 175, 'learning_rate': 0.17188650804059155, 'num_leaves': 237, 'max_depth': 14, 'min_child_samples': 23, 'subsample': 0.5515417281855344, 'colsample_bytree': 0.6349802799419768, 'reg_alpha': 8.229365154645329, 'reg_lambda': 4.492452283867802, 'min_split_gain': 0.0015094021850612872}. Best is trial 0 with value: 0.1910734538989327.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         237
1           Price_lag_2         175
14      pct_chg_Load_DA         160
3         Load_DA_lag_0         123
11           NGas_lag_2          99
2           Price_lag_7          98
10           Coal_lag_2          89
16  roll2d_median_Price          89
5       WindOn_DA_lag_0          85
13            EUA_lag_2          71
15       lag168_Load_DA          69
12            Oil_lag_2          43
9                  WD_7           9
7                  WD_1           7
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          31
10           Coal_lag_2           5
15       lag168_Load_DA           2
1           Price_lag_2           2
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 42.68% done
********************* END NS.. 311 ****************************************************
********************* START NS ... 312 ****************************************************
[I 2025-06-15 20:07:14,827] A new study created in memory with name: no-name-1a043174-54e5-4788-bf61-63c0a03bdfb6
[I 2025-06-15 20:07:16,305] Trial 0 finished with value: 0.013418840101621789 and parameters: {'n_estimators': 154, 'learning_rate': 0.014720277246935238, 'num_leaves': 50, 'max_depth': 8, 'min_child_samples': 7, 'subsample': 0.6907789533661295, 'colsample_bytree': 0.5929208359514486, 'reg_alpha': 5.4699841327119305, 'reg_lambda': 4.413465822264666, 'min_split_gain': 0.016122267361745973}. Best is trial 0 with value: 0.013418840101621789.
[Trial 0] New best: 0.01342
[I 2025-06-15 20:07:21,931] Trial 1 finished with value: 0.007388946996070185 and parameters: {'n_estimators': 251, 'learning_rate': 0.07245005605653049, 'num_leaves': 125, 'max_depth': 11, 'min_child_samples': 7, 'subsample': 0.8979383541440844, 'colsample_bytree': 0.8080491581883753, 'reg_alpha': 0.2840733261406647, 'reg_lambda': 8.542724483597397, 'min_split_gain': 0.001418338182350415}. Best is trial 1 with value: 0.007388946996070185.
[Trial 1] New best: 0.00739
[I 2025-06-15 20:07:22,186] Trial 2 finished with value: 0.01214074577310182 and parameters: {'n_estimators': 65, 'learning_rate': 0.09276974243153797, 'num_leaves': 193, 'max_depth': 4, 'min_child_samples': 15, 'subsample': 0.6785124308431552, 'colsample_bytree': 0.6924564607332782, 'reg_alpha': 1.6037868724807014, 'reg_lambda': 9.58384232402031, 'min_split_gain': 0.04030450545015332}. Best is trial 1 with value: 0.007388946996070185.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:07:23,005] Trial 3 finished with value: 0.010248432827039166 and parameters: {'n_estimators': 254, 'learning_rate': 0.09186127032239484, 'num_leaves': 188, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.5470219190669552, 'colsample_bytree': 0.7403158320425887, 'reg_alpha': 2.7807432674717694, 'reg_lambda': 1.2956471186646212, 'min_split_gain': 0.045202528280433155}. Best is trial 1 with value: 0.007388946996070185.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:07:24,200] Trial 4 finished with value: 0.008413189300894007 and parameters: {'n_estimators': 116, 'learning_rate': 0.11981730723246352, 'num_leaves': 242, 'max_depth': 16, 'min_child_samples': 11, 'subsample': 0.7155566927894982, 'colsample_bytree': 0.9471673965625127, 'reg_alpha': 0.9004021945796581, 'reg_lambda': 6.958898098963458, 'min_split_gain': 0.01884032761836541}. Best is trial 1 with value: 0.007388946996070185.
[I 2025-06-15 20:07:24,203] A new study created in memory with name: no-name-5372b69d-0b21-4a31-96d2-ec2fe598a9e2
[I 2025-06-15 20:07:24,234] Trial 0 finished with value: 0.1501893582219653 and parameters: {'n_estimators': 69, 'learning_rate': 0.22388773933610012, 'num_leaves': 50, 'max_depth': 9, 'min_child_samples': 26, 'subsample': 0.8586371975846634, 'colsample_bytree': 0.8575963093955343, 'reg_alpha': 8.965269948284243, 'reg_lambda': 8.40737699999002, 'min_split_gain': 0.0006566363540082665}. Best is trial 0 with value: 0.1501893582219653.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA        3830
0           Price_lag_1        3514
1           Price_lag_2        3183
3         Load_DA_lag_0        3172
5       WindOn_DA_lag_0        3053
15       lag168_Load_DA        2725
2           Price_lag_7        2377
16  roll2d_median_Price        1883
11           NGas_lag_2        1568
12            Oil_lag_2        1465
10           Coal_lag_2        1374
13            EUA_lag_2        1361
9                  WD_7          77
7                  WD_1          67
8                  WD_6          32
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          24
15       lag168_Load_DA           4
13            EUA_lag_2           3
3         Load_DA_lag_0           1
9                  WD_7           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 42.82% done
********************* END NS.. 312 ****************************************************
********************* START NS ... 313 ****************************************************
[I 2025-06-15 20:07:30,918] A new study created in memory with name: no-name-b6697e64-9aa1-4a58-af22-c45afe13705b
[I 2025-06-15 20:07:31,991] Trial 0 finished with value: 0.004311982948855124 and parameters: {'n_estimators': 198, 'learning_rate': 0.031097403683101155, 'num_leaves': 96, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.7503515403705046, 'colsample_bytree': 0.5752493734421602, 'reg_alpha': 8.697171544101304, 'reg_lambda': 9.490500116084824, 'min_split_gain': 0.0035315578694560313}. Best is trial 0 with value: 0.004311982948855124.
[Trial 0] New best: 0.00431
[I 2025-06-15 20:07:34,599] Trial 1 finished with value: 0.004081422242161729 and parameters: {'n_estimators': 158, 'learning_rate': 0.03775269221329651, 'num_leaves': 74, 'max_depth': 11, 'min_child_samples': 19, 'subsample': 0.7776273003991762, 'colsample_bytree': 0.7806263909095097, 'reg_alpha': 1.0204818148240113, 'reg_lambda': 6.503509040970298, 'min_split_gain': 0.019775998042679783}. Best is trial 1 with value: 0.004081422242161729.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:07:36,916] Trial 2 finished with value: 0.00448592312605316 and parameters: {'n_estimators': 274, 'learning_rate': 0.014946993133871175, 'num_leaves': 34, 'max_depth': 16, 'min_child_samples': 13, 'subsample': 0.5609219110238552, 'colsample_bytree': 0.5641763725874273, 'reg_alpha': 4.301038320878768, 'reg_lambda': 2.4203579087356673, 'min_split_gain': 0.04149912100709719}. Best is trial 1 with value: 0.004081422242161729.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:07:38,298] Trial 3 finished with value: 0.003961523557957459 and parameters: {'n_estimators': 294, 'learning_rate': 0.02826284283124996, 'num_leaves': 36, 'max_depth': 5, 'min_child_samples': 23, 'subsample': 0.7355499684961105, 'colsample_bytree': 0.6617704234914158, 'reg_alpha': 5.835654367456065, 'reg_lambda': 9.675772085059073, 'min_split_gain': 0.04184064530225889}. Best is trial 3 with value: 0.003961523557957459.
[I 2025-06-15 20:07:38,301] A new study created in memory with name: no-name-2ed3b04f-2c39-4442-9bbd-c5b3839c0d0d
[I 2025-06-15 20:07:38,383] Trial 0 finished with value: 0.0058676370932487796 and parameters: {'n_estimators': 237, 'learning_rate': 0.2971570889389391, 'num_leaves': 109, 'max_depth': 6, 'min_child_samples': 9, 'subsample': 0.6473476579507877, 'colsample_bytree': 0.8066110915404898, 'reg_alpha': 0.6832495497449675, 'reg_lambda': 5.036792717772851, 'min_split_gain': 0.008062356179930659}. Best is trial 0 with value: 0.0058676370932487796.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1451
1           Price_lag_2         861
14      pct_chg_Load_DA         707
3         Load_DA_lag_0         452
11           NGas_lag_2         314
2           Price_lag_7         287
16  roll2d_median_Price         240
15       lag168_Load_DA         226
5       WindOn_DA_lag_0         207
13            EUA_lag_2         148
10           Coal_lag_2         138
12            Oil_lag_2          88
7                  WD_1          41
9                  WD_7          25
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          29
15       lag168_Load_DA          10
16  roll2d_median_Price           4
3         Load_DA_lag_0           3
13            EUA_lag_2           3
14      pct_chg_Load_DA           3
5       WindOn_DA_lag_0           2
2           Price_lag_7           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
-> 42.95% done
********************* END NS.. 313 ****************************************************
********************* START NS ... 314 ****************************************************
[I 2025-06-15 20:07:41,385] A new study created in memory with name: no-name-6dd293e8-98ad-4f89-97b0-48e270dad864
[I 2025-06-15 20:07:44,576] Trial 0 finished with value: 0.0029170987627994555 and parameters: {'n_estimators': 297, 'learning_rate': 0.01352875545195025, 'num_leaves': 62, 'max_depth': 8, 'min_child_samples': 4, 'subsample': 0.8731054174718382, 'colsample_bytree': 0.9024179946449288, 'reg_alpha': 5.067053818496792, 'reg_lambda': 8.130263751661195, 'min_split_gain': 0.005948220376648972}. Best is trial 0 with value: 0.0029170987627994555.
[Trial 0] New best: 0.00292
[I 2025-06-15 20:07:45,588] Trial 1 finished with value: 0.010924002212165477 and parameters: {'n_estimators': 70, 'learning_rate': 0.012803360412658471, 'num_leaves': 68, 'max_depth': 11, 'min_child_samples': 24, 'subsample': 0.5881208886695598, 'colsample_bytree': 0.5486427801741637, 'reg_alpha': 0.5175996735474009, 'reg_lambda': 5.251969958199946, 'min_split_gain': 0.008518867382891383}. Best is trial 0 with value: 0.0029170987627994555.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:07:46,260] Trial 2 finished with value: 0.0029455446081559818 and parameters: {'n_estimators': 166, 'learning_rate': 0.06211946987931586, 'num_leaves': 190, 'max_depth': 5, 'min_child_samples': 22, 'subsample': 0.9225754233171135, 'colsample_bytree': 0.6803510164187037, 'reg_alpha': 8.88997803644982, 'reg_lambda': 4.417906944422669, 'min_split_gain': 0.011634635527711901}. Best is trial 0 with value: 0.0029170987627994555.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:07:47,829] Trial 3 finished with value: 0.0030365521500723067 and parameters: {'n_estimators': 267, 'learning_rate': 0.05060016045518062, 'num_leaves': 225, 'max_depth': 15, 'min_child_samples': 28, 'subsample': 0.6186487828655578, 'colsample_bytree': 0.8728658932014175, 'reg_alpha': 2.386304760585023, 'reg_lambda': 6.973566711643274, 'min_split_gain': 0.029547088081585855}. Best is trial 0 with value: 0.0029170987627994555.
[I 2025-06-15 20:07:47,833] A new study created in memory with name: no-name-9eab56ea-5730-4d2e-ac41-f125b300426e
[I 2025-06-15 20:07:47,881] Trial 0 finished with value: 0.006204252404291035 and parameters: {'n_estimators': 136, 'learning_rate': 0.19683404038766716, 'num_leaves': 98, 'max_depth': 7, 'min_child_samples': 13, 'subsample': 0.9572779071661621, 'colsample_bytree': 0.8527028055623356, 'reg_alpha': 3.174214702521174, 'reg_lambda': 4.234266134369216, 'min_split_gain': 0.0349750290967517}. Best is trial 0 with value: 0.006204252404291035.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3830
1           Price_lag_2        2552
14      pct_chg_Load_DA        1928
16  roll2d_median_Price        1187
2           Price_lag_7        1157
11           NGas_lag_2        1119
3         Load_DA_lag_0         881
10           Coal_lag_2         687
13            EUA_lag_2         588
5       WindOn_DA_lag_0         577
15       lag168_Load_DA         472
12            Oil_lag_2         399
9                  WD_7          39
8                  WD_6          12
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1          24
15       lag168_Load_DA           3
16  roll2d_median_Price           1
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 43.09% done
********************* END NS.. 314 ****************************************************
********************* START NS ... 315 ****************************************************
[I 2025-06-15 20:07:52,379] A new study created in memory with name: no-name-14260ac9-f1cf-4af3-a78b-11b198e9a67f
[I 2025-06-15 20:07:53,327] Trial 0 finished with value: 0.0034117143600869595 and parameters: {'n_estimators': 223, 'learning_rate': 0.15208199114955642, 'num_leaves': 130, 'max_depth': 14, 'min_child_samples': 5, 'subsample': 0.5439350961348055, 'colsample_bytree': 0.5442264667426446, 'reg_alpha': 7.04513302176355, 'reg_lambda': 6.140445831298072, 'min_split_gain': 0.005991647405753592}. Best is trial 0 with value: 0.0034117143600869595.
[Trial 0] New best: 0.00341
[I 2025-06-15 20:07:54,084] Trial 1 finished with value: 0.003611386668715358 and parameters: {'n_estimators': 106, 'learning_rate': 0.06745831331610704, 'num_leaves': 173, 'max_depth': 12, 'min_child_samples': 20, 'subsample': 0.6105327706773762, 'colsample_bytree': 0.8254934536049883, 'reg_alpha': 5.457317241910973, 'reg_lambda': 5.156093854557577, 'min_split_gain': 0.03268379277535231}. Best is trial 0 with value: 0.0034117143600869595.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:07:54,586] Trial 2 finished with value: 0.004020438374263565 and parameters: {'n_estimators': 201, 'learning_rate': 0.16632431162652175, 'num_leaves': 226, 'max_depth': 16, 'min_child_samples': 27, 'subsample': 0.8122368194906375, 'colsample_bytree': 0.8665353381527838, 'reg_alpha': 9.075208116120525, 'reg_lambda': 6.077815126930806, 'min_split_gain': 0.006865769021479684}. Best is trial 0 with value: 0.0034117143600869595.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:07:55,241] Trial 3 finished with value: 0.0038338363005890313 and parameters: {'n_estimators': 69, 'learning_rate': 0.03565814272973453, 'num_leaves': 40, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.9309870127694103, 'colsample_bytree': 0.8412937954758417, 'reg_alpha': 1.9496211176654388, 'reg_lambda': 5.15989482487269, 'min_split_gain': 0.011052990727589957}. Best is trial 0 with value: 0.0034117143600869595.
[I 2025-06-15 20:07:55,245] A new study created in memory with name: no-name-c366a976-bb19-4c93-b991-b812baa365e0
[I 2025-06-15 20:07:55,314] Trial 0 finished with value: 0.0164402519955754 and parameters: {'n_estimators': 126, 'learning_rate': 0.027765679196078946, 'num_leaves': 213, 'max_depth': 13, 'min_child_samples': 3, 'subsample': 0.6648279092944447, 'colsample_bytree': 0.6467921286692531, 'reg_alpha': 5.140164357264122, 'reg_lambda': 2.0858168061836224, 'min_split_gain': 0.048369810371917114}. Best is trial 0 with value: 0.0164402519955754.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         644
1           Price_lag_2         573
16  roll2d_median_Price         290
3         Load_DA_lag_0         286
2           Price_lag_7         278
14      pct_chg_Load_DA         276
11           NGas_lag_2         236
5       WindOn_DA_lag_0         224
13            EUA_lag_2         199
15       lag168_Load_DA         169
12            Oil_lag_2         164
10           Coal_lag_2         147
7                  WD_1          17
9                  WD_7          17
8                  WD_6          17
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          92
1           Price_lag_2          15
16  roll2d_median_Price          11
3         Load_DA_lag_0           4
15       lag168_Load_DA           4
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
2           Price_lag_7           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 43.23% done
********************* END NS.. 315 ****************************************************
********************* START NS ... 316 ****************************************************
[I 2025-06-15 20:07:58,016] A new study created in memory with name: no-name-ec8a601b-888d-495c-b09e-ac2217a10374
[I 2025-06-15 20:07:58,451] Trial 0 finished with value: 0.007541765312011679 and parameters: {'n_estimators': 66, 'learning_rate': 0.01392858072505416, 'num_leaves': 41, 'max_depth': 14, 'min_child_samples': 18, 'subsample': 0.8411770397439275, 'colsample_bytree': 0.9243302218188412, 'reg_alpha': 8.773458639964344, 'reg_lambda': 9.660838934877713, 'min_split_gain': 0.03284212232836588}. Best is trial 0 with value: 0.007541765312011679.
[Trial 0] New best: 0.00754
[I 2025-06-15 20:08:00,669] Trial 1 finished with value: 0.005763687698649586 and parameters: {'n_estimators': 280, 'learning_rate': 0.010126261375206847, 'num_leaves': 87, 'max_depth': 12, 'min_child_samples': 20, 'subsample': 0.586060295794632, 'colsample_bytree': 0.5637625018812682, 'reg_alpha': 7.071561393653155, 'reg_lambda': 4.863934685006491, 'min_split_gain': 0.033364351470950813}. Best is trial 1 with value: 0.005763687698649586.
[Trial 1] New best: 0.00576
[I 2025-06-15 20:08:01,353] Trial 2 finished with value: 0.003746010284755512 and parameters: {'n_estimators': 159, 'learning_rate': 0.13554667282856253, 'num_leaves': 251, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.8728731431935634, 'colsample_bytree': 0.581494071452981, 'reg_alpha': 1.6364148883297591, 'reg_lambda': 1.8079234553075862, 'min_split_gain': 0.03870191277104601}. Best is trial 2 with value: 0.003746010284755512.
[Trial 2] New best: 0.00375
[I 2025-06-15 20:08:01,886] Trial 3 finished with value: 0.005598003226994055 and parameters: {'n_estimators': 173, 'learning_rate': 0.15459798711872527, 'num_leaves': 246, 'max_depth': 4, 'min_child_samples': 3, 'subsample': 0.8488536263565142, 'colsample_bytree': 0.6521783758763937, 'reg_alpha': 0.4423363623761234, 'reg_lambda': 4.202066250327006, 'min_split_gain': 0.007256553723313047}. Best is trial 2 with value: 0.003746010284755512.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:08:04,087] Trial 4 finished with value: 0.004325700728019434 and parameters: {'n_estimators': 174, 'learning_rate': 0.025690992773140003, 'num_leaves': 155, 'max_depth': 15, 'min_child_samples': 22, 'subsample': 0.5492858139343011, 'colsample_bytree': 0.8687786781957987, 'reg_alpha': 5.116838405600443, 'reg_lambda': 1.9924886103877404, 'min_split_gain': 0.00013214206473451397}. Best is trial 2 with value: 0.003746010284755512.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:08:04,504] Trial 5 finished with value: 0.006876210160828601 and parameters: {'n_estimators': 125, 'learning_rate': 0.013953914884551529, 'num_leaves': 75, 'max_depth': 4, 'min_child_samples': 2, 'subsample': 0.5492301806482254, 'colsample_bytree': 0.7744712682163659, 'reg_alpha': 1.5228891634989972, 'reg_lambda': 4.512630493185327, 'min_split_gain': 0.014885984547055864}. Best is trial 2 with value: 0.003746010284755512.
[I 2025-06-15 20:08:04,506] A new study created in memory with name: no-name-9acbbd75-360a-4e64-956f-c47a02b0b164
[I 2025-06-15 20:08:04,537] Trial 0 finished with value: 0.017573403892863982 and parameters: {'n_estimators': 55, 'learning_rate': 0.09725310966116014, 'num_leaves': 227, 'max_depth': 15, 'min_child_samples': 30, 'subsample': 0.5710088806271555, 'colsample_bytree': 0.956601810319816, 'reg_alpha': 3.6709297790945374, 'reg_lambda': 1.73347519895932, 'min_split_gain': 0.022432303944668178}. Best is trial 0 with value: 0.017573403892863982.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         474
1           Price_lag_2         416
14      pct_chg_Load_DA         306
3         Load_DA_lag_0         241
2           Price_lag_7         212
11           NGas_lag_2         155
16  roll2d_median_Price         155
5       WindOn_DA_lag_0         143
15       lag168_Load_DA         130
13            EUA_lag_2         129
10           Coal_lag_2         128
12            Oil_lag_2          79
9                  WD_7          11
7                  WD_1           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          36
3         Load_DA_lag_0           3
1           Price_lag_2           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 43.37% done
********************* END NS.. 316 ****************************************************
********************* START NS ... 317 ****************************************************
[I 2025-06-15 20:08:06,712] A new study created in memory with name: no-name-25038693-4619-476c-8a28-c87ccde2df42
[I 2025-06-15 20:08:07,396] Trial 0 finished with value: 0.0050870789085182945 and parameters: {'n_estimators': 158, 'learning_rate': 0.025458192318569738, 'num_leaves': 71, 'max_depth': 5, 'min_child_samples': 11, 'subsample': 0.8526414607063437, 'colsample_bytree': 0.7673958628624462, 'reg_alpha': 5.711746963821889, 'reg_lambda': 6.057219887567028, 'min_split_gain': 0.042492308314767545}. Best is trial 0 with value: 0.0050870789085182945.
[Trial 0] New best: 0.00509
[I 2025-06-15 20:08:08,132] Trial 1 finished with value: 0.004717456347499218 and parameters: {'n_estimators': 251, 'learning_rate': 0.018558882993029067, 'num_leaves': 40, 'max_depth': 4, 'min_child_samples': 18, 'subsample': 0.9182152147395058, 'colsample_bytree': 0.9364721633012031, 'reg_alpha': 2.5240616836622207, 'reg_lambda': 6.04826231205967, 'min_split_gain': 0.029156577107166654}. Best is trial 1 with value: 0.004717456347499218.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:08:10,132] Trial 2 finished with value: 0.005678059080531022 and parameters: {'n_estimators': 295, 'learning_rate': 0.027803521546510754, 'num_leaves': 204, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.8579605050357034, 'colsample_bytree': 0.5522488238289633, 'reg_alpha': 4.777761787471672, 'reg_lambda': 7.890779575405872, 'min_split_gain': 0.045836738116818106}. Best is trial 1 with value: 0.004717456347499218.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:08:10,616] Trial 3 finished with value: 0.00508826805676385 and parameters: {'n_estimators': 274, 'learning_rate': 0.16999857423143785, 'num_leaves': 146, 'max_depth': 15, 'min_child_samples': 14, 'subsample': 0.7349073487860309, 'colsample_bytree': 0.5914477158653406, 'reg_alpha': 5.723715724455916, 'reg_lambda': 2.9722758536750438, 'min_split_gain': 0.048308616595287096}. Best is trial 1 with value: 0.004717456347499218.
[I 2025-06-15 20:08:10,617] A new study created in memory with name: no-name-f20a6367-db75-4742-be22-595b692b16db
[I 2025-06-15 20:08:10,691] Trial 0 finished with value: 0.025472165551296787 and parameters: {'n_estimators': 261, 'learning_rate': 0.29274442646345117, 'num_leaves': 234, 'max_depth': 10, 'min_child_samples': 23, 'subsample': 0.6397692898240344, 'colsample_bytree': 0.5964752383270875, 'reg_alpha': 7.080683273613415, 'reg_lambda': 4.967721576875725, 'min_split_gain': 0.04449714910626835}. Best is trial 0 with value: 0.025472165551296787.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1764
14      pct_chg_Load_DA         643
1           Price_lag_2         269
16  roll2d_median_Price         235
2           Price_lag_7         192
3         Load_DA_lag_0         149
11           NGas_lag_2         146
13            EUA_lag_2          78
15       lag168_Load_DA          54
12            Oil_lag_2          48
10           Coal_lag_2          31
5       WindOn_DA_lag_0          25
9                  WD_7          10
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 43.50% done
********************* END NS.. 317 ****************************************************
********************* START NS ... 318 ****************************************************
[I 2025-06-15 20:08:13,186] A new study created in memory with name: no-name-55930f7c-8cc4-433c-a3fc-7b2e27dcdb84
[I 2025-06-15 20:08:13,527] Trial 0 finished with value: 0.0032720582872980825 and parameters: {'n_estimators': 167, 'learning_rate': 0.17644426060956758, 'num_leaves': 244, 'max_depth': 16, 'min_child_samples': 3, 'subsample': 0.9785285813040546, 'colsample_bytree': 0.5712821569835853, 'reg_alpha': 6.996415727602004, 'reg_lambda': 5.355624985352084, 'min_split_gain': 0.04523007462815065}. Best is trial 0 with value: 0.0032720582872980825.
[Trial 0] New best: 0.00327
[I 2025-06-15 20:08:14,852] Trial 1 finished with value: 0.004127040506158457 and parameters: {'n_estimators': 235, 'learning_rate': 0.03506939085125239, 'num_leaves': 240, 'max_depth': 8, 'min_child_samples': 20, 'subsample': 0.6830578111112082, 'colsample_bytree': 0.5774834938555538, 'reg_alpha': 6.9723335106331685, 'reg_lambda': 1.722191519190891, 'min_split_gain': 0.040950354005674636}. Best is trial 0 with value: 0.0032720582872980825.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:08:15,489] Trial 2 finished with value: 0.004921263030772135 and parameters: {'n_estimators': 254, 'learning_rate': 0.1651062777616328, 'num_leaves': 157, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.5823393318317239, 'colsample_bytree': 0.5294362784378757, 'reg_alpha': 2.676730255252, 'reg_lambda': 1.332687216319124, 'min_split_gain': 0.02938246514634148}. Best is trial 0 with value: 0.0032720582872980825.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:08:15,869] Trial 3 finished with value: 0.002718825865933206 and parameters: {'n_estimators': 247, 'learning_rate': 0.29143966444315117, 'num_leaves': 196, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.7433824966025502, 'colsample_bytree': 0.6695027908364364, 'reg_alpha': 5.233928489874864, 'reg_lambda': 9.513166900831731, 'min_split_gain': 0.04874097014087449}. Best is trial 3 with value: 0.002718825865933206.
[I 2025-06-15 20:08:15,871] A new study created in memory with name: no-name-dd838ef1-ba71-4679-ba1c-8415809bff20
[I 2025-06-15 20:08:15,905] Trial 0 finished with value: 0.030554176740469657 and parameters: {'n_estimators': 88, 'learning_rate': 0.020639008517315656, 'num_leaves': 32, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.8920229792699772, 'colsample_bytree': 0.622585689389513, 'reg_alpha': 4.835726740417769, 'reg_lambda': 3.9140697152961215, 'min_split_gain': 0.045633412394291084}. Best is trial 0 with value: 0.030554176740469657.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         165
1           Price_lag_2          90
14      pct_chg_Load_DA          66
3         Load_DA_lag_0          52
11           NGas_lag_2          49
16  roll2d_median_Price          32
2           Price_lag_7          32
5       WindOn_DA_lag_0          26
10           Coal_lag_2          19
15       lag168_Load_DA          19
13            EUA_lag_2          14
12            Oil_lag_2           9
9                  WD_7           3
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          47
1           Price_lag_2          13
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 43.64% done
********************* END NS.. 318 ****************************************************
********************* START NS ... 319 ****************************************************
[I 2025-06-15 20:08:18,040] A new study created in memory with name: no-name-d456553f-2ba2-4b71-b62e-a98150fc2bf4
[I 2025-06-15 20:08:18,571] Trial 0 finished with value: 0.0035534588830697564 and parameters: {'n_estimators': 91, 'learning_rate': 0.033236332414572835, 'num_leaves': 215, 'max_depth': 10, 'min_child_samples': 7, 'subsample': 0.8048774889477531, 'colsample_bytree': 0.9308099533511954, 'reg_alpha': 7.7167785454474025, 'reg_lambda': 7.251754690239572, 'min_split_gain': 0.044859686424815706}. Best is trial 0 with value: 0.0035534588830697564.
[Trial 0] New best: 0.00355
[I 2025-06-15 20:08:19,293] Trial 1 finished with value: 0.002443383479475879 and parameters: {'n_estimators': 168, 'learning_rate': 0.10755545956260514, 'num_leaves': 84, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.9211954307934255, 'colsample_bytree': 0.6372914769287724, 'reg_alpha': 3.6847747542627163, 'reg_lambda': 6.295001767101994, 'min_split_gain': 0.018717419544010377}. Best is trial 1 with value: 0.002443383479475879.
[Trial 1] New best: 0.00244
[I 2025-06-15 20:08:19,576] Trial 2 finished with value: 0.002765493299622135 and parameters: {'n_estimators': 176, 'learning_rate': 0.24089652605109832, 'num_leaves': 188, 'max_depth': 6, 'min_child_samples': 29, 'subsample': 0.7283488463053913, 'colsample_bytree': 0.8016273678864927, 'reg_alpha': 6.457266951217042, 'reg_lambda': 3.698985837248535, 'min_split_gain': 0.03298957830471794}. Best is trial 1 with value: 0.002443383479475879.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:08:20,093] Trial 3 finished with value: 0.00368848774567807 and parameters: {'n_estimators': 154, 'learning_rate': 0.04610461812370136, 'num_leaves': 87, 'max_depth': 4, 'min_child_samples': 21, 'subsample': 0.7094969859283733, 'colsample_bytree': 0.6855066047101266, 'reg_alpha': 5.0775831014702995, 'reg_lambda': 1.8668526710118494, 'min_split_gain': 0.04343615437581657}. Best is trial 1 with value: 0.002443383479475879.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:08:21,577] Trial 4 finished with value: 0.0030258842470388242 and parameters: {'n_estimators': 195, 'learning_rate': 0.03160457389131584, 'num_leaves': 206, 'max_depth': 10, 'min_child_samples': 15, 'subsample': 0.9413929563764917, 'colsample_bytree': 0.6882581436282215, 'reg_alpha': 5.831709999161365, 'reg_lambda': 5.398360221809214, 'min_split_gain': 0.0374728428414076}. Best is trial 1 with value: 0.002443383479475879.
[I 2025-06-15 20:08:21,580] A new study created in memory with name: no-name-e598b7a5-5a30-4768-b548-34ff758a011d
[I 2025-06-15 20:08:21,643] Trial 0 finished with value: 0.046762967750368666 and parameters: {'n_estimators': 286, 'learning_rate': 0.0554906075210321, 'num_leaves': 116, 'max_depth': 11, 'min_child_samples': 23, 'subsample': 0.5949324386462123, 'colsample_bytree': 0.7630862580089405, 'reg_alpha': 8.570526884985247, 'reg_lambda': 4.620706624743583, 'min_split_gain': 0.00438664888785027}. Best is trial 0 with value: 0.046762967750368666.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         599
1           Price_lag_2         488
14      pct_chg_Load_DA         385
3         Load_DA_lag_0         325
11           NGas_lag_2         262
5       WindOn_DA_lag_0         252
15       lag168_Load_DA         229
2           Price_lag_7         224
16  roll2d_median_Price         203
10           Coal_lag_2         147
13            EUA_lag_2         139
12            Oil_lag_2          90
9                  WD_7          20
7                  WD_1          12
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          21
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 43.78% done
********************* END NS.. 319 ****************************************************
********************* START NS ... 320 ****************************************************
[I 2025-06-15 20:08:24,092] A new study created in memory with name: no-name-5106d5b3-dc18-4e3e-81f5-c1287a7a670b
[I 2025-06-15 20:08:25,936] Trial 0 finished with value: 0.008675063846178157 and parameters: {'n_estimators': 251, 'learning_rate': 0.03069362484221121, 'num_leaves': 123, 'max_depth': 6, 'min_child_samples': 12, 'subsample': 0.6435288873176079, 'colsample_bytree': 0.8117797802304817, 'reg_alpha': 3.996175166397349, 'reg_lambda': 7.336269117395673, 'min_split_gain': 0.017666997412993914}. Best is trial 0 with value: 0.008675063846178157.
[Trial 0] New best: 0.00868
[I 2025-06-15 20:08:26,470] Trial 1 finished with value: 0.010797801787441013 and parameters: {'n_estimators': 72, 'learning_rate': 0.04130071369511369, 'num_leaves': 174, 'max_depth': 7, 'min_child_samples': 26, 'subsample': 0.7515987623807268, 'colsample_bytree': 0.8243759653417957, 'reg_alpha': 5.807078328805816, 'reg_lambda': 9.50961867316287, 'min_split_gain': 0.016057236121692225}. Best is trial 0 with value: 0.008675063846178157.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:08:27,107] Trial 2 finished with value: 0.009696827077570053 and parameters: {'n_estimators': 139, 'learning_rate': 0.06824963028794338, 'num_leaves': 92, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.9309968550173733, 'colsample_bytree': 0.9145011360772772, 'reg_alpha': 0.5084632952219581, 'reg_lambda': 6.383053427224769, 'min_split_gain': 0.04459953154943824}. Best is trial 0 with value: 0.008675063846178157.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:08:27,585] Trial 3 finished with value: 0.00939781509359028 and parameters: {'n_estimators': 109, 'learning_rate': 0.2147217609534114, 'num_leaves': 171, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.5559631460078214, 'colsample_bytree': 0.673892169976328, 'reg_alpha': 0.7858972888626103, 'reg_lambda': 9.03807543525344, 'min_split_gain': 0.03172618620739489}. Best is trial 0 with value: 0.008675063846178157.
[I 2025-06-15 20:08:27,588] A new study created in memory with name: no-name-1dd8f71d-1d82-4c47-8053-46d0ba5c519f
[I 2025-06-15 20:08:27,619] Trial 0 finished with value: 0.13047793921969128 and parameters: {'n_estimators': 96, 'learning_rate': 0.019031316249677445, 'num_leaves': 170, 'max_depth': 8, 'min_child_samples': 17, 'subsample': 0.8009478915989559, 'colsample_bytree': 0.6365417853284911, 'reg_alpha': 5.153311791047473, 'reg_lambda': 9.562889063976765, 'min_split_gain': 0.043192598313678776}. Best is trial 0 with value: 0.13047793921969128.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1834
1           Price_lag_2        1274
14      pct_chg_Load_DA        1091
3         Load_DA_lag_0         668
11           NGas_lag_2         568
16  roll2d_median_Price         436
2           Price_lag_7         418
15       lag168_Load_DA         388
13            EUA_lag_2         348
5       WindOn_DA_lag_0         337
10           Coal_lag_2         256
12            Oil_lag_2         249
7                  WD_1          35
9                  WD_7          20
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          80
1           Price_lag_2          28
5       WindOn_DA_lag_0           8
3         Load_DA_lag_0           4
15       lag168_Load_DA           3
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 43.91% done
********************* END NS.. 320 ****************************************************
********************* START NS ... 321 ****************************************************
[I 2025-06-15 20:08:31,197] A new study created in memory with name: no-name-19589595-4c7b-4137-992f-a0ab8146e79e
[I 2025-06-15 20:08:34,977] Trial 0 finished with value: 0.00907148914769308 and parameters: {'n_estimators': 184, 'learning_rate': 0.022143376541967328, 'num_leaves': 161, 'max_depth': 9, 'min_child_samples': 8, 'subsample': 0.9492037683082558, 'colsample_bytree': 0.9829739468664951, 'reg_alpha': 0.15518283802929633, 'reg_lambda': 6.588640107253406, 'min_split_gain': 0.02207456533678579}. Best is trial 0 with value: 0.00907148914769308.
[Trial 0] New best: 0.00907
[I 2025-06-15 20:08:35,670] Trial 1 finished with value: 0.01006663672212689 and parameters: {'n_estimators': 230, 'learning_rate': 0.022310749647775543, 'num_leaves': 103, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.7115111202324704, 'colsample_bytree': 0.9191431068351001, 'reg_alpha': 8.185588983367445, 'reg_lambda': 5.638016674252622, 'min_split_gain': 0.004088939210409997}. Best is trial 0 with value: 0.00907148914769308.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:08:36,083] Trial 2 finished with value: 0.007719575509400505 and parameters: {'n_estimators': 224, 'learning_rate': 0.15502038480714253, 'num_leaves': 241, 'max_depth': 6, 'min_child_samples': 13, 'subsample': 0.9614717969554134, 'colsample_bytree': 0.9085660265777495, 'reg_alpha': 5.870642262884306, 'reg_lambda': 8.653355765686785, 'min_split_gain': 0.025750814717774413}. Best is trial 2 with value: 0.007719575509400505.
[Trial 2] New best: 0.00772
[I 2025-06-15 20:08:36,599] Trial 3 finished with value: 0.007989501123146019 and parameters: {'n_estimators': 67, 'learning_rate': 0.06209735791691019, 'num_leaves': 33, 'max_depth': 9, 'min_child_samples': 21, 'subsample': 0.6092673644080981, 'colsample_bytree': 0.9293121965218325, 'reg_alpha': 4.756353984221407, 'reg_lambda': 0.749956564119324, 'min_split_gain': 0.03166912057199267}. Best is trial 2 with value: 0.007719575509400505.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:08:38,265] Trial 4 finished with value: 0.011152960508132649 and parameters: {'n_estimators': 209, 'learning_rate': 0.013138361208977961, 'num_leaves': 171, 'max_depth': 10, 'min_child_samples': 12, 'subsample': 0.6205455155598001, 'colsample_bytree': 0.7826494972037124, 'reg_alpha': 9.637411276889656, 'reg_lambda': 8.971691507419667, 'min_split_gain': 0.023230854444405726}. Best is trial 2 with value: 0.007719575509400505.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:08:39,946] Trial 5 finished with value: 0.011448112182774778 and parameters: {'n_estimators': 184, 'learning_rate': 0.011188966320069379, 'num_leaves': 197, 'max_depth': 10, 'min_child_samples': 29, 'subsample': 0.7189252143944705, 'colsample_bytree': 0.9591288313762649, 'reg_alpha': 8.837501269581582, 'reg_lambda': 0.6366461076882124, 'min_split_gain': 0.016165939716625638}. Best is trial 2 with value: 0.007719575509400505.
[I 2025-06-15 20:08:39,948] A new study created in memory with name: no-name-1afca323-d392-4c5c-9794-20705c1c569c
[I 2025-06-15 20:08:40,014] Trial 0 finished with value: 0.1426055094536823 and parameters: {'n_estimators': 191, 'learning_rate': 0.11642693474998157, 'num_leaves': 57, 'max_depth': 13, 'min_child_samples': 11, 'subsample': 0.55283220525312, 'colsample_bytree': 0.5476124927812296, 'reg_alpha': 9.418603060589518, 'reg_lambda': 2.43386888066911, 'min_split_gain': 0.04361617440590836}. Best is trial 0 with value: 0.1426055094536823.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         330
14      pct_chg_Load_DA         186
1           Price_lag_2         149
3         Load_DA_lag_0         107
11           NGas_lag_2         104
2           Price_lag_7          90
16  roll2d_median_Price          79
5       WindOn_DA_lag_0          66
10           Coal_lag_2          55
13            EUA_lag_2          54
15       lag168_Load_DA          52
12            Oil_lag_2          34
7                  WD_1           8
9                  WD_7           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          12
1           Price_lag_2           2
15       lag168_Load_DA           1
16  roll2d_median_Price           1
3         Load_DA_lag_0           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 44.05% done
********************* END NS.. 321 ****************************************************
********************* START NS ... 322 ****************************************************
[I 2025-06-15 20:08:41,881] A new study created in memory with name: no-name-f0c50808-6063-4924-8293-011a98da92e2
[I 2025-06-15 20:08:42,458] Trial 0 finished with value: 0.0035610123893018465 and parameters: {'n_estimators': 57, 'learning_rate': 0.25758194523343114, 'num_leaves': 166, 'max_depth': 13, 'min_child_samples': 24, 'subsample': 0.8372030841741148, 'colsample_bytree': 0.7412627637294593, 'reg_alpha': 4.6040985889718105, 'reg_lambda': 0.9040941555088056, 'min_split_gain': 0.0011860025845108592}. Best is trial 0 with value: 0.0035610123893018465.
[Trial 0] New best: 0.00356
[I 2025-06-15 20:08:43,191] Trial 1 finished with value: 0.004817875588506537 and parameters: {'n_estimators': 241, 'learning_rate': 0.03319381711533534, 'num_leaves': 73, 'max_depth': 4, 'min_child_samples': 13, 'subsample': 0.8172140622340249, 'colsample_bytree': 0.5793809219632944, 'reg_alpha': 1.5285705884242318, 'reg_lambda': 6.642012308139793, 'min_split_gain': 0.029419777859019405}. Best is trial 0 with value: 0.0035610123893018465.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:08:43,998] Trial 2 finished with value: 0.0055535195039197105 and parameters: {'n_estimators': 172, 'learning_rate': 0.011489091377913476, 'num_leaves': 90, 'max_depth': 5, 'min_child_samples': 9, 'subsample': 0.7529469486096267, 'colsample_bytree': 0.796275707332796, 'reg_alpha': 5.691573777672778, 'reg_lambda': 8.881886886860668, 'min_split_gain': 0.034631671905807314}. Best is trial 0 with value: 0.0035610123893018465.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:08:44,270] Trial 3 finished with value: 0.0034955487834463484 and parameters: {'n_estimators': 67, 'learning_rate': 0.28376295024573134, 'num_leaves': 232, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.5477678143407229, 'colsample_bytree': 0.654577469604579, 'reg_alpha': 8.009552785312799, 'reg_lambda': 4.717461984555644, 'min_split_gain': 0.018361772915912407}. Best is trial 3 with value: 0.0034955487834463484.
[I 2025-06-15 20:08:44,273] A new study created in memory with name: no-name-2e79a2ed-d4bd-4c21-ba84-449067d7c89e
[I 2025-06-15 20:08:44,318] Trial 0 finished with value: 0.029070579277731823 and parameters: {'n_estimators': 181, 'learning_rate': 0.25507720517981314, 'num_leaves': 78, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.7114344928887022, 'colsample_bytree': 0.7129424774775333, 'reg_alpha': 5.4832882904782965, 'reg_lambda': 7.588969475016764, 'min_split_gain': 0.035807986600061986}. Best is trial 0 with value: 0.029070579277731823.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         199
1           Price_lag_2         102
14      pct_chg_Load_DA          82
11           NGas_lag_2          81
3         Load_DA_lag_0          78
2           Price_lag_7          68
10           Coal_lag_2          47
16  roll2d_median_Price          46
5       WindOn_DA_lag_0          41
15       lag168_Load_DA          41
13            EUA_lag_2          31
12            Oil_lag_2          15
9                  WD_7           6
8                  WD_6           4
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          12
15       lag168_Load_DA           1
3         Load_DA_lag_0           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 44.19% done
********************* END NS.. 322 ****************************************************
********************* START NS ... 323 ****************************************************
[I 2025-06-15 20:08:46,157] A new study created in memory with name: no-name-ae73a0ce-363d-48e7-a53b-8d6faabc5c34
[I 2025-06-15 20:08:46,641] Trial 0 finished with value: 0.024157671145181636 and parameters: {'n_estimators': 70, 'learning_rate': 0.011492328943385454, 'num_leaves': 60, 'max_depth': 8, 'min_child_samples': 28, 'subsample': 0.614159104648856, 'colsample_bytree': 0.7083539145963702, 'reg_alpha': 8.752690393873007, 'reg_lambda': 8.297066289020831, 'min_split_gain': 0.0065008885756322465}. Best is trial 0 with value: 0.024157671145181636.
[Trial 0] New best: 0.02416
[I 2025-06-15 20:08:47,901] Trial 1 finished with value: 0.008635174776295187 and parameters: {'n_estimators': 147, 'learning_rate': 0.010590340844379516, 'num_leaves': 59, 'max_depth': 9, 'min_child_samples': 2, 'subsample': 0.9038695847179361, 'colsample_bytree': 0.8322350584310112, 'reg_alpha': 9.337343779161298, 'reg_lambda': 2.6829134347102777, 'min_split_gain': 0.04105583771813497}. Best is trial 1 with value: 0.008635174776295187.
[Trial 1] New best: 0.00864
[I 2025-06-15 20:08:49,703] Trial 2 finished with value: 0.0031269347436861596 and parameters: {'n_estimators': 173, 'learning_rate': 0.03758704560528624, 'num_leaves': 60, 'max_depth': 8, 'min_child_samples': 16, 'subsample': 0.876224590620235, 'colsample_bytree': 0.9360056414162149, 'reg_alpha': 1.7372433353029992, 'reg_lambda': 9.522724440724806, 'min_split_gain': 0.036615426288732066}. Best is trial 2 with value: 0.0031269347436861596.
[Trial 2] New best: 0.00313
[I 2025-06-15 20:08:50,823] Trial 3 finished with value: 0.0033943754700617493 and parameters: {'n_estimators': 257, 'learning_rate': 0.08061869385603755, 'num_leaves': 213, 'max_depth': 11, 'min_child_samples': 24, 'subsample': 0.6114562540581654, 'colsample_bytree': 0.71688435138966, 'reg_alpha': 5.5521032633756295, 'reg_lambda': 5.233014200592734, 'min_split_gain': 0.013645647169832366}. Best is trial 2 with value: 0.0031269347436861596.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:08:51,582] Trial 4 finished with value: 0.0034922337282995587 and parameters: {'n_estimators': 92, 'learning_rate': 0.11618874301030735, 'num_leaves': 221, 'max_depth': 13, 'min_child_samples': 1, 'subsample': 0.6478906720458217, 'colsample_bytree': 0.6749928231106719, 'reg_alpha': 9.14377052166933, 'reg_lambda': 4.071948547901298, 'min_split_gain': 0.0059303199227394915}. Best is trial 2 with value: 0.0031269347436861596.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:08:52,938] Trial 5 finished with value: 0.004956312834346 and parameters: {'n_estimators': 182, 'learning_rate': 0.06494033680074768, 'num_leaves': 223, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.7167985352367806, 'colsample_bytree': 0.6009621047685919, 'reg_alpha': 0.5218576245271389, 'reg_lambda': 3.1459719493264284, 'min_split_gain': 0.03470475128348392}. Best is trial 2 with value: 0.0031269347436861596.
[I 2025-06-15 20:08:52,941] A new study created in memory with name: no-name-3803a913-2ea7-4e1b-a645-d2c2c8aacfef
[I 2025-06-15 20:08:53,035] Trial 0 finished with value: 0.06268443115101481 and parameters: {'n_estimators': 170, 'learning_rate': 0.014428473560010528, 'num_leaves': 69, 'max_depth': 8, 'min_child_samples': 3, 'subsample': 0.5908792690737769, 'colsample_bytree': 0.7300166288109424, 'reg_alpha': 5.970229226581494, 'reg_lambda': 8.410984042533396, 'min_split_gain': 0.04705030995692401}. Best is trial 0 with value: 0.06268443115101481.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1869
1           Price_lag_2        1339
14      pct_chg_Load_DA        1232
3         Load_DA_lag_0         769
11           NGas_lag_2         613
16  roll2d_median_Price         602
2           Price_lag_7         543
5       WindOn_DA_lag_0         363
15       lag168_Load_DA         362
10           Coal_lag_2         293
12            Oil_lag_2         252
13            EUA_lag_2         186
7                  WD_1          34
9                  WD_7          25
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         229
15       lag168_Load_DA          52
1           Price_lag_2          38
3         Load_DA_lag_0          14
5       WindOn_DA_lag_0          13
16  roll2d_median_Price           9
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 44.32% done
********************* END NS.. 323 ****************************************************
********************* START NS ... 324 ****************************************************
[I 2025-06-15 20:08:56,359] A new study created in memory with name: no-name-10fc93e6-6d56-4563-ad3e-80ec1844f36d
[I 2025-06-15 20:08:57,063] Trial 0 finished with value: 0.006386894496677591 and parameters: {'n_estimators': 103, 'learning_rate': 0.029572992662501876, 'num_leaves': 139, 'max_depth': 15, 'min_child_samples': 30, 'subsample': 0.9648599455108134, 'colsample_bytree': 0.8817714372739344, 'reg_alpha': 5.131856512619909, 'reg_lambda': 7.404040947199752, 'min_split_gain': 0.03486097603289452}. Best is trial 0 with value: 0.006386894496677591.
[Trial 0] New best: 0.00639
[I 2025-06-15 20:08:57,328] Trial 1 finished with value: 0.005017041241523283 and parameters: {'n_estimators': 227, 'learning_rate': 0.23117694197573793, 'num_leaves': 133, 'max_depth': 4, 'min_child_samples': 13, 'subsample': 0.9021417650526102, 'colsample_bytree': 0.8975515097702814, 'reg_alpha': 1.9119891250305165, 'reg_lambda': 9.355886008339436, 'min_split_gain': 0.0426512431772677}. Best is trial 1 with value: 0.005017041241523283.
[Trial 1] New best: 0.00502
[I 2025-06-15 20:08:59,024] Trial 2 finished with value: 0.007948431891518346 and parameters: {'n_estimators': 284, 'learning_rate': 0.0127260273038915, 'num_leaves': 158, 'max_depth': 6, 'min_child_samples': 5, 'subsample': 0.8558896639295341, 'colsample_bytree': 0.7168418085760059, 'reg_alpha': 7.2381580290799565, 'reg_lambda': 9.442231313306943, 'min_split_gain': 0.03578638825116118}. Best is trial 1 with value: 0.005017041241523283.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:09:00,606] Trial 3 finished with value: 0.004418644493184444 and parameters: {'n_estimators': 293, 'learning_rate': 0.0779196246714515, 'num_leaves': 85, 'max_depth': 8, 'min_child_samples': 28, 'subsample': 0.8005364472935766, 'colsample_bytree': 0.6195878925977181, 'reg_alpha': 1.9085899843793674, 'reg_lambda': 9.157038992749307, 'min_split_gain': 0.012434009765568155}. Best is trial 3 with value: 0.004418644493184444.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:09:01,784] Trial 4 finished with value: 0.006538388510743311 and parameters: {'n_estimators': 196, 'learning_rate': 0.06213447198400787, 'num_leaves': 72, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.751055673615441, 'colsample_bytree': 0.6244403825031846, 'reg_alpha': 6.512609500395371, 'reg_lambda': 7.838505921105289, 'min_split_gain': 0.025598117578234554}. Best is trial 3 with value: 0.004418644493184444.
[I 2025-06-15 20:09:01,785] A new study created in memory with name: no-name-9869b806-53c9-4675-a554-f2b20ea85f87
[I 2025-06-15 20:09:01,869] Trial 0 finished with value: 0.04244751874431856 and parameters: {'n_estimators': 170, 'learning_rate': 0.03001627966956627, 'num_leaves': 133, 'max_depth': 13, 'min_child_samples': 12, 'subsample': 0.8162214314520531, 'colsample_bytree': 0.9152976135445581, 'reg_alpha': 1.775889525482991, 'reg_lambda': 6.968076215926766, 'min_split_gain': 0.045700433484110195}. Best is trial 0 with value: 0.04244751874431856.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1136
1           Price_lag_2        1039
14      pct_chg_Load_DA         673
3         Load_DA_lag_0         603
2           Price_lag_7         533
11           NGas_lag_2         468
15       lag168_Load_DA         374
5       WindOn_DA_lag_0         367
16  roll2d_median_Price         307
10           Coal_lag_2         290
13            EUA_lag_2         254
12            Oil_lag_2         190
7                  WD_1          31
9                  WD_7          25
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         182
15       lag168_Load_DA          39
3         Load_DA_lag_0          15
16  roll2d_median_Price          15
2           Price_lag_7           4
1           Price_lag_2           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 44.46% done
********************* END NS.. 324 ****************************************************
********************* START NS ... 325 ****************************************************
[I 2025-06-15 20:09:04,675] A new study created in memory with name: no-name-848164a9-dfc4-4cfa-95c2-a02e3ca6a128
[I 2025-06-15 20:09:06,934] Trial 0 finished with value: 0.0054326962068146694 and parameters: {'n_estimators': 194, 'learning_rate': 0.03377091421446998, 'num_leaves': 190, 'max_depth': 11, 'min_child_samples': 5, 'subsample': 0.9699095594764717, 'colsample_bytree': 0.6240381828093442, 'reg_alpha': 2.383993114691455, 'reg_lambda': 8.773127968056192, 'min_split_gain': 0.04799137279698497}. Best is trial 0 with value: 0.0054326962068146694.
[Trial 0] New best: 0.00543
[I 2025-06-15 20:09:07,401] Trial 1 finished with value: 0.004698841391353853 and parameters: {'n_estimators': 228, 'learning_rate': 0.14243476183470447, 'num_leaves': 175, 'max_depth': 14, 'min_child_samples': 19, 'subsample': 0.7221017478486325, 'colsample_bytree': 0.6655721964998567, 'reg_alpha': 9.450278474507195, 'reg_lambda': 1.3482238309930261, 'min_split_gain': 0.02270573859532874}. Best is trial 1 with value: 0.004698841391353853.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:09:07,879] Trial 2 finished with value: 0.004464787752417955 and parameters: {'n_estimators': 139, 'learning_rate': 0.12765755076446417, 'num_leaves': 52, 'max_depth': 14, 'min_child_samples': 30, 'subsample': 0.7069464303208932, 'colsample_bytree': 0.9792147317379467, 'reg_alpha': 7.587053359030734, 'reg_lambda': 3.976978968687246, 'min_split_gain': 0.016940955889694843}. Best is trial 2 with value: 0.004464787752417955.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:09:09,009] Trial 3 finished with value: 0.004266463600137904 and parameters: {'n_estimators': 195, 'learning_rate': 0.052352329992440175, 'num_leaves': 81, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.689014724038522, 'colsample_bytree': 0.7547388429918414, 'reg_alpha': 3.950541851795508, 'reg_lambda': 3.7771323676015145, 'min_split_gain': 0.007264156761992791}. Best is trial 3 with value: 0.004266463600137904.
[Trial 3] New best: 0.00427
[I 2025-06-15 20:09:11,333] Trial 4 finished with value: 0.006775828117239415 and parameters: {'n_estimators': 249, 'learning_rate': 0.010343719335759177, 'num_leaves': 35, 'max_depth': 10, 'min_child_samples': 14, 'subsample': 0.6511341575097247, 'colsample_bytree': 0.7794607896146928, 'reg_alpha': 1.2788086600426585, 'reg_lambda': 7.884984722527728, 'min_split_gain': 0.030717657500763763}. Best is trial 3 with value: 0.004266463600137904.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:09:12,770] Trial 5 finished with value: 0.010445277732296455 and parameters: {'n_estimators': 80, 'learning_rate': 0.02567365246899617, 'num_leaves': 225, 'max_depth': 16, 'min_child_samples': 13, 'subsample': 0.5245574333036797, 'colsample_bytree': 0.5317396576321585, 'reg_alpha': 0.7023324170194933, 'reg_lambda': 5.806643010782392, 'min_split_gain': 0.04312752166146012}. Best is trial 3 with value: 0.004266463600137904.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:09:13,196] Trial 6 finished with value: 0.007287688708136244 and parameters: {'n_estimators': 82, 'learning_rate': 0.20648874455512944, 'num_leaves': 198, 'max_depth': 14, 'min_child_samples': 30, 'subsample': 0.6861126826870605, 'colsample_bytree': 0.5264448879222778, 'reg_alpha': 9.28098454096897, 'reg_lambda': 6.94234619498347, 'min_split_gain': 0.018059738252166235}. Best is trial 3 with value: 0.004266463600137904.
[I 2025-06-15 20:09:13,198] A new study created in memory with name: no-name-7983f4a5-6ab1-4dc2-9d19-2cf2b5dd8865
[I 2025-06-15 20:09:13,394] Trial 0 finished with value: 0.052167809540698475 and parameters: {'n_estimators': 191, 'learning_rate': 0.11093579200583097, 'num_leaves': 256, 'max_depth': 16, 'min_child_samples': 2, 'subsample': 0.7181043055823881, 'colsample_bytree': 0.5358474283700609, 'reg_alpha': 8.045749419178245, 'reg_lambda': 7.079048217130476, 'min_split_gain': 0.04906086317822077}. Best is trial 0 with value: 0.052167809540698475.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1224
1           Price_lag_2         892
14      pct_chg_Load_DA         720
3         Load_DA_lag_0         579
15       lag168_Load_DA         405
5       WindOn_DA_lag_0         393
11           NGas_lag_2         393
2           Price_lag_7         334
16  roll2d_median_Price         270
13            EUA_lag_2         263
10           Coal_lag_2         239
12            Oil_lag_2         173
9                  WD_7          37
7                  WD_1          30
8                  WD_6          21
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          21
15       lag168_Load_DA           6
3         Load_DA_lag_0           6
1           Price_lag_2           3
9                  WD_7           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 44.60% done
********************* END NS.. 325 ****************************************************
********************* START NS ... 326 ****************************************************
[I 2025-06-15 20:09:16,108] A new study created in memory with name: no-name-e187b704-3048-4786-90d6-80400bebf48b
[I 2025-06-15 20:09:17,969] Trial 0 finished with value: 0.004088320334322934 and parameters: {'n_estimators': 190, 'learning_rate': 0.010225200396186199, 'num_leaves': 161, 'max_depth': 7, 'min_child_samples': 4, 'subsample': 0.5548494547534641, 'colsample_bytree': 0.8955069771758539, 'reg_alpha': 5.668487654810303, 'reg_lambda': 7.482435051230684, 'min_split_gain': 0.04800684143026226}. Best is trial 0 with value: 0.004088320334322934.
[Trial 0] New best: 0.00409
[I 2025-06-15 20:09:18,529] Trial 1 finished with value: 0.002247267531666362 and parameters: {'n_estimators': 260, 'learning_rate': 0.1529389905350387, 'num_leaves': 130, 'max_depth': 6, 'min_child_samples': 1, 'subsample': 0.8367560068187677, 'colsample_bytree': 0.7986909530166515, 'reg_alpha': 4.528914789096941, 'reg_lambda': 8.831475607065732, 'min_split_gain': 0.015937896725412026}. Best is trial 1 with value: 0.002247267531666362.
[Trial 1] New best: 0.00225
[I 2025-06-15 20:09:20,081] Trial 2 finished with value: 0.003945039059988768 and parameters: {'n_estimators': 238, 'learning_rate': 0.11746521849910324, 'num_leaves': 185, 'max_depth': 11, 'min_child_samples': 10, 'subsample': 0.7718259223585612, 'colsample_bytree': 0.5178740089909306, 'reg_alpha': 4.391366186272807, 'reg_lambda': 9.475898825179893, 'min_split_gain': 0.007919456480914756}. Best is trial 1 with value: 0.002247267531666362.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:09:20,714] Trial 3 finished with value: 0.010703657710342202 and parameters: {'n_estimators': 100, 'learning_rate': 0.016152113817698356, 'num_leaves': 76, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.7271270073712551, 'colsample_bytree': 0.5520988646545095, 'reg_alpha': 5.2142681760450955, 'reg_lambda': 8.968944435709297, 'min_split_gain': 0.02202221624115183}. Best is trial 1 with value: 0.002247267531666362.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:09:22,370] Trial 4 finished with value: 0.004067267646173081 and parameters: {'n_estimators': 190, 'learning_rate': 0.01634602050280067, 'num_leaves': 250, 'max_depth': 11, 'min_child_samples': 4, 'subsample': 0.6584146310302745, 'colsample_bytree': 0.6369451931200247, 'reg_alpha': 9.638754012576117, 'reg_lambda': 7.821897042131182, 'min_split_gain': 0.004203296256472383}. Best is trial 1 with value: 0.002247267531666362.
[I 2025-06-15 20:09:22,373] A new study created in memory with name: no-name-28674802-d645-4a27-afe6-28f384ee4744
[I 2025-06-15 20:09:22,429] Trial 0 finished with value: 0.012683213788991293 and parameters: {'n_estimators': 134, 'learning_rate': 0.16056710162514592, 'num_leaves': 107, 'max_depth': 12, 'min_child_samples': 14, 'subsample': 0.7354067140157747, 'colsample_bytree': 0.8442259081324153, 'reg_alpha': 0.2772093670390352, 'reg_lambda': 6.486448870856868, 'min_split_gain': 0.009013210867787686}. Best is trial 0 with value: 0.012683213788991293.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         371
1           Price_lag_2         290
14      pct_chg_Load_DA         246
2           Price_lag_7         170
3         Load_DA_lag_0         163
11           NGas_lag_2         147
5       WindOn_DA_lag_0         131
13            EUA_lag_2         125
10           Coal_lag_2         103
16  roll2d_median_Price          98
15       lag168_Load_DA          97
12            Oil_lag_2          61
9                  WD_7           7
7                  WD_1           7
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          54
14      pct_chg_Load_DA          23
15       lag168_Load_DA          13
3         Load_DA_lag_0           8
5       WindOn_DA_lag_0           7
16  roll2d_median_Price           6
10           Coal_lag_2           4
12            Oil_lag_2           4
2           Price_lag_7           4
1           Price_lag_2           2
9                  WD_7           2
11           NGas_lag_2           2
6      WindOff_DA_lag_0           0
7                  WD_1           0
13            EUA_lag_2           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 44.73% done
********************* END NS.. 326 ****************************************************
********************* START NS ... 327 ****************************************************
[I 2025-06-15 20:09:24,646] A new study created in memory with name: no-name-8f61d7f9-3a4e-49c4-8dd7-dd15868736db
[I 2025-06-15 20:09:25,076] Trial 0 finished with value: 0.003924351142350398 and parameters: {'n_estimators': 101, 'learning_rate': 0.20171751706745442, 'num_leaves': 101, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.6349692411505241, 'colsample_bytree': 0.9634741828986924, 'reg_alpha': 6.37547227827271, 'reg_lambda': 8.056834819505095, 'min_split_gain': 0.03631871878391067}. Best is trial 0 with value: 0.003924351142350398.
[Trial 0] New best: 0.00392
[I 2025-06-15 20:09:29,124] Trial 1 finished with value: 0.008411703662140772 and parameters: {'n_estimators': 257, 'learning_rate': 0.011141399774063867, 'num_leaves': 102, 'max_depth': 9, 'min_child_samples': 16, 'subsample': 0.7096843419386103, 'colsample_bytree': 0.5357903994192039, 'reg_alpha': 2.2752692429124686, 'reg_lambda': 7.532190979689376, 'min_split_gain': 0.0003966040718680619}. Best is trial 0 with value: 0.003924351142350398.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:09:29,489] Trial 2 finished with value: 0.0036027234205808305 and parameters: {'n_estimators': 157, 'learning_rate': 0.19339924003023243, 'num_leaves': 201, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.6340077625950655, 'colsample_bytree': 0.9247204947372092, 'reg_alpha': 5.176197952989288, 'reg_lambda': 3.6245856655211006, 'min_split_gain': 0.046773187590286994}. Best is trial 2 with value: 0.0036027234205808305.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:09:33,203] Trial 3 finished with value: 0.0037221659373421387 and parameters: {'n_estimators': 199, 'learning_rate': 0.046107329742776426, 'num_leaves': 120, 'max_depth': 14, 'min_child_samples': 18, 'subsample': 0.7656024482859667, 'colsample_bytree': 0.6902326156873966, 'reg_alpha': 0.23060904766896262, 'reg_lambda': 6.338561987798261, 'min_split_gain': 0.02049795327856529}. Best is trial 2 with value: 0.0036027234205808305.
[I 2025-06-15 20:09:33,205] A new study created in memory with name: no-name-25034832-11be-48f8-b2b1-694f47c2d431
[I 2025-06-15 20:09:33,293] Trial 0 finished with value: 0.08342709587664658 and parameters: {'n_estimators': 300, 'learning_rate': 0.07511803844560248, 'num_leaves': 222, 'max_depth': 5, 'min_child_samples': 23, 'subsample': 0.876083307573539, 'colsample_bytree': 0.6959511191230253, 'reg_alpha': 9.181420999495467, 'reg_lambda': 5.639688379310601, 'min_split_gain': 0.00608177937374868}. Best is trial 0 with value: 0.08342709587664658.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         269
14      pct_chg_Load_DA         139
1           Price_lag_2         116
11           NGas_lag_2          78
3         Load_DA_lag_0          73
2           Price_lag_7          67
16  roll2d_median_Price          63
10           Coal_lag_2          46
5       WindOn_DA_lag_0          40
15       lag168_Load_DA          34
13            EUA_lag_2          20
12            Oil_lag_2          12
7                  WD_1           3
9                  WD_7           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          67
3         Load_DA_lag_0           9
1           Price_lag_2           9
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 44.87% done
********************* END NS.. 327 ****************************************************
********************* START NS ... 328 ****************************************************
[I 2025-06-15 20:09:35,212] A new study created in memory with name: no-name-c2e315d8-b377-4313-8515-55effd6fba16
[I 2025-06-15 20:09:36,017] Trial 0 finished with value: 0.0038774103930280545 and parameters: {'n_estimators': 170, 'learning_rate': 0.09221115995738725, 'num_leaves': 141, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.5572263099716309, 'colsample_bytree': 0.678690416853108, 'reg_alpha': 6.237581224709365, 'reg_lambda': 6.982225753123292, 'min_split_gain': 0.01237107574517602}. Best is trial 0 with value: 0.0038774103930280545.
[Trial 0] New best: 0.00388
[I 2025-06-15 20:09:37,426] Trial 1 finished with value: 0.0035752283592376485 and parameters: {'n_estimators': 147, 'learning_rate': 0.03124981915840161, 'num_leaves': 193, 'max_depth': 11, 'min_child_samples': 15, 'subsample': 0.6977651596885062, 'colsample_bytree': 0.9654323102012754, 'reg_alpha': 7.622550082341925, 'reg_lambda': 1.312652627423463, 'min_split_gain': 0.000598711997462681}. Best is trial 1 with value: 0.0035752283592376485.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:09:37,836] Trial 2 finished with value: 0.018573172495764966 and parameters: {'n_estimators': 63, 'learning_rate': 0.019182166040439874, 'num_leaves': 239, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.8230958707097875, 'colsample_bytree': 0.7334399147265811, 'reg_alpha': 9.515637095924195, 'reg_lambda': 0.7723016236021873, 'min_split_gain': 0.033424968810360986}. Best is trial 1 with value: 0.0035752283592376485.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:09:39,134] Trial 3 finished with value: 0.0039701425568166885 and parameters: {'n_estimators': 281, 'learning_rate': 0.023474728116042794, 'num_leaves': 168, 'max_depth': 5, 'min_child_samples': 30, 'subsample': 0.9433059019924578, 'colsample_bytree': 0.8110781715142633, 'reg_alpha': 6.256810919186662, 'reg_lambda': 9.706873836428008, 'min_split_gain': 0.022314015448115933}. Best is trial 1 with value: 0.0035752283592376485.
[I 2025-06-15 20:09:39,136] A new study created in memory with name: no-name-ba78a498-0f64-4a4b-8d8c-104723ad395a
[I 2025-06-15 20:09:39,236] Trial 0 finished with value: 0.027019139915141047 and parameters: {'n_estimators': 175, 'learning_rate': 0.017093007612838176, 'num_leaves': 157, 'max_depth': 15, 'min_child_samples': 23, 'subsample': 0.8254322170124011, 'colsample_bytree': 0.865079281593512, 'reg_alpha': 2.801464409021668, 'reg_lambda': 5.37645171275469, 'min_split_gain': 0.036640305508503855}. Best is trial 0 with value: 0.027019139915141047.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1677
1           Price_lag_2        1023
14      pct_chg_Load_DA         911
2           Price_lag_7         524
3         Load_DA_lag_0         474
16  roll2d_median_Price         462
11           NGas_lag_2         453
13            EUA_lag_2         291
15       lag168_Load_DA         266
5       WindOn_DA_lag_0         262
10           Coal_lag_2         227
12            Oil_lag_2         148
9                  WD_7          22
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         251
3         Load_DA_lag_0          48
12            Oil_lag_2          44
15       lag168_Load_DA          22
1           Price_lag_2          18
2           Price_lag_7           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 45.01% done
********************* END NS.. 328 ****************************************************
********************* START NS ... 329 ****************************************************
[I 2025-06-15 20:09:42,660] A new study created in memory with name: no-name-cf9470b3-4b81-4705-bc3c-57f774deed5c
[I 2025-06-15 20:09:43,105] Trial 0 finished with value: 0.0023278714864073268 and parameters: {'n_estimators': 104, 'learning_rate': 0.13238137825651472, 'num_leaves': 176, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.5707659709480923, 'colsample_bytree': 0.8969581349955434, 'reg_alpha': 4.2122714116653315, 'reg_lambda': 0.7174675802577501, 'min_split_gain': 0.015462613879477866}. Best is trial 0 with value: 0.0023278714864073268.
[Trial 0] New best: 0.00233
[I 2025-06-15 20:09:44,638] Trial 1 finished with value: 0.002920292745746955 and parameters: {'n_estimators': 147, 'learning_rate': 0.0687786470442228, 'num_leaves': 80, 'max_depth': 13, 'min_child_samples': 26, 'subsample': 0.701760828399004, 'colsample_bytree': 0.5127006850307511, 'reg_alpha': 0.12098241018226985, 'reg_lambda': 4.591665239099047, 'min_split_gain': 0.04301858853673875}. Best is trial 0 with value: 0.0023278714864073268.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:09:45,223] Trial 2 finished with value: 0.004932347616441728 and parameters: {'n_estimators': 127, 'learning_rate': 0.017768114508519933, 'num_leaves': 154, 'max_depth': 4, 'min_child_samples': 8, 'subsample': 0.9102281903048062, 'colsample_bytree': 0.6273270151806103, 'reg_alpha': 5.446156803810524, 'reg_lambda': 4.17116996499637, 'min_split_gain': 0.0009214445420563767}. Best is trial 0 with value: 0.0023278714864073268.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:09:45,573] Trial 3 finished with value: 0.002707895037883313 and parameters: {'n_estimators': 245, 'learning_rate': 0.2508571799157551, 'num_leaves': 255, 'max_depth': 15, 'min_child_samples': 22, 'subsample': 0.6178425362413952, 'colsample_bytree': 0.9820140234134587, 'reg_alpha': 4.31121715719394, 'reg_lambda': 1.8379692297792516, 'min_split_gain': 0.043665119226758184}. Best is trial 0 with value: 0.0023278714864073268.
[I 2025-06-15 20:09:45,576] A new study created in memory with name: no-name-5c580996-5efd-4e44-a1f1-71ed77cf2121
[I 2025-06-15 20:09:45,642] Trial 0 finished with value: 0.01726857125135865 and parameters: {'n_estimators': 228, 'learning_rate': 0.1771058209441405, 'num_leaves': 115, 'max_depth': 15, 'min_child_samples': 6, 'subsample': 0.711528369707466, 'colsample_bytree': 0.7052636271854862, 'reg_alpha': 5.354281152211573, 'reg_lambda': 1.7700278003625014, 'min_split_gain': 0.005725206447488008}. Best is trial 0 with value: 0.01726857125135865.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         448
1           Price_lag_2         322
14      pct_chg_Load_DA         294
3         Load_DA_lag_0         187
11           NGas_lag_2         159
2           Price_lag_7         155
16  roll2d_median_Price         145
15       lag168_Load_DA         142
5       WindOn_DA_lag_0         134
13            EUA_lag_2         100
10           Coal_lag_2          94
12            Oil_lag_2          66
7                  WD_1          11
9                  WD_7           7
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          37
15       lag168_Load_DA           6
3         Load_DA_lag_0           4
12            Oil_lag_2           3
1           Price_lag_2           3
5       WindOn_DA_lag_0           2
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 45.14% done
********************* END NS.. 329 ****************************************************
********************* START NS ... 330 ****************************************************
[I 2025-06-15 20:09:47,917] A new study created in memory with name: no-name-acb81dc3-ddf2-4910-8fdd-ff66433eb884
[I 2025-06-15 20:09:48,549] Trial 0 finished with value: 0.006876645080110561 and parameters: {'n_estimators': 96, 'learning_rate': 0.01202166272540365, 'num_leaves': 33, 'max_depth': 12, 'min_child_samples': 4, 'subsample': 0.9721399537974058, 'colsample_bytree': 0.8354069616233963, 'reg_alpha': 9.161584677641525, 'reg_lambda': 6.093357327425418, 'min_split_gain': 0.025220746679497676}. Best is trial 0 with value: 0.006876645080110561.
[Trial 0] New best: 0.00688
[I 2025-06-15 20:09:50,669] Trial 1 finished with value: 0.0028840621954733807 and parameters: {'n_estimators': 293, 'learning_rate': 0.030865253910087154, 'num_leaves': 201, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.6345809258870112, 'colsample_bytree': 0.743662867051051, 'reg_alpha': 4.604291854550696, 'reg_lambda': 6.315619138363612, 'min_split_gain': 0.02925184645957693}. Best is trial 1 with value: 0.0028840621954733807.
[Trial 1] New best: 0.00288
[I 2025-06-15 20:09:51,590] Trial 2 finished with value: 0.003278671242700197 and parameters: {'n_estimators': 293, 'learning_rate': 0.0213564659395536, 'num_leaves': 177, 'max_depth': 4, 'min_child_samples': 21, 'subsample': 0.6051654826386803, 'colsample_bytree': 0.8157270453006941, 'reg_alpha': 7.8864358701511925, 'reg_lambda': 0.9081355178463935, 'min_split_gain': 0.047456123345553575}. Best is trial 1 with value: 0.0028840621954733807.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:09:52,172] Trial 3 finished with value: 0.0028900324317508723 and parameters: {'n_estimators': 63, 'learning_rate': 0.1570604287344779, 'num_leaves': 202, 'max_depth': 15, 'min_child_samples': 9, 'subsample': 0.6099935070280913, 'colsample_bytree': 0.6585566462510916, 'reg_alpha': 1.9686300340678387, 'reg_lambda': 6.298871875519633, 'min_split_gain': 0.049400301178489155}. Best is trial 1 with value: 0.0028840621954733807.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:09:52,599] Trial 4 finished with value: 0.007782392056565834 and parameters: {'n_estimators': 107, 'learning_rate': 0.011633371611811492, 'num_leaves': 123, 'max_depth': 4, 'min_child_samples': 2, 'subsample': 0.6038918793686532, 'colsample_bytree': 0.7041198511652684, 'reg_alpha': 3.4302922457191354, 'reg_lambda': 6.69069843595717, 'min_split_gain': 0.02382195984673783}. Best is trial 1 with value: 0.0028840621954733807.
[I 2025-06-15 20:09:52,603] A new study created in memory with name: no-name-ddbcf909-afab-4351-aa32-67f46b7718a5
[I 2025-06-15 20:09:52,757] Trial 0 finished with value: 0.024389097954173874 and parameters: {'n_estimators': 264, 'learning_rate': 0.01772264817347961, 'num_leaves': 131, 'max_depth': 13, 'min_child_samples': 5, 'subsample': 0.5453562565477047, 'colsample_bytree': 0.9091545867875477, 'reg_alpha': 4.845136831413831, 'reg_lambda': 6.751766634926538, 'min_split_gain': 0.017516445173042444}. Best is trial 0 with value: 0.024389097954173874.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1839
1           Price_lag_2        1534
14      pct_chg_Load_DA        1211
3         Load_DA_lag_0         780
11           NGas_lag_2         597
2           Price_lag_7         486
16  roll2d_median_Price         464
15       lag168_Load_DA         453
5       WindOn_DA_lag_0         441
10           Coal_lag_2         379
13            EUA_lag_2         268
12            Oil_lag_2         190
9                  WD_7          38
7                  WD_1          27
8                  WD_6          21
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         314
3         Load_DA_lag_0          56
15       lag168_Load_DA          48
5       WindOn_DA_lag_0          35
1           Price_lag_2          16
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 45.28% done
********************* END NS.. 330 ****************************************************
********************* START NS ... 331 ****************************************************
[I 2025-06-15 20:09:56,793] A new study created in memory with name: no-name-968e0d9f-d0e1-4c54-b583-557038e7286e
[I 2025-06-15 20:09:57,499] Trial 0 finished with value: 0.005745667223131094 and parameters: {'n_estimators': 133, 'learning_rate': 0.03334917129646684, 'num_leaves': 194, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.7808090348221535, 'colsample_bytree': 0.6223856590542776, 'reg_alpha': 6.0964966471611195, 'reg_lambda': 9.787960354316521, 'min_split_gain': 0.029298748426804896}. Best is trial 0 with value: 0.005745667223131094.
[Trial 0] New best: 0.00575
[I 2025-06-15 20:09:59,954] Trial 1 finished with value: 0.004474909290075539 and parameters: {'n_estimators': 190, 'learning_rate': 0.04085317828162556, 'num_leaves': 249, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.8660114937781788, 'colsample_bytree': 0.9335007859292392, 'reg_alpha': 0.7837030128048106, 'reg_lambda': 7.587319413439301, 'min_split_gain': 0.029064847861239654}. Best is trial 1 with value: 0.004474909290075539.
[Trial 1] New best: 0.00447
[I 2025-06-15 20:10:00,279] Trial 2 finished with value: 0.004731591398123102 and parameters: {'n_estimators': 57, 'learning_rate': 0.23471052556512284, 'num_leaves': 173, 'max_depth': 14, 'min_child_samples': 18, 'subsample': 0.7890378426031439, 'colsample_bytree': 0.9265922579046486, 'reg_alpha': 4.3469825429647715, 'reg_lambda': 6.5489285979703995, 'min_split_gain': 0.028849291077831442}. Best is trial 1 with value: 0.004474909290075539.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:10:01,913] Trial 3 finished with value: 0.0057021535628948876 and parameters: {'n_estimators': 242, 'learning_rate': 0.010383773847043053, 'num_leaves': 134, 'max_depth': 8, 'min_child_samples': 24, 'subsample': 0.5373895888541818, 'colsample_bytree': 0.7356850761957778, 'reg_alpha': 8.803236451041014, 'reg_lambda': 2.779062497738317, 'min_split_gain': 0.020789361974850935}. Best is trial 1 with value: 0.004474909290075539.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:10:02,967] Trial 4 finished with value: 0.004715341479188707 and parameters: {'n_estimators': 112, 'learning_rate': 0.022249431174373672, 'num_leaves': 96, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.5863290486960273, 'colsample_bytree': 0.8926243211527523, 'reg_alpha': 5.599484563362695, 'reg_lambda': 3.8860862628486306, 'min_split_gain': 0.025996425382309386}. Best is trial 1 with value: 0.004474909290075539.
[I 2025-06-15 20:10:02,969] A new study created in memory with name: no-name-efeebcdc-af88-4095-88de-9abdd11d2f54
[I 2025-06-15 20:10:03,011] Trial 0 finished with value: 0.053569012993889165 and parameters: {'n_estimators': 77, 'learning_rate': 0.033345976830661525, 'num_leaves': 164, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.9390128057144278, 'colsample_bytree': 0.7683704472880255, 'reg_alpha': 5.896576354038095, 'reg_lambda': 8.399992824863068, 'min_split_gain': 0.04169925819827214}. Best is trial 0 with value: 0.053569012993889165.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2126
14      pct_chg_Load_DA        1756
1           Price_lag_2        1731
3         Load_DA_lag_0        1320
2           Price_lag_7         925
16  roll2d_median_Price         906
11           NGas_lag_2         870
5       WindOn_DA_lag_0         735
15       lag168_Load_DA         712
10           Coal_lag_2         533
12            Oil_lag_2         408
13            EUA_lag_2         324
9                  WD_7          46
7                  WD_1          39
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         100
1           Price_lag_2          17
2           Price_lag_7           5
5       WindOn_DA_lag_0           4
3         Load_DA_lag_0           2
15       lag168_Load_DA           2
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 45.42% done
********************* END NS.. 331 ****************************************************
********************* START NS ... 332 ****************************************************
[I 2025-06-15 20:10:06,695] A new study created in memory with name: no-name-045e0ae5-66e0-4173-9ad5-ffc18635c15d
[I 2025-06-15 20:10:07,957] Trial 0 finished with value: 0.00881849374211003 and parameters: {'n_estimators': 124, 'learning_rate': 0.06818809166055319, 'num_leaves': 155, 'max_depth': 12, 'min_child_samples': 20, 'subsample': 0.7609983663239699, 'colsample_bytree': 0.8704107098049481, 'reg_alpha': 4.000992465116574, 'reg_lambda': 8.279461644910393, 'min_split_gain': 0.013122797741624987}. Best is trial 0 with value: 0.00881849374211003.
[Trial 0] New best: 0.00882
[I 2025-06-15 20:10:08,654] Trial 1 finished with value: 0.013102177438727098 and parameters: {'n_estimators': 106, 'learning_rate': 0.24113223749597246, 'num_leaves': 240, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.6633011069541234, 'colsample_bytree': 0.5414658049740445, 'reg_alpha': 1.051029772203862, 'reg_lambda': 4.919228862136516, 'min_split_gain': 0.035114651758808485}. Best is trial 0 with value: 0.00881849374211003.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:10:12,754] Trial 2 finished with value: 0.00911769934150661 and parameters: {'n_estimators': 262, 'learning_rate': 0.037787696472437196, 'num_leaves': 227, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.8466862084511679, 'colsample_bytree': 0.9008919908165284, 'reg_alpha': 1.375270251533609, 'reg_lambda': 7.795255448669441, 'min_split_gain': 0.0011931114645712783}. Best is trial 0 with value: 0.00881849374211003.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:10:13,562] Trial 3 finished with value: 0.008009163250806904 and parameters: {'n_estimators': 161, 'learning_rate': 0.24212528990961704, 'num_leaves': 148, 'max_depth': 6, 'min_child_samples': 9, 'subsample': 0.5299004521556345, 'colsample_bytree': 0.5283479685697913, 'reg_alpha': 6.122754047338477, 'reg_lambda': 0.823662575026779, 'min_split_gain': 0.0003762769500369467}. Best is trial 3 with value: 0.008009163250806904.
[I 2025-06-15 20:10:13,565] A new study created in memory with name: no-name-bc3776b6-8a70-4cc8-9539-acb9153652eb
[I 2025-06-15 20:10:13,686] Trial 0 finished with value: 0.09322563890764209 and parameters: {'n_estimators': 287, 'learning_rate': 0.03912735599762097, 'num_leaves': 34, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.8765120245396372, 'colsample_bytree': 0.8546165915215814, 'reg_alpha': 1.038738140942097, 'reg_lambda': 2.5550834524470556, 'min_split_gain': 0.015506778850934905}. Best is trial 0 with value: 0.09322563890764209.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         538
1           Price_lag_2         511
16  roll2d_median_Price         328
3         Load_DA_lag_0         320
2           Price_lag_7         319
14      pct_chg_Load_DA         307
13            EUA_lag_2         236
5       WindOn_DA_lag_0         204
11           NGas_lag_2         190
12            Oil_lag_2         182
10           Coal_lag_2         176
15       lag168_Load_DA         168
9                  WD_7          22
8                  WD_6          12
7                  WD_1           9
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         189
15       lag168_Load_DA          51
3         Load_DA_lag_0          25
14      pct_chg_Load_DA          21
1           Price_lag_2          18
10           Coal_lag_2           8
2           Price_lag_7           7
12            Oil_lag_2           5
11           NGas_lag_2           2
16  roll2d_median_Price           1
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
13            EUA_lag_2           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 45.55% done
********************* END NS.. 332 ****************************************************
********************* START NS ... 333 ****************************************************
[I 2025-06-15 20:10:16,018] A new study created in memory with name: no-name-4ee07753-f4a4-45a5-8631-2589a535393d
[I 2025-06-15 20:10:16,634] Trial 0 finished with value: 0.03444993862954936 and parameters: {'n_estimators': 148, 'learning_rate': 0.21750292555311565, 'num_leaves': 176, 'max_depth': 10, 'min_child_samples': 1, 'subsample': 0.9852969351526548, 'colsample_bytree': 0.8361498683238405, 'reg_alpha': 0.634898552979235, 'reg_lambda': 2.797333761802976, 'min_split_gain': 0.04082569131796061}. Best is trial 0 with value: 0.03444993862954936.
[Trial 0] New best: 0.03445
[I 2025-06-15 20:10:17,529] Trial 1 finished with value: 0.033877798940502225 and parameters: {'n_estimators': 282, 'learning_rate': 0.08745320992978456, 'num_leaves': 118, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.9085939761862223, 'colsample_bytree': 0.58092551803391, 'reg_alpha': 1.8160285221592498, 'reg_lambda': 8.733041444982849, 'min_split_gain': 0.033450846758717505}. Best is trial 1 with value: 0.033877798940502225.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:10:18,794] Trial 2 finished with value: 0.04363674752343554 and parameters: {'n_estimators': 178, 'learning_rate': 0.018865102723490165, 'num_leaves': 181, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.7697181766230747, 'colsample_bytree': 0.7084061415470135, 'reg_alpha': 8.711215851413591, 'reg_lambda': 7.709974434984205, 'min_split_gain': 0.026141475756877225}. Best is trial 1 with value: 0.033877798940502225.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:10:20,289] Trial 3 finished with value: 0.037580301375157235 and parameters: {'n_estimators': 230, 'learning_rate': 0.029493423007528836, 'num_leaves': 52, 'max_depth': 12, 'min_child_samples': 18, 'subsample': 0.7670994023120538, 'colsample_bytree': 0.7390050990447801, 'reg_alpha': 8.535671074966036, 'reg_lambda': 7.673609758864835, 'min_split_gain': 0.04347350092196678}. Best is trial 1 with value: 0.033877798940502225.
[I 2025-06-15 20:10:20,291] A new study created in memory with name: no-name-bf34dcb6-311f-470a-8c65-9abffbc2ce44
[I 2025-06-15 20:10:20,424] Trial 0 finished with value: 0.22280037434045463 and parameters: {'n_estimators': 210, 'learning_rate': 0.014727742772846308, 'num_leaves': 71, 'max_depth': 14, 'min_child_samples': 15, 'subsample': 0.9475503660386936, 'colsample_bytree': 0.644207798644581, 'reg_alpha': 2.872497067507905, 'reg_lambda': 1.2161677990578967, 'min_split_gain': 0.014391499644794798}. Best is trial 0 with value: 0.22280037434045463.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         783
1           Price_lag_2         620
14      pct_chg_Load_DA         424
3         Load_DA_lag_0         367
11           NGas_lag_2         291
2           Price_lag_7         264
15       lag168_Load_DA         213
16  roll2d_median_Price         213
5       WindOn_DA_lag_0         193
13            EUA_lag_2         153
10           Coal_lag_2         146
12            Oil_lag_2         133
9                  WD_7          18
7                  WD_1          15
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         321
1           Price_lag_2          73
3         Load_DA_lag_0          69
15       lag168_Load_DA          25
2           Price_lag_7          23
10           Coal_lag_2          19
12            Oil_lag_2           6
16  roll2d_median_Price           4
11           NGas_lag_2           3
13            EUA_lag_2           1
7                  WD_1           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
14      pct_chg_Load_DA           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 45.69% done
********************* END NS.. 333 ****************************************************
********************* START NS ... 334 ****************************************************
[I 2025-06-15 20:10:22,991] A new study created in memory with name: no-name-6478c8d8-5f34-4209-a009-e2006468359d
[I 2025-06-15 20:10:24,458] Trial 0 finished with value: 0.05426264806557199 and parameters: {'n_estimators': 218, 'learning_rate': 0.11494047038562566, 'num_leaves': 103, 'max_depth': 8, 'min_child_samples': 4, 'subsample': 0.5385272135072494, 'colsample_bytree': 0.5377192936925159, 'reg_alpha': 0.3671419025852296, 'reg_lambda': 7.3455064145843165, 'min_split_gain': 0.012158201722676726}. Best is trial 0 with value: 0.05426264806557199.
[Trial 0] New best: 0.05426
[I 2025-06-15 20:10:24,952] Trial 1 finished with value: 0.07436981450288559 and parameters: {'n_estimators': 137, 'learning_rate': 0.215378874302438, 'num_leaves': 136, 'max_depth': 5, 'min_child_samples': 7, 'subsample': 0.7308731462831974, 'colsample_bytree': 0.5881748349298838, 'reg_alpha': 3.485784958018112, 'reg_lambda': 9.818202922915573, 'min_split_gain': 0.036774321355120744}. Best is trial 0 with value: 0.05426264806557199.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:10:26,124] Trial 2 finished with value: 0.05555626416315188 and parameters: {'n_estimators': 142, 'learning_rate': 0.07832740363474448, 'num_leaves': 156, 'max_depth': 10, 'min_child_samples': 5, 'subsample': 0.5667719421974611, 'colsample_bytree': 0.9470233677351796, 'reg_alpha': 1.5747961665395893, 'reg_lambda': 7.187543767616758, 'min_split_gain': 0.038142112394935125}. Best is trial 0 with value: 0.05426264806557199.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:10:27,135] Trial 3 finished with value: 0.059113545527031686 and parameters: {'n_estimators': 95, 'learning_rate': 0.041983545600073, 'num_leaves': 158, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.86250396810665, 'colsample_bytree': 0.9887152138416848, 'reg_alpha': 2.059483888248781, 'reg_lambda': 5.54204895082269, 'min_split_gain': 0.011860746862312517}. Best is trial 0 with value: 0.05426264806557199.
[I 2025-06-15 20:10:27,137] A new study created in memory with name: no-name-66dfa9d8-3a09-472b-acdf-d5c88feb15bc
[I 2025-06-15 20:10:27,194] Trial 0 finished with value: 0.3824313399423194 and parameters: {'n_estimators': 182, 'learning_rate': 0.12845753404608878, 'num_leaves': 145, 'max_depth': 7, 'min_child_samples': 13, 'subsample': 0.7162926463704483, 'colsample_bytree': 0.972870546637753, 'reg_alpha': 2.083344454656688, 'reg_lambda': 1.9800810772077415, 'min_split_gain': 0.02310718890075321}. Best is trial 0 with value: 0.3824313399423194.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1124
1           Price_lag_2        1046
14      pct_chg_Load_DA         867
3         Load_DA_lag_0         834
2           Price_lag_7         805
16  roll2d_median_Price         570
5       WindOn_DA_lag_0         570
11           NGas_lag_2         527
15       lag168_Load_DA         491
10           Coal_lag_2         464
12            Oil_lag_2         409
13            EUA_lag_2         407
9                  WD_7          31
7                  WD_1          19
8                  WD_6          17
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          52
3         Load_DA_lag_0          21
15       lag168_Load_DA           8
11           NGas_lag_2           6
16  roll2d_median_Price           5
14      pct_chg_Load_DA           4
2           Price_lag_7           2
5       WindOn_DA_lag_0           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
4        Solar_DA_lag_0           0
12            Oil_lag_2           0
13            EUA_lag_2           0
8                  WD_6           0
-> 45.83% done
********************* END NS.. 334 ****************************************************
********************* START NS ... 335 ****************************************************
[I 2025-06-15 20:10:30,162] A new study created in memory with name: no-name-63a50ed9-3f84-4127-8b24-ca97184cdfd1
[I 2025-06-15 20:10:31,056] Trial 0 finished with value: 0.06566845423261859 and parameters: {'n_estimators': 204, 'learning_rate': 0.0261669156216801, 'num_leaves': 214, 'max_depth': 5, 'min_child_samples': 24, 'subsample': 0.5417162603248226, 'colsample_bytree': 0.634206860806436, 'reg_alpha': 9.042030553497218, 'reg_lambda': 7.122398106287402, 'min_split_gain': 0.042167819162761015}. Best is trial 0 with value: 0.06566845423261859.
[Trial 0] New best: 0.06567
[I 2025-06-15 20:10:31,879] Trial 1 finished with value: 0.06905411956351289 and parameters: {'n_estimators': 63, 'learning_rate': 0.04578206466216442, 'num_leaves': 85, 'max_depth': 14, 'min_child_samples': 30, 'subsample': 0.9938289143734163, 'colsample_bytree': 0.6071728556545919, 'reg_alpha': 1.6326344454200792, 'reg_lambda': 4.810471775281347, 'min_split_gain': 0.024218420871343467}. Best is trial 0 with value: 0.06566845423261859.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:10:32,391] Trial 2 finished with value: 0.06221140799068987 and parameters: {'n_estimators': 161, 'learning_rate': 0.11495713409530844, 'num_leaves': 110, 'max_depth': 8, 'min_child_samples': 20, 'subsample': 0.8867748305899049, 'colsample_bytree': 0.9533297841705388, 'reg_alpha': 9.444737793489313, 'reg_lambda': 1.987115963726337, 'min_split_gain': 0.012360222023056878}. Best is trial 2 with value: 0.06221140799068987.
[Trial 2] New best: 0.06221
[I 2025-06-15 20:10:32,718] Trial 3 finished with value: 0.10626149198599348 and parameters: {'n_estimators': 98, 'learning_rate': 0.012450413753971084, 'num_leaves': 187, 'max_depth': 4, 'min_child_samples': 17, 'subsample': 0.9490230990342079, 'colsample_bytree': 0.9152686791409079, 'reg_alpha': 6.498306784054852, 'reg_lambda': 7.908425046174149, 'min_split_gain': 0.023442035276539992}. Best is trial 2 with value: 0.06221140799068987.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:10:35,862] Trial 4 finished with value: 0.06642701134694196 and parameters: {'n_estimators': 242, 'learning_rate': 0.01626945065853766, 'num_leaves': 68, 'max_depth': 15, 'min_child_samples': 28, 'subsample': 0.874662895374517, 'colsample_bytree': 0.5660369289637306, 'reg_alpha': 3.0832496605333812, 'reg_lambda': 9.087552512300492, 'min_split_gain': 0.021359768595973744}. Best is trial 2 with value: 0.06221140799068987.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:10:36,432] Trial 5 finished with value: 0.05312374226882286 and parameters: {'n_estimators': 174, 'learning_rate': 0.2793455503895471, 'num_leaves': 121, 'max_depth': 14, 'min_child_samples': 25, 'subsample': 0.9746995647619208, 'colsample_bytree': 0.5748601600083663, 'reg_alpha': 7.903564606692809, 'reg_lambda': 4.995988056136986, 'min_split_gain': 0.0020908459016163897}. Best is trial 5 with value: 0.05312374226882286.
[Trial 5] New best: 0.05312
[I 2025-06-15 20:10:40,005] Trial 6 finished with value: 0.04453303553321902 and parameters: {'n_estimators': 290, 'learning_rate': 0.033949434543041244, 'num_leaves': 211, 'max_depth': 12, 'min_child_samples': 30, 'subsample': 0.8496525895626106, 'colsample_bytree': 0.9390168353532249, 'reg_alpha': 2.0757433310510476, 'reg_lambda': 1.8164994401547585, 'min_split_gain': 0.010103705875830676}. Best is trial 6 with value: 0.04453303553321902.
[Trial 6] New best: 0.04453
[I 2025-06-15 20:10:43,325] Trial 7 finished with value: 0.0562574481252948 and parameters: {'n_estimators': 207, 'learning_rate': 0.029353747561741285, 'num_leaves': 173, 'max_depth': 11, 'min_child_samples': 13, 'subsample': 0.581532051717494, 'colsample_bytree': 0.9026944160592557, 'reg_alpha': 2.7430476193562447, 'reg_lambda': 6.714643273714003, 'min_split_gain': 0.00978870782280294}. Best is trial 6 with value: 0.04453303553321902.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:10:43,571] Trial 8 finished with value: 0.05296241513348734 and parameters: {'n_estimators': 78, 'learning_rate': 0.2828753734175734, 'num_leaves': 234, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.7219577988085952, 'colsample_bytree': 0.8441345707964372, 'reg_alpha': 5.412669216737324, 'reg_lambda': 3.0145798025681927, 'min_split_gain': 0.03340132063989448}. Best is trial 6 with value: 0.04453303553321902.
[Trial 8] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:10:44,618] Trial 9 finished with value: 0.060059270728266195 and parameters: {'n_estimators': 153, 'learning_rate': 0.03358016451428348, 'num_leaves': 84, 'max_depth': 6, 'min_child_samples': 15, 'subsample': 0.6773237721342888, 'colsample_bytree': 0.9142140220870019, 'reg_alpha': 6.280225565990389, 'reg_lambda': 4.034103553075765, 'min_split_gain': 0.04563923840840466}. Best is trial 6 with value: 0.04453303553321902.
[I 2025-06-15 20:10:44,621] A new study created in memory with name: no-name-dfca4d7d-7eda-49f3-aeaa-c1bd94ef56d0
[I 2025-06-15 20:10:44,719] Trial 0 finished with value: 0.23032735407168822 and parameters: {'n_estimators': 296, 'learning_rate': 0.19206961284853638, 'num_leaves': 37, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.6112361837806062, 'colsample_bytree': 0.5248073288976174, 'reg_alpha': 4.053434102047667, 'reg_lambda': 1.470118219525678, 'min_split_gain': 0.024683956390314128}. Best is trial 0 with value: 0.23032735407168822.
[Trial 9] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2777
14      pct_chg_Load_DA        2442
1           Price_lag_2        2194
3         Load_DA_lag_0        1657
15       lag168_Load_DA        1200
2           Price_lag_7        1106
5       WindOn_DA_lag_0        1076
16  roll2d_median_Price        1009
11           NGas_lag_2         990
13            EUA_lag_2         768
10           Coal_lag_2         723
12            Oil_lag_2         557
9                  WD_7          64
7                  WD_1          51
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          26
3         Load_DA_lag_0           9
5       WindOn_DA_lag_0           8
15       lag168_Load_DA           6
1           Price_lag_2           4
13            EUA_lag_2           3
16  roll2d_median_Price           3
14      pct_chg_Load_DA           1
7                  WD_1           0
6      WindOff_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
8                  WD_6           0
-> 45.96% done
********************* END NS.. 335 ****************************************************
********************* START NS ... 336 ****************************************************
[I 2025-06-15 20:10:49,883] A new study created in memory with name: no-name-a1be7833-d6a4-482a-8736-4efc26f8db93
[I 2025-06-15 20:10:51,515] Trial 0 finished with value: 0.017079473214562466 and parameters: {'n_estimators': 167, 'learning_rate': 0.02829735876851523, 'num_leaves': 134, 'max_depth': 13, 'min_child_samples': 3, 'subsample': 0.6188513378423182, 'colsample_bytree': 0.9747698692318111, 'reg_alpha': 6.790901609736145, 'reg_lambda': 8.839927554890215, 'min_split_gain': 0.02900070977630927}. Best is trial 0 with value: 0.017079473214562466.
[Trial 0] New best: 0.01708
[I 2025-06-15 20:10:52,228] Trial 1 finished with value: 0.01915580178159883 and parameters: {'n_estimators': 278, 'learning_rate': 0.08129291653162429, 'num_leaves': 102, 'max_depth': 5, 'min_child_samples': 26, 'subsample': 0.6693193055544955, 'colsample_bytree': 0.7555084715164704, 'reg_alpha': 7.97123839028076, 'reg_lambda': 1.3467866878658963, 'min_split_gain': 0.03030213688198228}. Best is trial 0 with value: 0.017079473214562466.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:10:52,771] Trial 2 finished with value: 0.01894553568176174 and parameters: {'n_estimators': 272, 'learning_rate': 0.29401480206622815, 'num_leaves': 247, 'max_depth': 11, 'min_child_samples': 23, 'subsample': 0.8026185669973704, 'colsample_bytree': 0.768448052616372, 'reg_alpha': 0.6744806060171737, 'reg_lambda': 6.251154529998221, 'min_split_gain': 0.03034630546303349}. Best is trial 0 with value: 0.017079473214562466.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:10:54,271] Trial 3 finished with value: 0.02570410040624153 and parameters: {'n_estimators': 183, 'learning_rate': 0.024857256172552966, 'num_leaves': 125, 'max_depth': 7, 'min_child_samples': 6, 'subsample': 0.6722823111370968, 'colsample_bytree': 0.5459195748353187, 'reg_alpha': 5.614218729367239, 'reg_lambda': 7.796553648794946, 'min_split_gain': 0.043194671256857914}. Best is trial 0 with value: 0.017079473214562466.
[I 2025-06-15 20:10:54,274] A new study created in memory with name: no-name-9fc0d50c-b325-4983-90a6-ca896a95387e
[I 2025-06-15 20:10:54,352] Trial 0 finished with value: 0.047965206708448005 and parameters: {'n_estimators': 133, 'learning_rate': 0.06086304260819575, 'num_leaves': 208, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.7657284962938841, 'colsample_bytree': 0.5475870337592634, 'reg_alpha': 3.9210418041672646, 'reg_lambda': 3.5748168300058913, 'min_split_gain': 0.01687855137203557}. Best is trial 0 with value: 0.047965206708448005.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1734
1           Price_lag_2         984
14      pct_chg_Load_DA         932
2           Price_lag_7         554
16  roll2d_median_Price         517
11           NGas_lag_2         454
3         Load_DA_lag_0         436
10           Coal_lag_2         282
5       WindOn_DA_lag_0         231
13            EUA_lag_2         224
15       lag168_Load_DA         127
12            Oil_lag_2         122
9                  WD_7          29
8                  WD_6          20
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          87
15       lag168_Load_DA          21
3         Load_DA_lag_0          13
16  roll2d_median_Price           5
1           Price_lag_2           3
5       WindOn_DA_lag_0           2
11           NGas_lag_2           2
13            EUA_lag_2           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
12            Oil_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 46.10% done
********************* END NS.. 336 ****************************************************
********************* START NS ... 337 ****************************************************
[I 2025-06-15 20:10:58,062] A new study created in memory with name: no-name-949d754c-9ac3-459a-8845-5b10e800a531
[I 2025-06-15 20:10:58,633] Trial 0 finished with value: 0.006929857760546704 and parameters: {'n_estimators': 105, 'learning_rate': 0.011043465061982961, 'num_leaves': 161, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.5727096316051103, 'colsample_bytree': 0.9756672518286111, 'reg_alpha': 6.474607909561715, 'reg_lambda': 1.1873702161216226, 'min_split_gain': 0.04965419563856267}. Best is trial 0 with value: 0.006929857760546704.
[Trial 0] New best: 0.00693
[I 2025-06-15 20:11:01,405] Trial 1 finished with value: 0.0037120901268396372 and parameters: {'n_estimators': 56, 'learning_rate': 0.12136858602218874, 'num_leaves': 187, 'max_depth': 15, 'min_child_samples': 5, 'subsample': 0.5799357776757017, 'colsample_bytree': 0.8817502958257808, 'reg_alpha': 0.22926867311097499, 'reg_lambda': 1.5765692208715287, 'min_split_gain': 0.010033095319894237}. Best is trial 1 with value: 0.0037120901268396372.
[Trial 1] New best: 0.00371
[I 2025-06-15 20:11:02,749] Trial 2 finished with value: 0.003421582688885883 and parameters: {'n_estimators': 261, 'learning_rate': 0.050852159559530034, 'num_leaves': 220, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.6788153434879801, 'colsample_bytree': 0.8326503969988333, 'reg_alpha': 8.866055941041184, 'reg_lambda': 3.6831913293742566, 'min_split_gain': 0.011052003293501568}. Best is trial 2 with value: 0.003421582688885883.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:11:02,997] Trial 3 finished with value: 0.003714294705209611 and parameters: {'n_estimators': 56, 'learning_rate': 0.051200477012810997, 'num_leaves': 194, 'max_depth': 4, 'min_child_samples': 2, 'subsample': 0.943287158615439, 'colsample_bytree': 0.906778390858844, 'reg_alpha': 4.593975728552236, 'reg_lambda': 0.7173253392938805, 'min_split_gain': 0.045753974027857434}. Best is trial 2 with value: 0.003421582688885883.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:11:03,484] Trial 4 finished with value: 0.01399118579937664 and parameters: {'n_estimators': 60, 'learning_rate': 0.01126955594279886, 'num_leaves': 137, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.7336645417594877, 'colsample_bytree': 0.5628955343492884, 'reg_alpha': 7.944393376020354, 'reg_lambda': 9.815121378698539, 'min_split_gain': 0.026479521263804547}. Best is trial 2 with value: 0.003421582688885883.
[I 2025-06-15 20:11:03,489] A new study created in memory with name: no-name-d47d2ea4-b4c2-43ac-9b22-77252d797bee
[I 2025-06-15 20:11:03,658] Trial 0 finished with value: 0.02818845419993098 and parameters: {'n_estimators': 281, 'learning_rate': 0.02721972828423997, 'num_leaves': 194, 'max_depth': 7, 'min_child_samples': 16, 'subsample': 0.6129884354694455, 'colsample_bytree': 0.7533559569474041, 'reg_alpha': 7.660712891416423, 'reg_lambda': 5.923388922257354, 'min_split_gain': 0.020040984722860606}. Best is trial 0 with value: 0.02818845419993098.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1096
1           Price_lag_2         815
14      pct_chg_Load_DA         670
3         Load_DA_lag_0         448
11           NGas_lag_2         376
2           Price_lag_7         364
16  roll2d_median_Price         347
5       WindOn_DA_lag_0         288
15       lag168_Load_DA         263
10           Coal_lag_2         252
13            EUA_lag_2         192
12            Oil_lag_2         132
9                  WD_7          54
7                  WD_1          25
8                  WD_6          19
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         201
1           Price_lag_2          32
15       lag168_Load_DA          31
3         Load_DA_lag_0          14
16  roll2d_median_Price           2
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 46.24% done
********************* END NS.. 337 ****************************************************
********************* START NS ... 338 ****************************************************
[I 2025-06-15 20:11:06,610] A new study created in memory with name: no-name-6a51f060-ce05-4e83-a7e7-58e7f4a6c917
[I 2025-06-15 20:11:07,343] Trial 0 finished with value: 0.05843678262981412 and parameters: {'n_estimators': 133, 'learning_rate': 0.12276902427620283, 'num_leaves': 69, 'max_depth': 11, 'min_child_samples': 12, 'subsample': 0.8906054029393866, 'colsample_bytree': 0.8245813153358762, 'reg_alpha': 9.3819389595712, 'reg_lambda': 1.139411164001306, 'min_split_gain': 0.014425268118799196}. Best is trial 0 with value: 0.05843678262981412.
[Trial 0] New best: 0.05844
[I 2025-06-15 20:11:07,838] Trial 1 finished with value: 0.07973479306512822 and parameters: {'n_estimators': 72, 'learning_rate': 0.02646517711134069, 'num_leaves': 188, 'max_depth': 12, 'min_child_samples': 25, 'subsample': 0.8207040773809979, 'colsample_bytree': 0.8450331955335303, 'reg_alpha': 9.65722422503262, 'reg_lambda': 9.63771600964255, 'min_split_gain': 0.0003676873280414939}. Best is trial 0 with value: 0.05843678262981412.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:11:08,462] Trial 2 finished with value: 0.06375823039797686 and parameters: {'n_estimators': 113, 'learning_rate': 0.04737228352360326, 'num_leaves': 109, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.8463026748647428, 'colsample_bytree': 0.7426501946601504, 'reg_alpha': 9.356927150619521, 'reg_lambda': 0.7961293254661816, 'min_split_gain': 0.006027929107104435}. Best is trial 0 with value: 0.05843678262981412.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:11:09,449] Trial 3 finished with value: 0.07392291347546785 and parameters: {'n_estimators': 152, 'learning_rate': 0.016683062131105448, 'num_leaves': 122, 'max_depth': 6, 'min_child_samples': 20, 'subsample': 0.6487350954396991, 'colsample_bytree': 0.9040792431954966, 'reg_alpha': 3.822354925737054, 'reg_lambda': 7.444352657897589, 'min_split_gain': 0.016888709341151997}. Best is trial 0 with value: 0.05843678262981412.
[I 2025-06-15 20:11:09,451] A new study created in memory with name: no-name-6671c572-d9f5-4a4c-b81e-e47ac3f6f56f
[I 2025-06-15 20:11:09,540] Trial 0 finished with value: 0.09033105734308162 and parameters: {'n_estimators': 239, 'learning_rate': 0.2950427706796835, 'num_leaves': 113, 'max_depth': 13, 'min_child_samples': 8, 'subsample': 0.8185904523632976, 'colsample_bytree': 0.5715483436429831, 'reg_alpha': 0.012789159283214113, 'reg_lambda': 4.887115439033419, 'min_split_gain': 0.04322810607584787}. Best is trial 0 with value: 0.09033105734308162.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         388
1           Price_lag_2         286
14      pct_chg_Load_DA         253
3         Load_DA_lag_0         162
11           NGas_lag_2         136
16  roll2d_median_Price         131
2           Price_lag_7         123
13            EUA_lag_2         121
5       WindOn_DA_lag_0          99
15       lag168_Load_DA          86
10           Coal_lag_2          80
12            Oil_lag_2          62
9                  WD_7          17
8                  WD_6           7
7                  WD_1           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          29
3         Load_DA_lag_0           9
14      pct_chg_Load_DA           9
16  roll2d_median_Price           9
1           Price_lag_2           7
2           Price_lag_7           7
15       lag168_Load_DA           7
5       WindOn_DA_lag_0           4
12            Oil_lag_2           4
11           NGas_lag_2           2
13            EUA_lag_2           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
8                  WD_6           0
-> 46.37% done
********************* END NS.. 338 ****************************************************
********************* START NS ... 339 ****************************************************
[I 2025-06-15 20:11:11,489] A new study created in memory with name: no-name-9626e61f-bbf4-4369-a3a2-1cfbb82896dc
[I 2025-06-15 20:11:11,997] Trial 0 finished with value: 0.13147254908044 and parameters: {'n_estimators': 201, 'learning_rate': 0.1931756974657822, 'num_leaves': 229, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.6058819768499708, 'colsample_bytree': 0.7661870128322202, 'reg_alpha': 4.3409681818420705, 'reg_lambda': 9.95183398778507, 'min_split_gain': 0.033737384097581195}. Best is trial 0 with value: 0.13147254908044.
[Trial 0] New best: 0.13147
[I 2025-06-15 20:11:16,139] Trial 1 finished with value: 0.16553715165898367 and parameters: {'n_estimators': 150, 'learning_rate': 0.020553398520773224, 'num_leaves': 160, 'max_depth': 16, 'min_child_samples': 2, 'subsample': 0.8982578492874826, 'colsample_bytree': 0.7636714600073222, 'reg_alpha': 1.5258882686666508, 'reg_lambda': 6.0032084611968095, 'min_split_gain': 0.006919967882174777}. Best is trial 0 with value: 0.13147254908044.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:11:17,452] Trial 2 finished with value: 0.12592326855794114 and parameters: {'n_estimators': 147, 'learning_rate': 0.11460219844418647, 'num_leaves': 119, 'max_depth': 9, 'min_child_samples': 3, 'subsample': 0.5917128558537015, 'colsample_bytree': 0.8305127261396517, 'reg_alpha': 7.5153791703099895, 'reg_lambda': 2.0197400668488372, 'min_split_gain': 0.0031501805061133928}. Best is trial 2 with value: 0.12592326855794114.
[Trial 2] New best: 0.12592
[I 2025-06-15 20:11:21,621] Trial 3 finished with value: 0.1622403841043709 and parameters: {'n_estimators': 236, 'learning_rate': 0.02150362148813145, 'num_leaves': 191, 'max_depth': 12, 'min_child_samples': 9, 'subsample': 0.6380305924122023, 'colsample_bytree': 0.5779978199929368, 'reg_alpha': 7.0767597255172, 'reg_lambda': 5.316188242690666, 'min_split_gain': 0.004602461469767416}. Best is trial 2 with value: 0.12592326855794114.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:11:24,010] Trial 4 finished with value: 0.14554680428245914 and parameters: {'n_estimators': 295, 'learning_rate': 0.012546012099354355, 'num_leaves': 165, 'max_depth': 6, 'min_child_samples': 29, 'subsample': 0.5021329119882529, 'colsample_bytree': 0.9785984384647359, 'reg_alpha': 1.651513989533584, 'reg_lambda': 1.8080594020024943, 'min_split_gain': 0.02887243147415502}. Best is trial 2 with value: 0.12592326855794114.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:11:24,569] Trial 5 finished with value: 0.1389613225957376 and parameters: {'n_estimators': 67, 'learning_rate': 0.08915446942870864, 'num_leaves': 102, 'max_depth': 12, 'min_child_samples': 4, 'subsample': 0.8403673121659475, 'colsample_bytree': 0.8770994271261319, 'reg_alpha': 9.128503399328865, 'reg_lambda': 9.080582346381952, 'min_split_gain': 0.01289743342931738}. Best is trial 2 with value: 0.12592326855794114.
[I 2025-06-15 20:11:24,572] A new study created in memory with name: no-name-4f86ffc6-a51b-4137-9c97-2bb91648b0f8
[I 2025-06-15 20:11:24,628] Trial 0 finished with value: 0.2332228000946374 and parameters: {'n_estimators': 134, 'learning_rate': 0.1441456310561041, 'num_leaves': 73, 'max_depth': 14, 'min_child_samples': 23, 'subsample': 0.7620657541291813, 'colsample_bytree': 0.6348701473858498, 'reg_alpha': 8.416976992920745, 'reg_lambda': 7.608972796676427, 'min_split_gain': 0.014074716894418433}. Best is trial 0 with value: 0.2332228000946374.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         678
0           Price_lag_1         657
14      pct_chg_Load_DA         530
3         Load_DA_lag_0         407
2           Price_lag_7         379
5       WindOn_DA_lag_0         367
16  roll2d_median_Price         293
15       lag168_Load_DA         279
11           NGas_lag_2         273
13            EUA_lag_2         260
10           Coal_lag_2         248
12            Oil_lag_2         121
9                  WD_7          27
7                  WD_1          13
8                  WD_6          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          39
15       lag168_Load_DA           7
1           Price_lag_2           2
3         Load_DA_lag_0           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 46.51% done
********************* END NS.. 339 ****************************************************
********************* START NS ... 340 ****************************************************
[I 2025-06-15 20:11:26,446] A new study created in memory with name: no-name-58306d13-da38-4167-8e26-9d63ad39dbd2
[I 2025-06-15 20:11:28,457] Trial 0 finished with value: 0.08179446923027117 and parameters: {'n_estimators': 171, 'learning_rate': 0.02544994282730942, 'num_leaves': 205, 'max_depth': 15, 'min_child_samples': 17, 'subsample': 0.5174733394861102, 'colsample_bytree': 0.7870769434063281, 'reg_alpha': 1.2631192623691057, 'reg_lambda': 5.063670522969072, 'min_split_gain': 0.03609362715897366}. Best is trial 0 with value: 0.08179446923027117.
[Trial 0] New best: 0.08179
[I 2025-06-15 20:11:28,801] Trial 1 finished with value: 0.07710627946168919 and parameters: {'n_estimators': 79, 'learning_rate': 0.24369866439921722, 'num_leaves': 216, 'max_depth': 13, 'min_child_samples': 23, 'subsample': 0.7606319716055501, 'colsample_bytree': 0.755431483926191, 'reg_alpha': 8.619513850362363, 'reg_lambda': 8.517578032086444, 'min_split_gain': 0.004527117376647366}. Best is trial 1 with value: 0.07710627946168919.
[Trial 1] New best: 0.07711
[I 2025-06-15 20:11:29,216] Trial 2 finished with value: 0.0832850791709963 and parameters: {'n_estimators': 255, 'learning_rate': 0.07495056906074354, 'num_leaves': 168, 'max_depth': 4, 'min_child_samples': 13, 'subsample': 0.6254253551629353, 'colsample_bytree': 0.9068685732234879, 'reg_alpha': 2.049925717787975, 'reg_lambda': 5.557374160354927, 'min_split_gain': 0.0427284921859723}. Best is trial 1 with value: 0.07710627946168919.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:11:29,858] Trial 3 finished with value: 0.08395054783851813 and parameters: {'n_estimators': 157, 'learning_rate': 0.07119756683771387, 'num_leaves': 172, 'max_depth': 8, 'min_child_samples': 4, 'subsample': 0.8465961616249391, 'colsample_bytree': 0.6725530991263573, 'reg_alpha': 6.630700634266305, 'reg_lambda': 3.7329957392458657, 'min_split_gain': 0.009610414480858354}. Best is trial 1 with value: 0.07710627946168919.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:11:30,216] Trial 4 finished with value: 0.0929824311670517 and parameters: {'n_estimators': 216, 'learning_rate': 0.19094289585983204, 'num_leaves': 179, 'max_depth': 10, 'min_child_samples': 20, 'subsample': 0.5712864537862621, 'colsample_bytree': 0.853907915576112, 'reg_alpha': 1.7312346400559486, 'reg_lambda': 2.995904706867145, 'min_split_gain': 0.019430649528078905}. Best is trial 1 with value: 0.07710627946168919.
[I 2025-06-15 20:11:30,219] A new study created in memory with name: no-name-ea55be93-4d02-419b-a94d-3e2ec0d4db6d
[I 2025-06-15 20:11:30,245] Trial 0 finished with value: 0.0983039937100177 and parameters: {'n_estimators': 121, 'learning_rate': 0.2781458376750181, 'num_leaves': 173, 'max_depth': 10, 'min_child_samples': 19, 'subsample': 0.609838730730285, 'colsample_bytree': 0.8830133144537174, 'reg_alpha': 2.0063670402285503, 'reg_lambda': 3.685091276510483, 'min_split_gain': 0.03941526564932765}. Best is trial 0 with value: 0.0983039937100177.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         276
1           Price_lag_2         203
14      pct_chg_Load_DA         167
13            EUA_lag_2         112
3         Load_DA_lag_0         109
11           NGas_lag_2          98
16  roll2d_median_Price          86
2           Price_lag_7          83
15       lag168_Load_DA          82
10           Coal_lag_2          72
5       WindOn_DA_lag_0          69
12            Oil_lag_2          50
9                  WD_7           9
7                  WD_1           8
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          20
2           Price_lag_7           6
3         Load_DA_lag_0           6
14      pct_chg_Load_DA           6
15       lag168_Load_DA           1
1           Price_lag_2           1
10           Coal_lag_2           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 46.65% done
********************* END NS.. 340 ****************************************************
********************* START NS ... 341 ****************************************************
[I 2025-06-15 20:11:31,492] A new study created in memory with name: no-name-0f421e34-06dd-4dff-9455-de3943f7b3b3
[I 2025-06-15 20:11:31,711] Trial 0 finished with value: 0.016034696640506024 and parameters: {'n_estimators': 228, 'learning_rate': 0.1479037968881446, 'num_leaves': 199, 'max_depth': 13, 'min_child_samples': 16, 'subsample': 0.9571529659986556, 'colsample_bytree': 0.5573305960422206, 'reg_alpha': 0.9377180400147767, 'reg_lambda': 0.48622715242074843, 'min_split_gain': 0.04541557551831497}. Best is trial 0 with value: 0.016034696640506024.
[Trial 0] New best: 0.01603
[I 2025-06-15 20:11:32,020] Trial 1 finished with value: 0.014106037108631042 and parameters: {'n_estimators': 95, 'learning_rate': 0.03188145542301801, 'num_leaves': 116, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.60022202518076, 'colsample_bytree': 0.5673673725864024, 'reg_alpha': 6.27197779749038, 'reg_lambda': 8.89038201150148, 'min_split_gain': 0.024124372752230862}. Best is trial 1 with value: 0.014106037108631042.
[Trial 1] New best: 0.01411
[I 2025-06-15 20:11:32,370] Trial 2 finished with value: 0.008370176972967708 and parameters: {'n_estimators': 208, 'learning_rate': 0.18891304942344697, 'num_leaves': 248, 'max_depth': 8, 'min_child_samples': 17, 'subsample': 0.8349221362857406, 'colsample_bytree': 0.7987888607222982, 'reg_alpha': 5.068868408080586, 'reg_lambda': 8.766947841215144, 'min_split_gain': 0.002679114066941407}. Best is trial 2 with value: 0.008370176972967708.
[Trial 2] New best: 0.00837
[I 2025-06-15 20:11:33,549] Trial 3 finished with value: 0.008557776217204558 and parameters: {'n_estimators': 255, 'learning_rate': 0.022330661045083085, 'num_leaves': 39, 'max_depth': 11, 'min_child_samples': 13, 'subsample': 0.5650628116072989, 'colsample_bytree': 0.8088648719954119, 'reg_alpha': 6.1749815229878635, 'reg_lambda': 6.7189297115454805, 'min_split_gain': 0.027241904003720327}. Best is trial 2 with value: 0.008370176972967708.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:11:33,861] Trial 4 finished with value: 0.010346036321766549 and parameters: {'n_estimators': 94, 'learning_rate': 0.09252254055279661, 'num_leaves': 61, 'max_depth': 14, 'min_child_samples': 8, 'subsample': 0.889435604849424, 'colsample_bytree': 0.7856692905753918, 'reg_alpha': 9.396776323825172, 'reg_lambda': 0.016071063302199873, 'min_split_gain': 0.03756511720797719}. Best is trial 2 with value: 0.008370176972967708.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:11:34,252] Trial 5 finished with value: 0.009558531617803866 and parameters: {'n_estimators': 78, 'learning_rate': 0.03153366703760681, 'num_leaves': 234, 'max_depth': 16, 'min_child_samples': 26, 'subsample': 0.8535264080328706, 'colsample_bytree': 0.879923722925581, 'reg_alpha': 3.1239524036694064, 'reg_lambda': 4.4387222722381825, 'min_split_gain': 0.0012290849269004202}. Best is trial 2 with value: 0.008370176972967708.
[I 2025-06-15 20:11:34,255] A new study created in memory with name: no-name-067a77ef-25e6-4ccd-ba04-6cbd1e9f28b8
[I 2025-06-15 20:11:34,277] Trial 0 finished with value: 0.021490271640978994 and parameters: {'n_estimators': 62, 'learning_rate': 0.1785689561010547, 'num_leaves': 192, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.5914818216002538, 'colsample_bytree': 0.628379208128355, 'reg_alpha': 0.851177559630778, 'reg_lambda': 1.8425573324226052, 'min_split_gain': 0.04163279740494188}. Best is trial 0 with value: 0.021490271640978994.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         506
1           Price_lag_2         413
14      pct_chg_Load_DA         329
3         Load_DA_lag_0         275
13            EUA_lag_2         240
15       lag168_Load_DA         219
5       WindOn_DA_lag_0         194
16  roll2d_median_Price         186
2           Price_lag_7         181
11           NGas_lag_2         171
12            Oil_lag_2         146
10           Coal_lag_2         125
9                  WD_7          16
7                  WD_1          11
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          27
2           Price_lag_7          13
3         Load_DA_lag_0          12
1           Price_lag_2          11
15       lag168_Load_DA           7
14      pct_chg_Load_DA           6
10           Coal_lag_2           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 46.79% done
********************* END NS.. 341 ****************************************************
********************* START NS ... 342 ****************************************************
[I 2025-06-15 20:11:35,384] A new study created in memory with name: no-name-39ec8204-c976-4f4b-8c54-8f3b7869f91b
[I 2025-06-15 20:11:35,626] Trial 0 finished with value: 0.005741843602608987 and parameters: {'n_estimators': 75, 'learning_rate': 0.15303690160171282, 'num_leaves': 49, 'max_depth': 14, 'min_child_samples': 18, 'subsample': 0.5683969990561242, 'colsample_bytree': 0.6881081616343013, 'reg_alpha': 1.2481015868213696, 'reg_lambda': 7.791096995455518, 'min_split_gain': 0.011008491809591486}. Best is trial 0 with value: 0.005741843602608987.
[Trial 0] New best: 0.00574
[I 2025-06-15 20:11:36,084] Trial 1 finished with value: 0.004261542367047339 and parameters: {'n_estimators': 298, 'learning_rate': 0.07642982232158138, 'num_leaves': 116, 'max_depth': 7, 'min_child_samples': 8, 'subsample': 0.5575618075644116, 'colsample_bytree': 0.9950714034026995, 'reg_alpha': 6.492461866022895, 'reg_lambda': 0.10972477164881256, 'min_split_gain': 0.04242961280866964}. Best is trial 1 with value: 0.004261542367047339.
[Trial 1] New best: 0.00426
[I 2025-06-15 20:11:37,151] Trial 2 finished with value: 0.004039995820249216 and parameters: {'n_estimators': 251, 'learning_rate': 0.013111643657584781, 'num_leaves': 162, 'max_depth': 16, 'min_child_samples': 9, 'subsample': 0.6262840540841452, 'colsample_bytree': 0.657424433246451, 'reg_alpha': 9.070005172168301, 'reg_lambda': 7.4363719467535825, 'min_split_gain': 0.0016546199392787509}. Best is trial 2 with value: 0.004039995820249216.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:11:37,906] Trial 3 finished with value: 0.0055730993275890095 and parameters: {'n_estimators': 217, 'learning_rate': 0.08453023185215904, 'num_leaves': 253, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.5173340874799764, 'colsample_bytree': 0.6903180871413419, 'reg_alpha': 8.431551905935532, 'reg_lambda': 3.5236822103270047, 'min_split_gain': 0.003110500689277801}. Best is trial 2 with value: 0.004039995820249216.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:11:38,224] Trial 4 finished with value: 0.006082538462233806 and parameters: {'n_estimators': 188, 'learning_rate': 0.02297869653806131, 'num_leaves': 127, 'max_depth': 4, 'min_child_samples': 28, 'subsample': 0.8178328611065858, 'colsample_bytree': 0.5428299749790995, 'reg_alpha': 5.697232112976632, 'reg_lambda': 2.9714366421019465, 'min_split_gain': 0.0407920386852712}. Best is trial 2 with value: 0.004039995820249216.
[I 2025-06-15 20:11:38,228] A new study created in memory with name: no-name-89b896ac-a9e5-4a79-88ec-d10903ab61ce
[I 2025-06-15 20:11:38,255] Trial 0 finished with value: 0.03457770288899673 and parameters: {'n_estimators': 179, 'learning_rate': 0.2866271497131566, 'num_leaves': 238, 'max_depth': 10, 'min_child_samples': 27, 'subsample': 0.8891103650215639, 'colsample_bytree': 0.6267435812642095, 'reg_alpha': 8.127660861571295, 'reg_lambda': 3.289333360542299, 'min_split_gain': 0.007175295052199205}. Best is trial 0 with value: 0.03457770288899673.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2350
1           Price_lag_2        1934
14      pct_chg_Load_DA        1181
16  roll2d_median_Price         899
2           Price_lag_7         827
11           NGas_lag_2         569
3         Load_DA_lag_0         455
10           Coal_lag_2         375
15       lag168_Load_DA         354
13            EUA_lag_2         351
12            Oil_lag_2         282
5       WindOn_DA_lag_0         251
9                  WD_7          21
8                  WD_6           7
7                  WD_1           3
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          11
3         Load_DA_lag_0           9
15       lag168_Load_DA           2
1           Price_lag_2           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 46.92% done
********************* END NS.. 342 ****************************************************
********************* START NS ... 343 ****************************************************
[I 2025-06-15 20:11:41,207] A new study created in memory with name: no-name-9a699993-dbc7-43ae-bd22-52f9c3048b22
[I 2025-06-15 20:11:42,093] Trial 0 finished with value: 0.002283068295989263 and parameters: {'n_estimators': 197, 'learning_rate': 0.05387512573989346, 'num_leaves': 57, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.6449057424789912, 'colsample_bytree': 0.6017129692546745, 'reg_alpha': 5.58287672597047, 'reg_lambda': 8.277208875248068, 'min_split_gain': 0.013759225569412692}. Best is trial 0 with value: 0.002283068295989263.
[Trial 0] New best: 0.00228
[I 2025-06-15 20:11:42,548] Trial 1 finished with value: 0.002118035035676077 and parameters: {'n_estimators': 116, 'learning_rate': 0.04381795916049481, 'num_leaves': 149, 'max_depth': 15, 'min_child_samples': 12, 'subsample': 0.7691970326156647, 'colsample_bytree': 0.6289291810767947, 'reg_alpha': 5.510346834997418, 'reg_lambda': 3.4183310231719566, 'min_split_gain': 0.04427898783988175}. Best is trial 1 with value: 0.002118035035676077.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:11:42,778] Trial 2 finished with value: 0.0017553413391040893 and parameters: {'n_estimators': 69, 'learning_rate': 0.05255786467054295, 'num_leaves': 70, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.7143230950591565, 'colsample_bytree': 0.6720064904592278, 'reg_alpha': 9.74239633110853, 'reg_lambda': 4.091262886692904, 'min_split_gain': 0.0278675299468444}. Best is trial 2 with value: 0.0017553413391040893.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:11:43,066] Trial 3 finished with value: 0.0013335854777882716 and parameters: {'n_estimators': 288, 'learning_rate': 0.24175154548480066, 'num_leaves': 60, 'max_depth': 14, 'min_child_samples': 19, 'subsample': 0.8678390268663396, 'colsample_bytree': 0.8224702532457682, 'reg_alpha': 8.429910202329902, 'reg_lambda': 2.631267852664841, 'min_split_gain': 0.011363561834083175}. Best is trial 3 with value: 0.0013335854777882716.
[I 2025-06-15 20:11:43,069] A new study created in memory with name: no-name-b1682e2b-c7a9-4622-b41f-13038f391713
[I 2025-06-15 20:11:43,101] Trial 0 finished with value: 0.053358698546962215 and parameters: {'n_estimators': 256, 'learning_rate': 0.15024373692135767, 'num_leaves': 46, 'max_depth': 7, 'min_child_samples': 26, 'subsample': 0.7094886338376196, 'colsample_bytree': 0.8732806314282038, 'reg_alpha': 1.227886411574488, 'reg_lambda': 4.824905035681187, 'min_split_gain': 0.025996872992891846}. Best is trial 0 with value: 0.053358698546962215.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         223
14      pct_chg_Load_DA         151
1           Price_lag_2         136
3         Load_DA_lag_0          83
2           Price_lag_7          74
16  roll2d_median_Price          68
13            EUA_lag_2          63
11           NGas_lag_2          57
15       lag168_Load_DA          54
10           Coal_lag_2          49
5       WindOn_DA_lag_0          43
12            Oil_lag_2          34
9                  WD_7           7
8                  WD_6           5
7                  WD_1           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          29
3         Load_DA_lag_0          16
5       WindOn_DA_lag_0           7
14      pct_chg_Load_DA           7
15       lag168_Load_DA           6
16  roll2d_median_Price           5
2           Price_lag_7           4
1           Price_lag_2           1
12            Oil_lag_2           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
13            EUA_lag_2           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 47.06% done
********************* END NS.. 343 ****************************************************
********************* START NS ... 344 ****************************************************
[I 2025-06-15 20:11:44,905] A new study created in memory with name: no-name-cd2c322e-96a1-44c9-b722-4e79ed8be9b4
[I 2025-06-15 20:11:45,511] Trial 0 finished with value: 0.0017405345025410166 and parameters: {'n_estimators': 198, 'learning_rate': 0.0428429819613372, 'num_leaves': 200, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.613509510668608, 'colsample_bytree': 0.9350114102857754, 'reg_alpha': 8.753075085181331, 'reg_lambda': 7.107426426691632, 'min_split_gain': 0.009979170961628864}. Best is trial 0 with value: 0.0017405345025410166.
[Trial 0] New best: 0.00174
[I 2025-06-15 20:11:45,754] Trial 1 finished with value: 0.0022274020421115585 and parameters: {'n_estimators': 223, 'learning_rate': 0.2665660790242977, 'num_leaves': 31, 'max_depth': 11, 'min_child_samples': 1, 'subsample': 0.9043836257586153, 'colsample_bytree': 0.5933604270911625, 'reg_alpha': 5.566728577594001, 'reg_lambda': 9.164486860982343, 'min_split_gain': 0.020352293673711705}. Best is trial 0 with value: 0.0017405345025410166.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:11:46,543] Trial 2 finished with value: 0.0032388690896324192 and parameters: {'n_estimators': 130, 'learning_rate': 0.018611710314800373, 'num_leaves': 71, 'max_depth': 12, 'min_child_samples': 25, 'subsample': 0.6097966299467339, 'colsample_bytree': 0.8300783065094441, 'reg_alpha': 2.5055107903631213, 'reg_lambda': 6.814868253942501, 'min_split_gain': 0.009860570811529763}. Best is trial 0 with value: 0.0017405345025410166.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:11:46,875] Trial 3 finished with value: 0.02184952163597437 and parameters: {'n_estimators': 79, 'learning_rate': 0.013127899014331201, 'num_leaves': 250, 'max_depth': 4, 'min_child_samples': 27, 'subsample': 0.5499791248373571, 'colsample_bytree': 0.5188886104405919, 'reg_alpha': 0.5706989344786517, 'reg_lambda': 3.8521498356163066, 'min_split_gain': 0.007002751562720677}. Best is trial 0 with value: 0.0017405345025410166.
[I 2025-06-15 20:11:46,879] A new study created in memory with name: no-name-f1266c36-b1fe-4dd7-a54a-3623599838c2
[I 2025-06-15 20:11:46,957] Trial 0 finished with value: 0.19005579090924477 and parameters: {'n_estimators': 142, 'learning_rate': 0.06385164032155242, 'num_leaves': 250, 'max_depth': 8, 'min_child_samples': 2, 'subsample': 0.6599127363203918, 'colsample_bytree': 0.7780516246204814, 'reg_alpha': 5.97663015142909, 'reg_lambda': 5.466457100832953, 'min_split_gain': 0.004742078047455945}. Best is trial 0 with value: 0.19005579090924477.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1312
1           Price_lag_2         900
14      pct_chg_Load_DA         848
11           NGas_lag_2         502
2           Price_lag_7         476
3         Load_DA_lag_0         468
16  roll2d_median_Price         463
5       WindOn_DA_lag_0         363
13            EUA_lag_2         360
10           Coal_lag_2         322
15       lag168_Load_DA         308
12            Oil_lag_2         208
9                  WD_7          56
8                  WD_6          24
7                  WD_1           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
3         Load_DA_lag_0         119
0           Price_lag_1         100
16  roll2d_median_Price          18
15       lag168_Load_DA          14
5       WindOn_DA_lag_0          13
1           Price_lag_2           8
10           Coal_lag_2           7
14      pct_chg_Load_DA           6
2           Price_lag_7           3
13            EUA_lag_2           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
-> 47.20% done
********************* END NS.. 344 ****************************************************
********************* START NS ... 345 ****************************************************
[I 2025-06-15 20:11:49,029] A new study created in memory with name: no-name-1879539e-937e-4e3e-9c86-bd73d500670f
[I 2025-06-15 20:11:49,566] Trial 0 finished with value: 0.01220896907455613 and parameters: {'n_estimators': 194, 'learning_rate': 0.010342169623572234, 'num_leaves': 44, 'max_depth': 10, 'min_child_samples': 2, 'subsample': 0.8109112129195555, 'colsample_bytree': 0.521602064091931, 'reg_alpha': 5.768827678389429, 'reg_lambda': 9.520196652524005, 'min_split_gain': 0.045890067511463614}. Best is trial 0 with value: 0.01220896907455613.
[Trial 0] New best: 0.01221
[I 2025-06-15 20:11:50,467] Trial 1 finished with value: 0.00329225869703263 and parameters: {'n_estimators': 172, 'learning_rate': 0.0754273127897827, 'num_leaves': 223, 'max_depth': 13, 'min_child_samples': 16, 'subsample': 0.9662193639609032, 'colsample_bytree': 0.674114715036583, 'reg_alpha': 3.797547645815235, 'reg_lambda': 2.8571346625490515, 'min_split_gain': 0.00331525645727182}. Best is trial 1 with value: 0.00329225869703263.
[Trial 1] New best: 0.00329
[I 2025-06-15 20:11:50,818] Trial 2 finished with value: 0.004707880386429462 and parameters: {'n_estimators': 160, 'learning_rate': 0.0860382578735387, 'num_leaves': 104, 'max_depth': 8, 'min_child_samples': 19, 'subsample': 0.6746141656788371, 'colsample_bytree': 0.5476884975505678, 'reg_alpha': 8.161499084178931, 'reg_lambda': 8.413705483340232, 'min_split_gain': 0.021466632557110777}. Best is trial 1 with value: 0.00329225869703263.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:11:51,209] Trial 3 finished with value: 0.003166357547146043 and parameters: {'n_estimators': 133, 'learning_rate': 0.07078606070266696, 'num_leaves': 251, 'max_depth': 7, 'min_child_samples': 26, 'subsample': 0.7884612604319843, 'colsample_bytree': 0.8416478355795829, 'reg_alpha': 2.805350677810603, 'reg_lambda': 9.051786051436958, 'min_split_gain': 0.030085438944500043}. Best is trial 3 with value: 0.003166357547146043.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:11:51,699] Trial 4 finished with value: 0.014644091164418246 and parameters: {'n_estimators': 120, 'learning_rate': 0.01039579699669075, 'num_leaves': 158, 'max_depth': 14, 'min_child_samples': 24, 'subsample': 0.6113457113177361, 'colsample_bytree': 0.8158447513703133, 'reg_alpha': 9.845281909531536, 'reg_lambda': 7.60995308972735, 'min_split_gain': 0.021701628660199986}. Best is trial 3 with value: 0.003166357547146043.
[I 2025-06-15 20:11:51,702] A new study created in memory with name: no-name-2d46cdb8-8d2f-4410-bf61-97d4119d401f
[I 2025-06-15 20:11:51,739] Trial 0 finished with value: 0.12958666423476864 and parameters: {'n_estimators': 214, 'learning_rate': 0.056207094879825115, 'num_leaves': 155, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.9834315713316868, 'colsample_bytree': 0.7126026254992448, 'reg_alpha': 8.198959254356136, 'reg_lambda': 0.03551944553458419, 'min_split_gain': 0.03673515237117971}. Best is trial 0 with value: 0.12958666423476864.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         903
1           Price_lag_2         649
14      pct_chg_Load_DA         630
3         Load_DA_lag_0         397
11           NGas_lag_2         269
16  roll2d_median_Price         248
2           Price_lag_7         235
5       WindOn_DA_lag_0         232
15       lag168_Load_DA         228
10           Coal_lag_2         145
13            EUA_lag_2         125
12            Oil_lag_2         117
9                  WD_7          17
7                  WD_1          15
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
3         Load_DA_lag_0          62
0           Price_lag_1          49
10           Coal_lag_2          19
15       lag168_Load_DA           6
1           Price_lag_2           2
13            EUA_lag_2           2
16  roll2d_median_Price           2
2           Price_lag_7           2
7                  WD_1           0
6      WindOff_DA_lag_0           0
9                  WD_7           0
5       WindOn_DA_lag_0           0
11           NGas_lag_2           0
12            Oil_lag_2           0
4        Solar_DA_lag_0           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 47.33% done
********************* END NS.. 345 ****************************************************
********************* START NS ... 346 ****************************************************
[I 2025-06-15 20:11:53,106] A new study created in memory with name: no-name-147879b7-27b7-48e9-bd6f-9349d657e011
[I 2025-06-15 20:11:53,860] Trial 0 finished with value: 0.005753501875549796 and parameters: {'n_estimators': 239, 'learning_rate': 0.02689365412142662, 'num_leaves': 130, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.9484840655486403, 'colsample_bytree': 0.5916352954925082, 'reg_alpha': 3.4592887268413564, 'reg_lambda': 4.6743627002354895, 'min_split_gain': 0.014437692617128794}. Best is trial 0 with value: 0.005753501875549796.
[I 2025-06-15 20:11:54,025] Trial 1 finished with value: 0.00902946222489626 and parameters: {'n_estimators': 58, 'learning_rate': 0.022436618730211043, 'num_leaves': 133, 'max_depth': 5, 'min_child_samples': 3, 'subsample': 0.8973627483393352, 'colsample_bytree': 0.9368661320772951, 'reg_alpha': 1.0307985238202333, 'reg_lambda': 9.364926783251494, 'min_split_gain': 0.0063076868984781155}. Best is trial 0 with value: 0.005753501875549796.
[Trial 0] New best: 0.00575
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:11:54,345] Trial 2 finished with value: 0.004429876662632399 and parameters: {'n_estimators': 128, 'learning_rate': 0.08602823690057812, 'num_leaves': 255, 'max_depth': 6, 'min_child_samples': 8, 'subsample': 0.6193369434411411, 'colsample_bytree': 0.8203640908228695, 'reg_alpha': 0.9632874925124568, 'reg_lambda': 6.897994466132667, 'min_split_gain': 0.028905643522782243}. Best is trial 2 with value: 0.004429876662632399.
[Trial 2] New best: 0.00443
[I 2025-06-15 20:11:54,611] Trial 3 finished with value: 0.012166202611020423 and parameters: {'n_estimators': 65, 'learning_rate': 0.01869576224808003, 'num_leaves': 52, 'max_depth': 12, 'min_child_samples': 4, 'subsample': 0.8231350638844837, 'colsample_bytree': 0.7327089527368669, 'reg_alpha': 7.644552116975699, 'reg_lambda': 1.1072101849508875, 'min_split_gain': 0.048309450971339374}. Best is trial 2 with value: 0.004429876662632399.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:11:55,797] Trial 4 finished with value: 0.00564690242421909 and parameters: {'n_estimators': 259, 'learning_rate': 0.019786545151611996, 'num_leaves': 72, 'max_depth': 14, 'min_child_samples': 5, 'subsample': 0.5429705734116412, 'colsample_bytree': 0.6601939294285759, 'reg_alpha': 9.966576474720688, 'reg_lambda': 2.09221515196502, 'min_split_gain': 0.0007794317130195183}. Best is trial 2 with value: 0.004429876662632399.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:11:56,010] Trial 5 finished with value: 0.004014683110362716 and parameters: {'n_estimators': 269, 'learning_rate': 0.26134973913278225, 'num_leaves': 78, 'max_depth': 7, 'min_child_samples': 25, 'subsample': 0.5510955307446506, 'colsample_bytree': 0.9989067898898027, 'reg_alpha': 5.546777539149107, 'reg_lambda': 3.243080449405488, 'min_split_gain': 0.02519106520035219}. Best is trial 5 with value: 0.004014683110362716.
[I 2025-06-15 20:11:56,012] A new study created in memory with name: no-name-3f777af2-33c2-4e4a-812c-8417e700060d
[I 2025-06-15 20:11:56,071] Trial 0 finished with value: 0.01653700163844383 and parameters: {'n_estimators': 159, 'learning_rate': 0.06878350454319025, 'num_leaves': 206, 'max_depth': 5, 'min_child_samples': 19, 'subsample': 0.8703217855450287, 'colsample_bytree': 0.5701960858417858, 'reg_alpha': 0.7819037406243368, 'reg_lambda': 2.8039391392151303, 'min_split_gain': 0.02618329212077718}. Best is trial 0 with value: 0.01653700163844383.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         210
14      pct_chg_Load_DA         130
1           Price_lag_2         113
3         Load_DA_lag_0          77
2           Price_lag_7          66
11           NGas_lag_2          65
5       WindOn_DA_lag_0          51
16  roll2d_median_Price          50
15       lag168_Load_DA          42
10           Coal_lag_2          30
13            EUA_lag_2          22
12            Oil_lag_2          17
7                  WD_1           4
8                  WD_6           3
9                  WD_7           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          80
3         Load_DA_lag_0          35
1           Price_lag_2          13
15       lag168_Load_DA          12
10           Coal_lag_2          10
14      pct_chg_Load_DA           6
2           Price_lag_7           5
5       WindOn_DA_lag_0           3
16  roll2d_median_Price           2
9                  WD_7           0
7                  WD_1           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 47.47% done
********************* END NS.. 346 ****************************************************
********************* START NS ... 347 ****************************************************
[I 2025-06-15 20:11:56,975] A new study created in memory with name: no-name-c950dbe5-d7b6-4bd7-b186-bd16da9d7f44
[I 2025-06-15 20:11:57,345] Trial 0 finished with value: 0.0037091767901421893 and parameters: {'n_estimators': 138, 'learning_rate': 0.06792096558740221, 'num_leaves': 216, 'max_depth': 9, 'min_child_samples': 20, 'subsample': 0.913674259985231, 'colsample_bytree': 0.5481470290716857, 'reg_alpha': 8.480642101527888, 'reg_lambda': 3.435398238544043, 'min_split_gain': 0.021286955310221286}. Best is trial 0 with value: 0.0037091767901421893.
[Trial 0] New best: 0.00371
[I 2025-06-15 20:11:57,577] Trial 1 finished with value: 0.002898522718380973 and parameters: {'n_estimators': 265, 'learning_rate': 0.091442868140347, 'num_leaves': 57, 'max_depth': 5, 'min_child_samples': 11, 'subsample': 0.5118151944004519, 'colsample_bytree': 0.8336702264069462, 'reg_alpha': 8.282752162667265, 'reg_lambda': 4.202153435905666, 'min_split_gain': 0.03342918922299614}. Best is trial 1 with value: 0.002898522718380973.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:11:57,928] Trial 2 finished with value: 0.0031565604037471066 and parameters: {'n_estimators': 279, 'learning_rate': 0.12442316406410492, 'num_leaves': 79, 'max_depth': 11, 'min_child_samples': 30, 'subsample': 0.9084448220389323, 'colsample_bytree': 0.894785493386353, 'reg_alpha': 5.737362620286987, 'reg_lambda': 6.504025234520019, 'min_split_gain': 0.020731004019208632}. Best is trial 1 with value: 0.002898522718380973.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:11:58,733] Trial 3 finished with value: 0.0033268159894686705 and parameters: {'n_estimators': 295, 'learning_rate': 0.023962134740901017, 'num_leaves': 196, 'max_depth': 16, 'min_child_samples': 19, 'subsample': 0.8997705557560015, 'colsample_bytree': 0.9465977195602938, 'reg_alpha': 5.957507080844847, 'reg_lambda': 6.866681740311929, 'min_split_gain': 0.04299002554924117}. Best is trial 1 with value: 0.002898522718380973.
[I 2025-06-15 20:11:58,736] A new study created in memory with name: no-name-5abb3001-cbb9-4c40-b0a2-23c58c89c306
[I 2025-06-15 20:11:58,765] Trial 0 finished with value: 0.03475140506924979 and parameters: {'n_estimators': 89, 'learning_rate': 0.024598075212957066, 'num_leaves': 209, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.8353529152518978, 'colsample_bytree': 0.6269318392333934, 'reg_alpha': 9.584437650767438, 'reg_lambda': 6.581796515019814, 'min_split_gain': 0.041191256629227124}. Best is trial 0 with value: 0.03475140506924979.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         453
14      pct_chg_Load_DA         245
1           Price_lag_2         188
3         Load_DA_lag_0         131
2           Price_lag_7         114
11           NGas_lag_2          98
16  roll2d_median_Price          94
15       lag168_Load_DA          64
10           Coal_lag_2          60
13            EUA_lag_2          55
5       WindOn_DA_lag_0          46
12            Oil_lag_2          34
7                  WD_1           6
9                  WD_7           5
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          49
3         Load_DA_lag_0          22
1           Price_lag_2           5
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 47.61% done
********************* END NS.. 347 ****************************************************
********************* START NS ... 348 ****************************************************
[I 2025-06-15 20:12:00,179] A new study created in memory with name: no-name-e65448ec-3e3f-4d02-a7e3-582d9ea4ca5c
[I 2025-06-15 20:12:00,479] Trial 0 finished with value: 0.0024477338175498916 and parameters: {'n_estimators': 220, 'learning_rate': 0.06823523513965235, 'num_leaves': 163, 'max_depth': 16, 'min_child_samples': 13, 'subsample': 0.6016081163933669, 'colsample_bytree': 0.720006429176087, 'reg_alpha': 9.958061069465066, 'reg_lambda': 1.7857848933642517, 'min_split_gain': 0.012821290272394904}. Best is trial 0 with value: 0.0024477338175498916.
[I 2025-06-15 20:12:00,606] Trial 1 finished with value: 0.0025600269837471727 and parameters: {'n_estimators': 74, 'learning_rate': 0.1466037673974848, 'num_leaves': 155, 'max_depth': 10, 'min_child_samples': 7, 'subsample': 0.9979023814133734, 'colsample_bytree': 0.8491088278055017, 'reg_alpha': 7.0224628229840285, 'reg_lambda': 4.748338021805681, 'min_split_gain': 0.048353716998242924}. Best is trial 0 with value: 0.0024477338175498916.
[Trial 0] New best: 0.00245
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:01,108] Trial 2 finished with value: 0.0038227665871530397 and parameters: {'n_estimators': 184, 'learning_rate': 0.15801485712453558, 'num_leaves': 47, 'max_depth': 8, 'min_child_samples': 22, 'subsample': 0.7916919577731163, 'colsample_bytree': 0.5899142148173016, 'reg_alpha': 0.05011256220401217, 'reg_lambda': 9.322380512911888, 'min_split_gain': 0.000643968215609414}. Best is trial 0 with value: 0.0024477338175498916.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:01,690] Trial 3 finished with value: 0.004613271537075643 and parameters: {'n_estimators': 86, 'learning_rate': 0.03521112992110572, 'num_leaves': 162, 'max_depth': 16, 'min_child_samples': 1, 'subsample': 0.8329334217675471, 'colsample_bytree': 0.5095000161069242, 'reg_alpha': 2.562618498396745, 'reg_lambda': 7.507892266165699, 'min_split_gain': 0.005605998817794256}. Best is trial 0 with value: 0.0024477338175498916.
[I 2025-06-15 20:12:01,692] A new study created in memory with name: no-name-552f82d1-0900-4cf5-865b-b521bb9cfc15
[I 2025-06-15 20:12:01,709] Trial 0 finished with value: 0.03126884533741097 and parameters: {'n_estimators': 51, 'learning_rate': 0.22974497535413294, 'num_leaves': 167, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.5418294649923141, 'colsample_bytree': 0.6340347278353036, 'reg_alpha': 6.760263105137467, 'reg_lambda': 6.828084008473246, 'min_split_gain': 0.016162664349156804}. Best is trial 0 with value: 0.03126884533741097.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         706
1           Price_lag_2         523
14      pct_chg_Load_DA         449
3         Load_DA_lag_0         277
11           NGas_lag_2         252
2           Price_lag_7         226
16  roll2d_median_Price         215
15       lag168_Load_DA         185
5       WindOn_DA_lag_0         183
13            EUA_lag_2         183
10           Coal_lag_2         151
12            Oil_lag_2         136
9                  WD_7          30
8                  WD_6          16
7                  WD_1          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          13
3         Load_DA_lag_0           4
15       lag168_Load_DA           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 47.74% done
********************* END NS.. 348 ****************************************************
********************* START NS ... 349 ****************************************************
[I 2025-06-15 20:12:03,727] A new study created in memory with name: no-name-af10118f-a206-4f55-924d-cd206c259718
[I 2025-06-15 20:12:03,977] Trial 0 finished with value: 0.00471749676456151 and parameters: {'n_estimators': 126, 'learning_rate': 0.0983235833694758, 'num_leaves': 55, 'max_depth': 10, 'min_child_samples': 5, 'subsample': 0.6120371547352597, 'colsample_bytree': 0.5697405115386522, 'reg_alpha': 5.5684212127834956, 'reg_lambda': 6.682135309790644, 'min_split_gain': 0.03299186632435075}. Best is trial 0 with value: 0.00471749676456151.
[Trial 0] New best: 0.00472
[I 2025-06-15 20:12:04,587] Trial 1 finished with value: 0.00457526245159661 and parameters: {'n_estimators': 177, 'learning_rate': 0.038267030550719595, 'num_leaves': 142, 'max_depth': 12, 'min_child_samples': 7, 'subsample': 0.9027414316111846, 'colsample_bytree': 0.5405070399547371, 'reg_alpha': 6.204202549744242, 'reg_lambda': 0.013974361476692954, 'min_split_gain': 0.044501011874951896}. Best is trial 1 with value: 0.00457526245159661.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:04,949] Trial 2 finished with value: 0.008001025856761795 and parameters: {'n_estimators': 53, 'learning_rate': 0.023953327471588903, 'num_leaves': 95, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.8833017692013014, 'colsample_bytree': 0.8138948151183987, 'reg_alpha': 3.1833914799392913, 'reg_lambda': 2.0803164749854064, 'min_split_gain': 0.010660741301620997}. Best is trial 1 with value: 0.00457526245159661.
[I 2025-06-15 20:12:05,144] Trial 3 finished with value: 0.004044292138672116 and parameters: {'n_estimators': 221, 'learning_rate': 0.29326639549154837, 'num_leaves': 254, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.5955415614392305, 'colsample_bytree': 0.8575844130037945, 'reg_alpha': 3.5539587964922035, 'reg_lambda': 2.0511436687764864, 'min_split_gain': 0.04853991951924245}. Best is trial 3 with value: 0.004044292138672116.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:05,146] A new study created in memory with name: no-name-04e9fa7b-11da-4167-a660-8a690f6caf7e
[I 2025-06-15 20:12:05,191] Trial 0 finished with value: 0.017328579219205567 and parameters: {'n_estimators': 134, 'learning_rate': 0.01351094880715571, 'num_leaves': 79, 'max_depth': 5, 'min_child_samples': 7, 'subsample': 0.9181792789074021, 'colsample_bytree': 0.7614113254552424, 'reg_alpha': 0.8884999449843523, 'reg_lambda': 9.307200759155792, 'min_split_gain': 0.033817728457284854}. Best is trial 0 with value: 0.017328579219205567.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         183
1           Price_lag_2         106
14      pct_chg_Load_DA         105
3         Load_DA_lag_0          76
2           Price_lag_7          46
11           NGas_lag_2          43
16  roll2d_median_Price          40
5       WindOn_DA_lag_0          36
10           Coal_lag_2          26
15       lag168_Load_DA          23
13            EUA_lag_2          17
12            Oil_lag_2          13
7                  WD_1           3
9                  WD_7           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         224
15       lag168_Load_DA          80
5       WindOn_DA_lag_0          30
1           Price_lag_2          28
2           Price_lag_7          15
3         Load_DA_lag_0          15
14      pct_chg_Load_DA          14
16  roll2d_median_Price          11
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 47.88% done
********************* END NS.. 349 ****************************************************
********************* START NS ... 350 ****************************************************
[I 2025-06-15 20:12:06,440] A new study created in memory with name: no-name-d522e666-0f78-46e1-a217-c3c2aeae315a
[I 2025-06-15 20:12:06,592] Trial 0 finished with value: 0.003826392262336589 and parameters: {'n_estimators': 168, 'learning_rate': 0.1402898863306622, 'num_leaves': 82, 'max_depth': 5, 'min_child_samples': 30, 'subsample': 0.6441538197855758, 'colsample_bytree': 0.5340832964582092, 'reg_alpha': 6.042691531073972, 'reg_lambda': 2.6836461928336544, 'min_split_gain': 0.009142791286958619}. Best is trial 0 with value: 0.003826392262336589.
[Trial 0] New best: 0.00383
[I 2025-06-15 20:12:07,176] Trial 1 finished with value: 0.0037888551028383657 and parameters: {'n_estimators': 155, 'learning_rate': 0.019365847832140983, 'num_leaves': 67, 'max_depth': 10, 'min_child_samples': 10, 'subsample': 0.6484141817011041, 'colsample_bytree': 0.6097096890366376, 'reg_alpha': 5.404104258703964, 'reg_lambda': 5.490208974698415, 'min_split_gain': 0.04599306623000712}. Best is trial 1 with value: 0.0037888551028383657.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:07,846] Trial 2 finished with value: 0.0022499889027729303 and parameters: {'n_estimators': 107, 'learning_rate': 0.036229251469590516, 'num_leaves': 187, 'max_depth': 15, 'min_child_samples': 11, 'subsample': 0.8297119425056876, 'colsample_bytree': 0.7542206981111395, 'reg_alpha': 4.9151927339654335, 'reg_lambda': 0.8913954372895805, 'min_split_gain': 0.01589107256746951}. Best is trial 2 with value: 0.0022499889027729303.
[Trial 2] New best: 0.00225
[I 2025-06-15 20:12:08,594] Trial 3 finished with value: 0.0023143222596189255 and parameters: {'n_estimators': 252, 'learning_rate': 0.022921021950564217, 'num_leaves': 215, 'max_depth': 9, 'min_child_samples': 29, 'subsample': 0.873566532595133, 'colsample_bytree': 0.7723970875218233, 'reg_alpha': 6.4156192198777635, 'reg_lambda': 4.74612455732752, 'min_split_gain': 0.03082014995306592}. Best is trial 2 with value: 0.0022499889027729303.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:09,208] Trial 4 finished with value: 0.002405660640955202 and parameters: {'n_estimators': 296, 'learning_rate': 0.0655908250983212, 'num_leaves': 84, 'max_depth': 8, 'min_child_samples': 12, 'subsample': 0.8846151485199188, 'colsample_bytree': 0.7837306221754394, 'reg_alpha': 8.289904583118464, 'reg_lambda': 0.6184355395082419, 'min_split_gain': 0.0456931786715207}. Best is trial 2 with value: 0.0022499889027729303.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:09,439] Trial 5 finished with value: 0.002672320304155469 and parameters: {'n_estimators': 157, 'learning_rate': 0.1377668306688311, 'num_leaves': 187, 'max_depth': 6, 'min_child_samples': 1, 'subsample': 0.7298900658898438, 'colsample_bytree': 0.5851986505634912, 'reg_alpha': 9.260326057943368, 'reg_lambda': 0.6403854278517118, 'min_split_gain': 0.027567262656034805}. Best is trial 2 with value: 0.0022499889027729303.
[I 2025-06-15 20:12:09,441] A new study created in memory with name: no-name-5a7ab299-f054-424a-b969-e7cb6420d5c4
[I 2025-06-15 20:12:09,479] Trial 0 finished with value: 0.004664598598089996 and parameters: {'n_estimators': 188, 'learning_rate': 0.057269774820054645, 'num_leaves': 97, 'max_depth': 15, 'min_child_samples': 12, 'subsample': 0.8418889164065138, 'colsample_bytree': 0.5682083296731373, 'reg_alpha': 1.4370907157589219, 'reg_lambda': 8.72329911823721, 'min_split_gain': 0.047902693042116}. Best is trial 0 with value: 0.004664598598089996.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1174
1           Price_lag_2        1049
14      pct_chg_Load_DA         709
2           Price_lag_7         370
16  roll2d_median_Price         367
3         Load_DA_lag_0         324
11           NGas_lag_2         252
15       lag168_Load_DA         167
10           Coal_lag_2         166
13            EUA_lag_2         142
5       WindOn_DA_lag_0         136
12            Oil_lag_2         135
9                  WD_7          17
8                  WD_6           9
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          57
15       lag168_Load_DA          36
5       WindOn_DA_lag_0          16
1           Price_lag_2          10
2           Price_lag_7           6
3         Load_DA_lag_0           5
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 48.02% done
********************* END NS.. 350 ****************************************************
********************* START NS ... 351 ****************************************************
[I 2025-06-15 20:12:10,602] A new study created in memory with name: no-name-3c088bfb-107d-4efc-acfb-9e3b1f267fb1
[I 2025-06-15 20:12:10,890] Trial 0 finished with value: 0.0015420632621047832 and parameters: {'n_estimators': 294, 'learning_rate': 0.13174083932333774, 'num_leaves': 50, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.6981457454044129, 'colsample_bytree': 0.9315790138392295, 'reg_alpha': 6.974417346951416, 'reg_lambda': 2.4927063600760313, 'min_split_gain': 0.008554610486411767}. Best is trial 0 with value: 0.0015420632621047832.
[I 2025-06-15 20:12:11,081] Trial 1 finished with value: 0.0017462496507607654 and parameters: {'n_estimators': 161, 'learning_rate': 0.10910108937270938, 'num_leaves': 86, 'max_depth': 5, 'min_child_samples': 2, 'subsample': 0.959544653039728, 'colsample_bytree': 0.6689199840693383, 'reg_alpha': 0.2626265484764667, 'reg_lambda': 5.6988807028636215, 'min_split_gain': 0.048193301957916296}. Best is trial 0 with value: 0.0015420632621047832.
[Trial 0] New best: 0.00154
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:12,273] Trial 2 finished with value: 0.0033265185369636963 and parameters: {'n_estimators': 285, 'learning_rate': 0.012218677117407147, 'num_leaves': 152, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.831246460674465, 'colsample_bytree': 0.6326582880962147, 'reg_alpha': 5.245563625359587, 'reg_lambda': 4.573365606760761, 'min_split_gain': 0.048688368436282985}. Best is trial 0 with value: 0.0015420632621047832.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:12,892] Trial 3 finished with value: 0.0016507618083669857 and parameters: {'n_estimators': 120, 'learning_rate': 0.03966387586861818, 'num_leaves': 210, 'max_depth': 13, 'min_child_samples': 17, 'subsample': 0.5014608560753552, 'colsample_bytree': 0.8548122191078894, 'reg_alpha': 2.778143860183624, 'reg_lambda': 4.08854686614041, 'min_split_gain': 0.04596360290052424}. Best is trial 0 with value: 0.0015420632621047832.
[I 2025-06-15 20:12:12,895] A new study created in memory with name: no-name-48e1e7da-31d4-43f9-8398-385bbbb7821b
[I 2025-06-15 20:12:12,967] Trial 0 finished with value: 0.13643911820277296 and parameters: {'n_estimators': 296, 'learning_rate': 0.018009238729866508, 'num_leaves': 254, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.9981010366764489, 'colsample_bytree': 0.5983975118645735, 'reg_alpha': 4.392380683106838, 'reg_lambda': 8.082059354636687, 'min_split_gain': 0.04656888091735853}. Best is trial 0 with value: 0.13643911820277296.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         457
14      pct_chg_Load_DA         339
1           Price_lag_2         279
3         Load_DA_lag_0         185
11           NGas_lag_2         166
2           Price_lag_7         156
16  roll2d_median_Price         153
5       WindOn_DA_lag_0         151
13            EUA_lag_2         139
15       lag168_Load_DA         123
10           Coal_lag_2         121
12            Oil_lag_2          86
9                  WD_7          18
7                  WD_1           8
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         141
15       lag168_Load_DA          73
5       WindOn_DA_lag_0          24
3         Load_DA_lag_0          18
1           Price_lag_2          17
2           Price_lag_7           5
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 48.15% done
********************* END NS.. 351 ****************************************************
********************* START NS ... 352 ****************************************************
[I 2025-06-15 20:12:14,488] A new study created in memory with name: no-name-39fae539-0caa-4e27-9cec-3b38cde9e4e2
[I 2025-06-15 20:12:14,743] Trial 0 finished with value: 0.002364411925036843 and parameters: {'n_estimators': 97, 'learning_rate': 0.06761617998628786, 'num_leaves': 138, 'max_depth': 10, 'min_child_samples': 20, 'subsample': 0.7772586122136516, 'colsample_bytree': 0.5079241727294697, 'reg_alpha': 6.584229750196827, 'reg_lambda': 4.879278091880064, 'min_split_gain': 0.03857399108005616}. Best is trial 0 with value: 0.002364411925036843.
[Trial 0] New best: 0.00236
[I 2025-06-15 20:12:15,051] Trial 1 finished with value: 0.0020899561794805364 and parameters: {'n_estimators': 87, 'learning_rate': 0.10641377636180549, 'num_leaves': 188, 'max_depth': 8, 'min_child_samples': 10, 'subsample': 0.8026522722460632, 'colsample_bytree': 0.8285517800094819, 'reg_alpha': 7.356483719521135, 'reg_lambda': 9.55286419309418, 'min_split_gain': 0.0005783928233651259}. Best is trial 1 with value: 0.0020899561794805364.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:15,504] Trial 2 finished with value: 0.001988714917545435 and parameters: {'n_estimators': 243, 'learning_rate': 0.058082547419337195, 'num_leaves': 185, 'max_depth': 14, 'min_child_samples': 4, 'subsample': 0.841063261907637, 'colsample_bytree': 0.7243817181890837, 'reg_alpha': 4.893181519302717, 'reg_lambda': 0.7536651542215533, 'min_split_gain': 0.042951756786420534}. Best is trial 2 with value: 0.001988714917545435.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:15,949] Trial 3 finished with value: 0.001967383443817043 and parameters: {'n_estimators': 68, 'learning_rate': 0.0496639411993269, 'num_leaves': 85, 'max_depth': 9, 'min_child_samples': 6, 'subsample': 0.647849462898807, 'colsample_bytree': 0.8442160275790318, 'reg_alpha': 2.8546602387505904, 'reg_lambda': 0.07053038551881552, 'min_split_gain': 0.0009936295911670667}. Best is trial 3 with value: 0.001967383443817043.
[I 2025-06-15 20:12:15,951] A new study created in memory with name: no-name-4aba8982-78ff-4a90-a31d-dbafe844be35
[I 2025-06-15 20:12:15,976] Trial 0 finished with value: 0.11589197805618234 and parameters: {'n_estimators': 64, 'learning_rate': 0.05599680364084594, 'num_leaves': 166, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.8055001876363798, 'colsample_bytree': 0.9536075140829319, 'reg_alpha': 7.496485994418287, 'reg_lambda': 6.481066331750377, 'min_split_gain': 0.02768044544308836}. Best is trial 0 with value: 0.11589197805618234.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1101
1           Price_lag_2         999
14      pct_chg_Load_DA         710
16  roll2d_median_Price         427
2           Price_lag_7         420
11           NGas_lag_2         300
3         Load_DA_lag_0         239
13            EUA_lag_2         220
10           Coal_lag_2         206
5       WindOn_DA_lag_0         181
12            Oil_lag_2         143
15       lag168_Load_DA         138
9                  WD_7           9
8                  WD_6           4
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1          37
15       lag168_Load_DA          17
5       WindOn_DA_lag_0          17
3         Load_DA_lag_0           2
1           Price_lag_2           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 48.29% done
********************* END NS.. 352 ****************************************************
********************* START NS ... 353 ****************************************************
[I 2025-06-15 20:12:17,670] A new study created in memory with name: no-name-2126f284-8607-4a8e-969f-a9c79f0e4b6b
[I 2025-06-15 20:12:18,632] Trial 0 finished with value: 0.0009799282718648914 and parameters: {'n_estimators': 216, 'learning_rate': 0.013902852507024197, 'num_leaves': 113, 'max_depth': 9, 'min_child_samples': 8, 'subsample': 0.5883331214012858, 'colsample_bytree': 0.8575485342933066, 'reg_alpha': 4.05707306774417, 'reg_lambda': 2.634435492736432, 'min_split_gain': 0.004344449569604741}. Best is trial 0 with value: 0.0009799282718648914.
[Trial 0] New best: 0.00098
[I 2025-06-15 20:12:18,984] Trial 1 finished with value: 0.0009370136911894539 and parameters: {'n_estimators': 202, 'learning_rate': 0.015607631812611683, 'num_leaves': 145, 'max_depth': 4, 'min_child_samples': 22, 'subsample': 0.5825384174870349, 'colsample_bytree': 0.6642515945794316, 'reg_alpha': 9.432373347546081, 'reg_lambda': 7.541816107431814, 'min_split_gain': 0.012082745568643617}. Best is trial 1 with value: 0.0009370136911894539.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:19,330] Trial 2 finished with value: 0.004915532553835284 and parameters: {'n_estimators': 104, 'learning_rate': 0.016787849940566615, 'num_leaves': 240, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.5367736582491305, 'colsample_bytree': 0.6950323292635645, 'reg_alpha': 9.849617339279858, 'reg_lambda': 3.3790839573873708, 'min_split_gain': 0.020429551650007385}. Best is trial 1 with value: 0.0009370136911894539.
[I 2025-06-15 20:12:19,491] Trial 3 finished with value: 0.0010120674631031769 and parameters: {'n_estimators': 162, 'learning_rate': 0.19451071928586627, 'num_leaves': 32, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.8982856809385991, 'colsample_bytree': 0.8849236013721725, 'reg_alpha': 8.892276818900038, 'reg_lambda': 8.533426040631625, 'min_split_gain': 0.024913162153058582}. Best is trial 1 with value: 0.0009370136911894539.
[I 2025-06-15 20:12:19,495] A new study created in memory with name: no-name-e99e2fbe-67cc-4e2e-909a-a7f1e8354e74
[I 2025-06-15 20:12:19,534] Trial 0 finished with value: 0.006675362743241577 and parameters: {'n_estimators': 183, 'learning_rate': 0.1938355616884109, 'num_leaves': 153, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.9482284959517144, 'colsample_bytree': 0.504581671713126, 'reg_alpha': 5.26100228604904, 'reg_lambda': 6.070033228491377, 'min_split_gain': 0.01549290009164931}. Best is trial 0 with value: 0.006675362743241577.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1286
1           Price_lag_2         531
14      pct_chg_Load_DA         314
16  roll2d_median_Price         169
2           Price_lag_7         133
11           NGas_lag_2          61
3         Load_DA_lag_0          60
10           Coal_lag_2          26
12            Oil_lag_2          14
15       lag168_Load_DA           9
9                  WD_7           5
13            EUA_lag_2           3
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          14
15       lag168_Load_DA           5
3         Load_DA_lag_0           3
5       WindOn_DA_lag_0           2
16  roll2d_median_Price           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 48.43% done
********************* END NS.. 353 ****************************************************
********************* START NS ... 354 ****************************************************
[I 2025-06-15 20:12:21,010] A new study created in memory with name: no-name-54ff45a7-5d19-4307-9488-4e748ebc7876
[I 2025-06-15 20:12:21,227] Trial 0 finished with value: 0.015698720940216532 and parameters: {'n_estimators': 72, 'learning_rate': 0.017412660246888256, 'num_leaves': 98, 'max_depth': 10, 'min_child_samples': 23, 'subsample': 0.9505834991266137, 'colsample_bytree': 0.8033308498806568, 'reg_alpha': 3.7592306231033836, 'reg_lambda': 0.2644325900964728, 'min_split_gain': 0.031021377883850693}. Best is trial 0 with value: 0.015698720940216532.
[Trial 0] New best: 0.01570
[I 2025-06-15 20:12:21,854] Trial 1 finished with value: 0.0014234291547712441 and parameters: {'n_estimators': 278, 'learning_rate': 0.023891196275720703, 'num_leaves': 92, 'max_depth': 6, 'min_child_samples': 8, 'subsample': 0.9973294149840327, 'colsample_bytree': 0.8534405556718831, 'reg_alpha': 9.221226899090784, 'reg_lambda': 1.7248427312030445, 'min_split_gain': 0.03628417026174253}. Best is trial 1 with value: 0.0014234291547712441.
[Trial 1] New best: 0.00142
[I 2025-06-15 20:12:22,166] Trial 2 finished with value: 0.0018391074175756205 and parameters: {'n_estimators': 80, 'learning_rate': 0.03960334860552514, 'num_leaves': 143, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.5512602626289451, 'colsample_bytree': 0.7687837208836592, 'reg_alpha': 9.327437808237898, 'reg_lambda': 7.395012360017096, 'min_split_gain': 0.0032570128142277834}. Best is trial 1 with value: 0.0014234291547712441.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:22,539] Trial 3 finished with value: 0.002156020973502995 and parameters: {'n_estimators': 83, 'learning_rate': 0.045341069634080744, 'num_leaves': 222, 'max_depth': 6, 'min_child_samples': 7, 'subsample': 0.7698045864561645, 'colsample_bytree': 0.5949170240131867, 'reg_alpha': 0.38353586643241067, 'reg_lambda': 4.117255058432459, 'min_split_gain': 0.02925414247818854}. Best is trial 1 with value: 0.0014234291547712441.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:23,267] Trial 4 finished with value: 0.0018398193619875174 and parameters: {'n_estimators': 206, 'learning_rate': 0.10069076517019514, 'num_leaves': 89, 'max_depth': 7, 'min_child_samples': 29, 'subsample': 0.6665554964591753, 'colsample_bytree': 0.5260696054931433, 'reg_alpha': 1.9693876485699513, 'reg_lambda': 4.4399957146767255, 'min_split_gain': 0.005440213246452392}. Best is trial 1 with value: 0.0014234291547712441.
[I 2025-06-15 20:12:23,269] A new study created in memory with name: no-name-31350803-7427-4d74-bed4-faa8ecd15ef6
[I 2025-06-15 20:12:23,344] Trial 0 finished with value: 0.017832115589723863 and parameters: {'n_estimators': 297, 'learning_rate': 0.027062627642918072, 'num_leaves': 229, 'max_depth': 5, 'min_child_samples': 2, 'subsample': 0.9543599080403251, 'colsample_bytree': 0.5922325001844346, 'reg_alpha': 7.3146528412803065, 'reg_lambda': 9.208308751148172, 'min_split_gain': 0.02596788693546999}. Best is trial 0 with value: 0.017832115589723863.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1855
1           Price_lag_2        1074
14      pct_chg_Load_DA         970
3         Load_DA_lag_0         494
16  roll2d_median_Price         428
11           NGas_lag_2         421
2           Price_lag_7         420
13            EUA_lag_2         234
15       lag168_Load_DA         232
10           Coal_lag_2         222
12            Oil_lag_2         166
5       WindOn_DA_lag_0         152
9                  WD_7          28
8                  WD_6           7
7                  WD_1           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          69
16  roll2d_median_Price          24
3         Load_DA_lag_0          11
1           Price_lag_2          11
15       lag168_Load_DA          11
10           Coal_lag_2           2
2           Price_lag_7           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 48.56% done
********************* END NS.. 354 ****************************************************
********************* START NS ... 355 ****************************************************
[I 2025-06-15 20:12:24,826] A new study created in memory with name: no-name-5453774d-b439-42bd-b5d1-7cf90a1c9e6a
[I 2025-06-15 20:12:25,752] Trial 0 finished with value: 0.010076838573990799 and parameters: {'n_estimators': 197, 'learning_rate': 0.010143206337675242, 'num_leaves': 228, 'max_depth': 13, 'min_child_samples': 21, 'subsample': 0.9789496650957418, 'colsample_bytree': 0.8004740490752245, 'reg_alpha': 3.652691672029411, 'reg_lambda': 1.8308208857756791, 'min_split_gain': 0.0005120844559532933}. Best is trial 0 with value: 0.010076838573990799.
[Trial 0] New best: 0.01008
[I 2025-06-15 20:12:26,031] Trial 1 finished with value: 0.003791012111118696 and parameters: {'n_estimators': 183, 'learning_rate': 0.10829684238459006, 'num_leaves': 207, 'max_depth': 4, 'min_child_samples': 24, 'subsample': 0.7675587863403376, 'colsample_bytree': 0.6060203092915195, 'reg_alpha': 5.321993025180346, 'reg_lambda': 0.608859168552317, 'min_split_gain': 0.03025739093934079}. Best is trial 1 with value: 0.003791012111118696.
[Trial 1] New best: 0.00379
[I 2025-06-15 20:12:26,435] Trial 2 finished with value: 0.003249258414812809 and parameters: {'n_estimators': 153, 'learning_rate': 0.0989368542126703, 'num_leaves': 88, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.8360859794058646, 'colsample_bytree': 0.9508141567968666, 'reg_alpha': 7.978352272992934, 'reg_lambda': 0.8482598044738432, 'min_split_gain': 0.003511493272513383}. Best is trial 2 with value: 0.003249258414812809.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:27,192] Trial 3 finished with value: 0.003900572403851011 and parameters: {'n_estimators': 178, 'learning_rate': 0.03960272733235151, 'num_leaves': 99, 'max_depth': 16, 'min_child_samples': 3, 'subsample': 0.8613144647200774, 'colsample_bytree': 0.5856880294575568, 'reg_alpha': 5.7645063895429915, 'reg_lambda': 6.246875679237746, 'min_split_gain': 0.036499500495154984}. Best is trial 2 with value: 0.003249258414812809.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:27,640] Trial 4 finished with value: 0.005269933703872207 and parameters: {'n_estimators': 71, 'learning_rate': 0.04137860047660859, 'num_leaves': 49, 'max_depth': 13, 'min_child_samples': 26, 'subsample': 0.8453799882316084, 'colsample_bytree': 0.8348100263919667, 'reg_alpha': 3.609358282131968, 'reg_lambda': 6.71873954263397, 'min_split_gain': 0.00025751665425872706}. Best is trial 2 with value: 0.003249258414812809.
[I 2025-06-15 20:12:27,644] A new study created in memory with name: no-name-bc923f72-b783-4c15-b0fd-386e94ec08ca
[I 2025-06-15 20:12:27,697] Trial 0 finished with value: 0.028909602032780746 and parameters: {'n_estimators': 247, 'learning_rate': 0.0509628333008409, 'num_leaves': 159, 'max_depth': 14, 'min_child_samples': 17, 'subsample': 0.5256527007355716, 'colsample_bytree': 0.7261826509852172, 'reg_alpha': 9.653135980687543, 'reg_lambda': 0.36503820990569036, 'min_split_gain': 0.004473370946998095}. Best is trial 0 with value: 0.028909602032780746.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         630
14      pct_chg_Load_DA         384
1           Price_lag_2         381
3         Load_DA_lag_0         266
2           Price_lag_7         199
5       WindOn_DA_lag_0         186
16  roll2d_median_Price         185
15       lag168_Load_DA         182
11           NGas_lag_2         179
10           Coal_lag_2         152
13            EUA_lag_2         126
12            Oil_lag_2          80
9                  WD_7          18
7                  WD_1          10
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 48.70% done
********************* END NS.. 355 ****************************************************
********************* START NS ... 356 ****************************************************
[I 2025-06-15 20:12:29,346] A new study created in memory with name: no-name-cc7757ea-59c3-49bc-a005-4b7359c732aa
[I 2025-06-15 20:12:29,693] Trial 0 finished with value: 0.038093067963249884 and parameters: {'n_estimators': 68, 'learning_rate': 0.017924291398953324, 'num_leaves': 186, 'max_depth': 15, 'min_child_samples': 15, 'subsample': 0.974465900896941, 'colsample_bytree': 0.9418357076288433, 'reg_alpha': 1.4742936437961962, 'reg_lambda': 2.2836031293060186, 'min_split_gain': 0.002986379065748751}. Best is trial 0 with value: 0.038093067963249884.
[Trial 0] New best: 0.03809
[I 2025-06-15 20:12:30,036] Trial 1 finished with value: 0.004274457103864086 and parameters: {'n_estimators': 157, 'learning_rate': 0.1967651454651279, 'num_leaves': 134, 'max_depth': 11, 'min_child_samples': 9, 'subsample': 0.6770774868398449, 'colsample_bytree': 0.7512814643684742, 'reg_alpha': 3.465350649552624, 'reg_lambda': 5.057385768549663, 'min_split_gain': 0.038577435893775824}. Best is trial 1 with value: 0.004274457103864086.
[Trial 1] New best: 0.00427
[I 2025-06-15 20:12:30,466] Trial 2 finished with value: 0.0042766033480913625 and parameters: {'n_estimators': 145, 'learning_rate': 0.1947486558545277, 'num_leaves': 210, 'max_depth': 10, 'min_child_samples': 3, 'subsample': 0.9171531561649764, 'colsample_bytree': 0.7377574204759672, 'reg_alpha': 2.1082032808327633, 'reg_lambda': 2.9328370923936795, 'min_split_gain': 0.004562622724505028}. Best is trial 1 with value: 0.004274457103864086.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:31,069] Trial 3 finished with value: 0.005116324969542959 and parameters: {'n_estimators': 153, 'learning_rate': 0.0373119128908842, 'num_leaves': 97, 'max_depth': 11, 'min_child_samples': 5, 'subsample': 0.6278690082110138, 'colsample_bytree': 0.6774068871554102, 'reg_alpha': 5.281495572364787, 'reg_lambda': 3.1345871489624946, 'min_split_gain': 0.03657802227640732}. Best is trial 1 with value: 0.004274457103864086.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:31,296] Trial 4 finished with value: 0.004045649299755396 and parameters: {'n_estimators': 190, 'learning_rate': 0.12666972375582797, 'num_leaves': 234, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.7474592213795599, 'colsample_bytree': 0.9695915988608135, 'reg_alpha': 6.1750406595146465, 'reg_lambda': 6.7169020854644454, 'min_split_gain': 0.014802039047728055}. Best is trial 4 with value: 0.004045649299755396.
[I 2025-06-15 20:12:31,298] A new study created in memory with name: no-name-10211d74-3364-46af-b139-c5fbc62d8b81
[I 2025-06-15 20:12:31,334] Trial 0 finished with value: 0.05122972845694252 and parameters: {'n_estimators': 225, 'learning_rate': 0.027646188970329336, 'num_leaves': 100, 'max_depth': 5, 'min_child_samples': 7, 'subsample': 0.8497200729843928, 'colsample_bytree': 0.8485908479280528, 'reg_alpha': 9.137621637621274, 'reg_lambda': 3.794204450643337, 'min_split_gain': 0.014630927403716028}. Best is trial 0 with value: 0.05122972845694252.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         390
14      pct_chg_Load_DA         242
1           Price_lag_2         156
3         Load_DA_lag_0         156
16  roll2d_median_Price         113
11           NGas_lag_2          99
2           Price_lag_7          95
15       lag168_Load_DA          86
5       WindOn_DA_lag_0          75
13            EUA_lag_2          60
10           Coal_lag_2          57
12            Oil_lag_2          29
7                  WD_1          11
9                  WD_7           6
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 48.84% done
********************* END NS.. 356 ****************************************************
********************* START NS ... 357 ****************************************************
[I 2025-06-15 20:12:32,345] A new study created in memory with name: no-name-49f9411a-947d-4995-bc10-b3149905f2ea
[I 2025-06-15 20:12:32,629] Trial 0 finished with value: 0.0062609181800412925 and parameters: {'n_estimators': 98, 'learning_rate': 0.030168378748319142, 'num_leaves': 102, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.9518842218941301, 'colsample_bytree': 0.9371440306018897, 'reg_alpha': 5.99315389879986, 'reg_lambda': 3.8186021464816666, 'min_split_gain': 0.005155042751284528}. Best is trial 0 with value: 0.0062609181800412925.
[Trial 0] New best: 0.00626
[I 2025-06-15 20:12:33,309] Trial 1 finished with value: 0.004790180169203402 and parameters: {'n_estimators': 168, 'learning_rate': 0.051588310574470934, 'num_leaves': 255, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.6439560095976484, 'colsample_bytree': 0.8032589892960331, 'reg_alpha': 1.7767612939356903, 'reg_lambda': 7.274382827602459, 'min_split_gain': 0.029555090414245463}. Best is trial 1 with value: 0.004790180169203402.
[Trial 1] New best: 0.00479
[I 2025-06-15 20:12:34,386] Trial 2 finished with value: 0.01034877779228551 and parameters: {'n_estimators': 259, 'learning_rate': 0.014315289757749813, 'num_leaves': 197, 'max_depth': 14, 'min_child_samples': 3, 'subsample': 0.9952146660948835, 'colsample_bytree': 0.5516202676491209, 'reg_alpha': 6.49717714800367, 'reg_lambda': 8.557589843919112, 'min_split_gain': 0.023784420478761555}. Best is trial 1 with value: 0.004790180169203402.
[I 2025-06-15 20:12:34,553] Trial 3 finished with value: 0.00490528980857377 and parameters: {'n_estimators': 66, 'learning_rate': 0.16689040474328498, 'num_leaves': 188, 'max_depth': 5, 'min_child_samples': 8, 'subsample': 0.8094774596558096, 'colsample_bytree': 0.6480060073187374, 'reg_alpha': 8.40163589703003, 'reg_lambda': 1.5086446560993116, 'min_split_gain': 0.018312966575782085}. Best is trial 1 with value: 0.004790180169203402.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:35,116] Trial 4 finished with value: 0.04514941928869274 and parameters: {'n_estimators': 73, 'learning_rate': 0.01654458789075739, 'num_leaves': 242, 'max_depth': 12, 'min_child_samples': 1, 'subsample': 0.5809197463923714, 'colsample_bytree': 0.853106063386269, 'reg_alpha': 1.9952699695808396, 'reg_lambda': 1.5572420032580547, 'min_split_gain': 0.020283686372036504}. Best is trial 1 with value: 0.004790180169203402.
[I 2025-06-15 20:12:35,118] A new study created in memory with name: no-name-a2220c45-970a-48a3-b2ff-c46b1ff95018
[I 2025-06-15 20:12:35,135] Trial 0 finished with value: 0.059579087485480436 and parameters: {'n_estimators': 60, 'learning_rate': 0.2709409173980077, 'num_leaves': 52, 'max_depth': 15, 'min_child_samples': 1, 'subsample': 0.6624858034735309, 'colsample_bytree': 0.5178954418074626, 'reg_alpha': 2.0608757579132906, 'reg_lambda': 0.5811555437564231, 'min_split_gain': 0.048076696036833635}. Best is trial 0 with value: 0.059579087485480436.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1461
1           Price_lag_2        1395
14      pct_chg_Load_DA        1324
3         Load_DA_lag_0         963
2           Price_lag_7         740
11           NGas_lag_2         667
5       WindOn_DA_lag_0         627
16  roll2d_median_Price         605
15       lag168_Load_DA         574
10           Coal_lag_2         386
13            EUA_lag_2         369
12            Oil_lag_2         330
9                  WD_7          44
7                  WD_1          22
8                  WD_6           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           8
3         Load_DA_lag_0           5
13            EUA_lag_2           3
15       lag168_Load_DA           1
1           Price_lag_2           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 48.97% done
********************* END NS.. 357 ****************************************************
********************* START NS ... 358 ****************************************************
[I 2025-06-15 20:12:36,512] A new study created in memory with name: no-name-aaa864da-bed2-459e-97fb-be7d98c85fa7
[I 2025-06-15 20:12:36,647] Trial 0 finished with value: 0.0037725228024614887 and parameters: {'n_estimators': 74, 'learning_rate': 0.1706670570198257, 'num_leaves': 229, 'max_depth': 15, 'min_child_samples': 28, 'subsample': 0.9830028649576347, 'colsample_bytree': 0.7945933209959208, 'reg_alpha': 2.372946647218972, 'reg_lambda': 7.200571321100995, 'min_split_gain': 0.04806284428790763}. Best is trial 0 with value: 0.0037725228024614887.
[Trial 0] New best: 0.00377
[I 2025-06-15 20:12:37,093] Trial 1 finished with value: 0.003978423364800279 and parameters: {'n_estimators': 289, 'learning_rate': 0.020769373149753557, 'num_leaves': 172, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.9610559961209291, 'colsample_bytree': 0.9882951129228411, 'reg_alpha': 8.10918048184495, 'reg_lambda': 3.0276069989519394, 'min_split_gain': 0.011073198775660814}. Best is trial 0 with value: 0.0037725228024614887.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:37,312] Trial 2 finished with value: 0.011501977167802097 and parameters: {'n_estimators': 62, 'learning_rate': 0.03401852142804212, 'num_leaves': 49, 'max_depth': 9, 'min_child_samples': 28, 'subsample': 0.9375347299337644, 'colsample_bytree': 0.9983186795147402, 'reg_alpha': 7.920962685870398, 'reg_lambda': 2.7710170606550566, 'min_split_gain': 0.04466746370496955}. Best is trial 0 with value: 0.0037725228024614887.
[I 2025-06-15 20:12:37,493] Trial 3 finished with value: 0.003993271814042592 and parameters: {'n_estimators': 102, 'learning_rate': 0.2696227258638204, 'num_leaves': 248, 'max_depth': 13, 'min_child_samples': 20, 'subsample': 0.7315835487737237, 'colsample_bytree': 0.7668178447211906, 'reg_alpha': 3.1511886431397973, 'reg_lambda': 5.490918403435426, 'min_split_gain': 0.030964321353189224}. Best is trial 0 with value: 0.0037725228024614887.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:37,495] A new study created in memory with name: no-name-41362a03-d755-46f6-8dff-870ad5242252
[I 2025-06-15 20:12:37,542] Trial 0 finished with value: 0.014321062824640907 and parameters: {'n_estimators': 244, 'learning_rate': 0.11689876641404377, 'num_leaves': 50, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.6047335531976081, 'colsample_bytree': 0.5819116879389867, 'reg_alpha': 0.13590842518069635, 'reg_lambda': 0.09524985511032757, 'min_split_gain': 0.023744667325211095}. Best is trial 0 with value: 0.014321062824640907.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         353
1           Price_lag_2         276
14      pct_chg_Load_DA         249
3         Load_DA_lag_0         130
11           NGas_lag_2         115
16  roll2d_median_Price         114
2           Price_lag_7         110
15       lag168_Load_DA          82
5       WindOn_DA_lag_0          72
13            EUA_lag_2          49
10           Coal_lag_2          40
12            Oil_lag_2          32
9                  WD_7           6
7                  WD_1           4
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          37
3         Load_DA_lag_0          10
14      pct_chg_Load_DA          10
13            EUA_lag_2           7
1           Price_lag_2           6
15       lag168_Load_DA           3
12            Oil_lag_2           1
16  roll2d_median_Price           1
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
8                  WD_6           0
-> 49.11% done
********************* END NS.. 358 ****************************************************
********************* START NS ... 359 ****************************************************
[I 2025-06-15 20:12:39,130] A new study created in memory with name: no-name-42d63d5e-7389-4220-8332-15f32107dcaa
[I 2025-06-15 20:12:39,611] Trial 0 finished with value: 0.003368725080552202 and parameters: {'n_estimators': 216, 'learning_rate': 0.03156638584568878, 'num_leaves': 169, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.8455932624953437, 'colsample_bytree': 0.7097590097380939, 'reg_alpha': 8.018676807516433, 'reg_lambda': 8.020998361614, 'min_split_gain': 0.04946690180642893}. Best is trial 0 with value: 0.003368725080552202.
[Trial 0] New best: 0.00337
[I 2025-06-15 20:12:40,283] Trial 1 finished with value: 0.0031960764516878403 and parameters: {'n_estimators': 294, 'learning_rate': 0.07879100130720879, 'num_leaves': 141, 'max_depth': 6, 'min_child_samples': 6, 'subsample': 0.8880760141269728, 'colsample_bytree': 0.7147390958805282, 'reg_alpha': 0.11545403012345301, 'reg_lambda': 8.174585231793696, 'min_split_gain': 0.003933083316013991}. Best is trial 1 with value: 0.0031960764516878403.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:41,475] Trial 2 finished with value: 0.003151209110200604 and parameters: {'n_estimators': 137, 'learning_rate': 0.03490200900881484, 'num_leaves': 204, 'max_depth': 13, 'min_child_samples': 20, 'subsample': 0.8071748810517232, 'colsample_bytree': 0.731044041895651, 'reg_alpha': 0.5783694082688551, 'reg_lambda': 4.551351157137667, 'min_split_gain': 0.005771620381133324}. Best is trial 2 with value: 0.003151209110200604.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:41,728] Trial 3 finished with value: 0.003267080977669257 and parameters: {'n_estimators': 77, 'learning_rate': 0.24862235579083516, 'num_leaves': 47, 'max_depth': 8, 'min_child_samples': 18, 'subsample': 0.9653969449461302, 'colsample_bytree': 0.9702520518460538, 'reg_alpha': 0.24292312797607463, 'reg_lambda': 7.049262917596392, 'min_split_gain': 0.003918506153322387}. Best is trial 2 with value: 0.003151209110200604.
[I 2025-06-15 20:12:41,731] A new study created in memory with name: no-name-80d69627-2b67-41ed-ab9f-e11856a7e231
[I 2025-06-15 20:12:41,771] Trial 0 finished with value: 0.005972509189581363 and parameters: {'n_estimators': 208, 'learning_rate': 0.07546810704178906, 'num_leaves': 199, 'max_depth': 11, 'min_child_samples': 12, 'subsample': 0.9131791210134967, 'colsample_bytree': 0.7589479750392951, 'reg_alpha': 4.155967704815109, 'reg_lambda': 3.4189457595204376, 'min_split_gain': 0.00979905247074328}. Best is trial 0 with value: 0.005972509189581363.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        3155
0           Price_lag_1        2740
14      pct_chg_Load_DA        2284
3         Load_DA_lag_0        1618
2           Price_lag_7        1548
16  roll2d_median_Price        1398
15       lag168_Load_DA        1249
11           NGas_lag_2        1092
10           Coal_lag_2        1052
5       WindOn_DA_lag_0        1013
13            EUA_lag_2         978
12            Oil_lag_2         797
7                  WD_1          40
9                  WD_7          38
8                  WD_6           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          52
3         Load_DA_lag_0          14
15       lag168_Load_DA           5
1           Price_lag_2           2
13            EUA_lag_2           1
16  roll2d_median_Price           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
5       WindOn_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
4        Solar_DA_lag_0           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 49.25% done
********************* END NS.. 359 ****************************************************
********************* START NS ... 360 ****************************************************
[I 2025-06-15 20:12:44,222] A new study created in memory with name: no-name-8cbb5186-979f-499c-9f12-48ee1a004eb4
[I 2025-06-15 20:12:44,344] Trial 0 finished with value: 0.002017206070176166 and parameters: {'n_estimators': 93, 'learning_rate': 0.0674540959557003, 'num_leaves': 176, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.7982722064532843, 'colsample_bytree': 0.93788202024488, 'reg_alpha': 5.586021793227146, 'reg_lambda': 3.867431279553589, 'min_split_gain': 0.00057710224297296}. Best is trial 0 with value: 0.002017206070176166.
[I 2025-06-15 20:12:44,509] Trial 1 finished with value: 0.002143861997242797 and parameters: {'n_estimators': 60, 'learning_rate': 0.22823449460119222, 'num_leaves': 181, 'max_depth': 11, 'min_child_samples': 24, 'subsample': 0.9198295815708593, 'colsample_bytree': 0.5707666639587943, 'reg_alpha': 9.21264737498354, 'reg_lambda': 5.04396333747567, 'min_split_gain': 0.028728639676361353}. Best is trial 0 with value: 0.002017206070176166.
[Trial 0] New best: 0.00202
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:45,146] Trial 2 finished with value: 0.0020442416206734146 and parameters: {'n_estimators': 280, 'learning_rate': 0.07293691023483105, 'num_leaves': 58, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.8663113665435263, 'colsample_bytree': 0.9577361266290705, 'reg_alpha': 8.525148312176135, 'reg_lambda': 5.079626053271994, 'min_split_gain': 0.0019804000180812355}. Best is trial 0 with value: 0.002017206070176166.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:45,491] Trial 3 finished with value: 0.00194666545703058 and parameters: {'n_estimators': 289, 'learning_rate': 0.15524858967089222, 'num_leaves': 138, 'max_depth': 13, 'min_child_samples': 1, 'subsample': 0.5607345402831887, 'colsample_bytree': 0.7369513473883429, 'reg_alpha': 7.3718195799365684, 'reg_lambda': 0.6218320431998892, 'min_split_gain': 0.03465859444086889}. Best is trial 3 with value: 0.00194666545703058.
[I 2025-06-15 20:12:45,495] A new study created in memory with name: no-name-b4784414-5a60-41fd-b399-f97eb4b706ef
[I 2025-06-15 20:12:45,525] Trial 0 finished with value: 0.003852317773752185 and parameters: {'n_estimators': 158, 'learning_rate': 0.04534103986209207, 'num_leaves': 134, 'max_depth': 13, 'min_child_samples': 8, 'subsample': 0.7417400516630122, 'colsample_bytree': 0.9925300689765306, 'reg_alpha': 1.820381183878318, 'reg_lambda': 9.981687976941501, 'min_split_gain': 0.04614967049897825}. Best is trial 0 with value: 0.003852317773752185.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         313
1           Price_lag_2         222
14      pct_chg_Load_DA         178
2           Price_lag_7         128
11           NGas_lag_2         127
16  roll2d_median_Price         108
3         Load_DA_lag_0         107
10           Coal_lag_2          88
15       lag168_Load_DA          68
13            EUA_lag_2          65
5       WindOn_DA_lag_0          60
12            Oil_lag_2          34
9                  WD_7           6
7                  WD_1           4
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          69
3         Load_DA_lag_0          35
10           Coal_lag_2           7
15       lag168_Load_DA           2
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 49.38% done
********************* END NS.. 360 ****************************************************
********************* START NS ... 361 ****************************************************
[I 2025-06-15 20:12:46,910] A new study created in memory with name: no-name-72ad6e73-b4f8-42c6-beb0-1e2d8648063f
[I 2025-06-15 20:12:47,052] Trial 0 finished with value: 0.00201608740904291 and parameters: {'n_estimators': 256, 'learning_rate': 0.25291684055311126, 'num_leaves': 214, 'max_depth': 4, 'min_child_samples': 30, 'subsample': 0.9930374398333351, 'colsample_bytree': 0.9937860013327238, 'reg_alpha': 6.942796702861042, 'reg_lambda': 2.3241497222764753, 'min_split_gain': 0.04563378606145917}. Best is trial 0 with value: 0.00201608740904291.
[Trial 0] New best: 0.00202
[I 2025-06-15 20:12:47,931] Trial 1 finished with value: 0.0018982420809533524 and parameters: {'n_estimators': 277, 'learning_rate': 0.02214300401808671, 'num_leaves': 134, 'max_depth': 9, 'min_child_samples': 12, 'subsample': 0.6809143474836744, 'colsample_bytree': 0.5558912937323669, 'reg_alpha': 4.826687845618731, 'reg_lambda': 5.173155192194844, 'min_split_gain': 0.04104000749313761}. Best is trial 1 with value: 0.0018982420809533524.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:48,866] Trial 2 finished with value: 0.0013868476242719964 and parameters: {'n_estimators': 235, 'learning_rate': 0.025481198551798055, 'num_leaves': 47, 'max_depth': 12, 'min_child_samples': 26, 'subsample': 0.9370973063178103, 'colsample_bytree': 0.8004258561920493, 'reg_alpha': 3.664503003154306, 'reg_lambda': 3.472837450986299, 'min_split_gain': 0.03177586186073313}. Best is trial 2 with value: 0.0013868476242719964.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:49,412] Trial 3 finished with value: 0.0019254456316969814 and parameters: {'n_estimators': 109, 'learning_rate': 0.04889040162947757, 'num_leaves': 81, 'max_depth': 9, 'min_child_samples': 13, 'subsample': 0.8904622203254883, 'colsample_bytree': 0.5524851067643433, 'reg_alpha': 7.132043972668266, 'reg_lambda': 8.14537649590198, 'min_split_gain': 0.03815677967644597}. Best is trial 2 with value: 0.0013868476242719964.
[I 2025-06-15 20:12:49,414] A new study created in memory with name: no-name-a23428d3-7b15-4afc-a4c7-6c6ae3b0e313
[I 2025-06-15 20:12:49,444] Trial 0 finished with value: 0.02928502950772736 and parameters: {'n_estimators': 106, 'learning_rate': 0.17648054331062676, 'num_leaves': 228, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.6848953864910119, 'colsample_bytree': 0.5763625931124854, 'reg_alpha': 3.5720007790212325, 'reg_lambda': 8.978691573065266, 'min_split_gain': 0.03376143457768768}. Best is trial 0 with value: 0.02928502950772736.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2264
1           Price_lag_2        1799
14      pct_chg_Load_DA        1309
3         Load_DA_lag_0         684
16  roll2d_median_Price         547
11           NGas_lag_2         542
2           Price_lag_7         531
15       lag168_Load_DA         447
5       WindOn_DA_lag_0         332
12            Oil_lag_2         286
10           Coal_lag_2         281
13            EUA_lag_2         273
9                  WD_7          27
7                  WD_1          17
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          13
3         Load_DA_lag_0           6
1           Price_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 49.52% done
********************* END NS.. 361 ****************************************************
********************* START NS ... 362 ****************************************************
[I 2025-06-15 20:12:51,369] A new study created in memory with name: no-name-89aceb1c-3b4b-4198-ba2d-b8320cb0182c
[I 2025-06-15 20:12:51,916] Trial 0 finished with value: 0.0028016172322170517 and parameters: {'n_estimators': 89, 'learning_rate': 0.03165378756550766, 'num_leaves': 89, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.6565041785199639, 'colsample_bytree': 0.8122644850972347, 'reg_alpha': 0.4550860964483727, 'reg_lambda': 3.02158151795924, 'min_split_gain': 0.04489945486592754}. Best is trial 0 with value: 0.0028016172322170517.
[Trial 0] New best: 0.00280
[I 2025-06-15 20:12:52,280] Trial 1 finished with value: 0.04263348067415521 and parameters: {'n_estimators': 58, 'learning_rate': 0.015068218756518594, 'num_leaves': 112, 'max_depth': 11, 'min_child_samples': 13, 'subsample': 0.75230170055304, 'colsample_bytree': 0.7077237411141937, 'reg_alpha': 0.9654738330218993, 'reg_lambda': 9.512943582952595, 'min_split_gain': 0.03327155215418886}. Best is trial 0 with value: 0.0028016172322170517.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:52,691] Trial 2 finished with value: 0.0039507800140389385 and parameters: {'n_estimators': 111, 'learning_rate': 0.021580967546951917, 'num_leaves': 71, 'max_depth': 8, 'min_child_samples': 28, 'subsample': 0.8640156737126474, 'colsample_bytree': 0.7677194395499916, 'reg_alpha': 9.336487870153263, 'reg_lambda': 1.9392754783961508, 'min_split_gain': 0.007026858994625513}. Best is trial 0 with value: 0.0028016172322170517.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:53,663] Trial 3 finished with value: 0.001691141347028531 and parameters: {'n_estimators': 100, 'learning_rate': 0.04576093291924688, 'num_leaves': 108, 'max_depth': 9, 'min_child_samples': 12, 'subsample': 0.9716930018085901, 'colsample_bytree': 0.9430939988981426, 'reg_alpha': 7.588070086445113, 'reg_lambda': 3.3338711660912956, 'min_split_gain': 0.012710110364437033}. Best is trial 3 with value: 0.001691141347028531.
[Trial 3] New best: 0.00169
[I 2025-06-15 20:12:54,036] Trial 4 finished with value: 0.001859336477558457 and parameters: {'n_estimators': 165, 'learning_rate': 0.15473960726993802, 'num_leaves': 228, 'max_depth': 6, 'min_child_samples': 26, 'subsample': 0.8569349559569324, 'colsample_bytree': 0.5070248493522934, 'reg_alpha': 7.426300873802759, 'reg_lambda': 2.3613184864407555, 'min_split_gain': 0.006581671376350795}. Best is trial 3 with value: 0.001691141347028531.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:54,287] Trial 5 finished with value: 0.0021062673637869695 and parameters: {'n_estimators': 52, 'learning_rate': 0.26504248871278174, 'num_leaves': 79, 'max_depth': 14, 'min_child_samples': 21, 'subsample': 0.7452093031785255, 'colsample_bytree': 0.8876912470931384, 'reg_alpha': 4.179666144188209, 'reg_lambda': 3.4162491040735112, 'min_split_gain': 0.005766950676961791}. Best is trial 3 with value: 0.001691141347028531.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:54,507] Trial 6 finished with value: 0.0021222162828342887 and parameters: {'n_estimators': 123, 'learning_rate': 0.27557381950529625, 'num_leaves': 117, 'max_depth': 12, 'min_child_samples': 26, 'subsample': 0.8003832694452251, 'colsample_bytree': 0.8593392224694425, 'reg_alpha': 8.346813068066348, 'reg_lambda': 9.990672235124084, 'min_split_gain': 0.016061777899068496}. Best is trial 3 with value: 0.001691141347028531.
[I 2025-06-15 20:12:54,508] A new study created in memory with name: no-name-4876a1a6-0559-4bbc-8f17-9f78ec2cf265
[I 2025-06-15 20:12:54,562] Trial 0 finished with value: 0.018663994746129453 and parameters: {'n_estimators': 153, 'learning_rate': 0.0610449893845483, 'num_leaves': 191, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.7473015334032214, 'colsample_bytree': 0.9619043321718362, 'reg_alpha': 2.0500354540787313, 'reg_lambda': 3.437866960660693, 'min_split_gain': 0.011609813909154915}. Best is trial 0 with value: 0.018663994746129453.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1046
1           Price_lag_2         624
14      pct_chg_Load_DA         534
16  roll2d_median_Price         302
2           Price_lag_7         274
3         Load_DA_lag_0         248
11           NGas_lag_2         215
13            EUA_lag_2         147
10           Coal_lag_2         126
15       lag168_Load_DA         113
5       WindOn_DA_lag_0          96
12            Oil_lag_2          83
9                  WD_7          15
7                  WD_1           1
8                  WD_6           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          81
3         Load_DA_lag_0          38
15       lag168_Load_DA          23
12            Oil_lag_2           2
1           Price_lag_2           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 49.66% done
********************* END NS.. 362 ****************************************************
********************* START NS ... 363 ****************************************************
[I 2025-06-15 20:12:55,906] A new study created in memory with name: no-name-99c807dd-c254-44eb-932c-04cae8c8d024
[I 2025-06-15 20:12:56,113] Trial 0 finished with value: 0.0010637225772825677 and parameters: {'n_estimators': 231, 'learning_rate': 0.13376866419988062, 'num_leaves': 63, 'max_depth': 10, 'min_child_samples': 25, 'subsample': 0.8547691160176555, 'colsample_bytree': 0.8903073310601743, 'reg_alpha': 2.315162729381152, 'reg_lambda': 9.377907524716521, 'min_split_gain': 0.03602154810664169}. Best is trial 0 with value: 0.0010637225772825677.
[I 2025-06-15 20:12:56,308] Trial 1 finished with value: 0.0014899271469455656 and parameters: {'n_estimators': 300, 'learning_rate': 0.13099169368792044, 'num_leaves': 38, 'max_depth': 12, 'min_child_samples': 11, 'subsample': 0.6135345697558516, 'colsample_bytree': 0.5257698210293891, 'reg_alpha': 8.919948169779866, 'reg_lambda': 4.156305086110608, 'min_split_gain': 0.04873470243940346}. Best is trial 0 with value: 0.0010637225772825677.
[Trial 0] New best: 0.00106
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:56,702] Trial 2 finished with value: 0.0025122263199037135 and parameters: {'n_estimators': 132, 'learning_rate': 0.02595086323922403, 'num_leaves': 212, 'max_depth': 12, 'min_child_samples': 23, 'subsample': 0.656146011353888, 'colsample_bytree': 0.5620470697175608, 'reg_alpha': 8.564676941788479, 'reg_lambda': 1.0409497058584005, 'min_split_gain': 0.0034462319383607334}. Best is trial 0 with value: 0.0010637225772825677.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:57,176] Trial 3 finished with value: 0.0011307409180700428 and parameters: {'n_estimators': 239, 'learning_rate': 0.12811841137806723, 'num_leaves': 107, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.981048559586349, 'colsample_bytree': 0.6513790114992519, 'reg_alpha': 1.6386386670627573, 'reg_lambda': 9.158147213740524, 'min_split_gain': 0.013873438993217635}. Best is trial 0 with value: 0.0010637225772825677.
[I 2025-06-15 20:12:57,179] A new study created in memory with name: no-name-000ffa05-8436-4fe4-961a-7b73784561b9
[I 2025-06-15 20:12:57,208] Trial 0 finished with value: 0.004090980072854367 and parameters: {'n_estimators': 241, 'learning_rate': 0.10177224774177032, 'num_leaves': 210, 'max_depth': 14, 'min_child_samples': 9, 'subsample': 0.7348585797014994, 'colsample_bytree': 0.9917530939583885, 'reg_alpha': 2.8784686677021476, 'reg_lambda': 4.664993166096067, 'min_split_gain': 0.005812000488118036}. Best is trial 0 with value: 0.004090980072854367.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         508
1           Price_lag_2         357
14      pct_chg_Load_DA         347
3         Load_DA_lag_0         222
16  roll2d_median_Price         155
11           NGas_lag_2         148
5       WindOn_DA_lag_0         144
2           Price_lag_7         142
15       lag168_Load_DA         126
10           Coal_lag_2         113
13            EUA_lag_2          84
12            Oil_lag_2          48
7                  WD_1           9
9                  WD_7           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          53
15       lag168_Load_DA          18
3         Load_DA_lag_0           5
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 49.79% done
********************* END NS.. 363 ****************************************************
********************* START NS ... 364 ****************************************************
[I 2025-06-15 20:12:58,604] A new study created in memory with name: no-name-8f1265a2-1475-4495-b186-fc875cf11802
[I 2025-06-15 20:12:58,817] Trial 0 finished with value: 0.0012808718093222245 and parameters: {'n_estimators': 235, 'learning_rate': 0.26167642802363106, 'num_leaves': 170, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.6520538184341602, 'colsample_bytree': 0.5720012071950424, 'reg_alpha': 4.057045174484124, 'reg_lambda': 6.639622862634616, 'min_split_gain': 0.0076542255977057}. Best is trial 0 with value: 0.0012808718093222245.
[I 2025-06-15 20:12:59,029] Trial 1 finished with value: 0.0009827505096208161 and parameters: {'n_estimators': 260, 'learning_rate': 0.21304840649054282, 'num_leaves': 111, 'max_depth': 11, 'min_child_samples': 7, 'subsample': 0.7019170251043043, 'colsample_bytree': 0.8096995114309871, 'reg_alpha': 2.47480447725838, 'reg_lambda': 9.495168922978369, 'min_split_gain': 0.03218022088368799}. Best is trial 1 with value: 0.0009827505096208161.
[Trial 0] New best: 0.00128
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:12:59,349] Trial 2 finished with value: 0.0010427722957658681 and parameters: {'n_estimators': 286, 'learning_rate': 0.16724263744916837, 'num_leaves': 93, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.7641085905276218, 'colsample_bytree': 0.8122037651357111, 'reg_alpha': 0.09353848088734296, 'reg_lambda': 8.286480962709707, 'min_split_gain': 0.03432227850270597}. Best is trial 1 with value: 0.0009827505096208161.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:12:59,877] Trial 3 finished with value: 0.0010293804014584 and parameters: {'n_estimators': 216, 'learning_rate': 0.06843014860068423, 'num_leaves': 252, 'max_depth': 6, 'min_child_samples': 30, 'subsample': 0.5236119274194023, 'colsample_bytree': 0.777703215524864, 'reg_alpha': 0.36950235046020463, 'reg_lambda': 1.7646094920103772, 'min_split_gain': 0.013997235687118283}. Best is trial 1 with value: 0.0009827505096208161.
[I 2025-06-15 20:12:59,880] A new study created in memory with name: no-name-08b3baa4-79ea-4e20-b293-cc1ff3516a02
[I 2025-06-15 20:12:59,913] Trial 0 finished with value: 0.0026190689505428866 and parameters: {'n_estimators': 195, 'learning_rate': 0.09075492221483458, 'num_leaves': 115, 'max_depth': 6, 'min_child_samples': 29, 'subsample': 0.7181225138942615, 'colsample_bytree': 0.8618387410784348, 'reg_alpha': 0.6002125765291333, 'reg_lambda': 2.7649559213561448, 'min_split_gain': 0.009529137417213913}. Best is trial 0 with value: 0.0026190689505428866.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         319
1           Price_lag_2         265
14      pct_chg_Load_DA         264
3         Load_DA_lag_0         165
16  roll2d_median_Price         150
2           Price_lag_7         125
5       WindOn_DA_lag_0         111
11           NGas_lag_2         111
15       lag168_Load_DA         111
10           Coal_lag_2          70
13            EUA_lag_2          52
12            Oil_lag_2          45
9                  WD_7           9
7                  WD_1           4
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          50
15       lag168_Load_DA          12
14      pct_chg_Load_DA           7
1           Price_lag_2           6
3         Load_DA_lag_0           3
10           Coal_lag_2           1
2           Price_lag_7           1
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 49.93% done
********************* END NS.. 364 ****************************************************
********************* START NS ... 365 ****************************************************
[I 2025-06-15 20:13:01,342] A new study created in memory with name: no-name-792607d1-0c7e-424a-9f57-9576dd789d6f
[I 2025-06-15 20:13:01,543] Trial 0 finished with value: 0.0009595411638420627 and parameters: {'n_estimators': 100, 'learning_rate': 0.11058131079666156, 'num_leaves': 56, 'max_depth': 9, 'min_child_samples': 12, 'subsample': 0.7720196929354555, 'colsample_bytree': 0.783098573180961, 'reg_alpha': 2.2792133663370864, 'reg_lambda': 3.6272679612151357, 'min_split_gain': 0.03783011713991119}. Best is trial 0 with value: 0.0009595411638420627.
[I 2025-06-15 20:13:01,723] Trial 1 finished with value: 0.04676656304944784 and parameters: {'n_estimators': 60, 'learning_rate': 0.012428153856610889, 'num_leaves': 71, 'max_depth': 7, 'min_child_samples': 15, 'subsample': 0.735615234945014, 'colsample_bytree': 0.6304226901060759, 'reg_alpha': 3.608426911760948, 'reg_lambda': 4.983425823051269, 'min_split_gain': 0.021369102073398566}. Best is trial 0 with value: 0.0009595411638420627.
[Trial 0] New best: 0.00096
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:02,150] Trial 2 finished with value: 0.0009210363697167032 and parameters: {'n_estimators': 242, 'learning_rate': 0.05821487123106947, 'num_leaves': 76, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.8798842324381608, 'colsample_bytree': 0.8355669859126668, 'reg_alpha': 6.439962297923677, 'reg_lambda': 1.6581791104760413, 'min_split_gain': 0.046914543134676245}. Best is trial 2 with value: 0.0009210363697167032.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:03,214] Trial 3 finished with value: 0.0009593697216004988 and parameters: {'n_estimators': 290, 'learning_rate': 0.02857931534035056, 'num_leaves': 162, 'max_depth': 8, 'min_child_samples': 28, 'subsample': 0.6024857178175063, 'colsample_bytree': 0.7700139091327782, 'reg_alpha': 0.9822771621350301, 'reg_lambda': 1.3500123953195264, 'min_split_gain': 0.04287000419328593}. Best is trial 2 with value: 0.0009210363697167032.
[I 2025-06-15 20:13:03,216] A new study created in memory with name: no-name-8b989704-e765-49dc-9325-cd6ae6b916a9
[I 2025-06-15 20:13:03,242] Trial 0 finished with value: 0.01111740313870715 and parameters: {'n_estimators': 246, 'learning_rate': 0.09746950177048049, 'num_leaves': 128, 'max_depth': 11, 'min_child_samples': 8, 'subsample': 0.7240815198914616, 'colsample_bytree': 0.9048231846839907, 'reg_alpha': 6.319482474218167, 'reg_lambda': 6.87681438668878, 'min_split_gain': 0.006197144980135367}. Best is trial 0 with value: 0.01111740313870715.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         878
1           Price_lag_2         661
14      pct_chg_Load_DA         566
11           NGas_lag_2         297
2           Price_lag_7         289
3         Load_DA_lag_0         275
16  roll2d_median_Price         248
10           Coal_lag_2         171
5       WindOn_DA_lag_0         169
15       lag168_Load_DA         159
13            EUA_lag_2         115
12            Oil_lag_2          83
9                  WD_7          15
7                  WD_1           7
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          15
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 50.07% done
********************* END NS.. 365 ****************************************************
********************* START NS ... 366 ****************************************************
[I 2025-06-15 20:13:04,667] A new study created in memory with name: no-name-d6313724-3fab-4765-90c3-07e51a2e5ec2
[I 2025-06-15 20:13:05,370] Trial 0 finished with value: 0.002985295212286026 and parameters: {'n_estimators': 253, 'learning_rate': 0.01410852926358105, 'num_leaves': 136, 'max_depth': 10, 'min_child_samples': 20, 'subsample': 0.5370136122028599, 'colsample_bytree': 0.9631975675431197, 'reg_alpha': 8.728506525613536, 'reg_lambda': 7.278703751831369, 'min_split_gain': 0.027714455023957286}. Best is trial 0 with value: 0.002985295212286026.
[Trial 0] New best: 0.00299
[I 2025-06-15 20:13:06,629] Trial 1 finished with value: 0.003482214209991789 and parameters: {'n_estimators': 181, 'learning_rate': 0.017565762183058192, 'num_leaves': 113, 'max_depth': 16, 'min_child_samples': 12, 'subsample': 0.6365552863120777, 'colsample_bytree': 0.6444896353041967, 'reg_alpha': 0.8471536259746537, 'reg_lambda': 1.7244724863832184, 'min_split_gain': 0.010362740953364613}. Best is trial 0 with value: 0.002985295212286026.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:06,941] Trial 2 finished with value: 0.002128644732965319 and parameters: {'n_estimators': 125, 'learning_rate': 0.17538441797875898, 'num_leaves': 201, 'max_depth': 14, 'min_child_samples': 2, 'subsample': 0.7399303806313422, 'colsample_bytree': 0.7015903755309358, 'reg_alpha': 8.393171947102969, 'reg_lambda': 3.1224055763817193, 'min_split_gain': 0.01174386582274411}. Best is trial 2 with value: 0.002128644732965319.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:07,520] Trial 3 finished with value: 0.0028047972715582723 and parameters: {'n_estimators': 232, 'learning_rate': 0.06464633357099257, 'num_leaves': 164, 'max_depth': 9, 'min_child_samples': 11, 'subsample': 0.8617553106819225, 'colsample_bytree': 0.5856596916518537, 'reg_alpha': 7.420096432568162, 'reg_lambda': 0.7735355310234193, 'min_split_gain': 0.03167683027296644}. Best is trial 2 with value: 0.002128644732965319.
[I 2025-06-15 20:13:07,523] A new study created in memory with name: no-name-e3667ef4-6529-4822-9a20-d12ac618a27f
[I 2025-06-15 20:13:07,546] Trial 0 finished with value: 0.027827532618337963 and parameters: {'n_estimators': 158, 'learning_rate': 0.024973673534516766, 'num_leaves': 140, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.5196064567578867, 'colsample_bytree': 0.9982098223114484, 'reg_alpha': 8.233202741838923, 'reg_lambda': 0.0748449725745326, 'min_split_gain': 0.02036869601062401}. Best is trial 0 with value: 0.027827532618337963.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         324
1           Price_lag_2         252
14      pct_chg_Load_DA         200
2           Price_lag_7         148
3         Load_DA_lag_0         134
11           NGas_lag_2         129
16  roll2d_median_Price         128
10           Coal_lag_2         107
15       lag168_Load_DA         104
5       WindOn_DA_lag_0         103
13            EUA_lag_2         102
12            Oil_lag_2          58
9                  WD_7          15
7                  WD_1           8
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 50.21% done
********************* END NS.. 366 ****************************************************
********************* START NS ... 367 ****************************************************
[I 2025-06-15 20:13:08,509] A new study created in memory with name: no-name-dabd50cb-e0ad-46cb-97b1-b074c2a647e7
[I 2025-06-15 20:13:08,704] Trial 0 finished with value: 0.0024568950730895745 and parameters: {'n_estimators': 104, 'learning_rate': 0.10300465720639428, 'num_leaves': 178, 'max_depth': 11, 'min_child_samples': 30, 'subsample': 0.875520891337563, 'colsample_bytree': 0.9928288509202161, 'reg_alpha': 5.3350267897357995, 'reg_lambda': 2.8016760950509614, 'min_split_gain': 0.04153575184665632}. Best is trial 0 with value: 0.0024568950730895745.
[I 2025-06-15 20:13:08,885] Trial 1 finished with value: 0.0031447199230731 and parameters: {'n_estimators': 50, 'learning_rate': 0.06545294440646307, 'num_leaves': 59, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.6500475136904449, 'colsample_bytree': 0.7827803990945212, 'reg_alpha': 0.6947111248818494, 'reg_lambda': 9.862862672891058, 'min_split_gain': 0.04993417245921364}. Best is trial 0 with value: 0.0024568950730895745.
[Trial 0] New best: 0.00246
[I 2025-06-15 20:13:09,092] Trial 2 finished with value: 0.0025193982429084763 and parameters: {'n_estimators': 68, 'learning_rate': 0.16193413206692922, 'num_leaves': 40, 'max_depth': 13, 'min_child_samples': 24, 'subsample': 0.7099919681217994, 'colsample_bytree': 0.6453489399572628, 'reg_alpha': 2.087679757561286, 'reg_lambda': 4.246367867982555, 'min_split_gain': 0.011864425416377345}. Best is trial 0 with value: 0.0024568950730895745.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:09,335] Trial 3 finished with value: 0.020216232239425803 and parameters: {'n_estimators': 80, 'learning_rate': 0.01731800911649193, 'num_leaves': 116, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.5236478672550928, 'colsample_bytree': 0.9711757568940066, 'reg_alpha': 6.86959386392695, 'reg_lambda': 7.917900675907644, 'min_split_gain': 0.03570976513794302}. Best is trial 0 with value: 0.0024568950730895745.
[I 2025-06-15 20:13:09,339] A new study created in memory with name: no-name-ec6182b5-bdcd-4bd3-8360-6e7c09b3f5a9
[I 2025-06-15 20:13:09,371] Trial 0 finished with value: 0.022688142197491424 and parameters: {'n_estimators': 166, 'learning_rate': 0.020726376104182744, 'num_leaves': 120, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.906855141361534, 'colsample_bytree': 0.7117932624484411, 'reg_alpha': 1.635183101966301, 'reg_lambda': 9.701916592426405, 'min_split_gain': 0.04524512831001198}. Best is trial 0 with value: 0.022688142197491424.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         522
14      pct_chg_Load_DA         327
1           Price_lag_2         233
3         Load_DA_lag_0         156
11           NGas_lag_2         149
16  roll2d_median_Price         124
2           Price_lag_7         117
5       WindOn_DA_lag_0          90
10           Coal_lag_2          70
15       lag168_Load_DA          64
13            EUA_lag_2          61
12            Oil_lag_2          45
7                  WD_1           7
9                  WD_7           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         128
15       lag168_Load_DA          48
1           Price_lag_2          19
3         Load_DA_lag_0           3
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 50.34% done
********************* END NS.. 367 ****************************************************
********************* START NS ... 368 ****************************************************
[I 2025-06-15 20:13:10,309] A new study created in memory with name: no-name-bf5d0daa-99cd-4fa5-a29d-e7593fccb8b3
[I 2025-06-15 20:13:10,458] Trial 0 finished with value: 0.0019617509114287393 and parameters: {'n_estimators': 54, 'learning_rate': 0.07484353613160383, 'num_leaves': 119, 'max_depth': 8, 'min_child_samples': 22, 'subsample': 0.8068959176341122, 'colsample_bytree': 0.5226667483060812, 'reg_alpha': 5.35385640865366, 'reg_lambda': 4.620338178685797, 'min_split_gain': 0.011389193217828298}. Best is trial 0 with value: 0.0019617509114287393.
[Trial 0] New best: 0.00196
[I 2025-06-15 20:13:10,708] Trial 1 finished with value: 0.0012411580908499294 and parameters: {'n_estimators': 228, 'learning_rate': 0.12343140578688633, 'num_leaves': 57, 'max_depth': 16, 'min_child_samples': 1, 'subsample': 0.5385437693935773, 'colsample_bytree': 0.8323068959193656, 'reg_alpha': 1.643503220844904, 'reg_lambda': 3.7970645910580316, 'min_split_gain': 0.03335473309038931}. Best is trial 1 with value: 0.0012411580908499294.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:11,105] Trial 2 finished with value: 0.0011120635657821838 and parameters: {'n_estimators': 224, 'learning_rate': 0.09129511743843996, 'num_leaves': 34, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.5897189059985436, 'colsample_bytree': 0.9298161996226263, 'reg_alpha': 7.299139593356002, 'reg_lambda': 9.253852266355892, 'min_split_gain': 0.023268576502361162}. Best is trial 2 with value: 0.0011120635657821838.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:11,580] Trial 3 finished with value: 0.0014518579043053424 and parameters: {'n_estimators': 286, 'learning_rate': 0.0843404322481311, 'num_leaves': 82, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.860474515320922, 'colsample_bytree': 0.5954748400405808, 'reg_alpha': 8.238694318749598, 'reg_lambda': 9.143661825785003, 'min_split_gain': 0.0387518363735891}. Best is trial 2 with value: 0.0011120635657821838.
[I 2025-06-15 20:13:11,583] A new study created in memory with name: no-name-7e4cb958-2316-40b0-ae60-81cd4027472b
[I 2025-06-15 20:13:11,602] Trial 0 finished with value: 0.029493228866553135 and parameters: {'n_estimators': 117, 'learning_rate': 0.07250144930103117, 'num_leaves': 67, 'max_depth': 14, 'min_child_samples': 23, 'subsample': 0.6704489936052187, 'colsample_bytree': 0.884408040066704, 'reg_alpha': 8.105793951612254, 'reg_lambda': 4.293561324821317, 'min_split_gain': 0.002484560022793103}. Best is trial 0 with value: 0.029493228866553135.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         569
14      pct_chg_Load_DA         352
1           Price_lag_2         297
3         Load_DA_lag_0         181
11           NGas_lag_2         167
16  roll2d_median_Price         129
2           Price_lag_7         124
10           Coal_lag_2         114
5       WindOn_DA_lag_0         111
15       lag168_Load_DA         101
13            EUA_lag_2          76
12            Oil_lag_2          49
7                  WD_1          12
9                  WD_7          12
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          18
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 50.48% done
********************* END NS.. 368 ****************************************************
********************* START NS ... 369 ****************************************************
[I 2025-06-15 20:13:12,885] A new study created in memory with name: no-name-5186f51f-7ef0-4945-bf58-d26f621d5db9
[I 2025-06-15 20:13:13,082] Trial 0 finished with value: 0.002628168866388636 and parameters: {'n_estimators': 236, 'learning_rate': 0.14573505912106594, 'num_leaves': 91, 'max_depth': 8, 'min_child_samples': 5, 'subsample': 0.5284623854308864, 'colsample_bytree': 0.9752200957167168, 'reg_alpha': 1.2591435947165164, 'reg_lambda': 6.1913282075311535, 'min_split_gain': 0.04227284832145021}. Best is trial 0 with value: 0.002628168866388636.
[Trial 0] New best: 0.00263
[I 2025-06-15 20:13:13,455] Trial 1 finished with value: 0.003849335060126 and parameters: {'n_estimators': 152, 'learning_rate': 0.03489177517476489, 'num_leaves': 93, 'max_depth': 6, 'min_child_samples': 12, 'subsample': 0.5940680325388374, 'colsample_bytree': 0.7290305424952443, 'reg_alpha': 4.371146777375069, 'reg_lambda': 3.5921656517370835, 'min_split_gain': 0.045033317298905716}. Best is trial 0 with value: 0.002628168866388636.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:13,989] Trial 2 finished with value: 0.003963124980285752 and parameters: {'n_estimators': 190, 'learning_rate': 0.033000589105435484, 'num_leaves': 91, 'max_depth': 6, 'min_child_samples': 26, 'subsample': 0.7851926644754382, 'colsample_bytree': 0.8618681524696921, 'reg_alpha': 8.1196908483846, 'reg_lambda': 1.7667056860658825, 'min_split_gain': 0.0047723824693351735}. Best is trial 0 with value: 0.002628168866388636.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:14,536] Trial 3 finished with value: 0.009898103904417498 and parameters: {'n_estimators': 134, 'learning_rate': 0.010970787820367471, 'num_leaves': 92, 'max_depth': 11, 'min_child_samples': 11, 'subsample': 0.7934292140310444, 'colsample_bytree': 0.6620818713859685, 'reg_alpha': 4.8437555106134695, 'reg_lambda': 9.079096196219432, 'min_split_gain': 0.007668611610208854}. Best is trial 0 with value: 0.002628168866388636.
[I 2025-06-15 20:13:14,539] A new study created in memory with name: no-name-fb7d1237-fe53-4890-94da-8bf5053db309
[I 2025-06-15 20:13:14,558] Trial 0 finished with value: 0.07692568461441845 and parameters: {'n_estimators': 97, 'learning_rate': 0.053621816724795573, 'num_leaves': 168, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.9304164775614316, 'colsample_bytree': 0.9925105467081056, 'reg_alpha': 2.2324972872107063, 'reg_lambda': 7.80936149056514, 'min_split_gain': 0.02496624271957852}. Best is trial 0 with value: 0.07692568461441845.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         519
14      pct_chg_Load_DA         477
1           Price_lag_2         432
3         Load_DA_lag_0         275
16  roll2d_median_Price         227
2           Price_lag_7         219
11           NGas_lag_2         195
5       WindOn_DA_lag_0         178
15       lag168_Load_DA         165
10           Coal_lag_2         102
12            Oil_lag_2          96
13            EUA_lag_2          88
9                  WD_7          10
7                  WD_1           7
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          89
3         Load_DA_lag_0          18
11           NGas_lag_2          15
15       lag168_Load_DA          14
5       WindOn_DA_lag_0           2
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 50.62% done
********************* END NS.. 369 ****************************************************
********************* START NS ... 370 ****************************************************
[I 2025-06-15 20:13:15,768] A new study created in memory with name: no-name-4d5e7880-77cf-42aa-a62b-2cca80463382
[I 2025-06-15 20:13:16,316] Trial 0 finished with value: 0.2911758887063089 and parameters: {'n_estimators': 137, 'learning_rate': 0.21297956157093423, 'num_leaves': 84, 'max_depth': 13, 'min_child_samples': 24, 'subsample': 0.9404402162554076, 'colsample_bytree': 0.6581875982689518, 'reg_alpha': 0.21550927102385264, 'reg_lambda': 4.820959673988101, 'min_split_gain': 0.0022777692544660223}. Best is trial 0 with value: 0.2911758887063089.
[Trial 0] New best: 0.29118
[I 2025-06-15 20:13:16,954] Trial 1 finished with value: 0.37728156402853547 and parameters: {'n_estimators': 295, 'learning_rate': 0.049982239441503554, 'num_leaves': 130, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.9832318350877096, 'colsample_bytree': 0.5648470691955171, 'reg_alpha': 9.405940014813458, 'reg_lambda': 3.2225337235314075, 'min_split_gain': 0.0132753303921974}. Best is trial 0 with value: 0.2911758887063089.
[I 2025-06-15 20:13:17,156] Trial 2 finished with value: 0.3409431011130633 and parameters: {'n_estimators': 187, 'learning_rate': 0.24120761670889657, 'num_leaves': 38, 'max_depth': 9, 'min_child_samples': 16, 'subsample': 0.8886498788094463, 'colsample_bytree': 0.5613772994213571, 'reg_alpha': 8.711766210180183, 'reg_lambda': 7.478367731161434, 'min_split_gain': 0.036236639107317725}. Best is trial 0 with value: 0.2911758887063089.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:18,103] Trial 3 finished with value: 0.421907774068114 and parameters: {'n_estimators': 214, 'learning_rate': 0.02523404636066712, 'num_leaves': 172, 'max_depth': 16, 'min_child_samples': 2, 'subsample': 0.8534694518798439, 'colsample_bytree': 0.5903420993861233, 'reg_alpha': 8.238227829464876, 'reg_lambda': 0.1473588963483774, 'min_split_gain': 0.01289801992296607}. Best is trial 0 with value: 0.2911758887063089.
[I 2025-06-15 20:13:18,106] A new study created in memory with name: no-name-a5a26cbe-0a3a-4379-8533-2a9ae4848363
[I 2025-06-15 20:13:18,134] Trial 0 finished with value: 1.753142678781689 and parameters: {'n_estimators': 60, 'learning_rate': 0.02822656553074805, 'num_leaves': 193, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.7733509437674341, 'colsample_bytree': 0.5119175667852403, 'reg_alpha': 0.03945291591277433, 'reg_lambda': 5.390968995316584, 'min_split_gain': 0.019984501724657877}. Best is trial 0 with value: 1.753142678781689.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1085
3         Load_DA_lag_0        1058
14      pct_chg_Load_DA         985
1           Price_lag_2         980
5       WindOn_DA_lag_0         841
15       lag168_Load_DA         772
2           Price_lag_7         612
16  roll2d_median_Price         578
11           NGas_lag_2         577
12            Oil_lag_2         471
13            EUA_lag_2         430
10           Coal_lag_2         425
9                  WD_7          17
7                  WD_1          16
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          75
3         Load_DA_lag_0          27
1           Price_lag_2          18
15       lag168_Load_DA          17
5       WindOn_DA_lag_0          12
16  roll2d_median_Price           7
12            Oil_lag_2           4
14      pct_chg_Load_DA           3
13            EUA_lag_2           1
2           Price_lag_7           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
-> 50.75% done
********************* END NS.. 370 ****************************************************
********************* START NS ... 371 ****************************************************
[I 2025-06-15 20:13:19,488] A new study created in memory with name: no-name-803b3b02-ccc9-4461-a6bc-b564a6db9494
[I 2025-06-15 20:13:20,080] Trial 0 finished with value: 0.29483117251567853 and parameters: {'n_estimators': 236, 'learning_rate': 0.01433913283010975, 'num_leaves': 58, 'max_depth': 14, 'min_child_samples': 30, 'subsample': 0.6841442822215232, 'colsample_bytree': 0.7913416099417143, 'reg_alpha': 9.704466008755627, 'reg_lambda': 8.933393813532014, 'min_split_gain': 0.01914981932600623}. Best is trial 0 with value: 0.29483117251567853.
[Trial 0] New best: 0.29483
[I 2025-06-15 20:13:20,348] Trial 1 finished with value: 0.26804903770154104 and parameters: {'n_estimators': 267, 'learning_rate': 0.1867359437730169, 'num_leaves': 70, 'max_depth': 16, 'min_child_samples': 13, 'subsample': 0.5055188423743548, 'colsample_bytree': 0.6209464081291427, 'reg_alpha': 5.07440538359198, 'reg_lambda': 2.4840154639528365, 'min_split_gain': 0.020926684616653247}. Best is trial 1 with value: 0.26804903770154104.
[Trial 1] New best: 0.26805
[I 2025-06-15 20:13:20,663] Trial 2 finished with value: 0.2335983269768216 and parameters: {'n_estimators': 259, 'learning_rate': 0.0732994265929028, 'num_leaves': 248, 'max_depth': 4, 'min_child_samples': 14, 'subsample': 0.9186849715031244, 'colsample_bytree': 0.6045287406670687, 'reg_alpha': 2.2380312998035192, 'reg_lambda': 2.7152542657898904, 'min_split_gain': 0.04754791306534094}. Best is trial 2 with value: 0.2335983269768216.
[I 2025-06-15 20:13:20,845] Trial 3 finished with value: 0.20624204962107898 and parameters: {'n_estimators': 52, 'learning_rate': 0.17023055920376495, 'num_leaves': 73, 'max_depth': 6, 'min_child_samples': 1, 'subsample': 0.845874855742077, 'colsample_bytree': 0.7223213868503894, 'reg_alpha': 3.945594566183016, 'reg_lambda': 0.7345785862777066, 'min_split_gain': 0.04921179908215524}. Best is trial 3 with value: 0.20624204962107898.
[Trial 2] New best: 0.23360
[Trial 3] New best: 0.20624
[I 2025-06-15 20:13:20,976] Trial 4 finished with value: 0.2610700729098023 and parameters: {'n_estimators': 55, 'learning_rate': 0.05429178586215371, 'num_leaves': 59, 'max_depth': 4, 'min_child_samples': 14, 'subsample': 0.9907919817206061, 'colsample_bytree': 0.953021430470357, 'reg_alpha': 9.859264277508776, 'reg_lambda': 8.371760112970644, 'min_split_gain': 0.007374223927721585}. Best is trial 3 with value: 0.20624204962107898.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:21,346] Trial 5 finished with value: 0.28517827567571624 and parameters: {'n_estimators': 218, 'learning_rate': 0.10601460084611802, 'num_leaves': 67, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.9451576867093394, 'colsample_bytree': 0.5492671376702072, 'reg_alpha': 8.061834069698564, 'reg_lambda': 7.570056927316983, 'min_split_gain': 0.029814061201022292}. Best is trial 3 with value: 0.20624204962107898.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:22,208] Trial 6 finished with value: 0.24249504277559306 and parameters: {'n_estimators': 264, 'learning_rate': 0.1312037202789797, 'num_leaves': 236, 'max_depth': 13, 'min_child_samples': 22, 'subsample': 0.9376297585086639, 'colsample_bytree': 0.747251137229912, 'reg_alpha': 1.5960602726743245, 'reg_lambda': 5.804251994112505, 'min_split_gain': 0.0017602612520444207}. Best is trial 3 with value: 0.20624204962107898.
[I 2025-06-15 20:13:22,211] A new study created in memory with name: no-name-59019f0f-4912-4358-a176-dfbc06513c2b
[I 2025-06-15 20:13:22,243] Trial 0 finished with value: 1.6590055599072546 and parameters: {'n_estimators': 297, 'learning_rate': 0.08818453036668385, 'num_leaves': 213, 'max_depth': 6, 'min_child_samples': 10, 'subsample': 0.5395814345063329, 'colsample_bytree': 0.7165210384004485, 'reg_alpha': 4.563045551106618, 'reg_lambda': 1.795563057842381, 'min_split_gain': 0.03633366465706967}. Best is trial 0 with value: 1.6590055599072546.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         328
1           Price_lag_2         202
14      pct_chg_Load_DA         185
3         Load_DA_lag_0         118
2           Price_lag_7          98
11           NGas_lag_2          92
16  roll2d_median_Price          89
10           Coal_lag_2          66
5       WindOn_DA_lag_0          61
15       lag168_Load_DA          60
13            EUA_lag_2          23
12            Oil_lag_2          17
9                  WD_7           4
7                  WD_1           3
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          49
3         Load_DA_lag_0          29
15       lag168_Load_DA          11
2           Price_lag_7          10
5       WindOn_DA_lag_0           7
11           NGas_lag_2           6
16  roll2d_median_Price           6
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
6      WindOff_DA_lag_0           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 50.89% done
********************* END NS.. 371 ****************************************************
********************* START NS ... 372 ****************************************************
[I 2025-06-15 20:13:23,089] A new study created in memory with name: no-name-d6fc039c-0fae-4d56-8be5-f9ec2375b147
[I 2025-06-15 20:13:23,300] Trial 0 finished with value: 0.12936239798457247 and parameters: {'n_estimators': 263, 'learning_rate': 0.263565170059049, 'num_leaves': 76, 'max_depth': 9, 'min_child_samples': 1, 'subsample': 0.9465838627624905, 'colsample_bytree': 0.5408856752053852, 'reg_alpha': 0.8354565570673111, 'reg_lambda': 2.568340290256762, 'min_split_gain': 0.013273026361377711}. Best is trial 0 with value: 0.12936239798457247.
[Trial 0] New best: 0.12936
[I 2025-06-15 20:13:23,641] Trial 1 finished with value: 0.006446200820136854 and parameters: {'n_estimators': 263, 'learning_rate': 0.0388521068691961, 'num_leaves': 152, 'max_depth': 4, 'min_child_samples': 10, 'subsample': 0.8662932769278833, 'colsample_bytree': 0.922388155790171, 'reg_alpha': 2.5053925904297145, 'reg_lambda': 5.802768125722082, 'min_split_gain': 0.004424663456711914}. Best is trial 1 with value: 0.006446200820136854.
[Trial 1] New best: 0.00645
[I 2025-06-15 20:13:24,274] Trial 2 finished with value: 0.002323284534285026 and parameters: {'n_estimators': 156, 'learning_rate': 0.03528443971290164, 'num_leaves': 138, 'max_depth': 11, 'min_child_samples': 12, 'subsample': 0.7550598318332378, 'colsample_bytree': 0.9860924272708611, 'reg_alpha': 8.156165657713396, 'reg_lambda': 1.6665291132631488, 'min_split_gain': 0.043302517009677834}. Best is trial 2 with value: 0.002323284534285026.
[Trial 2] New best: 0.00232
[I 2025-06-15 20:13:25,032] Trial 3 finished with value: 0.002963417288273042 and parameters: {'n_estimators': 182, 'learning_rate': 0.022167137584887336, 'num_leaves': 141, 'max_depth': 9, 'min_child_samples': 6, 'subsample': 0.860010876550459, 'colsample_bytree': 0.7038843727838315, 'reg_alpha': 5.585611725516731, 'reg_lambda': 1.0282480519460713, 'min_split_gain': 0.014838639609896193}. Best is trial 2 with value: 0.002323284534285026.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:25,627] Trial 4 finished with value: 0.01223254772850016 and parameters: {'n_estimators': 143, 'learning_rate': 0.09275134554244976, 'num_leaves': 251, 'max_depth': 7, 'min_child_samples': 29, 'subsample': 0.5810629652165396, 'colsample_bytree': 0.8967489240760815, 'reg_alpha': 2.1342372347888485, 'reg_lambda': 3.6062180138394506, 'min_split_gain': 0.02018321291911602}. Best is trial 2 with value: 0.002323284534285026.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:26,072] Trial 5 finished with value: 0.002009037453655977 and parameters: {'n_estimators': 73, 'learning_rate': 0.047221265114489756, 'num_leaves': 107, 'max_depth': 7, 'min_child_samples': 26, 'subsample': 0.8373603422943967, 'colsample_bytree': 0.8079952449090784, 'reg_alpha': 8.106209754263357, 'reg_lambda': 9.72203269165439, 'min_split_gain': 0.04808604112144026}. Best is trial 5 with value: 0.002009037453655977.
[I 2025-06-15 20:13:26,076] A new study created in memory with name: no-name-fc383550-8f81-4c49-bb62-aebff4aad872
[I 2025-06-15 20:13:26,188] Trial 0 finished with value: 0.19856933411872657 and parameters: {'n_estimators': 207, 'learning_rate': 0.021303774134924954, 'num_leaves': 186, 'max_depth': 14, 'min_child_samples': 19, 'subsample': 0.5859245540476353, 'colsample_bytree': 0.8664697379901876, 'reg_alpha': 9.697391557031063, 'reg_lambda': 1.398160854763919, 'min_split_gain': 0.04024667240826135}. Best is trial 0 with value: 0.19856933411872657.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         695
1           Price_lag_2         454
14      pct_chg_Load_DA         286
2           Price_lag_7         116
16  roll2d_median_Price         105
3         Load_DA_lag_0          86
11           NGas_lag_2          78
10           Coal_lag_2          31
12            Oil_lag_2          24
13            EUA_lag_2          24
15       lag168_Load_DA           7
5       WindOn_DA_lag_0           5
9                  WD_7           4
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         132
3         Load_DA_lag_0          76
1           Price_lag_2          40
15       lag168_Load_DA          19
16  roll2d_median_Price          10
5       WindOn_DA_lag_0           7
2           Price_lag_7           5
13            EUA_lag_2           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 51.03% done
********************* END NS.. 372 ****************************************************
********************* START NS ... 373 ****************************************************
[I 2025-06-15 20:13:27,562] A new study created in memory with name: no-name-3cfcf293-5736-4dcf-86b6-34e6e1b91f2b
[I 2025-06-15 20:13:28,151] Trial 0 finished with value: 0.0036022149470082165 and parameters: {'n_estimators': 275, 'learning_rate': 0.06116259444383005, 'num_leaves': 211, 'max_depth': 11, 'min_child_samples': 11, 'subsample': 0.5238401197965383, 'colsample_bytree': 0.842382993979619, 'reg_alpha': 1.8616950901318574, 'reg_lambda': 2.8742426304876223, 'min_split_gain': 0.03402476245157848}. Best is trial 0 with value: 0.0036022149470082165.
[I 2025-06-15 20:13:28,300] Trial 1 finished with value: 0.003317090709076844 and parameters: {'n_estimators': 128, 'learning_rate': 0.22978083356664475, 'num_leaves': 151, 'max_depth': 4, 'min_child_samples': 6, 'subsample': 0.7471316751653493, 'colsample_bytree': 0.7977305348930772, 'reg_alpha': 6.283638230789301, 'reg_lambda': 5.281300174489115, 'min_split_gain': 0.04462288215829269}. Best is trial 1 with value: 0.003317090709076844.
[Trial 0] New best: 0.00360
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:28,704] Trial 2 finished with value: 0.011645658498842188 and parameters: {'n_estimators': 50, 'learning_rate': 0.01265134229640999, 'num_leaves': 220, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.84345146604711, 'colsample_bytree': 0.8853101759009452, 'reg_alpha': 3.1632786972993987, 'reg_lambda': 3.2218431955350657, 'min_split_gain': 0.016363353739623564}. Best is trial 1 with value: 0.003317090709076844.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:29,208] Trial 3 finished with value: 0.0022927366072757207 and parameters: {'n_estimators': 163, 'learning_rate': 0.06103239266426157, 'num_leaves': 82, 'max_depth': 13, 'min_child_samples': 12, 'subsample': 0.6191118551977977, 'colsample_bytree': 0.9062599169868734, 'reg_alpha': 8.544606898498508, 'reg_lambda': 9.488414371013643, 'min_split_gain': 0.034622900911406414}. Best is trial 3 with value: 0.0022927366072757207.
[Trial 3] New best: 0.00229
[I 2025-06-15 20:13:29,791] Trial 4 finished with value: 0.005979879914736992 and parameters: {'n_estimators': 50, 'learning_rate': 0.020281190983740358, 'num_leaves': 95, 'max_depth': 15, 'min_child_samples': 19, 'subsample': 0.6271707225570113, 'colsample_bytree': 0.8336394915125731, 'reg_alpha': 0.22644545889719536, 'reg_lambda': 1.214743468716587, 'min_split_gain': 0.035987680577852635}. Best is trial 3 with value: 0.0022927366072757207.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:30,384] Trial 5 finished with value: 0.003511799838480817 and parameters: {'n_estimators': 259, 'learning_rate': 0.2257699297109077, 'num_leaves': 133, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.5956405183875386, 'colsample_bytree': 0.7245404806343908, 'reg_alpha': 3.4218019122006016, 'reg_lambda': 4.24029007862436, 'min_split_gain': 0.024122212257117505}. Best is trial 3 with value: 0.0022927366072757207.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:31,100] Trial 6 finished with value: 0.0024787058497103107 and parameters: {'n_estimators': 124, 'learning_rate': 0.021626723710672785, 'num_leaves': 183, 'max_depth': 7, 'min_child_samples': 18, 'subsample': 0.8204357846829388, 'colsample_bytree': 0.7730605517923912, 'reg_alpha': 2.8591337138221675, 'reg_lambda': 1.8165012742128106, 'min_split_gain': 0.02421671048960766}. Best is trial 3 with value: 0.0022927366072757207.
[I 2025-06-15 20:13:31,104] A new study created in memory with name: no-name-1b4b746b-1c08-4d99-8ff9-83fcd61f3e1e
[I 2025-06-15 20:13:31,168] Trial 0 finished with value: 0.0435777309995371 and parameters: {'n_estimators': 94, 'learning_rate': 0.10554555614170472, 'num_leaves': 68, 'max_depth': 13, 'min_child_samples': 2, 'subsample': 0.8334049170651512, 'colsample_bytree': 0.8281329124875969, 'reg_alpha': 1.379452398632337, 'reg_lambda': 5.613444329679425, 'min_split_gain': 0.011186559459281577}. Best is trial 0 with value: 0.0435777309995371.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         804
14      pct_chg_Load_DA         479
1           Price_lag_2         425
3         Load_DA_lag_0         230
16  roll2d_median_Price         226
11           NGas_lag_2         224
2           Price_lag_7         217
10           Coal_lag_2         126
13            EUA_lag_2         124
5       WindOn_DA_lag_0         113
15       lag168_Load_DA         113
12            Oil_lag_2          71
9                  WD_7          16
8                  WD_6           8
7                  WD_1           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         114
2           Price_lag_7          78
14      pct_chg_Load_DA          43
5       WindOn_DA_lag_0          42
3         Load_DA_lag_0          36
15       lag168_Load_DA          23
16  roll2d_median_Price          11
1           Price_lag_2           9
10           Coal_lag_2           3
11           NGas_lag_2           1
13            EUA_lag_2           1
7                  WD_1           0
9                  WD_7           0
12            Oil_lag_2           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 51.16% done
********************* END NS.. 373 ****************************************************
********************* START NS ... 374 ****************************************************
[I 2025-06-15 20:13:32,715] A new study created in memory with name: no-name-32ee9f61-0035-4872-8753-56d13117b8c2
[I 2025-06-15 20:13:32,977] Trial 0 finished with value: 0.01128172290986419 and parameters: {'n_estimators': 106, 'learning_rate': 0.010131509051500856, 'num_leaves': 41, 'max_depth': 6, 'min_child_samples': 5, 'subsample': 0.5344550918261624, 'colsample_bytree': 0.8377480134515651, 'reg_alpha': 6.813750669710704, 'reg_lambda': 0.9262660198740758, 'min_split_gain': 0.03196713851484131}. Best is trial 0 with value: 0.01128172290986419.
[I 2025-06-15 20:13:33,131] Trial 1 finished with value: 0.006810517734597757 and parameters: {'n_estimators': 132, 'learning_rate': 0.26131615047477463, 'num_leaves': 65, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.8197695253996158, 'colsample_bytree': 0.6769529714603548, 'reg_alpha': 9.237687598986518, 'reg_lambda': 5.186972783700662, 'min_split_gain': 0.03716072607023253}. Best is trial 1 with value: 0.006810517734597757.
[Trial 0] New best: 0.01128
[Trial 1] New best: 0.00681
[I 2025-06-15 20:13:33,611] Trial 2 finished with value: 0.005800125444181025 and parameters: {'n_estimators': 85, 'learning_rate': 0.03788225162380621, 'num_leaves': 148, 'max_depth': 11, 'min_child_samples': 30, 'subsample': 0.8389979632047356, 'colsample_bytree': 0.797042739254366, 'reg_alpha': 2.705075138234462, 'reg_lambda': 6.216806402879921, 'min_split_gain': 0.03564003329549276}. Best is trial 2 with value: 0.005800125444181025.
[Trial 2] New best: 0.00580
[I 2025-06-15 20:13:34,500] Trial 3 finished with value: 0.007350692997693277 and parameters: {'n_estimators': 191, 'learning_rate': 0.012252784815900385, 'num_leaves': 40, 'max_depth': 15, 'min_child_samples': 11, 'subsample': 0.7245866162307992, 'colsample_bytree': 0.7650098516660846, 'reg_alpha': 5.012093674303052, 'reg_lambda': 1.7547345136356096, 'min_split_gain': 0.002732562717742304}. Best is trial 2 with value: 0.005800125444181025.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:35,049] Trial 4 finished with value: 0.007458518329749041 and parameters: {'n_estimators': 116, 'learning_rate': 0.08762229614696089, 'num_leaves': 230, 'max_depth': 5, 'min_child_samples': 4, 'subsample': 0.6623984477167774, 'colsample_bytree': 0.5885201397238238, 'reg_alpha': 3.230156806723403, 'reg_lambda': 5.194831903921283, 'min_split_gain': 0.021089446644183402}. Best is trial 2 with value: 0.005800125444181025.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:35,619] Trial 5 finished with value: 0.009368409176041044 and parameters: {'n_estimators': 171, 'learning_rate': 0.017006149097038292, 'num_leaves': 160, 'max_depth': 14, 'min_child_samples': 6, 'subsample': 0.6129833307018983, 'colsample_bytree': 0.5238873414222756, 'reg_alpha': 9.85116287437873, 'reg_lambda': 3.212198121845894, 'min_split_gain': 0.006931758101493702}. Best is trial 2 with value: 0.005800125444181025.
[I 2025-06-15 20:13:35,621] A new study created in memory with name: no-name-16c651f3-73d7-48e8-8570-b4f0088b3312
[I 2025-06-15 20:13:35,650] Trial 0 finished with value: 0.011360632682739564 and parameters: {'n_estimators': 130, 'learning_rate': 0.02846874649759005, 'num_leaves': 176, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.6507021936857875, 'colsample_bytree': 0.9241608842157537, 'reg_alpha': 2.1574997914202463, 'reg_lambda': 8.672322644557244, 'min_split_gain': 0.03197793475439313}. Best is trial 0 with value: 0.011360632682739564.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1201
1           Price_lag_2         899
14      pct_chg_Load_DA         578
2           Price_lag_7         250
16  roll2d_median_Price         198
11           NGas_lag_2         173
3         Load_DA_lag_0         161
10           Coal_lag_2          74
12            Oil_lag_2          67
15       lag168_Load_DA          60
13            EUA_lag_2          54
5       WindOn_DA_lag_0          26
9                  WD_7           5
8                  WD_6           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         216
3         Load_DA_lag_0          56
14      pct_chg_Load_DA          19
13            EUA_lag_2          18
16  roll2d_median_Price          16
15       lag168_Load_DA          13
1           Price_lag_2           9
2           Price_lag_7           7
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 51.30% done
********************* END NS.. 374 ****************************************************
********************* START NS ... 375 ****************************************************
[I 2025-06-15 20:13:36,888] A new study created in memory with name: no-name-d8f1786b-f923-4411-ae68-d89514f97c34
[I 2025-06-15 20:13:37,004] Trial 0 finished with value: 0.010302185291582633 and parameters: {'n_estimators': 113, 'learning_rate': 0.13864778972394684, 'num_leaves': 55, 'max_depth': 13, 'min_child_samples': 18, 'subsample': 0.6328943271127419, 'colsample_bytree': 0.6742976925276292, 'reg_alpha': 8.784426568064998, 'reg_lambda': 5.709764556551131, 'min_split_gain': 0.04565541836909423}. Best is trial 0 with value: 0.010302185291582633.
[Trial 0] New best: 0.01030
[I 2025-06-15 20:13:37,213] Trial 1 finished with value: 0.010502319247714997 and parameters: {'n_estimators': 226, 'learning_rate': 0.12956011319850363, 'num_leaves': 34, 'max_depth': 11, 'min_child_samples': 25, 'subsample': 0.5915404664763382, 'colsample_bytree': 0.5044718382363882, 'reg_alpha': 5.6146684854678135, 'reg_lambda': 8.327598388064699, 'min_split_gain': 0.030783389784856337}. Best is trial 0 with value: 0.010302185291582633.
[I 2025-06-15 20:13:37,409] Trial 2 finished with value: 0.011025535843418524 and parameters: {'n_estimators': 107, 'learning_rate': 0.11296536606064556, 'num_leaves': 237, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.8793613091374585, 'colsample_bytree': 0.7587005517839978, 'reg_alpha': 6.702821670301242, 'reg_lambda': 5.040304018410219, 'min_split_gain': 0.022590159062756066}. Best is trial 0 with value: 0.010302185291582633.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:37,926] Trial 3 finished with value: 0.014123645971890503 and parameters: {'n_estimators': 158, 'learning_rate': 0.016708556185209014, 'num_leaves': 128, 'max_depth': 12, 'min_child_samples': 5, 'subsample': 0.5141658946171959, 'colsample_bytree': 0.5636361896786497, 'reg_alpha': 7.87187692803373, 'reg_lambda': 8.905479049872948, 'min_split_gain': 0.04272439781879094}. Best is trial 0 with value: 0.010302185291582633.
[I 2025-06-15 20:13:37,929] A new study created in memory with name: no-name-99ed254b-65c2-4f55-bb10-33bbbd4a247a
[I 2025-06-15 20:13:37,949] Trial 0 finished with value: 0.06377396636738057 and parameters: {'n_estimators': 55, 'learning_rate': 0.1991311914248711, 'num_leaves': 136, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.5546178336212106, 'colsample_bytree': 0.9733314357333258, 'reg_alpha': 9.8325648082447, 'reg_lambda': 6.080294820899046, 'min_split_gain': 0.046680934539940565}. Best is trial 0 with value: 0.06377396636738057.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         313
1           Price_lag_2         172
14      pct_chg_Load_DA         142
3         Load_DA_lag_0         109
11           NGas_lag_2          82
2           Price_lag_7          69
16  roll2d_median_Price          68
13            EUA_lag_2          56
15       lag168_Load_DA          50
10           Coal_lag_2          49
5       WindOn_DA_lag_0          42
12            Oil_lag_2          18
9                  WD_7           9
7                  WD_1           3
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          19
3         Load_DA_lag_0           7
15       lag168_Load_DA           1
5       WindOn_DA_lag_0           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 51.44% done
********************* END NS.. 375 ****************************************************
********************* START NS ... 376 ****************************************************
[I 2025-06-15 20:13:38,862] A new study created in memory with name: no-name-10f2ce94-6f5d-4f8f-ae98-9064de842a29
[I 2025-06-15 20:13:39,201] Trial 0 finished with value: 0.012129094183036529 and parameters: {'n_estimators': 113, 'learning_rate': 0.022007614070407796, 'num_leaves': 72, 'max_depth': 16, 'min_child_samples': 6, 'subsample': 0.8606421096775789, 'colsample_bytree': 0.8397391113583972, 'reg_alpha': 5.655784326130126, 'reg_lambda': 6.609524822232754, 'min_split_gain': 0.008046338627551287}. Best is trial 0 with value: 0.012129094183036529.
[Trial 0] New best: 0.01213
[I 2025-06-15 20:13:39,611] Trial 1 finished with value: 0.010231933080303297 and parameters: {'n_estimators': 300, 'learning_rate': 0.06357271997785946, 'num_leaves': 240, 'max_depth': 7, 'min_child_samples': 20, 'subsample': 0.8073671387746, 'colsample_bytree': 0.8383317848806902, 'reg_alpha': 8.023650282639073, 'reg_lambda': 8.775446886151027, 'min_split_gain': 0.018270166491629627}. Best is trial 1 with value: 0.010231933080303297.
[Trial 1] New best: 0.01023
[I 2025-06-15 20:13:39,977] Trial 2 finished with value: 0.010394697538105303 and parameters: {'n_estimators': 279, 'learning_rate': 0.13671279863166397, 'num_leaves': 166, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.7434761639229472, 'colsample_bytree': 0.8567342150224719, 'reg_alpha': 8.429609371638549, 'reg_lambda': 6.216821158888408, 'min_split_gain': 0.006264935690012164}. Best is trial 1 with value: 0.010231933080303297.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:40,451] Trial 3 finished with value: 0.010919653255945022 and parameters: {'n_estimators': 265, 'learning_rate': 0.09243444599744848, 'num_leaves': 72, 'max_depth': 5, 'min_child_samples': 25, 'subsample': 0.9946183023179787, 'colsample_bytree': 0.5811124043692915, 'reg_alpha': 1.8472009366656805, 'reg_lambda': 6.817530676062004, 'min_split_gain': 0.04379928734010763}. Best is trial 1 with value: 0.010231933080303297.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:41,775] Trial 4 finished with value: 0.009944277476856602 and parameters: {'n_estimators': 281, 'learning_rate': 0.0281668673560015, 'num_leaves': 110, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.620579528060331, 'colsample_bytree': 0.7629111824132285, 'reg_alpha': 6.3867188615651385, 'reg_lambda': 8.976182946803148, 'min_split_gain': 0.0018140462128747427}. Best is trial 4 with value: 0.009944277476856602.
[I 2025-06-15 20:13:41,779] A new study created in memory with name: no-name-46a82c9d-8385-4507-a92b-d9dbafdde06b
[I 2025-06-15 20:13:41,844] Trial 0 finished with value: 0.013330095534016014 and parameters: {'n_estimators': 285, 'learning_rate': 0.16447450749637274, 'num_leaves': 240, 'max_depth': 7, 'min_child_samples': 6, 'subsample': 0.9333280908930734, 'colsample_bytree': 0.7893083242461034, 'reg_alpha': 1.6341983760764944, 'reg_lambda': 8.886272954155858, 'min_split_gain': 0.045574316641220275}. Best is trial 0 with value: 0.013330095534016014.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2558
1           Price_lag_2        2040
14      pct_chg_Load_DA        1824
3         Load_DA_lag_0        1072
13            EUA_lag_2         951
2           Price_lag_7         922
5       WindOn_DA_lag_0         858
15       lag168_Load_DA         828
11           NGas_lag_2         774
16  roll2d_median_Price         701
12            Oil_lag_2         622
10           Coal_lag_2         542
9                  WD_7          70
8                  WD_6          56
7                  WD_1          39
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          41
14      pct_chg_Load_DA          20
3         Load_DA_lag_0          19
2           Price_lag_7          15
15       lag168_Load_DA          15
16  roll2d_median_Price           6
1           Price_lag_2           3
5       WindOn_DA_lag_0           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 51.57% done
********************* END NS.. 376 ****************************************************
********************* START NS ... 377 ****************************************************
[I 2025-06-15 20:13:44,046] A new study created in memory with name: no-name-9f6b62ae-5f78-471a-8719-835b95ddb7bb
[I 2025-06-15 20:13:44,415] Trial 0 finished with value: 0.0082328545997479 and parameters: {'n_estimators': 225, 'learning_rate': 0.18938134157059455, 'num_leaves': 56, 'max_depth': 11, 'min_child_samples': 30, 'subsample': 0.812373493429577, 'colsample_bytree': 0.5960151101355613, 'reg_alpha': 6.486930369041868, 'reg_lambda': 8.85604580126057, 'min_split_gain': 0.02230396075077412}. Best is trial 0 with value: 0.0082328545997479.
[Trial 0] New best: 0.00823
[I 2025-06-15 20:13:44,733] Trial 1 finished with value: 0.008280646051201936 and parameters: {'n_estimators': 154, 'learning_rate': 0.2137526283446405, 'num_leaves': 55, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.5644189437894762, 'colsample_bytree': 0.9154375938312418, 'reg_alpha': 7.36221736385703, 'reg_lambda': 6.6884721259357685, 'min_split_gain': 0.047601016016755204}. Best is trial 0 with value: 0.0082328545997479.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:45,233] Trial 2 finished with value: 0.006781963583268005 and parameters: {'n_estimators': 129, 'learning_rate': 0.10782634970017388, 'num_leaves': 57, 'max_depth': 9, 'min_child_samples': 16, 'subsample': 0.8200288659554791, 'colsample_bytree': 0.8144505319411455, 'reg_alpha': 2.8066516298147004, 'reg_lambda': 7.385260399295918, 'min_split_gain': 0.024198369060847654}. Best is trial 2 with value: 0.006781963583268005.
[Trial 2] New best: 0.00678
[I 2025-06-15 20:13:45,661] Trial 3 finished with value: 0.007397086517854558 and parameters: {'n_estimators': 239, 'learning_rate': 0.11300304085222983, 'num_leaves': 71, 'max_depth': 7, 'min_child_samples': 4, 'subsample': 0.9378927295275497, 'colsample_bytree': 0.8404998286714305, 'reg_alpha': 4.196862170794283, 'reg_lambda': 1.3937324232320603, 'min_split_gain': 0.03907429654864966}. Best is trial 2 with value: 0.006781963583268005.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:46,724] Trial 4 finished with value: 0.010636393707641306 and parameters: {'n_estimators': 243, 'learning_rate': 0.015155961705979049, 'num_leaves': 60, 'max_depth': 10, 'min_child_samples': 23, 'subsample': 0.849963010394751, 'colsample_bytree': 0.6090580580793667, 'reg_alpha': 7.107633714870109, 'reg_lambda': 7.610874893917876, 'min_split_gain': 0.04953364499329315}. Best is trial 2 with value: 0.006781963583268005.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:47,589] Trial 5 finished with value: 0.011748660637409541 and parameters: {'n_estimators': 151, 'learning_rate': 0.011811138131270823, 'num_leaves': 224, 'max_depth': 6, 'min_child_samples': 9, 'subsample': 0.9096981674890013, 'colsample_bytree': 0.7408524156121163, 'reg_alpha': 0.8304821528553541, 'reg_lambda': 4.000983026789097, 'min_split_gain': 0.04006159664384075}. Best is trial 2 with value: 0.006781963583268005.
[I 2025-06-15 20:13:47,592] A new study created in memory with name: no-name-c6904c4c-27bb-49c1-9d55-e7af2033208d
[I 2025-06-15 20:13:47,681] Trial 0 finished with value: 0.01997162145127876 and parameters: {'n_estimators': 274, 'learning_rate': 0.014231236238541672, 'num_leaves': 215, 'max_depth': 15, 'min_child_samples': 1, 'subsample': 0.50812721376678, 'colsample_bytree': 0.5386720469007791, 'reg_alpha': 1.3201973236546094, 'reg_lambda': 3.9807895006860794, 'min_split_gain': 0.04334394519845715}. Best is trial 0 with value: 0.01997162145127876.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         636
1           Price_lag_2         554
14      pct_chg_Load_DA         478
3         Load_DA_lag_0         362
2           Price_lag_7         238
15       lag168_Load_DA         229
5       WindOn_DA_lag_0         229
16  roll2d_median_Price         216
11           NGas_lag_2         205
10           Coal_lag_2         127
13            EUA_lag_2         119
12            Oil_lag_2         100
9                  WD_7          13
7                  WD_1          10
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         201
15       lag168_Load_DA          68
3         Load_DA_lag_0          68
1           Price_lag_2          20
10           Coal_lag_2           6
11           NGas_lag_2           4
12            Oil_lag_2           2
16  roll2d_median_Price           2
13            EUA_lag_2           1
7                  WD_1           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 51.71% done
********************* END NS.. 377 ****************************************************
********************* START NS ... 378 ****************************************************
[I 2025-06-15 20:13:48,893] A new study created in memory with name: no-name-729639c7-2297-4269-b043-eb8362de9d00
[I 2025-06-15 20:13:49,057] Trial 0 finished with value: 0.006200661828373513 and parameters: {'n_estimators': 138, 'learning_rate': 0.020068707719568105, 'num_leaves': 161, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.6679673480526431, 'colsample_bytree': 0.7285250304642744, 'reg_alpha': 0.8844670241702202, 'reg_lambda': 1.5115804450165604, 'min_split_gain': 0.004042944923926018}. Best is trial 0 with value: 0.006200661828373513.
[Trial 0] New best: 0.00620
[I 2025-06-15 20:13:49,728] Trial 1 finished with value: 0.003857173905717709 and parameters: {'n_estimators': 199, 'learning_rate': 0.02038183268696855, 'num_leaves': 91, 'max_depth': 8, 'min_child_samples': 14, 'subsample': 0.8516192874807165, 'colsample_bytree': 0.6594196579118357, 'reg_alpha': 4.1997646854168655, 'reg_lambda': 6.439797675898004, 'min_split_gain': 0.009263269312025679}. Best is trial 1 with value: 0.003857173905717709.
[Trial 1] New best: 0.00386
[I 2025-06-15 20:13:50,125] Trial 2 finished with value: 0.01011818589684744 and parameters: {'n_estimators': 117, 'learning_rate': 0.010363971423221742, 'num_leaves': 141, 'max_depth': 6, 'min_child_samples': 8, 'subsample': 0.7628641957480549, 'colsample_bytree': 0.9927107997655853, 'reg_alpha': 3.4162616935144774, 'reg_lambda': 6.536725754187417, 'min_split_gain': 0.040095890435095045}. Best is trial 1 with value: 0.003857173905717709.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:50,555] Trial 3 finished with value: 0.013443451015620889 and parameters: {'n_estimators': 116, 'learning_rate': 0.012527018448237223, 'num_leaves': 228, 'max_depth': 14, 'min_child_samples': 14, 'subsample': 0.7734739255817247, 'colsample_bytree': 0.6249948869095052, 'reg_alpha': 8.330183754647534, 'reg_lambda': 3.2676697925201026, 'min_split_gain': 0.0029151270616392224}. Best is trial 1 with value: 0.003857173905717709.
[I 2025-06-15 20:13:50,725] Trial 4 finished with value: 0.0035419270492081053 and parameters: {'n_estimators': 89, 'learning_rate': 0.191768832077027, 'num_leaves': 192, 'max_depth': 16, 'min_child_samples': 5, 'subsample': 0.9281299474332945, 'colsample_bytree': 0.7606489158284989, 'reg_alpha': 7.869463234788391, 'reg_lambda': 9.495640123400262, 'min_split_gain': 0.03086198117496693}. Best is trial 4 with value: 0.0035419270492081053.
[I 2025-06-15 20:13:50,728] A new study created in memory with name: no-name-059f6584-1fb4-4efc-be27-c8746a330eb1
[I 2025-06-15 20:13:50,751] Trial 0 finished with value: 0.06633707520722565 and parameters: {'n_estimators': 67, 'learning_rate': 0.09072396555489855, 'num_leaves': 127, 'max_depth': 9, 'min_child_samples': 3, 'subsample': 0.6736321003244928, 'colsample_bytree': 0.7631946790904134, 'reg_alpha': 7.5747192310586655, 'reg_lambda': 0.17118034244311353, 'min_split_gain': 0.04802322542899903}. Best is trial 0 with value: 0.06633707520722565.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         257
1           Price_lag_2         169
14      pct_chg_Load_DA         160
3         Load_DA_lag_0         100
11           NGas_lag_2          93
2           Price_lag_7          89
16  roll2d_median_Price          81
15       lag168_Load_DA          64
10           Coal_lag_2          53
13            EUA_lag_2          52
5       WindOn_DA_lag_0          50
12            Oil_lag_2          26
9                  WD_7           7
8                  WD_6           7
7                  WD_1           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          17
3         Load_DA_lag_0           1
1           Price_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 51.85% done
********************* END NS.. 378 ****************************************************
********************* START NS ... 379 ****************************************************
[I 2025-06-15 20:13:52,072] A new study created in memory with name: no-name-96a331e5-e36a-4362-b996-21f906e5f625
[I 2025-06-15 20:13:52,594] Trial 0 finished with value: 0.002133507962274203 and parameters: {'n_estimators': 195, 'learning_rate': 0.020190616529409112, 'num_leaves': 230, 'max_depth': 8, 'min_child_samples': 29, 'subsample': 0.9312074235039453, 'colsample_bytree': 0.6058295611172043, 'reg_alpha': 5.026783481261519, 'reg_lambda': 1.5220663293188896, 'min_split_gain': 0.020324289816320635}. Best is trial 0 with value: 0.002133507962274203.
[Trial 0] New best: 0.00213
[I 2025-06-15 20:13:53,111] Trial 1 finished with value: 0.0009397809224809036 and parameters: {'n_estimators': 166, 'learning_rate': 0.056365573050380396, 'num_leaves': 35, 'max_depth': 8, 'min_child_samples': 14, 'subsample': 0.9781762703443151, 'colsample_bytree': 0.8829808561798596, 'reg_alpha': 2.581325497699706, 'reg_lambda': 4.952552759273965, 'min_split_gain': 0.014555643155752807}. Best is trial 1 with value: 0.0009397809224809036.
[Trial 1] New best: 0.00094
[I 2025-06-15 20:13:53,644] Trial 2 finished with value: 0.0018771251804017917 and parameters: {'n_estimators': 221, 'learning_rate': 0.01852481170644854, 'num_leaves': 89, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.6229163971866534, 'colsample_bytree': 0.5386656428175558, 'reg_alpha': 5.197239736359784, 'reg_lambda': 3.5006095949335867, 'min_split_gain': 0.04240216727341845}. Best is trial 1 with value: 0.0009397809224809036.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:54,486] Trial 3 finished with value: 0.0021560806958333526 and parameters: {'n_estimators': 136, 'learning_rate': 0.02327684659724411, 'num_leaves': 82, 'max_depth': 13, 'min_child_samples': 16, 'subsample': 0.5260017962372456, 'colsample_bytree': 0.9737987304987132, 'reg_alpha': 0.2905133030451823, 'reg_lambda': 4.339471751382912, 'min_split_gain': 0.013808445035251755}. Best is trial 1 with value: 0.0009397809224809036.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:55,024] Trial 4 finished with value: 0.0014271281547219106 and parameters: {'n_estimators': 121, 'learning_rate': 0.06104238739236781, 'num_leaves': 142, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.6037683538782166, 'colsample_bytree': 0.8951466782848214, 'reg_alpha': 1.0583829993855176, 'reg_lambda': 1.7087039406472149, 'min_split_gain': 0.04338550988911391}. Best is trial 1 with value: 0.0009397809224809036.
[I 2025-06-15 20:13:55,026] A new study created in memory with name: no-name-7c2c9ff4-e73d-490b-9bda-8b009411c616
[I 2025-06-15 20:13:55,044] Trial 0 finished with value: 0.09041410410737434 and parameters: {'n_estimators': 59, 'learning_rate': 0.026033583874479065, 'num_leaves': 198, 'max_depth': 9, 'min_child_samples': 23, 'subsample': 0.5475838091594214, 'colsample_bytree': 0.5616276332188291, 'reg_alpha': 6.089818753645849, 'reg_lambda': 5.123380384261635, 'min_split_gain': 0.02795992154370204}. Best is trial 0 with value: 0.09041410410737434.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1118
14      pct_chg_Load_DA         755
1           Price_lag_2         738
3         Load_DA_lag_0         579
11           NGas_lag_2         376
5       WindOn_DA_lag_0         360
2           Price_lag_7         328
15       lag168_Load_DA         312
16  roll2d_median_Price         310
13            EUA_lag_2         284
10           Coal_lag_2         195
12            Oil_lag_2         174
9                  WD_7          29
7                  WD_1          26
8                  WD_6           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          47
3         Load_DA_lag_0          21
1           Price_lag_2           9
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 51.98% done
********************* END NS.. 379 ****************************************************
********************* START NS ... 380 ****************************************************
[I 2025-06-15 20:13:56,432] A new study created in memory with name: no-name-a5bb83e2-99dc-42df-8c59-0ac97bc228d5
[I 2025-06-15 20:13:56,586] Trial 0 finished with value: 0.0013494856588320808 and parameters: {'n_estimators': 140, 'learning_rate': 0.12986838036016873, 'num_leaves': 183, 'max_depth': 6, 'min_child_samples': 5, 'subsample': 0.773970178607401, 'colsample_bytree': 0.9775631433644227, 'reg_alpha': 5.700074923662023, 'reg_lambda': 9.854894306401071, 'min_split_gain': 0.03963299619631518}. Best is trial 0 with value: 0.0013494856588320808.
[Trial 0] New best: 0.00135
[I 2025-06-15 20:13:57,237] Trial 1 finished with value: 0.002029009452412255 and parameters: {'n_estimators': 227, 'learning_rate': 0.013296451812142033, 'num_leaves': 116, 'max_depth': 16, 'min_child_samples': 30, 'subsample': 0.6556059316445553, 'colsample_bytree': 0.6797809457339679, 'reg_alpha': 7.454467466808911, 'reg_lambda': 1.0962931523280628, 'min_split_gain': 0.01952614474910787}. Best is trial 0 with value: 0.0013494856588320808.
[I 2025-06-15 20:13:57,380] Trial 2 finished with value: 0.001606821384309378 and parameters: {'n_estimators': 161, 'learning_rate': 0.283535448491672, 'num_leaves': 53, 'max_depth': 6, 'min_child_samples': 23, 'subsample': 0.7532347108288804, 'colsample_bytree': 0.8606671454522357, 'reg_alpha': 9.098711805326383, 'reg_lambda': 3.4055816899872338, 'min_split_gain': 0.04249060174269669}. Best is trial 0 with value: 0.0013494856588320808.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:13:57,726] Trial 3 finished with value: 0.0027179083049540544 and parameters: {'n_estimators': 90, 'learning_rate': 0.029383371097525524, 'num_leaves': 195, 'max_depth': 9, 'min_child_samples': 2, 'subsample': 0.948216525414627, 'colsample_bytree': 0.5701509151314375, 'reg_alpha': 9.579699726539163, 'reg_lambda': 1.9446457642502435, 'min_split_gain': 0.004645554003920783}. Best is trial 0 with value: 0.0013494856588320808.
[I 2025-06-15 20:13:57,729] A new study created in memory with name: no-name-94f85ec1-50a0-4fd4-8125-532a6cb96ad3
[I 2025-06-15 20:13:57,761] Trial 0 finished with value: 0.043094928515499474 and parameters: {'n_estimators': 216, 'learning_rate': 0.295704415855542, 'num_leaves': 214, 'max_depth': 9, 'min_child_samples': 29, 'subsample': 0.8492137409971762, 'colsample_bytree': 0.536640564976331, 'reg_alpha': 9.788037635621093, 'reg_lambda': 1.6717671653315525, 'min_split_gain': 0.0012294444731641098}. Best is trial 0 with value: 0.043094928515499474.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         398
14      pct_chg_Load_DA         259
1           Price_lag_2         189
3         Load_DA_lag_0         130
16  roll2d_median_Price         115
11           NGas_lag_2         112
2           Price_lag_7         108
5       WindOn_DA_lag_0          80
15       lag168_Load_DA          70
10           Coal_lag_2          69
13            EUA_lag_2          48
12            Oil_lag_2          29
9                  WD_7           7
8                  WD_6           5
7                  WD_1           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          10
3         Load_DA_lag_0           1
1           Price_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 52.12% done
********************* END NS.. 380 ****************************************************
********************* START NS ... 381 ****************************************************
[I 2025-06-15 20:13:59,051] A new study created in memory with name: no-name-08748728-747f-4a24-96b7-0464bb9e815c
[I 2025-06-15 20:13:59,209] Trial 0 finished with value: 0.04847117492546102 and parameters: {'n_estimators': 109, 'learning_rate': 0.029479541510892406, 'num_leaves': 59, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.7553841254547957, 'colsample_bytree': 0.9212323130526809, 'reg_alpha': 4.646218022054828, 'reg_lambda': 3.4980168968579637, 'min_split_gain': 0.04330549521246162}. Best is trial 0 with value: 0.04847117492546102.
[Trial 0] New best: 0.04847
[I 2025-06-15 20:13:59,595] Trial 1 finished with value: 0.05958026961849161 and parameters: {'n_estimators': 165, 'learning_rate': 0.028755921604711573, 'num_leaves': 95, 'max_depth': 6, 'min_child_samples': 6, 'subsample': 0.8760336587332471, 'colsample_bytree': 0.5314745712699789, 'reg_alpha': 2.6652741012340506, 'reg_lambda': 3.885660750659259, 'min_split_gain': 0.030561345691681924}. Best is trial 0 with value: 0.04847117492546102.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:13:59,939] Trial 2 finished with value: 0.06171530287954157 and parameters: {'n_estimators': 151, 'learning_rate': 0.07209876368683309, 'num_leaves': 139, 'max_depth': 15, 'min_child_samples': 4, 'subsample': 0.5420715838341991, 'colsample_bytree': 0.650736197014447, 'reg_alpha': 9.350123363339351, 'reg_lambda': 8.820070632664939, 'min_split_gain': 0.04341502910391276}. Best is trial 0 with value: 0.04847117492546102.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:00,473] Trial 3 finished with value: 0.04224027991966447 and parameters: {'n_estimators': 296, 'learning_rate': 0.09935233713177706, 'num_leaves': 115, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.9556074940829561, 'colsample_bytree': 0.6101504366953399, 'reg_alpha': 8.056664000315681, 'reg_lambda': 4.486499149680512, 'min_split_gain': 0.013483823899741999}. Best is trial 3 with value: 0.04224027991966447.
[Trial 3] New best: 0.04224
[I 2025-06-15 20:14:01,001] Trial 4 finished with value: 0.038681002445380475 and parameters: {'n_estimators': 156, 'learning_rate': 0.060265499378002935, 'num_leaves': 162, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.5356796365989778, 'colsample_bytree': 0.5309898549096934, 'reg_alpha': 2.212638724549244, 'reg_lambda': 7.485239545057213, 'min_split_gain': 0.03097583635918788}. Best is trial 4 with value: 0.038681002445380475.
[Trial 4] New best: 0.03868
[I 2025-06-15 20:14:01,343] Trial 5 finished with value: 0.049196997950796954 and parameters: {'n_estimators': 67, 'learning_rate': 0.07675161917444949, 'num_leaves': 68, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.8282325781687505, 'colsample_bytree': 0.6368857634673976, 'reg_alpha': 2.930644613187355, 'reg_lambda': 6.602459699160722, 'min_split_gain': 0.005857220598879698}. Best is trial 4 with value: 0.038681002445380475.
[I 2025-06-15 20:14:01,548] Trial 6 finished with value: 0.057007501772739916 and parameters: {'n_estimators': 281, 'learning_rate': 0.2811046609222132, 'num_leaves': 80, 'max_depth': 12, 'min_child_samples': 10, 'subsample': 0.9640226383538084, 'colsample_bytree': 0.8929428900189007, 'reg_alpha': 6.85024584705871, 'reg_lambda': 8.86661228568483, 'min_split_gain': 0.020075333831450853}. Best is trial 4 with value: 0.038681002445380475.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:01,798] Trial 7 finished with value: 0.026398658471382713 and parameters: {'n_estimators': 91, 'learning_rate': 0.203080691752923, 'num_leaves': 88, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.8973011781510492, 'colsample_bytree': 0.6835085648492844, 'reg_alpha': 1.8570096949791859, 'reg_lambda': 7.41383986473648, 'min_split_gain': 0.012279282626442317}. Best is trial 7 with value: 0.026398658471382713.
[Trial 7] New best: 0.02640
[I 2025-06-15 20:14:02,531] Trial 8 finished with value: 0.06352223614143346 and parameters: {'n_estimators': 219, 'learning_rate': 0.010733327753142369, 'num_leaves': 42, 'max_depth': 9, 'min_child_samples': 14, 'subsample': 0.5028754301288912, 'colsample_bytree': 0.9342824956528718, 'reg_alpha': 8.605041401565472, 'reg_lambda': 5.035244697347081, 'min_split_gain': 0.001651229166555618}. Best is trial 7 with value: 0.026398658471382713.
[I 2025-06-15 20:14:02,738] Trial 9 finished with value: 0.041378124377855416 and parameters: {'n_estimators': 173, 'learning_rate': 0.17766659228635379, 'num_leaves': 75, 'max_depth': 4, 'min_child_samples': 9, 'subsample': 0.7870429853246859, 'colsample_bytree': 0.6521709602944207, 'reg_alpha': 7.136975833004229, 'reg_lambda': 5.006333270967426, 'min_split_gain': 0.004536007176222301}. Best is trial 7 with value: 0.026398658471382713.
[Trial 8] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 9] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:03,089] Trial 10 finished with value: 0.02708367625971535 and parameters: {'n_estimators': 63, 'learning_rate': 0.16192277849196168, 'num_leaves': 250, 'max_depth': 8, 'min_child_samples': 24, 'subsample': 0.6633368501981165, 'colsample_bytree': 0.773560957854875, 'reg_alpha': 0.18590338576748167, 'reg_lambda': 0.16130827779882217, 'min_split_gain': 0.016837671408807816}. Best is trial 7 with value: 0.026398658471382713.
[I 2025-06-15 20:14:03,091] A new study created in memory with name: no-name-a0397946-d4bd-4628-906b-732abb9afdb0
[I 2025-06-15 20:14:03,121] Trial 0 finished with value: 0.28977317687260074 and parameters: {'n_estimators': 213, 'learning_rate': 0.11150221480769827, 'num_leaves': 188, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.5926130789337329, 'colsample_bytree': 0.9196214032003593, 'reg_alpha': 5.41069602327628, 'reg_lambda': 9.097903802500063, 'min_split_gain': 0.0004657499339444482}. Best is trial 0 with value: 0.28977317687260074.
[Trial 10] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         441
1           Price_lag_2         342
3         Load_DA_lag_0         278
14      pct_chg_Load_DA         228
2           Price_lag_7         215
5       WindOn_DA_lag_0         153
16  roll2d_median_Price         151
11           NGas_lag_2         143
15       lag168_Load_DA         129
13            EUA_lag_2         128
10           Coal_lag_2         121
12            Oil_lag_2          65
9                  WD_7          13
7                  WD_1           7
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          58
3         Load_DA_lag_0          37
10           Coal_lag_2          19
12            Oil_lag_2           5
15       lag168_Load_DA           4
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 52.26% done
********************* END NS.. 381 ****************************************************
********************* START NS ... 382 ****************************************************
[I 2025-06-15 20:14:04,251] A new study created in memory with name: no-name-0044e482-85fb-4e7c-b941-43feeb46ff00
[I 2025-06-15 20:14:04,691] Trial 0 finished with value: 0.06352674245261504 and parameters: {'n_estimators': 199, 'learning_rate': 0.010980992012217349, 'num_leaves': 140, 'max_depth': 16, 'min_child_samples': 25, 'subsample': 0.5475653259230646, 'colsample_bytree': 0.9909483476261299, 'reg_alpha': 8.09212901103992, 'reg_lambda': 6.751017238365261, 'min_split_gain': 0.04984184520097057}. Best is trial 0 with value: 0.06352674245261504.
[Trial 0] New best: 0.06353
[I 2025-06-15 20:14:05,433] Trial 1 finished with value: 0.037343831403283116 and parameters: {'n_estimators': 289, 'learning_rate': 0.04413872694423209, 'num_leaves': 63, 'max_depth': 16, 'min_child_samples': 27, 'subsample': 0.6480102025081936, 'colsample_bytree': 0.5638216544948749, 'reg_alpha': 2.4712541339663083, 'reg_lambda': 8.41533766617432, 'min_split_gain': 0.02883694033099873}. Best is trial 1 with value: 0.037343831403283116.
[Trial 1] New best: 0.03734
[I 2025-06-15 20:14:05,738] Trial 2 finished with value: 0.055987170784948574 and parameters: {'n_estimators': 64, 'learning_rate': 0.04331417281381753, 'num_leaves': 68, 'max_depth': 8, 'min_child_samples': 30, 'subsample': 0.5270993009085696, 'colsample_bytree': 0.6662670374594977, 'reg_alpha': 1.0889485930057718, 'reg_lambda': 8.34151781339778, 'min_split_gain': 0.011752959845497785}. Best is trial 1 with value: 0.037343831403283116.
[I 2025-06-15 20:14:05,938] Trial 3 finished with value: 0.046880783154638 and parameters: {'n_estimators': 158, 'learning_rate': 0.14481875572558567, 'num_leaves': 215, 'max_depth': 5, 'min_child_samples': 16, 'subsample': 0.7020984257132001, 'colsample_bytree': 0.7704184433763858, 'reg_alpha': 4.4585696036552145, 'reg_lambda': 4.417774312955175, 'min_split_gain': 0.04768025112167129}. Best is trial 1 with value: 0.037343831403283116.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:06,255] Trial 4 finished with value: 0.08463461988489979 and parameters: {'n_estimators': 86, 'learning_rate': 0.017478706017966937, 'num_leaves': 242, 'max_depth': 13, 'min_child_samples': 20, 'subsample': 0.5604554483266291, 'colsample_bytree': 0.8020481015969712, 'reg_alpha': 7.9625359884515765, 'reg_lambda': 4.958239686965422, 'min_split_gain': 0.008319175326411538}. Best is trial 1 with value: 0.037343831403283116.
[I 2025-06-15 20:14:06,257] A new study created in memory with name: no-name-1443e7a5-b402-4925-bd97-f9065dd4bc8d
[I 2025-06-15 20:14:06,283] Trial 0 finished with value: 0.35464103229504573 and parameters: {'n_estimators': 104, 'learning_rate': 0.017406722990811783, 'num_leaves': 150, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.5763313734804886, 'colsample_bytree': 0.6748178919519183, 'reg_alpha': 5.88283641291489, 'reg_lambda': 5.654922178172032, 'min_split_gain': 0.042304805693578335}. Best is trial 0 with value: 0.35464103229504573.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1644
1           Price_lag_2        1527
14      pct_chg_Load_DA         927
3         Load_DA_lag_0         764
2           Price_lag_7         658
5       WindOn_DA_lag_0         561
11           NGas_lag_2         554
16  roll2d_median_Price         542
15       lag168_Load_DA         468
13            EUA_lag_2         400
10           Coal_lag_2         358
12            Oil_lag_2         244
9                  WD_7          35
7                  WD_1          31
8                  WD_6          18
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         148
3         Load_DA_lag_0          28
1           Price_lag_2          22
15       lag168_Load_DA           6
10           Coal_lag_2           1
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 52.39% done
********************* END NS.. 382 ****************************************************
********************* START NS ... 383 ****************************************************
[I 2025-06-15 20:14:07,851] A new study created in memory with name: no-name-09c4ca37-2d5d-418b-b64d-b6609d016871
[I 2025-06-15 20:14:08,262] Trial 0 finished with value: 0.0050116287875910425 and parameters: {'n_estimators': 158, 'learning_rate': 0.01923069635338084, 'num_leaves': 178, 'max_depth': 10, 'min_child_samples': 16, 'subsample': 0.7174538295134572, 'colsample_bytree': 0.7987001068839052, 'reg_alpha': 7.422786231937971, 'reg_lambda': 0.3468524773392756, 'min_split_gain': 0.016998241498758043}. Best is trial 0 with value: 0.0050116287875910425.
[Trial 0] New best: 0.00501
[I 2025-06-15 20:14:08,590] Trial 1 finished with value: 0.00780059901598901 and parameters: {'n_estimators': 122, 'learning_rate': 0.015937944147209487, 'num_leaves': 253, 'max_depth': 6, 'min_child_samples': 8, 'subsample': 0.5578791725015723, 'colsample_bytree': 0.7221585843112662, 'reg_alpha': 8.657676819521757, 'reg_lambda': 7.3706745424504785, 'min_split_gain': 0.03425788826367567}. Best is trial 0 with value: 0.0050116287875910425.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:14:09,071] Trial 2 finished with value: 0.006417882010966259 and parameters: {'n_estimators': 143, 'learning_rate': 0.09053597848003249, 'num_leaves': 103, 'max_depth': 16, 'min_child_samples': 19, 'subsample': 0.7010953086513221, 'colsample_bytree': 0.5252560029093856, 'reg_alpha': 5.555107064156641, 'reg_lambda': 6.915824866347856, 'min_split_gain': 0.0453132747276109}. Best is trial 0 with value: 0.0050116287875910425.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:09,566] Trial 3 finished with value: 0.007610565572615388 and parameters: {'n_estimators': 142, 'learning_rate': 0.01083360429448814, 'num_leaves': 252, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.8905716281959767, 'colsample_bytree': 0.8771566236245074, 'reg_alpha': 5.543575168257817, 'reg_lambda': 7.773035893288376, 'min_split_gain': 0.043317048727203655}. Best is trial 0 with value: 0.0050116287875910425.
[I 2025-06-15 20:14:09,569] A new study created in memory with name: no-name-18c15313-e4e5-433a-93e7-254a489829d2
[I 2025-06-15 20:14:09,599] Trial 0 finished with value: 0.020066040324153298 and parameters: {'n_estimators': 60, 'learning_rate': 0.0188598748368821, 'num_leaves': 185, 'max_depth': 8, 'min_child_samples': 28, 'subsample': 0.674254534975365, 'colsample_bytree': 0.5215993137515462, 'reg_alpha': 1.0327690748730223, 'reg_lambda': 1.2230502748641003, 'min_split_gain': 0.04464135193703388}. Best is trial 0 with value: 0.020066040324153298.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1888
1           Price_lag_2        1270
14      pct_chg_Load_DA         720
2           Price_lag_7         351
16  roll2d_median_Price         350
3         Load_DA_lag_0         219
11           NGas_lag_2         206
10           Coal_lag_2         120
13            EUA_lag_2          93
12            Oil_lag_2          92
15       lag168_Load_DA          72
5       WindOn_DA_lag_0          45
9                  WD_7           5
8                  WD_6           3
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          83
3         Load_DA_lag_0          22
15       lag168_Load_DA           8
1           Price_lag_2           7
14      pct_chg_Load_DA           4
2           Price_lag_7           2
12            Oil_lag_2           2
13            EUA_lag_2           1
16  roll2d_median_Price           1
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 52.53% done
********************* END NS.. 383 ****************************************************
********************* START NS ... 384 ****************************************************
[I 2025-06-15 20:14:11,135] A new study created in memory with name: no-name-d8be045c-abbb-4552-9d1f-ee413a68f2c7
[I 2025-06-15 20:14:11,498] Trial 0 finished with value: 0.003158824362237868 and parameters: {'n_estimators': 82, 'learning_rate': 0.09746258081776953, 'num_leaves': 228, 'max_depth': 15, 'min_child_samples': 4, 'subsample': 0.598623697529884, 'colsample_bytree': 0.889826542603877, 'reg_alpha': 1.1411459160807735, 'reg_lambda': 7.6614267203591, 'min_split_gain': 0.02892506360020146}. Best is trial 0 with value: 0.003158824362237868.
[Trial 0] New best: 0.00316
[I 2025-06-15 20:14:11,730] Trial 1 finished with value: 0.004379962903403835 and parameters: {'n_estimators': 80, 'learning_rate': 0.07690032070944637, 'num_leaves': 58, 'max_depth': 9, 'min_child_samples': 7, 'subsample': 0.5980823596687634, 'colsample_bytree': 0.7510859085353034, 'reg_alpha': 5.317374911272652, 'reg_lambda': 8.793556763149416, 'min_split_gain': 0.044251766906192876}. Best is trial 0 with value: 0.003158824362237868.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:14:11,980] Trial 2 finished with value: 0.0039140890244440145 and parameters: {'n_estimators': 191, 'learning_rate': 0.11450497756063634, 'num_leaves': 77, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.5857474583984521, 'colsample_bytree': 0.9041201183006218, 'reg_alpha': 4.5657461466871965, 'reg_lambda': 4.294792408837842, 'min_split_gain': 0.03472411752960835}. Best is trial 0 with value: 0.003158824362237868.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:12,936] Trial 3 finished with value: 0.006312744212716517 and parameters: {'n_estimators': 274, 'learning_rate': 0.017578854140914146, 'num_leaves': 104, 'max_depth': 14, 'min_child_samples': 28, 'subsample': 0.8724239195034458, 'colsample_bytree': 0.5225857636813551, 'reg_alpha': 7.187432943458893, 'reg_lambda': 7.435590109306278, 'min_split_gain': 0.03132738064925482}. Best is trial 0 with value: 0.003158824362237868.
[I 2025-06-15 20:14:12,938] A new study created in memory with name: no-name-72c887d6-92ba-4169-8a0e-383dd7a17fb2
[I 2025-06-15 20:14:12,999] Trial 0 finished with value: 0.025291622448797832 and parameters: {'n_estimators': 242, 'learning_rate': 0.017562491107104346, 'num_leaves': 47, 'max_depth': 16, 'min_child_samples': 12, 'subsample': 0.5376249216717621, 'colsample_bytree': 0.6218189157360424, 'reg_alpha': 9.708203341195285, 'reg_lambda': 7.095075534573413, 'min_split_gain': 0.040631648629149184}. Best is trial 0 with value: 0.025291622448797832.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA         962
1           Price_lag_2         946
0           Price_lag_1         889
3         Load_DA_lag_0         663
2           Price_lag_7         541
5       WindOn_DA_lag_0         536
15       lag168_Load_DA         479
16  roll2d_median_Price         479
11           NGas_lag_2         410
10           Coal_lag_2         288
12            Oil_lag_2         249
13            EUA_lag_2         207
9                  WD_7          21
7                  WD_1          15
8                  WD_6          10
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         162
3         Load_DA_lag_0          43
12            Oil_lag_2          12
1           Price_lag_2           9
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 52.67% done
********************* END NS.. 384 ****************************************************
********************* START NS ... 385 ****************************************************
[I 2025-06-15 20:14:14,416] A new study created in memory with name: no-name-a68f948d-9c19-4447-bd60-e0db7322c776
[I 2025-06-15 20:14:14,611] Trial 0 finished with value: 0.0043202700587585095 and parameters: {'n_estimators': 262, 'learning_rate': 0.09776589812366596, 'num_leaves': 79, 'max_depth': 5, 'min_child_samples': 23, 'subsample': 0.5964534195080575, 'colsample_bytree': 0.6465329267936, 'reg_alpha': 2.3405737878649036, 'reg_lambda': 0.605715200861604, 'min_split_gain': 0.020332063950285598}. Best is trial 0 with value: 0.0043202700587585095.
[Trial 0] New best: 0.00432
[I 2025-06-15 20:14:14,818] Trial 1 finished with value: 0.0034591382140621643 and parameters: {'n_estimators': 209, 'learning_rate': 0.21889580815951543, 'num_leaves': 150, 'max_depth': 14, 'min_child_samples': 10, 'subsample': 0.7185509093717158, 'colsample_bytree': 0.9022467336262262, 'reg_alpha': 8.757237026018194, 'reg_lambda': 7.821718389970361, 'min_split_gain': 0.006077485573309277}. Best is trial 1 with value: 0.0034591382140621643.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:14:15,158] Trial 2 finished with value: 0.005453056029336277 and parameters: {'n_estimators': 75, 'learning_rate': 0.07093132190772962, 'num_leaves': 181, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.7362371119964433, 'colsample_bytree': 0.6328506769492528, 'reg_alpha': 3.1916987156742085, 'reg_lambda': 3.6291976727942465, 'min_split_gain': 0.01619918023473409}. Best is trial 1 with value: 0.0034591382140621643.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:15,862] Trial 3 finished with value: 0.004373421935505024 and parameters: {'n_estimators': 249, 'learning_rate': 0.02306369005701405, 'num_leaves': 74, 'max_depth': 13, 'min_child_samples': 26, 'subsample': 0.6879562353627245, 'colsample_bytree': 0.6449532059107586, 'reg_alpha': 8.83200389408007, 'reg_lambda': 6.011649783604771, 'min_split_gain': 0.026854530994132014}. Best is trial 1 with value: 0.0034591382140621643.
[I 2025-06-15 20:14:15,864] A new study created in memory with name: no-name-084ae759-c1aa-43b7-8582-a7d5ec50abba
[I 2025-06-15 20:14:15,889] Trial 0 finished with value: 0.042208503719558886 and parameters: {'n_estimators': 154, 'learning_rate': 0.15932032776343502, 'num_leaves': 106, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.8575156509397207, 'colsample_bytree': 0.5509008119041489, 'reg_alpha': 8.02108931963551, 'reg_lambda': 8.533129936582949, 'min_split_gain': 0.038381709990328416}. Best is trial 0 with value: 0.042208503719558886.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         279
14      pct_chg_Load_DA         207
1           Price_lag_2         207
3         Load_DA_lag_0         129
5       WindOn_DA_lag_0         115
16  roll2d_median_Price         111
2           Price_lag_7         107
15       lag168_Load_DA          96
13            EUA_lag_2          93
11           NGas_lag_2          90
10           Coal_lag_2          75
12            Oil_lag_2          44
9                  WD_7          13
7                  WD_1           6
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          26
3         Load_DA_lag_0           6
15       lag168_Load_DA           5
12            Oil_lag_2           2
1           Price_lag_2           2
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 52.80% done
********************* END NS.. 385 ****************************************************
********************* START NS ... 386 ****************************************************
[I 2025-06-15 20:14:16,950] A new study created in memory with name: no-name-5d88b768-c339-4e0c-a81c-c021efc2a954
[I 2025-06-15 20:14:17,180] Trial 0 finished with value: 0.0014813390853597324 and parameters: {'n_estimators': 87, 'learning_rate': 0.04719840510647907, 'num_leaves': 108, 'max_depth': 13, 'min_child_samples': 27, 'subsample': 0.8350038005678329, 'colsample_bytree': 0.7534706790522143, 'reg_alpha': 4.656271839747998, 'reg_lambda': 3.066312368500234, 'min_split_gain': 0.04056472388672096}. Best is trial 0 with value: 0.0014813390853597324.
[I 2025-06-15 20:14:17,346] Trial 1 finished with value: 0.0021061480512048116 and parameters: {'n_estimators': 82, 'learning_rate': 0.05337458067808398, 'num_leaves': 228, 'max_depth': 5, 'min_child_samples': 12, 'subsample': 0.8620403653728476, 'colsample_bytree': 0.5800694954214924, 'reg_alpha': 3.458325261656495, 'reg_lambda': 3.637678036514842, 'min_split_gain': 0.031611625514049134}. Best is trial 0 with value: 0.0014813390853597324.
[Trial 0] New best: 0.00148
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:14:17,880] Trial 2 finished with value: 0.0012569735555457283 and parameters: {'n_estimators': 268, 'learning_rate': 0.10717734988958053, 'num_leaves': 35, 'max_depth': 15, 'min_child_samples': 3, 'subsample': 0.763617660504025, 'colsample_bytree': 0.8854985365347842, 'reg_alpha': 2.6713985762231838, 'reg_lambda': 5.677894390449164, 'min_split_gain': 0.017918848630567964}. Best is trial 2 with value: 0.0012569735555457283.
[I 2025-06-15 20:14:18,079] Trial 3 finished with value: 0.0016250916620300256 and parameters: {'n_estimators': 264, 'learning_rate': 0.2535238161502265, 'num_leaves': 228, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.7267741849451854, 'colsample_bytree': 0.6432256808685257, 'reg_alpha': 9.501040255097372, 'reg_lambda': 7.751258603047962, 'min_split_gain': 0.03595802351790847}. Best is trial 2 with value: 0.0012569735555457283.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:18,081] A new study created in memory with name: no-name-78087ec5-1358-4ed2-9df9-762b27f90670
[I 2025-06-15 20:14:18,118] Trial 0 finished with value: 0.06436233384999963 and parameters: {'n_estimators': 220, 'learning_rate': 0.06843315991514776, 'num_leaves': 230, 'max_depth': 5, 'min_child_samples': 8, 'subsample': 0.5996330416898537, 'colsample_bytree': 0.8609750392720643, 'reg_alpha': 5.514292433545748, 'reg_lambda': 5.8130376399920465, 'min_split_gain': 0.015486525807406665}. Best is trial 0 with value: 0.06436233384999963.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         583
1           Price_lag_2         372
14      pct_chg_Load_DA         371
3         Load_DA_lag_0         291
5       WindOn_DA_lag_0         218
2           Price_lag_7         206
11           NGas_lag_2         196
15       lag168_Load_DA         177
16  roll2d_median_Price         169
13            EUA_lag_2         155
12            Oil_lag_2         139
10           Coal_lag_2         125
9                  WD_7          16
7                  WD_1          13
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          83
15       lag168_Load_DA          36
3         Load_DA_lag_0          33
10           Coal_lag_2          10
16  roll2d_median_Price           5
14      pct_chg_Load_DA           1
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
4        Solar_DA_lag_0           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
2           Price_lag_7           0
8                  WD_6           0
-> 52.94% done
********************* END NS.. 386 ****************************************************
********************* START NS ... 387 ****************************************************
[I 2025-06-15 20:14:19,311] A new study created in memory with name: no-name-ec96df71-e8cd-4f12-a723-253419300743
[I 2025-06-15 20:14:19,653] Trial 0 finished with value: 0.003032011536323058 and parameters: {'n_estimators': 256, 'learning_rate': 0.05222467303635862, 'num_leaves': 34, 'max_depth': 4, 'min_child_samples': 3, 'subsample': 0.5897654934212537, 'colsample_bytree': 0.8239338327385755, 'reg_alpha': 0.3713250639800181, 'reg_lambda': 2.5589744374162393, 'min_split_gain': 0.003930636820981165}. Best is trial 0 with value: 0.003032011536323058.
[Trial 0] New best: 0.00303
[I 2025-06-15 20:14:20,386] Trial 1 finished with value: 0.0035053470608704096 and parameters: {'n_estimators': 80, 'learning_rate': 0.04483286587231809, 'num_leaves': 238, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.8341287230816329, 'colsample_bytree': 0.8024134146010113, 'reg_alpha': 0.6176610423577489, 'reg_lambda': 6.482067101554829, 'min_split_gain': 0.046382654628026675}. Best is trial 0 with value: 0.003032011536323058.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:14:20,980] Trial 2 finished with value: 0.0033945164781038206 and parameters: {'n_estimators': 114, 'learning_rate': 0.04019313652657422, 'num_leaves': 127, 'max_depth': 11, 'min_child_samples': 20, 'subsample': 0.9383645694749249, 'colsample_bytree': 0.8805825400183664, 'reg_alpha': 8.163788945000105, 'reg_lambda': 1.941704464093783, 'min_split_gain': 0.0274767450384162}. Best is trial 0 with value: 0.003032011536323058.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:21,358] Trial 3 finished with value: 0.0028120099026731154 and parameters: {'n_estimators': 273, 'learning_rate': 0.22330292596361312, 'num_leaves': 220, 'max_depth': 12, 'min_child_samples': 27, 'subsample': 0.5458239979023264, 'colsample_bytree': 0.8300849658648879, 'reg_alpha': 4.8050143482037, 'reg_lambda': 9.915813469991296, 'min_split_gain': 0.04358544090019483}. Best is trial 3 with value: 0.0028120099026731154.
[I 2025-06-15 20:14:21,360] A new study created in memory with name: no-name-1a443728-db6c-48a9-80dc-efd9b2da0c07
[I 2025-06-15 20:14:21,450] Trial 0 finished with value: 0.08993460360815363 and parameters: {'n_estimators': 296, 'learning_rate': 0.025391468223251185, 'num_leaves': 66, 'max_depth': 5, 'min_child_samples': 30, 'subsample': 0.5176108269385651, 'colsample_bytree': 0.8327963559743012, 'reg_alpha': 2.745377346974338, 'reg_lambda': 8.66251470280745, 'min_split_gain': 0.016266491950488456}. Best is trial 0 with value: 0.08993460360815363.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         236
14      pct_chg_Load_DA         162
1           Price_lag_2         149
3         Load_DA_lag_0          86
2           Price_lag_7          64
11           NGas_lag_2          55
16  roll2d_median_Price          52
5       WindOn_DA_lag_0          44
15       lag168_Load_DA          41
13            EUA_lag_2          33
10           Coal_lag_2          32
12            Oil_lag_2          19
9                  WD_7           5
7                  WD_1           2
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
3         Load_DA_lag_0         112
0           Price_lag_1         105
15       lag168_Load_DA          73
10           Coal_lag_2          54
16  roll2d_median_Price          40
2           Price_lag_7          36
5       WindOn_DA_lag_0          22
12            Oil_lag_2           5
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
4        Solar_DA_lag_0           0
11           NGas_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 53.08% done
********************* END NS.. 387 ****************************************************
********************* START NS ... 388 ****************************************************
[I 2025-06-15 20:14:23,177] A new study created in memory with name: no-name-7b2ccfa3-d005-4027-b7e4-3bc751b3d2b0
[I 2025-06-15 20:14:23,368] Trial 0 finished with value: 0.006919388555996622 and parameters: {'n_estimators': 165, 'learning_rate': 0.2417855250108717, 'num_leaves': 233, 'max_depth': 14, 'min_child_samples': 5, 'subsample': 0.754013189735486, 'colsample_bytree': 0.9830667692310655, 'reg_alpha': 9.816045913997119, 'reg_lambda': 0.4845150934508968, 'min_split_gain': 0.015179937534562139}. Best is trial 0 with value: 0.006919388555996622.
[Trial 0] New best: 0.00692
[I 2025-06-15 20:14:24,263] Trial 1 finished with value: 0.009251786974310986 and parameters: {'n_estimators': 175, 'learning_rate': 0.01850143473304037, 'num_leaves': 162, 'max_depth': 14, 'min_child_samples': 20, 'subsample': 0.7164476307217746, 'colsample_bytree': 0.756095865697678, 'reg_alpha': 4.601000035051435, 'reg_lambda': 5.591549388051263, 'min_split_gain': 0.009481545091043125}. Best is trial 0 with value: 0.006919388555996622.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:14:24,666] Trial 2 finished with value: 0.006419759713773396 and parameters: {'n_estimators': 255, 'learning_rate': 0.2571750326612473, 'num_leaves': 167, 'max_depth': 15, 'min_child_samples': 30, 'subsample': 0.841609989099037, 'colsample_bytree': 0.5396145158600206, 'reg_alpha': 1.8445249785143347, 'reg_lambda': 6.403132339577758, 'min_split_gain': 0.010193598365844615}. Best is trial 2 with value: 0.006419759713773396.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:25,439] Trial 3 finished with value: 0.013327736822219457 and parameters: {'n_estimators': 168, 'learning_rate': 0.012421183690871218, 'num_leaves': 101, 'max_depth': 9, 'min_child_samples': 15, 'subsample': 0.8631233009263999, 'colsample_bytree': 0.9461448911936634, 'reg_alpha': 2.907066057560521, 'reg_lambda': 4.629167513185591, 'min_split_gain': 0.005162158006550494}. Best is trial 2 with value: 0.006419759713773396.
[I 2025-06-15 20:14:25,442] A new study created in memory with name: no-name-43948dde-6b32-45b3-942c-b05f6a1d3ce7
[I 2025-06-15 20:14:25,508] Trial 0 finished with value: 0.06257274739337519 and parameters: {'n_estimators': 291, 'learning_rate': 0.015525878674367721, 'num_leaves': 169, 'max_depth': 14, 'min_child_samples': 4, 'subsample': 0.845747089740999, 'colsample_bytree': 0.6220056171431421, 'reg_alpha': 1.9735085070234593, 'reg_lambda': 7.763235328466615, 'min_split_gain': 0.019668373518058196}. Best is trial 0 with value: 0.06257274739337519.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         511
0           Price_lag_1         498
16  roll2d_median_Price         387
14      pct_chg_Load_DA         330
3         Load_DA_lag_0         313
2           Price_lag_7         232
5       WindOn_DA_lag_0         220
11           NGas_lag_2         210
15       lag168_Load_DA         184
10           Coal_lag_2         181
13            EUA_lag_2         166
12            Oil_lag_2         144
7                  WD_1          11
8                  WD_6          11
9                  WD_7          10
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         327
3         Load_DA_lag_0         116
15       lag168_Load_DA          68
1           Price_lag_2          40
16  roll2d_median_Price          40
10           Coal_lag_2          11
2           Price_lag_7          11
5       WindOn_DA_lag_0           4
14      pct_chg_Load_DA           3
13            EUA_lag_2           2
11           NGas_lag_2           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
12            Oil_lag_2           0
8                  WD_6           0
-> 53.21% done
********************* END NS.. 388 ****************************************************
********************* START NS ... 389 ****************************************************
[I 2025-06-15 20:14:27,224] A new study created in memory with name: no-name-d87f8e0e-69aa-44b8-8edd-c86fe0529567
[I 2025-06-15 20:14:27,579] Trial 0 finished with value: 0.005807852694978505 and parameters: {'n_estimators': 275, 'learning_rate': 0.18875400232337328, 'num_leaves': 155, 'max_depth': 12, 'min_child_samples': 20, 'subsample': 0.8042838620712314, 'colsample_bytree': 0.9686525377293278, 'reg_alpha': 2.596913393171482, 'reg_lambda': 9.521037146814933, 'min_split_gain': 0.03153879067540967}. Best is trial 0 with value: 0.005807852694978505.
[Trial 0] New best: 0.00581
[I 2025-06-15 20:14:28,097] Trial 1 finished with value: 0.01859469138233157 and parameters: {'n_estimators': 90, 'learning_rate': 0.019736221548502964, 'num_leaves': 145, 'max_depth': 5, 'min_child_samples': 17, 'subsample': 0.9826244268532978, 'colsample_bytree': 0.9042735037301359, 'reg_alpha': 2.0157002272948823, 'reg_lambda': 3.1103699453644564, 'min_split_gain': 0.028127483079190364}. Best is trial 0 with value: 0.005807852694978505.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:14:28,580] Trial 2 finished with value: 0.005921728252591501 and parameters: {'n_estimators': 70, 'learning_rate': 0.08031377534245507, 'num_leaves': 92, 'max_depth': 11, 'min_child_samples': 7, 'subsample': 0.7990516798275938, 'colsample_bytree': 0.9325433939133938, 'reg_alpha': 3.348541568227702, 'reg_lambda': 2.8374674168609704, 'min_split_gain': 0.043887635712819166}. Best is trial 0 with value: 0.005807852694978505.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:31,229] Trial 3 finished with value: 0.008280334041738365 and parameters: {'n_estimators': 235, 'learning_rate': 0.025491298633783178, 'num_leaves': 253, 'max_depth': 16, 'min_child_samples': 11, 'subsample': 0.7259631803018918, 'colsample_bytree': 0.5451273998982482, 'reg_alpha': 9.18116477314906, 'reg_lambda': 2.3145111505428186, 'min_split_gain': 0.010696721379203335}. Best is trial 0 with value: 0.005807852694978505.
[I 2025-06-15 20:14:31,232] A new study created in memory with name: no-name-ef32b60b-95d4-49cf-a360-0a3ab1a84708
[I 2025-06-15 20:14:31,340] Trial 0 finished with value: 0.036148673089281214 and parameters: {'n_estimators': 154, 'learning_rate': 0.032493335914656266, 'num_leaves': 168, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.6417417098493015, 'colsample_bytree': 0.5108611379013261, 'reg_alpha': 0.3951138662688214, 'reg_lambda': 4.8893046161951625, 'min_split_gain': 0.015064896361819558}. Best is trial 0 with value: 0.036148673089281214.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         394
14      pct_chg_Load_DA         285
1           Price_lag_2         266
3         Load_DA_lag_0         181
2           Price_lag_7         141
16  roll2d_median_Price         133
11           NGas_lag_2         129
5       WindOn_DA_lag_0         120
15       lag168_Load_DA         119
10           Coal_lag_2          78
13            EUA_lag_2          63
12            Oil_lag_2          45
9                  WD_7          10
7                  WD_1           7
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         173
3         Load_DA_lag_0          61
14      pct_chg_Load_DA          55
1           Price_lag_2          33
16  roll2d_median_Price          27
15       lag168_Load_DA          24
2           Price_lag_7           6
12            Oil_lag_2           4
10           Coal_lag_2           1
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
11           NGas_lag_2           0
13            EUA_lag_2           0
8                  WD_6           0
-> 53.35% done
********************* END NS.. 389 ****************************************************
********************* START NS ... 390 ****************************************************
[I 2025-06-15 20:14:33,826] A new study created in memory with name: no-name-533443f2-3e35-4811-9e53-afe2d66c82b9
[I 2025-06-15 20:14:34,169] Trial 0 finished with value: 0.007195889539778101 and parameters: {'n_estimators': 111, 'learning_rate': 0.07633896165302953, 'num_leaves': 247, 'max_depth': 4, 'min_child_samples': 15, 'subsample': 0.7364001274422003, 'colsample_bytree': 0.9809356469842969, 'reg_alpha': 1.0344537354496852, 'reg_lambda': 1.6300483939184751, 'min_split_gain': 0.02456920526765807}. Best is trial 0 with value: 0.007195889539778101.
[Trial 0] New best: 0.00720
[I 2025-06-15 20:14:35,679] Trial 1 finished with value: 0.006720249235880706 and parameters: {'n_estimators': 214, 'learning_rate': 0.06661577164018255, 'num_leaves': 253, 'max_depth': 9, 'min_child_samples': 11, 'subsample': 0.9143100495175245, 'colsample_bytree': 0.58865013110732, 'reg_alpha': 8.594277369438183, 'reg_lambda': 0.37844948938689504, 'min_split_gain': 0.0015047547826172293}. Best is trial 1 with value: 0.006720249235880706.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:14:36,433] Trial 2 finished with value: 0.006894278252959879 and parameters: {'n_estimators': 112, 'learning_rate': 0.07136195622662561, 'num_leaves': 164, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.5544320890075416, 'colsample_bytree': 0.6522422863725394, 'reg_alpha': 8.227875262750993, 'reg_lambda': 2.3061670514725394, 'min_split_gain': 0.030985702387046045}. Best is trial 1 with value: 0.006720249235880706.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:38,690] Trial 3 finished with value: 0.0071705416117184536 and parameters: {'n_estimators': 231, 'learning_rate': 0.0156094488558553, 'num_leaves': 79, 'max_depth': 9, 'min_child_samples': 1, 'subsample': 0.517394009076939, 'colsample_bytree': 0.7396088553955811, 'reg_alpha': 7.581702770854115, 'reg_lambda': 8.211723277438468, 'min_split_gain': 0.0335981968510891}. Best is trial 1 with value: 0.006720249235880706.
[I 2025-06-15 20:14:38,693] A new study created in memory with name: no-name-591a1728-7501-4756-b6bd-ea9bd789465b
[I 2025-06-15 20:14:38,768] Trial 0 finished with value: 0.06260299775998963 and parameters: {'n_estimators': 255, 'learning_rate': 0.017028238033782332, 'num_leaves': 196, 'max_depth': 10, 'min_child_samples': 20, 'subsample': 0.7169832044619159, 'colsample_bytree': 0.593634737954073, 'reg_alpha': 7.394689231390722, 'reg_lambda': 9.892391901666551, 'min_split_gain': 0.012894448679265808}. Best is trial 0 with value: 0.06260299775998963.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1280
1           Price_lag_2        1236
14      pct_chg_Load_DA         754
3         Load_DA_lag_0         536
13            EUA_lag_2         529
5       WindOn_DA_lag_0         523
11           NGas_lag_2         461
2           Price_lag_7         423
15       lag168_Load_DA         405
10           Coal_lag_2         339
16  roll2d_median_Price         331
12            Oil_lag_2         306
9                  WD_7          50
8                  WD_6          34
7                  WD_1          26
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         151
1           Price_lag_2          52
3         Load_DA_lag_0           3
2           Price_lag_7           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 53.49% done
********************* END NS.. 390 ****************************************************
********************* START NS ... 391 ****************************************************
[I 2025-06-15 20:14:41,897] A new study created in memory with name: no-name-70446161-e9bc-4152-b8a4-843125acf83e
[I 2025-06-15 20:14:42,305] Trial 0 finished with value: 0.005885785032168127 and parameters: {'n_estimators': 270, 'learning_rate': 0.09776434898842036, 'num_leaves': 156, 'max_depth': 4, 'min_child_samples': 24, 'subsample': 0.8839284779268979, 'colsample_bytree': 0.9866076158610806, 'reg_alpha': 3.8027626627779467, 'reg_lambda': 7.8732682878139935, 'min_split_gain': 0.03564242500569583}. Best is trial 0 with value: 0.005885785032168127.
[Trial 0] New best: 0.00589
[I 2025-06-15 20:14:42,800] Trial 1 finished with value: 0.006037240766389386 and parameters: {'n_estimators': 128, 'learning_rate': 0.07111088453746159, 'num_leaves': 173, 'max_depth': 5, 'min_child_samples': 8, 'subsample': 0.5712078666314176, 'colsample_bytree': 0.7318813437037845, 'reg_alpha': 4.698960421589083, 'reg_lambda': 1.9591884288723627, 'min_split_gain': 0.0468986419484564}. Best is trial 0 with value: 0.005885785032168127.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:14:43,361] Trial 2 finished with value: 0.03717174561216294 and parameters: {'n_estimators': 65, 'learning_rate': 0.013333912426291115, 'num_leaves': 106, 'max_depth': 10, 'min_child_samples': 16, 'subsample': 0.5559769271978416, 'colsample_bytree': 0.7799291611641235, 'reg_alpha': 8.23921879870641, 'reg_lambda': 2.619761988946969, 'min_split_gain': 0.006201795387366926}. Best is trial 0 with value: 0.005885785032168127.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:43,984] Trial 3 finished with value: 0.005554405072215965 and parameters: {'n_estimators': 166, 'learning_rate': 0.08462147489076197, 'num_leaves': 83, 'max_depth': 15, 'min_child_samples': 17, 'subsample': 0.6214052176075566, 'colsample_bytree': 0.7004421801387781, 'reg_alpha': 7.421042789206982, 'reg_lambda': 9.504820657042139, 'min_split_gain': 0.037813278382165486}. Best is trial 3 with value: 0.005554405072215965.
[I 2025-06-15 20:14:43,987] A new study created in memory with name: no-name-827f9921-e948-4d19-abf1-a0db06316f53
[I 2025-06-15 20:14:44,032] Trial 0 finished with value: 0.03774760393179278 and parameters: {'n_estimators': 158, 'learning_rate': 0.2229368088027073, 'num_leaves': 70, 'max_depth': 4, 'min_child_samples': 27, 'subsample': 0.5211856712293137, 'colsample_bytree': 0.7377126161990365, 'reg_alpha': 4.887895337365996, 'reg_lambda': 0.10779628061215352, 'min_split_gain': 0.022066435450207034}. Best is trial 0 with value: 0.03774760393179278.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         543
1           Price_lag_2         373
14      pct_chg_Load_DA         335
3         Load_DA_lag_0         199
2           Price_lag_7         153
11           NGas_lag_2         150
16  roll2d_median_Price         139
5       WindOn_DA_lag_0         109
15       lag168_Load_DA          99
10           Coal_lag_2          86
13            EUA_lag_2          85
12            Oil_lag_2          60
9                  WD_7          13
8                  WD_6           6
7                  WD_1           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          12
3         Load_DA_lag_0           3
1           Price_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 53.63% done
********************* END NS.. 391 ****************************************************
********************* START NS ... 392 ****************************************************
[I 2025-06-15 20:14:46,155] A new study created in memory with name: no-name-8662f0b2-1ef7-4dd8-a77c-3f6be49002fe
[I 2025-06-15 20:14:48,580] Trial 0 finished with value: 0.000542268039225493 and parameters: {'n_estimators': 114, 'learning_rate': 0.05765268769431576, 'num_leaves': 194, 'max_depth': 8, 'min_child_samples': 2, 'subsample': 0.8999374274476124, 'colsample_bytree': 0.9660770205026573, 'reg_alpha': 0.8609129395554627, 'reg_lambda': 1.1070465903273519, 'min_split_gain': 0.0002454427292373718}. Best is trial 0 with value: 0.000542268039225493.
[Trial 0] New best: 0.00054
[I 2025-06-15 20:14:49,306] Trial 1 finished with value: 0.0005918408449511623 and parameters: {'n_estimators': 134, 'learning_rate': 0.028730515789537458, 'num_leaves': 86, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.5414944888828537, 'colsample_bytree': 0.6941287908235481, 'reg_alpha': 1.3427739170548214, 'reg_lambda': 5.8147717258229425, 'min_split_gain': 0.022280335339943377}. Best is trial 0 with value: 0.000542268039225493.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:14:49,563] Trial 2 finished with value: 0.009219292943596217 and parameters: {'n_estimators': 72, 'learning_rate': 0.022602482012138297, 'num_leaves': 38, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.5940642763463335, 'colsample_bytree': 0.8358152465403387, 'reg_alpha': 6.3079297019531815, 'reg_lambda': 1.7601183784084884, 'min_split_gain': 0.03392983818474323}. Best is trial 0 with value: 0.000542268039225493.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:51,714] Trial 3 finished with value: 0.0010751656484282272 and parameters: {'n_estimators': 195, 'learning_rate': 0.033089128793842466, 'num_leaves': 110, 'max_depth': 11, 'min_child_samples': 23, 'subsample': 0.6887751910360544, 'colsample_bytree': 0.5583583278070018, 'reg_alpha': 9.371623644231807, 'reg_lambda': 8.958108941503138, 'min_split_gain': 0.0014780244841444967}. Best is trial 0 with value: 0.000542268039225493.
[I 2025-06-15 20:14:51,716] A new study created in memory with name: no-name-36ffd2f6-2494-4e8a-b7a7-694471eb5d94
[I 2025-06-15 20:14:51,752] Trial 0 finished with value: 0.0008472024412358622 and parameters: {'n_estimators': 55, 'learning_rate': 0.039759257078803525, 'num_leaves': 71, 'max_depth': 12, 'min_child_samples': 29, 'subsample': 0.5071616178120264, 'colsample_bytree': 0.8661404506494199, 'reg_alpha': 5.951417021349272, 'reg_lambda': 6.410574342796237, 'min_split_gain': 0.00500162747233513}. Best is trial 0 with value: 0.0008472024412358622.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        2189
0           Price_lag_1        2112
14      pct_chg_Load_DA        1969
3         Load_DA_lag_0        1422
2           Price_lag_7        1268
5       WindOn_DA_lag_0        1228
16  roll2d_median_Price        1148
15       lag168_Load_DA        1019
11           NGas_lag_2         816
10           Coal_lag_2         785
13            EUA_lag_2         714
12            Oil_lag_2         675
9                  WD_7          39
7                  WD_1          29
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          52
1           Price_lag_2           4
15       lag168_Load_DA           3
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 53.76% done
********************* END NS.. 392 ****************************************************
********************* START NS ... 393 ****************************************************
[I 2025-06-15 20:14:55,757] A new study created in memory with name: no-name-94c9e986-353b-4c84-873a-47c7022256a0
[I 2025-06-15 20:14:57,761] Trial 0 finished with value: 0.0005398734429922018 and parameters: {'n_estimators': 297, 'learning_rate': 0.05074197957910595, 'num_leaves': 210, 'max_depth': 15, 'min_child_samples': 12, 'subsample': 0.9413020705152646, 'colsample_bytree': 0.7545873375281567, 'reg_alpha': 3.073990406850622, 'reg_lambda': 3.5546998628938864, 'min_split_gain': 0.01778191212992406}. Best is trial 0 with value: 0.0005398734429922018.
[Trial 0] New best: 0.00054
[I 2025-06-15 20:14:58,215] Trial 1 finished with value: 0.0007244488074927584 and parameters: {'n_estimators': 266, 'learning_rate': 0.2984396408220317, 'num_leaves': 240, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.6776367965533249, 'colsample_bytree': 0.6331156563882181, 'reg_alpha': 4.564062380002499, 'reg_lambda': 2.160958041445391, 'min_split_gain': 0.027969994061279715}. Best is trial 0 with value: 0.0005398734429922018.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:14:58,902] Trial 2 finished with value: 0.00037849291062643805 and parameters: {'n_estimators': 86, 'learning_rate': 0.0498961601761813, 'num_leaves': 143, 'max_depth': 6, 'min_child_samples': 28, 'subsample': 0.9011703467818705, 'colsample_bytree': 0.7951172693578046, 'reg_alpha': 1.1891227571598684, 'reg_lambda': 1.8941123041679542, 'min_split_gain': 0.0046326607754574095}. Best is trial 2 with value: 0.00037849291062643805.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:14:59,245] Trial 3 finished with value: 0.05258399348156221 and parameters: {'n_estimators': 54, 'learning_rate': 0.01459695484958309, 'num_leaves': 193, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.6567954994339837, 'colsample_bytree': 0.9862540198341538, 'reg_alpha': 7.861464436934542, 'reg_lambda': 8.683851000693627, 'min_split_gain': 0.01049299876677941}. Best is trial 2 with value: 0.00037849291062643805.
[I 2025-06-15 20:14:59,247] A new study created in memory with name: no-name-d58f8db1-b4ee-4e0a-802b-51bf322770b1
[I 2025-06-15 20:14:59,297] Trial 0 finished with value: 0.0011800256960897333 and parameters: {'n_estimators': 186, 'learning_rate': 0.16798242946979405, 'num_leaves': 74, 'max_depth': 9, 'min_child_samples': 28, 'subsample': 0.7231393810619974, 'colsample_bytree': 0.9163722843027916, 'reg_alpha': 6.832691320196437, 'reg_lambda': 5.109845458577631, 'min_split_gain': 0.029333277235998264}. Best is trial 0 with value: 0.0011800256960897333.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1077
1           Price_lag_2         750
14      pct_chg_Load_DA         549
3         Load_DA_lag_0         223
16  roll2d_median_Price         208
2           Price_lag_7         191
11           NGas_lag_2         171
13            EUA_lag_2          98
15       lag168_Load_DA          93
10           Coal_lag_2          85
5       WindOn_DA_lag_0          72
12            Oil_lag_2          60
9                  WD_7           6
8                  WD_6           3
7                  WD_1           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 53.90% done
********************* END NS.. 393 ****************************************************
********************* START NS ... 394 ****************************************************
[I 2025-06-15 20:15:01,615] A new study created in memory with name: no-name-fb080f78-823c-42bd-83c6-f4fe3fd0d1c8
[I 2025-06-15 20:15:02,193] Trial 0 finished with value: 0.004375850256355972 and parameters: {'n_estimators': 112, 'learning_rate': 0.07677937926577684, 'num_leaves': 179, 'max_depth': 6, 'min_child_samples': 19, 'subsample': 0.6884858965612259, 'colsample_bytree': 0.9541432602514366, 'reg_alpha': 5.842433866670027, 'reg_lambda': 2.151370415580429, 'min_split_gain': 0.02126739368958599}. Best is trial 0 with value: 0.004375850256355972.
[Trial 0] New best: 0.00438
[I 2025-06-15 20:15:03,271] Trial 1 finished with value: 0.004920699464365775 and parameters: {'n_estimators': 165, 'learning_rate': 0.04219655035078158, 'num_leaves': 208, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.7331256450882772, 'colsample_bytree': 0.6760959967290785, 'reg_alpha': 8.549533901880652, 'reg_lambda': 1.8759857574821437, 'min_split_gain': 0.00863285147602103}. Best is trial 0 with value: 0.004375850256355972.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:15:03,995] Trial 2 finished with value: 0.004333820988976841 and parameters: {'n_estimators': 197, 'learning_rate': 0.10969178733188029, 'num_leaves': 158, 'max_depth': 5, 'min_child_samples': 18, 'subsample': 0.8280019844495249, 'colsample_bytree': 0.9999550756891675, 'reg_alpha': 3.9273308777271154, 'reg_lambda': 7.213690353519966, 'min_split_gain': 0.008712599067487349}. Best is trial 2 with value: 0.004333820988976841.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:15:04,759] Trial 3 finished with value: 0.004235196155355523 and parameters: {'n_estimators': 248, 'learning_rate': 0.12230996877300816, 'num_leaves': 185, 'max_depth': 10, 'min_child_samples': 20, 'subsample': 0.8716207543199446, 'colsample_bytree': 0.7730616454729933, 'reg_alpha': 6.937836351798609, 'reg_lambda': 5.027765434706568, 'min_split_gain': 0.012407159657399953}. Best is trial 3 with value: 0.004235196155355523.
[I 2025-06-15 20:15:04,762] A new study created in memory with name: no-name-7eaf812a-b8fc-4b65-aa1f-72a77cd9dfb7
[I 2025-06-15 20:15:04,823] Trial 0 finished with value: 0.0385391897397219 and parameters: {'n_estimators': 270, 'learning_rate': 0.05979001663915918, 'num_leaves': 241, 'max_depth': 7, 'min_child_samples': 30, 'subsample': 0.5002186499310609, 'colsample_bytree': 0.9130162723180142, 'reg_alpha': 9.203183567909806, 'reg_lambda': 2.782657572099747, 'min_split_gain': 0.0206896112702968}. Best is trial 0 with value: 0.0385391897397219.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         472
1           Price_lag_2         357
14      pct_chg_Load_DA         329
3         Load_DA_lag_0         202
13            EUA_lag_2         160
2           Price_lag_7         157
11           NGas_lag_2         151
5       WindOn_DA_lag_0         140
16  roll2d_median_Price         127
15       lag168_Load_DA         122
10           Coal_lag_2          75
12            Oil_lag_2          70
9                  WD_7          19
8                  WD_6          10
7                  WD_1           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 54.04% done
********************* END NS.. 394 ****************************************************
********************* START NS ... 395 ****************************************************
[I 2025-06-15 20:15:07,228] A new study created in memory with name: no-name-dae85cf1-cdaf-4735-8ad9-bcb28e028283
[I 2025-06-15 20:15:08,235] Trial 0 finished with value: 0.00720529044879694 and parameters: {'n_estimators': 281, 'learning_rate': 0.02548622566895306, 'num_leaves': 231, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.7881196311060218, 'colsample_bytree': 0.5720972965646223, 'reg_alpha': 7.216403335998908, 'reg_lambda': 2.504776863933432, 'min_split_gain': 0.04718709340283963}. Best is trial 0 with value: 0.00720529044879694.
[Trial 0] New best: 0.00721
[I 2025-06-15 20:15:08,600] Trial 1 finished with value: 0.00604071370032658 and parameters: {'n_estimators': 96, 'learning_rate': 0.17792459986562073, 'num_leaves': 229, 'max_depth': 5, 'min_child_samples': 30, 'subsample': 0.6242811717474073, 'colsample_bytree': 0.5090463512651092, 'reg_alpha': 6.069370900344762, 'reg_lambda': 1.513041517305852, 'min_split_gain': 0.0213889002194004}. Best is trial 1 with value: 0.00604071370032658.
[Trial 1] New best: 0.00604
[I 2025-06-15 20:15:09,236] Trial 2 finished with value: 0.005151655386914888 and parameters: {'n_estimators': 77, 'learning_rate': 0.06705837416874452, 'num_leaves': 123, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.7383510028918683, 'colsample_bytree': 0.8092394483934134, 'reg_alpha': 8.72539529599746, 'reg_lambda': 5.9019453341167205, 'min_split_gain': 0.01673600064389947}. Best is trial 2 with value: 0.005151655386914888.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:15:10,501] Trial 3 finished with value: 0.0045715928964485545 and parameters: {'n_estimators': 137, 'learning_rate': 0.13024051827588692, 'num_leaves': 81, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.6743692383041376, 'colsample_bytree': 0.7794205446353069, 'reg_alpha': 1.153821492053606, 'reg_lambda': 4.696630375349718, 'min_split_gain': 0.014848104305053594}. Best is trial 3 with value: 0.0045715928964485545.
[Trial 3] New best: 0.00457
[I 2025-06-15 20:15:11,469] Trial 4 finished with value: 0.004783504127212883 and parameters: {'n_estimators': 262, 'learning_rate': 0.06810821427786509, 'num_leaves': 250, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.8564742947178042, 'colsample_bytree': 0.8391332610203432, 'reg_alpha': 9.79298709620948, 'reg_lambda': 3.5047217934677324, 'min_split_gain': 0.02133248447778433}. Best is trial 3 with value: 0.0045715928964485545.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:15:12,176] Trial 5 finished with value: 0.0046254360385284314 and parameters: {'n_estimators': 250, 'learning_rate': 0.07621235096174597, 'num_leaves': 170, 'max_depth': 5, 'min_child_samples': 22, 'subsample': 0.8170472281721493, 'colsample_bytree': 0.8896820707918439, 'reg_alpha': 4.606160697463837, 'reg_lambda': 3.167089029368164, 'min_split_gain': 0.02165794332888526}. Best is trial 3 with value: 0.0045715928964485545.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:15:14,663] Trial 6 finished with value: 0.0071954341665361405 and parameters: {'n_estimators': 267, 'learning_rate': 0.014090077499023725, 'num_leaves': 122, 'max_depth': 7, 'min_child_samples': 3, 'subsample': 0.9394326987969228, 'colsample_bytree': 0.5115694123611532, 'reg_alpha': 5.721470180144237, 'reg_lambda': 7.92202908848317, 'min_split_gain': 0.040878931152502344}. Best is trial 3 with value: 0.0045715928964485545.
[I 2025-06-15 20:15:14,665] A new study created in memory with name: no-name-577b0e67-975e-4296-b469-09563f9d03d1
[I 2025-06-15 20:15:14,716] Trial 0 finished with value: 0.037061391285309914 and parameters: {'n_estimators': 204, 'learning_rate': 0.2936403707387536, 'num_leaves': 190, 'max_depth': 15, 'min_child_samples': 22, 'subsample': 0.9988598682264034, 'colsample_bytree': 0.7775347679251806, 'reg_alpha': 4.450169276391225, 'reg_lambda': 3.376331790975894, 'min_split_gain': 0.007212364078737011}. Best is trial 0 with value: 0.037061391285309914.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA         699
1           Price_lag_2         674
0           Price_lag_1         660
3         Load_DA_lag_0         499
5       WindOn_DA_lag_0         467
15       lag168_Load_DA         422
2           Price_lag_7         367
16  roll2d_median_Price         301
11           NGas_lag_2         273
13            EUA_lag_2         256
12            Oil_lag_2         202
10           Coal_lag_2         187
9                  WD_7          21
7                  WD_1          12
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          12
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 54.17% done
********************* END NS.. 395 ****************************************************
********************* START NS ... 396 ****************************************************
[I 2025-06-15 20:15:17,541] A new study created in memory with name: no-name-0940eb50-6222-4143-982f-5e18caa5ad1a
[I 2025-06-15 20:15:19,600] Trial 0 finished with value: 0.001318894949080055 and parameters: {'n_estimators': 203, 'learning_rate': 0.07032764872261549, 'num_leaves': 156, 'max_depth': 11, 'min_child_samples': 24, 'subsample': 0.7899839395670304, 'colsample_bytree': 0.6696399860917284, 'reg_alpha': 1.6187182369966846, 'reg_lambda': 8.438032343196895, 'min_split_gain': 0.01182712167898601}. Best is trial 0 with value: 0.001318894949080055.
[Trial 0] New best: 0.00132
[I 2025-06-15 20:15:20,298] Trial 1 finished with value: 0.0025220786276780264 and parameters: {'n_estimators': 226, 'learning_rate': 0.012063748773703357, 'num_leaves': 105, 'max_depth': 4, 'min_child_samples': 10, 'subsample': 0.8930714644610105, 'colsample_bytree': 0.7312459008002077, 'reg_alpha': 8.178491371429029, 'reg_lambda': 0.058412270618123374, 'min_split_gain': 0.04342534027014542}. Best is trial 0 with value: 0.001318894949080055.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:15:25,097] Trial 2 finished with value: 0.0014828635107768332 and parameters: {'n_estimators': 270, 'learning_rate': 0.01852991437452876, 'num_leaves': 237, 'max_depth': 14, 'min_child_samples': 19, 'subsample': 0.530550931829049, 'colsample_bytree': 0.530453064799758, 'reg_alpha': 3.9227908332205583, 'reg_lambda': 0.5161555112527882, 'min_split_gain': 0.00040823276290815615}. Best is trial 0 with value: 0.001318894949080055.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:15:25,807] Trial 3 finished with value: 0.0016945177337363932 and parameters: {'n_estimators': 213, 'learning_rate': 0.019635109932656405, 'num_leaves': 196, 'max_depth': 4, 'min_child_samples': 14, 'subsample': 0.5249587101606705, 'colsample_bytree': 0.5137143510932348, 'reg_alpha': 1.491739441237775, 'reg_lambda': 1.0896274691086583, 'min_split_gain': 0.04919125243173563}. Best is trial 0 with value: 0.001318894949080055.
[I 2025-06-15 20:15:25,810] A new study created in memory with name: no-name-49acf21b-fd8f-45fa-b0f6-c99e76fea960
[I 2025-06-15 20:15:25,841] Trial 0 finished with value: 0.006607062042533737 and parameters: {'n_estimators': 81, 'learning_rate': 0.13833645902353064, 'num_leaves': 179, 'max_depth': 7, 'min_child_samples': 14, 'subsample': 0.9678960827433185, 'colsample_bytree': 0.7045775680151591, 'reg_alpha': 5.111818482985523, 'reg_lambda': 4.915196039266602, 'min_split_gain': 0.013082815248733531}. Best is trial 0 with value: 0.006607062042533737.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1346
1           Price_lag_2        1242
14      pct_chg_Load_DA        1097
3         Load_DA_lag_0        1037
5       WindOn_DA_lag_0         687
15       lag168_Load_DA         672
11           NGas_lag_2         618
16  roll2d_median_Price         599
2           Price_lag_7         595
10           Coal_lag_2         408
13            EUA_lag_2         397
12            Oil_lag_2         333
9                  WD_7          44
7                  WD_1          33
8                  WD_6          19
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          12
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 54.31% done
********************* END NS.. 396 ****************************************************
********************* START NS ... 397 ****************************************************
[I 2025-06-15 20:15:29,020] A new study created in memory with name: no-name-517c3cd9-aafc-4361-8d1c-efd8f33449da
[I 2025-06-15 20:15:29,726] Trial 0 finished with value: 0.0014529838531177318 and parameters: {'n_estimators': 216, 'learning_rate': 0.11041532268869779, 'num_leaves': 226, 'max_depth': 10, 'min_child_samples': 22, 'subsample': 0.5750888102229355, 'colsample_bytree': 0.7684059041833636, 'reg_alpha': 8.790041425645237, 'reg_lambda': 6.003794445668423, 'min_split_gain': 0.031025318294127614}. Best is trial 0 with value: 0.0014529838531177318.
[Trial 0] New best: 0.00145
[I 2025-06-15 20:15:30,834] Trial 1 finished with value: 0.0009749204647140482 and parameters: {'n_estimators': 131, 'learning_rate': 0.057503163184888885, 'num_leaves': 133, 'max_depth': 13, 'min_child_samples': 24, 'subsample': 0.7351914965676396, 'colsample_bytree': 0.7439960557729351, 'reg_alpha': 6.010614063495804, 'reg_lambda': 7.430765712649148, 'min_split_gain': 0.014332076307619257}. Best is trial 1 with value: 0.0009749204647140482.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:15:31,547] Trial 2 finished with value: 0.0010978316535693246 and parameters: {'n_estimators': 270, 'learning_rate': 0.05362523818087623, 'num_leaves': 134, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.640882695544763, 'colsample_bytree': 0.7692957053082541, 'reg_alpha': 9.469033943953093, 'reg_lambda': 1.8347627003390832, 'min_split_gain': 0.030544294176688816}. Best is trial 1 with value: 0.0009749204647140482.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:15:31,954] Trial 3 finished with value: 0.0015134390649586662 and parameters: {'n_estimators': 203, 'learning_rate': 0.18546108168982797, 'num_leaves': 256, 'max_depth': 8, 'min_child_samples': 12, 'subsample': 0.8914930527554912, 'colsample_bytree': 0.7211082786901963, 'reg_alpha': 8.763544638157567, 'reg_lambda': 8.45296235244967, 'min_split_gain': 0.019375569434973074}. Best is trial 1 with value: 0.0009749204647140482.
[I 2025-06-15 20:15:31,956] A new study created in memory with name: no-name-858c6ae4-1d13-4daf-be29-332fd4c23e88
[I 2025-06-15 20:15:32,023] Trial 0 finished with value: 0.00899556685907099 and parameters: {'n_estimators': 297, 'learning_rate': 0.0503483988219289, 'num_leaves': 179, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.9282881063133199, 'colsample_bytree': 0.7952699501585403, 'reg_alpha': 8.503376159086422, 'reg_lambda': 7.719432605262555, 'min_split_gain': 0.03151445741746169}. Best is trial 0 with value: 0.00899556685907099.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         952
1           Price_lag_2         766
14      pct_chg_Load_DA         672
3         Load_DA_lag_0         447
2           Price_lag_7         282
11           NGas_lag_2         282
15       lag168_Load_DA         262
16  roll2d_median_Price         254
5       WindOn_DA_lag_0         233
13            EUA_lag_2         226
10           Coal_lag_2         175
12            Oil_lag_2         145
9                  WD_7          29
8                  WD_6          23
7                  WD_1          19
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 54.45% done
********************* END NS.. 397 ****************************************************
********************* START NS ... 398 ****************************************************
[I 2025-06-15 20:15:34,787] A new study created in memory with name: no-name-53f328d0-d8fe-45bc-9b28-89e88ffcd641
[I 2025-06-15 20:15:35,799] Trial 0 finished with value: 0.001060608605292724 and parameters: {'n_estimators': 114, 'learning_rate': 0.08441256946117735, 'num_leaves': 40, 'max_depth': 15, 'min_child_samples': 24, 'subsample': 0.8778615145050103, 'colsample_bytree': 0.6257339843675973, 'reg_alpha': 2.1757790002631996, 'reg_lambda': 8.506739913449312, 'min_split_gain': 0.032200674417526554}. Best is trial 0 with value: 0.001060608605292724.
[Trial 0] New best: 0.00106
[I 2025-06-15 20:15:37,482] Trial 1 finished with value: 0.0006455190013341296 and parameters: {'n_estimators': 83, 'learning_rate': 0.0929429641487633, 'num_leaves': 236, 'max_depth': 14, 'min_child_samples': 4, 'subsample': 0.8641771803339229, 'colsample_bytree': 0.9544825704407711, 'reg_alpha': 0.08467094673885955, 'reg_lambda': 3.9412319741770485, 'min_split_gain': 0.046461077933896304}. Best is trial 1 with value: 0.0006455190013341296.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:15:41,202] Trial 2 finished with value: 0.0009910381514046862 and parameters: {'n_estimators': 264, 'learning_rate': 0.032944816167888614, 'num_leaves': 132, 'max_depth': 10, 'min_child_samples': 20, 'subsample': 0.8020934293066317, 'colsample_bytree': 0.5698454533526145, 'reg_alpha': 0.5337293011167898, 'reg_lambda': 7.550619612328689, 'min_split_gain': 0.041031265304648096}. Best is trial 1 with value: 0.0006455190013341296.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:15:41,848] Trial 3 finished with value: 0.0010505071065703545 and parameters: {'n_estimators': 56, 'learning_rate': 0.07628975716665592, 'num_leaves': 192, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.7524524107362387, 'colsample_bytree': 0.741637368919961, 'reg_alpha': 3.636240729125019, 'reg_lambda': 3.8448980058706494, 'min_split_gain': 0.02718531088952303}. Best is trial 1 with value: 0.0006455190013341296.
[I 2025-06-15 20:15:41,850] A new study created in memory with name: no-name-3273029a-1562-4378-8568-10d969770adc
[I 2025-06-15 20:15:41,905] Trial 0 finished with value: 0.007895159939330548 and parameters: {'n_estimators': 178, 'learning_rate': 0.06086598719460247, 'num_leaves': 187, 'max_depth': 12, 'min_child_samples': 22, 'subsample': 0.6831193303727425, 'colsample_bytree': 0.5820346523454847, 'reg_alpha': 6.693193043689608, 'reg_lambda': 8.666091068790559, 'min_split_gain': 0.04579194030251869}. Best is trial 0 with value: 0.007895159939330548.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA        1451
1           Price_lag_2        1421
0           Price_lag_1        1115
3         Load_DA_lag_0         933
16  roll2d_median_Price         753
2           Price_lag_7         707
5       WindOn_DA_lag_0         692
15       lag168_Load_DA         556
11           NGas_lag_2         549
10           Coal_lag_2         457
12            Oil_lag_2         283
13            EUA_lag_2         264
9                  WD_7          21
7                  WD_1           7
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 54.58% done
********************* END NS.. 398 ****************************************************
********************* START NS ... 399 ****************************************************
[I 2025-06-15 20:15:44,991] A new study created in memory with name: no-name-b73cd2f8-ec41-443a-a165-1d42bdc2b6dc
[I 2025-06-15 20:15:47,710] Trial 0 finished with value: 0.0023314982681809284 and parameters: {'n_estimators': 176, 'learning_rate': 0.04962340214534651, 'num_leaves': 194, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.9952564160022936, 'colsample_bytree': 0.6565159668930729, 'reg_alpha': 0.4197282620479348, 'reg_lambda': 4.6341255534723, 'min_split_gain': 0.023054014602758213}. Best is trial 0 with value: 0.0023314982681809284.
[Trial 0] New best: 0.00233
[I 2025-06-15 20:15:48,553] Trial 1 finished with value: 0.0030815660655100947 and parameters: {'n_estimators': 297, 'learning_rate': 0.05922586235547097, 'num_leaves': 136, 'max_depth': 7, 'min_child_samples': 4, 'subsample': 0.908867026124289, 'colsample_bytree': 0.8169344890254052, 'reg_alpha': 7.783471717189347, 'reg_lambda': 5.143220554908224, 'min_split_gain': 0.039327707898712205}. Best is trial 0 with value: 0.0023314982681809284.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:15:49,773] Trial 2 finished with value: 0.0031881852527675804 and parameters: {'n_estimators': 291, 'learning_rate': 0.021632901919475166, 'num_leaves': 242, 'max_depth': 5, 'min_child_samples': 24, 'subsample': 0.7557923462315153, 'colsample_bytree': 0.829997781941484, 'reg_alpha': 6.070634089993298, 'reg_lambda': 9.711755083415547, 'min_split_gain': 0.01908947036410781}. Best is trial 0 with value: 0.0023314982681809284.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:15:56,934] Trial 3 finished with value: 0.002132699505682342 and parameters: {'n_estimators': 275, 'learning_rate': 0.023408580346306322, 'num_leaves': 161, 'max_depth': 14, 'min_child_samples': 20, 'subsample': 0.8639248934874535, 'colsample_bytree': 0.7987729269452264, 'reg_alpha': 0.5010962900798455, 'reg_lambda': 1.3150047606371684, 'min_split_gain': 0.0028099726824616757}. Best is trial 3 with value: 0.002132699505682342.
[I 2025-06-15 20:15:56,936] A new study created in memory with name: no-name-7525bb23-7555-4c80-8953-3b61ca06aa3b
[I 2025-06-15 20:15:56,989] Trial 0 finished with value: 0.017632561619257765 and parameters: {'n_estimators': 133, 'learning_rate': 0.12728291112768605, 'num_leaves': 122, 'max_depth': 12, 'min_child_samples': 24, 'subsample': 0.5470451628141755, 'colsample_bytree': 0.6165759276807725, 'reg_alpha': 0.6885749817529618, 'reg_lambda': 8.340091550622148, 'min_split_gain': 0.009519704802243973}. Best is trial 0 with value: 0.017632561619257765.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        6614
0           Price_lag_1        6017
14      pct_chg_Load_DA        5189
3         Load_DA_lag_0        3515
2           Price_lag_7        3100
16  roll2d_median_Price        3019
15       lag168_Load_DA        2715
5       WindOn_DA_lag_0        2703
11           NGas_lag_2        2370
10           Coal_lag_2        2027
13            EUA_lag_2        1920
12            Oil_lag_2        1829
9                  WD_7         119
7                  WD_1          88
8                  WD_6          30
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          22
15       lag168_Load_DA          12
2           Price_lag_7          10
5       WindOn_DA_lag_0          10
3         Load_DA_lag_0           8
7                  WD_1           8
1           Price_lag_2           4
14      pct_chg_Load_DA           2
13            EUA_lag_2           1
10           Coal_lag_2           1
12            Oil_lag_2           0
8                  WD_6           0
11           NGas_lag_2           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 54.72% done
********************* END NS.. 399 ****************************************************
********************* START NS ... 400 ****************************************************
[I 2025-06-15 20:16:04,792] A new study created in memory with name: no-name-0468e31a-0340-4424-bc3d-1aee832d71d7
[I 2025-06-15 20:16:05,310] Trial 0 finished with value: 0.004628515857616436 and parameters: {'n_estimators': 72, 'learning_rate': 0.10672145668552792, 'num_leaves': 40, 'max_depth': 12, 'min_child_samples': 6, 'subsample': 0.5275604902395531, 'colsample_bytree': 0.7842994858254597, 'reg_alpha': 0.1355234313674436, 'reg_lambda': 8.75761259380383, 'min_split_gain': 0.024905970670836303}. Best is trial 0 with value: 0.004628515857616436.
[Trial 0] New best: 0.00463
[I 2025-06-15 20:16:06,194] Trial 1 finished with value: 0.005082169150928334 and parameters: {'n_estimators': 190, 'learning_rate': 0.09719990424409287, 'num_leaves': 151, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.7453029686293101, 'colsample_bytree': 0.6428023005761536, 'reg_alpha': 1.2097197905858748, 'reg_lambda': 7.137783773165718, 'min_split_gain': 0.04431900612442168}. Best is trial 0 with value: 0.004628515857616436.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:16:07,089] Trial 2 finished with value: 0.007011317111903788 and parameters: {'n_estimators': 223, 'learning_rate': 0.05172615863578678, 'num_leaves': 181, 'max_depth': 5, 'min_child_samples': 10, 'subsample': 0.6270616244110827, 'colsample_bytree': 0.5841000598162325, 'reg_alpha': 7.909358447671271, 'reg_lambda': 1.2379829233455941, 'min_split_gain': 0.033250604833026894}. Best is trial 0 with value: 0.004628515857616436.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:16:07,382] Trial 3 finished with value: 0.004515359233159333 and parameters: {'n_estimators': 191, 'learning_rate': 0.23422280105525306, 'num_leaves': 129, 'max_depth': 6, 'min_child_samples': 29, 'subsample': 0.598690707538512, 'colsample_bytree': 0.9229008034220674, 'reg_alpha': 7.279351275756424, 'reg_lambda': 1.8763418965026557, 'min_split_gain': 0.02960761483259417}. Best is trial 3 with value: 0.004515359233159333.
[I 2025-06-15 20:16:07,385] A new study created in memory with name: no-name-5b88d7a1-fb34-45e3-b1f3-36a0389e1176
[I 2025-06-15 20:16:07,493] Trial 0 finished with value: 0.06297129230485395 and parameters: {'n_estimators': 158, 'learning_rate': 0.03222627257250547, 'num_leaves': 164, 'max_depth': 12, 'min_child_samples': 19, 'subsample': 0.7779894775223071, 'colsample_bytree': 0.8015706623313366, 'reg_alpha': 0.6920656086311283, 'reg_lambda': 4.841209295896853, 'min_split_gain': 0.0030839867787199274}. Best is trial 0 with value: 0.06297129230485395.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         211
14      pct_chg_Load_DA         115
1           Price_lag_2          91
3         Load_DA_lag_0          67
16  roll2d_median_Price          53
2           Price_lag_7          48
11           NGas_lag_2          47
13            EUA_lag_2          37
5       WindOn_DA_lag_0          31
10           Coal_lag_2          30
15       lag168_Load_DA          30
12            Oil_lag_2          16
9                  WD_7           3
7                  WD_1           2
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         129
15       lag168_Load_DA         122
5       WindOn_DA_lag_0          59
3         Load_DA_lag_0          46
1           Price_lag_2          42
2           Price_lag_7          34
13            EUA_lag_2          28
16  roll2d_median_Price          24
14      pct_chg_Load_DA          12
10           Coal_lag_2           8
7                  WD_1           4
12            Oil_lag_2           4
11           NGas_lag_2           1
9                  WD_7           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 54.86% done
********************* END NS.. 400 ****************************************************
********************* START NS ... 401 ****************************************************
[I 2025-06-15 20:16:09,571] A new study created in memory with name: no-name-c97ce6e3-6d1d-4b9e-b17d-450d81ba9343
[I 2025-06-15 20:16:10,129] Trial 0 finished with value: 0.0060991404289407435 and parameters: {'n_estimators': 156, 'learning_rate': 0.06872623553916136, 'num_leaves': 174, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.8009035595673077, 'colsample_bytree': 0.8604595495305045, 'reg_alpha': 5.576686770879439, 'reg_lambda': 7.432344629575923, 'min_split_gain': 0.01021858172243424}. Best is trial 0 with value: 0.0060991404289407435.
[Trial 0] New best: 0.00610
[I 2025-06-15 20:16:10,811] Trial 1 finished with value: 0.005609592728935725 and parameters: {'n_estimators': 186, 'learning_rate': 0.13461934795605007, 'num_leaves': 79, 'max_depth': 16, 'min_child_samples': 30, 'subsample': 0.5122077254687409, 'colsample_bytree': 0.9564188645357573, 'reg_alpha': 1.752546403217663, 'reg_lambda': 8.923381871079993, 'min_split_gain': 0.02653304982021798}. Best is trial 1 with value: 0.005609592728935725.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:16:13,369] Trial 2 finished with value: 0.01080601821202628 and parameters: {'n_estimators': 296, 'learning_rate': 0.010526780169198887, 'num_leaves': 125, 'max_depth': 12, 'min_child_samples': 24, 'subsample': 0.5077414257390251, 'colsample_bytree': 0.6842895007951602, 'reg_alpha': 4.932172981801334, 'reg_lambda': 8.650448390333796, 'min_split_gain': 0.04842061192933278}. Best is trial 1 with value: 0.005609592728935725.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:16:14,048] Trial 3 finished with value: 0.005782051995332123 and parameters: {'n_estimators': 285, 'learning_rate': 0.1413709791274082, 'num_leaves': 119, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.5650394340573022, 'colsample_bytree': 0.9934633602664643, 'reg_alpha': 0.9110145716036577, 'reg_lambda': 3.5123356712839913, 'min_split_gain': 0.04166792625131551}. Best is trial 1 with value: 0.005609592728935725.
[I 2025-06-15 20:16:14,050] A new study created in memory with name: no-name-2960aa48-96fa-413d-970d-57540bf2c8b5
[I 2025-06-15 20:16:14,100] Trial 0 finished with value: 0.04740963247721191 and parameters: {'n_estimators': 181, 'learning_rate': 0.20904493024015777, 'num_leaves': 140, 'max_depth': 13, 'min_child_samples': 16, 'subsample': 0.813979356785945, 'colsample_bytree': 0.8702747887368442, 'reg_alpha': 0.4515504402195136, 'reg_lambda': 7.034529922140282, 'min_split_gain': 0.035469576079085956}. Best is trial 0 with value: 0.04740963247721191.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         619
14      pct_chg_Load_DA         497
1           Price_lag_2         431
3         Load_DA_lag_0         306
5       WindOn_DA_lag_0         275
16  roll2d_median_Price         239
2           Price_lag_7         197
11           NGas_lag_2         191
15       lag168_Load_DA         186
10           Coal_lag_2         127
13            EUA_lag_2         106
12            Oil_lag_2          88
9                  WD_7          13
7                  WD_1           9
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          17
3         Load_DA_lag_0           4
15       lag168_Load_DA           4
16  roll2d_median_Price           3
5       WindOn_DA_lag_0           1
1           Price_lag_2           1
10           Coal_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 54.99% done
********************* END NS.. 401 ****************************************************
********************* START NS ... 402 ****************************************************
[I 2025-06-15 20:16:16,187] A new study created in memory with name: no-name-d6fca5e3-7596-43ed-b273-4c115c71ea49
[I 2025-06-15 20:16:16,462] Trial 0 finished with value: 0.0038911621128781327 and parameters: {'n_estimators': 170, 'learning_rate': 0.2730987747799622, 'num_leaves': 46, 'max_depth': 10, 'min_child_samples': 7, 'subsample': 0.6637373805766911, 'colsample_bytree': 0.8266813737773322, 'reg_alpha': 8.032891281082327, 'reg_lambda': 9.285557755710839, 'min_split_gain': 0.014775797615253489}. Best is trial 0 with value: 0.0038911621128781327.
[Trial 0] New best: 0.00389
[I 2025-06-15 20:16:17,089] Trial 1 finished with value: 0.02024965158116054 and parameters: {'n_estimators': 54, 'learning_rate': 0.02953786184166601, 'num_leaves': 167, 'max_depth': 9, 'min_child_samples': 16, 'subsample': 0.5108555655083116, 'colsample_bytree': 0.8529066905633471, 'reg_alpha': 2.8456118281600116, 'reg_lambda': 5.096404970488344, 'min_split_gain': 0.04743331705415334}. Best is trial 0 with value: 0.0038911621128781327.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:16:18,327] Trial 2 finished with value: 0.004834626417466729 and parameters: {'n_estimators': 106, 'learning_rate': 0.03859073653937699, 'num_leaves': 56, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.9620809071701903, 'colsample_bytree': 0.5579648926600475, 'reg_alpha': 0.7802935526079524, 'reg_lambda': 0.33370260623138415, 'min_split_gain': 0.014822086698809873}. Best is trial 0 with value: 0.0038911621128781327.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:16:20,387] Trial 3 finished with value: 0.005970991437056705 and parameters: {'n_estimators': 237, 'learning_rate': 0.0125908866453358, 'num_leaves': 218, 'max_depth': 15, 'min_child_samples': 21, 'subsample': 0.7542069175709707, 'colsample_bytree': 0.7259749670436574, 'reg_alpha': 6.778754477728393, 'reg_lambda': 2.472082727134015, 'min_split_gain': 0.025593695795525558}. Best is trial 0 with value: 0.0038911621128781327.
[I 2025-06-15 20:16:20,390] A new study created in memory with name: no-name-046caf4e-3759-4a1a-9dd7-0bac41e7575c
[I 2025-06-15 20:16:20,432] Trial 0 finished with value: 0.03927242854146851 and parameters: {'n_estimators': 149, 'learning_rate': 0.11577035989941638, 'num_leaves': 80, 'max_depth': 10, 'min_child_samples': 1, 'subsample': 0.6494464258953823, 'colsample_bytree': 0.7830817923204887, 'reg_alpha': 9.485548064880279, 'reg_lambda': 1.3572561463742105, 'min_split_gain': 0.002014362813852838}. Best is trial 0 with value: 0.03927242854146851.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         213
1           Price_lag_2         151
14      pct_chg_Load_DA         133
3         Load_DA_lag_0          76
2           Price_lag_7          69
11           NGas_lag_2          68
5       WindOn_DA_lag_0          62
13            EUA_lag_2          62
15       lag168_Load_DA          61
16  roll2d_median_Price          58
10           Coal_lag_2          37
12            Oil_lag_2          19
9                  WD_7          10
8                  WD_6           6
7                  WD_1           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 55.13% done
********************* END NS.. 402 ****************************************************
********************* START NS ... 403 ****************************************************
[I 2025-06-15 20:16:22,131] A new study created in memory with name: no-name-7ba555c9-2f4f-454b-9446-1258006f5555
[I 2025-06-15 20:16:22,643] Trial 0 finished with value: 0.017298213899970962 and parameters: {'n_estimators': 90, 'learning_rate': 0.01469922622154705, 'num_leaves': 92, 'max_depth': 15, 'min_child_samples': 30, 'subsample': 0.931486674282846, 'colsample_bytree': 0.7960572033881201, 'reg_alpha': 8.331786177467897, 'reg_lambda': 9.477272196706522, 'min_split_gain': 0.048816400241479915}. Best is trial 0 with value: 0.017298213899970962.
[Trial 0] New best: 0.01730
[I 2025-06-15 20:16:23,753] Trial 1 finished with value: 0.004352899701439168 and parameters: {'n_estimators': 239, 'learning_rate': 0.153865898565477, 'num_leaves': 233, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.7655651138760812, 'colsample_bytree': 0.5493156834448114, 'reg_alpha': 2.090129977612315, 'reg_lambda': 9.09140615643878, 'min_split_gain': 0.01632730181791773}. Best is trial 1 with value: 0.004352899701439168.
[Trial 1] New best: 0.00435
[I 2025-06-15 20:16:24,677] Trial 2 finished with value: 0.004299058920908275 and parameters: {'n_estimators': 105, 'learning_rate': 0.042169547730020754, 'num_leaves': 239, 'max_depth': 12, 'min_child_samples': 23, 'subsample': 0.8113949532434055, 'colsample_bytree': 0.7651580217522984, 'reg_alpha': 7.490323621299197, 'reg_lambda': 8.779952480814101, 'min_split_gain': 0.009058023541207423}. Best is trial 2 with value: 0.004299058920908275.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:16:25,399] Trial 3 finished with value: 0.0070451252449562594 and parameters: {'n_estimators': 68, 'learning_rate': 0.032412323055206096, 'num_leaves': 90, 'max_depth': 12, 'min_child_samples': 24, 'subsample': 0.9614410664588087, 'colsample_bytree': 0.5067262206442349, 'reg_alpha': 3.633177903737658, 'reg_lambda': 4.75386401940531, 'min_split_gain': 0.038480269302032605}. Best is trial 2 with value: 0.004299058920908275.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:16:26,070] Trial 4 finished with value: 0.0048867441431511205 and parameters: {'n_estimators': 77, 'learning_rate': 0.27392319498275214, 'num_leaves': 155, 'max_depth': 13, 'min_child_samples': 8, 'subsample': 0.6505009768709774, 'colsample_bytree': 0.5060287610361808, 'reg_alpha': 5.726735902865032, 'reg_lambda': 8.449211256142895, 'min_split_gain': 0.004828110333282315}. Best is trial 2 with value: 0.004299058920908275.
[I 2025-06-15 20:16:26,073] A new study created in memory with name: no-name-48ea7030-5e00-4a74-a932-e35dc0c4d80e
[I 2025-06-15 20:16:26,134] Trial 0 finished with value: 0.08763179356305299 and parameters: {'n_estimators': 217, 'learning_rate': 0.06020354435105843, 'num_leaves': 127, 'max_depth': 16, 'min_child_samples': 18, 'subsample': 0.8517577619190879, 'colsample_bytree': 0.6511071299842139, 'reg_alpha': 3.152611076049583, 'reg_lambda': 4.572950752475585, 'min_split_gain': 0.012376776798199984}. Best is trial 0 with value: 0.08763179356305299.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         961
1           Price_lag_2         752
14      pct_chg_Load_DA         552
2           Price_lag_7         289
3         Load_DA_lag_0         274
16  roll2d_median_Price         251
11           NGas_lag_2         189
13            EUA_lag_2         154
15       lag168_Load_DA         135
10           Coal_lag_2         119
5       WindOn_DA_lag_0         116
12            Oil_lag_2         116
9                  WD_7          24
8                  WD_6           9
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          72
3         Load_DA_lag_0          29
15       lag168_Load_DA          19
5       WindOn_DA_lag_0          10
16  roll2d_median_Price           6
12            Oil_lag_2           5
1           Price_lag_2           4
11           NGas_lag_2           1
13            EUA_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 55.27% done
********************* END NS.. 403 ****************************************************
********************* START NS ... 404 ****************************************************
[I 2025-06-15 20:16:28,442] A new study created in memory with name: no-name-ea2ec5fb-213f-44fa-a612-d0c1aa549e84
[I 2025-06-15 20:16:29,611] Trial 0 finished with value: 0.009007499922976314 and parameters: {'n_estimators': 177, 'learning_rate': 0.04677705904628938, 'num_leaves': 195, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.7511768410124221, 'colsample_bytree': 0.6270792700765262, 'reg_alpha': 7.709897028047777, 'reg_lambda': 9.186186843106016, 'min_split_gain': 0.017309892560078085}. Best is trial 0 with value: 0.009007499922976314.
[Trial 0] New best: 0.00901
[I 2025-06-15 20:16:30,971] Trial 1 finished with value: 0.0070453712902056485 and parameters: {'n_estimators': 91, 'learning_rate': 0.05538618180889206, 'num_leaves': 63, 'max_depth': 13, 'min_child_samples': 22, 'subsample': 0.8508207033240698, 'colsample_bytree': 0.6265312545775463, 'reg_alpha': 0.5946215007314293, 'reg_lambda': 5.854560949793488, 'min_split_gain': 0.04729239265677862}. Best is trial 1 with value: 0.0070453712902056485.
[Trial 1] New best: 0.00705
[I 2025-06-15 20:16:31,697] Trial 2 finished with value: 0.018827273348467472 and parameters: {'n_estimators': 54, 'learning_rate': 0.02177877911962073, 'num_leaves': 124, 'max_depth': 9, 'min_child_samples': 19, 'subsample': 0.6100588879367318, 'colsample_bytree': 0.6022885711411952, 'reg_alpha': 3.606160545874161, 'reg_lambda': 9.480824097871084, 'min_split_gain': 0.017194539516679826}. Best is trial 1 with value: 0.0070453712902056485.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:16:32,442] Trial 3 finished with value: 0.006443545682852085 and parameters: {'n_estimators': 234, 'learning_rate': 0.15204543754282362, 'num_leaves': 119, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.7711332909990802, 'colsample_bytree': 0.5694310411286538, 'reg_alpha': 2.227060360575621, 'reg_lambda': 1.0432265664631868, 'min_split_gain': 0.046090858281900494}. Best is trial 3 with value: 0.006443545682852085.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:16:32,801] Trial 4 finished with value: 0.006912976823276504 and parameters: {'n_estimators': 139, 'learning_rate': 0.2426863357467088, 'num_leaves': 99, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.7283721804765527, 'colsample_bytree': 0.870132126642635, 'reg_alpha': 4.145836981177179, 'reg_lambda': 7.935144336428873, 'min_split_gain': 0.02433128312883863}. Best is trial 3 with value: 0.006443545682852085.
[I 2025-06-15 20:16:32,803] A new study created in memory with name: no-name-69c0e611-c0f4-4d9b-a598-60e2394c9a26
[I 2025-06-15 20:16:32,896] Trial 0 finished with value: 0.1743200285164981 and parameters: {'n_estimators': 230, 'learning_rate': 0.01384501537510414, 'num_leaves': 117, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.5888051456812176, 'colsample_bytree': 0.7028547198272713, 'reg_alpha': 4.402414108553318, 'reg_lambda': 3.7120305839525614, 'min_split_gain': 0.026362715771648778}. Best is trial 0 with value: 0.1743200285164981.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         416
1           Price_lag_2         398
14      pct_chg_Load_DA         242
3         Load_DA_lag_0         233
2           Price_lag_7         225
11           NGas_lag_2         209
16  roll2d_median_Price         160
15       lag168_Load_DA         157
5       WindOn_DA_lag_0         146
10           Coal_lag_2         132
13            EUA_lag_2         132
12            Oil_lag_2         103
9                  WD_7          10
7                  WD_1           7
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         244
3         Load_DA_lag_0         110
15       lag168_Load_DA          58
5       WindOn_DA_lag_0          50
12            Oil_lag_2          13
1           Price_lag_2          10
16  roll2d_median_Price           1
7                  WD_1           0
6      WindOff_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
4        Solar_DA_lag_0           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 55.40% done
********************* END NS.. 404 ****************************************************
********************* START NS ... 405 ****************************************************
[I 2025-06-15 20:16:35,174] A new study created in memory with name: no-name-837db566-da71-4dc0-a45b-5d21508f2818
[I 2025-06-15 20:16:35,570] Trial 0 finished with value: 0.006190619125785953 and parameters: {'n_estimators': 229, 'learning_rate': 0.2132281768288742, 'num_leaves': 148, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.8133189488604429, 'colsample_bytree': 0.9527918117226166, 'reg_alpha': 7.15803629659737, 'reg_lambda': 4.720824883900406, 'min_split_gain': 0.0027566087089456962}. Best is trial 0 with value: 0.006190619125785953.
[Trial 0] New best: 0.00619
[I 2025-06-15 20:16:35,850] Trial 1 finished with value: 0.004878307713968665 and parameters: {'n_estimators': 190, 'learning_rate': 0.22924869036921183, 'num_leaves': 217, 'max_depth': 16, 'min_child_samples': 30, 'subsample': 0.9258970456038418, 'colsample_bytree': 0.6460522784275096, 'reg_alpha': 7.37513809015966, 'reg_lambda': 4.279845461200068, 'min_split_gain': 0.0389309212909426}. Best is trial 1 with value: 0.004878307713968665.
[Trial 1] New best: 0.00488
[I 2025-06-15 20:16:36,366] Trial 2 finished with value: 0.008568512803087602 and parameters: {'n_estimators': 123, 'learning_rate': 0.20100216999565773, 'num_leaves': 74, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.7898256338893367, 'colsample_bytree': 0.5731387364815543, 'reg_alpha': 3.528140405618748, 'reg_lambda': 1.8730990410440418, 'min_split_gain': 0.03484989732361247}. Best is trial 1 with value: 0.004878307713968665.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:16:37,447] Trial 3 finished with value: 0.006339773337798743 and parameters: {'n_estimators': 220, 'learning_rate': 0.045147348104823445, 'num_leaves': 229, 'max_depth': 16, 'min_child_samples': 10, 'subsample': 0.932625625076086, 'colsample_bytree': 0.8615382101851249, 'reg_alpha': 7.96363383509303, 'reg_lambda': 3.795718218648015, 'min_split_gain': 0.03911874168575936}. Best is trial 1 with value: 0.004878307713968665.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:16:38,103] Trial 4 finished with value: 0.015030825180678864 and parameters: {'n_estimators': 87, 'learning_rate': 0.01578501971400687, 'num_leaves': 48, 'max_depth': 11, 'min_child_samples': 23, 'subsample': 0.6555768678809191, 'colsample_bytree': 0.8136849222777287, 'reg_alpha': 7.226175791155632, 'reg_lambda': 7.661858050508409, 'min_split_gain': 0.007567758816152126}. Best is trial 1 with value: 0.004878307713968665.
[I 2025-06-15 20:16:38,105] A new study created in memory with name: no-name-4f05406b-31d4-44a3-a29a-e2590ce4a4e4
[I 2025-06-15 20:16:38,128] Trial 0 finished with value: 0.10725594404582557 and parameters: {'n_estimators': 51, 'learning_rate': 0.2620689820934551, 'num_leaves': 83, 'max_depth': 14, 'min_child_samples': 13, 'subsample': 0.8367752696861311, 'colsample_bytree': 0.7332919206977466, 'reg_alpha': 9.996167018399289, 'reg_lambda': 6.816836337905487, 'min_split_gain': 0.01380400135529788}. Best is trial 0 with value: 0.10725594404582557.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         210
1           Price_lag_2         117
14      pct_chg_Load_DA          81
3         Load_DA_lag_0          63
11           NGas_lag_2          52
16  roll2d_median_Price          48
2           Price_lag_7          43
5       WindOn_DA_lag_0          40
13            EUA_lag_2          35
15       lag168_Load_DA          35
10           Coal_lag_2          34
12            Oil_lag_2          11
7                  WD_1           5
9                  WD_7           5
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           8
3         Load_DA_lag_0           2
12            Oil_lag_2           2
5       WindOn_DA_lag_0           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 55.54% done
********************* END NS.. 405 ****************************************************
********************* START NS ... 406 ****************************************************
[I 2025-06-15 20:16:40,308] A new study created in memory with name: no-name-a7690c99-0043-44ad-b932-fe973af92099
[I 2025-06-15 20:16:40,648] Trial 0 finished with value: 0.0031102339637828994 and parameters: {'n_estimators': 182, 'learning_rate': 0.137798592412464, 'num_leaves': 39, 'max_depth': 6, 'min_child_samples': 19, 'subsample': 0.7685496615606191, 'colsample_bytree': 0.860236242503984, 'reg_alpha': 5.844510530376827, 'reg_lambda': 0.40187728483494944, 'min_split_gain': 0.009416557192385705}. Best is trial 0 with value: 0.0031102339637828994.
[Trial 0] New best: 0.00311
[I 2025-06-15 20:16:41,079] Trial 1 finished with value: 0.002804247305452288 and parameters: {'n_estimators': 72, 'learning_rate': 0.07732822559834604, 'num_leaves': 141, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.6078943791136016, 'colsample_bytree': 0.7027986461032881, 'reg_alpha': 1.3344265213634532, 'reg_lambda': 0.9948818207553667, 'min_split_gain': 0.027132205292365486}. Best is trial 1 with value: 0.002804247305452288.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:16:41,944] Trial 2 finished with value: 0.002015946438130928 and parameters: {'n_estimators': 297, 'learning_rate': 0.01891514084818123, 'num_leaves': 126, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.7926848003487363, 'colsample_bytree': 0.8499502807953572, 'reg_alpha': 3.5399601838855155, 'reg_lambda': 8.982877554375762, 'min_split_gain': 0.049073132882352}. Best is trial 2 with value: 0.002015946438130928.
[Trial 2] New best: 0.00202
[I 2025-06-15 20:16:42,369] Trial 3 finished with value: 0.07043866180921568 and parameters: {'n_estimators': 51, 'learning_rate': 0.01055720142487066, 'num_leaves': 164, 'max_depth': 15, 'min_child_samples': 16, 'subsample': 0.7857557148286928, 'colsample_bytree': 0.8601993857173917, 'reg_alpha': 6.494355799544051, 'reg_lambda': 2.145620371359125, 'min_split_gain': 0.03533737948657241}. Best is trial 2 with value: 0.002015946438130928.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:16:43,322] Trial 4 finished with value: 0.006022155339402866 and parameters: {'n_estimators': 98, 'learning_rate': 0.026335378509081152, 'num_leaves': 224, 'max_depth': 8, 'min_child_samples': 10, 'subsample': 0.7159172810255721, 'colsample_bytree': 0.5259847139499326, 'reg_alpha': 5.051366315083591, 'reg_lambda': 3.514201058561719, 'min_split_gain': 0.020836122620402547}. Best is trial 2 with value: 0.002015946438130928.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:16:44,747] Trial 5 finished with value: 0.003721826398478935 and parameters: {'n_estimators': 217, 'learning_rate': 0.04577248226351814, 'num_leaves': 200, 'max_depth': 11, 'min_child_samples': 15, 'subsample': 0.5583716443296549, 'colsample_bytree': 0.9220682620669052, 'reg_alpha': 3.9642192444906366, 'reg_lambda': 8.458000947791911, 'min_split_gain': 0.045698430011222355}. Best is trial 2 with value: 0.002015946438130928.
[I 2025-06-15 20:16:44,750] A new study created in memory with name: no-name-fbd6696b-6d92-40c0-bbb3-99cd2c1adcf1
[I 2025-06-15 20:16:44,793] Trial 0 finished with value: 0.0032384574331138145 and parameters: {'n_estimators': 87, 'learning_rate': 0.14792950036906985, 'num_leaves': 133, 'max_depth': 13, 'min_child_samples': 7, 'subsample': 0.9777251099563233, 'colsample_bytree': 0.9144167587585953, 'reg_alpha': 1.5499621163955968, 'reg_lambda': 9.179582696568357, 'min_split_gain': 0.020102318897246525}. Best is trial 0 with value: 0.0032384574331138145.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1690
14      pct_chg_Load_DA         651
1           Price_lag_2         458
3         Load_DA_lag_0         231
2           Price_lag_7         205
11           NGas_lag_2         205
16  roll2d_median_Price         189
15       lag168_Load_DA         107
13            EUA_lag_2          92
12            Oil_lag_2          82
5       WindOn_DA_lag_0          50
10           Coal_lag_2          48
9                  WD_7          15
8                  WD_6           2
7                  WD_1           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          42
3         Load_DA_lag_0          19
5       WindOn_DA_lag_0           6
15       lag168_Load_DA           4
12            Oil_lag_2           2
16  roll2d_median_Price           2
2           Price_lag_7           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
4        Solar_DA_lag_0           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 55.68% done
********************* END NS.. 406 ****************************************************
********************* START NS ... 407 ****************************************************
[I 2025-06-15 20:16:47,218] A new study created in memory with name: no-name-7c07a31b-bf24-4f61-b61f-1e3d27b49909
[I 2025-06-15 20:16:48,696] Trial 0 finished with value: 0.00222276914970101 and parameters: {'n_estimators': 256, 'learning_rate': 0.027116354410540333, 'num_leaves': 123, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.7539473614394712, 'colsample_bytree': 0.5743661464263696, 'reg_alpha': 6.4192560580480675, 'reg_lambda': 0.48302236687858935, 'min_split_gain': 0.021631788072640885}. Best is trial 0 with value: 0.00222276914970101.
[Trial 0] New best: 0.00222
[I 2025-06-15 20:16:50,809] Trial 1 finished with value: 0.0020325277587381996 and parameters: {'n_estimators': 130, 'learning_rate': 0.024759933985996633, 'num_leaves': 133, 'max_depth': 12, 'min_child_samples': 14, 'subsample': 0.87056758120277, 'colsample_bytree': 0.863738041791341, 'reg_alpha': 4.055778523236083, 'reg_lambda': 9.577140371501258, 'min_split_gain': 0.007641434536760089}. Best is trial 1 with value: 0.0020325277587381996.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:16:51,033] Trial 2 finished with value: 0.001763346150085997 and parameters: {'n_estimators': 64, 'learning_rate': 0.14918303183082088, 'num_leaves': 77, 'max_depth': 4, 'min_child_samples': 11, 'subsample': 0.9597023877735162, 'colsample_bytree': 0.9484700924957827, 'reg_alpha': 1.932947055357035, 'reg_lambda': 3.1087774824019556, 'min_split_gain': 0.029403460637943538}. Best is trial 2 with value: 0.001763346150085997.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:16:51,804] Trial 3 finished with value: 0.0018575648356218182 and parameters: {'n_estimators': 294, 'learning_rate': 0.10208261004085657, 'num_leaves': 197, 'max_depth': 11, 'min_child_samples': 28, 'subsample': 0.6030779601139278, 'colsample_bytree': 0.7884198875616928, 'reg_alpha': 5.9430770710595, 'reg_lambda': 8.216007546327136, 'min_split_gain': 0.014949685236622323}. Best is trial 2 with value: 0.001763346150085997.
[I 2025-06-15 20:16:51,806] A new study created in memory with name: no-name-be0525d4-47d8-4dec-b8ec-5f5c199f5bd4
[I 2025-06-15 20:16:51,855] Trial 0 finished with value: 0.008554414955013719 and parameters: {'n_estimators': 169, 'learning_rate': 0.2712594574782696, 'num_leaves': 192, 'max_depth': 4, 'min_child_samples': 9, 'subsample': 0.7059930249208607, 'colsample_bytree': 0.5914468632972278, 'reg_alpha': 8.90739407810316, 'reg_lambda': 9.91176408794333, 'min_split_gain': 0.01944650022376411}. Best is trial 0 with value: 0.008554414955013719.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         264
14      pct_chg_Load_DA         134
1           Price_lag_2          89
3         Load_DA_lag_0          78
16  roll2d_median_Price          58
2           Price_lag_7          56
11           NGas_lag_2          47
15       lag168_Load_DA          36
5       WindOn_DA_lag_0          34
12            Oil_lag_2          34
13            EUA_lag_2          29
10           Coal_lag_2          20
7                  WD_1           5
9                  WD_7           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           3
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 55.81% done
********************* END NS.. 407 ****************************************************
********************* START NS ... 408 ****************************************************
[I 2025-06-15 20:16:53,565] A new study created in memory with name: no-name-cf88b892-5740-4ce7-91ca-fead1e5fbb31
[I 2025-06-15 20:16:54,329] Trial 0 finished with value: 0.0051628541654672455 and parameters: {'n_estimators': 265, 'learning_rate': 0.03345244764056956, 'num_leaves': 224, 'max_depth': 4, 'min_child_samples': 15, 'subsample': 0.5728982331132177, 'colsample_bytree': 0.555098588863657, 'reg_alpha': 1.4029644741049774, 'reg_lambda': 9.380075835591851, 'min_split_gain': 0.03358580995923065}. Best is trial 0 with value: 0.0051628541654672455.
[Trial 0] New best: 0.00516
[I 2025-06-15 20:16:54,827] Trial 1 finished with value: 0.00343668040207692 and parameters: {'n_estimators': 219, 'learning_rate': 0.13664672208203588, 'num_leaves': 33, 'max_depth': 16, 'min_child_samples': 25, 'subsample': 0.8564628409010542, 'colsample_bytree': 0.67683933871471, 'reg_alpha': 7.320707762172349, 'reg_lambda': 8.352443190321818, 'min_split_gain': 0.034724605239156665}. Best is trial 1 with value: 0.00343668040207692.
[Trial 1] New best: 0.00344
[I 2025-06-15 20:16:55,464] Trial 2 finished with value: 0.003229243750937598 and parameters: {'n_estimators': 300, 'learning_rate': 0.22911960595598513, 'num_leaves': 223, 'max_depth': 11, 'min_child_samples': 11, 'subsample': 0.7384858081004534, 'colsample_bytree': 0.6830366225003469, 'reg_alpha': 3.58050824945228, 'reg_lambda': 1.4405717725792166, 'min_split_gain': 0.013558098073642633}. Best is trial 2 with value: 0.003229243750937598.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:16:55,715] Trial 3 finished with value: 0.0034247741271745285 and parameters: {'n_estimators': 71, 'learning_rate': 0.2457982622324983, 'num_leaves': 47, 'max_depth': 15, 'min_child_samples': 12, 'subsample': 0.5370680160702204, 'colsample_bytree': 0.793750699382868, 'reg_alpha': 7.42488579042017, 'reg_lambda': 3.282645709516699, 'min_split_gain': 0.031663322597300446}. Best is trial 2 with value: 0.003229243750937598.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:16:56,200] Trial 4 finished with value: 0.0030405744063014294 and parameters: {'n_estimators': 187, 'learning_rate': 0.2657159988974807, 'num_leaves': 84, 'max_depth': 9, 'min_child_samples': 15, 'subsample': 0.9581076255424964, 'colsample_bytree': 0.5695439705442704, 'reg_alpha': 8.585316025587423, 'reg_lambda': 3.3469559073118935, 'min_split_gain': 0.0076126343843708425}. Best is trial 4 with value: 0.0030405744063014294.
[I 2025-06-15 20:16:56,204] A new study created in memory with name: no-name-b4f2058b-a8fa-4e58-a117-9a5cb0007233
[I 2025-06-15 20:16:56,260] Trial 0 finished with value: 0.008397103917944161 and parameters: {'n_estimators': 80, 'learning_rate': 0.027368451605246296, 'num_leaves': 214, 'max_depth': 15, 'min_child_samples': 23, 'subsample': 0.8395605422055537, 'colsample_bytree': 0.6709171876262578, 'reg_alpha': 1.9658796387025868, 'reg_lambda': 7.345345426534452, 'min_split_gain': 0.02180129452349256}. Best is trial 0 with value: 0.008397103917944161.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         265
1           Price_lag_2         244
3         Load_DA_lag_0         130
14      pct_chg_Load_DA         111
2           Price_lag_7         100
5       WindOn_DA_lag_0          94
11           NGas_lag_2          93
16  roll2d_median_Price          83
15       lag168_Load_DA          72
13            EUA_lag_2          64
10           Coal_lag_2          53
12            Oil_lag_2          29
9                  WD_7          14
7                  WD_1           6
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          89
3         Load_DA_lag_0          29
5       WindOn_DA_lag_0          22
15       lag168_Load_DA           9
1           Price_lag_2           6
2           Price_lag_7           4
12            Oil_lag_2           3
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 55.95% done
********************* END NS.. 408 ****************************************************
********************* START NS ... 409 ****************************************************
[I 2025-06-15 20:16:58,562] A new study created in memory with name: no-name-94ff5aea-1b23-4cf9-a074-c8f5f32ba51a
[I 2025-06-15 20:16:59,199] Trial 0 finished with value: 0.00554746359461926 and parameters: {'n_estimators': 57, 'learning_rate': 0.18958866353181356, 'num_leaves': 162, 'max_depth': 15, 'min_child_samples': 28, 'subsample': 0.6744364509115568, 'colsample_bytree': 0.6231582669992538, 'reg_alpha': 4.8632332076122395, 'reg_lambda': 0.3157203201180725, 'min_split_gain': 0.002399426649030162}. Best is trial 0 with value: 0.00554746359461926.
[Trial 0] New best: 0.00555
[I 2025-06-15 20:16:59,740] Trial 1 finished with value: 0.005400832765080463 and parameters: {'n_estimators': 216, 'learning_rate': 0.12228818482367619, 'num_leaves': 77, 'max_depth': 9, 'min_child_samples': 23, 'subsample': 0.5111751409856105, 'colsample_bytree': 0.8393018267016115, 'reg_alpha': 8.566209564821008, 'reg_lambda': 0.7368918065099594, 'min_split_gain': 0.0182609882076942}. Best is trial 1 with value: 0.005400832765080463.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:17:01,144] Trial 2 finished with value: 0.015168581178239207 and parameters: {'n_estimators': 108, 'learning_rate': 0.013379796346844969, 'num_leaves': 244, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.5334840252105686, 'colsample_bytree': 0.6082570851148622, 'reg_alpha': 5.560577733983286, 'reg_lambda': 5.643741107045885, 'min_split_gain': 0.022281155135896343}. Best is trial 1 with value: 0.005400832765080463.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:17:02,503] Trial 3 finished with value: 0.005504045621235839 and parameters: {'n_estimators': 144, 'learning_rate': 0.1914218691655723, 'num_leaves': 149, 'max_depth': 9, 'min_child_samples': 13, 'subsample': 0.8163950032896461, 'colsample_bytree': 0.5930932669503695, 'reg_alpha': 7.5705938116952884, 'reg_lambda': 4.295278380150299, 'min_split_gain': 0.007585016486756336}. Best is trial 1 with value: 0.005400832765080463.
[I 2025-06-15 20:17:02,507] A new study created in memory with name: no-name-e9c299ad-fd65-4065-9a16-4cba6107650e
[I 2025-06-15 20:17:02,549] Trial 0 finished with value: 0.017157288342470593 and parameters: {'n_estimators': 85, 'learning_rate': 0.08175688405446414, 'num_leaves': 233, 'max_depth': 7, 'min_child_samples': 30, 'subsample': 0.6075503043385418, 'colsample_bytree': 0.9313744936768688, 'reg_alpha': 7.196451541433003, 'reg_lambda': 3.916458083669123, 'min_split_gain': 0.013240704849250258}. Best is trial 0 with value: 0.017157288342470593.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         413
14      pct_chg_Load_DA         254
1           Price_lag_2         253
11           NGas_lag_2         150
3         Load_DA_lag_0         134
2           Price_lag_7         126
16  roll2d_median_Price         124
5       WindOn_DA_lag_0         108
15       lag168_Load_DA          93
13            EUA_lag_2          82
10           Coal_lag_2          69
12            Oil_lag_2          51
9                  WD_7          12
7                  WD_1           8
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          14
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 56.09% done
********************* END NS.. 409 ****************************************************
********************* START NS ... 410 ****************************************************
[I 2025-06-15 20:17:05,072] A new study created in memory with name: no-name-c5ff2c3c-5aeb-4a96-baae-56df7d67dd97
[I 2025-06-15 20:17:06,003] Trial 0 finished with value: 0.003518010517679023 and parameters: {'n_estimators': 80, 'learning_rate': 0.041815505892754415, 'num_leaves': 92, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.5056313316407577, 'colsample_bytree': 0.9427799074456775, 'reg_alpha': 9.886419885825799, 'reg_lambda': 2.084022778028177, 'min_split_gain': 0.006823776334145615}. Best is trial 0 with value: 0.003518010517679023.
[Trial 0] New best: 0.00352
[I 2025-06-15 20:17:06,420] Trial 1 finished with value: 0.0031020815904264053 and parameters: {'n_estimators': 111, 'learning_rate': 0.2898118125416562, 'num_leaves': 215, 'max_depth': 14, 'min_child_samples': 22, 'subsample': 0.9011614521313036, 'colsample_bytree': 0.6724984438395032, 'reg_alpha': 2.8809890948519365, 'reg_lambda': 8.236400665908095, 'min_split_gain': 0.017855681352492703}. Best is trial 1 with value: 0.0031020815904264053.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:17:08,988] Trial 2 finished with value: 0.004231841835700518 and parameters: {'n_estimators': 218, 'learning_rate': 0.015739126078572238, 'num_leaves': 215, 'max_depth': 10, 'min_child_samples': 3, 'subsample': 0.834942219035239, 'colsample_bytree': 0.6008938161396387, 'reg_alpha': 7.231405917691568, 'reg_lambda': 4.385427834239413, 'min_split_gain': 0.023131272652441356}. Best is trial 1 with value: 0.0031020815904264053.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:17:10,855] Trial 3 finished with value: 0.004378009110488619 and parameters: {'n_estimators': 197, 'learning_rate': 0.012980997958201011, 'num_leaves': 128, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.9926103627770954, 'colsample_bytree': 0.9965628263916811, 'reg_alpha': 8.92417082884265, 'reg_lambda': 1.8983237180838264, 'min_split_gain': 0.01074620659546003}. Best is trial 1 with value: 0.0031020815904264053.
[I 2025-06-15 20:17:10,860] A new study created in memory with name: no-name-e79f0e3d-c62c-4bdc-a590-7e33b5f3c9c1
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 20:17:11,076] Trial 0 finished with value: 0.011901357037955348 and parameters: {'n_estimators': 232, 'learning_rate': 0.18637270653990265, 'num_leaves': 98, 'max_depth': 7, 'min_child_samples': 14, 'subsample': 0.7037475645864224, 'colsample_bytree': 0.9443826316933405, 'reg_alpha': 8.83022369956862, 'reg_lambda': 3.8775902976361, 'min_split_gain': 0.04759848604546145}. Best is trial 0 with value: 0.011901357037955348.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         269
1           Price_lag_2         213
3         Load_DA_lag_0         167
14      pct_chg_Load_DA         120
11           NGas_lag_2         107
15       lag168_Load_DA         106
5       WindOn_DA_lag_0         104
16  roll2d_median_Price          99
2           Price_lag_7          99
10           Coal_lag_2          65
13            EUA_lag_2          44
12            Oil_lag_2          35
7                  WD_1           8
8                  WD_6           4
9                  WD_7           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 56.22% done
********************* END NS.. 410 ****************************************************
********************* START NS ... 411 ****************************************************
[I 2025-06-15 20:17:13,631] A new study created in memory with name: no-name-f5904ec2-c8c0-4755-9de3-cf7fee354a34
[I 2025-06-15 20:17:16,040] Trial 0 finished with value: 0.0018908337906213511 and parameters: {'n_estimators': 186, 'learning_rate': 0.10541829593998851, 'num_leaves': 129, 'max_depth': 7, 'min_child_samples': 3, 'subsample': 0.8606541519807621, 'colsample_bytree': 0.6008126740567219, 'reg_alpha': 3.389350935784784, 'reg_lambda': 3.193523000576665, 'min_split_gain': 0.005855698066374266}. Best is trial 0 with value: 0.0018908337906213511.
[Trial 0] New best: 0.00189
[I 2025-06-15 20:17:18,624] Trial 1 finished with value: 0.0016368679902167016 and parameters: {'n_estimators': 136, 'learning_rate': 0.10449942249727412, 'num_leaves': 50, 'max_depth': 10, 'min_child_samples': 20, 'subsample': 0.7633161916466833, 'colsample_bytree': 0.8973123631790134, 'reg_alpha': 3.9725259152409675, 'reg_lambda': 3.047041001927547, 'min_split_gain': 0.03132688617187222}. Best is trial 1 with value: 0.0016368679902167016.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:17:19,399] Trial 2 finished with value: 0.0019465887890735799 and parameters: {'n_estimators': 52, 'learning_rate': 0.07633316978079387, 'num_leaves': 113, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.5248054625798907, 'colsample_bytree': 0.7824348274425639, 'reg_alpha': 4.6238694050702005, 'reg_lambda': 8.824871483470192, 'min_split_gain': 0.019788360383258638}. Best is trial 1 with value: 0.0016368679902167016.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:17:23,318] Trial 3 finished with value: 0.0017836120650503553 and parameters: {'n_estimators': 166, 'learning_rate': 0.15090370614198603, 'num_leaves': 195, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.6215303442206763, 'colsample_bytree': 0.643998501271354, 'reg_alpha': 6.6108579234836835, 'reg_lambda': 9.949401575869679, 'min_split_gain': 0.03068956393669168}. Best is trial 1 with value: 0.0016368679902167016.
[I 2025-06-15 20:17:23,330] A new study created in memory with name: no-name-f3f6cc97-b5bf-4aca-be80-a35b48f05459
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 20:17:23,970] Trial 0 finished with value: 0.0032760933787829103 and parameters: {'n_estimators': 79, 'learning_rate': 0.21237955140306713, 'num_leaves': 212, 'max_depth': 12, 'min_child_samples': 11, 'subsample': 0.9043721251265842, 'colsample_bytree': 0.7854066264426003, 'reg_alpha': 0.6557882987296015, 'reg_lambda': 2.632353051405756, 'min_split_gain': 0.030392356177078583}. Best is trial 0 with value: 0.0032760933787829103.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         587
14      pct_chg_Load_DA         408
1           Price_lag_2         397
3         Load_DA_lag_0         239
11           NGas_lag_2         178
2           Price_lag_7         174
16  roll2d_median_Price         157
15       lag168_Load_DA         156
5       WindOn_DA_lag_0         138
10           Coal_lag_2         102
13            EUA_lag_2          97
12            Oil_lag_2          87
9                  WD_7          11
7                  WD_1           9
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          10
3         Load_DA_lag_0           4
15       lag168_Load_DA           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 56.36% done
********************* END NS.. 411 ****************************************************
********************* START NS ... 412 ****************************************************
[I 2025-06-15 20:17:31,583] A new study created in memory with name: no-name-c168e05a-b842-4349-8995-6792f0b30450
[I 2025-06-15 20:17:32,664] Trial 0 finished with value: 0.0010485386901424745 and parameters: {'n_estimators': 72, 'learning_rate': 0.16463581585669584, 'num_leaves': 90, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.5075163478162805, 'colsample_bytree': 0.9121142690965838, 'reg_alpha': 2.876115876556804, 'reg_lambda': 1.1247253799032264, 'min_split_gain': 0.02482503397813084}. Best is trial 0 with value: 0.0010485386901424745.
[Trial 0] New best: 0.00105
[I 2025-06-15 20:17:33,978] Trial 1 finished with value: 0.0017419107692927208 and parameters: {'n_estimators': 173, 'learning_rate': 0.2113223726140021, 'num_leaves': 142, 'max_depth': 6, 'min_child_samples': 29, 'subsample': 0.9553818580400997, 'colsample_bytree': 0.5288297326346025, 'reg_alpha': 4.540795394871788, 'reg_lambda': 6.0488488345711575, 'min_split_gain': 0.00832980879979311}. Best is trial 0 with value: 0.0010485386901424745.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:17:50,408] Trial 2 finished with value: 0.001306352038723864 and parameters: {'n_estimators': 283, 'learning_rate': 0.014843327507539793, 'num_leaves': 112, 'max_depth': 13, 'min_child_samples': 1, 'subsample': 0.7382495317233213, 'colsample_bytree': 0.8675903010761901, 'reg_alpha': 4.386127023888918, 'reg_lambda': 3.1397829447793315, 'min_split_gain': 0.002129480904746639}. Best is trial 0 with value: 0.0010485386901424745.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:17:51,927] Trial 3 finished with value: 0.001046479091219414 and parameters: {'n_estimators': 204, 'learning_rate': 0.29651201085508844, 'num_leaves': 215, 'max_depth': 12, 'min_child_samples': 11, 'subsample': 0.5525193842731528, 'colsample_bytree': 0.9230219504889585, 'reg_alpha': 4.5137507746946355, 'reg_lambda': 7.079044534204631, 'min_split_gain': 0.020454969796786328}. Best is trial 3 with value: 0.001046479091219414.
[I 2025-06-15 20:17:51,931] A new study created in memory with name: no-name-9cb85fe4-50aa-412e-8fcb-3d02bd41a105
[I 2025-06-15 20:17:52,022] Trial 0 finished with value: 0.012486615066025141 and parameters: {'n_estimators': 283, 'learning_rate': 0.22457999741603277, 'num_leaves': 227, 'max_depth': 10, 'min_child_samples': 9, 'subsample': 0.9822624554306939, 'colsample_bytree': 0.6610419963005361, 'reg_alpha': 3.3338404713162726, 'reg_lambda': 4.833123182639772, 'min_split_gain': 0.03246077424134507}. Best is trial 0 with value: 0.012486615066025141.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         224
14      pct_chg_Load_DA         174
1           Price_lag_2         157
3         Load_DA_lag_0         112
2           Price_lag_7          92
11           NGas_lag_2          84
16  roll2d_median_Price          81
15       lag168_Load_DA          67
5       WindOn_DA_lag_0          63
13            EUA_lag_2          48
10           Coal_lag_2          42
12            Oil_lag_2          34
9                  WD_7           7
7                  WD_1           5
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           5
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 56.50% done
********************* END NS.. 412 ****************************************************
********************* START NS ... 413 ****************************************************
[I 2025-06-15 20:17:56,859] A new study created in memory with name: no-name-eeb4ade6-78a6-4878-a9dd-1f80dfa9e738
[I 2025-06-15 20:17:57,431] Trial 0 finished with value: 0.004730531004063086 and parameters: {'n_estimators': 98, 'learning_rate': 0.15922114885620622, 'num_leaves': 117, 'max_depth': 16, 'min_child_samples': 13, 'subsample': 0.878479117837411, 'colsample_bytree': 0.8944691154845568, 'reg_alpha': 4.751122802999038, 'reg_lambda': 5.943961158934714, 'min_split_gain': 0.021998250010869465}. Best is trial 0 with value: 0.004730531004063086.
[Trial 0] New best: 0.00473
[I 2025-06-15 20:17:57,763] Trial 1 finished with value: 0.004755536571947262 and parameters: {'n_estimators': 131, 'learning_rate': 0.238298562348227, 'num_leaves': 148, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.7043435808419583, 'colsample_bytree': 0.8014302250877254, 'reg_alpha': 6.2095730200676105, 'reg_lambda': 9.750567694628124, 'min_split_gain': 0.04394972670550825}. Best is trial 0 with value: 0.004730531004063086.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:17:58,816] Trial 2 finished with value: 0.004882327304762647 and parameters: {'n_estimators': 270, 'learning_rate': 0.07929763509120626, 'num_leaves': 84, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.8471350166932046, 'colsample_bytree': 0.7249802487039179, 'reg_alpha': 6.156880229849479, 'reg_lambda': 3.469168280445457, 'min_split_gain': 0.045099159440398245}. Best is trial 0 with value: 0.004730531004063086.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:17:59,762] Trial 3 finished with value: 0.0041540955381347295 and parameters: {'n_estimators': 224, 'learning_rate': 0.2220592732175004, 'num_leaves': 109, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.7140266527983222, 'colsample_bytree': 0.5402791009485675, 'reg_alpha': 2.315861791528786, 'reg_lambda': 8.947383938202588, 'min_split_gain': 0.007209002757889277}. Best is trial 3 with value: 0.0041540955381347295.
[I 2025-06-15 20:17:59,767] A new study created in memory with name: no-name-cbbe557d-b788-456c-b30e-90573806100f
[I 2025-06-15 20:17:59,815] Trial 0 finished with value: 0.02073184216353913 and parameters: {'n_estimators': 125, 'learning_rate': 0.01183182227295971, 'num_leaves': 204, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.7463774278611114, 'colsample_bytree': 0.782668896995852, 'reg_alpha': 4.036930543484257, 'reg_lambda': 9.964115557269272, 'min_split_gain': 0.03840558994546741}. Best is trial 0 with value: 0.02073184216353913.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         588
1           Price_lag_2         520
14      pct_chg_Load_DA         340
16  roll2d_median_Price         320
3         Load_DA_lag_0         254
2           Price_lag_7         243
13            EUA_lag_2         188
10           Coal_lag_2         187
11           NGas_lag_2         158
15       lag168_Load_DA         145
5       WindOn_DA_lag_0         126
12            Oil_lag_2          90
9                  WD_7          13
8                  WD_6          11
7                  WD_1          10
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          78
3         Load_DA_lag_0           5
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 56.63% done
********************* END NS.. 413 ****************************************************
********************* START NS ... 414 ****************************************************
[I 2025-06-15 20:18:02,243] A new study created in memory with name: no-name-5046ab20-b929-4cea-b632-54a31fa7bb3f
[I 2025-06-15 20:18:02,870] Trial 0 finished with value: 0.004861377090822802 and parameters: {'n_estimators': 293, 'learning_rate': 0.15937959685454187, 'num_leaves': 162, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.9727567526154488, 'colsample_bytree': 0.6680488476020381, 'reg_alpha': 5.284131830849185, 'reg_lambda': 4.659913568839768, 'min_split_gain': 0.047218317542905036}. Best is trial 0 with value: 0.004861377090822802.
[Trial 0] New best: 0.00486
[I 2025-06-15 20:18:03,541] Trial 1 finished with value: 0.005022827801398574 and parameters: {'n_estimators': 251, 'learning_rate': 0.17471291295523966, 'num_leaves': 110, 'max_depth': 16, 'min_child_samples': 23, 'subsample': 0.6872903537376901, 'colsample_bytree': 0.8383377815985614, 'reg_alpha': 3.006535306688659, 'reg_lambda': 0.08662663123480852, 'min_split_gain': 0.018901141591842}. Best is trial 0 with value: 0.004861377090822802.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:18:04,234] Trial 2 finished with value: 0.004992423299700914 and parameters: {'n_estimators': 230, 'learning_rate': 0.16216994035925608, 'num_leaves': 128, 'max_depth': 8, 'min_child_samples': 9, 'subsample': 0.6429471727968711, 'colsample_bytree': 0.5120641735648439, 'reg_alpha': 8.812237819650674, 'reg_lambda': 1.854876709830835, 'min_split_gain': 0.024249647313426638}. Best is trial 0 with value: 0.004861377090822802.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:18:05,001] Trial 3 finished with value: 0.005169453941397923 and parameters: {'n_estimators': 157, 'learning_rate': 0.17660528745624948, 'num_leaves': 183, 'max_depth': 16, 'min_child_samples': 12, 'subsample': 0.8743938200852963, 'colsample_bytree': 0.9731634924110444, 'reg_alpha': 4.456383193830002, 'reg_lambda': 6.453333619853092, 'min_split_gain': 0.015287228364617128}. Best is trial 0 with value: 0.004861377090822802.
[I 2025-06-15 20:18:05,004] A new study created in memory with name: no-name-c83e0213-8d06-4b0f-afba-ad872c770a24
[I 2025-06-15 20:18:05,030] Trial 0 finished with value: 0.024138038377906068 and parameters: {'n_estimators': 82, 'learning_rate': 0.017162716089927574, 'num_leaves': 37, 'max_depth': 15, 'min_child_samples': 17, 'subsample': 0.5705169553373455, 'colsample_bytree': 0.7571918828577238, 'reg_alpha': 7.796195021618665, 'reg_lambda': 1.211335706198784, 'min_split_gain': 0.03584371437400704}. Best is trial 0 with value: 0.024138038377906068.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         319
1           Price_lag_2         209
14      pct_chg_Load_DA         150
3         Load_DA_lag_0         105
11           NGas_lag_2          85
2           Price_lag_7          83
16  roll2d_median_Price          78
10           Coal_lag_2          59
5       WindOn_DA_lag_0          56
15       lag168_Load_DA          51
13            EUA_lag_2          49
12            Oil_lag_2          18
9                  WD_7           6
7                  WD_1           3
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 56.77% done
********************* END NS.. 414 ****************************************************
********************* START NS ... 415 ****************************************************
[I 2025-06-15 20:18:06,878] A new study created in memory with name: no-name-623ae4e4-ce5e-4c81-8efa-d647b53694bf
[I 2025-06-15 20:18:08,417] Trial 0 finished with value: 0.00538333768642383 and parameters: {'n_estimators': 110, 'learning_rate': 0.025053514326052802, 'num_leaves': 239, 'max_depth': 12, 'min_child_samples': 10, 'subsample': 0.9936820920716432, 'colsample_bytree': 0.859697958847182, 'reg_alpha': 2.1744466818230066, 'reg_lambda': 6.727297327770661, 'min_split_gain': 0.03343292955391634}. Best is trial 0 with value: 0.00538333768642383.
[Trial 0] New best: 0.00538
[I 2025-06-15 20:18:09,467] Trial 1 finished with value: 0.003041917328145629 and parameters: {'n_estimators': 132, 'learning_rate': 0.10736256109861565, 'num_leaves': 74, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.6272257246995389, 'colsample_bytree': 0.7793372543414814, 'reg_alpha': 0.7242879505828748, 'reg_lambda': 7.531969346284518, 'min_split_gain': 0.049962837247805834}. Best is trial 1 with value: 0.003041917328145629.
[Trial 1] New best: 0.00304
[I 2025-06-15 20:18:10,430] Trial 2 finished with value: 0.0031074723883625075 and parameters: {'n_estimators': 182, 'learning_rate': 0.14879745811467632, 'num_leaves': 206, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.9952854277259099, 'colsample_bytree': 0.7759279892600308, 'reg_alpha': 0.17625746399677844, 'reg_lambda': 5.994093761227658, 'min_split_gain': 0.037481330541153116}. Best is trial 1 with value: 0.003041917328145629.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:18:11,049] Trial 3 finished with value: 0.0029995963977787463 and parameters: {'n_estimators': 87, 'learning_rate': 0.19016240879084842, 'num_leaves': 119, 'max_depth': 8, 'min_child_samples': 19, 'subsample': 0.6285766440548478, 'colsample_bytree': 0.6936341496319491, 'reg_alpha': 0.11779044237094194, 'reg_lambda': 0.48974157717587397, 'min_split_gain': 0.0230346082282199}. Best is trial 3 with value: 0.0029995963977787463.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:18:14,889] Trial 4 finished with value: 0.0038831515362198776 and parameters: {'n_estimators': 249, 'learning_rate': 0.017890104449862186, 'num_leaves': 162, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.7309251014133242, 'colsample_bytree': 0.9698422805619802, 'reg_alpha': 3.1860104005041787, 'reg_lambda': 3.7468611277701336, 'min_split_gain': 0.00494418797819603}. Best is trial 3 with value: 0.0029995963977787463.
[I 2025-06-15 20:18:14,892] A new study created in memory with name: no-name-f814c58f-87ad-4a49-9ef6-a2424a8b9ce0
[I 2025-06-15 20:18:14,950] Trial 0 finished with value: 0.011027060512855784 and parameters: {'n_estimators': 219, 'learning_rate': 0.05435154980651305, 'num_leaves': 207, 'max_depth': 9, 'min_child_samples': 4, 'subsample': 0.704210533086018, 'colsample_bytree': 0.9072299075402122, 'reg_alpha': 1.3303394312256955, 'reg_lambda': 0.48476948280855914, 'min_split_gain': 0.02836570192032613}. Best is trial 0 with value: 0.011027060512855784.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         587
1           Price_lag_2         455
14      pct_chg_Load_DA         343
16  roll2d_median_Price         288
3         Load_DA_lag_0         266
2           Price_lag_7         232
11           NGas_lag_2         218
5       WindOn_DA_lag_0         204
10           Coal_lag_2         169
15       lag168_Load_DA         162
13            EUA_lag_2         128
12            Oil_lag_2         107
9                  WD_7          10
7                  WD_1           7
8                  WD_6           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          52
15       lag168_Load_DA          15
3         Load_DA_lag_0           3
1           Price_lag_2           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 56.91% done
********************* END NS.. 415 ****************************************************
********************* START NS ... 416 ****************************************************
[I 2025-06-15 20:18:17,125] A new study created in memory with name: no-name-b2acfe64-7eb4-4b6e-8b7b-2cdeaf4ad383
[I 2025-06-15 20:18:18,576] Trial 0 finished with value: 0.004213819063869088 and parameters: {'n_estimators': 136, 'learning_rate': 0.051328304723099905, 'num_leaves': 123, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.5259458407953643, 'colsample_bytree': 0.9706055721209175, 'reg_alpha': 3.0567561999043202, 'reg_lambda': 6.561245512192075, 'min_split_gain': 0.01632894408174379}. Best is trial 0 with value: 0.004213819063869088.
[Trial 0] New best: 0.00421
[I 2025-06-15 20:18:19,314] Trial 1 finished with value: 0.0047960082198787906 and parameters: {'n_estimators': 108, 'learning_rate': 0.15682334422035396, 'num_leaves': 78, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.8480536627441422, 'colsample_bytree': 0.5268509441284547, 'reg_alpha': 9.15836589024856, 'reg_lambda': 8.493969870696358, 'min_split_gain': 0.01778443851286079}. Best is trial 0 with value: 0.004213819063869088.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:18:20,493] Trial 2 finished with value: 0.004181464428428831 and parameters: {'n_estimators': 109, 'learning_rate': 0.08395597756195569, 'num_leaves': 155, 'max_depth': 15, 'min_child_samples': 5, 'subsample': 0.5601519063553488, 'colsample_bytree': 0.6223034094423974, 'reg_alpha': 5.295525763506591, 'reg_lambda': 0.2230476530739045, 'min_split_gain': 0.011225510492863239}. Best is trial 2 with value: 0.004181464428428831.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:18:21,054] Trial 3 finished with value: 0.0052002267836839845 and parameters: {'n_estimators': 51, 'learning_rate': 0.13140580837922983, 'num_leaves': 154, 'max_depth': 14, 'min_child_samples': 14, 'subsample': 0.7572658343010659, 'colsample_bytree': 0.5405024726381291, 'reg_alpha': 9.424992636652922, 'reg_lambda': 2.2252222058086666, 'min_split_gain': 0.004392069277972516}. Best is trial 2 with value: 0.004181464428428831.
[I 2025-06-15 20:18:21,056] A new study created in memory with name: no-name-420ad1b0-6c19-483a-9782-18c68a5ea42e
[I 2025-06-15 20:18:21,138] Trial 0 finished with value: 0.010389964521587088 and parameters: {'n_estimators': 269, 'learning_rate': 0.01982669577439609, 'num_leaves': 115, 'max_depth': 16, 'min_child_samples': 9, 'subsample': 0.8420877209197851, 'colsample_bytree': 0.9834984725533633, 'reg_alpha': 2.1905177393655215, 'reg_lambda': 1.8470972225103188, 'min_split_gain': 0.011272165188477196}. Best is trial 0 with value: 0.010389964521587088.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         816
1           Price_lag_2         812
14      pct_chg_Load_DA         566
3         Load_DA_lag_0         371
15       lag168_Load_DA         371
5       WindOn_DA_lag_0         369
11           NGas_lag_2         357
2           Price_lag_7         354
13            EUA_lag_2         353
16  roll2d_median_Price         292
10           Coal_lag_2         220
12            Oil_lag_2         168
9                  WD_7          34
8                  WD_6          24
7                  WD_1          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         121
15       lag168_Load_DA          75
3         Load_DA_lag_0          21
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 57.05% done
********************* END NS.. 416 ****************************************************
********************* START NS ... 417 ****************************************************
[I 2025-06-15 20:18:24,191] A new study created in memory with name: no-name-4c1420d3-db35-4dc1-afa8-80491fb965d5
[I 2025-06-15 20:18:24,522] Trial 0 finished with value: 0.002392787282085788 and parameters: {'n_estimators': 57, 'learning_rate': 0.20071151922507, 'num_leaves': 230, 'max_depth': 15, 'min_child_samples': 19, 'subsample': 0.953205579877634, 'colsample_bytree': 0.6235018469225656, 'reg_alpha': 9.059392608414845, 'reg_lambda': 0.5385734124244568, 'min_split_gain': 0.043949486775479954}. Best is trial 0 with value: 0.002392787282085788.
[Trial 0] New best: 0.00239
[I 2025-06-15 20:18:25,865] Trial 1 finished with value: 0.0021391795783043247 and parameters: {'n_estimators': 279, 'learning_rate': 0.048364655148776116, 'num_leaves': 32, 'max_depth': 7, 'min_child_samples': 6, 'subsample': 0.625681729923051, 'colsample_bytree': 0.6289129870812529, 'reg_alpha': 8.303025088966523, 'reg_lambda': 8.819945292653571, 'min_split_gain': 0.01844633000843582}. Best is trial 1 with value: 0.0021391795783043247.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:18:26,491] Trial 2 finished with value: 0.002243880136225856 and parameters: {'n_estimators': 170, 'learning_rate': 0.08930837226422472, 'num_leaves': 91, 'max_depth': 6, 'min_child_samples': 12, 'subsample': 0.6828740492348457, 'colsample_bytree': 0.6418347533802431, 'reg_alpha': 6.433502349394208, 'reg_lambda': 9.378933893975015, 'min_split_gain': 0.02769064326137598}. Best is trial 1 with value: 0.0021391795783043247.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:18:27,634] Trial 3 finished with value: 0.002444192120103957 and parameters: {'n_estimators': 239, 'learning_rate': 0.022457574181115044, 'num_leaves': 90, 'max_depth': 4, 'min_child_samples': 6, 'subsample': 0.8666626861980288, 'colsample_bytree': 0.9654112625749611, 'reg_alpha': 6.63629540054555, 'reg_lambda': 1.467765103723041, 'min_split_gain': 0.031541232045749455}. Best is trial 1 with value: 0.0021391795783043247.
[I 2025-06-15 20:18:27,638] A new study created in memory with name: no-name-0521239a-8484-4c1f-a0fc-d0b59956b064
[I 2025-06-15 20:18:27,689] Trial 0 finished with value: 0.005104027506483921 and parameters: {'n_estimators': 111, 'learning_rate': 0.2555510558282215, 'num_leaves': 95, 'max_depth': 13, 'min_child_samples': 3, 'subsample': 0.8151586821894621, 'colsample_bytree': 0.9696608431205074, 'reg_alpha': 0.49336637098748803, 'reg_lambda': 9.253250472445728, 'min_split_gain': 0.04979377524582502}. Best is trial 0 with value: 0.005104027506483921.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1180
1           Price_lag_2         917
14      pct_chg_Load_DA         570
3         Load_DA_lag_0         420
11           NGas_lag_2         351
2           Price_lag_7         346
5       WindOn_DA_lag_0         294
15       lag168_Load_DA         287
16  roll2d_median_Price         273
13            EUA_lag_2         201
10           Coal_lag_2         162
12            Oil_lag_2         125
9                  WD_7          57
8                  WD_6          32
7                  WD_1          27
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           8
15       lag168_Load_DA           4
5       WindOn_DA_lag_0           2
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 57.18% done
********************* END NS.. 417 ****************************************************
********************* START NS ... 418 ****************************************************
[I 2025-06-15 20:18:31,079] A new study created in memory with name: no-name-2b6180ac-be40-413a-a03e-42be677fd1dc
[I 2025-06-15 20:18:32,148] Trial 0 finished with value: 0.00222732517379617 and parameters: {'n_estimators': 178, 'learning_rate': 0.03439802038803805, 'num_leaves': 207, 'max_depth': 5, 'min_child_samples': 25, 'subsample': 0.8447720766579094, 'colsample_bytree': 0.7107281549438282, 'reg_alpha': 1.2256937769026721, 'reg_lambda': 5.366067778724295, 'min_split_gain': 0.032546594006074785}. Best is trial 0 with value: 0.00222732517379617.
[Trial 0] New best: 0.00223
[I 2025-06-15 20:18:33,485] Trial 1 finished with value: 0.004040466467407797 and parameters: {'n_estimators': 189, 'learning_rate': 0.014682243593756234, 'num_leaves': 235, 'max_depth': 7, 'min_child_samples': 6, 'subsample': 0.5649274597303247, 'colsample_bytree': 0.7963961415185302, 'reg_alpha': 8.331071443522074, 'reg_lambda': 7.3708650091802195, 'min_split_gain': 0.04092353939712414}. Best is trial 0 with value: 0.00222732517379617.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:18:35,028] Trial 2 finished with value: 0.0024863746647656626 and parameters: {'n_estimators': 198, 'learning_rate': 0.06386983528733374, 'num_leaves': 50, 'max_depth': 15, 'min_child_samples': 21, 'subsample': 0.98490859646166, 'colsample_bytree': 0.9981836451019448, 'reg_alpha': 2.5647829412920142, 'reg_lambda': 0.1493371427902357, 'min_split_gain': 0.01669126330049751}. Best is trial 0 with value: 0.00222732517379617.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:18:36,024] Trial 3 finished with value: 0.0035404424400410574 and parameters: {'n_estimators': 170, 'learning_rate': 0.017312980099716404, 'num_leaves': 99, 'max_depth': 5, 'min_child_samples': 18, 'subsample': 0.9829107036752867, 'colsample_bytree': 0.7311460359689146, 'reg_alpha': 1.4082727955170071, 'reg_lambda': 4.7265434800599815, 'min_split_gain': 0.023604295258014066}. Best is trial 0 with value: 0.00222732517379617.
[I 2025-06-15 20:18:36,027] A new study created in memory with name: no-name-adaefed4-801f-4332-a360-303022a22107
[I 2025-06-15 20:18:36,108] Trial 0 finished with value: 0.013914064967746024 and parameters: {'n_estimators': 293, 'learning_rate': 0.013668144114410417, 'num_leaves': 134, 'max_depth': 11, 'min_child_samples': 17, 'subsample': 0.8130674033940561, 'colsample_bytree': 0.5898491108401918, 'reg_alpha': 6.551437472095525, 'reg_lambda': 4.2047532192385315, 'min_split_gain': 0.020136144451461824}. Best is trial 0 with value: 0.013914064967746024.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1260
1           Price_lag_2         825
14      pct_chg_Load_DA         583
3         Load_DA_lag_0         323
16  roll2d_median_Price         227
11           NGas_lag_2         212
2           Price_lag_7         209
13            EUA_lag_2         150
15       lag168_Load_DA         138
10           Coal_lag_2         116
12            Oil_lag_2         105
5       WindOn_DA_lag_0          99
7                  WD_1          12
9                  WD_7           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 57.32% done
********************* END NS.. 418 ****************************************************
********************* START NS ... 419 ****************************************************
[I 2025-06-15 20:18:38,475] A new study created in memory with name: no-name-4d6006b5-df03-403d-bfbb-37db1dae5f7e
[I 2025-06-15 20:18:39,443] Trial 0 finished with value: 0.0021077330191497335 and parameters: {'n_estimators': 252, 'learning_rate': 0.017196910155284035, 'num_leaves': 203, 'max_depth': 4, 'min_child_samples': 30, 'subsample': 0.9331896609658492, 'colsample_bytree': 0.9939215344262703, 'reg_alpha': 9.054012057073944, 'reg_lambda': 7.7069739528589984, 'min_split_gain': 0.004237757935018943}. Best is trial 0 with value: 0.0021077330191497335.
[Trial 0] New best: 0.00211
[I 2025-06-15 20:18:39,834] Trial 1 finished with value: 0.002191778672942018 and parameters: {'n_estimators': 131, 'learning_rate': 0.074879594901652, 'num_leaves': 81, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.8275822921271847, 'colsample_bytree': 0.8714901839332438, 'reg_alpha': 4.848518499884147, 'reg_lambda': 2.4829996624133255, 'min_split_gain': 0.010953867954303904}. Best is trial 0 with value: 0.0021077330191497335.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:18:42,563] Trial 2 finished with value: 0.0022833997845522 and parameters: {'n_estimators': 299, 'learning_rate': 0.021114960390741087, 'num_leaves': 61, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.6619211532596431, 'colsample_bytree': 0.6532207155711331, 'reg_alpha': 3.9232526288188794, 'reg_lambda': 7.708172002594404, 'min_split_gain': 0.03923463561006152}. Best is trial 0 with value: 0.0021077330191497335.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:18:43,133] Trial 3 finished with value: 0.002292331264069486 and parameters: {'n_estimators': 119, 'learning_rate': 0.05809384699016331, 'num_leaves': 67, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.7628282521370313, 'colsample_bytree': 0.6642747020809274, 'reg_alpha': 8.094002104614452, 'reg_lambda': 7.111927323076802, 'min_split_gain': 0.0009083374461290872}. Best is trial 0 with value: 0.0021077330191497335.
[I 2025-06-15 20:18:43,137] A new study created in memory with name: no-name-6a19c369-4e09-4288-b8b3-81e5c24dcce6
[I 2025-06-15 20:18:43,176] Trial 0 finished with value: 0.01569724481710766 and parameters: {'n_estimators': 114, 'learning_rate': 0.19606215611404465, 'num_leaves': 84, 'max_depth': 11, 'min_child_samples': 24, 'subsample': 0.9383594124091849, 'colsample_bytree': 0.963397826152357, 'reg_alpha': 8.328212162238358, 'reg_lambda': 3.2642551227304772, 'min_split_gain': 0.021052189047914772}. Best is trial 0 with value: 0.01569724481710766.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1730
14      pct_chg_Load_DA         624
16  roll2d_median_Price         212
2           Price_lag_7         186
3         Load_DA_lag_0         111
11           NGas_lag_2         103
1           Price_lag_2          78
13            EUA_lag_2          70
12            Oil_lag_2          58
10           Coal_lag_2          44
15       lag168_Load_DA          35
5       WindOn_DA_lag_0          26
9                  WD_7           3
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 57.46% done
********************* END NS.. 419 ****************************************************
********************* START NS ... 420 ****************************************************
[I 2025-06-15 20:18:47,596] A new study created in memory with name: no-name-6546c222-a28a-4325-9f38-b526d4523440
[I 2025-06-15 20:18:49,829] Trial 0 finished with value: 0.0017987119489352697 and parameters: {'n_estimators': 124, 'learning_rate': 0.08527368512485621, 'num_leaves': 127, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.7088643630274618, 'colsample_bytree': 0.6467446251151348, 'reg_alpha': 0.7597933530734102, 'reg_lambda': 9.830469923740257, 'min_split_gain': 0.011173115846771088}. Best is trial 0 with value: 0.0017987119489352697.
[Trial 0] New best: 0.00180
[I 2025-06-15 20:18:50,376] Trial 1 finished with value: 0.0018516114512433714 and parameters: {'n_estimators': 73, 'learning_rate': 0.1109657460551508, 'num_leaves': 178, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.5541310658776045, 'colsample_bytree': 0.7464611655682629, 'reg_alpha': 6.531729749036195, 'reg_lambda': 8.251360428111404, 'min_split_gain': 0.038295078446407116}. Best is trial 0 with value: 0.0017987119489352697.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:18:52,686] Trial 2 finished with value: 0.002777232603273321 and parameters: {'n_estimators': 237, 'learning_rate': 0.012376914921380334, 'num_leaves': 39, 'max_depth': 11, 'min_child_samples': 24, 'subsample': 0.5911976053992046, 'colsample_bytree': 0.6771754002929402, 'reg_alpha': 0.003176539641649434, 'reg_lambda': 3.742766165814422, 'min_split_gain': 0.005864680050075788}. Best is trial 0 with value: 0.0017987119489352697.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:18:53,469] Trial 3 finished with value: 0.002054446056908139 and parameters: {'n_estimators': 243, 'learning_rate': 0.11270182503746637, 'num_leaves': 215, 'max_depth': 13, 'min_child_samples': 24, 'subsample': 0.8485749378502332, 'colsample_bytree': 0.7480169358898631, 'reg_alpha': 2.075350954422434, 'reg_lambda': 9.846228262192394, 'min_split_gain': 0.045118326715853724}. Best is trial 0 with value: 0.0017987119489352697.
[I 2025-06-15 20:18:53,472] A new study created in memory with name: no-name-4ee5b175-7977-45f7-8b99-acade777800d
[I 2025-06-15 20:18:53,505] Trial 0 finished with value: 0.00714947380160942 and parameters: {'n_estimators': 103, 'learning_rate': 0.021315884685942598, 'num_leaves': 103, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.7236311275880134, 'colsample_bytree': 0.8801071329961178, 'reg_alpha': 6.4384833514922, 'reg_lambda': 6.104916837009896, 'min_split_gain': 0.023218549359945363}. Best is trial 0 with value: 0.00714947380160942.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1383
0           Price_lag_1        1264
14      pct_chg_Load_DA        1165
3         Load_DA_lag_0        1045
5       WindOn_DA_lag_0         974
15       lag168_Load_DA         806
2           Price_lag_7         709
16  roll2d_median_Price         679
11           NGas_lag_2         588
13            EUA_lag_2         525
10           Coal_lag_2         507
12            Oil_lag_2         448
9                  WD_7          47
7                  WD_1          33
8                  WD_6          14
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 57.59% done
********************* END NS.. 420 ****************************************************
********************* START NS ... 421 ****************************************************
[I 2025-06-15 20:18:56,969] A new study created in memory with name: no-name-4be4f724-77b8-4066-aabd-532ec2399d14
[I 2025-06-15 20:18:59,445] Trial 0 finished with value: 0.003936782685032204 and parameters: {'n_estimators': 294, 'learning_rate': 0.015137056330545788, 'num_leaves': 119, 'max_depth': 6, 'min_child_samples': 26, 'subsample': 0.8456737422219409, 'colsample_bytree': 0.5649854950728076, 'reg_alpha': 6.187761835119804, 'reg_lambda': 0.8341296789961117, 'min_split_gain': 0.0058468751219432295}. Best is trial 0 with value: 0.003936782685032204.
[Trial 0] New best: 0.00394
[I 2025-06-15 20:19:00,570] Trial 1 finished with value: 0.0037746694042918665 and parameters: {'n_estimators': 110, 'learning_rate': 0.04203864992340716, 'num_leaves': 235, 'max_depth': 9, 'min_child_samples': 8, 'subsample': 0.5323483401478866, 'colsample_bytree': 0.7562365803588351, 'reg_alpha': 5.32536892581788, 'reg_lambda': 6.7042219996992145, 'min_split_gain': 0.014467529184760531}. Best is trial 1 with value: 0.0037746694042918665.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:19:01,814] Trial 2 finished with value: 0.003707368613290632 and parameters: {'n_estimators': 171, 'learning_rate': 0.08094686889391775, 'num_leaves': 93, 'max_depth': 13, 'min_child_samples': 8, 'subsample': 0.7571958794650031, 'colsample_bytree': 0.7054771472376248, 'reg_alpha': 4.269399948931484, 'reg_lambda': 8.927022713963327, 'min_split_gain': 0.016368751069035188}. Best is trial 2 with value: 0.003707368613290632.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:19:02,325] Trial 3 finished with value: 0.003571929280288173 and parameters: {'n_estimators': 160, 'learning_rate': 0.12825074045433996, 'num_leaves': 79, 'max_depth': 7, 'min_child_samples': 26, 'subsample': 0.6802141911065067, 'colsample_bytree': 0.9161909088767943, 'reg_alpha': 7.18899000432234, 'reg_lambda': 2.526797384616357, 'min_split_gain': 0.017879453769851266}. Best is trial 3 with value: 0.003571929280288173.
[I 2025-06-15 20:19:02,328] A new study created in memory with name: no-name-b3a70c25-2443-4a24-b47c-a16fd4cedff1
[I 2025-06-15 20:19:02,515] Trial 0 finished with value: 0.008199009299530642 and parameters: {'n_estimators': 278, 'learning_rate': 0.011630563179099662, 'num_leaves': 216, 'max_depth': 6, 'min_child_samples': 26, 'subsample': 0.5334036293541986, 'colsample_bytree': 0.8271834088280602, 'reg_alpha': 9.744714163359362, 'reg_lambda': 8.311355288505505, 'min_split_gain': 0.030201049475003506}. Best is trial 0 with value: 0.008199009299530642.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         420
14      pct_chg_Load_DA         283
1           Price_lag_2         264
3         Load_DA_lag_0         143
11           NGas_lag_2         125
5       WindOn_DA_lag_0         123
2           Price_lag_7         113
16  roll2d_median_Price         106
15       lag168_Load_DA          93
10           Coal_lag_2          79
13            EUA_lag_2          63
12            Oil_lag_2          47
7                  WD_1           9
9                  WD_7           5
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 57.73% done
********************* END NS.. 421 ****************************************************
********************* START NS ... 422 ****************************************************
[I 2025-06-15 20:19:06,180] A new study created in memory with name: no-name-053b46b8-9f5b-410d-983c-65388e579822
[I 2025-06-15 20:19:07,427] Trial 0 finished with value: 0.035819197230882 and parameters: {'n_estimators': 106, 'learning_rate': 0.010055976496133363, 'num_leaves': 184, 'max_depth': 16, 'min_child_samples': 12, 'subsample': 0.8577114869052727, 'colsample_bytree': 0.5156094350581383, 'reg_alpha': 6.927906964044288, 'reg_lambda': 5.919727271084433, 'min_split_gain': 0.0037020177674146506}. Best is trial 0 with value: 0.035819197230882.
[Trial 0] New best: 0.03582
[I 2025-06-15 20:19:08,221] Trial 1 finished with value: 0.0051010284576021485 and parameters: {'n_estimators': 88, 'learning_rate': 0.031572971536897886, 'num_leaves': 86, 'max_depth': 5, 'min_child_samples': 28, 'subsample': 0.7171901153591498, 'colsample_bytree': 0.9333631818768477, 'reg_alpha': 1.100120899320205, 'reg_lambda': 6.583978638970434, 'min_split_gain': 0.03425150586789842}. Best is trial 1 with value: 0.0051010284576021485.
[Trial 1] New best: 0.00510
[I 2025-06-15 20:19:08,681] Trial 2 finished with value: 0.004508646763529461 and parameters: {'n_estimators': 149, 'learning_rate': 0.08924776210941755, 'num_leaves': 218, 'max_depth': 4, 'min_child_samples': 28, 'subsample': 0.9607768502113051, 'colsample_bytree': 0.6767707649880021, 'reg_alpha': 0.9741578240757642, 'reg_lambda': 5.8314694415848205, 'min_split_gain': 0.019493239750929033}. Best is trial 2 with value: 0.004508646763529461.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:19:09,763] Trial 3 finished with value: 0.026800274153328283 and parameters: {'n_estimators': 97, 'learning_rate': 0.013004192925579808, 'num_leaves': 111, 'max_depth': 14, 'min_child_samples': 30, 'subsample': 0.8547834840860282, 'colsample_bytree': 0.752644904885929, 'reg_alpha': 2.754963877738507, 'reg_lambda': 6.816968340551803, 'min_split_gain': 0.012848898053627484}. Best is trial 2 with value: 0.004508646763529461.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:19:11,419] Trial 4 finished with value: 0.003710621997657024 and parameters: {'n_estimators': 243, 'learning_rate': 0.06892468525830986, 'num_leaves': 181, 'max_depth': 14, 'min_child_samples': 26, 'subsample': 0.5591564187476409, 'colsample_bytree': 0.9380628204773529, 'reg_alpha': 1.8041991420003478, 'reg_lambda': 7.281980108189905, 'min_split_gain': 0.027601062710018734}. Best is trial 4 with value: 0.003710621997657024.
[Trial 4] New best: 0.00371
[I 2025-06-15 20:19:12,615] Trial 5 finished with value: 0.003427906106907471 and parameters: {'n_estimators': 251, 'learning_rate': 0.24489048914778835, 'num_leaves': 169, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.6605555274610233, 'colsample_bytree': 0.9628085107560549, 'reg_alpha': 0.8041162663305446, 'reg_lambda': 5.871936662190199, 'min_split_gain': 0.018396795688136108}. Best is trial 5 with value: 0.003427906106907471.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:19:13,278] Trial 6 finished with value: 0.0034108832025603214 and parameters: {'n_estimators': 233, 'learning_rate': 0.2697939096438196, 'num_leaves': 185, 'max_depth': 13, 'min_child_samples': 26, 'subsample': 0.7549160180709797, 'colsample_bytree': 0.738884087249443, 'reg_alpha': 6.619785448748519, 'reg_lambda': 7.400721852287751, 'min_split_gain': 0.022525142693930675}. Best is trial 6 with value: 0.0034108832025603214.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:19:13,774] Trial 7 finished with value: 0.004246888498848932 and parameters: {'n_estimators': 80, 'learning_rate': 0.09567887736395385, 'num_leaves': 211, 'max_depth': 5, 'min_child_samples': 11, 'subsample': 0.8610854883775807, 'colsample_bytree': 0.6740170604211899, 'reg_alpha': 0.03458389057874656, 'reg_lambda': 3.9107031899791966, 'min_split_gain': 0.03470944748518279}. Best is trial 6 with value: 0.0034108832025603214.
[I 2025-06-15 20:19:13,776] A new study created in memory with name: no-name-cf751db2-9553-45c9-9f47-6a2592c73584
[I 2025-06-15 20:19:13,813] Trial 0 finished with value: 0.018414164972795213 and parameters: {'n_estimators': 111, 'learning_rate': 0.13108731509134777, 'num_leaves': 128, 'max_depth': 14, 'min_child_samples': 27, 'subsample': 0.8270597519892382, 'colsample_bytree': 0.6755356264612863, 'reg_alpha': 5.544274225058378, 'reg_lambda': 8.427993940891112, 'min_split_gain': 0.012883031322475159}. Best is trial 0 with value: 0.018414164972795213.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         219
14      pct_chg_Load_DA         125
1           Price_lag_2         118
3         Load_DA_lag_0          76
11           NGas_lag_2          61
2           Price_lag_7          60
16  roll2d_median_Price          57
13            EUA_lag_2          52
5       WindOn_DA_lag_0          39
10           Coal_lag_2          37
15       lag168_Load_DA          35
12            Oil_lag_2          24
9                  WD_7           7
7                  WD_1           3
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 57.87% done
********************* END NS.. 422 ****************************************************
********************* START NS ... 423 ****************************************************
[I 2025-06-15 20:19:15,807] A new study created in memory with name: no-name-1549cf04-3d2b-439e-a2ae-8a4e9cf85524
[I 2025-06-15 20:19:17,806] Trial 0 finished with value: 0.003443867347994353 and parameters: {'n_estimators': 271, 'learning_rate': 0.11465799450122112, 'num_leaves': 173, 'max_depth': 11, 'min_child_samples': 1, 'subsample': 0.5335474408361491, 'colsample_bytree': 0.7295059664572513, 'reg_alpha': 2.556864487237278, 'reg_lambda': 8.135971938712869, 'min_split_gain': 0.010221681971339203}. Best is trial 0 with value: 0.003443867347994353.
[Trial 0] New best: 0.00344
[I 2025-06-15 20:19:19,997] Trial 1 finished with value: 0.003814139887828086 and parameters: {'n_estimators': 262, 'learning_rate': 0.0329881337780696, 'num_leaves': 192, 'max_depth': 12, 'min_child_samples': 2, 'subsample': 0.5771835168385726, 'colsample_bytree': 0.960323750264641, 'reg_alpha': 7.653669134779394, 'reg_lambda': 2.4438950569097164, 'min_split_gain': 0.026864074227266523}. Best is trial 0 with value: 0.003443867347994353.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:19:21,339] Trial 2 finished with value: 0.0036140775238964807 and parameters: {'n_estimators': 204, 'learning_rate': 0.052271016333330686, 'num_leaves': 238, 'max_depth': 14, 'min_child_samples': 23, 'subsample': 0.979672230691431, 'colsample_bytree': 0.6990750141237736, 'reg_alpha': 6.83340584619936, 'reg_lambda': 6.697402037941949, 'min_split_gain': 0.03340292061711976}. Best is trial 0 with value: 0.003443867347994353.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:19:22,826] Trial 3 finished with value: 0.0035853165651059363 and parameters: {'n_estimators': 222, 'learning_rate': 0.05599850730177417, 'num_leaves': 117, 'max_depth': 12, 'min_child_samples': 23, 'subsample': 0.7932812040833648, 'colsample_bytree': 0.8780009590789242, 'reg_alpha': 5.200283739727631, 'reg_lambda': 9.692375031388865, 'min_split_gain': 0.042182157611463454}. Best is trial 0 with value: 0.003443867347994353.
[I 2025-06-15 20:19:22,830] A new study created in memory with name: no-name-1e0d1e69-9f1b-4bea-8914-ad6236836c99
[I 2025-06-15 20:19:22,870] Trial 0 finished with value: 0.022931216618513304 and parameters: {'n_estimators': 109, 'learning_rate': 0.05847241244706684, 'num_leaves': 86, 'max_depth': 7, 'min_child_samples': 8, 'subsample': 0.6218514409635643, 'colsample_bytree': 0.8467053092484915, 'reg_alpha': 4.381175546143111, 'reg_lambda': 8.558252403993611, 'min_split_gain': 0.03925381395925369}. Best is trial 0 with value: 0.022931216618513304.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         925
14      pct_chg_Load_DA         898
0           Price_lag_1         792
3         Load_DA_lag_0         692
2           Price_lag_7         559
5       WindOn_DA_lag_0         549
15       lag168_Load_DA         513
11           NGas_lag_2         445
16  roll2d_median_Price         421
13            EUA_lag_2         357
10           Coal_lag_2         320
12            Oil_lag_2         279
9                  WD_7          38
8                  WD_6          27
7                  WD_1          22
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 58.00% done
********************* END NS.. 423 ****************************************************
********************* START NS ... 424 ****************************************************
[I 2025-06-15 20:19:26,129] A new study created in memory with name: no-name-78c0622e-949e-4fcb-805e-9994a62842a4
[I 2025-06-15 20:19:26,646] Trial 0 finished with value: 0.0027432842973460214 and parameters: {'n_estimators': 51, 'learning_rate': 0.18422638838362346, 'num_leaves': 90, 'max_depth': 4, 'min_child_samples': 30, 'subsample': 0.7934514400391187, 'colsample_bytree': 0.5363566506612292, 'reg_alpha': 2.2186208896580117, 'reg_lambda': 4.757398187353845, 'min_split_gain': 0.011522440596178452}. Best is trial 0 with value: 0.0027432842973460214.
[Trial 0] New best: 0.00274
[I 2025-06-15 20:19:29,253] Trial 1 finished with value: 0.013708728388608598 and parameters: {'n_estimators': 130, 'learning_rate': 0.01312596445485713, 'num_leaves': 45, 'max_depth': 8, 'min_child_samples': 3, 'subsample': 0.8275689864368078, 'colsample_bytree': 0.5172759757249978, 'reg_alpha': 7.930222705454034, 'reg_lambda': 2.4277145862402505, 'min_split_gain': 0.010204016856013638}. Best is trial 0 with value: 0.0027432842973460214.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:19:31,004] Trial 2 finished with value: 0.002276633209762396 and parameters: {'n_estimators': 250, 'learning_rate': 0.0359849721750029, 'num_leaves': 88, 'max_depth': 13, 'min_child_samples': 2, 'subsample': 0.9133689356476451, 'colsample_bytree': 0.671909452372952, 'reg_alpha': 8.5251045051969, 'reg_lambda': 8.37573818661967, 'min_split_gain': 0.03816506006572047}. Best is trial 2 with value: 0.002276633209762396.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:19:31,976] Trial 3 finished with value: 0.002370877200371824 and parameters: {'n_estimators': 88, 'learning_rate': 0.0662693113801723, 'num_leaves': 34, 'max_depth': 10, 'min_child_samples': 13, 'subsample': 0.7466556416470871, 'colsample_bytree': 0.538374096187842, 'reg_alpha': 6.900265999448404, 'reg_lambda': 3.8124007593791376, 'min_split_gain': 0.002457307060958414}. Best is trial 2 with value: 0.002276633209762396.
[I 2025-06-15 20:19:31,978] A new study created in memory with name: no-name-9760df77-d5cc-43d2-a3f6-629b1e7eb644
[I 2025-06-15 20:19:32,026] Trial 0 finished with value: 0.009955458317789942 and parameters: {'n_estimators': 117, 'learning_rate': 0.16496843643016018, 'num_leaves': 217, 'max_depth': 5, 'min_child_samples': 9, 'subsample': 0.814110699764955, 'colsample_bytree': 0.6232593004740841, 'reg_alpha': 5.504528124173622, 'reg_lambda': 7.940703058810129, 'min_split_gain': 0.04081284644791345}. Best is trial 0 with value: 0.009955458317789942.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1298
1           Price_lag_2        1056
14      pct_chg_Load_DA         783
2           Price_lag_7         496
3         Load_DA_lag_0         464
11           NGas_lag_2         429
16  roll2d_median_Price         420
15       lag168_Load_DA         357
5       WindOn_DA_lag_0         303
10           Coal_lag_2         256
13            EUA_lag_2         241
12            Oil_lag_2         171
9                  WD_7          81
8                  WD_6          46
7                  WD_1          17
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 58.14% done
********************* END NS.. 424 ****************************************************
********************* START NS ... 425 ****************************************************
[I 2025-06-15 20:19:35,124] A new study created in memory with name: no-name-ed413088-219f-4b20-af97-5e9ff2d0ba30
[I 2025-06-15 20:19:35,885] Trial 0 finished with value: 0.000583373563878626 and parameters: {'n_estimators': 76, 'learning_rate': 0.1641361762405559, 'num_leaves': 87, 'max_depth': 9, 'min_child_samples': 23, 'subsample': 0.8624667071221526, 'colsample_bytree': 0.6520028124070415, 'reg_alpha': 9.70942175297324, 'reg_lambda': 9.562798113364655, 'min_split_gain': 0.0027752846975819204}. Best is trial 0 with value: 0.000583373563878626.
[Trial 0] New best: 0.00058
[I 2025-06-15 20:19:38,757] Trial 1 finished with value: 0.0006569453590334085 and parameters: {'n_estimators': 291, 'learning_rate': 0.012864783803823826, 'num_leaves': 223, 'max_depth': 13, 'min_child_samples': 3, 'subsample': 0.6365815479414371, 'colsample_bytree': 0.7072956157279291, 'reg_alpha': 8.243308743719469, 'reg_lambda': 9.038020183458164, 'min_split_gain': 0.028057000630821134}. Best is trial 0 with value: 0.000583373563878626.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:19:39,915] Trial 2 finished with value: 0.00043246830553707495 and parameters: {'n_estimators': 266, 'learning_rate': 0.23335163789358623, 'num_leaves': 80, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.8241971985027434, 'colsample_bytree': 0.788170589070157, 'reg_alpha': 2.5779343812482547, 'reg_lambda': 2.659958394678209, 'min_split_gain': 0.0019344115225492676}. Best is trial 2 with value: 0.00043246830553707495.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:19:41,008] Trial 3 finished with value: 0.022465035971909676 and parameters: {'n_estimators': 131, 'learning_rate': 0.01114416280191395, 'num_leaves': 92, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.5937596116584352, 'colsample_bytree': 0.5070630318986986, 'reg_alpha': 9.98434578511804, 'reg_lambda': 7.6049940545934005, 'min_split_gain': 0.022292717757341464}. Best is trial 2 with value: 0.00043246830553707495.
[I 2025-06-15 20:19:41,010] A new study created in memory with name: no-name-b4a787a2-a5f4-4166-bf2c-084b9a84c578
[I 2025-06-15 20:19:41,090] Trial 0 finished with value: 0.001176905900679167 and parameters: {'n_estimators': 159, 'learning_rate': 0.01608554992587444, 'num_leaves': 77, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.9890775550689679, 'colsample_bytree': 0.9888616137692015, 'reg_alpha': 0.79304298592071, 'reg_lambda': 7.575840752425393, 'min_split_gain': 0.027856968672350907}. Best is trial 0 with value: 0.001176905900679167.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         599
1           Price_lag_2         534
14      pct_chg_Load_DA         482
15       lag168_Load_DA         424
3         Load_DA_lag_0         410
5       WindOn_DA_lag_0         409
2           Price_lag_7         373
13            EUA_lag_2         253
16  roll2d_median_Price         241
11           NGas_lag_2         228
10           Coal_lag_2         223
12            Oil_lag_2         212
9                  WD_7          16
7                  WD_1           9
8                  WD_6           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         165
5       WindOn_DA_lag_0          22
15       lag168_Load_DA          20
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 58.28% done
********************* END NS.. 425 ****************************************************
********************* START NS ... 426 ****************************************************
[I 2025-06-15 20:19:43,797] A new study created in memory with name: no-name-a08cc9c3-1071-4e59-b8a3-be5136ccd1dd
[I 2025-06-15 20:19:45,519] Trial 0 finished with value: 0.0006287733732286766 and parameters: {'n_estimators': 164, 'learning_rate': 0.03644044996897016, 'num_leaves': 98, 'max_depth': 9, 'min_child_samples': 7, 'subsample': 0.8558342559714638, 'colsample_bytree': 0.5057330931409292, 'reg_alpha': 8.168081914167402, 'reg_lambda': 3.9586031978384826, 'min_split_gain': 0.0474074334068002}. Best is trial 0 with value: 0.0006287733732286766.
[Trial 0] New best: 0.00063
[I 2025-06-15 20:19:46,688] Trial 1 finished with value: 0.0005865659730423995 and parameters: {'n_estimators': 195, 'learning_rate': 0.0848927897459195, 'num_leaves': 190, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.83440947555321, 'colsample_bytree': 0.9563170586795284, 'reg_alpha': 1.3716521928935876, 'reg_lambda': 1.7604163799199057, 'min_split_gain': 0.008033882517805846}. Best is trial 1 with value: 0.0005865659730423995.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:19:47,511] Trial 2 finished with value: 0.0005095007358008163 and parameters: {'n_estimators': 105, 'learning_rate': 0.11484304483875306, 'num_leaves': 157, 'max_depth': 14, 'min_child_samples': 20, 'subsample': 0.7221517367323818, 'colsample_bytree': 0.9933158830279962, 'reg_alpha': 3.481328247264024, 'reg_lambda': 0.6988246840831758, 'min_split_gain': 0.02100107458068276}. Best is trial 2 with value: 0.0005095007358008163.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:19:47,915] Trial 3 finished with value: 0.0006614485008588489 and parameters: {'n_estimators': 67, 'learning_rate': 0.13628679435795468, 'num_leaves': 111, 'max_depth': 13, 'min_child_samples': 21, 'subsample': 0.7236014476541388, 'colsample_bytree': 0.7235657224914976, 'reg_alpha': 8.67945338239672, 'reg_lambda': 3.6835392817757553, 'min_split_gain': 0.02985039235417678}. Best is trial 2 with value: 0.0005095007358008163.
[I 2025-06-15 20:19:47,919] A new study created in memory with name: no-name-dd4a208a-3326-4943-ac93-d363386dd974
[I 2025-06-15 20:19:48,048] Trial 0 finished with value: 0.002331072029994972 and parameters: {'n_estimators': 298, 'learning_rate': 0.016951515445454077, 'num_leaves': 134, 'max_depth': 10, 'min_child_samples': 25, 'subsample': 0.7350087989212082, 'colsample_bytree': 0.6433202543481693, 'reg_alpha': 1.228465044848741, 'reg_lambda': 5.270136098018328, 'min_split_gain': 0.002524022980769952}. Best is trial 0 with value: 0.002331072029994972.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         638
14      pct_chg_Load_DA         519
1           Price_lag_2         420
3         Load_DA_lag_0         301
16  roll2d_median_Price         252
2           Price_lag_7         210
11           NGas_lag_2         203
15       lag168_Load_DA         185
5       WindOn_DA_lag_0         177
10           Coal_lag_2         120
13            EUA_lag_2          97
12            Oil_lag_2          96
7                  WD_1          14
9                  WD_7          11
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         201
15       lag168_Load_DA          76
5       WindOn_DA_lag_0          21
3         Load_DA_lag_0          14
1           Price_lag_2           7
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 58.41% done
********************* END NS.. 426 ****************************************************
********************* START NS ... 427 ****************************************************
[I 2025-06-15 20:19:50,127] A new study created in memory with name: no-name-d5a2e3b5-3a39-40db-a63a-e82e5b4f06d2
[I 2025-06-15 20:19:51,143] Trial 0 finished with value: 0.00072795716258129 and parameters: {'n_estimators': 164, 'learning_rate': 0.05426970486775409, 'num_leaves': 253, 'max_depth': 6, 'min_child_samples': 6, 'subsample': 0.6270896745283967, 'colsample_bytree': 0.5573878691667664, 'reg_alpha': 8.324706180017408, 'reg_lambda': 2.1931091701143357, 'min_split_gain': 0.039934243904394304}. Best is trial 0 with value: 0.00072795716258129.
[Trial 0] New best: 0.00073
[I 2025-06-15 20:19:51,506] Trial 1 finished with value: 0.0005764124106284027 and parameters: {'n_estimators': 165, 'learning_rate': 0.25455311424988336, 'num_leaves': 52, 'max_depth': 8, 'min_child_samples': 7, 'subsample': 0.9642429170941633, 'colsample_bytree': 0.8290851269902489, 'reg_alpha': 3.2650846066511843, 'reg_lambda': 5.776961131533209, 'min_split_gain': 0.039353320166255944}. Best is trial 1 with value: 0.0005764124106284027.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:19:51,969] Trial 2 finished with value: 0.000554432750865698 and parameters: {'n_estimators': 175, 'learning_rate': 0.22278114541710942, 'num_leaves': 127, 'max_depth': 14, 'min_child_samples': 25, 'subsample': 0.5206501541131296, 'colsample_bytree': 0.8799319753049408, 'reg_alpha': 8.115350427080669, 'reg_lambda': 5.084134875401979, 'min_split_gain': 0.007609379390357313}. Best is trial 2 with value: 0.000554432750865698.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:19:52,824] Trial 3 finished with value: 0.00039827150692142276 and parameters: {'n_estimators': 273, 'learning_rate': 0.013792676350497723, 'num_leaves': 33, 'max_depth': 4, 'min_child_samples': 20, 'subsample': 0.733702081837516, 'colsample_bytree': 0.977451594071667, 'reg_alpha': 3.4693223954454533, 'reg_lambda': 6.728479802000097, 'min_split_gain': 0.01607890642891693}. Best is trial 3 with value: 0.00039827150692142276.
[I 2025-06-15 20:19:52,827] A new study created in memory with name: no-name-6fe8d4f8-7f92-49e2-91a9-8019cd0d6a5f
[I 2025-06-15 20:19:52,884] Trial 0 finished with value: 0.0012891468359824236 and parameters: {'n_estimators': 225, 'learning_rate': 0.013259655884711258, 'num_leaves': 51, 'max_depth': 11, 'min_child_samples': 9, 'subsample': 0.9727420630325078, 'colsample_bytree': 0.6744156760712452, 'reg_alpha': 6.416469706958241, 'reg_lambda': 5.938670628807255, 'min_split_gain': 0.011732078944511127}. Best is trial 0 with value: 0.0012891468359824236.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2203
14      pct_chg_Load_DA         746
16  roll2d_median_Price         235
2           Price_lag_7         209
3         Load_DA_lag_0         107
1           Price_lag_2          92
11           NGas_lag_2          81
13            EUA_lag_2          75
12            Oil_lag_2          62
10           Coal_lag_2          22
15       lag168_Load_DA          20
5       WindOn_DA_lag_0          15
9                  WD_7           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 58.55% done
********************* END NS.. 427 ****************************************************
********************* START NS ... 428 ****************************************************
[I 2025-06-15 20:19:55,459] A new study created in memory with name: no-name-e7b302f1-2073-4b9b-abbd-86677796d9c5
[I 2025-06-15 20:19:56,917] Trial 0 finished with value: 0.0005651639548466003 and parameters: {'n_estimators': 165, 'learning_rate': 0.06824747022066253, 'num_leaves': 87, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.5630538805356492, 'colsample_bytree': 0.8843502948068782, 'reg_alpha': 9.694785204786452, 'reg_lambda': 9.397196456943076, 'min_split_gain': 0.0044526815583088636}. Best is trial 0 with value: 0.0005651639548466003.
[Trial 0] New best: 0.00057
[I 2025-06-15 20:19:57,484] Trial 1 finished with value: 0.00043353198443289563 and parameters: {'n_estimators': 125, 'learning_rate': 0.04287332061835092, 'num_leaves': 229, 'max_depth': 4, 'min_child_samples': 9, 'subsample': 0.5010165646962025, 'colsample_bytree': 0.8135352265999434, 'reg_alpha': 2.5569175435885603, 'reg_lambda': 0.3632768588120827, 'min_split_gain': 0.009697375586930969}. Best is trial 1 with value: 0.00043353198443289563.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:19:58,465] Trial 2 finished with value: 0.000571727938131945 and parameters: {'n_estimators': 182, 'learning_rate': 0.08282593890203992, 'num_leaves': 77, 'max_depth': 7, 'min_child_samples': 1, 'subsample': 0.8480563345866884, 'colsample_bytree': 0.5982835369781099, 'reg_alpha': 5.238735227781813, 'reg_lambda': 2.2157504743425482, 'min_split_gain': 0.026177642942270048}. Best is trial 1 with value: 0.00043353198443289563.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:20:00,788] Trial 3 finished with value: 0.00042070787964926487 and parameters: {'n_estimators': 263, 'learning_rate': 0.21174659880334867, 'num_leaves': 228, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.7549480670041497, 'colsample_bytree': 0.6205221543109185, 'reg_alpha': 0.2906242449317653, 'reg_lambda': 9.479770652105998, 'min_split_gain': 0.0048374243316854585}. Best is trial 3 with value: 0.00042070787964926487.
[I 2025-06-15 20:20:00,790] A new study created in memory with name: no-name-156efe9e-1294-4dc1-845d-3563d41b9699
[I 2025-06-15 20:20:00,833] Trial 0 finished with value: 0.0015443247185396578 and parameters: {'n_estimators': 129, 'learning_rate': 0.01705661953038339, 'num_leaves': 43, 'max_depth': 8, 'min_child_samples': 29, 'subsample': 0.824760044403338, 'colsample_bytree': 0.5950246174237019, 'reg_alpha': 7.898680300513136, 'reg_lambda': 2.100097526350664, 'min_split_gain': 0.015627617638951208}. Best is trial 0 with value: 0.0015443247185396578.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1209
3         Load_DA_lag_0        1200
5       WindOn_DA_lag_0        1117
0           Price_lag_1        1114
14      pct_chg_Load_DA        1073
15       lag168_Load_DA         898
2           Price_lag_7         870
16  roll2d_median_Price         705
10           Coal_lag_2         590
13            EUA_lag_2         587
11           NGas_lag_2         506
12            Oil_lag_2         458
9                  WD_7          35
7                  WD_1          22
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 58.69% done
********************* END NS.. 428 ****************************************************
********************* START NS ... 429 ****************************************************
[I 2025-06-15 20:20:04,480] A new study created in memory with name: no-name-194d37bb-f63a-437b-8e58-b533f82d1b00
[I 2025-06-15 20:20:08,566] Trial 0 finished with value: 0.0035570305740677253 and parameters: {'n_estimators': 262, 'learning_rate': 0.016359786660726825, 'num_leaves': 65, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.7367658991341388, 'colsample_bytree': 0.5785207349453731, 'reg_alpha': 0.5350272887843965, 'reg_lambda': 8.729930036328803, 'min_split_gain': 0.0022208566702780655}. Best is trial 0 with value: 0.0035570305740677253.
[Trial 0] New best: 0.00356
[I 2025-06-15 20:20:09,233] Trial 1 finished with value: 0.011646713016650545 and parameters: {'n_estimators': 105, 'learning_rate': 0.01701376438862496, 'num_leaves': 247, 'max_depth': 6, 'min_child_samples': 24, 'subsample': 0.961961797544852, 'colsample_bytree': 0.6058174899834494, 'reg_alpha': 8.613594883736006, 'reg_lambda': 3.39113535467039, 'min_split_gain': 0.014950869373906522}. Best is trial 0 with value: 0.0035570305740677253.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:20:10,171] Trial 2 finished with value: 0.0034252712011105858 and parameters: {'n_estimators': 166, 'learning_rate': 0.15975199499391754, 'num_leaves': 249, 'max_depth': 14, 'min_child_samples': 2, 'subsample': 0.7478337872977638, 'colsample_bytree': 0.8945031019964995, 'reg_alpha': 9.119505147529662, 'reg_lambda': 9.06301968709425, 'min_split_gain': 0.0017623539200876282}. Best is trial 2 with value: 0.0034252712011105858.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:20:10,439] Trial 3 finished with value: 0.0029486941456829036 and parameters: {'n_estimators': 57, 'learning_rate': 0.2927827964279701, 'num_leaves': 114, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.6798163944462683, 'colsample_bytree': 0.736993157079888, 'reg_alpha': 1.673979614290293, 'reg_lambda': 6.5010352932835325, 'min_split_gain': 0.005557495265438361}. Best is trial 3 with value: 0.0029486941456829036.
[I 2025-06-15 20:20:10,442] A new study created in memory with name: no-name-99fea547-0055-4507-adaa-77657cc65b43
[I 2025-06-15 20:20:10,514] Trial 0 finished with value: 0.013089746333204097 and parameters: {'n_estimators': 245, 'learning_rate': 0.11393925806158516, 'num_leaves': 229, 'max_depth': 11, 'min_child_samples': 25, 'subsample': 0.920940824257756, 'colsample_bytree': 0.7786418242860584, 'reg_alpha': 9.078238438911606, 'reg_lambda': 0.15150350910697497, 'min_split_gain': 0.009964451443537088}. Best is trial 0 with value: 0.013089746333204097.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         168
14      pct_chg_Load_DA         105
1           Price_lag_2         100
3         Load_DA_lag_0          72
16  roll2d_median_Price          50
2           Price_lag_7          47
15       lag168_Load_DA          41
11           NGas_lag_2          40
13            EUA_lag_2          40
5       WindOn_DA_lag_0          38
10           Coal_lag_2          24
12            Oil_lag_2          19
7                  WD_1           4
9                  WD_7           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 58.82% done
********************* END NS.. 429 ****************************************************
********************* START NS ... 430 ****************************************************
[I 2025-06-15 20:20:12,435] A new study created in memory with name: no-name-3e8b4458-46c5-401a-8144-62f7a24c5499
[I 2025-06-15 20:20:15,407] Trial 0 finished with value: 0.005169511309497841 and parameters: {'n_estimators': 137, 'learning_rate': 0.05961799915102177, 'num_leaves': 85, 'max_depth': 14, 'min_child_samples': 17, 'subsample': 0.6291492219588104, 'colsample_bytree': 0.8107312772888406, 'reg_alpha': 0.43303832484363314, 'reg_lambda': 0.39722883921995966, 'min_split_gain': 0.005774369266932539}. Best is trial 0 with value: 0.005169511309497841.
[Trial 0] New best: 0.00517
[I 2025-06-15 20:20:16,036] Trial 1 finished with value: 0.005301028964530108 and parameters: {'n_estimators': 172, 'learning_rate': 0.12498812342839292, 'num_leaves': 224, 'max_depth': 10, 'min_child_samples': 27, 'subsample': 0.5581544204600206, 'colsample_bytree': 0.7486267447821415, 'reg_alpha': 2.732817678541888, 'reg_lambda': 9.162605035485345, 'min_split_gain': 0.03441109954049933}. Best is trial 0 with value: 0.005169511309497841.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:20:17,962] Trial 2 finished with value: 0.004995995211930082 and parameters: {'n_estimators': 130, 'learning_rate': 0.051449610885107785, 'num_leaves': 75, 'max_depth': 9, 'min_child_samples': 3, 'subsample': 0.5198453345383389, 'colsample_bytree': 0.959991653736161, 'reg_alpha': 1.1093848500676151, 'reg_lambda': 9.037804815875255, 'min_split_gain': 0.042624935299013796}. Best is trial 2 with value: 0.004995995211930082.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:20:19,482] Trial 3 finished with value: 0.006701033725095573 and parameters: {'n_estimators': 160, 'learning_rate': 0.05053426360403895, 'num_leaves': 48, 'max_depth': 9, 'min_child_samples': 3, 'subsample': 0.6323017726307231, 'colsample_bytree': 0.5142570868339369, 'reg_alpha': 9.183760654520604, 'reg_lambda': 6.374921564688194, 'min_split_gain': 0.04687687913878233}. Best is trial 2 with value: 0.004995995211930082.
[I 2025-06-15 20:20:19,484] A new study created in memory with name: no-name-b414ca45-060d-4dfc-88b2-52070291c455
[I 2025-06-15 20:20:19,532] Trial 0 finished with value: 0.021591240777879935 and parameters: {'n_estimators': 182, 'learning_rate': 0.09963694597813622, 'num_leaves': 107, 'max_depth': 7, 'min_child_samples': 5, 'subsample': 0.9905952444073255, 'colsample_bytree': 0.9484577428447218, 'reg_alpha': 1.7260937606090776, 'reg_lambda': 3.4690122480464214, 'min_split_gain': 0.010590986433188915}. Best is trial 0 with value: 0.021591240777879935.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1580
1           Price_lag_2        1316
14      pct_chg_Load_DA        1291
3         Load_DA_lag_0         837
2           Price_lag_7         616
16  roll2d_median_Price         597
11           NGas_lag_2         576
5       WindOn_DA_lag_0         501
15       lag168_Load_DA         483
10           Coal_lag_2         357
12            Oil_lag_2         274
13            EUA_lag_2         262
9                  WD_7          22
7                  WD_1          20
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          22
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 58.96% done
********************* END NS.. 430 ****************************************************
********************* START NS ... 431 ****************************************************
[I 2025-06-15 20:20:23,027] A new study created in memory with name: no-name-6a1f6e8b-4c62-4cd4-97c2-ee065e009109
[I 2025-06-15 20:20:23,778] Trial 0 finished with value: 0.009106230105857028 and parameters: {'n_estimators': 128, 'learning_rate': 0.07724743134220728, 'num_leaves': 62, 'max_depth': 5, 'min_child_samples': 24, 'subsample': 0.6492105409073898, 'colsample_bytree': 0.5432552900503091, 'reg_alpha': 5.879190962292224, 'reg_lambda': 7.548165526236143, 'min_split_gain': 0.040832628003406875}. Best is trial 0 with value: 0.009106230105857028.
[Trial 0] New best: 0.00911
[I 2025-06-15 20:20:24,305] Trial 1 finished with value: 0.009901834340341031 and parameters: {'n_estimators': 70, 'learning_rate': 0.03916075331075656, 'num_leaves': 246, 'max_depth': 6, 'min_child_samples': 7, 'subsample': 0.5670095836149536, 'colsample_bytree': 0.536926810642115, 'reg_alpha': 2.8754604812088456, 'reg_lambda': 9.216530588212882, 'min_split_gain': 0.03928462114278289}. Best is trial 0 with value: 0.009106230105857028.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:20:24,714] Trial 2 finished with value: 0.007169010726791073 and parameters: {'n_estimators': 90, 'learning_rate': 0.17734165325429077, 'num_leaves': 180, 'max_depth': 6, 'min_child_samples': 13, 'subsample': 0.6163477646153754, 'colsample_bytree': 0.6183539013013656, 'reg_alpha': 5.112652180943998, 'reg_lambda': 0.7992130559676425, 'min_split_gain': 0.01678763766910098}. Best is trial 2 with value: 0.007169010726791073.
[Trial 2] New best: 0.00717
[I 2025-06-15 20:20:25,009] Trial 3 finished with value: 0.00677214798627205 and parameters: {'n_estimators': 88, 'learning_rate': 0.23797647175996373, 'num_leaves': 165, 'max_depth': 15, 'min_child_samples': 22, 'subsample': 0.990835441935175, 'colsample_bytree': 0.9940341345943765, 'reg_alpha': 8.57047839265411, 'reg_lambda': 0.9093451050337964, 'min_split_gain': 0.022650365267936685}. Best is trial 3 with value: 0.00677214798627205.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:20:25,895] Trial 4 finished with value: 0.006597859298454911 and parameters: {'n_estimators': 256, 'learning_rate': 0.08962667243233606, 'num_leaves': 125, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.989537622150471, 'colsample_bytree': 0.8252871220197204, 'reg_alpha': 5.096747416400643, 'reg_lambda': 9.989656568150776, 'min_split_gain': 0.028300250167944248}. Best is trial 4 with value: 0.006597859298454911.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:20:27,334] Trial 5 finished with value: 0.006921530935430294 and parameters: {'n_estimators': 176, 'learning_rate': 0.1274398149359182, 'num_leaves': 163, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.6156825783759835, 'colsample_bytree': 0.5601215765543986, 'reg_alpha': 4.898692269470333, 'reg_lambda': 0.10203449356681826, 'min_split_gain': 0.008831433664130394}. Best is trial 4 with value: 0.006597859298454911.
[I 2025-06-15 20:20:27,336] A new study created in memory with name: no-name-63c4420d-320b-4089-8b85-189cb727fa7a
[I 2025-06-15 20:20:27,415] Trial 0 finished with value: 0.05083157074790762 and parameters: {'n_estimators': 281, 'learning_rate': 0.09144737644838641, 'num_leaves': 186, 'max_depth': 14, 'min_child_samples': 14, 'subsample': 0.728266877825788, 'colsample_bytree': 0.5092394721671415, 'reg_alpha': 9.481878697224218, 'reg_lambda': 3.8765982057155446, 'min_split_gain': 0.032229837049769054}. Best is trial 0 with value: 0.05083157074790762.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         640
1           Price_lag_2         515
14      pct_chg_Load_DA         447
3         Load_DA_lag_0         259
11           NGas_lag_2         196
2           Price_lag_7         195
16  roll2d_median_Price         173
5       WindOn_DA_lag_0         160
15       lag168_Load_DA         141
10           Coal_lag_2         117
13            EUA_lag_2         116
12            Oil_lag_2          87
9                  WD_7          16
7                  WD_1          10
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 59.10% done
********************* END NS.. 431 ****************************************************
********************* START NS ... 432 ****************************************************
[I 2025-06-15 20:20:30,012] A new study created in memory with name: no-name-862ffe6f-e52f-4e08-8616-bc0eda40e83c
[I 2025-06-15 20:20:31,265] Trial 0 finished with value: 0.00923009339407822 and parameters: {'n_estimators': 287, 'learning_rate': 0.09050695170099159, 'num_leaves': 91, 'max_depth': 15, 'min_child_samples': 25, 'subsample': 0.5916177673861999, 'colsample_bytree': 0.5026100731927394, 'reg_alpha': 3.337253727141496, 'reg_lambda': 8.68393116774144, 'min_split_gain': 0.04145360520699792}. Best is trial 0 with value: 0.00923009339407822.
[Trial 0] New best: 0.00923
[I 2025-06-15 20:20:31,590] Trial 1 finished with value: 0.0095860065486427 and parameters: {'n_estimators': 94, 'learning_rate': 0.2694240149554409, 'num_leaves': 217, 'max_depth': 10, 'min_child_samples': 4, 'subsample': 0.7143651984655247, 'colsample_bytree': 0.7259201144394203, 'reg_alpha': 6.715640574447982, 'reg_lambda': 3.7099094638579233, 'min_split_gain': 0.02599672789370949}. Best is trial 0 with value: 0.00923009339407822.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:20:32,488] Trial 2 finished with value: 0.010795400983187506 and parameters: {'n_estimators': 98, 'learning_rate': 0.06618609053247668, 'num_leaves': 136, 'max_depth': 15, 'min_child_samples': 9, 'subsample': 0.7231831878573263, 'colsample_bytree': 0.6902222721164832, 'reg_alpha': 9.586817468310215, 'reg_lambda': 1.9934841474043985, 'min_split_gain': 0.037248262945326724}. Best is trial 0 with value: 0.00923009339407822.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:20:33,375] Trial 3 finished with value: 0.008986230211754672 and parameters: {'n_estimators': 172, 'learning_rate': 0.09896148028978062, 'num_leaves': 136, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.5211772780684067, 'colsample_bytree': 0.7215159999568344, 'reg_alpha': 2.0597569212340563, 'reg_lambda': 5.024382113175271, 'min_split_gain': 0.033425652266406806}. Best is trial 3 with value: 0.008986230211754672.
[I 2025-06-15 20:20:33,378] A new study created in memory with name: no-name-e96a8d79-2c0f-4ad6-92c6-2bc8fed24783
[I 2025-06-15 20:20:33,432] Trial 0 finished with value: 0.05860042747647637 and parameters: {'n_estimators': 217, 'learning_rate': 0.13788265288824603, 'num_leaves': 122, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.5738842836387237, 'colsample_bytree': 0.684626250100091, 'reg_alpha': 9.793370988833853, 'reg_lambda': 8.469835572995692, 'min_split_gain': 0.04157721467140167}. Best is trial 0 with value: 0.05860042747647637.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         665
1           Price_lag_2         624
14      pct_chg_Load_DA         534
3         Load_DA_lag_0         380
11           NGas_lag_2         264
2           Price_lag_7         261
16  roll2d_median_Price         226
5       WindOn_DA_lag_0         202
15       lag168_Load_DA         189
10           Coal_lag_2         181
13            EUA_lag_2         159
12            Oil_lag_2         120
9                  WD_7          22
7                  WD_1          10
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 59.23% done
********************* END NS.. 432 ****************************************************
********************* START NS ... 433 ****************************************************
[I 2025-06-15 20:20:35,690] A new study created in memory with name: no-name-a27b53d1-6d59-4158-9a2b-d795e870d1ea
[I 2025-06-15 20:20:37,112] Trial 0 finished with value: 0.006399277465011702 and parameters: {'n_estimators': 171, 'learning_rate': 0.09155773410601376, 'num_leaves': 99, 'max_depth': 10, 'min_child_samples': 14, 'subsample': 0.9112287590080344, 'colsample_bytree': 0.6922215507526108, 'reg_alpha': 9.997186029400043, 'reg_lambda': 4.127395066599106, 'min_split_gain': 0.001223019527534214}. Best is trial 0 with value: 0.006399277465011702.
[Trial 0] New best: 0.00640
[I 2025-06-15 20:20:38,789] Trial 1 finished with value: 0.00733141713838437 and parameters: {'n_estimators': 172, 'learning_rate': 0.036123549979815935, 'num_leaves': 231, 'max_depth': 12, 'min_child_samples': 11, 'subsample': 0.6133102117971181, 'colsample_bytree': 0.7180332486177494, 'reg_alpha': 6.336106773244081, 'reg_lambda': 6.256319550924124, 'min_split_gain': 0.017568008585649126}. Best is trial 0 with value: 0.006399277465011702.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:20:39,861] Trial 2 finished with value: 0.0076016335595346 and parameters: {'n_estimators': 165, 'learning_rate': 0.03692554970367438, 'num_leaves': 128, 'max_depth': 7, 'min_child_samples': 20, 'subsample': 0.9452065873906685, 'colsample_bytree': 0.7720803016636272, 'reg_alpha': 8.730440846171927, 'reg_lambda': 6.2774991840352365, 'min_split_gain': 0.02879309818651346}. Best is trial 0 with value: 0.006399277465011702.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:20:41,374] Trial 3 finished with value: 0.007030849915672511 and parameters: {'n_estimators': 247, 'learning_rate': 0.05418277841602424, 'num_leaves': 113, 'max_depth': 15, 'min_child_samples': 30, 'subsample': 0.9033633869734243, 'colsample_bytree': 0.835048338689758, 'reg_alpha': 2.7723780072394346, 'reg_lambda': 8.200343956530707, 'min_split_gain': 0.02655667647349799}. Best is trial 0 with value: 0.006399277465011702.
[I 2025-06-15 20:20:41,376] A new study created in memory with name: no-name-6796632a-72ac-437a-99c3-fd7b4362e443
[I 2025-06-15 20:20:41,409] Trial 0 finished with value: 0.010473419669633154 and parameters: {'n_estimators': 75, 'learning_rate': 0.1707916549574619, 'num_leaves': 87, 'max_depth': 12, 'min_child_samples': 6, 'subsample': 0.7473391297181896, 'colsample_bytree': 0.8052632110210192, 'reg_alpha': 1.580986332963814, 'reg_lambda': 5.274560324229949, 'min_split_gain': 0.03764735325014616}. Best is trial 0 with value: 0.010473419669633154.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         787
1           Price_lag_2         723
14      pct_chg_Load_DA         555
3         Load_DA_lag_0         539
5       WindOn_DA_lag_0         427
11           NGas_lag_2         380
2           Price_lag_7         365
16  roll2d_median_Price         325
13            EUA_lag_2         322
15       lag168_Load_DA         299
10           Coal_lag_2         280
12            Oil_lag_2         172
9                  WD_7          43
8                  WD_6          20
7                  WD_1          14
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          16
14      pct_chg_Load_DA           5
3         Load_DA_lag_0           2
15       lag168_Load_DA           1
9                  WD_7           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 59.37% done
********************* END NS.. 433 ****************************************************
********************* START NS ... 434 ****************************************************
[I 2025-06-15 20:20:44,120] A new study created in memory with name: no-name-e17eb496-266f-412e-aeea-6d1934580ad1
[I 2025-06-15 20:20:44,973] Trial 0 finished with value: 0.007656033204986884 and parameters: {'n_estimators': 63, 'learning_rate': 0.033428537013793606, 'num_leaves': 102, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.9112274390656905, 'colsample_bytree': 0.8876357174258931, 'reg_alpha': 5.110163071396503, 'reg_lambda': 1.248857418957372, 'min_split_gain': 0.018947297835068433}. Best is trial 0 with value: 0.007656033204986884.
[Trial 0] New best: 0.00766
[I 2025-06-15 20:20:46,436] Trial 1 finished with value: 0.0033131530589098766 and parameters: {'n_estimators': 142, 'learning_rate': 0.02649830258634139, 'num_leaves': 113, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.9456471723336034, 'colsample_bytree': 0.5237703140823731, 'reg_alpha': 6.851774679655474, 'reg_lambda': 1.290386553229006, 'min_split_gain': 0.012490095188566214}. Best is trial 1 with value: 0.0033131530589098766.
[Trial 1] New best: 0.00331
[I 2025-06-15 20:20:47,891] Trial 2 finished with value: 0.0023314417487209807 and parameters: {'n_estimators': 165, 'learning_rate': 0.05781684883836486, 'num_leaves': 252, 'max_depth': 12, 'min_child_samples': 9, 'subsample': 0.6706904099677115, 'colsample_bytree': 0.5095113616135046, 'reg_alpha': 4.841347627889712, 'reg_lambda': 7.478843662496784, 'min_split_gain': 0.04111666843344865}. Best is trial 2 with value: 0.0023314417487209807.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:20:48,249] Trial 3 finished with value: 0.002526161612135293 and parameters: {'n_estimators': 183, 'learning_rate': 0.2624124279823179, 'num_leaves': 205, 'max_depth': 12, 'min_child_samples': 24, 'subsample': 0.6201631629200655, 'colsample_bytree': 0.6792799509714003, 'reg_alpha': 9.82522191317294, 'reg_lambda': 3.7122058930544135, 'min_split_gain': 0.021777994245819606}. Best is trial 2 with value: 0.0023314417487209807.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:20:49,672] Trial 4 finished with value: 0.008499420684768432 and parameters: {'n_estimators': 84, 'learning_rate': 0.02493408537862452, 'num_leaves': 106, 'max_depth': 16, 'min_child_samples': 14, 'subsample': 0.8915829331290377, 'colsample_bytree': 0.5145282409250105, 'reg_alpha': 2.2075719329678933, 'reg_lambda': 1.6026792416087532, 'min_split_gain': 0.032726036245259695}. Best is trial 2 with value: 0.0023314417487209807.
[I 2025-06-15 20:20:49,675] A new study created in memory with name: no-name-7c5c0d79-922b-49d8-a576-900dc1ec9499
[I 2025-06-15 20:20:49,723] Trial 0 finished with value: 0.01774362880474594 and parameters: {'n_estimators': 175, 'learning_rate': 0.22597567929107573, 'num_leaves': 197, 'max_depth': 15, 'min_child_samples': 7, 'subsample': 0.6811000619571269, 'colsample_bytree': 0.8727897983565022, 'reg_alpha': 8.726120694436295, 'reg_lambda': 1.1752298140509354, 'min_split_gain': 0.03187384592421128}. Best is trial 0 with value: 0.01774362880474594.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         903
1           Price_lag_2         849
14      pct_chg_Load_DA         551
3         Load_DA_lag_0         413
2           Price_lag_7         396
11           NGas_lag_2         344
15       lag168_Load_DA         318
16  roll2d_median_Price         296
5       WindOn_DA_lag_0         257
13            EUA_lag_2         247
10           Coal_lag_2         227
12            Oil_lag_2         126
9                  WD_7          32
8                  WD_6          31
7                  WD_1          23
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 59.51% done
********************* END NS.. 434 ****************************************************
********************* START NS ... 435 ****************************************************
[I 2025-06-15 20:20:52,401] A new study created in memory with name: no-name-bb0a970b-7755-42ff-aa70-3eba7d226155
[I 2025-06-15 20:20:53,792] Trial 0 finished with value: 0.003551935385637009 and parameters: {'n_estimators': 94, 'learning_rate': 0.027147737297793198, 'num_leaves': 80, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.679061608329254, 'colsample_bytree': 0.8064870023129318, 'reg_alpha': 1.8292588653627007, 'reg_lambda': 3.322844558358402, 'min_split_gain': 0.0045903126050562465}. Best is trial 0 with value: 0.003551935385637009.
[Trial 0] New best: 0.00355
[I 2025-06-15 20:20:54,588] Trial 1 finished with value: 0.0019188742398861963 and parameters: {'n_estimators': 59, 'learning_rate': 0.05794620093407842, 'num_leaves': 45, 'max_depth': 15, 'min_child_samples': 27, 'subsample': 0.5733300969797572, 'colsample_bytree': 0.6133598790351472, 'reg_alpha': 2.967210115114718, 'reg_lambda': 0.48312530680827304, 'min_split_gain': 9.989305995114495e-05}. Best is trial 1 with value: 0.0019188742398861963.
[Trial 1] New best: 0.00192
[I 2025-06-15 20:20:55,811] Trial 2 finished with value: 0.0009646316606260747 and parameters: {'n_estimators': 127, 'learning_rate': 0.04387010141440319, 'num_leaves': 125, 'max_depth': 7, 'min_child_samples': 10, 'subsample': 0.5451043650288288, 'colsample_bytree': 0.9505250913615996, 'reg_alpha': 2.2230596529591495, 'reg_lambda': 9.494749794487237, 'min_split_gain': 0.03090742461803104}. Best is trial 2 with value: 0.0009646316606260747.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:20:58,972] Trial 3 finished with value: 0.0022696899704569827 and parameters: {'n_estimators': 285, 'learning_rate': 0.0107301481775884, 'num_leaves': 54, 'max_depth': 12, 'min_child_samples': 28, 'subsample': 0.9759792872268451, 'colsample_bytree': 0.7298970369444833, 'reg_alpha': 2.2906507916607497, 'reg_lambda': 5.387131286775704, 'min_split_gain': 0.030518165216717498}. Best is trial 2 with value: 0.0009646316606260747.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:20:59,353] Trial 4 finished with value: 0.001259734003409554 and parameters: {'n_estimators': 286, 'learning_rate': 0.26568152002626005, 'num_leaves': 130, 'max_depth': 5, 'min_child_samples': 11, 'subsample': 0.8137542666762434, 'colsample_bytree': 0.6034159416884977, 'reg_alpha': 1.0741830734783353, 'reg_lambda': 9.197661940103638, 'min_split_gain': 0.049075526333288134}. Best is trial 2 with value: 0.0009646316606260747.
[I 2025-06-15 20:20:59,355] A new study created in memory with name: no-name-4d372de0-6477-49f2-a7ab-b022e784c766
[I 2025-06-15 20:20:59,393] Trial 0 finished with value: 0.046062718331686237 and parameters: {'n_estimators': 137, 'learning_rate': 0.20599607433957318, 'num_leaves': 35, 'max_depth': 14, 'min_child_samples': 3, 'subsample': 0.799346719265073, 'colsample_bytree': 0.6582169218501559, 'reg_alpha': 9.068983869019503, 'reg_lambda': 3.1192396684292643, 'min_split_gain': 0.007911542451447595}. Best is trial 0 with value: 0.046062718331686237.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1426
1           Price_lag_2        1010
14      pct_chg_Load_DA         925
2           Price_lag_7         483
3         Load_DA_lag_0         483
16  roll2d_median_Price         425
11           NGas_lag_2         410
15       lag168_Load_DA         245
5       WindOn_DA_lag_0         229
13            EUA_lag_2         226
10           Coal_lag_2         213
12            Oil_lag_2         183
7                  WD_1          11
9                  WD_7           8
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 59.64% done
********************* END NS.. 435 ****************************************************
********************* START NS ... 436 ****************************************************
[I 2025-06-15 20:21:01,839] A new study created in memory with name: no-name-002ad2cb-8218-4e3c-baba-6e3a0aab75c1
[I 2025-06-15 20:21:02,168] Trial 0 finished with value: 0.0010706484253673882 and parameters: {'n_estimators': 232, 'learning_rate': 0.20951890312148386, 'num_leaves': 181, 'max_depth': 8, 'min_child_samples': 20, 'subsample': 0.6838227226892172, 'colsample_bytree': 0.9777714981995739, 'reg_alpha': 9.763033857333227, 'reg_lambda': 4.935659742122826, 'min_split_gain': 0.023511030999417367}. Best is trial 0 with value: 0.0010706484253673882.
[Trial 0] New best: 0.00107
[I 2025-06-15 20:21:03,280] Trial 1 finished with value: 0.0038779307453898106 and parameters: {'n_estimators': 110, 'learning_rate': 0.02173885980540526, 'num_leaves': 160, 'max_depth': 14, 'min_child_samples': 2, 'subsample': 0.7453653975807095, 'colsample_bytree': 0.8780369485435211, 'reg_alpha': 7.417944787571592, 'reg_lambda': 3.0034458835080056, 'min_split_gain': 0.03342225273848406}. Best is trial 0 with value: 0.0010706484253673882.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:21:06,848] Trial 2 finished with value: 0.0010270210132089126 and parameters: {'n_estimators': 292, 'learning_rate': 0.020342221575493803, 'num_leaves': 201, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.7521467936570924, 'colsample_bytree': 0.7977109619234464, 'reg_alpha': 4.298394491227169, 'reg_lambda': 8.21053256069523, 'min_split_gain': 0.006071843441311498}. Best is trial 2 with value: 0.0010270210132089126.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:21:07,281] Trial 3 finished with value: 0.0012941638219230312 and parameters: {'n_estimators': 55, 'learning_rate': 0.2179602510653783, 'num_leaves': 148, 'max_depth': 10, 'min_child_samples': 14, 'subsample': 0.6019767833895809, 'colsample_bytree': 0.599682578857051, 'reg_alpha': 6.622064643710649, 'reg_lambda': 1.7454177688143357, 'min_split_gain': 0.01513368668575531}. Best is trial 2 with value: 0.0010270210132089126.
[I 2025-06-15 20:21:07,283] A new study created in memory with name: no-name-ab9b431f-e74c-4815-97cf-7a5bd501a59b
[I 2025-06-15 20:21:07,333] Trial 0 finished with value: 0.030471272737368726 and parameters: {'n_estimators': 183, 'learning_rate': 0.16328969341609717, 'num_leaves': 152, 'max_depth': 5, 'min_child_samples': 2, 'subsample': 0.9709047398279993, 'colsample_bytree': 0.8006323494924789, 'reg_alpha': 9.191804812979042, 'reg_lambda': 9.403235362176845, 'min_split_gain': 0.024474895026825396}. Best is trial 0 with value: 0.030471272737368726.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3291
1           Price_lag_2        2915
14      pct_chg_Load_DA        2318
2           Price_lag_7        1232
3         Load_DA_lag_0        1209
16  roll2d_median_Price        1135
11           NGas_lag_2        1057
5       WindOn_DA_lag_0         983
13            EUA_lag_2         937
15       lag168_Load_DA         919
12            Oil_lag_2         692
10           Coal_lag_2         642
9                  WD_7          83
8                  WD_6          44
7                  WD_1          27
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 59.78% done
********************* END NS.. 436 ****************************************************
********************* START NS ... 437 ****************************************************
[I 2025-06-15 20:21:11,736] A new study created in memory with name: no-name-35e87cdb-f654-4163-bd57-61718fd25242
[I 2025-06-15 20:21:14,264] Trial 0 finished with value: 0.004517787380210312 and parameters: {'n_estimators': 264, 'learning_rate': 0.010816989601916828, 'num_leaves': 241, 'max_depth': 13, 'min_child_samples': 9, 'subsample': 0.7969030986602738, 'colsample_bytree': 0.5227617251015517, 'reg_alpha': 9.91872535844475, 'reg_lambda': 8.737202723313793, 'min_split_gain': 0.03770364973607219}. Best is trial 0 with value: 0.004517787380210312.
[Trial 0] New best: 0.00452
[I 2025-06-15 20:21:17,138] Trial 1 finished with value: 0.004142475273338586 and parameters: {'n_estimators': 275, 'learning_rate': 0.040150142453431036, 'num_leaves': 206, 'max_depth': 14, 'min_child_samples': 28, 'subsample': 0.6933221692673024, 'colsample_bytree': 0.6908017001225893, 'reg_alpha': 6.579040610006034, 'reg_lambda': 0.7882892958384491, 'min_split_gain': 0.0035390339115908887}. Best is trial 1 with value: 0.004142475273338586.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:21:18,637] Trial 2 finished with value: 0.004879809790411873 and parameters: {'n_estimators': 142, 'learning_rate': 0.021793764928389518, 'num_leaves': 135, 'max_depth': 12, 'min_child_samples': 30, 'subsample': 0.5864187093188651, 'colsample_bytree': 0.9209725865549937, 'reg_alpha': 8.574959512287968, 'reg_lambda': 3.1590659560899956, 'min_split_gain': 0.039511867186531185}. Best is trial 1 with value: 0.004142475273338586.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:21:24,176] Trial 3 finished with value: 0.00431806495025108 and parameters: {'n_estimators': 258, 'learning_rate': 0.02287628669235993, 'num_leaves': 54, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.954526632908623, 'colsample_bytree': 0.9775736541439718, 'reg_alpha': 4.892132529339566, 'reg_lambda': 4.50419756682126, 'min_split_gain': 0.018248153957721613}. Best is trial 1 with value: 0.004142475273338586.
[I 2025-06-15 20:21:24,180] A new study created in memory with name: no-name-9fa98c31-9b49-4453-8835-8ae5073bbbea
[I 2025-06-15 20:21:24,235] Trial 0 finished with value: 0.00590214463416557 and parameters: {'n_estimators': 87, 'learning_rate': 0.11170403111483035, 'num_leaves': 227, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.6376952065949262, 'colsample_bytree': 0.860211218462077, 'reg_alpha': 2.114274081167924, 'reg_lambda': 7.460764656361071, 'min_split_gain': 0.02236553920967024}. Best is trial 0 with value: 0.00590214463416557.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1951
1           Price_lag_2        1713
14      pct_chg_Load_DA        1255
3         Load_DA_lag_0         987
5       WindOn_DA_lag_0         832
13            EUA_lag_2         790
15       lag168_Load_DA         745
2           Price_lag_7         684
11           NGas_lag_2         620
16  roll2d_median_Price         573
12            Oil_lag_2         471
10           Coal_lag_2         460
9                  WD_7          63
7                  WD_1          46
8                  WD_6          30
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          40
15       lag168_Load_DA           7
3         Load_DA_lag_0           1
1           Price_lag_2           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 59.92% done
********************* END NS.. 437 ****************************************************
********************* START NS ... 438 ****************************************************
[I 2025-06-15 20:21:28,678] A new study created in memory with name: no-name-1bee145c-daad-4143-9a4e-a93775b5e8a2
[I 2025-06-15 20:21:30,222] Trial 0 finished with value: 0.0053387572798920075 and parameters: {'n_estimators': 83, 'learning_rate': 0.1713194400942949, 'num_leaves': 91, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.6008661352540393, 'colsample_bytree': 0.7612630637589181, 'reg_alpha': 2.855588410358939, 'reg_lambda': 8.123533864393996, 'min_split_gain': 0.004714435576174969}. Best is trial 0 with value: 0.0053387572798920075.
[Trial 0] New best: 0.00534
[I 2025-06-15 20:21:33,180] Trial 1 finished with value: 0.004776465104757092 and parameters: {'n_estimators': 295, 'learning_rate': 0.04842087145144959, 'num_leaves': 96, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.9672187851637772, 'colsample_bytree': 0.5995661454424455, 'reg_alpha': 1.5345415831316433, 'reg_lambda': 1.5450258933708616, 'min_split_gain': 0.01718902740911642}. Best is trial 1 with value: 0.004776465104757092.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:21:33,984] Trial 2 finished with value: 0.005444014521183207 and parameters: {'n_estimators': 231, 'learning_rate': 0.14842733305595504, 'num_leaves': 182, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.7971793975610555, 'colsample_bytree': 0.8404727641502683, 'reg_alpha': 7.904131695203808, 'reg_lambda': 7.869610284732136, 'min_split_gain': 0.01728327541800035}. Best is trial 1 with value: 0.004776465104757092.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:21:35,201] Trial 3 finished with value: 0.005074385722661517 and parameters: {'n_estimators': 70, 'learning_rate': 0.04851728572988265, 'num_leaves': 74, 'max_depth': 13, 'min_child_samples': 26, 'subsample': 0.9467797465246046, 'colsample_bytree': 0.6008483982520446, 'reg_alpha': 0.8962685138817783, 'reg_lambda': 6.560055671281604, 'min_split_gain': 0.03705742520117567}. Best is trial 1 with value: 0.004776465104757092.
[I 2025-06-15 20:21:35,204] A new study created in memory with name: no-name-87d3fe10-f4e3-4221-a2f0-aa9a6a9f4238
[I 2025-06-15 20:21:35,281] Trial 0 finished with value: 0.023739937337735856 and parameters: {'n_estimators': 299, 'learning_rate': 0.026059354624207307, 'num_leaves': 172, 'max_depth': 5, 'min_child_samples': 19, 'subsample': 0.755587426147609, 'colsample_bytree': 0.6446028683987267, 'reg_alpha': 5.1609393048680685, 'reg_lambda': 0.24938990746928513, 'min_split_gain': 0.015362981267223925}. Best is trial 0 with value: 0.023739937337735856.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        2033
0           Price_lag_1        1914
14      pct_chg_Load_DA        1343
3         Load_DA_lag_0        1097
5       WindOn_DA_lag_0         971
2           Price_lag_7         899
11           NGas_lag_2         888
16  roll2d_median_Price         789
15       lag168_Load_DA         766
10           Coal_lag_2         633
13            EUA_lag_2         608
12            Oil_lag_2         470
9                  WD_7          56
7                  WD_1          37
8                  WD_6          14
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          53
1           Price_lag_2          11
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 60.05% done
********************* END NS.. 438 ****************************************************
********************* START NS ... 439 ****************************************************
[I 2025-06-15 20:21:39,948] A new study created in memory with name: no-name-174b85e7-83cc-42a6-8bbc-327bd1293a14
[I 2025-06-15 20:21:41,633] Trial 0 finished with value: 0.0014908162372977085 and parameters: {'n_estimators': 295, 'learning_rate': 0.047725323128954666, 'num_leaves': 246, 'max_depth': 5, 'min_child_samples': 30, 'subsample': 0.7322985078691884, 'colsample_bytree': 0.7746341574150848, 'reg_alpha': 1.807316811682853, 'reg_lambda': 2.0866970137128726, 'min_split_gain': 0.011628847480163497}. Best is trial 0 with value: 0.0014908162372977085.
[Trial 0] New best: 0.00149
[I 2025-06-15 20:21:42,081] Trial 1 finished with value: 0.00147017354161767 and parameters: {'n_estimators': 92, 'learning_rate': 0.19157487137856455, 'num_leaves': 152, 'max_depth': 4, 'min_child_samples': 27, 'subsample': 0.9392165248471587, 'colsample_bytree': 0.8254303352885819, 'reg_alpha': 4.46299297637119, 'reg_lambda': 7.667942552073471, 'min_split_gain': 0.041503248863742966}. Best is trial 1 with value: 0.00147017354161767.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:21:42,681] Trial 2 finished with value: 0.0016966935541451135 and parameters: {'n_estimators': 111, 'learning_rate': 0.11111581141274182, 'num_leaves': 70, 'max_depth': 7, 'min_child_samples': 22, 'subsample': 0.7007297411635315, 'colsample_bytree': 0.559260059873242, 'reg_alpha': 9.538354165954313, 'reg_lambda': 1.2744058132147684, 'min_split_gain': 0.025405031775787287}. Best is trial 1 with value: 0.00147017354161767.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:21:43,687] Trial 3 finished with value: 0.0022101264444473723 and parameters: {'n_estimators': 133, 'learning_rate': 0.027154147045428945, 'num_leaves': 156, 'max_depth': 6, 'min_child_samples': 6, 'subsample': 0.5194383646974029, 'colsample_bytree': 0.5323065807112068, 'reg_alpha': 9.849551193899726, 'reg_lambda': 1.2502115641774825, 'min_split_gain': 0.010097411999477314}. Best is trial 1 with value: 0.00147017354161767.
[I 2025-06-15 20:21:43,690] A new study created in memory with name: no-name-c8aa0f6f-21d7-49c7-8a44-ce420041985f
[I 2025-06-15 20:21:43,733] Trial 0 finished with value: 0.005852068957047845 and parameters: {'n_estimators': 112, 'learning_rate': 0.030647311147543686, 'num_leaves': 58, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.9382745249112161, 'colsample_bytree': 0.8245326256625187, 'reg_alpha': 5.613062659599005, 'reg_lambda': 4.979059961200736, 'min_split_gain': 0.025035249072013828}. Best is trial 0 with value: 0.005852068957047845.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         198
1           Price_lag_2         103
14      pct_chg_Load_DA          90
3         Load_DA_lag_0          72
11           NGas_lag_2          44
2           Price_lag_7          40
16  roll2d_median_Price          36
15       lag168_Load_DA          29
5       WindOn_DA_lag_0          24
10           Coal_lag_2          21
13            EUA_lag_2          14
12            Oil_lag_2           9
7                  WD_1           6
9                  WD_7           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          17
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 60.19% done
********************* END NS.. 439 ****************************************************
********************* START NS ... 440 ****************************************************
[I 2025-06-15 20:21:45,996] A new study created in memory with name: no-name-7da4bcba-a683-4194-9d4e-9e480f9fd0db
[I 2025-06-15 20:21:47,880] Trial 0 finished with value: 0.0009803082778963173 and parameters: {'n_estimators': 169, 'learning_rate': 0.018917578294948615, 'num_leaves': 54, 'max_depth': 7, 'min_child_samples': 30, 'subsample': 0.9941192601159855, 'colsample_bytree': 0.5074146729556596, 'reg_alpha': 3.5778448837496866, 'reg_lambda': 3.4653926630714205, 'min_split_gain': 0.0017528721259308768}. Best is trial 0 with value: 0.0009803082778963173.
[Trial 0] New best: 0.00098
[I 2025-06-15 20:21:48,527] Trial 1 finished with value: 0.07793100714110134 and parameters: {'n_estimators': 61, 'learning_rate': 0.01315717661556692, 'num_leaves': 139, 'max_depth': 10, 'min_child_samples': 28, 'subsample': 0.5293503177756269, 'colsample_bytree': 0.7054545483713638, 'reg_alpha': 3.414697860432574, 'reg_lambda': 6.029438417791001, 'min_split_gain': 0.02772963687869113}. Best is trial 0 with value: 0.0009803082778963173.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:21:49,531] Trial 2 finished with value: 0.0004105178588252785 and parameters: {'n_estimators': 104, 'learning_rate': 0.07271538052567114, 'num_leaves': 159, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.6768650466572921, 'colsample_bytree': 0.7322163738702618, 'reg_alpha': 3.1296738104478763, 'reg_lambda': 7.260148843184528, 'min_split_gain': 0.030584444342679713}. Best is trial 2 with value: 0.0004105178588252785.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:21:50,073] Trial 3 finished with value: 0.007004486819092398 and parameters: {'n_estimators': 63, 'learning_rate': 0.032236497364966754, 'num_leaves': 192, 'max_depth': 16, 'min_child_samples': 19, 'subsample': 0.8016754433904469, 'colsample_bytree': 0.8009698758476973, 'reg_alpha': 8.000073577652664, 'reg_lambda': 0.9806724925320376, 'min_split_gain': 0.011296104655366785}. Best is trial 2 with value: 0.0004105178588252785.
[I 2025-06-15 20:21:50,076] A new study created in memory with name: no-name-da6aa55e-e3d1-4d44-bfe1-014efab4e236
[I 2025-06-15 20:21:50,145] Trial 0 finished with value: 0.004634383015293055 and parameters: {'n_estimators': 280, 'learning_rate': 0.042799815066663666, 'num_leaves': 140, 'max_depth': 14, 'min_child_samples': 27, 'subsample': 0.6292756384241545, 'colsample_bytree': 0.8021165432893216, 'reg_alpha': 3.142631408482152, 'reg_lambda': 9.436549635288573, 'min_split_gain': 0.04903479490896092}. Best is trial 0 with value: 0.004634383015293055.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         813
1           Price_lag_2         727
14      pct_chg_Load_DA         607
3         Load_DA_lag_0         426
11           NGas_lag_2         294
2           Price_lag_7         277
16  roll2d_median_Price         268
15       lag168_Load_DA         236
5       WindOn_DA_lag_0         228
10           Coal_lag_2         187
13            EUA_lag_2         184
12            Oil_lag_2         132
9                  WD_7          20
7                  WD_1          13
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          23
10           Coal_lag_2           3
1           Price_lag_2           3
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 60.33% done
********************* END NS.. 440 ****************************************************
********************* START NS ... 441 ****************************************************
[I 2025-06-15 20:21:52,636] A new study created in memory with name: no-name-1592dbe1-4d43-41d8-95fd-e65a00fbcd7f
[I 2025-06-15 20:21:54,968] Trial 0 finished with value: 0.0008563200894364606 and parameters: {'n_estimators': 184, 'learning_rate': 0.04952082060007674, 'num_leaves': 69, 'max_depth': 14, 'min_child_samples': 18, 'subsample': 0.9055220231953673, 'colsample_bytree': 0.5542885540259275, 'reg_alpha': 2.982038585993548, 'reg_lambda': 5.722863390329808, 'min_split_gain': 0.016631898412710816}. Best is trial 0 with value: 0.0008563200894364606.
[Trial 0] New best: 0.00086
[I 2025-06-15 20:21:56,653] Trial 1 finished with value: 0.0007563162982031963 and parameters: {'n_estimators': 235, 'learning_rate': 0.11597194544583592, 'num_leaves': 72, 'max_depth': 16, 'min_child_samples': 30, 'subsample': 0.5769103471891037, 'colsample_bytree': 0.6074054797187851, 'reg_alpha': 9.54186299131656, 'reg_lambda': 3.7212417689705104, 'min_split_gain': 8.668610890661955e-05}. Best is trial 1 with value: 0.0007563162982031963.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:21:57,848] Trial 2 finished with value: 0.0007000249804573417 and parameters: {'n_estimators': 206, 'learning_rate': 0.02433820741918348, 'num_leaves': 160, 'max_depth': 5, 'min_child_samples': 22, 'subsample': 0.9384477940425882, 'colsample_bytree': 0.7939102146566601, 'reg_alpha': 3.876737101437031, 'reg_lambda': 8.287991613654768, 'min_split_gain': 0.012771459015842946}. Best is trial 2 with value: 0.0007000249804573417.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:21:59,813] Trial 3 finished with value: 0.0019933191720941158 and parameters: {'n_estimators': 229, 'learning_rate': 0.012279422188700527, 'num_leaves': 80, 'max_depth': 13, 'min_child_samples': 23, 'subsample': 0.5882929623356283, 'colsample_bytree': 0.9807973616514133, 'reg_alpha': 5.262134164900216, 'reg_lambda': 6.821682915304052, 'min_split_gain': 0.040172053030363225}. Best is trial 2 with value: 0.0007000249804573417.
[I 2025-06-15 20:21:59,818] A new study created in memory with name: no-name-50fc925f-856e-4294-b260-5619bf1c2f4a
[I 2025-06-15 20:21:59,884] Trial 0 finished with value: 0.014485631166850615 and parameters: {'n_estimators': 272, 'learning_rate': 0.03174314836397848, 'num_leaves': 45, 'max_depth': 5, 'min_child_samples': 16, 'subsample': 0.8592743286526521, 'colsample_bytree': 0.8436070993223772, 'reg_alpha': 5.784870801307078, 'reg_lambda': 2.2983343078999363, 'min_split_gain': 0.03472793507238055}. Best is trial 0 with value: 0.014485631166850615.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1657
1           Price_lag_2         801
14      pct_chg_Load_DA         704
3         Load_DA_lag_0         289
2           Price_lag_7         233
11           NGas_lag_2         223
16  roll2d_median_Price         197
13            EUA_lag_2         147
15       lag168_Load_DA         132
12            Oil_lag_2         120
5       WindOn_DA_lag_0          89
10           Coal_lag_2          73
9                  WD_7          15
8                  WD_6           5
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           7
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 60.47% done
********************* END NS.. 441 ****************************************************
********************* START NS ... 442 ****************************************************
[I 2025-06-15 20:22:02,697] A new study created in memory with name: no-name-893db75f-906e-4072-bd07-55a3ce1d9ed7
[I 2025-06-15 20:22:03,989] Trial 0 finished with value: 0.004004322030199073 and parameters: {'n_estimators': 98, 'learning_rate': 0.038198009366630196, 'num_leaves': 193, 'max_depth': 6, 'min_child_samples': 3, 'subsample': 0.7894250871675814, 'colsample_bytree': 0.8952581580449743, 'reg_alpha': 0.6964922253047401, 'reg_lambda': 5.94986027781419, 'min_split_gain': 0.007657441992320236}. Best is trial 0 with value: 0.004004322030199073.
[Trial 0] New best: 0.00400
[I 2025-06-15 20:22:06,224] Trial 1 finished with value: 0.0038169622296201425 and parameters: {'n_estimators': 224, 'learning_rate': 0.025363263147752373, 'num_leaves': 116, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.7414422603200331, 'colsample_bytree': 0.775263381690269, 'reg_alpha': 5.925343467561531, 'reg_lambda': 3.9343620268513133, 'min_split_gain': 0.022262046960056705}. Best is trial 1 with value: 0.0038169622296201425.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:22:06,840] Trial 2 finished with value: 0.0034926847738131817 and parameters: {'n_estimators': 179, 'learning_rate': 0.16288226509111745, 'num_leaves': 245, 'max_depth': 10, 'min_child_samples': 3, 'subsample': 0.9116871420593924, 'colsample_bytree': 0.9940392192268497, 'reg_alpha': 7.202587189591921, 'reg_lambda': 3.793336129119619, 'min_split_gain': 0.01854818110436111}. Best is trial 2 with value: 0.0034926847738131817.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:22:07,316] Trial 3 finished with value: 0.004076518797274155 and parameters: {'n_estimators': 118, 'learning_rate': 0.10511678777654392, 'num_leaves': 228, 'max_depth': 7, 'min_child_samples': 15, 'subsample': 0.9069958223780107, 'colsample_bytree': 0.7687483568026583, 'reg_alpha': 8.112080346407723, 'reg_lambda': 8.084719676914158, 'min_split_gain': 0.04598006474894003}. Best is trial 2 with value: 0.0034926847738131817.
[I 2025-06-15 20:22:07,318] A new study created in memory with name: no-name-c4a391d9-d202-40c3-8532-505d2c1fdcbc
[I 2025-06-15 20:22:07,376] Trial 0 finished with value: 0.010476639756608416 and parameters: {'n_estimators': 254, 'learning_rate': 0.2969877930897528, 'num_leaves': 52, 'max_depth': 16, 'min_child_samples': 18, 'subsample': 0.7037296355542206, 'colsample_bytree': 0.7482145206065325, 'reg_alpha': 2.2331404782333673, 'reg_lambda': 0.12646956977255153, 'min_split_gain': 0.0056576392345941695}. Best is trial 0 with value: 0.010476639756608416.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         378
1           Price_lag_2         271
14      pct_chg_Load_DA         262
11           NGas_lag_2         145
2           Price_lag_7         140
3         Load_DA_lag_0         137
16  roll2d_median_Price         112
5       WindOn_DA_lag_0         108
15       lag168_Load_DA         103
13            EUA_lag_2          86
10           Coal_lag_2          83
12            Oil_lag_2          60
9                  WD_7          12
7                  WD_1           6
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          15
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 60.60% done
********************* END NS.. 442 ****************************************************
********************* START NS ... 443 ****************************************************
[I 2025-06-15 20:22:09,420] A new study created in memory with name: no-name-5f98579a-209e-4ed9-baaa-bf72f231e062
[I 2025-06-15 20:22:10,478] Trial 0 finished with value: 0.00475609487397205 and parameters: {'n_estimators': 229, 'learning_rate': 0.03789932092305441, 'num_leaves': 34, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.9004484836057645, 'colsample_bytree': 0.972349579326139, 'reg_alpha': 1.2222945920235595, 'reg_lambda': 7.131305743071344, 'min_split_gain': 0.041170169462961044}. Best is trial 0 with value: 0.00475609487397205.
[Trial 0] New best: 0.00476
[I 2025-06-15 20:22:11,132] Trial 1 finished with value: 0.00451650602471716 and parameters: {'n_estimators': 176, 'learning_rate': 0.23485863864831802, 'num_leaves': 157, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.7980829365948959, 'colsample_bytree': 0.992459943211489, 'reg_alpha': 1.8910817856831452, 'reg_lambda': 3.609264724704835, 'min_split_gain': 0.01742353691656209}. Best is trial 1 with value: 0.00451650602471716.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:22:11,421] Trial 2 finished with value: 0.004621798127958246 and parameters: {'n_estimators': 232, 'learning_rate': 0.22903960961368253, 'num_leaves': 119, 'max_depth': 4, 'min_child_samples': 24, 'subsample': 0.5488989267220625, 'colsample_bytree': 0.8755284831615795, 'reg_alpha': 8.99770340607649, 'reg_lambda': 8.416156650509052, 'min_split_gain': 0.04277525886976247}. Best is trial 1 with value: 0.00451650602471716.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:22:14,919] Trial 3 finished with value: 0.005454492631686132 and parameters: {'n_estimators': 286, 'learning_rate': 0.01315457374698191, 'num_leaves': 129, 'max_depth': 16, 'min_child_samples': 4, 'subsample': 0.7987407945677716, 'colsample_bytree': 0.9107520136363986, 'reg_alpha': 5.6520966387206695, 'reg_lambda': 4.967498927195888, 'min_split_gain': 0.02125765825984695}. Best is trial 1 with value: 0.00451650602471716.
[I 2025-06-15 20:22:14,922] A new study created in memory with name: no-name-7eb5c38c-4523-4b6b-8f6e-82e8d0f3b41b
[I 2025-06-15 20:22:14,977] Trial 0 finished with value: 0.011825811675179418 and parameters: {'n_estimators': 244, 'learning_rate': 0.2363331698923169, 'num_leaves': 242, 'max_depth': 12, 'min_child_samples': 1, 'subsample': 0.6570766685625837, 'colsample_bytree': 0.6671252229475508, 'reg_alpha': 7.5423708614036595, 'reg_lambda': 5.8814181342481175, 'min_split_gain': 0.016658671027777374}. Best is trial 0 with value: 0.011825811675179418.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         372
14      pct_chg_Load_DA         292
1           Price_lag_2         265
3         Load_DA_lag_0         207
5       WindOn_DA_lag_0         150
11           NGas_lag_2         143
15       lag168_Load_DA         139
16  roll2d_median_Price         136
2           Price_lag_7         124
13            EUA_lag_2          87
10           Coal_lag_2          73
12            Oil_lag_2          64
7                  WD_1           8
9                  WD_7           6
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 60.74% done
********************* END NS.. 443 ****************************************************
********************* START NS ... 444 ****************************************************
[I 2025-06-15 20:22:16,962] A new study created in memory with name: no-name-751fa6eb-0d98-43e3-8d7f-e9b154532ca4
[I 2025-06-15 20:22:19,465] Trial 0 finished with value: 0.006297662664573889 and parameters: {'n_estimators': 167, 'learning_rate': 0.018828871946576314, 'num_leaves': 42, 'max_depth': 16, 'min_child_samples': 19, 'subsample': 0.8040325941084501, 'colsample_bytree': 0.6954709474312193, 'reg_alpha': 5.160981078242905, 'reg_lambda': 4.529524039775331, 'min_split_gain': 0.025703143665919276}. Best is trial 0 with value: 0.006297662664573889.
[Trial 0] New best: 0.00630
[I 2025-06-15 20:22:22,737] Trial 1 finished with value: 0.006002007986373697 and parameters: {'n_estimators': 286, 'learning_rate': 0.012683993106551782, 'num_leaves': 137, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.8395163275227489, 'colsample_bytree': 0.8253701545401115, 'reg_alpha': 3.764120827311551, 'reg_lambda': 1.2907759524709372, 'min_split_gain': 0.015132224031640412}. Best is trial 1 with value: 0.006002007986373697.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:22:23,490] Trial 2 finished with value: 0.005280085622587325 and parameters: {'n_estimators': 85, 'learning_rate': 0.07154274225912427, 'num_leaves': 183, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.8543246795758732, 'colsample_bytree': 0.6885509019898982, 'reg_alpha': 8.35431896859645, 'reg_lambda': 0.733434730970498, 'min_split_gain': 0.03335842451933061}. Best is trial 2 with value: 0.005280085622587325.
[Trial 2] New best: 0.00528
[I 2025-06-15 20:22:27,810] Trial 3 finished with value: 0.005529880421730272 and parameters: {'n_estimators': 281, 'learning_rate': 0.015734749079864427, 'num_leaves': 227, 'max_depth': 12, 'min_child_samples': 3, 'subsample': 0.7868843015300442, 'colsample_bytree': 0.7084651975062761, 'reg_alpha': 6.246886759032582, 'reg_lambda': 1.3309868785154877, 'min_split_gain': 0.0009918993454282432}. Best is trial 2 with value: 0.005280085622587325.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:22:29,375] Trial 4 finished with value: 0.010471902130292752 and parameters: {'n_estimators': 148, 'learning_rate': 0.013764574652667639, 'num_leaves': 45, 'max_depth': 16, 'min_child_samples': 11, 'subsample': 0.9594622587287253, 'colsample_bytree': 0.7934620606234356, 'reg_alpha': 2.473593674929797, 'reg_lambda': 4.653699099312374, 'min_split_gain': 0.010518686593565491}. Best is trial 2 with value: 0.005280085622587325.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:22:31,685] Trial 5 finished with value: 0.0056379351716982495 and parameters: {'n_estimators': 271, 'learning_rate': 0.022192836937559683, 'num_leaves': 216, 'max_depth': 8, 'min_child_samples': 14, 'subsample': 0.6711144274813152, 'colsample_bytree': 0.6185734764601043, 'reg_alpha': 7.72366386992501, 'reg_lambda': 5.255267588602997, 'min_split_gain': 0.03504765737012185}. Best is trial 2 with value: 0.005280085622587325.
[I 2025-06-15 20:22:31,686] A new study created in memory with name: no-name-3722bb42-c532-4b5e-b687-b968e066cc06
[I 2025-06-15 20:22:31,720] Trial 0 finished with value: 0.015758526405079746 and parameters: {'n_estimators': 108, 'learning_rate': 0.143488312945676, 'num_leaves': 174, 'max_depth': 16, 'min_child_samples': 1, 'subsample': 0.5444607952878342, 'colsample_bytree': 0.7581934529646832, 'reg_alpha': 7.162908680035275, 'reg_lambda': 0.8182754558650651, 'min_split_gain': 0.03155534651251072}. Best is trial 0 with value: 0.015758526405079746.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         597
1           Price_lag_2         459
14      pct_chg_Load_DA         340
11           NGas_lag_2         192
2           Price_lag_7         188
3         Load_DA_lag_0         187
16  roll2d_median_Price         184
15       lag168_Load_DA         143
10           Coal_lag_2         118
5       WindOn_DA_lag_0         113
13            EUA_lag_2          99
12            Oil_lag_2          61
9                  WD_7          26
8                  WD_6          20
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 60.88% done
********************* END NS.. 444 ****************************************************
********************* START NS ... 445 ****************************************************
[I 2025-06-15 20:22:33,560] A new study created in memory with name: no-name-ffc30b64-659c-49bf-8e80-1b6a30a14b13
[I 2025-06-15 20:22:33,932] Trial 0 finished with value: 0.027742735085601766 and parameters: {'n_estimators': 154, 'learning_rate': 0.2055632402812382, 'num_leaves': 161, 'max_depth': 11, 'min_child_samples': 23, 'subsample': 0.6240221848454562, 'colsample_bytree': 0.8359131947263267, 'reg_alpha': 9.15548627359529, 'reg_lambda': 7.072092180210017, 'min_split_gain': 0.021513257927379778}. Best is trial 0 with value: 0.027742735085601766.
[Trial 0] New best: 0.02774
[I 2025-06-15 20:22:34,457] Trial 1 finished with value: 0.02998618217117029 and parameters: {'n_estimators': 151, 'learning_rate': 0.2325753647873244, 'num_leaves': 206, 'max_depth': 15, 'min_child_samples': 3, 'subsample': 0.848523173481641, 'colsample_bytree': 0.8511117622400519, 'reg_alpha': 6.708324659964551, 'reg_lambda': 8.38545387899211, 'min_split_gain': 0.040342479073468274}. Best is trial 0 with value: 0.027742735085601766.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:22:36,034] Trial 2 finished with value: 0.028518028799004645 and parameters: {'n_estimators': 210, 'learning_rate': 0.017393376214795326, 'num_leaves': 227, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.5531980645703359, 'colsample_bytree': 0.9707932498268601, 'reg_alpha': 7.813364983226404, 'reg_lambda': 7.5697497601517565, 'min_split_gain': 0.0369482000863524}. Best is trial 0 with value: 0.027742735085601766.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:22:37,123] Trial 3 finished with value: 0.030126153496209576 and parameters: {'n_estimators': 100, 'learning_rate': 0.07237572836933241, 'num_leaves': 249, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.5791884337785247, 'colsample_bytree': 0.8314165865679997, 'reg_alpha': 7.513004262974372, 'reg_lambda': 8.71386561055979, 'min_split_gain': 0.0243527080518829}. Best is trial 0 with value: 0.027742735085601766.
[I 2025-06-15 20:22:37,125] A new study created in memory with name: no-name-547407c0-a6ef-44cc-9543-4da9c6d64be1
[I 2025-06-15 20:22:37,191] Trial 0 finished with value: 0.06508096766391815 and parameters: {'n_estimators': 268, 'learning_rate': 0.08025067246456738, 'num_leaves': 96, 'max_depth': 7, 'min_child_samples': 26, 'subsample': 0.7124017921535409, 'colsample_bytree': 0.7591203968718119, 'reg_alpha': 5.479463401374715, 'reg_lambda': 4.0828148226855765, 'min_split_gain': 0.04281326606272331}. Best is trial 0 with value: 0.06508096766391815.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         234
14      pct_chg_Load_DA         162
1           Price_lag_2         140
3         Load_DA_lag_0          81
2           Price_lag_7          72
16  roll2d_median_Price          68
11           NGas_lag_2          60
15       lag168_Load_DA          44
13            EUA_lag_2          43
5       WindOn_DA_lag_0          41
10           Coal_lag_2          37
12            Oil_lag_2          15
9                  WD_7           8
7                  WD_1           4
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 61.01% done
********************* END NS.. 445 ****************************************************
********************* START NS ... 446 ****************************************************
[I 2025-06-15 20:22:39,087] A new study created in memory with name: no-name-933bae3c-0888-485b-aeac-a6a27ef32c33
[I 2025-06-15 20:22:40,088] Trial 0 finished with value: 0.032211572969912915 and parameters: {'n_estimators': 69, 'learning_rate': 0.06890367115085487, 'num_leaves': 48, 'max_depth': 14, 'min_child_samples': 15, 'subsample': 0.735959929502803, 'colsample_bytree': 0.592042250326579, 'reg_alpha': 2.588077684244179, 'reg_lambda': 0.4993712539954098, 'min_split_gain': 0.04610346524669656}. Best is trial 0 with value: 0.032211572969912915.
[Trial 0] New best: 0.03221
[I 2025-06-15 20:22:40,432] Trial 1 finished with value: 0.06729424328179905 and parameters: {'n_estimators': 55, 'learning_rate': 0.01552068164502718, 'num_leaves': 242, 'max_depth': 5, 'min_child_samples': 4, 'subsample': 0.894282833793751, 'colsample_bytree': 0.755241081151654, 'reg_alpha': 9.209135599697632, 'reg_lambda': 5.926093508560718, 'min_split_gain': 0.04088359360222786}. Best is trial 0 with value: 0.032211572969912915.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:22:40,869] Trial 2 finished with value: 0.03438097715182468 and parameters: {'n_estimators': 121, 'learning_rate': 0.28070773637154334, 'num_leaves': 198, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.5555097652615568, 'colsample_bytree': 0.6287728051893359, 'reg_alpha': 8.953215198741757, 'reg_lambda': 3.922850686270367, 'min_split_gain': 0.010845022558772982}. Best is trial 0 with value: 0.032211572969912915.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:22:43,338] Trial 3 finished with value: 0.034119117440547735 and parameters: {'n_estimators': 107, 'learning_rate': 0.01524035555561702, 'num_leaves': 103, 'max_depth': 12, 'min_child_samples': 28, 'subsample': 0.9579643812377747, 'colsample_bytree': 0.9961712774452423, 'reg_alpha': 0.39699768290389903, 'reg_lambda': 2.8328858975003435, 'min_split_gain': 0.031134588372813507}. Best is trial 0 with value: 0.032211572969912915.
[I 2025-06-15 20:22:43,340] A new study created in memory with name: no-name-16f5ede2-1e06-4897-9145-1ef186c6426a
[I 2025-06-15 20:22:43,381] Trial 0 finished with value: 0.08856053179166229 and parameters: {'n_estimators': 100, 'learning_rate': 0.020102367634584298, 'num_leaves': 221, 'max_depth': 12, 'min_child_samples': 28, 'subsample': 0.9385445176930404, 'colsample_bytree': 0.5988752315227054, 'reg_alpha': 6.3521938698480245, 'reg_lambda': 2.7781054841624977, 'min_split_gain': 0.03403053322650568}. Best is trial 0 with value: 0.08856053179166229.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         661
1           Price_lag_2         610
14      pct_chg_Load_DA         355
3         Load_DA_lag_0         273
16  roll2d_median_Price         198
15       lag168_Load_DA         185
11           NGas_lag_2         180
2           Price_lag_7         176
13            EUA_lag_2         121
5       WindOn_DA_lag_0         118
10           Coal_lag_2         109
12            Oil_lag_2          82
9                  WD_7          12
8                  WD_6           2
7                  WD_1           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          33
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 61.15% done
********************* END NS.. 446 ****************************************************
********************* START NS ... 447 ****************************************************
[I 2025-06-15 20:22:45,468] A new study created in memory with name: no-name-1ca412ad-8033-444b-a6ba-158f13efac11
[I 2025-06-15 20:22:46,864] Trial 0 finished with value: 0.010449327124451797 and parameters: {'n_estimators': 158, 'learning_rate': 0.023169777930613256, 'num_leaves': 90, 'max_depth': 15, 'min_child_samples': 28, 'subsample': 0.9272122118820186, 'colsample_bytree': 0.7199234044237208, 'reg_alpha': 6.23987493347213, 'reg_lambda': 5.565803897823641, 'min_split_gain': 0.049624527615458075}. Best is trial 0 with value: 0.010449327124451797.
[Trial 0] New best: 0.01045
[I 2025-06-15 20:22:49,365] Trial 1 finished with value: 0.010712027826982122 and parameters: {'n_estimators': 270, 'learning_rate': 0.014597159430808396, 'num_leaves': 44, 'max_depth': 13, 'min_child_samples': 27, 'subsample': 0.9608777539328438, 'colsample_bytree': 0.5139846348805409, 'reg_alpha': 5.705308038680018, 'reg_lambda': 1.6889165432200537, 'min_split_gain': 0.046807912043444894}. Best is trial 0 with value: 0.010449327124451797.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:22:52,214] Trial 2 finished with value: 0.009537033888578487 and parameters: {'n_estimators': 212, 'learning_rate': 0.018162015273973084, 'num_leaves': 111, 'max_depth': 15, 'min_child_samples': 21, 'subsample': 0.5236810746123108, 'colsample_bytree': 0.5647257248928743, 'reg_alpha': 2.9040465788736283, 'reg_lambda': 0.02445879487026348, 'min_split_gain': 0.033316982944669275}. Best is trial 2 with value: 0.009537033888578487.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:22:55,078] Trial 3 finished with value: 0.00777084692060474 and parameters: {'n_estimators': 218, 'learning_rate': 0.024910829427006478, 'num_leaves': 64, 'max_depth': 12, 'min_child_samples': 3, 'subsample': 0.8291775674750212, 'colsample_bytree': 0.7143076909466293, 'reg_alpha': 2.5364527072533685, 'reg_lambda': 0.6290429278205778, 'min_split_gain': 0.01247105606416834}. Best is trial 3 with value: 0.00777084692060474.
[Trial 3] New best: 0.00777
[I 2025-06-15 20:22:56,375] Trial 4 finished with value: 0.008822008122773438 and parameters: {'n_estimators': 192, 'learning_rate': 0.05597100089886359, 'num_leaves': 155, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.731319130357575, 'colsample_bytree': 0.5472890186600046, 'reg_alpha': 3.7312759173598886, 'reg_lambda': 7.3574856457426066, 'min_split_gain': 0.02694632395976221}. Best is trial 3 with value: 0.00777084692060474.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:22:57,302] Trial 5 finished with value: 0.007045040775181946 and parameters: {'n_estimators': 179, 'learning_rate': 0.17420641530382283, 'num_leaves': 110, 'max_depth': 9, 'min_child_samples': 20, 'subsample': 0.5660898451765377, 'colsample_bytree': 0.6123870692832771, 'reg_alpha': 5.704959364930803, 'reg_lambda': 1.9736598859428456, 'min_split_gain': 0.003619575879962528}. Best is trial 5 with value: 0.007045040775181946.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:22:58,039] Trial 6 finished with value: 0.011803576045879088 and parameters: {'n_estimators': 250, 'learning_rate': 0.012149989610013083, 'num_leaves': 81, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.5705314410227361, 'colsample_bytree': 0.8514877579948423, 'reg_alpha': 1.9826523541679175, 'reg_lambda': 1.5421681480786442, 'min_split_gain': 0.0049708394908179855}. Best is trial 5 with value: 0.007045040775181946.
[I 2025-06-15 20:22:58,042] A new study created in memory with name: no-name-5d089a0f-2a62-46be-b380-b405a181f0fa
[I 2025-06-15 20:22:58,103] Trial 0 finished with value: 0.031089306953133448 and parameters: {'n_estimators': 268, 'learning_rate': 0.22244430801722787, 'num_leaves': 52, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.6988359786776115, 'colsample_bytree': 0.7803649916412904, 'reg_alpha': 9.406841458509737, 'reg_lambda': 9.70308772667289, 'min_split_gain': 0.00849905022328369}. Best is trial 0 with value: 0.031089306953133448.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         589
1           Price_lag_2         574
14      pct_chg_Load_DA         281
3         Load_DA_lag_0         268
2           Price_lag_7         211
15       lag168_Load_DA         204
5       WindOn_DA_lag_0         204
13            EUA_lag_2         192
16  roll2d_median_Price         186
11           NGas_lag_2         172
12            Oil_lag_2         149
10           Coal_lag_2         139
9                  WD_7          21
7                  WD_1          13
8                  WD_6          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 61.29% done
********************* END NS.. 447 ****************************************************
********************* START NS ... 448 ****************************************************
[I 2025-06-15 20:22:59,919] A new study created in memory with name: no-name-057e8f24-5f6e-46fa-958b-b9009be109db
[I 2025-06-15 20:23:00,879] Trial 0 finished with value: 0.002939879741614525 and parameters: {'n_estimators': 199, 'learning_rate': 0.02163724990588745, 'num_leaves': 102, 'max_depth': 5, 'min_child_samples': 14, 'subsample': 0.7501537073471392, 'colsample_bytree': 0.9549793514203055, 'reg_alpha': 2.2376338984630726, 'reg_lambda': 0.4232547384151075, 'min_split_gain': 0.03804754567940413}. Best is trial 0 with value: 0.002939879741614525.
[Trial 0] New best: 0.00294
[I 2025-06-15 20:23:01,476] Trial 1 finished with value: 0.0030528612806099836 and parameters: {'n_estimators': 89, 'learning_rate': 0.18833464089629334, 'num_leaves': 249, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.5046725643062195, 'colsample_bytree': 0.5362805013133217, 'reg_alpha': 7.6793378859097325, 'reg_lambda': 6.753250316481662, 'min_split_gain': 0.003918454912440372}. Best is trial 0 with value: 0.002939879741614525.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:23:03,776] Trial 2 finished with value: 0.005044005267496179 and parameters: {'n_estimators': 281, 'learning_rate': 0.012242641573752884, 'num_leaves': 117, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.6875301765705776, 'colsample_bytree': 0.606470875918796, 'reg_alpha': 9.397037015576167, 'reg_lambda': 3.980342301475347, 'min_split_gain': 0.03064492450013911}. Best is trial 0 with value: 0.002939879741614525.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:23:04,132] Trial 3 finished with value: 0.0025767003827587144 and parameters: {'n_estimators': 235, 'learning_rate': 0.29680533325371095, 'num_leaves': 197, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.6569185989484474, 'colsample_bytree': 0.5389427731639849, 'reg_alpha': 5.395407723472139, 'reg_lambda': 7.087796555729767, 'min_split_gain': 0.02526899233325197}. Best is trial 3 with value: 0.0025767003827587144.
[I 2025-06-15 20:23:04,135] A new study created in memory with name: no-name-3dc56e9b-c934-4969-accd-bd3e3b035f1a
[I 2025-06-15 20:23:04,219] Trial 0 finished with value: 0.0502668953196977 and parameters: {'n_estimators': 196, 'learning_rate': 0.01660869224902295, 'num_leaves': 90, 'max_depth': 4, 'min_child_samples': 14, 'subsample': 0.9938582303086763, 'colsample_bytree': 0.7441680476256761, 'reg_alpha': 3.3008070724285266, 'reg_lambda': 3.0125970586251984, 'min_split_gain': 0.014085042713264957}. Best is trial 0 with value: 0.0502668953196977.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         245
1           Price_lag_2         213
16  roll2d_median_Price         145
2           Price_lag_7          63
14      pct_chg_Load_DA          62
10           Coal_lag_2          52
3         Load_DA_lag_0          47
11           NGas_lag_2          45
12            Oil_lag_2          32
15       lag168_Load_DA          28
5       WindOn_DA_lag_0          27
13            EUA_lag_2          27
9                  WD_7           6
8                  WD_6           5
7                  WD_1           4
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         235
3         Load_DA_lag_0          64
11           NGas_lag_2          36
1           Price_lag_2          17
13            EUA_lag_2           2
16  roll2d_median_Price           2
12            Oil_lag_2           1
15       lag168_Load_DA           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
5       WindOn_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 61.42% done
********************* END NS.. 448 ****************************************************
********************* START NS ... 449 ****************************************************
[I 2025-06-15 20:23:05,842] A new study created in memory with name: no-name-5625eb05-d581-43f1-9831-6b9f487ccd66
[I 2025-06-15 20:23:07,387] Trial 0 finished with value: 0.003688352501322491 and parameters: {'n_estimators': 183, 'learning_rate': 0.035538579284443095, 'num_leaves': 40, 'max_depth': 9, 'min_child_samples': 1, 'subsample': 0.7334517757752821, 'colsample_bytree': 0.5986994364202372, 'reg_alpha': 7.535473655718627, 'reg_lambda': 3.409164054454814, 'min_split_gain': 0.031009712846125432}. Best is trial 0 with value: 0.003688352501322491.
[Trial 0] New best: 0.00369
[I 2025-06-15 20:23:09,602] Trial 1 finished with value: 0.00300857046737643 and parameters: {'n_estimators': 263, 'learning_rate': 0.05170217473145253, 'num_leaves': 55, 'max_depth': 9, 'min_child_samples': 21, 'subsample': 0.6037407933233174, 'colsample_bytree': 0.7922694567899005, 'reg_alpha': 2.4762617232303175, 'reg_lambda': 0.4341242553865199, 'min_split_gain': 0.009384911172276595}. Best is trial 1 with value: 0.00300857046737643.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:23:12,493] Trial 2 finished with value: 0.004453597089797426 and parameters: {'n_estimators': 226, 'learning_rate': 0.014802178744507443, 'num_leaves': 230, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.9864164966385953, 'colsample_bytree': 0.5856925667858763, 'reg_alpha': 2.4497694512848742, 'reg_lambda': 0.9060307458235595, 'min_split_gain': 0.03816825511358335}. Best is trial 1 with value: 0.00300857046737643.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:23:13,337] Trial 3 finished with value: 0.002833245431758782 and parameters: {'n_estimators': 112, 'learning_rate': 0.14417443113514591, 'num_leaves': 179, 'max_depth': 13, 'min_child_samples': 22, 'subsample': 0.9795946500465704, 'colsample_bytree': 0.9765458291417988, 'reg_alpha': 2.7618949603042475, 'reg_lambda': 7.639149687339336, 'min_split_gain': 0.011004256489279558}. Best is trial 3 with value: 0.002833245431758782.
[I 2025-06-15 20:23:13,339] A new study created in memory with name: no-name-90957b39-11ca-433a-b186-448831222a3a
[I 2025-06-15 20:23:13,425] Trial 0 finished with value: 0.05495474409436018 and parameters: {'n_estimators': 96, 'learning_rate': 0.017678181193428587, 'num_leaves': 74, 'max_depth': 11, 'min_child_samples': 7, 'subsample': 0.6488487763594937, 'colsample_bytree': 0.6604301807279263, 'reg_alpha': 1.9362577409026405, 'reg_lambda': 8.962570800299364, 'min_split_gain': 0.005856061383622407}. Best is trial 0 with value: 0.05495474409436018.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         624
14      pct_chg_Load_DA         571
1           Price_lag_2         467
3         Load_DA_lag_0         393
5       WindOn_DA_lag_0         311
2           Price_lag_7         275
16  roll2d_median_Price         261
15       lag168_Load_DA         251
11           NGas_lag_2         250
13            EUA_lag_2         184
10           Coal_lag_2         155
12            Oil_lag_2         124
7                  WD_1          18
9                  WD_7          15
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         142
3         Load_DA_lag_0          55
14      pct_chg_Load_DA          49
1           Price_lag_2          36
16  roll2d_median_Price          25
11           NGas_lag_2          23
15       lag168_Load_DA          20
5       WindOn_DA_lag_0           8
13            EUA_lag_2           8
2           Price_lag_7           8
12            Oil_lag_2           5
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
8                  WD_6           0
-> 61.56% done
********************* END NS.. 449 ****************************************************
********************* START NS ... 450 ****************************************************
[I 2025-06-15 20:23:15,444] A new study created in memory with name: no-name-841536f5-dbc3-4e1c-b482-269a4d1b480a
[I 2025-06-15 20:23:15,833] Trial 0 finished with value: 0.015270654368536599 and parameters: {'n_estimators': 221, 'learning_rate': 0.20409180796403584, 'num_leaves': 31, 'max_depth': 5, 'min_child_samples': 2, 'subsample': 0.6042781973354874, 'colsample_bytree': 0.9261343822174508, 'reg_alpha': 4.829554711459041, 'reg_lambda': 4.902194483418116, 'min_split_gain': 0.004430411575671311}. Best is trial 0 with value: 0.015270654368536599.
[Trial 0] New best: 0.01527
[I 2025-06-15 20:23:17,472] Trial 1 finished with value: 0.01750909982168852 and parameters: {'n_estimators': 242, 'learning_rate': 0.016163943435743042, 'num_leaves': 37, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.5298204149488732, 'colsample_bytree': 0.7017421534873911, 'reg_alpha': 1.9609097786488638, 'reg_lambda': 6.901294569092324, 'min_split_gain': 0.03872140229615448}. Best is trial 0 with value: 0.015270654368536599.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:23:17,937] Trial 2 finished with value: 0.020621553823882185 and parameters: {'n_estimators': 107, 'learning_rate': 0.11156814004029753, 'num_leaves': 152, 'max_depth': 4, 'min_child_samples': 8, 'subsample': 0.6159438514642193, 'colsample_bytree': 0.8099877021822908, 'reg_alpha': 8.030731301483225, 'reg_lambda': 2.35654680176403, 'min_split_gain': 0.025478626132818108}. Best is trial 0 with value: 0.015270654368536599.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:23:20,436] Trial 3 finished with value: 0.01728165138064086 and parameters: {'n_estimators': 250, 'learning_rate': 0.038441075060971906, 'num_leaves': 69, 'max_depth': 12, 'min_child_samples': 11, 'subsample': 0.6815546175663485, 'colsample_bytree': 0.7741468826251121, 'reg_alpha': 9.296121329230985, 'reg_lambda': 4.703739298387478, 'min_split_gain': 0.002937581099803233}. Best is trial 0 with value: 0.015270654368536599.
[I 2025-06-15 20:23:20,439] A new study created in memory with name: no-name-32e63abf-fc88-4058-90ac-b0f199642c43
[I 2025-06-15 20:23:20,508] Trial 0 finished with value: 0.05645141199110396 and parameters: {'n_estimators': 202, 'learning_rate': 0.0681383137013674, 'num_leaves': 163, 'max_depth': 5, 'min_child_samples': 28, 'subsample': 0.8750450997738599, 'colsample_bytree': 0.7503440751694724, 'reg_alpha': 1.766046954663384, 'reg_lambda': 1.276790113027103, 'min_split_gain': 0.01416145571937787}. Best is trial 0 with value: 0.05645141199110396.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         358
1           Price_lag_2         248
14      pct_chg_Load_DA         230
3         Load_DA_lag_0         156
2           Price_lag_7         122
5       WindOn_DA_lag_0         120
11           NGas_lag_2         114
16  roll2d_median_Price         101
15       lag168_Load_DA          99
13            EUA_lag_2          78
10           Coal_lag_2          74
12            Oil_lag_2          37
7                  WD_1           7
9                  WD_7           7
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          61
14      pct_chg_Load_DA          13
3         Load_DA_lag_0          13
15       lag168_Load_DA          12
2           Price_lag_7          10
16  roll2d_median_Price           8
1           Price_lag_2           7
12            Oil_lag_2           2
5       WindOn_DA_lag_0           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
13            EUA_lag_2           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 61.70% done
********************* END NS.. 450 ****************************************************
********************* START NS ... 451 ****************************************************
[I 2025-06-15 20:23:22,134] A new study created in memory with name: no-name-a4ac6aca-4749-4443-975c-b74772b84a4a
[I 2025-06-15 20:23:23,116] Trial 0 finished with value: 0.017732975939749193 and parameters: {'n_estimators': 123, 'learning_rate': 0.08568013486928665, 'num_leaves': 45, 'max_depth': 16, 'min_child_samples': 3, 'subsample': 0.7282080214993445, 'colsample_bytree': 0.5016706288584429, 'reg_alpha': 2.1469294889292847, 'reg_lambda': 3.9065418222828607, 'min_split_gain': 0.025936010171811727}. Best is trial 0 with value: 0.017732975939749193.
[Trial 0] New best: 0.01773
[I 2025-06-15 20:23:25,217] Trial 1 finished with value: 0.01671711166684183 and parameters: {'n_estimators': 240, 'learning_rate': 0.051193958597631074, 'num_leaves': 141, 'max_depth': 12, 'min_child_samples': 27, 'subsample': 0.8382127532272434, 'colsample_bytree': 0.6959039662234645, 'reg_alpha': 2.0803827609017658, 'reg_lambda': 4.1671548929255025, 'min_split_gain': 0.02050368747346667}. Best is trial 1 with value: 0.01671711166684183.
[Trial 1] New best: 0.01672
[I 2025-06-15 20:23:26,239] Trial 2 finished with value: 0.04553613663210981 and parameters: {'n_estimators': 93, 'learning_rate': 0.0121553488402747, 'num_leaves': 78, 'max_depth': 15, 'min_child_samples': 12, 'subsample': 0.6550342393505932, 'colsample_bytree': 0.5512545884143125, 'reg_alpha': 7.633211226604457, 'reg_lambda': 0.4614775239640967, 'min_split_gain': 0.04115534325685472}. Best is trial 1 with value: 0.01671711166684183.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:23:26,751] Trial 3 finished with value: 0.01811505378463413 and parameters: {'n_estimators': 190, 'learning_rate': 0.12519976245328485, 'num_leaves': 185, 'max_depth': 8, 'min_child_samples': 13, 'subsample': 0.6295766131508926, 'colsample_bytree': 0.6471969574975563, 'reg_alpha': 6.426868385108975, 'reg_lambda': 5.909617069123808, 'min_split_gain': 0.03664838714865664}. Best is trial 1 with value: 0.01671711166684183.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:23:28,850] Trial 4 finished with value: 0.022379734613783217 and parameters: {'n_estimators': 222, 'learning_rate': 0.010383525504413236, 'num_leaves': 41, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.5166626168625741, 'colsample_bytree': 0.7486980776469581, 'reg_alpha': 1.0429222891099255, 'reg_lambda': 0.5142328008074404, 'min_split_gain': 0.044330161977388684}. Best is trial 1 with value: 0.01671711166684183.
[I 2025-06-15 20:23:28,853] A new study created in memory with name: no-name-be6346a1-607f-4e90-bae0-1129423883eb
[I 2025-06-15 20:23:28,993] Trial 0 finished with value: 0.05989997177795048 and parameters: {'n_estimators': 245, 'learning_rate': 0.011851950900803014, 'num_leaves': 75, 'max_depth': 9, 'min_child_samples': 4, 'subsample': 0.6286805875315336, 'colsample_bytree': 0.5768250019117038, 'reg_alpha': 4.88117744000356, 'reg_lambda': 9.422392060990113, 'min_split_gain': 0.02199374472152621}. Best is trial 0 with value: 0.05989997177795048.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1462
1           Price_lag_2        1425
14      pct_chg_Load_DA        1197
3         Load_DA_lag_0         896
5       WindOn_DA_lag_0         642
15       lag168_Load_DA         614
16  roll2d_median_Price         565
2           Price_lag_7         560
11           NGas_lag_2         559
13            EUA_lag_2         465
10           Coal_lag_2         405
12            Oil_lag_2         288
9                  WD_7          45
7                  WD_1          30
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         253
2           Price_lag_7          67
1           Price_lag_2          54
15       lag168_Load_DA          44
3         Load_DA_lag_0          43
12            Oil_lag_2          18
16  roll2d_median_Price          15
7                  WD_1           0
6      WindOff_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
5       WindOn_DA_lag_0           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 61.83% done
********************* END NS.. 451 ****************************************************
********************* START NS ... 452 ****************************************************
[I 2025-06-15 20:23:32,132] A new study created in memory with name: no-name-d6172955-b7d4-4ca0-b7aa-c04e40692d13
[I 2025-06-15 20:23:32,440] Trial 0 finished with value: 0.0012968336727483764 and parameters: {'n_estimators': 197, 'learning_rate': 0.291135846316139, 'num_leaves': 75, 'max_depth': 12, 'min_child_samples': 6, 'subsample': 0.8523051842586415, 'colsample_bytree': 0.9174566090558143, 'reg_alpha': 5.6080570805346, 'reg_lambda': 8.893517981041576, 'min_split_gain': 0.03265972142206267}. Best is trial 0 with value: 0.0012968336727483764.
[Trial 0] New best: 0.00130
[I 2025-06-15 20:23:32,953] Trial 1 finished with value: 0.001272565088658828 and parameters: {'n_estimators': 139, 'learning_rate': 0.1509583048291965, 'num_leaves': 60, 'max_depth': 7, 'min_child_samples': 1, 'subsample': 0.871492855345859, 'colsample_bytree': 0.6034968529416567, 'reg_alpha': 6.552207114346698, 'reg_lambda': 9.930262465001624, 'min_split_gain': 0.024439416073450678}. Best is trial 1 with value: 0.001272565088658828.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:23:34,575] Trial 2 finished with value: 0.003096037652274001 and parameters: {'n_estimators': 163, 'learning_rate': 0.016258345734949928, 'num_leaves': 44, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.9935783811015735, 'colsample_bytree': 0.6845891352789011, 'reg_alpha': 3.5850402435950004, 'reg_lambda': 3.586698759406252, 'min_split_gain': 0.024852477387575074}. Best is trial 1 with value: 0.001272565088658828.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:23:35,321] Trial 3 finished with value: 0.054572550985723285 and parameters: {'n_estimators': 63, 'learning_rate': 0.013859714760988637, 'num_leaves': 228, 'max_depth': 15, 'min_child_samples': 16, 'subsample': 0.9236182293156614, 'colsample_bytree': 0.6579671129010536, 'reg_alpha': 9.072489425645285, 'reg_lambda': 1.2793570525642017, 'min_split_gain': 0.04323474527285351}. Best is trial 1 with value: 0.001272565088658828.
[I 2025-06-15 20:23:35,323] A new study created in memory with name: no-name-41eff3d2-4177-4ea3-9d07-30fbde18dd88
[I 2025-06-15 20:23:35,416] Trial 0 finished with value: 0.0027822443320142166 and parameters: {'n_estimators': 216, 'learning_rate': 0.011796921117820883, 'num_leaves': 214, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.8954690802437048, 'colsample_bytree': 0.5212080097666558, 'reg_alpha': 8.180586334388497, 'reg_lambda': 1.6238346694785244, 'min_split_gain': 0.0044680236698242505}. Best is trial 0 with value: 0.0027822443320142166.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         393
1           Price_lag_2         363
14      pct_chg_Load_DA         191
2           Price_lag_7         178
3         Load_DA_lag_0         173
11           NGas_lag_2         153
16  roll2d_median_Price         123
15       lag168_Load_DA         112
5       WindOn_DA_lag_0         107
13            EUA_lag_2         101
10           Coal_lag_2          75
12            Oil_lag_2          53
9                  WD_7          19
8                  WD_6           9
7                  WD_1           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         137
1           Price_lag_2          40
12            Oil_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 61.97% done
********************* END NS.. 452 ****************************************************
********************* START NS ... 453 ****************************************************
[I 2025-06-15 20:23:37,258] A new study created in memory with name: no-name-6b1040f6-7eb0-451e-8592-0527570df8bb
[I 2025-06-15 20:23:37,665] Trial 0 finished with value: 0.0006547963548987542 and parameters: {'n_estimators': 182, 'learning_rate': 0.11817380300334775, 'num_leaves': 221, 'max_depth': 12, 'min_child_samples': 24, 'subsample': 0.7666242432953959, 'colsample_bytree': 0.7609203004487912, 'reg_alpha': 8.934873393000613, 'reg_lambda': 9.03912489061918, 'min_split_gain': 0.04471022755555315}. Best is trial 0 with value: 0.0006547963548987542.
[Trial 0] New best: 0.00065
[I 2025-06-15 20:23:39,342] Trial 1 finished with value: 0.0004494660077670903 and parameters: {'n_estimators': 126, 'learning_rate': 0.12992375807554324, 'num_leaves': 135, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.9038029907677316, 'colsample_bytree': 0.651696879056656, 'reg_alpha': 0.6178374276001231, 'reg_lambda': 5.258644202809892, 'min_split_gain': 0.011767670640905637}. Best is trial 1 with value: 0.0004494660077670903.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:23:40,802] Trial 2 finished with value: 0.004468391458150472 and parameters: {'n_estimators': 120, 'learning_rate': 0.018932167773557475, 'num_leaves': 135, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.8803552630101, 'colsample_bytree': 0.7795658329045319, 'reg_alpha': 2.1254723185172333, 'reg_lambda': 0.6231305408188703, 'min_split_gain': 0.04513291205432057}. Best is trial 1 with value: 0.0004494660077670903.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:23:43,903] Trial 3 finished with value: 0.0004981055593673702 and parameters: {'n_estimators': 271, 'learning_rate': 0.035245179452340965, 'num_leaves': 186, 'max_depth': 9, 'min_child_samples': 28, 'subsample': 0.7895567911033767, 'colsample_bytree': 0.8647487360765171, 'reg_alpha': 0.1739433924839906, 'reg_lambda': 9.130378003512641, 'min_split_gain': 0.027204775078101873}. Best is trial 1 with value: 0.0004494660077670903.
[I 2025-06-15 20:23:43,907] A new study created in memory with name: no-name-5a3a0220-ee32-42f2-ba57-4781f4a640c3
[I 2025-06-15 20:23:43,937] Trial 0 finished with value: 0.0014976401384479907 and parameters: {'n_estimators': 91, 'learning_rate': 0.17518848877650556, 'num_leaves': 137, 'max_depth': 16, 'min_child_samples': 23, 'subsample': 0.7578670751933376, 'colsample_bytree': 0.7880480558361515, 'reg_alpha': 3.4638502050399667, 'reg_lambda': 3.6609993838658283, 'min_split_gain': 0.0461738617174172}. Best is trial 0 with value: 0.0014976401384479907.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1008
0           Price_lag_1         950
3         Load_DA_lag_0         919
14      pct_chg_Load_DA         919
5       WindOn_DA_lag_0         898
15       lag168_Load_DA         603
16  roll2d_median_Price         584
2           Price_lag_7         557
13            EUA_lag_2         442
11           NGas_lag_2         413
10           Coal_lag_2         375
12            Oil_lag_2         259
9                  WD_7          33
7                  WD_1          23
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          16
11           NGas_lag_2           1
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 62.11% done
********************* END NS.. 453 ****************************************************
********************* START NS ... 454 ****************************************************
[I 2025-06-15 20:23:46,679] A new study created in memory with name: no-name-590f6ac5-bfaa-4d98-ab83-9644eb2e6637
[I 2025-06-15 20:23:47,651] Trial 0 finished with value: 0.004316636134253833 and parameters: {'n_estimators': 158, 'learning_rate': 0.2050849757337143, 'num_leaves': 109, 'max_depth': 16, 'min_child_samples': 1, 'subsample': 0.5343295861564243, 'colsample_bytree': 0.5218064023488832, 'reg_alpha': 7.471816519802889, 'reg_lambda': 1.3781004667032537, 'min_split_gain': 0.0072096238982310434}. Best is trial 0 with value: 0.004316636134253833.
[Trial 0] New best: 0.00432
[I 2025-06-15 20:23:48,371] Trial 1 finished with value: 0.0037859099441230402 and parameters: {'n_estimators': 172, 'learning_rate': 0.05502369917880396, 'num_leaves': 173, 'max_depth': 5, 'min_child_samples': 18, 'subsample': 0.7234010904887376, 'colsample_bytree': 0.8878438502074812, 'reg_alpha': 3.667075053664358, 'reg_lambda': 9.540337252887438, 'min_split_gain': 0.0192085754090693}. Best is trial 1 with value: 0.0037859099441230402.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:23:49,131] Trial 2 finished with value: 0.005255736669918813 and parameters: {'n_estimators': 76, 'learning_rate': 0.04510099109254626, 'num_leaves': 62, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.5980825424961201, 'colsample_bytree': 0.7627072493927449, 'reg_alpha': 9.600676129093115, 'reg_lambda': 2.4438689494432717, 'min_split_gain': 0.03483304832100719}. Best is trial 1 with value: 0.0037859099441230402.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:23:49,768] Trial 3 finished with value: 0.004234715306058113 and parameters: {'n_estimators': 134, 'learning_rate': 0.19203482907713867, 'num_leaves': 75, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.9877141782476956, 'colsample_bytree': 0.5806970495516018, 'reg_alpha': 0.11496829927947316, 'reg_lambda': 2.159830590803117, 'min_split_gain': 0.027757156360909353}. Best is trial 1 with value: 0.0037859099441230402.
[I 2025-06-15 20:23:49,771] A new study created in memory with name: no-name-1eba2734-efa0-46d8-a376-d75ddbd9656c
[I 2025-06-15 20:23:49,837] Trial 0 finished with value: 0.017158312142980057 and parameters: {'n_estimators': 170, 'learning_rate': 0.0347490434835321, 'num_leaves': 186, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.7581097060727665, 'colsample_bytree': 0.926109652847806, 'reg_alpha': 2.46168543486881, 'reg_lambda': 1.121674161749645, 'min_split_gain': 0.017640133481889725}. Best is trial 0 with value: 0.017158312142980057.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         907
14      pct_chg_Load_DA         546
1           Price_lag_2         499
3         Load_DA_lag_0         387
11           NGas_lag_2         278
2           Price_lag_7         212
16  roll2d_median_Price         206
15       lag168_Load_DA         182
5       WindOn_DA_lag_0         181
10           Coal_lag_2         137
13            EUA_lag_2          83
12            Oil_lag_2          68
7                  WD_1          20
9                  WD_7          12
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         147
10           Coal_lag_2          15
1           Price_lag_2          11
15       lag168_Load_DA           8
3         Load_DA_lag_0           3
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 62.24% done
********************* END NS.. 454 ****************************************************
********************* START NS ... 455 ****************************************************
[I 2025-06-15 20:23:51,696] A new study created in memory with name: no-name-0528f039-e451-47d2-abbd-124ebc0437c9
[I 2025-06-15 20:23:53,675] Trial 0 finished with value: 0.005749241973281285 and parameters: {'n_estimators': 191, 'learning_rate': 0.02472864505692011, 'num_leaves': 125, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.8283231023980615, 'colsample_bytree': 0.7534948042406828, 'reg_alpha': 9.34848010333343, 'reg_lambda': 8.96721524147709, 'min_split_gain': 0.006110364030675941}. Best is trial 0 with value: 0.005749241973281285.
[Trial 0] New best: 0.00575
[I 2025-06-15 20:23:53,996] Trial 1 finished with value: 0.004946888331945746 and parameters: {'n_estimators': 195, 'learning_rate': 0.2716222142011918, 'num_leaves': 58, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.5952474721644552, 'colsample_bytree': 0.7282593256661992, 'reg_alpha': 3.4955604848481348, 'reg_lambda': 5.203451650538005, 'min_split_gain': 0.04748875843286126}. Best is trial 1 with value: 0.004946888331945746.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:23:54,353] Trial 2 finished with value: 0.004734216310631439 and parameters: {'n_estimators': 182, 'learning_rate': 0.18135232055116898, 'num_leaves': 77, 'max_depth': 16, 'min_child_samples': 26, 'subsample': 0.5210525319313528, 'colsample_bytree': 0.6400821723268065, 'reg_alpha': 7.066624295208674, 'reg_lambda': 5.571383442592273, 'min_split_gain': 0.04803740670661673}. Best is trial 2 with value: 0.004734216310631439.
[Trial 2] New best: 0.00473
[I 2025-06-15 20:23:54,759] Trial 3 finished with value: 0.0045052405098408655 and parameters: {'n_estimators': 178, 'learning_rate': 0.19821139311485542, 'num_leaves': 200, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.666865054770568, 'colsample_bytree': 0.9896091612525959, 'reg_alpha': 6.02068624400599, 'reg_lambda': 7.6470932072911, 'min_split_gain': 0.019592596679204318}. Best is trial 3 with value: 0.0045052405098408655.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:23:59,270] Trial 4 finished with value: 0.005429268284717095 and parameters: {'n_estimators': 199, 'learning_rate': 0.025888664380720735, 'num_leaves': 173, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.8570497469495093, 'colsample_bytree': 0.5393630071453218, 'reg_alpha': 1.4595055546726177, 'reg_lambda': 1.9345572947207135, 'min_split_gain': 0.0330155269083949}. Best is trial 3 with value: 0.0045052405098408655.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:23:59,914] Trial 5 finished with value: 0.043843331648663476 and parameters: {'n_estimators': 97, 'learning_rate': 0.01382123849139844, 'num_leaves': 65, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.5738707332547649, 'colsample_bytree': 0.6600510486186508, 'reg_alpha': 1.165461937078418, 'reg_lambda': 7.4449526648402085, 'min_split_gain': 0.02211854427907485}. Best is trial 3 with value: 0.0045052405098408655.
[I 2025-06-15 20:23:59,917] A new study created in memory with name: no-name-7699d8e6-3b97-4439-99a1-b7849731251a
[I 2025-06-15 20:23:59,988] Trial 0 finished with value: 0.022028609002706156 and parameters: {'n_estimators': 234, 'learning_rate': 0.20311131743969946, 'num_leaves': 78, 'max_depth': 12, 'min_child_samples': 9, 'subsample': 0.8852849865255693, 'colsample_bytree': 0.5567014784040536, 'reg_alpha': 0.5500579981648368, 'reg_lambda': 6.795458128071702, 'min_split_gain': 0.040017958322954815}. Best is trial 0 with value: 0.022028609002706156.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         290
14      pct_chg_Load_DA         225
1           Price_lag_2         177
3         Load_DA_lag_0         109
2           Price_lag_7          99
11           NGas_lag_2          98
16  roll2d_median_Price          98
5       WindOn_DA_lag_0          86
15       lag168_Load_DA          80
13            EUA_lag_2          62
10           Coal_lag_2          56
12            Oil_lag_2          39
9                  WD_7           6
7                  WD_1           4
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          22
1           Price_lag_2           5
15       lag168_Load_DA           4
11           NGas_lag_2           3
16  roll2d_median_Price           2
7                  WD_1           1
3         Load_DA_lag_0           1
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 62.38% done
********************* END NS.. 455 ****************************************************
********************* START NS ... 456 ****************************************************
[I 2025-06-15 20:24:01,808] A new study created in memory with name: no-name-b2cca90e-a775-45fa-951c-76e7b1901655
[I 2025-06-15 20:24:04,721] Trial 0 finished with value: 0.0030627398855025355 and parameters: {'n_estimators': 270, 'learning_rate': 0.021109042020226912, 'num_leaves': 240, 'max_depth': 13, 'min_child_samples': 16, 'subsample': 0.821964940859224, 'colsample_bytree': 0.7423819637577632, 'reg_alpha': 9.637310065834995, 'reg_lambda': 6.653138137777778, 'min_split_gain': 0.0024242215785316382}. Best is trial 0 with value: 0.0030627398855025355.
[Trial 0] New best: 0.00306
[I 2025-06-15 20:24:05,621] Trial 1 finished with value: 0.0037731910225241414 and parameters: {'n_estimators': 52, 'learning_rate': 0.057817690740814574, 'num_leaves': 113, 'max_depth': 12, 'min_child_samples': 14, 'subsample': 0.601818523992306, 'colsample_bytree': 0.7166437030255364, 'reg_alpha': 0.4479239525619516, 'reg_lambda': 6.735231166399044, 'min_split_gain': 0.039840032960156695}. Best is trial 0 with value: 0.0030627398855025355.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:24:06,350] Trial 2 finished with value: 0.002639695326570601 and parameters: {'n_estimators': 208, 'learning_rate': 0.1563781911350318, 'num_leaves': 106, 'max_depth': 15, 'min_child_samples': 29, 'subsample': 0.5107782318539108, 'colsample_bytree': 0.548685438888094, 'reg_alpha': 4.3580274259906435, 'reg_lambda': 2.8006602614959153, 'min_split_gain': 0.018827519327160116}. Best is trial 2 with value: 0.002639695326570601.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:24:07,011] Trial 3 finished with value: 0.002751852687182822 and parameters: {'n_estimators': 78, 'learning_rate': 0.04377337652418864, 'num_leaves': 250, 'max_depth': 12, 'min_child_samples': 18, 'subsample': 0.9238656804550616, 'colsample_bytree': 0.8776727963944791, 'reg_alpha': 4.930700963187633, 'reg_lambda': 9.948508876143729, 'min_split_gain': 0.03766029394896401}. Best is trial 2 with value: 0.002639695326570601.
[I 2025-06-15 20:24:07,013] A new study created in memory with name: no-name-0336639a-131a-40ec-85a7-722252262829
[I 2025-06-15 20:24:07,061] Trial 0 finished with value: 0.0045964989970814565 and parameters: {'n_estimators': 111, 'learning_rate': 0.07909946035759918, 'num_leaves': 127, 'max_depth': 10, 'min_child_samples': 27, 'subsample': 0.7047722315514127, 'colsample_bytree': 0.8780047639097264, 'reg_alpha': 0.7211768413094988, 'reg_lambda': 2.124463148227369, 'min_split_gain': 0.010428643922375769}. Best is trial 0 with value: 0.0045964989970814565.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         544
1           Price_lag_2         508
16  roll2d_median_Price         238
3         Load_DA_lag_0         234
14      pct_chg_Load_DA         223
5       WindOn_DA_lag_0         177
11           NGas_lag_2         162
2           Price_lag_7         161
15       lag168_Load_DA         140
13            EUA_lag_2         132
12            Oil_lag_2         125
10           Coal_lag_2          95
9                  WD_7          16
8                  WD_6          15
7                  WD_1          11
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          52
15       lag168_Load_DA          22
16  roll2d_median_Price          11
10           Coal_lag_2          10
5       WindOn_DA_lag_0           6
1           Price_lag_2           6
2           Price_lag_7           4
3         Load_DA_lag_0           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 62.52% done
********************* END NS.. 456 ****************************************************
********************* START NS ... 457 ****************************************************
[I 2025-06-15 20:24:08,934] A new study created in memory with name: no-name-cdf8214b-5ecc-4d23-ae74-783a79f263f5
[I 2025-06-15 20:24:10,116] Trial 0 finished with value: 0.0019239017582110274 and parameters: {'n_estimators': 271, 'learning_rate': 0.13133488291794823, 'num_leaves': 149, 'max_depth': 11, 'min_child_samples': 1, 'subsample': 0.5622853938442551, 'colsample_bytree': 0.8708648510105604, 'reg_alpha': 5.50360039700954, 'reg_lambda': 6.753331994322382, 'min_split_gain': 0.012825622842954366}. Best is trial 0 with value: 0.0019239017582110274.
[Trial 0] New best: 0.00192
[I 2025-06-15 20:24:10,405] Trial 1 finished with value: 0.0018219046524790572 and parameters: {'n_estimators': 90, 'learning_rate': 0.27069879767386573, 'num_leaves': 31, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.9097785741556683, 'colsample_bytree': 0.8635350960194252, 'reg_alpha': 2.955057002777035, 'reg_lambda': 0.6606708212714119, 'min_split_gain': 0.017485602692640528}. Best is trial 1 with value: 0.0018219046524790572.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:24:11,097] Trial 2 finished with value: 0.001588011633898151 and parameters: {'n_estimators': 83, 'learning_rate': 0.21006816284198035, 'num_leaves': 157, 'max_depth': 11, 'min_child_samples': 10, 'subsample': 0.9736911232086778, 'colsample_bytree': 0.9011993043174447, 'reg_alpha': 3.712252838480196, 'reg_lambda': 2.6518198260289747, 'min_split_gain': 0.011084667980544388}. Best is trial 2 with value: 0.001588011633898151.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:24:12,784] Trial 3 finished with value: 0.001455038253011873 and parameters: {'n_estimators': 209, 'learning_rate': 0.0830640867928774, 'num_leaves': 113, 'max_depth': 7, 'min_child_samples': 3, 'subsample': 0.8420650147641304, 'colsample_bytree': 0.7927051569208758, 'reg_alpha': 2.3008302719215568, 'reg_lambda': 0.1762111418537149, 'min_split_gain': 0.0029422001125459286}. Best is trial 3 with value: 0.001455038253011873.
[I 2025-06-15 20:24:12,787] A new study created in memory with name: no-name-3e2043d0-d7f4-4526-adfe-2af3595bcf71
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 20:24:13,021] Trial 0 finished with value: 0.0017560168880688591 and parameters: {'n_estimators': 197, 'learning_rate': 0.017702268301852682, 'num_leaves': 232, 'max_depth': 7, 'min_child_samples': 29, 'subsample': 0.5971330347721862, 'colsample_bytree': 0.6908691055833327, 'reg_alpha': 1.9426180844375085, 'reg_lambda': 9.666217072120316, 'min_split_gain': 0.009097860502598466}. Best is trial 0 with value: 0.0017560168880688591.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1402
1           Price_lag_2        1281
14      pct_chg_Load_DA        1138
3         Load_DA_lag_0         939
5       WindOn_DA_lag_0         867
15       lag168_Load_DA         731
2           Price_lag_7         704
11           NGas_lag_2         591
16  roll2d_median_Price         571
13            EUA_lag_2         494
12            Oil_lag_2         488
10           Coal_lag_2         403
9                  WD_7          40
7                  WD_1          19
8                  WD_6          15
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         193
1           Price_lag_2          29
16  roll2d_median_Price          11
3         Load_DA_lag_0           2
2           Price_lag_7           1
12            Oil_lag_2           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 62.65% done
********************* END NS.. 457 ****************************************************
********************* START NS ... 458 ****************************************************
[I 2025-06-15 20:24:15,985] A new study created in memory with name: no-name-cfc0165c-2b4f-46b2-8b2f-2ab29d4b9fb6
[I 2025-06-15 20:24:18,366] Trial 0 finished with value: 0.010652087484293374 and parameters: {'n_estimators': 204, 'learning_rate': 0.010994483202088378, 'num_leaves': 154, 'max_depth': 13, 'min_child_samples': 7, 'subsample': 0.6485787269823593, 'colsample_bytree': 0.7536716971545723, 'reg_alpha': 3.09352659898415, 'reg_lambda': 8.52146534224007, 'min_split_gain': 0.03870351235664442}. Best is trial 0 with value: 0.010652087484293374.
[Trial 0] New best: 0.01065
[I 2025-06-15 20:24:20,584] Trial 1 finished with value: 0.0043472085598789895 and parameters: {'n_estimators': 266, 'learning_rate': 0.027320822691655148, 'num_leaves': 204, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.5358644573361186, 'colsample_bytree': 0.7962572580881979, 'reg_alpha': 3.7950360317071286, 'reg_lambda': 2.703067131522295, 'min_split_gain': 0.040248156533635185}. Best is trial 1 with value: 0.0043472085598789895.
[Trial 1] New best: 0.00435
[I 2025-06-15 20:24:20,837] Trial 2 finished with value: 0.005189947356022184 and parameters: {'n_estimators': 189, 'learning_rate': 0.20231924083770778, 'num_leaves': 44, 'max_depth': 4, 'min_child_samples': 8, 'subsample': 0.8069114366335016, 'colsample_bytree': 0.8498299792227946, 'reg_alpha': 5.00994340739355, 'reg_lambda': 2.5990023623130476, 'min_split_gain': 0.04265355722710492}. Best is trial 1 with value: 0.0043472085598789895.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:24:21,753] Trial 3 finished with value: 0.004940627169319361 and parameters: {'n_estimators': 200, 'learning_rate': 0.021638622791475652, 'num_leaves': 52, 'max_depth': 5, 'min_child_samples': 19, 'subsample': 0.6015501187443795, 'colsample_bytree': 0.6799409162281642, 'reg_alpha': 2.7560644892360218, 'reg_lambda': 6.722583581989938, 'min_split_gain': 0.04043306670391481}. Best is trial 1 with value: 0.0043472085598789895.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:24:22,209] Trial 4 finished with value: 0.005310347791016789 and parameters: {'n_estimators': 77, 'learning_rate': 0.09938866103670305, 'num_leaves': 157, 'max_depth': 8, 'min_child_samples': 16, 'subsample': 0.8788940018948039, 'colsample_bytree': 0.6638836030177873, 'reg_alpha': 9.936784974748525, 'reg_lambda': 0.4208716266234436, 'min_split_gain': 0.049906853904593394}. Best is trial 1 with value: 0.0043472085598789895.
[I 2025-06-15 20:24:22,211] A new study created in memory with name: no-name-0eb734ca-fbe1-49db-9cef-8747170eac76
[I 2025-06-15 20:24:22,246] Trial 0 finished with value: 0.016440268780743655 and parameters: {'n_estimators': 149, 'learning_rate': 0.01205179279961127, 'num_leaves': 36, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.8947593331154394, 'colsample_bytree': 0.7391129427481045, 'reg_alpha': 6.0712776077059605, 'reg_lambda': 6.139649286476611, 'min_split_gain': 0.043712181909143694}. Best is trial 0 with value: 0.016440268780743655.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2170
1           Price_lag_2        1820
14      pct_chg_Load_DA        1409
3         Load_DA_lag_0         802
11           NGas_lag_2         660
16  roll2d_median_Price         633
2           Price_lag_7         613
15       lag168_Load_DA         511
5       WindOn_DA_lag_0         467
10           Coal_lag_2         317
12            Oil_lag_2         286
13            EUA_lag_2         281
9                  WD_7          33
7                  WD_1          29
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          11
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 62.79% done
********************* END NS.. 458 ****************************************************
********************* START NS ... 459 ****************************************************
[I 2025-06-15 20:24:25,643] A new study created in memory with name: no-name-0dac5e84-8bf8-4b4f-b28a-03e2a0d47e1e
[I 2025-06-15 20:24:25,895] Trial 0 finished with value: 0.011882139156209408 and parameters: {'n_estimators': 88, 'learning_rate': 0.037159805650644515, 'num_leaves': 119, 'max_depth': 4, 'min_child_samples': 21, 'subsample': 0.513430918610772, 'colsample_bytree': 0.7571495083629487, 'reg_alpha': 7.700585826977866, 'reg_lambda': 9.36565356784059, 'min_split_gain': 0.02372152463797081}. Best is trial 0 with value: 0.011882139156209408.
[Trial 0] New best: 0.01188
[I 2025-06-15 20:24:29,073] Trial 1 finished with value: 0.011395013833737767 and parameters: {'n_estimators': 269, 'learning_rate': 0.010958671489010535, 'num_leaves': 64, 'max_depth': 12, 'min_child_samples': 3, 'subsample': 0.6054089303407246, 'colsample_bytree': 0.5788390901030307, 'reg_alpha': 5.628650384440618, 'reg_lambda': 2.6896702873658374, 'min_split_gain': 0.009449753158200581}. Best is trial 1 with value: 0.011395013833737767.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:24:29,415] Trial 2 finished with value: 0.008844443928195793 and parameters: {'n_estimators': 222, 'learning_rate': 0.19187722076265873, 'num_leaves': 227, 'max_depth': 12, 'min_child_samples': 18, 'subsample': 0.923878186984685, 'colsample_bytree': 0.8320130587605978, 'reg_alpha': 6.601594755059884, 'reg_lambda': 6.195713527450853, 'min_split_gain': 0.04595982005878374}. Best is trial 2 with value: 0.008844443928195793.
[Trial 2] New best: 0.00884
[I 2025-06-15 20:24:30,168] Trial 3 finished with value: 0.008357464644499761 and parameters: {'n_estimators': 230, 'learning_rate': 0.25532270186027045, 'num_leaves': 230, 'max_depth': 14, 'min_child_samples': 14, 'subsample': 0.712892173802748, 'colsample_bytree': 0.8910560057727432, 'reg_alpha': 7.244498832028476, 'reg_lambda': 5.281832312024765, 'min_split_gain': 0.0007537513663452068}. Best is trial 3 with value: 0.008357464644499761.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:24:31,010] Trial 4 finished with value: 0.008571421670235516 and parameters: {'n_estimators': 120, 'learning_rate': 0.047762732989582314, 'num_leaves': 126, 'max_depth': 13, 'min_child_samples': 27, 'subsample': 0.692048397230429, 'colsample_bytree': 0.8974409315467529, 'reg_alpha': 6.935013919844746, 'reg_lambda': 0.07853123630351844, 'min_split_gain': 0.03167458203498804}. Best is trial 3 with value: 0.008357464644499761.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:24:31,880] Trial 5 finished with value: 0.010722507914952023 and parameters: {'n_estimators': 92, 'learning_rate': 0.04212421580881269, 'num_leaves': 162, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.8337749044484574, 'colsample_bytree': 0.6028640369620593, 'reg_alpha': 5.462680706583734, 'reg_lambda': 5.030332808704964, 'min_split_gain': 0.038552795319325615}. Best is trial 3 with value: 0.008357464644499761.
[I 2025-06-15 20:24:31,882] A new study created in memory with name: no-name-1dd63cdc-1eaa-4649-8c5f-c7096ea4a480
[I 2025-06-15 20:24:31,908] Trial 0 finished with value: 0.054261683129278794 and parameters: {'n_estimators': 72, 'learning_rate': 0.11956844528448672, 'num_leaves': 100, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.7794425784088659, 'colsample_bytree': 0.8040386304815197, 'reg_alpha': 6.82470983053583, 'reg_lambda': 2.215705395619442, 'min_split_gain': 0.0027458301365190178}. Best is trial 0 with value: 0.054261683129278794.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         421
1           Price_lag_2         347
14      pct_chg_Load_DA         318
5       WindOn_DA_lag_0         299
3         Load_DA_lag_0         271
15       lag168_Load_DA         240
13            EUA_lag_2         224
2           Price_lag_7         208
16  roll2d_median_Price         196
11           NGas_lag_2         173
10           Coal_lag_2         160
12            Oil_lag_2         150
9                  WD_7          18
8                  WD_6          10
7                  WD_1           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          15
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 62.93% done
********************* END NS.. 459 ****************************************************
********************* START NS ... 460 ****************************************************
[I 2025-06-15 20:24:33,873] A new study created in memory with name: no-name-9ed03975-fd38-4d60-a5e6-4f07d2c7f051
[I 2025-06-15 20:24:38,160] Trial 0 finished with value: 0.007090185774685444 and parameters: {'n_estimators': 283, 'learning_rate': 0.015005296550363775, 'num_leaves': 92, 'max_depth': 9, 'min_child_samples': 15, 'subsample': 0.5007521281689615, 'colsample_bytree': 0.9077224345491465, 'reg_alpha': 0.8259695751689833, 'reg_lambda': 5.7962319071975665, 'min_split_gain': 0.008306143925702952}. Best is trial 0 with value: 0.007090185774685444.
[Trial 0] New best: 0.00709
[I 2025-06-15 20:24:38,673] Trial 1 finished with value: 0.007203225340609819 and parameters: {'n_estimators': 54, 'learning_rate': 0.17637158107546408, 'num_leaves': 172, 'max_depth': 11, 'min_child_samples': 8, 'subsample': 0.8471957420867783, 'colsample_bytree': 0.8958695428123588, 'reg_alpha': 2.5011696186180576, 'reg_lambda': 0.9643157748167874, 'min_split_gain': 0.03493150142414683}. Best is trial 0 with value: 0.007090185774685444.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:24:41,295] Trial 2 finished with value: 0.008229285455068158 and parameters: {'n_estimators': 236, 'learning_rate': 0.027374616024873892, 'num_leaves': 245, 'max_depth': 13, 'min_child_samples': 12, 'subsample': 0.777897307656928, 'colsample_bytree': 0.6279502640750283, 'reg_alpha': 5.288454211348323, 'reg_lambda': 4.672232025464341, 'min_split_gain': 0.017934538369308053}. Best is trial 0 with value: 0.007090185774685444.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:24:42,535] Trial 3 finished with value: 0.012688854109052766 and parameters: {'n_estimators': 120, 'learning_rate': 0.017173083072614784, 'num_leaves': 198, 'max_depth': 14, 'min_child_samples': 20, 'subsample': 0.5284643154965003, 'colsample_bytree': 0.7036823698148575, 'reg_alpha': 4.404963012337112, 'reg_lambda': 1.5987268680823508, 'min_split_gain': 0.0038190709474943464}. Best is trial 0 with value: 0.007090185774685444.
[I 2025-06-15 20:24:42,536] A new study created in memory with name: no-name-0e3e4f89-abf1-4e12-a7b3-d9d8c5eb1f4d
[I 2025-06-15 20:24:42,580] Trial 0 finished with value: 0.037837959063909944 and parameters: {'n_estimators': 72, 'learning_rate': 0.16736969220532502, 'num_leaves': 57, 'max_depth': 14, 'min_child_samples': 21, 'subsample': 0.8026369151068973, 'colsample_bytree': 0.9630150677758054, 'reg_alpha': 1.4796500664044576, 'reg_lambda': 1.3186446919298267, 'min_split_gain': 0.0004673204538467757}. Best is trial 0 with value: 0.037837959063909944.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        5262
1           Price_lag_2        4566
14      pct_chg_Load_DA        3436
16  roll2d_median_Price        1860
2           Price_lag_7        1730
11           NGas_lag_2        1582
3         Load_DA_lag_0        1537
13            EUA_lag_2         966
10           Coal_lag_2         848
15       lag168_Load_DA         844
5       WindOn_DA_lag_0         815
12            Oil_lag_2         656
9                  WD_7          38
7                  WD_1          28
8                  WD_6          10
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          52
15       lag168_Load_DA          17
3         Load_DA_lag_0          13
11           NGas_lag_2           4
2           Price_lag_7           2
5       WindOn_DA_lag_0           2
1           Price_lag_2           2
10           Coal_lag_2           1
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 63.06% done
********************* END NS.. 460 ****************************************************
********************* START NS ... 461 ****************************************************
[I 2025-06-15 20:24:48,135] A new study created in memory with name: no-name-6cd9fbdd-4033-4f25-9be4-389f31035ef7
[I 2025-06-15 20:24:49,869] Trial 0 finished with value: 0.003350434746599838 and parameters: {'n_estimators': 167, 'learning_rate': 0.10592347944678995, 'num_leaves': 133, 'max_depth': 7, 'min_child_samples': 14, 'subsample': 0.932451742573339, 'colsample_bytree': 0.7810033317839502, 'reg_alpha': 2.8574146268328904, 'reg_lambda': 7.12787016572387, 'min_split_gain': 0.00032026714274550064}. Best is trial 0 with value: 0.003350434746599838.
[Trial 0] New best: 0.00335
[I 2025-06-15 20:24:50,323] Trial 1 finished with value: 0.0031220627440687273 and parameters: {'n_estimators': 173, 'learning_rate': 0.2174664469400381, 'num_leaves': 72, 'max_depth': 15, 'min_child_samples': 10, 'subsample': 0.9756253063748432, 'colsample_bytree': 0.9440154042950859, 'reg_alpha': 5.167301604355961, 'reg_lambda': 9.86355416697908, 'min_split_gain': 0.027482489108252958}. Best is trial 1 with value: 0.0031220627440687273.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:24:51,321] Trial 2 finished with value: 0.003356823317374078 and parameters: {'n_estimators': 153, 'learning_rate': 0.04419017489878433, 'num_leaves': 84, 'max_depth': 6, 'min_child_samples': 29, 'subsample': 0.6168914960940459, 'colsample_bytree': 0.8040716620844788, 'reg_alpha': 1.9453692562668168, 'reg_lambda': 1.8017124607087054, 'min_split_gain': 0.027455211315440126}. Best is trial 1 with value: 0.0031220627440687273.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:24:53,593] Trial 3 finished with value: 0.003723675553743188 and parameters: {'n_estimators': 229, 'learning_rate': 0.016263114861604703, 'num_leaves': 91, 'max_depth': 16, 'min_child_samples': 20, 'subsample': 0.9802065867504965, 'colsample_bytree': 0.9376143237667918, 'reg_alpha': 4.838995187470724, 'reg_lambda': 8.233072968923658, 'min_split_gain': 0.027881435939913712}. Best is trial 1 with value: 0.0031220627440687273.
[I 2025-06-15 20:24:53,596] A new study created in memory with name: no-name-929b32aa-d7ec-4d67-8fb8-b60314dcaccf
[I 2025-06-15 20:24:53,646] Trial 0 finished with value: 0.012790272279354899 and parameters: {'n_estimators': 207, 'learning_rate': 0.11664720224920729, 'num_leaves': 138, 'max_depth': 10, 'min_child_samples': 8, 'subsample': 0.8351502120509682, 'colsample_bytree': 0.9520503431980228, 'reg_alpha': 6.930088631041471, 'reg_lambda': 1.9106384256408526, 'min_split_gain': 0.038348907873630385}. Best is trial 0 with value: 0.012790272279354899.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         260
1           Price_lag_2         186
14      pct_chg_Load_DA         180
3         Load_DA_lag_0         126
16  roll2d_median_Price          99
2           Price_lag_7          96
11           NGas_lag_2          88
5       WindOn_DA_lag_0          66
15       lag168_Load_DA          61
13            EUA_lag_2          51
10           Coal_lag_2          41
12            Oil_lag_2          30
9                  WD_7           8
7                  WD_1           7
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 63.20% done
********************* END NS.. 461 ****************************************************
********************* START NS ... 462 ****************************************************
[I 2025-06-15 20:24:55,357] A new study created in memory with name: no-name-ded71c51-273b-4931-8e35-52f638faad15
[I 2025-06-15 20:24:58,074] Trial 0 finished with value: 0.004573531033771463 and parameters: {'n_estimators': 237, 'learning_rate': 0.012257003040556831, 'num_leaves': 248, 'max_depth': 8, 'min_child_samples': 29, 'subsample': 0.5914431610465067, 'colsample_bytree': 0.7545357074258596, 'reg_alpha': 1.7400816057507695, 'reg_lambda': 6.05219915936577, 'min_split_gain': 0.00612308396227006}. Best is trial 0 with value: 0.004573531033771463.
[Trial 0] New best: 0.00457
[I 2025-06-15 20:24:59,252] Trial 1 finished with value: 0.0038378571396930007 and parameters: {'n_estimators': 96, 'learning_rate': 0.034351973245444395, 'num_leaves': 68, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.6633237992569352, 'colsample_bytree': 0.6310637463870858, 'reg_alpha': 3.5412513168392703, 'reg_lambda': 0.6465582017715021, 'min_split_gain': 0.04815075038270048}. Best is trial 1 with value: 0.0038378571396930007.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:25:01,393] Trial 2 finished with value: 0.003391641684408733 and parameters: {'n_estimators': 150, 'learning_rate': 0.022583985496353643, 'num_leaves': 156, 'max_depth': 14, 'min_child_samples': 30, 'subsample': 0.9597368275053805, 'colsample_bytree': 0.7608031811892082, 'reg_alpha': 2.5324044175610516, 'reg_lambda': 1.7078332156921727, 'min_split_gain': 0.009027339217805542}. Best is trial 2 with value: 0.003391641684408733.
[Trial 2] New best: 0.00339
[I 2025-06-15 20:25:02,946] Trial 3 finished with value: 0.006686107091935765 and parameters: {'n_estimators': 205, 'learning_rate': 0.012023771409301139, 'num_leaves': 45, 'max_depth': 15, 'min_child_samples': 24, 'subsample': 0.8855435858797212, 'colsample_bytree': 0.9092506133601359, 'reg_alpha': 6.553497757191962, 'reg_lambda': 3.7866713230612445, 'min_split_gain': 0.014585563506995648}. Best is trial 2 with value: 0.003391641684408733.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:25:05,525] Trial 4 finished with value: 0.0023209539774256345 and parameters: {'n_estimators': 195, 'learning_rate': 0.03287314932968176, 'num_leaves': 204, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.7527860922806439, 'colsample_bytree': 0.7668603832108376, 'reg_alpha': 5.89251069177149, 'reg_lambda': 3.51246394458832, 'min_split_gain': 0.002339474965290528}. Best is trial 4 with value: 0.0023209539774256345.
[Trial 4] New best: 0.00232
[I 2025-06-15 20:25:05,769] Trial 5 finished with value: 0.0020067787630773993 and parameters: {'n_estimators': 231, 'learning_rate': 0.2990135403512833, 'num_leaves': 124, 'max_depth': 5, 'min_child_samples': 29, 'subsample': 0.8847452616376713, 'colsample_bytree': 0.8067156532903794, 'reg_alpha': 5.376332474319858, 'reg_lambda': 6.315381568769425, 'min_split_gain': 0.03817510178405058}. Best is trial 5 with value: 0.0020067787630773993.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:25:06,487] Trial 6 finished with value: 0.0043574462191015234 and parameters: {'n_estimators': 123, 'learning_rate': 0.02358937067548689, 'num_leaves': 200, 'max_depth': 6, 'min_child_samples': 19, 'subsample': 0.9892729037955462, 'colsample_bytree': 0.9170421930220485, 'reg_alpha': 9.516113721277481, 'reg_lambda': 1.3386727254326891, 'min_split_gain': 0.0048231205026734395}. Best is trial 5 with value: 0.0020067787630773993.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:25:06,924] Trial 7 finished with value: 0.02948684246126071 and parameters: {'n_estimators': 55, 'learning_rate': 0.02759634413762911, 'num_leaves': 35, 'max_depth': 12, 'min_child_samples': 7, 'subsample': 0.56660238054907, 'colsample_bytree': 0.5855602626182417, 'reg_alpha': 9.126190410118008, 'reg_lambda': 9.264374024036213, 'min_split_gain': 0.03512860283663618}. Best is trial 5 with value: 0.0020067787630773993.
[I 2025-06-15 20:25:06,927] A new study created in memory with name: no-name-d7bb71e6-a0dd-4198-b39a-454e5a83b025
[I 2025-06-15 20:25:07,110] Trial 0 finished with value: 0.00482855390947518 and parameters: {'n_estimators': 150, 'learning_rate': 0.015205214263234908, 'num_leaves': 211, 'max_depth': 10, 'min_child_samples': 1, 'subsample': 0.9205261144068021, 'colsample_bytree': 0.6755990752021521, 'reg_alpha': 0.32258894000791627, 'reg_lambda': 6.126371806084111, 'min_split_gain': 0.006429295111202904}. Best is trial 0 with value: 0.00482855390947518.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         149
14      pct_chg_Load_DA          85
1           Price_lag_2          60
3         Load_DA_lag_0          56
11           NGas_lag_2          37
15       lag168_Load_DA          30
16  roll2d_median_Price          30
2           Price_lag_7          27
5       WindOn_DA_lag_0          21
13            EUA_lag_2          18
10           Coal_lag_2          12
12            Oil_lag_2          10
9                  WD_7           3
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         265
15       lag168_Load_DA         130
5       WindOn_DA_lag_0          95
14      pct_chg_Load_DA          63
11           NGas_lag_2          57
3         Load_DA_lag_0          56
16  roll2d_median_Price          55
2           Price_lag_7          55
1           Price_lag_2          52
13            EUA_lag_2          19
10           Coal_lag_2           5
7                  WD_1           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
12            Oil_lag_2           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 63.34% done
********************* END NS.. 462 ****************************************************
********************* START NS ... 463 ****************************************************
[I 2025-06-15 20:25:08,834] A new study created in memory with name: no-name-48cc8af4-13c1-4524-a923-b2988210fd80
[I 2025-06-15 20:25:09,468] Trial 0 finished with value: 0.21137427769694214 and parameters: {'n_estimators': 64, 'learning_rate': 0.012054552712407681, 'num_leaves': 94, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.6630450468690681, 'colsample_bytree': 0.6555723576191893, 'reg_alpha': 4.050438598741209, 'reg_lambda': 1.9837438931945783, 'min_split_gain': 0.03655408193622991}. Best is trial 0 with value: 0.21137427769694214.
[Trial 0] New best: 0.21137
[I 2025-06-15 20:25:09,846] Trial 1 finished with value: 0.004612788726159962 and parameters: {'n_estimators': 218, 'learning_rate': 0.1538289971331155, 'num_leaves': 124, 'max_depth': 5, 'min_child_samples': 28, 'subsample': 0.6059637352256123, 'colsample_bytree': 0.9817569163199944, 'reg_alpha': 9.206907038428776, 'reg_lambda': 3.100539077622039, 'min_split_gain': 0.019129000267635372}. Best is trial 1 with value: 0.004612788726159962.
[Trial 1] New best: 0.00461
[I 2025-06-15 20:25:10,182] Trial 2 finished with value: 0.004906151658140106 and parameters: {'n_estimators': 289, 'learning_rate': 0.2686674116456661, 'num_leaves': 70, 'max_depth': 5, 'min_child_samples': 2, 'subsample': 0.5796627293080063, 'colsample_bytree': 0.7198184267296568, 'reg_alpha': 8.637122743097988, 'reg_lambda': 0.9480526247528753, 'min_split_gain': 0.030491156177527387}. Best is trial 1 with value: 0.004612788726159962.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:25:12,002] Trial 3 finished with value: 0.017752884308103584 and parameters: {'n_estimators': 250, 'learning_rate': 0.011372066667826346, 'num_leaves': 38, 'max_depth': 7, 'min_child_samples': 21, 'subsample': 0.674898969249491, 'colsample_bytree': 0.5077467691200691, 'reg_alpha': 7.039445955509819, 'reg_lambda': 8.849000154398276, 'min_split_gain': 0.049980024395065256}. Best is trial 1 with value: 0.004612788726159962.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:25:13,403] Trial 4 finished with value: 0.04977925231216381 and parameters: {'n_estimators': 165, 'learning_rate': 0.011058806309649276, 'num_leaves': 148, 'max_depth': 7, 'min_child_samples': 4, 'subsample': 0.7868176626757448, 'colsample_bytree': 0.5450708411151434, 'reg_alpha': 8.48371525679714, 'reg_lambda': 4.486482651649226, 'min_split_gain': 0.026581156157957697}. Best is trial 1 with value: 0.004612788726159962.
[I 2025-06-15 20:25:13,406] A new study created in memory with name: no-name-f4241156-cf4b-47e1-8a70-cd6b71537eed
[I 2025-06-15 20:25:13,493] Trial 0 finished with value: 0.0963709423695265 and parameters: {'n_estimators': 223, 'learning_rate': 0.09176133051278089, 'num_leaves': 125, 'max_depth': 13, 'min_child_samples': 22, 'subsample': 0.8644048326654463, 'colsample_bytree': 0.6107141074045963, 'reg_alpha': 0.09523771172687323, 'reg_lambda': 5.087846689226687, 'min_split_gain': 0.009895019796291316}. Best is trial 0 with value: 0.0963709423695265.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         277
14      pct_chg_Load_DA         174
1           Price_lag_2          97
3         Load_DA_lag_0          89
11           NGas_lag_2          79
16  roll2d_median_Price          73
2           Price_lag_7          52
15       lag168_Load_DA          49
5       WindOn_DA_lag_0          48
10           Coal_lag_2          38
13            EUA_lag_2          36
12            Oil_lag_2          16
7                  WD_1           4
9                  WD_7           4
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          59
15       lag168_Load_DA          34
16  roll2d_median_Price          23
3         Load_DA_lag_0          19
2           Price_lag_7          13
14      pct_chg_Load_DA          12
5       WindOn_DA_lag_0          11
1           Price_lag_2           8
11           NGas_lag_2           8
9                  WD_7           2
10           Coal_lag_2           2
13            EUA_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
12            Oil_lag_2           0
8                  WD_6           0
-> 63.47% done
********************* END NS.. 463 ****************************************************
********************* START NS ... 464 ****************************************************
[I 2025-06-15 20:25:15,628] A new study created in memory with name: no-name-b49edd84-4d58-4b74-96ef-b70900e0d4b8
[I 2025-06-15 20:25:16,218] Trial 0 finished with value: 0.00535578913119622 and parameters: {'n_estimators': 109, 'learning_rate': 0.03737352285526661, 'num_leaves': 148, 'max_depth': 6, 'min_child_samples': 14, 'subsample': 0.8639965370238696, 'colsample_bytree': 0.7999959725786465, 'reg_alpha': 4.625613462658825, 'reg_lambda': 8.927363351978316, 'min_split_gain': 0.04475242006684904}. Best is trial 0 with value: 0.00535578913119622.
[Trial 0] New best: 0.00536
[I 2025-06-15 20:25:16,945] Trial 1 finished with value: 0.0049585430666624665 and parameters: {'n_estimators': 215, 'learning_rate': 0.04645257682621282, 'num_leaves': 210, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.7836995137402182, 'colsample_bytree': 0.6728851849719348, 'reg_alpha': 5.653374595792955, 'reg_lambda': 3.877918094372381, 'min_split_gain': 0.03273887288604533}. Best is trial 1 with value: 0.0049585430666624665.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:25:17,444] Trial 2 finished with value: 0.003672311618713091 and parameters: {'n_estimators': 115, 'learning_rate': 0.10884986455758193, 'num_leaves': 168, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.6646379880053226, 'colsample_bytree': 0.9874057526474946, 'reg_alpha': 4.745942513976727, 'reg_lambda': 4.546907286808506, 'min_split_gain': 0.006892302868296263}. Best is trial 2 with value: 0.003672311618713091.
[Trial 2] New best: 0.00367
[I 2025-06-15 20:25:17,732] Trial 3 finished with value: 0.0038419501091449316 and parameters: {'n_estimators': 144, 'learning_rate': 0.23941999214355045, 'num_leaves': 182, 'max_depth': 4, 'min_child_samples': 14, 'subsample': 0.8142003234995936, 'colsample_bytree': 0.8524786661577224, 'reg_alpha': 7.291793639694376, 'reg_lambda': 2.778338522187007, 'min_split_gain': 0.004246596298984407}. Best is trial 2 with value: 0.003672311618713091.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:25:19,076] Trial 4 finished with value: 0.005512145406085404 and parameters: {'n_estimators': 256, 'learning_rate': 0.03758766375290359, 'num_leaves': 251, 'max_depth': 8, 'min_child_samples': 9, 'subsample': 0.860248693338632, 'colsample_bytree': 0.6769462378912878, 'reg_alpha': 9.495113368916291, 'reg_lambda': 0.2326084543547302, 'min_split_gain': 0.03211371006374709}. Best is trial 2 with value: 0.003672311618713091.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:25:19,860] Trial 5 finished with value: 0.006878551956580281 and parameters: {'n_estimators': 198, 'learning_rate': 0.1870141914365311, 'num_leaves': 103, 'max_depth': 9, 'min_child_samples': 14, 'subsample': 0.7883061530704476, 'colsample_bytree': 0.5554457933803859, 'reg_alpha': 9.241535120635927, 'reg_lambda': 2.2181095785074554, 'min_split_gain': 0.02751877806939872}. Best is trial 2 with value: 0.003672311618713091.
[I 2025-06-15 20:25:19,862] A new study created in memory with name: no-name-dbbe584f-3a55-4b49-94e5-0d96b73665b3
[I 2025-06-15 20:25:19,932] Trial 0 finished with value: 0.13329225416461032 and parameters: {'n_estimators': 297, 'learning_rate': 0.0297796698747188, 'num_leaves': 94, 'max_depth': 11, 'min_child_samples': 4, 'subsample': 0.7104538045840294, 'colsample_bytree': 0.8098437674443537, 'reg_alpha': 8.102581822239712, 'reg_lambda': 7.554198445011283, 'min_split_gain': 0.030446844712394446}. Best is trial 0 with value: 0.13329225416461032.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         542
14      pct_chg_Load_DA         381
1           Price_lag_2         281
3         Load_DA_lag_0         220
16  roll2d_median_Price         163
11           NGas_lag_2         157
2           Price_lag_7         151
5       WindOn_DA_lag_0         149
15       lag168_Load_DA         135
10           Coal_lag_2          98
13            EUA_lag_2          94
12            Oil_lag_2          49
7                  WD_1          11
9                  WD_7           6
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          60
1           Price_lag_2          10
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 63.61% done
********************* END NS.. 464 ****************************************************
********************* START NS ... 465 ****************************************************
[I 2025-06-15 20:25:22,021] A new study created in memory with name: no-name-e64f09fc-fec8-4c6f-a847-f4be0d08a6bd
[I 2025-06-15 20:25:22,519] Trial 0 finished with value: 0.003254011376072703 and parameters: {'n_estimators': 280, 'learning_rate': 0.2072248367711019, 'num_leaves': 58, 'max_depth': 15, 'min_child_samples': 17, 'subsample': 0.7833206081688026, 'colsample_bytree': 0.6777848426381583, 'reg_alpha': 2.2754000020525647, 'reg_lambda': 4.4663204338847065, 'min_split_gain': 0.019551038724475684}. Best is trial 0 with value: 0.003254011376072703.
[Trial 0] New best: 0.00325
[I 2025-06-15 20:25:25,691] Trial 1 finished with value: 0.0032513150281489565 and parameters: {'n_estimators': 251, 'learning_rate': 0.020574495694079948, 'num_leaves': 67, 'max_depth': 9, 'min_child_samples': 12, 'subsample': 0.9654247355429075, 'colsample_bytree': 0.9765862835899268, 'reg_alpha': 0.4688601575437057, 'reg_lambda': 7.797286071225081, 'min_split_gain': 0.047601587938918034}. Best is trial 1 with value: 0.0032513150281489565.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:25:26,540] Trial 2 finished with value: 0.0033827019318661768 and parameters: {'n_estimators': 56, 'learning_rate': 0.08514442430520883, 'num_leaves': 71, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.5188487999179967, 'colsample_bytree': 0.6094093765085185, 'reg_alpha': 5.090199776969017, 'reg_lambda': 1.7571104127362203, 'min_split_gain': 0.002038768412915859}. Best is trial 1 with value: 0.0032513150281489565.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:25:27,113] Trial 3 finished with value: 0.0030931782031584803 and parameters: {'n_estimators': 260, 'learning_rate': 0.21300521284682317, 'num_leaves': 212, 'max_depth': 14, 'min_child_samples': 21, 'subsample': 0.6258842723706052, 'colsample_bytree': 0.7330882566339694, 'reg_alpha': 2.791314886213989, 'reg_lambda': 4.470710165240616, 'min_split_gain': 0.032710789708948765}. Best is trial 3 with value: 0.0030931782031584803.
[I 2025-06-15 20:25:27,115] A new study created in memory with name: no-name-0f5d9f5b-87de-4322-b7a3-73994dc75276
[I 2025-06-15 20:25:27,171] Trial 0 finished with value: 0.007779288389875516 and parameters: {'n_estimators': 173, 'learning_rate': 0.10353716598647203, 'num_leaves': 129, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.7881662383404614, 'colsample_bytree': 0.7619802912605294, 'reg_alpha': 3.776674163138302, 'reg_lambda': 8.380892016392847, 'min_split_gain': 0.03297884791688992}. Best is trial 0 with value: 0.007779288389875516.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         320
1           Price_lag_2         229
14      pct_chg_Load_DA         199
3         Load_DA_lag_0         160
16  roll2d_median_Price         123
11           NGas_lag_2         106
2           Price_lag_7         101
15       lag168_Load_DA          96
5       WindOn_DA_lag_0          75
13            EUA_lag_2          49
10           Coal_lag_2          47
12            Oil_lag_2          36
9                  WD_7          10
7                  WD_1           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          42
9                  WD_7           5
2           Price_lag_7           3
1           Price_lag_2           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
16  roll2d_median_Price           0
-> 63.75% done
********************* END NS.. 465 ****************************************************
********************* START NS ... 466 ****************************************************
[I 2025-06-15 20:25:29,078] A new study created in memory with name: no-name-fca6919e-5d46-4f7c-8fa3-e144079f370c
[I 2025-06-15 20:25:30,970] Trial 0 finished with value: 0.02559260150192443 and parameters: {'n_estimators': 265, 'learning_rate': 0.022577824479471577, 'num_leaves': 157, 'max_depth': 7, 'min_child_samples': 29, 'subsample': 0.8551519528207857, 'colsample_bytree': 0.7305019919117337, 'reg_alpha': 5.297622263465741, 'reg_lambda': 5.442820884165769, 'min_split_gain': 0.014537529354787799}. Best is trial 0 with value: 0.02559260150192443.
[Trial 0] New best: 0.02559
[I 2025-06-15 20:25:32,300] Trial 1 finished with value: 0.03107989583592161 and parameters: {'n_estimators': 201, 'learning_rate': 0.012554211301199216, 'num_leaves': 223, 'max_depth': 6, 'min_child_samples': 5, 'subsample': 0.6789648081414392, 'colsample_bytree': 0.7350282852102559, 'reg_alpha': 4.118796995071046, 'reg_lambda': 2.3830427977519184, 'min_split_gain': 0.017265224605516726}. Best is trial 0 with value: 0.02559260150192443.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:25:34,776] Trial 2 finished with value: 0.025481673965567105 and parameters: {'n_estimators': 176, 'learning_rate': 0.04849482453490582, 'num_leaves': 224, 'max_depth': 9, 'min_child_samples': 14, 'subsample': 0.6156892145388781, 'colsample_bytree': 0.5049260485838115, 'reg_alpha': 2.270140618150003, 'reg_lambda': 8.72402661814242, 'min_split_gain': 0.015018102084312946}. Best is trial 2 with value: 0.025481673965567105.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:25:36,053] Trial 3 finished with value: 0.024976320186261072 and parameters: {'n_estimators': 80, 'learning_rate': 0.057455808050772925, 'num_leaves': 110, 'max_depth': 10, 'min_child_samples': 16, 'subsample': 0.6845130319482853, 'colsample_bytree': 0.6249845660412491, 'reg_alpha': 0.6788455187603859, 'reg_lambda': 6.9692967246811115, 'min_split_gain': 0.04746554948179568}. Best is trial 3 with value: 0.024976320186261072.
[I 2025-06-15 20:25:36,056] A new study created in memory with name: no-name-090f62eb-236b-4b80-9d76-b82c49d14024
[I 2025-06-15 20:25:36,128] Trial 0 finished with value: 0.04517132334497226 and parameters: {'n_estimators': 245, 'learning_rate': 0.2525105226946138, 'num_leaves': 161, 'max_depth': 9, 'min_child_samples': 4, 'subsample': 0.7443936696885896, 'colsample_bytree': 0.5546186545002444, 'reg_alpha': 4.649520128702674, 'reg_lambda': 2.0854444902120353, 'min_split_gain': 0.022091363709977465}. Best is trial 0 with value: 0.04517132334497226.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1167
0           Price_lag_1        1045
14      pct_chg_Load_DA         656
3         Load_DA_lag_0         483
11           NGas_lag_2         472
15       lag168_Load_DA         438
2           Price_lag_7         409
16  roll2d_median_Price         394
5       WindOn_DA_lag_0         297
10           Coal_lag_2         288
13            EUA_lag_2         288
12            Oil_lag_2         215
9                  WD_7          21
7                  WD_1          12
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          12
1           Price_lag_2           3
9                  WD_7           1
16  roll2d_median_Price           1
3         Load_DA_lag_0           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
2           Price_lag_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 63.89% done
********************* END NS.. 466 ****************************************************
********************* START NS ... 467 ****************************************************
[I 2025-06-15 20:25:39,049] A new study created in memory with name: no-name-697f9c59-c508-49b5-a49d-1168f02c1b7a
[I 2025-06-15 20:25:39,251] Trial 0 finished with value: 0.024804247212588507 and parameters: {'n_estimators': 58, 'learning_rate': 0.11031745279526751, 'num_leaves': 165, 'max_depth': 5, 'min_child_samples': 14, 'subsample': 0.8478973803459444, 'colsample_bytree': 0.7290502928714765, 'reg_alpha': 9.271942508201649, 'reg_lambda': 2.560404833664779, 'min_split_gain': 0.00952509596126096}. Best is trial 0 with value: 0.024804247212588507.
[Trial 0] New best: 0.02480
[I 2025-06-15 20:25:40,590] Trial 1 finished with value: 0.026616839399761238 and parameters: {'n_estimators': 269, 'learning_rate': 0.043843996481133816, 'num_leaves': 146, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.866011915481673, 'colsample_bytree': 0.5843012741404752, 'reg_alpha': 9.1782935850071, 'reg_lambda': 8.436306811577092, 'min_split_gain': 0.027382663563559107}. Best is trial 0 with value: 0.024804247212588507.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:25:41,901] Trial 2 finished with value: 0.02141956616279136 and parameters: {'n_estimators': 110, 'learning_rate': 0.03997572642488718, 'num_leaves': 75, 'max_depth': 9, 'min_child_samples': 13, 'subsample': 0.5532738019279468, 'colsample_bytree': 0.8565484272865114, 'reg_alpha': 6.9025639297366315, 'reg_lambda': 6.511914879452646, 'min_split_gain': 0.00016588019156734137}. Best is trial 2 with value: 0.02141956616279136.
[Trial 2] New best: 0.02142
[I 2025-06-15 20:25:42,962] Trial 3 finished with value: 0.045838031110553255 and parameters: {'n_estimators': 140, 'learning_rate': 0.012467882506802652, 'num_leaves': 143, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.7536605999267164, 'colsample_bytree': 0.7559747879323504, 'reg_alpha': 1.1412127125068905, 'reg_lambda': 5.613323601893766, 'min_split_gain': 0.04714804987970944}. Best is trial 2 with value: 0.02141956616279136.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:25:43,295] Trial 4 finished with value: 0.023100894287345606 and parameters: {'n_estimators': 62, 'learning_rate': 0.21378864742277698, 'num_leaves': 163, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.8743503859105632, 'colsample_bytree': 0.7560595839775122, 'reg_alpha': 7.3858302361191335, 'reg_lambda': 3.0588809254697047, 'min_split_gain': 0.015806343302509487}. Best is trial 2 with value: 0.02141956616279136.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:25:43,689] Trial 5 finished with value: 0.022495473925166546 and parameters: {'n_estimators': 175, 'learning_rate': 0.1801099506795648, 'num_leaves': 182, 'max_depth': 12, 'min_child_samples': 16, 'subsample': 0.7034826613042355, 'colsample_bytree': 0.8390922676158736, 'reg_alpha': 7.257505137464138, 'reg_lambda': 0.7091853403445303, 'min_split_gain': 0.023064862758311766}. Best is trial 2 with value: 0.02141956616279136.
[I 2025-06-15 20:25:43,691] A new study created in memory with name: no-name-1defa583-16f9-4d63-81be-2c0381aa2963
[I 2025-06-15 20:25:43,739] Trial 0 finished with value: 0.040142076285257754 and parameters: {'n_estimators': 86, 'learning_rate': 0.0666433259365621, 'num_leaves': 91, 'max_depth': 8, 'min_child_samples': 4, 'subsample': 0.8430498452748955, 'colsample_bytree': 0.6147303609895589, 'reg_alpha': 2.8810501326124727, 'reg_lambda': 3.3287372991685062, 'min_split_gain': 0.021210609932878682}. Best is trial 0 with value: 0.040142076285257754.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1253
1           Price_lag_2         998
14      pct_chg_Load_DA         819
2           Price_lag_7         461
16  roll2d_median_Price         378
3         Load_DA_lag_0         338
11           NGas_lag_2         337
13            EUA_lag_2         289
5       WindOn_DA_lag_0         204
12            Oil_lag_2         184
15       lag168_Load_DA         178
10           Coal_lag_2         154
9                  WD_7          18
8                  WD_6          11
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          61
1           Price_lag_2          13
9                  WD_7           8
2           Price_lag_7           2
12            Oil_lag_2           1
11           NGas_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
16  roll2d_median_Price           0
-> 64.02% done
********************* END NS.. 467 ****************************************************
********************* START NS ... 468 ****************************************************
[I 2025-06-15 20:25:46,254] A new study created in memory with name: no-name-aabd11b3-06e3-4461-9d32-1dbe2ff1cd0f
[I 2025-06-15 20:25:49,825] Trial 0 finished with value: 0.008069177455366615 and parameters: {'n_estimators': 272, 'learning_rate': 0.015641139785478226, 'num_leaves': 131, 'max_depth': 14, 'min_child_samples': 18, 'subsample': 0.5051060000778052, 'colsample_bytree': 0.5412536163763959, 'reg_alpha': 2.5536087558160236, 'reg_lambda': 7.878656216170292, 'min_split_gain': 0.033461674985033364}. Best is trial 0 with value: 0.008069177455366615.
[Trial 0] New best: 0.00807
[I 2025-06-15 20:25:50,252] Trial 1 finished with value: 0.005471563406778384 and parameters: {'n_estimators': 93, 'learning_rate': 0.13559025004252936, 'num_leaves': 256, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.5595518294499559, 'colsample_bytree': 0.9624254764269056, 'reg_alpha': 0.15418256818551024, 'reg_lambda': 7.577042489858506, 'min_split_gain': 0.002575927042057835}. Best is trial 1 with value: 0.005471563406778384.
[Trial 1] New best: 0.00547
[I 2025-06-15 20:25:52,658] Trial 2 finished with value: 0.006716337452655269 and parameters: {'n_estimators': 250, 'learning_rate': 0.020321226988614372, 'num_leaves': 38, 'max_depth': 12, 'min_child_samples': 18, 'subsample': 0.7050445794922579, 'colsample_bytree': 0.6178262523026503, 'reg_alpha': 1.645100041403208, 'reg_lambda': 2.9069034316157207, 'min_split_gain': 0.02480359937669821}. Best is trial 1 with value: 0.005471563406778384.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:25:53,073] Trial 3 finished with value: 0.0059189243986824125 and parameters: {'n_estimators': 184, 'learning_rate': 0.2957655684208501, 'num_leaves': 241, 'max_depth': 9, 'min_child_samples': 23, 'subsample': 0.5537830341236059, 'colsample_bytree': 0.7958873696982784, 'reg_alpha': 3.121713842303496, 'reg_lambda': 4.59668736812897, 'min_split_gain': 0.0167756095783373}. Best is trial 1 with value: 0.005471563406778384.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:25:53,863] Trial 4 finished with value: 0.008579146048785211 and parameters: {'n_estimators': 84, 'learning_rate': 0.036469627017782384, 'num_leaves': 213, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.5008506369437125, 'colsample_bytree': 0.9085052938887042, 'reg_alpha': 5.488124487501804, 'reg_lambda': 0.29345009423858204, 'min_split_gain': 0.024542607906634647}. Best is trial 1 with value: 0.005471563406778384.
[I 2025-06-15 20:25:53,865] A new study created in memory with name: no-name-0c9a0bd9-de1b-42cd-bca8-b9aa53a7c9b1
[I 2025-06-15 20:25:54,017] Trial 0 finished with value: 0.009347481054148727 and parameters: {'n_estimators': 258, 'learning_rate': 0.011665612516401554, 'num_leaves': 215, 'max_depth': 11, 'min_child_samples': 30, 'subsample': 0.6292194090907679, 'colsample_bytree': 0.5795429249940903, 'reg_alpha': 2.7039575094971955, 'reg_lambda': 6.286096737651904, 'min_split_gain': 0.021424625303888653}. Best is trial 0 with value: 0.009347481054148727.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         489
14      pct_chg_Load_DA         289
1           Price_lag_2         239
3         Load_DA_lag_0         180
16  roll2d_median_Price         170
11           NGas_lag_2         143
2           Price_lag_7         141
5       WindOn_DA_lag_0         127
15       lag168_Load_DA         118
12            Oil_lag_2          82
10           Coal_lag_2          73
13            EUA_lag_2          63
9                  WD_7           9
7                  WD_1           8
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         259
1           Price_lag_2          57
11           NGas_lag_2          23
12            Oil_lag_2           9
13            EUA_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 64.16% done
********************* END NS.. 468 ****************************************************
********************* START NS ... 469 ****************************************************
[I 2025-06-15 20:25:55,983] A new study created in memory with name: no-name-4164c25d-d64c-43bf-8629-9dddc0171aa3
[I 2025-06-15 20:25:56,393] Trial 0 finished with value: 0.015761773522241878 and parameters: {'n_estimators': 245, 'learning_rate': 0.1622515388261861, 'num_leaves': 207, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.6657206847239487, 'colsample_bytree': 0.7049569516227572, 'reg_alpha': 7.1636688294559026, 'reg_lambda': 5.717962229689551, 'min_split_gain': 0.04486656365661457}. Best is trial 0 with value: 0.015761773522241878.
[Trial 0] New best: 0.01576
[I 2025-06-15 20:25:57,067] Trial 1 finished with value: 0.01336991520451334 and parameters: {'n_estimators': 54, 'learning_rate': 0.11142172784237425, 'num_leaves': 46, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.9486483990950418, 'colsample_bytree': 0.8918391552535001, 'reg_alpha': 0.0018770334806772482, 'reg_lambda': 4.859555109248125, 'min_split_gain': 0.006934477453944599}. Best is trial 1 with value: 0.01336991520451334.
[Trial 1] New best: 0.01337
[I 2025-06-15 20:26:03,800] Trial 2 finished with value: 0.016875670407398763 and parameters: {'n_estimators': 239, 'learning_rate': 0.013956290566010115, 'num_leaves': 192, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.7273684929046096, 'colsample_bytree': 0.9161195937487181, 'reg_alpha': 0.014492793860573228, 'reg_lambda': 9.904319727937574, 'min_split_gain': 0.021178757211175726}. Best is trial 1 with value: 0.01336991520451334.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:26:06,923] Trial 3 finished with value: 0.015521786294911114 and parameters: {'n_estimators': 164, 'learning_rate': 0.03408327754140213, 'num_leaves': 54, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.8309290822496872, 'colsample_bytree': 0.6085326087344927, 'reg_alpha': 1.6544461599747884, 'reg_lambda': 7.962128907540041, 'min_split_gain': 0.0017259673884612303}. Best is trial 1 with value: 0.01336991520451334.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:26:07,808] Trial 4 finished with value: 0.016255871053948726 and parameters: {'n_estimators': 186, 'learning_rate': 0.03635500329927198, 'num_leaves': 94, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.5804823992765387, 'colsample_bytree': 0.882943725523595, 'reg_alpha': 6.070800322113474, 'reg_lambda': 9.405698464563534, 'min_split_gain': 0.03452447814026073}. Best is trial 1 with value: 0.01336991520451334.
[I 2025-06-15 20:26:07,812] A new study created in memory with name: no-name-a5e09b86-9e0f-4df5-9934-6a83059b69da
[I 2025-06-15 20:26:07,873] Trial 0 finished with value: 0.05032070539175524 and parameters: {'n_estimators': 198, 'learning_rate': 0.20298018230894685, 'num_leaves': 155, 'max_depth': 10, 'min_child_samples': 20, 'subsample': 0.9519332086421246, 'colsample_bytree': 0.9987289049638728, 'reg_alpha': 6.2689649961779015, 'reg_lambda': 5.973553451487436, 'min_split_gain': 0.03410212333009841}. Best is trial 0 with value: 0.05032070539175524.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         572
1           Price_lag_2         376
14      pct_chg_Load_DA         328
3         Load_DA_lag_0         170
16  roll2d_median_Price         153
11           NGas_lag_2         152
2           Price_lag_7         141
5       WindOn_DA_lag_0         140
15       lag168_Load_DA         122
10           Coal_lag_2          88
12            Oil_lag_2          88
13            EUA_lag_2          87
9                  WD_7           8
7                  WD_1           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          16
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 64.30% done
********************* END NS.. 469 ****************************************************
********************* START NS ... 470 ****************************************************
[I 2025-06-15 20:26:10,305] A new study created in memory with name: no-name-cb600f59-ff5a-4b91-b400-8fa6fb440157
[I 2025-06-15 20:26:13,993] Trial 0 finished with value: 0.026235076673938665 and parameters: {'n_estimators': 235, 'learning_rate': 0.01082279149559303, 'num_leaves': 95, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.5213836625853395, 'colsample_bytree': 0.9071151158145797, 'reg_alpha': 4.351941900443902, 'reg_lambda': 6.588456094170424, 'min_split_gain': 0.01392263113674121}. Best is trial 0 with value: 0.026235076673938665.
[Trial 0] New best: 0.02624
[I 2025-06-15 20:26:16,295] Trial 1 finished with value: 0.01235854181542468 and parameters: {'n_estimators': 254, 'learning_rate': 0.021932845483826395, 'num_leaves': 190, 'max_depth': 6, 'min_child_samples': 8, 'subsample': 0.8863115740768457, 'colsample_bytree': 0.88184259171689, 'reg_alpha': 3.368586265981741, 'reg_lambda': 7.148088954802687, 'min_split_gain': 0.04382558318572531}. Best is trial 1 with value: 0.01235854181542468.
[Trial 1] New best: 0.01236
[I 2025-06-15 20:26:16,694] Trial 2 finished with value: 0.012895618215088443 and parameters: {'n_estimators': 278, 'learning_rate': 0.2152126516559294, 'num_leaves': 75, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.5139464475899553, 'colsample_bytree': 0.7305332747943427, 'reg_alpha': 9.391872326813125, 'reg_lambda': 9.015785334148026, 'min_split_gain': 0.00816247793837343}. Best is trial 1 with value: 0.01235854181542468.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:26:17,310] Trial 3 finished with value: 0.011595981664148566 and parameters: {'n_estimators': 284, 'learning_rate': 0.27685411811057903, 'num_leaves': 128, 'max_depth': 5, 'min_child_samples': 16, 'subsample': 0.5527799690130428, 'colsample_bytree': 0.9966068090813821, 'reg_alpha': 7.192447414266882, 'reg_lambda': 3.2339221911586336, 'min_split_gain': 0.028372827698204248}. Best is trial 3 with value: 0.011595981664148566.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:26:17,726] Trial 4 finished with value: 0.012614527585132792 and parameters: {'n_estimators': 128, 'learning_rate': 0.13628780356659614, 'num_leaves': 134, 'max_depth': 6, 'min_child_samples': 6, 'subsample': 0.5458838021774652, 'colsample_bytree': 0.6833270850321835, 'reg_alpha': 5.10206558210143, 'reg_lambda': 3.9483534986542725, 'min_split_gain': 0.04967953823322746}. Best is trial 3 with value: 0.011595981664148566.
[I 2025-06-15 20:26:17,729] A new study created in memory with name: no-name-695f9198-812e-41fb-ba64-7083b8d8b562
[I 2025-06-15 20:26:17,773] Trial 0 finished with value: 0.14260316629107692 and parameters: {'n_estimators': 114, 'learning_rate': 0.0285374429300024, 'num_leaves': 198, 'max_depth': 13, 'min_child_samples': 24, 'subsample': 0.8954671021181952, 'colsample_bytree': 0.9128253646652675, 'reg_alpha': 9.145834710302154, 'reg_lambda': 0.9580518718984321, 'min_split_gain': 0.01499214309256891}. Best is trial 0 with value: 0.14260316629107692.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         155
14      pct_chg_Load_DA         107
1           Price_lag_2          59
3         Load_DA_lag_0          49
2           Price_lag_7          42
11           NGas_lag_2          38
16  roll2d_median_Price          35
5       WindOn_DA_lag_0          29
15       lag168_Load_DA          20
13            EUA_lag_2          19
10           Coal_lag_2          18
12            Oil_lag_2          12
7                  WD_1           2
9                  WD_7           2
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          74
1           Price_lag_2           5
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 64.43% done
********************* END NS.. 470 ****************************************************
********************* START NS ... 471 ****************************************************
[I 2025-06-15 20:26:19,618] A new study created in memory with name: no-name-4cf4e0eb-2cf9-4cd1-bc23-bb85a462f9eb
[I 2025-06-15 20:26:24,096] Trial 0 finished with value: 0.003970290316961632 and parameters: {'n_estimators': 277, 'learning_rate': 0.014311377119572736, 'num_leaves': 200, 'max_depth': 16, 'min_child_samples': 20, 'subsample': 0.8247055750749728, 'colsample_bytree': 0.5801889444263821, 'reg_alpha': 1.4480478611017589, 'reg_lambda': 8.130389826672724, 'min_split_gain': 0.036769267509872285}. Best is trial 0 with value: 0.003970290316961632.
[Trial 0] New best: 0.00397
[I 2025-06-15 20:26:24,988] Trial 1 finished with value: 0.003275216818408516 and parameters: {'n_estimators': 73, 'learning_rate': 0.19513470452603043, 'num_leaves': 186, 'max_depth': 15, 'min_child_samples': 1, 'subsample': 0.5853470154088791, 'colsample_bytree': 0.698723310303534, 'reg_alpha': 1.3697225487576303, 'reg_lambda': 6.507756129016497, 'min_split_gain': 0.022280213877771245}. Best is trial 1 with value: 0.003275216818408516.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:26:26,104] Trial 2 finished with value: 0.005937769571454764 and parameters: {'n_estimators': 135, 'learning_rate': 0.0208729643318857, 'num_leaves': 100, 'max_depth': 6, 'min_child_samples': 21, 'subsample': 0.6416984128817528, 'colsample_bytree': 0.5762997122970739, 'reg_alpha': 0.10772400059703036, 'reg_lambda': 5.772306563794729, 'min_split_gain': 0.027809658627030387}. Best is trial 1 with value: 0.003275216818408516.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:26:26,456] Trial 3 finished with value: 0.0033497434459439806 and parameters: {'n_estimators': 228, 'learning_rate': 0.27943293350054094, 'num_leaves': 35, 'max_depth': 12, 'min_child_samples': 25, 'subsample': 0.6481399623652608, 'colsample_bytree': 0.6589416377513688, 'reg_alpha': 8.539770329074551, 'reg_lambda': 6.0994802123548455, 'min_split_gain': 0.015729721781362505}. Best is trial 1 with value: 0.003275216818408516.
[I 2025-06-15 20:26:26,458] A new study created in memory with name: no-name-8265a4ff-561e-4cbf-a03a-c80bb42f2498
[I 2025-06-15 20:26:26,554] Trial 0 finished with value: 0.02294367218791349 and parameters: {'n_estimators': 273, 'learning_rate': 0.046807889501053086, 'num_leaves': 103, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.9133587415547217, 'colsample_bytree': 0.5314611439191661, 'reg_alpha': 5.117111866861443, 'reg_lambda': 0.4642450466569126, 'min_split_gain': 0.013177062729444022}. Best is trial 0 with value: 0.02294367218791349.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         506
0           Price_lag_1         504
3         Load_DA_lag_0         419
14      pct_chg_Load_DA         366
5       WindOn_DA_lag_0         342
2           Price_lag_7         320
15       lag168_Load_DA         281
11           NGas_lag_2         270
16  roll2d_median_Price         249
10           Coal_lag_2         211
13            EUA_lag_2         187
12            Oil_lag_2         128
9                  WD_7          18
7                  WD_1           8
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          79
1           Price_lag_2          16
3         Load_DA_lag_0          10
10           Coal_lag_2           5
16  roll2d_median_Price           1
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
2           Price_lag_7           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 64.57% done
********************* END NS.. 471 ****************************************************
********************* START NS ... 472 ****************************************************
[I 2025-06-15 20:26:28,688] A new study created in memory with name: no-name-f5ca495c-2105-4764-8b9c-a8492e256ce6
[I 2025-06-15 20:26:30,232] Trial 0 finished with value: 0.004872747020920378 and parameters: {'n_estimators': 225, 'learning_rate': 0.014579914363358867, 'num_leaves': 56, 'max_depth': 5, 'min_child_samples': 4, 'subsample': 0.5341063145292608, 'colsample_bytree': 0.9624287215150473, 'reg_alpha': 3.5527072307502428, 'reg_lambda': 3.8348850434193196, 'min_split_gain': 0.019591851735652078}. Best is trial 0 with value: 0.004872747020920378.
[Trial 0] New best: 0.00487
[I 2025-06-15 20:26:30,754] Trial 1 finished with value: 0.0044098001792612085 and parameters: {'n_estimators': 246, 'learning_rate': 0.19740576281862834, 'num_leaves': 204, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.8173000391273175, 'colsample_bytree': 0.6131733613384934, 'reg_alpha': 6.550605356360602, 'reg_lambda': 3.987796394776697, 'min_split_gain': 0.048722973331734594}. Best is trial 1 with value: 0.0044098001792612085.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:26:31,788] Trial 2 finished with value: 0.003693763789437761 and parameters: {'n_estimators': 202, 'learning_rate': 0.08176974019814404, 'num_leaves': 40, 'max_depth': 10, 'min_child_samples': 7, 'subsample': 0.6332220529963313, 'colsample_bytree': 0.8791979783258623, 'reg_alpha': 4.635717003864287, 'reg_lambda': 6.977970263251284, 'min_split_gain': 0.019382252125285094}. Best is trial 2 with value: 0.003693763789437761.
[Trial 2] New best: 0.00369
[I 2025-06-15 20:26:34,414] Trial 3 finished with value: 0.004072501540914725 and parameters: {'n_estimators': 233, 'learning_rate': 0.03773575794120556, 'num_leaves': 68, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.9572594800647981, 'colsample_bytree': 0.631969821272761, 'reg_alpha': 2.0062247865493257, 'reg_lambda': 4.595565582760852, 'min_split_gain': 0.03452363083037035}. Best is trial 2 with value: 0.003693763789437761.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:26:35,296] Trial 4 finished with value: 0.003484879751212346 and parameters: {'n_estimators': 248, 'learning_rate': 0.1453595312869204, 'num_leaves': 194, 'max_depth': 12, 'min_child_samples': 5, 'subsample': 0.6403037360188568, 'colsample_bytree': 0.8065648736632309, 'reg_alpha': 1.517045263152248, 'reg_lambda': 1.1789894654198085, 'min_split_gain': 0.03162326608192017}. Best is trial 4 with value: 0.003484879751212346.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:26:38,955] Trial 5 finished with value: 0.003450359569652058 and parameters: {'n_estimators': 286, 'learning_rate': 0.04808978758112166, 'num_leaves': 167, 'max_depth': 10, 'min_child_samples': 5, 'subsample': 0.6340586854955967, 'colsample_bytree': 0.5924092583630589, 'reg_alpha': 2.2407463769447835, 'reg_lambda': 2.1309253790751903, 'min_split_gain': 0.006759188560269691}. Best is trial 5 with value: 0.003450359569652058.
[I 2025-06-15 20:26:38,958] A new study created in memory with name: no-name-b51b3448-4440-44d4-bfab-8558a88124eb
[I 2025-06-15 20:26:39,036] Trial 0 finished with value: 0.016438183625742286 and parameters: {'n_estimators': 245, 'learning_rate': 0.03296415086193775, 'num_leaves': 206, 'max_depth': 8, 'min_child_samples': 3, 'subsample': 0.7012072847944935, 'colsample_bytree': 0.8644368644205842, 'reg_alpha': 5.272840359631021, 'reg_lambda': 8.027775904835892, 'min_split_gain': 0.02179708616221182}. Best is trial 0 with value: 0.016438183625742286.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        2593
0           Price_lag_1        2480
14      pct_chg_Load_DA        1984
3         Load_DA_lag_0        1635
5       WindOn_DA_lag_0        1554
2           Price_lag_7        1252
11           NGas_lag_2        1219
15       lag168_Load_DA        1204
16  roll2d_median_Price        1172
13            EUA_lag_2        1073
10           Coal_lag_2         889
12            Oil_lag_2         713
9                  WD_7          96
7                  WD_1          38
8                  WD_6          36
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         116
3         Load_DA_lag_0          23
1           Price_lag_2          10
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 64.71% done
********************* END NS.. 472 ****************************************************
********************* START NS ... 473 ****************************************************
[I 2025-06-15 20:26:43,771] A new study created in memory with name: no-name-1229fbe3-f1f6-44a3-b380-79b0a54dadca
[I 2025-06-15 20:26:46,679] Trial 0 finished with value: 0.002404341876572529 and parameters: {'n_estimators': 114, 'learning_rate': 0.0373037471825625, 'num_leaves': 196, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.504435262874097, 'colsample_bytree': 0.8312121385190611, 'reg_alpha': 1.1565491011226725, 'reg_lambda': 9.481684123578997, 'min_split_gain': 0.005092275580136996}. Best is trial 0 with value: 0.002404341876572529.
[Trial 0] New best: 0.00240
[I 2025-06-15 20:26:47,275] Trial 1 finished with value: 0.0023721660523017044 and parameters: {'n_estimators': 206, 'learning_rate': 0.11186210693051171, 'num_leaves': 233, 'max_depth': 4, 'min_child_samples': 8, 'subsample': 0.5913614407549875, 'colsample_bytree': 0.8901718672030794, 'reg_alpha': 1.2898038551358426, 'reg_lambda': 2.586076510686026, 'min_split_gain': 0.0005448033622957038}. Best is trial 1 with value: 0.0023721660523017044.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:26:48,043] Trial 2 finished with value: 0.002716713878568119 and parameters: {'n_estimators': 273, 'learning_rate': 0.06646776389430482, 'num_leaves': 196, 'max_depth': 7, 'min_child_samples': 29, 'subsample': 0.8836009103471103, 'colsample_bytree': 0.9221564360434611, 'reg_alpha': 9.344082540648436, 'reg_lambda': 4.784618642222092, 'min_split_gain': 0.022137187597581716}. Best is trial 1 with value: 0.0023721660523017044.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:26:49,465] Trial 3 finished with value: 0.002214791840784223 and parameters: {'n_estimators': 142, 'learning_rate': 0.044756011373310874, 'num_leaves': 55, 'max_depth': 11, 'min_child_samples': 28, 'subsample': 0.7076340939439771, 'colsample_bytree': 0.9875149353524298, 'reg_alpha': 2.484726805297408, 'reg_lambda': 3.2692662097206604, 'min_split_gain': 0.03218987410940469}. Best is trial 3 with value: 0.002214791840784223.
[I 2025-06-15 20:26:49,467] A new study created in memory with name: no-name-3ed5cb92-d419-4aba-9a09-153c7c93477c
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 20:26:49,699] Trial 0 finished with value: 0.06554669944062795 and parameters: {'n_estimators': 276, 'learning_rate': 0.05889127219437792, 'num_leaves': 135, 'max_depth': 11, 'min_child_samples': 12, 'subsample': 0.8661907087295226, 'colsample_bytree': 0.7879172717074125, 'reg_alpha': 7.903935037357142, 'reg_lambda': 2.7696644920997846, 'min_split_gain': 0.023313563196103167}. Best is trial 0 with value: 0.06554669944062795.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1603
14      pct_chg_Load_DA        1020
1           Price_lag_2         998
3         Load_DA_lag_0         557
16  roll2d_median_Price         482
2           Price_lag_7         457
11           NGas_lag_2         426
5       WindOn_DA_lag_0         316
15       lag168_Load_DA         291
13            EUA_lag_2         186
10           Coal_lag_2         185
12            Oil_lag_2         177
7                  WD_1          23
9                  WD_7          15
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          35
1           Price_lag_2           5
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 64.84% done
********************* END NS.. 473 ****************************************************
********************* START NS ... 474 ****************************************************
[I 2025-06-15 20:26:52,909] A new study created in memory with name: no-name-d1d2c940-db80-476e-b5d3-aebc28c8c3c7
[I 2025-06-15 20:26:54,804] Trial 0 finished with value: 0.01693228188578927 and parameters: {'n_estimators': 258, 'learning_rate': 0.053431843646727754, 'num_leaves': 172, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.5714683746711067, 'colsample_bytree': 0.9942886613659573, 'reg_alpha': 2.203921156478037, 'reg_lambda': 0.8689141945811363, 'min_split_gain': 0.03463485716241059}. Best is trial 0 with value: 0.01693228188578927.
[Trial 0] New best: 0.01693
[I 2025-06-15 20:26:55,192] Trial 1 finished with value: 0.016521193632528238 and parameters: {'n_estimators': 141, 'learning_rate': 0.2815351989817288, 'num_leaves': 130, 'max_depth': 11, 'min_child_samples': 12, 'subsample': 0.5758846725916471, 'colsample_bytree': 0.7026078537651879, 'reg_alpha': 9.655485750776934, 'reg_lambda': 4.635031192107824, 'min_split_gain': 0.008704522922846463}. Best is trial 1 with value: 0.016521193632528238.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:26:56,913] Trial 2 finished with value: 0.017556811811865945 and parameters: {'n_estimators': 224, 'learning_rate': 0.015778719313057438, 'num_leaves': 239, 'max_depth': 7, 'min_child_samples': 20, 'subsample': 0.8240381975509905, 'colsample_bytree': 0.641923337497821, 'reg_alpha': 6.834725036409271, 'reg_lambda': 1.2210339008064097, 'min_split_gain': 0.007459993247121511}. Best is trial 1 with value: 0.016521193632528238.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:26:58,327] Trial 3 finished with value: 0.017928345531753927 and parameters: {'n_estimators': 233, 'learning_rate': 0.027743870474802205, 'num_leaves': 81, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.8353373974990217, 'colsample_bytree': 0.618371429517988, 'reg_alpha': 2.0404980481387325, 'reg_lambda': 4.738495195738964, 'min_split_gain': 0.003097935924407719}. Best is trial 1 with value: 0.016521193632528238.
[I 2025-06-15 20:26:58,330] A new study created in memory with name: no-name-c5bd448a-ab58-47d6-aad2-ce50a83194a6
[I 2025-06-15 20:26:58,426] Trial 0 finished with value: 0.14143741440183547 and parameters: {'n_estimators': 269, 'learning_rate': 0.06367552174189488, 'num_leaves': 99, 'max_depth': 4, 'min_child_samples': 1, 'subsample': 0.7093203355385076, 'colsample_bytree': 0.5150641395575308, 'reg_alpha': 5.885458812575628, 'reg_lambda': 5.29905475970097, 'min_split_gain': 0.04204624740319487}. Best is trial 0 with value: 0.14143741440183547.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         203
1           Price_lag_2         140
14      pct_chg_Load_DA         129
3         Load_DA_lag_0          87
5       WindOn_DA_lag_0          75
2           Price_lag_7          72
11           NGas_lag_2          71
16  roll2d_median_Price          66
13            EUA_lag_2          61
15       lag168_Load_DA          54
10           Coal_lag_2          48
12            Oil_lag_2          20
9                  WD_7           8
8                  WD_6           6
7                  WD_1           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          38
16  roll2d_median_Price          10
1           Price_lag_2           9
12            Oil_lag_2           8
3         Load_DA_lag_0           6
5       WindOn_DA_lag_0           3
13            EUA_lag_2           1
2           Price_lag_7           0
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 64.98% done
********************* END NS.. 474 ****************************************************
********************* START NS ... 475 ****************************************************
[I 2025-06-15 20:27:00,681] A new study created in memory with name: no-name-001fce42-0bde-4b1b-972b-d53275799669
[I 2025-06-15 20:27:01,583] Trial 0 finished with value: 0.02104337514079374 and parameters: {'n_estimators': 204, 'learning_rate': 0.015214301536343592, 'num_leaves': 38, 'max_depth': 4, 'min_child_samples': 1, 'subsample': 0.9032142052159355, 'colsample_bytree': 0.8984540485388078, 'reg_alpha': 7.355744615133563, 'reg_lambda': 8.818292105789352, 'min_split_gain': 0.03227933107965005}. Best is trial 0 with value: 0.02104337514079374.
[Trial 0] New best: 0.02104
[I 2025-06-15 20:27:02,794] Trial 1 finished with value: 0.017343867032719337 and parameters: {'n_estimators': 156, 'learning_rate': 0.11791859398463167, 'num_leaves': 44, 'max_depth': 10, 'min_child_samples': 30, 'subsample': 0.6247247047021525, 'colsample_bytree': 0.8143034320407805, 'reg_alpha': 3.3118313789997433, 'reg_lambda': 4.7576727793995355, 'min_split_gain': 0.003468211258657816}. Best is trial 1 with value: 0.017343867032719337.
[Trial 1] New best: 0.01734
[I 2025-06-15 20:27:03,638] Trial 2 finished with value: 0.02447694674013727 and parameters: {'n_estimators': 81, 'learning_rate': 0.022229401562334338, 'num_leaves': 232, 'max_depth': 12, 'min_child_samples': 22, 'subsample': 0.9357771786479878, 'colsample_bytree': 0.6444415140268669, 'reg_alpha': 4.053226035703284, 'reg_lambda': 2.7127493784691383, 'min_split_gain': 0.03810420480843243}. Best is trial 1 with value: 0.017343867032719337.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:27:04,955] Trial 3 finished with value: 0.017472858523607936 and parameters: {'n_estimators': 264, 'learning_rate': 0.23424642307277163, 'num_leaves': 48, 'max_depth': 8, 'min_child_samples': 14, 'subsample': 0.7778152583091467, 'colsample_bytree': 0.7377471313502029, 'reg_alpha': 1.1639226748713905, 'reg_lambda': 4.274204227051132, 'min_split_gain': 0.0013132522214559517}. Best is trial 1 with value: 0.017343867032719337.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:27:06,665] Trial 4 finished with value: 0.01604939357995123 and parameters: {'n_estimators': 135, 'learning_rate': 0.10099494831691055, 'num_leaves': 217, 'max_depth': 16, 'min_child_samples': 13, 'subsample': 0.9106491283080032, 'colsample_bytree': 0.5605044501650109, 'reg_alpha': 7.704355661696896, 'reg_lambda': 8.3430965801306, 'min_split_gain': 0.004723918975869068}. Best is trial 4 with value: 0.01604939357995123.
[Trial 4] New best: 0.01605
[I 2025-06-15 20:27:10,574] Trial 5 finished with value: 0.01767588544615366 and parameters: {'n_estimators': 251, 'learning_rate': 0.01591813605305276, 'num_leaves': 72, 'max_depth': 9, 'min_child_samples': 14, 'subsample': 0.8039007203594664, 'colsample_bytree': 0.6265180864983064, 'reg_alpha': 2.098481302373987, 'reg_lambda': 9.467773824354962, 'min_split_gain': 0.010804956837105957}. Best is trial 4 with value: 0.01604939357995123.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:27:11,805] Trial 6 finished with value: 0.02280171730095264 and parameters: {'n_estimators': 110, 'learning_rate': 0.01689072243157036, 'num_leaves': 140, 'max_depth': 15, 'min_child_samples': 6, 'subsample': 0.9886476359760852, 'colsample_bytree': 0.9513685642304338, 'reg_alpha': 4.419629470118769, 'reg_lambda': 1.5450639154973833, 'min_split_gain': 0.032604666744759196}. Best is trial 4 with value: 0.01604939357995123.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:27:12,301] Trial 7 finished with value: 0.017284971194762355 and parameters: {'n_estimators': 159, 'learning_rate': 0.2818156477249296, 'num_leaves': 185, 'max_depth': 11, 'min_child_samples': 8, 'subsample': 0.914452349306484, 'colsample_bytree': 0.589212161924271, 'reg_alpha': 1.496948688214006, 'reg_lambda': 9.672851109807079, 'min_split_gain': 0.0414531498101407}. Best is trial 4 with value: 0.01604939357995123.
[I 2025-06-15 20:27:12,304] A new study created in memory with name: no-name-5ee0253b-967c-48bf-a1f5-6973646ae6e9
[I 2025-06-15 20:27:12,388] Trial 0 finished with value: 0.05582466671312349 and parameters: {'n_estimators': 126, 'learning_rate': 0.01767848490161596, 'num_leaves': 127, 'max_depth': 8, 'min_child_samples': 23, 'subsample': 0.865685069442357, 'colsample_bytree': 0.939766970070111, 'reg_alpha': 0.42806729133161636, 'reg_lambda': 6.5065106303949936, 'min_split_gain': 0.009829921359304506}. Best is trial 0 with value: 0.05582466671312349.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         792
1           Price_lag_2         791
14      pct_chg_Load_DA         424
3         Load_DA_lag_0         381
5       WindOn_DA_lag_0         348
13            EUA_lag_2         338
2           Price_lag_7         291
15       lag168_Load_DA         288
12            Oil_lag_2         280
11           NGas_lag_2         277
16  roll2d_median_Price         277
10           Coal_lag_2         212
8                  WD_6          41
9                  WD_7          32
7                  WD_1          20
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         305
12            Oil_lag_2          39
1           Price_lag_2          22
16  roll2d_median_Price          20
15       lag168_Load_DA          16
5       WindOn_DA_lag_0          10
2           Price_lag_7          10
3         Load_DA_lag_0           4
14      pct_chg_Load_DA           3
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
13            EUA_lag_2           0
8                  WD_6           0
-> 65.12% done
********************* END NS.. 475 ****************************************************
********************* START NS ... 476 ****************************************************
[I 2025-06-15 20:27:15,147] A new study created in memory with name: no-name-6016bc7e-4f08-4fec-a0a6-8b2a697180b0
[I 2025-06-15 20:27:18,072] Trial 0 finished with value: 0.001694461737384633 and parameters: {'n_estimators': 158, 'learning_rate': 0.07497656170659628, 'num_leaves': 134, 'max_depth': 15, 'min_child_samples': 11, 'subsample': 0.7558222673651295, 'colsample_bytree': 0.907623641852412, 'reg_alpha': 3.891589588641966, 'reg_lambda': 1.7972752126410896, 'min_split_gain': 0.002234291790383419}. Best is trial 0 with value: 0.001694461737384633.
[Trial 0] New best: 0.00169
[I 2025-06-15 20:27:18,787] Trial 1 finished with value: 0.00853421566457872 and parameters: {'n_estimators': 133, 'learning_rate': 0.015244952210024634, 'num_leaves': 187, 'max_depth': 5, 'min_child_samples': 30, 'subsample': 0.6326030504159688, 'colsample_bytree': 0.6809053148856795, 'reg_alpha': 7.5876677853813215, 'reg_lambda': 1.8650087309338614, 'min_split_gain': 0.02142091577854729}. Best is trial 0 with value: 0.001694461737384633.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:27:19,630] Trial 2 finished with value: 0.0018096475904146984 and parameters: {'n_estimators': 286, 'learning_rate': 0.13492261236759326, 'num_leaves': 48, 'max_depth': 14, 'min_child_samples': 3, 'subsample': 0.7529085692689943, 'colsample_bytree': 0.8342823715304958, 'reg_alpha': 7.4666813423479095, 'reg_lambda': 4.4799315847205, 'min_split_gain': 0.005204690999850825}. Best is trial 0 with value: 0.001694461737384633.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:27:20,374] Trial 3 finished with value: 0.0019481773499721534 and parameters: {'n_estimators': 174, 'learning_rate': 0.2110799351842888, 'num_leaves': 245, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.6223097383282619, 'colsample_bytree': 0.6021570910937253, 'reg_alpha': 2.9114707233880797, 'reg_lambda': 7.7616525618986945, 'min_split_gain': 0.02288069185095412}. Best is trial 0 with value: 0.001694461737384633.
[I 2025-06-15 20:27:20,376] A new study created in memory with name: no-name-d7ded66a-dd38-4aa3-beea-516463800658
[I 2025-06-15 20:27:20,452] Trial 0 finished with value: 0.004692270461811313 and parameters: {'n_estimators': 81, 'learning_rate': 0.01511299521945276, 'num_leaves': 78, 'max_depth': 5, 'min_child_samples': 22, 'subsample': 0.8677217697994901, 'colsample_bytree': 0.5371032739499111, 'reg_alpha': 2.455539193526678, 'reg_lambda': 8.327855504152904, 'min_split_gain': 0.020692217434124468}. Best is trial 0 with value: 0.004692270461811313.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1543
14      pct_chg_Load_DA        1472
1           Price_lag_2        1276
3         Load_DA_lag_0        1142
5       WindOn_DA_lag_0        1141
15       lag168_Load_DA         874
2           Price_lag_7         775
16  roll2d_median_Price         685
11           NGas_lag_2         680
13            EUA_lag_2         653
12            Oil_lag_2         644
10           Coal_lag_2         500
9                  WD_7          53
8                  WD_6          29
7                  WD_1          23
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         115
1           Price_lag_2          51
3         Load_DA_lag_0          25
16  roll2d_median_Price          18
2           Price_lag_7           2
10           Coal_lag_2           2
12            Oil_lag_2           2
11           NGas_lag_2           1
13            EUA_lag_2           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 65.25% done
********************* END NS.. 476 ****************************************************
********************* START NS ... 477 ****************************************************
[I 2025-06-15 20:27:24,311] A new study created in memory with name: no-name-d0c0c388-8717-481a-80e5-79bbd8afdffe
[I 2025-06-15 20:27:26,678] Trial 0 finished with value: 0.002074369886623601 and parameters: {'n_estimators': 241, 'learning_rate': 0.05731729966511426, 'num_leaves': 239, 'max_depth': 10, 'min_child_samples': 10, 'subsample': 0.5211708802529456, 'colsample_bytree': 0.9637119542780357, 'reg_alpha': 2.712897509319545, 'reg_lambda': 5.658512710415008, 'min_split_gain': 0.010920568349540489}. Best is trial 0 with value: 0.002074369886623601.
[Trial 0] New best: 0.00207
[I 2025-06-15 20:27:27,510] Trial 1 finished with value: 0.002859344482780052 and parameters: {'n_estimators': 108, 'learning_rate': 0.03384594771991102, 'num_leaves': 117, 'max_depth': 10, 'min_child_samples': 17, 'subsample': 0.7790715095999695, 'colsample_bytree': 0.611390599924069, 'reg_alpha': 9.04333547601302, 'reg_lambda': 8.90653296543418, 'min_split_gain': 0.03587987091157742}. Best is trial 0 with value: 0.002074369886623601.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:27:28,000] Trial 2 finished with value: 0.002266040224381885 and parameters: {'n_estimators': 62, 'learning_rate': 0.10755332225384728, 'num_leaves': 81, 'max_depth': 6, 'min_child_samples': 12, 'subsample': 0.6729567967226382, 'colsample_bytree': 0.885612971190784, 'reg_alpha': 8.627998345652237, 'reg_lambda': 5.782930797166296, 'min_split_gain': 0.030646116066637082}. Best is trial 0 with value: 0.002074369886623601.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:27:28,850] Trial 3 finished with value: 0.002782900172229208 and parameters: {'n_estimators': 276, 'learning_rate': 0.09008742569396565, 'num_leaves': 120, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.9089533377605843, 'colsample_bytree': 0.5859151264106259, 'reg_alpha': 4.1580362208901995, 'reg_lambda': 4.766895117469106, 'min_split_gain': 0.04086927141535089}. Best is trial 0 with value: 0.002074369886623601.
[I 2025-06-15 20:27:28,852] A new study created in memory with name: no-name-461719dc-2bff-49fd-bdd1-7b323dad7390
[I 2025-06-15 20:27:28,917] Trial 0 finished with value: 0.011530802346019274 and parameters: {'n_estimators': 279, 'learning_rate': 0.0979587075392479, 'num_leaves': 114, 'max_depth': 12, 'min_child_samples': 24, 'subsample': 0.8230397168868486, 'colsample_bytree': 0.8499168524905012, 'reg_alpha': 7.506845049133116, 'reg_lambda': 1.0472512913894405, 'min_split_gain': 0.017761270507602067}. Best is trial 0 with value: 0.011530802346019274.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1656
14      pct_chg_Load_DA        1597
1           Price_lag_2        1424
3         Load_DA_lag_0        1032
5       WindOn_DA_lag_0         913
15       lag168_Load_DA         811
2           Price_lag_7         752
16  roll2d_median_Price         696
11           NGas_lag_2         691
10           Coal_lag_2         468
13            EUA_lag_2         466
12            Oil_lag_2         332
9                  WD_7          46
7                  WD_1          37
8                  WD_6          19
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           5
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 65.39% done
********************* END NS.. 477 ****************************************************
********************* START NS ... 478 ****************************************************
[I 2025-06-15 20:27:32,824] A new study created in memory with name: no-name-32e5ced1-0e57-4139-96da-64b84c18092f
[I 2025-06-15 20:27:33,614] Trial 0 finished with value: 0.005762128502150095 and parameters: {'n_estimators': 133, 'learning_rate': 0.02304085545930796, 'num_leaves': 216, 'max_depth': 8, 'min_child_samples': 18, 'subsample': 0.6763153230760779, 'colsample_bytree': 0.9034031899307005, 'reg_alpha': 9.920595448899613, 'reg_lambda': 8.674985511166476, 'min_split_gain': 0.013470741829377098}. Best is trial 0 with value: 0.005762128502150095.
[Trial 0] New best: 0.00576
[I 2025-06-15 20:27:33,967] Trial 1 finished with value: 0.005235321375228317 and parameters: {'n_estimators': 139, 'learning_rate': 0.2000470528853203, 'num_leaves': 218, 'max_depth': 6, 'min_child_samples': 26, 'subsample': 0.8330496163432932, 'colsample_bytree': 0.6916507483662142, 'reg_alpha': 3.884360691786868, 'reg_lambda': 2.491282059946478, 'min_split_gain': 0.02687118806214325}. Best is trial 1 with value: 0.005235321375228317.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:27:34,798] Trial 2 finished with value: 0.005077985601848337 and parameters: {'n_estimators': 113, 'learning_rate': 0.0754249775953526, 'num_leaves': 240, 'max_depth': 9, 'min_child_samples': 28, 'subsample': 0.6290209947829131, 'colsample_bytree': 0.8184488223051618, 'reg_alpha': 6.129844902310482, 'reg_lambda': 1.9191206257756843, 'min_split_gain': 0.022732756242906785}. Best is trial 2 with value: 0.005077985601848337.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:27:36,333] Trial 3 finished with value: 0.004529645994600578 and parameters: {'n_estimators': 213, 'learning_rate': 0.1412459440493645, 'num_leaves': 210, 'max_depth': 15, 'min_child_samples': 16, 'subsample': 0.7022290806610958, 'colsample_bytree': 0.8440102528792394, 'reg_alpha': 0.1543226496691008, 'reg_lambda': 1.4507628248465743, 'min_split_gain': 0.021527978391917852}. Best is trial 3 with value: 0.004529645994600578.
[Trial 3] New best: 0.00453
[I 2025-06-15 20:27:37,399] Trial 4 finished with value: 0.004700790421060635 and parameters: {'n_estimators': 136, 'learning_rate': 0.14683297633547646, 'num_leaves': 74, 'max_depth': 15, 'min_child_samples': 6, 'subsample': 0.8022524186914968, 'colsample_bytree': 0.8564519879862837, 'reg_alpha': 1.3530795926033368, 'reg_lambda': 5.153915841087211, 'min_split_gain': 0.015234568215332124}. Best is trial 3 with value: 0.004529645994600578.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:27:37,909] Trial 5 finished with value: 0.005067236464188277 and parameters: {'n_estimators': 106, 'learning_rate': 0.16461657060520093, 'num_leaves': 197, 'max_depth': 15, 'min_child_samples': 22, 'subsample': 0.6937365920294993, 'colsample_bytree': 0.5809253993158634, 'reg_alpha': 5.517227026406224, 'reg_lambda': 5.904778567259546, 'min_split_gain': 0.021613716620140406}. Best is trial 3 with value: 0.004529645994600578.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:27:38,614] Trial 6 finished with value: 0.004904777788202779 and parameters: {'n_estimators': 83, 'learning_rate': 0.10188153254885421, 'num_leaves': 53, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.5970274805630766, 'colsample_bytree': 0.983375646026962, 'reg_alpha': 5.379929084302423, 'reg_lambda': 2.9200206015497567, 'min_split_gain': 0.004454468946503637}. Best is trial 3 with value: 0.004529645994600578.
[I 2025-06-15 20:27:38,617] A new study created in memory with name: no-name-fb7905ca-1308-4aa1-a78f-4a7aeb53a277
[I 2025-06-15 20:27:38,687] Trial 0 finished with value: 0.022619962224629586 and parameters: {'n_estimators': 262, 'learning_rate': 0.07849575224287841, 'num_leaves': 73, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.5165455871018554, 'colsample_bytree': 0.5792116090891496, 'reg_alpha': 6.385377058923827, 'reg_lambda': 4.727097760969775, 'min_split_gain': 0.03879788320608239}. Best is trial 0 with value: 0.022619962224629586.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1118
14      pct_chg_Load_DA         989
0           Price_lag_1         879
16  roll2d_median_Price         618
3         Load_DA_lag_0         608
2           Price_lag_7         532
5       WindOn_DA_lag_0         512
15       lag168_Load_DA         461
11           NGas_lag_2         435
10           Coal_lag_2         362
13            EUA_lag_2         235
12            Oil_lag_2         205
9                  WD_7          19
7                  WD_1          12
8                  WD_6           3
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 65.53% done
********************* END NS.. 478 ****************************************************
********************* START NS ... 479 ****************************************************
[I 2025-06-15 20:27:41,521] A new study created in memory with name: no-name-d32807c5-770f-465a-abea-0a6dd5999c93
[I 2025-06-15 20:27:41,828] Trial 0 finished with value: 0.03261047821611287 and parameters: {'n_estimators': 76, 'learning_rate': 0.19502719527205442, 'num_leaves': 255, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.5873271015854729, 'colsample_bytree': 0.7430050993162522, 'reg_alpha': 4.3799418040509615, 'reg_lambda': 2.668736535652606, 'min_split_gain': 0.009788583401583223}. Best is trial 0 with value: 0.03261047821611287.
[Trial 0] New best: 0.03261
[I 2025-06-15 20:27:43,033] Trial 1 finished with value: 0.029158443455447436 and parameters: {'n_estimators': 165, 'learning_rate': 0.03021325693380013, 'num_leaves': 43, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.8704452043630277, 'colsample_bytree': 0.639567175239339, 'reg_alpha': 8.634014797048998, 'reg_lambda': 1.6168400432780106, 'min_split_gain': 0.027330549567534834}. Best is trial 1 with value: 0.029158443455447436.
[Trial 1] New best: 0.02916
[I 2025-06-15 20:27:43,323] Trial 2 finished with value: 0.03207196017202366 and parameters: {'n_estimators': 166, 'learning_rate': 0.29030881984158574, 'num_leaves': 38, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.9566282493942817, 'colsample_bytree': 0.7781844159996297, 'reg_alpha': 5.720074722787729, 'reg_lambda': 6.251783269788654, 'min_split_gain': 0.02611822141181221}. Best is trial 1 with value: 0.029158443455447436.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:27:44,382] Trial 3 finished with value: 0.03068617300347128 and parameters: {'n_estimators': 104, 'learning_rate': 0.0877540752449381, 'num_leaves': 200, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.6178838548244603, 'colsample_bytree': 0.679079117838805, 'reg_alpha': 3.719452410476932, 'reg_lambda': 1.5656077241267075, 'min_split_gain': 0.013316972858789728}. Best is trial 1 with value: 0.029158443455447436.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:27:47,175] Trial 4 finished with value: 0.029887926996965464 and parameters: {'n_estimators': 289, 'learning_rate': 0.015064236755854476, 'num_leaves': 77, 'max_depth': 11, 'min_child_samples': 18, 'subsample': 0.8005174128137451, 'colsample_bytree': 0.8079589653761617, 'reg_alpha': 9.801044721864447, 'reg_lambda': 9.7716410292135, 'min_split_gain': 0.000378898343555395}. Best is trial 1 with value: 0.029158443455447436.
[I 2025-06-15 20:27:47,177] A new study created in memory with name: no-name-454dddc2-fbc2-4b3f-b48c-877b9379fbbf
[I 2025-06-15 20:27:47,201] Trial 0 finished with value: 0.06674493465955382 and parameters: {'n_estimators': 65, 'learning_rate': 0.12621888741392526, 'num_leaves': 187, 'max_depth': 8, 'min_child_samples': 7, 'subsample': 0.9607443991606575, 'colsample_bytree': 0.8296796816929433, 'reg_alpha': 7.175976008642754, 'reg_lambda': 9.698887147678706, 'min_split_gain': 0.0017433921066864644}. Best is trial 0 with value: 0.06674493465955382.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1267
1           Price_lag_2         953
14      pct_chg_Load_DA         669
3         Load_DA_lag_0         337
16  roll2d_median_Price         324
2           Price_lag_7         322
11           NGas_lag_2         249
15       lag168_Load_DA         203
13            EUA_lag_2         162
10           Coal_lag_2         127
12            Oil_lag_2         111
5       WindOn_DA_lag_0         109
9                  WD_7          41
8                  WD_6          27
7                  WD_1           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          11
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 65.66% done
********************* END NS.. 479 ****************************************************
********************* START NS ... 480 ****************************************************
[I 2025-06-15 20:27:49,656] A new study created in memory with name: no-name-c55eb05e-024d-4e68-b412-8c28f658ef6c
[I 2025-06-15 20:27:50,347] Trial 0 finished with value: 0.025818363510043598 and parameters: {'n_estimators': 190, 'learning_rate': 0.10465414461164894, 'num_leaves': 213, 'max_depth': 12, 'min_child_samples': 2, 'subsample': 0.7676321414992638, 'colsample_bytree': 0.8668281100445201, 'reg_alpha': 8.29797810450456, 'reg_lambda': 5.602546610524083, 'min_split_gain': 0.04179022355924261}. Best is trial 0 with value: 0.025818363510043598.
[Trial 0] New best: 0.02582
[I 2025-06-15 20:27:51,406] Trial 1 finished with value: 0.04698361396812167 and parameters: {'n_estimators': 66, 'learning_rate': 0.013677780480907611, 'num_leaves': 140, 'max_depth': 13, 'min_child_samples': 24, 'subsample': 0.9974749981681876, 'colsample_bytree': 0.7391748465823637, 'reg_alpha': 0.7817296912323579, 'reg_lambda': 7.083047635888206, 'min_split_gain': 0.005748809630455005}. Best is trial 0 with value: 0.025818363510043598.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:27:51,752] Trial 2 finished with value: 0.029583553911650618 and parameters: {'n_estimators': 150, 'learning_rate': 0.09448446166395016, 'num_leaves': 210, 'max_depth': 4, 'min_child_samples': 15, 'subsample': 0.7064678641069992, 'colsample_bytree': 0.9186702311793611, 'reg_alpha': 6.473086404263882, 'reg_lambda': 2.254474764782789, 'min_split_gain': 0.03721817559862328}. Best is trial 0 with value: 0.025818363510043598.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:27:52,660] Trial 3 finished with value: 0.027970897194864976 and parameters: {'n_estimators': 204, 'learning_rate': 0.05796004437792475, 'num_leaves': 212, 'max_depth': 16, 'min_child_samples': 18, 'subsample': 0.7567551272037589, 'colsample_bytree': 0.7547438123897398, 'reg_alpha': 9.110856808430718, 'reg_lambda': 3.7836853363088583, 'min_split_gain': 0.02616910650847741}. Best is trial 0 with value: 0.025818363510043598.
[I 2025-06-15 20:27:52,663] A new study created in memory with name: no-name-c567e019-096b-4096-a11b-ed7f0deda049
[I 2025-06-15 20:27:52,694] Trial 0 finished with value: 0.058951948728349794 and parameters: {'n_estimators': 115, 'learning_rate': 0.07930714689899328, 'num_leaves': 39, 'max_depth': 8, 'min_child_samples': 29, 'subsample': 0.7639878865285332, 'colsample_bytree': 0.6511365427474384, 'reg_alpha': 5.183619410734589, 'reg_lambda': 9.087632783708143, 'min_split_gain': 0.009829129322109165}. Best is trial 0 with value: 0.058951948728349794.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         465
1           Price_lag_2         315
14      pct_chg_Load_DA         273
11           NGas_lag_2         172
2           Price_lag_7         162
16  roll2d_median_Price         136
3         Load_DA_lag_0         135
15       lag168_Load_DA          98
10           Coal_lag_2          93
5       WindOn_DA_lag_0          92
13            EUA_lag_2          70
12            Oil_lag_2          58
9                  WD_7           9
8                  WD_6           8
7                  WD_1           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          23
1           Price_lag_2           4
15       lag168_Load_DA           2
3         Load_DA_lag_0           2
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 65.80% done
********************* END NS.. 480 ****************************************************
********************* START NS ... 481 ****************************************************
[I 2025-06-15 20:27:54,884] A new study created in memory with name: no-name-d97f9a1a-6a98-4467-b621-e9562b3d4c74
[I 2025-06-15 20:27:55,458] Trial 0 finished with value: 0.00303769347987662 and parameters: {'n_estimators': 241, 'learning_rate': 0.23815525790022632, 'num_leaves': 89, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.5109617030601101, 'colsample_bytree': 0.5682570293092228, 'reg_alpha': 1.8385448060893106, 'reg_lambda': 9.219518756211937, 'min_split_gain': 0.002244702521666825}. Best is trial 0 with value: 0.00303769347987662.
[Trial 0] New best: 0.00304
[I 2025-06-15 20:27:56,285] Trial 1 finished with value: 0.008124543766088232 and parameters: {'n_estimators': 100, 'learning_rate': 0.018916189045345484, 'num_leaves': 234, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.891307855651141, 'colsample_bytree': 0.845112723436561, 'reg_alpha': 6.4569213930076295, 'reg_lambda': 5.761903255255671, 'min_split_gain': 0.02028129167322742}. Best is trial 0 with value: 0.00303769347987662.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:28:00,618] Trial 2 finished with value: 0.0045540498911347095 and parameters: {'n_estimators': 140, 'learning_rate': 0.018813541400085697, 'num_leaves': 167, 'max_depth': 9, 'min_child_samples': 2, 'subsample': 0.834900324948652, 'colsample_bytree': 0.5968046595630551, 'reg_alpha': 0.09853904403167468, 'reg_lambda': 0.8500536346528031, 'min_split_gain': 0.005996838905093449}. Best is trial 0 with value: 0.00303769347987662.
[I 2025-06-15 20:28:00,809] Trial 3 finished with value: 0.00351944913985191 and parameters: {'n_estimators': 51, 'learning_rate': 0.13216366832767232, 'num_leaves': 83, 'max_depth': 4, 'min_child_samples': 30, 'subsample': 0.8466472554698493, 'colsample_bytree': 0.6871208719508369, 'reg_alpha': 3.0489768782626223, 'reg_lambda': 8.436498175478617, 'min_split_gain': 0.044899001548957165}. Best is trial 0 with value: 0.00303769347987662.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:28:00,811] A new study created in memory with name: no-name-037a4abe-7641-439a-9ae7-0659d5ed8394
[I 2025-06-15 20:28:00,933] Trial 0 finished with value: 0.005266571744203093 and parameters: {'n_estimators': 200, 'learning_rate': 0.017017601343274635, 'num_leaves': 120, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.7286199701103144, 'colsample_bytree': 0.6654987242253592, 'reg_alpha': 0.8275461738278911, 'reg_lambda': 9.775290603476538, 'min_split_gain': 0.038375058268056504}. Best is trial 0 with value: 0.005266571744203093.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         558
1           Price_lag_2         483
14      pct_chg_Load_DA         360
3         Load_DA_lag_0         301
16  roll2d_median_Price         285
15       lag168_Load_DA         284
5       WindOn_DA_lag_0         268
11           NGas_lag_2         256
13            EUA_lag_2         232
2           Price_lag_7         207
10           Coal_lag_2         175
12            Oil_lag_2         160
9                  WD_7          12
7                  WD_1           8
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         184
15       lag168_Load_DA          67
3         Load_DA_lag_0          66
1           Price_lag_2          25
10           Coal_lag_2          12
11           NGas_lag_2           6
13            EUA_lag_2           3
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 65.94% done
********************* END NS.. 481 ****************************************************
********************* START NS ... 482 ****************************************************
[I 2025-06-15 20:28:03,246] A new study created in memory with name: no-name-6b75acc6-09b8-47b6-85b8-038db0bd7526
[I 2025-06-15 20:28:04,591] Trial 0 finished with value: 0.007268569591040438 and parameters: {'n_estimators': 235, 'learning_rate': 0.0835795953882078, 'num_leaves': 120, 'max_depth': 9, 'min_child_samples': 28, 'subsample': 0.6482522315266694, 'colsample_bytree': 0.9486270835475381, 'reg_alpha': 3.8494416066727846, 'reg_lambda': 2.16812856040712, 'min_split_gain': 0.005339048875778074}. Best is trial 0 with value: 0.007268569591040438.
[Trial 0] New best: 0.00727
[I 2025-06-15 20:28:06,090] Trial 1 finished with value: 0.009154180911106797 and parameters: {'n_estimators': 289, 'learning_rate': 0.014069510612795611, 'num_leaves': 46, 'max_depth': 5, 'min_child_samples': 12, 'subsample': 0.8961420655473096, 'colsample_bytree': 0.9196219740231204, 'reg_alpha': 5.230230824521102, 'reg_lambda': 7.211969462107751, 'min_split_gain': 0.012940145825171162}. Best is trial 0 with value: 0.007268569591040438.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:28:07,375] Trial 2 finished with value: 0.011108462120987858 and parameters: {'n_estimators': 258, 'learning_rate': 0.03053343892554683, 'num_leaves': 95, 'max_depth': 5, 'min_child_samples': 12, 'subsample': 0.6760676345857894, 'colsample_bytree': 0.5506869143651036, 'reg_alpha': 9.781652400768087, 'reg_lambda': 6.5361142606261025, 'min_split_gain': 0.01595088032283076}. Best is trial 0 with value: 0.007268569591040438.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:28:08,178] Trial 3 finished with value: 0.009859549268784311 and parameters: {'n_estimators': 234, 'learning_rate': 0.06500333003757372, 'num_leaves': 107, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.5036001719816182, 'colsample_bytree': 0.5046096615045195, 'reg_alpha': 9.035720818010006, 'reg_lambda': 7.322263421501675, 'min_split_gain': 0.015446687692356166}. Best is trial 0 with value: 0.007268569591040438.
[I 2025-06-15 20:28:08,181] A new study created in memory with name: no-name-d91d9f7f-7741-495c-a4eb-6ab81e778fc5
[I 2025-06-15 20:28:08,283] Trial 0 finished with value: 0.029070679051304806 and parameters: {'n_estimators': 276, 'learning_rate': 0.02237980339087364, 'num_leaves': 86, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.8961344049848351, 'colsample_bytree': 0.7634126399547938, 'reg_alpha': 3.0092011345665846, 'reg_lambda': 7.238237009298645, 'min_split_gain': 0.010868107990111137}. Best is trial 0 with value: 0.029070679051304806.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1118
14      pct_chg_Load_DA         868
1           Price_lag_2         806
3         Load_DA_lag_0         609
15       lag168_Load_DA         519
5       WindOn_DA_lag_0         501
11           NGas_lag_2         408
16  roll2d_median_Price         382
2           Price_lag_7         362
10           Coal_lag_2         268
13            EUA_lag_2         264
12            Oil_lag_2         250
9                  WD_7          25
7                  WD_1          21
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         109
15       lag168_Load_DA          76
1           Price_lag_2          17
2           Price_lag_7          17
10           Coal_lag_2          16
3         Load_DA_lag_0           7
13            EUA_lag_2           6
11           NGas_lag_2           4
16  roll2d_median_Price           3
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
12            Oil_lag_2           0
5       WindOn_DA_lag_0           0
14      pct_chg_Load_DA           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 66.07% done
********************* END NS.. 482 ****************************************************
********************* START NS ... 483 ****************************************************
[I 2025-06-15 20:28:11,094] A new study created in memory with name: no-name-ceb824b1-868a-4bf7-b981-07f7ab6fdd1a
[I 2025-06-15 20:28:11,676] Trial 0 finished with value: 0.007612459674089527 and parameters: {'n_estimators': 179, 'learning_rate': 0.12859629847829057, 'num_leaves': 99, 'max_depth': 6, 'min_child_samples': 1, 'subsample': 0.6312925099415789, 'colsample_bytree': 0.9204973827909597, 'reg_alpha': 0.6672656473639738, 'reg_lambda': 8.327155630415486, 'min_split_gain': 0.036934677859104204}. Best is trial 0 with value: 0.007612459674089527.
[Trial 0] New best: 0.00761
[I 2025-06-15 20:28:12,427] Trial 1 finished with value: 0.007605020714397431 and parameters: {'n_estimators': 285, 'learning_rate': 0.10271000563609538, 'num_leaves': 195, 'max_depth': 11, 'min_child_samples': 18, 'subsample': 0.8183789140595996, 'colsample_bytree': 0.9128910598328521, 'reg_alpha': 7.579894279172926, 'reg_lambda': 4.804153010481768, 'min_split_gain': 0.01038442596332177}. Best is trial 1 with value: 0.007605020714397431.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:28:14,025] Trial 2 finished with value: 0.0067667946265614135 and parameters: {'n_estimators': 238, 'learning_rate': 0.09086450545016182, 'num_leaves': 104, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.8965133485898393, 'colsample_bytree': 0.8045627316607702, 'reg_alpha': 9.602777319371162, 'reg_lambda': 6.718825436384192, 'min_split_gain': 0.0008487743982073415}. Best is trial 2 with value: 0.0067667946265614135.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:28:14,744] Trial 3 finished with value: 0.012196207726192697 and parameters: {'n_estimators': 57, 'learning_rate': 0.035350716260530955, 'num_leaves': 136, 'max_depth': 12, 'min_child_samples': 16, 'subsample': 0.7416695431370721, 'colsample_bytree': 0.8983305184156084, 'reg_alpha': 5.355395030705226, 'reg_lambda': 5.590969481451628, 'min_split_gain': 0.009264808878086795}. Best is trial 2 with value: 0.0067667946265614135.
[I 2025-06-15 20:28:14,747] A new study created in memory with name: no-name-9ed327fc-8b46-4842-8990-9590a744d0dc
[I 2025-06-15 20:28:14,792] Trial 0 finished with value: 0.03681034744348308 and parameters: {'n_estimators': 104, 'learning_rate': 0.01627856009816903, 'num_leaves': 83, 'max_depth': 10, 'min_child_samples': 4, 'subsample': 0.8785431836119999, 'colsample_bytree': 0.641481353528754, 'reg_alpha': 3.1700018172035636, 'reg_lambda': 7.303832433646856, 'min_split_gain': 0.03593286916153456}. Best is trial 0 with value: 0.03681034744348308.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         940
1           Price_lag_2         938
14      pct_chg_Load_DA         756
5       WindOn_DA_lag_0         701
3         Load_DA_lag_0         612
2           Price_lag_7         514
11           NGas_lag_2         495
16  roll2d_median_Price         474
15       lag168_Load_DA         472
13            EUA_lag_2         356
10           Coal_lag_2         319
12            Oil_lag_2         259
9                  WD_7          44
8                  WD_6          23
7                  WD_1          21
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          66
15       lag168_Load_DA          65
1           Price_lag_2          14
3         Load_DA_lag_0          12
2           Price_lag_7           4
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 66.21% done
********************* END NS.. 483 ****************************************************
********************* START NS ... 484 ****************************************************
[I 2025-06-15 20:28:17,919] A new study created in memory with name: no-name-6d27003c-4dcc-4d0b-8cdc-b8a9885f9bd5
[I 2025-06-15 20:28:20,142] Trial 0 finished with value: 0.007682503795824498 and parameters: {'n_estimators': 207, 'learning_rate': 0.013322624795668833, 'num_leaves': 156, 'max_depth': 13, 'min_child_samples': 4, 'subsample': 0.6468172576083793, 'colsample_bytree': 0.9787965365709446, 'reg_alpha': 8.846167145258692, 'reg_lambda': 2.815769418249486, 'min_split_gain': 0.02285259092442582}. Best is trial 0 with value: 0.007682503795824498.
[Trial 0] New best: 0.00768
[I 2025-06-15 20:28:20,736] Trial 1 finished with value: 0.004886963893834851 and parameters: {'n_estimators': 186, 'learning_rate': 0.15232330867825647, 'num_leaves': 239, 'max_depth': 14, 'min_child_samples': 10, 'subsample': 0.7868575364233104, 'colsample_bytree': 0.8103393073745144, 'reg_alpha': 5.001649943532475, 'reg_lambda': 8.935946295745739, 'min_split_gain': 0.03379702626540761}. Best is trial 1 with value: 0.004886963893834851.
[Trial 1] New best: 0.00489
[I 2025-06-15 20:28:21,068] Trial 2 finished with value: 0.004508119638278732 and parameters: {'n_estimators': 100, 'learning_rate': 0.25584429091789995, 'num_leaves': 139, 'max_depth': 5, 'min_child_samples': 10, 'subsample': 0.714668055611199, 'colsample_bytree': 0.9331247729889978, 'reg_alpha': 3.1177465618011757, 'reg_lambda': 1.8595896523991484, 'min_split_gain': 0.01837497503104335}. Best is trial 2 with value: 0.004508119638278732.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:28:25,100] Trial 3 finished with value: 0.005951110408930894 and parameters: {'n_estimators': 118, 'learning_rate': 0.0375016535293722, 'num_leaves': 244, 'max_depth': 12, 'min_child_samples': 2, 'subsample': 0.9972180109798932, 'colsample_bytree': 0.7527748373564649, 'reg_alpha': 0.592587022282125, 'reg_lambda': 1.1720067010751956, 'min_split_gain': 0.02396568107380058}. Best is trial 2 with value: 0.004508119638278732.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:28:27,419] Trial 4 finished with value: 0.009324918246480615 and parameters: {'n_estimators': 143, 'learning_rate': 0.01802868586325211, 'num_leaves': 160, 'max_depth': 16, 'min_child_samples': 11, 'subsample': 0.9066899250439495, 'colsample_bytree': 0.8667498190456243, 'reg_alpha': 1.817709878315198, 'reg_lambda': 3.8654658344491932, 'min_split_gain': 0.020461600796788104}. Best is trial 2 with value: 0.004508119638278732.
[I 2025-06-15 20:28:27,421] A new study created in memory with name: no-name-76d5e313-d343-4651-861c-056072334178
[I 2025-06-15 20:28:27,463] Trial 0 finished with value: 0.1399148934513794 and parameters: {'n_estimators': 157, 'learning_rate': 0.05241867096638667, 'num_leaves': 80, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.7377213080390157, 'colsample_bytree': 0.7953095803130227, 'reg_alpha': 5.556661739007208, 'reg_lambda': 3.967717825082, 'min_split_gain': 0.03812660320125092}. Best is trial 0 with value: 0.1399148934513794.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         234
1           Price_lag_2         128
14      pct_chg_Load_DA         116
2           Price_lag_7          88
3         Load_DA_lag_0          78
11           NGas_lag_2          76
15       lag168_Load_DA          65
16  roll2d_median_Price          59
5       WindOn_DA_lag_0          59
10           Coal_lag_2          47
12            Oil_lag_2          25
13            EUA_lag_2          16
9                  WD_7           5
7                  WD_1           3
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          40
15       lag168_Load_DA           5
1           Price_lag_2           5
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 66.35% done
********************* END NS.. 484 ****************************************************
********************* START NS ... 485 ****************************************************
[I 2025-06-15 20:28:29,282] A new study created in memory with name: no-name-e85f903b-2a1b-43f3-bf22-d7e985012905
[I 2025-06-15 20:28:30,040] Trial 0 finished with value: 0.00982374043339009 and parameters: {'n_estimators': 293, 'learning_rate': 0.029643503415666014, 'num_leaves': 215, 'max_depth': 4, 'min_child_samples': 27, 'subsample': 0.549076726559758, 'colsample_bytree': 0.6156877033382668, 'reg_alpha': 7.637329625619456, 'reg_lambda': 2.2983140562896995, 'min_split_gain': 0.03583824599269879}. Best is trial 0 with value: 0.00982374043339009.
[Trial 0] New best: 0.00982
[I 2025-06-15 20:28:31,234] Trial 1 finished with value: 0.008445354893029387 and parameters: {'n_estimators': 252, 'learning_rate': 0.10768104335593491, 'num_leaves': 255, 'max_depth': 8, 'min_child_samples': 20, 'subsample': 0.527291296635791, 'colsample_bytree': 0.8428499203761018, 'reg_alpha': 8.152540568952201, 'reg_lambda': 2.6228695170872074, 'min_split_gain': 0.0012001768872836772}. Best is trial 1 with value: 0.008445354893029387.
[Trial 1] New best: 0.00845
[I 2025-06-15 20:28:32,281] Trial 2 finished with value: 0.017490017345006954 and parameters: {'n_estimators': 139, 'learning_rate': 0.015439415092459453, 'num_leaves': 62, 'max_depth': 12, 'min_child_samples': 26, 'subsample': 0.6522806810935717, 'colsample_bytree': 0.850051377751119, 'reg_alpha': 7.362559048187315, 'reg_lambda': 4.087879945752835, 'min_split_gain': 0.04657483811330596}. Best is trial 1 with value: 0.008445354893029387.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:28:32,648] Trial 3 finished with value: 0.008539442313684257 and parameters: {'n_estimators': 156, 'learning_rate': 0.21680983616381397, 'num_leaves': 76, 'max_depth': 9, 'min_child_samples': 29, 'subsample': 0.5863481291379059, 'colsample_bytree': 0.5790537657473458, 'reg_alpha': 5.666965781288106, 'reg_lambda': 4.151846759862505, 'min_split_gain': 0.034341697137751474}. Best is trial 1 with value: 0.008445354893029387.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:28:33,616] Trial 4 finished with value: 0.010498727753208264 and parameters: {'n_estimators': 102, 'learning_rate': 0.03810576482439876, 'num_leaves': 135, 'max_depth': 9, 'min_child_samples': 24, 'subsample': 0.9420142626881954, 'colsample_bytree': 0.5704665688073471, 'reg_alpha': 7.12285538081079, 'reg_lambda': 4.794668131518781, 'min_split_gain': 0.04899591793644794}. Best is trial 1 with value: 0.008445354893029387.
[I 2025-06-15 20:28:33,618] A new study created in memory with name: no-name-03b9637b-8bd1-4df0-9b95-e32e99c40e66
[I 2025-06-15 20:28:33,665] Trial 0 finished with value: 0.1368039653477749 and parameters: {'n_estimators': 187, 'learning_rate': 0.2853478618277702, 'num_leaves': 210, 'max_depth': 11, 'min_child_samples': 25, 'subsample': 0.6718166821318765, 'colsample_bytree': 0.6813916628897784, 'reg_alpha': 5.321804209953934, 'reg_lambda': 7.420768892452814, 'min_split_gain': 0.005775533826362861}. Best is trial 0 with value: 0.1368039653477749.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         726
1           Price_lag_2         666
14      pct_chg_Load_DA         529
3         Load_DA_lag_0         392
5       WindOn_DA_lag_0         369
16  roll2d_median_Price         297
2           Price_lag_7         296
13            EUA_lag_2         293
15       lag168_Load_DA         283
11           NGas_lag_2         261
10           Coal_lag_2         225
12            Oil_lag_2         175
9                  WD_7          34
8                  WD_6          18
7                  WD_1          13
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          14
15       lag168_Load_DA           3
1           Price_lag_2           2
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 66.48% done
********************* END NS.. 485 ****************************************************
********************* START NS ... 486 ****************************************************
[I 2025-06-15 20:28:36,387] A new study created in memory with name: no-name-e25746e0-6746-4065-86cd-40152151412d
[I 2025-06-15 20:28:37,702] Trial 0 finished with value: 0.0077631141701033175 and parameters: {'n_estimators': 294, 'learning_rate': 0.06499922135686635, 'num_leaves': 134, 'max_depth': 15, 'min_child_samples': 23, 'subsample': 0.8227676292607047, 'colsample_bytree': 0.5913815828445063, 'reg_alpha': 3.0117730511592544, 'reg_lambda': 2.0025819514782617, 'min_split_gain': 0.03122941942318986}. Best is trial 0 with value: 0.0077631141701033175.
[Trial 0] New best: 0.00776
[I 2025-06-15 20:28:39,741] Trial 1 finished with value: 0.006954355648242348 and parameters: {'n_estimators': 197, 'learning_rate': 0.07552490302146142, 'num_leaves': 87, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.9245401735374345, 'colsample_bytree': 0.6838501808420634, 'reg_alpha': 6.307583776487483, 'reg_lambda': 6.272454439643424, 'min_split_gain': 0.001322587879904863}. Best is trial 1 with value: 0.006954355648242348.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:28:42,089] Trial 2 finished with value: 0.0067233016647946856 and parameters: {'n_estimators': 235, 'learning_rate': 0.07982300396732349, 'num_leaves': 75, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.8820228091423988, 'colsample_bytree': 0.8778358378362956, 'reg_alpha': 4.0269948294033275, 'reg_lambda': 0.0319551834590559, 'min_split_gain': 0.0007317023622612218}. Best is trial 2 with value: 0.0067233016647946856.
[Trial 2] New best: 0.00672
[I 2025-06-15 20:28:42,975] Trial 3 finished with value: 0.018484283171297668 and parameters: {'n_estimators': 102, 'learning_rate': 0.01873872514602345, 'num_leaves': 98, 'max_depth': 14, 'min_child_samples': 24, 'subsample': 0.5165072456820794, 'colsample_bytree': 0.6400309409694932, 'reg_alpha': 7.716699532948866, 'reg_lambda': 4.320301252495108, 'min_split_gain': 0.03970955924560304}. Best is trial 2 with value: 0.0067233016647946856.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:28:43,630] Trial 4 finished with value: 0.00931062135664367 and parameters: {'n_estimators': 51, 'learning_rate': 0.060935187556622694, 'num_leaves': 232, 'max_depth': 9, 'min_child_samples': 8, 'subsample': 0.8698586891729249, 'colsample_bytree': 0.7530838202421757, 'reg_alpha': 4.204958761525587, 'reg_lambda': 4.183044455499443, 'min_split_gain': 0.039848485673046136}. Best is trial 2 with value: 0.0067233016647946856.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:28:44,871] Trial 5 finished with value: 0.007484893654136814 and parameters: {'n_estimators': 174, 'learning_rate': 0.08534304487242432, 'num_leaves': 175, 'max_depth': 13, 'min_child_samples': 19, 'subsample': 0.8610589568476409, 'colsample_bytree': 0.9207091822906597, 'reg_alpha': 6.674498217072565, 'reg_lambda': 2.519206030989236, 'min_split_gain': 0.005959519574785049}. Best is trial 2 with value: 0.0067233016647946856.
[I 2025-06-15 20:28:44,873] A new study created in memory with name: no-name-30cbf55a-b95d-4141-b533-a27c042ed615
[I 2025-06-15 20:28:44,958] Trial 0 finished with value: 0.011731260773818903 and parameters: {'n_estimators': 125, 'learning_rate': 0.05035670511158348, 'num_leaves': 172, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.7770979530068505, 'colsample_bytree': 0.5780482842322802, 'reg_alpha': 1.3671801232963454, 'reg_lambda': 8.713421025888707, 'min_split_gain': 0.03280199988609795}. Best is trial 0 with value: 0.011731260773818903.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1629
1           Price_lag_2        1339
14      pct_chg_Load_DA        1185
3         Load_DA_lag_0        1005
5       WindOn_DA_lag_0         973
15       lag168_Load_DA         788
16  roll2d_median_Price         696
11           NGas_lag_2         694
2           Price_lag_7         693
12            Oil_lag_2         601
13            EUA_lag_2         538
10           Coal_lag_2         464
9                  WD_7          54
7                  WD_1          29
8                  WD_6          28
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          90
16  roll2d_median_Price          29
1           Price_lag_2          17
5       WindOn_DA_lag_0          12
15       lag168_Load_DA           8
2           Price_lag_7           4
3         Load_DA_lag_0           3
9                  WD_7           2
10           Coal_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 66.62% done
********************* END NS.. 486 ****************************************************
********************* START NS ... 487 ****************************************************
[I 2025-06-15 20:28:48,913] A new study created in memory with name: no-name-1928111d-0e7a-4274-9e71-990ebce65bae
[I 2025-06-15 20:28:50,320] Trial 0 finished with value: 0.015375811096818523 and parameters: {'n_estimators': 221, 'learning_rate': 0.010402632150890602, 'num_leaves': 219, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.6073730743242177, 'colsample_bytree': 0.9227286787347383, 'reg_alpha': 9.962215869117298, 'reg_lambda': 4.521300922905493, 'min_split_gain': 0.0477884059203144}. Best is trial 0 with value: 0.015375811096818523.
[Trial 0] New best: 0.01538
[I 2025-06-15 20:28:51,095] Trial 1 finished with value: 0.007651463717541672 and parameters: {'n_estimators': 84, 'learning_rate': 0.1018929681001002, 'num_leaves': 108, 'max_depth': 8, 'min_child_samples': 3, 'subsample': 0.5564050501974889, 'colsample_bytree': 0.5822949754448552, 'reg_alpha': 7.599029521828405, 'reg_lambda': 0.7494076730853616, 'min_split_gain': 0.02802345169545231}. Best is trial 1 with value: 0.007651463717541672.
[Trial 1] New best: 0.00765
[I 2025-06-15 20:28:53,431] Trial 2 finished with value: 0.011373469352002083 and parameters: {'n_estimators': 232, 'learning_rate': 0.014836674774355732, 'num_leaves': 219, 'max_depth': 8, 'min_child_samples': 3, 'subsample': 0.924676072321421, 'colsample_bytree': 0.5485079772672956, 'reg_alpha': 6.927280197417957, 'reg_lambda': 5.552398160489674, 'min_split_gain': 0.03565205606392429}. Best is trial 1 with value: 0.007651463717541672.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:28:54,346] Trial 3 finished with value: 0.006655426309007023 and parameters: {'n_estimators': 139, 'learning_rate': 0.03032211922189669, 'num_leaves': 208, 'max_depth': 6, 'min_child_samples': 30, 'subsample': 0.6727192691197451, 'colsample_bytree': 0.925333895818563, 'reg_alpha': 5.617112601330289, 'reg_lambda': 5.201504904272241, 'min_split_gain': 0.009572281433498775}. Best is trial 3 with value: 0.006655426309007023.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:28:54,950] Trial 4 finished with value: 0.0057588341187467884 and parameters: {'n_estimators': 148, 'learning_rate': 0.1642177210815279, 'num_leaves': 59, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.8180769810987907, 'colsample_bytree': 0.8249950780046903, 'reg_alpha': 1.7806900305382511, 'reg_lambda': 8.04867716992806, 'min_split_gain': 0.04572450152600484}. Best is trial 4 with value: 0.0057588341187467884.
[Trial 4] New best: 0.00576
[I 2025-06-15 20:28:58,110] Trial 5 finished with value: 0.009183138458967566 and parameters: {'n_estimators': 257, 'learning_rate': 0.015889257003763233, 'num_leaves': 161, 'max_depth': 11, 'min_child_samples': 12, 'subsample': 0.8069294548857033, 'colsample_bytree': 0.5658412172273959, 'reg_alpha': 4.51468138429584, 'reg_lambda': 8.84377326611724, 'min_split_gain': 0.0326558365366191}. Best is trial 4 with value: 0.0057588341187467884.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:28:58,902] Trial 6 finished with value: 0.00634845488304249 and parameters: {'n_estimators': 176, 'learning_rate': 0.07173441469077343, 'num_leaves': 202, 'max_depth': 6, 'min_child_samples': 3, 'subsample': 0.6048522592912362, 'colsample_bytree': 0.7861954968634269, 'reg_alpha': 6.897749638022073, 'reg_lambda': 3.4674620155134948, 'min_split_gain': 0.0381171563231336}. Best is trial 4 with value: 0.0057588341187467884.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:28:59,173] Trial 7 finished with value: 0.006342484963148325 and parameters: {'n_estimators': 63, 'learning_rate': 0.23452643852248026, 'num_leaves': 139, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.8733365021042208, 'colsample_bytree': 0.6632933663010279, 'reg_alpha': 6.9633651236883285, 'reg_lambda': 6.64927190166975, 'min_split_gain': 0.03146872824793333}. Best is trial 4 with value: 0.0057588341187467884.
[I 2025-06-15 20:28:59,175] A new study created in memory with name: no-name-35aeb1e0-3598-4a7b-9ebf-71fcbe9b2ef7
[I 2025-06-15 20:28:59,253] Trial 0 finished with value: 0.07072168001247024 and parameters: {'n_estimators': 113, 'learning_rate': 0.011257356462285413, 'num_leaves': 242, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.8903501675395697, 'colsample_bytree': 0.9875913454606916, 'reg_alpha': 3.4597075939330155, 'reg_lambda': 1.6757177148898605, 'min_split_gain': 0.042867448571681656}. Best is trial 0 with value: 0.07072168001247024.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         406
1           Price_lag_2         363
14      pct_chg_Load_DA         306
3         Load_DA_lag_0         170
15       lag168_Load_DA         164
2           Price_lag_7         154
16  roll2d_median_Price         145
11           NGas_lag_2         144
5       WindOn_DA_lag_0         134
10           Coal_lag_2          82
12            Oil_lag_2          59
13            EUA_lag_2          52
7                  WD_1          10
9                  WD_7           8
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         281
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 66.76% done
********************* END NS.. 487 ****************************************************
********************* START NS ... 488 ****************************************************
[I 2025-06-15 20:29:01,281] A new study created in memory with name: no-name-f24a60cb-9646-499f-8fb1-1755ae76a6f9
[I 2025-06-15 20:29:02,922] Trial 0 finished with value: 0.010873658868318323 and parameters: {'n_estimators': 287, 'learning_rate': 0.0151664411521192, 'num_leaves': 159, 'max_depth': 5, 'min_child_samples': 25, 'subsample': 0.6545952461046871, 'colsample_bytree': 0.5335684301614481, 'reg_alpha': 6.535076759717717, 'reg_lambda': 6.198324614909795, 'min_split_gain': 0.030903573521530333}. Best is trial 0 with value: 0.010873658868318323.
[Trial 0] New best: 0.01087
[I 2025-06-15 20:29:05,479] Trial 1 finished with value: 0.01271093533273434 and parameters: {'n_estimators': 164, 'learning_rate': 0.01924205336792283, 'num_leaves': 58, 'max_depth': 13, 'min_child_samples': 7, 'subsample': 0.708341410642693, 'colsample_bytree': 0.5176158469265921, 'reg_alpha': 1.4645511183187676, 'reg_lambda': 1.6174495931601274, 'min_split_gain': 0.01595938558962853}. Best is trial 0 with value: 0.010873658868318323.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:29:07,752] Trial 2 finished with value: 0.018122415591850217 and parameters: {'n_estimators': 220, 'learning_rate': 0.010364841319428181, 'num_leaves': 232, 'max_depth': 8, 'min_child_samples': 24, 'subsample': 0.799621387775024, 'colsample_bytree': 0.9621771029724737, 'reg_alpha': 3.4739903349163246, 'reg_lambda': 2.7950499228573977, 'min_split_gain': 0.017788627523032462}. Best is trial 0 with value: 0.010873658868318323.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:29:09,963] Trial 3 finished with value: 0.005544629767411539 and parameters: {'n_estimators': 228, 'learning_rate': 0.09273069445184273, 'num_leaves': 157, 'max_depth': 14, 'min_child_samples': 13, 'subsample': 0.568157568895684, 'colsample_bytree': 0.6450663307900559, 'reg_alpha': 0.2764343166752281, 'reg_lambda': 9.09794767434253, 'min_split_gain': 0.018376288117147494}. Best is trial 3 with value: 0.005544629767411539.
[Trial 3] New best: 0.00554
[I 2025-06-15 20:29:12,291] Trial 4 finished with value: 0.08280240053331184 and parameters: {'n_estimators': 110, 'learning_rate': 0.011892138527884793, 'num_leaves': 139, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.6311890419621462, 'colsample_bytree': 0.6374670946656835, 'reg_alpha': 1.4445821113408008, 'reg_lambda': 0.3508306948959572, 'min_split_gain': 0.01901961554722515}. Best is trial 3 with value: 0.005544629767411539.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:29:14,129] Trial 5 finished with value: 0.008982000567097635 and parameters: {'n_estimators': 255, 'learning_rate': 0.025132471978640165, 'num_leaves': 35, 'max_depth': 11, 'min_child_samples': 23, 'subsample': 0.739278000902531, 'colsample_bytree': 0.6087990302660309, 'reg_alpha': 7.625176688779652, 'reg_lambda': 8.329190045159844, 'min_split_gain': 0.030709965639574746}. Best is trial 3 with value: 0.005544629767411539.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:29:14,903] Trial 6 finished with value: 0.00699199650030737 and parameters: {'n_estimators': 162, 'learning_rate': 0.12304512218263446, 'num_leaves': 157, 'max_depth': 8, 'min_child_samples': 20, 'subsample': 0.915980917222244, 'colsample_bytree': 0.5830761267347995, 'reg_alpha': 8.845450727969688, 'reg_lambda': 0.7519257912921073, 'min_split_gain': 0.015076738588361789}. Best is trial 3 with value: 0.005544629767411539.
[I 2025-06-15 20:29:14,906] A new study created in memory with name: no-name-c1fb86cc-9227-4b85-aaa5-257827233fb0
[I 2025-06-15 20:29:14,954] Trial 0 finished with value: 0.16102139926894557 and parameters: {'n_estimators': 189, 'learning_rate': 0.15115860965043554, 'num_leaves': 54, 'max_depth': 9, 'min_child_samples': 27, 'subsample': 0.8573562941055228, 'colsample_bytree': 0.6928190554002479, 'reg_alpha': 9.161459203636618, 'reg_lambda': 8.621028351774521, 'min_split_gain': 0.04867200499236075}. Best is trial 0 with value: 0.16102139926894557.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1443
14      pct_chg_Load_DA        1306
0           Price_lag_1        1295
3         Load_DA_lag_0         974
5       WindOn_DA_lag_0         943
15       lag168_Load_DA         881
2           Price_lag_7         799
16  roll2d_median_Price         770
11           NGas_lag_2         653
10           Coal_lag_2         560
13            EUA_lag_2         496
12            Oil_lag_2         391
9                  WD_7          37
7                  WD_1          36
8                  WD_6          19
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          14
1           Price_lag_2           2
16  roll2d_median_Price           1
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
2           Price_lag_7           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 66.89% done
********************* END NS.. 488 ****************************************************
********************* START NS ... 489 ****************************************************
[I 2025-06-15 20:29:18,680] A new study created in memory with name: no-name-9fb0d6e5-6968-45c5-b8da-6b701f7d63b7
[I 2025-06-15 20:29:20,480] Trial 0 finished with value: 0.0027794170677970568 and parameters: {'n_estimators': 124, 'learning_rate': 0.057048297512574765, 'num_leaves': 225, 'max_depth': 8, 'min_child_samples': 7, 'subsample': 0.7234991210854173, 'colsample_bytree': 0.9849231015766, 'reg_alpha': 3.428803868907412, 'reg_lambda': 7.3028155824634435, 'min_split_gain': 0.014992733036173017}. Best is trial 0 with value: 0.0027794170677970568.
[Trial 0] New best: 0.00278
[I 2025-06-15 20:29:22,068] Trial 1 finished with value: 0.005836863930230689 and parameters: {'n_estimators': 156, 'learning_rate': 0.018643061247185816, 'num_leaves': 207, 'max_depth': 9, 'min_child_samples': 28, 'subsample': 0.6501895210747735, 'colsample_bytree': 0.9803135906194794, 'reg_alpha': 4.327982384191794, 'reg_lambda': 0.18904757968130292, 'min_split_gain': 0.0017910447353109205}. Best is trial 0 with value: 0.0027794170677970568.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:29:22,947] Trial 2 finished with value: 0.0030707010047731193 and parameters: {'n_estimators': 92, 'learning_rate': 0.06772236340930728, 'num_leaves': 236, 'max_depth': 16, 'min_child_samples': 30, 'subsample': 0.8868619048060578, 'colsample_bytree': 0.9366990670195459, 'reg_alpha': 4.818787901628841, 'reg_lambda': 6.892957458145079, 'min_split_gain': 0.04305618947815184}. Best is trial 0 with value: 0.0027794170677970568.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:29:24,170] Trial 3 finished with value: 0.0025881265938768046 and parameters: {'n_estimators': 152, 'learning_rate': 0.09370964445081068, 'num_leaves': 247, 'max_depth': 10, 'min_child_samples': 12, 'subsample': 0.69104393155371, 'colsample_bytree': 0.9146451414559627, 'reg_alpha': 0.6080166284786137, 'reg_lambda': 0.8556555803405264, 'min_split_gain': 0.04159857477964447}. Best is trial 3 with value: 0.0025881265938768046.
[I 2025-06-15 20:29:24,174] A new study created in memory with name: no-name-bfa19194-aaab-4f9c-9501-63157d702c35
[I 2025-06-15 20:29:24,225] Trial 0 finished with value: 0.011579965191154193 and parameters: {'n_estimators': 124, 'learning_rate': 0.09728512181594912, 'num_leaves': 201, 'max_depth': 15, 'min_child_samples': 5, 'subsample': 0.612465575195821, 'colsample_bytree': 0.9562156227321668, 'reg_alpha': 2.207805679030048, 'reg_lambda': 5.739053479330437, 'min_split_gain': 0.03128458415369064}. Best is trial 0 with value: 0.011579965191154193.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         973
1           Price_lag_2         922
14      pct_chg_Load_DA         880
3         Load_DA_lag_0         553
16  roll2d_median_Price         489
2           Price_lag_7         466
11           NGas_lag_2         417
5       WindOn_DA_lag_0         408
15       lag168_Load_DA         341
10           Coal_lag_2         200
13            EUA_lag_2         167
12            Oil_lag_2         122
9                  WD_7          19
7                  WD_1           9
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          58
12            Oil_lag_2           5
3         Load_DA_lag_0           1
5       WindOn_DA_lag_0           1
1           Price_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 67.03% done
********************* END NS.. 489 ****************************************************
********************* START NS ... 490 ****************************************************
[I 2025-06-15 20:29:26,800] A new study created in memory with name: no-name-109c3fdb-50ba-4616-861a-1628e52eadd5
[I 2025-06-15 20:29:27,640] Trial 0 finished with value: 0.0023596050077014283 and parameters: {'n_estimators': 232, 'learning_rate': 0.03375401602087454, 'num_leaves': 124, 'max_depth': 4, 'min_child_samples': 7, 'subsample': 0.7992502907237506, 'colsample_bytree': 0.5946118750132792, 'reg_alpha': 4.658469409566527, 'reg_lambda': 6.426955322831464, 'min_split_gain': 0.03651269945019863}. Best is trial 0 with value: 0.0023596050077014283.
[Trial 0] New best: 0.00236
[I 2025-06-15 20:29:28,359] Trial 1 finished with value: 0.014012727855994958 and parameters: {'n_estimators': 84, 'learning_rate': 0.024718939454326013, 'num_leaves': 240, 'max_depth': 16, 'min_child_samples': 20, 'subsample': 0.8363565627948725, 'colsample_bytree': 0.6330537536261946, 'reg_alpha': 7.321874099693004, 'reg_lambda': 2.106695357481465, 'min_split_gain': 0.027415241446055186}. Best is trial 0 with value: 0.0023596050077014283.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:29:28,829] Trial 2 finished with value: 0.054426662107141976 and parameters: {'n_estimators': 102, 'learning_rate': 0.013104706766568974, 'num_leaves': 245, 'max_depth': 5, 'min_child_samples': 13, 'subsample': 0.5883611881561152, 'colsample_bytree': 0.6076031982452101, 'reg_alpha': 9.23363699569678, 'reg_lambda': 6.202632623354273, 'min_split_gain': 0.041958427606918064}. Best is trial 0 with value: 0.0023596050077014283.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:29:29,803] Trial 3 finished with value: 0.0019224433070667822 and parameters: {'n_estimators': 120, 'learning_rate': 0.25968025376976606, 'num_leaves': 163, 'max_depth': 7, 'min_child_samples': 21, 'subsample': 0.6800018006693808, 'colsample_bytree': 0.8495059634134297, 'reg_alpha': 0.04711951103477441, 'reg_lambda': 0.672137455813353, 'min_split_gain': 0.0039901921087485024}. Best is trial 3 with value: 0.0019224433070667822.
[I 2025-06-15 20:29:29,805] A new study created in memory with name: no-name-bc36cbbe-f387-4c9d-893b-3550b8ad6b24
[I 2025-06-15 20:29:29,882] Trial 0 finished with value: 0.0028011181109086664 and parameters: {'n_estimators': 260, 'learning_rate': 0.09058636445707179, 'num_leaves': 88, 'max_depth': 9, 'min_child_samples': 16, 'subsample': 0.7359026337795691, 'colsample_bytree': 0.6653934894122024, 'reg_alpha': 4.202991545677235, 'reg_lambda': 8.83103180825066, 'min_split_gain': 0.016977101680463753}. Best is trial 0 with value: 0.0028011181109086664.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         665
14      pct_chg_Load_DA         662
1           Price_lag_2         609
5       WindOn_DA_lag_0         410
2           Price_lag_7         389
3         Load_DA_lag_0         379
16  roll2d_median_Price         359
15       lag168_Load_DA         355
11           NGas_lag_2         318
10           Coal_lag_2         232
12            Oil_lag_2         219
13            EUA_lag_2         209
9                  WD_7          14
7                  WD_1           7
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          50
1           Price_lag_2           5
3         Load_DA_lag_0           2
12            Oil_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 67.17% done
********************* END NS.. 490 ****************************************************
********************* START NS ... 491 ****************************************************
[I 2025-06-15 20:29:32,472] A new study created in memory with name: no-name-01eb60c5-df45-4f61-b1d9-5f7befb66ac0
[I 2025-06-15 20:29:35,027] Trial 0 finished with value: 0.004124742275118966 and parameters: {'n_estimators': 210, 'learning_rate': 0.01979917688690922, 'num_leaves': 72, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.7393032755547315, 'colsample_bytree': 0.8567419078104641, 'reg_alpha': 5.7898293552868205, 'reg_lambda': 5.519907798383952, 'min_split_gain': 0.018409484767509633}. Best is trial 0 with value: 0.004124742275118966.
[Trial 0] New best: 0.00412
[I 2025-06-15 20:29:35,572] Trial 1 finished with value: 0.0035704165687416496 and parameters: {'n_estimators': 262, 'learning_rate': 0.2393475332361047, 'num_leaves': 157, 'max_depth': 7, 'min_child_samples': 6, 'subsample': 0.5377209027284735, 'colsample_bytree': 0.9267433591438174, 'reg_alpha': 6.1136856152778645, 'reg_lambda': 3.1175945345155642, 'min_split_gain': 0.007890073352271139}. Best is trial 1 with value: 0.0035704165687416496.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:29:35,980] Trial 2 finished with value: 0.003650347454434221 and parameters: {'n_estimators': 178, 'learning_rate': 0.26329196052268095, 'num_leaves': 73, 'max_depth': 13, 'min_child_samples': 17, 'subsample': 0.5012137028025682, 'colsample_bytree': 0.9596560844742117, 'reg_alpha': 4.2614809920104, 'reg_lambda': 0.7687661317027161, 'min_split_gain': 0.0252357119946168}. Best is trial 1 with value: 0.0035704165687416496.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:29:38,424] Trial 3 finished with value: 0.0036814722663136897 and parameters: {'n_estimators': 227, 'learning_rate': 0.08598597505243703, 'num_leaves': 143, 'max_depth': 16, 'min_child_samples': 12, 'subsample': 0.9025766133480646, 'colsample_bytree': 0.8960733091294003, 'reg_alpha': 4.999575715200364, 'reg_lambda': 4.81820467893254, 'min_split_gain': 0.0017081119341923968}. Best is trial 1 with value: 0.0035704165687416496.
[I 2025-06-15 20:29:38,427] A new study created in memory with name: no-name-b23041b5-d5bb-4036-b532-b7f1def403d1
[I 2025-06-15 20:29:38,483] Trial 0 finished with value: 0.02735634308066201 and parameters: {'n_estimators': 191, 'learning_rate': 0.27097143022928966, 'num_leaves': 54, 'max_depth': 12, 'min_child_samples': 3, 'subsample': 0.7935573523209876, 'colsample_bytree': 0.6090244585732086, 'reg_alpha': 8.864904491435304, 'reg_lambda': 1.0229108660159802, 'min_split_gain': 0.04154936577227057}. Best is trial 0 with value: 0.02735634308066201.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         286
1           Price_lag_2         206
14      pct_chg_Load_DA         184
2           Price_lag_7         122
3         Load_DA_lag_0         122
5       WindOn_DA_lag_0         102
16  roll2d_median_Price         100
11           NGas_lag_2          94
15       lag168_Load_DA          92
10           Coal_lag_2          76
13            EUA_lag_2          59
12            Oil_lag_2          51
7                  WD_1           7
9                  WD_7           6
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           3
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 67.31% done
********************* END NS.. 491 ****************************************************
********************* START NS ... 492 ****************************************************
[I 2025-06-15 20:29:40,538] A new study created in memory with name: no-name-b16a43d6-7c3e-45ff-909c-847f146b184c
[I 2025-06-15 20:29:41,119] Trial 0 finished with value: 0.00490744204308246 and parameters: {'n_estimators': 232, 'learning_rate': 0.06470512328157739, 'num_leaves': 85, 'max_depth': 4, 'min_child_samples': 8, 'subsample': 0.6970144767224578, 'colsample_bytree': 0.8631669979500045, 'reg_alpha': 5.064207487331029, 'reg_lambda': 0.5167336096868425, 'min_split_gain': 0.03254773894838722}. Best is trial 0 with value: 0.00490744204308246.
[Trial 0] New best: 0.00491
[I 2025-06-15 20:29:42,094] Trial 1 finished with value: 0.004301394510234625 and parameters: {'n_estimators': 247, 'learning_rate': 0.06328553330353703, 'num_leaves': 136, 'max_depth': 5, 'min_child_samples': 23, 'subsample': 0.7612563093053595, 'colsample_bytree': 0.8624026782749941, 'reg_alpha': 1.5597703948426334, 'reg_lambda': 5.4992846868358995, 'min_split_gain': 0.012562592066476103}. Best is trial 1 with value: 0.004301394510234625.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:29:42,572] Trial 2 finished with value: 0.005335303989773779 and parameters: {'n_estimators': 232, 'learning_rate': 0.25586340553535203, 'num_leaves': 142, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.7673401465780185, 'colsample_bytree': 0.5228479887106827, 'reg_alpha': 2.573810407478241, 'reg_lambda': 1.1853023797623707, 'min_split_gain': 0.0369522679605183}. Best is trial 1 with value: 0.004301394510234625.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:29:43,624] Trial 3 finished with value: 0.005835812357608773 and parameters: {'n_estimators': 113, 'learning_rate': 0.04963151006554141, 'num_leaves': 43, 'max_depth': 7, 'min_child_samples': 23, 'subsample': 0.7724289666767724, 'colsample_bytree': 0.5449810954839337, 'reg_alpha': 0.6534004558165241, 'reg_lambda': 0.33782908511067355, 'min_split_gain': 0.02781763642744184}. Best is trial 1 with value: 0.004301394510234625.
[I 2025-06-15 20:29:43,626] A new study created in memory with name: no-name-a4872964-c9ab-4167-a904-f3c0e6df2b81
[I 2025-06-15 20:29:43,777] Trial 0 finished with value: 0.01194211809364113 and parameters: {'n_estimators': 284, 'learning_rate': 0.013679735043504945, 'num_leaves': 122, 'max_depth': 13, 'min_child_samples': 7, 'subsample': 0.9563129651893961, 'colsample_bytree': 0.8994124871595129, 'reg_alpha': 0.3592524278997733, 'reg_lambda': 7.413378599663535, 'min_split_gain': 0.049357545166926794}. Best is trial 0 with value: 0.01194211809364113.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1112
1           Price_lag_2         725
14      pct_chg_Load_DA         618
3         Load_DA_lag_0         543
11           NGas_lag_2         357
5       WindOn_DA_lag_0         313
16  roll2d_median_Price         281
15       lag168_Load_DA         272
2           Price_lag_7         270
10           Coal_lag_2         268
13            EUA_lag_2         208
12            Oil_lag_2          77
7                  WD_1          25
9                  WD_7          20
8                  WD_6           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         370
12            Oil_lag_2          30
3         Load_DA_lag_0          29
1           Price_lag_2          16
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 67.44% done
********************* END NS.. 492 ****************************************************
********************* START NS ... 493 ****************************************************
[I 2025-06-15 20:29:46,577] A new study created in memory with name: no-name-6c7c5582-c32a-4c63-97d8-9fe79d02f46a
[I 2025-06-15 20:29:47,812] Trial 0 finished with value: 0.013053823631180156 and parameters: {'n_estimators': 167, 'learning_rate': 0.04726132082132248, 'num_leaves': 73, 'max_depth': 8, 'min_child_samples': 28, 'subsample': 0.8285908567603651, 'colsample_bytree': 0.725062004512557, 'reg_alpha': 5.196934992037402, 'reg_lambda': 4.839917511359833, 'min_split_gain': 0.021633336238335167}. Best is trial 0 with value: 0.013053823631180156.
[Trial 0] New best: 0.01305
[I 2025-06-15 20:29:49,980] Trial 1 finished with value: 0.016422598263889327 and parameters: {'n_estimators': 180, 'learning_rate': 0.015494181198125432, 'num_leaves': 113, 'max_depth': 14, 'min_child_samples': 12, 'subsample': 0.6653572306760716, 'colsample_bytree': 0.7354422710927964, 'reg_alpha': 8.065120918245015, 'reg_lambda': 2.409201493941169, 'min_split_gain': 0.010254141957781883}. Best is trial 0 with value: 0.013053823631180156.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:29:50,522] Trial 2 finished with value: 0.013781021516560613 and parameters: {'n_estimators': 172, 'learning_rate': 0.0503098827354893, 'num_leaves': 250, 'max_depth': 4, 'min_child_samples': 24, 'subsample': 0.6332143921418987, 'colsample_bytree': 0.908244352318122, 'reg_alpha': 9.980285519335354, 'reg_lambda': 7.186528628747276, 'min_split_gain': 0.009839488198212966}. Best is trial 0 with value: 0.013053823631180156.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:29:52,465] Trial 3 finished with value: 0.011539538922056461 and parameters: {'n_estimators': 243, 'learning_rate': 0.07214764129158587, 'num_leaves': 174, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.602397344594739, 'colsample_bytree': 0.9078210072802466, 'reg_alpha': 1.6300293593754933, 'reg_lambda': 7.741634766060291, 'min_split_gain': 0.022815607512408832}. Best is trial 3 with value: 0.011539538922056461.
[Trial 3] New best: 0.01154
[I 2025-06-15 20:29:52,975] Trial 4 finished with value: 0.013659063096909863 and parameters: {'n_estimators': 116, 'learning_rate': 0.18483382236678095, 'num_leaves': 194, 'max_depth': 8, 'min_child_samples': 20, 'subsample': 0.8567529735133104, 'colsample_bytree': 0.8076769856618523, 'reg_alpha': 7.471079676997011, 'reg_lambda': 0.8523352434959475, 'min_split_gain': 0.040894016608737774}. Best is trial 3 with value: 0.011539538922056461.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:29:53,870] Trial 5 finished with value: 0.011965918868356943 and parameters: {'n_estimators': 253, 'learning_rate': 0.08627715919503895, 'num_leaves': 217, 'max_depth': 8, 'min_child_samples': 3, 'subsample': 0.694518778005647, 'colsample_bytree': 0.9894904673883844, 'reg_alpha': 3.399634728570339, 'reg_lambda': 1.3646042172469985, 'min_split_gain': 0.045124625721003946}. Best is trial 3 with value: 0.011539538922056461.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:29:54,481] Trial 6 finished with value: 0.04160239098311919 and parameters: {'n_estimators': 98, 'learning_rate': 0.012706460062062276, 'num_leaves': 167, 'max_depth': 6, 'min_child_samples': 23, 'subsample': 0.9377096560432583, 'colsample_bytree': 0.9448678730480209, 'reg_alpha': 4.527120983988125, 'reg_lambda': 6.387102896476681, 'min_split_gain': 0.004879447176839902}. Best is trial 3 with value: 0.011539538922056461.
[I 2025-06-15 20:29:54,484] A new study created in memory with name: no-name-f46941e0-e01b-459a-92c8-c5c82a180ccd
[I 2025-06-15 20:29:54,507] Trial 0 finished with value: 0.1626953259793452 and parameters: {'n_estimators': 51, 'learning_rate': 0.09432662859167244, 'num_leaves': 35, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.7624226732840247, 'colsample_bytree': 0.9176596343195822, 'reg_alpha': 6.705995456143121, 'reg_lambda': 7.132987069829307, 'min_split_gain': 0.008130458856424938}. Best is trial 0 with value: 0.1626953259793452.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1245
14      pct_chg_Load_DA        1170
1           Price_lag_2         997
3         Load_DA_lag_0         748
5       WindOn_DA_lag_0         581
16  roll2d_median_Price         534
11           NGas_lag_2         513
2           Price_lag_7         490
15       lag168_Load_DA         462
13            EUA_lag_2         259
10           Coal_lag_2         215
12            Oil_lag_2         188
9                  WD_7          26
7                  WD_1          19
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          40
10           Coal_lag_2          13
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 67.58% done
********************* END NS.. 493 ****************************************************
********************* START NS ... 494 ****************************************************
[I 2025-06-15 20:29:57,518] A new study created in memory with name: no-name-ff2d13a2-371c-4828-9eff-5bfb45c6aaf2
[I 2025-06-15 20:29:58,082] Trial 0 finished with value: 0.025673753013622746 and parameters: {'n_estimators': 67, 'learning_rate': 0.02809256519585804, 'num_leaves': 240, 'max_depth': 16, 'min_child_samples': 26, 'subsample': 0.7838365777273633, 'colsample_bytree': 0.5728935672297725, 'reg_alpha': 7.105402209991095, 'reg_lambda': 6.640967882971901, 'min_split_gain': 0.040039167266274975}. Best is trial 0 with value: 0.025673753013622746.
[Trial 0] New best: 0.02567
[I 2025-06-15 20:29:59,521] Trial 1 finished with value: 0.017381631991131612 and parameters: {'n_estimators': 231, 'learning_rate': 0.06618459664993653, 'num_leaves': 218, 'max_depth': 11, 'min_child_samples': 17, 'subsample': 0.8922433154923144, 'colsample_bytree': 0.6881071215744675, 'reg_alpha': 4.803186273197327, 'reg_lambda': 8.884434109189728, 'min_split_gain': 0.012765183945377585}. Best is trial 1 with value: 0.017381631991131612.
[Trial 1] New best: 0.01738
[I 2025-06-15 20:30:01,285] Trial 2 finished with value: 0.021221125907155452 and parameters: {'n_estimators': 224, 'learning_rate': 0.035467273088441796, 'num_leaves': 254, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.843151785976358, 'colsample_bytree': 0.5356054499879827, 'reg_alpha': 8.902521821050435, 'reg_lambda': 1.0406217977921184, 'min_split_gain': 0.018490916452714185}. Best is trial 1 with value: 0.017381631991131612.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:30:03,381] Trial 3 finished with value: 0.01765243045960747 and parameters: {'n_estimators': 242, 'learning_rate': 0.0404722553717789, 'num_leaves': 39, 'max_depth': 15, 'min_child_samples': 19, 'subsample': 0.8574405491665011, 'colsample_bytree': 0.9073148841709398, 'reg_alpha': 0.7313253605348347, 'reg_lambda': 9.983456193464779, 'min_split_gain': 0.045161953302828975}. Best is trial 1 with value: 0.017381631991131612.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:30:03,930] Trial 4 finished with value: 0.01726655177942735 and parameters: {'n_estimators': 148, 'learning_rate': 0.16297208209789074, 'num_leaves': 69, 'max_depth': 8, 'min_child_samples': 9, 'subsample': 0.5646114080182065, 'colsample_bytree': 0.8936300940693982, 'reg_alpha': 7.143061993591968, 'reg_lambda': 1.221476841836483, 'min_split_gain': 0.008884214784932638}. Best is trial 4 with value: 0.01726655177942735.
[I 2025-06-15 20:30:03,934] A new study created in memory with name: no-name-7d923747-3f73-43ff-b57a-f1b6204ea37c
[I 2025-06-15 20:30:04,061] Trial 0 finished with value: 0.12797938692319216 and parameters: {'n_estimators': 207, 'learning_rate': 0.02704437822420668, 'num_leaves': 89, 'max_depth': 14, 'min_child_samples': 6, 'subsample': 0.6830277711429122, 'colsample_bytree': 0.5547833529537729, 'reg_alpha': 0.3528391419202792, 'reg_lambda': 6.607471120473201, 'min_split_gain': 0.031109546586605325}. Best is trial 0 with value: 0.12797938692319216.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         389
1           Price_lag_2         294
14      pct_chg_Load_DA         275
3         Load_DA_lag_0         184
2           Price_lag_7         150
11           NGas_lag_2         147
16  roll2d_median_Price         141
5       WindOn_DA_lag_0         131
15       lag168_Load_DA         131
13            EUA_lag_2         106
10           Coal_lag_2          83
12            Oil_lag_2          55
9                  WD_7          13
7                  WD_1           7
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         220
14      pct_chg_Load_DA          69
2           Price_lag_7          52
3         Load_DA_lag_0          49
1           Price_lag_2          43
16  roll2d_median_Price          19
10           Coal_lag_2          17
11           NGas_lag_2          12
13            EUA_lag_2          11
12            Oil_lag_2           8
15       lag168_Load_DA           4
5       WindOn_DA_lag_0           2
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 67.72% done
********************* END NS.. 494 ****************************************************
********************* START NS ... 495 ****************************************************
[I 2025-06-15 20:30:06,375] A new study created in memory with name: no-name-7d276163-056a-49d2-90f1-27c890f8c1e7
[I 2025-06-15 20:30:06,782] Trial 0 finished with value: 0.01269501298954634 and parameters: {'n_estimators': 114, 'learning_rate': 0.07196737034622142, 'num_leaves': 169, 'max_depth': 4, 'min_child_samples': 14, 'subsample': 0.6677429537732416, 'colsample_bytree': 0.9318806230288037, 'reg_alpha': 7.688667465282335, 'reg_lambda': 7.915526707761983, 'min_split_gain': 0.026435047046833027}. Best is trial 0 with value: 0.01269501298954634.
[Trial 0] New best: 0.01270
[I 2025-06-15 20:30:07,306] Trial 1 finished with value: 0.011592506640827139 and parameters: {'n_estimators': 108, 'learning_rate': 0.15137470055466873, 'num_leaves': 144, 'max_depth': 16, 'min_child_samples': 23, 'subsample': 0.5544117120566481, 'colsample_bytree': 0.8956588224697629, 'reg_alpha': 5.792146673216881, 'reg_lambda': 8.828615652550406, 'min_split_gain': 0.011396695168352705}. Best is trial 1 with value: 0.011592506640827139.
[Trial 1] New best: 0.01159
[I 2025-06-15 20:30:07,966] Trial 2 finished with value: 0.012947103987791114 and parameters: {'n_estimators': 83, 'learning_rate': 0.11645082074909761, 'num_leaves': 237, 'max_depth': 15, 'min_child_samples': 5, 'subsample': 0.6192450133173837, 'colsample_bytree': 0.7182472457832982, 'reg_alpha': 8.427995590798986, 'reg_lambda': 0.8150562818154283, 'min_split_gain': 0.010193908223348131}. Best is trial 1 with value: 0.011592506640827139.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:30:08,914] Trial 3 finished with value: 0.010960695265012919 and parameters: {'n_estimators': 216, 'learning_rate': 0.15746576596770423, 'num_leaves': 235, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.6217543276790121, 'colsample_bytree': 0.6866422216360095, 'reg_alpha': 0.8891452864750793, 'reg_lambda': 7.704060212280145, 'min_split_gain': 0.026840825174179513}. Best is trial 3 with value: 0.010960695265012919.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:30:09,343] Trial 4 finished with value: 0.012726469494933237 and parameters: {'n_estimators': 84, 'learning_rate': 0.24548705993064301, 'num_leaves': 223, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.8057534890474729, 'colsample_bytree': 0.6743421498356226, 'reg_alpha': 7.538410776833578, 'reg_lambda': 4.873705310828965, 'min_split_gain': 0.009232604009764356}. Best is trial 3 with value: 0.010960695265012919.
[I 2025-06-15 20:30:09,346] A new study created in memory with name: no-name-267c79c4-9d2b-4c13-aef2-c8a5c882d16b
[I 2025-06-15 20:30:09,419] Trial 0 finished with value: 0.07870719037389277 and parameters: {'n_estimators': 253, 'learning_rate': 0.050219001007237946, 'num_leaves': 75, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.8170979811021003, 'colsample_bytree': 0.8373282168061313, 'reg_alpha': 6.176424381141309, 'reg_lambda': 1.734490802648193, 'min_split_gain': 0.0022602137349262753}. Best is trial 0 with value: 0.07870719037389277.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         590
1           Price_lag_2         561
14      pct_chg_Load_DA         419
3         Load_DA_lag_0         390
5       WindOn_DA_lag_0         351
15       lag168_Load_DA         342
16  roll2d_median_Price         303
2           Price_lag_7         267
11           NGas_lag_2         254
10           Coal_lag_2         189
13            EUA_lag_2         178
12            Oil_lag_2         118
9                  WD_7          16
7                  WD_1          11
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          99
3         Load_DA_lag_0          35
13            EUA_lag_2           6
1           Price_lag_2           4
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 67.85% done
********************* END NS.. 495 ****************************************************
********************* START NS ... 496 ****************************************************
[I 2025-06-15 20:30:12,135] A new study created in memory with name: no-name-17c08742-bb11-4170-a08c-2c95427e4cdd
[I 2025-06-15 20:30:12,673] Trial 0 finished with value: 0.01908092803989318 and parameters: {'n_estimators': 88, 'learning_rate': 0.026215214532664222, 'num_leaves': 202, 'max_depth': 6, 'min_child_samples': 30, 'subsample': 0.9172952133421566, 'colsample_bytree': 0.772557899385305, 'reg_alpha': 6.873917097506659, 'reg_lambda': 1.770417864579924, 'min_split_gain': 0.04322528759778189}. Best is trial 0 with value: 0.01908092803989318.
[Trial 0] New best: 0.01908
[I 2025-06-15 20:30:15,590] Trial 1 finished with value: 0.01077436915797883 and parameters: {'n_estimators': 292, 'learning_rate': 0.01332908646032973, 'num_leaves': 109, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.9013482938800358, 'colsample_bytree': 0.8307946220127822, 'reg_alpha': 5.557569552925764, 'reg_lambda': 3.288464056076199, 'min_split_gain': 0.0050845939159270825}. Best is trial 1 with value: 0.01077436915797883.
[Trial 1] New best: 0.01077
[I 2025-06-15 20:30:16,040] Trial 2 finished with value: 0.01225236609113784 and parameters: {'n_estimators': 147, 'learning_rate': 0.1866170918369829, 'num_leaves': 109, 'max_depth': 9, 'min_child_samples': 28, 'subsample': 0.9096872100365725, 'colsample_bytree': 0.5610689001216038, 'reg_alpha': 5.711645324096804, 'reg_lambda': 1.7415570903570299, 'min_split_gain': 0.04908330051909186}. Best is trial 1 with value: 0.01077436915797883.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:30:16,472] Trial 3 finished with value: 0.009764798636306417 and parameters: {'n_estimators': 297, 'learning_rate': 0.296725161915276, 'num_leaves': 62, 'max_depth': 13, 'min_child_samples': 3, 'subsample': 0.8600880200304901, 'colsample_bytree': 0.6885494852522926, 'reg_alpha': 4.41385278773089, 'reg_lambda': 7.031949266046096, 'min_split_gain': 0.030134294512655303}. Best is trial 3 with value: 0.009764798636306417.
[Trial 3] New best: 0.00976
[I 2025-06-15 20:30:16,921] Trial 4 finished with value: 0.008955767326404693 and parameters: {'n_estimators': 111, 'learning_rate': 0.15075148329011423, 'num_leaves': 247, 'max_depth': 8, 'min_child_samples': 29, 'subsample': 0.9548479252283183, 'colsample_bytree': 0.8656400564700717, 'reg_alpha': 2.7385014901645413, 'reg_lambda': 2.8465100997890227, 'min_split_gain': 0.04374988782992437}. Best is trial 4 with value: 0.008955767326404693.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:30:18,036] Trial 5 finished with value: 0.008474758548254876 and parameters: {'n_estimators': 222, 'learning_rate': 0.1982628450545167, 'num_leaves': 94, 'max_depth': 9, 'min_child_samples': 6, 'subsample': 0.871751978305789, 'colsample_bytree': 0.7089446522730052, 'reg_alpha': 2.537389844949308, 'reg_lambda': 5.278293133739664, 'min_split_gain': 0.00474175041892207}. Best is trial 5 with value: 0.008474758548254876.
[Trial 5] New best: 0.00847
[I 2025-06-15 20:30:18,967] Trial 6 finished with value: 0.008554343604972396 and parameters: {'n_estimators': 247, 'learning_rate': 0.14381693413324664, 'num_leaves': 196, 'max_depth': 13, 'min_child_samples': 9, 'subsample': 0.7548545277171061, 'colsample_bytree': 0.961876391380631, 'reg_alpha': 8.332638944423525, 'reg_lambda': 0.2769960909871494, 'min_split_gain': 0.0026343288345591965}. Best is trial 5 with value: 0.008474758548254876.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:30:20,603] Trial 7 finished with value: 0.04990394962578457 and parameters: {'n_estimators': 127, 'learning_rate': 0.0119135675294188, 'num_leaves': 131, 'max_depth': 15, 'min_child_samples': 5, 'subsample': 0.8310236597968752, 'colsample_bytree': 0.6706480066411742, 'reg_alpha': 4.901497692508087, 'reg_lambda': 8.199453660554125, 'min_split_gain': 0.030612148198461298}. Best is trial 5 with value: 0.008474758548254876.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:30:21,401] Trial 8 finished with value: 0.0069838269522068544 and parameters: {'n_estimators': 75, 'learning_rate': 0.23392370756756586, 'num_leaves': 36, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.8226551415187036, 'colsample_bytree': 0.8756953509071217, 'reg_alpha': 0.1833070417017746, 'reg_lambda': 9.490594042889937, 'min_split_gain': 0.0011900096827385743}. Best is trial 8 with value: 0.0069838269522068544.
[I 2025-06-15 20:30:21,588] Trial 9 finished with value: 0.010628116925289706 and parameters: {'n_estimators': 51, 'learning_rate': 0.25620032234109136, 'num_leaves': 95, 'max_depth': 4, 'min_child_samples': 20, 'subsample': 0.8491078514521886, 'colsample_bytree': 0.6376948261675944, 'reg_alpha': 1.9461319874320582, 'reg_lambda': 0.7593893785072414, 'min_split_gain': 0.03115391650609301}. Best is trial 8 with value: 0.0069838269522068544.
[Trial 8] New best: 0.00698
[Trial 9] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:30:23,042] Trial 10 finished with value: 0.007740355969024541 and parameters: {'n_estimators': 187, 'learning_rate': 0.07348159943808935, 'num_leaves': 40, 'max_depth': 11, 'min_child_samples': 19, 'subsample': 0.5515021542433375, 'colsample_bytree': 0.999106618705476, 'reg_alpha': 0.3266338497578908, 'reg_lambda': 9.266678470660032, 'min_split_gain': 0.01498245247697165}. Best is trial 8 with value: 0.0069838269522068544.
[Trial 10] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:30:24,592] Trial 11 finished with value: 0.007699879297497779 and parameters: {'n_estimators': 183, 'learning_rate': 0.07185406948923839, 'num_leaves': 35, 'max_depth': 11, 'min_child_samples': 19, 'subsample': 0.529044030506219, 'colsample_bytree': 0.9944668690603444, 'reg_alpha': 0.26860532286788347, 'reg_lambda': 9.864602997045692, 'min_split_gain': 0.01535080342570036}. Best is trial 8 with value: 0.0069838269522068544.
[I 2025-06-15 20:30:24,596] A new study created in memory with name: no-name-97268b64-7ff7-4915-870b-35079d399881
[Trial 11] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 20:30:24,890] Trial 0 finished with value: 0.04100347635185508 and parameters: {'n_estimators': 246, 'learning_rate': 0.0292971037125401, 'num_leaves': 90, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.6285822500536031, 'colsample_bytree': 0.8101760780769776, 'reg_alpha': 8.744123002393598, 'reg_lambda': 9.287886942526013, 'min_split_gain': 0.013121571941158695}. Best is trial 0 with value: 0.04100347635185508.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         448
1           Price_lag_2         344
14      pct_chg_Load_DA         293
3         Load_DA_lag_0         237
5       WindOn_DA_lag_0         208
16  roll2d_median_Price         189
15       lag168_Load_DA         175
11           NGas_lag_2         163
2           Price_lag_7         163
12            Oil_lag_2         121
10           Coal_lag_2         106
13            EUA_lag_2         103
9                  WD_7          11
7                  WD_1           9
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         117
1           Price_lag_2          16
12            Oil_lag_2           5
3         Load_DA_lag_0           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 67.99% done
********************* END NS.. 496 ****************************************************
********************* START NS ... 497 ****************************************************
[I 2025-06-15 20:30:27,385] A new study created in memory with name: no-name-5b8089ca-5584-42fc-8993-fb99aff7c1e4
[I 2025-06-15 20:30:28,260] Trial 0 finished with value: 0.007070031901123736 and parameters: {'n_estimators': 217, 'learning_rate': 0.08673938287384363, 'num_leaves': 176, 'max_depth': 4, 'min_child_samples': 17, 'subsample': 0.5187621283715942, 'colsample_bytree': 0.6833608947006018, 'reg_alpha': 1.4398791888284745, 'reg_lambda': 2.364681891007466, 'min_split_gain': 0.006497227422906993}. Best is trial 0 with value: 0.007070031901123736.
[Trial 0] New best: 0.00707
[I 2025-06-15 20:30:31,942] Trial 1 finished with value: 0.010460906755049097 and parameters: {'n_estimators': 216, 'learning_rate': 0.012468748446257094, 'num_leaves': 211, 'max_depth': 12, 'min_child_samples': 23, 'subsample': 0.7292228361608868, 'colsample_bytree': 0.9743204528578995, 'reg_alpha': 1.2036736403907589, 'reg_lambda': 7.114213510106556, 'min_split_gain': 0.009472189061810899}. Best is trial 0 with value: 0.007070031901123736.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:30:32,631] Trial 2 finished with value: 0.006680435109783969 and parameters: {'n_estimators': 280, 'learning_rate': 0.15992627540186555, 'num_leaves': 34, 'max_depth': 14, 'min_child_samples': 6, 'subsample': 0.8247162030179314, 'colsample_bytree': 0.9002635824298411, 'reg_alpha': 7.066910403267653, 'reg_lambda': 6.28960034117413, 'min_split_gain': 0.004308824716441051}. Best is trial 2 with value: 0.006680435109783969.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:30:35,075] Trial 3 finished with value: 0.01004675179181575 and parameters: {'n_estimators': 175, 'learning_rate': 0.03189594929200614, 'num_leaves': 171, 'max_depth': 11, 'min_child_samples': 3, 'subsample': 0.5274321535073031, 'colsample_bytree': 0.5935177266036606, 'reg_alpha': 3.920712291206654, 'reg_lambda': 1.8284664807817663, 'min_split_gain': 0.04051354127759625}. Best is trial 2 with value: 0.006680435109783969.
[I 2025-06-15 20:30:35,078] A new study created in memory with name: no-name-fc687020-e3c1-46fe-ba15-6840e205ecdb
[I 2025-06-15 20:30:35,255] Trial 0 finished with value: 0.034291608865160646 and parameters: {'n_estimators': 245, 'learning_rate': 0.010754127665751524, 'num_leaves': 174, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.8826825293769993, 'colsample_bytree': 0.9206981263445977, 'reg_alpha': 5.0638641525745145, 'reg_lambda': 3.0066308804229203, 'min_split_gain': 0.01218953315919143}. Best is trial 0 with value: 0.034291608865160646.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         426
14      pct_chg_Load_DA         296
1           Price_lag_2         291
3         Load_DA_lag_0         220
5       WindOn_DA_lag_0         212
2           Price_lag_7         181
16  roll2d_median_Price         167
15       lag168_Load_DA         158
13            EUA_lag_2         147
11           NGas_lag_2         142
10           Coal_lag_2         112
12            Oil_lag_2          75
9                  WD_7          14
8                  WD_6           9
7                  WD_1           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         501
3         Load_DA_lag_0          89
13            EUA_lag_2          40
2           Price_lag_7          20
1           Price_lag_2          11
12            Oil_lag_2           8
10           Coal_lag_2           4
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 68.13% done
********************* END NS.. 497 ****************************************************
********************* START NS ... 498 ****************************************************
[I 2025-06-15 20:30:37,671] A new study created in memory with name: no-name-b3b2eca9-52cb-4c44-b665-8d225843e15c
[I 2025-06-15 20:30:38,227] Trial 0 finished with value: 0.009702755003290221 and parameters: {'n_estimators': 63, 'learning_rate': 0.15302760030799492, 'num_leaves': 87, 'max_depth': 11, 'min_child_samples': 14, 'subsample': 0.7627694313598847, 'colsample_bytree': 0.518200479300925, 'reg_alpha': 5.116606150627229, 'reg_lambda': 2.2209914373007997, 'min_split_gain': 0.03377704718100168}. Best is trial 0 with value: 0.009702755003290221.
[Trial 0] New best: 0.00970
[I 2025-06-15 20:30:38,956] Trial 1 finished with value: 0.007955026429793628 and parameters: {'n_estimators': 277, 'learning_rate': 0.08937338254539923, 'num_leaves': 107, 'max_depth': 9, 'min_child_samples': 13, 'subsample': 0.6590429821398226, 'colsample_bytree': 0.6458674194826303, 'reg_alpha': 9.748474653779871, 'reg_lambda': 6.79392352957745, 'min_split_gain': 0.03353655936099897}. Best is trial 1 with value: 0.007955026429793628.
[Trial 1] New best: 0.00796
[I 2025-06-15 20:30:39,519] Trial 2 finished with value: 0.007205030657993876 and parameters: {'n_estimators': 177, 'learning_rate': 0.12556926115304384, 'num_leaves': 99, 'max_depth': 12, 'min_child_samples': 10, 'subsample': 0.877715550079087, 'colsample_bytree': 0.868004303238797, 'reg_alpha': 7.66312083034689, 'reg_lambda': 7.355219209769295, 'min_split_gain': 0.020063748793647807}. Best is trial 2 with value: 0.007205030657993876.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:30:44,371] Trial 3 finished with value: 0.008346973142324687 and parameters: {'n_estimators': 267, 'learning_rate': 0.02044295949588786, 'num_leaves': 156, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.8173305685065544, 'colsample_bytree': 0.5779285851670195, 'reg_alpha': 3.1615398836018627, 'reg_lambda': 7.330052937550322, 'min_split_gain': 0.011050158490836971}. Best is trial 2 with value: 0.007205030657993876.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:30:45,920] Trial 4 finished with value: 0.024210399991706585 and parameters: {'n_estimators': 162, 'learning_rate': 0.013318918900794582, 'num_leaves': 110, 'max_depth': 11, 'min_child_samples': 12, 'subsample': 0.7017858557515927, 'colsample_bytree': 0.9805208448687279, 'reg_alpha': 6.1545337684452575, 'reg_lambda': 3.652311649213366, 'min_split_gain': 0.04059127295115841}. Best is trial 2 with value: 0.007205030657993876.
[I 2025-06-15 20:30:45,926] A new study created in memory with name: no-name-089d97bf-ab5b-47f1-9cc2-3986b65e47ae
[I 2025-06-15 20:30:45,984] Trial 0 finished with value: 0.12313945562237717 and parameters: {'n_estimators': 192, 'learning_rate': 0.15283810853374744, 'num_leaves': 217, 'max_depth': 16, 'min_child_samples': 15, 'subsample': 0.7247668233104647, 'colsample_bytree': 0.7304793509423232, 'reg_alpha': 7.059527438883949, 'reg_lambda': 3.1318281746991117, 'min_split_gain': 0.004435616483348059}. Best is trial 0 with value: 0.12313945562237717.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         425
1           Price_lag_2         304
14      pct_chg_Load_DA         288
3         Load_DA_lag_0         166
2           Price_lag_7         151
11           NGas_lag_2         145
16  roll2d_median_Price         134
5       WindOn_DA_lag_0         120
15       lag168_Load_DA         113
10           Coal_lag_2          98
13            EUA_lag_2          81
12            Oil_lag_2          30
9                  WD_7          16
8                  WD_6           9
7                  WD_1           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          40
3         Load_DA_lag_0          10
12            Oil_lag_2           3
1           Price_lag_2           3
13            EUA_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 68.26% done
********************* END NS.. 498 ****************************************************
********************* START NS ... 499 ****************************************************
[I 2025-06-15 20:30:48,371] A new study created in memory with name: no-name-27dcea4f-7558-4458-9f34-e09c3e6bcf8e
[I 2025-06-15 20:30:52,632] Trial 0 finished with value: 0.011505297637226982 and parameters: {'n_estimators': 299, 'learning_rate': 0.01195142347538434, 'num_leaves': 193, 'max_depth': 8, 'min_child_samples': 1, 'subsample': 0.8578194708409222, 'colsample_bytree': 0.5474927150766188, 'reg_alpha': 5.158945835745957, 'reg_lambda': 8.187144083214811, 'min_split_gain': 0.0007100785526902853}. Best is trial 0 with value: 0.011505297637226982.
[Trial 0] New best: 0.01151
[I 2025-06-15 20:30:53,319] Trial 1 finished with value: 0.007138422027175345 and parameters: {'n_estimators': 137, 'learning_rate': 0.072877598250788, 'num_leaves': 227, 'max_depth': 12, 'min_child_samples': 19, 'subsample': 0.8450299626482372, 'colsample_bytree': 0.7528223646790123, 'reg_alpha': 9.877863847103693, 'reg_lambda': 7.231326527841913, 'min_split_gain': 0.027590155005416075}. Best is trial 1 with value: 0.007138422027175345.
[Trial 1] New best: 0.00714
[I 2025-06-15 20:30:53,856] Trial 2 finished with value: 0.00798230857084445 and parameters: {'n_estimators': 162, 'learning_rate': 0.025158399792410625, 'num_leaves': 250, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.8175374089943224, 'colsample_bytree': 0.9963543566428161, 'reg_alpha': 5.2407228644002135, 'reg_lambda': 1.800292031861589, 'min_split_gain': 0.02196697724011781}. Best is trial 1 with value: 0.007138422027175345.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:30:54,266] Trial 3 finished with value: 0.00562968039859801 and parameters: {'n_estimators': 75, 'learning_rate': 0.09224511203690379, 'num_leaves': 219, 'max_depth': 5, 'min_child_samples': 4, 'subsample': 0.6083091533297618, 'colsample_bytree': 0.836882342636794, 'reg_alpha': 0.9105249560916973, 'reg_lambda': 2.468529091693825, 'min_split_gain': 0.03390416183580969}. Best is trial 3 with value: 0.00562968039859801.
[Trial 3] New best: 0.00563
[I 2025-06-15 20:30:54,941] Trial 4 finished with value: 0.0679788441107456 and parameters: {'n_estimators': 57, 'learning_rate': 0.026021108013395487, 'num_leaves': 231, 'max_depth': 16, 'min_child_samples': 10, 'subsample': 0.9076210816026253, 'colsample_bytree': 0.5354308299417938, 'reg_alpha': 3.79985598929083, 'reg_lambda': 6.1231246526575065, 'min_split_gain': 0.0370629145919859}. Best is trial 3 with value: 0.00562968039859801.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:30:55,564] Trial 5 finished with value: 0.006412179091948689 and parameters: {'n_estimators': 250, 'learning_rate': 0.2554424116850031, 'num_leaves': 146, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.5201724741504778, 'colsample_bytree': 0.7930777043968886, 'reg_alpha': 7.570605611729045, 'reg_lambda': 4.992865236730443, 'min_split_gain': 0.0074321925670553625}. Best is trial 3 with value: 0.00562968039859801.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:30:56,274] Trial 6 finished with value: 0.006632075517188629 and parameters: {'n_estimators': 214, 'learning_rate': 0.08598911011037708, 'num_leaves': 125, 'max_depth': 15, 'min_child_samples': 23, 'subsample': 0.6837782113623602, 'colsample_bytree': 0.820069741413326, 'reg_alpha': 5.498063157485392, 'reg_lambda': 3.9498935406566247, 'min_split_gain': 0.04174727527295263}. Best is trial 3 with value: 0.00562968039859801.
[I 2025-06-15 20:30:56,277] A new study created in memory with name: no-name-5eae9090-6d07-4199-a873-da7fc6b9c230
[I 2025-06-15 20:30:56,364] Trial 0 finished with value: 0.06374858473864863 and parameters: {'n_estimators': 143, 'learning_rate': 0.020668908899446416, 'num_leaves': 44, 'max_depth': 16, 'min_child_samples': 6, 'subsample': 0.9436353484869273, 'colsample_bytree': 0.7775408488013378, 'reg_alpha': 4.605776143043983, 'reg_lambda': 7.660919912841422, 'min_split_gain': 0.04018088624537469}. Best is trial 0 with value: 0.06374858473864863.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         545
1           Price_lag_2         285
14      pct_chg_Load_DA         284
3         Load_DA_lag_0         158
11           NGas_lag_2         134
2           Price_lag_7         130
16  roll2d_median_Price         125
15       lag168_Load_DA          85
5       WindOn_DA_lag_0          78
10           Coal_lag_2          66
13            EUA_lag_2          61
12            Oil_lag_2          59
9                  WD_7           7
7                  WD_1           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         222
3         Load_DA_lag_0          37
1           Price_lag_2          23
2           Price_lag_7          11
13            EUA_lag_2           9
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 68.40% done
********************* END NS.. 499 ****************************************************
********************* START NS ... 500 ****************************************************
[I 2025-06-15 20:30:58,419] A new study created in memory with name: no-name-3f7f8dac-056d-4bcf-bb01-d369217c690e
[I 2025-06-15 20:30:59,853] Trial 0 finished with value: 0.006699892179922614 and parameters: {'n_estimators': 211, 'learning_rate': 0.023704856786920765, 'num_leaves': 51, 'max_depth': 6, 'min_child_samples': 6, 'subsample': 0.9388868079184739, 'colsample_bytree': 0.8379037894726351, 'reg_alpha': 5.788489062385862, 'reg_lambda': 8.313281231686506, 'min_split_gain': 0.010442291848464586}. Best is trial 0 with value: 0.006699892179922614.
[Trial 0] New best: 0.00670
[I 2025-06-15 20:31:00,850] Trial 1 finished with value: 0.014195819887569816 and parameters: {'n_estimators': 263, 'learning_rate': 0.010251494519384613, 'num_leaves': 197, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.9821173132551653, 'colsample_bytree': 0.7197020805168017, 'reg_alpha': 0.7712694124751007, 'reg_lambda': 7.034958089990274, 'min_split_gain': 0.046243275895981456}. Best is trial 0 with value: 0.006699892179922614.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:31:02,142] Trial 2 finished with value: 0.04208454632233801 and parameters: {'n_estimators': 146, 'learning_rate': 0.011782549640586414, 'num_leaves': 38, 'max_depth': 13, 'min_child_samples': 26, 'subsample': 0.6959815751528573, 'colsample_bytree': 0.9381555454226184, 'reg_alpha': 7.448562393417658, 'reg_lambda': 3.3563492209734402, 'min_split_gain': 0.008744272533003623}. Best is trial 0 with value: 0.006699892179922614.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:31:03,430] Trial 3 finished with value: 0.006074286074367951 and parameters: {'n_estimators': 130, 'learning_rate': 0.05797814885449842, 'num_leaves': 123, 'max_depth': 14, 'min_child_samples': 15, 'subsample': 0.612209031643129, 'colsample_bytree': 0.8363790825334939, 'reg_alpha': 4.865189900208348, 'reg_lambda': 6.896511114036988, 'min_split_gain': 0.029704907105405194}. Best is trial 3 with value: 0.006074286074367951.
[I 2025-06-15 20:31:03,433] A new study created in memory with name: no-name-f20db15c-382f-4b7a-9612-07d1a73e43b4
[I 2025-06-15 20:31:03,466] Trial 0 finished with value: 0.018795329606288085 and parameters: {'n_estimators': 72, 'learning_rate': 0.26111032542074375, 'num_leaves': 119, 'max_depth': 4, 'min_child_samples': 21, 'subsample': 0.5546523946875611, 'colsample_bytree': 0.6843307018655942, 'reg_alpha': 3.8083328864000787, 'reg_lambda': 4.022068396932084, 'min_split_gain': 0.030482568532573}. Best is trial 0 with value: 0.018795329606288085.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1021
1           Price_lag_2         732
14      pct_chg_Load_DA         665
3         Load_DA_lag_0         458
11           NGas_lag_2         345
2           Price_lag_7         303
16  roll2d_median_Price         251
5       WindOn_DA_lag_0         227
15       lag168_Load_DA         208
13            EUA_lag_2         165
10           Coal_lag_2         143
12            Oil_lag_2         125
9                  WD_7          16
7                  WD_1          11
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          19
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 68.54% done
********************* END NS.. 500 ****************************************************
********************* START NS ... 501 ****************************************************
[I 2025-06-15 20:31:05,972] A new study created in memory with name: no-name-f2dc1e3b-6c9d-4e23-a366-4188c6f124c4
[I 2025-06-15 20:31:09,879] Trial 0 finished with value: 0.006165322898334315 and parameters: {'n_estimators': 294, 'learning_rate': 0.013908237299211888, 'num_leaves': 92, 'max_depth': 12, 'min_child_samples': 2, 'subsample': 0.7083027837021008, 'colsample_bytree': 0.9076208748119676, 'reg_alpha': 6.265980505061979, 'reg_lambda': 4.99807950496791, 'min_split_gain': 0.014031272380000799}. Best is trial 0 with value: 0.006165322898334315.
[Trial 0] New best: 0.00617
[I 2025-06-15 20:31:12,061] Trial 1 finished with value: 0.005904860940239663 and parameters: {'n_estimators': 195, 'learning_rate': 0.022119729291506414, 'num_leaves': 101, 'max_depth': 13, 'min_child_samples': 23, 'subsample': 0.8682980711353986, 'colsample_bytree': 0.8033286596275226, 'reg_alpha': 3.760041574225835, 'reg_lambda': 2.046526193476934, 'min_split_gain': 0.0433477128324903}. Best is trial 1 with value: 0.005904860940239663.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:31:12,472] Trial 2 finished with value: 0.00593834121430408 and parameters: {'n_estimators': 113, 'learning_rate': 0.1941693728519908, 'num_leaves': 109, 'max_depth': 15, 'min_child_samples': 12, 'subsample': 0.6750380597371762, 'colsample_bytree': 0.6883606512467507, 'reg_alpha': 8.852875178476319, 'reg_lambda': 9.532962204294815, 'min_split_gain': 0.019453133942815632}. Best is trial 1 with value: 0.005904860940239663.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:31:13,030] Trial 3 finished with value: 0.006595838645934645 and parameters: {'n_estimators': 51, 'learning_rate': 0.08125203028207481, 'num_leaves': 133, 'max_depth': 12, 'min_child_samples': 7, 'subsample': 0.7106148224724351, 'colsample_bytree': 0.6364059113252691, 'reg_alpha': 7.6976790746483985, 'reg_lambda': 0.5947426005348078, 'min_split_gain': 0.008637511178596263}. Best is trial 1 with value: 0.005904860940239663.
[I 2025-06-15 20:31:13,033] A new study created in memory with name: no-name-974069ee-5d86-4a02-9069-1fe67f11f33a
[I 2025-06-15 20:31:13,107] Trial 0 finished with value: 0.036399141195684104 and parameters: {'n_estimators': 281, 'learning_rate': 0.17107162278347743, 'num_leaves': 69, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.7886854406961397, 'colsample_bytree': 0.9586041837192901, 'reg_alpha': 9.025494632394638, 'reg_lambda': 0.9108594468957165, 'min_split_gain': 0.04411550573581488}. Best is trial 0 with value: 0.036399141195684104.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2265
1           Price_lag_2        1786
14      pct_chg_Load_DA        1168
3         Load_DA_lag_0         485
16  roll2d_median_Price         465
2           Price_lag_7         449
11           NGas_lag_2         377
15       lag168_Load_DA         194
12            Oil_lag_2         165
5       WindOn_DA_lag_0         161
13            EUA_lag_2         158
10           Coal_lag_2         151
9                  WD_7          18
8                  WD_6          10
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           7
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 68.67% done
********************* END NS.. 501 ****************************************************
********************* START NS ... 502 ****************************************************
[I 2025-06-15 20:31:16,210] A new study created in memory with name: no-name-ad7616f8-5066-402d-b82e-fb0a0a271c67
[I 2025-06-15 20:31:20,187] Trial 0 finished with value: 0.005112911040212844 and parameters: {'n_estimators': 296, 'learning_rate': 0.01264116825783619, 'num_leaves': 88, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.8076505786040292, 'colsample_bytree': 0.5484026158163549, 'reg_alpha': 1.8212536810163538, 'reg_lambda': 4.8019831136904205, 'min_split_gain': 0.0328882890205862}. Best is trial 0 with value: 0.005112911040212844.
[Trial 0] New best: 0.00511
[I 2025-06-15 20:31:23,253] Trial 1 finished with value: 0.0032032434884035886 and parameters: {'n_estimators': 212, 'learning_rate': 0.043182544185180194, 'num_leaves': 249, 'max_depth': 13, 'min_child_samples': 12, 'subsample': 0.5850068510114403, 'colsample_bytree': 0.7390415824839298, 'reg_alpha': 0.8821534751603555, 'reg_lambda': 4.267691724213543, 'min_split_gain': 0.02469708575922555}. Best is trial 1 with value: 0.0032032434884035886.
[Trial 1] New best: 0.00320
[I 2025-06-15 20:31:23,577] Trial 2 finished with value: 0.003734987456090034 and parameters: {'n_estimators': 90, 'learning_rate': 0.18516148566659704, 'num_leaves': 89, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.9633878211570229, 'colsample_bytree': 0.8861042204422774, 'reg_alpha': 4.805858001777393, 'reg_lambda': 9.991904186281069, 'min_split_gain': 0.04926165564880388}. Best is trial 1 with value: 0.0032032434884035886.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:31:25,265] Trial 3 finished with value: 0.0025955332613051703 and parameters: {'n_estimators': 241, 'learning_rate': 0.06443792147369377, 'num_leaves': 253, 'max_depth': 7, 'min_child_samples': 10, 'subsample': 0.7739071290038366, 'colsample_bytree': 0.904601736885425, 'reg_alpha': 0.8759687070941924, 'reg_lambda': 4.409301387741063, 'min_split_gain': 0.013812770682040771}. Best is trial 3 with value: 0.0025955332613051703.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:31:26,343] Trial 4 finished with value: 0.004277098143378903 and parameters: {'n_estimators': 260, 'learning_rate': 0.10048812285300142, 'num_leaves': 85, 'max_depth': 14, 'min_child_samples': 8, 'subsample': 0.5172379237567404, 'colsample_bytree': 0.5575521466286062, 'reg_alpha': 5.9648414692783005, 'reg_lambda': 7.795175139095098, 'min_split_gain': 0.03583499756472126}. Best is trial 3 with value: 0.0025955332613051703.
[I 2025-06-15 20:31:26,347] A new study created in memory with name: no-name-740bb29d-4f06-459b-9785-d84cb389c587
[I 2025-06-15 20:31:26,397] Trial 0 finished with value: 0.009634662926156836 and parameters: {'n_estimators': 175, 'learning_rate': 0.06427081175431348, 'num_leaves': 58, 'max_depth': 11, 'min_child_samples': 30, 'subsample': 0.6157051940507046, 'colsample_bytree': 0.9885087499572701, 'reg_alpha': 3.954514079402851, 'reg_lambda': 9.174775609287016, 'min_split_gain': 0.042834980353289316}. Best is trial 0 with value: 0.009634662926156836.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1487
14      pct_chg_Load_DA        1196
1           Price_lag_2        1118
3         Load_DA_lag_0         748
5       WindOn_DA_lag_0         664
15       lag168_Load_DA         654
16  roll2d_median_Price         605
11           NGas_lag_2         595
2           Price_lag_7         591
10           Coal_lag_2         392
13            EUA_lag_2         290
12            Oil_lag_2         257
9                  WD_7          33
7                  WD_1          26
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          48
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 68.81% done
********************* END NS.. 502 ****************************************************
********************* START NS ... 503 ****************************************************
[I 2025-06-15 20:31:29,332] A new study created in memory with name: no-name-cf4550dd-e6ba-44f9-aef0-a86987a58613
[I 2025-06-15 20:31:30,248] Trial 0 finished with value: 0.0023053905299861405 and parameters: {'n_estimators': 159, 'learning_rate': 0.2009268427961025, 'num_leaves': 66, 'max_depth': 15, 'min_child_samples': 20, 'subsample': 0.6847021258468491, 'colsample_bytree': 0.8119485425200333, 'reg_alpha': 1.0000384344880475, 'reg_lambda': 2.2935984247513996, 'min_split_gain': 0.022695301077724946}. Best is trial 0 with value: 0.0023053905299861405.
[Trial 0] New best: 0.00231
[I 2025-06-15 20:31:31,512] Trial 1 finished with value: 0.0023485278876928324 and parameters: {'n_estimators': 287, 'learning_rate': 0.04409272862278097, 'num_leaves': 183, 'max_depth': 6, 'min_child_samples': 26, 'subsample': 0.5911325724033591, 'colsample_bytree': 0.7975495629901485, 'reg_alpha': 8.39893503514579, 'reg_lambda': 1.0450987974193493, 'min_split_gain': 0.015679948708256737}. Best is trial 0 with value: 0.0023053905299861405.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:31:33,937] Trial 2 finished with value: 0.0024679515464069808 and parameters: {'n_estimators': 228, 'learning_rate': 0.022103844599728556, 'num_leaves': 55, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.8890721202287621, 'colsample_bytree': 0.7492810991808032, 'reg_alpha': 3.876700441264116, 'reg_lambda': 4.97205956022153, 'min_split_gain': 0.03444779743727991}. Best is trial 0 with value: 0.0023053905299861405.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:31:34,683] Trial 3 finished with value: 0.002570463135018063 and parameters: {'n_estimators': 118, 'learning_rate': 0.13274265009239714, 'num_leaves': 125, 'max_depth': 8, 'min_child_samples': 4, 'subsample': 0.6789516527597601, 'colsample_bytree': 0.5554829983096419, 'reg_alpha': 2.5142158248439115, 'reg_lambda': 9.21988195228245, 'min_split_gain': 0.03985768679071584}. Best is trial 0 with value: 0.0023053905299861405.
[I 2025-06-15 20:31:34,686] A new study created in memory with name: no-name-a796971e-832a-49b7-a27e-3c9f92999949
[I 2025-06-15 20:31:34,768] Trial 0 finished with value: 0.0048354470477540885 and parameters: {'n_estimators': 199, 'learning_rate': 0.045369187662973855, 'num_leaves': 128, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.796046158231404, 'colsample_bytree': 0.8725539382035432, 'reg_alpha': 5.353687607712116, 'reg_lambda': 4.840188878338462, 'min_split_gain': 0.0018575924285144819}. Best is trial 0 with value: 0.0048354470477540885.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         437
1           Price_lag_2         342
14      pct_chg_Load_DA         317
3         Load_DA_lag_0         199
2           Price_lag_7         162
15       lag168_Load_DA         160
5       WindOn_DA_lag_0         157
16  roll2d_median_Price         154
11           NGas_lag_2         151
10           Coal_lag_2          88
13            EUA_lag_2          80
12            Oil_lag_2          53
9                  WD_7          11
7                  WD_1           5
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         103
1           Price_lag_2           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 68.95% done
********************* END NS.. 503 ****************************************************
********************* START NS ... 504 ****************************************************
[I 2025-06-15 20:31:36,651] A new study created in memory with name: no-name-60d1e7fa-301b-40ef-be59-1e090f03dde4
[I 2025-06-15 20:31:37,874] Trial 0 finished with value: 0.00346285952915179 and parameters: {'n_estimators': 136, 'learning_rate': 0.033030688173153494, 'num_leaves': 83, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.9336240789185111, 'colsample_bytree': 0.5074825260467573, 'reg_alpha': 8.572842286644192, 'reg_lambda': 5.746882793062722, 'min_split_gain': 0.018893204547669358}. Best is trial 0 with value: 0.00346285952915179.
[Trial 0] New best: 0.00346
[I 2025-06-15 20:31:38,212] Trial 1 finished with value: 0.0030266318372099287 and parameters: {'n_estimators': 141, 'learning_rate': 0.17619127725327258, 'num_leaves': 131, 'max_depth': 6, 'min_child_samples': 21, 'subsample': 0.8812365919250458, 'colsample_bytree': 0.7483515283149123, 'reg_alpha': 8.641363325898872, 'reg_lambda': 0.06197479145100826, 'min_split_gain': 0.031967846063395824}. Best is trial 1 with value: 0.0030266318372099287.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:31:38,711] Trial 2 finished with value: 0.007071680284564977 and parameters: {'n_estimators': 53, 'learning_rate': 0.05148651613813768, 'num_leaves': 64, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.6503708562042954, 'colsample_bytree': 0.8098576821545221, 'reg_alpha': 8.338181442733042, 'reg_lambda': 1.2093805370749255, 'min_split_gain': 0.02048443625469239}. Best is trial 1 with value: 0.0030266318372099287.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:31:40,183] Trial 3 finished with value: 0.003008173154268251 and parameters: {'n_estimators': 233, 'learning_rate': 0.07206211172760725, 'num_leaves': 199, 'max_depth': 12, 'min_child_samples': 5, 'subsample': 0.8436263415474594, 'colsample_bytree': 0.7496963473429792, 'reg_alpha': 5.897273740384408, 'reg_lambda': 8.94660226085856, 'min_split_gain': 0.0077843244340793875}. Best is trial 3 with value: 0.003008173154268251.
[I 2025-06-15 20:31:40,185] A new study created in memory with name: no-name-953b04fa-1eec-4441-8572-ef2e23d1e48a
[I 2025-06-15 20:31:40,255] Trial 0 finished with value: 0.004836687263864901 and parameters: {'n_estimators': 251, 'learning_rate': 0.050053933520030375, 'num_leaves': 209, 'max_depth': 6, 'min_child_samples': 3, 'subsample': 0.5585586111203948, 'colsample_bytree': 0.7094130426704339, 'reg_alpha': 4.250488461040725, 'reg_lambda': 3.5154024685649468, 'min_split_gain': 0.024329464451007185}. Best is trial 0 with value: 0.004836687263864901.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         976
1           Price_lag_2         947
14      pct_chg_Load_DA         804
5       WindOn_DA_lag_0         517
3         Load_DA_lag_0         485
11           NGas_lag_2         483
15       lag168_Load_DA         461
2           Price_lag_7         410
13            EUA_lag_2         367
16  roll2d_median_Price         336
10           Coal_lag_2         296
12            Oil_lag_2         239
9                  WD_7          47
8                  WD_6          28
7                  WD_1          19
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          48
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 69.08% done
********************* END NS.. 504 ****************************************************
********************* START NS ... 505 ****************************************************
[I 2025-06-15 20:31:43,229] A new study created in memory with name: no-name-899c5257-76d5-459e-859a-ad4ced9f29f0
[I 2025-06-15 20:31:44,252] Trial 0 finished with value: 0.004655074362282036 and parameters: {'n_estimators': 249, 'learning_rate': 0.06512268419327781, 'num_leaves': 76, 'max_depth': 6, 'min_child_samples': 20, 'subsample': 0.958418137704627, 'colsample_bytree': 0.723931345684462, 'reg_alpha': 5.503339677654607, 'reg_lambda': 9.820387427144722, 'min_split_gain': 0.04400216637700918}. Best is trial 0 with value: 0.004655074362282036.
[Trial 0] New best: 0.00466
[I 2025-06-15 20:31:44,645] Trial 1 finished with value: 0.0048882355073596295 and parameters: {'n_estimators': 124, 'learning_rate': 0.25216097043263186, 'num_leaves': 137, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.7291854672960885, 'colsample_bytree': 0.506749751186981, 'reg_alpha': 5.315662103469211, 'reg_lambda': 4.950497476990751, 'min_split_gain': 0.024739525616761505}. Best is trial 0 with value: 0.004655074362282036.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:31:45,078] Trial 2 finished with value: 0.004704869957984186 and parameters: {'n_estimators': 60, 'learning_rate': 0.2708672626927136, 'num_leaves': 253, 'max_depth': 9, 'min_child_samples': 10, 'subsample': 0.9200054505241804, 'colsample_bytree': 0.5922202276704471, 'reg_alpha': 1.3854030943093432, 'reg_lambda': 9.977966137066144, 'min_split_gain': 0.030447986124850685}. Best is trial 0 with value: 0.004655074362282036.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:31:46,750] Trial 3 finished with value: 0.00436684086277993 and parameters: {'n_estimators': 141, 'learning_rate': 0.03785098653398387, 'num_leaves': 236, 'max_depth': 12, 'min_child_samples': 21, 'subsample': 0.7854806014258868, 'colsample_bytree': 0.8801299803953784, 'reg_alpha': 1.8437423980168843, 'reg_lambda': 7.624709910712063, 'min_split_gain': 0.03168759435668071}. Best is trial 3 with value: 0.00436684086277993.
[I 2025-06-15 20:31:46,753] A new study created in memory with name: no-name-679911fb-ec6d-4e30-97f4-837f7a1ac30b
[I 2025-06-15 20:31:46,835] Trial 0 finished with value: 0.011285534439318595 and parameters: {'n_estimators': 162, 'learning_rate': 0.028904984838532696, 'num_leaves': 199, 'max_depth': 7, 'min_child_samples': 5, 'subsample': 0.5941588115300811, 'colsample_bytree': 0.6891992610948434, 'reg_alpha': 2.345580529060738, 'reg_lambda': 2.544581606234888, 'min_split_gain': 0.024252922832052062}. Best is trial 0 with value: 0.011285534439318595.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1798
1           Price_lag_2        1598
14      pct_chg_Load_DA        1162
3         Load_DA_lag_0         692
11           NGas_lag_2         572
2           Price_lag_7         561
16  roll2d_median_Price         488
15       lag168_Load_DA         349
5       WindOn_DA_lag_0         335
10           Coal_lag_2         244
12            Oil_lag_2         243
13            EUA_lag_2         224
9                  WD_7          11
7                  WD_1           9
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         109
1           Price_lag_2           8
2           Price_lag_7           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
16  roll2d_median_Price           0
-> 69.22% done
********************* END NS.. 505 ****************************************************
********************* START NS ... 506 ****************************************************
[I 2025-06-15 20:31:49,691] A new study created in memory with name: no-name-9aa89ea5-1763-48d6-8d4d-831ea860182a
[I 2025-06-15 20:31:50,321] Trial 0 finished with value: 0.004828029860294208 and parameters: {'n_estimators': 209, 'learning_rate': 0.20297988539953712, 'num_leaves': 149, 'max_depth': 6, 'min_child_samples': 28, 'subsample': 0.8716894464109444, 'colsample_bytree': 0.7180675448567658, 'reg_alpha': 4.671576502139178, 'reg_lambda': 1.3647148623807703, 'min_split_gain': 0.04084753674559993}. Best is trial 0 with value: 0.004828029860294208.
[Trial 0] New best: 0.00483
[I 2025-06-15 20:31:54,483] Trial 1 finished with value: 0.007087969399849536 and parameters: {'n_estimators': 168, 'learning_rate': 0.018452286140342562, 'num_leaves': 151, 'max_depth': 16, 'min_child_samples': 18, 'subsample': 0.8341286600523455, 'colsample_bytree': 0.7057994783295387, 'reg_alpha': 0.4051027602797297, 'reg_lambda': 3.7991490239645964, 'min_split_gain': 0.013637807297966653}. Best is trial 0 with value: 0.004828029860294208.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:31:55,724] Trial 2 finished with value: 0.005905914417403374 and parameters: {'n_estimators': 217, 'learning_rate': 0.038179512605462027, 'num_leaves': 78, 'max_depth': 16, 'min_child_samples': 18, 'subsample': 0.8167024334048703, 'colsample_bytree': 0.6644009680920271, 'reg_alpha': 9.447485356971477, 'reg_lambda': 5.990586138556266, 'min_split_gain': 0.02279220486038538}. Best is trial 0 with value: 0.004828029860294208.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:31:56,643] Trial 3 finished with value: 0.004393286833752871 and parameters: {'n_estimators': 130, 'learning_rate': 0.10064113421941287, 'num_leaves': 218, 'max_depth': 8, 'min_child_samples': 18, 'subsample': 0.5330463613292062, 'colsample_bytree': 0.9431543055654935, 'reg_alpha': 6.8830517429646845, 'reg_lambda': 0.6477898117697833, 'min_split_gain': 0.011770763798145407}. Best is trial 3 with value: 0.004393286833752871.
[I 2025-06-15 20:31:56,645] A new study created in memory with name: no-name-2d16ec30-fe61-4007-b702-438b0cbc078c
[I 2025-06-15 20:31:56,688] Trial 0 finished with value: 0.01978400001524087 and parameters: {'n_estimators': 172, 'learning_rate': 0.27714215272765225, 'num_leaves': 112, 'max_depth': 14, 'min_child_samples': 8, 'subsample': 0.7387784597906106, 'colsample_bytree': 0.718190564368621, 'reg_alpha': 9.54239694879923, 'reg_lambda': 6.584080723099133, 'min_split_gain': 0.03690600360114829}. Best is trial 0 with value: 0.01978400001524087.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         588
14      pct_chg_Load_DA         389
1           Price_lag_2         334
3         Load_DA_lag_0         260
11           NGas_lag_2         181
16  roll2d_median_Price         178
5       WindOn_DA_lag_0         158
2           Price_lag_7         156
15       lag168_Load_DA         145
10           Coal_lag_2         124
13            EUA_lag_2          95
12            Oil_lag_2          78
9                  WD_7          13
7                  WD_1          12
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 69.36% done
********************* END NS.. 506 ****************************************************
********************* START NS ... 507 ****************************************************
[I 2025-06-15 20:31:58,384] A new study created in memory with name: no-name-b22d1804-4b08-4e10-a66f-0d58e14b06f3
[I 2025-06-15 20:31:58,869] Trial 0 finished with value: 0.0039048855236659004 and parameters: {'n_estimators': 203, 'learning_rate': 0.06500793128532975, 'num_leaves': 232, 'max_depth': 4, 'min_child_samples': 9, 'subsample': 0.5515547817622137, 'colsample_bytree': 0.9238828601283764, 'reg_alpha': 7.19271012797566, 'reg_lambda': 9.298275542534576, 'min_split_gain': 0.03988166250299641}. Best is trial 0 with value: 0.0039048855236659004.
[Trial 0] New best: 0.00390
[I 2025-06-15 20:32:00,826] Trial 1 finished with value: 0.0043702139800423714 and parameters: {'n_estimators': 235, 'learning_rate': 0.06221570274176373, 'num_leaves': 123, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.7792341645154539, 'colsample_bytree': 0.5668814029950762, 'reg_alpha': 1.7899637882294694, 'reg_lambda': 5.686874817091677, 'min_split_gain': 0.038773395500640406}. Best is trial 0 with value: 0.0039048855236659004.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:32:01,583] Trial 2 finished with value: 0.003849131241703481 and parameters: {'n_estimators': 160, 'learning_rate': 0.11271146328581043, 'num_leaves': 236, 'max_depth': 10, 'min_child_samples': 28, 'subsample': 0.8365184688711051, 'colsample_bytree': 0.8822956382243301, 'reg_alpha': 9.463592170663475, 'reg_lambda': 1.2900081066730573, 'min_split_gain': 0.028380350721881033}. Best is trial 2 with value: 0.003849131241703481.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:32:01,988] Trial 3 finished with value: 0.0046051789104378325 and parameters: {'n_estimators': 140, 'learning_rate': 0.20435615401051374, 'num_leaves': 104, 'max_depth': 15, 'min_child_samples': 11, 'subsample': 0.952772816244918, 'colsample_bytree': 0.6291381739162296, 'reg_alpha': 8.400048169570061, 'reg_lambda': 4.514941476338913, 'min_split_gain': 0.0325392410311715}. Best is trial 2 with value: 0.003849131241703481.
[I 2025-06-15 20:32:01,990] A new study created in memory with name: no-name-d64103b1-1c5d-4633-9ca4-9f1a9f0fae0c
[I 2025-06-15 20:32:02,063] Trial 0 finished with value: 0.009340522351461442 and parameters: {'n_estimators': 89, 'learning_rate': 0.023482835158153192, 'num_leaves': 141, 'max_depth': 12, 'min_child_samples': 20, 'subsample': 0.8023085672704218, 'colsample_bytree': 0.6186625484662869, 'reg_alpha': 0.25452746706718266, 'reg_lambda': 2.0330038271190896, 'min_split_gain': 0.02866802623657454}. Best is trial 0 with value: 0.009340522351461442.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         396
14      pct_chg_Load_DA         227
1           Price_lag_2         208
3         Load_DA_lag_0         121
11           NGas_lag_2         112
16  roll2d_median_Price          96
2           Price_lag_7          93
15       lag168_Load_DA          76
5       WindOn_DA_lag_0          66
10           Coal_lag_2          59
13            EUA_lag_2          56
12            Oil_lag_2          22
7                  WD_1          10
9                  WD_7           9
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          94
5       WindOn_DA_lag_0          36
1           Price_lag_2          31
15       lag168_Load_DA          17
2           Price_lag_7           6
16  roll2d_median_Price           4
11           NGas_lag_2           2
14      pct_chg_Load_DA           2
10           Coal_lag_2           1
13            EUA_lag_2           1
3         Load_DA_lag_0           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
12            Oil_lag_2           0
8                  WD_6           0
-> 69.49% done
********************* END NS.. 507 ****************************************************
********************* START NS ... 508 ****************************************************
[I 2025-06-15 20:32:03,677] A new study created in memory with name: no-name-c5972ddf-fb8d-4cef-9ffa-70657f5061fe
[I 2025-06-15 20:32:05,760] Trial 0 finished with value: 0.0023682841965293757 and parameters: {'n_estimators': 277, 'learning_rate': 0.018379660133959783, 'num_leaves': 234, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.6720205134048931, 'colsample_bytree': 0.620560805064223, 'reg_alpha': 7.879263621172012, 'reg_lambda': 1.6844947178077885, 'min_split_gain': 0.03681052978371693}. Best is trial 0 with value: 0.0023682841965293757.
[Trial 0] New best: 0.00237
[I 2025-06-15 20:32:06,654] Trial 1 finished with value: 0.0022475653641166123 and parameters: {'n_estimators': 162, 'learning_rate': 0.05020866200508828, 'num_leaves': 97, 'max_depth': 10, 'min_child_samples': 15, 'subsample': 0.9014704353214453, 'colsample_bytree': 0.951990510232211, 'reg_alpha': 7.744098501929244, 'reg_lambda': 0.5554685359786748, 'min_split_gain': 0.03611901870648447}. Best is trial 1 with value: 0.0022475653641166123.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:32:07,144] Trial 2 finished with value: 0.0022585716050588967 and parameters: {'n_estimators': 151, 'learning_rate': 0.17895436032231474, 'num_leaves': 229, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.9135936898117141, 'colsample_bytree': 0.752814451408178, 'reg_alpha': 1.2854608234699083, 'reg_lambda': 3.4807653435648414, 'min_split_gain': 0.04135452636552666}. Best is trial 1 with value: 0.0022475653641166123.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:32:07,858] Trial 3 finished with value: 0.002549634252345901 and parameters: {'n_estimators': 114, 'learning_rate': 0.10115057929702022, 'num_leaves': 203, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.8022404196466293, 'colsample_bytree': 0.7850685502329371, 'reg_alpha': 6.17373424166138, 'reg_lambda': 3.453932644138785, 'min_split_gain': 0.015514677478022126}. Best is trial 1 with value: 0.0022475653641166123.
[I 2025-06-15 20:32:07,862] A new study created in memory with name: no-name-c2635d78-e339-4ad6-8cf7-ce4ecaf2c4b3
[I 2025-06-15 20:32:07,903] Trial 0 finished with value: 0.01858090221623629 and parameters: {'n_estimators': 141, 'learning_rate': 0.29778462472426853, 'num_leaves': 174, 'max_depth': 8, 'min_child_samples': 4, 'subsample': 0.7021320260669001, 'colsample_bytree': 0.5731278262632549, 'reg_alpha': 4.907725714231087, 'reg_lambda': 3.6188263713809956, 'min_split_gain': 0.0005518075421936553}. Best is trial 0 with value: 0.01858090221623629.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1015
14      pct_chg_Load_DA         616
1           Price_lag_2         534
3         Load_DA_lag_0         308
11           NGas_lag_2         282
2           Price_lag_7         257
16  roll2d_median_Price         238
15       lag168_Load_DA         148
5       WindOn_DA_lag_0         139
10           Coal_lag_2         125
13            EUA_lag_2          73
12            Oil_lag_2          68
7                  WD_1          15
9                  WD_7          10
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          10
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 69.63% done
********************* END NS.. 508 ****************************************************
********************* START NS ... 509 ****************************************************
[I 2025-06-15 20:32:09,931] A new study created in memory with name: no-name-0aeadb01-d136-4514-92c0-82655cbf8d79
[I 2025-06-15 20:32:10,470] Trial 0 finished with value: 0.000993736965222014 and parameters: {'n_estimators': 75, 'learning_rate': 0.05416008006018311, 'num_leaves': 111, 'max_depth': 6, 'min_child_samples': 14, 'subsample': 0.9980620019742236, 'colsample_bytree': 0.8064201671393267, 'reg_alpha': 7.197753599165556, 'reg_lambda': 1.2424568336169672, 'min_split_gain': 0.03943335202093918}. Best is trial 0 with value: 0.000993736965222014.
[Trial 0] New best: 0.00099
[I 2025-06-15 20:32:11,487] Trial 1 finished with value: 0.0009979366747763707 and parameters: {'n_estimators': 134, 'learning_rate': 0.03033993832161253, 'num_leaves': 37, 'max_depth': 11, 'min_child_samples': 25, 'subsample': 0.577804125530871, 'colsample_bytree': 0.7529965356448982, 'reg_alpha': 8.9931214510022, 'reg_lambda': 0.9791299035905898, 'min_split_gain': 0.0029726825834010874}. Best is trial 0 with value: 0.000993736965222014.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:32:12,158] Trial 2 finished with value: 0.0013556809441213017 and parameters: {'n_estimators': 158, 'learning_rate': 0.1355324512201834, 'num_leaves': 236, 'max_depth': 8, 'min_child_samples': 9, 'subsample': 0.8462728437609806, 'colsample_bytree': 0.5828028120899992, 'reg_alpha': 9.896722924181839, 'reg_lambda': 7.02531112115163, 'min_split_gain': 0.03243902238149884}. Best is trial 0 with value: 0.000993736965222014.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:32:12,706] Trial 3 finished with value: 0.047921529267969304 and parameters: {'n_estimators': 67, 'learning_rate': 0.020151934058943347, 'num_leaves': 39, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.6774714666656998, 'colsample_bytree': 0.7763031936423634, 'reg_alpha': 5.513032353638173, 'reg_lambda': 3.788105047069471, 'min_split_gain': 0.03946741419271468}. Best is trial 0 with value: 0.000993736965222014.
[I 2025-06-15 20:32:12,709] A new study created in memory with name: no-name-0b2c43ed-a652-4726-953a-1af1ebedb96a
[I 2025-06-15 20:32:12,732] Trial 0 finished with value: 0.010329175360850961 and parameters: {'n_estimators': 63, 'learning_rate': 0.24211117593874237, 'num_leaves': 117, 'max_depth': 9, 'min_child_samples': 21, 'subsample': 0.7734608961854718, 'colsample_bytree': 0.8146652409010333, 'reg_alpha': 6.348185474955434, 'reg_lambda': 6.959594921030754, 'min_split_gain': 0.032065000352669286}. Best is trial 0 with value: 0.010329175360850961.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         692
1           Price_lag_2         414
14      pct_chg_Load_DA         325
16  roll2d_median_Price         132
2           Price_lag_7         120
3         Load_DA_lag_0         109
11           NGas_lag_2         102
15       lag168_Load_DA          41
10           Coal_lag_2          35
13            EUA_lag_2          35
12            Oil_lag_2          33
5       WindOn_DA_lag_0          24
9                  WD_7           8
8                  WD_6           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 69.77% done
********************* END NS.. 509 ****************************************************
********************* START NS ... 510 ****************************************************
[I 2025-06-15 20:32:14,375] A new study created in memory with name: no-name-45e23b80-18a5-408a-a0c8-c1fc91ce7c30
[I 2025-06-15 20:32:15,008] Trial 0 finished with value: 0.00335566140390698 and parameters: {'n_estimators': 67, 'learning_rate': 0.06556280450136867, 'num_leaves': 190, 'max_depth': 8, 'min_child_samples': 4, 'subsample': 0.9646181206864046, 'colsample_bytree': 0.6322648336888486, 'reg_alpha': 5.429226454826459, 'reg_lambda': 6.985304648335382, 'min_split_gain': 0.03827788752159977}. Best is trial 0 with value: 0.00335566140390698.
[Trial 0] New best: 0.00336
[I 2025-06-15 20:32:18,562] Trial 1 finished with value: 0.004193390874870913 and parameters: {'n_estimators': 243, 'learning_rate': 0.020186822016073767, 'num_leaves': 76, 'max_depth': 16, 'min_child_samples': 15, 'subsample': 0.665453747828043, 'colsample_bytree': 0.8919404013439466, 'reg_alpha': 2.479849752373017, 'reg_lambda': 0.6246557639143036, 'min_split_gain': 0.010855336700422463}. Best is trial 0 with value: 0.00335566140390698.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:32:20,342] Trial 2 finished with value: 0.03820505300229022 and parameters: {'n_estimators': 144, 'learning_rate': 0.01027425587847497, 'num_leaves': 206, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.596046556242531, 'colsample_bytree': 0.7008035067253482, 'reg_alpha': 2.1552221845925135, 'reg_lambda': 5.400835829943841, 'min_split_gain': 0.002656655327738594}. Best is trial 0 with value: 0.00335566140390698.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:32:22,190] Trial 3 finished with value: 0.0038077543764476044 and parameters: {'n_estimators': 77, 'learning_rate': 0.036939321780111424, 'num_leaves': 143, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.5136377881523209, 'colsample_bytree': 0.5466262553342562, 'reg_alpha': 0.5115424836911697, 'reg_lambda': 1.5477215009654055, 'min_split_gain': 0.0435543599727599}. Best is trial 0 with value: 0.00335566140390698.
[I 2025-06-15 20:32:22,194] A new study created in memory with name: no-name-0f8341bb-b6d3-4b33-b315-a3fdeb10f957
[I 2025-06-15 20:32:22,360] Trial 0 finished with value: 0.017374950678058176 and parameters: {'n_estimators': 192, 'learning_rate': 0.2203026759268651, 'num_leaves': 116, 'max_depth': 15, 'min_child_samples': 24, 'subsample': 0.7799442523901639, 'colsample_bytree': 0.8787897377119356, 'reg_alpha': 8.659348719691323, 'reg_lambda': 6.986741591142389, 'min_split_gain': 0.03437177854834216}. Best is trial 0 with value: 0.017374950678058176.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         527
0           Price_lag_1         510
14      pct_chg_Load_DA         290
16  roll2d_median_Price         216
2           Price_lag_7         200
3         Load_DA_lag_0         199
11           NGas_lag_2         190
15       lag168_Load_DA         141
13            EUA_lag_2         133
5       WindOn_DA_lag_0         101
10           Coal_lag_2          91
12            Oil_lag_2          84
9                  WD_7          11
8                  WD_6          10
7                  WD_1           4
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 69.90% done
********************* END NS.. 510 ****************************************************
********************* START NS ... 511 ****************************************************
[I 2025-06-15 20:32:24,405] A new study created in memory with name: no-name-463e524b-f129-4491-be91-a2f54a477557
[I 2025-06-15 20:32:26,991] Trial 0 finished with value: 0.0054796807166962724 and parameters: {'n_estimators': 125, 'learning_rate': 0.06324119134307439, 'num_leaves': 255, 'max_depth': 14, 'min_child_samples': 15, 'subsample': 0.8347845812204593, 'colsample_bytree': 0.6686796812481743, 'reg_alpha': 0.9535081017883718, 'reg_lambda': 6.3506275414825195, 'min_split_gain': 0.012979742461717065}. Best is trial 0 with value: 0.0054796807166962724.
[Trial 0] New best: 0.00548
[I 2025-06-15 20:32:27,967] Trial 1 finished with value: 0.006460951766715687 and parameters: {'n_estimators': 276, 'learning_rate': 0.07202841374588395, 'num_leaves': 195, 'max_depth': 6, 'min_child_samples': 13, 'subsample': 0.8415476993549722, 'colsample_bytree': 0.8508496979305114, 'reg_alpha': 3.276984718962014, 'reg_lambda': 3.904395287042708, 'min_split_gain': 0.01243192802253414}. Best is trial 0 with value: 0.0054796807166962724.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:32:29,477] Trial 2 finished with value: 0.00591334041379665 and parameters: {'n_estimators': 216, 'learning_rate': 0.05923124446349127, 'num_leaves': 164, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.8657779957845004, 'colsample_bytree': 0.8056917470433177, 'reg_alpha': 2.236985936883793, 'reg_lambda': 2.822797538673142, 'min_split_gain': 0.025860364353587892}. Best is trial 0 with value: 0.0054796807166962724.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:32:32,131] Trial 3 finished with value: 0.0069128356047555755 and parameters: {'n_estimators': 274, 'learning_rate': 0.013617613412665314, 'num_leaves': 96, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.7558096265926436, 'colsample_bytree': 0.8550258958908992, 'reg_alpha': 6.87062799970913, 'reg_lambda': 2.0335618565033933, 'min_split_gain': 0.007874347486972138}. Best is trial 0 with value: 0.0054796807166962724.
[I 2025-06-15 20:32:32,134] A new study created in memory with name: no-name-fd66a1fd-abcb-408d-817b-50376d64dcc3
[I 2025-06-15 20:32:32,176] Trial 0 finished with value: 0.016639936729565652 and parameters: {'n_estimators': 174, 'learning_rate': 0.12422991522040892, 'num_leaves': 35, 'max_depth': 4, 'min_child_samples': 13, 'subsample': 0.7660214664322965, 'colsample_bytree': 0.8701027556016605, 'reg_alpha': 7.465778358153702, 'reg_lambda': 7.251519246355138, 'min_split_gain': 0.005273037364165395}. Best is trial 0 with value: 0.016639936729565652.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1704
0           Price_lag_1        1566
14      pct_chg_Load_DA        1455
3         Load_DA_lag_0        1288
5       WindOn_DA_lag_0        1053
15       lag168_Load_DA        1045
2           Price_lag_7         898
11           NGas_lag_2         845
16  roll2d_median_Price         822
13            EUA_lag_2         613
10           Coal_lag_2         569
12            Oil_lag_2         488
7                  WD_1          39
9                  WD_7          35
8                  WD_6          16
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 70.04% done
********************* END NS.. 511 ****************************************************
********************* START NS ... 512 ****************************************************
[I 2025-06-15 20:32:35,579] A new study created in memory with name: no-name-725d7c2f-3bdc-4f21-b65b-45acd6b5e9ce
[I 2025-06-15 20:32:37,676] Trial 0 finished with value: 0.004972299887986255 and parameters: {'n_estimators': 250, 'learning_rate': 0.017531544869094216, 'num_leaves': 214, 'max_depth': 7, 'min_child_samples': 1, 'subsample': 0.6968235160904975, 'colsample_bytree': 0.5740350088181353, 'reg_alpha': 6.511013936853597, 'reg_lambda': 7.6168641761489795, 'min_split_gain': 0.025952825693844217}. Best is trial 0 with value: 0.004972299887986255.
[Trial 0] New best: 0.00497
[I 2025-06-15 20:32:38,449] Trial 1 finished with value: 0.0031808221065150687 and parameters: {'n_estimators': 132, 'learning_rate': 0.15077480350829836, 'num_leaves': 53, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.7681394513985615, 'colsample_bytree': 0.521831777554963, 'reg_alpha': 1.9162852081444104, 'reg_lambda': 0.07044049319191559, 'min_split_gain': 0.01645230723494382}. Best is trial 1 with value: 0.0031808221065150687.
[Trial 1] New best: 0.00318
[I 2025-06-15 20:32:39,297] Trial 2 finished with value: 0.049132636047172995 and parameters: {'n_estimators': 130, 'learning_rate': 0.012584810153516882, 'num_leaves': 129, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.9282046395893815, 'colsample_bytree': 0.9975107875966656, 'reg_alpha': 7.986199910074269, 'reg_lambda': 6.518355706229295, 'min_split_gain': 0.04316034248424902}. Best is trial 1 with value: 0.0031808221065150687.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:32:40,573] Trial 3 finished with value: 0.003729191361289888 and parameters: {'n_estimators': 142, 'learning_rate': 0.08319297080938776, 'num_leaves': 88, 'max_depth': 14, 'min_child_samples': 17, 'subsample': 0.9049475493428123, 'colsample_bytree': 0.6277364688786742, 'reg_alpha': 0.32308548877008736, 'reg_lambda': 6.374220443815676, 'min_split_gain': 0.048769473276730216}. Best is trial 1 with value: 0.0031808221065150687.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:32:42,872] Trial 4 finished with value: 0.004721745923965982 and parameters: {'n_estimators': 222, 'learning_rate': 0.0275637984305609, 'num_leaves': 50, 'max_depth': 8, 'min_child_samples': 1, 'subsample': 0.6920691264833596, 'colsample_bytree': 0.5776208205886896, 'reg_alpha': 6.00494975259541, 'reg_lambda': 0.7941377779550507, 'min_split_gain': 0.028997970973252553}. Best is trial 1 with value: 0.0031808221065150687.
[I 2025-06-15 20:32:42,874] A new study created in memory with name: no-name-5e998b33-45b0-4ca7-9369-bbd283eba2ec
[I 2025-06-15 20:32:42,933] Trial 0 finished with value: 0.031542510287765176 and parameters: {'n_estimators': 275, 'learning_rate': 0.06517738266905104, 'num_leaves': 210, 'max_depth': 7, 'min_child_samples': 1, 'subsample': 0.7255884365836754, 'colsample_bytree': 0.865813837944219, 'reg_alpha': 2.1954547919167924, 'reg_lambda': 4.519146845075415, 'min_split_gain': 0.014771790099974137}. Best is trial 0 with value: 0.031542510287765176.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         595
1           Price_lag_2         577
3         Load_DA_lag_0         271
16  roll2d_median_Price         246
14      pct_chg_Load_DA         242
2           Price_lag_7         210
11           NGas_lag_2         180
5       WindOn_DA_lag_0         179
15       lag168_Load_DA         151
12            Oil_lag_2         133
10           Coal_lag_2         129
13            EUA_lag_2         103
9                  WD_7          16
7                  WD_1          15
8                  WD_6           7
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          36
2           Price_lag_7           5
1           Price_lag_2           3
15       lag168_Load_DA           2
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
16  roll2d_median_Price           0
-> 70.18% done
********************* END NS.. 512 ****************************************************
********************* START NS ... 513 ****************************************************
[I 2025-06-15 20:32:44,913] A new study created in memory with name: no-name-0b4b6e0f-1a85-487a-9230-18ff7b9bf61e
[I 2025-06-15 20:32:45,951] Trial 0 finished with value: 0.0029585868441671832 and parameters: {'n_estimators': 242, 'learning_rate': 0.031053462337614024, 'num_leaves': 87, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.916210734485178, 'colsample_bytree': 0.6962269762231195, 'reg_alpha': 7.899020661869636, 'reg_lambda': 6.112875188963035, 'min_split_gain': 0.010733433675010451}. Best is trial 0 with value: 0.0029585868441671832.
[Trial 0] New best: 0.00296
[I 2025-06-15 20:32:46,569] Trial 1 finished with value: 0.0026852181119479995 and parameters: {'n_estimators': 187, 'learning_rate': 0.17315407995502985, 'num_leaves': 93, 'max_depth': 10, 'min_child_samples': 1, 'subsample': 0.6498395433823073, 'colsample_bytree': 0.6712993589027303, 'reg_alpha': 4.11035593790697, 'reg_lambda': 7.612039647275156, 'min_split_gain': 0.020366387690888273}. Best is trial 1 with value: 0.0026852181119479995.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:32:48,335] Trial 2 finished with value: 0.0028888776084063975 and parameters: {'n_estimators': 164, 'learning_rate': 0.04092237543607433, 'num_leaves': 245, 'max_depth': 9, 'min_child_samples': 24, 'subsample': 0.9299372855581635, 'colsample_bytree': 0.6491730814197887, 'reg_alpha': 1.2527536251569504, 'reg_lambda': 1.7732783358781268, 'min_split_gain': 0.03885840198572203}. Best is trial 1 with value: 0.0026852181119479995.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:32:48,729] Trial 3 finished with value: 0.0026401626801314877 and parameters: {'n_estimators': 106, 'learning_rate': 0.22434389539660735, 'num_leaves': 253, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.9370460792444211, 'colsample_bytree': 0.6477595309238042, 'reg_alpha': 4.488687628172812, 'reg_lambda': 4.702400827385992, 'min_split_gain': 0.049131370936607174}. Best is trial 3 with value: 0.0026401626801314877.
[I 2025-06-15 20:32:48,732] A new study created in memory with name: no-name-c257356c-eacb-440e-8ae6-0e4a076bc273
[I 2025-06-15 20:32:48,770] Trial 0 finished with value: 0.023150135990583267 and parameters: {'n_estimators': 94, 'learning_rate': 0.020657229360202655, 'num_leaves': 65, 'max_depth': 5, 'min_child_samples': 3, 'subsample': 0.8780282406737141, 'colsample_bytree': 0.5613278830855978, 'reg_alpha': 1.5202864193633248, 'reg_lambda': 9.913390845510225, 'min_split_gain': 0.04605213889305409}. Best is trial 0 with value: 0.023150135990583267.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         241
1           Price_lag_2         130
14      pct_chg_Load_DA         113
3         Load_DA_lag_0          78
5       WindOn_DA_lag_0          62
11           NGas_lag_2          61
2           Price_lag_7          60
15       lag168_Load_DA          51
16  roll2d_median_Price          47
10           Coal_lag_2          31
13            EUA_lag_2          19
12            Oil_lag_2          11
9                  WD_7           4
8                  WD_6           4
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          50
1           Price_lag_2          25
9                  WD_7           2
2           Price_lag_7           1
3         Load_DA_lag_0           1
11           NGas_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 70.31% done
********************* END NS.. 513 ****************************************************
********************* START NS ... 514 ****************************************************
[I 2025-06-15 20:32:50,201] A new study created in memory with name: no-name-46c7563a-a607-4215-b1cf-5ac29e92dde2
[I 2025-06-15 20:32:51,047] Trial 0 finished with value: 0.0012800892066734896 and parameters: {'n_estimators': 295, 'learning_rate': 0.12033974869291836, 'num_leaves': 84, 'max_depth': 8, 'min_child_samples': 10, 'subsample': 0.6659452870376668, 'colsample_bytree': 0.8353876074831225, 'reg_alpha': 2.3719788964118873, 'reg_lambda': 5.707981248430566, 'min_split_gain': 0.015729822041917797}. Best is trial 0 with value: 0.0012800892066734896.
[Trial 0] New best: 0.00128
[I 2025-06-15 20:32:51,414] Trial 1 finished with value: 0.001184399361987429 and parameters: {'n_estimators': 251, 'learning_rate': 0.17024149550414075, 'num_leaves': 33, 'max_depth': 8, 'min_child_samples': 24, 'subsample': 0.9309536688599659, 'colsample_bytree': 0.696848396760011, 'reg_alpha': 9.604964056605446, 'reg_lambda': 1.3071279020909554, 'min_split_gain': 0.04827733909051765}. Best is trial 1 with value: 0.001184399361987429.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:32:52,103] Trial 2 finished with value: 0.0011700090215223844 and parameters: {'n_estimators': 134, 'learning_rate': 0.07572615726604669, 'num_leaves': 65, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.783886440637136, 'colsample_bytree': 0.5288397297794063, 'reg_alpha': 0.08910893504202044, 'reg_lambda': 3.6168622254550153, 'min_split_gain': 0.029955099847290535}. Best is trial 2 with value: 0.0011700090215223844.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:32:52,591] Trial 3 finished with value: 0.0009982418222008566 and parameters: {'n_estimators': 160, 'learning_rate': 0.08930242392343786, 'num_leaves': 194, 'max_depth': 12, 'min_child_samples': 6, 'subsample': 0.887048806037797, 'colsample_bytree': 0.8204654850719242, 'reg_alpha': 9.77267044052988, 'reg_lambda': 9.288744296735494, 'min_split_gain': 0.04913745788806456}. Best is trial 3 with value: 0.0009982418222008566.
[I 2025-06-15 20:32:52,593] A new study created in memory with name: no-name-f5a5fb86-f2c7-4c9a-a4b9-8a26017e27fe
[I 2025-06-15 20:32:52,662] Trial 0 finished with value: 0.0050327166536695426 and parameters: {'n_estimators': 287, 'learning_rate': 0.22141156881955842, 'num_leaves': 36, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.7275238351490144, 'colsample_bytree': 0.6631360417657977, 'reg_alpha': 4.856661302372061, 'reg_lambda': 5.523452422062945, 'min_split_gain': 0.0127631316794667}. Best is trial 0 with value: 0.0050327166536695426.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         491
1           Price_lag_2         283
14      pct_chg_Load_DA         272
11           NGas_lag_2         146
3         Load_DA_lag_0         132
2           Price_lag_7         123
16  roll2d_median_Price         121
15       lag168_Load_DA          82
10           Coal_lag_2          69
13            EUA_lag_2          65
5       WindOn_DA_lag_0          53
12            Oil_lag_2          31
9                  WD_7          14
8                  WD_6           9
7                  WD_1           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 70.45% done
********************* END NS.. 514 ****************************************************
********************* START NS ... 515 ****************************************************
[I 2025-06-15 20:32:54,363] A new study created in memory with name: no-name-0a60c8c2-da69-4db6-944a-e2ccc071ee04
[I 2025-06-15 20:32:55,078] Trial 0 finished with value: 0.00561914471188229 and parameters: {'n_estimators': 81, 'learning_rate': 0.04421356486740641, 'num_leaves': 166, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.9381700729395632, 'colsample_bytree': 0.9725244591332864, 'reg_alpha': 7.550741925466645, 'reg_lambda': 4.629043550272539, 'min_split_gain': 0.044326319592073594}. Best is trial 0 with value: 0.00561914471188229.
[Trial 0] New best: 0.00562
[I 2025-06-15 20:32:56,523] Trial 1 finished with value: 0.004884877129764335 and parameters: {'n_estimators': 158, 'learning_rate': 0.03428944252446431, 'num_leaves': 248, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.5768501084478546, 'colsample_bytree': 0.727759399161443, 'reg_alpha': 1.716974375188619, 'reg_lambda': 4.621445546440167, 'min_split_gain': 0.03473184719858787}. Best is trial 1 with value: 0.004884877129764335.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:32:56,948] Trial 2 finished with value: 0.004810888439796623 and parameters: {'n_estimators': 209, 'learning_rate': 0.21515317079207294, 'num_leaves': 220, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.5085031005769733, 'colsample_bytree': 0.6847137144009885, 'reg_alpha': 0.8265132320340962, 'reg_lambda': 4.785757972328849, 'min_split_gain': 0.03830866274194141}. Best is trial 2 with value: 0.004810888439796623.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:32:58,000] Trial 3 finished with value: 0.004977739350570945 and parameters: {'n_estimators': 149, 'learning_rate': 0.04103338758191474, 'num_leaves': 208, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.9879280857221708, 'colsample_bytree': 0.9307215987767872, 'reg_alpha': 4.934612112659557, 'reg_lambda': 5.148572074445844, 'min_split_gain': 0.02155147706341939}. Best is trial 2 with value: 0.004810888439796623.
[I 2025-06-15 20:32:58,002] A new study created in memory with name: no-name-dfd70c06-0fd7-4142-ba14-17b9756994f1
[I 2025-06-15 20:32:58,052] Trial 0 finished with value: 0.02938859075694335 and parameters: {'n_estimators': 165, 'learning_rate': 0.028776807890597236, 'num_leaves': 126, 'max_depth': 10, 'min_child_samples': 4, 'subsample': 0.9622642450341385, 'colsample_bytree': 0.5864157103012932, 'reg_alpha': 9.732747960688688, 'reg_lambda': 1.3902827420480146, 'min_split_gain': 0.0457918958430447}. Best is trial 0 with value: 0.02938859075694335.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         262
1           Price_lag_2         165
14      pct_chg_Load_DA          96
3         Load_DA_lag_0          95
11           NGas_lag_2          72
10           Coal_lag_2          72
16  roll2d_median_Price          65
5       WindOn_DA_lag_0          55
2           Price_lag_7          52
15       lag168_Load_DA          47
12            Oil_lag_2          12
13            EUA_lag_2          10
7                  WD_1           6
9                  WD_7           4
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 70.59% done
********************* END NS.. 515 ****************************************************
********************* START NS ... 516 ****************************************************
[I 2025-06-15 20:32:59,866] A new study created in memory with name: no-name-8b90da71-a0b1-4352-92bf-06670e31a9dc
[I 2025-06-15 20:33:00,074] Trial 0 finished with value: 0.010520934292162692 and parameters: {'n_estimators': 79, 'learning_rate': 0.23774541390333978, 'num_leaves': 45, 'max_depth': 8, 'min_child_samples': 20, 'subsample': 0.7390793309589474, 'colsample_bytree': 0.8787576089642564, 'reg_alpha': 4.255945452560889, 'reg_lambda': 0.2195013971141635, 'min_split_gain': 0.04578375461372175}. Best is trial 0 with value: 0.010520934292162692.
[Trial 0] New best: 0.01052
[I 2025-06-15 20:33:00,862] Trial 1 finished with value: 0.010734580544378383 and parameters: {'n_estimators': 106, 'learning_rate': 0.03662068623696868, 'num_leaves': 50, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.5064722400003275, 'colsample_bytree': 0.5392383882559993, 'reg_alpha': 9.531619877700749, 'reg_lambda': 5.164881123758017, 'min_split_gain': 0.04593891208364324}. Best is trial 0 with value: 0.010520934292162692.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:33:02,282] Trial 2 finished with value: 0.009444221316144532 and parameters: {'n_estimators': 179, 'learning_rate': 0.05731890872985024, 'num_leaves': 168, 'max_depth': 7, 'min_child_samples': 11, 'subsample': 0.6226685839674999, 'colsample_bytree': 0.7576467185528253, 'reg_alpha': 0.4146489779006546, 'reg_lambda': 1.3862426184328636, 'min_split_gain': 0.019525609072019758}. Best is trial 2 with value: 0.009444221316144532.
[Trial 2] New best: 0.00944
[I 2025-06-15 20:33:03,685] Trial 3 finished with value: 0.06616027800274261 and parameters: {'n_estimators': 86, 'learning_rate': 0.01216385367632577, 'num_leaves': 121, 'max_depth': 14, 'min_child_samples': 8, 'subsample': 0.5886764956915269, 'colsample_bytree': 0.505131890226448, 'reg_alpha': 1.9317562591690518, 'reg_lambda': 8.780132451478572, 'min_split_gain': 0.009206378082049127}. Best is trial 2 with value: 0.009444221316144532.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:33:04,283] Trial 4 finished with value: 0.02841642383890115 and parameters: {'n_estimators': 57, 'learning_rate': 0.028486512739853694, 'num_leaves': 214, 'max_depth': 12, 'min_child_samples': 4, 'subsample': 0.5170246766108212, 'colsample_bytree': 0.94995447430931, 'reg_alpha': 9.691606834706523, 'reg_lambda': 6.05065563688715, 'min_split_gain': 0.03436028399858773}. Best is trial 2 with value: 0.009444221316144532.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:33:05,499] Trial 5 finished with value: 0.010056582272138763 and parameters: {'n_estimators': 138, 'learning_rate': 0.03434205246945889, 'num_leaves': 209, 'max_depth': 9, 'min_child_samples': 10, 'subsample': 0.9051020906044951, 'colsample_bytree': 0.9742707537176999, 'reg_alpha': 3.7842315233570343, 'reg_lambda': 9.059955207201806, 'min_split_gain': 0.04719310644920721}. Best is trial 2 with value: 0.009444221316144532.
[I 2025-06-15 20:33:05,501] A new study created in memory with name: no-name-3254f399-2d23-4d5c-bb26-a346aad7cb6d
[I 2025-06-15 20:33:05,561] Trial 0 finished with value: 0.06626034095040491 and parameters: {'n_estimators': 288, 'learning_rate': 0.17955513697548248, 'num_leaves': 36, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.6459337072127435, 'colsample_bytree': 0.8154843741489344, 'reg_alpha': 7.895680056603702, 'reg_lambda': 2.38314640055897, 'min_split_gain': 0.01825018833783298}. Best is trial 0 with value: 0.06626034095040491.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1519
1           Price_lag_2        1360
14      pct_chg_Load_DA        1173
3         Load_DA_lag_0         806
5       WindOn_DA_lag_0         661
2           Price_lag_7         626
16  roll2d_median_Price         621
11           NGas_lag_2         564
10           Coal_lag_2         508
15       lag168_Load_DA         489
13            EUA_lag_2         350
12            Oil_lag_2         332
9                  WD_7          41
7                  WD_1          34
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 70.73% done
********************* END NS.. 516 ****************************************************
********************* START NS ... 517 ****************************************************
[I 2025-06-15 20:33:08,531] A new study created in memory with name: no-name-e9a431f7-3138-4cac-b92a-29f66f82ebb4
[I 2025-06-15 20:33:11,789] Trial 0 finished with value: 0.009674442330308846 and parameters: {'n_estimators': 265, 'learning_rate': 0.05656720468221653, 'num_leaves': 150, 'max_depth': 9, 'min_child_samples': 1, 'subsample': 0.5225528856973611, 'colsample_bytree': 0.7243864214660873, 'reg_alpha': 1.7411364172121802, 'reg_lambda': 1.509048117814582, 'min_split_gain': 0.007828012217373032}. Best is trial 0 with value: 0.009674442330308846.
[Trial 0] New best: 0.00967
[I 2025-06-15 20:33:16,474] Trial 1 finished with value: 0.009494356299839671 and parameters: {'n_estimators': 250, 'learning_rate': 0.021883229805438418, 'num_leaves': 156, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.6290405525476619, 'colsample_bytree': 0.9586465467830255, 'reg_alpha': 1.534282437454032, 'reg_lambda': 1.8159924872561273, 'min_split_gain': 0.006955281954306481}. Best is trial 1 with value: 0.009494356299839671.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:33:17,232] Trial 2 finished with value: 0.009532125002068329 and parameters: {'n_estimators': 234, 'learning_rate': 0.07351367413175848, 'num_leaves': 177, 'max_depth': 6, 'min_child_samples': 23, 'subsample': 0.7225059767959283, 'colsample_bytree': 0.5165965530734422, 'reg_alpha': 5.974464543830589, 'reg_lambda': 4.202321794098528, 'min_split_gain': 0.036596307805901854}. Best is trial 1 with value: 0.009494356299839671.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:33:18,214] Trial 3 finished with value: 0.010863541978002128 and parameters: {'n_estimators': 138, 'learning_rate': 0.04034045567049202, 'num_leaves': 131, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.9553526070700323, 'colsample_bytree': 0.5442411653766586, 'reg_alpha': 8.940297007305164, 'reg_lambda': 5.652219703001763, 'min_split_gain': 0.040133750332887866}. Best is trial 1 with value: 0.009494356299839671.
[I 2025-06-15 20:33:18,217] A new study created in memory with name: no-name-439e34f0-3952-4b23-a9a9-aea72f7ed111
[I 2025-06-15 20:33:18,277] Trial 0 finished with value: 0.044868378824419075 and parameters: {'n_estimators': 77, 'learning_rate': 0.0132132820114527, 'num_leaves': 236, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.981320243149824, 'colsample_bytree': 0.5531256613538414, 'reg_alpha': 1.4823672621159845, 'reg_lambda': 0.08633178420381804, 'min_split_gain': 0.026868007874594077}. Best is trial 0 with value: 0.044868378824419075.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        4283
1           Price_lag_2        3641
14      pct_chg_Load_DA        3223
3         Load_DA_lag_0        1808
16  roll2d_median_Price        1546
2           Price_lag_7        1493
11           NGas_lag_2        1367
15       lag168_Load_DA        1337
13            EUA_lag_2        1105
5       WindOn_DA_lag_0        1101
10           Coal_lag_2         817
12            Oil_lag_2         816
9                  WD_7          60
7                  WD_1          40
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         138
11           NGas_lag_2          19
15       lag168_Load_DA          14
3         Load_DA_lag_0          10
16  roll2d_median_Price           7
10           Coal_lag_2           5
2           Price_lag_7           4
1           Price_lag_2           3
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
4        Solar_DA_lag_0           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 70.86% done
********************* END NS.. 517 ****************************************************
********************* START NS ... 518 ****************************************************
[I 2025-06-15 20:33:21,550] A new study created in memory with name: no-name-e07949f3-b395-472f-9acb-357111c63197
[I 2025-06-15 20:33:21,786] Trial 0 finished with value: 0.006643717963431682 and parameters: {'n_estimators': 144, 'learning_rate': 0.14341208374966616, 'num_leaves': 114, 'max_depth': 9, 'min_child_samples': 20, 'subsample': 0.5070438683632496, 'colsample_bytree': 0.8215131918544565, 'reg_alpha': 4.742160505598028, 'reg_lambda': 0.7893392791085119, 'min_split_gain': 0.035647236886195054}. Best is trial 0 with value: 0.006643717963431682.
[Trial 0] New best: 0.00664
[I 2025-06-15 20:33:22,208] Trial 1 finished with value: 0.006471745229978708 and parameters: {'n_estimators': 228, 'learning_rate': 0.07680684609348876, 'num_leaves': 218, 'max_depth': 14, 'min_child_samples': 17, 'subsample': 0.6595980119890285, 'colsample_bytree': 0.880272933689664, 'reg_alpha': 6.2736265629321135, 'reg_lambda': 3.4799881812372746, 'min_split_gain': 0.01222829434962346}. Best is trial 1 with value: 0.006471745229978708.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:33:23,115] Trial 2 finished with value: 0.008318148567518377 and parameters: {'n_estimators': 231, 'learning_rate': 0.013950352210359205, 'num_leaves': 216, 'max_depth': 8, 'min_child_samples': 10, 'subsample': 0.972037553809519, 'colsample_bytree': 0.948252548843968, 'reg_alpha': 4.690507149991413, 'reg_lambda': 2.224287327312502, 'min_split_gain': 0.0036662766384984616}. Best is trial 1 with value: 0.006471745229978708.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:33:24,341] Trial 3 finished with value: 0.006274547583233803 and parameters: {'n_estimators': 248, 'learning_rate': 0.024725197332031778, 'num_leaves': 99, 'max_depth': 9, 'min_child_samples': 3, 'subsample': 0.6425264853061684, 'colsample_bytree': 0.8263892958914225, 'reg_alpha': 3.0070562229453444, 'reg_lambda': 3.1851100063264948, 'min_split_gain': 0.016663204096101086}. Best is trial 3 with value: 0.006274547583233803.
[I 2025-06-15 20:33:24,344] A new study created in memory with name: no-name-71615dc7-dfbc-4b31-acb4-9e85f6e0bc7b
[I 2025-06-15 20:33:24,453] Trial 0 finished with value: 0.009610937308519576 and parameters: {'n_estimators': 272, 'learning_rate': 0.015042964464091797, 'num_leaves': 162, 'max_depth': 14, 'min_child_samples': 4, 'subsample': 0.8550741166639104, 'colsample_bytree': 0.5857342883029121, 'reg_alpha': 0.2896139274288301, 'reg_lambda': 7.910503732222515, 'min_split_gain': 0.029584663535330208}. Best is trial 0 with value: 0.009610937308519576.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3024
1           Price_lag_2        2846
14      pct_chg_Load_DA        2194
3         Load_DA_lag_0        1281
2           Price_lag_7        1253
11           NGas_lag_2        1250
16  roll2d_median_Price        1155
5       WindOn_DA_lag_0         905
15       lag168_Load_DA         873
13            EUA_lag_2         667
12            Oil_lag_2         656
10           Coal_lag_2         570
9                  WD_7          51
8                  WD_6          39
7                  WD_1          24
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         311
3         Load_DA_lag_0          61
16  roll2d_median_Price          47
2           Price_lag_7          44
15       lag168_Load_DA          38
1           Price_lag_2          33
5       WindOn_DA_lag_0          24
11           NGas_lag_2          19
10           Coal_lag_2          17
14      pct_chg_Load_DA           3
9                  WD_7           2
12            Oil_lag_2           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
13            EUA_lag_2           0
8                  WD_6           0
-> 71.00% done
********************* END NS.. 518 ****************************************************
********************* START NS ... 519 ****************************************************
[I 2025-06-15 20:33:26,837] A new study created in memory with name: no-name-07429add-c7c0-4488-895e-25dacc768527
[I 2025-06-15 20:33:27,300] Trial 0 finished with value: 0.004981052717164367 and parameters: {'n_estimators': 137, 'learning_rate': 0.034368723770506336, 'num_leaves': 69, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.5361910222788511, 'colsample_bytree': 0.8619109344960794, 'reg_alpha': 9.986535762446495, 'reg_lambda': 0.7208446626916354, 'min_split_gain': 0.04933096576547998}. Best is trial 0 with value: 0.004981052717164367.
[Trial 0] New best: 0.00498
[I 2025-06-15 20:33:27,702] Trial 1 finished with value: 0.004153511566490349 and parameters: {'n_estimators': 124, 'learning_rate': 0.05463006516681498, 'num_leaves': 37, 'max_depth': 7, 'min_child_samples': 27, 'subsample': 0.6308912838064672, 'colsample_bytree': 0.8755344650246034, 'reg_alpha': 3.2636157534476142, 'reg_lambda': 1.3913032528608815, 'min_split_gain': 0.01361710977696281}. Best is trial 1 with value: 0.004153511566490349.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:33:28,634] Trial 2 finished with value: 0.006956562678991664 and parameters: {'n_estimators': 288, 'learning_rate': 0.019468481008378342, 'num_leaves': 57, 'max_depth': 14, 'min_child_samples': 26, 'subsample': 0.6639373557370509, 'colsample_bytree': 0.5566503876480293, 'reg_alpha': 5.443145040973779, 'reg_lambda': 3.436594643182591, 'min_split_gain': 0.03148856556645826}. Best is trial 1 with value: 0.004153511566490349.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:33:28,971] Trial 3 finished with value: 0.004541178125814145 and parameters: {'n_estimators': 285, 'learning_rate': 0.10265054169757651, 'num_leaves': 103, 'max_depth': 5, 'min_child_samples': 8, 'subsample': 0.8736791356173738, 'colsample_bytree': 0.785888002723697, 'reg_alpha': 4.471870787245202, 'reg_lambda': 9.073764788803645, 'min_split_gain': 0.03122720160193392}. Best is trial 1 with value: 0.004153511566490349.
[I 2025-06-15 20:33:28,973] A new study created in memory with name: no-name-c38441b4-ae36-48e7-881e-a2b7b5240b7b
[I 2025-06-15 20:33:28,993] Trial 0 finished with value: 0.06891651306669817 and parameters: {'n_estimators': 112, 'learning_rate': 0.27511082252737973, 'num_leaves': 204, 'max_depth': 16, 'min_child_samples': 17, 'subsample': 0.9706171066991238, 'colsample_bytree': 0.9117968577283746, 'reg_alpha': 6.488341712034865, 'reg_lambda': 4.393538582240209, 'min_split_gain': 0.03497600311903632}. Best is trial 0 with value: 0.06891651306669817.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1102
1           Price_lag_2         740
14      pct_chg_Load_DA         660
3         Load_DA_lag_0         366
11           NGas_lag_2         300
2           Price_lag_7         235
16  roll2d_median_Price         228
15       lag168_Load_DA         194
5       WindOn_DA_lag_0         182
10           Coal_lag_2         132
13            EUA_lag_2         124
12            Oil_lag_2         116
7                  WD_1          15
9                  WD_7           9
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           3
3         Load_DA_lag_0           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 71.14% done
********************* END NS.. 519 ****************************************************
********************* START NS ... 520 ****************************************************
[I 2025-06-15 20:33:29,961] A new study created in memory with name: no-name-ba29e5fc-d9b7-4c74-a6ea-b45b16802aa6
[I 2025-06-15 20:33:30,266] Trial 0 finished with value: 0.0043874405256462195 and parameters: {'n_estimators': 262, 'learning_rate': 0.1107435914470032, 'num_leaves': 227, 'max_depth': 13, 'min_child_samples': 27, 'subsample': 0.9826856699999129, 'colsample_bytree': 0.5258729042317182, 'reg_alpha': 4.033857106697506, 'reg_lambda': 6.847023644316122, 'min_split_gain': 0.03126456800593869}. Best is trial 0 with value: 0.0043874405256462195.
[Trial 0] New best: 0.00439
[I 2025-06-15 20:33:30,544] Trial 1 finished with value: 0.003681814015910089 and parameters: {'n_estimators': 102, 'learning_rate': 0.2987618801130975, 'num_leaves': 134, 'max_depth': 16, 'min_child_samples': 21, 'subsample': 0.7822159310909376, 'colsample_bytree': 0.5679501670471182, 'reg_alpha': 0.44492391688577215, 'reg_lambda': 3.10651180514001, 'min_split_gain': 0.01185183179173393}. Best is trial 1 with value: 0.003681814015910089.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:33:33,503] Trial 2 finished with value: 0.004114070609021336 and parameters: {'n_estimators': 244, 'learning_rate': 0.01659444552577857, 'num_leaves': 191, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.9235748249643566, 'colsample_bytree': 0.8448641713225298, 'reg_alpha': 0.27826467752013806, 'reg_lambda': 1.9932864988491705, 'min_split_gain': 0.04699099136913712}. Best is trial 1 with value: 0.003681814015910089.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:33:33,983] Trial 3 finished with value: 0.0037589951564847394 and parameters: {'n_estimators': 231, 'learning_rate': 0.0896715630002689, 'num_leaves': 80, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.9223526961211991, 'colsample_bytree': 0.817080466836547, 'reg_alpha': 3.3863209217782675, 'reg_lambda': 1.4185076601252, 'min_split_gain': 0.019955368977999197}. Best is trial 1 with value: 0.003681814015910089.
[I 2025-06-15 20:33:33,986] A new study created in memory with name: no-name-8a7bbaa8-2b7e-4553-9499-e63ee8f8fd75
[I 2025-06-15 20:33:34,070] Trial 0 finished with value: 0.030040305162858966 and parameters: {'n_estimators': 258, 'learning_rate': 0.021514107356962495, 'num_leaves': 180, 'max_depth': 14, 'min_child_samples': 9, 'subsample': 0.6654005401868062, 'colsample_bytree': 0.507253412234438, 'reg_alpha': 0.2638106506660065, 'reg_lambda': 1.108548147857643, 'min_split_gain': 0.025544572736500456}. Best is trial 0 with value: 0.030040305162858966.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         485
1           Price_lag_2         461
3         Load_DA_lag_0         309
5       WindOn_DA_lag_0         300
2           Price_lag_7         280
14      pct_chg_Load_DA         261
15       lag168_Load_DA         246
11           NGas_lag_2         203
16  roll2d_median_Price         195
10           Coal_lag_2         192
13            EUA_lag_2         170
12            Oil_lag_2         120
9                  WD_7          10
7                  WD_1           7
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         184
3         Load_DA_lag_0          74
1           Price_lag_2          39
2           Price_lag_7          39
16  roll2d_median_Price          26
14      pct_chg_Load_DA          18
9                  WD_7          15
11           NGas_lag_2          15
15       lag168_Load_DA          15
5       WindOn_DA_lag_0           6
10           Coal_lag_2           2
12            Oil_lag_2           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
13            EUA_lag_2           0
8                  WD_6           0
-> 71.27% done
********************* END NS.. 520 ****************************************************
********************* START NS ... 521 ****************************************************
[I 2025-06-15 20:33:35,001] A new study created in memory with name: no-name-bf94a90d-c037-4193-8063-7aaec7e6ecc3
[I 2025-06-15 20:33:35,371] Trial 0 finished with value: 0.003897502754667336 and parameters: {'n_estimators': 73, 'learning_rate': 0.07642220427426434, 'num_leaves': 68, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.5436219736171061, 'colsample_bytree': 0.5783307891811084, 'reg_alpha': 1.6665814873491747, 'reg_lambda': 6.231485473080179, 'min_split_gain': 0.04355659723456128}. Best is trial 0 with value: 0.003897502754667336.
[Trial 0] New best: 0.00390
[I 2025-06-15 20:33:35,595] Trial 1 finished with value: 0.00404221954451273 and parameters: {'n_estimators': 54, 'learning_rate': 0.11832582431723675, 'num_leaves': 229, 'max_depth': 7, 'min_child_samples': 12, 'subsample': 0.9706829630058538, 'colsample_bytree': 0.5121149766613078, 'reg_alpha': 2.19140815639307, 'reg_lambda': 1.4022005365758272, 'min_split_gain': 0.015050188100462276}. Best is trial 0 with value: 0.003897502754667336.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:33:35,884] Trial 2 finished with value: 0.004106858604037081 and parameters: {'n_estimators': 259, 'learning_rate': 0.20215307560596335, 'num_leaves': 121, 'max_depth': 12, 'min_child_samples': 6, 'subsample': 0.6528000220655547, 'colsample_bytree': 0.9670697168054949, 'reg_alpha': 4.603500707939295, 'reg_lambda': 0.5440410297574694, 'min_split_gain': 0.03592185137327817}. Best is trial 0 with value: 0.003897502754667336.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:33:36,167] Trial 3 finished with value: 0.052355703348304444 and parameters: {'n_estimators': 88, 'learning_rate': 0.012974624305892381, 'num_leaves': 223, 'max_depth': 9, 'min_child_samples': 24, 'subsample': 0.6151941633631788, 'colsample_bytree': 0.5649901142949805, 'reg_alpha': 9.485652120124415, 'reg_lambda': 7.152172985733292, 'min_split_gain': 0.029297353204050677}. Best is trial 0 with value: 0.003897502754667336.
[I 2025-06-15 20:33:36,169] A new study created in memory with name: no-name-935f52b1-54b8-4a14-80ba-0f4f838c92b9
[I 2025-06-15 20:33:36,195] Trial 0 finished with value: 0.011563776904780757 and parameters: {'n_estimators': 118, 'learning_rate': 0.0838073636124264, 'num_leaves': 87, 'max_depth': 11, 'min_child_samples': 14, 'subsample': 0.7478400899345512, 'colsample_bytree': 0.6192121680236147, 'reg_alpha': 5.124541745194971, 'reg_lambda': 3.582240165664796, 'min_split_gain': 0.033092740671247754}. Best is trial 0 with value: 0.011563776904780757.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         781
0           Price_lag_1         722
14      pct_chg_Load_DA         404
3         Load_DA_lag_0         323
15       lag168_Load_DA         252
2           Price_lag_7         229
11           NGas_lag_2         228
16  roll2d_median_Price         210
5       WindOn_DA_lag_0         200
10           Coal_lag_2         162
13            EUA_lag_2         155
12            Oil_lag_2         108
9                  WD_7          16
7                  WD_1           7
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          20
3         Load_DA_lag_0          11
15       lag168_Load_DA           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 71.41% done
********************* END NS.. 521 ****************************************************
********************* START NS ... 522 ****************************************************
[I 2025-06-15 20:33:36,999] A new study created in memory with name: no-name-ddac11a9-3cd5-4c73-8893-1c0724333263
[I 2025-06-15 20:33:37,776] Trial 0 finished with value: 0.0066554799289959415 and parameters: {'n_estimators': 185, 'learning_rate': 0.057339049174149066, 'num_leaves': 186, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.5745454282610398, 'colsample_bytree': 0.8438541918014103, 'reg_alpha': 1.360095585391472, 'reg_lambda': 6.929366848653457, 'min_split_gain': 0.01889020505441066}. Best is trial 0 with value: 0.0066554799289959415.
[Trial 0] New best: 0.00666
[I 2025-06-15 20:33:38,128] Trial 1 finished with value: 0.006587221514652237 and parameters: {'n_estimators': 97, 'learning_rate': 0.176966537700267, 'num_leaves': 58, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.6213927994264188, 'colsample_bytree': 0.8253313604333357, 'reg_alpha': 2.517962061022454, 'reg_lambda': 1.8334677183187675, 'min_split_gain': 0.01256550284316535}. Best is trial 1 with value: 0.006587221514652237.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:33:38,660] Trial 2 finished with value: 0.005730681914369704 and parameters: {'n_estimators': 222, 'learning_rate': 0.056149257639059674, 'num_leaves': 117, 'max_depth': 5, 'min_child_samples': 7, 'subsample': 0.8216030434402501, 'colsample_bytree': 0.7811526225175762, 'reg_alpha': 1.4330842523567011, 'reg_lambda': 6.822543931805646, 'min_split_gain': 0.013570187806981765}. Best is trial 2 with value: 0.005730681914369704.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:33:38,993] Trial 3 finished with value: 0.005854403439764279 and parameters: {'n_estimators': 202, 'learning_rate': 0.09365289631278706, 'num_leaves': 112, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.5246040843748128, 'colsample_bytree': 0.5858016143480964, 'reg_alpha': 9.201220621772496, 'reg_lambda': 2.742935532306956, 'min_split_gain': 0.044586282787964115}. Best is trial 2 with value: 0.005730681914369704.
[I 2025-06-15 20:33:38,996] A new study created in memory with name: no-name-155a5cd4-1605-4577-a958-df6cea3f431e
[I 2025-06-15 20:33:39,018] Trial 0 finished with value: 0.006453057506134774 and parameters: {'n_estimators': 59, 'learning_rate': 0.03228287047616589, 'num_leaves': 99, 'max_depth': 9, 'min_child_samples': 20, 'subsample': 0.9843764149963621, 'colsample_bytree': 0.7770865660911015, 'reg_alpha': 4.559947580675305, 'reg_lambda': 3.7373012891980038, 'min_split_gain': 0.007403798802141032}. Best is trial 0 with value: 0.006453057506134774.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1141
1           Price_lag_2         797
14      pct_chg_Load_DA         671
3         Load_DA_lag_0         481
11           NGas_lag_2         387
15       lag168_Load_DA         349
5       WindOn_DA_lag_0         343
16  roll2d_median_Price         335
2           Price_lag_7         323
10           Coal_lag_2         212
13            EUA_lag_2         181
12            Oil_lag_2         130
7                  WD_1          27
9                  WD_7          20
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          95
3         Load_DA_lag_0          19
15       lag168_Load_DA          17
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 71.55% done
********************* END NS.. 522 ****************************************************
********************* START NS ... 523 ****************************************************
[I 2025-06-15 20:33:40,232] A new study created in memory with name: no-name-8999000b-1a69-409e-9829-a3e9e9b3339a
[I 2025-06-15 20:33:40,583] Trial 0 finished with value: 0.0046924187684420065 and parameters: {'n_estimators': 84, 'learning_rate': 0.14156878481794655, 'num_leaves': 160, 'max_depth': 13, 'min_child_samples': 8, 'subsample': 0.71594827266266, 'colsample_bytree': 0.7074801975828938, 'reg_alpha': 7.883476874235886, 'reg_lambda': 1.4622902279903305, 'min_split_gain': 0.01051577746937445}. Best is trial 0 with value: 0.0046924187684420065.
[I 2025-06-15 20:33:40,729] Trial 1 finished with value: 0.00517112385988256 and parameters: {'n_estimators': 78, 'learning_rate': 0.20125125368207292, 'num_leaves': 137, 'max_depth': 5, 'min_child_samples': 16, 'subsample': 0.690148695282276, 'colsample_bytree': 0.5115816795607868, 'reg_alpha': 8.20973795676285, 'reg_lambda': 3.441694846360882, 'min_split_gain': 0.01952146179655939}. Best is trial 0 with value: 0.0046924187684420065.
[Trial 0] New best: 0.00469
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:33:41,808] Trial 2 finished with value: 0.00464338252290846 and parameters: {'n_estimators': 278, 'learning_rate': 0.028344952869439347, 'num_leaves': 88, 'max_depth': 10, 'min_child_samples': 4, 'subsample': 0.6676233740326389, 'colsample_bytree': 0.7738430642537153, 'reg_alpha': 7.0559547072040925, 'reg_lambda': 4.200580577018669, 'min_split_gain': 0.008771171494847657}. Best is trial 2 with value: 0.00464338252290846.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:33:42,381] Trial 3 finished with value: 0.004210691886946747 and parameters: {'n_estimators': 247, 'learning_rate': 0.0479245813258518, 'num_leaves': 80, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.7350047419793584, 'colsample_bytree': 0.6994406046094094, 'reg_alpha': 6.049488885968043, 'reg_lambda': 0.9628466648086398, 'min_split_gain': 0.022277346496195356}. Best is trial 3 with value: 0.004210691886946747.
[I 2025-06-15 20:33:42,383] A new study created in memory with name: no-name-06789460-4cfa-4dc6-b46f-a4eb4f3e9a14
[I 2025-06-15 20:33:42,416] Trial 0 finished with value: 0.008632634177590895 and parameters: {'n_estimators': 300, 'learning_rate': 0.06049450976202475, 'num_leaves': 209, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.9865246383131695, 'colsample_bytree': 0.8938146014918386, 'reg_alpha': 7.3900909675933715, 'reg_lambda': 5.395426250661959, 'min_split_gain': 0.02673736350420656}. Best is trial 0 with value: 0.008632634177590895.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1166
1           Price_lag_2         895
14      pct_chg_Load_DA         727
3         Load_DA_lag_0         435
11           NGas_lag_2         360
15       lag168_Load_DA         327
16  roll2d_median_Price         312
2           Price_lag_7         309
5       WindOn_DA_lag_0         302
10           Coal_lag_2         207
12            Oil_lag_2         169
13            EUA_lag_2         142
7                  WD_1          28
9                  WD_7          28
8                  WD_6          20
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          10
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 71.68% done
********************* END NS.. 523 ****************************************************
********************* START NS ... 524 ****************************************************
[I 2025-06-15 20:33:43,406] A new study created in memory with name: no-name-357efa78-9e56-4550-8c53-e53115376153
[I 2025-06-15 20:33:43,708] Trial 0 finished with value: 0.00270919766662725 and parameters: {'n_estimators': 203, 'learning_rate': 0.057160241487548814, 'num_leaves': 148, 'max_depth': 6, 'min_child_samples': 5, 'subsample': 0.8768407865606407, 'colsample_bytree': 0.8215497153808777, 'reg_alpha': 4.723471022736236, 'reg_lambda': 0.23107260241577166, 'min_split_gain': 0.031217552755911787}. Best is trial 0 with value: 0.00270919766662725.
[Trial 0] New best: 0.00271
[I 2025-06-15 20:33:43,928] Trial 1 finished with value: 0.04744646787831752 and parameters: {'n_estimators': 69, 'learning_rate': 0.01906434958447297, 'num_leaves': 63, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.798394809366635, 'colsample_bytree': 0.7814801691195455, 'reg_alpha': 8.293263327424453, 'reg_lambda': 3.9888859452963366, 'min_split_gain': 0.03939890325156378}. Best is trial 0 with value: 0.00270919766662725.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:33:44,426] Trial 2 finished with value: 0.0063841570874841255 and parameters: {'n_estimators': 107, 'learning_rate': 0.025607091791592252, 'num_leaves': 35, 'max_depth': 11, 'min_child_samples': 3, 'subsample': 0.6716839361286675, 'colsample_bytree': 0.5058422173483367, 'reg_alpha': 2.1135186902438576, 'reg_lambda': 5.086615734620164, 'min_split_gain': 0.018013869537701465}. Best is trial 0 with value: 0.00270919766662725.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:33:45,005] Trial 3 finished with value: 0.05900614047849486 and parameters: {'n_estimators': 83, 'learning_rate': 0.014573546688892207, 'num_leaves': 39, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.6047508305321504, 'colsample_bytree': 0.7845694392651532, 'reg_alpha': 0.17696751997244298, 'reg_lambda': 4.960181823808884, 'min_split_gain': 0.009762754072429759}. Best is trial 0 with value: 0.00270919766662725.
[I 2025-06-15 20:33:45,008] A new study created in memory with name: no-name-99dc736c-d29a-4ed9-b899-2bc6edd6cfb2
[I 2025-06-15 20:33:45,050] Trial 0 finished with value: 0.004053968038108145 and parameters: {'n_estimators': 161, 'learning_rate': 0.02491221475344392, 'num_leaves': 123, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.7673028780835129, 'colsample_bytree': 0.733516697757651, 'reg_alpha': 2.5254755096614345, 'reg_lambda': 3.6617449014270944, 'min_split_gain': 0.039265957965027715}. Best is trial 0 with value: 0.004053968038108145.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         951
1           Price_lag_2         654
14      pct_chg_Load_DA         580
3         Load_DA_lag_0         369
11           NGas_lag_2         303
2           Price_lag_7         250
16  roll2d_median_Price         230
15       lag168_Load_DA         202
5       WindOn_DA_lag_0         200
10           Coal_lag_2         150
12            Oil_lag_2          87
13            EUA_lag_2          83
7                  WD_1          14
9                  WD_7          14
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          91
3         Load_DA_lag_0          31
1           Price_lag_2           9
10           Coal_lag_2           3
15       lag168_Load_DA           3
11           NGas_lag_2           2
12            Oil_lag_2           1
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 71.82% done
********************* END NS.. 524 ****************************************************
********************* START NS ... 525 ****************************************************
[I 2025-06-15 20:33:46,841] A new study created in memory with name: no-name-f6b85e5f-c5c5-43f6-88cf-2d299bef6090
[I 2025-06-15 20:33:47,011] Trial 0 finished with value: 0.00440296797344535 and parameters: {'n_estimators': 171, 'learning_rate': 0.11247266924350896, 'num_leaves': 69, 'max_depth': 6, 'min_child_samples': 20, 'subsample': 0.6404062682668378, 'colsample_bytree': 0.9696129732965618, 'reg_alpha': 1.5658197555262598, 'reg_lambda': 0.713197858706528, 'min_split_gain': 0.03590345341103159}. Best is trial 0 with value: 0.00440296797344535.
[Trial 0] New best: 0.00440
[I 2025-06-15 20:33:47,711] Trial 1 finished with value: 0.004549389648576936 and parameters: {'n_estimators': 188, 'learning_rate': 0.06783917838042333, 'num_leaves': 129, 'max_depth': 9, 'min_child_samples': 10, 'subsample': 0.6391969395093061, 'colsample_bytree': 0.9853414075119356, 'reg_alpha': 2.081265461661185, 'reg_lambda': 4.508345650897622, 'min_split_gain': 0.02769953715204872}. Best is trial 0 with value: 0.00440296797344535.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:33:47,998] Trial 2 finished with value: 0.00517142603371413 and parameters: {'n_estimators': 97, 'learning_rate': 0.22301114552553164, 'num_leaves': 184, 'max_depth': 8, 'min_child_samples': 14, 'subsample': 0.6913313758646318, 'colsample_bytree': 0.9708792697562941, 'reg_alpha': 7.961113654055783, 'reg_lambda': 5.825093785808722, 'min_split_gain': 0.007133545481480164}. Best is trial 0 with value: 0.00440296797344535.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:33:48,255] Trial 3 finished with value: 0.005299786132686728 and parameters: {'n_estimators': 228, 'learning_rate': 0.23199967520556708, 'num_leaves': 95, 'max_depth': 8, 'min_child_samples': 9, 'subsample': 0.9108530925250229, 'colsample_bytree': 0.7294192661679493, 'reg_alpha': 4.5656862917621135, 'reg_lambda': 9.251895226222066, 'min_split_gain': 0.034324085925442976}. Best is trial 0 with value: 0.00440296797344535.
[I 2025-06-15 20:33:48,257] A new study created in memory with name: no-name-1d0b5894-437b-41ff-bea9-dccb208c9872
[I 2025-06-15 20:33:48,294] Trial 0 finished with value: 0.02454199398120129 and parameters: {'n_estimators': 130, 'learning_rate': 0.1923414807797542, 'num_leaves': 119, 'max_depth': 8, 'min_child_samples': 13, 'subsample': 0.9051735882995187, 'colsample_bytree': 0.9110551265160568, 'reg_alpha': 3.030578821448114, 'reg_lambda': 8.632761557262082, 'min_split_gain': 0.005300086328600468}. Best is trial 0 with value: 0.02454199398120129.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         589
14      pct_chg_Load_DA         391
1           Price_lag_2         281
3         Load_DA_lag_0         218
11           NGas_lag_2         177
16  roll2d_median_Price         167
2           Price_lag_7         146
5       WindOn_DA_lag_0         114
15       lag168_Load_DA         109
10           Coal_lag_2          87
12            Oil_lag_2          50
13            EUA_lag_2          33
7                  WD_1          11
9                  WD_7           7
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          28
15       lag168_Load_DA           8
3         Load_DA_lag_0           2
10           Coal_lag_2           1
2           Price_lag_7           1
5       WindOn_DA_lag_0           1
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 71.96% done
********************* END NS.. 525 ****************************************************
********************* START NS ... 526 ****************************************************
[I 2025-06-15 20:33:50,224] A new study created in memory with name: no-name-1d80f4d4-c070-4fd4-ab3e-d3df90ed97aa
[I 2025-06-15 20:33:51,327] Trial 0 finished with value: 0.024317925193123452 and parameters: {'n_estimators': 172, 'learning_rate': 0.020538582641554913, 'num_leaves': 63, 'max_depth': 11, 'min_child_samples': 15, 'subsample': 0.7054908452212879, 'colsample_bytree': 0.633631400540269, 'reg_alpha': 6.235354632103512, 'reg_lambda': 7.402004241143231, 'min_split_gain': 0.03175008991844938}. Best is trial 0 with value: 0.024317925193123452.
[Trial 0] New best: 0.02432
[I 2025-06-15 20:33:51,651] Trial 1 finished with value: 0.010822080868965106 and parameters: {'n_estimators': 52, 'learning_rate': 0.2662920645165952, 'num_leaves': 248, 'max_depth': 10, 'min_child_samples': 15, 'subsample': 0.8665617475751154, 'colsample_bytree': 0.6565583655768126, 'reg_alpha': 0.11733735381264432, 'reg_lambda': 4.408670598087894, 'min_split_gain': 0.03265073328284068}. Best is trial 1 with value: 0.010822080868965106.
[Trial 1] New best: 0.01082
[I 2025-06-15 20:33:52,101] Trial 2 finished with value: 0.011228366712223324 and parameters: {'n_estimators': 141, 'learning_rate': 0.12128923731383294, 'num_leaves': 35, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.6297818468488305, 'colsample_bytree': 0.6414088462883711, 'reg_alpha': 0.4865361468388274, 'reg_lambda': 1.3974497794394125, 'min_split_gain': 0.02152340617211461}. Best is trial 1 with value: 0.010822080868965106.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:33:53,295] Trial 3 finished with value: 0.018637875819264054 and parameters: {'n_estimators': 172, 'learning_rate': 0.02308497036861297, 'num_leaves': 93, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.5181501059484092, 'colsample_bytree': 0.6369574025163078, 'reg_alpha': 2.0495150111977587, 'reg_lambda': 0.0051003756898537755, 'min_split_gain': 0.027927357685492177}. Best is trial 1 with value: 0.010822080868965106.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:33:53,650] Trial 4 finished with value: 0.06173726633604363 and parameters: {'n_estimators': 152, 'learning_rate': 0.012898569697245099, 'num_leaves': 169, 'max_depth': 4, 'min_child_samples': 8, 'subsample': 0.7237783007727596, 'colsample_bytree': 0.8374193992101238, 'reg_alpha': 9.933940496772484, 'reg_lambda': 7.692732497126942, 'min_split_gain': 0.029670315612984985}. Best is trial 1 with value: 0.010822080868965106.
[I 2025-06-15 20:33:53,653] A new study created in memory with name: no-name-f017819f-5e7a-4bbb-9b99-f3a830ca9bdc
[I 2025-06-15 20:33:53,684] Trial 0 finished with value: 0.14000419824850843 and parameters: {'n_estimators': 125, 'learning_rate': 0.29507589130048606, 'num_leaves': 86, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.9902583676239685, 'colsample_bytree': 0.8830508698899664, 'reg_alpha': 9.019335216103846, 'reg_lambda': 1.3222694830588466, 'min_split_gain': 0.03804240651510202}. Best is trial 0 with value: 0.14000419824850843.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         420
1           Price_lag_2         381
3         Load_DA_lag_0         239
14      pct_chg_Load_DA         218
11           NGas_lag_2         211
2           Price_lag_7         206
16  roll2d_median_Price         188
5       WindOn_DA_lag_0         166
10           Coal_lag_2         125
12            Oil_lag_2         124
15       lag168_Load_DA         120
13            EUA_lag_2         107
9                  WD_7           7
7                  WD_1           3
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           3
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 72.09% done
********************* END NS.. 526 ****************************************************
********************* START NS ... 527 ****************************************************
[I 2025-06-15 20:33:55,168] A new study created in memory with name: no-name-f2c4ccf4-9db0-4398-8941-2b11efe44c15
[I 2025-06-15 20:33:55,958] Trial 0 finished with value: 0.008702325285680435 and parameters: {'n_estimators': 158, 'learning_rate': 0.07777005754935931, 'num_leaves': 195, 'max_depth': 10, 'min_child_samples': 25, 'subsample': 0.5879330862197187, 'colsample_bytree': 0.7778552668625307, 'reg_alpha': 0.7528467265427297, 'reg_lambda': 9.764312243473121, 'min_split_gain': 0.009930968287008358}. Best is trial 0 with value: 0.008702325285680435.
[Trial 0] New best: 0.00870
[I 2025-06-15 20:33:56,294] Trial 1 finished with value: 0.008881498009675906 and parameters: {'n_estimators': 90, 'learning_rate': 0.1764855464275794, 'num_leaves': 218, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.7349002123965143, 'colsample_bytree': 0.7176914011217966, 'reg_alpha': 0.6419579576514645, 'reg_lambda': 0.35960000000459846, 'min_split_gain': 0.007033092222744897}. Best is trial 0 with value: 0.008702325285680435.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:33:56,992] Trial 2 finished with value: 0.013833082399635467 and parameters: {'n_estimators': 293, 'learning_rate': 0.015690885443930633, 'num_leaves': 80, 'max_depth': 5, 'min_child_samples': 11, 'subsample': 0.8373138724167731, 'colsample_bytree': 0.9368211887151734, 'reg_alpha': 8.436627936352627, 'reg_lambda': 7.64861588805489, 'min_split_gain': 2.64233964116567e-05}. Best is trial 0 with value: 0.008702325285680435.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:33:58,234] Trial 3 finished with value: 0.011943520545342956 and parameters: {'n_estimators': 295, 'learning_rate': 0.022745584461040785, 'num_leaves': 134, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.9583968974865797, 'colsample_bytree': 0.9934171211307871, 'reg_alpha': 7.944235161697508, 'reg_lambda': 3.1315640904407527, 'min_split_gain': 0.039047126215254496}. Best is trial 0 with value: 0.008702325285680435.
[I 2025-06-15 20:33:58,236] A new study created in memory with name: no-name-e6378ca7-538b-4525-9f88-2a2282e7da2b
[I 2025-06-15 20:33:58,264] Trial 0 finished with value: 0.07463784947693496 and parameters: {'n_estimators': 120, 'learning_rate': 0.011013722577310021, 'num_leaves': 84, 'max_depth': 4, 'min_child_samples': 30, 'subsample': 0.7448294700031658, 'colsample_bytree': 0.7188895395213593, 'reg_alpha': 3.654424998280459, 'reg_lambda': 8.365459796224465, 'min_split_gain': 0.03248702490568572}. Best is trial 0 with value: 0.07463784947693496.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1685
1           Price_lag_2        1533
14      pct_chg_Load_DA        1401
3         Load_DA_lag_0         997
15       lag168_Load_DA         843
5       WindOn_DA_lag_0         807
2           Price_lag_7         708
11           NGas_lag_2         704
16  roll2d_median_Price         667
13            EUA_lag_2         475
10           Coal_lag_2         437
12            Oil_lag_2         328
9                  WD_7          43
7                  WD_1          31
8                  WD_6          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         149
15       lag168_Load_DA          23
1           Price_lag_2          23
2           Price_lag_7           5
5       WindOn_DA_lag_0           3
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
16  roll2d_median_Price           0
-> 72.23% done
********************* END NS.. 527 ****************************************************
********************* START NS ... 528 ****************************************************
[I 2025-06-15 20:34:00,654] A new study created in memory with name: no-name-025f9408-a054-4cb5-b423-aa5d41108824
[I 2025-06-15 20:34:00,931] Trial 0 finished with value: 0.009223008962255315 and parameters: {'n_estimators': 294, 'learning_rate': 0.22361695256491446, 'num_leaves': 57, 'max_depth': 14, 'min_child_samples': 20, 'subsample': 0.9279993847133993, 'colsample_bytree': 0.6670853438229823, 'reg_alpha': 5.69342283467516, 'reg_lambda': 0.584700212250463, 'min_split_gain': 0.047544924307529615}. Best is trial 0 with value: 0.009223008962255315.
[I 2025-06-15 20:34:01,100] Trial 1 finished with value: 0.007844099804661081 and parameters: {'n_estimators': 143, 'learning_rate': 0.19965320324739982, 'num_leaves': 90, 'max_depth': 12, 'min_child_samples': 29, 'subsample': 0.7125993432463277, 'colsample_bytree': 0.8804825835383497, 'reg_alpha': 8.216979797946959, 'reg_lambda': 1.986268487294539, 'min_split_gain': 0.04765062322822298}. Best is trial 1 with value: 0.007844099804661081.
[Trial 0] New best: 0.00922
[Trial 1] New best: 0.00784
[I 2025-06-15 20:34:01,339] Trial 2 finished with value: 0.007547033764361451 and parameters: {'n_estimators': 67, 'learning_rate': 0.27114604317657376, 'num_leaves': 190, 'max_depth': 9, 'min_child_samples': 11, 'subsample': 0.6689601181746597, 'colsample_bytree': 0.8102957637876923, 'reg_alpha': 4.7077058037912, 'reg_lambda': 9.326890706290422, 'min_split_gain': 0.017276865867378765}. Best is trial 2 with value: 0.007547033764361451.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:34:01,716] Trial 3 finished with value: 0.007988566627706964 and parameters: {'n_estimators': 113, 'learning_rate': 0.07870785535434795, 'num_leaves': 62, 'max_depth': 8, 'min_child_samples': 7, 'subsample': 0.831491946391743, 'colsample_bytree': 0.9432741379358907, 'reg_alpha': 7.4220777412697645, 'reg_lambda': 9.396231637820659, 'min_split_gain': 0.03085572588743478}. Best is trial 2 with value: 0.007547033764361451.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:34:04,494] Trial 4 finished with value: 0.0072110513736960745 and parameters: {'n_estimators': 298, 'learning_rate': 0.017566732361791904, 'num_leaves': 227, 'max_depth': 12, 'min_child_samples': 5, 'subsample': 0.8945131779042981, 'colsample_bytree': 0.7416318522359748, 'reg_alpha': 2.5848382285331484, 'reg_lambda': 0.6544917640134418, 'min_split_gain': 0.004731086791530398}. Best is trial 4 with value: 0.0072110513736960745.
[I 2025-06-15 20:34:04,497] A new study created in memory with name: no-name-d15887fb-ead8-4574-8c88-7433430059bf
[I 2025-06-15 20:34:04,532] Trial 0 finished with value: 0.04395765922265549 and parameters: {'n_estimators': 154, 'learning_rate': 0.02497127209960146, 'num_leaves': 227, 'max_depth': 11, 'min_child_samples': 3, 'subsample': 0.9567898784756494, 'colsample_bytree': 0.5995886526428169, 'reg_alpha': 8.782907822897878, 'reg_lambda': 7.287647904328681, 'min_split_gain': 0.04385612848996422}. Best is trial 0 with value: 0.04395765922265549.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        4754
0           Price_lag_1        4611
14      pct_chg_Load_DA        4167
3         Load_DA_lag_0        2391
2           Price_lag_7        2358
16  roll2d_median_Price        2152
11           NGas_lag_2        2106
5       WindOn_DA_lag_0        1909
15       lag168_Load_DA        1887
13            EUA_lag_2        1705
12            Oil_lag_2        1400
10           Coal_lag_2        1256
9                  WD_7         118
8                  WD_6          49
7                  WD_1          44
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          28
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 72.37% done
********************* END NS.. 528 ****************************************************
********************* START NS ... 529 ****************************************************
[I 2025-06-15 20:34:07,713] A new study created in memory with name: no-name-30317506-d7e7-419e-9933-a23d542e3557
[I 2025-06-15 20:34:07,956] Trial 0 finished with value: 0.016526491581180774 and parameters: {'n_estimators': 211, 'learning_rate': 0.234988239060155, 'num_leaves': 116, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.9554092252474864, 'colsample_bytree': 0.6958496228130171, 'reg_alpha': 0.46550268220717395, 'reg_lambda': 0.22950293191881577, 'min_split_gain': 0.001210226417324739}. Best is trial 0 with value: 0.016526491581180774.
[I 2025-06-15 20:34:08,058] Trial 1 finished with value: 0.01958452018302617 and parameters: {'n_estimators': 122, 'learning_rate': 0.23850461965085568, 'num_leaves': 37, 'max_depth': 15, 'min_child_samples': 10, 'subsample': 0.928323395555446, 'colsample_bytree': 0.8292148241113848, 'reg_alpha': 4.7305056489494195, 'reg_lambda': 9.32624228645254, 'min_split_gain': 0.0432852619318981}. Best is trial 0 with value: 0.016526491581180774.
[Trial 0] New best: 0.01653
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:34:08,521] Trial 2 finished with value: 0.01777156840147621 and parameters: {'n_estimators': 274, 'learning_rate': 0.0841268768228679, 'num_leaves': 148, 'max_depth': 15, 'min_child_samples': 29, 'subsample': 0.808777474625586, 'colsample_bytree': 0.526378250234029, 'reg_alpha': 1.2138199640581737, 'reg_lambda': 9.259133586848145, 'min_split_gain': 0.03636989828353651}. Best is trial 0 with value: 0.016526491581180774.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:34:08,966] Trial 3 finished with value: 0.10461852315399452 and parameters: {'n_estimators': 75, 'learning_rate': 0.011930586661458036, 'num_leaves': 175, 'max_depth': 7, 'min_child_samples': 22, 'subsample': 0.809385419756771, 'colsample_bytree': 0.6839205174180378, 'reg_alpha': 3.337287441422876, 'reg_lambda': 3.805088455906879, 'min_split_gain': 0.027019537934337756}. Best is trial 0 with value: 0.016526491581180774.
[I 2025-06-15 20:34:08,969] A new study created in memory with name: no-name-cdeca5cc-8c21-49a5-88f5-557475e7b2b4
[I 2025-06-15 20:34:08,999] Trial 0 finished with value: 0.07222357195908606 and parameters: {'n_estimators': 143, 'learning_rate': 0.11668771359100248, 'num_leaves': 155, 'max_depth': 10, 'min_child_samples': 11, 'subsample': 0.8246349448625874, 'colsample_bytree': 0.545258736515823, 'reg_alpha': 9.378807820201382, 'reg_lambda': 2.025470194110227, 'min_split_gain': 0.019243871165833995}. Best is trial 0 with value: 0.07222357195908606.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         520
1           Price_lag_2         395
14      pct_chg_Load_DA         285
3         Load_DA_lag_0         276
5       WindOn_DA_lag_0         242
2           Price_lag_7         239
16  roll2d_median_Price         235
15       lag168_Load_DA         202
11           NGas_lag_2         182
10           Coal_lag_2         155
13            EUA_lag_2         128
12            Oil_lag_2         111
9                  WD_7          14
7                  WD_1           9
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          10
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 72.50% done
********************* END NS.. 529 ****************************************************
********************* START NS ... 530 ****************************************************
[I 2025-06-15 20:34:10,153] A new study created in memory with name: no-name-9fdf9b8b-85ff-42e0-804c-8f3220b4d031
[I 2025-06-15 20:34:10,611] Trial 0 finished with value: 0.018379698780193344 and parameters: {'n_estimators': 201, 'learning_rate': 0.023464252291201096, 'num_leaves': 229, 'max_depth': 10, 'min_child_samples': 22, 'subsample': 0.6411705583000958, 'colsample_bytree': 0.6775065640394123, 'reg_alpha': 6.89781563914948, 'reg_lambda': 3.049914488977956, 'min_split_gain': 0.02888649949477611}. Best is trial 0 with value: 0.018379698780193344.
[Trial 0] New best: 0.01838
[I 2025-06-15 20:34:10,959] Trial 1 finished with value: 0.016759556657736673 and parameters: {'n_estimators': 258, 'learning_rate': 0.09183676973075329, 'num_leaves': 74, 'max_depth': 6, 'min_child_samples': 28, 'subsample': 0.9683494312697418, 'colsample_bytree': 0.5089002885670306, 'reg_alpha': 7.416070235273427, 'reg_lambda': 9.690333468365463, 'min_split_gain': 0.010847116514242833}. Best is trial 1 with value: 0.016759556657736673.
[Trial 1] New best: 0.01676
[I 2025-06-15 20:34:11,186] Trial 2 finished with value: 0.01637224733772943 and parameters: {'n_estimators': 122, 'learning_rate': 0.16516239286513126, 'num_leaves': 74, 'max_depth': 8, 'min_child_samples': 7, 'subsample': 0.7725093188051431, 'colsample_bytree': 0.8801124238095595, 'reg_alpha': 0.5162902863229546, 'reg_lambda': 6.753181680397789, 'min_split_gain': 0.03934968333498178}. Best is trial 2 with value: 0.01637224733772943.
[I 2025-06-15 20:34:11,381] Trial 3 finished with value: 0.01708441688814458 and parameters: {'n_estimators': 262, 'learning_rate': 0.2671092710103201, 'num_leaves': 200, 'max_depth': 4, 'min_child_samples': 27, 'subsample': 0.5194814566733843, 'colsample_bytree': 0.6864435474212651, 'reg_alpha': 6.548339059324043, 'reg_lambda': 6.587982743590337, 'min_split_gain': 0.004507824204173222}. Best is trial 2 with value: 0.01637224733772943.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:34:11,833] Trial 4 finished with value: 0.016563397105142937 and parameters: {'n_estimators': 237, 'learning_rate': 0.06725183659044473, 'num_leaves': 138, 'max_depth': 6, 'min_child_samples': 10, 'subsample': 0.5120134448404198, 'colsample_bytree': 0.5334621632042211, 'reg_alpha': 0.9226798946439729, 'reg_lambda': 7.709703254515009, 'min_split_gain': 0.030965131855948697}. Best is trial 2 with value: 0.01637224733772943.
[I 2025-06-15 20:34:11,836] A new study created in memory with name: no-name-a5c9c8ad-9620-485c-9def-e2e8c8f0d3db
[I 2025-06-15 20:34:11,868] Trial 0 finished with value: 0.06186796552793583 and parameters: {'n_estimators': 159, 'learning_rate': 0.032609129770031296, 'num_leaves': 158, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.7753467177159272, 'colsample_bytree': 0.6617833630097171, 'reg_alpha': 7.529482278826754, 'reg_lambda': 3.3714037747913705, 'min_split_gain': 0.00907795355476188}. Best is trial 0 with value: 0.06186796552793583.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         494
1           Price_lag_2         411
14      pct_chg_Load_DA         381
3         Load_DA_lag_0         212
16  roll2d_median_Price         202
2           Price_lag_7         201
5       WindOn_DA_lag_0         179
11           NGas_lag_2         159
15       lag168_Load_DA         148
10           Coal_lag_2         120
12            Oil_lag_2          83
13            EUA_lag_2          68
9                  WD_7          13
7                  WD_1          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          92
1           Price_lag_2          11
15       lag168_Load_DA           7
11           NGas_lag_2           3
2           Price_lag_7           2
12            Oil_lag_2           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
16  roll2d_median_Price           0
-> 72.64% done
********************* END NS.. 530 ****************************************************
********************* START NS ... 531 ****************************************************
[I 2025-06-15 20:34:13,145] A new study created in memory with name: no-name-7d401f8c-093d-4342-b836-af2363e26ca5
[I 2025-06-15 20:34:13,281] Trial 0 finished with value: 0.005872004400692905 and parameters: {'n_estimators': 113, 'learning_rate': 0.22743948230401054, 'num_leaves': 183, 'max_depth': 13, 'min_child_samples': 27, 'subsample': 0.9870159632142712, 'colsample_bytree': 0.8981615204647566, 'reg_alpha': 7.592916330281902, 'reg_lambda': 5.105864591769802, 'min_split_gain': 0.026139618131660852}. Best is trial 0 with value: 0.005872004400692905.
[Trial 0] New best: 0.00587
[I 2025-06-15 20:34:13,511] Trial 1 finished with value: 0.010468869038140491 and parameters: {'n_estimators': 103, 'learning_rate': 0.02417922932465803, 'num_leaves': 212, 'max_depth': 13, 'min_child_samples': 8, 'subsample': 0.5876961121314455, 'colsample_bytree': 0.5287060800992589, 'reg_alpha': 8.465927014430822, 'reg_lambda': 9.751814291695265, 'min_split_gain': 0.021736867657958438}. Best is trial 0 with value: 0.005872004400692905.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:34:13,753] Trial 2 finished with value: 0.005710987050912909 and parameters: {'n_estimators': 95, 'learning_rate': 0.09246084228403716, 'num_leaves': 211, 'max_depth': 16, 'min_child_samples': 17, 'subsample': 0.7129569132998088, 'colsample_bytree': 0.932181269625034, 'reg_alpha': 7.613107236239309, 'reg_lambda': 3.4305185973820618, 'min_split_gain': 0.042947939177240074}. Best is trial 2 with value: 0.005710987050912909.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:34:14,369] Trial 3 finished with value: 0.005996918442214276 and parameters: {'n_estimators': 196, 'learning_rate': 0.0401695578144052, 'num_leaves': 248, 'max_depth': 11, 'min_child_samples': 28, 'subsample': 0.7855147524337979, 'colsample_bytree': 0.5379892226041214, 'reg_alpha': 5.9655709184151675, 'reg_lambda': 3.5466648464976904, 'min_split_gain': 0.030870109818731718}. Best is trial 2 with value: 0.005710987050912909.
[I 2025-06-15 20:34:14,371] A new study created in memory with name: no-name-153f6320-92e2-4a09-846c-03df63176119
[I 2025-06-15 20:34:14,413] Trial 0 finished with value: 0.028864654563827696 and parameters: {'n_estimators': 298, 'learning_rate': 0.021596185701699796, 'num_leaves': 83, 'max_depth': 4, 'min_child_samples': 3, 'subsample': 0.8182848676104129, 'colsample_bytree': 0.7400635688894042, 'reg_alpha': 9.206930195984018, 'reg_lambda': 6.090935766522304, 'min_split_gain': 0.036760732576654555}. Best is trial 0 with value: 0.028864654563827696.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         522
14      pct_chg_Load_DA         312
1           Price_lag_2         232
3         Load_DA_lag_0         162
11           NGas_lag_2         144
16  roll2d_median_Price         130
2           Price_lag_7         120
5       WindOn_DA_lag_0          64
15       lag168_Load_DA          62
10           Coal_lag_2          57
12            Oil_lag_2          50
13            EUA_lag_2          33
9                  WD_7           8
7                  WD_1           6
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          39
1           Price_lag_2           8
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 72.78% done
********************* END NS.. 531 ****************************************************
********************* START NS ... 532 ****************************************************
[I 2025-06-15 20:34:15,448] A new study created in memory with name: no-name-13783cbf-542a-4acd-a4dd-64a0de0c8350
[I 2025-06-15 20:34:16,142] Trial 0 finished with value: 0.0017107919795472735 and parameters: {'n_estimators': 215, 'learning_rate': 0.022136008418989005, 'num_leaves': 52, 'max_depth': 9, 'min_child_samples': 16, 'subsample': 0.7231272978568565, 'colsample_bytree': 0.8875315678325675, 'reg_alpha': 4.258650837967616, 'reg_lambda': 9.823618226832915, 'min_split_gain': 0.027626399014080174}. Best is trial 0 with value: 0.0017107919795472735.
[Trial 0] New best: 0.00171
[I 2025-06-15 20:34:16,339] Trial 1 finished with value: 0.0013830291530180503 and parameters: {'n_estimators': 91, 'learning_rate': 0.1418728059464874, 'num_leaves': 212, 'max_depth': 14, 'min_child_samples': 12, 'subsample': 0.6122458686671117, 'colsample_bytree': 0.8941104668658838, 'reg_alpha': 7.48885802494729, 'reg_lambda': 4.1685180207413906, 'min_split_gain': 0.029476203802022906}. Best is trial 1 with value: 0.0013830291530180503.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:34:17,107] Trial 2 finished with value: 0.0025992493731859393 and parameters: {'n_estimators': 226, 'learning_rate': 0.026480079459746526, 'num_leaves': 94, 'max_depth': 15, 'min_child_samples': 9, 'subsample': 0.5529414313585116, 'colsample_bytree': 0.5688813045144879, 'reg_alpha': 5.11815694738118, 'reg_lambda': 3.372954386669047, 'min_split_gain': 0.0379396557585892}. Best is trial 1 with value: 0.0013830291530180503.
[I 2025-06-15 20:34:17,260] Trial 3 finished with value: 0.0019906155681604946 and parameters: {'n_estimators': 60, 'learning_rate': 0.2927006333162244, 'num_leaves': 253, 'max_depth': 8, 'min_child_samples': 19, 'subsample': 0.6918312807080687, 'colsample_bytree': 0.6865264561879464, 'reg_alpha': 8.467374345685199, 'reg_lambda': 6.804493398790444, 'min_split_gain': 0.03672205244889236}. Best is trial 1 with value: 0.0013830291530180503.
[I 2025-06-15 20:34:17,263] A new study created in memory with name: no-name-ad585f05-c445-4725-b14e-1150b3f67df2
[I 2025-06-15 20:34:17,298] Trial 0 finished with value: 0.04427982486730205 and parameters: {'n_estimators': 220, 'learning_rate': 0.01706870388371783, 'num_leaves': 227, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.8175207331813524, 'colsample_bytree': 0.5683796911031449, 'reg_alpha': 8.763496976602342, 'reg_lambda': 9.916851906936538, 'min_split_gain': 0.005870224988210354}. Best is trial 0 with value: 0.04427982486730205.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         356
14      pct_chg_Load_DA         212
1           Price_lag_2         204
3         Load_DA_lag_0         136
16  roll2d_median_Price         120
11           NGas_lag_2         115
2           Price_lag_7         105
5       WindOn_DA_lag_0          75
15       lag168_Load_DA          68
10           Coal_lag_2          50
12            Oil_lag_2          42
13            EUA_lag_2          38
9                  WD_7          12
8                  WD_6           7
7                  WD_1           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         148
1           Price_lag_2          42
2           Price_lag_7           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
16  roll2d_median_Price           0
-> 72.91% done
********************* END NS.. 532 ****************************************************
********************* START NS ... 533 ****************************************************
[I 2025-06-15 20:34:18,877] A new study created in memory with name: no-name-e4ef1af4-b1fd-455d-b93c-0b505340b9c0
[I 2025-06-15 20:34:19,141] Trial 0 finished with value: 0.08235121048338175 and parameters: {'n_estimators': 83, 'learning_rate': 0.016652349448378425, 'num_leaves': 254, 'max_depth': 12, 'min_child_samples': 5, 'subsample': 0.8947961716448003, 'colsample_bytree': 0.9113376346520582, 'reg_alpha': 5.720190405270058, 'reg_lambda': 6.125257959381721, 'min_split_gain': 0.02928312694351041}. Best is trial 0 with value: 0.08235121048338175.
[Trial 0] New best: 0.08235
[I 2025-06-15 20:34:19,414] Trial 1 finished with value: 0.002558752599552492 and parameters: {'n_estimators': 240, 'learning_rate': 0.1774116712270111, 'num_leaves': 136, 'max_depth': 15, 'min_child_samples': 23, 'subsample': 0.659023187968911, 'colsample_bytree': 0.9819455392664416, 'reg_alpha': 1.6531494759180332, 'reg_lambda': 0.3845479591742196, 'min_split_gain': 0.04293882087231661}. Best is trial 1 with value: 0.002558752599552492.
[Trial 1] New best: 0.00256
[I 2025-06-15 20:34:22,266] Trial 2 finished with value: 0.003965917961633013 and parameters: {'n_estimators': 293, 'learning_rate': 0.020557283017537446, 'num_leaves': 114, 'max_depth': 12, 'min_child_samples': 7, 'subsample': 0.7305131169919522, 'colsample_bytree': 0.6597050596437406, 'reg_alpha': 1.5174190530337794, 'reg_lambda': 1.2915696020164091, 'min_split_gain': 0.03460026370703919}. Best is trial 1 with value: 0.002558752599552492.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:34:23,497] Trial 3 finished with value: 0.004785989727222939 and parameters: {'n_estimators': 299, 'learning_rate': 0.012036872093018664, 'num_leaves': 107, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.762717473536934, 'colsample_bytree': 0.8963551553168662, 'reg_alpha': 5.228053039208104, 'reg_lambda': 3.428225471885927, 'min_split_gain': 0.048936196401093555}. Best is trial 1 with value: 0.002558752599552492.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:34:23,799] Trial 4 finished with value: 0.002747428311745855 and parameters: {'n_estimators': 261, 'learning_rate': 0.22817439684160928, 'num_leaves': 51, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.946259801573119, 'colsample_bytree': 0.690206187559046, 'reg_alpha': 1.2393730926255453, 'reg_lambda': 1.198803615829287, 'min_split_gain': 0.049157306878968035}. Best is trial 1 with value: 0.002558752599552492.
[I 2025-06-15 20:34:23,802] A new study created in memory with name: no-name-aeddecd3-4882-4645-824b-b79242c74eef
[I 2025-06-15 20:34:23,833] Trial 0 finished with value: 0.1348776974136376 and parameters: {'n_estimators': 123, 'learning_rate': 0.017044797469900196, 'num_leaves': 126, 'max_depth': 5, 'min_child_samples': 28, 'subsample': 0.6736283681336265, 'colsample_bytree': 0.7092566702111452, 'reg_alpha': 5.1000942009153665, 'reg_lambda': 1.2726609059812766, 'min_split_gain': 0.0017702129215360852}. Best is trial 0 with value: 0.1348776974136376.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         429
14      pct_chg_Load_DA         332
1           Price_lag_2         286
3         Load_DA_lag_0         208
16  roll2d_median_Price         159
11           NGas_lag_2         143
2           Price_lag_7         137
5       WindOn_DA_lag_0         119
15       lag168_Load_DA         115
10           Coal_lag_2          63
12            Oil_lag_2          57
13            EUA_lag_2          25
9                  WD_7           7
7                  WD_1           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         166
3         Load_DA_lag_0          42
1           Price_lag_2          22
11           NGas_lag_2          14
13            EUA_lag_2           2
16  roll2d_median_Price           2
6      WindOff_DA_lag_0           0
7                  WD_1           0
5       WindOn_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
4        Solar_DA_lag_0           0
12            Oil_lag_2           0
2           Price_lag_7           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 73.05% done
********************* END NS.. 533 ****************************************************
********************* START NS ... 534 ****************************************************
[I 2025-06-15 20:34:25,509] A new study created in memory with name: no-name-2ef5e791-7c30-4564-bd0a-b6d6d2cff629
[I 2025-06-15 20:34:25,609] Trial 0 finished with value: 0.03148374006363162 and parameters: {'n_estimators': 114, 'learning_rate': 0.2746238693460296, 'num_leaves': 36, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.836345675290057, 'colsample_bytree': 0.6312537818354842, 'reg_alpha': 4.664008670609938, 'reg_lambda': 9.78246087949252, 'min_split_gain': 0.04662110556543906}. Best is trial 0 with value: 0.03148374006363162.
[Trial 0] New best: 0.03148
[I 2025-06-15 20:34:25,854] Trial 1 finished with value: 0.04567021251785586 and parameters: {'n_estimators': 98, 'learning_rate': 0.020184934092562102, 'num_leaves': 158, 'max_depth': 16, 'min_child_samples': 12, 'subsample': 0.8185161565593284, 'colsample_bytree': 0.7686611272305747, 'reg_alpha': 8.662119252995183, 'reg_lambda': 8.946562504350398, 'min_split_gain': 0.04932067736112563}. Best is trial 0 with value: 0.03148374006363162.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:34:26,728] Trial 2 finished with value: 0.03777057924413669 and parameters: {'n_estimators': 172, 'learning_rate': 0.01346327622100803, 'num_leaves': 64, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.8506417555281276, 'colsample_bytree': 0.595761596269517, 'reg_alpha': 3.2906552353003304, 'reg_lambda': 5.609119869192258, 'min_split_gain': 0.012026813725002878}. Best is trial 0 with value: 0.03148374006363162.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:34:27,075] Trial 3 finished with value: 0.21969141021483796 and parameters: {'n_estimators': 63, 'learning_rate': 0.01140972306533313, 'num_leaves': 68, 'max_depth': 11, 'min_child_samples': 29, 'subsample': 0.8726973712165487, 'colsample_bytree': 0.7777643299340907, 'reg_alpha': 7.064689061039935, 'reg_lambda': 6.957119566533679, 'min_split_gain': 0.014335603460231224}. Best is trial 0 with value: 0.03148374006363162.
[I 2025-06-15 20:34:27,077] A new study created in memory with name: no-name-85771638-f41e-4202-9181-9193b535ecd8
[I 2025-06-15 20:34:27,126] Trial 0 finished with value: 0.08525183044823724 and parameters: {'n_estimators': 150, 'learning_rate': 0.02349532804849558, 'num_leaves': 234, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.7932971100073238, 'colsample_bytree': 0.5800922326231999, 'reg_alpha': 3.8032847953067495, 'reg_lambda': 0.57687640588439, 'min_split_gain': 0.03592915435199861}. Best is trial 0 with value: 0.08525183044823724.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         194
1           Price_lag_2         137
14      pct_chg_Load_DA          78
2           Price_lag_7          62
11           NGas_lag_2          61
3         Load_DA_lag_0          58
16  roll2d_median_Price          52
15       lag168_Load_DA          47
5       WindOn_DA_lag_0          39
10           Coal_lag_2          33
13            EUA_lag_2          28
12            Oil_lag_2           9
7                  WD_1           4
8                  WD_6           4
9                  WD_7           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         127
3         Load_DA_lag_0          30
1           Price_lag_2          25
11           NGas_lag_2          25
15       lag168_Load_DA           4
16  roll2d_median_Price           4
2           Price_lag_7           3
10           Coal_lag_2           1
13            EUA_lag_2           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
5       WindOn_DA_lag_0           0
9                  WD_7           0
4        Solar_DA_lag_0           0
12            Oil_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 73.19% done
********************* END NS.. 534 ****************************************************
********************* START NS ... 535 ****************************************************
[I 2025-06-15 20:34:28,725] A new study created in memory with name: no-name-dad10142-bce1-43f5-a07d-47a94c183f00
[I 2025-06-15 20:34:29,744] Trial 0 finished with value: 0.03560525812987469 and parameters: {'n_estimators': 254, 'learning_rate': 0.03968637452257015, 'num_leaves': 215, 'max_depth': 16, 'min_child_samples': 30, 'subsample': 0.9977211855953534, 'colsample_bytree': 0.7814539431407714, 'reg_alpha': 9.81170191074014, 'reg_lambda': 3.883366192491077, 'min_split_gain': 0.019856983546156094}. Best is trial 0 with value: 0.03560525812987469.
[I 2025-06-15 20:34:29,932] Trial 1 finished with value: 0.041897434897552766 and parameters: {'n_estimators': 54, 'learning_rate': 0.04609535188731061, 'num_leaves': 113, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.6623040779166978, 'colsample_bytree': 0.6777234215849608, 'reg_alpha': 5.894637345472079, 'reg_lambda': 4.836100308120086, 'min_split_gain': 0.0025383377691811262}. Best is trial 0 with value: 0.03560525812987469.
[Trial 0] New best: 0.03561
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:34:30,230] Trial 2 finished with value: 0.034209387907252614 and parameters: {'n_estimators': 182, 'learning_rate': 0.2182863023455571, 'num_leaves': 71, 'max_depth': 15, 'min_child_samples': 4, 'subsample': 0.9862103579356398, 'colsample_bytree': 0.7189647719333674, 'reg_alpha': 0.3553468815037186, 'reg_lambda': 3.28892597056842, 'min_split_gain': 0.026441334863664057}. Best is trial 2 with value: 0.034209387907252614.
[Trial 2] New best: 0.03421
[I 2025-06-15 20:34:32,198] Trial 3 finished with value: 0.03357746216981122 and parameters: {'n_estimators': 254, 'learning_rate': 0.015590062124554022, 'num_leaves': 244, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.6357591912505427, 'colsample_bytree': 0.5904525096064901, 'reg_alpha': 0.10673107794423542, 'reg_lambda': 1.965251595083689, 'min_split_gain': 0.006711142597373582}. Best is trial 3 with value: 0.03357746216981122.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:34:32,662] Trial 4 finished with value: 0.043569990412800935 and parameters: {'n_estimators': 174, 'learning_rate': 0.013485467588126333, 'num_leaves': 156, 'max_depth': 4, 'min_child_samples': 17, 'subsample': 0.7543421835600986, 'colsample_bytree': 0.733982878940523, 'reg_alpha': 6.484102056366647, 'reg_lambda': 9.570626904343849, 'min_split_gain': 0.030309417343765917}. Best is trial 3 with value: 0.03357746216981122.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:34:32,978] Trial 5 finished with value: 0.03362897697932273 and parameters: {'n_estimators': 204, 'learning_rate': 0.26442056051519786, 'num_leaves': 35, 'max_depth': 13, 'min_child_samples': 17, 'subsample': 0.8460244697530556, 'colsample_bytree': 0.6808325152277375, 'reg_alpha': 1.8749584956441512, 'reg_lambda': 5.999169403887469, 'min_split_gain': 0.01808516774264119}. Best is trial 3 with value: 0.03357746216981122.
[I 2025-06-15 20:34:32,981] A new study created in memory with name: no-name-4883f86e-9b92-45f5-bc9a-13e6caea492f
[I 2025-06-15 20:34:33,017] Trial 0 finished with value: 0.0618955795225569 and parameters: {'n_estimators': 238, 'learning_rate': 0.2546883106769372, 'num_leaves': 198, 'max_depth': 6, 'min_child_samples': 1, 'subsample': 0.6097522531978427, 'colsample_bytree': 0.5768562695994013, 'reg_alpha': 3.173390640849676, 'reg_lambda': 7.651403086540613, 'min_split_gain': 0.004081455938415241}. Best is trial 0 with value: 0.0618955795225569.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3884
1           Price_lag_2        3768
14      pct_chg_Load_DA        2206
16  roll2d_median_Price        1882
3         Load_DA_lag_0        1655
2           Price_lag_7        1505
15       lag168_Load_DA        1285
11           NGas_lag_2        1238
10           Coal_lag_2        1166
13            EUA_lag_2         920
5       WindOn_DA_lag_0         907
12            Oil_lag_2         887
9                  WD_7          97
7                  WD_1          61
8                  WD_6          22
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          31
1           Price_lag_2           8
3         Load_DA_lag_0           7
2           Price_lag_7           4
11           NGas_lag_2           3
12            Oil_lag_2           2
16  roll2d_median_Price           2
15       lag168_Load_DA           1
7                  WD_1           0
6      WindOff_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 73.32% done
********************* END NS.. 535 ****************************************************
********************* START NS ... 536 ****************************************************
[I 2025-06-15 20:34:35,192] A new study created in memory with name: no-name-a14fc1a6-1b20-41a6-a6f9-6d6e5aeca1f1
[I 2025-06-15 20:34:35,405] Trial 0 finished with value: 0.038314463079258405 and parameters: {'n_estimators': 69, 'learning_rate': 0.024996613258552104, 'num_leaves': 84, 'max_depth': 11, 'min_child_samples': 28, 'subsample': 0.5256479312047545, 'colsample_bytree': 0.9606869210225693, 'reg_alpha': 9.995973422742603, 'reg_lambda': 4.880321328965405, 'min_split_gain': 0.044992397445486186}. Best is trial 0 with value: 0.038314463079258405.
[Trial 0] New best: 0.03831
[I 2025-06-15 20:34:36,138] Trial 1 finished with value: 0.02739543079461701 and parameters: {'n_estimators': 138, 'learning_rate': 0.01566776232469813, 'num_leaves': 121, 'max_depth': 12, 'min_child_samples': 16, 'subsample': 0.6065125988450478, 'colsample_bytree': 0.581218138746648, 'reg_alpha': 2.003260350708155, 'reg_lambda': 2.954700103913507, 'min_split_gain': 0.014891797317624418}. Best is trial 1 with value: 0.02739543079461701.
[Trial 1] New best: 0.02740
[I 2025-06-15 20:34:36,921] Trial 2 finished with value: 0.018848456391832965 and parameters: {'n_estimators': 190, 'learning_rate': 0.055448198697382954, 'num_leaves': 207, 'max_depth': 16, 'min_child_samples': 23, 'subsample': 0.7456828910904312, 'colsample_bytree': 0.8015572278317767, 'reg_alpha': 0.9477426967223213, 'reg_lambda': 3.773898719770572, 'min_split_gain': 0.04176282730758311}. Best is trial 2 with value: 0.018848456391832965.
[Trial 2] New best: 0.01885
[I 2025-06-15 20:34:37,182] Trial 3 finished with value: 0.021160150748183018 and parameters: {'n_estimators': 234, 'learning_rate': 0.11157176838028134, 'num_leaves': 203, 'max_depth': 6, 'min_child_samples': 10, 'subsample': 0.8207757976662098, 'colsample_bytree': 0.6602001108751657, 'reg_alpha': 6.291040319111591, 'reg_lambda': 6.702568527782558, 'min_split_gain': 0.04213719230273804}. Best is trial 2 with value: 0.018848456391832965.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:34:37,848] Trial 4 finished with value: 0.0240821673777705 and parameters: {'n_estimators': 225, 'learning_rate': 0.012624814890147572, 'num_leaves': 242, 'max_depth': 5, 'min_child_samples': 28, 'subsample': 0.6362723839627263, 'colsample_bytree': 0.598069903645749, 'reg_alpha': 3.225213967509848, 'reg_lambda': 0.8573500646223653, 'min_split_gain': 0.03892138032719411}. Best is trial 2 with value: 0.018848456391832965.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:34:38,494] Trial 5 finished with value: 0.020278943622089773 and parameters: {'n_estimators': 214, 'learning_rate': 0.04148505387369727, 'num_leaves': 256, 'max_depth': 9, 'min_child_samples': 28, 'subsample': 0.731243584043541, 'colsample_bytree': 0.7492764951246674, 'reg_alpha': 6.354735124979696, 'reg_lambda': 9.420725547920204, 'min_split_gain': 0.018353206793690718}. Best is trial 2 with value: 0.018848456391832965.
[I 2025-06-15 20:34:38,497] A new study created in memory with name: no-name-b3675864-9fe2-4c6e-8f14-fb8bc288da3c
[I 2025-06-15 20:34:38,539] Trial 0 finished with value: 0.04322232772788642 and parameters: {'n_estimators': 290, 'learning_rate': 0.017809126930297084, 'num_leaves': 162, 'max_depth': 9, 'min_child_samples': 15, 'subsample': 0.919732832915636, 'colsample_bytree': 0.845173330987637, 'reg_alpha': 8.770011930549499, 'reg_lambda': 7.214494236382487, 'min_split_gain': 0.007094049810152364}. Best is trial 0 with value: 0.04322232772788642.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1413
1           Price_lag_2        1386
14      pct_chg_Load_DA        1182
3         Load_DA_lag_0         894
2           Price_lag_7         538
11           NGas_lag_2         506
16  roll2d_median_Price         506
15       lag168_Load_DA         488
5       WindOn_DA_lag_0         482
10           Coal_lag_2         266
12            Oil_lag_2         194
13            EUA_lag_2         179
9                  WD_7          24
7                  WD_1          15
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         161
3         Load_DA_lag_0          22
11           NGas_lag_2          12
1           Price_lag_2          12
12            Oil_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 73.46% done
********************* END NS.. 536 ****************************************************
********************* START NS ... 537 ****************************************************
[I 2025-06-15 20:34:40,198] A new study created in memory with name: no-name-457dc5dc-8ec6-466b-beb1-fe6058be423b
[I 2025-06-15 20:34:40,433] Trial 0 finished with value: 0.040494653067959664 and parameters: {'n_estimators': 84, 'learning_rate': 0.019008388667771763, 'num_leaves': 171, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.7592678336488576, 'colsample_bytree': 0.8415102100040175, 'reg_alpha': 9.418601493603976, 'reg_lambda': 0.7542826148919346, 'min_split_gain': 0.014199242608619773}. Best is trial 0 with value: 0.040494653067959664.
[Trial 0] New best: 0.04049
[I 2025-06-15 20:34:42,575] Trial 1 finished with value: 0.019555502915256796 and parameters: {'n_estimators': 232, 'learning_rate': 0.010200334654550667, 'num_leaves': 205, 'max_depth': 15, 'min_child_samples': 19, 'subsample': 0.8077707003539264, 'colsample_bytree': 0.6019277097130478, 'reg_alpha': 0.371924659926266, 'reg_lambda': 4.374245923335964, 'min_split_gain': 0.04520191400123153}. Best is trial 1 with value: 0.019555502915256796.
[Trial 1] New best: 0.01956
[I 2025-06-15 20:34:43,161] Trial 2 finished with value: 0.015602608354479408 and parameters: {'n_estimators': 157, 'learning_rate': 0.08279040220857127, 'num_leaves': 172, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.5229522155249573, 'colsample_bytree': 0.5372526502234736, 'reg_alpha': 4.476177610941794, 'reg_lambda': 0.40871165018811273, 'min_split_gain': 0.03909835719602117}. Best is trial 2 with value: 0.015602608354479408.
[Trial 2] New best: 0.01560
[I 2025-06-15 20:34:43,461] Trial 3 finished with value: 0.016210298369261647 and parameters: {'n_estimators': 121, 'learning_rate': 0.1607620713037733, 'num_leaves': 46, 'max_depth': 14, 'min_child_samples': 2, 'subsample': 0.8508407935446473, 'colsample_bytree': 0.9450222689060765, 'reg_alpha': 0.12698596274122065, 'reg_lambda': 1.8615052454058922, 'min_split_gain': 0.03907066501854288}. Best is trial 2 with value: 0.015602608354479408.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:34:44,113] Trial 4 finished with value: 0.016646742846215017 and parameters: {'n_estimators': 186, 'learning_rate': 0.03707474285593534, 'num_leaves': 235, 'max_depth': 6, 'min_child_samples': 9, 'subsample': 0.7051847451330887, 'colsample_bytree': 0.6138670367477518, 'reg_alpha': 2.639372998013921, 'reg_lambda': 3.9846979954126605, 'min_split_gain': 0.02749027757275444}. Best is trial 2 with value: 0.015602608354479408.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:34:44,729] Trial 5 finished with value: 0.018011074239169086 and parameters: {'n_estimators': 214, 'learning_rate': 0.026582284106480454, 'num_leaves': 74, 'max_depth': 8, 'min_child_samples': 17, 'subsample': 0.5194232868122501, 'colsample_bytree': 0.9594443627425457, 'reg_alpha': 8.782251589622179, 'reg_lambda': 6.358585823426894, 'min_split_gain': 0.03687171387575916}. Best is trial 2 with value: 0.015602608354479408.
[I 2025-06-15 20:34:44,731] A new study created in memory with name: no-name-e559615b-bf0f-4f26-9e90-ba72a76f0e2e
[I 2025-06-15 20:34:44,833] Trial 0 finished with value: 0.019367872203383676 and parameters: {'n_estimators': 149, 'learning_rate': 0.033453165083345865, 'num_leaves': 75, 'max_depth': 14, 'min_child_samples': 1, 'subsample': 0.7591111337039655, 'colsample_bytree': 0.889328954923329, 'reg_alpha': 0.9740010263303545, 'reg_lambda': 9.310176604875886, 'min_split_gain': 0.025293511866874077}. Best is trial 0 with value: 0.019367872203383676.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         777
1           Price_lag_2         688
14      pct_chg_Load_DA         313
3         Load_DA_lag_0         255
2           Price_lag_7         234
11           NGas_lag_2         216
16  roll2d_median_Price         209
15       lag168_Load_DA         185
5       WindOn_DA_lag_0         168
10           Coal_lag_2         103
13            EUA_lag_2          83
12            Oil_lag_2          74
9                  WD_7          19
7                  WD_1          17
8                  WD_6          13
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         220
3         Load_DA_lag_0          42
1           Price_lag_2          31
2           Price_lag_7          28
15       lag168_Load_DA          23
12            Oil_lag_2          14
11           NGas_lag_2           9
5       WindOn_DA_lag_0           5
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 73.60% done
********************* END NS.. 537 ****************************************************
********************* START NS ... 538 ****************************************************
[I 2025-06-15 20:34:46,313] A new study created in memory with name: no-name-c85654c0-9ed8-4f78-84eb-f87e093d26f0
[I 2025-06-15 20:34:46,700] Trial 0 finished with value: 0.0031481422944632125 and parameters: {'n_estimators': 248, 'learning_rate': 0.1584681127585625, 'num_leaves': 110, 'max_depth': 5, 'min_child_samples': 7, 'subsample': 0.9922507538680552, 'colsample_bytree': 0.8338178740064583, 'reg_alpha': 3.393186420201879, 'reg_lambda': 0.7815618158078608, 'min_split_gain': 0.027656812803377503}. Best is trial 0 with value: 0.0031481422944632125.
[Trial 0] New best: 0.00315
[I 2025-06-15 20:34:48,184] Trial 1 finished with value: 0.004229898056522759 and parameters: {'n_estimators': 164, 'learning_rate': 0.023145118017046228, 'num_leaves': 123, 'max_depth': 12, 'min_child_samples': 24, 'subsample': 0.5606795720568755, 'colsample_bytree': 0.9986922690424673, 'reg_alpha': 3.376837872012378, 'reg_lambda': 3.997455184305255, 'min_split_gain': 0.04878252932939407}. Best is trial 0 with value: 0.0031481422944632125.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:34:49,852] Trial 2 finished with value: 0.003550094233970249 and parameters: {'n_estimators': 289, 'learning_rate': 0.038211118538674343, 'num_leaves': 136, 'max_depth': 10, 'min_child_samples': 4, 'subsample': 0.6688772898731861, 'colsample_bytree': 0.7218170516251629, 'reg_alpha': 4.469002722968131, 'reg_lambda': 5.216096539549364, 'min_split_gain': 0.030732045277692938}. Best is trial 0 with value: 0.0031481422944632125.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:34:50,490] Trial 3 finished with value: 0.0030221777477446813 and parameters: {'n_estimators': 117, 'learning_rate': 0.03647420094958998, 'num_leaves': 109, 'max_depth': 11, 'min_child_samples': 14, 'subsample': 0.7915560193798636, 'colsample_bytree': 0.615055951160191, 'reg_alpha': 7.06405887121273, 'reg_lambda': 0.9913884016370322, 'min_split_gain': 0.008247008365446284}. Best is trial 3 with value: 0.0030221777477446813.
[I 2025-06-15 20:34:50,492] A new study created in memory with name: no-name-5b4b0209-ac2f-4866-a8e9-4ab3f185ac62
[I 2025-06-15 20:34:50,523] Trial 0 finished with value: 0.00825144597142248 and parameters: {'n_estimators': 269, 'learning_rate': 0.2489426714410786, 'num_leaves': 143, 'max_depth': 11, 'min_child_samples': 30, 'subsample': 0.9194514442860168, 'colsample_bytree': 0.8063663604975406, 'reg_alpha': 8.864119572516689, 'reg_lambda': 1.4806395780362769, 'min_split_gain': 0.020498866053906994}. Best is trial 0 with value: 0.00825144597142248.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         974
1           Price_lag_2         932
14      pct_chg_Load_DA         606
2           Price_lag_7         401
3         Load_DA_lag_0         380
11           NGas_lag_2         347
16  roll2d_median_Price         333
15       lag168_Load_DA         277
13            EUA_lag_2         263
10           Coal_lag_2         170
5       WindOn_DA_lag_0         161
12            Oil_lag_2         140
8                  WD_6          27
9                  WD_7          24
7                  WD_1           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           6
3         Load_DA_lag_0           1
11           NGas_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 73.73% done
********************* END NS.. 538 ****************************************************
********************* START NS ... 539 ****************************************************
[I 2025-06-15 20:34:51,703] A new study created in memory with name: no-name-c80b2dd4-7716-4d9b-9ddf-3ca47f11a2d3
[I 2025-06-15 20:34:51,924] Trial 0 finished with value: 0.0042818776389422645 and parameters: {'n_estimators': 273, 'learning_rate': 0.15932439406570192, 'num_leaves': 204, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.5339617717548342, 'colsample_bytree': 0.9004296518502016, 'reg_alpha': 3.302492924017412, 'reg_lambda': 7.078946891784339, 'min_split_gain': 0.02461479107976068}. Best is trial 0 with value: 0.0042818776389422645.
[Trial 0] New best: 0.00428
[I 2025-06-15 20:34:52,381] Trial 1 finished with value: 0.003615533115782763 and parameters: {'n_estimators': 293, 'learning_rate': 0.0819817030284024, 'num_leaves': 235, 'max_depth': 6, 'min_child_samples': 3, 'subsample': 0.9725831389191706, 'colsample_bytree': 0.9163445224638975, 'reg_alpha': 7.585730934713055, 'reg_lambda': 2.83285241312647, 'min_split_gain': 0.008130054979512275}. Best is trial 1 with value: 0.003615533115782763.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:34:52,839] Trial 2 finished with value: 0.0038246758060914757 and parameters: {'n_estimators': 280, 'learning_rate': 0.16208911548515792, 'num_leaves': 170, 'max_depth': 8, 'min_child_samples': 25, 'subsample': 0.6366787079985197, 'colsample_bytree': 0.7547648220104999, 'reg_alpha': 0.6370011979817924, 'reg_lambda': 8.942023004057996, 'min_split_gain': 0.04505232864067674}. Best is trial 1 with value: 0.003615533115782763.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:34:53,193] Trial 3 finished with value: 0.01191540336474269 and parameters: {'n_estimators': 89, 'learning_rate': 0.024862720101708317, 'num_leaves': 152, 'max_depth': 9, 'min_child_samples': 3, 'subsample': 0.8948383342441131, 'colsample_bytree': 0.6264336883568657, 'reg_alpha': 9.330656787812703, 'reg_lambda': 2.4730452662955194, 'min_split_gain': 0.03232973878295951}. Best is trial 1 with value: 0.003615533115782763.
[I 2025-06-15 20:34:53,195] A new study created in memory with name: no-name-9ca8611c-2158-4c34-84d2-cf419a59bd2e
[I 2025-06-15 20:34:53,244] Trial 0 finished with value: 0.008200245927273567 and parameters: {'n_estimators': 276, 'learning_rate': 0.018695264243672842, 'num_leaves': 227, 'max_depth': 6, 'min_child_samples': 30, 'subsample': 0.7157377535263527, 'colsample_bytree': 0.9154782886145008, 'reg_alpha': 3.006878321036115, 'reg_lambda': 9.166762249596669, 'min_split_gain': 0.011926352602368668}. Best is trial 0 with value: 0.008200245927273567.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         688
1           Price_lag_2         454
14      pct_chg_Load_DA         438
3         Load_DA_lag_0         304
2           Price_lag_7         270
11           NGas_lag_2         240
15       lag168_Load_DA         206
5       WindOn_DA_lag_0         205
10           Coal_lag_2         166
16  roll2d_median_Price         165
13            EUA_lag_2         130
12            Oil_lag_2         118
9                  WD_7          22
7                  WD_1          13
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         232
3         Load_DA_lag_0          93
1           Price_lag_2          12
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 73.87% done
********************* END NS.. 539 ****************************************************
********************* START NS ... 540 ****************************************************
[I 2025-06-15 20:34:54,571] A new study created in memory with name: no-name-d5c60544-0f50-4ccb-9565-63671cfd9990
[I 2025-06-15 20:34:54,815] Trial 0 finished with value: 0.006329951077644152 and parameters: {'n_estimators': 142, 'learning_rate': 0.03076029067604675, 'num_leaves': 207, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.6234468309272259, 'colsample_bytree': 0.79177242210295, 'reg_alpha': 2.7470907941664855, 'reg_lambda': 1.5757014047901496, 'min_split_gain': 0.01401627522591974}. Best is trial 0 with value: 0.006329951077644152.
[Trial 0] New best: 0.00633
[I 2025-06-15 20:34:55,328] Trial 1 finished with value: 0.00637733458460659 and parameters: {'n_estimators': 251, 'learning_rate': 0.061802727356494766, 'num_leaves': 60, 'max_depth': 16, 'min_child_samples': 9, 'subsample': 0.6743326633578768, 'colsample_bytree': 0.8752578589494697, 'reg_alpha': 5.747800682804236, 'reg_lambda': 7.698631404164856, 'min_split_gain': 0.03749587486600977}. Best is trial 0 with value: 0.006329951077644152.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:34:56,731] Trial 2 finished with value: 0.006521193460428944 and parameters: {'n_estimators': 244, 'learning_rate': 0.01646705267684172, 'num_leaves': 126, 'max_depth': 10, 'min_child_samples': 13, 'subsample': 0.5499761708505528, 'colsample_bytree': 0.8640316270655333, 'reg_alpha': 2.0511398631236775, 'reg_lambda': 5.830229253092053, 'min_split_gain': 0.028647342312909076}. Best is trial 0 with value: 0.006329951077644152.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:34:57,413] Trial 3 finished with value: 0.0065628860923062915 and parameters: {'n_estimators': 283, 'learning_rate': 0.09721206066648731, 'num_leaves': 67, 'max_depth': 5, 'min_child_samples': 7, 'subsample': 0.9278001275280865, 'colsample_bytree': 0.5869332887049664, 'reg_alpha': 6.876303913462397, 'reg_lambda': 0.7736425170378725, 'min_split_gain': 0.002327038047006308}. Best is trial 0 with value: 0.006329951077644152.
[I 2025-06-15 20:34:57,415] A new study created in memory with name: no-name-2f201ab5-552d-4460-b4b8-024eeab6188b
[I 2025-06-15 20:34:57,448] Trial 0 finished with value: 0.058248931702226205 and parameters: {'n_estimators': 230, 'learning_rate': 0.013976938695160855, 'num_leaves': 195, 'max_depth': 5, 'min_child_samples': 21, 'subsample': 0.566248094479044, 'colsample_bytree': 0.661931805117209, 'reg_alpha': 4.116415671871026, 'reg_lambda': 3.703118726977773, 'min_split_gain': 0.04360367654865593}. Best is trial 0 with value: 0.058248931702226205.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1351
1           Price_lag_2         671
14      pct_chg_Load_DA         548
2           Price_lag_7         171
3         Load_DA_lag_0         170
16  roll2d_median_Price         165
11           NGas_lag_2         126
12            Oil_lag_2          70
13            EUA_lag_2          66
10           Coal_lag_2          63
15       lag168_Load_DA          56
5       WindOn_DA_lag_0          36
9                  WD_7           3
8                  WD_6           2
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         123
3         Load_DA_lag_0          34
15       lag168_Load_DA           9
10           Coal_lag_2           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 74.01% done
********************* END NS.. 540 ****************************************************
********************* START NS ... 541 ****************************************************
[I 2025-06-15 20:34:59,321] A new study created in memory with name: no-name-1b6facb6-0d3e-4b45-bee2-ee17b78eefe8
[I 2025-06-15 20:34:59,572] Trial 0 finished with value: 0.00320733785015883 and parameters: {'n_estimators': 121, 'learning_rate': 0.03693426776550301, 'num_leaves': 95, 'max_depth': 6, 'min_child_samples': 30, 'subsample': 0.5845043966223474, 'colsample_bytree': 0.9409207871229862, 'reg_alpha': 6.9351091142823265, 'reg_lambda': 1.4996894683675543, 'min_split_gain': 0.010350905102512482}. Best is trial 0 with value: 0.00320733785015883.
[Trial 0] New best: 0.00321
[I 2025-06-15 20:34:59,815] Trial 1 finished with value: 0.0329209587072344 and parameters: {'n_estimators': 66, 'learning_rate': 0.024495442421386595, 'num_leaves': 32, 'max_depth': 16, 'min_child_samples': 19, 'subsample': 0.5758757932670364, 'colsample_bytree': 0.8724605735096862, 'reg_alpha': 8.839385613270473, 'reg_lambda': 3.216439351869945, 'min_split_gain': 0.04559800362576583}. Best is trial 0 with value: 0.00320733785015883.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:00,441] Trial 2 finished with value: 0.0021552646325233454 and parameters: {'n_estimators': 76, 'learning_rate': 0.2671402226544351, 'num_leaves': 177, 'max_depth': 14, 'min_child_samples': 23, 'subsample': 0.5823475847850996, 'colsample_bytree': 0.80781197027916, 'reg_alpha': 0.5586272103380596, 'reg_lambda': 0.23044672230594365, 'min_split_gain': 0.0010479447147265064}. Best is trial 2 with value: 0.0021552646325233454.
[Trial 2] New best: 0.00216
[I 2025-06-15 20:35:00,834] Trial 3 finished with value: 0.05266640597714662 and parameters: {'n_estimators': 110, 'learning_rate': 0.012366406778938473, 'num_leaves': 205, 'max_depth': 6, 'min_child_samples': 14, 'subsample': 0.5407165530428648, 'colsample_bytree': 0.7149250506192985, 'reg_alpha': 3.3603724043951075, 'reg_lambda': 8.676446307535704, 'min_split_gain': 0.021450756385849225}. Best is trial 2 with value: 0.0021552646325233454.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:01,125] Trial 4 finished with value: 0.010696080713675065 and parameters: {'n_estimators': 110, 'learning_rate': 0.021599817319613707, 'num_leaves': 60, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.6803313176907084, 'colsample_bytree': 0.5102908105171313, 'reg_alpha': 2.277796854316727, 'reg_lambda': 4.143507523692721, 'min_split_gain': 0.040100602788206374}. Best is trial 2 with value: 0.0021552646325233454.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:01,557] Trial 5 finished with value: 0.08207091981486324 and parameters: {'n_estimators': 111, 'learning_rate': 0.01019221795780241, 'num_leaves': 234, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.848095755738316, 'colsample_bytree': 0.8424367905133298, 'reg_alpha': 7.375199006196982, 'reg_lambda': 1.507453052301948, 'min_split_gain': 0.031664288950977615}. Best is trial 2 with value: 0.0021552646325233454.
[I 2025-06-15 20:35:01,560] A new study created in memory with name: no-name-066f4466-28da-400f-94cb-c74244024f4b
[I 2025-06-15 20:35:01,592] Trial 0 finished with value: 0.02554363755807319 and parameters: {'n_estimators': 237, 'learning_rate': 0.026628385804556058, 'num_leaves': 195, 'max_depth': 10, 'min_child_samples': 1, 'subsample': 0.7425802396852169, 'colsample_bytree': 0.9638190540505214, 'reg_alpha': 6.345006572017528, 'reg_lambda': 7.6500851896649005, 'min_split_gain': 0.01534281894177475}. Best is trial 0 with value: 0.02554363755807319.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA        1430
1           Price_lag_2        1150
0           Price_lag_1        1094
3         Load_DA_lag_0        1051
15       lag168_Load_DA        1038
5       WindOn_DA_lag_0         923
2           Price_lag_7         746
16  roll2d_median_Price         630
13            EUA_lag_2         610
11           NGas_lag_2         574
12            Oil_lag_2         545
10           Coal_lag_2         383
9                  WD_7          17
7                  WD_1          15
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 74.15% done
********************* END NS.. 541 ****************************************************
********************* START NS ... 542 ****************************************************
[I 2025-06-15 20:35:03,184] A new study created in memory with name: no-name-31dae770-e7bb-4c29-953e-7454bc3db099
[I 2025-06-15 20:35:03,392] Trial 0 finished with value: 0.004382265007747203 and parameters: {'n_estimators': 180, 'learning_rate': 0.014810558626059454, 'num_leaves': 253, 'max_depth': 4, 'min_child_samples': 10, 'subsample': 0.8809532556272542, 'colsample_bytree': 0.70107933851817, 'reg_alpha': 0.8688255112416443, 'reg_lambda': 0.9051905916168335, 'min_split_gain': 0.04669662801387081}. Best is trial 0 with value: 0.004382265007747203.
[Trial 0] New best: 0.00438
[I 2025-06-15 20:35:03,884] Trial 1 finished with value: 0.0009743688097982847 and parameters: {'n_estimators': 136, 'learning_rate': 0.03380432043384367, 'num_leaves': 137, 'max_depth': 16, 'min_child_samples': 10, 'subsample': 0.6210396344292672, 'colsample_bytree': 0.6487870545255641, 'reg_alpha': 6.845767467732426, 'reg_lambda': 5.48873919944553, 'min_split_gain': 0.011172692584678923}. Best is trial 1 with value: 0.0009743688097982847.
[Trial 1] New best: 0.00097
[I 2025-06-15 20:35:04,645] Trial 2 finished with value: 0.0008851559248555285 and parameters: {'n_estimators': 177, 'learning_rate': 0.06927381536638012, 'num_leaves': 154, 'max_depth': 14, 'min_child_samples': 21, 'subsample': 0.5165641053979173, 'colsample_bytree': 0.6000848214470424, 'reg_alpha': 3.718279693730228, 'reg_lambda': 9.940256487332231, 'min_split_gain': 0.013079682209107935}. Best is trial 2 with value: 0.0008851559248555285.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:04,884] Trial 3 finished with value: 0.005819171895763246 and parameters: {'n_estimators': 77, 'learning_rate': 0.0319372930220246, 'num_leaves': 105, 'max_depth': 5, 'min_child_samples': 26, 'subsample': 0.8567518950752965, 'colsample_bytree': 0.7042804211244967, 'reg_alpha': 4.555323155482943, 'reg_lambda': 3.7442103010714134, 'min_split_gain': 0.028428707521616522}. Best is trial 2 with value: 0.0008851559248555285.
[I 2025-06-15 20:35:05,051] Trial 4 finished with value: 0.0011046049266763035 and parameters: {'n_estimators': 124, 'learning_rate': 0.20790348738978157, 'num_leaves': 110, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.9266718051014489, 'colsample_bytree': 0.529159938305567, 'reg_alpha': 8.569127078586371, 'reg_lambda': 4.084236607479964, 'min_split_gain': 0.03721360974411764}. Best is trial 2 with value: 0.0008851559248555285.
[I 2025-06-15 20:35:05,054] A new study created in memory with name: no-name-c698f0dc-87c5-4573-b03c-c78e769b8d16
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 20:35:05,104] Trial 0 finished with value: 0.004480620212208749 and parameters: {'n_estimators': 144, 'learning_rate': 0.018901302046987256, 'num_leaves': 146, 'max_depth': 15, 'min_child_samples': 10, 'subsample': 0.7646162611022752, 'colsample_bytree': 0.7064612640943964, 'reg_alpha': 0.9963564241694611, 'reg_lambda': 3.5765353063926097, 'min_split_gain': 0.004195923660863493}. Best is trial 0 with value: 0.004480620212208749.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1278
0           Price_lag_1        1230
14      pct_chg_Load_DA         665
3         Load_DA_lag_0         613
5       WindOn_DA_lag_0         459
11           NGas_lag_2         436
15       lag168_Load_DA         429
2           Price_lag_7         416
16  roll2d_median_Price         404
13            EUA_lag_2         335
10           Coal_lag_2         257
12            Oil_lag_2         227
9                  WD_7          36
7                  WD_1          22
8                  WD_6          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         179
3         Load_DA_lag_0          87
2           Price_lag_7          86
15       lag168_Load_DA          62
16  roll2d_median_Price          37
1           Price_lag_2          35
10           Coal_lag_2          35
11           NGas_lag_2          21
12            Oil_lag_2           8
13            EUA_lag_2           2
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
14      pct_chg_Load_DA           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 74.28% done
********************* END NS.. 542 ****************************************************
********************* START NS ... 543 ****************************************************
[I 2025-06-15 20:35:06,341] A new study created in memory with name: no-name-8f94612e-da2e-4f6b-94de-ea0587f4ba55
[I 2025-06-15 20:35:06,532] Trial 0 finished with value: 0.0005281131750502599 and parameters: {'n_estimators': 248, 'learning_rate': 0.10663725506293703, 'num_leaves': 45, 'max_depth': 14, 'min_child_samples': 25, 'subsample': 0.905279914318283, 'colsample_bytree': 0.9827337167541206, 'reg_alpha': 8.229124468552268, 'reg_lambda': 0.12571493435763692, 'min_split_gain': 0.025919075364495444}. Best is trial 0 with value: 0.0005281131750502599.
[Trial 0] New best: 0.00053
[I 2025-06-15 20:35:06,737] Trial 1 finished with value: 0.00042271822502909273 and parameters: {'n_estimators': 260, 'learning_rate': 0.09755248046263656, 'num_leaves': 141, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.6860184692308748, 'colsample_bytree': 0.9807353393547513, 'reg_alpha': 1.3277766502406119, 'reg_lambda': 5.0153071174400745, 'min_split_gain': 0.0288518451746561}. Best is trial 1 with value: 0.00042271822502909273.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:07,109] Trial 2 finished with value: 0.0005909455773711325 and parameters: {'n_estimators': 100, 'learning_rate': 0.28297108921267056, 'num_leaves': 187, 'max_depth': 10, 'min_child_samples': 12, 'subsample': 0.5356173840530696, 'colsample_bytree': 0.9984218478834912, 'reg_alpha': 3.281320975080492, 'reg_lambda': 1.6156846965311389, 'min_split_gain': 0.0010179331468758658}. Best is trial 1 with value: 0.00042271822502909273.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:07,629] Trial 3 finished with value: 0.0034194348913340385 and parameters: {'n_estimators': 267, 'learning_rate': 0.010868823472678264, 'num_leaves': 39, 'max_depth': 4, 'min_child_samples': 18, 'subsample': 0.9525469749959798, 'colsample_bytree': 0.9299932234204702, 'reg_alpha': 5.694799794621762, 'reg_lambda': 7.3658504438938035, 'min_split_gain': 0.017143799880188916}. Best is trial 1 with value: 0.00042271822502909273.
[I 2025-06-15 20:35:07,632] A new study created in memory with name: no-name-018fe554-4509-4be7-926d-419876a90c5e
[I 2025-06-15 20:35:07,666] Trial 0 finished with value: 0.002976124367299739 and parameters: {'n_estimators': 139, 'learning_rate': 0.022745358923459358, 'num_leaves': 37, 'max_depth': 12, 'min_child_samples': 4, 'subsample': 0.8676061140829178, 'colsample_bytree': 0.7704404797162114, 'reg_alpha': 1.9898336627846946, 'reg_lambda': 2.2590754269089186, 'min_split_gain': 0.011534715282111141}. Best is trial 0 with value: 0.002976124367299739.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         542
14      pct_chg_Load_DA         260
1           Price_lag_2         211
3         Load_DA_lag_0         170
16  roll2d_median_Price         137
11           NGas_lag_2         127
5       WindOn_DA_lag_0         107
2           Price_lag_7          99
10           Coal_lag_2          90
15       lag168_Load_DA          65
12            Oil_lag_2          62
13            EUA_lag_2          27
7                  WD_1          11
9                  WD_7           7
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         133
1           Price_lag_2          14
15       lag168_Load_DA          12
12            Oil_lag_2           4
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 74.42% done
********************* END NS.. 543 ****************************************************
********************* START NS ... 544 ****************************************************
[I 2025-06-15 20:35:08,755] A new study created in memory with name: no-name-7dea9c77-1288-440c-bd69-0d8e7df26f23
[I 2025-06-15 20:35:09,518] Trial 0 finished with value: 0.0005471590687427277 and parameters: {'n_estimators': 167, 'learning_rate': 0.03417868987550428, 'num_leaves': 104, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.7266372963928054, 'colsample_bytree': 0.5611121243037911, 'reg_alpha': 3.3099705931545245, 'reg_lambda': 8.869781369620554, 'min_split_gain': 0.006793741999937381}. Best is trial 0 with value: 0.0005471590687427277.
[Trial 0] New best: 0.00055
[I 2025-06-15 20:35:10,622] Trial 1 finished with value: 0.0004624369123773714 and parameters: {'n_estimators': 251, 'learning_rate': 0.028765079324159382, 'num_leaves': 107, 'max_depth': 13, 'min_child_samples': 27, 'subsample': 0.580298457571221, 'colsample_bytree': 0.7151656015104899, 'reg_alpha': 1.3754231934088046, 'reg_lambda': 4.401008490392727, 'min_split_gain': 0.035833017434848236}. Best is trial 1 with value: 0.0004624369123773714.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:10,946] Trial 2 finished with value: 0.00042117270233992714 and parameters: {'n_estimators': 190, 'learning_rate': 0.09344797529605675, 'num_leaves': 152, 'max_depth': 11, 'min_child_samples': 15, 'subsample': 0.6238896356900927, 'colsample_bytree': 0.7026120359036708, 'reg_alpha': 9.332413820881685, 'reg_lambda': 3.3175495459518145, 'min_split_gain': 0.04932553516778092}. Best is trial 2 with value: 0.00042117270233992714.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:11,273] Trial 3 finished with value: 0.0005062277498177774 and parameters: {'n_estimators': 64, 'learning_rate': 0.06314212571048865, 'num_leaves': 52, 'max_depth': 10, 'min_child_samples': 1, 'subsample': 0.7535934336670479, 'colsample_bytree': 0.6939324197200574, 'reg_alpha': 1.8925462528791248, 'reg_lambda': 3.6129645240739383, 'min_split_gain': 0.04471435091390917}. Best is trial 2 with value: 0.00042117270233992714.
[I 2025-06-15 20:35:11,276] A new study created in memory with name: no-name-60be5a54-6015-4d91-8bb1-871c601bde1e
[I 2025-06-15 20:35:11,309] Trial 0 finished with value: 0.000517437111833259 and parameters: {'n_estimators': 300, 'learning_rate': 0.010646011374122495, 'num_leaves': 215, 'max_depth': 14, 'min_child_samples': 9, 'subsample': 0.9177359905099169, 'colsample_bytree': 0.7363136159453059, 'reg_alpha': 4.779456556610585, 'reg_lambda': 4.430203025732281, 'min_split_gain': 0.006069183088770514}. Best is trial 0 with value: 0.000517437111833259.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         442
14      pct_chg_Load_DA         255
1           Price_lag_2         246
3         Load_DA_lag_0         149
11           NGas_lag_2         131
16  roll2d_median_Price         106
2           Price_lag_7         102
15       lag168_Load_DA          59
10           Coal_lag_2          58
5       WindOn_DA_lag_0          49
13            EUA_lag_2          35
12            Oil_lag_2          26
9                  WD_7           9
7                  WD_1           8
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 74.56% done
********************* END NS.. 544 ****************************************************
********************* START NS ... 545 ****************************************************
[I 2025-06-15 20:35:12,729] A new study created in memory with name: no-name-61b9d32f-e2d4-4b96-82c3-5693bcee2ada
[I 2025-06-15 20:35:13,001] Trial 0 finished with value: 0.003180982941042269 and parameters: {'n_estimators': 231, 'learning_rate': 0.1829846727658909, 'num_leaves': 131, 'max_depth': 8, 'min_child_samples': 28, 'subsample': 0.6204059612665007, 'colsample_bytree': 0.5388614463704479, 'reg_alpha': 6.794750182512821, 'reg_lambda': 6.324150620423589, 'min_split_gain': 0.04929971599240448}. Best is trial 0 with value: 0.003180982941042269.
[I 2025-06-15 20:35:13,158] Trial 1 finished with value: 0.002303927093296794 and parameters: {'n_estimators': 119, 'learning_rate': 0.16235914538710558, 'num_leaves': 130, 'max_depth': 16, 'min_child_samples': 13, 'subsample': 0.6117947782802147, 'colsample_bytree': 0.6628148907960898, 'reg_alpha': 8.607092875875475, 'reg_lambda': 4.338733244611048, 'min_split_gain': 0.010218443723884375}. Best is trial 1 with value: 0.002303927093296794.
[Trial 0] New best: 0.00318
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:13,326] Trial 2 finished with value: 0.0021080815916645497 and parameters: {'n_estimators': 195, 'learning_rate': 0.2788603990538499, 'num_leaves': 56, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.7666144791356093, 'colsample_bytree': 0.7733514921513662, 'reg_alpha': 6.161314879175297, 'reg_lambda': 9.803271649178596, 'min_split_gain': 0.006006082780784178}. Best is trial 2 with value: 0.0021080815916645497.
[Trial 2] New best: 0.00211
[I 2025-06-15 20:35:13,993] Trial 3 finished with value: 0.0021796829707612067 and parameters: {'n_estimators': 100, 'learning_rate': 0.06790782282269445, 'num_leaves': 141, 'max_depth': 11, 'min_child_samples': 15, 'subsample': 0.9998404219285875, 'colsample_bytree': 0.9734341888874026, 'reg_alpha': 1.6588387413229122, 'reg_lambda': 6.728010726443319, 'min_split_gain': 0.012969522413946106}. Best is trial 2 with value: 0.0021080815916645497.
[I 2025-06-15 20:35:14,175] Trial 4 finished with value: 0.0021374361028675057 and parameters: {'n_estimators': 263, 'learning_rate': 0.22557408009733107, 'num_leaves': 161, 'max_depth': 4, 'min_child_samples': 11, 'subsample': 0.9166632326197715, 'colsample_bytree': 0.7900693092302746, 'reg_alpha': 4.926760668604622, 'reg_lambda': 1.756186109183847, 'min_split_gain': 0.04042048097117314}. Best is trial 2 with value: 0.0021080815916645497.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:14,743] Trial 5 finished with value: 0.0024166905155173887 and parameters: {'n_estimators': 284, 'learning_rate': 0.06385525069685931, 'num_leaves': 241, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.689014375957494, 'colsample_bytree': 0.6400481846992353, 'reg_alpha': 2.497886020838936, 'reg_lambda': 4.56477576557607, 'min_split_gain': 0.03564154215698416}. Best is trial 2 with value: 0.0021080815916645497.
[I 2025-06-15 20:35:14,746] A new study created in memory with name: no-name-0affefa1-2b5d-4e3e-a29f-df9168ca869b
[I 2025-06-15 20:35:14,763] Trial 0 finished with value: 0.014412020473428768 and parameters: {'n_estimators': 87, 'learning_rate': 0.010159853893448161, 'num_leaves': 242, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.883074548529418, 'colsample_bytree': 0.6215524011128555, 'reg_alpha': 8.777030718093423, 'reg_lambda': 4.123927210621772, 'min_split_gain': 0.04067118673318675}. Best is trial 0 with value: 0.014412020473428768.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         236
1           Price_lag_2         209
14      pct_chg_Load_DA         166
3         Load_DA_lag_0         122
16  roll2d_median_Price          99
2           Price_lag_7          89
15       lag168_Load_DA          76
5       WindOn_DA_lag_0          75
11           NGas_lag_2          68
13            EUA_lag_2          67
10           Coal_lag_2          56
12            Oil_lag_2          40
9                  WD_7          10
8                  WD_6           5
7                  WD_1           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 74.69% done
********************* END NS.. 545 ****************************************************
********************* START NS ... 546 ****************************************************
[I 2025-06-15 20:35:16,188] A new study created in memory with name: no-name-880a6275-4cfb-4974-8ea7-4119a078c946
[I 2025-06-15 20:35:16,554] Trial 0 finished with value: 0.004253451793569603 and parameters: {'n_estimators': 171, 'learning_rate': 0.031340145707240454, 'num_leaves': 207, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.5718971952362629, 'colsample_bytree': 0.8038325481343593, 'reg_alpha': 5.120348055771322, 'reg_lambda': 8.45535935102952, 'min_split_gain': 0.03908247579202828}. Best is trial 0 with value: 0.004253451793569603.
[Trial 0] New best: 0.00425
[I 2025-06-15 20:35:17,054] Trial 1 finished with value: 0.0034151491835949315 and parameters: {'n_estimators': 190, 'learning_rate': 0.1375241184529818, 'num_leaves': 130, 'max_depth': 14, 'min_child_samples': 19, 'subsample': 0.7569884429717291, 'colsample_bytree': 0.8441434319270996, 'reg_alpha': 2.722354998762021, 'reg_lambda': 4.090750613417536, 'min_split_gain': 0.005751758282614806}. Best is trial 1 with value: 0.0034151491835949315.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:17,712] Trial 2 finished with value: 0.014866138166912707 and parameters: {'n_estimators': 196, 'learning_rate': 0.011761774270178316, 'num_leaves': 56, 'max_depth': 14, 'min_child_samples': 16, 'subsample': 0.8389395403952584, 'colsample_bytree': 0.6653539769603031, 'reg_alpha': 9.710543381632364, 'reg_lambda': 1.059810587764134, 'min_split_gain': 0.007030094905975143}. Best is trial 1 with value: 0.0034151491835949315.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:18,478] Trial 3 finished with value: 0.003880630208103997 and parameters: {'n_estimators': 190, 'learning_rate': 0.02592473960775905, 'num_leaves': 60, 'max_depth': 12, 'min_child_samples': 29, 'subsample': 0.8028496519801853, 'colsample_bytree': 0.9465461471319209, 'reg_alpha': 5.806407274503035, 'reg_lambda': 6.882302766422546, 'min_split_gain': 0.010102060497410898}. Best is trial 1 with value: 0.0034151491835949315.
[I 2025-06-15 20:35:18,481] A new study created in memory with name: no-name-b74bdb4c-4c5b-424e-abcd-3ba5b2a49f43
[I 2025-06-15 20:35:18,507] Trial 0 finished with value: 0.023568492209974756 and parameters: {'n_estimators': 225, 'learning_rate': 0.013834953474479033, 'num_leaves': 199, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.6517851746076433, 'colsample_bytree': 0.8783442454462209, 'reg_alpha': 6.920794082990924, 'reg_lambda': 0.5887837155289222, 'min_split_gain': 0.04403590206068362}. Best is trial 0 with value: 0.023568492209974756.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         781
14      pct_chg_Load_DA         688
1           Price_lag_2         652
3         Load_DA_lag_0         474
5       WindOn_DA_lag_0         441
15       lag168_Load_DA         401
2           Price_lag_7         356
16  roll2d_median_Price         290
11           NGas_lag_2         271
12            Oil_lag_2         245
10           Coal_lag_2         235
13            EUA_lag_2         231
9                  WD_7          25
7                  WD_1          24
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 74.83% done
********************* END NS.. 546 ****************************************************
********************* START NS ... 547 ****************************************************
[I 2025-06-15 20:35:19,736] A new study created in memory with name: no-name-9d0db9aa-bfee-4d7e-a33a-71a4aaa07cd5
[I 2025-06-15 20:35:20,037] Trial 0 finished with value: 0.002958453951067645 and parameters: {'n_estimators': 128, 'learning_rate': 0.1752250437302008, 'num_leaves': 231, 'max_depth': 16, 'min_child_samples': 1, 'subsample': 0.7515546231581607, 'colsample_bytree': 0.6077784018977501, 'reg_alpha': 5.519277457645665, 'reg_lambda': 6.2075252272822805, 'min_split_gain': 0.004082463880304615}. Best is trial 0 with value: 0.002958453951067645.
[I 2025-06-15 20:35:20,150] Trial 1 finished with value: 0.004235468449352184 and parameters: {'n_estimators': 75, 'learning_rate': 0.07382323319849408, 'num_leaves': 65, 'max_depth': 4, 'min_child_samples': 21, 'subsample': 0.6910105542928024, 'colsample_bytree': 0.7631639962744332, 'reg_alpha': 5.93719827301368, 'reg_lambda': 8.64382118565178, 'min_split_gain': 0.004108781258776473}. Best is trial 0 with value: 0.002958453951067645.
[Trial 0] New best: 0.00296
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:21,602] Trial 2 finished with value: 0.005225543156446842 and parameters: {'n_estimators': 218, 'learning_rate': 0.014892882538114573, 'num_leaves': 255, 'max_depth': 9, 'min_child_samples': 24, 'subsample': 0.9801998599126476, 'colsample_bytree': 0.5866553744274128, 'reg_alpha': 0.4047119578230296, 'reg_lambda': 9.700843735126485, 'min_split_gain': 0.04435732283358021}. Best is trial 0 with value: 0.002958453951067645.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:22,084] Trial 3 finished with value: 0.0033322711296377497 and parameters: {'n_estimators': 265, 'learning_rate': 0.06905006359548913, 'num_leaves': 225, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.8204813647212779, 'colsample_bytree': 0.7132211100855079, 'reg_alpha': 3.721821909047801, 'reg_lambda': 1.2679681347559912, 'min_split_gain': 0.01729363964176861}. Best is trial 0 with value: 0.002958453951067645.
[I 2025-06-15 20:35:22,086] A new study created in memory with name: no-name-5eb17052-d884-4fbd-b668-7c46616bc362
[I 2025-06-15 20:35:22,105] Trial 0 finished with value: 0.022173475862240888 and parameters: {'n_estimators': 151, 'learning_rate': 0.04903680920667182, 'num_leaves': 243, 'max_depth': 4, 'min_child_samples': 13, 'subsample': 0.6281332312312364, 'colsample_bytree': 0.8167129681881932, 'reg_alpha': 5.395381762269727, 'reg_lambda': 9.495272709346299, 'min_split_gain': 0.04339215824201487}. Best is trial 0 with value: 0.022173475862240888.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         693
0           Price_lag_1         570
3         Load_DA_lag_0         363
2           Price_lag_7         357
14      pct_chg_Load_DA         349
15       lag168_Load_DA         332
13            EUA_lag_2         329
11           NGas_lag_2         321
5       WindOn_DA_lag_0         292
16  roll2d_median_Price         282
10           Coal_lag_2         243
12            Oil_lag_2         191
9                  WD_7          27
8                  WD_6          14
7                  WD_1          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 74.97% done
********************* END NS.. 547 ****************************************************
********************* START NS ... 548 ****************************************************
[I 2025-06-15 20:35:23,182] A new study created in memory with name: no-name-3462e1c2-276f-4f34-9054-753856d23352
[I 2025-06-15 20:35:23,500] Trial 0 finished with value: 0.007013765339476785 and parameters: {'n_estimators': 124, 'learning_rate': 0.020655724004865702, 'num_leaves': 108, 'max_depth': 14, 'min_child_samples': 8, 'subsample': 0.8751072166362668, 'colsample_bytree': 0.9647715913163017, 'reg_alpha': 7.582572031522195, 'reg_lambda': 9.771164176162285, 'min_split_gain': 0.048921711870298284}. Best is trial 0 with value: 0.007013765339476785.
[Trial 0] New best: 0.00701
[I 2025-06-15 20:35:23,831] Trial 1 finished with value: 0.006683417530010459 and parameters: {'n_estimators': 78, 'learning_rate': 0.03204176871563938, 'num_leaves': 93, 'max_depth': 14, 'min_child_samples': 20, 'subsample': 0.8426116399676515, 'colsample_bytree': 0.8813413497366224, 'reg_alpha': 4.572442036662788, 'reg_lambda': 2.9904098240818087, 'min_split_gain': 0.023897210796883075}. Best is trial 1 with value: 0.006683417530010459.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:24,611] Trial 2 finished with value: 0.008306866742045576 and parameters: {'n_estimators': 183, 'learning_rate': 0.01339442935541787, 'num_leaves': 212, 'max_depth': 15, 'min_child_samples': 4, 'subsample': 0.772349584862685, 'colsample_bytree': 0.7556027699248562, 'reg_alpha': 6.147292497737055, 'reg_lambda': 0.9348362943489352, 'min_split_gain': 0.02462381390050654}. Best is trial 1 with value: 0.006683417530010459.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:25,242] Trial 3 finished with value: 0.10300251559991287 and parameters: {'n_estimators': 95, 'learning_rate': 0.011184868820930645, 'num_leaves': 141, 'max_depth': 8, 'min_child_samples': 16, 'subsample': 0.7850807015626082, 'colsample_bytree': 0.6841046557500732, 'reg_alpha': 1.7329879541517745, 'reg_lambda': 3.5442947313923865, 'min_split_gain': 0.04543420851577986}. Best is trial 1 with value: 0.006683417530010459.
[I 2025-06-15 20:35:25,246] A new study created in memory with name: no-name-2c640e25-05b1-4c44-bfc7-43e228e945bc
[I 2025-06-15 20:35:25,267] Trial 0 finished with value: 0.014250213678947174 and parameters: {'n_estimators': 112, 'learning_rate': 0.1567032504457767, 'num_leaves': 232, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.5087952000174172, 'colsample_bytree': 0.904376470805077, 'reg_alpha': 9.094618923296093, 'reg_lambda': 9.300600460884302, 'min_split_gain': 0.00934963777242856}. Best is trial 0 with value: 0.014250213678947174.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1139
1           Price_lag_2         745
14      pct_chg_Load_DA         435
16  roll2d_median_Price         242
2           Price_lag_7         191
11           NGas_lag_2         124
3         Load_DA_lag_0          85
10           Coal_lag_2          46
13            EUA_lag_2          29
15       lag168_Load_DA          29
12            Oil_lag_2          24
5       WindOn_DA_lag_0           7
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 75.10% done
********************* END NS.. 548 ****************************************************
********************* START NS ... 549 ****************************************************
[I 2025-06-15 20:35:26,747] A new study created in memory with name: no-name-814e5e8d-f8db-4d03-8720-1881aca9644a
[I 2025-06-15 20:35:26,884] Trial 0 finished with value: 0.00035521062355477927 and parameters: {'n_estimators': 129, 'learning_rate': 0.24846072812033468, 'num_leaves': 41, 'max_depth': 15, 'min_child_samples': 20, 'subsample': 0.8585940041328163, 'colsample_bytree': 0.7497194848725388, 'reg_alpha': 4.479323785160717, 'reg_lambda': 7.293589027616597, 'min_split_gain': 0.03366980581645865}. Best is trial 0 with value: 0.00035521062355477927.
[Trial 0] New best: 0.00036
[I 2025-06-15 20:35:27,636] Trial 1 finished with value: 0.0004976504546424377 and parameters: {'n_estimators': 261, 'learning_rate': 0.03112193105676234, 'num_leaves': 103, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.5317450863557714, 'colsample_bytree': 0.5712476233921133, 'reg_alpha': 5.279057914632417, 'reg_lambda': 3.1445804709951575, 'min_split_gain': 0.043283770772608476}. Best is trial 0 with value: 0.00035521062355477927.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:28,129] Trial 2 finished with value: 0.00025984733989354203 and parameters: {'n_estimators': 293, 'learning_rate': 0.036355814972106854, 'num_leaves': 34, 'max_depth': 4, 'min_child_samples': 28, 'subsample': 0.7986622128415326, 'colsample_bytree': 0.9874012196393417, 'reg_alpha': 5.926370472685947, 'reg_lambda': 2.9392863464184273, 'min_split_gain': 0.04656939874419749}. Best is trial 2 with value: 0.00025984733989354203.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:28,409] Trial 3 finished with value: 0.001142334166130191 and parameters: {'n_estimators': 60, 'learning_rate': 0.0535486741322797, 'num_leaves': 119, 'max_depth': 15, 'min_child_samples': 20, 'subsample': 0.6694960824017124, 'colsample_bytree': 0.6620357781628193, 'reg_alpha': 5.518985833044297, 'reg_lambda': 5.929416516139892, 'min_split_gain': 0.048088493774981256}. Best is trial 2 with value: 0.00025984733989354203.
[I 2025-06-15 20:35:28,411] A new study created in memory with name: no-name-ed5e442b-8f42-4936-9261-a0d80df47cb8
[I 2025-06-15 20:35:28,474] Trial 0 finished with value: 0.009217041640315482 and parameters: {'n_estimators': 258, 'learning_rate': 0.1763268240714341, 'num_leaves': 103, 'max_depth': 16, 'min_child_samples': 12, 'subsample': 0.9997277649056739, 'colsample_bytree': 0.5295094013148613, 'reg_alpha': 5.77256260262908, 'reg_lambda': 3.3422863648597048, 'min_split_gain': 0.02824532377164334}. Best is trial 0 with value: 0.009217041640315482.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1031
14      pct_chg_Load_DA         553
3         Load_DA_lag_0         270
11           NGas_lag_2         237
1           Price_lag_2         207
16  roll2d_median_Price         165
2           Price_lag_7         157
15       lag168_Load_DA         134
10           Coal_lag_2         110
5       WindOn_DA_lag_0          85
12            Oil_lag_2          72
13            EUA_lag_2          31
7                  WD_1           8
9                  WD_7           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 75.24% done
********************* END NS.. 549 ****************************************************
********************* START NS ... 550 ****************************************************
[I 2025-06-15 20:35:29,991] A new study created in memory with name: no-name-c8ceba5c-8eba-4cb1-a7f0-f4831c9c1142
[I 2025-06-15 20:35:30,165] Trial 0 finished with value: 0.0005139624015679842 and parameters: {'n_estimators': 63, 'learning_rate': 0.1389432815481359, 'num_leaves': 111, 'max_depth': 15, 'min_child_samples': 6, 'subsample': 0.6748855558315285, 'colsample_bytree': 0.772644501238297, 'reg_alpha': 2.8868839874936327, 'reg_lambda': 9.508975949419092, 'min_split_gain': 0.03496672448419252}. Best is trial 0 with value: 0.0005139624015679842.
[Trial 0] New best: 0.00051
[I 2025-06-15 20:35:31,019] Trial 1 finished with value: 0.0004457123893971666 and parameters: {'n_estimators': 193, 'learning_rate': 0.06925361596443624, 'num_leaves': 164, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.8798606021250216, 'colsample_bytree': 0.6283343697858824, 'reg_alpha': 0.1410865713527587, 'reg_lambda': 0.4897429276686538, 'min_split_gain': 0.02090104427512718}. Best is trial 1 with value: 0.0004457123893971666.
[I 2025-06-15 20:35:31,172] Trial 2 finished with value: 0.0004222573048700428 and parameters: {'n_estimators': 84, 'learning_rate': 0.29982023230241467, 'num_leaves': 115, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.9480597216779354, 'colsample_bytree': 0.8174774893411012, 'reg_alpha': 8.917862779076593, 'reg_lambda': 5.3162419986435445, 'min_split_gain': 0.012195051558219311}. Best is trial 2 with value: 0.0004222573048700428.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:31,409] Trial 3 finished with value: 0.0004125210697243876 and parameters: {'n_estimators': 67, 'learning_rate': 0.2071267546203425, 'num_leaves': 80, 'max_depth': 12, 'min_child_samples': 14, 'subsample': 0.6834324227651076, 'colsample_bytree': 0.5864354529150305, 'reg_alpha': 0.4471855048468487, 'reg_lambda': 0.7683869845317925, 'min_split_gain': 0.023692255461570052}. Best is trial 3 with value: 0.0004125210697243876.
[I 2025-06-15 20:35:31,412] A new study created in memory with name: no-name-236ba937-dab0-4965-ae1c-dfc572d0064c
[I 2025-06-15 20:35:31,441] Trial 0 finished with value: 0.009799991885445914 and parameters: {'n_estimators': 233, 'learning_rate': 0.06214272837076296, 'num_leaves': 199, 'max_depth': 11, 'min_child_samples': 15, 'subsample': 0.8535191656356782, 'colsample_bytree': 0.8206421956337329, 'reg_alpha': 3.938525177531628, 'reg_lambda': 0.3747536947227992, 'min_split_gain': 0.008595696736302073}. Best is trial 0 with value: 0.009799991885445914.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         463
1           Price_lag_2         389
3         Load_DA_lag_0         256
14      pct_chg_Load_DA         225
2           Price_lag_7         211
5       WindOn_DA_lag_0         196
16  roll2d_median_Price         194
10           Coal_lag_2         179
11           NGas_lag_2         177
15       lag168_Load_DA         167
12            Oil_lag_2         115
13            EUA_lag_2          66
9                  WD_7          13
7                  WD_1           8
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          22
1           Price_lag_2           3
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 75.38% done
********************* END NS.. 550 ****************************************************
********************* START NS ... 551 ****************************************************
[I 2025-06-15 20:35:33,175] A new study created in memory with name: no-name-30bc7ced-4b9f-4aca-b8ec-26cba5a7b87e
[I 2025-06-15 20:35:34,072] Trial 0 finished with value: 0.004552890019359562 and parameters: {'n_estimators': 292, 'learning_rate': 0.0189912984735699, 'num_leaves': 76, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.6461319688518876, 'colsample_bytree': 0.9810249118252792, 'reg_alpha': 4.563803120973691, 'reg_lambda': 9.029583543622952, 'min_split_gain': 0.0028192382043882138}. Best is trial 0 with value: 0.004552890019359562.
[Trial 0] New best: 0.00455
[I 2025-06-15 20:35:35,040] Trial 1 finished with value: 0.054829920311816015 and parameters: {'n_estimators': 147, 'learning_rate': 0.010110291180322802, 'num_leaves': 210, 'max_depth': 10, 'min_child_samples': 30, 'subsample': 0.5435174529040978, 'colsample_bytree': 0.8641239504536754, 'reg_alpha': 0.8092463298315422, 'reg_lambda': 2.448617972160705, 'min_split_gain': 0.0030408386523856413}. Best is trial 0 with value: 0.004552890019359562.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:35,614] Trial 2 finished with value: 0.004516095673255769 and parameters: {'n_estimators': 206, 'learning_rate': 0.043714649499518234, 'num_leaves': 32, 'max_depth': 9, 'min_child_samples': 16, 'subsample': 0.542330539880342, 'colsample_bytree': 0.7694031592541146, 'reg_alpha': 3.552320342233093, 'reg_lambda': 4.178034066564294, 'min_split_gain': 0.02918018659492191}. Best is trial 2 with value: 0.004516095673255769.
[I 2025-06-15 20:35:35,813] Trial 3 finished with value: 0.21419271037619855 and parameters: {'n_estimators': 50, 'learning_rate': 0.014536728007989993, 'num_leaves': 215, 'max_depth': 8, 'min_child_samples': 29, 'subsample': 0.9837168694378142, 'colsample_bytree': 0.755301213498966, 'reg_alpha': 8.893918345321776, 'reg_lambda': 6.299523648855966, 'min_split_gain': 0.04084218245493765}. Best is trial 2 with value: 0.004516095673255769.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:35,816] A new study created in memory with name: no-name-33eaaa65-7f92-437b-82bf-106b0f47f105
[I 2025-06-15 20:35:35,853] Trial 0 finished with value: 0.017170579155075655 and parameters: {'n_estimators': 173, 'learning_rate': 0.08706464567393918, 'num_leaves': 37, 'max_depth': 6, 'min_child_samples': 1, 'subsample': 0.8586664921171411, 'colsample_bytree': 0.8725553113916527, 'reg_alpha': 2.063488956803231, 'reg_lambda': 7.822306905280733, 'min_split_gain': 0.0022399596762887053}. Best is trial 0 with value: 0.017170579155075655.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1311
1           Price_lag_2         917
14      pct_chg_Load_DA         791
3         Load_DA_lag_0         593
11           NGas_lag_2         341
5       WindOn_DA_lag_0         333
2           Price_lag_7         322
16  roll2d_median_Price         294
15       lag168_Load_DA         292
10           Coal_lag_2         221
12            Oil_lag_2         124
13            EUA_lag_2          87
7                  WD_1          22
9                  WD_7          18
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          79
2           Price_lag_7          28
3         Load_DA_lag_0          10
1           Price_lag_2           6
14      pct_chg_Load_DA           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 75.51% done
********************* END NS.. 551 ****************************************************
********************* START NS ... 552 ****************************************************
[I 2025-06-15 20:35:37,576] A new study created in memory with name: no-name-0778cf3e-67d9-44d0-ad74-e2a1f1565935
[I 2025-06-15 20:35:38,637] Trial 0 finished with value: 0.011207140459547989 and parameters: {'n_estimators': 273, 'learning_rate': 0.014531743791971436, 'num_leaves': 198, 'max_depth': 11, 'min_child_samples': 9, 'subsample': 0.892450299478398, 'colsample_bytree': 0.6970136695182724, 'reg_alpha': 6.7836686980363154, 'reg_lambda': 8.525370377934616, 'min_split_gain': 0.04144244356131001}. Best is trial 0 with value: 0.011207140459547989.
[Trial 0] New best: 0.01121
[I 2025-06-15 20:35:38,863] Trial 1 finished with value: 0.007530758623189819 and parameters: {'n_estimators': 134, 'learning_rate': 0.19869504248354283, 'num_leaves': 185, 'max_depth': 9, 'min_child_samples': 8, 'subsample': 0.5900510567471688, 'colsample_bytree': 0.9994082376577219, 'reg_alpha': 6.592309025641152, 'reg_lambda': 2.743765456256125, 'min_split_gain': 0.020536132099891524}. Best is trial 1 with value: 0.007530758623189819.
[I 2025-06-15 20:35:39,035] Trial 2 finished with value: 0.007449222903369859 and parameters: {'n_estimators': 155, 'learning_rate': 0.298944542398833, 'num_leaves': 41, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.7280537461581573, 'colsample_bytree': 0.6593715077201276, 'reg_alpha': 1.4541831364481306, 'reg_lambda': 0.9514948800988587, 'min_split_gain': 0.03816005634440353}. Best is trial 2 with value: 0.007449222903369859.
[Trial 1] New best: 0.00753
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:39,238] Trial 3 finished with value: 0.007749963089274715 and parameters: {'n_estimators': 217, 'learning_rate': 0.1473117107484631, 'num_leaves': 139, 'max_depth': 7, 'min_child_samples': 21, 'subsample': 0.9591240140680386, 'colsample_bytree': 0.8559738101439032, 'reg_alpha': 7.396102623980807, 'reg_lambda': 9.758615475223134, 'min_split_gain': 0.044259349042394265}. Best is trial 2 with value: 0.007449222903369859.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:40,244] Trial 4 finished with value: 0.008127705801560338 and parameters: {'n_estimators': 267, 'learning_rate': 0.022117561470285534, 'num_leaves': 81, 'max_depth': 7, 'min_child_samples': 3, 'subsample': 0.7134909457113239, 'colsample_bytree': 0.8535535833262933, 'reg_alpha': 3.217578256177549, 'reg_lambda': 5.7611172116532465, 'min_split_gain': 0.03272282557346733}. Best is trial 2 with value: 0.007449222903369859.
[I 2025-06-15 20:35:40,246] A new study created in memory with name: no-name-e3e94e46-dd25-4ff0-8bf1-5a9068f3cf5c
[I 2025-06-15 20:35:40,280] Trial 0 finished with value: 0.04111455239385196 and parameters: {'n_estimators': 247, 'learning_rate': 0.19188341012586488, 'num_leaves': 63, 'max_depth': 9, 'min_child_samples': 27, 'subsample': 0.9800259509012328, 'colsample_bytree': 0.7732060905121925, 'reg_alpha': 2.940649181647771, 'reg_lambda': 0.36309382789837197, 'min_split_gain': 0.005087001831123561}. Best is trial 0 with value: 0.04111455239385196.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         240
1           Price_lag_2         123
3         Load_DA_lag_0          93
14      pct_chg_Load_DA          89
11           NGas_lag_2          78
5       WindOn_DA_lag_0          69
16  roll2d_median_Price          67
2           Price_lag_7          57
10           Coal_lag_2          54
15       lag168_Load_DA          37
13            EUA_lag_2          23
12            Oil_lag_2          18
9                  WD_7           5
7                  WD_1           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          19
1           Price_lag_2           3
10           Coal_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 75.65% done
********************* END NS.. 552 ****************************************************
********************* START NS ... 553 ****************************************************
[I 2025-06-15 20:35:41,763] A new study created in memory with name: no-name-8deee459-8ec2-465c-9a6a-35fd4d2f7768
[I 2025-06-15 20:35:41,970] Trial 0 finished with value: 0.005093811150346061 and parameters: {'n_estimators': 269, 'learning_rate': 0.16161195258438665, 'num_leaves': 148, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.664030964717687, 'colsample_bytree': 0.7093780067237814, 'reg_alpha': 8.024636545505508, 'reg_lambda': 6.493914660792889, 'min_split_gain': 0.013914722835037814}. Best is trial 0 with value: 0.005093811150346061.
[Trial 0] New best: 0.00509
[I 2025-06-15 20:35:42,506] Trial 1 finished with value: 0.03863083383127726 and parameters: {'n_estimators': 108, 'learning_rate': 0.017089010682007643, 'num_leaves': 124, 'max_depth': 8, 'min_child_samples': 5, 'subsample': 0.9368107408809362, 'colsample_bytree': 0.792257811171854, 'reg_alpha': 8.415420311143421, 'reg_lambda': 5.544893306059971, 'min_split_gain': 0.02061694405326302}. Best is trial 0 with value: 0.005093811150346061.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:43,066] Trial 2 finished with value: 0.005149262930813253 and parameters: {'n_estimators': 295, 'learning_rate': 0.06841273852983455, 'num_leaves': 31, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.8512866444529164, 'colsample_bytree': 0.748718900657904, 'reg_alpha': 0.514768946145685, 'reg_lambda': 1.2434101223453575, 'min_split_gain': 0.04955186522465495}. Best is trial 0 with value: 0.005093811150346061.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:43,420] Trial 3 finished with value: 0.005001635579492317 and parameters: {'n_estimators': 151, 'learning_rate': 0.07753925155872744, 'num_leaves': 242, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.8034627113027563, 'colsample_bytree': 0.8757492036490551, 'reg_alpha': 6.698720268401787, 'reg_lambda': 7.083190838228884, 'min_split_gain': 0.019909881909364765}. Best is trial 3 with value: 0.005001635579492317.
[I 2025-06-15 20:35:43,422] A new study created in memory with name: no-name-b9ce4c28-eb5b-4c96-bf8f-3fc450250df0
[I 2025-06-15 20:35:43,439] Trial 0 finished with value: 0.032842128537048564 and parameters: {'n_estimators': 74, 'learning_rate': 0.14874774004005287, 'num_leaves': 214, 'max_depth': 11, 'min_child_samples': 18, 'subsample': 0.7423907742694178, 'colsample_bytree': 0.6526920638605793, 'reg_alpha': 3.7288318590094773, 'reg_lambda': 1.175631527899389, 'min_split_gain': 0.02357759412367845}. Best is trial 0 with value: 0.032842128537048564.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         674
1           Price_lag_2         459
14      pct_chg_Load_DA         448
3         Load_DA_lag_0         251
11           NGas_lag_2         245
2           Price_lag_7         243
15       lag168_Load_DA         192
5       WindOn_DA_lag_0         171
16  roll2d_median_Price         170
10           Coal_lag_2         139
13            EUA_lag_2          89
12            Oil_lag_2          86
9                  WD_7          15
7                  WD_1          12
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          16
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 75.79% done
********************* END NS.. 553 ****************************************************
********************* START NS ... 554 ****************************************************
[I 2025-06-15 20:35:44,414] A new study created in memory with name: no-name-7750569f-79b2-4a95-8a8a-bdaf2e5133bd
[I 2025-06-15 20:35:45,325] Trial 0 finished with value: 0.004820150509269022 and parameters: {'n_estimators': 202, 'learning_rate': 0.019380921752878307, 'num_leaves': 181, 'max_depth': 8, 'min_child_samples': 29, 'subsample': 0.788831797986536, 'colsample_bytree': 0.7794908899645429, 'reg_alpha': 0.26793801031585796, 'reg_lambda': 9.994172216949487, 'min_split_gain': 0.025777750662727073}. Best is trial 0 with value: 0.004820150509269022.
[I 2025-06-15 20:35:45,524] Trial 1 finished with value: 0.004511007230706315 and parameters: {'n_estimators': 131, 'learning_rate': 0.14972881283654085, 'num_leaves': 133, 'max_depth': 6, 'min_child_samples': 10, 'subsample': 0.8994610808193417, 'colsample_bytree': 0.7916338229861593, 'reg_alpha': 4.657899607996324, 'reg_lambda': 4.582409164313663, 'min_split_gain': 0.01780532793445932}. Best is trial 1 with value: 0.004511007230706315.
[Trial 0] New best: 0.00482
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:46,056] Trial 2 finished with value: 0.0058152030574195115 and parameters: {'n_estimators': 172, 'learning_rate': 0.03979695564605919, 'num_leaves': 121, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.752586552702768, 'colsample_bytree': 0.6937765561215432, 'reg_alpha': 9.845035827785189, 'reg_lambda': 0.70667079877482, 'min_split_gain': 0.03123224378508669}. Best is trial 1 with value: 0.004511007230706315.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:46,544] Trial 3 finished with value: 0.0036091260202728063 and parameters: {'n_estimators': 158, 'learning_rate': 0.17035861143930545, 'num_leaves': 244, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.9587442748506656, 'colsample_bytree': 0.5215369216042791, 'reg_alpha': 1.1507124412306324, 'reg_lambda': 5.3739706027468515, 'min_split_gain': 0.010316083808880862}. Best is trial 3 with value: 0.0036091260202728063.
[I 2025-06-15 20:35:46,741] Trial 4 finished with value: 0.0035765788879699376 and parameters: {'n_estimators': 215, 'learning_rate': 0.28226091915472595, 'num_leaves': 155, 'max_depth': 4, 'min_child_samples': 15, 'subsample': 0.5116295030784166, 'colsample_bytree': 0.84495998290073, 'reg_alpha': 4.973498906006806, 'reg_lambda': 8.395109706243847, 'min_split_gain': 0.003170271832279592}. Best is trial 4 with value: 0.0035765788879699376.
[Trial 3] New best: 0.00361
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:47,396] Trial 5 finished with value: 0.00556173474585684 and parameters: {'n_estimators': 297, 'learning_rate': 0.046739714916340235, 'num_leaves': 72, 'max_depth': 9, 'min_child_samples': 26, 'subsample': 0.7684215933901947, 'colsample_bytree': 0.5763223548173321, 'reg_alpha': 7.000912339876687, 'reg_lambda': 7.354281715605485, 'min_split_gain': 0.049036734392174895}. Best is trial 4 with value: 0.0035765788879699376.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:47,870] Trial 6 finished with value: 0.004044588006132521 and parameters: {'n_estimators': 185, 'learning_rate': 0.07756258288347613, 'num_leaves': 97, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.6321546007155794, 'colsample_bytree': 0.8507644780156065, 'reg_alpha': 3.9469158418858608, 'reg_lambda': 3.1692521144885557, 'min_split_gain': 0.02818679662673213}. Best is trial 4 with value: 0.0035765788879699376.
[I 2025-06-15 20:35:47,873] A new study created in memory with name: no-name-0488f3e9-1dc1-4fe4-a87b-5f6c0d28f308
[I 2025-06-15 20:35:47,902] Trial 0 finished with value: 0.05896778978426349 and parameters: {'n_estimators': 122, 'learning_rate': 0.04646249156511179, 'num_leaves': 203, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.5360694797421135, 'colsample_bytree': 0.5272908164922099, 'reg_alpha': 4.361818260817807, 'reg_lambda': 1.0766134863385435, 'min_split_gain': 0.04372437750318592}. Best is trial 0 with value: 0.05896778978426349.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         249
1           Price_lag_2         203
14      pct_chg_Load_DA         124
5       WindOn_DA_lag_0          93
3         Load_DA_lag_0          85
11           NGas_lag_2          76
13            EUA_lag_2          68
2           Price_lag_7          67
12            Oil_lag_2          63
16  roll2d_median_Price          59
10           Coal_lag_2          53
15       lag168_Load_DA          52
7                  WD_1           6
9                  WD_7           4
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          35
1           Price_lag_2          15
10           Coal_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 75.92% done
********************* END NS.. 554 ****************************************************
********************* START NS ... 555 ****************************************************
[I 2025-06-15 20:35:49,506] A new study created in memory with name: no-name-883e7cbd-4d4b-423d-ac62-2131eeaeae2d
[I 2025-06-15 20:35:49,896] Trial 0 finished with value: 0.006459338538356574 and parameters: {'n_estimators': 93, 'learning_rate': 0.043387374480411904, 'num_leaves': 145, 'max_depth': 13, 'min_child_samples': 21, 'subsample': 0.6953404360602395, 'colsample_bytree': 0.5008064909626689, 'reg_alpha': 7.953967720595024, 'reg_lambda': 9.552730614286043, 'min_split_gain': 0.0005987457121036377}. Best is trial 0 with value: 0.006459338538356574.
[I 2025-06-15 20:35:50,050] Trial 1 finished with value: 0.0037491201440289085 and parameters: {'n_estimators': 105, 'learning_rate': 0.1709074114793768, 'num_leaves': 201, 'max_depth': 6, 'min_child_samples': 19, 'subsample': 0.8765165380693991, 'colsample_bytree': 0.8779720424787982, 'reg_alpha': 9.892928103109064, 'reg_lambda': 0.7843195361597921, 'min_split_gain': 0.004163890495809264}. Best is trial 1 with value: 0.0037491201440289085.
[Trial 0] New best: 0.00646
[Trial 1] New best: 0.00375
[I 2025-06-15 20:35:50,749] Trial 2 finished with value: 0.010516233001194765 and parameters: {'n_estimators': 263, 'learning_rate': 0.010668163133518696, 'num_leaves': 45, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.8973883683670014, 'colsample_bytree': 0.9447465736025091, 'reg_alpha': 5.302212984139119, 'reg_lambda': 3.8455861845678063, 'min_split_gain': 0.00849369264815925}. Best is trial 1 with value: 0.0037491201440289085.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:51,118] Trial 3 finished with value: 0.003120109666783009 and parameters: {'n_estimators': 100, 'learning_rate': 0.19169999635247836, 'num_leaves': 164, 'max_depth': 10, 'min_child_samples': 26, 'subsample': 0.5421171521603292, 'colsample_bytree': 0.811796837762376, 'reg_alpha': 5.281582697409069, 'reg_lambda': 0.1438000067116718, 'min_split_gain': 0.0021523952912388734}. Best is trial 3 with value: 0.003120109666783009.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:51,855] Trial 4 finished with value: 0.0045566165078989715 and parameters: {'n_estimators': 166, 'learning_rate': 0.02929212100080071, 'num_leaves': 65, 'max_depth': 13, 'min_child_samples': 7, 'subsample': 0.7628788111164445, 'colsample_bytree': 0.8388047818702105, 'reg_alpha': 9.155252961850257, 'reg_lambda': 0.960316775180271, 'min_split_gain': 0.0035143690192619816}. Best is trial 3 with value: 0.003120109666783009.
[I 2025-06-15 20:35:51,857] A new study created in memory with name: no-name-a8e84fb3-ca06-4074-bc84-93dc4b71f151
[I 2025-06-15 20:35:51,888] Trial 0 finished with value: 0.01132143511813049 and parameters: {'n_estimators': 120, 'learning_rate': 0.13851473743193887, 'num_leaves': 68, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.5372928107789261, 'colsample_bytree': 0.9812067048211324, 'reg_alpha': 2.6227999345093678, 'reg_lambda': 6.168591357603042, 'min_split_gain': 0.0017357525134509832}. Best is trial 0 with value: 0.01132143511813049.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         535
1           Price_lag_2         521
14      pct_chg_Load_DA         380
3         Load_DA_lag_0         314
15       lag168_Load_DA         279
5       WindOn_DA_lag_0         226
2           Price_lag_7         225
13            EUA_lag_2         191
16  roll2d_median_Price         176
11           NGas_lag_2         168
12            Oil_lag_2         151
10           Coal_lag_2         136
9                  WD_7          13
7                  WD_1          12
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          41
5       WindOn_DA_lag_0           3
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 76.06% done
********************* END NS.. 555 ****************************************************
********************* START NS ... 556 ****************************************************
[I 2025-06-15 20:35:52,785] A new study created in memory with name: no-name-753025d9-ebe9-4bea-9621-69e4225eeef1
[I 2025-06-15 20:35:52,996] Trial 0 finished with value: 0.0005011516931473164 and parameters: {'n_estimators': 120, 'learning_rate': 0.21724318138655704, 'num_leaves': 136, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.7199104858909253, 'colsample_bytree': 0.6778407966399558, 'reg_alpha': 3.66714433477759, 'reg_lambda': 8.451296407819836, 'min_split_gain': 0.010106509902860451}. Best is trial 0 with value: 0.0005011516931473164.
[I 2025-06-15 20:35:53,198] Trial 1 finished with value: 0.0014258620219551044 and parameters: {'n_estimators': 164, 'learning_rate': 0.2789331728893855, 'num_leaves': 229, 'max_depth': 13, 'min_child_samples': 2, 'subsample': 0.8710235156303385, 'colsample_bytree': 0.5444848880677616, 'reg_alpha': 3.4992762258011343, 'reg_lambda': 6.490892671625729, 'min_split_gain': 0.02820367838331694}. Best is trial 0 with value: 0.0005011516931473164.
[Trial 0] New best: 0.00050
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:35:53,774] Trial 2 finished with value: 0.0005085506015945992 and parameters: {'n_estimators': 103, 'learning_rate': 0.041048992426936846, 'num_leaves': 140, 'max_depth': 13, 'min_child_samples': 16, 'subsample': 0.8161521720496212, 'colsample_bytree': 0.7068630730850847, 'reg_alpha': 1.5395457007138968, 'reg_lambda': 2.28476631296598, 'min_split_gain': 0.029509458165726716}. Best is trial 0 with value: 0.0005011516931473164.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:54,274] Trial 3 finished with value: 0.0004902503157376777 and parameters: {'n_estimators': 299, 'learning_rate': 0.07873225940282502, 'num_leaves': 80, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.699422134515209, 'colsample_bytree': 0.8867480096496692, 'reg_alpha': 0.8253482933038947, 'reg_lambda': 2.1521725751616283, 'min_split_gain': 0.035888559634294694}. Best is trial 3 with value: 0.0004902503157376777.
[I 2025-06-15 20:35:54,277] A new study created in memory with name: no-name-05983e2b-3a5d-4907-819e-0984c592dc35
[I 2025-06-15 20:35:54,302] Trial 0 finished with value: 0.033336165236264426 and parameters: {'n_estimators': 156, 'learning_rate': 0.12558941987907823, 'num_leaves': 135, 'max_depth': 9, 'min_child_samples': 4, 'subsample': 0.6232821222245927, 'colsample_bytree': 0.9930188482632276, 'reg_alpha': 9.021972074347357, 'reg_lambda': 8.807200277719607, 'min_split_gain': 0.029239025981443685}. Best is trial 0 with value: 0.033336165236264426.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         950
1           Price_lag_2         765
14      pct_chg_Load_DA         686
3         Load_DA_lag_0         376
16  roll2d_median_Price         367
2           Price_lag_7         325
11           NGas_lag_2         306
5       WindOn_DA_lag_0         260
15       lag168_Load_DA         259
10           Coal_lag_2         185
13            EUA_lag_2         113
12            Oil_lag_2          88
9                  WD_7          15
7                  WD_1          14
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          12
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 76.20% done
********************* END NS.. 556 ****************************************************
********************* START NS ... 557 ****************************************************
[I 2025-06-15 20:35:55,482] A new study created in memory with name: no-name-b594c505-011f-4570-baab-6b61657ec448
[I 2025-06-15 20:35:55,911] Trial 0 finished with value: 0.04397792872914325 and parameters: {'n_estimators': 131, 'learning_rate': 0.010014189822188458, 'num_leaves': 80, 'max_depth': 10, 'min_child_samples': 29, 'subsample': 0.6510285197940617, 'colsample_bytree': 0.5258818405077559, 'reg_alpha': 2.198924627993444, 'reg_lambda': 6.997020257097582, 'min_split_gain': 0.004418186722833522}. Best is trial 0 with value: 0.04397792872914325.
[Trial 0] New best: 0.04398
[I 2025-06-15 20:35:56,353] Trial 1 finished with value: 0.0019545396526193907 and parameters: {'n_estimators': 98, 'learning_rate': 0.02879746168825753, 'num_leaves': 211, 'max_depth': 9, 'min_child_samples': 19, 'subsample': 0.9351726488601928, 'colsample_bytree': 0.6610721448399957, 'reg_alpha': 3.3280644012605642, 'reg_lambda': 2.6112721956807174, 'min_split_gain': 0.006505556701815929}. Best is trial 1 with value: 0.0019545396526193907.
[Trial 1] New best: 0.00195
[I 2025-06-15 20:35:57,739] Trial 2 finished with value: 0.0003762995349974562 and parameters: {'n_estimators': 235, 'learning_rate': 0.07954118855246552, 'num_leaves': 161, 'max_depth': 15, 'min_child_samples': 10, 'subsample': 0.5567945151088352, 'colsample_bytree': 0.5744138372995657, 'reg_alpha': 0.3905269211315887, 'reg_lambda': 8.314560937032544, 'min_split_gain': 0.0047014183285571725}. Best is trial 2 with value: 0.0003762995349974562.
[Trial 2] New best: 0.00038
[I 2025-06-15 20:35:58,994] Trial 3 finished with value: 0.0005552458715530686 and parameters: {'n_estimators': 278, 'learning_rate': 0.013395473668732224, 'num_leaves': 38, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.6294847817473073, 'colsample_bytree': 0.9639195931036861, 'reg_alpha': 9.780909331332609, 'reg_lambda': 1.7150089026637094, 'min_split_gain': 0.009386592274084199}. Best is trial 2 with value: 0.0003762995349974562.
[I 2025-06-15 20:35:59,150] Trial 4 finished with value: 0.0007757804270199954 and parameters: {'n_estimators': 65, 'learning_rate': 0.2768264530391792, 'num_leaves': 52, 'max_depth': 13, 'min_child_samples': 4, 'subsample': 0.7549857418878287, 'colsample_bytree': 0.5174850449961703, 'reg_alpha': 9.358557208617267, 'reg_lambda': 4.947285612086317, 'min_split_gain': 0.04593891873198927}. Best is trial 2 with value: 0.0003762995349974562.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:35:59,678] Trial 5 finished with value: 0.002026319646342309 and parameters: {'n_estimators': 157, 'learning_rate': 0.01794064135542069, 'num_leaves': 129, 'max_depth': 7, 'min_child_samples': 14, 'subsample': 0.7663408695448067, 'colsample_bytree': 0.7695733064399567, 'reg_alpha': 5.487652560916104, 'reg_lambda': 1.5257027890220065, 'min_split_gain': 0.04367633327238129}. Best is trial 2 with value: 0.0003762995349974562.
[I 2025-06-15 20:35:59,681] A new study created in memory with name: no-name-3c4797c9-8aa5-426c-9439-a88eb5fe61c0
[I 2025-06-15 20:35:59,708] Trial 0 finished with value: 0.007861283923341426 and parameters: {'n_estimators': 110, 'learning_rate': 0.09794032342802204, 'num_leaves': 71, 'max_depth': 8, 'min_child_samples': 16, 'subsample': 0.7834408319286941, 'colsample_bytree': 0.5356626762094441, 'reg_alpha': 0.9388969828529015, 'reg_lambda': 3.515927979833142, 'min_split_gain': 0.03920642242500297}. Best is trial 0 with value: 0.007861283923341426.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        2685
14      pct_chg_Load_DA        2532
0           Price_lag_1        2343
5       WindOn_DA_lag_0        2273
3         Load_DA_lag_0        2186
15       lag168_Load_DA        1855
2           Price_lag_7        1698
16  roll2d_median_Price        1582
11           NGas_lag_2        1438
13            EUA_lag_2        1301
10           Coal_lag_2        1135
12            Oil_lag_2        1031
9                  WD_7          72
7                  WD_1          38
8                  WD_6          25
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          24
1           Price_lag_2          11
5       WindOn_DA_lag_0           5
15       lag168_Load_DA           3
14      pct_chg_Load_DA           2
10           Coal_lag_2           1
12            Oil_lag_2           1
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 76.33% done
********************* END NS.. 557 ****************************************************
********************* START NS ... 558 ****************************************************
[I 2025-06-15 20:36:01,689] A new study created in memory with name: no-name-803a9c1d-71bf-495d-b897-c1bc3ef6d2e2
[I 2025-06-15 20:36:02,185] Trial 0 finished with value: 0.00023094749734982206 and parameters: {'n_estimators': 300, 'learning_rate': 0.23291752336465227, 'num_leaves': 160, 'max_depth': 12, 'min_child_samples': 18, 'subsample': 0.6406083964250638, 'colsample_bytree': 0.6438478722813163, 'reg_alpha': 0.4815183890540986, 'reg_lambda': 3.627417241641482, 'min_split_gain': 0.0027970627278844475}. Best is trial 0 with value: 0.00023094749734982206.
[Trial 0] New best: 0.00023
[I 2025-06-15 20:36:03,230] Trial 1 finished with value: 0.000666369390737611 and parameters: {'n_estimators': 258, 'learning_rate': 0.012890899565979592, 'num_leaves': 64, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.765205394339322, 'colsample_bytree': 0.8657021212016763, 'reg_alpha': 3.887921192483671, 'reg_lambda': 1.8558747927031805, 'min_split_gain': 0.01159379640819419}. Best is trial 0 with value: 0.00023094749734982206.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:03,703] Trial 2 finished with value: 0.11069243743689194 and parameters: {'n_estimators': 63, 'learning_rate': 0.013628508487485721, 'num_leaves': 213, 'max_depth': 16, 'min_child_samples': 1, 'subsample': 0.5151158880464721, 'colsample_bytree': 0.9821102059441532, 'reg_alpha': 5.837160545509608, 'reg_lambda': 7.34762422073648, 'min_split_gain': 0.012988935328622676}. Best is trial 0 with value: 0.00023094749734982206.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:04,058] Trial 3 finished with value: 0.0003485058966894907 and parameters: {'n_estimators': 175, 'learning_rate': 0.08934811928257677, 'num_leaves': 197, 'max_depth': 4, 'min_child_samples': 28, 'subsample': 0.8162726500417141, 'colsample_bytree': 0.8214473006449111, 'reg_alpha': 5.6401383967832945, 'reg_lambda': 1.5238609250334911, 'min_split_gain': 0.01439234326950361}. Best is trial 0 with value: 0.00023094749734982206.
[I 2025-06-15 20:36:04,060] A new study created in memory with name: no-name-52ae347b-2eb5-4dd8-a4e9-8a84ade2e6a3
[I 2025-06-15 20:36:04,100] Trial 0 finished with value: 0.022583875092396885 and parameters: {'n_estimators': 220, 'learning_rate': 0.1550585783265909, 'num_leaves': 109, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.9972715411916344, 'colsample_bytree': 0.5708495769327553, 'reg_alpha': 9.002072735503926, 'reg_lambda': 8.125991581127174, 'min_split_gain': 0.0063527809698892546}. Best is trial 0 with value: 0.022583875092396885.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         940
3         Load_DA_lag_0         919
1           Price_lag_2         898
14      pct_chg_Load_DA         886
5       WindOn_DA_lag_0         791
15       lag168_Load_DA         775
2           Price_lag_7         656
16  roll2d_median_Price         533
11           NGas_lag_2         519
13            EUA_lag_2         411
10           Coal_lag_2         386
12            Oil_lag_2         380
9                  WD_7          20
7                  WD_1          13
8                  WD_6          13
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          12
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 76.47% done
********************* END NS.. 558 ****************************************************
********************* START NS ... 559 ****************************************************
[I 2025-06-15 20:36:06,018] A new study created in memory with name: no-name-254b1abd-7c96-4c55-82d5-15d9561ac2d8
[I 2025-06-15 20:36:06,411] Trial 0 finished with value: 0.00012097941016617852 and parameters: {'n_estimators': 123, 'learning_rate': 0.03634284374388934, 'num_leaves': 151, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.7689430670677155, 'colsample_bytree': 0.9871683905676231, 'reg_alpha': 4.5140524027207025, 'reg_lambda': 6.727460350871661, 'min_split_gain': 0.014316363085172479}. Best is trial 0 with value: 0.00012097941016617852.
[Trial 0] New best: 0.00012
[I 2025-06-15 20:36:07,221] Trial 1 finished with value: 0.00021320040319722103 and parameters: {'n_estimators': 216, 'learning_rate': 0.03882338632530732, 'num_leaves': 66, 'max_depth': 16, 'min_child_samples': 1, 'subsample': 0.5519691508925425, 'colsample_bytree': 0.8710556918904078, 'reg_alpha': 3.859533062024272, 'reg_lambda': 8.653892109312846, 'min_split_gain': 0.024092362707663875}. Best is trial 0 with value: 0.00012097941016617852.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:08,157] Trial 2 finished with value: 0.00010732849738701889 and parameters: {'n_estimators': 271, 'learning_rate': 0.016771884506594295, 'num_leaves': 187, 'max_depth': 7, 'min_child_samples': 14, 'subsample': 0.9982055864032013, 'colsample_bytree': 0.7959248830461009, 'reg_alpha': 5.14315603705368, 'reg_lambda': 9.173611323869906, 'min_split_gain': 0.008724955568578862}. Best is trial 2 with value: 0.00010732849738701889.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:08,383] Trial 3 finished with value: 0.00024909197074494456 and parameters: {'n_estimators': 100, 'learning_rate': 0.22785844535776106, 'num_leaves': 74, 'max_depth': 8, 'min_child_samples': 1, 'subsample': 0.6413742281066463, 'colsample_bytree': 0.8599727201589367, 'reg_alpha': 2.7263020454316345, 'reg_lambda': 6.117622408874891, 'min_split_gain': 0.014570411014978975}. Best is trial 2 with value: 0.00010732849738701889.
[I 2025-06-15 20:36:08,384] A new study created in memory with name: no-name-811ad0eb-aa3e-44b1-a73a-f0bf46b79fcd
[I 2025-06-15 20:36:08,415] Trial 0 finished with value: 9.762130846067872e-05 and parameters: {'n_estimators': 188, 'learning_rate': 0.16553254844792048, 'num_leaves': 210, 'max_depth': 4, 'min_child_samples': 6, 'subsample': 0.6102197132221361, 'colsample_bytree': 0.8656202310631333, 'reg_alpha': 0.08297449988314831, 'reg_lambda': 6.419962676622094, 'min_split_gain': 0.03142815766469994}. Best is trial 0 with value: 9.762130846067872e-05.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2938
1           Price_lag_2        2143
14      pct_chg_Load_DA        1544
16  roll2d_median_Price         724
2           Price_lag_7         693
3         Load_DA_lag_0         623
11           NGas_lag_2         610
13            EUA_lag_2         397
15       lag168_Load_DA         394
12            Oil_lag_2         345
5       WindOn_DA_lag_0         274
10           Coal_lag_2         216
9                  WD_7          31
8                  WD_6          31
7                  WD_1           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          21
15       lag168_Load_DA           3
5       WindOn_DA_lag_0           2
1           Price_lag_2           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 76.61% done
********************* END NS.. 559 ****************************************************
********************* START NS ... 560 ****************************************************
[I 2025-06-15 20:36:10,619] A new study created in memory with name: no-name-c89f1f59-2905-4105-99ff-9845dcd5a8d4
[I 2025-06-15 20:36:10,811] Trial 0 finished with value: 0.0024803832694554337 and parameters: {'n_estimators': 84, 'learning_rate': 0.10082874824744897, 'num_leaves': 97, 'max_depth': 9, 'min_child_samples': 29, 'subsample': 0.5463866559363818, 'colsample_bytree': 0.5560297872067741, 'reg_alpha': 9.746204293693628, 'reg_lambda': 1.3918682332072518, 'min_split_gain': 0.035286285945634405}. Best is trial 0 with value: 0.0024803832694554337.
[Trial 0] New best: 0.00248
[I 2025-06-15 20:36:11,326] Trial 1 finished with value: 0.0021814492607985804 and parameters: {'n_estimators': 124, 'learning_rate': 0.05163885342377649, 'num_leaves': 199, 'max_depth': 16, 'min_child_samples': 14, 'subsample': 0.9337961220310367, 'colsample_bytree': 0.6835159597796794, 'reg_alpha': 9.594481558522034, 'reg_lambda': 2.6646886277580686, 'min_split_gain': 0.0034483702402957565}. Best is trial 1 with value: 0.0021814492607985804.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:11,654] Trial 2 finished with value: 0.0018116005498184928 and parameters: {'n_estimators': 297, 'learning_rate': 0.08721874988586094, 'num_leaves': 186, 'max_depth': 6, 'min_child_samples': 21, 'subsample': 0.6911230788859732, 'colsample_bytree': 0.7830684667507337, 'reg_alpha': 3.964834839149556, 'reg_lambda': 4.808974288051519, 'min_split_gain': 0.034752364939506855}. Best is trial 2 with value: 0.0018116005498184928.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:12,050] Trial 3 finished with value: 0.19364474835293308 and parameters: {'n_estimators': 53, 'learning_rate': 0.013829751815469518, 'num_leaves': 133, 'max_depth': 12, 'min_child_samples': 4, 'subsample': 0.6219534897226684, 'colsample_bytree': 0.6933212163934424, 'reg_alpha': 2.287402158331411, 'reg_lambda': 0.12842269174301757, 'min_split_gain': 0.042126109785890896}. Best is trial 2 with value: 0.0018116005498184928.
[I 2025-06-15 20:36:12,052] A new study created in memory with name: no-name-28792ae9-198c-4e04-a14c-020c39548184
[I 2025-06-15 20:36:12,069] Trial 0 finished with value: 0.014678014287043467 and parameters: {'n_estimators': 65, 'learning_rate': 0.09654715467505186, 'num_leaves': 227, 'max_depth': 16, 'min_child_samples': 24, 'subsample': 0.5237281638210152, 'colsample_bytree': 0.9215414772098509, 'reg_alpha': 5.110292311591458, 'reg_lambda': 8.094364956281412, 'min_split_gain': 0.017572166943255235}. Best is trial 0 with value: 0.014678014287043467.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         644
1           Price_lag_2         404
14      pct_chg_Load_DA         365
3         Load_DA_lag_0         222
11           NGas_lag_2         177
16  roll2d_median_Price         139
2           Price_lag_7         135
15       lag168_Load_DA         119
5       WindOn_DA_lag_0         114
10           Coal_lag_2          78
13            EUA_lag_2          63
12            Oil_lag_2          56
9                  WD_7           8
7                  WD_1           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          20
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 76.74% done
********************* END NS.. 560 ****************************************************
********************* START NS ... 561 ****************************************************
[I 2025-06-15 20:36:13,293] A new study created in memory with name: no-name-a3701145-d994-4c86-b469-43b4477b71b6
[I 2025-06-15 20:36:13,430] Trial 0 finished with value: 0.0041212604564216035 and parameters: {'n_estimators': 256, 'learning_rate': 0.2815868919612816, 'num_leaves': 86, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.652907773924253, 'colsample_bytree': 0.9731940480491319, 'reg_alpha': 8.565143615907738, 'reg_lambda': 4.602684237254789, 'min_split_gain': 0.03884289930603586}. Best is trial 0 with value: 0.0041212604564216035.
[Trial 0] New best: 0.00412
[I 2025-06-15 20:36:13,787] Trial 1 finished with value: 0.0035026164515740664 and parameters: {'n_estimators': 259, 'learning_rate': 0.04575798908012462, 'num_leaves': 61, 'max_depth': 9, 'min_child_samples': 6, 'subsample': 0.9410346303634107, 'colsample_bytree': 0.9190570222416623, 'reg_alpha': 7.013035948064834, 'reg_lambda': 4.020769949414583, 'min_split_gain': 0.03958430145161182}. Best is trial 1 with value: 0.0035026164515740664.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:14,679] Trial 2 finished with value: 0.004779012993925799 and parameters: {'n_estimators': 181, 'learning_rate': 0.01838025387623863, 'num_leaves': 92, 'max_depth': 14, 'min_child_samples': 15, 'subsample': 0.546642162284667, 'colsample_bytree': 0.9868949555087873, 'reg_alpha': 1.4287740626867296, 'reg_lambda': 4.2912417470243325, 'min_split_gain': 0.03242333390284923}. Best is trial 1 with value: 0.0035026164515740664.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:14,930] Trial 3 finished with value: 0.0022034135146580553 and parameters: {'n_estimators': 111, 'learning_rate': 0.13481249954482888, 'num_leaves': 251, 'max_depth': 5, 'min_child_samples': 13, 'subsample': 0.7574495593684691, 'colsample_bytree': 0.9445997191617637, 'reg_alpha': 1.5624551102318196, 'reg_lambda': 7.202253543851418, 'min_split_gain': 0.010504401099959027}. Best is trial 3 with value: 0.0022034135146580553.
[Trial 3] New best: 0.00220
[I 2025-06-15 20:36:15,191] Trial 4 finished with value: 0.0037820429366035614 and parameters: {'n_estimators': 265, 'learning_rate': 0.14400490263162685, 'num_leaves': 229, 'max_depth': 5, 'min_child_samples': 4, 'subsample': 0.8478112864996598, 'colsample_bytree': 0.523514581531586, 'reg_alpha': 3.134018109460963, 'reg_lambda': 3.650088521186027, 'min_split_gain': 0.03169127294367866}. Best is trial 3 with value: 0.0022034135146580553.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:15,493] Trial 5 finished with value: 0.003800352060018896 and parameters: {'n_estimators': 155, 'learning_rate': 0.0915457069031825, 'num_leaves': 108, 'max_depth': 9, 'min_child_samples': 24, 'subsample': 0.9444193757830288, 'colsample_bytree': 0.6564800907335575, 'reg_alpha': 4.215679580855857, 'reg_lambda': 3.5298637049737636, 'min_split_gain': 0.03243133782811091}. Best is trial 3 with value: 0.0022034135146580553.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:16,043] Trial 6 finished with value: 0.0041211186372313285 and parameters: {'n_estimators': 74, 'learning_rate': 0.06231858577105922, 'num_leaves': 133, 'max_depth': 15, 'min_child_samples': 9, 'subsample': 0.8752883042414674, 'colsample_bytree': 0.5839504029985503, 'reg_alpha': 1.0203418028520472, 'reg_lambda': 2.7380460120835237, 'min_split_gain': 0.012485575652202825}. Best is trial 3 with value: 0.0022034135146580553.
[I 2025-06-15 20:36:16,045] A new study created in memory with name: no-name-1566de76-8adc-4217-8622-83bbc606ec95
[I 2025-06-15 20:36:16,074] Trial 0 finished with value: 0.05578785510093431 and parameters: {'n_estimators': 274, 'learning_rate': 0.03413435249792689, 'num_leaves': 160, 'max_depth': 15, 'min_child_samples': 17, 'subsample': 0.7995081186534707, 'colsample_bytree': 0.7071428700159874, 'reg_alpha': 5.818288638556675, 'reg_lambda': 2.2578356161590785, 'min_split_gain': 0.04107995379420049}. Best is trial 0 with value: 0.05578785510093431.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         528
14      pct_chg_Load_DA         366
1           Price_lag_2         326
3         Load_DA_lag_0         232
5       WindOn_DA_lag_0         178
2           Price_lag_7         177
11           NGas_lag_2         166
15       lag168_Load_DA         162
16  roll2d_median_Price         162
10           Coal_lag_2         101
12            Oil_lag_2          85
13            EUA_lag_2          79
7                  WD_1          11
9                  WD_7          10
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 76.88% done
********************* END NS.. 561 ****************************************************
********************* START NS ... 562 ****************************************************
[I 2025-06-15 20:36:17,400] A new study created in memory with name: no-name-7ab25e7d-7a90-4d6d-8871-bd23a0c50b53
[I 2025-06-15 20:36:17,567] Trial 0 finished with value: 0.0025894675847890907 and parameters: {'n_estimators': 223, 'learning_rate': 0.11093582239300138, 'num_leaves': 149, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.8182522181359677, 'colsample_bytree': 0.9107182152744349, 'reg_alpha': 5.856416624419634, 'reg_lambda': 0.9691162024265487, 'min_split_gain': 0.04216566144280809}. Best is trial 0 with value: 0.0025894675847890907.
[I 2025-06-15 20:36:17,643] Trial 1 finished with value: 0.0027890091660697724 and parameters: {'n_estimators': 69, 'learning_rate': 0.2370141072296512, 'num_leaves': 190, 'max_depth': 15, 'min_child_samples': 19, 'subsample': 0.7049238920660923, 'colsample_bytree': 0.7080955451628292, 'reg_alpha': 3.217466355535076, 'reg_lambda': 5.436552987484388, 'min_split_gain': 0.04828651289170358}. Best is trial 0 with value: 0.0025894675847890907.
[Trial 0] New best: 0.00259
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:17,956] Trial 2 finished with value: 0.0031173768507486168 and parameters: {'n_estimators': 292, 'learning_rate': 0.017747463670730837, 'num_leaves': 239, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.9622940086917307, 'colsample_bytree': 0.982195096053776, 'reg_alpha': 9.82059383651629, 'reg_lambda': 2.026891066930796, 'min_split_gain': 0.03981104473925245}. Best is trial 0 with value: 0.0025894675847890907.
[I 2025-06-15 20:36:18,137] Trial 3 finished with value: 0.0027965011231926317 and parameters: {'n_estimators': 154, 'learning_rate': 0.12438621881945092, 'num_leaves': 239, 'max_depth': 14, 'min_child_samples': 14, 'subsample': 0.8251603158655547, 'colsample_bytree': 0.7372670742198932, 'reg_alpha': 8.660811772416674, 'reg_lambda': 5.07116102083374, 'min_split_gain': 0.04471116890154833}. Best is trial 0 with value: 0.0025894675847890907.
[I 2025-06-15 20:36:18,140] A new study created in memory with name: no-name-c078d843-fd1a-448c-a42e-46b4edcb9fb1
[I 2025-06-15 20:36:18,158] Trial 0 finished with value: 0.02693154312990505 and parameters: {'n_estimators': 59, 'learning_rate': 0.14935239830288413, 'num_leaves': 162, 'max_depth': 16, 'min_child_samples': 20, 'subsample': 0.7111052636982856, 'colsample_bytree': 0.5598449232662934, 'reg_alpha': 2.0667465006170316, 'reg_lambda': 3.702757666139452, 'min_split_gain': 0.026860965628714917}. Best is trial 0 with value: 0.02693154312990505.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         405
14      pct_chg_Load_DA         238
1           Price_lag_2         133
3         Load_DA_lag_0         103
11           NGas_lag_2          98
2           Price_lag_7          91
16  roll2d_median_Price          77
15       lag168_Load_DA          67
5       WindOn_DA_lag_0          66
10           Coal_lag_2          44
12            Oil_lag_2          34
13            EUA_lag_2          21
9                  WD_7           6
7                  WD_1           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          12
1           Price_lag_2           3
11           NGas_lag_2           1
16  roll2d_median_Price           1
2           Price_lag_7           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
3         Load_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 77.02% done
********************* END NS.. 562 ****************************************************
********************* START NS ... 563 ****************************************************
[I 2025-06-15 20:36:19,375] A new study created in memory with name: no-name-e4b7e101-9e63-4992-82df-af71e7d956b5
[I 2025-06-15 20:36:19,662] Trial 0 finished with value: 0.0010889475796943178 and parameters: {'n_estimators': 288, 'learning_rate': 0.061900419034544324, 'num_leaves': 253, 'max_depth': 14, 'min_child_samples': 27, 'subsample': 0.7955174793847859, 'colsample_bytree': 0.9718467571362291, 'reg_alpha': 2.7397519252785854, 'reg_lambda': 3.7701567181621467, 'min_split_gain': 0.0472940634590146}. Best is trial 0 with value: 0.0010889475796943178.
[Trial 0] New best: 0.00109
[I 2025-06-15 20:36:20,026] Trial 1 finished with value: 0.0011155000243988134 and parameters: {'n_estimators': 296, 'learning_rate': 0.1227443836716046, 'num_leaves': 217, 'max_depth': 11, 'min_child_samples': 19, 'subsample': 0.539403013920437, 'colsample_bytree': 0.5865588689085139, 'reg_alpha': 7.745282700861006, 'reg_lambda': 4.760646690491481, 'min_split_gain': 0.006300578943206636}. Best is trial 0 with value: 0.0010889475796943178.
[I 2025-06-15 20:36:20,205] Trial 2 finished with value: 0.001175122141577596 and parameters: {'n_estimators': 177, 'learning_rate': 0.25856729320852556, 'num_leaves': 157, 'max_depth': 12, 'min_child_samples': 30, 'subsample': 0.6511048729342988, 'colsample_bytree': 0.7525136376419703, 'reg_alpha': 2.4858648642291303, 'reg_lambda': 7.638284728047252, 'min_split_gain': 0.03096854301744233}. Best is trial 0 with value: 0.0010889475796943178.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:20,654] Trial 3 finished with value: 0.0013539581013769081 and parameters: {'n_estimators': 169, 'learning_rate': 0.02693168561750786, 'num_leaves': 238, 'max_depth': 8, 'min_child_samples': 25, 'subsample': 0.6448799716390655, 'colsample_bytree': 0.7414004279239577, 'reg_alpha': 9.564127948787787, 'reg_lambda': 8.663792351663874, 'min_split_gain': 0.0335890816901848}. Best is trial 0 with value: 0.0010889475796943178.
[I 2025-06-15 20:36:20,656] A new study created in memory with name: no-name-8c20917e-5768-4773-8678-56e1299e4b60
[I 2025-06-15 20:36:20,684] Trial 0 finished with value: 0.004500551962880767 and parameters: {'n_estimators': 233, 'learning_rate': 0.12356882815097092, 'num_leaves': 191, 'max_depth': 4, 'min_child_samples': 26, 'subsample': 0.8145120747188506, 'colsample_bytree': 0.5874346680738443, 'reg_alpha': 6.028883978977596, 'reg_lambda': 6.144976847407964, 'min_split_gain': 0.023974585439088203}. Best is trial 0 with value: 0.004500551962880767.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1087
14      pct_chg_Load_DA         700
1           Price_lag_2         593
3         Load_DA_lag_0         401
2           Price_lag_7         324
16  roll2d_median_Price         287
11           NGas_lag_2         280
5       WindOn_DA_lag_0         206
15       lag168_Load_DA         191
10           Coal_lag_2         131
12            Oil_lag_2         110
13            EUA_lag_2          67
7                  WD_1          15
9                  WD_7          14
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 77.15% done
********************* END NS.. 563 ****************************************************
********************* START NS ... 564 ****************************************************
[I 2025-06-15 20:36:21,836] A new study created in memory with name: no-name-e3473214-be8e-4a4d-aca2-cde9303603b5
[I 2025-06-15 20:36:22,180] Trial 0 finished with value: 0.0012574842110574214 and parameters: {'n_estimators': 293, 'learning_rate': 0.029593990880629303, 'num_leaves': 237, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.7732685965438534, 'colsample_bytree': 0.9310223205583745, 'reg_alpha': 8.285279171930327, 'reg_lambda': 9.946809799678189, 'min_split_gain': 0.035534824605146835}. Best is trial 0 with value: 0.0012574842110574214.
[I 2025-06-15 20:36:22,376] Trial 1 finished with value: 0.0013431964636831826 and parameters: {'n_estimators': 111, 'learning_rate': 0.10114535694769557, 'num_leaves': 33, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.9928911126784827, 'colsample_bytree': 0.5491571880076227, 'reg_alpha': 4.613176914237758, 'reg_lambda': 8.044753890519713, 'min_split_gain': 0.047074351323531376}. Best is trial 0 with value: 0.0012574842110574214.
[Trial 0] New best: 0.00126
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:22,606] Trial 2 finished with value: 0.0012950604283815642 and parameters: {'n_estimators': 268, 'learning_rate': 0.1380761637391244, 'num_leaves': 236, 'max_depth': 12, 'min_child_samples': 10, 'subsample': 0.9275472216378385, 'colsample_bytree': 0.9168359732145352, 'reg_alpha': 7.1823188251624845, 'reg_lambda': 3.058644527362885, 'min_split_gain': 0.025855464508056137}. Best is trial 0 with value: 0.0012574842110574214.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:22,859] Trial 3 finished with value: 0.00106604206497223 and parameters: {'n_estimators': 89, 'learning_rate': 0.18020779483743288, 'num_leaves': 46, 'max_depth': 8, 'min_child_samples': 3, 'subsample': 0.6941910769286002, 'colsample_bytree': 0.8232859040610813, 'reg_alpha': 6.229800078125733, 'reg_lambda': 8.534737400536741, 'min_split_gain': 0.005622333128594165}. Best is trial 3 with value: 0.00106604206497223.
[I 2025-06-15 20:36:22,861] A new study created in memory with name: no-name-b317ddcb-68cb-4b1c-8ce7-c87fc4c84faa
[I 2025-06-15 20:36:22,896] Trial 0 finished with value: 0.0017564672883034758 and parameters: {'n_estimators': 53, 'learning_rate': 0.031796323910771154, 'num_leaves': 44, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.5261406289379267, 'colsample_bytree': 0.6112441753647595, 'reg_alpha': 2.4426467288265052, 'reg_lambda': 6.529212442178678, 'min_split_gain': 0.030196620393579378}. Best is trial 0 with value: 0.0017564672883034758.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         416
1           Price_lag_2         384
14      pct_chg_Load_DA         256
3         Load_DA_lag_0         210
2           Price_lag_7         169
5       WindOn_DA_lag_0         169
16  roll2d_median_Price         159
15       lag168_Load_DA         149
11           NGas_lag_2         142
10           Coal_lag_2         117
13            EUA_lag_2         114
12            Oil_lag_2          91
9                  WD_7          21
7                  WD_1          11
8                  WD_6          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          34
1           Price_lag_2          15
16  roll2d_median_Price           1
2           Price_lag_7           1
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 77.29% done
********************* END NS.. 564 ****************************************************
********************* START NS ... 565 ****************************************************
[I 2025-06-15 20:36:24,104] A new study created in memory with name: no-name-44d1db51-beaa-4570-bf7c-ff330a06de0a
[I 2025-06-15 20:36:24,337] Trial 0 finished with value: 0.0009233411332364071 and parameters: {'n_estimators': 290, 'learning_rate': 0.05334746968849613, 'num_leaves': 201, 'max_depth': 11, 'min_child_samples': 12, 'subsample': 0.7112497962604393, 'colsample_bytree': 0.974555502126417, 'reg_alpha': 9.656629778621589, 'reg_lambda': 9.973783580889954, 'min_split_gain': 0.031079192830256366}. Best is trial 0 with value: 0.0009233411332364071.
[Trial 0] New best: 0.00092
[I 2025-06-15 20:36:24,659] Trial 1 finished with value: 0.0008862861266543809 and parameters: {'n_estimators': 97, 'learning_rate': 0.0502473646004405, 'num_leaves': 46, 'max_depth': 15, 'min_child_samples': 11, 'subsample': 0.8108029212607453, 'colsample_bytree': 0.8748252129235077, 'reg_alpha': 0.18260475471883808, 'reg_lambda': 4.380175522415738, 'min_split_gain': 0.038803400570405495}. Best is trial 1 with value: 0.0008862861266543809.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:25,493] Trial 2 finished with value: 0.0012884130321761716 and parameters: {'n_estimators': 190, 'learning_rate': 0.02030242116413121, 'num_leaves': 134, 'max_depth': 14, 'min_child_samples': 13, 'subsample': 0.9996828451510925, 'colsample_bytree': 0.8269338698327274, 'reg_alpha': 7.262892146723585, 'reg_lambda': 1.3663863061414794, 'min_split_gain': 0.002083953433354008}. Best is trial 1 with value: 0.0008862861266543809.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:25,922] Trial 3 finished with value: 0.0030160970674083555 and parameters: {'n_estimators': 116, 'learning_rate': 0.0261352938960254, 'num_leaves': 92, 'max_depth': 11, 'min_child_samples': 19, 'subsample': 0.737891153063973, 'colsample_bytree': 0.7877691971650492, 'reg_alpha': 8.836894073666539, 'reg_lambda': 8.840952973607118, 'min_split_gain': 0.04047548646886323}. Best is trial 1 with value: 0.0008862861266543809.
[I 2025-06-15 20:36:25,925] A new study created in memory with name: no-name-2296e387-2232-44ac-9521-fbf7d00018b1
[I 2025-06-15 20:36:25,943] Trial 0 finished with value: 0.001118824832176921 and parameters: {'n_estimators': 139, 'learning_rate': 0.011395908802609121, 'num_leaves': 66, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.7841430912026877, 'colsample_bytree': 0.8384647469797466, 'reg_alpha': 9.878812897954994, 'reg_lambda': 1.0692882248548596, 'min_split_gain': 0.015072947802057996}. Best is trial 0 with value: 0.001118824832176921.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1157
1           Price_lag_2         740
14      pct_chg_Load_DA         632
16  roll2d_median_Price         313
3         Load_DA_lag_0         285
2           Price_lag_7         270
11           NGas_lag_2         247
5       WindOn_DA_lag_0         184
15       lag168_Load_DA         150
10           Coal_lag_2         133
12            Oil_lag_2         123
13            EUA_lag_2         119
7                  WD_1           6
9                  WD_7           6
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 77.43% done
********************* END NS.. 565 ****************************************************
********************* START NS ... 566 ****************************************************
[I 2025-06-15 20:36:27,490] A new study created in memory with name: no-name-95591923-77a6-4e4b-aaf4-7036da81052d
[I 2025-06-15 20:36:27,641] Trial 0 finished with value: 0.010426324890698844 and parameters: {'n_estimators': 70, 'learning_rate': 0.03248804035667742, 'num_leaves': 86, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.9225663310869988, 'colsample_bytree': 0.9619681707041352, 'reg_alpha': 9.138292394873412, 'reg_lambda': 8.307123147588209, 'min_split_gain': 0.011479669859761694}. Best is trial 0 with value: 0.010426324890698844.
[Trial 0] New best: 0.01043
[I 2025-06-15 20:36:27,972] Trial 1 finished with value: 0.012429494043616998 and parameters: {'n_estimators': 126, 'learning_rate': 0.017466566504723578, 'num_leaves': 111, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.8985615016929466, 'colsample_bytree': 0.905636011848384, 'reg_alpha': 3.4246943043273435, 'reg_lambda': 3.3478634158312417, 'min_split_gain': 0.02979831708277539}. Best is trial 0 with value: 0.010426324890698844.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:28,471] Trial 2 finished with value: 0.00017620691272573932 and parameters: {'n_estimators': 210, 'learning_rate': 0.07783432669008372, 'num_leaves': 88, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.9172286457151224, 'colsample_bytree': 0.7923506616376901, 'reg_alpha': 3.447662724044479, 'reg_lambda': 0.8868092786948323, 'min_split_gain': 0.01458109589456131}. Best is trial 2 with value: 0.00017620691272573932.
[Trial 2] New best: 0.00018
[I 2025-06-15 20:36:29,324] Trial 3 finished with value: 0.00013799235838535953 and parameters: {'n_estimators': 213, 'learning_rate': 0.024626897683892535, 'num_leaves': 178, 'max_depth': 10, 'min_child_samples': 7, 'subsample': 0.5950678942613646, 'colsample_bytree': 0.6227989262529111, 'reg_alpha': 7.783066088917026, 'reg_lambda': 6.778063974535399, 'min_split_gain': 0.021296120529382567}. Best is trial 3 with value: 0.00013799235838535953.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:30,086] Trial 4 finished with value: 0.019940632355106628 and parameters: {'n_estimators': 167, 'learning_rate': 0.011546648775418267, 'num_leaves': 198, 'max_depth': 15, 'min_child_samples': 14, 'subsample': 0.6085618393843819, 'colsample_bytree': 0.6990654755338574, 'reg_alpha': 8.05746513904769, 'reg_lambda': 7.45137028175494, 'min_split_gain': 0.034095744115920035}. Best is trial 3 with value: 0.00013799235838535953.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:30,738] Trial 5 finished with value: 0.019860943787234863 and parameters: {'n_estimators': 128, 'learning_rate': 0.01531306692074051, 'num_leaves': 188, 'max_depth': 16, 'min_child_samples': 15, 'subsample': 0.962178785495798, 'colsample_bytree': 0.8816168805181694, 'reg_alpha': 7.258826784048162, 'reg_lambda': 4.547037142520241, 'min_split_gain': 0.024126769432912056}. Best is trial 3 with value: 0.00013799235838535953.
[I 2025-06-15 20:36:30,741] A new study created in memory with name: no-name-49af33c7-216c-4cce-8267-dc9bb0b1bff3
[I 2025-06-15 20:36:30,931] Trial 0 finished with value: 0.0005391497213774237 and parameters: {'n_estimators': 136, 'learning_rate': 0.010266482386647601, 'num_leaves': 154, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.5783616566613812, 'colsample_bytree': 0.664228213792226, 'reg_alpha': 0.1574532158650932, 'reg_lambda': 5.590486937710629, 'min_split_gain': 0.038227169750028156}. Best is trial 0 with value: 0.0005391497213774237.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1560
1           Price_lag_2        1428
14      pct_chg_Load_DA         970
2           Price_lag_7         711
3         Load_DA_lag_0         630
16  roll2d_median_Price         610
11           NGas_lag_2         478
15       lag168_Load_DA         441
13            EUA_lag_2         389
5       WindOn_DA_lag_0         376
10           Coal_lag_2         297
12            Oil_lag_2         210
9                  WD_7          63
8                  WD_6          37
7                  WD_1          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         123
1           Price_lag_2          42
14      pct_chg_Load_DA          12
10           Coal_lag_2           5
16  roll2d_median_Price           5
13            EUA_lag_2           3
11           NGas_lag_2           2
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
3         Load_DA_lag_0           0
12            Oil_lag_2           0
2           Price_lag_7           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 77.56% done
********************* END NS.. 566 ****************************************************
********************* START NS ... 567 ****************************************************
[I 2025-06-15 20:36:33,098] A new study created in memory with name: no-name-e66b3749-053b-4120-a818-d6cd9c748f28
[I 2025-06-15 20:36:34,665] Trial 0 finished with value: 0.0014659623050857495 and parameters: {'n_estimators': 206, 'learning_rate': 0.026617043515121078, 'num_leaves': 96, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.9131749898583799, 'colsample_bytree': 0.7320543820269788, 'reg_alpha': 1.9059785314880506, 'reg_lambda': 3.0034734591915626, 'min_split_gain': 0.0378623818506028}. Best is trial 0 with value: 0.0014659623050857495.
[Trial 0] New best: 0.00147
[I 2025-06-15 20:36:35,159] Trial 1 finished with value: 0.0010618830388229965 and parameters: {'n_estimators': 95, 'learning_rate': 0.24531170343486355, 'num_leaves': 200, 'max_depth': 7, 'min_child_samples': 3, 'subsample': 0.937838365723815, 'colsample_bytree': 0.8349691294352604, 'reg_alpha': 1.73590185369223, 'reg_lambda': 2.456839297571838, 'min_split_gain': 0.023987704448257596}. Best is trial 1 with value: 0.0010618830388229965.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:35,551] Trial 2 finished with value: 0.006475649028104481 and parameters: {'n_estimators': 52, 'learning_rate': 0.049478327185234906, 'num_leaves': 126, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.7024841032891341, 'colsample_bytree': 0.8702516309664878, 'reg_alpha': 0.3140650970131831, 'reg_lambda': 5.594262460934044, 'min_split_gain': 0.033533484544215286}. Best is trial 1 with value: 0.0010618830388229965.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:36,149] Trial 3 finished with value: 0.0013869105140628668 and parameters: {'n_estimators': 221, 'learning_rate': 0.07358202334198038, 'num_leaves': 81, 'max_depth': 16, 'min_child_samples': 12, 'subsample': 0.9067138000180333, 'colsample_bytree': 0.7369222734006563, 'reg_alpha': 7.386726711299484, 'reg_lambda': 3.1373173959108547, 'min_split_gain': 0.008843889852795784}. Best is trial 1 with value: 0.0010618830388229965.
[I 2025-06-15 20:36:36,151] A new study created in memory with name: no-name-a221e401-e10a-4781-a640-9571f09a0657
[I 2025-06-15 20:36:36,179] Trial 0 finished with value: 0.006130270462080847 and parameters: {'n_estimators': 97, 'learning_rate': 0.237476863560413, 'num_leaves': 222, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.8434199705639736, 'colsample_bytree': 0.5853405715854844, 'reg_alpha': 3.267216521607994, 'reg_lambda': 2.2968335365166426, 'min_split_gain': 0.014601732366134641}. Best is trial 0 with value: 0.006130270462080847.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         323
14      pct_chg_Load_DA         249
1           Price_lag_2         236
3         Load_DA_lag_0         157
16  roll2d_median_Price         146
2           Price_lag_7         121
5       WindOn_DA_lag_0         104
11           NGas_lag_2         102
15       lag168_Load_DA          97
10           Coal_lag_2          77
12            Oil_lag_2          66
13            EUA_lag_2          39
9                  WD_7           8
7                  WD_1           6
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 77.70% done
********************* END NS.. 567 ****************************************************
********************* START NS ... 568 ****************************************************
[I 2025-06-15 20:36:37,481] A new study created in memory with name: no-name-fea73027-52fe-45c6-b0cc-44fb25c53367
[I 2025-06-15 20:36:39,979] Trial 0 finished with value: 0.0048830879896598995 and parameters: {'n_estimators': 289, 'learning_rate': 0.010632964817906652, 'num_leaves': 109, 'max_depth': 11, 'min_child_samples': 5, 'subsample': 0.5422010649791407, 'colsample_bytree': 0.8918308159716022, 'reg_alpha': 1.0983031613078786, 'reg_lambda': 6.682024107454881, 'min_split_gain': 0.017915286597314905}. Best is trial 0 with value: 0.0048830879896598995.
[Trial 0] New best: 0.00488
[I 2025-06-15 20:36:40,642] Trial 1 finished with value: 0.002915294818083064 and parameters: {'n_estimators': 197, 'learning_rate': 0.03503022636880705, 'num_leaves': 88, 'max_depth': 8, 'min_child_samples': 24, 'subsample': 0.5932639378567784, 'colsample_bytree': 0.7568108770192534, 'reg_alpha': 3.0138909223081756, 'reg_lambda': 0.24713035061804223, 'min_split_gain': 0.02638516785254479}. Best is trial 1 with value: 0.002915294818083064.
[Trial 1] New best: 0.00292
[I 2025-06-15 20:36:41,075] Trial 2 finished with value: 0.0030902896681337965 and parameters: {'n_estimators': 130, 'learning_rate': 0.03469341504430167, 'num_leaves': 251, 'max_depth': 11, 'min_child_samples': 18, 'subsample': 0.764232489381575, 'colsample_bytree': 0.8385459876497093, 'reg_alpha': 8.613458078558446, 'reg_lambda': 7.7963727546163, 'min_split_gain': 0.013972218855671938}. Best is trial 1 with value: 0.002915294818083064.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:41,569] Trial 3 finished with value: 0.002091297730625182 and parameters: {'n_estimators': 139, 'learning_rate': 0.09842020005998982, 'num_leaves': 215, 'max_depth': 12, 'min_child_samples': 24, 'subsample': 0.763995013316473, 'colsample_bytree': 0.724719488769592, 'reg_alpha': 1.072382325138156, 'reg_lambda': 8.993178327941433, 'min_split_gain': 0.013045135585049362}. Best is trial 3 with value: 0.002091297730625182.
[I 2025-06-15 20:36:41,775] Trial 4 finished with value: 0.0032464833851667887 and parameters: {'n_estimators': 244, 'learning_rate': 0.2572960630087105, 'num_leaves': 169, 'max_depth': 7, 'min_child_samples': 10, 'subsample': 0.502860262243449, 'colsample_bytree': 0.5530007103679161, 'reg_alpha': 7.014817394801018, 'reg_lambda': 8.232124125706848, 'min_split_gain': 0.03649549533421926}. Best is trial 3 with value: 0.002091297730625182.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:41,778] A new study created in memory with name: no-name-4e8e5ba9-89fe-4b05-bc50-672d97c31848
[I 2025-06-15 20:36:41,829] Trial 0 finished with value: 0.01592497736198986 and parameters: {'n_estimators': 166, 'learning_rate': 0.029864344040454946, 'num_leaves': 193, 'max_depth': 11, 'min_child_samples': 25, 'subsample': 0.6395643727903219, 'colsample_bytree': 0.557696881407886, 'reg_alpha': 4.881241503290031, 'reg_lambda': 2.3037279261836483, 'min_split_gain': 0.045246256079186115}. Best is trial 0 with value: 0.01592497736198986.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1077
0           Price_lag_1        1044
14      pct_chg_Load_DA         999
3         Load_DA_lag_0         850
5       WindOn_DA_lag_0         535
15       lag168_Load_DA         512
2           Price_lag_7         506
16  roll2d_median_Price         461
11           NGas_lag_2         426
10           Coal_lag_2         357
13            EUA_lag_2         335
12            Oil_lag_2         237
9                  WD_7          29
7                  WD_1          24
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 77.84% done
********************* END NS.. 568 ****************************************************
********************* START NS ... 569 ****************************************************
[I 2025-06-15 20:36:43,052] A new study created in memory with name: no-name-668d553a-c5bf-43b8-85cd-e264872dac9f
[I 2025-06-15 20:36:43,416] Trial 0 finished with value: 0.001525249568331016 and parameters: {'n_estimators': 141, 'learning_rate': 0.06182494893000652, 'num_leaves': 230, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.8191528099473993, 'colsample_bytree': 0.7675690276065709, 'reg_alpha': 2.1405424074048196, 'reg_lambda': 8.692920673265728, 'min_split_gain': 0.030611423942866152}. Best is trial 0 with value: 0.001525249568331016.
[I 2025-06-15 20:36:43,571] Trial 1 finished with value: 0.0014719384980714414 and parameters: {'n_estimators': 214, 'learning_rate': 0.22203307051238597, 'num_leaves': 166, 'max_depth': 11, 'min_child_samples': 25, 'subsample': 0.5026258750589361, 'colsample_bytree': 0.6551964702939455, 'reg_alpha': 2.9000137141135287, 'reg_lambda': 6.108997855884074, 'min_split_gain': 0.04821538508305517}. Best is trial 1 with value: 0.0014719384980714414.
[Trial 0] New best: 0.00153
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:44,083] Trial 2 finished with value: 0.0018742319777446462 and parameters: {'n_estimators': 189, 'learning_rate': 0.03761734099091219, 'num_leaves': 60, 'max_depth': 12, 'min_child_samples': 11, 'subsample': 0.7919468698417003, 'colsample_bytree': 0.692145148171467, 'reg_alpha': 7.176046071762665, 'reg_lambda': 7.667519237142829, 'min_split_gain': 0.03856764026889261}. Best is trial 1 with value: 0.0014719384980714414.
[I 2025-06-15 20:36:44,239] Trial 3 finished with value: 0.0017682419804798365 and parameters: {'n_estimators': 72, 'learning_rate': 0.08442424184815568, 'num_leaves': 72, 'max_depth': 4, 'min_child_samples': 30, 'subsample': 0.8716503702094562, 'colsample_bytree': 0.774864784221168, 'reg_alpha': 1.6366557324750597, 'reg_lambda': 7.009081752652592, 'min_split_gain': 0.010711138431286737}. Best is trial 1 with value: 0.0014719384980714414.
[I 2025-06-15 20:36:44,241] A new study created in memory with name: no-name-492f0162-26d6-428c-ba82-769ad2e7e61e
[I 2025-06-15 20:36:44,262] Trial 0 finished with value: 0.007325371061784371 and parameters: {'n_estimators': 50, 'learning_rate': 0.01037589248783901, 'num_leaves': 102, 'max_depth': 7, 'min_child_samples': 24, 'subsample': 0.7150993423698822, 'colsample_bytree': 0.5364551301853336, 'reg_alpha': 1.599860108131137, 'reg_lambda': 8.82798845259554, 'min_split_gain': 0.003044665759580656}. Best is trial 0 with value: 0.007325371061784371.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         292
1           Price_lag_2         178
14      pct_chg_Load_DA         123
3         Load_DA_lag_0         112
11           NGas_lag_2         104
2           Price_lag_7          70
16  roll2d_median_Price          68
15       lag168_Load_DA          59
10           Coal_lag_2          49
5       WindOn_DA_lag_0          40
13            EUA_lag_2          21
12            Oil_lag_2          18
7                  WD_1           5
9                  WD_7           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          44
16  roll2d_median_Price          17
1           Price_lag_2           8
12            Oil_lag_2           7
13            EUA_lag_2           3
2           Price_lag_7           0
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 77.98% done
********************* END NS.. 569 ****************************************************
********************* START NS ... 570 ****************************************************
[I 2025-06-15 20:36:45,457] A new study created in memory with name: no-name-fa9d7670-b271-4a99-951e-dd2ec7cae96f
[I 2025-06-15 20:36:45,579] Trial 0 finished with value: 0.00035655619495636334 and parameters: {'n_estimators': 86, 'learning_rate': 0.2612599892499612, 'num_leaves': 98, 'max_depth': 8, 'min_child_samples': 13, 'subsample': 0.5243997415121391, 'colsample_bytree': 0.8057177918282132, 'reg_alpha': 4.813707492251052, 'reg_lambda': 9.291752855391351, 'min_split_gain': 0.02929109109976166}. Best is trial 0 with value: 0.00035655619495636334.
[I 2025-06-15 20:36:45,688] Trial 1 finished with value: 0.0003880793213846385 and parameters: {'n_estimators': 285, 'learning_rate': 0.14285730330246013, 'num_leaves': 169, 'max_depth': 4, 'min_child_samples': 9, 'subsample': 0.6516713560153817, 'colsample_bytree': 0.8180181415751986, 'reg_alpha': 8.544932887116957, 'reg_lambda': 1.678145312658611, 'min_split_gain': 0.04316376017231707}. Best is trial 0 with value: 0.00035655619495636334.
[Trial 0] New best: 0.00036
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:46,264] Trial 2 finished with value: 0.0002557136605062401 and parameters: {'n_estimators': 274, 'learning_rate': 0.030779846979184004, 'num_leaves': 32, 'max_depth': 9, 'min_child_samples': 15, 'subsample': 0.8541758343186205, 'colsample_bytree': 0.8206726070029549, 'reg_alpha': 2.3419983881753526, 'reg_lambda': 4.66825971357911, 'min_split_gain': 0.03786115220328748}. Best is trial 2 with value: 0.0002557136605062401.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:47,384] Trial 3 finished with value: 0.001047214064341651 and parameters: {'n_estimators': 263, 'learning_rate': 0.013071616253439678, 'num_leaves': 164, 'max_depth': 13, 'min_child_samples': 3, 'subsample': 0.8578049765835187, 'colsample_bytree': 0.836903128725034, 'reg_alpha': 4.65290342335218, 'reg_lambda': 8.21828742769171, 'min_split_gain': 0.01050252544077881}. Best is trial 2 with value: 0.0002557136605062401.
[I 2025-06-15 20:36:47,387] A new study created in memory with name: no-name-98e5aa1c-a6e7-4be2-a746-e9eb381d50dc
[I 2025-06-15 20:36:47,412] Trial 0 finished with value: 0.00027997975981964977 and parameters: {'n_estimators': 182, 'learning_rate': 0.08701503014486123, 'num_leaves': 161, 'max_depth': 15, 'min_child_samples': 1, 'subsample': 0.9584312506096295, 'colsample_bytree': 0.8229639204184956, 'reg_alpha': 9.951255851658678, 'reg_lambda': 1.639427554089905, 'min_split_gain': 0.03723329913963772}. Best is trial 0 with value: 0.00027997975981964977.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1853
1           Price_lag_2        1255
14      pct_chg_Load_DA        1130
3         Load_DA_lag_0         699
11           NGas_lag_2         478
2           Price_lag_7         453
5       WindOn_DA_lag_0         450
16  roll2d_median_Price         391
15       lag168_Load_DA         388
10           Coal_lag_2         252
12            Oil_lag_2         157
13            EUA_lag_2         137
7                  WD_1          23
9                  WD_7          23
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 78.11% done
********************* END NS.. 570 ****************************************************
********************* START NS ... 571 ****************************************************
[I 2025-06-15 20:36:49,370] A new study created in memory with name: no-name-f0a3e03d-e5b6-4abb-90e3-c38a37c514a6
[I 2025-06-15 20:36:49,644] Trial 0 finished with value: 0.005770624301513326 and parameters: {'n_estimators': 119, 'learning_rate': 0.021822578476164114, 'num_leaves': 256, 'max_depth': 5, 'min_child_samples': 28, 'subsample': 0.6525143050143172, 'colsample_bytree': 0.8691962928949117, 'reg_alpha': 4.5023631089269855, 'reg_lambda': 2.033681869317853, 'min_split_gain': 0.016077540968961107}. Best is trial 0 with value: 0.005770624301513326.
[Trial 0] New best: 0.00577
[I 2025-06-15 20:36:50,686] Trial 1 finished with value: 0.0008254025635486857 and parameters: {'n_estimators': 174, 'learning_rate': 0.025533966126728622, 'num_leaves': 239, 'max_depth': 12, 'min_child_samples': 25, 'subsample': 0.5444912533566384, 'colsample_bytree': 0.6871673228080287, 'reg_alpha': 2.007392189335132, 'reg_lambda': 1.2321878699103372, 'min_split_gain': 0.007125407018604657}. Best is trial 1 with value: 0.0008254025635486857.
[I 2025-06-15 20:36:50,867] Trial 2 finished with value: 0.0007265789045398773 and parameters: {'n_estimators': 58, 'learning_rate': 0.21140198934364676, 'num_leaves': 226, 'max_depth': 10, 'min_child_samples': 28, 'subsample': 0.8466685414244384, 'colsample_bytree': 0.7082823151434494, 'reg_alpha': 9.528621355190372, 'reg_lambda': 8.8486783995839, 'min_split_gain': 0.020003131887237974}. Best is trial 2 with value: 0.0007265789045398773.
[Trial 1] New best: 0.00083
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:51,255] Trial 3 finished with value: 0.0006240878257616768 and parameters: {'n_estimators': 198, 'learning_rate': 0.1181013965862364, 'num_leaves': 31, 'max_depth': 8, 'min_child_samples': 5, 'subsample': 0.6929500424805652, 'colsample_bytree': 0.5437005261367138, 'reg_alpha': 2.415481841655447, 'reg_lambda': 7.030911884377722, 'min_split_gain': 0.012438904411168367}. Best is trial 3 with value: 0.0006240878257616768.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:51,498] Trial 4 finished with value: 0.2593419337957287 and parameters: {'n_estimators': 65, 'learning_rate': 0.01005980907260727, 'num_leaves': 63, 'max_depth': 11, 'min_child_samples': 29, 'subsample': 0.6088572859929298, 'colsample_bytree': 0.6373986170136854, 'reg_alpha': 8.491691750971409, 'reg_lambda': 0.4366754234874348, 'min_split_gain': 0.01089666971995722}. Best is trial 3 with value: 0.0006240878257616768.
[I 2025-06-15 20:36:51,501] A new study created in memory with name: no-name-cae8bd76-f1fc-491f-be65-156cb8caeeaf
[I 2025-06-15 20:36:51,545] Trial 0 finished with value: 0.0019599305469493364 and parameters: {'n_estimators': 300, 'learning_rate': 0.218520955844884, 'num_leaves': 191, 'max_depth': 16, 'min_child_samples': 19, 'subsample': 0.9145636129175359, 'colsample_bytree': 0.9884147659322977, 'reg_alpha': 9.101247430336363, 'reg_lambda': 0.7705094669980173, 'min_split_gain': 0.03290167038948054}. Best is trial 0 with value: 0.0019599305469493364.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         792
1           Price_lag_2         729
14      pct_chg_Load_DA         367
16  roll2d_median_Price         257
2           Price_lag_7         244
11           NGas_lag_2         235
5       WindOn_DA_lag_0         199
3         Load_DA_lag_0         194
13            EUA_lag_2         186
15       lag168_Load_DA         163
10           Coal_lag_2         147
12            Oil_lag_2         143
9                  WD_7          15
8                  WD_6          13
7                  WD_1          11
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 78.25% done
********************* END NS.. 571 ****************************************************
********************* START NS ... 572 ****************************************************
[I 2025-06-15 20:36:53,083] A new study created in memory with name: no-name-8dd69253-b119-4eed-a39a-84005af071b1
[I 2025-06-15 20:36:53,892] Trial 0 finished with value: 0.0008666155407791965 and parameters: {'n_estimators': 243, 'learning_rate': 0.053170330088833546, 'num_leaves': 135, 'max_depth': 14, 'min_child_samples': 5, 'subsample': 0.6406888055085429, 'colsample_bytree': 0.8312394153568765, 'reg_alpha': 9.035175710613348, 'reg_lambda': 5.815419779158807, 'min_split_gain': 0.0015684627990931955}. Best is trial 0 with value: 0.0008666155407791965.
[Trial 0] New best: 0.00087
[I 2025-06-15 20:36:54,376] Trial 1 finished with value: 0.0008360572825156748 and parameters: {'n_estimators': 277, 'learning_rate': 0.10668905881089154, 'num_leaves': 160, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.6678973726063682, 'colsample_bytree': 0.8639759363359989, 'reg_alpha': 8.403766715090045, 'reg_lambda': 1.6775229067863673, 'min_split_gain': 0.003079771088964645}. Best is trial 1 with value: 0.0008360572825156748.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:54,644] Trial 2 finished with value: 0.0008986653632887231 and parameters: {'n_estimators': 95, 'learning_rate': 0.201447205710325, 'num_leaves': 234, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.5898527024424816, 'colsample_bytree': 0.6410830277244559, 'reg_alpha': 0.6192841678264194, 'reg_lambda': 5.652934360583891, 'min_split_gain': 0.03981409877991047}. Best is trial 1 with value: 0.0008360572825156748.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:36:54,916] Trial 3 finished with value: 0.0010015832733079224 and parameters: {'n_estimators': 66, 'learning_rate': 0.15181536739380666, 'num_leaves': 160, 'max_depth': 16, 'min_child_samples': 20, 'subsample': 0.7430361465363207, 'colsample_bytree': 0.6538175852650849, 'reg_alpha': 5.439370733871201, 'reg_lambda': 7.236589158860651, 'min_split_gain': 0.03227380521563304}. Best is trial 1 with value: 0.0008360572825156748.
[I 2025-06-15 20:36:54,919] A new study created in memory with name: no-name-db0050b9-c936-468e-9efa-3ea08288ac1e
[I 2025-06-15 20:36:54,942] Trial 0 finished with value: 0.00198110427581998 and parameters: {'n_estimators': 162, 'learning_rate': 0.12083621958294022, 'num_leaves': 65, 'max_depth': 4, 'min_child_samples': 13, 'subsample': 0.5865917136541605, 'colsample_bytree': 0.9658098379821785, 'reg_alpha': 6.780794176271017, 'reg_lambda': 3.945193459050149, 'min_split_gain': 0.027258039142419532}. Best is trial 0 with value: 0.00198110427581998.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         624
1           Price_lag_2         563
14      pct_chg_Load_DA         440
3         Load_DA_lag_0         338
2           Price_lag_7         314
5       WindOn_DA_lag_0         275
13            EUA_lag_2         256
16  roll2d_median_Price         255
15       lag168_Load_DA         254
11           NGas_lag_2         238
10           Coal_lag_2         215
12            Oil_lag_2         178
9                  WD_7          25
8                  WD_6          16
7                  WD_1          13
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 78.39% done
********************* END NS.. 572 ****************************************************
********************* START NS ... 573 ****************************************************
[I 2025-06-15 20:36:56,493] A new study created in memory with name: no-name-c507e8ef-7594-4693-b1a9-2f1cde448478
[I 2025-06-15 20:36:56,750] Trial 0 finished with value: 0.020835770382604844 and parameters: {'n_estimators': 113, 'learning_rate': 0.01696108280444454, 'num_leaves': 194, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.9288378372788907, 'colsample_bytree': 0.9941889993191315, 'reg_alpha': 9.22114128697688, 'reg_lambda': 7.877820729955573, 'min_split_gain': 0.013600462047102508}. Best is trial 0 with value: 0.020835770382604844.
[Trial 0] New best: 0.02084
[I 2025-06-15 20:36:57,131] Trial 1 finished with value: 0.0012283702695047225 and parameters: {'n_estimators': 158, 'learning_rate': 0.021461495411196137, 'num_leaves': 58, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.9818231359807673, 'colsample_bytree': 0.914802415323686, 'reg_alpha': 7.66047913535954, 'reg_lambda': 2.7084501231526215, 'min_split_gain': 0.036780550853355756}. Best is trial 1 with value: 0.0012283702695047225.
[Trial 1] New best: 0.00123
[I 2025-06-15 20:36:57,413] Trial 2 finished with value: 0.042644738867270805 and parameters: {'n_estimators': 110, 'learning_rate': 0.014145011670584297, 'num_leaves': 54, 'max_depth': 5, 'min_child_samples': 13, 'subsample': 0.5343414100023188, 'colsample_bytree': 0.6419740111601633, 'reg_alpha': 4.910727121785871, 'reg_lambda': 9.635657185769835, 'min_split_gain': 0.046183341716450485}. Best is trial 1 with value: 0.0012283702695047225.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:58,454] Trial 3 finished with value: 0.00021015252615313223 and parameters: {'n_estimators': 173, 'learning_rate': 0.031029644042853213, 'num_leaves': 189, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.8729566463397058, 'colsample_bytree': 0.8827606644196498, 'reg_alpha': 0.567395708597751, 'reg_lambda': 0.47626216445741054, 'min_split_gain': 0.039215543498708955}. Best is trial 3 with value: 0.00021015252615313223.
[Trial 3] New best: 0.00021
[I 2025-06-15 20:36:58,876] Trial 4 finished with value: 0.0002623291091904234 and parameters: {'n_estimators': 161, 'learning_rate': 0.09335533312699032, 'num_leaves': 57, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.8047265955687088, 'colsample_bytree': 0.5525134480885538, 'reg_alpha': 2.7473621517381686, 'reg_lambda': 2.818335493863634, 'min_split_gain': 0.01953874987989769}. Best is trial 3 with value: 0.00021015252615313223.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:36:59,190] Trial 5 finished with value: 0.00024309980968415697 and parameters: {'n_estimators': 198, 'learning_rate': 0.07757598868934291, 'num_leaves': 251, 'max_depth': 5, 'min_child_samples': 23, 'subsample': 0.7765161308508735, 'colsample_bytree': 0.547259419684743, 'reg_alpha': 0.6183385063709912, 'reg_lambda': 0.4853162449641035, 'min_split_gain': 0.040183464465749684}. Best is trial 3 with value: 0.00021015252615313223.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:00,099] Trial 6 finished with value: 0.0009379686980298766 and parameters: {'n_estimators': 124, 'learning_rate': 0.028080377164163006, 'num_leaves': 141, 'max_depth': 12, 'min_child_samples': 25, 'subsample': 0.8524674693325995, 'colsample_bytree': 0.6672864016659592, 'reg_alpha': 0.40091671407948115, 'reg_lambda': 9.215637104915395, 'min_split_gain': 0.025948596465217817}. Best is trial 3 with value: 0.00021015252615313223.
[I 2025-06-15 20:37:00,101] A new study created in memory with name: no-name-4a888360-ec64-4362-8969-36a8a20dcd16
[I 2025-06-15 20:37:00,130] Trial 0 finished with value: 0.0004919458008238684 and parameters: {'n_estimators': 249, 'learning_rate': 0.12117564804201622, 'num_leaves': 246, 'max_depth': 16, 'min_child_samples': 5, 'subsample': 0.7594577600202611, 'colsample_bytree': 0.5220641244787272, 'reg_alpha': 9.390719934914925, 'reg_lambda': 8.989451468749193, 'min_split_gain': 0.029207819021233497}. Best is trial 0 with value: 0.0004919458008238684.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        2932
0           Price_lag_1        2890
14      pct_chg_Load_DA        2083
3         Load_DA_lag_0        1140
16  roll2d_median_Price        1095
2           Price_lag_7         984
11           NGas_lag_2         844
15       lag168_Load_DA         640
5       WindOn_DA_lag_0         628
10           Coal_lag_2         452
13            EUA_lag_2         351
12            Oil_lag_2         350
9                  WD_7          28
7                  WD_1          13
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 78.52% done
********************* END NS.. 573 ****************************************************
********************* START NS ... 574 ****************************************************
[I 2025-06-15 20:37:02,022] A new study created in memory with name: no-name-6f9659d9-2d0d-4628-8bf2-924a8e6986b9
[I 2025-06-15 20:37:02,752] Trial 0 finished with value: 0.00011633978494542063 and parameters: {'n_estimators': 284, 'learning_rate': 0.024199641611605087, 'num_leaves': 58, 'max_depth': 7, 'min_child_samples': 22, 'subsample': 0.9660613635338127, 'colsample_bytree': 0.6373864791845367, 'reg_alpha': 1.0311696505381718, 'reg_lambda': 9.52875199548347, 'min_split_gain': 0.03146901467330956}. Best is trial 0 with value: 0.00011633978494542063.
[Trial 0] New best: 0.00012
[I 2025-06-15 20:37:03,233] Trial 1 finished with value: 0.00014593086909847844 and parameters: {'n_estimators': 213, 'learning_rate': 0.09283580527330179, 'num_leaves': 109, 'max_depth': 6, 'min_child_samples': 21, 'subsample': 0.6976673056282185, 'colsample_bytree': 0.6223267357946035, 'reg_alpha': 6.862916381181812, 'reg_lambda': 1.4311265511575255, 'min_split_gain': 0.012121000728430715}. Best is trial 0 with value: 0.00011633978494542063.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:03,869] Trial 2 finished with value: 0.023317188417712947 and parameters: {'n_estimators': 116, 'learning_rate': 0.015908230811328795, 'num_leaves': 234, 'max_depth': 8, 'min_child_samples': 13, 'subsample': 0.5744133269028961, 'colsample_bytree': 0.6274646796319131, 'reg_alpha': 1.959715172709141, 'reg_lambda': 2.8657978798950436, 'min_split_gain': 0.005649395262872026}. Best is trial 0 with value: 0.00011633978494542063.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:04,646] Trial 3 finished with value: 9.751929099436836e-05 and parameters: {'n_estimators': 135, 'learning_rate': 0.04390243371922007, 'num_leaves': 132, 'max_depth': 13, 'min_child_samples': 26, 'subsample': 0.6589503865316177, 'colsample_bytree': 0.7208894567391104, 'reg_alpha': 1.3912624370606208, 'reg_lambda': 4.167697235149731, 'min_split_gain': 0.03133103555271486}. Best is trial 3 with value: 9.751929099436836e-05.
[I 2025-06-15 20:37:04,649] A new study created in memory with name: no-name-53ac25c0-c42f-4802-963e-e712636954ca
[I 2025-06-15 20:37:04,675] Trial 0 finished with value: 0.0004251874068691511 and parameters: {'n_estimators': 142, 'learning_rate': 0.047480398758803985, 'num_leaves': 215, 'max_depth': 8, 'min_child_samples': 14, 'subsample': 0.8829615052328668, 'colsample_bytree': 0.7176657988540804, 'reg_alpha': 2.2728904209919722, 'reg_lambda': 1.594778270634527, 'min_split_gain': 0.042111152056787615}. Best is trial 0 with value: 0.0004251874068691511.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1604
1           Price_lag_2        1493
14      pct_chg_Load_DA        1148
3         Load_DA_lag_0         829
11           NGas_lag_2         529
16  roll2d_median_Price         521
2           Price_lag_7         519
15       lag168_Load_DA         518
5       WindOn_DA_lag_0         402
10           Coal_lag_2         358
13            EUA_lag_2         284
12            Oil_lag_2         255
9                  WD_7          24
7                  WD_1          17
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 78.66% done
********************* END NS.. 574 ****************************************************
********************* START NS ... 575 ****************************************************
[I 2025-06-15 20:37:06,679] A new study created in memory with name: no-name-b18f4c98-34fc-4e61-afe9-2e121fbfeb78
[I 2025-06-15 20:37:06,828] Trial 0 finished with value: 0.010215045431607934 and parameters: {'n_estimators': 193, 'learning_rate': 0.18818236594231882, 'num_leaves': 59, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.934746504922539, 'colsample_bytree': 0.8183560274709971, 'reg_alpha': 5.290009568464955, 'reg_lambda': 3.5297486936887967, 'min_split_gain': 0.012843355434156795}. Best is trial 0 with value: 0.010215045431607934.
[I 2025-06-15 20:37:07,025] Trial 1 finished with value: 0.08775679906062424 and parameters: {'n_estimators': 80, 'learning_rate': 0.01828635822491246, 'num_leaves': 48, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.5737666527103948, 'colsample_bytree': 0.6094053231739476, 'reg_alpha': 8.269163880154945, 'reg_lambda': 1.6494525679552208, 'min_split_gain': 0.03490104078170461}. Best is trial 0 with value: 0.010215045431607934.
[Trial 0] New best: 0.01022
[I 2025-06-15 20:37:07,202] Trial 2 finished with value: 0.009002868618210877 and parameters: {'n_estimators': 51, 'learning_rate': 0.19804294367935504, 'num_leaves': 151, 'max_depth': 16, 'min_child_samples': 12, 'subsample': 0.843220342597532, 'colsample_bytree': 0.7987346109880489, 'reg_alpha': 1.7735079856165326, 'reg_lambda': 9.527497126423958, 'min_split_gain': 0.03770947006415455}. Best is trial 2 with value: 0.009002868618210877.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] New best: 0.00900
[I 2025-06-15 20:37:07,700] Trial 3 finished with value: 0.010181833156112223 and parameters: {'n_estimators': 217, 'learning_rate': 0.05472640204093154, 'num_leaves': 187, 'max_depth': 11, 'min_child_samples': 9, 'subsample': 0.8969658246262415, 'colsample_bytree': 0.8991705810021609, 'reg_alpha': 4.194037132290158, 'reg_lambda': 2.180163679992093, 'min_split_gain': 0.027209923606448607}. Best is trial 2 with value: 0.009002868618210877.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:08,716] Trial 4 finished with value: 0.006409464578855502 and parameters: {'n_estimators': 104, 'learning_rate': 0.12993144863910278, 'num_leaves': 248, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.8732315710407414, 'colsample_bytree': 0.6320524307824837, 'reg_alpha': 0.24522893026179715, 'reg_lambda': 7.891245276296127, 'min_split_gain': 0.000689660336457193}. Best is trial 4 with value: 0.006409464578855502.
[Trial 4] New best: 0.00641
[I 2025-06-15 20:37:09,240] Trial 5 finished with value: 0.011671703516981833 and parameters: {'n_estimators': 270, 'learning_rate': 0.09352917118125438, 'num_leaves': 254, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.9391372973133632, 'colsample_bytree': 0.7032710620226282, 'reg_alpha': 7.220998084282446, 'reg_lambda': 4.846395330340645, 'min_split_gain': 0.04511019327968909}. Best is trial 4 with value: 0.006409464578855502.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:09,743] Trial 6 finished with value: 0.031217137231717304 and parameters: {'n_estimators': 179, 'learning_rate': 0.013170957694434032, 'num_leaves': 145, 'max_depth': 15, 'min_child_samples': 29, 'subsample': 0.5333484246628868, 'colsample_bytree': 0.7966990289219361, 'reg_alpha': 8.43260377791106, 'reg_lambda': 5.320660148034334, 'min_split_gain': 0.04418729515530533}. Best is trial 4 with value: 0.006409464578855502.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:10,233] Trial 7 finished with value: 0.13372496836583142 and parameters: {'n_estimators': 94, 'learning_rate': 0.012579902594694967, 'num_leaves': 163, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.8201252662530107, 'colsample_bytree': 0.8993009261944996, 'reg_alpha': 1.5220982576297082, 'reg_lambda': 9.34648068722882, 'min_split_gain': 0.0024610426935848697}. Best is trial 4 with value: 0.006409464578855502.
[I 2025-06-15 20:37:10,236] A new study created in memory with name: no-name-01404b75-f1b0-4c29-84c2-4e57e15ce026
[I 2025-06-15 20:37:10,267] Trial 0 finished with value: 0.036046508675024964 and parameters: {'n_estimators': 263, 'learning_rate': 0.20829645505481956, 'num_leaves': 100, 'max_depth': 12, 'min_child_samples': 25, 'subsample': 0.9996190911961529, 'colsample_bytree': 0.5701069569431498, 'reg_alpha': 2.535202768754311, 'reg_lambda': 6.243649758709534, 'min_split_gain': 0.004938105124425158}. Best is trial 0 with value: 0.036046508675024964.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
3         Load_DA_lag_0        2111
1           Price_lag_2        1969
0           Price_lag_1        1919
5       WindOn_DA_lag_0        1817
15       lag168_Load_DA        1780
14      pct_chg_Load_DA        1677
2           Price_lag_7        1646
13            EUA_lag_2        1296
11           NGas_lag_2        1231
16  roll2d_median_Price        1230
10           Coal_lag_2        1122
12            Oil_lag_2         924
9                  WD_7          53
7                  WD_1          44
8                  WD_6          29
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           6
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 78.80% done
********************* END NS.. 575 ****************************************************
********************* START NS ... 576 ****************************************************
[I 2025-06-15 20:37:11,922] A new study created in memory with name: no-name-110cdc41-420c-437a-9318-d80f39e0e2ca
[I 2025-06-15 20:37:12,127] Trial 0 finished with value: 0.013116424928160274 and parameters: {'n_estimators': 219, 'learning_rate': 0.24933926123300534, 'num_leaves': 118, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.8571283934184244, 'colsample_bytree': 0.7430159726616776, 'reg_alpha': 6.8644573839162835, 'reg_lambda': 3.6004429163195883, 'min_split_gain': 0.04645093126810594}. Best is trial 0 with value: 0.013116424928160274.
[I 2025-06-15 20:37:12,307] Trial 1 finished with value: 0.012159459154295685 and parameters: {'n_estimators': 291, 'learning_rate': 0.24840021069539406, 'num_leaves': 227, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.8289922914463761, 'colsample_bytree': 0.6346281648366647, 'reg_alpha': 6.142532557220262, 'reg_lambda': 3.3659266926106133, 'min_split_gain': 0.005953752830841957}. Best is trial 1 with value: 0.012159459154295685.
[Trial 0] New best: 0.01312
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:12,571] Trial 2 finished with value: 0.02422367516896196 and parameters: {'n_estimators': 175, 'learning_rate': 0.016128330832780098, 'num_leaves': 131, 'max_depth': 4, 'min_child_samples': 6, 'subsample': 0.8852511457944511, 'colsample_bytree': 0.863596765600247, 'reg_alpha': 5.025932116690628, 'reg_lambda': 9.620451444677563, 'min_split_gain': 0.004947407320156733}. Best is trial 1 with value: 0.012159459154295685.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:13,162] Trial 3 finished with value: 0.015604831962778056 and parameters: {'n_estimators': 137, 'learning_rate': 0.029631072223633342, 'num_leaves': 144, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.7170967372336625, 'colsample_bytree': 0.9890027154960181, 'reg_alpha': 5.35745292754363, 'reg_lambda': 6.525116789213242, 'min_split_gain': 0.020998880327451615}. Best is trial 1 with value: 0.012159459154295685.
[I 2025-06-15 20:37:13,164] A new study created in memory with name: no-name-d19e2e04-7278-4cae-bd23-9d16780d640d
[I 2025-06-15 20:37:13,187] Trial 0 finished with value: 0.0386369731099313 and parameters: {'n_estimators': 227, 'learning_rate': 0.08825758473776651, 'num_leaves': 135, 'max_depth': 8, 'min_child_samples': 29, 'subsample': 0.6764286497303565, 'colsample_bytree': 0.8464839110079203, 'reg_alpha': 2.3557522898132643, 'reg_lambda': 2.523302032723408, 'min_split_gain': 0.0017232924907570114}. Best is trial 0 with value: 0.0386369731099313.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         290
1           Price_lag_2         225
3         Load_DA_lag_0         129
5       WindOn_DA_lag_0         113
2           Price_lag_7         112
14      pct_chg_Load_DA         111
11           NGas_lag_2          96
16  roll2d_median_Price          88
10           Coal_lag_2          86
15       lag168_Load_DA          85
13            EUA_lag_2          80
12            Oil_lag_2          33
9                  WD_7          17
7                  WD_1           9
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          23
1           Price_lag_2           1
16  roll2d_median_Price           1
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
2           Price_lag_7           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 78.93% done
********************* END NS.. 576 ****************************************************
********************* START NS ... 577 ****************************************************
[I 2025-06-15 20:37:14,537] A new study created in memory with name: no-name-994a43c0-c4d1-4bcf-a092-700c43aa4e6e
[I 2025-06-15 20:37:14,732] Trial 0 finished with value: 0.0036241628833821662 and parameters: {'n_estimators': 244, 'learning_rate': 0.2767470226840626, 'num_leaves': 145, 'max_depth': 10, 'min_child_samples': 12, 'subsample': 0.7467795569740235, 'colsample_bytree': 0.8655006935014425, 'reg_alpha': 2.3612785910828227, 'reg_lambda': 3.167431433079794, 'min_split_gain': 0.028654345889713752}. Best is trial 0 with value: 0.0036241628833821662.
[I 2025-06-15 20:37:14,942] Trial 1 finished with value: 0.004216549068190348 and parameters: {'n_estimators': 107, 'learning_rate': 0.2718448886620801, 'num_leaves': 173, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.8228399267651578, 'colsample_bytree': 0.5042733788667366, 'reg_alpha': 2.7174548102428995, 'reg_lambda': 0.3249976479783834, 'min_split_gain': 0.02713574555947684}. Best is trial 0 with value: 0.0036241628833821662.
[Trial 0] New best: 0.00362
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:15,868] Trial 2 finished with value: 0.0040860933415862175 and parameters: {'n_estimators': 258, 'learning_rate': 0.035675863303111274, 'num_leaves': 245, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.6797411548832653, 'colsample_bytree': 0.7154886048370346, 'reg_alpha': 1.4202415915814515, 'reg_lambda': 7.171472593994489, 'min_split_gain': 0.044070694417068304}. Best is trial 0 with value: 0.0036241628833821662.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:17,509] Trial 3 finished with value: 0.0038266026461848787 and parameters: {'n_estimators': 223, 'learning_rate': 0.03237361871147124, 'num_leaves': 60, 'max_depth': 13, 'min_child_samples': 4, 'subsample': 0.7021540744986662, 'colsample_bytree': 0.8599399095061171, 'reg_alpha': 7.47778762298598, 'reg_lambda': 6.621115461980175, 'min_split_gain': 0.035160685134768443}. Best is trial 0 with value: 0.0036241628833821662.
[I 2025-06-15 20:37:17,511] A new study created in memory with name: no-name-e0cab615-b03c-4eeb-a848-6a12e6c515f2
[I 2025-06-15 20:37:17,577] Trial 0 finished with value: 0.007050598808347991 and parameters: {'n_estimators': 276, 'learning_rate': 0.1162765846003522, 'num_leaves': 138, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.9988486788085438, 'colsample_bytree': 0.8611766696593012, 'reg_alpha': 7.362799148684369, 'reg_lambda': 6.332965476107905, 'min_split_gain': 0.024349779145388457}. Best is trial 0 with value: 0.007050598808347991.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         268
1           Price_lag_2         186
14      pct_chg_Load_DA         183
3         Load_DA_lag_0         137
2           Price_lag_7         102
16  roll2d_median_Price          98
5       WindOn_DA_lag_0          87
15       lag168_Load_DA          77
11           NGas_lag_2          75
12            Oil_lag_2          56
10           Coal_lag_2          52
13            EUA_lag_2          48
9                  WD_7           6
7                  WD_1           4
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 79.07% done
********************* END NS.. 577 ****************************************************
********************* START NS ... 578 ****************************************************
[I 2025-06-15 20:37:19,089] A new study created in memory with name: no-name-34711c4c-28cd-4fb4-bbcb-87ae777a7aac
[I 2025-06-15 20:37:21,714] Trial 0 finished with value: 0.002518840303383168 and parameters: {'n_estimators': 227, 'learning_rate': 0.026476373876119938, 'num_leaves': 244, 'max_depth': 15, 'min_child_samples': 10, 'subsample': 0.8362905114796875, 'colsample_bytree': 0.9641531503446812, 'reg_alpha': 0.9681210955507791, 'reg_lambda': 0.7113068756328267, 'min_split_gain': 0.008502653522080766}. Best is trial 0 with value: 0.002518840303383168.
[Trial 0] New best: 0.00252
[I 2025-06-15 20:37:22,001] Trial 1 finished with value: 0.0021870077956815583 and parameters: {'n_estimators': 237, 'learning_rate': 0.13289485007117144, 'num_leaves': 34, 'max_depth': 16, 'min_child_samples': 5, 'subsample': 0.9473434015880204, 'colsample_bytree': 0.9197168103963521, 'reg_alpha': 2.3407943959300126, 'reg_lambda': 2.130176663666913, 'min_split_gain': 0.028723784566803558}. Best is trial 1 with value: 0.0021870077956815583.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:22,832] Trial 2 finished with value: 0.002319718024409319 and parameters: {'n_estimators': 238, 'learning_rate': 0.05584886573165018, 'num_leaves': 218, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.6367173502750263, 'colsample_bytree': 0.6431355877949301, 'reg_alpha': 4.866545278273532, 'reg_lambda': 3.705656560205001, 'min_split_gain': 0.005389449284726955}. Best is trial 1 with value: 0.0021870077956815583.
[I 2025-06-15 20:37:23,003] Trial 3 finished with value: 0.0024762096054427316 and parameters: {'n_estimators': 99, 'learning_rate': 0.19240090014898584, 'num_leaves': 216, 'max_depth': 7, 'min_child_samples': 25, 'subsample': 0.6674722397941006, 'colsample_bytree': 0.7862202942913764, 'reg_alpha': 6.112011393074761, 'reg_lambda': 8.55831429661779, 'min_split_gain': 0.028984256088977402}. Best is trial 1 with value: 0.0021870077956815583.
[I 2025-06-15 20:37:23,006] A new study created in memory with name: no-name-b521171a-ddb4-44ae-949d-86e2815cb310
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 20:37:23,024] Trial 0 finished with value: 0.004612123873233644 and parameters: {'n_estimators': 66, 'learning_rate': 0.13446465031985072, 'num_leaves': 90, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.9257156799571334, 'colsample_bytree': 0.5397247987988043, 'reg_alpha': 2.9964306652577166, 'reg_lambda': 0.49652959671783736, 'min_split_gain': 0.028212499672260128}. Best is trial 0 with value: 0.004612123873233644.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         448
14      pct_chg_Load_DA         293
1           Price_lag_2         211
3         Load_DA_lag_0         196
5       WindOn_DA_lag_0         139
2           Price_lag_7         130
11           NGas_lag_2         128
16  roll2d_median_Price         112
15       lag168_Load_DA         104
10           Coal_lag_2          74
13            EUA_lag_2          57
12            Oil_lag_2          50
7                  WD_1           7
9                  WD_7           5
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 79.21% done
********************* END NS.. 578 ****************************************************
********************* START NS ... 579 ****************************************************
[I 2025-06-15 20:37:24,200] A new study created in memory with name: no-name-7e0c320e-9542-4f97-8223-d52b773a44b5
[I 2025-06-15 20:37:24,505] Trial 0 finished with value: 0.0005402196991534657 and parameters: {'n_estimators': 77, 'learning_rate': 0.28173420143674505, 'num_leaves': 171, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.5915076303087887, 'colsample_bytree': 0.6262947440143527, 'reg_alpha': 4.613593789835405, 'reg_lambda': 3.1857739142412145, 'min_split_gain': 0.005402215674349203}. Best is trial 0 with value: 0.0005402196991534657.
[Trial 0] New best: 0.00054
[I 2025-06-15 20:37:25,015] Trial 1 finished with value: 0.0003602421019295054 and parameters: {'n_estimators': 160, 'learning_rate': 0.07572630287687815, 'num_leaves': 37, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.7577165604486968, 'colsample_bytree': 0.9519795924345904, 'reg_alpha': 0.6277119366811723, 'reg_lambda': 2.495278399423726, 'min_split_gain': 0.01774173753502815}. Best is trial 1 with value: 0.0003602421019295054.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:26,224] Trial 2 finished with value: 0.0005057359224664653 and parameters: {'n_estimators': 283, 'learning_rate': 0.014845669814062793, 'num_leaves': 44, 'max_depth': 14, 'min_child_samples': 17, 'subsample': 0.7222851220434966, 'colsample_bytree': 0.6587387264328498, 'reg_alpha': 6.499740228967368, 'reg_lambda': 6.472440142533287, 'min_split_gain': 0.00535640622777216}. Best is trial 1 with value: 0.0003602421019295054.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:26,593] Trial 3 finished with value: 0.00044116128968318843 and parameters: {'n_estimators': 218, 'learning_rate': 0.0846268471078423, 'num_leaves': 212, 'max_depth': 15, 'min_child_samples': 6, 'subsample': 0.9067985204364291, 'colsample_bytree': 0.5137245091923908, 'reg_alpha': 6.001685637018623, 'reg_lambda': 4.396696307920822, 'min_split_gain': 0.03877656514780617}. Best is trial 1 with value: 0.0003602421019295054.
[I 2025-06-15 20:37:26,596] A new study created in memory with name: no-name-702776f4-9b10-4315-af29-072194ce036c
[I 2025-06-15 20:37:26,620] Trial 0 finished with value: 0.004010934890956206 and parameters: {'n_estimators': 151, 'learning_rate': 0.073042195030609, 'num_leaves': 169, 'max_depth': 13, 'min_child_samples': 1, 'subsample': 0.787073250232484, 'colsample_bytree': 0.5287234404424678, 'reg_alpha': 5.130565580366183, 'reg_lambda': 1.72587365006592, 'min_split_gain': 0.01844810468634028}. Best is trial 0 with value: 0.004010934890956206.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1034
14      pct_chg_Load_DA         634
1           Price_lag_2         608
3         Load_DA_lag_0         398
11           NGas_lag_2         338
16  roll2d_median_Price         308
5       WindOn_DA_lag_0         281
2           Price_lag_7         234
15       lag168_Load_DA         219
10           Coal_lag_2         181
12            Oil_lag_2         100
13            EUA_lag_2          81
7                  WD_1          19
9                  WD_7          18
8                  WD_6           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 79.34% done
********************* END NS.. 579 ****************************************************
********************* START NS ... 580 ****************************************************
[I 2025-06-15 20:37:27,608] A new study created in memory with name: no-name-a234a246-88ac-4dff-83b0-64aa8acd02ae
[I 2025-06-15 20:37:28,361] Trial 0 finished with value: 8.87772734522185e-05 and parameters: {'n_estimators': 128, 'learning_rate': 0.0638289196275625, 'num_leaves': 243, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.931023465306091, 'colsample_bytree': 0.8910134210411539, 'reg_alpha': 1.058386996385151, 'reg_lambda': 0.055738020706728175, 'min_split_gain': 0.020202002025462015}. Best is trial 0 with value: 8.87772734522185e-05.
[Trial 0] New best: 0.00009
[I 2025-06-15 20:37:29,254] Trial 1 finished with value: 0.00011701606740277521 and parameters: {'n_estimators': 278, 'learning_rate': 0.02643965305128693, 'num_leaves': 120, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.7294172543555926, 'colsample_bytree': 0.7735279201258687, 'reg_alpha': 3.8442638513139227, 'reg_lambda': 0.3927824661479207, 'min_split_gain': 0.025403597874779017}. Best is trial 0 with value: 8.87772734522185e-05.
[I 2025-06-15 20:37:29,413] Trial 2 finished with value: 0.016113265189012117 and parameters: {'n_estimators': 58, 'learning_rate': 0.03418355277265652, 'num_leaves': 159, 'max_depth': 5, 'min_child_samples': 24, 'subsample': 0.7318008319609197, 'colsample_bytree': 0.6195930216702649, 'reg_alpha': 7.421263079137321, 'reg_lambda': 1.8198188805788151, 'min_split_gain': 0.004865904662333654}. Best is trial 0 with value: 8.87772734522185e-05.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:29,668] Trial 3 finished with value: 0.00018452066515610712 and parameters: {'n_estimators': 284, 'learning_rate': 0.13150803341636297, 'num_leaves': 210, 'max_depth': 9, 'min_child_samples': 19, 'subsample': 0.8641370753284344, 'colsample_bytree': 0.8677557445823276, 'reg_alpha': 6.289744420295929, 'reg_lambda': 3.193125197355796, 'min_split_gain': 0.04653560079889748}. Best is trial 0 with value: 8.87772734522185e-05.
[I 2025-06-15 20:37:29,671] A new study created in memory with name: no-name-56db7ce5-475e-441e-902b-1866d0cb5d49
[I 2025-06-15 20:37:29,708] Trial 0 finished with value: 0.0037939678272536892 and parameters: {'n_estimators': 291, 'learning_rate': 0.05953823117471186, 'num_leaves': 253, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.888559996250507, 'colsample_bytree': 0.9082331257086043, 'reg_alpha': 7.528236862843495, 'reg_lambda': 5.317799162526068, 'min_split_gain': 0.04861835460047705}. Best is trial 0 with value: 0.0037939678272536892.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1641
14      pct_chg_Load_DA        1557
0           Price_lag_1        1512
3         Load_DA_lag_0        1121
2           Price_lag_7         912
5       WindOn_DA_lag_0         864
16  roll2d_median_Price         805
15       lag168_Load_DA         733
11           NGas_lag_2         677
10           Coal_lag_2         399
13            EUA_lag_2         320
12            Oil_lag_2         310
9                  WD_7          33
7                  WD_1          25
8                  WD_6          10
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 79.48% done
********************* END NS.. 580 ****************************************************
********************* START NS ... 581 ****************************************************
[I 2025-06-15 20:37:31,271] A new study created in memory with name: no-name-b4133e99-a8f1-44b5-b865-c5296498dafe
[I 2025-06-15 20:37:31,527] Trial 0 finished with value: 0.001055155256736694 and parameters: {'n_estimators': 197, 'learning_rate': 0.07169380269703074, 'num_leaves': 248, 'max_depth': 16, 'min_child_samples': 27, 'subsample': 0.6394943804686677, 'colsample_bytree': 0.7978206846315146, 'reg_alpha': 6.133744704235987, 'reg_lambda': 3.037801106901561, 'min_split_gain': 0.036696840200008844}. Best is trial 0 with value: 0.001055155256736694.
[I 2025-06-15 20:37:31,678] Trial 1 finished with value: 0.0011128558173480576 and parameters: {'n_estimators': 146, 'learning_rate': 0.2076793959494004, 'num_leaves': 192, 'max_depth': 7, 'min_child_samples': 24, 'subsample': 0.7694231088414534, 'colsample_bytree': 0.5106006907224807, 'reg_alpha': 1.074916148024786, 'reg_lambda': 3.7183296850010725, 'min_split_gain': 0.03994866624121151}. Best is trial 0 with value: 0.001055155256736694.
[Trial 0] New best: 0.00106
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:32,367] Trial 2 finished with value: 0.0010830909802682926 and parameters: {'n_estimators': 187, 'learning_rate': 0.023986129494644272, 'num_leaves': 64, 'max_depth': 15, 'min_child_samples': 6, 'subsample': 0.6987116656467918, 'colsample_bytree': 0.6434193938736832, 'reg_alpha': 3.6212062264503633, 'reg_lambda': 1.5366163378479891, 'min_split_gain': 0.04630194574389767}. Best is trial 0 with value: 0.001055155256736694.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:32,677] Trial 3 finished with value: 0.0011992998468377907 and parameters: {'n_estimators': 275, 'learning_rate': 0.07109901216301946, 'num_leaves': 136, 'max_depth': 4, 'min_child_samples': 1, 'subsample': 0.5923311246065175, 'colsample_bytree': 0.5192965369790894, 'reg_alpha': 9.033197412294466, 'reg_lambda': 3.0959659009775464, 'min_split_gain': 0.01582463516120276}. Best is trial 0 with value: 0.001055155256736694.
[I 2025-06-15 20:37:32,680] A new study created in memory with name: no-name-e29c6710-f28c-427a-8d00-60c07b4c9eca
[I 2025-06-15 20:37:32,703] Trial 0 finished with value: 0.0027792107513066946 and parameters: {'n_estimators': 129, 'learning_rate': 0.07972879492092026, 'num_leaves': 35, 'max_depth': 14, 'min_child_samples': 26, 'subsample': 0.9222652650174463, 'colsample_bytree': 0.9746941260662307, 'reg_alpha': 6.08417408459179, 'reg_lambda': 5.5872677918558065, 'min_split_gain': 0.02606022876040986}. Best is trial 0 with value: 0.0027792107513066946.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         720
1           Price_lag_2         496
14      pct_chg_Load_DA         431
3         Load_DA_lag_0         253
11           NGas_lag_2         194
16  roll2d_median_Price         176
2           Price_lag_7         168
5       WindOn_DA_lag_0         119
15       lag168_Load_DA         111
10           Coal_lag_2          80
13            EUA_lag_2          57
12            Oil_lag_2          56
9                  WD_7          10
7                  WD_1           8
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 79.62% done
********************* END NS.. 581 ****************************************************
********************* START NS ... 582 ****************************************************
[I 2025-06-15 20:37:33,995] A new study created in memory with name: no-name-c2f23749-ac56-4292-be2d-3715835529bc
[I 2025-06-15 20:37:34,191] Trial 0 finished with value: 0.00270368045554979 and parameters: {'n_estimators': 69, 'learning_rate': 0.06615451416044786, 'num_leaves': 117, 'max_depth': 15, 'min_child_samples': 11, 'subsample': 0.7677375052214849, 'colsample_bytree': 0.5193456925919939, 'reg_alpha': 6.856075327936656, 'reg_lambda': 5.9153864979859065, 'min_split_gain': 0.01947950152060478}. Best is trial 0 with value: 0.00270368045554979.
[I 2025-06-15 20:37:34,403] Trial 1 finished with value: 0.0019016227064277448 and parameters: {'n_estimators': 91, 'learning_rate': 0.05956424257071644, 'num_leaves': 188, 'max_depth': 10, 'min_child_samples': 10, 'subsample': 0.715468552367513, 'colsample_bytree': 0.7083113173421551, 'reg_alpha': 9.996765589787916, 'reg_lambda': 4.931935132135524, 'min_split_gain': 0.034781824672737126}. Best is trial 1 with value: 0.0019016227064277448.
[Trial 0] New best: 0.00270
[I 2025-06-15 20:37:34,580] Trial 2 finished with value: 0.0019514465409530164 and parameters: {'n_estimators': 53, 'learning_rate': 0.15236068521790122, 'num_leaves': 252, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.8739804437405996, 'colsample_bytree': 0.7034536460496212, 'reg_alpha': 5.4132909904712925, 'reg_lambda': 6.815735188957994, 'min_split_gain': 0.020469540627921962}. Best is trial 1 with value: 0.0019016227064277448.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:34,948] Trial 3 finished with value: 0.03192217560482682 and parameters: {'n_estimators': 113, 'learning_rate': 0.015501074403163206, 'num_leaves': 42, 'max_depth': 6, 'min_child_samples': 23, 'subsample': 0.8580401350570712, 'colsample_bytree': 0.8809229405800506, 'reg_alpha': 6.6494336697392775, 'reg_lambda': 2.157503349093175, 'min_split_gain': 0.024551806449221322}. Best is trial 1 with value: 0.0019016227064277448.
[I 2025-06-15 20:37:34,951] A new study created in memory with name: no-name-ea082986-bc96-4136-a4e5-c5200564e7cc
[I 2025-06-15 20:37:34,968] Trial 0 finished with value: 0.00481558366339183 and parameters: {'n_estimators': 56, 'learning_rate': 0.27321108695145985, 'num_leaves': 224, 'max_depth': 8, 'min_child_samples': 19, 'subsample': 0.6455711432930878, 'colsample_bytree': 0.9161321625502589, 'reg_alpha': 0.29637317929075313, 'reg_lambda': 9.810271249110734, 'min_split_gain': 0.04428027214875164}. Best is trial 0 with value: 0.00481558366339183.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         624
1           Price_lag_2         425
14      pct_chg_Load_DA         325
16  roll2d_median_Price         181
3         Load_DA_lag_0         160
2           Price_lag_7         159
11           NGas_lag_2         151
15       lag168_Load_DA          87
10           Coal_lag_2          80
13            EUA_lag_2          79
5       WindOn_DA_lag_0          62
12            Oil_lag_2          53
9                  WD_7          16
8                  WD_6          11
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 79.75% done
********************* END NS.. 582 ****************************************************
********************* START NS ... 583 ****************************************************
[I 2025-06-15 20:37:35,879] A new study created in memory with name: no-name-1f2a3203-eae7-43c8-afe4-064d25ac9b8a
[I 2025-06-15 20:37:37,411] Trial 0 finished with value: 0.0010721447790604068 and parameters: {'n_estimators': 278, 'learning_rate': 0.019965115246702282, 'num_leaves': 235, 'max_depth': 8, 'min_child_samples': 28, 'subsample': 0.8178046969495215, 'colsample_bytree': 0.5815556156404704, 'reg_alpha': 0.3631728536900447, 'reg_lambda': 4.894376325779594, 'min_split_gain': 0.029415958025864143}. Best is trial 0 with value: 0.0010721447790604068.
[Trial 0] New best: 0.00107
[I 2025-06-15 20:37:37,971] Trial 1 finished with value: 0.03552463261178595 and parameters: {'n_estimators': 113, 'learning_rate': 0.014704888876254343, 'num_leaves': 164, 'max_depth': 16, 'min_child_samples': 26, 'subsample': 0.5566052790764835, 'colsample_bytree': 0.7804469329690842, 'reg_alpha': 9.429649154265787, 'reg_lambda': 7.398997161099146, 'min_split_gain': 0.013787303559423631}. Best is trial 0 with value: 0.0010721447790604068.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:38,216] Trial 2 finished with value: 0.000982691266751671 and parameters: {'n_estimators': 287, 'learning_rate': 0.24541109962674096, 'num_leaves': 140, 'max_depth': 12, 'min_child_samples': 3, 'subsample': 0.5261732326441433, 'colsample_bytree': 0.8783786567895113, 'reg_alpha': 4.230275617930575, 'reg_lambda': 7.372267022161287, 'min_split_gain': 0.04736292198455986}. Best is trial 2 with value: 0.000982691266751671.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:38,642] Trial 3 finished with value: 0.000903955564420421 and parameters: {'n_estimators': 150, 'learning_rate': 0.08743415748645916, 'num_leaves': 156, 'max_depth': 12, 'min_child_samples': 28, 'subsample': 0.9082782643103582, 'colsample_bytree': 0.7222452352226825, 'reg_alpha': 4.6748199368046714, 'reg_lambda': 5.642108021747313, 'min_split_gain': 0.021924304293236135}. Best is trial 3 with value: 0.000903955564420421.
[I 2025-06-15 20:37:38,645] A new study created in memory with name: no-name-d2273ae5-988a-4f54-bc4c-f016b2fb6f79
[I 2025-06-15 20:37:38,693] Trial 0 finished with value: 0.005373536201017815 and parameters: {'n_estimators': 284, 'learning_rate': 0.01692902933655014, 'num_leaves': 36, 'max_depth': 10, 'min_child_samples': 9, 'subsample': 0.8553848181692143, 'colsample_bytree': 0.8052296024690269, 'reg_alpha': 9.220035190479425, 'reg_lambda': 3.801967993189259, 'min_split_gain': 0.011382081806897698}. Best is trial 0 with value: 0.005373536201017815.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         678
1           Price_lag_2         539
14      pct_chg_Load_DA         436
3         Load_DA_lag_0         349
11           NGas_lag_2         219
16  roll2d_median_Price         205
2           Price_lag_7         194
5       WindOn_DA_lag_0         183
15       lag168_Load_DA         162
10           Coal_lag_2         121
12            Oil_lag_2          97
13            EUA_lag_2          79
9                  WD_7          16
7                  WD_1          14
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 79.89% done
********************* END NS.. 583 ****************************************************
********************* START NS ... 584 ****************************************************
[I 2025-06-15 20:37:40,154] A new study created in memory with name: no-name-728e789a-4ff4-405a-b0a6-12ef988a377b
[I 2025-06-15 20:37:40,243] Trial 0 finished with value: 0.000570598629749686 and parameters: {'n_estimators': 67, 'learning_rate': 0.058978623749933394, 'num_leaves': 233, 'max_depth': 4, 'min_child_samples': 17, 'subsample': 0.9984682769166807, 'colsample_bytree': 0.8352829213259749, 'reg_alpha': 5.0540324912409655, 'reg_lambda': 3.0912106731767333, 'min_split_gain': 0.011185728321687106}. Best is trial 0 with value: 0.000570598629749686.
[Trial 0] New best: 0.00057
[I 2025-06-15 20:37:40,511] Trial 1 finished with value: 0.01035626849197628 and parameters: {'n_estimators': 85, 'learning_rate': 0.0257529407635268, 'num_leaves': 219, 'max_depth': 16, 'min_child_samples': 24, 'subsample': 0.9681603471839885, 'colsample_bytree': 0.5152961851135973, 'reg_alpha': 4.451354246263931, 'reg_lambda': 1.9349473835929676, 'min_split_gain': 0.04100375190708619}. Best is trial 0 with value: 0.000570598629749686.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:41,160] Trial 2 finished with value: 0.000310933340367821 and parameters: {'n_estimators': 273, 'learning_rate': 0.04873147486132598, 'num_leaves': 63, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.6968932385916408, 'colsample_bytree': 0.523566913261794, 'reg_alpha': 3.0233469849512415, 'reg_lambda': 3.1541338031875563, 'min_split_gain': 0.04732910109185484}. Best is trial 2 with value: 0.000310933340367821.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:41,518] Trial 3 finished with value: 0.0002596070981659468 and parameters: {'n_estimators': 211, 'learning_rate': 0.2294620699202465, 'num_leaves': 239, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.6507859756709526, 'colsample_bytree': 0.5169610169040466, 'reg_alpha': 4.719812651858209, 'reg_lambda': 0.6798355885665441, 'min_split_gain': 0.0017679678520848563}. Best is trial 3 with value: 0.0002596070981659468.
[I 2025-06-15 20:37:41,521] A new study created in memory with name: no-name-48144192-0bfd-46bf-b182-f4c7ddb4a1be
[I 2025-06-15 20:37:41,534] Trial 0 finished with value: 0.0016567995194291631 and parameters: {'n_estimators': 55, 'learning_rate': 0.09264029293342044, 'num_leaves': 221, 'max_depth': 11, 'min_child_samples': 29, 'subsample': 0.8249266796815702, 'colsample_bytree': 0.7685160307908077, 'reg_alpha': 3.5405009883550367, 'reg_lambda': 6.7061389003950245, 'min_split_gain': 0.007819154202010875}. Best is trial 0 with value: 0.0016567995194291631.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         612
1           Price_lag_2         596
14      pct_chg_Load_DA         318
3         Load_DA_lag_0         308
2           Price_lag_7         301
13            EUA_lag_2         269
16  roll2d_median_Price         258
5       WindOn_DA_lag_0         225
15       lag168_Load_DA         221
11           NGas_lag_2         217
10           Coal_lag_2         214
12            Oil_lag_2         199
7                  WD_1          17
8                  WD_6          12
9                  WD_7          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 80.03% done
********************* END NS.. 584 ****************************************************
********************* START NS ... 585 ****************************************************
[I 2025-06-15 20:37:42,658] A new study created in memory with name: no-name-e07d5d21-6371-48f0-a5ac-e6cd5fdbf30e
[I 2025-06-15 20:37:43,422] Trial 0 finished with value: 0.0004010902868097509 and parameters: {'n_estimators': 158, 'learning_rate': 0.027286109191168424, 'num_leaves': 120, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.5651302440089965, 'colsample_bytree': 0.8825156782121351, 'reg_alpha': 2.2218464377868874, 'reg_lambda': 5.952916134225916, 'min_split_gain': 0.025083471159339378}. Best is trial 0 with value: 0.0004010902868097509.
[Trial 0] New best: 0.00040
[I 2025-06-15 20:37:43,705] Trial 1 finished with value: 0.00030876463395763384 and parameters: {'n_estimators': 114, 'learning_rate': 0.12579288175990785, 'num_leaves': 134, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.5213475375769852, 'colsample_bytree': 0.9807594420353931, 'reg_alpha': 0.48683090081615754, 'reg_lambda': 6.538277218483807, 'min_split_gain': 0.04832994050298251}. Best is trial 1 with value: 0.00030876463395763384.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:44,037] Trial 2 finished with value: 0.00031843381341870443 and parameters: {'n_estimators': 137, 'learning_rate': 0.15757801865900606, 'num_leaves': 101, 'max_depth': 16, 'min_child_samples': 6, 'subsample': 0.7892793311270863, 'colsample_bytree': 0.9035099363685328, 'reg_alpha': 1.0560582848929756, 'reg_lambda': 2.5158834147065767, 'min_split_gain': 0.04478627855179425}. Best is trial 1 with value: 0.00030876463395763384.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:44,375] Trial 3 finished with value: 0.06323590911116715 and parameters: {'n_estimators': 76, 'learning_rate': 0.01744597003284343, 'num_leaves': 209, 'max_depth': 9, 'min_child_samples': 15, 'subsample': 0.698345183291879, 'colsample_bytree': 0.5795380773295535, 'reg_alpha': 3.0735737776930305, 'reg_lambda': 3.469027838821599, 'min_split_gain': 0.0062938869448766754}. Best is trial 1 with value: 0.00030876463395763384.
[I 2025-06-15 20:37:44,378] A new study created in memory with name: no-name-d57ade25-c698-43a2-b0d9-0a085703dd80
[I 2025-06-15 20:37:44,397] Trial 0 finished with value: 0.0006844401194116735 and parameters: {'n_estimators': 131, 'learning_rate': 0.08083341618853501, 'num_leaves': 140, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.8095148697373301, 'colsample_bytree': 0.6909849041010043, 'reg_alpha': 4.553522384960555, 'reg_lambda': 3.4562396028910056, 'min_split_gain': 0.0014791614434703948}. Best is trial 0 with value: 0.0006844401194116735.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         687
14      pct_chg_Load_DA         573
1           Price_lag_2         524
3         Load_DA_lag_0         349
16  roll2d_median_Price         332
2           Price_lag_7         258
11           NGas_lag_2         212
5       WindOn_DA_lag_0         202
15       lag168_Load_DA         198
10           Coal_lag_2         120
12            Oil_lag_2          83
13            EUA_lag_2          76
7                  WD_1           9
9                  WD_7           8
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 80.16% done
********************* END NS.. 585 ****************************************************
********************* START NS ... 586 ****************************************************
[I 2025-06-15 20:37:45,197] A new study created in memory with name: no-name-fba02672-7e04-4314-858b-66f3b6770747
[I 2025-06-15 20:37:45,483] Trial 0 finished with value: 0.00013103986336911912 and parameters: {'n_estimators': 77, 'learning_rate': 0.14067733188330764, 'num_leaves': 125, 'max_depth': 14, 'min_child_samples': 13, 'subsample': 0.5902299959655845, 'colsample_bytree': 0.967496190253669, 'reg_alpha': 9.539051720975534, 'reg_lambda': 1.4257689891938652, 'min_split_gain': 0.0026143939030551957}. Best is trial 0 with value: 0.00013103986336911912.
[Trial 0] New best: 0.00013
[I 2025-06-15 20:37:45,771] Trial 1 finished with value: 0.00010275018529757247 and parameters: {'n_estimators': 228, 'learning_rate': 0.14719284100935456, 'num_leaves': 248, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.6049321308610247, 'colsample_bytree': 0.821728797294456, 'reg_alpha': 2.226690656538061, 'reg_lambda': 8.931771272193195, 'min_split_gain': 0.01292285530517099}. Best is trial 1 with value: 0.00010275018529757247.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:46,454] Trial 2 finished with value: 0.0027297987047021784 and parameters: {'n_estimators': 225, 'learning_rate': 0.012569892995114032, 'num_leaves': 197, 'max_depth': 13, 'min_child_samples': 20, 'subsample': 0.9353996788196608, 'colsample_bytree': 0.7912908667644486, 'reg_alpha': 7.42083234502492, 'reg_lambda': 8.487297677888035, 'min_split_gain': 0.01446049158409426}. Best is trial 1 with value: 0.00010275018529757247.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:47,216] Trial 3 finished with value: 0.00013766010189213856 and parameters: {'n_estimators': 289, 'learning_rate': 0.03595996162859625, 'num_leaves': 197, 'max_depth': 8, 'min_child_samples': 3, 'subsample': 0.7264965418590272, 'colsample_bytree': 0.5886112917276122, 'reg_alpha': 2.73233309269181, 'reg_lambda': 7.780198107104665, 'min_split_gain': 0.036364930019322526}. Best is trial 1 with value: 0.00010275018529757247.
[I 2025-06-15 20:37:47,219] A new study created in memory with name: no-name-43042d7f-1972-471d-810b-e1f9603dfcf6
[I 2025-06-15 20:37:47,245] Trial 0 finished with value: 0.00016533172188242602 and parameters: {'n_estimators': 279, 'learning_rate': 0.06774634127419797, 'num_leaves': 51, 'max_depth': 13, 'min_child_samples': 7, 'subsample': 0.9837247219912912, 'colsample_bytree': 0.7736141691149134, 'reg_alpha': 4.016852039422593, 'reg_lambda': 6.3880510694569645, 'min_split_gain': 0.029821872783710564}. Best is trial 0 with value: 0.00016533172188242602.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         571
1           Price_lag_2         504
14      pct_chg_Load_DA         397
3         Load_DA_lag_0         355
2           Price_lag_7         254
5       WindOn_DA_lag_0         247
15       lag168_Load_DA         211
11           NGas_lag_2         194
16  roll2d_median_Price         185
12            Oil_lag_2         151
10           Coal_lag_2         120
13            EUA_lag_2          91
9                  WD_7          13
7                  WD_1           8
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 80.30% done
********************* END NS.. 586 ****************************************************
********************* START NS ... 587 ****************************************************
[I 2025-06-15 20:37:48,201] A new study created in memory with name: no-name-84bf41d2-a67e-4a32-a3e9-ca012ae3efcc
[I 2025-06-15 20:37:48,379] Trial 0 finished with value: 0.0012212553332022968 and parameters: {'n_estimators': 147, 'learning_rate': 0.1739010965663898, 'num_leaves': 181, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.7085039091812637, 'colsample_bytree': 0.8903817663384911, 'reg_alpha': 0.13535721587944693, 'reg_lambda': 7.192453785692966, 'min_split_gain': 0.040105920136248974}. Best is trial 0 with value: 0.0012212553332022968.
[Trial 0] New best: 0.00122
[I 2025-06-15 20:37:48,902] Trial 1 finished with value: 0.001585816635929668 and parameters: {'n_estimators': 140, 'learning_rate': 0.033932007092102034, 'num_leaves': 152, 'max_depth': 7, 'min_child_samples': 11, 'subsample': 0.8709734473216026, 'colsample_bytree': 0.8269681715430042, 'reg_alpha': 8.203760596907838, 'reg_lambda': 2.0779252872389975, 'min_split_gain': 0.014429017243677783}. Best is trial 0 with value: 0.0012212553332022968.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:49,376] Trial 2 finished with value: 0.01203329853654545 and parameters: {'n_estimators': 108, 'learning_rate': 0.021584665167844224, 'num_leaves': 81, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.887451668159815, 'colsample_bytree': 0.9071203995088102, 'reg_alpha': 9.403804890038256, 'reg_lambda': 0.31976362451808904, 'min_split_gain': 0.0429311656109352}. Best is trial 0 with value: 0.0012212553332022968.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:49,865] Trial 3 finished with value: 0.061669297145896175 and parameters: {'n_estimators': 139, 'learning_rate': 0.010259603010276971, 'num_leaves': 139, 'max_depth': 10, 'min_child_samples': 29, 'subsample': 0.7611697098732824, 'colsample_bytree': 0.6817069503920345, 'reg_alpha': 7.6605020081023465, 'reg_lambda': 5.8782219345675655, 'min_split_gain': 0.011879697459670402}. Best is trial 0 with value: 0.0012212553332022968.
[I 2025-06-15 20:37:49,868] A new study created in memory with name: no-name-1322d3b5-f09f-451d-995e-3ce2de75c3e0
[I 2025-06-15 20:37:49,894] Trial 0 finished with value: 0.00905804668894061 and parameters: {'n_estimators': 212, 'learning_rate': 0.012756048758143925, 'num_leaves': 116, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.7056851581381931, 'colsample_bytree': 0.643519807867293, 'reg_alpha': 9.777217547732974, 'reg_lambda': 1.7564547800665697, 'min_split_gain': 0.018661914729430875}. Best is trial 0 with value: 0.00905804668894061.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         563
1           Price_lag_2         520
14      pct_chg_Load_DA         426
16  roll2d_median_Price         304
3         Load_DA_lag_0         265
2           Price_lag_7         220
5       WindOn_DA_lag_0         199
11           NGas_lag_2         186
15       lag168_Load_DA         163
10           Coal_lag_2         134
12            Oil_lag_2          82
13            EUA_lag_2          76
7                  WD_1           8
9                  WD_7           6
8                  WD_6           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 80.44% done
********************* END NS.. 587 ****************************************************
********************* START NS ... 588 ****************************************************
[I 2025-06-15 20:37:50,759] A new study created in memory with name: no-name-f65c0e27-1c12-4eb6-be33-83c96137a655
[I 2025-06-15 20:37:50,987] Trial 0 finished with value: 0.07998718857186003 and parameters: {'n_estimators': 74, 'learning_rate': 0.022556746787004603, 'num_leaves': 74, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.7585637530663255, 'colsample_bytree': 0.8919144124043656, 'reg_alpha': 9.09569805850272, 'reg_lambda': 1.9185287225675585, 'min_split_gain': 0.019608142715383792}. Best is trial 0 with value: 0.07998718857186003.
[I 2025-06-15 20:37:51,152] Trial 1 finished with value: 0.01257726588305755 and parameters: {'n_estimators': 95, 'learning_rate': 0.10252985415141562, 'num_leaves': 50, 'max_depth': 14, 'min_child_samples': 14, 'subsample': 0.5756557958398896, 'colsample_bytree': 0.6361801296801788, 'reg_alpha': 8.353695765504556, 'reg_lambda': 8.808937459700012, 'min_split_gain': 0.033698102339123286}. Best is trial 1 with value: 0.01257726588305755.
[Trial 0] New best: 0.07999
[Trial 1] New best: 0.01258
[I 2025-06-15 20:37:51,238] Trial 2 finished with value: 0.012842521730844615 and parameters: {'n_estimators': 67, 'learning_rate': 0.18232628467366266, 'num_leaves': 38, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.8801645839704431, 'colsample_bytree': 0.9834250745480408, 'reg_alpha': 8.788036383129768, 'reg_lambda': 6.495032727897034, 'min_split_gain': 0.04134369865631206}. Best is trial 1 with value: 0.01257726588305755.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:51,731] Trial 3 finished with value: 0.07138589296583268 and parameters: {'n_estimators': 97, 'learning_rate': 0.017960097336307473, 'num_leaves': 209, 'max_depth': 10, 'min_child_samples': 23, 'subsample': 0.7337500774853642, 'colsample_bytree': 0.922750762820614, 'reg_alpha': 6.136606463795275, 'reg_lambda': 1.9752751874776397, 'min_split_gain': 0.017585619679123216}. Best is trial 1 with value: 0.01257726588305755.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:52,090] Trial 4 finished with value: 0.013860218018070298 and parameters: {'n_estimators': 260, 'learning_rate': 0.16410945342400296, 'num_leaves': 227, 'max_depth': 11, 'min_child_samples': 3, 'subsample': 0.6796184407315486, 'colsample_bytree': 0.5393324739267332, 'reg_alpha': 9.993850485494292, 'reg_lambda': 4.866065996572736, 'min_split_gain': 0.021980235945459253}. Best is trial 1 with value: 0.01257726588305755.
[I 2025-06-15 20:37:52,093] A new study created in memory with name: no-name-5cf41db8-3963-45e1-bf49-36398cd67465
[I 2025-06-15 20:37:52,114] Trial 0 finished with value: 0.056152050093405036 and parameters: {'n_estimators': 105, 'learning_rate': 0.02405300659919182, 'num_leaves': 52, 'max_depth': 7, 'min_child_samples': 27, 'subsample': 0.5062809103563746, 'colsample_bytree': 0.9712280323120721, 'reg_alpha': 9.621507268763708, 'reg_lambda': 0.6177315128499516, 'min_split_gain': 0.03824691692485564}. Best is trial 0 with value: 0.056152050093405036.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         425
1           Price_lag_2         274
14      pct_chg_Load_DA         224
3         Load_DA_lag_0         174
11           NGas_lag_2         139
16  roll2d_median_Price         125
2           Price_lag_7         119
15       lag168_Load_DA          95
5       WindOn_DA_lag_0          93
10           Coal_lag_2          82
13            EUA_lag_2          46
12            Oil_lag_2          35
9                  WD_7          14
8                  WD_6           9
7                  WD_1           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 80.57% done
********************* END NS.. 588 ****************************************************
********************* START NS ... 589 ****************************************************
[I 2025-06-15 20:37:52,914] A new study created in memory with name: no-name-5da30bd5-fcf5-4e47-8c78-c15202532a09
[I 2025-06-15 20:37:53,150] Trial 0 finished with value: 0.026629604101405132 and parameters: {'n_estimators': 99, 'learning_rate': 0.07158243020099067, 'num_leaves': 178, 'max_depth': 10, 'min_child_samples': 8, 'subsample': 0.931735381141798, 'colsample_bytree': 0.7025153730355276, 'reg_alpha': 8.042154560807615, 'reg_lambda': 7.254738506781755, 'min_split_gain': 0.020400960848199118}. Best is trial 0 with value: 0.026629604101405132.
[Trial 0] New best: 0.02663
[I 2025-06-15 20:37:53,680] Trial 1 finished with value: 0.03270065072764682 and parameters: {'n_estimators': 207, 'learning_rate': 0.016377294985516894, 'num_leaves': 242, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.5601282926849203, 'colsample_bytree': 0.9549930733468012, 'reg_alpha': 2.6399746934113324, 'reg_lambda': 1.7318105870057499, 'min_split_gain': 0.046496054424677274}. Best is trial 0 with value: 0.026629604101405132.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:53,978] Trial 2 finished with value: 0.023831564653925608 and parameters: {'n_estimators': 137, 'learning_rate': 0.26828357657071317, 'num_leaves': 71, 'max_depth': 9, 'min_child_samples': 21, 'subsample': 0.8738003192686693, 'colsample_bytree': 0.8274586743449137, 'reg_alpha': 5.005839463573802, 'reg_lambda': 2.2377396131711027, 'min_split_gain': 0.024632299270716657}. Best is trial 2 with value: 0.023831564653925608.
[Trial 2] New best: 0.02383
[I 2025-06-15 20:37:54,229] Trial 3 finished with value: 0.026658139282202162 and parameters: {'n_estimators': 130, 'learning_rate': 0.11019574483709926, 'num_leaves': 104, 'max_depth': 8, 'min_child_samples': 7, 'subsample': 0.8876368593731565, 'colsample_bytree': 0.9728128287720007, 'reg_alpha': 9.028734940498849, 'reg_lambda': 7.09671316370378, 'min_split_gain': 0.01570741925935354}. Best is trial 2 with value: 0.023831564653925608.
[I 2025-06-15 20:37:54,436] Trial 4 finished with value: 0.023734878551336384 and parameters: {'n_estimators': 78, 'learning_rate': 0.16438541174661267, 'num_leaves': 159, 'max_depth': 10, 'min_child_samples': 22, 'subsample': 0.6950007037657567, 'colsample_bytree': 0.8625643708687463, 'reg_alpha': 4.259814416173092, 'reg_lambda': 3.7699404917727843, 'min_split_gain': 0.02676647894420199}. Best is trial 4 with value: 0.023734878551336384.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:55,069] Trial 5 finished with value: 0.026293287861344713 and parameters: {'n_estimators': 226, 'learning_rate': 0.0330005052874403, 'num_leaves': 42, 'max_depth': 7, 'min_child_samples': 3, 'subsample': 0.9511531571616934, 'colsample_bytree': 0.5718370909797075, 'reg_alpha': 6.605159671618664, 'reg_lambda': 3.124610065711615, 'min_split_gain': 0.002014030410899753}. Best is trial 4 with value: 0.023734878551336384.
[I 2025-06-15 20:37:55,072] A new study created in memory with name: no-name-22de4264-6de2-4c12-be42-7da7d480810e
[I 2025-06-15 20:37:55,102] Trial 0 finished with value: 0.09164636276458614 and parameters: {'n_estimators': 220, 'learning_rate': 0.09313419743703642, 'num_leaves': 210, 'max_depth': 5, 'min_child_samples': 3, 'subsample': 0.5045937003663332, 'colsample_bytree': 0.503414551739817, 'reg_alpha': 6.178578290853938, 'reg_lambda': 1.3347148399159114, 'min_split_gain': 0.020044630474847853}. Best is trial 0 with value: 0.09164636276458614.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         373
14      pct_chg_Load_DA         262
1           Price_lag_2         246
3         Load_DA_lag_0         152
2           Price_lag_7         136
5       WindOn_DA_lag_0          97
16  roll2d_median_Price          94
11           NGas_lag_2          93
15       lag168_Load_DA          92
10           Coal_lag_2          55
12            Oil_lag_2          53
13            EUA_lag_2          35
9                  WD_7           9
7                  WD_1           8
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          11
10           Coal_lag_2           4
11           NGas_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 80.71% done
********************* END NS.. 589 ****************************************************
********************* START NS ... 590 ****************************************************
[I 2025-06-15 20:37:56,379] A new study created in memory with name: no-name-30d0abfa-c633-4dfb-9864-e32152d03250
[I 2025-06-15 20:37:56,625] Trial 0 finished with value: 0.23168561547181785 and parameters: {'n_estimators': 63, 'learning_rate': 0.015076345302863966, 'num_leaves': 207, 'max_depth': 14, 'min_child_samples': 12, 'subsample': 0.6441754564956248, 'colsample_bytree': 0.9987299641752505, 'reg_alpha': 3.3636945849163533, 'reg_lambda': 5.591642178152628, 'min_split_gain': 0.021463978832859228}. Best is trial 0 with value: 0.23168561547181785.
[Trial 0] New best: 0.23169
[I 2025-06-15 20:37:56,956] Trial 1 finished with value: 0.013036247325115309 and parameters: {'n_estimators': 155, 'learning_rate': 0.10726226967526585, 'num_leaves': 144, 'max_depth': 15, 'min_child_samples': 5, 'subsample': 0.9487499558754591, 'colsample_bytree': 0.5321573721122186, 'reg_alpha': 5.688414541690739, 'reg_lambda': 4.8891408007329185, 'min_split_gain': 0.035179779052484324}. Best is trial 1 with value: 0.013036247325115309.
[Trial 1] New best: 0.01304
[I 2025-06-15 20:37:57,206] Trial 2 finished with value: 0.012728555291420312 and parameters: {'n_estimators': 166, 'learning_rate': 0.1476321998915798, 'num_leaves': 47, 'max_depth': 12, 'min_child_samples': 26, 'subsample': 0.8539684814306705, 'colsample_bytree': 0.503860351412459, 'reg_alpha': 4.8199994698619255, 'reg_lambda': 6.349103093435302, 'min_split_gain': 0.031657081167835284}. Best is trial 2 with value: 0.012728555291420312.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:37:57,740] Trial 3 finished with value: 0.04023328431810699 and parameters: {'n_estimators': 185, 'learning_rate': 0.012508554831728236, 'num_leaves': 244, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.9717402507474772, 'colsample_bytree': 0.7684421988894066, 'reg_alpha': 6.40582307758149, 'reg_lambda': 6.5450574822413525, 'min_split_gain': 0.040973303005826266}. Best is trial 2 with value: 0.012728555291420312.
[I 2025-06-15 20:37:57,929] Trial 4 finished with value: 0.014274239593185743 and parameters: {'n_estimators': 240, 'learning_rate': 0.29370755889529265, 'num_leaves': 151, 'max_depth': 16, 'min_child_samples': 25, 'subsample': 0.9577992184618633, 'colsample_bytree': 0.8412374999201093, 'reg_alpha': 6.694086076634886, 'reg_lambda': 6.7793889291418274, 'min_split_gain': 0.042788664420160605}. Best is trial 2 with value: 0.012728555291420312.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:37:57,930] A new study created in memory with name: no-name-eefc4ee4-6603-4c5e-a5c6-5a63b1907875
[I 2025-06-15 20:37:57,959] Trial 0 finished with value: 0.03870764112987307 and parameters: {'n_estimators': 240, 'learning_rate': 0.12863844498309046, 'num_leaves': 110, 'max_depth': 10, 'min_child_samples': 5, 'subsample': 0.8427819271209995, 'colsample_bytree': 0.5164033622520485, 'reg_alpha': 8.725082035763052, 'reg_lambda': 5.545904279745414, 'min_split_gain': 0.027505655253723837}. Best is trial 0 with value: 0.03870764112987307.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         473
1           Price_lag_2         421
14      pct_chg_Load_DA         175
3         Load_DA_lag_0         167
16  roll2d_median_Price         154
2           Price_lag_7         147
5       WindOn_DA_lag_0         110
11           NGas_lag_2         109
15       lag168_Load_DA         105
12            Oil_lag_2          85
13            EUA_lag_2          70
10           Coal_lag_2          55
8                  WD_6          15
7                  WD_1          12
9                  WD_7          10
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 80.85% done
********************* END NS.. 590 ****************************************************
********************* START NS ... 591 ****************************************************
[I 2025-06-15 20:37:59,013] A new study created in memory with name: no-name-2c04c74d-024b-41cd-8b0a-21fdee085d27
[I 2025-06-15 20:37:59,659] Trial 0 finished with value: 0.0004893400495650373 and parameters: {'n_estimators': 178, 'learning_rate': 0.053333351654678576, 'num_leaves': 153, 'max_depth': 15, 'min_child_samples': 20, 'subsample': 0.7353521395678589, 'colsample_bytree': 0.5140662337423156, 'reg_alpha': 0.4572707616428462, 'reg_lambda': 4.050622035071579, 'min_split_gain': 0.026562127228154316}. Best is trial 0 with value: 0.0004893400495650373.
[I 2025-06-15 20:37:59,775] Trial 1 finished with value: 0.0007351673971755794 and parameters: {'n_estimators': 93, 'learning_rate': 0.23762439018048834, 'num_leaves': 62, 'max_depth': 9, 'min_child_samples': 10, 'subsample': 0.9514244888698884, 'colsample_bytree': 0.7674037557482581, 'reg_alpha': 8.476126056360926, 'reg_lambda': 3.078213548448997, 'min_split_gain': 0.02603425768255721}. Best is trial 0 with value: 0.0004893400495650373.
[Trial 0] New best: 0.00049
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:00,118] Trial 2 finished with value: 0.09519003251717091 and parameters: {'n_estimators': 85, 'learning_rate': 0.014170215788468539, 'num_leaves': 179, 'max_depth': 9, 'min_child_samples': 7, 'subsample': 0.9669452254347299, 'colsample_bytree': 0.6079544456937065, 'reg_alpha': 7.312074027357864, 'reg_lambda': 7.958663978941348, 'min_split_gain': 0.0007782027992086105}. Best is trial 0 with value: 0.0004893400495650373.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:00,411] Trial 3 finished with value: 0.0004619000389480588 and parameters: {'n_estimators': 161, 'learning_rate': 0.06631848765391692, 'num_leaves': 105, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.5793831799657652, 'colsample_bytree': 0.9827183589922555, 'reg_alpha': 0.05734291692328197, 'reg_lambda': 9.419201765592172, 'min_split_gain': 0.007660941706485686}. Best is trial 3 with value: 0.0004619000389480588.
[I 2025-06-15 20:38:00,413] A new study created in memory with name: no-name-4a06a112-2b0a-4e07-a8d4-adac9b5a55ac
[I 2025-06-15 20:38:00,445] Trial 0 finished with value: 0.008329537201555546 and parameters: {'n_estimators': 257, 'learning_rate': 0.06476051332560008, 'num_leaves': 235, 'max_depth': 11, 'min_child_samples': 18, 'subsample': 0.9016118744564674, 'colsample_bytree': 0.9455080211743886, 'reg_alpha': 8.400767798973764, 'reg_lambda': 1.3198086955680743, 'min_split_gain': 0.04887190029735881}. Best is trial 0 with value: 0.008329537201555546.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         941
14      pct_chg_Load_DA         547
1           Price_lag_2         410
3         Load_DA_lag_0         316
16  roll2d_median_Price         277
11           NGas_lag_2         251
2           Price_lag_7         218
5       WindOn_DA_lag_0         209
15       lag168_Load_DA         168
12            Oil_lag_2         151
13            EUA_lag_2         140
10           Coal_lag_2         108
7                  WD_1          19
9                  WD_7          13
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 80.98% done
********************* END NS.. 591 ****************************************************
********************* START NS ... 592 ****************************************************
[I 2025-06-15 20:38:01,752] A new study created in memory with name: no-name-e9cb7985-9fec-4bf4-ac61-df5d7681e1ec
[I 2025-06-15 20:38:02,101] Trial 0 finished with value: 0.0016147648007250606 and parameters: {'n_estimators': 152, 'learning_rate': 0.02152479890423278, 'num_leaves': 87, 'max_depth': 6, 'min_child_samples': 7, 'subsample': 0.7626446570551868, 'colsample_bytree': 0.6462407223581069, 'reg_alpha': 0.5420659642899928, 'reg_lambda': 0.5320515518773994, 'min_split_gain': 0.02270921216059717}. Best is trial 0 with value: 0.0016147648007250606.
[I 2025-06-15 20:38:02,274] Trial 1 finished with value: 0.00032697974720076575 and parameters: {'n_estimators': 178, 'learning_rate': 0.15977052959044494, 'num_leaves': 40, 'max_depth': 6, 'min_child_samples': 15, 'subsample': 0.8596148346250536, 'colsample_bytree': 0.5323964128089387, 'reg_alpha': 5.746206012735621, 'reg_lambda': 8.657415122591086, 'min_split_gain': 0.014991254913711555}. Best is trial 1 with value: 0.00032697974720076575.
[Trial 0] New best: 0.00161
[Trial 1] New best: 0.00033
[I 2025-06-15 20:38:03,065] Trial 2 finished with value: 0.003897252432633971 and parameters: {'n_estimators': 179, 'learning_rate': 0.015594713639549805, 'num_leaves': 176, 'max_depth': 10, 'min_child_samples': 30, 'subsample': 0.8372420276239128, 'colsample_bytree': 0.7463167994494794, 'reg_alpha': 2.301187554192258, 'reg_lambda': 8.710076447868587, 'min_split_gain': 0.02714545157188369}. Best is trial 1 with value: 0.00032697974720076575.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:03,275] Trial 3 finished with value: 0.23597970330526133 and parameters: {'n_estimators': 56, 'learning_rate': 0.01255888127104733, 'num_leaves': 244, 'max_depth': 8, 'min_child_samples': 29, 'subsample': 0.7720555508772765, 'colsample_bytree': 0.5369685441416776, 'reg_alpha': 6.366044230776868, 'reg_lambda': 9.80689743712615, 'min_split_gain': 0.03524609177368961}. Best is trial 1 with value: 0.00032697974720076575.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:03,854] Trial 4 finished with value: 0.018137932281110578 and parameters: {'n_estimators': 146, 'learning_rate': 0.013780096614437352, 'num_leaves': 208, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.5438050073584006, 'colsample_bytree': 0.8973790700154753, 'reg_alpha': 7.780410491728037, 'reg_lambda': 4.062371673609507, 'min_split_gain': 0.022389850610342845}. Best is trial 1 with value: 0.00032697974720076575.
[I 2025-06-15 20:38:03,857] A new study created in memory with name: no-name-c9ce6080-99c9-4765-ac2e-fe1a98669766
[I 2025-06-15 20:38:03,879] Trial 0 finished with value: 0.012483830139197386 and parameters: {'n_estimators': 66, 'learning_rate': 0.01063889924472814, 'num_leaves': 250, 'max_depth': 14, 'min_child_samples': 21, 'subsample': 0.8375895819145123, 'colsample_bytree': 0.7343162124064699, 'reg_alpha': 7.027329368071088, 'reg_lambda': 8.507414571236488, 'min_split_gain': 0.010584096690605067}. Best is trial 0 with value: 0.012483830139197386.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         496
1           Price_lag_2         446
16  roll2d_median_Price         179
3         Load_DA_lag_0         174
14      pct_chg_Load_DA         161
2           Price_lag_7         132
11           NGas_lag_2         130
5       WindOn_DA_lag_0         108
15       lag168_Load_DA          94
13            EUA_lag_2          92
12            Oil_lag_2          78
10           Coal_lag_2          49
8                  WD_6          21
9                  WD_7          17
7                  WD_1          12
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          44
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 81.12% done
********************* END NS.. 592 ****************************************************
********************* START NS ... 593 ****************************************************
[I 2025-06-15 20:38:04,880] A new study created in memory with name: no-name-86cf7c67-c4e6-4230-a783-e892711c6415
[I 2025-06-15 20:38:05,040] Trial 0 finished with value: 0.0005909509030366885 and parameters: {'n_estimators': 173, 'learning_rate': 0.13876170930738504, 'num_leaves': 197, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.7003259912780784, 'colsample_bytree': 0.8466258182302315, 'reg_alpha': 4.643146493014831, 'reg_lambda': 4.181519178444807, 'min_split_gain': 0.04021305588204856}. Best is trial 0 with value: 0.0005909509030366885.
[Trial 0] New best: 0.00059
[I 2025-06-15 20:38:05,258] Trial 1 finished with value: 0.02124979810769971 and parameters: {'n_estimators': 131, 'learning_rate': 0.014818366370536741, 'num_leaves': 111, 'max_depth': 5, 'min_child_samples': 28, 'subsample': 0.5667833473509788, 'colsample_bytree': 0.7625753601477174, 'reg_alpha': 4.419065340105521, 'reg_lambda': 0.258373213307872, 'min_split_gain': 0.04874330874615743}. Best is trial 0 with value: 0.0005909509030366885.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:05,592] Trial 2 finished with value: 0.00066999846755541 and parameters: {'n_estimators': 207, 'learning_rate': 0.2571075991799118, 'num_leaves': 145, 'max_depth': 6, 'min_child_samples': 14, 'subsample': 0.9147644987374242, 'colsample_bytree': 0.5370355643817982, 'reg_alpha': 3.225352399726271, 'reg_lambda': 7.040844983588378, 'min_split_gain': 0.001189376242329343}. Best is trial 0 with value: 0.0005909509030366885.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:06,610] Trial 3 finished with value: 0.0004802885902532052 and parameters: {'n_estimators': 297, 'learning_rate': 0.05068526651549465, 'num_leaves': 57, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.8880341350440766, 'colsample_bytree': 0.5302956921740181, 'reg_alpha': 0.635881470933457, 'reg_lambda': 0.9564913028299704, 'min_split_gain': 0.005621443996224601}. Best is trial 3 with value: 0.0004802885902532052.
[I 2025-06-15 20:38:06,613] A new study created in memory with name: no-name-8ce5031c-4a8e-40ab-a313-b785bb210004
[I 2025-06-15 20:38:06,635] Trial 0 finished with value: 0.010045571744963106 and parameters: {'n_estimators': 158, 'learning_rate': 0.013350382511699965, 'num_leaves': 225, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.7276902092437387, 'colsample_bytree': 0.7933104535355616, 'reg_alpha': 6.8234952402688585, 'reg_lambda': 7.92669828930089, 'min_split_gain': 0.024298904736628907}. Best is trial 0 with value: 0.010045571744963106.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2531
1           Price_lag_2        2451
14      pct_chg_Load_DA        1961
3         Load_DA_lag_0        1375
5       WindOn_DA_lag_0        1215
15       lag168_Load_DA        1142
16  roll2d_median_Price        1096
2           Price_lag_7        1094
11           NGas_lag_2        1040
13            EUA_lag_2         797
12            Oil_lag_2         739
10           Coal_lag_2         659
9                  WD_7          49
7                  WD_1          44
8                  WD_6          19
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 81.26% done
********************* END NS.. 593 ****************************************************
********************* START NS ... 594 ****************************************************
[I 2025-06-15 20:38:08,516] A new study created in memory with name: no-name-37771c0b-7b1c-46f5-a1de-470e93b68452
[I 2025-06-15 20:38:08,912] Trial 0 finished with value: 0.0016138118356798722 and parameters: {'n_estimators': 184, 'learning_rate': 0.06215655659845799, 'num_leaves': 52, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.6935824876107869, 'colsample_bytree': 0.6775782868336646, 'reg_alpha': 3.2966798154344703, 'reg_lambda': 6.999945666376876, 'min_split_gain': 0.0162730249229714}. Best is trial 0 with value: 0.0016138118356798722.
[Trial 0] New best: 0.00161
[I 2025-06-15 20:38:09,686] Trial 1 finished with value: 0.004962197477730588 and parameters: {'n_estimators': 292, 'learning_rate': 0.010253247296147365, 'num_leaves': 204, 'max_depth': 13, 'min_child_samples': 12, 'subsample': 0.7469376626107487, 'colsample_bytree': 0.822430879715588, 'reg_alpha': 8.956748463208239, 'reg_lambda': 6.550907576332942, 'min_split_gain': 0.02456829224908566}. Best is trial 0 with value: 0.0016138118356798722.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:10,050] Trial 2 finished with value: 0.0016258910155230082 and parameters: {'n_estimators': 108, 'learning_rate': 0.06173770336158332, 'num_leaves': 56, 'max_depth': 16, 'min_child_samples': 13, 'subsample': 0.8013325154039244, 'colsample_bytree': 0.521587035421679, 'reg_alpha': 6.074988886115955, 'reg_lambda': 1.6483400358833622, 'min_split_gain': 0.036709945588045}. Best is trial 0 with value: 0.0016138118356798722.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:10,684] Trial 3 finished with value: 0.02954666032087309 and parameters: {'n_estimators': 111, 'learning_rate': 0.016430956523197218, 'num_leaves': 176, 'max_depth': 7, 'min_child_samples': 3, 'subsample': 0.6084127265313433, 'colsample_bytree': 0.9260131917854699, 'reg_alpha': 7.677361849916531, 'reg_lambda': 3.121763373784232, 'min_split_gain': 0.016971345332871125}. Best is trial 0 with value: 0.0016138118356798722.
[I 2025-06-15 20:38:10,687] A new study created in memory with name: no-name-143f203c-a2e2-4f75-a731-b428ee2ebbee
[I 2025-06-15 20:38:10,708] Trial 0 finished with value: 0.01011289804271463 and parameters: {'n_estimators': 102, 'learning_rate': 0.05156338438080529, 'num_leaves': 116, 'max_depth': 11, 'min_child_samples': 20, 'subsample': 0.7125790582427316, 'colsample_bytree': 0.8364914613201903, 'reg_alpha': 6.637729634729865, 'reg_lambda': 7.118258176557387, 'min_split_gain': 0.021888701500676247}. Best is trial 0 with value: 0.01011289804271463.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1172
1           Price_lag_2        1030
14      pct_chg_Load_DA         716
3         Load_DA_lag_0         567
11           NGas_lag_2         431
2           Price_lag_7         425
5       WindOn_DA_lag_0         414
16  roll2d_median_Price         371
15       lag168_Load_DA         362
10           Coal_lag_2         303
12            Oil_lag_2         219
13            EUA_lag_2         190
9                  WD_7          38
7                  WD_1          24
8                  WD_6          17
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 81.40% done
********************* END NS.. 594 ****************************************************
********************* START NS ... 595 ****************************************************
[I 2025-06-15 20:38:12,027] A new study created in memory with name: no-name-4c8b4db8-a0aa-4ee9-97af-acdd731522f1
[I 2025-06-15 20:38:12,544] Trial 0 finished with value: 0.001470434003421948 and parameters: {'n_estimators': 291, 'learning_rate': 0.0913401517962349, 'num_leaves': 83, 'max_depth': 16, 'min_child_samples': 2, 'subsample': 0.8652561437447955, 'colsample_bytree': 0.981977828777669, 'reg_alpha': 0.6389717703143538, 'reg_lambda': 5.824011188391954, 'min_split_gain': 0.013254263658699451}. Best is trial 0 with value: 0.001470434003421948.
[Trial 0] New best: 0.00147
[I 2025-06-15 20:38:13,120] Trial 1 finished with value: 0.0020145690465156276 and parameters: {'n_estimators': 127, 'learning_rate': 0.03543508097731718, 'num_leaves': 119, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.8676832249617594, 'colsample_bytree': 0.6003426162369716, 'reg_alpha': 2.291789305945481, 'reg_lambda': 3.746144824810865, 'min_split_gain': 0.03210671258846633}. Best is trial 0 with value: 0.001470434003421948.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:13,678] Trial 2 finished with value: 0.012357505268539522 and parameters: {'n_estimators': 162, 'learning_rate': 0.015041075135646073, 'num_leaves': 98, 'max_depth': 15, 'min_child_samples': 5, 'subsample': 0.7901235472174672, 'colsample_bytree': 0.9034315007306597, 'reg_alpha': 6.133574442954992, 'reg_lambda': 1.6722120628893267, 'min_split_gain': 0.038191629133024124}. Best is trial 0 with value: 0.001470434003421948.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:14,560] Trial 3 finished with value: 0.0014575035089752747 and parameters: {'n_estimators': 231, 'learning_rate': 0.04403572361365089, 'num_leaves': 107, 'max_depth': 12, 'min_child_samples': 22, 'subsample': 0.7142860716214386, 'colsample_bytree': 0.8640064783811869, 'reg_alpha': 6.6309833191333745, 'reg_lambda': 0.04756613514724006, 'min_split_gain': 0.00385761611659371}. Best is trial 3 with value: 0.0014575035089752747.
[I 2025-06-15 20:38:14,563] A new study created in memory with name: no-name-cf816976-9e99-4898-a304-57680401764f
[I 2025-06-15 20:38:14,584] Trial 0 finished with value: 0.004244818519208603 and parameters: {'n_estimators': 202, 'learning_rate': 0.2946264316496515, 'num_leaves': 180, 'max_depth': 5, 'min_child_samples': 14, 'subsample': 0.8633014913082104, 'colsample_bytree': 0.6844867018064222, 'reg_alpha': 7.302004377525609, 'reg_lambda': 4.127335171490609, 'min_split_gain': 0.006326674014597378}. Best is trial 0 with value: 0.004244818519208603.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1643
1           Price_lag_2        1328
14      pct_chg_Load_DA        1178
3         Load_DA_lag_0         871
2           Price_lag_7         667
5       WindOn_DA_lag_0         635
15       lag168_Load_DA         633
11           NGas_lag_2         545
16  roll2d_median_Price         520
12            Oil_lag_2         457
13            EUA_lag_2         430
10           Coal_lag_2         357
9                  WD_7          38
7                  WD_1          27
8                  WD_6          26
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 81.53% done
********************* END NS.. 595 ****************************************************
********************* START NS ... 596 ****************************************************
[I 2025-06-15 20:38:15,919] A new study created in memory with name: no-name-0b35b9eb-c953-48e6-8386-e34d6e40a01d
[I 2025-06-15 20:38:16,281] Trial 0 finished with value: 0.0001708105353707292 and parameters: {'n_estimators': 151, 'learning_rate': 0.19903424539539497, 'num_leaves': 234, 'max_depth': 10, 'min_child_samples': 12, 'subsample': 0.65114753528198, 'colsample_bytree': 0.7988370941559395, 'reg_alpha': 0.4083305247824176, 'reg_lambda': 2.9721929669015767, 'min_split_gain': 0.009644888101852933}. Best is trial 0 with value: 0.0001708105353707292.
[I 2025-06-15 20:38:16,451] Trial 1 finished with value: 0.000361649254847279 and parameters: {'n_estimators': 229, 'learning_rate': 0.11046832532117949, 'num_leaves': 96, 'max_depth': 5, 'min_child_samples': 23, 'subsample': 0.8284412970600132, 'colsample_bytree': 0.9208888120597225, 'reg_alpha': 9.043711381353793, 'reg_lambda': 9.726343825093688, 'min_split_gain': 0.047019785172821356}. Best is trial 0 with value: 0.0001708105353707292.
[Trial 0] New best: 0.00017
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:16,787] Trial 2 finished with value: 0.0004169963142170348 and parameters: {'n_estimators': 229, 'learning_rate': 0.1824359785780045, 'num_leaves': 99, 'max_depth': 14, 'min_child_samples': 12, 'subsample': 0.7175618527344487, 'colsample_bytree': 0.711230972190791, 'reg_alpha': 8.27644329171611, 'reg_lambda': 2.3701851262191043, 'min_split_gain': 0.002199297767053654}. Best is trial 0 with value: 0.0001708105353707292.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:17,026] Trial 3 finished with value: 0.0002463959838620673 and parameters: {'n_estimators': 78, 'learning_rate': 0.09009523783958082, 'num_leaves': 127, 'max_depth': 6, 'min_child_samples': 12, 'subsample': 0.6262071454155647, 'colsample_bytree': 0.9419989556174346, 'reg_alpha': 4.874953937645648, 'reg_lambda': 4.690859027743239, 'min_split_gain': 0.005102154307368989}. Best is trial 0 with value: 0.0001708105353707292.
[I 2025-06-15 20:38:17,029] A new study created in memory with name: no-name-15cd100a-33bd-467a-b456-4e2f84ad5b6d
[I 2025-06-15 20:38:17,070] Trial 0 finished with value: 0.004922563153681296 and parameters: {'n_estimators': 241, 'learning_rate': 0.029801194371515925, 'num_leaves': 221, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.6333656541382013, 'colsample_bytree': 0.6209466300553397, 'reg_alpha': 5.403466695292979, 'reg_lambda': 3.9881104644355303, 'min_split_gain': 0.004670658200466887}. Best is trial 0 with value: 0.004922563153681296.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         704
0           Price_lag_1         692
14      pct_chg_Load_DA         630
3         Load_DA_lag_0         468
16  roll2d_median_Price         415
2           Price_lag_7         390
15       lag168_Load_DA         357
5       WindOn_DA_lag_0         342
11           NGas_lag_2         336
10           Coal_lag_2         233
12            Oil_lag_2         194
13            EUA_lag_2         186
9                  WD_7          15
7                  WD_1          12
8                  WD_6           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 81.67% done
********************* END NS.. 596 ****************************************************
********************* START NS ... 597 ****************************************************
[I 2025-06-15 20:38:18,176] A new study created in memory with name: no-name-32878623-c2f4-45bd-ad92-872ea0e59d5e
[I 2025-06-15 20:38:18,412] Trial 0 finished with value: 0.05486470441896324 and parameters: {'n_estimators': 119, 'learning_rate': 0.013515149650325482, 'num_leaves': 170, 'max_depth': 7, 'min_child_samples': 11, 'subsample': 0.8522170490253103, 'colsample_bytree': 0.6669837981841169, 'reg_alpha': 6.45002632534224, 'reg_lambda': 6.335432531700384, 'min_split_gain': 0.03152314499554144}. Best is trial 0 with value: 0.05486470441896324.
[I 2025-06-15 20:38:18,589] Trial 1 finished with value: 0.0004177144732988967 and parameters: {'n_estimators': 177, 'learning_rate': 0.20693081064271257, 'num_leaves': 80, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.7251332350287967, 'colsample_bytree': 0.7660370202529867, 'reg_alpha': 0.8241040242211939, 'reg_lambda': 2.450846509245096, 'min_split_gain': 0.0417063636367895}. Best is trial 1 with value: 0.0004177144732988967.
[Trial 0] New best: 0.05486
[Trial 1] New best: 0.00042
[I 2025-06-15 20:38:19,008] Trial 2 finished with value: 0.013566138697363651 and parameters: {'n_estimators': 172, 'learning_rate': 0.013932373489209237, 'num_leaves': 180, 'max_depth': 6, 'min_child_samples': 30, 'subsample': 0.75971202060517, 'colsample_bytree': 0.7457653121062864, 'reg_alpha': 6.719326557844192, 'reg_lambda': 8.027071136727846, 'min_split_gain': 0.04266224264317525}. Best is trial 1 with value: 0.0004177144732988967.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:19,391] Trial 3 finished with value: 0.00034543944500906406 and parameters: {'n_estimators': 259, 'learning_rate': 0.24854341629039328, 'num_leaves': 215, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.7949840356528528, 'colsample_bytree': 0.8008034784040221, 'reg_alpha': 0.06536720643561478, 'reg_lambda': 6.411036529070557, 'min_split_gain': 0.020554370757519702}. Best is trial 3 with value: 0.00034543944500906406.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:19,853] Trial 4 finished with value: 0.0009713584309708934 and parameters: {'n_estimators': 100, 'learning_rate': 0.14413071386515852, 'num_leaves': 199, 'max_depth': 9, 'min_child_samples': 15, 'subsample': 0.8837246433137061, 'colsample_bytree': 0.5573298318784847, 'reg_alpha': 0.324651873552978, 'reg_lambda': 0.630453117763895, 'min_split_gain': 0.04692497721041893}. Best is trial 3 with value: 0.00034543944500906406.
[I 2025-06-15 20:38:19,856] A new study created in memory with name: no-name-c148b779-98e4-43fd-8218-27e9ab71aab1
[I 2025-06-15 20:38:19,875] Trial 0 finished with value: 0.00572338960380402 and parameters: {'n_estimators': 70, 'learning_rate': 0.019549460531797156, 'num_leaves': 163, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.7380158587813466, 'colsample_bytree': 0.6509894227044395, 'reg_alpha': 4.300115590145964, 'reg_lambda': 4.664398510710873, 'min_split_gain': 0.04086990058080581}. Best is trial 0 with value: 0.00572338960380402.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         319
1           Price_lag_2         233
14      pct_chg_Load_DA         213
3         Load_DA_lag_0         169
16  roll2d_median_Price         121
2           Price_lag_7         109
5       WindOn_DA_lag_0         103
11           NGas_lag_2          94
10           Coal_lag_2          86
15       lag168_Load_DA          69
13            EUA_lag_2          49
12            Oil_lag_2          42
7                  WD_1           6
9                  WD_7           5
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 81.81% done
********************* END NS.. 597 ****************************************************
********************* START NS ... 598 ****************************************************
[I 2025-06-15 20:38:21,087] A new study created in memory with name: no-name-e32e0f92-2ccd-4457-a241-7dfbf31d0bfc
[I 2025-06-15 20:38:21,435] Trial 0 finished with value: 0.0009004980305846915 and parameters: {'n_estimators': 262, 'learning_rate': 0.026254297828961902, 'num_leaves': 102, 'max_depth': 5, 'min_child_samples': 4, 'subsample': 0.5712131261493285, 'colsample_bytree': 0.7921207413830632, 'reg_alpha': 2.6435751915914176, 'reg_lambda': 1.4780273332612726, 'min_split_gain': 0.035499751100200906}. Best is trial 0 with value: 0.0009004980305846915.
[Trial 0] New best: 0.00090
[I 2025-06-15 20:38:22,083] Trial 1 finished with value: 0.0008769268985180751 and parameters: {'n_estimators': 205, 'learning_rate': 0.04521799914377044, 'num_leaves': 46, 'max_depth': 12, 'min_child_samples': 2, 'subsample': 0.6146571767474268, 'colsample_bytree': 0.5225439722888612, 'reg_alpha': 1.0186218576427153, 'reg_lambda': 3.5910261026781787, 'min_split_gain': 0.016944564585233687}. Best is trial 1 with value: 0.0008769268985180751.
[I 2025-06-15 20:38:22,237] Trial 2 finished with value: 0.0008251468309000476 and parameters: {'n_estimators': 119, 'learning_rate': 0.25036571086908727, 'num_leaves': 195, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.8798537245159754, 'colsample_bytree': 0.9666686112014613, 'reg_alpha': 2.1167286263261156, 'reg_lambda': 9.995924218606469, 'min_split_gain': 0.04820518113913729}. Best is trial 2 with value: 0.0008251468309000476.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:23,143] Trial 3 finished with value: 0.0009676238279351046 and parameters: {'n_estimators': 271, 'learning_rate': 0.029149216639546578, 'num_leaves': 237, 'max_depth': 10, 'min_child_samples': 19, 'subsample': 0.7129685643403394, 'colsample_bytree': 0.587610810692933, 'reg_alpha': 4.218476526885466, 'reg_lambda': 6.074404707988123, 'min_split_gain': 0.02262224576073318}. Best is trial 2 with value: 0.0008251468309000476.
[I 2025-06-15 20:38:23,145] A new study created in memory with name: no-name-07433bc7-2d53-496b-93e5-c8f8e163e00f
[I 2025-06-15 20:38:23,174] Trial 0 finished with value: 0.0061804096504980395 and parameters: {'n_estimators': 255, 'learning_rate': 0.018799867816133065, 'num_leaves': 201, 'max_depth': 14, 'min_child_samples': 4, 'subsample': 0.8484268387578769, 'colsample_bytree': 0.9022309255856431, 'reg_alpha': 1.4216335650785783, 'reg_lambda': 1.494476760643898, 'min_split_gain': 0.037707815661522454}. Best is trial 0 with value: 0.0061804096504980395.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         262
14      pct_chg_Load_DA         179
1           Price_lag_2         151
3         Load_DA_lag_0         101
16  roll2d_median_Price          77
2           Price_lag_7          74
11           NGas_lag_2          67
5       WindOn_DA_lag_0          56
15       lag168_Load_DA          56
10           Coal_lag_2          34
12            Oil_lag_2          30
13            EUA_lag_2          23
7                  WD_1           5
9                  WD_7           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          34
16  roll2d_median_Price           2
2           Price_lag_7           0
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 81.94% done
********************* END NS.. 598 ****************************************************
********************* START NS ... 599 ****************************************************
[I 2025-06-15 20:38:24,157] A new study created in memory with name: no-name-1af0dacc-b4aa-451b-b888-74bab71f5aed
[I 2025-06-15 20:38:24,695] Trial 0 finished with value: 0.0009970721733400254 and parameters: {'n_estimators': 272, 'learning_rate': 0.026086376693292987, 'num_leaves': 196, 'max_depth': 13, 'min_child_samples': 9, 'subsample': 0.8841767072412665, 'colsample_bytree': 0.9834239613850827, 'reg_alpha': 7.53833873241193, 'reg_lambda': 0.16723304919372128, 'min_split_gain': 0.027907966374910644}. Best is trial 0 with value: 0.0009970721733400254.
[Trial 0] New best: 0.00100
[I 2025-06-15 20:38:25,055] Trial 1 finished with value: 0.013125195389529434 and parameters: {'n_estimators': 110, 'learning_rate': 0.023061280535503218, 'num_leaves': 180, 'max_depth': 10, 'min_child_samples': 12, 'subsample': 0.683148080926147, 'colsample_bytree': 0.9615853702128606, 'reg_alpha': 9.976436540445595, 'reg_lambda': 4.441759342668945, 'min_split_gain': 0.025374185093850407}. Best is trial 0 with value: 0.0009970721733400254.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:25,660] Trial 2 finished with value: 0.0017695387551252169 and parameters: {'n_estimators': 128, 'learning_rate': 0.05259745511677703, 'num_leaves': 169, 'max_depth': 11, 'min_child_samples': 20, 'subsample': 0.7991281175355778, 'colsample_bytree': 0.8762127195662235, 'reg_alpha': 5.204354007038198, 'reg_lambda': 0.25826732300233357, 'min_split_gain': 0.012642179349078797}. Best is trial 0 with value: 0.0009970721733400254.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:26,127] Trial 3 finished with value: 0.0012766806659276467 and parameters: {'n_estimators': 152, 'learning_rate': 0.03634843987157266, 'num_leaves': 156, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.6425669487814671, 'colsample_bytree': 0.5045352187046035, 'reg_alpha': 5.520125304320633, 'reg_lambda': 6.657399065220354, 'min_split_gain': 0.030777801464876044}. Best is trial 0 with value: 0.0009970721733400254.
[I 2025-06-15 20:38:26,129] A new study created in memory with name: no-name-45029a60-2417-4c1e-bb01-6cb45d178565
[I 2025-06-15 20:38:26,156] Trial 0 finished with value: 0.008655535734389035 and parameters: {'n_estimators': 182, 'learning_rate': 0.09718404620393867, 'num_leaves': 57, 'max_depth': 15, 'min_child_samples': 15, 'subsample': 0.931494694568398, 'colsample_bytree': 0.7285735835347102, 'reg_alpha': 9.921333326102985, 'reg_lambda': 4.369609574899773, 'min_split_gain': 0.012640101493172573}. Best is trial 0 with value: 0.008655535734389035.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2122
14      pct_chg_Load_DA        1213
1           Price_lag_2        1071
2           Price_lag_7         631
3         Load_DA_lag_0         599
11           NGas_lag_2         589
16  roll2d_median_Price         562
10           Coal_lag_2         334
5       WindOn_DA_lag_0         321
13            EUA_lag_2         259
15       lag168_Load_DA         225
12            Oil_lag_2         200
9                  WD_7          37
7                  WD_1          17
8                  WD_6          17
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 82.08% done
********************* END NS.. 599 ****************************************************
********************* START NS ... 600 ****************************************************
[I 2025-06-15 20:38:27,573] A new study created in memory with name: no-name-aac9253a-81b9-4cec-a093-fef90462c1ee
[I 2025-06-15 20:38:27,812] Trial 0 finished with value: 0.032641833639118635 and parameters: {'n_estimators': 69, 'learning_rate': 0.027903955762035606, 'num_leaves': 250, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.5386831819345415, 'colsample_bytree': 0.7008249278114075, 'reg_alpha': 8.370792746997624, 'reg_lambda': 2.1001040213069775, 'min_split_gain': 0.0008969197448343569}. Best is trial 0 with value: 0.032641833639118635.
[Trial 0] New best: 0.03264
[I 2025-06-15 20:38:28,141] Trial 1 finished with value: 0.02658175865048575 and parameters: {'n_estimators': 130, 'learning_rate': 0.015845659411861605, 'num_leaves': 143, 'max_depth': 12, 'min_child_samples': 24, 'subsample': 0.788089352027151, 'colsample_bytree': 0.9089210592275894, 'reg_alpha': 9.3095892698301, 'reg_lambda': 3.4031256483979035, 'min_split_gain': 0.02188695716135159}. Best is trial 1 with value: 0.02658175865048575.
[I 2025-06-15 20:38:28,328] Trial 2 finished with value: 0.0009422562722410398 and parameters: {'n_estimators': 79, 'learning_rate': 0.15891937765362765, 'num_leaves': 124, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.5800505576300861, 'colsample_bytree': 0.7720242569564432, 'reg_alpha': 7.340455324162054, 'reg_lambda': 6.109726841927239, 'min_split_gain': 0.013781449083448522}. Best is trial 2 with value: 0.0009422562722410398.
[Trial 1] New best: 0.02658
[Trial 2] New best: 0.00094
[I 2025-06-15 20:38:28,959] Trial 3 finished with value: 0.0010462527124514272 and parameters: {'n_estimators': 259, 'learning_rate': 0.060450455584121494, 'num_leaves': 131, 'max_depth': 9, 'min_child_samples': 5, 'subsample': 0.8474674324983487, 'colsample_bytree': 0.6889824212112563, 'reg_alpha': 9.300399106584639, 'reg_lambda': 0.10062152439228811, 'min_split_gain': 0.007881893947100023}. Best is trial 2 with value: 0.0009422562722410398.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:29,335] Trial 4 finished with value: 0.0012085673359745828 and parameters: {'n_estimators': 160, 'learning_rate': 0.05693599941298385, 'num_leaves': 249, 'max_depth': 16, 'min_child_samples': 18, 'subsample': 0.9451093163381539, 'colsample_bytree': 0.756412344256647, 'reg_alpha': 6.184022212285862, 'reg_lambda': 8.148242763405872, 'min_split_gain': 0.03904254678612759}. Best is trial 2 with value: 0.0009422562722410398.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:30,135] Trial 5 finished with value: 0.008686788774257428 and parameters: {'n_estimators': 174, 'learning_rate': 0.015972076309818114, 'num_leaves': 142, 'max_depth': 10, 'min_child_samples': 5, 'subsample': 0.8645813143186427, 'colsample_bytree': 0.9303661858233232, 'reg_alpha': 7.219448646011484, 'reg_lambda': 4.215817917870121, 'min_split_gain': 0.0014095740341520226}. Best is trial 2 with value: 0.0009422562722410398.
[I 2025-06-15 20:38:30,138] A new study created in memory with name: no-name-f593a92e-4592-4610-a2ec-ff73b307624a
[I 2025-06-15 20:38:30,163] Trial 0 finished with value: 0.006019254764412664 and parameters: {'n_estimators': 264, 'learning_rate': 0.012786874743185923, 'num_leaves': 190, 'max_depth': 13, 'min_child_samples': 12, 'subsample': 0.981531547409547, 'colsample_bytree': 0.9199731117241801, 'reg_alpha': 1.8546782977278564, 'reg_lambda': 9.037456307154585, 'min_split_gain': 0.0495210582906716}. Best is trial 0 with value: 0.006019254764412664.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         347
1           Price_lag_2         273
14      pct_chg_Load_DA         231
3         Load_DA_lag_0         147
16  roll2d_median_Price         118
2           Price_lag_7         114
11           NGas_lag_2         103
5       WindOn_DA_lag_0          94
15       lag168_Load_DA          94
13            EUA_lag_2          62
10           Coal_lag_2          56
12            Oil_lag_2          46
9                  WD_7          10
7                  WD_1           5
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 82.22% done
********************* END NS.. 600 ****************************************************
********************* START NS ... 601 ****************************************************
[I 2025-06-15 20:38:31,027] A new study created in memory with name: no-name-ff39ace2-695b-40e0-9367-7fd9a32bb519
[I 2025-06-15 20:38:31,474] Trial 0 finished with value: 0.0009309152799785986 and parameters: {'n_estimators': 217, 'learning_rate': 0.025506513841658116, 'num_leaves': 33, 'max_depth': 11, 'min_child_samples': 28, 'subsample': 0.651797575319061, 'colsample_bytree': 0.740479180147726, 'reg_alpha': 7.085555320045339, 'reg_lambda': 3.997691069141367, 'min_split_gain': 0.006318649506782881}. Best is trial 0 with value: 0.0009309152799785986.
[Trial 0] New best: 0.00093
[I 2025-06-15 20:38:31,754] Trial 1 finished with value: 0.12523454533183798 and parameters: {'n_estimators': 76, 'learning_rate': 0.01540853683221799, 'num_leaves': 60, 'max_depth': 11, 'min_child_samples': 4, 'subsample': 0.8096645019966878, 'colsample_bytree': 0.785387089730756, 'reg_alpha': 7.157694601024679, 'reg_lambda': 6.475890510165341, 'min_split_gain': 0.004015861462965931}. Best is trial 0 with value: 0.0009309152799785986.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:32,617] Trial 2 finished with value: 0.0007560074641671971 and parameters: {'n_estimators': 239, 'learning_rate': 0.03767043926250723, 'num_leaves': 46, 'max_depth': 10, 'min_child_samples': 25, 'subsample': 0.8561321437754336, 'colsample_bytree': 0.8862714246632551, 'reg_alpha': 1.739836391804389, 'reg_lambda': 3.547499552887131, 'min_split_gain': 0.022195926581394188}. Best is trial 2 with value: 0.0007560074641671971.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:32,886] Trial 3 finished with value: 0.0007739385620625429 and parameters: {'n_estimators': 103, 'learning_rate': 0.08943423386543994, 'num_leaves': 199, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.7799601637747198, 'colsample_bytree': 0.9459467340695811, 'reg_alpha': 7.2041969112973785, 'reg_lambda': 1.1974144463027636, 'min_split_gain': 0.032098498381667805}. Best is trial 2 with value: 0.0007560074641671971.
[I 2025-06-15 20:38:32,889] A new study created in memory with name: no-name-49c4895c-acb8-4af7-affc-1992644b2be1
[I 2025-06-15 20:38:32,916] Trial 0 finished with value: 0.003113129222151905 and parameters: {'n_estimators': 154, 'learning_rate': 0.03280008860642306, 'num_leaves': 220, 'max_depth': 16, 'min_child_samples': 17, 'subsample': 0.8811039179259745, 'colsample_bytree': 0.6971877894479536, 'reg_alpha': 1.0648450422719569, 'reg_lambda': 5.074512123463137, 'min_split_gain': 0.03198257180211613}. Best is trial 0 with value: 0.003113129222151905.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1972
1           Price_lag_2        1503
14      pct_chg_Load_DA        1390
3         Load_DA_lag_0         891
11           NGas_lag_2         624
15       lag168_Load_DA         602
2           Price_lag_7         589
5       WindOn_DA_lag_0         556
16  roll2d_median_Price         484
10           Coal_lag_2         319
13            EUA_lag_2         267
12            Oil_lag_2         258
9                  WD_7          35
7                  WD_1          33
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          11
16  roll2d_median_Price           1
2           Price_lag_7           0
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 82.35% done
********************* END NS.. 601 ****************************************************
********************* START NS ... 602 ****************************************************
[I 2025-06-15 20:38:34,185] A new study created in memory with name: no-name-99cfcf9b-865c-4bec-b693-e6c0d864f259
[I 2025-06-15 20:38:34,346] Trial 0 finished with value: 0.048248100040024944 and parameters: {'n_estimators': 78, 'learning_rate': 0.022063998951643512, 'num_leaves': 58, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.8485215465602289, 'colsample_bytree': 0.7799743953995435, 'reg_alpha': 8.5543292464892, 'reg_lambda': 8.895725413081012, 'min_split_gain': 0.005931661278532563}. Best is trial 0 with value: 0.048248100040024944.
[Trial 0] New best: 0.04825
[I 2025-06-15 20:38:34,791] Trial 1 finished with value: 0.0027850540950986285 and parameters: {'n_estimators': 249, 'learning_rate': 0.013865080295719177, 'num_leaves': 53, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.7472212619845757, 'colsample_bytree': 0.6224113141772865, 'reg_alpha': 9.338185788636332, 'reg_lambda': 1.275556654108484, 'min_split_gain': 0.01094230671263099}. Best is trial 1 with value: 0.0027850540950986285.
[Trial 1] New best: 0.00279
[I 2025-06-15 20:38:35,019] Trial 2 finished with value: 0.00036706999286001196 and parameters: {'n_estimators': 206, 'learning_rate': 0.13472401373479317, 'num_leaves': 252, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.973607410729706, 'colsample_bytree': 0.6914668560193115, 'reg_alpha': 7.6591626495947125, 'reg_lambda': 1.806434665144101, 'min_split_gain': 0.016224612427508323}. Best is trial 2 with value: 0.00036706999286001196.
[Trial 2] New best: 0.00037
[I 2025-06-15 20:38:35,243] Trial 3 finished with value: 0.000403272898721471 and parameters: {'n_estimators': 161, 'learning_rate': 0.0457594172736926, 'num_leaves': 51, 'max_depth': 4, 'min_child_samples': 18, 'subsample': 0.8795094933802194, 'colsample_bytree': 0.8477337221091489, 'reg_alpha': 6.740981219110047, 'reg_lambda': 8.673854474679032, 'min_split_gain': 0.03449344821894658}. Best is trial 2 with value: 0.00036706999286001196.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:36,008] Trial 4 finished with value: 0.0003786374388456842 and parameters: {'n_estimators': 248, 'learning_rate': 0.044960941897704026, 'num_leaves': 241, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.8814069022831837, 'colsample_bytree': 0.6705139816761269, 'reg_alpha': 5.656544695264958, 'reg_lambda': 6.871405906103494, 'min_split_gain': 0.0024949969569406983}. Best is trial 2 with value: 0.00036706999286001196.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:36,552] Trial 5 finished with value: 0.0003409099797422499 and parameters: {'n_estimators': 204, 'learning_rate': 0.039497781253256814, 'num_leaves': 212, 'max_depth': 6, 'min_child_samples': 6, 'subsample': 0.752216367175669, 'colsample_bytree': 0.9109891326181685, 'reg_alpha': 9.716686994289322, 'reg_lambda': 0.16446467759926753, 'min_split_gain': 0.03062440094532323}. Best is trial 5 with value: 0.0003409099797422499.
[I 2025-06-15 20:38:36,555] A new study created in memory with name: no-name-3ff7ab1d-20ca-4c6b-a20e-8dd2802f2b16
[I 2025-06-15 20:38:36,577] Trial 0 finished with value: 0.0031441652479232453 and parameters: {'n_estimators': 112, 'learning_rate': 0.07098695466894443, 'num_leaves': 81, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.8777780280197875, 'colsample_bytree': 0.5789463753704681, 'reg_alpha': 2.616032190100992, 'reg_lambda': 0.9638743662857774, 'min_split_gain': 0.011936092796023723}. Best is trial 0 with value: 0.0031441652479232453.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1185
14      pct_chg_Load_DA         674
1           Price_lag_2         518
11           NGas_lag_2         316
2           Price_lag_7         310
3         Load_DA_lag_0         305
16  roll2d_median_Price         237
10           Coal_lag_2         166
5       WindOn_DA_lag_0         159
15       lag168_Load_DA         133
12            Oil_lag_2         105
13            EUA_lag_2          86
9                  WD_7          26
8                  WD_6          10
7                  WD_1           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 82.49% done
********************* END NS.. 602 ****************************************************
********************* START NS ... 603 ****************************************************
[I 2025-06-15 20:38:37,828] A new study created in memory with name: no-name-d4cd32a1-42ca-4bf1-b363-96b8ad2bf71f
[I 2025-06-15 20:38:38,187] Trial 0 finished with value: 0.001636026425825116 and parameters: {'n_estimators': 293, 'learning_rate': 0.1386327166889482, 'num_leaves': 249, 'max_depth': 16, 'min_child_samples': 15, 'subsample': 0.8668393923208507, 'colsample_bytree': 0.5192753381424684, 'reg_alpha': 1.0640418115751193, 'reg_lambda': 4.79890638981364, 'min_split_gain': 0.009749499450030619}. Best is trial 0 with value: 0.001636026425825116.
[Trial 0] New best: 0.00164
[I 2025-06-15 20:38:38,870] Trial 1 finished with value: 0.01045722805648544 and parameters: {'n_estimators': 223, 'learning_rate': 0.013720535944770502, 'num_leaves': 56, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.5921253682989831, 'colsample_bytree': 0.6785163487024279, 'reg_alpha': 8.376517203739283, 'reg_lambda': 0.012168801151650754, 'min_split_gain': 0.016096270281857568}. Best is trial 0 with value: 0.001636026425825116.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:39,634] Trial 2 finished with value: 0.0014174990033297228 and parameters: {'n_estimators': 247, 'learning_rate': 0.02311073810412684, 'num_leaves': 56, 'max_depth': 13, 'min_child_samples': 17, 'subsample': 0.5423925027150289, 'colsample_bytree': 0.9841399628646834, 'reg_alpha': 3.517475508772141, 'reg_lambda': 4.112351589803983, 'min_split_gain': 0.04134325686934826}. Best is trial 2 with value: 0.0014174990033297228.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:39,921] Trial 3 finished with value: 0.0018714623251782881 and parameters: {'n_estimators': 226, 'learning_rate': 0.2613481106680927, 'num_leaves': 75, 'max_depth': 11, 'min_child_samples': 2, 'subsample': 0.7647775483340988, 'colsample_bytree': 0.588362563840792, 'reg_alpha': 9.00958419861103, 'reg_lambda': 2.292401590928476, 'min_split_gain': 0.006728117110418075}. Best is trial 2 with value: 0.0014174990033297228.
[I 2025-06-15 20:38:39,925] A new study created in memory with name: no-name-e41d6077-3d01-4998-86a6-e26a28e30727
[I 2025-06-15 20:38:40,041] Trial 0 finished with value: 0.0071699868061907744 and parameters: {'n_estimators': 215, 'learning_rate': 0.037955015613232224, 'num_leaves': 162, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.8804437113609462, 'colsample_bytree': 0.6653985500306084, 'reg_alpha': 2.4718327999539778, 'reg_lambda': 0.6178342045409768, 'min_split_gain': 0.04045615868358087}. Best is trial 0 with value: 0.0071699868061907744.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2600
14      pct_chg_Load_DA        1552
1           Price_lag_2        1282
3         Load_DA_lag_0         746
16  roll2d_median_Price         695
2           Price_lag_7         693
11           NGas_lag_2         598
15       lag168_Load_DA         377
5       WindOn_DA_lag_0         349
12            Oil_lag_2         297
13            EUA_lag_2         282
10           Coal_lag_2         257
9                  WD_7          20
7                  WD_1          13
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 82.63% done
********************* END NS.. 603 ****************************************************
********************* START NS ... 604 ****************************************************
[I 2025-06-15 20:38:41,863] A new study created in memory with name: no-name-a5eb98b7-0e8c-41d1-acf2-8e20a5d7b97e
[I 2025-06-15 20:38:42,920] Trial 0 finished with value: 0.0009381646103621173 and parameters: {'n_estimators': 204, 'learning_rate': 0.032679091173792744, 'num_leaves': 80, 'max_depth': 16, 'min_child_samples': 2, 'subsample': 0.5229483255979821, 'colsample_bytree': 0.7876159961810987, 'reg_alpha': 1.55453590806928, 'reg_lambda': 6.97091316922991, 'min_split_gain': 0.017142615962603896}. Best is trial 0 with value: 0.0009381646103621173.
[Trial 0] New best: 0.00094
[I 2025-06-15 20:38:43,226] Trial 1 finished with value: 0.002280257860484518 and parameters: {'n_estimators': 125, 'learning_rate': 0.0904980038773716, 'num_leaves': 47, 'max_depth': 4, 'min_child_samples': 3, 'subsample': 0.8219163566324488, 'colsample_bytree': 0.5439382737011622, 'reg_alpha': 8.734818476794059, 'reg_lambda': 7.820083311095855, 'min_split_gain': 0.023203396855710906}. Best is trial 0 with value: 0.0009381646103621173.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:44,270] Trial 2 finished with value: 0.0010149649201128819 and parameters: {'n_estimators': 233, 'learning_rate': 0.03478764518505874, 'num_leaves': 36, 'max_depth': 12, 'min_child_samples': 6, 'subsample': 0.7936103314675995, 'colsample_bytree': 0.7273398483782425, 'reg_alpha': 3.2138379631304215, 'reg_lambda': 8.167160053311358, 'min_split_gain': 0.012925108773991952}. Best is trial 0 with value: 0.0009381646103621173.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:44,793] Trial 3 finished with value: 0.0019524126500944314 and parameters: {'n_estimators': 171, 'learning_rate': 0.05450506685974979, 'num_leaves': 64, 'max_depth': 10, 'min_child_samples': 22, 'subsample': 0.9440320386138152, 'colsample_bytree': 0.8490520654841653, 'reg_alpha': 9.279605246986968, 'reg_lambda': 1.4834026562134772, 'min_split_gain': 0.010260850208294692}. Best is trial 0 with value: 0.0009381646103621173.
[I 2025-06-15 20:38:44,797] A new study created in memory with name: no-name-01de0066-446a-46aa-bf5b-883a1e53a036
[I 2025-06-15 20:38:44,843] Trial 0 finished with value: 0.00474435955806098 and parameters: {'n_estimators': 278, 'learning_rate': 0.03160989829889033, 'num_leaves': 46, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.7464226075105099, 'colsample_bytree': 0.9484365272670685, 'reg_alpha': 5.967382443252184, 'reg_lambda': 3.4061724853473505, 'min_split_gain': 0.020412976947520617}. Best is trial 0 with value: 0.00474435955806098.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2502
1           Price_lag_2        2435
14      pct_chg_Load_DA        1909
3         Load_DA_lag_0        1348
2           Price_lag_7        1255
5       WindOn_DA_lag_0        1082
16  roll2d_median_Price        1058
11           NGas_lag_2        1045
15       lag168_Load_DA        1017
12            Oil_lag_2         665
10           Coal_lag_2         653
13            EUA_lag_2         554
9                  WD_7          51
7                  WD_1          29
8                  WD_6          22
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 82.76% done
********************* END NS.. 604 ****************************************************
********************* START NS ... 605 ****************************************************
[I 2025-06-15 20:38:47,072] A new study created in memory with name: no-name-8cd98f6b-cf15-49c6-9690-df08480dd8fa
[I 2025-06-15 20:38:47,542] Trial 0 finished with value: 0.0001867638700802191 and parameters: {'n_estimators': 194, 'learning_rate': 0.04077508051344874, 'num_leaves': 132, 'max_depth': 7, 'min_child_samples': 16, 'subsample': 0.8399219654129533, 'colsample_bytree': 0.9065453287052156, 'reg_alpha': 4.341124446774573, 'reg_lambda': 0.6867071652417656, 'min_split_gain': 0.005160092367474434}. Best is trial 0 with value: 0.0001867638700802191.
[Trial 0] New best: 0.00019
[I 2025-06-15 20:38:47,857] Trial 1 finished with value: 0.00045620550702249593 and parameters: {'n_estimators': 113, 'learning_rate': 0.04083686377420848, 'num_leaves': 32, 'max_depth': 9, 'min_child_samples': 20, 'subsample': 0.5196164917673334, 'colsample_bytree': 0.5918771758589827, 'reg_alpha': 5.526852298975243, 'reg_lambda': 7.019494570507621, 'min_split_gain': 0.03710161023269533}. Best is trial 0 with value: 0.0001867638700802191.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:48,478] Trial 2 finished with value: 0.00044132487128508 and parameters: {'n_estimators': 145, 'learning_rate': 0.03297142539878633, 'num_leaves': 174, 'max_depth': 8, 'min_child_samples': 24, 'subsample': 0.5803177011160285, 'colsample_bytree': 0.8787198510087848, 'reg_alpha': 2.109524694153, 'reg_lambda': 7.218554740186315, 'min_split_gain': 0.0435788007460741}. Best is trial 0 with value: 0.0001867638700802191.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:49,141] Trial 3 finished with value: 0.0005632451806222976 and parameters: {'n_estimators': 250, 'learning_rate': 0.017212681698038718, 'num_leaves': 209, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.9785172845198099, 'colsample_bytree': 0.5698539924488281, 'reg_alpha': 5.603825419728329, 'reg_lambda': 9.809064652865004, 'min_split_gain': 0.010140156501096759}. Best is trial 0 with value: 0.0001867638700802191.
[I 2025-06-15 20:38:49,143] A new study created in memory with name: no-name-c38c643b-6a9a-47c6-91db-37c44704763e
[I 2025-06-15 20:38:49,159] Trial 0 finished with value: 0.00032221141003820547 and parameters: {'n_estimators': 101, 'learning_rate': 0.07705913940311881, 'num_leaves': 62, 'max_depth': 5, 'min_child_samples': 20, 'subsample': 0.7992106681140791, 'colsample_bytree': 0.8682574127280271, 'reg_alpha': 1.923322917619379, 'reg_lambda': 1.060763851985751, 'min_split_gain': 0.033258864983664725}. Best is trial 0 with value: 0.00032221141003820547.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1763
1           Price_lag_2        1129
14      pct_chg_Load_DA        1110
3         Load_DA_lag_0         753
2           Price_lag_7         641
11           NGas_lag_2         521
16  roll2d_median_Price         505
5       WindOn_DA_lag_0         464
15       lag168_Load_DA         449
12            Oil_lag_2         442
13            EUA_lag_2         423
10           Coal_lag_2         327
7                  WD_1          27
9                  WD_7          21
8                  WD_6           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 82.90% done
********************* END NS.. 605 ****************************************************
********************* START NS ... 606 ****************************************************
[I 2025-06-15 20:38:50,582] A new study created in memory with name: no-name-939aa355-8750-48f7-8f09-5c2af07ea55c
[I 2025-06-15 20:38:50,789] Trial 0 finished with value: 0.0002209609209212805 and parameters: {'n_estimators': 220, 'learning_rate': 0.14616090282550276, 'num_leaves': 102, 'max_depth': 14, 'min_child_samples': 5, 'subsample': 0.8739880713677016, 'colsample_bytree': 0.5576735018518522, 'reg_alpha': 3.8512719817814887, 'reg_lambda': 3.217411123084173, 'min_split_gain': 0.024845016961055146}. Best is trial 0 with value: 0.0002209609209212805.
[I 2025-06-15 20:38:50,938] Trial 1 finished with value: 0.0001776409485878498 and parameters: {'n_estimators': 148, 'learning_rate': 0.24638871195972095, 'num_leaves': 45, 'max_depth': 13, 'min_child_samples': 8, 'subsample': 0.5127968128196115, 'colsample_bytree': 0.5065295760482242, 'reg_alpha': 1.7465057156072372, 'reg_lambda': 8.565671086492102, 'min_split_gain': 0.02712054949552742}. Best is trial 1 with value: 0.0001776409485878498.
[Trial 0] New best: 0.00022
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:51,539] Trial 2 finished with value: 0.00034872117020018725 and parameters: {'n_estimators': 248, 'learning_rate': 0.023784091713143267, 'num_leaves': 71, 'max_depth': 13, 'min_child_samples': 2, 'subsample': 0.7600157427770771, 'colsample_bytree': 0.7234166139410001, 'reg_alpha': 8.924421085170785, 'reg_lambda': 6.74982150052355, 'min_split_gain': 0.03059992544609641}. Best is trial 1 with value: 0.0001776409485878498.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:51,838] Trial 3 finished with value: 0.043541570827508015 and parameters: {'n_estimators': 94, 'learning_rate': 0.018357368832436768, 'num_leaves': 67, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.954621293512597, 'colsample_bytree': 0.8288837260956003, 'reg_alpha': 8.740784720565307, 'reg_lambda': 9.72981280373389, 'min_split_gain': 0.033993664783100015}. Best is trial 1 with value: 0.0001776409485878498.
[I 2025-06-15 20:38:51,841] A new study created in memory with name: no-name-1ccd322d-630b-45ad-b44c-b863acf0f7eb
[I 2025-06-15 20:38:51,908] Trial 0 finished with value: 0.0008564879232773841 and parameters: {'n_estimators': 148, 'learning_rate': 0.13428318241530904, 'num_leaves': 256, 'max_depth': 13, 'min_child_samples': 16, 'subsample': 0.5936290984235757, 'colsample_bytree': 0.6673975400970408, 'reg_alpha': 7.305461835033583, 'reg_lambda': 4.9873835163936695, 'min_split_gain': 0.007431764951970055}. Best is trial 0 with value: 0.0008564879232773841.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         325
1           Price_lag_2         285
16  roll2d_median_Price         185
3         Load_DA_lag_0         155
2           Price_lag_7         151
14      pct_chg_Load_DA         134
12            Oil_lag_2         115
13            EUA_lag_2         104
15       lag168_Load_DA          92
5       WindOn_DA_lag_0          87
11           NGas_lag_2          77
10           Coal_lag_2          53
8                  WD_6          13
9                  WD_7          10
7                  WD_1           9
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 83.04% done
********************* END NS.. 606 ****************************************************
********************* START NS ... 607 ****************************************************
[I 2025-06-15 20:38:52,956] A new study created in memory with name: no-name-6e20202c-0244-487a-8912-f26591647747
[I 2025-06-15 20:38:53,050] Trial 0 finished with value: 0.00031247133949419977 and parameters: {'n_estimators': 126, 'learning_rate': 0.27919958751735746, 'num_leaves': 47, 'max_depth': 9, 'min_child_samples': 26, 'subsample': 0.8471152691004327, 'colsample_bytree': 0.6327335690791709, 'reg_alpha': 5.745958833230397, 'reg_lambda': 7.50607485188373, 'min_split_gain': 0.04708413956058133}. Best is trial 0 with value: 0.00031247133949419977.
[I 2025-06-15 20:38:53,220] Trial 1 finished with value: 0.0002007477760214369 and parameters: {'n_estimators': 241, 'learning_rate': 0.12741309406451898, 'num_leaves': 97, 'max_depth': 9, 'min_child_samples': 21, 'subsample': 0.9697551360937762, 'colsample_bytree': 0.5074057790584194, 'reg_alpha': 7.296961329443738, 'reg_lambda': 1.430810726712629, 'min_split_gain': 0.02189200038266666}. Best is trial 1 with value: 0.0002007477760214369.
[Trial 0] New best: 0.00031
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:53,408] Trial 2 finished with value: 0.00031275012023761835 and parameters: {'n_estimators': 248, 'learning_rate': 0.11172304517183265, 'num_leaves': 170, 'max_depth': 13, 'min_child_samples': 21, 'subsample': 0.632646051752094, 'colsample_bytree': 0.7866166511893288, 'reg_alpha': 9.575705276812348, 'reg_lambda': 8.766747199670565, 'min_split_gain': 0.03928918700174689}. Best is trial 1 with value: 0.0002007477760214369.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:53,786] Trial 3 finished with value: 0.0003140470586572157 and parameters: {'n_estimators': 277, 'learning_rate': 0.16501481624964312, 'num_leaves': 118, 'max_depth': 5, 'min_child_samples': 29, 'subsample': 0.559041364539228, 'colsample_bytree': 0.7311079739642166, 'reg_alpha': 5.153058348100707, 'reg_lambda': 0.7601221080573339, 'min_split_gain': 0.004351089842997308}. Best is trial 1 with value: 0.0002007477760214369.
[I 2025-06-15 20:38:53,789] A new study created in memory with name: no-name-fb196d1f-16f0-43b1-a9ec-217c6c0f58a8
[I 2025-06-15 20:38:53,813] Trial 0 finished with value: 0.001685461094636749 and parameters: {'n_estimators': 71, 'learning_rate': 0.010850843943222273, 'num_leaves': 190, 'max_depth': 7, 'min_child_samples': 15, 'subsample': 0.5030575669648281, 'colsample_bytree': 0.7110707381555423, 'reg_alpha': 0.7605146015221487, 'reg_lambda': 9.758996981040898, 'min_split_gain': 0.015222113297899659}. Best is trial 0 with value: 0.001685461094636749.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         518
1           Price_lag_2         451
14      pct_chg_Load_DA         187
2           Price_lag_7         163
3         Load_DA_lag_0         147
16  roll2d_median_Price         138
11           NGas_lag_2         126
15       lag168_Load_DA         112
12            Oil_lag_2          99
5       WindOn_DA_lag_0          90
10           Coal_lag_2          66
13            EUA_lag_2          54
8                  WD_6          20
9                  WD_7          18
7                  WD_1          15
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          39
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 83.17% done
********************* END NS.. 607 ****************************************************
********************* START NS ... 608 ****************************************************
[I 2025-06-15 20:38:55,053] A new study created in memory with name: no-name-149a15ae-bfb9-4de6-9dfb-59dfd01c2b99
[I 2025-06-15 20:38:55,216] Trial 0 finished with value: 0.00020920958307653513 and parameters: {'n_estimators': 205, 'learning_rate': 0.21564159029481975, 'num_leaves': 189, 'max_depth': 13, 'min_child_samples': 13, 'subsample': 0.5152369482915723, 'colsample_bytree': 0.5006734990796708, 'reg_alpha': 2.936322779842432, 'reg_lambda': 7.310065489245643, 'min_split_gain': 0.02613359832509401}. Best is trial 0 with value: 0.00020920958307653513.
[I 2025-06-15 20:38:55,323] Trial 1 finished with value: 0.00022857650486971946 and parameters: {'n_estimators': 191, 'learning_rate': 0.23593270778011063, 'num_leaves': 57, 'max_depth': 13, 'min_child_samples': 26, 'subsample': 0.8354365234859894, 'colsample_bytree': 0.5686481003331239, 'reg_alpha': 6.077891521286113, 'reg_lambda': 1.5820825150973084, 'min_split_gain': 0.039869065531153686}. Best is trial 0 with value: 0.00020920958307653513.
[Trial 0] New best: 0.00021
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:55,656] Trial 2 finished with value: 0.0002003962108378684 and parameters: {'n_estimators': 177, 'learning_rate': 0.07408115605802483, 'num_leaves': 133, 'max_depth': 11, 'min_child_samples': 17, 'subsample': 0.6594661177578125, 'colsample_bytree': 0.7737022256677857, 'reg_alpha': 5.013703006946921, 'reg_lambda': 5.706008577470776, 'min_split_gain': 0.04858310690248593}. Best is trial 2 with value: 0.0002003962108378684.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:38:56,091] Trial 3 finished with value: 0.00014779963132714104 and parameters: {'n_estimators': 220, 'learning_rate': 0.03248908637918862, 'num_leaves': 63, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.6017210435794751, 'colsample_bytree': 0.5956079526619489, 'reg_alpha': 7.816127929342097, 'reg_lambda': 9.868909349943403, 'min_split_gain': 0.04559575026312668}. Best is trial 3 with value: 0.00014779963132714104.
[I 2025-06-15 20:38:56,094] A new study created in memory with name: no-name-043c133a-4dbc-4000-accf-0c51004ce956
[I 2025-06-15 20:38:56,124] Trial 0 finished with value: 0.0011851906076085028 and parameters: {'n_estimators': 277, 'learning_rate': 0.07057539346492817, 'num_leaves': 231, 'max_depth': 8, 'min_child_samples': 14, 'subsample': 0.8124872684112532, 'colsample_bytree': 0.7420388289898979, 'reg_alpha': 5.02942959455048, 'reg_lambda': 5.988656408888029, 'min_split_gain': 0.010646178571236026}. Best is trial 0 with value: 0.0011851906076085028.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1153
1           Price_lag_2         769
14      pct_chg_Load_DA         579
3         Load_DA_lag_0         367
2           Price_lag_7         254
16  roll2d_median_Price         251
15       lag168_Load_DA         250
11           NGas_lag_2         245
5       WindOn_DA_lag_0         157
10           Coal_lag_2         133
13            EUA_lag_2         102
12            Oil_lag_2          74
9                  WD_7          35
8                  WD_6          19
7                  WD_1          17
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 83.31% done
********************* END NS.. 608 ****************************************************
********************* START NS ... 609 ****************************************************
[I 2025-06-15 20:38:57,519] A new study created in memory with name: no-name-2ccbb64b-b2a1-4dd8-abeb-e51d57faac7a
[I 2025-06-15 20:38:58,058] Trial 0 finished with value: 0.000615729237497148 and parameters: {'n_estimators': 260, 'learning_rate': 0.014628477551794193, 'num_leaves': 155, 'max_depth': 11, 'min_child_samples': 28, 'subsample': 0.8059583320769421, 'colsample_bytree': 0.5326448442428046, 'reg_alpha': 5.29847058460081, 'reg_lambda': 1.910320953749931, 'min_split_gain': 0.031150494842920692}. Best is trial 0 with value: 0.000615729237497148.
[Trial 0] New best: 0.00062
[I 2025-06-15 20:38:58,579] Trial 1 finished with value: 0.00012658657907901652 and parameters: {'n_estimators': 173, 'learning_rate': 0.09510085817006572, 'num_leaves': 251, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.696268386472185, 'colsample_bytree': 0.969426204455663, 'reg_alpha': 1.9806048111454078, 'reg_lambda': 0.576431147660903, 'min_split_gain': 0.009394434501166322}. Best is trial 1 with value: 0.00012658657907901652.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:38:59,318] Trial 2 finished with value: 0.0001647343296279582 and parameters: {'n_estimators': 143, 'learning_rate': 0.039868134503067976, 'num_leaves': 89, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.52019608014234, 'colsample_bytree': 0.5972170143384943, 'reg_alpha': 0.9000224170296878, 'reg_lambda': 5.947302425505974, 'min_split_gain': 0.042934421910372514}. Best is trial 1 with value: 0.00012658657907901652.
[I 2025-06-15 20:38:59,490] Trial 3 finished with value: 0.000183788881303309 and parameters: {'n_estimators': 86, 'learning_rate': 0.1083574024676624, 'num_leaves': 137, 'max_depth': 8, 'min_child_samples': 24, 'subsample': 0.5803260511244699, 'colsample_bytree': 0.7282427031818831, 'reg_alpha': 9.608568923646898, 'reg_lambda': 5.299128830990948, 'min_split_gain': 0.041872196280190656}. Best is trial 1 with value: 0.00012658657907901652.
[I 2025-06-15 20:38:59,493] A new study created in memory with name: no-name-9e965919-cbd1-469b-99ef-220668a68911
[I 2025-06-15 20:38:59,515] Trial 0 finished with value: 0.00060353706296085 and parameters: {'n_estimators': 88, 'learning_rate': 0.059209288776180764, 'num_leaves': 172, 'max_depth': 4, 'min_child_samples': 10, 'subsample': 0.9087072727686127, 'colsample_bytree': 0.5900209827652094, 'reg_alpha': 3.308213815852695, 'reg_lambda': 6.350168991613511, 'min_split_gain': 0.005878625496370371}. Best is trial 0 with value: 0.00060353706296085.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA        1092
0           Price_lag_1        1081
1           Price_lag_2         815
3         Load_DA_lag_0         681
5       WindOn_DA_lag_0         607
16  roll2d_median_Price         495
15       lag168_Load_DA         474
2           Price_lag_7         445
11           NGas_lag_2         418
13            EUA_lag_2         345
10           Coal_lag_2         285
12            Oil_lag_2         253
9                  WD_7          27
7                  WD_1          19
8                  WD_6          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 83.45% done
********************* END NS.. 609 ****************************************************
********************* START NS ... 610 ****************************************************
[I 2025-06-15 20:39:00,971] A new study created in memory with name: no-name-7866e7d5-99e5-4643-abf1-5ca1a44de6d1
[I 2025-06-15 20:39:01,074] Trial 0 finished with value: 0.0026013449567070268 and parameters: {'n_estimators': 70, 'learning_rate': 0.2307281749428678, 'num_leaves': 245, 'max_depth': 6, 'min_child_samples': 7, 'subsample': 0.7357816479194003, 'colsample_bytree': 0.7370277310134083, 'reg_alpha': 9.18892432956878, 'reg_lambda': 4.294039265279067, 'min_split_gain': 0.01474660609417734}. Best is trial 0 with value: 0.0026013449567070268.
[I 2025-06-15 20:39:01,238] Trial 1 finished with value: 0.09507092500739238 and parameters: {'n_estimators': 88, 'learning_rate': 0.013546624685974501, 'num_leaves': 123, 'max_depth': 13, 'min_child_samples': 5, 'subsample': 0.9811317093365213, 'colsample_bytree': 0.7360983972159922, 'reg_alpha': 8.085328753493865, 'reg_lambda': 5.784654636179375, 'min_split_gain': 0.0274543027400135}. Best is trial 0 with value: 0.0026013449567070268.
[Trial 0] New best: 0.00260
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:01,365] Trial 2 finished with value: 0.0041559842710154294 and parameters: {'n_estimators': 70, 'learning_rate': 0.21503512464143018, 'num_leaves': 50, 'max_depth': 8, 'min_child_samples': 4, 'subsample': 0.9687237022676713, 'colsample_bytree': 0.528294430966481, 'reg_alpha': 1.3116537862732425, 'reg_lambda': 8.402304311667779, 'min_split_gain': 0.023935994802039708}. Best is trial 0 with value: 0.0026013449567070268.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:39:01,731] Trial 3 finished with value: 0.0026875319696025523 and parameters: {'n_estimators': 97, 'learning_rate': 0.05264122061205409, 'num_leaves': 149, 'max_depth': 8, 'min_child_samples': 22, 'subsample': 0.8390876900112281, 'colsample_bytree': 0.6323921146731473, 'reg_alpha': 5.470673586760633, 'reg_lambda': 2.358688708407981, 'min_split_gain': 0.037801553335916865}. Best is trial 0 with value: 0.0026013449567070268.
[I 2025-06-15 20:39:01,732] A new study created in memory with name: no-name-90c4e044-eb5b-44e6-9d85-a74102260d82
[I 2025-06-15 20:39:01,752] Trial 0 finished with value: 0.03362353175076855 and parameters: {'n_estimators': 90, 'learning_rate': 0.13049339533811707, 'num_leaves': 152, 'max_depth': 5, 'min_child_samples': 3, 'subsample': 0.9032746225655175, 'colsample_bytree': 0.942482242717587, 'reg_alpha': 3.3695487506258592, 'reg_lambda': 4.993040882055702, 'min_split_gain': 0.04512255886455977}. Best is trial 0 with value: 0.03362353175076855.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         195
1           Price_lag_2         112
14      pct_chg_Load_DA         110
3         Load_DA_lag_0          76
2           Price_lag_7          71
11           NGas_lag_2          61
15       lag168_Load_DA          47
10           Coal_lag_2          45
16  roll2d_median_Price          45
13            EUA_lag_2          34
5       WindOn_DA_lag_0          34
12            Oil_lag_2          26
9                  WD_7          11
8                  WD_6           4
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 83.58% done
********************* END NS.. 610 ****************************************************
********************* START NS ... 611 ****************************************************
[I 2025-06-15 20:39:02,940] A new study created in memory with name: no-name-539be713-858a-400b-980e-c7328e87a2f2
[I 2025-06-15 20:39:03,354] Trial 0 finished with value: 0.02153919205754155 and parameters: {'n_estimators': 181, 'learning_rate': 0.012140437132874236, 'num_leaves': 146, 'max_depth': 6, 'min_child_samples': 13, 'subsample': 0.7573553860534606, 'colsample_bytree': 0.7213683670853709, 'reg_alpha': 2.1205104327083935, 'reg_lambda': 8.971627773245563, 'min_split_gain': 0.041498309568246974}. Best is trial 0 with value: 0.02153919205754155.
[Trial 0] New best: 0.02154
[I 2025-06-15 20:39:03,584] Trial 1 finished with value: 0.0309348905573814 and parameters: {'n_estimators': 100, 'learning_rate': 0.018330421034217177, 'num_leaves': 59, 'max_depth': 7, 'min_child_samples': 6, 'subsample': 0.7489318805528224, 'colsample_bytree': 0.9666343055900524, 'reg_alpha': 6.992496356316899, 'reg_lambda': 0.5717438822374155, 'min_split_gain': 0.04300374939221469}. Best is trial 0 with value: 0.02153919205754155.
[I 2025-06-15 20:39:03,703] Trial 2 finished with value: 0.013609697156192814 and parameters: {'n_estimators': 77, 'learning_rate': 0.1461552449870347, 'num_leaves': 169, 'max_depth': 4, 'min_child_samples': 19, 'subsample': 0.7582422400334043, 'colsample_bytree': 0.5599134649602988, 'reg_alpha': 1.1620217217073825, 'reg_lambda': 0.922464224380507, 'min_split_gain': 0.003104814565084524}. Best is trial 2 with value: 0.013609697156192814.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 2] New best: 0.01361
[I 2025-06-15 20:39:04,064] Trial 3 finished with value: 0.010948826424500312 and parameters: {'n_estimators': 271, 'learning_rate': 0.08150469492011482, 'num_leaves': 224, 'max_depth': 13, 'min_child_samples': 23, 'subsample': 0.9680029020823093, 'colsample_bytree': 0.8439764947023551, 'reg_alpha': 6.137138573377089, 'reg_lambda': 2.9434342463300514, 'min_split_gain': 0.03260879809424878}. Best is trial 3 with value: 0.010948826424500312.
[Trial 3] New best: 0.01095
[I 2025-06-15 20:39:04,389] Trial 4 finished with value: 0.011016241880316848 and parameters: {'n_estimators': 220, 'learning_rate': 0.13181070242046838, 'num_leaves': 125, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.611726969594498, 'colsample_bytree': 0.9685802359230775, 'reg_alpha': 7.366300772725902, 'reg_lambda': 7.103376399596151, 'min_split_gain': 0.048529718674154655}. Best is trial 3 with value: 0.010948826424500312.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:04,750] Trial 5 finished with value: 0.01166156776984006 and parameters: {'n_estimators': 248, 'learning_rate': 0.061913899686418916, 'num_leaves': 218, 'max_depth': 9, 'min_child_samples': 12, 'subsample': 0.9974506628461881, 'colsample_bytree': 0.8246885598650464, 'reg_alpha': 7.2067603615412414, 'reg_lambda': 5.844840763184978, 'min_split_gain': 0.046755755797582066}. Best is trial 3 with value: 0.010948826424500312.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:39:05,014] Trial 6 finished with value: 0.014013948231238844 and parameters: {'n_estimators': 276, 'learning_rate': 0.2190651090610518, 'num_leaves': 213, 'max_depth': 16, 'min_child_samples': 2, 'subsample': 0.570028185951687, 'colsample_bytree': 0.5595400294752995, 'reg_alpha': 5.626919679734263, 'reg_lambda': 2.7058416292120633, 'min_split_gain': 0.036013465438443214}. Best is trial 3 with value: 0.010948826424500312.
[I 2025-06-15 20:39:05,017] A new study created in memory with name: no-name-329d66fd-79ca-4f6e-8de2-138d4b04947a
[I 2025-06-15 20:39:05,049] Trial 0 finished with value: 0.12586176410834096 and parameters: {'n_estimators': 298, 'learning_rate': 0.21626633094785302, 'num_leaves': 145, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.6250539655052594, 'colsample_bytree': 0.915259130802031, 'reg_alpha': 3.9393661455572593, 'reg_lambda': 4.813470674719305, 'min_split_gain': 0.001015523046718525}. Best is trial 0 with value: 0.12586176410834096.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         630
14      pct_chg_Load_DA         386
1           Price_lag_2         361
3         Load_DA_lag_0         203
11           NGas_lag_2         171
2           Price_lag_7         168
16  roll2d_median_Price         150
15       lag168_Load_DA         120
5       WindOn_DA_lag_0          95
10           Coal_lag_2          84
12            Oil_lag_2          45
13            EUA_lag_2          39
9                  WD_7          14
7                  WD_1           6
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          15
16  roll2d_median_Price           5
2           Price_lag_7           0
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 83.72% done
********************* END NS.. 611 ****************************************************
********************* START NS ... 612 ****************************************************
[I 2025-06-15 20:39:06,575] A new study created in memory with name: no-name-0f25e8ac-f4c1-4f39-92b3-2961f0cc44d4
[I 2025-06-15 20:39:07,287] Trial 0 finished with value: 0.01724129307618903 and parameters: {'n_estimators': 248, 'learning_rate': 0.014458019384408053, 'num_leaves': 139, 'max_depth': 16, 'min_child_samples': 4, 'subsample': 0.973990939340512, 'colsample_bytree': 0.6479150606294355, 'reg_alpha': 5.196397700565319, 'reg_lambda': 3.999573089494164, 'min_split_gain': 0.0069070107025283435}. Best is trial 0 with value: 0.01724129307618903.
[Trial 0] New best: 0.01724
[I 2025-06-15 20:39:07,646] Trial 1 finished with value: 0.0141752760301002 and parameters: {'n_estimators': 246, 'learning_rate': 0.09615615605545376, 'num_leaves': 131, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.6873129245598919, 'colsample_bytree': 0.8830621456802495, 'reg_alpha': 2.0113907860506486, 'reg_lambda': 6.951341797823058, 'min_split_gain': 0.033981219968135114}. Best is trial 1 with value: 0.0141752760301002.
[Trial 1] New best: 0.01418
[I 2025-06-15 20:39:08,053] Trial 2 finished with value: 0.015879716599097413 and parameters: {'n_estimators': 98, 'learning_rate': 0.038221409535124554, 'num_leaves': 84, 'max_depth': 9, 'min_child_samples': 16, 'subsample': 0.8603474342745718, 'colsample_bytree': 0.7023532384328139, 'reg_alpha': 3.8961325642451214, 'reg_lambda': 1.352871047766322, 'min_split_gain': 0.020142177864953183}. Best is trial 1 with value: 0.0141752760301002.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:08,339] Trial 3 finished with value: 0.014748557730938411 and parameters: {'n_estimators': 172, 'learning_rate': 0.10637457857023029, 'num_leaves': 209, 'max_depth': 15, 'min_child_samples': 7, 'subsample': 0.7936271635229422, 'colsample_bytree': 0.7852791873037532, 'reg_alpha': 4.399915087382285, 'reg_lambda': 0.7534168133999442, 'min_split_gain': 0.02491398376532568}. Best is trial 1 with value: 0.0141752760301002.
[I 2025-06-15 20:39:08,503] Trial 4 finished with value: 0.014190550171592329 and parameters: {'n_estimators': 113, 'learning_rate': 0.17276067150871613, 'num_leaves': 166, 'max_depth': 16, 'min_child_samples': 18, 'subsample': 0.5998095625977595, 'colsample_bytree': 0.7886122343767561, 'reg_alpha': 8.646498320509151, 'reg_lambda': 7.121675993869886, 'min_split_gain': 0.020544537379390056}. Best is trial 1 with value: 0.0141752760301002.
[I 2025-06-15 20:39:08,506] A new study created in memory with name: no-name-779a57e7-2fcf-407d-935a-3dca49731b97
[I 2025-06-15 20:39:08,537] Trial 0 finished with value: 0.12726719714120482 and parameters: {'n_estimators': 283, 'learning_rate': 0.02899452982078697, 'num_leaves': 90, 'max_depth': 12, 'min_child_samples': 7, 'subsample': 0.6013404703386134, 'colsample_bytree': 0.5848515434042847, 'reg_alpha': 5.394528007532462, 'reg_lambda': 4.212046200192646, 'min_split_gain': 0.032836108566387155}. Best is trial 0 with value: 0.12726719714120482.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         711
1           Price_lag_2         568
14      pct_chg_Load_DA         515
3         Load_DA_lag_0         332
2           Price_lag_7         264
16  roll2d_median_Price         235
15       lag168_Load_DA         229
11           NGas_lag_2         224
5       WindOn_DA_lag_0         196
10           Coal_lag_2         120
12            Oil_lag_2         106
13            EUA_lag_2         101
9                  WD_7          15
7                  WD_1          10
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          56
16  roll2d_median_Price          21
3         Load_DA_lag_0          20
1           Price_lag_2          10
10           Coal_lag_2           1
12            Oil_lag_2           1
2           Price_lag_7           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
11           NGas_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 83.86% done
********************* END NS.. 612 ****************************************************
********************* START NS ... 613 ****************************************************
[I 2025-06-15 20:39:09,733] A new study created in memory with name: no-name-ad238c52-5356-4a1a-b5ac-8b7fa670815c
[I 2025-06-15 20:39:10,231] Trial 0 finished with value: 0.00965855607632477 and parameters: {'n_estimators': 212, 'learning_rate': 0.017890715724634444, 'num_leaves': 97, 'max_depth': 7, 'min_child_samples': 13, 'subsample': 0.9595901133067057, 'colsample_bytree': 0.5014549121489049, 'reg_alpha': 1.6392981685051444, 'reg_lambda': 5.1521101824185696, 'min_split_gain': 0.04134574339218048}. Best is trial 0 with value: 0.00965855607632477.
[I 2025-06-15 20:39:10,348] Trial 1 finished with value: 0.008797057220270325 and parameters: {'n_estimators': 110, 'learning_rate': 0.21075586753632017, 'num_leaves': 231, 'max_depth': 14, 'min_child_samples': 22, 'subsample': 0.8261359035026632, 'colsample_bytree': 0.6825272830329517, 'reg_alpha': 9.192238844067315, 'reg_lambda': 9.368253476435878, 'min_split_gain': 0.041824258628058386}. Best is trial 1 with value: 0.008797057220270325.
[Trial 0] New best: 0.00966
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:10,680] Trial 2 finished with value: 0.008517224409016871 and parameters: {'n_estimators': 75, 'learning_rate': 0.17518946953399242, 'num_leaves': 178, 'max_depth': 11, 'min_child_samples': 5, 'subsample': 0.6401681891912911, 'colsample_bytree': 0.8062266156629221, 'reg_alpha': 1.168613941586214, 'reg_lambda': 7.6722723323449395, 'min_split_gain': 0.03230956409939587}. Best is trial 2 with value: 0.008517224409016871.
[I 2025-06-15 20:39:10,867] Trial 3 finished with value: 0.008319635450863689 and parameters: {'n_estimators': 110, 'learning_rate': 0.2119647588154496, 'num_leaves': 70, 'max_depth': 4, 'min_child_samples': 3, 'subsample': 0.866856918530212, 'colsample_bytree': 0.5919581849055597, 'reg_alpha': 5.3456507482000015, 'reg_lambda': 1.167414866048373, 'min_split_gain': 0.013038419829494836}. Best is trial 3 with value: 0.008319635450863689.
[Trial 2] New best: 0.00852
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:11,258] Trial 4 finished with value: 0.012832455543579811 and parameters: {'n_estimators': 130, 'learning_rate': 0.022230546112098803, 'num_leaves': 177, 'max_depth': 16, 'min_child_samples': 9, 'subsample': 0.5644632547175783, 'colsample_bytree': 0.6789721124212695, 'reg_alpha': 9.119421135939538, 'reg_lambda': 7.809248660633308, 'min_split_gain': 0.012069223724226918}. Best is trial 3 with value: 0.008319635450863689.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:39:11,524] Trial 5 finished with value: 0.008735554874898624 and parameters: {'n_estimators': 208, 'learning_rate': 0.08792585115157346, 'num_leaves': 90, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.7346830256178536, 'colsample_bytree': 0.9187555323611661, 'reg_alpha': 6.596227120603947, 'reg_lambda': 6.8321719572064765, 'min_split_gain': 0.024464507498741614}. Best is trial 3 with value: 0.008319635450863689.
[I 2025-06-15 20:39:11,526] A new study created in memory with name: no-name-14d9e168-cbd1-465a-8b9c-16bb5d7e4884
[I 2025-06-15 20:39:11,619] Trial 0 finished with value: 0.04369978679776056 and parameters: {'n_estimators': 78, 'learning_rate': 0.15051632473590917, 'num_leaves': 157, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.5524632864167727, 'colsample_bytree': 0.5520911822600919, 'reg_alpha': 6.686303673263527, 'reg_lambda': 7.571503422743183, 'min_split_gain': 0.035424416537969176}. Best is trial 0 with value: 0.04369978679776056.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         277
1           Price_lag_2         219
14      pct_chg_Load_DA          91
11           NGas_lag_2          78
3         Load_DA_lag_0          70
2           Price_lag_7          52
16  roll2d_median_Price          51
15       lag168_Load_DA          45
10           Coal_lag_2          42
13            EUA_lag_2          35
5       WindOn_DA_lag_0          28
12            Oil_lag_2          11
8                  WD_6           6
7                  WD_1           5
9                  WD_7           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          11
16  roll2d_median_Price           4
3         Load_DA_lag_0           2
1           Price_lag_2           1
2           Price_lag_7           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 83.99% done
********************* END NS.. 613 ****************************************************
********************* START NS ... 614 ****************************************************
[I 2025-06-15 20:39:12,504] A new study created in memory with name: no-name-58b8db06-f706-492d-b1da-0fd0ffd387df
[I 2025-06-15 20:39:13,110] Trial 0 finished with value: 0.006516569282757116 and parameters: {'n_estimators': 139, 'learning_rate': 0.05218845132342287, 'num_leaves': 153, 'max_depth': 16, 'min_child_samples': 10, 'subsample': 0.7997343923914262, 'colsample_bytree': 0.7618568156556702, 'reg_alpha': 1.722221005733685, 'reg_lambda': 3.829779872066207, 'min_split_gain': 0.014310985849303421}. Best is trial 0 with value: 0.006516569282757116.
[Trial 0] New best: 0.00652
[I 2025-06-15 20:39:13,727] Trial 1 finished with value: 0.03394872330426805 and parameters: {'n_estimators': 165, 'learning_rate': 0.010479742629552657, 'num_leaves': 93, 'max_depth': 9, 'min_child_samples': 7, 'subsample': 0.9651509489502947, 'colsample_bytree': 0.5629794116981669, 'reg_alpha': 2.6487998230516285, 'reg_lambda': 1.6865030990839458, 'min_split_gain': 0.010371190025786355}. Best is trial 0 with value: 0.006516569282757116.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:14,158] Trial 2 finished with value: 0.0062161450632746236 and parameters: {'n_estimators': 288, 'learning_rate': 0.291119475931648, 'num_leaves': 139, 'max_depth': 14, 'min_child_samples': 8, 'subsample': 0.7826573161174746, 'colsample_bytree': 0.8941409191141063, 'reg_alpha': 2.8325934154343537, 'reg_lambda': 5.220774682836181, 'min_split_gain': 0.016789385334927503}. Best is trial 2 with value: 0.0062161450632746236.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:39:14,582] Trial 3 finished with value: 0.006741969361838483 and parameters: {'n_estimators': 231, 'learning_rate': 0.05351492888804246, 'num_leaves': 201, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.9647893562680209, 'colsample_bytree': 0.8275295397773724, 'reg_alpha': 7.903143748244384, 'reg_lambda': 0.024908694825663824, 'min_split_gain': 0.013689336609729586}. Best is trial 2 with value: 0.0062161450632746236.
[I 2025-06-15 20:39:14,584] A new study created in memory with name: no-name-78a8af78-ac38-4b71-9879-6d98e058ae15
[I 2025-06-15 20:39:14,600] Trial 0 finished with value: 0.025193114815871036 and parameters: {'n_estimators': 99, 'learning_rate': 0.14056065168597567, 'num_leaves': 199, 'max_depth': 14, 'min_child_samples': 25, 'subsample': 0.509364101488501, 'colsample_bytree': 0.6533215147043405, 'reg_alpha': 9.360822588631384, 'reg_lambda': 1.910243527646378, 'min_split_gain': 0.035273248691900794}. Best is trial 0 with value: 0.025193114815871036.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         265
1           Price_lag_2         216
14      pct_chg_Load_DA         201
3         Load_DA_lag_0         155
5       WindOn_DA_lag_0         150
2           Price_lag_7         122
16  roll2d_median_Price         109
15       lag168_Load_DA         100
11           NGas_lag_2          82
13            EUA_lag_2          61
10           Coal_lag_2          59
12            Oil_lag_2          58
9                  WD_7           8
7                  WD_1           5
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           8
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 84.13% done
********************* END NS.. 614 ****************************************************
********************* START NS ... 615 ****************************************************
[I 2025-06-15 20:39:15,790] A new study created in memory with name: no-name-ae918286-7757-466c-829c-f1459713e5c6
[I 2025-06-15 20:39:16,052] Trial 0 finished with value: 0.004194547272582684 and parameters: {'n_estimators': 209, 'learning_rate': 0.05694614536264065, 'num_leaves': 240, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.8693592922531905, 'colsample_bytree': 0.7430743900403085, 'reg_alpha': 7.628590300890327, 'reg_lambda': 0.41544746224433826, 'min_split_gain': 0.040088702078953625}. Best is trial 0 with value: 0.004194547272582684.
[Trial 0] New best: 0.00419
[I 2025-06-15 20:39:16,293] Trial 1 finished with value: 0.004642347033053505 and parameters: {'n_estimators': 231, 'learning_rate': 0.06268606442078524, 'num_leaves': 165, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.5085650187808897, 'colsample_bytree': 0.5170515626989365, 'reg_alpha': 9.99214021163656, 'reg_lambda': 8.4743524026997, 'min_split_gain': 0.04855964591143491}. Best is trial 0 with value: 0.004194547272582684.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:17,076] Trial 2 finished with value: 0.003939110523401935 and parameters: {'n_estimators': 285, 'learning_rate': 0.050201868619956215, 'num_leaves': 166, 'max_depth': 10, 'min_child_samples': 19, 'subsample': 0.503803682789957, 'colsample_bytree': 0.8615207980513906, 'reg_alpha': 0.6526411901521023, 'reg_lambda': 5.904403653708588, 'min_split_gain': 0.026403652937471097}. Best is trial 2 with value: 0.003939110523401935.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:39:17,522] Trial 3 finished with value: 0.0038395489437218927 and parameters: {'n_estimators': 293, 'learning_rate': 0.04230920386472208, 'num_leaves': 66, 'max_depth': 13, 'min_child_samples': 9, 'subsample': 0.6494531628898861, 'colsample_bytree': 0.985163316589508, 'reg_alpha': 6.497012457865926, 'reg_lambda': 3.953059511291883, 'min_split_gain': 0.04922920882852115}. Best is trial 3 with value: 0.0038395489437218927.
[I 2025-06-15 20:39:17,524] A new study created in memory with name: no-name-903a1268-fadf-43ce-b5f5-51c14aff2c8c
[I 2025-06-15 20:39:17,554] Trial 0 finished with value: 0.010410847902899137 and parameters: {'n_estimators': 170, 'learning_rate': 0.11866406834630226, 'num_leaves': 209, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.6580697140394074, 'colsample_bytree': 0.5198631664444622, 'reg_alpha': 4.275746782933759, 'reg_lambda': 6.689271664876593, 'min_split_gain': 0.008875625498864155}. Best is trial 0 with value: 0.010410847902899137.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1235
14      pct_chg_Load_DA         717
1           Price_lag_2         519
16  roll2d_median_Price         326
2           Price_lag_7         323
3         Load_DA_lag_0         315
11           NGas_lag_2         300
5       WindOn_DA_lag_0         190
15       lag168_Load_DA         169
10           Coal_lag_2         159
13            EUA_lag_2         113
12            Oil_lag_2         100
9                  WD_7          16
7                  WD_1           8
8                  WD_6           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          28
3         Load_DA_lag_0          10
1           Price_lag_2           5
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 84.27% done
********************* END NS.. 615 ****************************************************
********************* START NS ... 616 ****************************************************
[I 2025-06-15 20:39:18,576] A new study created in memory with name: no-name-820e8f0c-89b0-408b-9f6f-159af2f6e9eb
[I 2025-06-15 20:39:19,234] Trial 0 finished with value: 0.0012018387940254491 and parameters: {'n_estimators': 257, 'learning_rate': 0.02102718428880648, 'num_leaves': 50, 'max_depth': 9, 'min_child_samples': 26, 'subsample': 0.5235018096760938, 'colsample_bytree': 0.8132684252277174, 'reg_alpha': 5.579600653573472, 'reg_lambda': 4.385393293695454, 'min_split_gain': 0.037989557377580314}. Best is trial 0 with value: 0.0012018387940254491.
[Trial 0] New best: 0.00120
[I 2025-06-15 20:39:20,037] Trial 1 finished with value: 0.001501966627293614 and parameters: {'n_estimators': 285, 'learning_rate': 0.04037868234684018, 'num_leaves': 50, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.7275846089632324, 'colsample_bytree': 0.8292986891214158, 'reg_alpha': 5.729811040645696, 'reg_lambda': 1.0502257697730755, 'min_split_gain': 0.007889156977780715}. Best is trial 0 with value: 0.0012018387940254491.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:20,262] Trial 2 finished with value: 0.0009141544917423225 and parameters: {'n_estimators': 123, 'learning_rate': 0.2267689892413899, 'num_leaves': 212, 'max_depth': 9, 'min_child_samples': 6, 'subsample': 0.5338414385918189, 'colsample_bytree': 0.68252086645997, 'reg_alpha': 9.334469752993934, 'reg_lambda': 1.5880442020134145, 'min_split_gain': 0.011853164768882164}. Best is trial 2 with value: 0.0009141544917423225.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:39:21,053] Trial 3 finished with value: 0.0021261384814868635 and parameters: {'n_estimators': 204, 'learning_rate': 0.021565000029435868, 'num_leaves': 45, 'max_depth': 13, 'min_child_samples': 16, 'subsample': 0.8112563969061088, 'colsample_bytree': 0.5929250443779808, 'reg_alpha': 8.410160676999196, 'reg_lambda': 8.024923762055204, 'min_split_gain': 0.03064975939008094}. Best is trial 2 with value: 0.0009141544917423225.
[I 2025-06-15 20:39:21,056] A new study created in memory with name: no-name-7f2e27dc-463b-4af0-8a2d-c5597a3eb9b2
[I 2025-06-15 20:39:21,080] Trial 0 finished with value: 0.01715829799452611 and parameters: {'n_estimators': 82, 'learning_rate': 0.04617747048941663, 'num_leaves': 167, 'max_depth': 11, 'min_child_samples': 19, 'subsample': 0.9548426361810858, 'colsample_bytree': 0.7217052431032231, 'reg_alpha': 3.7803904898764396, 'reg_lambda': 8.331954714918345, 'min_split_gain': 0.0423689472308786}. Best is trial 0 with value: 0.01715829799452611.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         223
1           Price_lag_2         143
14      pct_chg_Load_DA          96
2           Price_lag_7          93
11           NGas_lag_2          86
3         Load_DA_lag_0          85
16  roll2d_median_Price          76
15       lag168_Load_DA          59
5       WindOn_DA_lag_0          58
10           Coal_lag_2          53
13            EUA_lag_2          42
12            Oil_lag_2          30
9                  WD_7          17
8                  WD_6           9
7                  WD_1           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          46
3         Load_DA_lag_0          16
1           Price_lag_2           3
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 84.40% done
********************* END NS.. 616 ****************************************************
********************* START NS ... 617 ****************************************************
[I 2025-06-15 20:39:22,339] A new study created in memory with name: no-name-1a3ff9cf-872b-4a90-9191-3c100eb0e91d
[I 2025-06-15 20:39:22,706] Trial 0 finished with value: 0.0006813769281883505 and parameters: {'n_estimators': 231, 'learning_rate': 0.1848224902389054, 'num_leaves': 167, 'max_depth': 14, 'min_child_samples': 21, 'subsample': 0.5507321688791206, 'colsample_bytree': 0.5805352254765795, 'reg_alpha': 7.527543548703904, 'reg_lambda': 6.077273715021575, 'min_split_gain': 0.03747977015589126}. Best is trial 0 with value: 0.0006813769281883505.
[Trial 0] New best: 0.00068
[I 2025-06-15 20:39:23,178] Trial 1 finished with value: 0.0006581198572558471 and parameters: {'n_estimators': 248, 'learning_rate': 0.08865525351367212, 'num_leaves': 31, 'max_depth': 4, 'min_child_samples': 22, 'subsample': 0.9419982477185579, 'colsample_bytree': 0.8712522086728534, 'reg_alpha': 4.903716751569469, 'reg_lambda': 0.01470252059747823, 'min_split_gain': 0.021723270306676607}. Best is trial 1 with value: 0.0006581198572558471.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:23,592] Trial 2 finished with value: 0.0006525118362289253 and parameters: {'n_estimators': 164, 'learning_rate': 0.04979035552411461, 'num_leaves': 125, 'max_depth': 16, 'min_child_samples': 11, 'subsample': 0.7507111699362021, 'colsample_bytree': 0.7446628955046655, 'reg_alpha': 9.308952409933383, 'reg_lambda': 3.4981078109420114, 'min_split_gain': 0.04960107991025589}. Best is trial 2 with value: 0.0006525118362289253.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:39:23,826] Trial 3 finished with value: 0.0007729848214011502 and parameters: {'n_estimators': 204, 'learning_rate': 0.2300880999620631, 'num_leaves': 170, 'max_depth': 14, 'min_child_samples': 12, 'subsample': 0.6680641350263508, 'colsample_bytree': 0.7776579579462788, 'reg_alpha': 9.335733060070442, 'reg_lambda': 9.95669199907136, 'min_split_gain': 0.03500240881961871}. Best is trial 2 with value: 0.0006525118362289253.
[I 2025-06-15 20:39:23,830] A new study created in memory with name: no-name-b8958ddd-9408-41f0-bd9d-1a41dbdc8500
[I 2025-06-15 20:39:23,872] Trial 0 finished with value: 0.04777579594703444 and parameters: {'n_estimators': 54, 'learning_rate': 0.019312372093699497, 'num_leaves': 38, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.9237932783299231, 'colsample_bytree': 0.6331749409352978, 'reg_alpha': 5.944766648444887, 'reg_lambda': 4.265060393036247, 'min_split_gain': 0.040572151836306546}. Best is trial 0 with value: 0.04777579594703444.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         817
1           Price_lag_2         527
14      pct_chg_Load_DA         452
3         Load_DA_lag_0         229
2           Price_lag_7         206
11           NGas_lag_2         197
16  roll2d_median_Price         188
15       lag168_Load_DA         152
5       WindOn_DA_lag_0         114
10           Coal_lag_2         108
13            EUA_lag_2          58
12            Oil_lag_2          41
9                  WD_7          26
8                  WD_6          17
7                  WD_1           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
3         Load_DA_lag_0          31
0           Price_lag_1          21
15       lag168_Load_DA           4
12            Oil_lag_2           2
11           NGas_lag_2           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 84.54% done
********************* END NS.. 617 ****************************************************
********************* START NS ... 618 ****************************************************
[I 2025-06-15 20:39:25,537] A new study created in memory with name: no-name-b0238999-e5fa-40f7-866f-8cb4dfb55070
[I 2025-06-15 20:39:25,923] Trial 0 finished with value: 0.00036684079184328436 and parameters: {'n_estimators': 165, 'learning_rate': 0.04899015915636934, 'num_leaves': 34, 'max_depth': 12, 'min_child_samples': 21, 'subsample': 0.9895328383526419, 'colsample_bytree': 0.7361583975791356, 'reg_alpha': 4.56774294565954, 'reg_lambda': 2.152744802089485, 'min_split_gain': 0.008854461497104716}. Best is trial 0 with value: 0.00036684079184328436.
[Trial 0] New best: 0.00037
[I 2025-06-15 20:39:26,647] Trial 1 finished with value: 0.0015036938651381976 and parameters: {'n_estimators': 211, 'learning_rate': 0.05106160500645278, 'num_leaves': 215, 'max_depth': 16, 'min_child_samples': 1, 'subsample': 0.5213054603730067, 'colsample_bytree': 0.6244306526437067, 'reg_alpha': 2.3268023715582595, 'reg_lambda': 0.7038464952433654, 'min_split_gain': 0.02897366744672716}. Best is trial 0 with value: 0.00036684079184328436.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:27,163] Trial 2 finished with value: 0.0004981996395197917 and parameters: {'n_estimators': 264, 'learning_rate': 0.03176785341677158, 'num_leaves': 216, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.915148212259068, 'colsample_bytree': 0.718872529872842, 'reg_alpha': 8.297271085829049, 'reg_lambda': 2.265783670157724, 'min_split_gain': 0.020790772179978847}. Best is trial 0 with value: 0.00036684079184328436.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:39:29,611] Trial 3 finished with value: 0.0005169788535531646 and parameters: {'n_estimators': 282, 'learning_rate': 0.0282334897501211, 'num_leaves': 60, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.8193100005947386, 'colsample_bytree': 0.6661843129895697, 'reg_alpha': 1.7999566276034806, 'reg_lambda': 1.3320622186944653, 'min_split_gain': 0.0489577633354982}. Best is trial 0 with value: 0.00036684079184328436.
[I 2025-06-15 20:39:29,614] A new study created in memory with name: no-name-49dab9fb-efcd-4525-ba3f-56f7f56db16a
[I 2025-06-15 20:39:29,635] Trial 0 finished with value: 0.034749920753992435 and parameters: {'n_estimators': 58, 'learning_rate': 0.03866635588542094, 'num_leaves': 67, 'max_depth': 7, 'min_child_samples': 24, 'subsample': 0.9129435343591168, 'colsample_bytree': 0.7452671277570915, 'reg_alpha': 7.966709084715985, 'reg_lambda': 1.8388749674528049, 'min_split_gain': 0.02967853495177494}. Best is trial 0 with value: 0.034749920753992435.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1176
1           Price_lag_2         902
14      pct_chg_Load_DA         673
3         Load_DA_lag_0         437
5       WindOn_DA_lag_0         311
2           Price_lag_7         307
13            EUA_lag_2         288
11           NGas_lag_2         278
15       lag168_Load_DA         270
16  roll2d_median_Price         255
10           Coal_lag_2         220
12            Oil_lag_2         195
9                  WD_7          27
7                  WD_1          19
8                  WD_6          16
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           8
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 84.68% done
********************* END NS.. 618 ****************************************************
********************* START NS ... 619 ****************************************************
[I 2025-06-15 20:39:31,958] A new study created in memory with name: no-name-7fa393c5-52ab-4440-90bf-7037b9338b33
[I 2025-06-15 20:39:32,328] Trial 0 finished with value: 0.00548467926656034 and parameters: {'n_estimators': 66, 'learning_rate': 0.042632536318515844, 'num_leaves': 252, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.8628168669058087, 'colsample_bytree': 0.554285036088042, 'reg_alpha': 7.658996944749282, 'reg_lambda': 0.15892124520206274, 'min_split_gain': 0.0028272883094792815}. Best is trial 0 with value: 0.00548467926656034.
[Trial 0] New best: 0.00548
[I 2025-06-15 20:39:33,021] Trial 1 finished with value: 0.0007719554027547489 and parameters: {'n_estimators': 293, 'learning_rate': 0.11270507741855791, 'num_leaves': 153, 'max_depth': 7, 'min_child_samples': 3, 'subsample': 0.7037512847855045, 'colsample_bytree': 0.5695112982144288, 'reg_alpha': 7.131526577986705, 'reg_lambda': 0.05125489809101991, 'min_split_gain': 0.036219136805276324}. Best is trial 1 with value: 0.0007719554027547489.
[Trial 1] New best: 0.00077
[I 2025-06-15 20:39:33,503] Trial 2 finished with value: 0.0002597985908523711 and parameters: {'n_estimators': 189, 'learning_rate': 0.08116269458404882, 'num_leaves': 77, 'max_depth': 4, 'min_child_samples': 28, 'subsample': 0.5529637464132195, 'colsample_bytree': 0.5022693459220637, 'reg_alpha': 1.3825732458613127, 'reg_lambda': 9.579310347878792, 'min_split_gain': 0.03434909572639138}. Best is trial 2 with value: 0.0002597985908523711.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:34,155] Trial 3 finished with value: 0.0007792139213889576 and parameters: {'n_estimators': 188, 'learning_rate': 0.025274410726494514, 'num_leaves': 222, 'max_depth': 4, 'min_child_samples': 14, 'subsample': 0.9912540978245592, 'colsample_bytree': 0.8318437543876349, 'reg_alpha': 9.031070729754173, 'reg_lambda': 2.828201644480235, 'min_split_gain': 0.009482298223837622}. Best is trial 2 with value: 0.0002597985908523711.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:39:34,780] Trial 4 finished with value: 0.0005824363872568998 and parameters: {'n_estimators': 185, 'learning_rate': 0.031461397863680955, 'num_leaves': 242, 'max_depth': 4, 'min_child_samples': 23, 'subsample': 0.5499275117855735, 'colsample_bytree': 0.8697999626356707, 'reg_alpha': 9.071824708883112, 'reg_lambda': 0.897187297541957, 'min_split_gain': 0.020810898624574387}. Best is trial 2 with value: 0.0002597985908523711.
[I 2025-06-15 20:39:34,782] A new study created in memory with name: no-name-10255b20-67a1-4485-83d4-f32e53763d47
[I 2025-06-15 20:39:34,802] Trial 0 finished with value: 0.02013716464154102 and parameters: {'n_estimators': 50, 'learning_rate': 0.011600803576732816, 'num_leaves': 81, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.5443502487721491, 'colsample_bytree': 0.7582965583610122, 'reg_alpha': 9.938389407686936, 'reg_lambda': 9.587198379102729, 'min_split_gain': 0.04677728367663281}. Best is trial 0 with value: 0.02013716464154102.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         535
1           Price_lag_2         404
14      pct_chg_Load_DA         201
3         Load_DA_lag_0         179
16  roll2d_median_Price         168
2           Price_lag_7         158
11           NGas_lag_2         132
15       lag168_Load_DA         101
13            EUA_lag_2          96
5       WindOn_DA_lag_0          75
12            Oil_lag_2          66
10           Coal_lag_2          59
7                  WD_1          15
9                  WD_7          11
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 84.82% done
********************* END NS.. 619 ****************************************************
********************* START NS ... 620 ****************************************************
[I 2025-06-15 20:39:36,353] A new study created in memory with name: no-name-ccb0d1fc-f802-4e46-a5a5-aaa2fa2a1e12
[I 2025-06-15 20:39:36,718] Trial 0 finished with value: 0.06376372056997795 and parameters: {'n_estimators': 80, 'learning_rate': 0.018832806433872425, 'num_leaves': 188, 'max_depth': 4, 'min_child_samples': 8, 'subsample': 0.748657770217793, 'colsample_bytree': 0.7060205460701899, 'reg_alpha': 7.6506816094872345, 'reg_lambda': 8.296297955058717, 'min_split_gain': 0.013917849867496635}. Best is trial 0 with value: 0.06376372056997795.
[Trial 0] New best: 0.06376
[I 2025-06-15 20:39:37,408] Trial 1 finished with value: 0.0029238489718605085 and parameters: {'n_estimators': 264, 'learning_rate': 0.02088185315848194, 'num_leaves': 68, 'max_depth': 4, 'min_child_samples': 21, 'subsample': 0.8332272626094859, 'colsample_bytree': 0.7949226923748144, 'reg_alpha': 3.688734889758156, 'reg_lambda': 7.0145472645769145, 'min_split_gain': 0.02218489807536618}. Best is trial 1 with value: 0.0029238489718605085.
[Trial 1] New best: 0.00292
[I 2025-06-15 20:39:38,274] Trial 2 finished with value: 0.0028400153892740782 and parameters: {'n_estimators': 74, 'learning_rate': 0.07142452652271603, 'num_leaves': 126, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.7949365891989264, 'colsample_bytree': 0.7566454183886495, 'reg_alpha': 2.3890685618997165, 'reg_lambda': 8.725056536375737, 'min_split_gain': 0.011349216668579083}. Best is trial 2 with value: 0.0028400153892740782.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:38,815] Trial 3 finished with value: 0.0028270665817034628 and parameters: {'n_estimators': 207, 'learning_rate': 0.11496099052084228, 'num_leaves': 59, 'max_depth': 10, 'min_child_samples': 11, 'subsample': 0.9759189119256229, 'colsample_bytree': 0.7725651096793786, 'reg_alpha': 5.776765222027656, 'reg_lambda': 9.08908324763488, 'min_split_gain': 0.024065581907859804}. Best is trial 3 with value: 0.0028270665817034628.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:39:39,131] Trial 4 finished with value: 0.002928673410520563 and parameters: {'n_estimators': 296, 'learning_rate': 0.20961146156822463, 'num_leaves': 175, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.9840301783694081, 'colsample_bytree': 0.7966333391146243, 'reg_alpha': 9.650122615111185, 'reg_lambda': 5.181880541286454, 'min_split_gain': 0.031376381399760865}. Best is trial 3 with value: 0.0028270665817034628.
[I 2025-06-15 20:39:39,133] A new study created in memory with name: no-name-fdf82ea9-ce6b-422e-926f-82a2f9e4878d
[I 2025-06-15 20:39:39,159] Trial 0 finished with value: 0.012615087566375485 and parameters: {'n_estimators': 65, 'learning_rate': 0.027059373078094588, 'num_leaves': 108, 'max_depth': 12, 'min_child_samples': 2, 'subsample': 0.7066338958846556, 'colsample_bytree': 0.8969002035507216, 'reg_alpha': 5.871838526834604, 'reg_lambda': 4.841978706393383, 'min_split_gain': 0.04595743985236333}. Best is trial 0 with value: 0.012615087566375485.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         446
1           Price_lag_2         349
14      pct_chg_Load_DA         294
3         Load_DA_lag_0         198
2           Price_lag_7         171
16  roll2d_median_Price         151
11           NGas_lag_2         140
5       WindOn_DA_lag_0         120
15       lag168_Load_DA         110
10           Coal_lag_2          85
13            EUA_lag_2          73
12            Oil_lag_2          47
9                  WD_7          12
7                  WD_1           6
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 84.95% done
********************* END NS.. 620 ****************************************************
********************* START NS ... 621 ****************************************************
[I 2025-06-15 20:39:40,930] A new study created in memory with name: no-name-fc85dc89-6887-40c0-8dae-70096054fcd2
[I 2025-06-15 20:39:41,711] Trial 0 finished with value: 0.11765453351758577 and parameters: {'n_estimators': 93, 'learning_rate': 0.012036892202064322, 'num_leaves': 42, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.9350405298079496, 'colsample_bytree': 0.7845883519272909, 'reg_alpha': 1.1709558183063062, 'reg_lambda': 7.414142676568115, 'min_split_gain': 0.02920796630169721}. Best is trial 0 with value: 0.11765453351758577.
[Trial 0] New best: 0.11765
[I 2025-06-15 20:39:43,235] Trial 1 finished with value: 0.0034836444323042767 and parameters: {'n_estimators': 290, 'learning_rate': 0.06201397206478116, 'num_leaves': 118, 'max_depth': 8, 'min_child_samples': 16, 'subsample': 0.6941631788689384, 'colsample_bytree': 0.7073447753411866, 'reg_alpha': 0.07776127410919265, 'reg_lambda': 9.95652655831518, 'min_split_gain': 0.039372695308302966}. Best is trial 1 with value: 0.0034836444323042767.
[Trial 1] New best: 0.00348
[I 2025-06-15 20:39:44,553] Trial 2 finished with value: 0.0034403472713608334 and parameters: {'n_estimators': 71, 'learning_rate': 0.09678407519881951, 'num_leaves': 178, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.9421163343402874, 'colsample_bytree': 0.9148297384671598, 'reg_alpha': 0.6367953871861365, 'reg_lambda': 1.614452832335589, 'min_split_gain': 0.03556484271090636}. Best is trial 2 with value: 0.0034403472713608334.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:45,171] Trial 3 finished with value: 0.00366338682657744 and parameters: {'n_estimators': 298, 'learning_rate': 0.23092834823362365, 'num_leaves': 201, 'max_depth': 16, 'min_child_samples': 21, 'subsample': 0.7509552595684855, 'colsample_bytree': 0.9484078746174704, 'reg_alpha': 2.106115493006675, 'reg_lambda': 8.978789518213823, 'min_split_gain': 0.010829517693814584}. Best is trial 2 with value: 0.0034403472713608334.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:39:46,120] Trial 4 finished with value: 0.003550516138791692 and parameters: {'n_estimators': 91, 'learning_rate': 0.06808313947419883, 'num_leaves': 172, 'max_depth': 11, 'min_child_samples': 20, 'subsample': 0.8245612296226816, 'colsample_bytree': 0.6290596085995227, 'reg_alpha': 2.76638641139501, 'reg_lambda': 8.727184187274231, 'min_split_gain': 0.02736190638209053}. Best is trial 2 with value: 0.0034403472713608334.
[I 2025-06-15 20:39:46,122] A new study created in memory with name: no-name-891d7dc2-d8d6-4ce3-8703-57d97f506126
[I 2025-06-15 20:39:46,184] Trial 0 finished with value: 0.009490081029037045 and parameters: {'n_estimators': 186, 'learning_rate': 0.05403813243414322, 'num_leaves': 199, 'max_depth': 11, 'min_child_samples': 2, 'subsample': 0.8785149706892439, 'colsample_bytree': 0.6273481338331348, 'reg_alpha': 0.6073510594778919, 'reg_lambda': 9.111466253448636, 'min_split_gain': 0.021805559864321374}. Best is trial 0 with value: 0.009490081029037045.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         930
1           Price_lag_2         914
14      pct_chg_Load_DA         870
3         Load_DA_lag_0         573
16  roll2d_median_Price         513
2           Price_lag_7         488
5       WindOn_DA_lag_0         391
11           NGas_lag_2         355
15       lag168_Load_DA         353
10           Coal_lag_2         228
13            EUA_lag_2         168
12            Oil_lag_2         146
9                  WD_7          17
7                  WD_1          10
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          28
1           Price_lag_2           8
15       lag168_Load_DA           1
11           NGas_lag_2           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 85.09% done
********************* END NS.. 621 ****************************************************
********************* START NS ... 622 ****************************************************
[I 2025-06-15 20:39:48,461] A new study created in memory with name: no-name-b2936ad5-4524-41f4-bada-ae0f497d3421
[I 2025-06-15 20:39:49,242] Trial 0 finished with value: 0.021941039947428492 and parameters: {'n_estimators': 102, 'learning_rate': 0.023585477651164914, 'num_leaves': 161, 'max_depth': 14, 'min_child_samples': 13, 'subsample': 0.816000959950244, 'colsample_bytree': 0.6788492947783684, 'reg_alpha': 4.6227876979687945, 'reg_lambda': 3.0104337349554244, 'min_split_gain': 0.0342856959058316}. Best is trial 0 with value: 0.021941039947428492.
[Trial 0] New best: 0.02194
[I 2025-06-15 20:39:50,426] Trial 1 finished with value: 0.15884030699749765 and parameters: {'n_estimators': 68, 'learning_rate': 0.014288389720358317, 'num_leaves': 226, 'max_depth': 14, 'min_child_samples': 18, 'subsample': 0.9127147684370582, 'colsample_bytree': 0.6016674957354231, 'reg_alpha': 0.38973771452817996, 'reg_lambda': 5.710046553809259, 'min_split_gain': 0.03465825792518038}. Best is trial 0 with value: 0.021941039947428492.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:50,805] Trial 2 finished with value: 0.011751202476592263 and parameters: {'n_estimators': 202, 'learning_rate': 0.1904711439837753, 'num_leaves': 230, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.687170000606666, 'colsample_bytree': 0.6244978286114201, 'reg_alpha': 9.900727859772083, 'reg_lambda': 2.0427077944901817, 'min_split_gain': 0.01764652280188823}. Best is trial 2 with value: 0.011751202476592263.
[Trial 2] New best: 0.01175
[I 2025-06-15 20:39:51,220] Trial 3 finished with value: 0.01039486046070804 and parameters: {'n_estimators': 249, 'learning_rate': 0.15908186944335664, 'num_leaves': 36, 'max_depth': 5, 'min_child_samples': 24, 'subsample': 0.8777405813977084, 'colsample_bytree': 0.8340678978172766, 'reg_alpha': 4.187962377248069, 'reg_lambda': 7.884603038407077, 'min_split_gain': 0.011004506183135088}. Best is trial 3 with value: 0.01039486046070804.
[Trial 3] New best: 0.01039
[I 2025-06-15 20:39:52,538] Trial 4 finished with value: 0.011415938378512647 and parameters: {'n_estimators': 248, 'learning_rate': 0.06232643143238827, 'num_leaves': 255, 'max_depth': 15, 'min_child_samples': 16, 'subsample': 0.8870401472180229, 'colsample_bytree': 0.9478408841795709, 'reg_alpha': 1.795935296676059, 'reg_lambda': 5.239216832142115, 'min_split_gain': 0.03490685734967174}. Best is trial 3 with value: 0.01039486046070804.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:53,607] Trial 5 finished with value: 0.0373964889766342 and parameters: {'n_estimators': 109, 'learning_rate': 0.017479843938411278, 'num_leaves': 120, 'max_depth': 9, 'min_child_samples': 19, 'subsample': 0.7262275475429065, 'colsample_bytree': 0.7360977840056921, 'reg_alpha': 7.393188172109564, 'reg_lambda': 4.633758835368511, 'min_split_gain': 0.029941671567383622}. Best is trial 3 with value: 0.01039486046070804.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:39:55,305] Trial 6 finished with value: 0.011785679051754044 and parameters: {'n_estimators': 120, 'learning_rate': 0.12249298026082396, 'num_leaves': 142, 'max_depth': 12, 'min_child_samples': 13, 'subsample': 0.893300219340542, 'colsample_bytree': 0.6954721258792118, 'reg_alpha': 1.7057474114775661, 'reg_lambda': 3.594575861242044, 'min_split_gain': 0.0035402755135889976}. Best is trial 3 with value: 0.01039486046070804.
[I 2025-06-15 20:39:55,307] A new study created in memory with name: no-name-ad88ba47-a540-44c0-8a70-f57e4d34670e
[I 2025-06-15 20:39:55,331] Trial 0 finished with value: 0.030973010295006844 and parameters: {'n_estimators': 57, 'learning_rate': 0.11885609778334254, 'num_leaves': 201, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.7922983205791367, 'colsample_bytree': 0.60569622143489, 'reg_alpha': 4.403991128583074, 'reg_lambda': 7.590363672031658, 'min_split_gain': 0.010756674877989642}. Best is trial 0 with value: 0.030973010295006844.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         372
1           Price_lag_2         197
14      pct_chg_Load_DA         196
3         Load_DA_lag_0         147
15       lag168_Load_DA         112
2           Price_lag_7          92
11           NGas_lag_2          90
13            EUA_lag_2          89
16  roll2d_median_Price          86
5       WindOn_DA_lag_0          82
10           Coal_lag_2          72
12            Oil_lag_2          21
7                  WD_1           7
9                  WD_7           4
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 85.23% done
********************* END NS.. 622 ****************************************************
********************* START NS ... 623 ****************************************************
[I 2025-06-15 20:39:57,006] A new study created in memory with name: no-name-20563e59-03e1-4bcd-ab7e-2d2d2b89ba30
[I 2025-06-15 20:39:57,262] Trial 0 finished with value: 0.010485959091290376 and parameters: {'n_estimators': 68, 'learning_rate': 0.18184414975064633, 'num_leaves': 92, 'max_depth': 6, 'min_child_samples': 7, 'subsample': 0.5177153211008731, 'colsample_bytree': 0.7974450434957161, 'reg_alpha': 8.827929056499654, 'reg_lambda': 5.957102409259752, 'min_split_gain': 0.008988557134045078}. Best is trial 0 with value: 0.010485959091290376.
[Trial 0] New best: 0.01049
[I 2025-06-15 20:39:57,493] Trial 1 finished with value: 0.040261464931491804 and parameters: {'n_estimators': 59, 'learning_rate': 0.03104050835024592, 'num_leaves': 62, 'max_depth': 5, 'min_child_samples': 10, 'subsample': 0.7587799240995197, 'colsample_bytree': 0.5687574376611224, 'reg_alpha': 6.1421910352516385, 'reg_lambda': 2.243771732537514, 'min_split_gain': 0.03517040995069442}. Best is trial 0 with value: 0.010485959091290376.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:39:57,714] Trial 2 finished with value: 0.2173429006934735 and parameters: {'n_estimators': 76, 'learning_rate': 0.010461001226329635, 'num_leaves': 165, 'max_depth': 4, 'min_child_samples': 30, 'subsample': 0.8382734491876633, 'colsample_bytree': 0.8206908296686044, 'reg_alpha': 3.2506552301690905, 'reg_lambda': 8.524320913414304, 'min_split_gain': 0.04963671718187371}. Best is trial 0 with value: 0.010485959091290376.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:39:58,554] Trial 3 finished with value: 0.012636207157854676 and parameters: {'n_estimators': 257, 'learning_rate': 0.02595486743509948, 'num_leaves': 232, 'max_depth': 4, 'min_child_samples': 30, 'subsample': 0.6349326923288597, 'colsample_bytree': 0.750308254855791, 'reg_alpha': 9.110725710968106, 'reg_lambda': 3.6542113721053493, 'min_split_gain': 0.010772430152063306}. Best is trial 0 with value: 0.010485959091290376.
[I 2025-06-15 20:39:58,557] A new study created in memory with name: no-name-8cb7f02f-f9b4-4563-82b8-c900c9d6bdf8
[I 2025-06-15 20:39:58,587] Trial 0 finished with value: 0.02995609084771705 and parameters: {'n_estimators': 101, 'learning_rate': 0.2558998072287577, 'num_leaves': 87, 'max_depth': 5, 'min_child_samples': 11, 'subsample': 0.8718797305213601, 'colsample_bytree': 0.6793642487333988, 'reg_alpha': 3.1776990884492906, 'reg_lambda': 0.6596246718831822, 'min_split_gain': 0.045814044010921345}. Best is trial 0 with value: 0.02995609084771705.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         279
1           Price_lag_2         196
14      pct_chg_Load_DA         169
3         Load_DA_lag_0         115
11           NGas_lag_2          86
2           Price_lag_7          81
15       lag168_Load_DA          75
16  roll2d_median_Price          65
5       WindOn_DA_lag_0          64
10           Coal_lag_2          62
13            EUA_lag_2          52
12            Oil_lag_2          46
9                  WD_7          11
7                  WD_1           7
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 85.36% done
********************* END NS.. 623 ****************************************************
********************* START NS ... 624 ****************************************************
[I 2025-06-15 20:40:00,251] A new study created in memory with name: no-name-8bec45f1-15ce-4ac8-8d70-16bdb19d926d
[I 2025-06-15 20:40:01,625] Trial 0 finished with value: 0.0009646056787577873 and parameters: {'n_estimators': 192, 'learning_rate': 0.02241569215362867, 'num_leaves': 216, 'max_depth': 15, 'min_child_samples': 19, 'subsample': 0.5077659718448925, 'colsample_bytree': 0.6256444153231656, 'reg_alpha': 8.285957414821203, 'reg_lambda': 6.416306268080579, 'min_split_gain': 0.009168758271780908}. Best is trial 0 with value: 0.0009646056787577873.
[Trial 0] New best: 0.00096
[I 2025-06-15 20:40:02,286] Trial 1 finished with value: 0.000498933627652629 and parameters: {'n_estimators': 91, 'learning_rate': 0.20808927398690288, 'num_leaves': 147, 'max_depth': 10, 'min_child_samples': 28, 'subsample': 0.9955366053560537, 'colsample_bytree': 0.8091506575126527, 'reg_alpha': 1.3559760357591144, 'reg_lambda': 8.355216382122727, 'min_split_gain': 0.017830873948718406}. Best is trial 1 with value: 0.000498933627652629.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:40:03,016] Trial 2 finished with value: 0.0004592729997417793 and parameters: {'n_estimators': 58, 'learning_rate': 0.10653881748558314, 'num_leaves': 89, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.5095263811591834, 'colsample_bytree': 0.8790630936116413, 'reg_alpha': 4.516317237915045, 'reg_lambda': 2.9423447960290194, 'min_split_gain': 0.004761903868002882}. Best is trial 2 with value: 0.0004592729997417793.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:40:03,467] Trial 3 finished with value: 0.000435355604676864 and parameters: {'n_estimators': 238, 'learning_rate': 0.12299597468256378, 'num_leaves': 117, 'max_depth': 12, 'min_child_samples': 25, 'subsample': 0.948713661143221, 'colsample_bytree': 0.9338029337715796, 'reg_alpha': 6.975765501719278, 'reg_lambda': 4.413950568371029, 'min_split_gain': 0.03239771272521579}. Best is trial 3 with value: 0.000435355604676864.
[I 2025-06-15 20:40:03,469] A new study created in memory with name: no-name-6646b942-aa3c-425e-a731-49b97afe4e9b
[I 2025-06-15 20:40:03,516] Trial 0 finished with value: 0.0010723590658722225 and parameters: {'n_estimators': 173, 'learning_rate': 0.023028678449791787, 'num_leaves': 186, 'max_depth': 6, 'min_child_samples': 23, 'subsample': 0.9966170317214317, 'colsample_bytree': 0.7116703212775373, 'reg_alpha': 3.22248326403244, 'reg_lambda': 8.850237738871042, 'min_split_gain': 0.01740249292438529}. Best is trial 0 with value: 0.0010723590658722225.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         399
14      pct_chg_Load_DA         244
1           Price_lag_2         169
3         Load_DA_lag_0         125
11           NGas_lag_2         107
16  roll2d_median_Price         106
2           Price_lag_7         101
10           Coal_lag_2          69
5       WindOn_DA_lag_0          56
15       lag168_Load_DA          51
13            EUA_lag_2          40
12            Oil_lag_2          29
7                  WD_1           4
9                  WD_7           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          27
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 85.50% done
********************* END NS.. 624 ****************************************************
********************* START NS ... 625 ****************************************************
[I 2025-06-15 20:40:05,361] A new study created in memory with name: no-name-af1b80e5-07f7-4224-b08e-6ab9c714aee1
[I 2025-06-15 20:40:07,306] Trial 0 finished with value: 0.018897396907303663 and parameters: {'n_estimators': 262, 'learning_rate': 0.028125659166093483, 'num_leaves': 132, 'max_depth': 15, 'min_child_samples': 4, 'subsample': 0.6773838989029194, 'colsample_bytree': 0.5179417726124216, 'reg_alpha': 6.739253947694122, 'reg_lambda': 7.854313807986494, 'min_split_gain': 0.04599710641031519}. Best is trial 0 with value: 0.018897396907303663.
[Trial 0] New best: 0.01890
[I 2025-06-15 20:40:08,031] Trial 1 finished with value: 0.19112786446339466 and parameters: {'n_estimators': 77, 'learning_rate': 0.01008492921779994, 'num_leaves': 155, 'max_depth': 9, 'min_child_samples': 6, 'subsample': 0.9677458473346481, 'colsample_bytree': 0.9925607090212841, 'reg_alpha': 5.557984249554028, 'reg_lambda': 1.813697822308502, 'min_split_gain': 0.03945168988184164}. Best is trial 0 with value: 0.018897396907303663.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:40:08,314] Trial 2 finished with value: 0.015372548064161632 and parameters: {'n_estimators': 245, 'learning_rate': 0.2404869136044316, 'num_leaves': 234, 'max_depth': 10, 'min_child_samples': 30, 'subsample': 0.6544221446540874, 'colsample_bytree': 0.9009042517058445, 'reg_alpha': 8.147161325345428, 'reg_lambda': 8.338655239862096, 'min_split_gain': 0.0389858391278434}. Best is trial 2 with value: 0.015372548064161632.
[Trial 2] New best: 0.01537
[I 2025-06-15 20:40:08,978] Trial 3 finished with value: 0.018899297736176577 and parameters: {'n_estimators': 140, 'learning_rate': 0.04353941723831683, 'num_leaves': 59, 'max_depth': 4, 'min_child_samples': 21, 'subsample': 0.7185642401528705, 'colsample_bytree': 0.8277840147101871, 'reg_alpha': 2.4205476366515066, 'reg_lambda': 3.060438099688221, 'min_split_gain': 0.04181653753299024}. Best is trial 2 with value: 0.015372548064161632.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:40:09,669] Trial 4 finished with value: 0.015138527559561745 and parameters: {'n_estimators': 125, 'learning_rate': 0.10533152977801137, 'num_leaves': 194, 'max_depth': 6, 'min_child_samples': 12, 'subsample': 0.9938527064698646, 'colsample_bytree': 0.843668675097641, 'reg_alpha': 3.0081008492989545, 'reg_lambda': 3.162801076715318, 'min_split_gain': 0.0071632889966766035}. Best is trial 4 with value: 0.015138527559561745.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:40:10,855] Trial 5 finished with value: 0.016190574745420204 and parameters: {'n_estimators': 292, 'learning_rate': 0.05617114601381702, 'num_leaves': 183, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.9563483579010484, 'colsample_bytree': 0.7354124667115396, 'reg_alpha': 3.542932291375286, 'reg_lambda': 3.5689635817759866, 'min_split_gain': 0.043761814797534465}. Best is trial 4 with value: 0.015138527559561745.
[I 2025-06-15 20:40:10,858] A new study created in memory with name: no-name-73320d9b-aa25-4e90-834d-42e6cbf397c8
[I 2025-06-15 20:40:10,911] Trial 0 finished with value: 0.07646912624576253 and parameters: {'n_estimators': 178, 'learning_rate': 0.05115412213458874, 'num_leaves': 43, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.7994018118336701, 'colsample_bytree': 0.5780986653594768, 'reg_alpha': 9.22448820089866, 'reg_lambda': 7.132083074514296, 'min_split_gain': 0.018251716645721595}. Best is trial 0 with value: 0.07646912624576253.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         722
14      pct_chg_Load_DA         512
1           Price_lag_2         507
3         Load_DA_lag_0         368
5       WindOn_DA_lag_0         300
15       lag168_Load_DA         277
16  roll2d_median_Price         276
2           Price_lag_7         248
13            EUA_lag_2         216
11           NGas_lag_2         215
10           Coal_lag_2         155
12            Oil_lag_2         127
9                  WD_7          24
7                  WD_1          15
8                  WD_6          10
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 85.64% done
********************* END NS.. 625 ****************************************************
********************* START NS ... 626 ****************************************************
[I 2025-06-15 20:40:13,145] A new study created in memory with name: no-name-5e9749cf-1b4b-4a34-a021-1957453b3bc0
[I 2025-06-15 20:40:14,250] Trial 0 finished with value: 0.02049453391667475 and parameters: {'n_estimators': 178, 'learning_rate': 0.0523047997323868, 'num_leaves': 192, 'max_depth': 15, 'min_child_samples': 20, 'subsample': 0.6312446623421584, 'colsample_bytree': 0.9864699054175112, 'reg_alpha': 1.784299216493449, 'reg_lambda': 3.8158504575552255, 'min_split_gain': 0.047219247557177935}. Best is trial 0 with value: 0.02049453391667475.
[Trial 0] New best: 0.02049
[I 2025-06-15 20:40:14,833] Trial 1 finished with value: 0.02194699881270939 and parameters: {'n_estimators': 185, 'learning_rate': 0.19932757367595705, 'num_leaves': 119, 'max_depth': 16, 'min_child_samples': 24, 'subsample': 0.9007159786451743, 'colsample_bytree': 0.6190186839170044, 'reg_alpha': 1.7058057556296835, 'reg_lambda': 2.2418616352541343, 'min_split_gain': 0.024534521344637086}. Best is trial 0 with value: 0.02049453391667475.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:40:15,625] Trial 2 finished with value: 0.01790410855346136 and parameters: {'n_estimators': 186, 'learning_rate': 0.29651197525067013, 'num_leaves': 94, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.9439406677728572, 'colsample_bytree': 0.5117440865406693, 'reg_alpha': 0.29436791706964116, 'reg_lambda': 5.05302405446902, 'min_split_gain': 0.010746094151235081}. Best is trial 2 with value: 0.01790410855346136.
[Trial 2] New best: 0.01790
[I 2025-06-15 20:40:17,398] Trial 3 finished with value: 0.02464661329092443 and parameters: {'n_estimators': 300, 'learning_rate': 0.0311145040620068, 'num_leaves': 197, 'max_depth': 6, 'min_child_samples': 1, 'subsample': 0.6597334021133447, 'colsample_bytree': 0.5176714634042732, 'reg_alpha': 4.994002742703671, 'reg_lambda': 1.4686072990430332, 'min_split_gain': 0.029946064222258675}. Best is trial 2 with value: 0.01790410855346136.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:40:17,763] Trial 4 finished with value: 0.04783165819404645 and parameters: {'n_estimators': 56, 'learning_rate': 0.030105303484569284, 'num_leaves': 65, 'max_depth': 4, 'min_child_samples': 22, 'subsample': 0.6317523943152814, 'colsample_bytree': 0.9620074928307721, 'reg_alpha': 1.0969923105276969, 'reg_lambda': 7.324291517302589, 'min_split_gain': 0.016948321828928144}. Best is trial 2 with value: 0.01790410855346136.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:40:18,490] Trial 5 finished with value: 0.023082697826553714 and parameters: {'n_estimators': 235, 'learning_rate': 0.07919758294042753, 'num_leaves': 107, 'max_depth': 9, 'min_child_samples': 12, 'subsample': 0.8163389580289591, 'colsample_bytree': 0.7674273245693348, 'reg_alpha': 6.424280341444458, 'reg_lambda': 5.510692738248282, 'min_split_gain': 0.026844531459542492}. Best is trial 2 with value: 0.01790410855346136.
[I 2025-06-15 20:40:18,492] A new study created in memory with name: no-name-8f1ebc7f-df4c-47fd-a2eb-95f5d89e8dea
[I 2025-06-15 20:40:18,555] Trial 0 finished with value: 0.10543860773417513 and parameters: {'n_estimators': 298, 'learning_rate': 0.036876580743899746, 'num_leaves': 105, 'max_depth': 8, 'min_child_samples': 25, 'subsample': 0.6613733702158895, 'colsample_bytree': 0.7880806204943146, 'reg_alpha': 7.651607483834387, 'reg_lambda': 6.928839204844914, 'min_split_gain': 0.019735243581536246}. Best is trial 0 with value: 0.10543860773417513.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         455
1           Price_lag_2         396
3         Load_DA_lag_0         346
16  roll2d_median_Price         320
2           Price_lag_7         282
5       WindOn_DA_lag_0         281
14      pct_chg_Load_DA         281
12            Oil_lag_2         253
15       lag168_Load_DA         219
13            EUA_lag_2         159
11           NGas_lag_2         155
10           Coal_lag_2         153
9                  WD_7          12
7                  WD_1           6
8                  WD_6           6
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          34
3         Load_DA_lag_0           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 85.77% done
********************* END NS.. 626 ****************************************************
********************* START NS ... 627 ****************************************************
[I 2025-06-15 20:40:20,431] A new study created in memory with name: no-name-42e4b940-b368-4ea0-aece-34d6026a4a46
[I 2025-06-15 20:40:20,853] Trial 0 finished with value: 0.01848787274230489 and parameters: {'n_estimators': 76, 'learning_rate': 0.03369605473726557, 'num_leaves': 166, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.9778483437053347, 'colsample_bytree': 0.8957329477903876, 'reg_alpha': 9.452020217914244, 'reg_lambda': 6.103875815655993, 'min_split_gain': 0.02073497254435318}. Best is trial 0 with value: 0.01848787274230489.
[Trial 0] New best: 0.01849
[I 2025-06-15 20:40:21,549] Trial 1 finished with value: 0.10188206900801818 and parameters: {'n_estimators': 61, 'learning_rate': 0.017276473835465628, 'num_leaves': 205, 'max_depth': 13, 'min_child_samples': 18, 'subsample': 0.5505577111823197, 'colsample_bytree': 0.5282494314196093, 'reg_alpha': 2.0799813079567087, 'reg_lambda': 7.186630506559349, 'min_split_gain': 0.04156795003106442}. Best is trial 0 with value: 0.01848787274230489.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:40:22,454] Trial 2 finished with value: 0.01073500327811131 and parameters: {'n_estimators': 294, 'learning_rate': 0.12834003322036353, 'num_leaves': 208, 'max_depth': 14, 'min_child_samples': 19, 'subsample': 0.6345323530782145, 'colsample_bytree': 0.5269468865006899, 'reg_alpha': 5.384158079191348, 'reg_lambda': 2.855698495680301, 'min_split_gain': 0.019276980910415034}. Best is trial 2 with value: 0.01073500327811131.
[Trial 2] New best: 0.01074
[I 2025-06-15 20:40:23,881] Trial 3 finished with value: 0.011966444959561418 and parameters: {'n_estimators': 297, 'learning_rate': 0.05108067346117985, 'num_leaves': 126, 'max_depth': 10, 'min_child_samples': 1, 'subsample': 0.5317620248247025, 'colsample_bytree': 0.5522021316454607, 'reg_alpha': 6.481573740486734, 'reg_lambda': 3.3453394738541977, 'min_split_gain': 0.031092381250774338}. Best is trial 2 with value: 0.01073500327811131.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:40:24,687] Trial 4 finished with value: 0.015598452325893233 and parameters: {'n_estimators': 233, 'learning_rate': 0.013397054938141495, 'num_leaves': 143, 'max_depth': 4, 'min_child_samples': 22, 'subsample': 0.5876329134166769, 'colsample_bytree': 0.9645106280521288, 'reg_alpha': 9.371552220645501, 'reg_lambda': 9.301616360215657, 'min_split_gain': 0.020875454681256102}. Best is trial 2 with value: 0.01073500327811131.
[I 2025-06-15 20:40:24,864] Trial 5 finished with value: 0.01272298373845059 and parameters: {'n_estimators': 115, 'learning_rate': 0.2188441911438134, 'num_leaves': 154, 'max_depth': 4, 'min_child_samples': 15, 'subsample': 0.7315465269891883, 'colsample_bytree': 0.7811421962101313, 'reg_alpha': 9.004966741794462, 'reg_lambda': 6.282126674046152, 'min_split_gain': 0.029565101712857523}. Best is trial 2 with value: 0.01073500327811131.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:40:24,866] A new study created in memory with name: no-name-42b30776-bf68-429e-afa7-11bf1311b88b
[I 2025-06-15 20:40:25,034] Trial 0 finished with value: 0.0447970206821503 and parameters: {'n_estimators': 196, 'learning_rate': 0.02131325313148793, 'num_leaves': 168, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.6755896696383777, 'colsample_bytree': 0.5479593585009437, 'reg_alpha': 6.160600441622808, 'reg_lambda': 8.236627828481186, 'min_split_gain': 0.031187189561368026}. Best is trial 0 with value: 0.0447970206821503.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         585
1           Price_lag_2         536
3         Load_DA_lag_0         223
14      pct_chg_Load_DA         202
2           Price_lag_7         183
16  roll2d_median_Price         162
15       lag168_Load_DA         146
11           NGas_lag_2         145
5       WindOn_DA_lag_0         130
12            Oil_lag_2         116
13            EUA_lag_2          95
10           Coal_lag_2          79
8                  WD_6          18
9                  WD_7          17
7                  WD_1          15
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          69
3         Load_DA_lag_0          29
12            Oil_lag_2           6
1           Price_lag_2           3
15       lag168_Load_DA           3
13            EUA_lag_2           2
16  roll2d_median_Price           1
7                  WD_1           0
6      WindOff_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 85.91% done
********************* END NS.. 627 ****************************************************
********************* START NS ... 628 ****************************************************
[I 2025-06-15 20:40:27,011] A new study created in memory with name: no-name-9b3889f2-b493-45db-825f-66e987a2eb9b
[I 2025-06-15 20:40:27,703] Trial 0 finished with value: 0.005712362107751146 and parameters: {'n_estimators': 246, 'learning_rate': 0.07713498125180131, 'num_leaves': 243, 'max_depth': 7, 'min_child_samples': 17, 'subsample': 0.7294501902395325, 'colsample_bytree': 0.9163767753210745, 'reg_alpha': 2.9016432367203002, 'reg_lambda': 0.5366075596642972, 'min_split_gain': 0.036512663593400434}. Best is trial 0 with value: 0.005712362107751146.
[Trial 0] New best: 0.00571
[I 2025-06-15 20:40:28,889] Trial 1 finished with value: 0.010311525143520477 and parameters: {'n_estimators': 223, 'learning_rate': 0.013048339159308734, 'num_leaves': 200, 'max_depth': 6, 'min_child_samples': 30, 'subsample': 0.9194318874946545, 'colsample_bytree': 0.8153638511813729, 'reg_alpha': 6.5289323934532115, 'reg_lambda': 4.352233125434296, 'min_split_gain': 0.004124480957868571}. Best is trial 0 with value: 0.005712362107751146.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:40:31,390] Trial 2 finished with value: 0.007902103689269996 and parameters: {'n_estimators': 240, 'learning_rate': 0.013974669104136637, 'num_leaves': 42, 'max_depth': 10, 'min_child_samples': 11, 'subsample': 0.9451010128143778, 'colsample_bytree': 0.8850366758406529, 'reg_alpha': 1.7043049736614069, 'reg_lambda': 6.908609625886807, 'min_split_gain': 0.03920993696074397}. Best is trial 0 with value: 0.005712362107751146.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:40:31,695] Trial 3 finished with value: 0.007610156464884281 and parameters: {'n_estimators': 81, 'learning_rate': 0.05410073099515724, 'num_leaves': 247, 'max_depth': 4, 'min_child_samples': 18, 'subsample': 0.9444799016888596, 'colsample_bytree': 0.6316108628358537, 'reg_alpha': 9.219133816155013, 'reg_lambda': 3.7390154408966136, 'min_split_gain': 0.035821030999570375}. Best is trial 0 with value: 0.005712362107751146.
[I 2025-06-15 20:40:31,697] A new study created in memory with name: no-name-e3b747c7-cbd6-4058-8411-4859a1827b9e
[I 2025-06-15 20:40:31,759] Trial 0 finished with value: 0.011780264905419646 and parameters: {'n_estimators': 143, 'learning_rate': 0.04279874278787598, 'num_leaves': 129, 'max_depth': 10, 'min_child_samples': 28, 'subsample': 0.5785359044877876, 'colsample_bytree': 0.8970973846345661, 'reg_alpha': 1.5064337458031651, 'reg_lambda': 5.423432555748326, 'min_split_gain': 0.014641972823287375}. Best is trial 0 with value: 0.011780264905419646.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         820
14      pct_chg_Load_DA         569
1           Price_lag_2         486
3         Load_DA_lag_0         329
11           NGas_lag_2         238
2           Price_lag_7         223
16  roll2d_median_Price         222
15       lag168_Load_DA         178
5       WindOn_DA_lag_0         134
10           Coal_lag_2         104
13            EUA_lag_2          89
12            Oil_lag_2          87
9                  WD_7          10
7                  WD_1           5
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          55
3         Load_DA_lag_0          40
14      pct_chg_Load_DA          26
12            Oil_lag_2          13
15       lag168_Load_DA          10
9                  WD_7           0
13            EUA_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 86.05% done
********************* END NS.. 628 ****************************************************
********************* START NS ... 629 ****************************************************
[I 2025-06-15 20:40:34,025] A new study created in memory with name: no-name-3ed21db1-96f6-4fa1-a2af-bd2e7d709dfc
[I 2025-06-15 20:40:34,263] Trial 0 finished with value: 0.0007531568768857491 and parameters: {'n_estimators': 289, 'learning_rate': 0.27282055319914056, 'num_leaves': 237, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.8075558116039234, 'colsample_bytree': 0.766559068140318, 'reg_alpha': 9.958457033565155, 'reg_lambda': 6.089181678813302, 'min_split_gain': 0.012691650051040016}. Best is trial 0 with value: 0.0007531568768857491.
[Trial 0] New best: 0.00075
[I 2025-06-15 20:40:34,945] Trial 1 finished with value: 0.0020825973986351508 and parameters: {'n_estimators': 105, 'learning_rate': 0.03214831343199623, 'num_leaves': 149, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.8867403148510952, 'colsample_bytree': 0.6957045937542745, 'reg_alpha': 8.694313535216114, 'reg_lambda': 1.2300307272168143, 'min_split_gain': 0.03071382202001089}. Best is trial 0 with value: 0.0007531568768857491.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:40:42,186] Trial 2 finished with value: 0.002123387454858318 and parameters: {'n_estimators': 292, 'learning_rate': 0.030817037210557167, 'num_leaves': 185, 'max_depth': 15, 'min_child_samples': 21, 'subsample': 0.9972655114835832, 'colsample_bytree': 0.8572375961745577, 'reg_alpha': 0.07677761173874642, 'reg_lambda': 5.924734708812117, 'min_split_gain': 0.0023152204658106914}. Best is trial 0 with value: 0.0007531568768857491.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:40:43,277] Trial 3 finished with value: 0.002014974032606636 and parameters: {'n_estimators': 78, 'learning_rate': 0.04383703668625962, 'num_leaves': 119, 'max_depth': 14, 'min_child_samples': 9, 'subsample': 0.5804307177645007, 'colsample_bytree': 0.8680116136995301, 'reg_alpha': 1.4180919690379112, 'reg_lambda': 6.61717858423061, 'min_split_gain': 0.03909058801461548}. Best is trial 0 with value: 0.0007531568768857491.
[I 2025-06-15 20:40:43,278] A new study created in memory with name: no-name-12bb546e-16fd-4378-8e4b-42c18f75218f
[I 2025-06-15 20:40:43,314] Trial 0 finished with value: 0.0037649736245488675 and parameters: {'n_estimators': 117, 'learning_rate': 0.20137372015953559, 'num_leaves': 213, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.6425347532055752, 'colsample_bytree': 0.778072691452542, 'reg_alpha': 6.281489862122488, 'reg_lambda': 4.3578107176465295, 'min_split_gain': 0.032534717995087115}. Best is trial 0 with value: 0.0037649736245488675.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         165
14      pct_chg_Load_DA          95
1           Price_lag_2          90
3         Load_DA_lag_0          74
11           NGas_lag_2          58
2           Price_lag_7          50
16  roll2d_median_Price          46
10           Coal_lag_2          35
15       lag168_Load_DA          35
13            EUA_lag_2          26
5       WindOn_DA_lag_0          23
12            Oil_lag_2          21
7                  WD_1           3
9                  WD_7           3
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           5
3         Load_DA_lag_0           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 86.18% done
********************* END NS.. 629 ****************************************************
********************* START NS ... 630 ****************************************************
[I 2025-06-15 20:40:45,077] A new study created in memory with name: no-name-4dfab15b-fbe2-4d7d-a4fd-22040d9308bd
[I 2025-06-15 20:40:45,370] Trial 0 finished with value: 0.000627297947666175 and parameters: {'n_estimators': 272, 'learning_rate': 0.06467704422553347, 'num_leaves': 242, 'max_depth': 4, 'min_child_samples': 12, 'subsample': 0.8390863255713332, 'colsample_bytree': 0.7485381770139745, 'reg_alpha': 9.20792332529944, 'reg_lambda': 4.058841584804494, 'min_split_gain': 0.0338567362169823}. Best is trial 0 with value: 0.000627297947666175.
[Trial 0] New best: 0.00063
[I 2025-06-15 20:40:45,610] Trial 1 finished with value: 0.0007535915093005576 and parameters: {'n_estimators': 152, 'learning_rate': 0.2612828127434647, 'num_leaves': 236, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.5132615864518328, 'colsample_bytree': 0.633658042274865, 'reg_alpha': 4.089778810645211, 'reg_lambda': 0.08262222849139467, 'min_split_gain': 0.01207793377379125}. Best is trial 0 with value: 0.000627297947666175.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:40:46,636] Trial 2 finished with value: 0.0006334828722076783 and parameters: {'n_estimators': 252, 'learning_rate': 0.05388185536254362, 'num_leaves': 198, 'max_depth': 5, 'min_child_samples': 29, 'subsample': 0.5630962588483615, 'colsample_bytree': 0.8365440903020676, 'reg_alpha': 4.374279830645173, 'reg_lambda': 8.986522608563112, 'min_split_gain': 0.0071300441016461215}. Best is trial 0 with value: 0.000627297947666175.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:40:47,278] Trial 3 finished with value: 0.0024813168411561884 and parameters: {'n_estimators': 107, 'learning_rate': 0.030659192826208655, 'num_leaves': 31, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.8232644650451104, 'colsample_bytree': 0.9612024844887009, 'reg_alpha': 2.8980656151366713, 'reg_lambda': 8.224677579222943, 'min_split_gain': 0.014176960684330553}. Best is trial 0 with value: 0.000627297947666175.
[I 2025-06-15 20:40:47,283] A new study created in memory with name: no-name-58d9a5b5-412a-4d2b-8357-950684fdfdbc
[I 2025-06-15 20:40:47,352] Trial 0 finished with value: 0.0051959649187734444 and parameters: {'n_estimators': 276, 'learning_rate': 0.2802981602559115, 'num_leaves': 91, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.7948533786118497, 'colsample_bytree': 0.8681112696754337, 'reg_alpha': 1.7950641136749423, 'reg_lambda': 4.565005982412976, 'min_split_gain': 0.04906765142139213}. Best is trial 0 with value: 0.0051959649187734444.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         548
14      pct_chg_Load_DA         261
1           Price_lag_2         168
3         Load_DA_lag_0         163
11           NGas_lag_2          99
2           Price_lag_7          84
15       lag168_Load_DA          77
16  roll2d_median_Price          72
10           Coal_lag_2          59
5       WindOn_DA_lag_0          39
13            EUA_lag_2          30
12            Oil_lag_2          17
9                  WD_7          11
7                  WD_1           8
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           7
3         Load_DA_lag_0           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 86.32% done
********************* END NS.. 630 ****************************************************
********************* START NS ... 631 ****************************************************
[I 2025-06-15 20:40:49,403] A new study created in memory with name: no-name-d0e66e80-0fe6-4996-8390-afea39674c73
[I 2025-06-15 20:40:49,825] Trial 0 finished with value: 0.000544861303794385 and parameters: {'n_estimators': 188, 'learning_rate': 0.1442062512693742, 'num_leaves': 123, 'max_depth': 13, 'min_child_samples': 15, 'subsample': 0.768790188218704, 'colsample_bytree': 0.6687382691566173, 'reg_alpha': 2.5667686010163546, 'reg_lambda': 2.429498495297814, 'min_split_gain': 0.03344317767774949}. Best is trial 0 with value: 0.000544861303794385.
[Trial 0] New best: 0.00054
[I 2025-06-15 20:40:51,259] Trial 1 finished with value: 0.0036602452184108674 and parameters: {'n_estimators': 170, 'learning_rate': 0.01662467312618187, 'num_leaves': 129, 'max_depth': 7, 'min_child_samples': 23, 'subsample': 0.739502519621762, 'colsample_bytree': 0.5396679955703485, 'reg_alpha': 4.646037964030838, 'reg_lambda': 9.863959996799476, 'min_split_gain': 0.02574709463081697}. Best is trial 0 with value: 0.000544861303794385.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:40:53,110] Trial 2 finished with value: 0.0005849322867814605 and parameters: {'n_estimators': 144, 'learning_rate': 0.045813737741525806, 'num_leaves': 99, 'max_depth': 8, 'min_child_samples': 4, 'subsample': 0.9740544917404255, 'colsample_bytree': 0.7152062981834016, 'reg_alpha': 3.339712162877225, 'reg_lambda': 2.0567157156257867, 'min_split_gain': 0.016536036448406923}. Best is trial 0 with value: 0.000544861303794385.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:40:59,230] Trial 3 finished with value: 0.0005580090609601342 and parameters: {'n_estimators': 285, 'learning_rate': 0.02368847868975273, 'num_leaves': 248, 'max_depth': 12, 'min_child_samples': 8, 'subsample': 0.5223415271636744, 'colsample_bytree': 0.5816956784951988, 'reg_alpha': 0.6389198497125048, 'reg_lambda': 8.917362461819238, 'min_split_gain': 0.030460443403261584}. Best is trial 0 with value: 0.000544861303794385.
[I 2025-06-15 20:40:59,233] A new study created in memory with name: no-name-760bcba9-12cb-40ca-8c81-34f12946734e
[I 2025-06-15 20:40:59,303] Trial 0 finished with value: 0.015437782590564025 and parameters: {'n_estimators': 241, 'learning_rate': 0.024737233913387058, 'num_leaves': 167, 'max_depth': 10, 'min_child_samples': 23, 'subsample': 0.6572039216294601, 'colsample_bytree': 0.9799390765570819, 'reg_alpha': 9.056634750909755, 'reg_lambda': 9.353399021905304, 'min_split_gain': 0.021122163329216844}. Best is trial 0 with value: 0.015437782590564025.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         431
1           Price_lag_2         311
14      pct_chg_Load_DA         259
3         Load_DA_lag_0         215
2           Price_lag_7         148
15       lag168_Load_DA         146
16  roll2d_median_Price         140
11           NGas_lag_2         127
10           Coal_lag_2          98
5       WindOn_DA_lag_0          81
12            Oil_lag_2          65
13            EUA_lag_2          61
9                  WD_7           7
7                  WD_1           6
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 86.46% done
********************* END NS.. 631 ****************************************************
********************* START NS ... 632 ****************************************************
[I 2025-06-15 20:41:01,194] A new study created in memory with name: no-name-29d421fd-7973-4bfb-bca4-eb6070f095ec
[I 2025-06-15 20:41:03,337] Trial 0 finished with value: 0.0017910230844396084 and parameters: {'n_estimators': 272, 'learning_rate': 0.05107575504996491, 'num_leaves': 76, 'max_depth': 16, 'min_child_samples': 9, 'subsample': 0.5615236552740452, 'colsample_bytree': 0.6721478486631705, 'reg_alpha': 0.7971422377272641, 'reg_lambda': 2.1023784675254866, 'min_split_gain': 0.04843409330583379}. Best is trial 0 with value: 0.0017910230844396084.
[Trial 0] New best: 0.00179
[I 2025-06-15 20:41:04,774] Trial 1 finished with value: 0.008205212852446131 and parameters: {'n_estimators': 132, 'learning_rate': 0.018455798937370767, 'num_leaves': 128, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.649908407718182, 'colsample_bytree': 0.8821126021833773, 'reg_alpha': 1.1088602750246268, 'reg_lambda': 2.3486578782942678, 'min_split_gain': 0.022435106578190967}. Best is trial 0 with value: 0.0017910230844396084.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:41:06,017] Trial 2 finished with value: 0.0018290332235775142 and parameters: {'n_estimators': 194, 'learning_rate': 0.030321902940517315, 'num_leaves': 45, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.7895315603782912, 'colsample_bytree': 0.9434033607179538, 'reg_alpha': 4.784314838900889, 'reg_lambda': 3.636910590931428, 'min_split_gain': 0.03249468527448276}. Best is trial 0 with value: 0.0017910230844396084.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:41:06,253] Trial 3 finished with value: 0.0018267546336788217 and parameters: {'n_estimators': 81, 'learning_rate': 0.2182505844203253, 'num_leaves': 79, 'max_depth': 16, 'min_child_samples': 23, 'subsample': 0.6106880387388541, 'colsample_bytree': 0.9931304809288394, 'reg_alpha': 6.7778175489535, 'reg_lambda': 0.986186067069531, 'min_split_gain': 0.04099520847614273}. Best is trial 0 with value: 0.0017910230844396084.
[I 2025-06-15 20:41:06,254] A new study created in memory with name: no-name-d298072d-cfdc-43c9-9d11-beff7a6a0010
[I 2025-06-15 20:41:06,412] Trial 0 finished with value: 0.003454466192736986 and parameters: {'n_estimators': 240, 'learning_rate': 0.12867616973744428, 'num_leaves': 154, 'max_depth': 11, 'min_child_samples': 2, 'subsample': 0.7041356718144591, 'colsample_bytree': 0.7774357701209027, 'reg_alpha': 5.882958237953341, 'reg_lambda': 1.218161189274396, 'min_split_gain': 0.03682157622146473}. Best is trial 0 with value: 0.003454466192736986.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1309
1           Price_lag_2        1260
14      pct_chg_Load_DA        1061
3         Load_DA_lag_0         827
15       lag168_Load_DA         568
16  roll2d_median_Price         562
5       WindOn_DA_lag_0         552
2           Price_lag_7         546
11           NGas_lag_2         493
10           Coal_lag_2         314
13            EUA_lag_2         274
12            Oil_lag_2         231
9                  WD_7          37
7                  WD_1          14
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 86.59% done
********************* END NS.. 632 ****************************************************
********************* START NS ... 633 ****************************************************
[I 2025-06-15 20:41:09,176] A new study created in memory with name: no-name-41ebba04-edb5-4ed1-aa3c-b73b5f7fae94
[I 2025-06-15 20:41:09,659] Trial 0 finished with value: 0.0019780233048846143 and parameters: {'n_estimators': 257, 'learning_rate': 0.13125082846743, 'num_leaves': 118, 'max_depth': 12, 'min_child_samples': 28, 'subsample': 0.9709917660643592, 'colsample_bytree': 0.7476353457923892, 'reg_alpha': 9.637689478712371, 'reg_lambda': 4.001363433062794, 'min_split_gain': 0.034228745137049744}. Best is trial 0 with value: 0.0019780233048846143.
[Trial 0] New best: 0.00198
[I 2025-06-15 20:41:10,355] Trial 1 finished with value: 0.0019972748291123745 and parameters: {'n_estimators': 263, 'learning_rate': 0.23666880859170208, 'num_leaves': 146, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.9049438443019099, 'colsample_bytree': 0.8974239298559656, 'reg_alpha': 9.250634418056782, 'reg_lambda': 3.21789046909446, 'min_split_gain': 0.00691987354631311}. Best is trial 0 with value: 0.0019780233048846143.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:41:11,507] Trial 2 finished with value: 0.001992965965390925 and parameters: {'n_estimators': 175, 'learning_rate': 0.041389456135676496, 'num_leaves': 97, 'max_depth': 14, 'min_child_samples': 6, 'subsample': 0.5760513683028077, 'colsample_bytree': 0.8349854977114499, 'reg_alpha': 6.402378383071249, 'reg_lambda': 7.231442415573307, 'min_split_gain': 0.03444951935641165}. Best is trial 0 with value: 0.0019780233048846143.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:41:12,350] Trial 3 finished with value: 0.0019452499107137887 and parameters: {'n_estimators': 205, 'learning_rate': 0.10345539241412156, 'num_leaves': 71, 'max_depth': 6, 'min_child_samples': 1, 'subsample': 0.6884407062863533, 'colsample_bytree': 0.5532368509518949, 'reg_alpha': 5.394385874073117, 'reg_lambda': 9.363550016518342, 'min_split_gain': 0.00835037922327877}. Best is trial 3 with value: 0.0019452499107137887.
[I 2025-06-15 20:41:12,352] A new study created in memory with name: no-name-3c3aecd2-0776-4f05-9bdb-858eceb44217
[I 2025-06-15 20:41:12,387] Trial 0 finished with value: 0.0037273088731206334 and parameters: {'n_estimators': 128, 'learning_rate': 0.018636980103865414, 'num_leaves': 136, 'max_depth': 9, 'min_child_samples': 8, 'subsample': 0.8305272991687781, 'colsample_bytree': 0.7507972108912286, 'reg_alpha': 7.658691014503614, 'reg_lambda': 7.326693916468279, 'min_split_gain': 0.02377346438409927}. Best is trial 0 with value: 0.0037273088731206334.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         772
0           Price_lag_1         771
14      pct_chg_Load_DA         337
2           Price_lag_7         306
3         Load_DA_lag_0         271
11           NGas_lag_2         258
16  roll2d_median_Price         230
5       WindOn_DA_lag_0         212
13            EUA_lag_2         210
15       lag168_Load_DA         200
12            Oil_lag_2         193
10           Coal_lag_2         147
9                  WD_7          26
8                  WD_6          21
7                  WD_1          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 86.73% done
********************* END NS.. 633 ****************************************************
********************* START NS ... 634 ****************************************************
[I 2025-06-15 20:41:14,359] A new study created in memory with name: no-name-8b9a49c8-b7d5-4706-99cd-cf20088a51c8
[I 2025-06-15 20:41:16,831] Trial 0 finished with value: 0.00042226223626613287 and parameters: {'n_estimators': 284, 'learning_rate': 0.039966012418936096, 'num_leaves': 190, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.5716513432239317, 'colsample_bytree': 0.6671763290325978, 'reg_alpha': 8.490210518875308, 'reg_lambda': 5.467903720297251, 'min_split_gain': 0.0066056654668127605}. Best is trial 0 with value: 0.00042226223626613287.
[Trial 0] New best: 0.00042
[I 2025-06-15 20:41:17,525] Trial 1 finished with value: 0.05776500893619719 and parameters: {'n_estimators': 107, 'learning_rate': 0.01252459110258423, 'num_leaves': 234, 'max_depth': 5, 'min_child_samples': 19, 'subsample': 0.5301180076855687, 'colsample_bytree': 0.5304824047684527, 'reg_alpha': 4.740393598169263, 'reg_lambda': 9.922623973723756, 'min_split_gain': 0.030338625729021365}. Best is trial 0 with value: 0.00042226223626613287.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:41:17,765] Trial 2 finished with value: 0.0004164592488583384 and parameters: {'n_estimators': 151, 'learning_rate': 0.27971689767639923, 'num_leaves': 160, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.9442357641144543, 'colsample_bytree': 0.9046537612397048, 'reg_alpha': 8.98866021734525, 'reg_lambda': 6.147273138287078, 'min_split_gain': 0.038425019174896824}. Best is trial 2 with value: 0.0004164592488583384.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:41:18,549] Trial 3 finished with value: 0.12755312686121684 and parameters: {'n_estimators': 87, 'learning_rate': 0.01085617614678583, 'num_leaves': 76, 'max_depth': 14, 'min_child_samples': 2, 'subsample': 0.5831878705556769, 'colsample_bytree': 0.7425349846221747, 'reg_alpha': 7.291914239592544, 'reg_lambda': 8.456215391339677, 'min_split_gain': 0.04059539888882449}. Best is trial 2 with value: 0.0004164592488583384.
[I 2025-06-15 20:41:18,553] A new study created in memory with name: no-name-cbe1e89e-d16d-42c8-8699-9495aa402e00
[I 2025-06-15 20:41:18,604] Trial 0 finished with value: 0.0018293216891949301 and parameters: {'n_estimators': 216, 'learning_rate': 0.014494141466360213, 'num_leaves': 132, 'max_depth': 7, 'min_child_samples': 27, 'subsample': 0.7279416460941013, 'colsample_bytree': 0.7712354402694976, 'reg_alpha': 9.770854516124496, 'reg_lambda': 7.5391538248735115, 'min_split_gain': 0.017389828815612202}. Best is trial 0 with value: 0.0018293216891949301.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         146
14      pct_chg_Load_DA          92
1           Price_lag_2          60
2           Price_lag_7          40
3         Load_DA_lag_0          39
11           NGas_lag_2          36
16  roll2d_median_Price          32
5       WindOn_DA_lag_0          23
10           Coal_lag_2          19
15       lag168_Load_DA          19
13            EUA_lag_2          13
12            Oil_lag_2          10
9                  WD_7           3
8                  WD_6           1
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 86.87% done
********************* END NS.. 634 ****************************************************
********************* START NS ... 635 ****************************************************
[I 2025-06-15 20:41:20,288] A new study created in memory with name: no-name-4bb9dd6a-c90c-4283-b170-786cfa6a7de1
[I 2025-06-15 20:41:22,691] Trial 0 finished with value: 0.0007967999537238035 and parameters: {'n_estimators': 288, 'learning_rate': 0.011831599335870486, 'num_leaves': 81, 'max_depth': 12, 'min_child_samples': 24, 'subsample': 0.574813502134517, 'colsample_bytree': 0.5678227824704556, 'reg_alpha': 7.8624874131302604, 'reg_lambda': 1.914982127166095, 'min_split_gain': 0.023138701595164554}. Best is trial 0 with value: 0.0007967999537238035.
[Trial 0] New best: 0.00080
[I 2025-06-15 20:41:23,440] Trial 1 finished with value: 0.00018704039289719747 and parameters: {'n_estimators': 285, 'learning_rate': 0.06872434749234693, 'num_leaves': 207, 'max_depth': 16, 'min_child_samples': 15, 'subsample': 0.6821485639541436, 'colsample_bytree': 0.9929424641038285, 'reg_alpha': 7.697702250730258, 'reg_lambda': 0.2670848663663494, 'min_split_gain': 0.033050992718473504}. Best is trial 1 with value: 0.00018704039289719747.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:41:27,896] Trial 2 finished with value: 0.000351235375100735 and parameters: {'n_estimators': 217, 'learning_rate': 0.030769134304726872, 'num_leaves': 218, 'max_depth': 16, 'min_child_samples': 25, 'subsample': 0.9423901055294546, 'colsample_bytree': 0.6165449164790631, 'reg_alpha': 2.709872670054402, 'reg_lambda': 3.9933409089714345, 'min_split_gain': 0.0008448324101295091}. Best is trial 1 with value: 0.00018704039289719747.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:41:28,841] Trial 3 finished with value: 0.00020416345291179286 and parameters: {'n_estimators': 168, 'learning_rate': 0.05090398981115206, 'num_leaves': 165, 'max_depth': 14, 'min_child_samples': 10, 'subsample': 0.7164965918392852, 'colsample_bytree': 0.858265719615662, 'reg_alpha': 8.248249446815258, 'reg_lambda': 2.1043159445439197, 'min_split_gain': 0.025304840362644132}. Best is trial 1 with value: 0.00018704039289719747.
[I 2025-06-15 20:41:28,843] A new study created in memory with name: no-name-92c3011b-d8c0-4514-a70a-996b2aeada5c
[I 2025-06-15 20:41:28,883] Trial 0 finished with value: 0.0014257723656530394 and parameters: {'n_estimators': 91, 'learning_rate': 0.24072755368246343, 'num_leaves': 204, 'max_depth': 7, 'min_child_samples': 14, 'subsample': 0.6529246480678816, 'colsample_bytree': 0.5582637201944043, 'reg_alpha': 4.179219612006177, 'reg_lambda': 8.25361575752776, 'min_split_gain': 0.012924837740553592}. Best is trial 0 with value: 0.0014257723656530394.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         726
14      pct_chg_Load_DA         435
1           Price_lag_2         322
2           Price_lag_7         194
3         Load_DA_lag_0         190
11           NGas_lag_2         187
16  roll2d_median_Price         184
15       lag168_Load_DA         101
10           Coal_lag_2          92
5       WindOn_DA_lag_0          89
13            EUA_lag_2          69
12            Oil_lag_2          66
9                  WD_7          10
7                  WD_1           5
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 87.00% done
********************* END NS.. 635 ****************************************************
********************* START NS ... 636 ****************************************************
[I 2025-06-15 20:41:30,826] A new study created in memory with name: no-name-1e3e8c98-9446-4017-b919-7744a087bd8f
[I 2025-06-15 20:41:33,059] Trial 0 finished with value: 0.0011546077566667913 and parameters: {'n_estimators': 275, 'learning_rate': 0.07882550844086063, 'num_leaves': 206, 'max_depth': 11, 'min_child_samples': 10, 'subsample': 0.5609591271467855, 'colsample_bytree': 0.7913248331042968, 'reg_alpha': 0.15639061888597872, 'reg_lambda': 9.067921887362596, 'min_split_gain': 0.030453294376649367}. Best is trial 0 with value: 0.0011546077566667913.
[Trial 0] New best: 0.00115
[I 2025-06-15 20:41:33,941] Trial 1 finished with value: 0.0016154604151259594 and parameters: {'n_estimators': 98, 'learning_rate': 0.037523676725557766, 'num_leaves': 166, 'max_depth': 16, 'min_child_samples': 24, 'subsample': 0.7837926375970795, 'colsample_bytree': 0.6338732043347338, 'reg_alpha': 4.916731179995071, 'reg_lambda': 0.05733636588942437, 'min_split_gain': 0.019435312749698147}. Best is trial 0 with value: 0.0011546077566667913.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:41:34,835] Trial 2 finished with value: 0.001389593552556496 and parameters: {'n_estimators': 134, 'learning_rate': 0.03203227979873894, 'num_leaves': 156, 'max_depth': 6, 'min_child_samples': 14, 'subsample': 0.7834106782041392, 'colsample_bytree': 0.5470512717435396, 'reg_alpha': 4.396840004743893, 'reg_lambda': 7.759735995549991, 'min_split_gain': 0.009058720296276352}. Best is trial 0 with value: 0.0011546077566667913.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:41:35,386] Trial 3 finished with value: 0.0009872276407374408 and parameters: {'n_estimators': 192, 'learning_rate': 0.09581201493387989, 'num_leaves': 228, 'max_depth': 10, 'min_child_samples': 4, 'subsample': 0.7596286592808519, 'colsample_bytree': 0.7129827356275054, 'reg_alpha': 9.563976698477282, 'reg_lambda': 4.288862454250117, 'min_split_gain': 0.025657398178637453}. Best is trial 3 with value: 0.0009872276407374408.
[I 2025-06-15 20:41:35,388] A new study created in memory with name: no-name-4746781b-cc6f-4eb2-a460-458d00556cd4
[I 2025-06-15 20:41:35,429] Trial 0 finished with value: 0.004428226463256169 and parameters: {'n_estimators': 144, 'learning_rate': 0.25410453929990967, 'num_leaves': 239, 'max_depth': 16, 'min_child_samples': 9, 'subsample': 0.5850126764901054, 'colsample_bytree': 0.7308256326259546, 'reg_alpha': 6.776206555708745, 'reg_lambda': 4.324923716818033, 'min_split_gain': 0.03211169210683644}. Best is trial 0 with value: 0.004428226463256169.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         445
1           Price_lag_2         299
14      pct_chg_Load_DA         254
2           Price_lag_7         174
11           NGas_lag_2         174
3         Load_DA_lag_0         162
16  roll2d_median_Price         133
15       lag168_Load_DA          96
10           Coal_lag_2          93
5       WindOn_DA_lag_0          83
13            EUA_lag_2          62
12            Oil_lag_2          49
9                  WD_7          23
8                  WD_6          12
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 87.14% done
********************* END NS.. 636 ****************************************************
********************* START NS ... 637 ****************************************************
[I 2025-06-15 20:41:37,339] A new study created in memory with name: no-name-706a0938-5b24-43a7-aa5e-d0de15dc7f42
[I 2025-06-15 20:41:38,303] Trial 0 finished with value: 0.004384160456368515 and parameters: {'n_estimators': 115, 'learning_rate': 0.026733923575243783, 'num_leaves': 194, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.9792397224942297, 'colsample_bytree': 0.9031711344406672, 'reg_alpha': 0.03968602454036518, 'reg_lambda': 9.264997916224228, 'min_split_gain': 0.04414435606900391}. Best is trial 0 with value: 0.004384160456368515.
[Trial 0] New best: 0.00438
[I 2025-06-15 20:41:39,746] Trial 1 finished with value: 0.003154937873122501 and parameters: {'n_estimators': 247, 'learning_rate': 0.01739078386843852, 'num_leaves': 92, 'max_depth': 6, 'min_child_samples': 10, 'subsample': 0.5864743194869022, 'colsample_bytree': 0.5917246834251941, 'reg_alpha': 9.388120063440041, 'reg_lambda': 2.9461319879558676, 'min_split_gain': 0.0077575106949213934}. Best is trial 1 with value: 0.003154937873122501.
[Trial 1] New best: 0.00315
[I 2025-06-15 20:41:42,249] Trial 2 finished with value: 0.0027009023058122508 and parameters: {'n_estimators': 268, 'learning_rate': 0.018318974862066812, 'num_leaves': 57, 'max_depth': 10, 'min_child_samples': 13, 'subsample': 0.6616776064307308, 'colsample_bytree': 0.7824095975701523, 'reg_alpha': 7.093468194718655, 'reg_lambda': 4.870963195757769, 'min_split_gain': 0.011528767883198267}. Best is trial 2 with value: 0.0027009023058122508.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:41:43,922] Trial 3 finished with value: 0.0023452884310325556 and parameters: {'n_estimators': 212, 'learning_rate': 0.02294400127932816, 'num_leaves': 37, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.5859798335319613, 'colsample_bytree': 0.9919976497105691, 'reg_alpha': 5.993228799032702, 'reg_lambda': 1.1356320332246628, 'min_split_gain': 0.041800674985747265}. Best is trial 3 with value: 0.0023452884310325556.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:41:45,656] Trial 4 finished with value: 0.002551440550502945 and parameters: {'n_estimators': 249, 'learning_rate': 0.020529405367702695, 'num_leaves': 130, 'max_depth': 13, 'min_child_samples': 20, 'subsample': 0.7667466027337074, 'colsample_bytree': 0.922352793482905, 'reg_alpha': 9.804477432293336, 'reg_lambda': 3.6017496650432523, 'min_split_gain': 0.01708883100080742}. Best is trial 3 with value: 0.0023452884310325556.
[I 2025-06-15 20:41:45,658] A new study created in memory with name: no-name-796cf722-568c-4385-8242-daa9f70ed11e
[I 2025-06-15 20:41:45,710] Trial 0 finished with value: 0.009499581308444823 and parameters: {'n_estimators': 158, 'learning_rate': 0.030245592409139155, 'num_leaves': 204, 'max_depth': 15, 'min_child_samples': 24, 'subsample': 0.5552360468989135, 'colsample_bytree': 0.6044052087250501, 'reg_alpha': 6.037818880451145, 'reg_lambda': 8.36580194727555, 'min_split_gain': 0.008819549560178825}. Best is trial 0 with value: 0.009499581308444823.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2109
14      pct_chg_Load_DA        1069
1           Price_lag_2         929
2           Price_lag_7         555
16  roll2d_median_Price         537
11           NGas_lag_2         423
3         Load_DA_lag_0         327
13            EUA_lag_2         213
10           Coal_lag_2         198
12            Oil_lag_2         188
5       WindOn_DA_lag_0         175
15       lag168_Load_DA         168
9                  WD_7          14
7                  WD_1           1
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 87.28% done
********************* END NS.. 637 ****************************************************
********************* START NS ... 638 ****************************************************
[I 2025-06-15 20:41:48,714] A new study created in memory with name: no-name-163881a9-e665-4cf5-8ec5-4877da0a2d74
[I 2025-06-15 20:41:50,158] Trial 0 finished with value: 0.0017995636104915522 and parameters: {'n_estimators': 108, 'learning_rate': 0.05370807139570568, 'num_leaves': 73, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.6740912104945957, 'colsample_bytree': 0.7676681184863572, 'reg_alpha': 0.5910471441293874, 'reg_lambda': 5.853585956294496, 'min_split_gain': 0.00575565689501999}. Best is trial 0 with value: 0.0017995636104915522.
[Trial 0] New best: 0.00180
[I 2025-06-15 20:41:51,047] Trial 1 finished with value: 0.0019816273209231107 and parameters: {'n_estimators': 128, 'learning_rate': 0.052565555550241956, 'num_leaves': 110, 'max_depth': 16, 'min_child_samples': 16, 'subsample': 0.6057829479569838, 'colsample_bytree': 0.854037975023488, 'reg_alpha': 8.260931916930385, 'reg_lambda': 7.5143443018075065, 'min_split_gain': 0.03202385638646995}. Best is trial 0 with value: 0.0017995636104915522.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:41:51,665] Trial 2 finished with value: 0.002049826080581118 and parameters: {'n_estimators': 194, 'learning_rate': 0.09810494308441124, 'num_leaves': 171, 'max_depth': 15, 'min_child_samples': 15, 'subsample': 0.8703813515044633, 'colsample_bytree': 0.8332951794394323, 'reg_alpha': 3.8984541550569842, 'reg_lambda': 1.1100684449053677, 'min_split_gain': 0.03878679599373935}. Best is trial 0 with value: 0.0017995636104915522.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:41:52,764] Trial 3 finished with value: 0.002094074882295756 and parameters: {'n_estimators': 143, 'learning_rate': 0.07559921612673055, 'num_leaves': 189, 'max_depth': 15, 'min_child_samples': 13, 'subsample': 0.5513930042236546, 'colsample_bytree': 0.5901656124149125, 'reg_alpha': 2.9259440063991615, 'reg_lambda': 0.7852418144682594, 'min_split_gain': 0.02863389809205824}. Best is trial 0 with value: 0.0017995636104915522.
[I 2025-06-15 20:41:52,766] A new study created in memory with name: no-name-33273445-29b2-4ced-bda1-f66e1d45d8f4
[I 2025-06-15 20:41:52,840] Trial 0 finished with value: 0.012922590572206974 and parameters: {'n_estimators': 268, 'learning_rate': 0.02084318719356611, 'num_leaves': 144, 'max_depth': 7, 'min_child_samples': 6, 'subsample': 0.9698925588312228, 'colsample_bytree': 0.5180195034414969, 'reg_alpha': 9.087587436106228, 'reg_lambda': 5.160699230582463, 'min_split_gain': 0.04478549428009547}. Best is trial 0 with value: 0.012922590572206974.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1537
1           Price_lag_2        1421
14      pct_chg_Load_DA         945
3         Load_DA_lag_0         538
16  roll2d_median_Price         532
2           Price_lag_7         505
11           NGas_lag_2         477
15       lag168_Load_DA         420
5       WindOn_DA_lag_0         388
10           Coal_lag_2         324
13            EUA_lag_2         317
12            Oil_lag_2         254
9                  WD_7          29
7                  WD_1          13
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 87.41% done
********************* END NS.. 638 ****************************************************
********************* START NS ... 639 ****************************************************
[I 2025-06-15 20:41:55,862] A new study created in memory with name: no-name-1105a868-bdf4-42ad-bcd8-0e0457bc8e8b
[I 2025-06-15 20:41:57,428] Trial 0 finished with value: 0.002990738609959852 and parameters: {'n_estimators': 249, 'learning_rate': 0.043844872832520095, 'num_leaves': 242, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.6862602587276413, 'colsample_bytree': 0.8422174590643146, 'reg_alpha': 6.151257089600902, 'reg_lambda': 7.7482086054927946, 'min_split_gain': 0.011977536264733524}. Best is trial 0 with value: 0.002990738609959852.
[Trial 0] New best: 0.00299
[I 2025-06-15 20:41:59,141] Trial 1 finished with value: 0.004970027507815813 and parameters: {'n_estimators': 202, 'learning_rate': 0.01487162728482873, 'num_leaves': 149, 'max_depth': 6, 'min_child_samples': 12, 'subsample': 0.5149587943790499, 'colsample_bytree': 0.8310947568681015, 'reg_alpha': 1.0463274100696907, 'reg_lambda': 3.941826764472159, 'min_split_gain': 0.005886634502915567}. Best is trial 0 with value: 0.002990738609959852.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:42:00,119] Trial 2 finished with value: 0.0036343196793606866 and parameters: {'n_estimators': 101, 'learning_rate': 0.04884255742628682, 'num_leaves': 112, 'max_depth': 8, 'min_child_samples': 9, 'subsample': 0.5101909294128385, 'colsample_bytree': 0.8291528447766097, 'reg_alpha': 4.678771697780827, 'reg_lambda': 1.8449019217194584, 'min_split_gain': 0.04397475958207311}. Best is trial 0 with value: 0.002990738609959852.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:42:01,965] Trial 3 finished with value: 0.0027986283578261026 and parameters: {'n_estimators': 262, 'learning_rate': 0.0601089577052068, 'num_leaves': 224, 'max_depth': 11, 'min_child_samples': 18, 'subsample': 0.6882810107625306, 'colsample_bytree': 0.5965258791359924, 'reg_alpha': 1.2168954637718699, 'reg_lambda': 3.028409549074116, 'min_split_gain': 0.02715524544584377}. Best is trial 3 with value: 0.0027986283578261026.
[I 2025-06-15 20:42:01,968] A new study created in memory with name: no-name-72b32f19-bf65-4c7f-bc67-495e1dfc1935
[I 2025-06-15 20:42:02,153] Trial 0 finished with value: 0.02203448032308337 and parameters: {'n_estimators': 164, 'learning_rate': 0.1318520891751129, 'num_leaves': 215, 'max_depth': 16, 'min_child_samples': 13, 'subsample': 0.6580983640157018, 'colsample_bytree': 0.6044330424841011, 'reg_alpha': 1.9587927247038261, 'reg_lambda': 8.635574654990313, 'min_split_gain': 0.038763785457519816}. Best is trial 0 with value: 0.02203448032308337.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1385
0           Price_lag_1        1292
14      pct_chg_Load_DA         956
3         Load_DA_lag_0         727
15       lag168_Load_DA         596
5       WindOn_DA_lag_0         562
11           NGas_lag_2         560
2           Price_lag_7         517
16  roll2d_median_Price         468
10           Coal_lag_2         387
13            EUA_lag_2         293
12            Oil_lag_2         250
9                  WD_7          38
7                  WD_1          16
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           8
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 87.55% done
********************* END NS.. 639 ****************************************************
********************* START NS ... 640 ****************************************************
[I 2025-06-15 20:42:04,930] A new study created in memory with name: no-name-85cefc39-36c9-4d6d-a9e3-28c30d545e3e
[I 2025-06-15 20:42:05,340] Trial 0 finished with value: 0.0036984156571997456 and parameters: {'n_estimators': 154, 'learning_rate': 0.06310176323986737, 'num_leaves': 70, 'max_depth': 4, 'min_child_samples': 14, 'subsample': 0.6524825125565132, 'colsample_bytree': 0.7385276699602181, 'reg_alpha': 1.8438486248134867, 'reg_lambda': 9.988307767404285, 'min_split_gain': 0.041798160424355635}. Best is trial 0 with value: 0.0036984156571997456.
[Trial 0] New best: 0.00370
[I 2025-06-15 20:42:05,666] Trial 1 finished with value: 0.014552631827932852 and parameters: {'n_estimators': 51, 'learning_rate': 0.04210987964265392, 'num_leaves': 150, 'max_depth': 13, 'min_child_samples': 18, 'subsample': 0.5812595938762379, 'colsample_bytree': 0.6098589516281312, 'reg_alpha': 9.578101999822827, 'reg_lambda': 4.54015956192893, 'min_split_gain': 0.03814854818614019}. Best is trial 0 with value: 0.0036984156571997456.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:42:07,771] Trial 2 finished with value: 0.004847860126976703 and parameters: {'n_estimators': 207, 'learning_rate': 0.03431835508952964, 'num_leaves': 205, 'max_depth': 9, 'min_child_samples': 24, 'subsample': 0.9985746375012413, 'colsample_bytree': 0.5508909875568846, 'reg_alpha': 4.74078517856337, 'reg_lambda': 5.619652425167692, 'min_split_gain': 0.016396438576304056}. Best is trial 0 with value: 0.0036984156571997456.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:42:10,452] Trial 3 finished with value: 0.013969912028918972 and parameters: {'n_estimators': 206, 'learning_rate': 0.010549723815162873, 'num_leaves': 92, 'max_depth': 11, 'min_child_samples': 18, 'subsample': 0.55674248142437, 'colsample_bytree': 0.6203370157632221, 'reg_alpha': 2.822219918390986, 'reg_lambda': 4.97596757310381, 'min_split_gain': 0.0162425690923185}. Best is trial 0 with value: 0.0036984156571997456.
[I 2025-06-15 20:42:10,454] A new study created in memory with name: no-name-33167a9d-24d8-4469-9cda-ded4f992c0d7
[I 2025-06-15 20:42:10,509] Trial 0 finished with value: 0.034583733988279804 and parameters: {'n_estimators': 222, 'learning_rate': 0.04545099109028411, 'num_leaves': 156, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.9041381149420895, 'colsample_bytree': 0.8530252668541728, 'reg_alpha': 5.816338647815957, 'reg_lambda': 1.6423369895170448, 'min_split_gain': 0.0004958579094882621}. Best is trial 0 with value: 0.034583733988279804.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         615
1           Price_lag_2         279
14      pct_chg_Load_DA         263
3         Load_DA_lag_0         168
11           NGas_lag_2         143
16  roll2d_median_Price         106
2           Price_lag_7          95
5       WindOn_DA_lag_0          84
13            EUA_lag_2          84
15       lag168_Load_DA          82
10           Coal_lag_2          48
12            Oil_lag_2          40
9                  WD_7          13
7                  WD_1          11
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          64
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 87.69% done
********************* END NS.. 640 ****************************************************
********************* START NS ... 641 ****************************************************
[I 2025-06-15 20:42:12,607] A new study created in memory with name: no-name-f032f0bc-68a3-42cd-9b52-2a69b79ef649
[I 2025-06-15 20:42:13,232] Trial 0 finished with value: 0.009902807608699174 and parameters: {'n_estimators': 282, 'learning_rate': 0.05040749955997963, 'num_leaves': 165, 'max_depth': 5, 'min_child_samples': 24, 'subsample': 0.9945280243018626, 'colsample_bytree': 0.6296212753025514, 'reg_alpha': 2.6827108877373074, 'reg_lambda': 4.864966553543093, 'min_split_gain': 0.040402925136460044}. Best is trial 0 with value: 0.009902807608699174.
[Trial 0] New best: 0.00990
[I 2025-06-15 20:42:13,743] Trial 1 finished with value: 0.008496647179833816 and parameters: {'n_estimators': 169, 'learning_rate': 0.0963610182224196, 'num_leaves': 242, 'max_depth': 6, 'min_child_samples': 12, 'subsample': 0.5270780650531073, 'colsample_bytree': 0.7631566925307474, 'reg_alpha': 7.89724538900559, 'reg_lambda': 2.999920279946627, 'min_split_gain': 0.014121566243749507}. Best is trial 1 with value: 0.008496647179833816.
[Trial 1] New best: 0.00850
[I 2025-06-15 20:42:14,198] Trial 2 finished with value: 0.008484612156365813 and parameters: {'n_estimators': 95, 'learning_rate': 0.17899928317040228, 'num_leaves': 166, 'max_depth': 13, 'min_child_samples': 24, 'subsample': 0.8276900278153803, 'colsample_bytree': 0.603984292268658, 'reg_alpha': 5.812646789259036, 'reg_lambda': 7.991587435342199, 'min_split_gain': 0.030993934075476057}. Best is trial 2 with value: 0.008484612156365813.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:42:17,021] Trial 3 finished with value: 0.011211892699185355 and parameters: {'n_estimators': 273, 'learning_rate': 0.01043434041518271, 'num_leaves': 235, 'max_depth': 10, 'min_child_samples': 10, 'subsample': 0.5792658594259211, 'colsample_bytree': 0.5810335114889262, 'reg_alpha': 7.20920298692672, 'reg_lambda': 9.260473438677106, 'min_split_gain': 0.0034068705930876634}. Best is trial 2 with value: 0.008484612156365813.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:42:20,351] Trial 4 finished with value: 0.006545071868721408 and parameters: {'n_estimators': 296, 'learning_rate': 0.015820591525094078, 'num_leaves': 162, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.9378959107941085, 'colsample_bytree': 0.9834328223360843, 'reg_alpha': 0.8315149119240761, 'reg_lambda': 8.66925928771847, 'min_split_gain': 0.038159441370381374}. Best is trial 4 with value: 0.006545071868721408.
[Trial 4] New best: 0.00655
[I 2025-06-15 20:42:20,756] Trial 5 finished with value: 0.008179496919407133 and parameters: {'n_estimators': 255, 'learning_rate': 0.1541839587847292, 'num_leaves': 219, 'max_depth': 14, 'min_child_samples': 15, 'subsample': 0.9867819702700512, 'colsample_bytree': 0.8263758466132869, 'reg_alpha': 6.942913344033625, 'reg_lambda': 1.8635990399070435, 'min_split_gain': 0.043373719276286984}. Best is trial 4 with value: 0.006545071868721408.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:42:22,702] Trial 6 finished with value: 0.005254460198621368 and parameters: {'n_estimators': 99, 'learning_rate': 0.2550080326552594, 'num_leaves': 237, 'max_depth': 14, 'min_child_samples': 5, 'subsample': 0.5249179672567956, 'colsample_bytree': 0.9333942587529432, 'reg_alpha': 1.2609408757865748, 'reg_lambda': 9.572972498840024, 'min_split_gain': 0.001209589040350867}. Best is trial 6 with value: 0.005254460198621368.
[Trial 6] New best: 0.00525
[I 2025-06-15 20:42:24,245] Trial 7 finished with value: 0.00800618096737833 and parameters: {'n_estimators': 209, 'learning_rate': 0.01964847885879358, 'num_leaves': 91, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.9863870703462321, 'colsample_bytree': 0.9678267043300322, 'reg_alpha': 9.532306212240824, 'reg_lambda': 8.96920598170057, 'min_split_gain': 0.043249051314579165}. Best is trial 6 with value: 0.005254460198621368.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:42:25,078] Trial 8 finished with value: 0.011165724230758442 and parameters: {'n_estimators': 68, 'learning_rate': 0.04023047682456057, 'num_leaves': 237, 'max_depth': 13, 'min_child_samples': 27, 'subsample': 0.5855152902219567, 'colsample_bytree': 0.6972316186365703, 'reg_alpha': 3.214490345690707, 'reg_lambda': 0.5233120639158484, 'min_split_gain': 0.02895975964560092}. Best is trial 6 with value: 0.005254460198621368.
[Trial 8] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:42:28,190] Trial 9 finished with value: 0.007622032149280411 and parameters: {'n_estimators': 249, 'learning_rate': 0.017236409960597176, 'num_leaves': 144, 'max_depth': 7, 'min_child_samples': 10, 'subsample': 0.5206125420249477, 'colsample_bytree': 0.6992032604302401, 'reg_alpha': 0.3864287087876894, 'reg_lambda': 9.403979735505382, 'min_split_gain': 0.018886938331653752}. Best is trial 6 with value: 0.005254460198621368.
[I 2025-06-15 20:42:28,193] A new study created in memory with name: no-name-649dab62-1539-45eb-85e3-de1319f5cfbd
[I 2025-06-15 20:42:28,248] Trial 0 finished with value: 0.09385651615500125 and parameters: {'n_estimators': 245, 'learning_rate': 0.03642336919381992, 'num_leaves': 31, 'max_depth': 12, 'min_child_samples': 23, 'subsample': 0.52104455562098, 'colsample_bytree': 0.9105049730125507, 'reg_alpha': 6.932159814711073, 'reg_lambda': 3.5994030854142025, 'min_split_gain': 0.034446675148688786}. Best is trial 0 with value: 0.09385651615500125.
[Trial 9] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA        1082
5       WindOn_DA_lag_0        1010
0           Price_lag_1        1002
3         Load_DA_lag_0         984
15       lag168_Load_DA         984
1           Price_lag_2         934
2           Price_lag_7         721
16  roll2d_median_Price         613
12            Oil_lag_2         553
13            EUA_lag_2         515
11           NGas_lag_2         503
10           Coal_lag_2         390
9                  WD_7          26
8                  WD_6          13
7                  WD_1          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          34
1           Price_lag_2           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 87.82% done
********************* END NS.. 641 ****************************************************
********************* START NS ... 642 ****************************************************
[I 2025-06-15 20:42:31,261] A new study created in memory with name: no-name-4cb36947-1e59-4ac8-aa97-f43a235f5cfa
[I 2025-06-15 20:42:34,876] Trial 0 finished with value: 0.007429312934504932 and parameters: {'n_estimators': 203, 'learning_rate': 0.04154882467407678, 'num_leaves': 214, 'max_depth': 13, 'min_child_samples': 11, 'subsample': 0.661790880168392, 'colsample_bytree': 0.7935684796140767, 'reg_alpha': 0.2798641100367616, 'reg_lambda': 8.125551270255198, 'min_split_gain': 0.027644523722764422}. Best is trial 0 with value: 0.007429312934504932.
[Trial 0] New best: 0.00743
[I 2025-06-15 20:42:36,108] Trial 1 finished with value: 0.010005321115338384 and parameters: {'n_estimators': 150, 'learning_rate': 0.029918554907545174, 'num_leaves': 202, 'max_depth': 15, 'min_child_samples': 29, 'subsample': 0.5675681473540931, 'colsample_bytree': 0.6368957148742462, 'reg_alpha': 5.222277979087183, 'reg_lambda': 4.815510741980384, 'min_split_gain': 0.031968558015309026}. Best is trial 0 with value: 0.007429312934504932.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:42:36,677] Trial 2 finished with value: 0.008699165238356258 and parameters: {'n_estimators': 181, 'learning_rate': 0.0867208073886174, 'num_leaves': 63, 'max_depth': 11, 'min_child_samples': 26, 'subsample': 0.9954623503794933, 'colsample_bytree': 0.8593136210510908, 'reg_alpha': 4.953807904101632, 'reg_lambda': 2.346178399557205, 'min_split_gain': 0.04831686487216108}. Best is trial 0 with value: 0.007429312934504932.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:42:37,492] Trial 3 finished with value: 0.009077903592812958 and parameters: {'n_estimators': 222, 'learning_rate': 0.12069155468055805, 'num_leaves': 252, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.5890357802783153, 'colsample_bytree': 0.5281227718656734, 'reg_alpha': 7.005679497690132, 'reg_lambda': 4.875422648224545, 'min_split_gain': 0.01262469502637128}. Best is trial 0 with value: 0.007429312934504932.
[I 2025-06-15 20:42:37,494] A new study created in memory with name: no-name-7297110f-d7b8-409c-8ea0-80c1d7e49597
[I 2025-06-15 20:42:37,531] Trial 0 finished with value: 0.09200531491282184 and parameters: {'n_estimators': 81, 'learning_rate': 0.038183455451966955, 'num_leaves': 125, 'max_depth': 7, 'min_child_samples': 8, 'subsample': 0.9740523917501145, 'colsample_bytree': 0.6852064059722436, 'reg_alpha': 3.7579438629716835, 'reg_lambda': 9.884885005967895, 'min_split_gain': 0.0355523818650012}. Best is trial 0 with value: 0.09200531491282184.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2645
1           Price_lag_2        2566
14      pct_chg_Load_DA        2344
3         Load_DA_lag_0        1898
5       WindOn_DA_lag_0        1370
2           Price_lag_7        1352
16  roll2d_median_Price        1351
15       lag168_Load_DA        1265
11           NGas_lag_2        1066
10           Coal_lag_2         759
13            EUA_lag_2         643
12            Oil_lag_2         552
9                  WD_7          58
7                  WD_1          28
8                  WD_6          16
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          78
16  roll2d_median_Price          15
11           NGas_lag_2           5
1           Price_lag_2           3
12            Oil_lag_2           2
2           Price_lag_7           1
3         Load_DA_lag_0           1
13            EUA_lag_2           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 87.96% done
********************* END NS.. 642 ****************************************************
********************* START NS ... 643 ****************************************************
[I 2025-06-15 20:42:41,983] A new study created in memory with name: no-name-f59614db-e51f-4366-a6bc-e4578a3e74b6
[I 2025-06-15 20:42:43,132] Trial 0 finished with value: 0.029414598946997234 and parameters: {'n_estimators': 154, 'learning_rate': 0.06636827561011714, 'num_leaves': 146, 'max_depth': 8, 'min_child_samples': 12, 'subsample': 0.9642034191018929, 'colsample_bytree': 0.6284838520327787, 'reg_alpha': 0.885346067934778, 'reg_lambda': 0.05338388552468154, 'min_split_gain': 0.03916877110029963}. Best is trial 0 with value: 0.029414598946997234.
[Trial 0] New best: 0.02941
[I 2025-06-15 20:42:43,553] Trial 1 finished with value: 0.02681232771906548 and parameters: {'n_estimators': 118, 'learning_rate': 0.27748228864137003, 'num_leaves': 197, 'max_depth': 15, 'min_child_samples': 23, 'subsample': 0.7126950879342833, 'colsample_bytree': 0.7816770945031888, 'reg_alpha': 1.352444877549962, 'reg_lambda': 0.42348680158169505, 'min_split_gain': 0.02051187324455696}. Best is trial 1 with value: 0.02681232771906548.
[Trial 1] New best: 0.02681
[I 2025-06-15 20:42:44,018] Trial 2 finished with value: 0.031608302466933406 and parameters: {'n_estimators': 228, 'learning_rate': 0.2616905549715599, 'num_leaves': 250, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.9079829742775496, 'colsample_bytree': 0.9108565677555153, 'reg_alpha': 5.083300661422143, 'reg_lambda': 4.802218749323409, 'min_split_gain': 0.014211953791625771}. Best is trial 1 with value: 0.02681232771906548.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:42:44,417] Trial 3 finished with value: 0.030649834879104693 and parameters: {'n_estimators': 219, 'learning_rate': 0.15017538334352373, 'num_leaves': 50, 'max_depth': 9, 'min_child_samples': 24, 'subsample': 0.7724352026784117, 'colsample_bytree': 0.8608385093826708, 'reg_alpha': 8.329527309291336, 'reg_lambda': 6.814594932229472, 'min_split_gain': 0.029956238863390706}. Best is trial 1 with value: 0.02681232771906548.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:42:46,189] Trial 4 finished with value: 0.03380639766182187 and parameters: {'n_estimators': 180, 'learning_rate': 0.01949860693601442, 'num_leaves': 256, 'max_depth': 16, 'min_child_samples': 20, 'subsample': 0.786642417924204, 'colsample_bytree': 0.6156965960804873, 'reg_alpha': 5.540790265932626, 'reg_lambda': 5.089538950957243, 'min_split_gain': 0.012611935413374043}. Best is trial 1 with value: 0.02681232771906548.
[I 2025-06-15 20:42:46,192] A new study created in memory with name: no-name-55326053-e630-4bc3-9b0f-3ccce5cab987
[I 2025-06-15 20:42:46,219] Trial 0 finished with value: 0.10666400704358513 and parameters: {'n_estimators': 76, 'learning_rate': 0.26621297105492714, 'num_leaves': 79, 'max_depth': 4, 'min_child_samples': 4, 'subsample': 0.7001819428663117, 'colsample_bytree': 0.8439210480591663, 'reg_alpha': 8.331515874358058, 'reg_lambda': 4.716332408169298, 'min_split_gain': 0.00010878076097494694}. Best is trial 0 with value: 0.10666400704358513.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         321
14      pct_chg_Load_DA         237
1           Price_lag_2         231
3         Load_DA_lag_0         173
16  roll2d_median_Price         126
5       WindOn_DA_lag_0         119
15       lag168_Load_DA         117
2           Price_lag_7         113
11           NGas_lag_2         100
10           Coal_lag_2          63
13            EUA_lag_2          59
12            Oil_lag_2          39
7                  WD_1           5
9                  WD_7           4
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          27
16  roll2d_median_Price           8
2           Price_lag_7           2
15       lag168_Load_DA           2
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 88.10% done
********************* END NS.. 643 ****************************************************
********************* START NS ... 644 ****************************************************
[I 2025-06-15 20:42:48,089] A new study created in memory with name: no-name-052a7735-a857-4cc0-b06a-60241e795ebd
[I 2025-06-15 20:42:49,539] Trial 0 finished with value: 0.02672102963881558 and parameters: {'n_estimators': 267, 'learning_rate': 0.06145476215963478, 'num_leaves': 41, 'max_depth': 6, 'min_child_samples': 9, 'subsample': 0.9677456671732447, 'colsample_bytree': 0.9510477715003978, 'reg_alpha': 5.962474593997394, 'reg_lambda': 5.478386173098655, 'min_split_gain': 0.001339936833499611}. Best is trial 0 with value: 0.02672102963881558.
[Trial 0] New best: 0.02672
[I 2025-06-15 20:42:50,872] Trial 1 finished with value: 0.026471566239343065 and parameters: {'n_estimators': 119, 'learning_rate': 0.09809598387214552, 'num_leaves': 172, 'max_depth': 12, 'min_child_samples': 30, 'subsample': 0.7075016513430719, 'colsample_bytree': 0.6373326102224075, 'reg_alpha': 0.07818027150585438, 'reg_lambda': 7.179671822536885, 'min_split_gain': 0.029727857888896744}. Best is trial 1 with value: 0.026471566239343065.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:42:52,042] Trial 2 finished with value: 0.0286281923012414 and parameters: {'n_estimators': 118, 'learning_rate': 0.058868825448343896, 'num_leaves': 154, 'max_depth': 14, 'min_child_samples': 27, 'subsample': 0.6351879468452089, 'colsample_bytree': 0.8953307500385299, 'reg_alpha': 3.144016955449004, 'reg_lambda': 3.4820887474053297, 'min_split_gain': 0.027221253446855942}. Best is trial 1 with value: 0.026471566239343065.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:42:52,767] Trial 3 finished with value: 0.03695356689526546 and parameters: {'n_estimators': 255, 'learning_rate': 0.013917614475183224, 'num_leaves': 35, 'max_depth': 4, 'min_child_samples': 27, 'subsample': 0.8843068157082048, 'colsample_bytree': 0.5727834608982323, 'reg_alpha': 9.652050461493676, 'reg_lambda': 6.875430726478292, 'min_split_gain': 0.027445793129679798}. Best is trial 1 with value: 0.026471566239343065.
[I 2025-06-15 20:42:52,770] A new study created in memory with name: no-name-ddb7b189-1438-46c9-9feb-99b29144bc7c
[I 2025-06-15 20:42:52,806] Trial 0 finished with value: 0.06700180335715807 and parameters: {'n_estimators': 109, 'learning_rate': 0.25931860645630206, 'num_leaves': 44, 'max_depth': 14, 'min_child_samples': 12, 'subsample': 0.9473145777544738, 'colsample_bytree': 0.7116838873435846, 'reg_alpha': 7.004491541223348, 'reg_lambda': 0.6820028361095787, 'min_split_gain': 0.0012042053433766076}. Best is trial 0 with value: 0.06700180335715807.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2         933
0           Price_lag_1         925
14      pct_chg_Load_DA         694
3         Load_DA_lag_0         575
2           Price_lag_7         443
16  roll2d_median_Price         442
15       lag168_Load_DA         421
5       WindOn_DA_lag_0         330
11           NGas_lag_2         322
10           Coal_lag_2         296
13            EUA_lag_2         260
12            Oil_lag_2         202
9                  WD_7          15
7                  WD_1          12
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          15
16  roll2d_median_Price           2
11           NGas_lag_2           1
2           Price_lag_7           0
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 88.24% done
********************* END NS.. 644 ****************************************************
********************* START NS ... 645 ****************************************************
[I 2025-06-15 20:42:55,065] A new study created in memory with name: no-name-d02e8019-1b22-4bf5-902f-b88d3bd9a80e
[I 2025-06-15 20:42:57,155] Trial 0 finished with value: 0.00040512575362959996 and parameters: {'n_estimators': 175, 'learning_rate': 0.031855955285756, 'num_leaves': 65, 'max_depth': 7, 'min_child_samples': 11, 'subsample': 0.9699411638632365, 'colsample_bytree': 0.9183607422183988, 'reg_alpha': 1.242291443393705, 'reg_lambda': 0.5129961889115997, 'min_split_gain': 0.01445749419528798}. Best is trial 0 with value: 0.00040512575362959996.
[Trial 0] New best: 0.00041
[I 2025-06-15 20:42:57,722] Trial 1 finished with value: 0.053052460882823345 and parameters: {'n_estimators': 59, 'learning_rate': 0.018632935343578405, 'num_leaves': 42, 'max_depth': 16, 'min_child_samples': 25, 'subsample': 0.5641286154662966, 'colsample_bytree': 0.6999595085830146, 'reg_alpha': 1.8687570189279723, 'reg_lambda': 0.6861086474545952, 'min_split_gain': 0.004692397527131936}. Best is trial 0 with value: 0.00040512575362959996.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:42:58,096] Trial 2 finished with value: 0.0003968708196440413 and parameters: {'n_estimators': 133, 'learning_rate': 0.13141705102566167, 'num_leaves': 215, 'max_depth': 7, 'min_child_samples': 27, 'subsample': 0.6916428295513475, 'colsample_bytree': 0.9407409762520365, 'reg_alpha': 6.914003454628977, 'reg_lambda': 6.006593509677668, 'min_split_gain': 0.034637240375391086}. Best is trial 2 with value: 0.0003968708196440413.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:42:59,988] Trial 3 finished with value: 0.006273776329896675 and parameters: {'n_estimators': 166, 'learning_rate': 0.012841791145768297, 'num_leaves': 72, 'max_depth': 10, 'min_child_samples': 15, 'subsample': 0.757743667477712, 'colsample_bytree': 0.9753569282252803, 'reg_alpha': 2.727972045372613, 'reg_lambda': 0.5827464544654426, 'min_split_gain': 0.024208524692996305}. Best is trial 2 with value: 0.0003968708196440413.
[I 2025-06-15 20:42:59,992] A new study created in memory with name: no-name-5d384897-e25e-48b4-a781-0928988e3b99
[I 2025-06-15 20:43:00,054] Trial 0 finished with value: 0.0029133838308747755 and parameters: {'n_estimators': 101, 'learning_rate': 0.08941323866150507, 'num_leaves': 121, 'max_depth': 7, 'min_child_samples': 4, 'subsample': 0.522650816719828, 'colsample_bytree': 0.5975548531034496, 'reg_alpha': 4.183653840768221, 'reg_lambda': 6.813581918200534, 'min_split_gain': 0.011533247874339948}. Best is trial 0 with value: 0.0029133838308747755.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         360
14      pct_chg_Load_DA         220
1           Price_lag_2         151
3         Load_DA_lag_0         102
16  roll2d_median_Price          91
11           NGas_lag_2          90
2           Price_lag_7          82
5       WindOn_DA_lag_0          60
10           Coal_lag_2          52
15       lag168_Load_DA          39
13            EUA_lag_2          30
12            Oil_lag_2          26
9                  WD_7           5
7                  WD_1           4
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          36
1           Price_lag_2           8
3         Load_DA_lag_0           4
5       WindOn_DA_lag_0           2
11           NGas_lag_2           2
16  roll2d_median_Price           2
2           Price_lag_7           1
13            EUA_lag_2           1
15       lag168_Load_DA           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
12            Oil_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 88.37% done
********************* END NS.. 645 ****************************************************
********************* START NS ... 646 ****************************************************
[I 2025-06-15 20:43:02,191] A new study created in memory with name: no-name-e90bfdc1-ad0e-42d8-9786-949af00a6d36
[I 2025-06-15 20:43:03,210] Trial 0 finished with value: 0.0006497015011994243 and parameters: {'n_estimators': 271, 'learning_rate': 0.019109750483595624, 'num_leaves': 33, 'max_depth': 5, 'min_child_samples': 21, 'subsample': 0.678485872345842, 'colsample_bytree': 0.595573402770408, 'reg_alpha': 9.040445350040912, 'reg_lambda': 0.6651919616514856, 'min_split_gain': 0.027204863140809072}. Best is trial 0 with value: 0.0006497015011994243.
[Trial 0] New best: 0.00065
[I 2025-06-15 20:43:05,640] Trial 1 finished with value: 0.0006569198229172574 and parameters: {'n_estimators': 260, 'learning_rate': 0.054965780009595816, 'num_leaves': 150, 'max_depth': 14, 'min_child_samples': 17, 'subsample': 0.5190344582164438, 'colsample_bytree': 0.5723873613367169, 'reg_alpha': 0.5613436620812562, 'reg_lambda': 7.324378522053243, 'min_split_gain': 0.026831094431092772}. Best is trial 0 with value: 0.0006497015011994243.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:43:06,098] Trial 2 finished with value: 0.0007014810985603309 and parameters: {'n_estimators': 148, 'learning_rate': 0.049597880346886405, 'num_leaves': 110, 'max_depth': 4, 'min_child_samples': 2, 'subsample': 0.8323241732481588, 'colsample_bytree': 0.6127180906644372, 'reg_alpha': 8.88736859839739, 'reg_lambda': 4.687234428771928, 'min_split_gain': 0.013899447210368088}. Best is trial 0 with value: 0.0006497015011994243.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:43:06,748] Trial 3 finished with value: 0.0006438360568602174 and parameters: {'n_estimators': 139, 'learning_rate': 0.18734195313931848, 'num_leaves': 243, 'max_depth': 9, 'min_child_samples': 4, 'subsample': 0.6145032298692488, 'colsample_bytree': 0.5156090246695847, 'reg_alpha': 1.6596520073152132, 'reg_lambda': 3.8309063954916978, 'min_split_gain': 0.03011503091258947}. Best is trial 3 with value: 0.0006438360568602174.
[I 2025-06-15 20:43:06,751] A new study created in memory with name: no-name-49965ff9-b62d-487e-b032-fe3bb1c2e43b
[I 2025-06-15 20:43:06,810] Trial 0 finished with value: 0.006319674379313078 and parameters: {'n_estimators': 172, 'learning_rate': 0.12476687130683246, 'num_leaves': 106, 'max_depth': 8, 'min_child_samples': 25, 'subsample': 0.9724348461821621, 'colsample_bytree': 0.8873463245971114, 'reg_alpha': 1.934589051597022, 'reg_lambda': 3.9323099401757053, 'min_split_gain': 0.015135202021884404}. Best is trial 0 with value: 0.006319674379313078.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         468
1           Price_lag_2         412
3         Load_DA_lag_0         256
16  roll2d_median_Price         234
14      pct_chg_Load_DA         211
15       lag168_Load_DA         183
2           Price_lag_7         176
5       WindOn_DA_lag_0         158
11           NGas_lag_2         143
13            EUA_lag_2         141
10           Coal_lag_2         140
12            Oil_lag_2         101
8                  WD_6          17
9                  WD_7          14
7                  WD_1           9
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          24
3         Load_DA_lag_0           9
14      pct_chg_Load_DA           6
2           Price_lag_7           2
5       WindOn_DA_lag_0           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 88.51% done
********************* END NS.. 646 ****************************************************
********************* START NS ... 647 ****************************************************
[I 2025-06-15 20:43:08,704] A new study created in memory with name: no-name-84a797a7-78de-4651-aecb-288c6552378b
[I 2025-06-15 20:43:10,936] Trial 0 finished with value: 0.005260178348917796 and parameters: {'n_estimators': 205, 'learning_rate': 0.012192004755820062, 'num_leaves': 135, 'max_depth': 11, 'min_child_samples': 20, 'subsample': 0.885664246130511, 'colsample_bytree': 0.5019822233030933, 'reg_alpha': 3.405221089811522, 'reg_lambda': 5.799750032689306, 'min_split_gain': 0.03174504369302494}. Best is trial 0 with value: 0.005260178348917796.
[Trial 0] New best: 0.00526
[I 2025-06-15 20:43:11,549] Trial 1 finished with value: 0.0031348341305782687 and parameters: {'n_estimators': 131, 'learning_rate': 0.03080381967612519, 'num_leaves': 180, 'max_depth': 5, 'min_child_samples': 9, 'subsample': 0.6550373765047173, 'colsample_bytree': 0.9083407971710746, 'reg_alpha': 5.671446620368007, 'reg_lambda': 8.303643726213133, 'min_split_gain': 0.0068061441497032195}. Best is trial 1 with value: 0.0031348341305782687.
[Trial 1] New best: 0.00313
[I 2025-06-15 20:43:13,668] Trial 2 finished with value: 0.0028726683500543936 and parameters: {'n_estimators': 207, 'learning_rate': 0.023600207249200235, 'num_leaves': 75, 'max_depth': 10, 'min_child_samples': 25, 'subsample': 0.5191593836891393, 'colsample_bytree': 0.6184208880564261, 'reg_alpha': 3.348006888711874, 'reg_lambda': 8.281408613595605, 'min_split_gain': 0.044651137823762836}. Best is trial 2 with value: 0.0028726683500543936.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:43:17,333] Trial 3 finished with value: 0.003187561025904237 and parameters: {'n_estimators': 246, 'learning_rate': 0.016758035121533156, 'num_leaves': 255, 'max_depth': 14, 'min_child_samples': 23, 'subsample': 0.6861984151811493, 'colsample_bytree': 0.8227491622645615, 'reg_alpha': 2.3542943293268834, 'reg_lambda': 1.929745431895109, 'min_split_gain': 0.003643452765232025}. Best is trial 2 with value: 0.0028726683500543936.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:43:17,651] Trial 4 finished with value: 0.08803521727622454 and parameters: {'n_estimators': 50, 'learning_rate': 0.01577351278366751, 'num_leaves': 155, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.5342084628451442, 'colsample_bytree': 0.6799808406382707, 'reg_alpha': 7.989788673895047, 'reg_lambda': 1.9958664637147716, 'min_split_gain': 0.03506117171757603}. Best is trial 2 with value: 0.0028726683500543936.
[I 2025-06-15 20:43:17,654] A new study created in memory with name: no-name-d3883be8-330b-493b-98f5-cf538be670b2
[I 2025-06-15 20:43:17,724] Trial 0 finished with value: 0.008653913074900663 and parameters: {'n_estimators': 242, 'learning_rate': 0.10354812344195419, 'num_leaves': 111, 'max_depth': 4, 'min_child_samples': 13, 'subsample': 0.8515004742558122, 'colsample_bytree': 0.5705485085536091, 'reg_alpha': 6.132462273436227, 'reg_lambda': 7.39656684443783, 'min_split_gain': 0.03436620530582244}. Best is trial 0 with value: 0.008653913074900663.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1829
1           Price_lag_2        1603
14      pct_chg_Load_DA         979
3         Load_DA_lag_0         592
2           Price_lag_7         510
16  roll2d_median_Price         480
15       lag168_Load_DA         439
11           NGas_lag_2         378
5       WindOn_DA_lag_0         343
10           Coal_lag_2         294
13            EUA_lag_2         251
12            Oil_lag_2         222
9                  WD_7          48
8                  WD_6          13
7                  WD_1           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 88.65% done
********************* END NS.. 647 ****************************************************
********************* START NS ... 648 ****************************************************
[I 2025-06-15 20:43:20,789] A new study created in memory with name: no-name-fb6d2609-4e69-43e9-bf77-ed2a30ae917b
[I 2025-06-15 20:43:23,598] Trial 0 finished with value: 0.003117732993470908 and parameters: {'n_estimators': 267, 'learning_rate': 0.03307457670164334, 'num_leaves': 62, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.7507997080069324, 'colsample_bytree': 0.8370494975210138, 'reg_alpha': 0.45724346363946844, 'reg_lambda': 5.5125427596156875, 'min_split_gain': 0.04318175648827291}. Best is trial 0 with value: 0.003117732993470908.
[Trial 0] New best: 0.00312
[I 2025-06-15 20:43:24,900] Trial 1 finished with value: 0.004722210285140006 and parameters: {'n_estimators': 195, 'learning_rate': 0.013655183714157611, 'num_leaves': 86, 'max_depth': 10, 'min_child_samples': 19, 'subsample': 0.5426048556888725, 'colsample_bytree': 0.8045535141173417, 'reg_alpha': 8.34859942701897, 'reg_lambda': 7.277939482841678, 'min_split_gain': 0.01695182216250663}. Best is trial 0 with value: 0.003117732993470908.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:43:25,301] Trial 2 finished with value: 0.003244303604926785 and parameters: {'n_estimators': 54, 'learning_rate': 0.061187830711753835, 'num_leaves': 177, 'max_depth': 11, 'min_child_samples': 12, 'subsample': 0.920652154372201, 'colsample_bytree': 0.766061122391035, 'reg_alpha': 9.82576524446808, 'reg_lambda': 5.593348819122788, 'min_split_gain': 0.018250432326275785}. Best is trial 0 with value: 0.003117732993470908.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:43:26,733] Trial 3 finished with value: 0.0034674435927731103 and parameters: {'n_estimators': 251, 'learning_rate': 0.057980344423053255, 'num_leaves': 215, 'max_depth': 13, 'min_child_samples': 16, 'subsample': 0.6244785715904859, 'colsample_bytree': 0.5155692448262728, 'reg_alpha': 4.716175357075635, 'reg_lambda': 8.296726302315909, 'min_split_gain': 0.04917908981467309}. Best is trial 0 with value: 0.003117732993470908.
[I 2025-06-15 20:43:26,736] A new study created in memory with name: no-name-3a999580-8f43-4538-b8fb-602a6b76e6d8
[I 2025-06-15 20:43:26,795] Trial 0 finished with value: 0.0029206956524921008 and parameters: {'n_estimators': 209, 'learning_rate': 0.20728784648908108, 'num_leaves': 225, 'max_depth': 7, 'min_child_samples': 16, 'subsample': 0.5427919193585256, 'colsample_bytree': 0.976801569657654, 'reg_alpha': 0.7951713874736466, 'reg_lambda': 8.73886205915585, 'min_split_gain': 0.02815092482941515}. Best is trial 0 with value: 0.0029206956524921008.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2475
1           Price_lag_2        1870
14      pct_chg_Load_DA        1699
3         Load_DA_lag_0        1142
16  roll2d_median_Price         914
5       WindOn_DA_lag_0         847
2           Price_lag_7         779
15       lag168_Load_DA         693
11           NGas_lag_2         647
10           Coal_lag_2         435
13            EUA_lag_2         374
12            Oil_lag_2         361
9                  WD_7          49
7                  WD_1          23
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           9
15       lag168_Load_DA           5
5       WindOn_DA_lag_0           3
3         Load_DA_lag_0           3
14      pct_chg_Load_DA           3
16  roll2d_median_Price           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
8                  WD_6           0
-> 88.78% done
********************* END NS.. 648 ****************************************************
********************* START NS ... 649 ****************************************************
[I 2025-06-15 20:43:30,565] A new study created in memory with name: no-name-84708b94-44a7-4af7-b194-3e26f0e3b90a
[I 2025-06-15 20:43:31,530] Trial 0 finished with value: 0.0030041972831315964 and parameters: {'n_estimators': 165, 'learning_rate': 0.01726683130264837, 'num_leaves': 146, 'max_depth': 5, 'min_child_samples': 16, 'subsample': 0.8683648762250535, 'colsample_bytree': 0.9382185682767276, 'reg_alpha': 5.717849889274127, 'reg_lambda': 7.627063027422295, 'min_split_gain': 0.0013146360951008518}. Best is trial 0 with value: 0.0030041972831315964.
[Trial 0] New best: 0.00300
[I 2025-06-15 20:43:32,762] Trial 1 finished with value: 0.01830945579161071 and parameters: {'n_estimators': 165, 'learning_rate': 0.010244394142947318, 'num_leaves': 79, 'max_depth': 14, 'min_child_samples': 11, 'subsample': 0.7454088080759913, 'colsample_bytree': 0.7651596118424219, 'reg_alpha': 9.58382841867882, 'reg_lambda': 2.6295404332464143, 'min_split_gain': 0.015075645752559381}. Best is trial 0 with value: 0.0030041972831315964.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:43:33,813] Trial 2 finished with value: 0.0013345038355986336 and parameters: {'n_estimators': 137, 'learning_rate': 0.08519642060904643, 'num_leaves': 35, 'max_depth': 12, 'min_child_samples': 30, 'subsample': 0.7829704481261218, 'colsample_bytree': 0.7313966830253731, 'reg_alpha': 8.68563320867214, 'reg_lambda': 7.279311172788563, 'min_split_gain': 0.003876624727324707}. Best is trial 2 with value: 0.0013345038355986336.
[Trial 2] New best: 0.00133
[I 2025-06-15 20:43:34,672] Trial 3 finished with value: 0.0016569178663228172 and parameters: {'n_estimators': 139, 'learning_rate': 0.12975116069718404, 'num_leaves': 172, 'max_depth': 10, 'min_child_samples': 16, 'subsample': 0.9529573503717641, 'colsample_bytree': 0.5688446541502985, 'reg_alpha': 1.006115029469431, 'reg_lambda': 1.386003354987121, 'min_split_gain': 0.039781918622220996}. Best is trial 2 with value: 0.0013345038355986336.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:43:34,994] Trial 4 finished with value: 0.001206249990273091 and parameters: {'n_estimators': 105, 'learning_rate': 0.27545791217175786, 'num_leaves': 252, 'max_depth': 8, 'min_child_samples': 2, 'subsample': 0.5452181585079922, 'colsample_bytree': 0.8453178669326076, 'reg_alpha': 4.924663590546211, 'reg_lambda': 4.346067670831619, 'min_split_gain': 0.024425075022551607}. Best is trial 4 with value: 0.001206249990273091.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:43:35,734] Trial 5 finished with value: 0.055375644128669445 and parameters: {'n_estimators': 69, 'learning_rate': 0.016014959056371673, 'num_leaves': 134, 'max_depth': 12, 'min_child_samples': 16, 'subsample': 0.9585909770350882, 'colsample_bytree': 0.5797474291965379, 'reg_alpha': 4.305776251116492, 'reg_lambda': 1.2434670076596, 'min_split_gain': 0.014062360920196066}. Best is trial 4 with value: 0.001206249990273091.
[I 2025-06-15 20:43:35,738] A new study created in memory with name: no-name-52f45e7f-abe8-4bc8-864b-e5bd3954e665
[I 2025-06-15 20:43:35,781] Trial 0 finished with value: 0.0053741040456965275 and parameters: {'n_estimators': 110, 'learning_rate': 0.04087080531531645, 'num_leaves': 249, 'max_depth': 10, 'min_child_samples': 23, 'subsample': 0.8618928551623704, 'colsample_bytree': 0.5644541498090567, 'reg_alpha': 2.8998223618356542, 'reg_lambda': 9.686385232404072, 'min_split_gain': 0.04919939809399951}. Best is trial 0 with value: 0.0053741040456965275.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         213
1           Price_lag_2         174
14      pct_chg_Load_DA         144
3         Load_DA_lag_0          83
2           Price_lag_7          81
11           NGas_lag_2          80
16  roll2d_median_Price          70
5       WindOn_DA_lag_0          60
15       lag168_Load_DA          54
13            EUA_lag_2          37
10           Coal_lag_2          29
12            Oil_lag_2          24
9                  WD_7           5
8                  WD_6           5
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          20
1           Price_lag_2           2
3         Load_DA_lag_0           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 88.92% done
********************* END NS.. 649 ****************************************************
********************* START NS ... 650 ****************************************************
[I 2025-06-15 20:43:37,490] A new study created in memory with name: no-name-0722e388-1892-4b48-affc-554268342a8a
[I 2025-06-15 20:43:37,677] Trial 0 finished with value: 0.0022039239322110674 and parameters: {'n_estimators': 70, 'learning_rate': 0.0420932060735678, 'num_leaves': 206, 'max_depth': 4, 'min_child_samples': 10, 'subsample': 0.5848621149939192, 'colsample_bytree': 0.8403874344403766, 'reg_alpha': 0.7671949146752177, 'reg_lambda': 4.564286977463357, 'min_split_gain': 0.004201923930108087}. Best is trial 0 with value: 0.0022039239322110674.
[Trial 0] New best: 0.00220
[I 2025-06-15 20:43:40,876] Trial 1 finished with value: 0.0013027706216713472 and parameters: {'n_estimators': 271, 'learning_rate': 0.06200339069610338, 'num_leaves': 216, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.7649240526798513, 'colsample_bytree': 0.997557240530942, 'reg_alpha': 2.3889123295102097, 'reg_lambda': 6.160443728128549, 'min_split_gain': 0.00396867234761158}. Best is trial 1 with value: 0.0013027706216713472.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:43:41,679] Trial 2 finished with value: 0.1387284448635594 and parameters: {'n_estimators': 55, 'learning_rate': 0.01108950184156233, 'num_leaves': 166, 'max_depth': 8, 'min_child_samples': 3, 'subsample': 0.9226367815905161, 'colsample_bytree': 0.6731399346830067, 'reg_alpha': 0.8256856541172686, 'reg_lambda': 3.647652441584124, 'min_split_gain': 0.003961709818691589}. Best is trial 1 with value: 0.0013027706216713472.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:43:45,756] Trial 3 finished with value: 0.0014000459500549407 and parameters: {'n_estimators': 199, 'learning_rate': 0.04212715447139404, 'num_leaves': 176, 'max_depth': 14, 'min_child_samples': 11, 'subsample': 0.8862909928061713, 'colsample_bytree': 0.513800839296245, 'reg_alpha': 3.888001990687605, 'reg_lambda': 9.905996842344658, 'min_split_gain': 0.0002232421174718502}. Best is trial 1 with value: 0.0013027706216713472.
[I 2025-06-15 20:43:45,757] A new study created in memory with name: no-name-1de99dd3-95c2-4640-8281-ac7cd08b5c93
[I 2025-06-15 20:43:45,829] Trial 0 finished with value: 0.0035510104370602767 and parameters: {'n_estimators': 265, 'learning_rate': 0.10442539429568182, 'num_leaves': 245, 'max_depth': 15, 'min_child_samples': 4, 'subsample': 0.6904907676724046, 'colsample_bytree': 0.8484068296885741, 'reg_alpha': 1.8734779661976386, 'reg_lambda': 2.938525196378179, 'min_split_gain': 0.009964142055568604}. Best is trial 0 with value: 0.0035510104370602767.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1956
14      pct_chg_Load_DA        1676
1           Price_lag_2        1372
3         Load_DA_lag_0        1287
5       WindOn_DA_lag_0        1109
15       lag168_Load_DA        1037
2           Price_lag_7         827
16  roll2d_median_Price         798
11           NGas_lag_2         778
13            EUA_lag_2         669
12            Oil_lag_2         609
10           Coal_lag_2         559
9                  WD_7          59
7                  WD_1          31
8                  WD_6          22
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           4
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 89.06% done
********************* END NS.. 650 ****************************************************
********************* START NS ... 651 ****************************************************
[I 2025-06-15 20:43:49,944] A new study created in memory with name: no-name-0e5c4b5c-47d9-436b-a2d1-b0f87ea472d2
[I 2025-06-15 20:43:50,363] Trial 0 finished with value: 0.0009833166755323114 and parameters: {'n_estimators': 121, 'learning_rate': 0.13842107667147519, 'num_leaves': 235, 'max_depth': 15, 'min_child_samples': 28, 'subsample': 0.7270986391154355, 'colsample_bytree': 0.7273389533986627, 'reg_alpha': 2.1119045617572727, 'reg_lambda': 2.0138174672917253, 'min_split_gain': 0.018637282399358118}. Best is trial 0 with value: 0.0009833166755323114.
[Trial 0] New best: 0.00098
[I 2025-06-15 20:43:51,387] Trial 1 finished with value: 0.0009755167559093886 and parameters: {'n_estimators': 296, 'learning_rate': 0.08625247962920976, 'num_leaves': 68, 'max_depth': 15, 'min_child_samples': 6, 'subsample': 0.873337116265291, 'colsample_bytree': 0.6099862799358523, 'reg_alpha': 3.546326963540264, 'reg_lambda': 2.97740235194203, 'min_split_gain': 0.040760399016915305}. Best is trial 1 with value: 0.0009755167559093886.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:43:52,983] Trial 2 finished with value: 0.0010825140299056165 and parameters: {'n_estimators': 257, 'learning_rate': 0.013906486977108437, 'num_leaves': 213, 'max_depth': 8, 'min_child_samples': 12, 'subsample': 0.8934839316234442, 'colsample_bytree': 0.9856124801682811, 'reg_alpha': 8.7903806226446, 'reg_lambda': 8.453253418442465, 'min_split_gain': 0.048838499305848486}. Best is trial 1 with value: 0.0009755167559093886.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:43:54,197] Trial 3 finished with value: 0.0010948980941116685 and parameters: {'n_estimators': 254, 'learning_rate': 0.07825267105921671, 'num_leaves': 234, 'max_depth': 8, 'min_child_samples': 16, 'subsample': 0.8769788387596509, 'colsample_bytree': 0.7554349757442906, 'reg_alpha': 8.942950752615657, 'reg_lambda': 4.784651820368449, 'min_split_gain': 0.0035007433943481884}. Best is trial 1 with value: 0.0009755167559093886.
[I 2025-06-15 20:43:54,200] A new study created in memory with name: no-name-b4342bde-a11b-4ec0-ad35-c2c540856707
[I 2025-06-15 20:43:54,270] Trial 0 finished with value: 0.0035353683675786334 and parameters: {'n_estimators': 244, 'learning_rate': 0.05200101272891202, 'num_leaves': 60, 'max_depth': 5, 'min_child_samples': 18, 'subsample': 0.5565792980797809, 'colsample_bytree': 0.5394609551546998, 'reg_alpha': 6.226350974854862, 'reg_lambda': 4.608310301029862, 'min_split_gain': 0.03761389922058848}. Best is trial 0 with value: 0.0035353683675786334.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         662
1           Price_lag_2         537
14      pct_chg_Load_DA         346
3         Load_DA_lag_0         296
11           NGas_lag_2         240
15       lag168_Load_DA         227
16  roll2d_median_Price         207
2           Price_lag_7         192
5       WindOn_DA_lag_0         191
13            EUA_lag_2         131
12            Oil_lag_2         122
10           Coal_lag_2         115
9                  WD_7          11
8                  WD_6           9
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 89.19% done
********************* END NS.. 651 ****************************************************
********************* START NS ... 652 ****************************************************
[I 2025-06-15 20:43:56,409] A new study created in memory with name: no-name-e547ef5a-fc61-49cb-991e-20a369a91d1d
[I 2025-06-15 20:43:57,483] Trial 0 finished with value: 0.0032729815238196317 and parameters: {'n_estimators': 171, 'learning_rate': 0.08239049165612905, 'num_leaves': 216, 'max_depth': 14, 'min_child_samples': 25, 'subsample': 0.6681092712042807, 'colsample_bytree': 0.9315366640310747, 'reg_alpha': 4.444772008383422, 'reg_lambda': 1.65307072829031, 'min_split_gain': 0.006078993936481986}. Best is trial 0 with value: 0.0032729815238196317.
[Trial 0] New best: 0.00327
[I 2025-06-15 20:43:57,897] Trial 1 finished with value: 0.0035899699869651213 and parameters: {'n_estimators': 169, 'learning_rate': 0.21475051743442453, 'num_leaves': 48, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.8766981963829342, 'colsample_bytree': 0.9946697041451085, 'reg_alpha': 1.0382040845107954, 'reg_lambda': 2.9365112893191037, 'min_split_gain': 0.026816473868839553}. Best is trial 0 with value: 0.0032729815238196317.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:43:58,173] Trial 2 finished with value: 0.0031481759444837542 and parameters: {'n_estimators': 194, 'learning_rate': 0.24303403538979368, 'num_leaves': 73, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.6597166553890583, 'colsample_bytree': 0.8907232744552253, 'reg_alpha': 3.275163365289666, 'reg_lambda': 0.07353790680229011, 'min_split_gain': 0.04751690677316113}. Best is trial 2 with value: 0.0031481759444837542.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:43:59,209] Trial 3 finished with value: 0.0033834741635401615 and parameters: {'n_estimators': 242, 'learning_rate': 0.1949432681294774, 'num_leaves': 174, 'max_depth': 16, 'min_child_samples': 12, 'subsample': 0.6259209292196972, 'colsample_bytree': 0.6752960371289984, 'reg_alpha': 0.4860463001621651, 'reg_lambda': 9.011852607634971, 'min_split_gain': 0.02012754760695741}. Best is trial 2 with value: 0.0031481759444837542.
[I 2025-06-15 20:43:59,211] A new study created in memory with name: no-name-9dd9c088-fdba-4953-96ef-c4a29766c94c
[I 2025-06-15 20:43:59,251] Trial 0 finished with value: 0.0511297565191411 and parameters: {'n_estimators': 158, 'learning_rate': 0.06093643008994662, 'num_leaves': 178, 'max_depth': 9, 'min_child_samples': 24, 'subsample': 0.5211794669638128, 'colsample_bytree': 0.6959202349354576, 'reg_alpha': 5.482006784900422, 'reg_lambda': 3.1870910762512974, 'min_split_gain': 0.009079518029284018}. Best is trial 0 with value: 0.0511297565191411.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         209
14      pct_chg_Load_DA         121
1           Price_lag_2         106
3         Load_DA_lag_0          61
2           Price_lag_7          53
16  roll2d_median_Price          51
11           NGas_lag_2          44
15       lag168_Load_DA          32
5       WindOn_DA_lag_0          26
10           Coal_lag_2          26
13            EUA_lag_2          25
12            Oil_lag_2          11
7                  WD_1           2
9                  WD_7           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           9
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 89.33% done
********************* END NS.. 652 ****************************************************
********************* START NS ... 653 ****************************************************
[I 2025-06-15 20:44:01,071] A new study created in memory with name: no-name-51166b95-2adb-498c-ba35-56a41255feff
[I 2025-06-15 20:44:02,106] Trial 0 finished with value: 0.004898193074467654 and parameters: {'n_estimators': 176, 'learning_rate': 0.01760191305207007, 'num_leaves': 33, 'max_depth': 8, 'min_child_samples': 18, 'subsample': 0.5523789880778475, 'colsample_bytree': 0.6804978349866082, 'reg_alpha': 6.267663544898733, 'reg_lambda': 2.8758223295342544, 'min_split_gain': 0.020327476550407255}. Best is trial 0 with value: 0.004898193074467654.
[Trial 0] New best: 0.00490
[I 2025-06-15 20:44:02,543] Trial 1 finished with value: 0.0027617907072836722 and parameters: {'n_estimators': 188, 'learning_rate': 0.10958751128604034, 'num_leaves': 203, 'max_depth': 9, 'min_child_samples': 21, 'subsample': 0.7095438330309347, 'colsample_bytree': 0.8456654341320642, 'reg_alpha': 9.290735111213998, 'reg_lambda': 0.1312120026014829, 'min_split_gain': 0.028401021379674707}. Best is trial 1 with value: 0.0027617907072836722.
[Trial 1] New best: 0.00276
[I 2025-06-15 20:44:03,020] Trial 2 finished with value: 0.002842063233507918 and parameters: {'n_estimators': 210, 'learning_rate': 0.13193830868031964, 'num_leaves': 64, 'max_depth': 15, 'min_child_samples': 20, 'subsample': 0.6013220672452626, 'colsample_bytree': 0.8593550984578725, 'reg_alpha': 7.129258375703383, 'reg_lambda': 9.082923093053614, 'min_split_gain': 0.02036312480703257}. Best is trial 1 with value: 0.0027617907072836722.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:44:03,361] Trial 3 finished with value: 0.002857797439125325 and parameters: {'n_estimators': 175, 'learning_rate': 0.24625662564606732, 'num_leaves': 163, 'max_depth': 7, 'min_child_samples': 15, 'subsample': 0.7367375902578959, 'colsample_bytree': 0.7244996317017048, 'reg_alpha': 1.3394907565248027, 'reg_lambda': 7.595374079000356, 'min_split_gain': 0.046221100934162235}. Best is trial 1 with value: 0.0027617907072836722.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:44:05,267] Trial 4 finished with value: 0.00285678416620445 and parameters: {'n_estimators': 223, 'learning_rate': 0.030389736986115187, 'num_leaves': 92, 'max_depth': 6, 'min_child_samples': 8, 'subsample': 0.8204844130373391, 'colsample_bytree': 0.8428745592245528, 'reg_alpha': 3.0118408969964596, 'reg_lambda': 0.7207332415538104, 'min_split_gain': 0.012167547915244515}. Best is trial 1 with value: 0.0027617907072836722.
[I 2025-06-15 20:44:05,270] A new study created in memory with name: no-name-dfc84915-1ad2-4c69-9316-f2104d48792e
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 20:44:05,464] Trial 0 finished with value: 0.04694529111937351 and parameters: {'n_estimators': 229, 'learning_rate': 0.02679639650928908, 'num_leaves': 237, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.6437711887784396, 'colsample_bytree': 0.500119042061488, 'reg_alpha': 7.070256095939909, 'reg_lambda': 6.096959520959557, 'min_split_gain': 0.019656838351753594}. Best is trial 0 with value: 0.04694529111937351.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         396
14      pct_chg_Load_DA         232
1           Price_lag_2         186
3         Load_DA_lag_0         115
11           NGas_lag_2         101
2           Price_lag_7          99
16  roll2d_median_Price          88
15       lag168_Load_DA          60
10           Coal_lag_2          56
5       WindOn_DA_lag_0          48
13            EUA_lag_2          46
12            Oil_lag_2          29
9                  WD_7          11
7                  WD_1           4
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 89.47% done
********************* END NS.. 653 ****************************************************
********************* START NS ... 654 ****************************************************
[I 2025-06-15 20:44:07,248] A new study created in memory with name: no-name-937f5f4b-62e1-4539-966e-f2312ad4e7ca
[I 2025-06-15 20:44:07,957] Trial 0 finished with value: 0.025491371665125665 and parameters: {'n_estimators': 229, 'learning_rate': 0.07083206160840971, 'num_leaves': 102, 'max_depth': 8, 'min_child_samples': 3, 'subsample': 0.7875288968513872, 'colsample_bytree': 0.8877816684247244, 'reg_alpha': 4.410138863135796, 'reg_lambda': 4.433299910840253, 'min_split_gain': 0.038997862723187365}. Best is trial 0 with value: 0.025491371665125665.
[Trial 0] New best: 0.02549
[I 2025-06-15 20:44:09,202] Trial 1 finished with value: 0.02778746660777941 and parameters: {'n_estimators': 149, 'learning_rate': 0.04316482876508517, 'num_leaves': 172, 'max_depth': 6, 'min_child_samples': 29, 'subsample': 0.6220425444411136, 'colsample_bytree': 0.6385831386596914, 'reg_alpha': 0.39064748379958214, 'reg_lambda': 6.243076466359656, 'min_split_gain': 0.02039537646155852}. Best is trial 0 with value: 0.025491371665125665.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:44:09,590] Trial 2 finished with value: 0.018563942489394605 and parameters: {'n_estimators': 179, 'learning_rate': 0.15072724351598382, 'num_leaves': 181, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.7235311142247899, 'colsample_bytree': 0.9398026059249098, 'reg_alpha': 4.228325153553946, 'reg_lambda': 7.584713298773416, 'min_split_gain': 0.011755621449288524}. Best is trial 2 with value: 0.018563942489394605.
[Trial 2] New best: 0.01856
[I 2025-06-15 20:44:11,096] Trial 3 finished with value: 0.028549685890981596 and parameters: {'n_estimators': 211, 'learning_rate': 0.02893729983869191, 'num_leaves': 52, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.635218892713425, 'colsample_bytree': 0.6482302429651179, 'reg_alpha': 2.6956510974583017, 'reg_lambda': 7.161945558107145, 'min_split_gain': 0.006212767121355417}. Best is trial 2 with value: 0.018563942489394605.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:44:13,041] Trial 4 finished with value: 0.02945017636273689 and parameters: {'n_estimators': 178, 'learning_rate': 0.030247664407260663, 'num_leaves': 144, 'max_depth': 8, 'min_child_samples': 28, 'subsample': 0.7713000863833199, 'colsample_bytree': 0.5525972145444134, 'reg_alpha': 2.817886076489372, 'reg_lambda': 0.8028425002667672, 'min_split_gain': 0.0337596996919601}. Best is trial 2 with value: 0.018563942489394605.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:44:13,732] Trial 5 finished with value: 0.028933178490731973 and parameters: {'n_estimators': 133, 'learning_rate': 0.1315515654689612, 'num_leaves': 87, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.6447057654351633, 'colsample_bytree': 0.7162184259218083, 'reg_alpha': 4.378742287433211, 'reg_lambda': 3.6126858876374337, 'min_split_gain': 0.022223664306253083}. Best is trial 2 with value: 0.018563942489394605.
[I 2025-06-15 20:44:13,734] A new study created in memory with name: no-name-0f378295-720f-4555-96c9-54036880fd46
[I 2025-06-15 20:44:13,781] Trial 0 finished with value: 0.08440981746982339 and parameters: {'n_estimators': 194, 'learning_rate': 0.04028630230135804, 'num_leaves': 107, 'max_depth': 4, 'min_child_samples': 17, 'subsample': 0.6906716626351352, 'colsample_bytree': 0.7343179720926021, 'reg_alpha': 8.608669364427673, 'reg_lambda': 9.601038315393758, 'min_split_gain': 0.005193357282496797}. Best is trial 0 with value: 0.08440981746982339.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         375
14      pct_chg_Load_DA         206
1           Price_lag_2         196
3         Load_DA_lag_0         168
11           NGas_lag_2         100
16  roll2d_median_Price          99
2           Price_lag_7          97
15       lag168_Load_DA          88
5       WindOn_DA_lag_0          79
13            EUA_lag_2          67
10           Coal_lag_2          56
12            Oil_lag_2          38
7                  WD_1           5
9                  WD_7           5
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 89.60% done
********************* END NS.. 654 ****************************************************
********************* START NS ... 655 ****************************************************
[I 2025-06-15 20:44:15,568] A new study created in memory with name: no-name-f09dd630-af59-4d8b-9a9d-46a034c7a654
[I 2025-06-15 20:44:15,864] Trial 0 finished with value: 0.023933049769622943 and parameters: {'n_estimators': 156, 'learning_rate': 0.25819407504126807, 'num_leaves': 75, 'max_depth': 4, 'min_child_samples': 23, 'subsample': 0.505162945735732, 'colsample_bytree': 0.9163518494070941, 'reg_alpha': 1.1073411996169025, 'reg_lambda': 9.582544463475905, 'min_split_gain': 0.014561177079001442}. Best is trial 0 with value: 0.023933049769622943.
[Trial 0] New best: 0.02393
[I 2025-06-15 20:44:16,646] Trial 1 finished with value: 0.03027155301678547 and parameters: {'n_estimators': 61, 'learning_rate': 0.252764621581244, 'num_leaves': 100, 'max_depth': 7, 'min_child_samples': 11, 'subsample': 0.7589014525212374, 'colsample_bytree': 0.5480414398327583, 'reg_alpha': 6.07887563248029, 'reg_lambda': 6.769675719796268, 'min_split_gain': 0.03474283924885504}. Best is trial 0 with value: 0.023933049769622943.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:44:17,573] Trial 2 finished with value: 0.03134890668228077 and parameters: {'n_estimators': 180, 'learning_rate': 0.1283980424762612, 'num_leaves': 107, 'max_depth': 7, 'min_child_samples': 1, 'subsample': 0.7421116073177396, 'colsample_bytree': 0.6765836033917765, 'reg_alpha': 0.31991103427261525, 'reg_lambda': 4.905749828628241, 'min_split_gain': 0.02254298968646636}. Best is trial 0 with value: 0.023933049769622943.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:44:18,357] Trial 3 finished with value: 0.027932989858576507 and parameters: {'n_estimators': 142, 'learning_rate': 0.18204284872971652, 'num_leaves': 173, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.8372894115409197, 'colsample_bytree': 0.5333286948019207, 'reg_alpha': 5.021350567965043, 'reg_lambda': 2.324603395515039, 'min_split_gain': 0.007796048097435993}. Best is trial 0 with value: 0.023933049769622943.
[I 2025-06-15 20:44:18,360] A new study created in memory with name: no-name-fa071683-db0d-48b4-b0f3-45fb4b50a1ea
[I 2025-06-15 20:44:18,397] Trial 0 finished with value: 0.09069296071482319 and parameters: {'n_estimators': 139, 'learning_rate': 0.14910194555328246, 'num_leaves': 248, 'max_depth': 14, 'min_child_samples': 9, 'subsample': 0.7086254335006672, 'colsample_bytree': 0.6857991795270373, 'reg_alpha': 5.707886991605696, 'reg_lambda': 3.626483687045777, 'min_split_gain': 0.02874541945745115}. Best is trial 0 with value: 0.09069296071482319.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         275
1           Price_lag_2         146
14      pct_chg_Load_DA         130
3         Load_DA_lag_0          82
16  roll2d_median_Price          76
11           NGas_lag_2          70
2           Price_lag_7          64
5       WindOn_DA_lag_0          57
15       lag168_Load_DA          53
10           Coal_lag_2          45
12            Oil_lag_2          39
13            EUA_lag_2          36
9                  WD_7           6
7                  WD_1           4
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          11
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 89.74% done
********************* END NS.. 655 ****************************************************
********************* START NS ... 656 ****************************************************
[I 2025-06-15 20:44:20,220] A new study created in memory with name: no-name-dcc1a20c-fe02-4b44-9369-6a5e319b5175
[I 2025-06-15 20:44:20,906] Trial 0 finished with value: 0.0036932931885416613 and parameters: {'n_estimators': 152, 'learning_rate': 0.21381630837083876, 'num_leaves': 171, 'max_depth': 16, 'min_child_samples': 21, 'subsample': 0.8106898845619768, 'colsample_bytree': 0.7712113296939769, 'reg_alpha': 1.9857726736103098, 'reg_lambda': 2.112130707038946, 'min_split_gain': 0.003948275293065867}. Best is trial 0 with value: 0.0036932931885416613.
[Trial 0] New best: 0.00369
[I 2025-06-15 20:44:22,687] Trial 1 finished with value: 0.00527906138963798 and parameters: {'n_estimators': 236, 'learning_rate': 0.03419014787200062, 'num_leaves': 205, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.9072446908007668, 'colsample_bytree': 0.5019235955941295, 'reg_alpha': 0.004050570988711755, 'reg_lambda': 6.815725373797987, 'min_split_gain': 0.03781060324622295}. Best is trial 0 with value: 0.0036932931885416613.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:44:24,008] Trial 2 finished with value: 0.06208844661884456 and parameters: {'n_estimators': 107, 'learning_rate': 0.011347056514907097, 'num_leaves': 191, 'max_depth': 9, 'min_child_samples': 6, 'subsample': 0.7405976193403379, 'colsample_bytree': 0.785859525377095, 'reg_alpha': 2.771661235196831, 'reg_lambda': 7.535062804499332, 'min_split_gain': 0.012942361419112286}. Best is trial 0 with value: 0.0036932931885416613.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:44:26,578] Trial 3 finished with value: 0.0066207533904055526 and parameters: {'n_estimators': 286, 'learning_rate': 0.015444261992838996, 'num_leaves': 127, 'max_depth': 14, 'min_child_samples': 24, 'subsample': 0.8431994698404213, 'colsample_bytree': 0.6874060505242409, 'reg_alpha': 7.246595799530304, 'reg_lambda': 8.93780966667048, 'min_split_gain': 0.02057297764405431}. Best is trial 0 with value: 0.0036932931885416613.
[I 2025-06-15 20:44:26,580] A new study created in memory with name: no-name-3d26b51c-2e98-4ac0-9832-76043a72038c
[I 2025-06-15 20:44:26,637] Trial 0 finished with value: 0.02961755416311417 and parameters: {'n_estimators': 219, 'learning_rate': 0.07664673380310992, 'num_leaves': 146, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.9481751159882001, 'colsample_bytree': 0.8826453969059971, 'reg_alpha': 9.589295764473833, 'reg_lambda': 3.5329940742877, 'min_split_gain': 0.048205927499030285}. Best is trial 0 with value: 0.02961755416311417.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         598
1           Price_lag_2         543
14      pct_chg_Load_DA         495
3         Load_DA_lag_0         442
15       lag168_Load_DA         383
5       WindOn_DA_lag_0         273
2           Price_lag_7         257
13            EUA_lag_2         245
16  roll2d_median_Price         239
11           NGas_lag_2         222
12            Oil_lag_2         210
10           Coal_lag_2         157
9                  WD_7          20
7                  WD_1           9
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 89.88% done
********************* END NS.. 656 ****************************************************
********************* START NS ... 657 ****************************************************
[I 2025-06-15 20:44:28,855] A new study created in memory with name: no-name-27f9b2fd-57ef-42cd-9682-1d92b636ab9f
[I 2025-06-15 20:44:29,597] Trial 0 finished with value: 0.06323966659671365 and parameters: {'n_estimators': 84, 'learning_rate': 0.012482444645559488, 'num_leaves': 35, 'max_depth': 13, 'min_child_samples': 11, 'subsample': 0.7074808191916591, 'colsample_bytree': 0.5779636786470252, 'reg_alpha': 6.866927346063951, 'reg_lambda': 8.849783871128862, 'min_split_gain': 0.009967397595353667}. Best is trial 0 with value: 0.06323966659671365.
[Trial 0] New best: 0.06324
[I 2025-06-15 20:44:31,004] Trial 1 finished with value: 0.0016860478473218511 and parameters: {'n_estimators': 182, 'learning_rate': 0.020816223903223623, 'num_leaves': 233, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.7589502756604185, 'colsample_bytree': 0.5893698893109284, 'reg_alpha': 8.80904268730433, 'reg_lambda': 6.603753261435706, 'min_split_gain': 0.010563994361479158}. Best is trial 1 with value: 0.0016860478473218511.
[Trial 1] New best: 0.00169
[I 2025-06-15 20:44:31,564] Trial 2 finished with value: 0.0013072906833899373 and parameters: {'n_estimators': 54, 'learning_rate': 0.1757036024560261, 'num_leaves': 53, 'max_depth': 12, 'min_child_samples': 26, 'subsample': 0.6884854300668979, 'colsample_bytree': 0.9948895518121539, 'reg_alpha': 2.6972781874191343, 'reg_lambda': 4.510233910416945, 'min_split_gain': 0.010557589805889762}. Best is trial 2 with value: 0.0013072906833899373.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:44:32,558] Trial 3 finished with value: 0.0014972781768047506 and parameters: {'n_estimators': 166, 'learning_rate': 0.044163812818255294, 'num_leaves': 156, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.7879693797875242, 'colsample_bytree': 0.5273568867822818, 'reg_alpha': 9.593592447499681, 'reg_lambda': 2.5544838446861675, 'min_split_gain': 0.03132333898081606}. Best is trial 2 with value: 0.0013072906833899373.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:44:33,106] Trial 4 finished with value: 0.0012476420382660535 and parameters: {'n_estimators': 296, 'learning_rate': 0.18601663969286295, 'num_leaves': 214, 'max_depth': 8, 'min_child_samples': 13, 'subsample': 0.6613237728996173, 'colsample_bytree': 0.9174091035694036, 'reg_alpha': 6.2113860455789185, 'reg_lambda': 1.460134538593757, 'min_split_gain': 0.04748348922539411}. Best is trial 4 with value: 0.0012476420382660535.
[I 2025-06-15 20:44:33,108] A new study created in memory with name: no-name-8e88a4d6-bacc-40f1-b52b-3935ea8c79a1
[I 2025-06-15 20:44:33,190] Trial 0 finished with value: 0.008869338584401592 and parameters: {'n_estimators': 257, 'learning_rate': 0.10503352235267045, 'num_leaves': 182, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.8191235794463458, 'colsample_bytree': 0.5699469918008024, 'reg_alpha': 9.849663172106336, 'reg_lambda': 6.587206912607522, 'min_split_gain': 0.008144494301249866}. Best is trial 0 with value: 0.008869338584401592.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         253
14      pct_chg_Load_DA         153
1           Price_lag_2         110
2           Price_lag_7          73
3         Load_DA_lag_0          72
16  roll2d_median_Price          68
11           NGas_lag_2          65
15       lag168_Load_DA          34
13            EUA_lag_2          25
5       WindOn_DA_lag_0          22
10           Coal_lag_2          22
12            Oil_lag_2          13
9                  WD_7           3
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 90.01% done
********************* END NS.. 657 ****************************************************
********************* START NS ... 658 ****************************************************
[I 2025-06-15 20:44:35,140] A new study created in memory with name: no-name-a06317fc-dbac-46aa-9f99-14257789b163
[I 2025-06-15 20:44:35,359] Trial 0 finished with value: 0.000471866882716861 and parameters: {'n_estimators': 269, 'learning_rate': 0.1810275189161022, 'num_leaves': 73, 'max_depth': 4, 'min_child_samples': 30, 'subsample': 0.634698405996875, 'colsample_bytree': 0.6563838818227361, 'reg_alpha': 7.287538604422846, 'reg_lambda': 3.4983917934120523, 'min_split_gain': 0.025501285609595845}. Best is trial 0 with value: 0.000471866882716861.
[Trial 0] New best: 0.00047
[I 2025-06-15 20:44:35,961] Trial 1 finished with value: 0.019859137821385497 and parameters: {'n_estimators': 55, 'learning_rate': 0.028075163818694692, 'num_leaves': 59, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.57041524335094, 'colsample_bytree': 0.8298138180513193, 'reg_alpha': 2.5550252691658537, 'reg_lambda': 1.4371470419357024, 'min_split_gain': 0.004495449397554563}. Best is trial 0 with value: 0.000471866882716861.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:44:37,440] Trial 2 finished with value: 0.000324516288253369 and parameters: {'n_estimators': 175, 'learning_rate': 0.03511825824860713, 'num_leaves': 51, 'max_depth': 16, 'min_child_samples': 2, 'subsample': 0.5449938286125386, 'colsample_bytree': 0.7993678056129385, 'reg_alpha': 6.71614074358999, 'reg_lambda': 0.25403272532175336, 'min_split_gain': 0.04433641473222201}. Best is trial 2 with value: 0.000324516288253369.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:44:37,932] Trial 3 finished with value: 0.0003760043017014319 and parameters: {'n_estimators': 136, 'learning_rate': 0.1752396208633178, 'num_leaves': 160, 'max_depth': 11, 'min_child_samples': 11, 'subsample': 0.7550848302578058, 'colsample_bytree': 0.9862177906638019, 'reg_alpha': 8.834185866517826, 'reg_lambda': 0.4514648759875728, 'min_split_gain': 0.040582780423758874}. Best is trial 2 with value: 0.000324516288253369.
[I 2025-06-15 20:44:37,934] A new study created in memory with name: no-name-0ac978f6-c168-49aa-99ff-be7de4e002ee
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 20:44:38,200] Trial 0 finished with value: 0.0005229476677263989 and parameters: {'n_estimators': 183, 'learning_rate': 0.11846864569771706, 'num_leaves': 167, 'max_depth': 5, 'min_child_samples': 18, 'subsample': 0.9901167935043926, 'colsample_bytree': 0.9640841577861431, 'reg_alpha': 1.8272199095828245, 'reg_lambda': 4.939105962484058, 'min_split_gain': 0.025507566190344513}. Best is trial 0 with value: 0.0005229476677263989.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1315
1           Price_lag_2         947
14      pct_chg_Load_DA         720
2           Price_lag_7         443
3         Load_DA_lag_0         405
11           NGas_lag_2         376
16  roll2d_median_Price         313
10           Coal_lag_2         197
5       WindOn_DA_lag_0         152
12            Oil_lag_2         151
15       lag168_Load_DA         127
13            EUA_lag_2         124
9                  WD_7          26
8                  WD_6          19
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          35
14      pct_chg_Load_DA           7
15       lag168_Load_DA           3
5       WindOn_DA_lag_0           3
1           Price_lag_2           1
10           Coal_lag_2           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 90.15% done
********************* END NS.. 658 ****************************************************
********************* START NS ... 659 ****************************************************
[I 2025-06-15 20:44:40,542] A new study created in memory with name: no-name-eca19ecb-0aa8-43f7-ae7a-8963f461c68d
[I 2025-06-15 20:44:43,329] Trial 0 finished with value: 0.0037349387323799693 and parameters: {'n_estimators': 256, 'learning_rate': 0.036698500680358424, 'num_leaves': 72, 'max_depth': 14, 'min_child_samples': 6, 'subsample': 0.8630085500396749, 'colsample_bytree': 0.9819448549123759, 'reg_alpha': 2.065283492198126, 'reg_lambda': 1.1546939987875005, 'min_split_gain': 0.015412958521223624}. Best is trial 0 with value: 0.0037349387323799693.
[Trial 0] New best: 0.00373
[I 2025-06-15 20:44:44,494] Trial 1 finished with value: 0.0042488440717378135 and parameters: {'n_estimators': 190, 'learning_rate': 0.08393507441019239, 'num_leaves': 228, 'max_depth': 16, 'min_child_samples': 4, 'subsample': 0.6114194880355968, 'colsample_bytree': 0.5585462532669948, 'reg_alpha': 4.230244007547038, 'reg_lambda': 2.473146648539487, 'min_split_gain': 0.017545802915549337}. Best is trial 0 with value: 0.0037349387323799693.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:44:46,258] Trial 2 finished with value: 0.003319306111378772 and parameters: {'n_estimators': 282, 'learning_rate': 0.07575976130400255, 'num_leaves': 101, 'max_depth': 6, 'min_child_samples': 24, 'subsample': 0.518291179459548, 'colsample_bytree': 0.8232835430421512, 'reg_alpha': 1.1745620918272937, 'reg_lambda': 2.3150728669531384, 'min_split_gain': 0.0039376478367339155}. Best is trial 2 with value: 0.003319306111378772.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:44:46,840] Trial 3 finished with value: 0.004271430917167446 and parameters: {'n_estimators': 206, 'learning_rate': 0.2740012714987634, 'num_leaves': 53, 'max_depth': 7, 'min_child_samples': 17, 'subsample': 0.7399281625876845, 'colsample_bytree': 0.5581066138299997, 'reg_alpha': 3.321308215289979, 'reg_lambda': 6.88919772858114, 'min_split_gain': 0.049310541981983744}. Best is trial 2 with value: 0.003319306111378772.
[I 2025-06-15 20:44:46,842] A new study created in memory with name: no-name-46061862-f07b-42d9-9eb5-d868742916a1
[I 2025-06-15 20:44:46,865] Trial 0 finished with value: 0.026191702335937418 and parameters: {'n_estimators': 53, 'learning_rate': 0.21254348779782023, 'num_leaves': 238, 'max_depth': 4, 'min_child_samples': 17, 'subsample': 0.5113035689128087, 'colsample_bytree': 0.9325696447378208, 'reg_alpha': 6.987101144491205, 'reg_lambda': 6.169783602965122, 'min_split_gain': 0.02032799710040807}. Best is trial 0 with value: 0.026191702335937418.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1386
1           Price_lag_2        1060
14      pct_chg_Load_DA         984
3         Load_DA_lag_0         678
15       lag168_Load_DA         562
5       WindOn_DA_lag_0         547
2           Price_lag_7         487
16  roll2d_median_Price         481
11           NGas_lag_2         464
13            EUA_lag_2         430
12            Oil_lag_2         368
10           Coal_lag_2         302
9                  WD_7          31
8                  WD_6          16
7                  WD_1          14
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           2
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 90.29% done
********************* END NS.. 659 ****************************************************
********************* START NS ... 660 ****************************************************
[I 2025-06-15 20:44:49,634] A new study created in memory with name: no-name-99497961-2ad3-49c8-8bab-8584f834f800
[I 2025-06-15 20:44:50,126] Trial 0 finished with value: 0.005050132443870782 and parameters: {'n_estimators': 102, 'learning_rate': 0.10907564484342505, 'num_leaves': 115, 'max_depth': 5, 'min_child_samples': 21, 'subsample': 0.5351506509543597, 'colsample_bytree': 0.9120171771054065, 'reg_alpha': 4.5525297768880115, 'reg_lambda': 6.511685099567437, 'min_split_gain': 0.0026163684700059643}. Best is trial 0 with value: 0.005050132443870782.
[Trial 0] New best: 0.00505
[I 2025-06-15 20:44:52,453] Trial 1 finished with value: 0.006346645498638201 and parameters: {'n_estimators': 229, 'learning_rate': 0.015686238382611586, 'num_leaves': 195, 'max_depth': 7, 'min_child_samples': 13, 'subsample': 0.9137403479468518, 'colsample_bytree': 0.6674840330207265, 'reg_alpha': 3.494662478157282, 'reg_lambda': 1.3157433831540999, 'min_split_gain': 0.011676033391976082}. Best is trial 0 with value: 0.005050132443870782.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:44:55,142] Trial 2 finished with value: 0.005201402990084116 and parameters: {'n_estimators': 249, 'learning_rate': 0.043856049776478355, 'num_leaves': 102, 'max_depth': 16, 'min_child_samples': 10, 'subsample': 0.7479729814999696, 'colsample_bytree': 0.5483298940245295, 'reg_alpha': 3.7464465378683434, 'reg_lambda': 6.855263651129434, 'min_split_gain': 0.013657646963325498}. Best is trial 0 with value: 0.005050132443870782.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:44:58,126] Trial 3 finished with value: 0.00820856582194966 and parameters: {'n_estimators': 290, 'learning_rate': 0.010012392133203422, 'num_leaves': 147, 'max_depth': 8, 'min_child_samples': 7, 'subsample': 0.9089252829077086, 'colsample_bytree': 0.7306692725552635, 'reg_alpha': 3.132096157281322, 'reg_lambda': 5.585170323595032, 'min_split_gain': 0.03586730852686489}. Best is trial 0 with value: 0.005050132443870782.
[I 2025-06-15 20:44:58,128] A new study created in memory with name: no-name-24e5cd57-bf19-40d4-ba94-a2c1a97c28e6
[I 2025-06-15 20:44:58,179] Trial 0 finished with value: 0.04693431242167604 and parameters: {'n_estimators': 226, 'learning_rate': 0.020443185637637952, 'num_leaves': 242, 'max_depth': 12, 'min_child_samples': 25, 'subsample': 0.9920372293992613, 'colsample_bytree': 0.6991175835292137, 'reg_alpha': 6.780410809797802, 'reg_lambda': 7.26048017782114, 'min_split_gain': 0.02132815313497535}. Best is trial 0 with value: 0.04693431242167604.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         510
14      pct_chg_Load_DA         334
1           Price_lag_2         254
3         Load_DA_lag_0         193
13            EUA_lag_2         136
5       WindOn_DA_lag_0         134
2           Price_lag_7         130
16  roll2d_median_Price         122
11           NGas_lag_2         121
15       lag168_Load_DA         115
12            Oil_lag_2         111
10           Coal_lag_2          81
9                  WD_7          16
7                  WD_1           7
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          56
1           Price_lag_2           6
5       WindOn_DA_lag_0           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 90.42% done
********************* END NS.. 660 ****************************************************
********************* START NS ... 661 ****************************************************
[I 2025-06-15 20:44:59,909] A new study created in memory with name: no-name-3cd06847-15e0-40ae-9f6e-6f1ee2f5711c
[I 2025-06-15 20:45:00,624] Trial 0 finished with value: 0.001280215709787773 and parameters: {'n_estimators': 271, 'learning_rate': 0.11113266394214018, 'num_leaves': 217, 'max_depth': 7, 'min_child_samples': 17, 'subsample': 0.9528041039606945, 'colsample_bytree': 0.6311672291371775, 'reg_alpha': 2.515384867445236, 'reg_lambda': 7.988702072104706, 'min_split_gain': 0.031196403117423332}. Best is trial 0 with value: 0.001280215709787773.
[Trial 0] New best: 0.00128
[I 2025-06-15 20:45:01,432] Trial 1 finished with value: 0.0009394247281378989 and parameters: {'n_estimators': 247, 'learning_rate': 0.036126351125460116, 'num_leaves': 236, 'max_depth': 4, 'min_child_samples': 28, 'subsample': 0.6736329932403567, 'colsample_bytree': 0.7347688426152008, 'reg_alpha': 5.659848687083344, 'reg_lambda': 7.510782797364323, 'min_split_gain': 0.049333142129373786}. Best is trial 1 with value: 0.0009394247281378989.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:45:01,795] Trial 2 finished with value: 0.0013550487923109447 and parameters: {'n_estimators': 58, 'learning_rate': 0.1636573082014793, 'num_leaves': 75, 'max_depth': 10, 'min_child_samples': 29, 'subsample': 0.8290623306221239, 'colsample_bytree': 0.9889923336099476, 'reg_alpha': 8.601101686077053, 'reg_lambda': 9.63174416393986, 'min_split_gain': 0.03870449378417809}. Best is trial 1 with value: 0.0009394247281378989.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:45:02,640] Trial 3 finished with value: 0.0013104022116361962 and parameters: {'n_estimators': 101, 'learning_rate': 0.06660193758770083, 'num_leaves': 221, 'max_depth': 7, 'min_child_samples': 18, 'subsample': 0.639361362554703, 'colsample_bytree': 0.5834838721782927, 'reg_alpha': 3.9518119759752803, 'reg_lambda': 7.016763043998297, 'min_split_gain': 0.02053584024292314}. Best is trial 1 with value: 0.0009394247281378989.
[I 2025-06-15 20:45:02,643] A new study created in memory with name: no-name-973752dd-1dad-4527-b23f-258386f6b332
[I 2025-06-15 20:45:02,706] Trial 0 finished with value: 0.012293499114744559 and parameters: {'n_estimators': 215, 'learning_rate': 0.1211927842248018, 'num_leaves': 128, 'max_depth': 7, 'min_child_samples': 25, 'subsample': 0.9960470571893291, 'colsample_bytree': 0.6829810314345475, 'reg_alpha': 4.288412487380154, 'reg_lambda': 8.685764437191194, 'min_split_gain': 0.02061194709307468}. Best is trial 0 with value: 0.012293499114744559.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1000
14      pct_chg_Load_DA         432
1           Price_lag_2         388
3         Load_DA_lag_0         256
11           NGas_lag_2         156
16  roll2d_median_Price         131
2           Price_lag_7         121
5       WindOn_DA_lag_0         111
15       lag168_Load_DA          83
10           Coal_lag_2          76
13            EUA_lag_2          50
12            Oil_lag_2          42
9                  WD_7          16
7                  WD_1          12
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          16
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 90.56% done
********************* END NS.. 661 ****************************************************
********************* START NS ... 662 ****************************************************
[I 2025-06-15 20:45:05,353] A new study created in memory with name: no-name-e73b9f7b-30ee-4b97-95f0-e050786256e8
[I 2025-06-15 20:45:06,790] Trial 0 finished with value: 0.0002915625312038616 and parameters: {'n_estimators': 167, 'learning_rate': 0.11206003905691483, 'num_leaves': 131, 'max_depth': 9, 'min_child_samples': 1, 'subsample': 0.6528536735456218, 'colsample_bytree': 0.9447602455796202, 'reg_alpha': 0.3875143471021303, 'reg_lambda': 2.2567021601232504, 'min_split_gain': 0.019720689595760444}. Best is trial 0 with value: 0.0002915625312038616.
[Trial 0] New best: 0.00029
[I 2025-06-15 20:45:08,080] Trial 1 finished with value: 0.0004545750684450175 and parameters: {'n_estimators': 117, 'learning_rate': 0.03804887605522337, 'num_leaves': 134, 'max_depth': 7, 'min_child_samples': 3, 'subsample': 0.8413948300123781, 'colsample_bytree': 0.5824386367124988, 'reg_alpha': 5.328907833054524, 'reg_lambda': 4.833937333636229, 'min_split_gain': 0.016389574056571373}. Best is trial 0 with value: 0.0002915625312038616.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:45:09,401] Trial 2 finished with value: 0.0003747322913267093 and parameters: {'n_estimators': 200, 'learning_rate': 0.04181081163688655, 'num_leaves': 173, 'max_depth': 9, 'min_child_samples': 12, 'subsample': 0.6467610270859105, 'colsample_bytree': 0.8924047211349094, 'reg_alpha': 7.451578130860464, 'reg_lambda': 4.1176259862383935, 'min_split_gain': 0.03722093334701246}. Best is trial 0 with value: 0.0002915625312038616.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:45:10,147] Trial 3 finished with value: 0.0003633287015598607 and parameters: {'n_estimators': 238, 'learning_rate': 0.07087995088370953, 'num_leaves': 116, 'max_depth': 7, 'min_child_samples': 18, 'subsample': 0.939125835094709, 'colsample_bytree': 0.7582735226328732, 'reg_alpha': 6.646666039053777, 'reg_lambda': 8.873767882015871, 'min_split_gain': 0.03337501315363639}. Best is trial 0 with value: 0.0002915625312038616.
[I 2025-06-15 20:45:10,150] A new study created in memory with name: no-name-12f6793a-cab9-4a98-b136-ff18b734255d
[I 2025-06-15 20:45:10,200] Trial 0 finished with value: 0.00034600455505100154 and parameters: {'n_estimators': 163, 'learning_rate': 0.08564626260222619, 'num_leaves': 64, 'max_depth': 14, 'min_child_samples': 9, 'subsample': 0.7500447348614958, 'colsample_bytree': 0.8893439460510043, 'reg_alpha': 2.4814285153064986, 'reg_lambda': 0.5595934620819187, 'min_split_gain': 0.029363558809322995}. Best is trial 0 with value: 0.00034600455505100154.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA        1033
1           Price_lag_2         956
0           Price_lag_1         925
3         Load_DA_lag_0         662
16  roll2d_median_Price         580
5       WindOn_DA_lag_0         530
2           Price_lag_7         511
15       lag168_Load_DA         399
11           NGas_lag_2         370
10           Coal_lag_2         328
13            EUA_lag_2         275
12            Oil_lag_2         262
9                  WD_7          18
8                  WD_6          13
7                  WD_1           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          25
1           Price_lag_2           1
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 90.70% done
********************* END NS.. 662 ****************************************************
********************* START NS ... 663 ****************************************************
[I 2025-06-15 20:45:12,881] A new study created in memory with name: no-name-602b3ae4-ea8c-42c6-aeef-f4110b4b593b
[I 2025-06-15 20:45:13,138] Trial 0 finished with value: 0.0008852487493284474 and parameters: {'n_estimators': 200, 'learning_rate': 0.1622484095728967, 'num_leaves': 141, 'max_depth': 4, 'min_child_samples': 2, 'subsample': 0.9461302178735428, 'colsample_bytree': 0.7524425628411936, 'reg_alpha': 4.791305762596752, 'reg_lambda': 3.04868815749877, 'min_split_gain': 0.0094920357349367}. Best is trial 0 with value: 0.0008852487493284474.
[Trial 0] New best: 0.00089
[I 2025-06-15 20:45:14,428] Trial 1 finished with value: 0.0008030124507871214 and parameters: {'n_estimators': 146, 'learning_rate': 0.09958037459008687, 'num_leaves': 153, 'max_depth': 10, 'min_child_samples': 7, 'subsample': 0.7888167714911657, 'colsample_bytree': 0.9622062931525439, 'reg_alpha': 3.3487315851024446, 'reg_lambda': 8.992279984592997, 'min_split_gain': 0.007851009217218752}. Best is trial 1 with value: 0.0008030124507871214.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:45:17,374] Trial 2 finished with value: 0.0007935244668001575 and parameters: {'n_estimators': 295, 'learning_rate': 0.024221461895645023, 'num_leaves': 201, 'max_depth': 12, 'min_child_samples': 29, 'subsample': 0.964030581908158, 'colsample_bytree': 0.9352543259573882, 'reg_alpha': 7.4331977531397, 'reg_lambda': 0.36254678343972446, 'min_split_gain': 0.0062507582017434175}. Best is trial 2 with value: 0.0007935244668001575.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:45:17,917] Trial 3 finished with value: 0.0016027154869636309 and parameters: {'n_estimators': 51, 'learning_rate': 0.0792293595461737, 'num_leaves': 57, 'max_depth': 13, 'min_child_samples': 7, 'subsample': 0.9709150355731012, 'colsample_bytree': 0.5173369881327619, 'reg_alpha': 4.4641010646854475, 'reg_lambda': 5.118383405533336, 'min_split_gain': 0.016273368741512733}. Best is trial 2 with value: 0.0007935244668001575.
[I 2025-06-15 20:45:17,920] A new study created in memory with name: no-name-0b970160-6c84-4f46-8198-f66a3f7271ae
[I 2025-06-15 20:45:17,958] Trial 0 finished with value: 0.0044366141107043495 and parameters: {'n_estimators': 134, 'learning_rate': 0.07187357854760124, 'num_leaves': 152, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.541334987395357, 'colsample_bytree': 0.9104861070694878, 'reg_alpha': 5.659926347828631, 'reg_lambda': 1.059001261147513, 'min_split_gain': 0.002279149628050897}. Best is trial 0 with value: 0.0044366141107043495.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2240
14      pct_chg_Load_DA        1445
1           Price_lag_2        1249
3         Load_DA_lag_0         795
16  roll2d_median_Price         623
2           Price_lag_7         617
13            EUA_lag_2         546
5       WindOn_DA_lag_0         531
11           NGas_lag_2         512
10           Coal_lag_2         428
15       lag168_Load_DA         417
12            Oil_lag_2         364
8                  WD_6          35
9                  WD_7          29
7                  WD_1          15
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 90.83% done
********************* END NS.. 663 ****************************************************
********************* START NS ... 664 ****************************************************
[I 2025-06-15 20:45:21,429] A new study created in memory with name: no-name-fbab2ae6-8e83-4db7-8896-edcfc1bb56d2
[I 2025-06-15 20:45:22,200] Trial 0 finished with value: 0.0033849000320908336 and parameters: {'n_estimators': 98, 'learning_rate': 0.03645793144878243, 'num_leaves': 240, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.6194607438341371, 'colsample_bytree': 0.5364903166575051, 'reg_alpha': 7.661689462145899, 'reg_lambda': 8.66516113432247, 'min_split_gain': 0.01078516269699808}. Best is trial 0 with value: 0.0033849000320908336.
[Trial 0] New best: 0.00338
[I 2025-06-15 20:45:23,439] Trial 1 finished with value: 0.03569215921527821 and parameters: {'n_estimators': 147, 'learning_rate': 0.010311961870501119, 'num_leaves': 71, 'max_depth': 9, 'min_child_samples': 30, 'subsample': 0.8543314406681234, 'colsample_bytree': 0.8046476140547204, 'reg_alpha': 5.755823147772171, 'reg_lambda': 1.2335230081904835, 'min_split_gain': 0.0022700929852145723}. Best is trial 0 with value: 0.0033849000320908336.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:45:24,610] Trial 2 finished with value: 0.003940412643550025 and parameters: {'n_estimators': 148, 'learning_rate': 0.022644576309447752, 'num_leaves': 127, 'max_depth': 9, 'min_child_samples': 14, 'subsample': 0.7622549250289291, 'colsample_bytree': 0.5510819682195741, 'reg_alpha': 7.027540084143778, 'reg_lambda': 7.643441233822912, 'min_split_gain': 0.020455342267812118}. Best is trial 0 with value: 0.0033849000320908336.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:45:25,645] Trial 3 finished with value: 0.005601375610094522 and parameters: {'n_estimators': 204, 'learning_rate': 0.014262808105964185, 'num_leaves': 108, 'max_depth': 5, 'min_child_samples': 27, 'subsample': 0.651276851557347, 'colsample_bytree': 0.5459070460356411, 'reg_alpha': 1.8044180274507426, 'reg_lambda': 1.9403438874726187, 'min_split_gain': 0.0070152793039214905}. Best is trial 0 with value: 0.0033849000320908336.
[I 2025-06-15 20:45:25,647] A new study created in memory with name: no-name-6ae28cb8-081d-4dcd-b848-4560b57274d6
[I 2025-06-15 20:45:25,672] Trial 0 finished with value: 0.012902325141333723 and parameters: {'n_estimators': 53, 'learning_rate': 0.23318071470196022, 'num_leaves': 256, 'max_depth': 7, 'min_child_samples': 21, 'subsample': 0.791502107444048, 'colsample_bytree': 0.9178379692963623, 'reg_alpha': 4.671481984618869, 'reg_lambda': 4.749695032701813, 'min_split_gain': 0.007571217059659491}. Best is trial 0 with value: 0.012902325141333723.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         649
1           Price_lag_2         645
2           Price_lag_7         386
14      pct_chg_Load_DA         352
16  roll2d_median_Price         276
11           NGas_lag_2         263
3         Load_DA_lag_0         249
10           Coal_lag_2         179
13            EUA_lag_2         175
15       lag168_Load_DA         155
5       WindOn_DA_lag_0         120
12            Oil_lag_2         106
9                  WD_7          21
8                  WD_6          14
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           6
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 90.97% done
********************* END NS.. 664 ****************************************************
********************* START NS ... 665 ****************************************************
[I 2025-06-15 20:45:27,941] A new study created in memory with name: no-name-ce05fd51-f10f-411c-8eae-6feb36411c80
[I 2025-06-15 20:45:28,388] Trial 0 finished with value: 0.0014739952470778916 and parameters: {'n_estimators': 110, 'learning_rate': 0.12023794973041851, 'num_leaves': 194, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.5675138864823359, 'colsample_bytree': 0.6964886760447251, 'reg_alpha': 5.577039232469934, 'reg_lambda': 2.202236083782431, 'min_split_gain': 0.011017895535372158}. Best is trial 0 with value: 0.0014739952470778916.
[Trial 0] New best: 0.00147
[I 2025-06-15 20:45:30,021] Trial 1 finished with value: 0.003694446670576664 and parameters: {'n_estimators': 199, 'learning_rate': 0.015704890429590206, 'num_leaves': 102, 'max_depth': 10, 'min_child_samples': 3, 'subsample': 0.6363815443034426, 'colsample_bytree': 0.9802107640720542, 'reg_alpha': 7.00191662347607, 'reg_lambda': 7.041858159835595, 'min_split_gain': 0.042848784006500676}. Best is trial 0 with value: 0.0014739952470778916.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:45:30,611] Trial 2 finished with value: 0.0012198588161374197 and parameters: {'n_estimators': 125, 'learning_rate': 0.28724172220436706, 'num_leaves': 77, 'max_depth': 5, 'min_child_samples': 1, 'subsample': 0.7008422505954799, 'colsample_bytree': 0.9590665807762144, 'reg_alpha': 5.662696252814531, 'reg_lambda': 9.088245255163889, 'min_split_gain': 0.00039073833538517125}. Best is trial 2 with value: 0.0012198588161374197.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:45:32,859] Trial 3 finished with value: 0.001629181264860855 and parameters: {'n_estimators': 244, 'learning_rate': 0.028903191524595702, 'num_leaves': 135, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.736029895981188, 'colsample_bytree': 0.8374484728448948, 'reg_alpha': 1.99582235205353, 'reg_lambda': 9.350863837900347, 'min_split_gain': 0.040185314980890785}. Best is trial 2 with value: 0.0012198588161374197.
[I 2025-06-15 20:45:32,862] A new study created in memory with name: no-name-c6a729f8-adb3-4e4d-8b8a-db025f2050bf
[I 2025-06-15 20:45:32,932] Trial 0 finished with value: 0.00815432535749091 and parameters: {'n_estimators': 122, 'learning_rate': 0.013093800879268659, 'num_leaves': 172, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.8325710087078403, 'colsample_bytree': 0.6966874551608997, 'reg_alpha': 2.1204752158671436, 'reg_lambda': 0.903947492903191, 'min_split_gain': 0.014426990153154053}. Best is trial 0 with value: 0.00815432535749091.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         320
1           Price_lag_2         293
14      pct_chg_Load_DA         187
2           Price_lag_7         143
3         Load_DA_lag_0         131
5       WindOn_DA_lag_0         128
16  roll2d_median_Price         124
11           NGas_lag_2         110
13            EUA_lag_2         105
15       lag168_Load_DA         100
10           Coal_lag_2          77
12            Oil_lag_2          65
9                  WD_7          10
8                  WD_6           7
7                  WD_1           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         104
1           Price_lag_2          30
15       lag168_Load_DA          21
3         Load_DA_lag_0          14
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 91.11% done
********************* END NS.. 665 ****************************************************
********************* START NS ... 666 ****************************************************
[I 2025-06-15 20:45:34,892] A new study created in memory with name: no-name-2d85d797-cce8-48b7-895f-8ba4b686a5e9
[I 2025-06-15 20:45:35,785] Trial 0 finished with value: 0.0014566351857431107 and parameters: {'n_estimators': 185, 'learning_rate': 0.026390030805808568, 'num_leaves': 201, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.7410541945507236, 'colsample_bytree': 0.8987126855295517, 'reg_alpha': 2.798794824504501, 'reg_lambda': 8.615842089583834, 'min_split_gain': 0.04562023097910592}. Best is trial 0 with value: 0.0014566351857431107.
[Trial 0] New best: 0.00146
[I 2025-06-15 20:45:36,362] Trial 1 finished with value: 0.0011119265227476432 and parameters: {'n_estimators': 137, 'learning_rate': 0.07967107451635137, 'num_leaves': 106, 'max_depth': 9, 'min_child_samples': 20, 'subsample': 0.6999336786834895, 'colsample_bytree': 0.694772855197354, 'reg_alpha': 9.812504869461037, 'reg_lambda': 0.3459121058477532, 'min_split_gain': 0.022054443496028028}. Best is trial 1 with value: 0.0011119265227476432.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:45:36,867] Trial 2 finished with value: 0.002659980067441344 and parameters: {'n_estimators': 69, 'learning_rate': 0.051440232197603, 'num_leaves': 78, 'max_depth': 7, 'min_child_samples': 13, 'subsample': 0.549987171542594, 'colsample_bytree': 0.7443530149829933, 'reg_alpha': 5.0364912583506545, 'reg_lambda': 2.6530388797443702, 'min_split_gain': 0.044794376659144375}. Best is trial 1 with value: 0.0011119265227476432.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:45:37,392] Trial 3 finished with value: 0.0015467982332574206 and parameters: {'n_estimators': 169, 'learning_rate': 0.026969282561155217, 'num_leaves': 128, 'max_depth': 4, 'min_child_samples': 14, 'subsample': 0.7288793798991706, 'colsample_bytree': 0.9967901169968679, 'reg_alpha': 7.030713389536879, 'reg_lambda': 7.853175275627383, 'min_split_gain': 0.01673952914398441}. Best is trial 1 with value: 0.0011119265227476432.
[I 2025-06-15 20:45:37,394] A new study created in memory with name: no-name-75cc172c-b8fa-43b2-b8ef-dc3fd07cd659
[I 2025-06-15 20:45:37,470] Trial 0 finished with value: 0.0322862311222109 and parameters: {'n_estimators': 236, 'learning_rate': 0.1226429288048741, 'num_leaves': 59, 'max_depth': 12, 'min_child_samples': 16, 'subsample': 0.9889424329776013, 'colsample_bytree': 0.505088092860288, 'reg_alpha': 8.112894011831852, 'reg_lambda': 5.886182057250742, 'min_split_gain': 0.03387947153289322}. Best is trial 0 with value: 0.0322862311222109.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         515
14      pct_chg_Load_DA         296
1           Price_lag_2         285
3         Load_DA_lag_0         174
11           NGas_lag_2         140
2           Price_lag_7         137
16  roll2d_median_Price         126
15       lag168_Load_DA         103
5       WindOn_DA_lag_0          94
13            EUA_lag_2          80
10           Coal_lag_2          76
12            Oil_lag_2          61
9                  WD_7          16
8                  WD_6          12
7                  WD_1           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 91.24% done
********************* END NS.. 666 ****************************************************
********************* START NS ... 667 ****************************************************
[I 2025-06-15 20:45:39,554] A new study created in memory with name: no-name-3981fd23-4bc3-4fd9-a879-659a71f0e570
[I 2025-06-15 20:45:40,596] Trial 0 finished with value: 0.004634892970345112 and parameters: {'n_estimators': 130, 'learning_rate': 0.02117412664836699, 'num_leaves': 109, 'max_depth': 10, 'min_child_samples': 6, 'subsample': 0.967874367601159, 'colsample_bytree': 0.6413804329739339, 'reg_alpha': 7.403399388369701, 'reg_lambda': 2.0178165615951436, 'min_split_gain': 0.022509538339506314}. Best is trial 0 with value: 0.004634892970345112.
[Trial 0] New best: 0.00463
[I 2025-06-15 20:45:42,249] Trial 1 finished with value: 0.0012726957199332117 and parameters: {'n_estimators': 184, 'learning_rate': 0.02129675172028211, 'num_leaves': 133, 'max_depth': 15, 'min_child_samples': 23, 'subsample': 0.5477475416036395, 'colsample_bytree': 0.8669571839196855, 'reg_alpha': 2.8210827723146714, 'reg_lambda': 7.652284833520034, 'min_split_gain': 0.03673137970928122}. Best is trial 1 with value: 0.0012726957199332117.
[Trial 1] New best: 0.00127
[I 2025-06-15 20:45:45,925] Trial 2 finished with value: 0.0008717530503855522 and parameters: {'n_estimators': 277, 'learning_rate': 0.02767841049603621, 'num_leaves': 173, 'max_depth': 16, 'min_child_samples': 21, 'subsample': 0.519338065581125, 'colsample_bytree': 0.5198423280314646, 'reg_alpha': 3.6359478508680176, 'reg_lambda': 4.642858379259879, 'min_split_gain': 0.007870208738665391}. Best is trial 2 with value: 0.0008717530503855522.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:45:46,591] Trial 3 finished with value: 0.12341700558587072 and parameters: {'n_estimators': 60, 'learning_rate': 0.016706514163770118, 'num_leaves': 155, 'max_depth': 13, 'min_child_samples': 2, 'subsample': 0.6726437451093583, 'colsample_bytree': 0.5276204077560294, 'reg_alpha': 8.985136349710118, 'reg_lambda': 5.481978882813518, 'min_split_gain': 0.009552381957686774}. Best is trial 2 with value: 0.0008717530503855522.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:45:47,556] Trial 4 finished with value: 0.0008975511322437384 and parameters: {'n_estimators': 101, 'learning_rate': 0.047658838844368265, 'num_leaves': 187, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.7664049733446999, 'colsample_bytree': 0.703511108058146, 'reg_alpha': 6.668598019753023, 'reg_lambda': 0.29338137088797733, 'min_split_gain': 0.0022604936729327453}. Best is trial 2 with value: 0.0008717530503855522.
[I 2025-06-15 20:45:47,559] A new study created in memory with name: no-name-2e11ee33-4a63-4098-b5a5-769fb6873890
[I 2025-06-15 20:45:47,622] Trial 0 finished with value: 0.010769689463821217 and parameters: {'n_estimators': 262, 'learning_rate': 0.15407867115710802, 'num_leaves': 150, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.9993709971825496, 'colsample_bytree': 0.8300064039836124, 'reg_alpha': 0.256445149241894, 'reg_lambda': 5.160509859239399, 'min_split_gain': 0.028572883569599708}. Best is trial 0 with value: 0.010769689463821217.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        2504
0           Price_lag_1        2501
14      pct_chg_Load_DA        1503
3         Load_DA_lag_0        1173
2           Price_lag_7        1022
16  roll2d_median_Price        1017
5       WindOn_DA_lag_0         947
15       lag168_Load_DA         912
13            EUA_lag_2         845
10           Coal_lag_2         744
12            Oil_lag_2         724
11           NGas_lag_2         696
9                  WD_7          76
8                  WD_6          60
7                  WD_1          45
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          19
15       lag168_Load_DA           3
5       WindOn_DA_lag_0           1
13            EUA_lag_2           1
1           Price_lag_2           1
16  roll2d_median_Price           1
3         Load_DA_lag_0           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 91.38% done
********************* END NS.. 667 ****************************************************
********************* START NS ... 668 ****************************************************
[I 2025-06-15 20:45:51,892] A new study created in memory with name: no-name-11f16f34-a256-4095-8555-1862e88b2342
[I 2025-06-15 20:45:53,147] Trial 0 finished with value: 0.06407679147892238 and parameters: {'n_estimators': 159, 'learning_rate': 0.011061799714595276, 'num_leaves': 190, 'max_depth': 14, 'min_child_samples': 23, 'subsample': 0.9637588064057357, 'colsample_bytree': 0.5494627368833954, 'reg_alpha': 5.827580743802651, 'reg_lambda': 3.7003180120012935, 'min_split_gain': 0.029860565033722777}. Best is trial 0 with value: 0.06407679147892238.
[Trial 0] New best: 0.06408
[I 2025-06-15 20:45:54,478] Trial 1 finished with value: 0.0574906053884868 and parameters: {'n_estimators': 165, 'learning_rate': 0.010211208047933958, 'num_leaves': 75, 'max_depth': 16, 'min_child_samples': 27, 'subsample': 0.5033832590730667, 'colsample_bytree': 0.8275257735274142, 'reg_alpha': 6.146278502057115, 'reg_lambda': 0.2586250286930003, 'min_split_gain': 0.049199302779499084}. Best is trial 1 with value: 0.0574906053884868.
[Trial 1] New best: 0.05749
[I 2025-06-15 20:45:55,064] Trial 2 finished with value: 0.03846972924756896 and parameters: {'n_estimators': 180, 'learning_rate': 0.023179722917301585, 'num_leaves': 183, 'max_depth': 4, 'min_child_samples': 28, 'subsample': 0.9153616869686654, 'colsample_bytree': 0.6823539975888718, 'reg_alpha': 5.1182992883267, 'reg_lambda': 3.2725653979636435, 'min_split_gain': 0.001011666822940821}. Best is trial 2 with value: 0.03846972924756896.
[Trial 2] New best: 0.03847
[I 2025-06-15 20:45:56,238] Trial 3 finished with value: 0.036411855130732244 and parameters: {'n_estimators': 216, 'learning_rate': 0.05032647894064975, 'num_leaves': 212, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.8467611615248452, 'colsample_bytree': 0.5760588439789394, 'reg_alpha': 7.216637475185427, 'reg_lambda': 0.24078663352552043, 'min_split_gain': 0.03304439192455224}. Best is trial 3 with value: 0.036411855130732244.
[Trial 3] New best: 0.03641
[I 2025-06-15 20:45:57,693] Trial 4 finished with value: 0.13594285263794234 and parameters: {'n_estimators': 67, 'learning_rate': 0.013004896256638797, 'num_leaves': 149, 'max_depth': 10, 'min_child_samples': 21, 'subsample': 0.8681548680322471, 'colsample_bytree': 0.9751507162003186, 'reg_alpha': 0.27808387574241644, 'reg_lambda': 1.0201946626897762, 'min_split_gain': 0.03243034944644114}. Best is trial 3 with value: 0.036411855130732244.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:45:58,664] Trial 5 finished with value: 0.06192842697590948 and parameters: {'n_estimators': 120, 'learning_rate': 0.013598439557613782, 'num_leaves': 98, 'max_depth': 5, 'min_child_samples': 12, 'subsample': 0.7131705318418312, 'colsample_bytree': 0.7635195870386009, 'reg_alpha': 1.4479822177691781, 'reg_lambda': 5.057756138651184, 'min_split_gain': 0.017527979745277283}. Best is trial 3 with value: 0.036411855130732244.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:45:59,189] Trial 6 finished with value: 0.11100235109013727 and parameters: {'n_estimators': 87, 'learning_rate': 0.012830488278104674, 'num_leaves': 99, 'max_depth': 5, 'min_child_samples': 15, 'subsample': 0.5034031472619717, 'colsample_bytree': 0.5641777743635518, 'reg_alpha': 5.264498317347611, 'reg_lambda': 9.943421822127668, 'min_split_gain': 0.026898790782253185}. Best is trial 3 with value: 0.036411855130732244.
[I 2025-06-15 20:45:59,191] A new study created in memory with name: no-name-49f761a3-00d7-49f7-9c20-d0ee42605778
[I 2025-06-15 20:45:59,340] Trial 0 finished with value: 0.1303200535775814 and parameters: {'n_estimators': 85, 'learning_rate': 0.09618842171064466, 'num_leaves': 141, 'max_depth': 10, 'min_child_samples': 18, 'subsample': 0.9096347198749251, 'colsample_bytree': 0.5438175736787962, 'reg_alpha': 2.3002419570803436, 'reg_lambda': 3.525103125124632, 'min_split_gain': 0.010546561309905523}. Best is trial 0 with value: 0.1303200535775814.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         907
1           Price_lag_2         708
14      pct_chg_Load_DA         475
3         Load_DA_lag_0         309
2           Price_lag_7         228
16  roll2d_median_Price         203
11           NGas_lag_2         200
5       WindOn_DA_lag_0         185
15       lag168_Load_DA         174
10           Coal_lag_2         130
13            EUA_lag_2         108
12            Oil_lag_2          87
9                  WD_7          22
8                  WD_6          17
7                  WD_1           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          39
5       WindOn_DA_lag_0          16
3         Load_DA_lag_0          12
12            Oil_lag_2          10
16  roll2d_median_Price           9
1           Price_lag_2           7
15       lag168_Load_DA           6
10           Coal_lag_2           5
13            EUA_lag_2           5
2           Price_lag_7           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 91.52% done
********************* END NS.. 668 ****************************************************
********************* START NS ... 669 ****************************************************
[I 2025-06-15 20:46:01,740] A new study created in memory with name: no-name-518cd02d-9ae6-4c62-b5bb-1aa2ee4e2ae5
[I 2025-06-15 20:46:02,179] Trial 0 finished with value: 0.03494239146627611 and parameters: {'n_estimators': 267, 'learning_rate': 0.1064855668110169, 'num_leaves': 205, 'max_depth': 4, 'min_child_samples': 9, 'subsample': 0.9765941994689751, 'colsample_bytree': 0.6910856561055826, 'reg_alpha': 3.73319881952629, 'reg_lambda': 2.887982314584878, 'min_split_gain': 0.006692843466042925}. Best is trial 0 with value: 0.03494239146627611.
[I 2025-06-15 20:46:02,364] Trial 1 finished with value: 0.21551148727370664 and parameters: {'n_estimators': 64, 'learning_rate': 0.010412857501932871, 'num_leaves': 192, 'max_depth': 4, 'min_child_samples': 20, 'subsample': 0.7877779845410424, 'colsample_bytree': 0.7720808504300991, 'reg_alpha': 4.34706179163626, 'reg_lambda': 1.8917518709654357, 'min_split_gain': 0.009726959032347483}. Best is trial 0 with value: 0.03494239146627611.
[Trial 0] New best: 0.03494
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:46:04,220] Trial 2 finished with value: 0.04248933018134717 and parameters: {'n_estimators': 224, 'learning_rate': 0.014120046497432256, 'num_leaves': 184, 'max_depth': 15, 'min_child_samples': 27, 'subsample': 0.9583980535545056, 'colsample_bytree': 0.6451077291160603, 'reg_alpha': 5.264993917555408, 'reg_lambda': 8.751092415524463, 'min_split_gain': 0.019304336528950573}. Best is trial 0 with value: 0.03494239146627611.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:46:04,858] Trial 3 finished with value: 0.03684795600716101 and parameters: {'n_estimators': 187, 'learning_rate': 0.13154432510385136, 'num_leaves': 143, 'max_depth': 15, 'min_child_samples': 24, 'subsample': 0.6676900861797729, 'colsample_bytree': 0.6891393844755833, 'reg_alpha': 6.060083843468282, 'reg_lambda': 9.705167205993655, 'min_split_gain': 0.02722851854661525}. Best is trial 0 with value: 0.03494239146627611.
[I 2025-06-15 20:46:04,860] A new study created in memory with name: no-name-9f8e59ca-229d-4660-b998-43318e8a74a4
[I 2025-06-15 20:46:04,910] Trial 0 finished with value: 0.15686394017731783 and parameters: {'n_estimators': 188, 'learning_rate': 0.04230471099940757, 'num_leaves': 234, 'max_depth': 9, 'min_child_samples': 8, 'subsample': 0.7236841053186085, 'colsample_bytree': 0.6764175653217439, 'reg_alpha': 9.301836363928105, 'reg_lambda': 5.381630832451817, 'min_split_gain': 0.049721762829411535}. Best is trial 0 with value: 0.15686394017731783.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         648
1           Price_lag_2         432
14      pct_chg_Load_DA         291
3         Load_DA_lag_0         166
11           NGas_lag_2         152
2           Price_lag_7         134
15       lag168_Load_DA         133
13            EUA_lag_2         131
5       WindOn_DA_lag_0         126
10           Coal_lag_2         120
16  roll2d_median_Price         120
12            Oil_lag_2          71
9                  WD_7          20
7                  WD_1          12
8                  WD_6           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 91.66% done
********************* END NS.. 669 ****************************************************
********************* START NS ... 670 ****************************************************
[I 2025-06-15 20:46:06,912] A new study created in memory with name: no-name-af79a5e0-6137-4e5e-aa5d-56446526bcfc
[I 2025-06-15 20:46:07,388] Trial 0 finished with value: 0.0029508058825247784 and parameters: {'n_estimators': 210, 'learning_rate': 0.1434155948264211, 'num_leaves': 151, 'max_depth': 16, 'min_child_samples': 26, 'subsample': 0.9318912803530941, 'colsample_bytree': 0.9011447874152698, 'reg_alpha': 6.374189086238701, 'reg_lambda': 5.497559395263534, 'min_split_gain': 0.006669564830357255}. Best is trial 0 with value: 0.0029508058825247784.
[Trial 0] New best: 0.00295
[I 2025-06-15 20:46:08,508] Trial 1 finished with value: 0.0028525788469992482 and parameters: {'n_estimators': 115, 'learning_rate': 0.06672956710899494, 'num_leaves': 48, 'max_depth': 16, 'min_child_samples': 11, 'subsample': 0.9865830265723725, 'colsample_bytree': 0.9870812185204544, 'reg_alpha': 0.7078810863479312, 'reg_lambda': 9.349397573171132, 'min_split_gain': 0.021663533316117464}. Best is trial 1 with value: 0.0028525788469992482.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:46:09,203] Trial 2 finished with value: 0.0035615050787126657 and parameters: {'n_estimators': 110, 'learning_rate': 0.18388849898908857, 'num_leaves': 123, 'max_depth': 6, 'min_child_samples': 26, 'subsample': 0.7943114793490993, 'colsample_bytree': 0.5622944770451836, 'reg_alpha': 1.5009729270306738, 'reg_lambda': 1.628810012494758, 'min_split_gain': 0.012851013915616723}. Best is trial 1 with value: 0.0028525788469992482.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:46:09,472] Trial 3 finished with value: 0.002899235086899814 and parameters: {'n_estimators': 138, 'learning_rate': 0.22757749975899935, 'num_leaves': 240, 'max_depth': 5, 'min_child_samples': 2, 'subsample': 0.8045487375321885, 'colsample_bytree': 0.9813213527260161, 'reg_alpha': 2.921033337752669, 'reg_lambda': 7.829211776358084, 'min_split_gain': 0.023248228502058734}. Best is trial 1 with value: 0.0028525788469992482.
[I 2025-06-15 20:46:09,474] A new study created in memory with name: no-name-b296c7ca-f357-4c90-8bf6-9c676f7b0bce
[I 2025-06-15 20:46:09,505] Trial 0 finished with value: 0.020908902208825648 and parameters: {'n_estimators': 85, 'learning_rate': 0.027190103074731863, 'num_leaves': 227, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.618151744838316, 'colsample_bytree': 0.9442734343114382, 'reg_alpha': 9.86989770431544, 'reg_lambda': 3.6052675368079212, 'min_split_gain': 0.033315443543288574}. Best is trial 0 with value: 0.020908902208825648.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1142
14      pct_chg_Load_DA         821
1           Price_lag_2         616
3         Load_DA_lag_0         472
16  roll2d_median_Price         409
5       WindOn_DA_lag_0         371
2           Price_lag_7         354
11           NGas_lag_2         330
15       lag168_Load_DA         272
13            EUA_lag_2         224
10           Coal_lag_2         190
12            Oil_lag_2         175
7                  WD_1           9
9                  WD_7           9
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 91.79% done
********************* END NS.. 670 ****************************************************
********************* START NS ... 671 ****************************************************
[I 2025-06-15 20:46:11,960] A new study created in memory with name: no-name-5f6e45bf-8caa-4ae0-93df-72ec3427b415
[I 2025-06-15 20:46:12,198] Trial 0 finished with value: 0.001790817014372943 and parameters: {'n_estimators': 254, 'learning_rate': 0.2524308729480483, 'num_leaves': 253, 'max_depth': 10, 'min_child_samples': 27, 'subsample': 0.5754794206900304, 'colsample_bytree': 0.7812140287888008, 'reg_alpha': 5.421180687176004, 'reg_lambda': 6.31812550825496, 'min_split_gain': 0.029602940411660034}. Best is trial 0 with value: 0.001790817014372943.
[Trial 0] New best: 0.00179
[I 2025-06-15 20:46:12,939] Trial 1 finished with value: 0.0013964311393917883 and parameters: {'n_estimators': 284, 'learning_rate': 0.0934603477196282, 'num_leaves': 143, 'max_depth': 11, 'min_child_samples': 29, 'subsample': 0.9679787822899775, 'colsample_bytree': 0.6000054863830433, 'reg_alpha': 8.9367822583575, 'reg_lambda': 4.216716179511249, 'min_split_gain': 0.024225251276190005}. Best is trial 1 with value: 0.0013964311393917883.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:46:14,360] Trial 2 finished with value: 0.0009352718449485582 and parameters: {'n_estimators': 77, 'learning_rate': 0.10513601133789033, 'num_leaves': 124, 'max_depth': 15, 'min_child_samples': 9, 'subsample': 0.8261868609771008, 'colsample_bytree': 0.9173643603015438, 'reg_alpha': 0.6774085746322855, 'reg_lambda': 7.831990090705655, 'min_split_gain': 0.017919553521438352}. Best is trial 2 with value: 0.0009352718449485582.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:46:15,026] Trial 3 finished with value: 0.030324495596405727 and parameters: {'n_estimators': 63, 'learning_rate': 0.028518950948485108, 'num_leaves': 73, 'max_depth': 13, 'min_child_samples': 23, 'subsample': 0.609082431617024, 'colsample_bytree': 0.8876306927696238, 'reg_alpha': 9.960084001763367, 'reg_lambda': 4.640428887058966, 'min_split_gain': 0.018339051860836536}. Best is trial 2 with value: 0.0009352718449485582.
[I 2025-06-15 20:46:15,028] A new study created in memory with name: no-name-846abbc9-d345-41d7-b4e7-d21aafcc79ab
[I 2025-06-15 20:46:15,074] Trial 0 finished with value: 0.023289985462197165 and parameters: {'n_estimators': 153, 'learning_rate': 0.015403106748066, 'num_leaves': 180, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.8457688678204925, 'colsample_bytree': 0.8368619051392072, 'reg_alpha': 7.409280339580269, 'reg_lambda': 2.4445515160405717, 'min_split_gain': 0.01915779723190138}. Best is trial 0 with value: 0.023289985462197165.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
14      pct_chg_Load_DA        1020
0           Price_lag_1        1006
1           Price_lag_2         887
3         Load_DA_lag_0         678
5       WindOn_DA_lag_0         634
16  roll2d_median_Price         524
15       lag168_Load_DA         512
2           Price_lag_7         469
11           NGas_lag_2         380
10           Coal_lag_2         303
13            EUA_lag_2         299
12            Oil_lag_2         219
9                  WD_7          17
7                  WD_1          12
8                  WD_6          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          87
1           Price_lag_2           7
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 91.93% done
********************* END NS.. 671 ****************************************************
********************* START NS ... 672 ****************************************************
[I 2025-06-15 20:46:18,377] A new study created in memory with name: no-name-bccbcf39-51aa-43db-88e3-55367aef7281
[I 2025-06-15 20:46:18,507] Trial 0 finished with value: 0.0031825005977414236 and parameters: {'n_estimators': 53, 'learning_rate': 0.07167051004289764, 'num_leaves': 104, 'max_depth': 4, 'min_child_samples': 18, 'subsample': 0.5840258188211549, 'colsample_bytree': 0.7530442536605529, 'reg_alpha': 8.488013184216946, 'reg_lambda': 0.25267438750175564, 'min_split_gain': 0.0068568723040272565}. Best is trial 0 with value: 0.0031825005977414236.
[Trial 0] New best: 0.00318
[I 2025-06-15 20:46:19,097] Trial 1 finished with value: 0.1333055301811482 and parameters: {'n_estimators': 61, 'learning_rate': 0.01621396815527268, 'num_leaves': 105, 'max_depth': 9, 'min_child_samples': 20, 'subsample': 0.8377536058875368, 'colsample_bytree': 0.8290169257335109, 'reg_alpha': 3.8733801062709072, 'reg_lambda': 7.118693093380916, 'min_split_gain': 0.010084757314144654}. Best is trial 0 with value: 0.0031825005977414236.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:46:19,365] Trial 2 finished with value: 0.0019935779471203182 and parameters: {'n_estimators': 113, 'learning_rate': 0.20150207499810138, 'num_leaves': 91, 'max_depth': 16, 'min_child_samples': 14, 'subsample': 0.7569453553036134, 'colsample_bytree': 0.7912826356790911, 'reg_alpha': 8.015964775351293, 'reg_lambda': 7.732837666015044, 'min_split_gain': 0.03731411193734923}. Best is trial 2 with value: 0.0019935779471203182.
[Trial 2] New best: 0.00199
[I 2025-06-15 20:46:19,843] Trial 3 finished with value: 0.022064113694373638 and parameters: {'n_estimators': 240, 'learning_rate': 0.20752308407878756, 'num_leaves': 221, 'max_depth': 10, 'min_child_samples': 16, 'subsample': 0.9019362193070026, 'colsample_bytree': 0.5554171773645392, 'reg_alpha': 8.112845608848485, 'reg_lambda': 4.476537560798293, 'min_split_gain': 0.0399848910232956}. Best is trial 2 with value: 0.0019935779471203182.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:46:22,155] Trial 4 finished with value: 0.0022132589856694463 and parameters: {'n_estimators': 172, 'learning_rate': 0.032697203449465986, 'num_leaves': 168, 'max_depth': 12, 'min_child_samples': 2, 'subsample': 0.7756547181832689, 'colsample_bytree': 0.9523626111181684, 'reg_alpha': 3.4060984804425374, 'reg_lambda': 5.2287683928122455, 'min_split_gain': 0.025304746660216315}. Best is trial 2 with value: 0.0019935779471203182.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:46:22,404] Trial 5 finished with value: 0.0018348412709741028 and parameters: {'n_estimators': 118, 'learning_rate': 0.2319000840184158, 'num_leaves': 221, 'max_depth': 4, 'min_child_samples': 14, 'subsample': 0.9394702904596963, 'colsample_bytree': 0.6326184881186506, 'reg_alpha': 9.538748600601908, 'reg_lambda': 2.1366279767927887, 'min_split_gain': 0.00651690175827912}. Best is trial 5 with value: 0.0018348412709741028.
[I 2025-06-15 20:46:22,406] A new study created in memory with name: no-name-86b4cf3c-8883-4a0d-826a-cd253b920283
[I 2025-06-15 20:46:22,517] Trial 0 finished with value: 0.010628742254688068 and parameters: {'n_estimators': 163, 'learning_rate': 0.01744855184891074, 'num_leaves': 65, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.7048094682065373, 'colsample_bytree': 0.5253233503529742, 'reg_alpha': 0.3906038242315679, 'reg_lambda': 9.460313476569087, 'min_split_gain': 0.02226831176233772}. Best is trial 0 with value: 0.010628742254688068.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         229
1           Price_lag_2         203
14      pct_chg_Load_DA          80
3         Load_DA_lag_0          70
11           NGas_lag_2          52
5       WindOn_DA_lag_0          40
10           Coal_lag_2          38
16  roll2d_median_Price          37
15       lag168_Load_DA          33
2           Price_lag_7          27
13            EUA_lag_2           9
9                  WD_7           7
8                  WD_6           5
12            Oil_lag_2           4
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         159
5       WindOn_DA_lag_0          47
1           Price_lag_2          34
15       lag168_Load_DA          24
10           Coal_lag_2          24
14      pct_chg_Load_DA          16
3         Load_DA_lag_0          14
12            Oil_lag_2          11
13            EUA_lag_2           8
11           NGas_lag_2           5
2           Price_lag_7           5
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 92.07% done
********************* END NS.. 672 ****************************************************
********************* START NS ... 673 ****************************************************
[I 2025-06-15 20:46:23,838] A new study created in memory with name: no-name-50060744-3f79-4faf-897c-cc659f2c153c
[I 2025-06-15 20:46:24,454] Trial 0 finished with value: 0.14734379676130854 and parameters: {'n_estimators': 89, 'learning_rate': 0.010224103889550785, 'num_leaves': 242, 'max_depth': 11, 'min_child_samples': 20, 'subsample': 0.8472648695767115, 'colsample_bytree': 0.9343202553484335, 'reg_alpha': 4.5633426293852235, 'reg_lambda': 5.063546836199626, 'min_split_gain': 0.022278120575379052}. Best is trial 0 with value: 0.14734379676130854.
[Trial 0] New best: 0.14734
[I 2025-06-15 20:46:26,036] Trial 1 finished with value: 0.009206095188413713 and parameters: {'n_estimators': 165, 'learning_rate': 0.015431964494799087, 'num_leaves': 195, 'max_depth': 10, 'min_child_samples': 4, 'subsample': 0.5253014186053823, 'colsample_bytree': 0.7725289865409887, 'reg_alpha': 4.818236702069539, 'reg_lambda': 6.701690434439308, 'min_split_gain': 0.03427837965422757}. Best is trial 1 with value: 0.009206095188413713.
[Trial 1] New best: 0.00921
[I 2025-06-15 20:46:28,215] Trial 2 finished with value: 0.0018673185302480763 and parameters: {'n_estimators': 225, 'learning_rate': 0.06530916331204888, 'num_leaves': 173, 'max_depth': 11, 'min_child_samples': 28, 'subsample': 0.9489006203782046, 'colsample_bytree': 0.8704299410321419, 'reg_alpha': 5.408408051996679, 'reg_lambda': 0.13580674155242778, 'min_split_gain': 0.002207901482743069}. Best is trial 2 with value: 0.0018673185302480763.
[Trial 2] New best: 0.00187
[I 2025-06-15 20:46:28,856] Trial 3 finished with value: 0.04046141360728719 and parameters: {'n_estimators': 106, 'learning_rate': 0.015085559221129063, 'num_leaves': 246, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.8943039589396427, 'colsample_bytree': 0.9876446763546364, 'reg_alpha': 9.569361136702812, 'reg_lambda': 0.4167633236510082, 'min_split_gain': 0.028475651086776983}. Best is trial 2 with value: 0.0018673185302480763.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:46:31,027] Trial 4 finished with value: 0.004666589973209951 and parameters: {'n_estimators': 250, 'learning_rate': 0.012990537823197082, 'num_leaves': 39, 'max_depth': 14, 'min_child_samples': 2, 'subsample': 0.92566054120677, 'colsample_bytree': 0.6067563816131678, 'reg_alpha': 5.319849018998384, 'reg_lambda': 8.5780504947702, 'min_split_gain': 0.01396357443221638}. Best is trial 2 with value: 0.0018673185302480763.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:46:32,578] Trial 5 finished with value: 0.0033063474525173407 and parameters: {'n_estimators': 121, 'learning_rate': 0.0286726426647592, 'num_leaves': 232, 'max_depth': 11, 'min_child_samples': 13, 'subsample': 0.8874126511350334, 'colsample_bytree': 0.7934229910838775, 'reg_alpha': 1.2972255224392149, 'reg_lambda': 2.4682310805689402, 'min_split_gain': 0.049159879705935855}. Best is trial 2 with value: 0.0018673185302480763.
[I 2025-06-15 20:46:32,580] A new study created in memory with name: no-name-a80149e6-ce63-48f1-a369-1725197dac57
[I 2025-06-15 20:46:32,631] Trial 0 finished with value: 0.009400598865114514 and parameters: {'n_estimators': 67, 'learning_rate': 0.019977409942461256, 'num_leaves': 184, 'max_depth': 16, 'min_child_samples': 6, 'subsample': 0.9230639189177812, 'colsample_bytree': 0.6472364796949333, 'reg_alpha': 5.18197611309918, 'reg_lambda': 1.2762264138887336, 'min_split_gain': 0.027891875727271826}. Best is trial 0 with value: 0.009400598865114514.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1321
1           Price_lag_2        1098
14      pct_chg_Load_DA         902
3         Load_DA_lag_0         786
5       WindOn_DA_lag_0         671
15       lag168_Load_DA         630
2           Price_lag_7         610
11           NGas_lag_2         469
13            EUA_lag_2         458
16  roll2d_median_Price         430
10           Coal_lag_2         374
12            Oil_lag_2         355
9                  WD_7          44
8                  WD_6          39
7                  WD_1          17
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          65
1           Price_lag_2          20
3         Load_DA_lag_0          17
5       WindOn_DA_lag_0          13
12            Oil_lag_2           5
15       lag168_Load_DA           2
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 92.20% done
********************* END NS.. 673 ****************************************************
********************* START NS ... 674 ****************************************************
[I 2025-06-15 20:46:35,263] A new study created in memory with name: no-name-f1fcafac-ab43-487e-baa7-60f02ede2a7b
[I 2025-06-15 20:46:35,667] Trial 0 finished with value: 0.016141104094144664 and parameters: {'n_estimators': 201, 'learning_rate': 0.1313881971262388, 'num_leaves': 32, 'max_depth': 15, 'min_child_samples': 21, 'subsample': 0.8292241893240199, 'colsample_bytree': 0.821921961969882, 'reg_alpha': 3.7710839784295125, 'reg_lambda': 0.299571345023496, 'min_split_gain': 0.02219438442631775}. Best is trial 0 with value: 0.016141104094144664.
[Trial 0] New best: 0.01614
[I 2025-06-15 20:46:37,800] Trial 1 finished with value: 0.04797503799425717 and parameters: {'n_estimators': 148, 'learning_rate': 0.01013497182701875, 'num_leaves': 134, 'max_depth': 9, 'min_child_samples': 23, 'subsample': 0.5850495257942896, 'colsample_bytree': 0.5400984625696947, 'reg_alpha': 1.563110174476675, 'reg_lambda': 2.2742642249866107, 'min_split_gain': 0.015815497357619224}. Best is trial 0 with value: 0.016141104094144664.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:46:38,561] Trial 2 finished with value: 0.09164935526856223 and parameters: {'n_estimators': 99, 'learning_rate': 0.01114853104313496, 'num_leaves': 162, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.9086391361686629, 'colsample_bytree': 0.8539197947106232, 'reg_alpha': 5.45534035767285, 'reg_lambda': 4.887017944686906, 'min_split_gain': 0.03080080426953706}. Best is trial 0 with value: 0.016141104094144664.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:46:39,090] Trial 3 finished with value: 0.016511637191875744 and parameters: {'n_estimators': 170, 'learning_rate': 0.10450038480603295, 'num_leaves': 166, 'max_depth': 11, 'min_child_samples': 29, 'subsample': 0.6751398220572902, 'colsample_bytree': 0.8158240276414208, 'reg_alpha': 4.04724295056571, 'reg_lambda': 5.594646697991827, 'min_split_gain': 0.039497234683964176}. Best is trial 0 with value: 0.016141104094144664.
[I 2025-06-15 20:46:39,093] A new study created in memory with name: no-name-cc33a42c-e28a-441e-b2f1-fd5394336efe
[I 2025-06-15 20:46:39,126] Trial 0 finished with value: 0.06697920952518674 and parameters: {'n_estimators': 89, 'learning_rate': 0.28316022241742816, 'num_leaves': 200, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.9699598587669346, 'colsample_bytree': 0.5568186635799508, 'reg_alpha': 9.95735927386786, 'reg_lambda': 8.90237666654807, 'min_split_gain': 0.014221010518117345}. Best is trial 0 with value: 0.06697920952518674.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         398
1           Price_lag_2         262
14      pct_chg_Load_DA         244
3         Load_DA_lag_0         165
16  roll2d_median_Price         108
2           Price_lag_7         105
5       WindOn_DA_lag_0          99
11           NGas_lag_2          96
13            EUA_lag_2          71
15       lag168_Load_DA          66
10           Coal_lag_2          49
12            Oil_lag_2          33
9                  WD_7           8
7                  WD_1           6
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           7
3         Load_DA_lag_0           1
1           Price_lag_2           1
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 92.34% done
********************* END NS.. 674 ****************************************************
********************* START NS ... 675 ****************************************************
[I 2025-06-15 20:46:40,972] A new study created in memory with name: no-name-285ed70b-8c8a-40f8-93d2-8c0475089196
[I 2025-06-15 20:46:41,444] Trial 0 finished with value: 0.01667286687347942 and parameters: {'n_estimators': 294, 'learning_rate': 0.13724120055864483, 'num_leaves': 66, 'max_depth': 11, 'min_child_samples': 8, 'subsample': 0.5540923616011544, 'colsample_bytree': 0.970643877408862, 'reg_alpha': 4.791111022382269, 'reg_lambda': 1.5735216081988002, 'min_split_gain': 0.036591802074214703}. Best is trial 0 with value: 0.01667286687347942.
[Trial 0] New best: 0.01667
[I 2025-06-15 20:46:46,034] Trial 1 finished with value: 0.014625346107958724 and parameters: {'n_estimators': 245, 'learning_rate': 0.026562054365657787, 'num_leaves': 227, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.7818815544541142, 'colsample_bytree': 0.5198784923838562, 'reg_alpha': 0.6828206385472357, 'reg_lambda': 2.5430931233471865, 'min_split_gain': 0.0210061552835841}. Best is trial 1 with value: 0.014625346107958724.
[Trial 1] New best: 0.01463
[I 2025-06-15 20:46:46,336] Trial 2 finished with value: 0.016890998378266464 and parameters: {'n_estimators': 89, 'learning_rate': 0.1750254031122822, 'num_leaves': 142, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.6401492606479796, 'colsample_bytree': 0.8057306001438763, 'reg_alpha': 5.873848638956818, 'reg_lambda': 8.031391998182588, 'min_split_gain': 0.02730114857851626}. Best is trial 1 with value: 0.014625346107958724.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:46:47,122] Trial 3 finished with value: 0.016594753671283498 and parameters: {'n_estimators': 222, 'learning_rate': 0.08792738727317492, 'num_leaves': 135, 'max_depth': 8, 'min_child_samples': 24, 'subsample': 0.7446599123434972, 'colsample_bytree': 0.8407656430832599, 'reg_alpha': 5.494479822915937, 'reg_lambda': 8.22884275967665, 'min_split_gain': 0.01067713898090491}. Best is trial 1 with value: 0.014625346107958724.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:46:48,418] Trial 4 finished with value: 0.016767101885124615 and parameters: {'n_estimators': 239, 'learning_rate': 0.04066693207145574, 'num_leaves': 188, 'max_depth': 12, 'min_child_samples': 11, 'subsample': 0.5404260594408243, 'colsample_bytree': 0.9464476819999275, 'reg_alpha': 4.610539923468663, 'reg_lambda': 8.492097866792635, 'min_split_gain': 0.03345566872338634}. Best is trial 1 with value: 0.014625346107958724.
[I 2025-06-15 20:46:48,421] A new study created in memory with name: no-name-5d78d67c-049c-44a3-bfa3-f3e88c4063d5
[I 2025-06-15 20:46:48,605] Trial 0 finished with value: 0.06665357888776889 and parameters: {'n_estimators': 194, 'learning_rate': 0.04795042277562006, 'num_leaves': 91, 'max_depth': 5, 'min_child_samples': 29, 'subsample': 0.554492245390056, 'colsample_bytree': 0.5275737635392117, 'reg_alpha': 0.23432232860009172, 'reg_lambda': 3.8516004437443065, 'min_split_gain': 0.012382831859109544}. Best is trial 0 with value: 0.06665357888776889.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        3527
0           Price_lag_1        3096
14      pct_chg_Load_DA        2374
3         Load_DA_lag_0        2178
16  roll2d_median_Price        1800
2           Price_lag_7        1728
5       WindOn_DA_lag_0        1581
15       lag168_Load_DA        1516
10           Coal_lag_2        1313
11           NGas_lag_2        1223
13            EUA_lag_2        1147
12            Oil_lag_2         936
9                  WD_7          93
7                  WD_1          48
8                  WD_6          42
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          59
3         Load_DA_lag_0          31
16  roll2d_median_Price          28
15       lag168_Load_DA          25
5       WindOn_DA_lag_0          23
13            EUA_lag_2          12
1           Price_lag_2           5
11           NGas_lag_2           4
10           Coal_lag_2           3
2           Price_lag_7           0
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
12            Oil_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 92.48% done
********************* END NS.. 675 ****************************************************
********************* START NS ... 676 ****************************************************
[I 2025-06-15 20:46:53,612] A new study created in memory with name: no-name-7b28752e-0cbc-4bb3-9e73-ef2fa4762585
[I 2025-06-15 20:46:54,035] Trial 0 finished with value: 0.0006519351334639484 and parameters: {'n_estimators': 81, 'learning_rate': 0.13014915602017313, 'num_leaves': 172, 'max_depth': 12, 'min_child_samples': 20, 'subsample': 0.518907852444024, 'colsample_bytree': 0.7778873737146489, 'reg_alpha': 9.893698665695704, 'reg_lambda': 4.277181329048884, 'min_split_gain': 0.00822106357162905}. Best is trial 0 with value: 0.0006519351334639484.
[Trial 0] New best: 0.00065
[I 2025-06-15 20:46:54,366] Trial 1 finished with value: 0.0005469691418298737 and parameters: {'n_estimators': 90, 'learning_rate': 0.11695471252429847, 'num_leaves': 245, 'max_depth': 14, 'min_child_samples': 21, 'subsample': 0.7065297060961935, 'colsample_bytree': 0.7992237201244595, 'reg_alpha': 9.980922636225454, 'reg_lambda': 6.27574951618851, 'min_split_gain': 0.048912216201126665}. Best is trial 1 with value: 0.0005469691418298737.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:46:54,597] Trial 2 finished with value: 0.0004942404150330594 and parameters: {'n_estimators': 71, 'learning_rate': 0.06694084628851857, 'num_leaves': 175, 'max_depth': 4, 'min_child_samples': 20, 'subsample': 0.9541768400552688, 'colsample_bytree': 0.6907799354042499, 'reg_alpha': 8.473820297186121, 'reg_lambda': 2.8713101779950123, 'min_split_gain': 0.013961157874742043}. Best is trial 2 with value: 0.0004942404150330594.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:46:55,393] Trial 3 finished with value: 0.08754321006523946 and parameters: {'n_estimators': 79, 'learning_rate': 0.01124057661864214, 'num_leaves': 234, 'max_depth': 9, 'min_child_samples': 17, 'subsample': 0.6165013893903719, 'colsample_bytree': 0.9559635086247678, 'reg_alpha': 2.1536022065386815, 'reg_lambda': 4.133276934380879, 'min_split_gain': 0.0295877395044678}. Best is trial 2 with value: 0.0004942404150330594.
[I 2025-06-15 20:46:55,395] A new study created in memory with name: no-name-9e9ddd77-28b5-4b80-b25e-2460995f8739
[I 2025-06-15 20:46:55,419] Trial 0 finished with value: 0.0022322144860201145 and parameters: {'n_estimators': 53, 'learning_rate': 0.1851041769258515, 'num_leaves': 163, 'max_depth': 4, 'min_child_samples': 16, 'subsample': 0.8994030164903015, 'colsample_bytree': 0.9615796581360162, 'reg_alpha': 3.7827035095968107, 'reg_lambda': 4.142610857921457, 'min_split_gain': 0.030949785376200314}. Best is trial 0 with value: 0.0022322144860201145.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         396
14      pct_chg_Load_DA         144
1           Price_lag_2         122
16  roll2d_median_Price          65
3         Load_DA_lag_0          44
2           Price_lag_7          44
11           NGas_lag_2          31
13            EUA_lag_2          23
15       lag168_Load_DA          22
10           Coal_lag_2          19
12            Oil_lag_2          13
5       WindOn_DA_lag_0          12
9                  WD_7           7
8                  WD_6           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1           9
5       WindOn_DA_lag_0           3
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 92.61% done
********************* END NS.. 676 ****************************************************
********************* START NS ... 677 ****************************************************
[I 2025-06-15 20:46:56,887] A new study created in memory with name: no-name-79524d7d-62ba-4b34-87bd-c76fce6217dd
[I 2025-06-15 20:46:57,249] Trial 0 finished with value: 0.0010992068209374083 and parameters: {'n_estimators': 126, 'learning_rate': 0.2247692475679934, 'num_leaves': 242, 'max_depth': 6, 'min_child_samples': 14, 'subsample': 0.9259477205669162, 'colsample_bytree': 0.5608733803531901, 'reg_alpha': 8.112409114267633, 'reg_lambda': 0.8209804025501988, 'min_split_gain': 0.04324486874308213}. Best is trial 0 with value: 0.0010992068209374083.
[Trial 0] New best: 0.00110
[I 2025-06-15 20:46:57,797] Trial 1 finished with value: 0.000981299720739337 and parameters: {'n_estimators': 293, 'learning_rate': 0.1630577201073912, 'num_leaves': 145, 'max_depth': 7, 'min_child_samples': 1, 'subsample': 0.5258336416170015, 'colsample_bytree': 0.6101386420610492, 'reg_alpha': 4.626687950544113, 'reg_lambda': 8.727794753288428, 'min_split_gain': 0.021218493459936785}. Best is trial 1 with value: 0.000981299720739337.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:46:59,083] Trial 2 finished with value: 0.0010211681644467924 and parameters: {'n_estimators': 106, 'learning_rate': 0.04065156810639193, 'num_leaves': 56, 'max_depth': 13, 'min_child_samples': 6, 'subsample': 0.5762994452280825, 'colsample_bytree': 0.558687193259763, 'reg_alpha': 2.4329648704503737, 'reg_lambda': 5.871483951583313, 'min_split_gain': 0.0030400377739818354}. Best is trial 1 with value: 0.000981299720739337.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:46:59,554] Trial 3 finished with value: 0.0005550427120398032 and parameters: {'n_estimators': 168, 'learning_rate': 0.029237281936353127, 'num_leaves': 52, 'max_depth': 4, 'min_child_samples': 17, 'subsample': 0.8921292475104295, 'colsample_bytree': 0.9487423115480178, 'reg_alpha': 8.961774980582934, 'reg_lambda': 7.821646647359635, 'min_split_gain': 0.004310152945877305}. Best is trial 3 with value: 0.0005550427120398032.
[I 2025-06-15 20:46:59,557] A new study created in memory with name: no-name-73c14a11-e873-4580-ac3a-e0afb9b8aaf0
[I 2025-06-15 20:46:59,584] Trial 0 finished with value: 0.006108167646063534 and parameters: {'n_estimators': 55, 'learning_rate': 0.10675198955037134, 'num_leaves': 37, 'max_depth': 4, 'min_child_samples': 29, 'subsample': 0.9954533642763475, 'colsample_bytree': 0.9023793220203358, 'reg_alpha': 6.549317395752194, 'reg_lambda': 4.728542871331657, 'min_split_gain': 0.002294182463054534}. Best is trial 0 with value: 0.006108167646063534.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1015
14      pct_chg_Load_DA         409
1           Price_lag_2         129
2           Price_lag_7         128
3         Load_DA_lag_0         121
16  roll2d_median_Price         113
11           NGas_lag_2          69
13            EUA_lag_2          63
12            Oil_lag_2          54
5       WindOn_DA_lag_0          38
15       lag168_Load_DA          27
10           Coal_lag_2          23
9                  WD_7          13
8                  WD_6           4
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          21
5       WindOn_DA_lag_0           8
15       lag168_Load_DA           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 92.75% done
********************* END NS.. 677 ****************************************************
********************* START NS ... 678 ****************************************************
[I 2025-06-15 20:47:01,443] A new study created in memory with name: no-name-58fb202d-6210-4970-8802-cc51e6a813fe
[I 2025-06-15 20:47:03,047] Trial 0 finished with value: 0.0010427558469264656 and parameters: {'n_estimators': 201, 'learning_rate': 0.017324430498220276, 'num_leaves': 64, 'max_depth': 9, 'min_child_samples': 13, 'subsample': 0.9778652485465353, 'colsample_bytree': 0.868998237794105, 'reg_alpha': 3.216840656257862, 'reg_lambda': 3.8531160020536026, 'min_split_gain': 0.0492842083257851}. Best is trial 0 with value: 0.0010427558469264656.
[Trial 0] New best: 0.00104
[I 2025-06-15 20:47:03,985] Trial 1 finished with value: 0.0006593764387487567 and parameters: {'n_estimators': 113, 'learning_rate': 0.09226603708544284, 'num_leaves': 254, 'max_depth': 15, 'min_child_samples': 17, 'subsample': 0.6719227942069246, 'colsample_bytree': 0.7497418001865706, 'reg_alpha': 1.350371988421838, 'reg_lambda': 6.009924399558752, 'min_split_gain': 0.043969299235205705}. Best is trial 1 with value: 0.0006593764387487567.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:47:04,271] Trial 2 finished with value: 0.0006657754863128649 and parameters: {'n_estimators': 297, 'learning_rate': 0.2749516496226669, 'num_leaves': 75, 'max_depth': 16, 'min_child_samples': 2, 'subsample': 0.5209555869324378, 'colsample_bytree': 0.9259766460044678, 'reg_alpha': 9.768778302119102, 'reg_lambda': 1.3469586693584112, 'min_split_gain': 0.049323447210776646}. Best is trial 1 with value: 0.0006593764387487567.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:47:04,839] Trial 3 finished with value: 0.0009431835457260675 and parameters: {'n_estimators': 66, 'learning_rate': 0.1459814201997499, 'num_leaves': 180, 'max_depth': 8, 'min_child_samples': 14, 'subsample': 0.8773608553266428, 'colsample_bytree': 0.6199321214580044, 'reg_alpha': 0.766644709186034, 'reg_lambda': 5.444032652912954, 'min_split_gain': 0.03968441696549098}. Best is trial 1 with value: 0.0006593764387487567.
[I 2025-06-15 20:47:04,841] A new study created in memory with name: no-name-f167de07-b096-424f-a67e-17289f32f389
[I 2025-06-15 20:47:04,878] Trial 0 finished with value: 0.001953843185508398 and parameters: {'n_estimators': 86, 'learning_rate': 0.21357806991345277, 'num_leaves': 214, 'max_depth': 4, 'min_child_samples': 10, 'subsample': 0.5449552089465723, 'colsample_bytree': 0.6070110012037622, 'reg_alpha': 1.3962170290921072, 'reg_lambda': 9.349332384021231, 'min_split_gain': 0.01647579631328591}. Best is trial 0 with value: 0.001953843185508398.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         685
1           Price_lag_2         583
14      pct_chg_Load_DA         528
3         Load_DA_lag_0         403
16  roll2d_median_Price         262
5       WindOn_DA_lag_0         235
2           Price_lag_7         232
11           NGas_lag_2         212
15       lag168_Load_DA         210
13            EUA_lag_2         130
10           Coal_lag_2         119
12            Oil_lag_2          84
9                  WD_7          13
7                  WD_1           6
8                  WD_6           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          10
15       lag168_Load_DA           6
5       WindOn_DA_lag_0           5
3         Load_DA_lag_0           3
7                  WD_1           3
16  roll2d_median_Price           2
1           Price_lag_2           1
10           Coal_lag_2           1
11           NGas_lag_2           1
12            Oil_lag_2           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
9                  WD_7           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 92.89% done
********************* END NS.. 678 ****************************************************
********************* START NS ... 679 ****************************************************
[I 2025-06-15 20:47:06,931] A new study created in memory with name: no-name-b6e26f3a-5598-4cbf-80f5-a79468cd8554
[I 2025-06-15 20:47:08,567] Trial 0 finished with value: 0.005118681010670666 and parameters: {'n_estimators': 193, 'learning_rate': 0.019420742209980732, 'num_leaves': 60, 'max_depth': 9, 'min_child_samples': 19, 'subsample': 0.6058823199325827, 'colsample_bytree': 0.6737390212136573, 'reg_alpha': 8.371903961642026, 'reg_lambda': 8.246876834141299, 'min_split_gain': 0.0017591943756128882}. Best is trial 0 with value: 0.005118681010670666.
[Trial 0] New best: 0.00512
[I 2025-06-15 20:47:09,978] Trial 1 finished with value: 0.005277891614097592 and parameters: {'n_estimators': 246, 'learning_rate': 0.07416837427478831, 'num_leaves': 91, 'max_depth': 15, 'min_child_samples': 14, 'subsample': 0.5029894533337238, 'colsample_bytree': 0.9517246934065042, 'reg_alpha': 2.207351647736453, 'reg_lambda': 6.70769800458685, 'min_split_gain': 0.0159037939670968}. Best is trial 0 with value: 0.005118681010670666.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:47:10,360] Trial 2 finished with value: 0.005155333280214299 and parameters: {'n_estimators': 85, 'learning_rate': 0.044309265522016035, 'num_leaves': 255, 'max_depth': 5, 'min_child_samples': 2, 'subsample': 0.5733127179666143, 'colsample_bytree': 0.8727259239902248, 'reg_alpha': 7.527164918204977, 'reg_lambda': 5.5490854532875105, 'min_split_gain': 0.04847002570631812}. Best is trial 0 with value: 0.005118681010670666.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:47:11,304] Trial 3 finished with value: 0.02984393610551413 and parameters: {'n_estimators': 62, 'learning_rate': 0.023450120956196703, 'num_leaves': 225, 'max_depth': 15, 'min_child_samples': 1, 'subsample': 0.7679287907085192, 'colsample_bytree': 0.8305291543963302, 'reg_alpha': 2.8007696158995943, 'reg_lambda': 3.6167809904082624, 'min_split_gain': 0.04685774486734523}. Best is trial 0 with value: 0.005118681010670666.
[I 2025-06-15 20:47:11,306] A new study created in memory with name: no-name-c2bd5133-37ba-4541-a5e3-6259b64f801b
[I 2025-06-15 20:47:11,355] Trial 0 finished with value: 0.01738708401525776 and parameters: {'n_estimators': 174, 'learning_rate': 0.24081706965603095, 'num_leaves': 51, 'max_depth': 4, 'min_child_samples': 14, 'subsample': 0.7957586029050909, 'colsample_bytree': 0.9160974188013888, 'reg_alpha': 0.02470267526544001, 'reg_lambda': 7.348378121223611, 'min_split_gain': 0.023265828994574002}. Best is trial 0 with value: 0.01738708401525776.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1682
1           Price_lag_2        1255
14      pct_chg_Load_DA         816
16  roll2d_median_Price         487
2           Price_lag_7         460
3         Load_DA_lag_0         359
11           NGas_lag_2         271
13            EUA_lag_2         265
10           Coal_lag_2         228
15       lag168_Load_DA         221
5       WindOn_DA_lag_0         163
12            Oil_lag_2         157
9                  WD_7          42
8                  WD_6          30
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1          15
15       lag168_Load_DA           8
11           NGas_lag_2           4
7                  WD_1           3
16  roll2d_median_Price           3
3         Load_DA_lag_0           2
10           Coal_lag_2           1
12            Oil_lag_2           1
2           Price_lag_7           1
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
1           Price_lag_2           0
9                  WD_7           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 93.02% done
********************* END NS.. 679 ****************************************************
********************* START NS ... 680 ****************************************************
[I 2025-06-15 20:47:13,944] A new study created in memory with name: no-name-e0cefd7a-db41-4a5c-a044-16182f11e9eb
[I 2025-06-15 20:47:14,178] Trial 0 finished with value: 0.007131558290529086 and parameters: {'n_estimators': 61, 'learning_rate': 0.1760834905972148, 'num_leaves': 183, 'max_depth': 15, 'min_child_samples': 25, 'subsample': 0.9916078336836347, 'colsample_bytree': 0.8832476640610016, 'reg_alpha': 8.492479279450274, 'reg_lambda': 6.054018380281999, 'min_split_gain': 0.018593125268519972}. Best is trial 0 with value: 0.007131558290529086.
[Trial 0] New best: 0.00713
[I 2025-06-15 20:47:15,174] Trial 1 finished with value: 0.006647740537626102 and parameters: {'n_estimators': 183, 'learning_rate': 0.0475782175098221, 'num_leaves': 62, 'max_depth': 14, 'min_child_samples': 8, 'subsample': 0.9104791438009223, 'colsample_bytree': 0.7610256404632088, 'reg_alpha': 5.547496611512276, 'reg_lambda': 9.711041901187246, 'min_split_gain': 0.047825079864606594}. Best is trial 1 with value: 0.006647740537626102.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:47:15,712] Trial 2 finished with value: 0.006187178415992225 and parameters: {'n_estimators': 198, 'learning_rate': 0.08784830078057294, 'num_leaves': 138, 'max_depth': 5, 'min_child_samples': 2, 'subsample': 0.6274093976658548, 'colsample_bytree': 0.5791069967098275, 'reg_alpha': 9.29769596532813, 'reg_lambda': 5.587512212216247, 'min_split_gain': 0.015532595019724755}. Best is trial 2 with value: 0.006187178415992225.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:47:17,989] Trial 3 finished with value: 0.0070066854166801204 and parameters: {'n_estimators': 243, 'learning_rate': 0.07169912373905081, 'num_leaves': 153, 'max_depth': 16, 'min_child_samples': 3, 'subsample': 0.559832776244872, 'colsample_bytree': 0.9293573265233506, 'reg_alpha': 5.474199690643572, 'reg_lambda': 2.7402259489277525, 'min_split_gain': 0.0034595571156416705}. Best is trial 2 with value: 0.006187178415992225.
[I 2025-06-15 20:47:17,990] A new study created in memory with name: no-name-6df0f7ed-7dee-4228-ace3-eb18430725d3
[I 2025-06-15 20:47:18,022] Trial 0 finished with value: 0.011367408012626893 and parameters: {'n_estimators': 103, 'learning_rate': 0.13212156384578708, 'num_leaves': 79, 'max_depth': 12, 'min_child_samples': 6, 'subsample': 0.8482129909383294, 'colsample_bytree': 0.7115230531325067, 'reg_alpha': 2.4603936610245136, 'reg_lambda': 4.761131332764085, 'min_split_gain': 0.04074679067474417}. Best is trial 0 with value: 0.011367408012626893.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         575
1           Price_lag_2         397
14      pct_chg_Load_DA         226
2           Price_lag_7         181
3         Load_DA_lag_0         152
11           NGas_lag_2         130
5       WindOn_DA_lag_0         118
15       lag168_Load_DA         112
10           Coal_lag_2          91
16  roll2d_median_Price          88
12            Oil_lag_2          82
13            EUA_lag_2          66
9                  WD_7          19
8                  WD_6          14
7                  WD_1           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           4
15       lag168_Load_DA           1
9                  WD_7           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 93.16% done
********************* END NS.. 680 ****************************************************
********************* START NS ... 681 ****************************************************
[I 2025-06-15 20:47:19,909] A new study created in memory with name: no-name-3f573e59-6e51-41b4-a873-c17e31bd5f6d
[I 2025-06-15 20:47:20,220] Trial 0 finished with value: 0.004967330975276061 and parameters: {'n_estimators': 129, 'learning_rate': 0.22810148304751152, 'num_leaves': 100, 'max_depth': 13, 'min_child_samples': 8, 'subsample': 0.5808262370213181, 'colsample_bytree': 0.6375697980454089, 'reg_alpha': 9.411411373678767, 'reg_lambda': 4.009848778339785, 'min_split_gain': 0.015035731262770469}. Best is trial 0 with value: 0.004967330975276061.
[Trial 0] New best: 0.00497
[I 2025-06-15 20:47:20,704] Trial 1 finished with value: 0.004965735377294983 and parameters: {'n_estimators': 287, 'learning_rate': 0.10483328297750745, 'num_leaves': 68, 'max_depth': 14, 'min_child_samples': 6, 'subsample': 0.8683650414053611, 'colsample_bytree': 0.6203326713108026, 'reg_alpha': 9.676919368508191, 'reg_lambda': 1.3428046959401851, 'min_split_gain': 0.04324736880469046}. Best is trial 1 with value: 0.004965735377294983.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:47:23,235] Trial 2 finished with value: 0.00502805950956696 and parameters: {'n_estimators': 135, 'learning_rate': 0.04498003874460962, 'num_leaves': 238, 'max_depth': 11, 'min_child_samples': 19, 'subsample': 0.7364562056746475, 'colsample_bytree': 0.5002598023470364, 'reg_alpha': 2.62803824060318, 'reg_lambda': 7.997394696940092, 'min_split_gain': 0.0003127017770944984}. Best is trial 1 with value: 0.004965735377294983.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:47:23,571] Trial 3 finished with value: 0.005012030842244602 and parameters: {'n_estimators': 149, 'learning_rate': 0.16928614639062442, 'num_leaves': 235, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.6528917115983859, 'colsample_bytree': 0.8756181631392943, 'reg_alpha': 6.535896710212068, 'reg_lambda': 2.2605967137492544, 'min_split_gain': 0.014562632486094303}. Best is trial 1 with value: 0.004965735377294983.
[I 2025-06-15 20:47:23,573] A new study created in memory with name: no-name-461121f1-b2b5-403a-8588-71d220f257b0
[I 2025-06-15 20:47:23,594] Trial 0 finished with value: 0.012844946293768552 and parameters: {'n_estimators': 53, 'learning_rate': 0.12687078529629386, 'num_leaves': 120, 'max_depth': 13, 'min_child_samples': 12, 'subsample': 0.9978277210784754, 'colsample_bytree': 0.8230729974742274, 'reg_alpha': 9.376028953427792, 'reg_lambda': 6.926622473732297, 'min_split_gain': 0.010803331642818444}. Best is trial 0 with value: 0.012844946293768552.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         374
1           Price_lag_2         239
14      pct_chg_Load_DA         194
3         Load_DA_lag_0         117
11           NGas_lag_2         110
2           Price_lag_7         103
16  roll2d_median_Price          90
15       lag168_Load_DA          69
5       WindOn_DA_lag_0          63
10           Coal_lag_2          54
13            EUA_lag_2          45
12            Oil_lag_2          27
8                  WD_6          13
9                  WD_7          12
7                  WD_1           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 93.30% done
********************* END NS.. 681 ****************************************************
********************* START NS ... 682 ****************************************************
[I 2025-06-15 20:47:25,259] A new study created in memory with name: no-name-eabe118d-c365-4c1e-8dfb-76aed21c74c0
[I 2025-06-15 20:47:25,847] Trial 0 finished with value: 0.03255876663107254 and parameters: {'n_estimators': 120, 'learning_rate': 0.019669129972135196, 'num_leaves': 210, 'max_depth': 6, 'min_child_samples': 13, 'subsample': 0.910939775747106, 'colsample_bytree': 0.7856186689552331, 'reg_alpha': 7.319133021893692, 'reg_lambda': 6.524812875861928, 'min_split_gain': 0.04292358963603423}. Best is trial 0 with value: 0.03255876663107254.
[Trial 0] New best: 0.03256
[I 2025-06-15 20:47:26,431] Trial 1 finished with value: 0.02643984846331893 and parameters: {'n_estimators': 167, 'learning_rate': 0.16821249514154385, 'num_leaves': 119, 'max_depth': 12, 'min_child_samples': 28, 'subsample': 0.7332751700035542, 'colsample_bytree': 0.8718544473824787, 'reg_alpha': 1.6382703779506724, 'reg_lambda': 8.316724234187825, 'min_split_gain': 0.039595368504447476}. Best is trial 1 with value: 0.02643984846331893.
[Trial 1] New best: 0.02644
[I 2025-06-15 20:47:26,789] Trial 2 finished with value: 0.03000694401643826 and parameters: {'n_estimators': 217, 'learning_rate': 0.20413508265523245, 'num_leaves': 187, 'max_depth': 11, 'min_child_samples': 1, 'subsample': 0.9365815484248059, 'colsample_bytree': 0.8505870071891555, 'reg_alpha': 8.728982460887403, 'reg_lambda': 5.086196716852744, 'min_split_gain': 0.02220019957743702}. Best is trial 1 with value: 0.02643984846331893.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:47:28,036] Trial 3 finished with value: 0.02861663958548812 and parameters: {'n_estimators': 287, 'learning_rate': 0.10171990511659158, 'num_leaves': 119, 'max_depth': 12, 'min_child_samples': 15, 'subsample': 0.994667072361515, 'colsample_bytree': 0.5404755367339682, 'reg_alpha': 2.5398808352351554, 'reg_lambda': 4.344534684954899, 'min_split_gain': 0.020262777231983956}. Best is trial 1 with value: 0.02643984846331893.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:47:28,954] Trial 4 finished with value: 0.028422199827729682 and parameters: {'n_estimators': 199, 'learning_rate': 0.06177580482190501, 'num_leaves': 234, 'max_depth': 9, 'min_child_samples': 3, 'subsample': 0.6597788701817366, 'colsample_bytree': 0.7628997157143885, 'reg_alpha': 6.503805386277622, 'reg_lambda': 7.99417167467995, 'min_split_gain': 0.0368262813033722}. Best is trial 1 with value: 0.02643984846331893.
[I 2025-06-15 20:47:28,957] A new study created in memory with name: no-name-676cd1ff-af08-4535-b0b1-2e5a78a509b8
[I 2025-06-15 20:47:29,005] Trial 0 finished with value: 0.15639173303145282 and parameters: {'n_estimators': 207, 'learning_rate': 0.09008889682624469, 'num_leaves': 94, 'max_depth': 6, 'min_child_samples': 4, 'subsample': 0.5986792733027355, 'colsample_bytree': 0.9142222017428434, 'reg_alpha': 4.035306827645798, 'reg_lambda': 5.858059236001924, 'min_split_gain': 0.03854626428789016}. Best is trial 0 with value: 0.15639173303145282.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         385
1           Price_lag_2         285
14      pct_chg_Load_DA         272
3         Load_DA_lag_0         156
16  roll2d_median_Price         128
2           Price_lag_7         117
15       lag168_Load_DA         109
5       WindOn_DA_lag_0         108
11           NGas_lag_2          93
13            EUA_lag_2          63
10           Coal_lag_2          57
12            Oil_lag_2          45
7                  WD_1           6
9                  WD_7           6
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          23
15       lag168_Load_DA           9
16  roll2d_median_Price           4
3         Load_DA_lag_0           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 93.43% done
********************* END NS.. 682 ****************************************************
********************* START NS ... 683 ****************************************************
[I 2025-06-15 20:47:30,728] A new study created in memory with name: no-name-3556673f-1472-40c4-8a7e-9adcbc748ba4
[I 2025-06-15 20:47:31,770] Trial 0 finished with value: 0.03430108875600357 and parameters: {'n_estimators': 175, 'learning_rate': 0.011152507329675947, 'num_leaves': 241, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.7351795395542433, 'colsample_bytree': 0.7164101253614346, 'reg_alpha': 8.669331125144705, 'reg_lambda': 6.049206263523211, 'min_split_gain': 0.04851789793269754}. Best is trial 0 with value: 0.03430108875600357.
[Trial 0] New best: 0.03430
[I 2025-06-15 20:47:34,692] Trial 1 finished with value: 0.025196474072953667 and parameters: {'n_estimators': 214, 'learning_rate': 0.06448597474237534, 'num_leaves': 202, 'max_depth': 16, 'min_child_samples': 21, 'subsample': 0.8810786227892725, 'colsample_bytree': 0.673170005382596, 'reg_alpha': 3.844456616695211, 'reg_lambda': 5.379566737771874, 'min_split_gain': 0.0012947035561561183}. Best is trial 1 with value: 0.025196474072953667.
[Trial 1] New best: 0.02520
[I 2025-06-15 20:47:36,006] Trial 2 finished with value: 0.02668515921886543 and parameters: {'n_estimators': 294, 'learning_rate': 0.0458841331571458, 'num_leaves': 171, 'max_depth': 10, 'min_child_samples': 1, 'subsample': 0.5209898678014567, 'colsample_bytree': 0.8006735838730018, 'reg_alpha': 3.521393926425327, 'reg_lambda': 8.674591794387595, 'min_split_gain': 0.048408013234070235}. Best is trial 1 with value: 0.025196474072953667.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:47:36,291] Trial 3 finished with value: 0.02564296736591103 and parameters: {'n_estimators': 183, 'learning_rate': 0.19518905771778217, 'num_leaves': 138, 'max_depth': 8, 'min_child_samples': 14, 'subsample': 0.8895991943422172, 'colsample_bytree': 0.675042540141865, 'reg_alpha': 9.317169304003633, 'reg_lambda': 3.8333838588066325, 'min_split_gain': 0.03545478316136802}. Best is trial 1 with value: 0.025196474072953667.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:47:37,048] Trial 4 finished with value: 0.07265160214307796 and parameters: {'n_estimators': 62, 'learning_rate': 0.014702281799527997, 'num_leaves': 39, 'max_depth': 14, 'min_child_samples': 14, 'subsample': 0.5307904281293085, 'colsample_bytree': 0.8951292700420392, 'reg_alpha': 9.758867316025663, 'reg_lambda': 0.9268301448824967, 'min_split_gain': 0.03496010947026115}. Best is trial 1 with value: 0.025196474072953667.
[I 2025-06-15 20:47:37,051] A new study created in memory with name: no-name-55f7a6f8-16b4-4747-844b-a9efbe59835f
[I 2025-06-15 20:47:37,113] Trial 0 finished with value: 0.12913739109296454 and parameters: {'n_estimators': 271, 'learning_rate': 0.08719436675401311, 'num_leaves': 31, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.554949491832759, 'colsample_bytree': 0.9649060752127706, 'reg_alpha': 0.926521283669155, 'reg_lambda': 6.457908651350094, 'min_split_gain': 0.043115217955809054}. Best is trial 0 with value: 0.12913739109296454.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1825
1           Price_lag_2        1730
14      pct_chg_Load_DA        1331
5       WindOn_DA_lag_0        1229
15       lag168_Load_DA        1178
3         Load_DA_lag_0        1101
13            EUA_lag_2         892
2           Price_lag_7         889
11           NGas_lag_2         698
12            Oil_lag_2         680
16  roll2d_median_Price         670
10           Coal_lag_2         607
9                  WD_7          71
8                  WD_6          47
7                  WD_1          31
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          38
15       lag168_Load_DA          13
11           NGas_lag_2          11
2           Price_lag_7           8
3         Load_DA_lag_0           8
16  roll2d_median_Price           6
12            Oil_lag_2           3
1           Price_lag_2           2
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
5       WindOn_DA_lag_0           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 93.57% done
********************* END NS.. 683 ****************************************************
********************* START NS ... 684 ****************************************************
[I 2025-06-15 20:47:40,903] A new study created in memory with name: no-name-7f4fb629-bad0-4f21-8a5a-47c808fc3672
[I 2025-06-15 20:47:41,370] Trial 0 finished with value: 0.0013705868610621754 and parameters: {'n_estimators': 244, 'learning_rate': 0.07015294299102176, 'num_leaves': 221, 'max_depth': 9, 'min_child_samples': 2, 'subsample': 0.7833283153252297, 'colsample_bytree': 0.7969004529622326, 'reg_alpha': 9.177716770239323, 'reg_lambda': 9.920216937364653, 'min_split_gain': 0.03812933684067506}. Best is trial 0 with value: 0.0013705868610621754.
[Trial 0] New best: 0.00137
[I 2025-06-15 20:47:42,115] Trial 1 finished with value: 0.0016183932769975823 and parameters: {'n_estimators': 276, 'learning_rate': 0.06579257503996432, 'num_leaves': 169, 'max_depth': 5, 'min_child_samples': 12, 'subsample': 0.8735873112251303, 'colsample_bytree': 0.7366113262549199, 'reg_alpha': 3.143306818018685, 'reg_lambda': 2.9802467948950087, 'min_split_gain': 0.02644491619425149}. Best is trial 0 with value: 0.0013705868610621754.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:47:44,187] Trial 2 finished with value: 0.0015827557149483103 and parameters: {'n_estimators': 162, 'learning_rate': 0.068019891155634, 'num_leaves': 109, 'max_depth': 12, 'min_child_samples': 1, 'subsample': 0.9334782941009105, 'colsample_bytree': 0.8553446731350614, 'reg_alpha': 1.7667848120742624, 'reg_lambda': 5.492752055411465, 'min_split_gain': 0.013624741393329582}. Best is trial 0 with value: 0.0013705868610621754.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:47:44,654] Trial 3 finished with value: 0.0681793955388003 and parameters: {'n_estimators': 75, 'learning_rate': 0.011452748407029516, 'num_leaves': 224, 'max_depth': 10, 'min_child_samples': 30, 'subsample': 0.6043011888700451, 'colsample_bytree': 0.8175686920998578, 'reg_alpha': 8.647847110964051, 'reg_lambda': 8.495172239871309, 'min_split_gain': 0.03708125647056797}. Best is trial 0 with value: 0.0013705868610621754.
[I 2025-06-15 20:47:44,656] A new study created in memory with name: no-name-c699b316-0da3-487c-b578-817b209b35f9
[I 2025-06-15 20:47:44,701] Trial 0 finished with value: 0.004041085591541787 and parameters: {'n_estimators': 164, 'learning_rate': 0.11660664724097279, 'num_leaves': 122, 'max_depth': 10, 'min_child_samples': 29, 'subsample': 0.7873871829209207, 'colsample_bytree': 0.7464948930677437, 'reg_alpha': 5.0735055331173236, 'reg_lambda': 2.141875308110184, 'min_split_gain': 0.010454076081817698}. Best is trial 0 with value: 0.004041085591541787.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         619
1           Price_lag_2         383
14      pct_chg_Load_DA         369
2           Price_lag_7         210
11           NGas_lag_2         199
3         Load_DA_lag_0         182
16  roll2d_median_Price         160
5       WindOn_DA_lag_0         123
15       lag168_Load_DA         104
10           Coal_lag_2         103
13            EUA_lag_2          85
12            Oil_lag_2          55
9                  WD_7          18
8                  WD_6          12
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          21
3         Load_DA_lag_0           4
16  roll2d_median_Price           1
2           Price_lag_7           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
15       lag168_Load_DA           0
8                  WD_6           0
-> 93.71% done
********************* END NS.. 684 ****************************************************
********************* START NS ... 685 ****************************************************
[I 2025-06-15 20:47:46,688] A new study created in memory with name: no-name-cd680412-dfd2-498c-8f23-aea142b7fae2
[I 2025-06-15 20:47:47,857] Trial 0 finished with value: 0.0010486840325523415 and parameters: {'n_estimators': 212, 'learning_rate': 0.026410001872359357, 'num_leaves': 247, 'max_depth': 14, 'min_child_samples': 4, 'subsample': 0.8398258612481272, 'colsample_bytree': 0.9301955245587783, 'reg_alpha': 9.689280259363093, 'reg_lambda': 5.656842156551435, 'min_split_gain': 0.033383802058885886}. Best is trial 0 with value: 0.0010486840325523415.
[Trial 0] New best: 0.00105
[I 2025-06-15 20:47:48,362] Trial 1 finished with value: 0.0011145891950854235 and parameters: {'n_estimators': 238, 'learning_rate': 0.09069617592520114, 'num_leaves': 47, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.5511945171147552, 'colsample_bytree': 0.739920429536511, 'reg_alpha': 6.394048724561582, 'reg_lambda': 3.515031948721737, 'min_split_gain': 0.028592401567208554}. Best is trial 0 with value: 0.0010486840325523415.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:47:48,925] Trial 2 finished with value: 0.0010430508175285026 and parameters: {'n_estimators': 110, 'learning_rate': 0.1666074839654503, 'num_leaves': 79, 'max_depth': 10, 'min_child_samples': 29, 'subsample': 0.7716847821826177, 'colsample_bytree': 0.8779052729281376, 'reg_alpha': 1.7147300528054032, 'reg_lambda': 2.5753412434745226, 'min_split_gain': 0.014472942350422797}. Best is trial 2 with value: 0.0010430508175285026.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:47:50,564] Trial 3 finished with value: 0.001709754905137914 and parameters: {'n_estimators': 271, 'learning_rate': 0.01268560923933517, 'num_leaves': 57, 'max_depth': 15, 'min_child_samples': 25, 'subsample': 0.7799962090372733, 'colsample_bytree': 0.693961178633933, 'reg_alpha': 9.953787529466016, 'reg_lambda': 2.2914305161063964, 'min_split_gain': 0.00677914209736934}. Best is trial 2 with value: 0.0010430508175285026.
[I 2025-06-15 20:47:50,567] A new study created in memory with name: no-name-94b8eaec-6566-42c3-b3e3-d623777515e4
[I 2025-06-15 20:47:50,607] Trial 0 finished with value: 0.0034688119830681464 and parameters: {'n_estimators': 160, 'learning_rate': 0.11243350181369687, 'num_leaves': 236, 'max_depth': 9, 'min_child_samples': 28, 'subsample': 0.7488089825700757, 'colsample_bytree': 0.7960231370814247, 'reg_alpha': 9.385091276611137, 'reg_lambda': 2.9144442160685182, 'min_split_gain': 0.020914478956281752}. Best is trial 0 with value: 0.0034688119830681464.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         476
1           Price_lag_2         372
14      pct_chg_Load_DA         352
3         Load_DA_lag_0         223
16  roll2d_median_Price         184
15       lag168_Load_DA         179
2           Price_lag_7         177
5       WindOn_DA_lag_0         174
11           NGas_lag_2         161
13            EUA_lag_2         118
10           Coal_lag_2         116
12            Oil_lag_2          54
9                  WD_7          13
7                  WD_1          12
8                  WD_6           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 93.84% done
********************* END NS.. 685 ****************************************************
********************* START NS ... 686 ****************************************************
[I 2025-06-15 20:47:52,416] A new study created in memory with name: no-name-91a009ae-8419-4924-8d6b-c0509f7adb42
[I 2025-06-15 20:47:53,470] Trial 0 finished with value: 0.0005546944398328975 and parameters: {'n_estimators': 203, 'learning_rate': 0.035459768394700185, 'num_leaves': 241, 'max_depth': 10, 'min_child_samples': 9, 'subsample': 0.6300426133092698, 'colsample_bytree': 0.8019987355842, 'reg_alpha': 5.774586560502376, 'reg_lambda': 8.768470100213996, 'min_split_gain': 0.03694015287769544}. Best is trial 0 with value: 0.0005546944398328975.
[Trial 0] New best: 0.00055
[I 2025-06-15 20:47:54,924] Trial 1 finished with value: 0.0006125744053905412 and parameters: {'n_estimators': 184, 'learning_rate': 0.05341238462357922, 'num_leaves': 172, 'max_depth': 7, 'min_child_samples': 29, 'subsample': 0.8854708205751116, 'colsample_bytree': 0.7929864427085442, 'reg_alpha': 0.6253425336075391, 'reg_lambda': 8.370118834932072, 'min_split_gain': 0.0004860346670923166}. Best is trial 0 with value: 0.0005546944398328975.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:47:56,167] Trial 2 finished with value: 0.002441222018358494 and parameters: {'n_estimators': 195, 'learning_rate': 0.013411787853964536, 'num_leaves': 170, 'max_depth': 8, 'min_child_samples': 22, 'subsample': 0.5522465814291619, 'colsample_bytree': 0.919817176168084, 'reg_alpha': 8.914540178044835, 'reg_lambda': 9.201408520496955, 'min_split_gain': 0.04651027951135545}. Best is trial 0 with value: 0.0005546944398328975.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:47:56,504] Trial 3 finished with value: 0.0008232676815033792 and parameters: {'n_estimators': 156, 'learning_rate': 0.17110392999521162, 'num_leaves': 38, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.6504553739387378, 'colsample_bytree': 0.6222567277577544, 'reg_alpha': 0.025541961315718176, 'reg_lambda': 8.229264763258952, 'min_split_gain': 0.037442446654816865}. Best is trial 0 with value: 0.0005546944398328975.
[I 2025-06-15 20:47:56,506] A new study created in memory with name: no-name-f598c03f-2d4a-42b6-aa37-b2e51f3d9be1
[I 2025-06-15 20:47:56,575] Trial 0 finished with value: 0.0033261934782584327 and parameters: {'n_estimators': 268, 'learning_rate': 0.2804184077477833, 'num_leaves': 169, 'max_depth': 13, 'min_child_samples': 8, 'subsample': 0.5973169042265882, 'colsample_bytree': 0.645693403843073, 'reg_alpha': 2.4347670868061675, 'reg_lambda': 8.016612101209898, 'min_split_gain': 0.027544904093759405}. Best is trial 0 with value: 0.0033261934782584327.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1381
1           Price_lag_2         952
14      pct_chg_Load_DA         825
3         Load_DA_lag_0         489
2           Price_lag_7         376
16  roll2d_median_Price         359
11           NGas_lag_2         301
5       WindOn_DA_lag_0         222
15       lag168_Load_DA         191
10           Coal_lag_2         184
13            EUA_lag_2         167
12            Oil_lag_2         116
9                  WD_7          31
8                  WD_6          28
7                  WD_1           6
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          12
3         Load_DA_lag_0           4
15       lag168_Load_DA           2
13            EUA_lag_2           1
9                  WD_7           0
14      pct_chg_Load_DA           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 93.98% done
********************* END NS.. 686 ****************************************************
********************* START NS ... 687 ****************************************************
[I 2025-06-15 20:47:58,998] A new study created in memory with name: no-name-92794253-3bbd-4a9b-bbe6-206a47471220
[I 2025-06-15 20:48:00,539] Trial 0 finished with value: 0.0004086819498064812 and parameters: {'n_estimators': 300, 'learning_rate': 0.10039262836704012, 'num_leaves': 68, 'max_depth': 10, 'min_child_samples': 10, 'subsample': 0.6121972105769249, 'colsample_bytree': 0.7431971565913066, 'reg_alpha': 1.5946747149751472, 'reg_lambda': 5.026114690143673, 'min_split_gain': 0.004448336120614083}. Best is trial 0 with value: 0.0004086819498064812.
[Trial 0] New best: 0.00041
[I 2025-06-15 20:48:01,169] Trial 1 finished with value: 0.04263636863552123 and parameters: {'n_estimators': 66, 'learning_rate': 0.017646094265399568, 'num_leaves': 108, 'max_depth': 16, 'min_child_samples': 4, 'subsample': 0.8943955038618625, 'colsample_bytree': 0.7036286873848959, 'reg_alpha': 9.841222788711072, 'reg_lambda': 0.49549574461952295, 'min_split_gain': 0.0014626362779020963}. Best is trial 0 with value: 0.0004086819498064812.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:48:01,881] Trial 2 finished with value: 0.0015517640521283856 and parameters: {'n_estimators': 68, 'learning_rate': 0.04293647958074325, 'num_leaves': 90, 'max_depth': 14, 'min_child_samples': 21, 'subsample': 0.5252639662417362, 'colsample_bytree': 0.5149826696932637, 'reg_alpha': 2.9590988432814855, 'reg_lambda': 5.048277642865381, 'min_split_gain': 0.02553380948487914}. Best is trial 0 with value: 0.0004086819498064812.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:48:02,400] Trial 3 finished with value: 0.00037786586094354645 and parameters: {'n_estimators': 279, 'learning_rate': 0.2162571970573768, 'num_leaves': 186, 'max_depth': 9, 'min_child_samples': 25, 'subsample': 0.7583724137662534, 'colsample_bytree': 0.938320047395103, 'reg_alpha': 0.09416741731556932, 'reg_lambda': 1.6109816367421481, 'min_split_gain': 0.04463696618957504}. Best is trial 3 with value: 0.00037786586094354645.
[I 2025-06-15 20:48:02,402] A new study created in memory with name: no-name-47638e1f-744d-4b02-a210-93485b8ddb8d
[I 2025-06-15 20:48:02,474] Trial 0 finished with value: 0.0016359786023586315 and parameters: {'n_estimators': 240, 'learning_rate': 0.11044210060616011, 'num_leaves': 184, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.8365802880771955, 'colsample_bytree': 0.8047788331684876, 'reg_alpha': 0.28221925309941276, 'reg_lambda': 0.5888985371084299, 'min_split_gain': 0.03747171604805351}. Best is trial 0 with value: 0.0016359786023586315.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         443
14      pct_chg_Load_DA         339
1           Price_lag_2         337
16  roll2d_median_Price         197
3         Load_DA_lag_0         190
2           Price_lag_7         161
5       WindOn_DA_lag_0         124
11           NGas_lag_2         112
15       lag168_Load_DA         100
10           Coal_lag_2          80
13            EUA_lag_2          67
12            Oil_lag_2          59
7                  WD_1           3
9                  WD_7           2
8                  WD_6           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          31
2           Price_lag_7          16
3         Load_DA_lag_0          13
14      pct_chg_Load_DA           7
15       lag168_Load_DA           4
5       WindOn_DA_lag_0           3
1           Price_lag_2           2
13            EUA_lag_2           1
11           NGas_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 94.12% done
********************* END NS.. 687 ****************************************************
********************* START NS ... 688 ****************************************************
[I 2025-06-15 20:48:04,390] A new study created in memory with name: no-name-1ed805db-5443-4b4e-9fd9-e93034861068
[I 2025-06-15 20:48:04,649] Trial 0 finished with value: 0.016924911405266585 and parameters: {'n_estimators': 79, 'learning_rate': 0.1325619988911317, 'num_leaves': 178, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.7924970622944709, 'colsample_bytree': 0.9613582116047236, 'reg_alpha': 6.455532625789471, 'reg_lambda': 3.3576061260943932, 'min_split_gain': 0.03427510170860259}. Best is trial 0 with value: 0.016924911405266585.
[Trial 0] New best: 0.01692
[I 2025-06-15 20:48:05,029] Trial 1 finished with value: 0.018660657150695797 and parameters: {'n_estimators': 64, 'learning_rate': 0.17022382618765552, 'num_leaves': 74, 'max_depth': 15, 'min_child_samples': 12, 'subsample': 0.9068898546019655, 'colsample_bytree': 0.6196426836368416, 'reg_alpha': 7.588159968219328, 'reg_lambda': 9.223385350952167, 'min_split_gain': 0.046428910667387906}. Best is trial 0 with value: 0.016924911405266585.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:48:05,947] Trial 2 finished with value: 0.017318991777113404 and parameters: {'n_estimators': 282, 'learning_rate': 0.06287324719231993, 'num_leaves': 127, 'max_depth': 16, 'min_child_samples': 25, 'subsample': 0.7403461343486232, 'colsample_bytree': 0.735350344319615, 'reg_alpha': 7.493849476326955, 'reg_lambda': 4.942194798901972, 'min_split_gain': 0.01470268117816681}. Best is trial 0 with value: 0.016924911405266585.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:48:06,658] Trial 3 finished with value: 0.016819885556247093 and parameters: {'n_estimators': 222, 'learning_rate': 0.11782631361545302, 'num_leaves': 139, 'max_depth': 12, 'min_child_samples': 10, 'subsample': 0.728314112452396, 'colsample_bytree': 0.9738904288591348, 'reg_alpha': 1.1470074284006926, 'reg_lambda': 0.3290116495386497, 'min_split_gain': 0.049934864777529786}. Best is trial 3 with value: 0.016819885556247093.
[I 2025-06-15 20:48:06,660] A new study created in memory with name: no-name-20c252dd-948a-4209-a5a9-d38593146fad
[I 2025-06-15 20:48:06,715] Trial 0 finished with value: 0.06990074860236724 and parameters: {'n_estimators': 255, 'learning_rate': 0.2824178957587703, 'num_leaves': 222, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.5438583396779264, 'colsample_bytree': 0.672208061356671, 'reg_alpha': 3.0002005158484146, 'reg_lambda': 3.207398565347205, 'min_split_gain': 0.04836843574718355}. Best is trial 0 with value: 0.06990074860236724.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         625
14      pct_chg_Load_DA         555
1           Price_lag_2         441
3         Load_DA_lag_0         307
16  roll2d_median_Price         269
2           Price_lag_7         248
5       WindOn_DA_lag_0         166
11           NGas_lag_2         162
15       lag168_Load_DA         126
10           Coal_lag_2          88
12            Oil_lag_2          83
13            EUA_lag_2          80
7                  WD_1           6
9                  WD_7           5
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1           9
3         Load_DA_lag_0           5
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 94.25% done
********************* END NS.. 688 ****************************************************
********************* START NS ... 689 ****************************************************
[I 2025-06-15 20:48:08,902] A new study created in memory with name: no-name-a7a4ff48-ff07-405c-a688-4becd5ba7b74
[I 2025-06-15 20:48:09,258] Trial 0 finished with value: 0.03159505561183126 and parameters: {'n_estimators': 72, 'learning_rate': 0.019476311842699004, 'num_leaves': 100, 'max_depth': 12, 'min_child_samples': 29, 'subsample': 0.5079104338376284, 'colsample_bytree': 0.7523942289882513, 'reg_alpha': 8.87478534154713, 'reg_lambda': 8.415380085688742, 'min_split_gain': 0.0019330951400505449}. Best is trial 0 with value: 0.03159505561183126.
[Trial 0] New best: 0.03160
[I 2025-06-15 20:48:13,222] Trial 1 finished with value: 0.02675279829272273 and parameters: {'n_estimators': 270, 'learning_rate': 0.023153045886222714, 'num_leaves': 212, 'max_depth': 9, 'min_child_samples': 4, 'subsample': 0.8260997871118765, 'colsample_bytree': 0.6856588254838323, 'reg_alpha': 0.8416695339606461, 'reg_lambda': 4.478949080654999, 'min_split_gain': 0.03736682597698306}. Best is trial 1 with value: 0.02675279829272273.
[Trial 1] New best: 0.02675
[I 2025-06-15 20:48:15,050] Trial 2 finished with value: 0.025006726331831156 and parameters: {'n_estimators': 281, 'learning_rate': 0.056644654264843344, 'num_leaves': 230, 'max_depth': 9, 'min_child_samples': 9, 'subsample': 0.8722597525809199, 'colsample_bytree': 0.8169612538044917, 'reg_alpha': 2.358497841614049, 'reg_lambda': 3.8868139306499883, 'min_split_gain': 0.013661369608223861}. Best is trial 2 with value: 0.025006726331831156.
[Trial 2] New best: 0.02501
[I 2025-06-15 20:48:15,720] Trial 3 finished with value: 0.024660666556178035 and parameters: {'n_estimators': 236, 'learning_rate': 0.15618651723770288, 'num_leaves': 40, 'max_depth': 16, 'min_child_samples': 30, 'subsample': 0.8783370901332233, 'colsample_bytree': 0.8161338664242572, 'reg_alpha': 2.7013856783653525, 'reg_lambda': 7.156919660269535, 'min_split_gain': 0.02786922458539652}. Best is trial 3 with value: 0.024660666556178035.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:48:17,320] Trial 4 finished with value: 0.023963675354658747 and parameters: {'n_estimators': 150, 'learning_rate': 0.03938064469226041, 'num_leaves': 136, 'max_depth': 9, 'min_child_samples': 10, 'subsample': 0.7549611605611332, 'colsample_bytree': 0.8222184875573625, 'reg_alpha': 3.942475318672013, 'reg_lambda': 6.925145412463561, 'min_split_gain': 0.012711903488487847}. Best is trial 4 with value: 0.023963675354658747.
[Trial 4] New best: 0.02396
[I 2025-06-15 20:48:17,625] Trial 5 finished with value: 0.04664292342655781 and parameters: {'n_estimators': 63, 'learning_rate': 0.010131060906684733, 'num_leaves': 169, 'max_depth': 5, 'min_child_samples': 5, 'subsample': 0.5890956109991132, 'colsample_bytree': 0.9956821174932395, 'reg_alpha': 8.709500044283672, 'reg_lambda': 6.974478805045891, 'min_split_gain': 0.020234850239861314}. Best is trial 4 with value: 0.023963675354658747.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:48:18,672] Trial 6 finished with value: 0.024888337281966152 and parameters: {'n_estimators': 85, 'learning_rate': 0.05586554388237497, 'num_leaves': 46, 'max_depth': 9, 'min_child_samples': 29, 'subsample': 0.8558650547368456, 'colsample_bytree': 0.8946844763676105, 'reg_alpha': 2.711317014275271, 'reg_lambda': 9.710158243822073, 'min_split_gain': 0.019708449805383877}. Best is trial 4 with value: 0.023963675354658747.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:48:19,342] Trial 7 finished with value: 0.02468017365161269 and parameters: {'n_estimators': 130, 'learning_rate': 0.05607610364088021, 'num_leaves': 169, 'max_depth': 6, 'min_child_samples': 14, 'subsample': 0.9626719887984297, 'colsample_bytree': 0.86094715419, 'reg_alpha': 6.360096891184664, 'reg_lambda': 6.518526728943393, 'min_split_gain': 0.04907323202132854}. Best is trial 4 with value: 0.023963675354658747.
[I 2025-06-15 20:48:19,345] A new study created in memory with name: no-name-cb1067ad-c4f5-4738-aeab-5f491eb94ef8
[I 2025-06-15 20:48:19,409] Trial 0 finished with value: 0.4113904693215642 and parameters: {'n_estimators': 254, 'learning_rate': 0.09389956331307796, 'num_leaves': 202, 'max_depth': 7, 'min_child_samples': 18, 'subsample': 0.8150016159595641, 'colsample_bytree': 0.65408442782305, 'reg_alpha': 6.223125366183716, 'reg_lambda': 3.533448110013765, 'min_split_gain': 0.007113488731382534}. Best is trial 0 with value: 0.4113904693215642.
[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1524
1           Price_lag_2        1211
14      pct_chg_Load_DA         998
3         Load_DA_lag_0         677
2           Price_lag_7         516
16  roll2d_median_Price         493
11           NGas_lag_2         405
13            EUA_lag_2         398
5       WindOn_DA_lag_0         339
15       lag168_Load_DA         321
12            Oil_lag_2         244
10           Coal_lag_2         237
9                  WD_7          37
8                  WD_6          35
7                  WD_1           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          36
3         Load_DA_lag_0          16
15       lag168_Load_DA           4
12            Oil_lag_2           4
1           Price_lag_2           3
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 94.39% done
********************* END NS.. 689 ****************************************************
********************* START NS ... 690 ****************************************************
[I 2025-06-15 20:48:22,182] A new study created in memory with name: no-name-9b9859d5-a36e-49c5-8bc3-f4ac927f5cb0
[I 2025-06-15 20:48:22,417] Trial 0 finished with value: 0.011445611835017439 and parameters: {'n_estimators': 73, 'learning_rate': 0.20290949985338103, 'num_leaves': 118, 'max_depth': 4, 'min_child_samples': 18, 'subsample': 0.7130683624608066, 'colsample_bytree': 0.7411266315827538, 'reg_alpha': 6.41329384962689, 'reg_lambda': 0.8137012137265565, 'min_split_gain': 0.006805329595503701}. Best is trial 0 with value: 0.011445611835017439.
[Trial 0] New best: 0.01145
[I 2025-06-15 20:48:24,198] Trial 1 finished with value: 0.015496301172812884 and parameters: {'n_estimators': 185, 'learning_rate': 0.010966600157137013, 'num_leaves': 84, 'max_depth': 7, 'min_child_samples': 2, 'subsample': 0.5845701246586521, 'colsample_bytree': 0.77054639731739, 'reg_alpha': 2.40861700510942, 'reg_lambda': 5.832578325009672, 'min_split_gain': 0.047028091473608265}. Best is trial 0 with value: 0.011445611835017439.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:48:25,265] Trial 2 finished with value: 0.015006942691983367 and parameters: {'n_estimators': 102, 'learning_rate': 0.06650769521153203, 'num_leaves': 169, 'max_depth': 9, 'min_child_samples': 26, 'subsample': 0.8142969809311222, 'colsample_bytree': 0.5898121906832363, 'reg_alpha': 4.204549399181021, 'reg_lambda': 2.16338911981385, 'min_split_gain': 0.014614693881284469}. Best is trial 0 with value: 0.011445611835017439.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:48:25,721] Trial 3 finished with value: 0.014115892877268485 and parameters: {'n_estimators': 298, 'learning_rate': 0.11759227412711114, 'num_leaves': 166, 'max_depth': 10, 'min_child_samples': 22, 'subsample': 0.6100530996777138, 'colsample_bytree': 0.994990868915948, 'reg_alpha': 6.527315194198046, 'reg_lambda': 9.47356828283865, 'min_split_gain': 0.044781338518252306}. Best is trial 0 with value: 0.011445611835017439.
[I 2025-06-15 20:48:25,723] A new study created in memory with name: no-name-de705584-d590-4f1d-8538-22fd5dba95e3
[I 2025-06-15 20:48:25,755] Trial 0 finished with value: 0.40883889528216466 and parameters: {'n_estimators': 115, 'learning_rate': 0.03143742780121606, 'num_leaves': 37, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.9350728785009088, 'colsample_bytree': 0.9005588630215886, 'reg_alpha': 8.668661885531767, 'reg_lambda': 3.854320654445571, 'min_split_gain': 0.006074037205809901}. Best is trial 0 with value: 0.40883889528216466.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         220
1           Price_lag_2         120
14      pct_chg_Load_DA          96
3         Load_DA_lag_0          92
2           Price_lag_7          67
13            EUA_lag_2          61
11           NGas_lag_2          49
15       lag168_Load_DA          42
5       WindOn_DA_lag_0          41
16  roll2d_median_Price          33
12            Oil_lag_2          19
10           Coal_lag_2          18
9                  WD_7           7
8                  WD_6           4
7                  WD_1           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         153
15       lag168_Load_DA          29
3         Load_DA_lag_0          28
1           Price_lag_2           7
16  roll2d_median_Price           7
2           Price_lag_7           5
12            Oil_lag_2           3
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 94.53% done
********************* END NS.. 690 ****************************************************
********************* START NS ... 691 ****************************************************
[I 2025-06-15 20:48:27,163] A new study created in memory with name: no-name-f848c7d6-d4af-41e9-b110-a24f40a3a57d
[I 2025-06-15 20:48:28,924] Trial 0 finished with value: 0.0130889825639954 and parameters: {'n_estimators': 180, 'learning_rate': 0.03904373953022899, 'num_leaves': 191, 'max_depth': 16, 'min_child_samples': 27, 'subsample': 0.9026970393375509, 'colsample_bytree': 0.5953104543910483, 'reg_alpha': 2.694024019810952, 'reg_lambda': 3.2388781749682005, 'min_split_gain': 0.03077494845474442}. Best is trial 0 with value: 0.0130889825639954.
[Trial 0] New best: 0.01309
[I 2025-06-15 20:48:32,483] Trial 1 finished with value: 0.014179997933148086 and parameters: {'n_estimators': 297, 'learning_rate': 0.010957264764707717, 'num_leaves': 104, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.8625844911061323, 'colsample_bytree': 0.6970238377743598, 'reg_alpha': 1.2337697609545917, 'reg_lambda': 8.03178825878353, 'min_split_gain': 0.03489873764923445}. Best is trial 0 with value: 0.0130889825639954.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:48:33,549] Trial 2 finished with value: 0.02151253105522194 and parameters: {'n_estimators': 147, 'learning_rate': 0.010834233166280703, 'num_leaves': 38, 'max_depth': 7, 'min_child_samples': 26, 'subsample': 0.5622594953886333, 'colsample_bytree': 0.5997672251436202, 'reg_alpha': 5.4698460664170145, 'reg_lambda': 1.7206947815903373, 'min_split_gain': 0.0008560187169752488}. Best is trial 0 with value: 0.0130889825639954.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:48:33,864] Trial 3 finished with value: 0.012811339791971228 and parameters: {'n_estimators': 110, 'learning_rate': 0.2092570386169598, 'num_leaves': 178, 'max_depth': 15, 'min_child_samples': 29, 'subsample': 0.534933252473462, 'colsample_bytree': 0.8932864490368648, 'reg_alpha': 2.9880693382304724, 'reg_lambda': 7.605854593091794, 'min_split_gain': 0.03383496808870719}. Best is trial 3 with value: 0.012811339791971228.
[I 2025-06-15 20:48:33,866] A new study created in memory with name: no-name-fa76a3e9-f38c-4a38-8b5c-43e258283cdf
[I 2025-06-15 20:48:33,992] Trial 0 finished with value: 0.08776310222643231 and parameters: {'n_estimators': 258, 'learning_rate': 0.020474891052458012, 'num_leaves': 175, 'max_depth': 11, 'min_child_samples': 8, 'subsample': 0.5715780808699278, 'colsample_bytree': 0.8240857267553909, 'reg_alpha': 5.573066570218977, 'reg_lambda': 4.41800702802595, 'min_split_gain': 0.010057739037276359}. Best is trial 0 with value: 0.08776310222643231.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         274
14      pct_chg_Load_DA         213
1           Price_lag_2         155
3         Load_DA_lag_0         101
16  roll2d_median_Price          83
2           Price_lag_7          80
11           NGas_lag_2          73
5       WindOn_DA_lag_0          59
15       lag168_Load_DA          49
13            EUA_lag_2          43
10           Coal_lag_2          41
12            Oil_lag_2          21
9                  WD_7           7
7                  WD_1           3
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         234
3         Load_DA_lag_0          74
15       lag168_Load_DA          71
2           Price_lag_7          54
1           Price_lag_2          11
12            Oil_lag_2          10
5       WindOn_DA_lag_0           3
13            EUA_lag_2           1
11           NGas_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 94.66% done
********************* END NS.. 691 ****************************************************
********************* START NS ... 692 ****************************************************
[I 2025-06-15 20:48:35,514] A new study created in memory with name: no-name-8ede4ac3-99a3-487b-ae93-64c1ed013509
[I 2025-06-15 20:48:36,508] Trial 0 finished with value: 0.015293415748471983 and parameters: {'n_estimators': 83, 'learning_rate': 0.0558083364621499, 'num_leaves': 249, 'max_depth': 15, 'min_child_samples': 2, 'subsample': 0.8624675696120083, 'colsample_bytree': 0.8927742422140084, 'reg_alpha': 1.8942321167738474, 'reg_lambda': 4.92821567185824, 'min_split_gain': 0.04732427162591106}. Best is trial 0 with value: 0.015293415748471983.
[Trial 0] New best: 0.01529
[I 2025-06-15 20:48:37,195] Trial 1 finished with value: 0.014861053866321708 and parameters: {'n_estimators': 165, 'learning_rate': 0.1017216981138024, 'num_leaves': 76, 'max_depth': 12, 'min_child_samples': 28, 'subsample': 0.7865278410774836, 'colsample_bytree': 0.9823373759655611, 'reg_alpha': 9.216043380173613, 'reg_lambda': 6.4047769041096, 'min_split_gain': 0.009214988416415399}. Best is trial 1 with value: 0.014861053866321708.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:48:38,857] Trial 2 finished with value: 0.01573603194276547 and parameters: {'n_estimators': 222, 'learning_rate': 0.019652239895265863, 'num_leaves': 147, 'max_depth': 9, 'min_child_samples': 7, 'subsample': 0.5826946581475984, 'colsample_bytree': 0.793559832695136, 'reg_alpha': 7.788279117547355, 'reg_lambda': 8.97817612630809, 'min_split_gain': 0.021730755180363395}. Best is trial 1 with value: 0.014861053866321708.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:48:39,467] Trial 3 finished with value: 0.014118823608415052 and parameters: {'n_estimators': 264, 'learning_rate': 0.1195266365275314, 'num_leaves': 32, 'max_depth': 6, 'min_child_samples': 15, 'subsample': 0.9986409167510483, 'colsample_bytree': 0.8947352280887112, 'reg_alpha': 3.5204294161090246, 'reg_lambda': 8.902714356037398, 'min_split_gain': 0.031682772682900706}. Best is trial 3 with value: 0.014118823608415052.
[Trial 3] New best: 0.01412
[I 2025-06-15 20:48:40,286] Trial 4 finished with value: 0.015282296475810586 and parameters: {'n_estimators': 202, 'learning_rate': 0.0917799261114505, 'num_leaves': 171, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.8731020368165359, 'colsample_bytree': 0.6709226834349223, 'reg_alpha': 6.2162846710652895, 'reg_lambda': 9.792200354284152, 'min_split_gain': 0.019587924518294154}. Best is trial 3 with value: 0.014118823608415052.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:48:40,905] Trial 5 finished with value: 0.013838662425629917 and parameters: {'n_estimators': 83, 'learning_rate': 0.1252477552044312, 'num_leaves': 57, 'max_depth': 9, 'min_child_samples': 19, 'subsample': 0.6975917539164169, 'colsample_bytree': 0.763712038647304, 'reg_alpha': 4.432894045142976, 'reg_lambda': 0.8985801487092993, 'min_split_gain': 0.009334389248608839}. Best is trial 5 with value: 0.013838662425629917.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:48:41,754] Trial 6 finished with value: 0.014719067625148486 and parameters: {'n_estimators': 246, 'learning_rate': 0.07910048147633943, 'num_leaves': 103, 'max_depth': 13, 'min_child_samples': 23, 'subsample': 0.9501106372412872, 'colsample_bytree': 0.9349212563317257, 'reg_alpha': 3.112688053828727, 'reg_lambda': 2.9991175442369333, 'min_split_gain': 0.04691626252365771}. Best is trial 5 with value: 0.013838662425629917.
[I 2025-06-15 20:48:41,755] A new study created in memory with name: no-name-91bfcb82-0ca0-4bc6-a05a-5105a59904cd
[I 2025-06-15 20:48:41,885] Trial 0 finished with value: 0.034657828207225405 and parameters: {'n_estimators': 197, 'learning_rate': 0.012408443919153876, 'num_leaves': 200, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.6433224560730189, 'colsample_bytree': 0.7862564478594234, 'reg_alpha': 3.751284841497516, 'reg_lambda': 8.145498013642516, 'min_split_gain': 0.02440182248440126}. Best is trial 0 with value: 0.034657828207225405.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         532
1           Price_lag_2         401
14      pct_chg_Load_DA         341
3         Load_DA_lag_0         290
11           NGas_lag_2         181
5       WindOn_DA_lag_0         174
16  roll2d_median_Price         168
2           Price_lag_7         168
13            EUA_lag_2         163
15       lag168_Load_DA         155
10           Coal_lag_2         109
12            Oil_lag_2          88
9                  WD_7          17
7                  WD_1          13
8                  WD_6          13
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         252
15       lag168_Load_DA         126
3         Load_DA_lag_0          63
1           Price_lag_2          39
5       WindOn_DA_lag_0          13
11           NGas_lag_2          10
16  roll2d_median_Price          10
2           Price_lag_7           5
12            Oil_lag_2           1
7                  WD_1           0
6      WindOff_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
4        Solar_DA_lag_0           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 94.80% done
********************* END NS.. 692 ****************************************************
********************* START NS ... 693 ****************************************************
[I 2025-06-15 20:48:43,647] A new study created in memory with name: no-name-6a537ac0-6bff-450e-82ad-f72c735c1be5
[I 2025-06-15 20:48:43,874] Trial 0 finished with value: 0.05129475692666241 and parameters: {'n_estimators': 61, 'learning_rate': 0.011405359290382812, 'num_leaves': 209, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.6544610113083908, 'colsample_bytree': 0.707257903688149, 'reg_alpha': 9.444053853436913, 'reg_lambda': 5.3551798356203895, 'min_split_gain': 0.041183027708806476}. Best is trial 0 with value: 0.05129475692666241.
[Trial 0] New best: 0.05129
[I 2025-06-15 20:48:45,456] Trial 1 finished with value: 0.008357881506051433 and parameters: {'n_estimators': 284, 'learning_rate': 0.022257383128199276, 'num_leaves': 167, 'max_depth': 15, 'min_child_samples': 30, 'subsample': 0.770194295902424, 'colsample_bytree': 0.920907253029692, 'reg_alpha': 7.3590196900842555, 'reg_lambda': 2.749484176339856, 'min_split_gain': 0.027432756525255366}. Best is trial 1 with value: 0.008357881506051433.
[Trial 1] New best: 0.00836
[I 2025-06-15 20:48:46,913] Trial 2 finished with value: 0.008283941189032413 and parameters: {'n_estimators': 282, 'learning_rate': 0.04859994380209379, 'num_leaves': 67, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.9962871168010066, 'colsample_bytree': 0.8143028039547029, 'reg_alpha': 5.722882304302258, 'reg_lambda': 1.0054013348812985, 'min_split_gain': 0.013543593061526766}. Best is trial 2 with value: 0.008283941189032413.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:48:47,563] Trial 3 finished with value: 0.009240271996182546 and parameters: {'n_estimators': 89, 'learning_rate': 0.10123784030125602, 'num_leaves': 170, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.5983516608246677, 'colsample_bytree': 0.6049101881244783, 'reg_alpha': 6.979303906022414, 'reg_lambda': 8.540768234771539, 'min_split_gain': 0.018704460067900577}. Best is trial 2 with value: 0.008283941189032413.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:48:48,960] Trial 4 finished with value: 0.013590971943554008 and parameters: {'n_estimators': 204, 'learning_rate': 0.011794635008507626, 'num_leaves': 204, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.7890580931442119, 'colsample_bytree': 0.7164019596106745, 'reg_alpha': 9.022518968360576, 'reg_lambda': 2.0313405423820194, 'min_split_gain': 0.049310987463919126}. Best is trial 2 with value: 0.008283941189032413.
[I 2025-06-15 20:48:48,963] A new study created in memory with name: no-name-4df975a3-74d3-404b-89b6-c824e352ac85
[I 2025-06-15 20:48:49,016] Trial 0 finished with value: 0.0443737222943943 and parameters: {'n_estimators': 83, 'learning_rate': 0.011454991301348454, 'num_leaves': 114, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.6252088309327746, 'colsample_bytree': 0.6469573233430499, 'reg_alpha': 4.615885901335862, 'reg_lambda': 7.2739436445639605, 'min_split_gain': 0.034875532550880786}. Best is trial 0 with value: 0.0443737222943943.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1229
1           Price_lag_2         932
14      pct_chg_Load_DA         772
3         Load_DA_lag_0         542
11           NGas_lag_2         383
2           Price_lag_7         379
5       WindOn_DA_lag_0         368
15       lag168_Load_DA         359
16  roll2d_median_Price         329
10           Coal_lag_2         253
13            EUA_lag_2         220
12            Oil_lag_2         160
9                  WD_7          32
7                  WD_1          19
8                  WD_6          18
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          93
15       lag168_Load_DA          54
1           Price_lag_2          18
16  roll2d_median_Price           6
3         Load_DA_lag_0           4
2           Price_lag_7           1
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 94.94% done
********************* END NS.. 693 ****************************************************
********************* START NS ... 694 ****************************************************
[I 2025-06-15 20:48:51,977] A new study created in memory with name: no-name-64786a78-5155-46c8-8bd7-ee4724328387
[I 2025-06-15 20:48:52,128] Trial 0 finished with value: 0.0029100445987952964 and parameters: {'n_estimators': 115, 'learning_rate': 0.17708406930061066, 'num_leaves': 53, 'max_depth': 5, 'min_child_samples': 12, 'subsample': 0.9515992301346761, 'colsample_bytree': 0.6891883053530955, 'reg_alpha': 4.573465323193919, 'reg_lambda': 6.550085409125758, 'min_split_gain': 0.04243662878455332}. Best is trial 0 with value: 0.0029100445987952964.
[Trial 0] New best: 0.00291
[I 2025-06-15 20:48:52,701] Trial 1 finished with value: 0.0037463812309305997 and parameters: {'n_estimators': 231, 'learning_rate': 0.21144567800824415, 'num_leaves': 200, 'max_depth': 11, 'min_child_samples': 20, 'subsample': 0.6991910598321375, 'colsample_bytree': 0.6070454182375942, 'reg_alpha': 1.0598989548734628, 'reg_lambda': 3.356488345195978, 'min_split_gain': 0.029014281650140347}. Best is trial 0 with value: 0.0029100445987952964.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:48:53,795] Trial 2 finished with value: 0.017667612625762483 and parameters: {'n_estimators': 142, 'learning_rate': 0.015806510704740734, 'num_leaves': 233, 'max_depth': 6, 'min_child_samples': 1, 'subsample': 0.6241791242762373, 'colsample_bytree': 0.5666116585823688, 'reg_alpha': 2.214750050114591, 'reg_lambda': 6.255317004939601, 'min_split_gain': 0.02456474175352289}. Best is trial 0 with value: 0.0029100445987952964.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:48:54,197] Trial 3 finished with value: 0.00367561285854722 and parameters: {'n_estimators': 141, 'learning_rate': 0.17265365934434587, 'num_leaves': 116, 'max_depth': 9, 'min_child_samples': 27, 'subsample': 0.7245502855912647, 'colsample_bytree': 0.6139954234178483, 'reg_alpha': 6.574306541128156, 'reg_lambda': 9.42278321432989, 'min_split_gain': 0.029664134983181423}. Best is trial 0 with value: 0.0029100445987952964.
[I 2025-06-15 20:48:54,200] A new study created in memory with name: no-name-0a60a702-4850-4426-953d-30978c6cc8a5
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 20:48:54,412] Trial 0 finished with value: 0.15253608974962016 and parameters: {'n_estimators': 260, 'learning_rate': 0.013466634387523189, 'num_leaves': 49, 'max_depth': 10, 'min_child_samples': 15, 'subsample': 0.8408564908890925, 'colsample_bytree': 0.7367644563764169, 'reg_alpha': 1.3089772074936623, 'reg_lambda': 6.360142817514771, 'min_split_gain': 0.0045341679860151805}. Best is trial 0 with value: 0.15253608974962016.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         258
1           Price_lag_2         101
14      pct_chg_Load_DA          87
3         Load_DA_lag_0          85
16  roll2d_median_Price          56
2           Price_lag_7          53
11           NGas_lag_2          52
5       WindOn_DA_lag_0          38
10           Coal_lag_2          36
13            EUA_lag_2          34
15       lag168_Load_DA          33
12            Oil_lag_2          14
9                  WD_7          10
8                  WD_6           3
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         430
15       lag168_Load_DA         184
2           Price_lag_7         121
1           Price_lag_2          58
14      pct_chg_Load_DA          57
16  roll2d_median_Price          32
11           NGas_lag_2          21
10           Coal_lag_2          16
3         Load_DA_lag_0          15
9                  WD_7          11
12            Oil_lag_2           6
5       WindOn_DA_lag_0           2
8                  WD_6           1
6      WindOff_DA_lag_0           0
7                  WD_1           0
13            EUA_lag_2           0
4        Solar_DA_lag_0           0
-> 95.08% done
********************* END NS.. 694 ****************************************************
********************* START NS ... 695 ****************************************************
[I 2025-06-15 20:48:56,174] A new study created in memory with name: no-name-7ef99a0e-37dc-41bb-93ce-838a5fd5126d
[I 2025-06-15 20:48:57,098] Trial 0 finished with value: 0.0024061247264150214 and parameters: {'n_estimators': 155, 'learning_rate': 0.03882912245646051, 'num_leaves': 152, 'max_depth': 11, 'min_child_samples': 28, 'subsample': 0.7364668973098955, 'colsample_bytree': 0.8867484624634092, 'reg_alpha': 4.865270243574709, 'reg_lambda': 3.4686198381977675, 'min_split_gain': 0.013567502631842837}. Best is trial 0 with value: 0.0024061247264150214.
[Trial 0] New best: 0.00241
[I 2025-06-15 20:48:57,331] Trial 1 finished with value: 0.001669452108445678 and parameters: {'n_estimators': 70, 'learning_rate': 0.14877764924678627, 'num_leaves': 154, 'max_depth': 5, 'min_child_samples': 26, 'subsample': 0.739898503678118, 'colsample_bytree': 0.8425656396921039, 'reg_alpha': 3.2914833696042467, 'reg_lambda': 1.1420811808984477, 'min_split_gain': 0.04202249864275355}. Best is trial 1 with value: 0.001669452108445678.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:48:58,958] Trial 2 finished with value: 0.012248447252618882 and parameters: {'n_estimators': 199, 'learning_rate': 0.011998503801531477, 'num_leaves': 147, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.9748815997023904, 'colsample_bytree': 0.5773580094272339, 'reg_alpha': 3.8984404460155764, 'reg_lambda': 9.340609054184426, 'min_split_gain': 0.03649942709015645}. Best is trial 1 with value: 0.001669452108445678.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:49:00,392] Trial 3 finished with value: 0.0026269615034386195 and parameters: {'n_estimators': 265, 'learning_rate': 0.0484729252596742, 'num_leaves': 124, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.8822727502282344, 'colsample_bytree': 0.6496683103582036, 'reg_alpha': 5.575876638467754, 'reg_lambda': 0.8463553975275218, 'min_split_gain': 0.02092273694265305}. Best is trial 1 with value: 0.001669452108445678.
[I 2025-06-15 20:49:00,395] A new study created in memory with name: no-name-95bcc1f1-52bb-44f6-816f-e625e26ce599
[I 2025-06-15 20:49:00,537] Trial 0 finished with value: 0.23403906290606344 and parameters: {'n_estimators': 287, 'learning_rate': 0.030110481074203336, 'num_leaves': 210, 'max_depth': 11, 'min_child_samples': 24, 'subsample': 0.8697485120939068, 'colsample_bytree': 0.5201839699948241, 'reg_alpha': 3.0780277320647476, 'reg_lambda': 5.187515761903178, 'min_split_gain': 0.003017305144995941}. Best is trial 0 with value: 0.23403906290606344.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         309
14      pct_chg_Load_DA         187
1           Price_lag_2         120
3         Load_DA_lag_0          81
16  roll2d_median_Price          63
2           Price_lag_7          59
11           NGas_lag_2          56
15       lag168_Load_DA          56
5       WindOn_DA_lag_0          37
10           Coal_lag_2          25
13            EUA_lag_2          19
12            Oil_lag_2          18
9                  WD_7           4
7                  WD_1           3
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         207
15       lag168_Load_DA          99
1           Price_lag_2          52
2           Price_lag_7          41
10           Coal_lag_2          22
16  roll2d_median_Price          11
3         Load_DA_lag_0           8
11           NGas_lag_2           2
12            Oil_lag_2           2
6      WindOff_DA_lag_0           0
7                  WD_1           0
5       WindOn_DA_lag_0           0
9                  WD_7           0
4        Solar_DA_lag_0           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 95.21% done
********************* END NS.. 695 ****************************************************
********************* START NS ... 696 ****************************************************
[I 2025-06-15 20:49:02,292] A new study created in memory with name: no-name-29434540-94ab-4984-b6de-e24be39c0b54
[I 2025-06-15 20:49:02,730] Trial 0 finished with value: 0.0007314379658607275 and parameters: {'n_estimators': 281, 'learning_rate': 0.07583835679614936, 'num_leaves': 251, 'max_depth': 11, 'min_child_samples': 4, 'subsample': 0.7066023227767274, 'colsample_bytree': 0.8133934426632532, 'reg_alpha': 9.79435586397462, 'reg_lambda': 8.435330611890064, 'min_split_gain': 0.04560346719794185}. Best is trial 0 with value: 0.0007314379658607275.
[Trial 0] New best: 0.00073
[I 2025-06-15 20:49:03,498] Trial 1 finished with value: 0.0011219768143786047 and parameters: {'n_estimators': 125, 'learning_rate': 0.028911840522171505, 'num_leaves': 176, 'max_depth': 7, 'min_child_samples': 19, 'subsample': 0.5285807248791404, 'colsample_bytree': 0.7530383835620278, 'reg_alpha': 9.853163467144334, 'reg_lambda': 4.183439723876845, 'min_split_gain': 0.02492200233155581}. Best is trial 0 with value: 0.0007314379658607275.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:49:07,387] Trial 2 finished with value: 0.0005609506456202307 and parameters: {'n_estimators': 276, 'learning_rate': 0.022645216448963904, 'num_leaves': 80, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.960166993667823, 'colsample_bytree': 0.787101761265171, 'reg_alpha': 0.5289984157936123, 'reg_lambda': 8.159861087739781, 'min_split_gain': 0.023264451186010966}. Best is trial 2 with value: 0.0005609506456202307.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:49:07,627] Trial 3 finished with value: 0.003801983445038728 and parameters: {'n_estimators': 78, 'learning_rate': 0.030473086691251178, 'num_leaves': 213, 'max_depth': 4, 'min_child_samples': 10, 'subsample': 0.5174448161407299, 'colsample_bytree': 0.7765434694206441, 'reg_alpha': 8.291626492519073, 'reg_lambda': 3.915051934046734, 'min_split_gain': 0.013286636606133129}. Best is trial 2 with value: 0.0005609506456202307.
[I 2025-06-15 20:49:07,630] A new study created in memory with name: no-name-c5cfa40d-8acd-4abb-99f3-f2a271354433
[I 2025-06-15 20:49:07,762] Trial 0 finished with value: 0.010634745568874468 and parameters: {'n_estimators': 260, 'learning_rate': 0.045484774921061864, 'num_leaves': 46, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.9041170913878864, 'colsample_bytree': 0.8449715577785105, 'reg_alpha': 3.370353171270495, 'reg_lambda': 0.5602414785274246, 'min_split_gain': 0.00012367949940964018}. Best is trial 0 with value: 0.010634745568874468.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        3896
1           Price_lag_2        3515
14      pct_chg_Load_DA        2618
3         Load_DA_lag_0        1611
16  roll2d_median_Price        1395
15       lag168_Load_DA        1180
2           Price_lag_7        1159
11           NGas_lag_2        1008
5       WindOn_DA_lag_0         951
13            EUA_lag_2         825
12            Oil_lag_2         682
10           Coal_lag_2         681
9                  WD_7          46
7                  WD_1          36
8                  WD_6          17
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         166
15       lag168_Load_DA         165
16  roll2d_median_Price         158
2           Price_lag_7          17
3         Load_DA_lag_0          14
11           NGas_lag_2          10
10           Coal_lag_2           5
1           Price_lag_2           4
5       WindOn_DA_lag_0           2
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 95.35% done
********************* END NS.. 696 ****************************************************
********************* START NS ... 697 ****************************************************
[I 2025-06-15 20:49:12,613] A new study created in memory with name: no-name-c93711f1-0582-4e43-95ff-d3eaf9552cae
[I 2025-06-15 20:49:13,332] Trial 0 finished with value: 0.004102188170551395 and parameters: {'n_estimators': 234, 'learning_rate': 0.07474796550308305, 'num_leaves': 176, 'max_depth': 12, 'min_child_samples': 20, 'subsample': 0.7537746595807733, 'colsample_bytree': 0.7852636893487559, 'reg_alpha': 9.054855499905578, 'reg_lambda': 8.434989425642785, 'min_split_gain': 0.012001871016447063}. Best is trial 0 with value: 0.004102188170551395.
[Trial 0] New best: 0.00410
[I 2025-06-15 20:49:14,365] Trial 1 finished with value: 0.0049478354155102905 and parameters: {'n_estimators': 154, 'learning_rate': 0.01808906147715901, 'num_leaves': 122, 'max_depth': 12, 'min_child_samples': 23, 'subsample': 0.7792740306623072, 'colsample_bytree': 0.9826925559224673, 'reg_alpha': 8.141620101874485, 'reg_lambda': 1.6342535362967214, 'min_split_gain': 0.02220029141493057}. Best is trial 0 with value: 0.004102188170551395.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:49:14,903] Trial 2 finished with value: 0.014177681308479897 and parameters: {'n_estimators': 124, 'learning_rate': 0.013084125843841258, 'num_leaves': 141, 'max_depth': 5, 'min_child_samples': 10, 'subsample': 0.521741677647825, 'colsample_bytree': 0.5096305828345149, 'reg_alpha': 8.101182780685289, 'reg_lambda': 4.911534200447978, 'min_split_gain': 0.035749542997854315}. Best is trial 0 with value: 0.004102188170551395.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:49:15,584] Trial 3 finished with value: 0.004256207121784224 and parameters: {'n_estimators': 288, 'learning_rate': 0.06155392250377599, 'num_leaves': 124, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.7729207633746972, 'colsample_bytree': 0.7705281589266229, 'reg_alpha': 8.662392589138593, 'reg_lambda': 6.891863647431977, 'min_split_gain': 0.04154269555301467}. Best is trial 0 with value: 0.004102188170551395.
[I 2025-06-15 20:49:15,587] A new study created in memory with name: no-name-26254c06-e67b-4ddf-ad26-8089a2aa617e
[I 2025-06-15 20:49:15,677] Trial 0 finished with value: 0.01206180947628177 and parameters: {'n_estimators': 259, 'learning_rate': 0.03387779384159196, 'num_leaves': 174, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.6046090660982113, 'colsample_bytree': 0.9404238285688267, 'reg_alpha': 8.89104293245793, 'reg_lambda': 0.6210768114714427, 'min_split_gain': 0.018821317552254344}. Best is trial 0 with value: 0.01206180947628177.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         648
14      pct_chg_Load_DA         405
1           Price_lag_2         393
3         Load_DA_lag_0         288
2           Price_lag_7         199
11           NGas_lag_2         187
16  roll2d_median_Price         182
13            EUA_lag_2         163
15       lag168_Load_DA         151
10           Coal_lag_2         131
5       WindOn_DA_lag_0         124
12            Oil_lag_2         107
9                  WD_7          22
8                  WD_6          15
7                  WD_1           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         111
16  roll2d_median_Price          23
15       lag168_Load_DA          17
3         Load_DA_lag_0          11
1           Price_lag_2           6
2           Price_lag_7           0
4        Solar_DA_lag_0           0
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 95.49% done
********************* END NS.. 697 ****************************************************
********************* START NS ... 698 ****************************************************
[I 2025-06-15 20:49:17,613] A new study created in memory with name: no-name-b31d4537-7368-491e-9d98-769f0f212e10
[I 2025-06-15 20:49:17,935] Trial 0 finished with value: 0.010078593561288533 and parameters: {'n_estimators': 257, 'learning_rate': 0.28000908741191843, 'num_leaves': 123, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.7647600423356804, 'colsample_bytree': 0.6864428440567625, 'reg_alpha': 4.19560558748543, 'reg_lambda': 0.8832067467021065, 'min_split_gain': 0.009393786734202948}. Best is trial 0 with value: 0.010078593561288533.
[Trial 0] New best: 0.01008
[I 2025-06-15 20:49:18,231] Trial 1 finished with value: 0.011551779698324957 and parameters: {'n_estimators': 163, 'learning_rate': 0.13465327105742278, 'num_leaves': 78, 'max_depth': 16, 'min_child_samples': 27, 'subsample': 0.6241839349628924, 'colsample_bytree': 0.5685055436783322, 'reg_alpha': 6.919889644133876, 'reg_lambda': 2.3999374622891025, 'min_split_gain': 0.049325803166382465}. Best is trial 0 with value: 0.010078593561288533.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:49:18,767] Trial 2 finished with value: 0.013113438893782468 and parameters: {'n_estimators': 65, 'learning_rate': 0.042185616510149015, 'num_leaves': 195, 'max_depth': 15, 'min_child_samples': 8, 'subsample': 0.5085256603470676, 'colsample_bytree': 0.7321994009961341, 'reg_alpha': 9.314175787501503, 'reg_lambda': 9.931625050044458, 'min_split_gain': 0.04900428671532714}. Best is trial 0 with value: 0.010078593561288533.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:49:19,261] Trial 3 finished with value: 0.011118436777784277 and parameters: {'n_estimators': 281, 'learning_rate': 0.08776490291891426, 'num_leaves': 203, 'max_depth': 4, 'min_child_samples': 15, 'subsample': 0.7850097213305434, 'colsample_bytree': 0.7144123730219258, 'reg_alpha': 5.459856565841832, 'reg_lambda': 8.891217710540865, 'min_split_gain': 0.04803483364232576}. Best is trial 0 with value: 0.010078593561288533.
[I 2025-06-15 20:49:19,263] A new study created in memory with name: no-name-a56d2f2d-378d-4269-a10b-55f4d007531b
[I 2025-06-15 20:49:19,332] Trial 0 finished with value: 0.08235862415899509 and parameters: {'n_estimators': 179, 'learning_rate': 0.04903649396714378, 'num_leaves': 171, 'max_depth': 10, 'min_child_samples': 10, 'subsample': 0.994956247929617, 'colsample_bytree': 0.6190540693245488, 'reg_alpha': 6.216419094220288, 'reg_lambda': 0.21983784791237082, 'min_split_gain': 0.04923810320868066}. Best is trial 0 with value: 0.08235862415899509.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         275
1           Price_lag_2         184
3         Load_DA_lag_0         137
14      pct_chg_Load_DA         101
2           Price_lag_7          93
11           NGas_lag_2          90
16  roll2d_median_Price          84
15       lag168_Load_DA          77
5       WindOn_DA_lag_0          64
10           Coal_lag_2          61
13            EUA_lag_2          59
12            Oil_lag_2          21
9                  WD_7           9
7                  WD_1           5
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          50
1           Price_lag_2          14
15       lag168_Load_DA          11
3         Load_DA_lag_0           9
16  roll2d_median_Price           1
2           Price_lag_7           1
5       WindOn_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 95.62% done
********************* END NS.. 698 ****************************************************
********************* START NS ... 699 ****************************************************
[I 2025-06-15 20:49:21,187] A new study created in memory with name: no-name-122eff4f-d509-46e5-b620-599641a34068
[I 2025-06-15 20:49:23,608] Trial 0 finished with value: 0.006162357365762962 and parameters: {'n_estimators': 274, 'learning_rate': 0.07550485565410778, 'num_leaves': 68, 'max_depth': 14, 'min_child_samples': 7, 'subsample': 0.8164719127692885, 'colsample_bytree': 0.5667392946693351, 'reg_alpha': 1.3119328351989323, 'reg_lambda': 3.7172228256864406, 'min_split_gain': 0.003075842317387867}. Best is trial 0 with value: 0.006162357365762962.
[Trial 0] New best: 0.00616
[I 2025-06-15 20:49:24,346] Trial 1 finished with value: 0.006277780064718064 and parameters: {'n_estimators': 252, 'learning_rate': 0.1928391977279337, 'num_leaves': 198, 'max_depth': 12, 'min_child_samples': 3, 'subsample': 0.5013765041423883, 'colsample_bytree': 0.9824031025085249, 'reg_alpha': 5.421056752918351, 'reg_lambda': 6.8248273671651365, 'min_split_gain': 0.0029803653693997625}. Best is trial 0 with value: 0.006162357365762962.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:49:26,243] Trial 2 finished with value: 0.007677426334326012 and parameters: {'n_estimators': 247, 'learning_rate': 0.014614116210635967, 'num_leaves': 38, 'max_depth': 7, 'min_child_samples': 8, 'subsample': 0.75961508727474, 'colsample_bytree': 0.8587273264176101, 'reg_alpha': 5.1050268552117695, 'reg_lambda': 2.483685817198041, 'min_split_gain': 0.030473961224758678}. Best is trial 0 with value: 0.006162357365762962.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:49:26,659] Trial 3 finished with value: 0.010466042022265186 and parameters: {'n_estimators': 55, 'learning_rate': 0.23953911236767164, 'num_leaves': 48, 'max_depth': 11, 'min_child_samples': 10, 'subsample': 0.6358759542769623, 'colsample_bytree': 0.5664014232329126, 'reg_alpha': 7.911704679638838, 'reg_lambda': 9.2812518719298, 'min_split_gain': 0.0448204604990059}. Best is trial 0 with value: 0.006162357365762962.
[I 2025-06-15 20:49:26,661] A new study created in memory with name: no-name-e15acd08-3465-42ba-983c-3819275cea1d
[I 2025-06-15 20:49:26,724] Trial 0 finished with value: 0.10419412383911547 and parameters: {'n_estimators': 261, 'learning_rate': 0.24329342320637573, 'num_leaves': 245, 'max_depth': 9, 'min_child_samples': 11, 'subsample': 0.8353752305202327, 'colsample_bytree': 0.8929361657673347, 'reg_alpha': 9.358726340868975, 'reg_lambda': 2.387252808301633, 'min_split_gain': 0.009808202048050808}. Best is trial 0 with value: 0.10419412383911547.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
1           Price_lag_2        1678
0           Price_lag_1        1666
14      pct_chg_Load_DA        1304
15       lag168_Load_DA        1125
5       WindOn_DA_lag_0        1097
3         Load_DA_lag_0         969
2           Price_lag_7         943
11           NGas_lag_2         806
16  roll2d_median_Price         678
13            EUA_lag_2         676
12            Oil_lag_2         637
10           Coal_lag_2         608
9                  WD_7          52
8                  WD_6          35
7                  WD_1          16
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          10
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 95.76% done
********************* END NS.. 699 ****************************************************
********************* START NS ... 700 ****************************************************
[I 2025-06-15 20:49:30,759] A new study created in memory with name: no-name-dbb6a380-9990-45bb-9f90-a7ac9d73c9ab
[I 2025-06-15 20:49:31,361] Trial 0 finished with value: 0.0011483404510221837 and parameters: {'n_estimators': 130, 'learning_rate': 0.06809781685238923, 'num_leaves': 139, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.8532947808192035, 'colsample_bytree': 0.6561260974915605, 'reg_alpha': 3.868621459877415, 'reg_lambda': 7.697231479624448, 'min_split_gain': 0.041670717524293355}. Best is trial 0 with value: 0.0011483404510221837.
[Trial 0] New best: 0.00115
[I 2025-06-15 20:49:32,416] Trial 1 finished with value: 0.002546030526033386 and parameters: {'n_estimators': 144, 'learning_rate': 0.022565407082039243, 'num_leaves': 49, 'max_depth': 9, 'min_child_samples': 18, 'subsample': 0.5415744288446871, 'colsample_bytree': 0.7128991207751297, 'reg_alpha': 8.407689540630479, 'reg_lambda': 4.885360381293378, 'min_split_gain': 0.035680185789110634}. Best is trial 0 with value: 0.0011483404510221837.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:49:33,193] Trial 2 finished with value: 0.001249701386841582 and parameters: {'n_estimators': 165, 'learning_rate': 0.04625659028644635, 'num_leaves': 241, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.5894931794731143, 'colsample_bytree': 0.7285038780645028, 'reg_alpha': 7.284403878947092, 'reg_lambda': 9.722710965952054, 'min_split_gain': 0.008983924749507611}. Best is trial 0 with value: 0.0011483404510221837.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:49:33,565] Trial 3 finished with value: 0.0008246649497903753 and parameters: {'n_estimators': 91, 'learning_rate': 0.13671239364773838, 'num_leaves': 107, 'max_depth': 5, 'min_child_samples': 24, 'subsample': 0.9324595230426077, 'colsample_bytree': 0.9686976368898216, 'reg_alpha': 6.768526767115423, 'reg_lambda': 4.6499737038540445, 'min_split_gain': 0.003846244631377327}. Best is trial 3 with value: 0.0008246649497903753.
[I 2025-06-15 20:49:33,568] A new study created in memory with name: no-name-d4107f99-3bdc-4eb9-88e7-2b95b7f063ce
[I 2025-06-15 20:49:33,616] Trial 0 finished with value: 0.012265418126427253 and parameters: {'n_estimators': 96, 'learning_rate': 0.06273089154284667, 'num_leaves': 252, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.6562638345162677, 'colsample_bytree': 0.7870930126042373, 'reg_alpha': 3.9758396143826893, 'reg_lambda': 7.24903491559226, 'min_split_gain': 0.008908845489401906}. Best is trial 0 with value: 0.012265418126427253.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         394
14      pct_chg_Load_DA         229
1           Price_lag_2         211
3         Load_DA_lag_0         171
2           Price_lag_7         132
5       WindOn_DA_lag_0         117
13            EUA_lag_2         108
16  roll2d_median_Price         103
15       lag168_Load_DA         100
11           NGas_lag_2          91
10           Coal_lag_2          58
12            Oil_lag_2          32
9                  WD_7          11
8                  WD_6           4
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          72
15       lag168_Load_DA          19
11           NGas_lag_2           8
1           Price_lag_2           5
3         Load_DA_lag_0           1
5       WindOn_DA_lag_0           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 95.90% done
********************* END NS.. 700 ****************************************************
********************* START NS ... 701 ****************************************************
[I 2025-06-15 20:49:35,400] A new study created in memory with name: no-name-e4fe90fa-a6c5-48b0-b0a3-cbf3c6f17f8b
[I 2025-06-15 20:49:35,825] Trial 0 finished with value: 0.00014655061772410696 and parameters: {'n_estimators': 165, 'learning_rate': 0.0872739848432912, 'num_leaves': 70, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.8448164607293305, 'colsample_bytree': 0.909878476998742, 'reg_alpha': 3.2573486188193934, 'reg_lambda': 8.449501143239566, 'min_split_gain': 0.03734332459688171}. Best is trial 0 with value: 0.00014655061772410696.
[Trial 0] New best: 0.00015
[I 2025-06-15 20:49:39,063] Trial 1 finished with value: 0.00031970197477296995 and parameters: {'n_estimators': 149, 'learning_rate': 0.0237928028754878, 'num_leaves': 235, 'max_depth': 16, 'min_child_samples': 8, 'subsample': 0.5434553843841499, 'colsample_bytree': 0.9527759073963098, 'reg_alpha': 0.21239504504910744, 'reg_lambda': 3.6162413425079585, 'min_split_gain': 0.03922662646628287}. Best is trial 0 with value: 0.00014655061772410696.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:49:39,329] Trial 2 finished with value: 0.0001515847078405553 and parameters: {'n_estimators': 206, 'learning_rate': 0.21070199039541282, 'num_leaves': 56, 'max_depth': 13, 'min_child_samples': 26, 'subsample': 0.7082982348650002, 'colsample_bytree': 0.8284724013658253, 'reg_alpha': 9.776486691136352, 'reg_lambda': 2.2856301716330907, 'min_split_gain': 0.022624044147323726}. Best is trial 0 with value: 0.00014655061772410696.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:49:41,099] Trial 3 finished with value: 0.00020344165050405628 and parameters: {'n_estimators': 217, 'learning_rate': 0.03822380406540924, 'num_leaves': 198, 'max_depth': 16, 'min_child_samples': 3, 'subsample': 0.8903950561147902, 'colsample_bytree': 0.7841028476550962, 'reg_alpha': 5.7903506222683765, 'reg_lambda': 9.573184970328743, 'min_split_gain': 0.01518511265054271}. Best is trial 0 with value: 0.00014655061772410696.
[I 2025-06-15 20:49:41,102] A new study created in memory with name: no-name-73c16091-8986-4b81-8c3e-868ed433534c
[I 2025-06-15 20:49:41,189] Trial 0 finished with value: 0.0005306401023969353 and parameters: {'n_estimators': 212, 'learning_rate': 0.040636225000481335, 'num_leaves': 239, 'max_depth': 14, 'min_child_samples': 27, 'subsample': 0.8100796394708046, 'colsample_bytree': 0.9666234435641454, 'reg_alpha': 1.1757254093775482, 'reg_lambda': 6.205545602473031, 'min_split_gain': 0.016578221711084796}. Best is trial 0 with value: 0.0005306401023969353.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         616
14      pct_chg_Load_DA         419
1           Price_lag_2         323
3         Load_DA_lag_0         206
16  roll2d_median_Price         162
2           Price_lag_7         161
11           NGas_lag_2         157
5       WindOn_DA_lag_0         119
13            EUA_lag_2         114
15       lag168_Load_DA          88
10           Coal_lag_2          65
12            Oil_lag_2          51
9                  WD_7          10
7                  WD_1           5
8                  WD_6           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          83
3         Load_DA_lag_0          33
5       WindOn_DA_lag_0          21
14      pct_chg_Load_DA          18
15       lag168_Load_DA           5
10           Coal_lag_2           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 96.03% done
********************* END NS.. 701 ****************************************************
********************* START NS ... 702 ****************************************************
[I 2025-06-15 20:49:43,035] A new study created in memory with name: no-name-65668e71-f7bd-4a96-8e3f-6fe67ead844b
[I 2025-06-15 20:49:43,337] Trial 0 finished with value: 0.00044730474549777976 and parameters: {'n_estimators': 219, 'learning_rate': 0.26588444335364114, 'num_leaves': 211, 'max_depth': 14, 'min_child_samples': 28, 'subsample': 0.9713052122674801, 'colsample_bytree': 0.5941514919523978, 'reg_alpha': 3.7026029034071186, 'reg_lambda': 4.942993847841613, 'min_split_gain': 0.03339461410863313}. Best is trial 0 with value: 0.00044730474549777976.
[Trial 0] New best: 0.00045
[I 2025-06-15 20:49:43,785] Trial 1 finished with value: 0.00045740505946952075 and parameters: {'n_estimators': 267, 'learning_rate': 0.1362567697713194, 'num_leaves': 186, 'max_depth': 9, 'min_child_samples': 29, 'subsample': 0.6133975857225928, 'colsample_bytree': 0.5793194253631406, 'reg_alpha': 7.212214978612282, 'reg_lambda': 8.465284258260834, 'min_split_gain': 0.03725302016207806}. Best is trial 0 with value: 0.00044730474549777976.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:49:44,091] Trial 2 finished with value: 0.002072021241628497 and parameters: {'n_estimators': 197, 'learning_rate': 0.2932742980254683, 'num_leaves': 232, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.60076145272838, 'colsample_bytree': 0.5359305240736814, 'reg_alpha': 7.117965198030522, 'reg_lambda': 3.0431091695447208, 'min_split_gain': 0.047911888001806666}. Best is trial 0 with value: 0.00044730474549777976.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:49:45,141] Trial 3 finished with value: 0.0022659500466755867 and parameters: {'n_estimators': 155, 'learning_rate': 0.016648860334799016, 'num_leaves': 112, 'max_depth': 14, 'min_child_samples': 22, 'subsample': 0.5589662693674691, 'colsample_bytree': 0.724481892786784, 'reg_alpha': 5.966373864112232, 'reg_lambda': 7.38129887407831, 'min_split_gain': 0.013033986658838743}. Best is trial 0 with value: 0.00044730474549777976.
[I 2025-06-15 20:49:45,143] A new study created in memory with name: no-name-57f9c701-1943-47e3-bc2b-22d29f0718b1
[I 2025-06-15 20:49:45,211] Trial 0 finished with value: 0.002042502146250513 and parameters: {'n_estimators': 183, 'learning_rate': 0.062019241501339695, 'num_leaves': 186, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.725167950191814, 'colsample_bytree': 0.6960613589550056, 'reg_alpha': 0.17906665343202288, 'reg_lambda': 9.036232266102965, 'min_split_gain': 0.016802624454160132}. Best is trial 0 with value: 0.002042502146250513.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         220
1           Price_lag_2         157
14      pct_chg_Load_DA          92
3         Load_DA_lag_0          75
2           Price_lag_7          63
16  roll2d_median_Price          58
11           NGas_lag_2          55
15       lag168_Load_DA          38
13            EUA_lag_2          37
10           Coal_lag_2          36
5       WindOn_DA_lag_0          29
12            Oil_lag_2          28
9                  WD_7           4
7                  WD_1           3
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          51
3         Load_DA_lag_0          30
14      pct_chg_Load_DA          16
5       WindOn_DA_lag_0          14
11           NGas_lag_2           6
15       lag168_Load_DA           4
1           Price_lag_2           3
10           Coal_lag_2           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 96.17% done
********************* END NS.. 702 ****************************************************
********************* START NS ... 703 ****************************************************
[I 2025-06-15 20:49:46,883] A new study created in memory with name: no-name-15a46ef5-6489-4881-bcf0-50cbeba05319
[I 2025-06-15 20:49:47,438] Trial 0 finished with value: 0.013691495520042082 and parameters: {'n_estimators': 89, 'learning_rate': 0.10989298420709064, 'num_leaves': 171, 'max_depth': 16, 'min_child_samples': 26, 'subsample': 0.8163710380571199, 'colsample_bytree': 0.7531790210132443, 'reg_alpha': 5.620489584556239, 'reg_lambda': 7.240196645060685, 'min_split_gain': 0.0034448327436703897}. Best is trial 0 with value: 0.013691495520042082.
[Trial 0] New best: 0.01369
[I 2025-06-15 20:49:47,847] Trial 1 finished with value: 0.015782510690224707 and parameters: {'n_estimators': 60, 'learning_rate': 0.05493467304774985, 'num_leaves': 155, 'max_depth': 13, 'min_child_samples': 30, 'subsample': 0.904229675555219, 'colsample_bytree': 0.6895437925368071, 'reg_alpha': 6.010233599846428, 'reg_lambda': 2.6282704301919493, 'min_split_gain': 0.013739552292298713}. Best is trial 0 with value: 0.013691495520042082.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:49:48,901] Trial 2 finished with value: 0.013902264801241506 and parameters: {'n_estimators': 173, 'learning_rate': 0.043899745098827866, 'num_leaves': 160, 'max_depth': 16, 'min_child_samples': 13, 'subsample': 0.8162791085730665, 'colsample_bytree': 0.7391393460809539, 'reg_alpha': 3.9079011714063827, 'reg_lambda': 8.321244625008381, 'min_split_gain': 0.03989753229146603}. Best is trial 0 with value: 0.013691495520042082.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:49:51,042] Trial 3 finished with value: 0.01400471757872295 and parameters: {'n_estimators': 237, 'learning_rate': 0.018862206967108726, 'num_leaves': 155, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.7096426024923976, 'colsample_bytree': 0.6664043194379012, 'reg_alpha': 1.1595240127557738, 'reg_lambda': 4.768295089194013, 'min_split_gain': 0.049765311976016476}. Best is trial 0 with value: 0.013691495520042082.
[I 2025-06-15 20:49:51,044] A new study created in memory with name: no-name-7b561870-e820-412a-8959-eb3bdac2818a
[I 2025-06-15 20:49:51,169] Trial 0 finished with value: 0.02779575757179249 and parameters: {'n_estimators': 223, 'learning_rate': 0.010586195353310242, 'num_leaves': 204, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.6024239511727709, 'colsample_bytree': 0.7803209954103217, 'reg_alpha': 1.1072035675203273, 'reg_lambda': 1.7330107752125068, 'min_split_gain': 0.024349010408885277}. Best is trial 0 with value: 0.02779575757179249.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         603
1           Price_lag_2         470
14      pct_chg_Load_DA         388
3         Load_DA_lag_0         326
13            EUA_lag_2         279
5       WindOn_DA_lag_0         237
15       lag168_Load_DA         228
2           Price_lag_7         206
12            Oil_lag_2         206
11           NGas_lag_2         181
16  roll2d_median_Price         180
10           Coal_lag_2         159
8                  WD_6          19
9                  WD_7          17
7                  WD_1          10
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         279
3         Load_DA_lag_0         126
2           Price_lag_7          35
1           Price_lag_2          16
5       WindOn_DA_lag_0          15
14      pct_chg_Load_DA           4
11           NGas_lag_2           0
15       lag168_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
10           Coal_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 96.31% done
********************* END NS.. 703 ****************************************************
********************* START NS ... 704 ****************************************************
[I 2025-06-15 20:49:52,974] A new study created in memory with name: no-name-2e30a7dc-0597-4c51-9172-862f5d67730b
[I 2025-06-15 20:49:55,023] Trial 0 finished with value: 0.04891749888203284 and parameters: {'n_estimators': 288, 'learning_rate': 0.011282697876665075, 'num_leaves': 133, 'max_depth': 16, 'min_child_samples': 26, 'subsample': 0.5995930086701515, 'colsample_bytree': 0.899588871386559, 'reg_alpha': 7.786672246485712, 'reg_lambda': 9.624527014127667, 'min_split_gain': 0.005729642548238406}. Best is trial 0 with value: 0.04891749888203284.
[Trial 0] New best: 0.04892
[I 2025-06-15 20:49:56,025] Trial 1 finished with value: 0.04227975881035631 and parameters: {'n_estimators': 176, 'learning_rate': 0.06391396724550182, 'num_leaves': 206, 'max_depth': 7, 'min_child_samples': 28, 'subsample': 0.7372153856017755, 'colsample_bytree': 0.7170510465191633, 'reg_alpha': 0.42267908068621685, 'reg_lambda': 8.921481587372732, 'min_split_gain': 0.021940222345417953}. Best is trial 1 with value: 0.04227975881035631.
[Trial 1] New best: 0.04228
[I 2025-06-15 20:49:56,507] Trial 2 finished with value: 0.050987197092898805 and parameters: {'n_estimators': 227, 'learning_rate': 0.13166248624632787, 'num_leaves': 146, 'max_depth': 10, 'min_child_samples': 30, 'subsample': 0.709227949237106, 'colsample_bytree': 0.5492422859639268, 'reg_alpha': 9.73387082394665, 'reg_lambda': 6.102275279201191, 'min_split_gain': 0.02617161311110386}. Best is trial 1 with value: 0.04227975881035631.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:49:56,796] Trial 3 finished with value: 0.044583661257021234 and parameters: {'n_estimators': 208, 'learning_rate': 0.18272572143237542, 'num_leaves': 115, 'max_depth': 12, 'min_child_samples': 26, 'subsample': 0.761284420558949, 'colsample_bytree': 0.7613258306299735, 'reg_alpha': 9.277005475707764, 'reg_lambda': 7.853046044615226, 'min_split_gain': 0.022437776036528908}. Best is trial 1 with value: 0.04227975881035631.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:49:57,629] Trial 4 finished with value: 0.04495954945093539 and parameters: {'n_estimators': 91, 'learning_rate': 0.05633326561384442, 'num_leaves': 136, 'max_depth': 7, 'min_child_samples': 16, 'subsample': 0.7110581190191025, 'colsample_bytree': 0.5836237164403442, 'reg_alpha': 1.457962323096661, 'reg_lambda': 3.9728884374222826, 'min_split_gain': 0.015991666587019126}. Best is trial 1 with value: 0.04227975881035631.
[I 2025-06-15 20:49:57,632] A new study created in memory with name: no-name-646ec8e9-7b14-48e1-8f08-48898daed33a
[I 2025-06-15 20:49:57,667] Trial 0 finished with value: 0.2811786715867379 and parameters: {'n_estimators': 59, 'learning_rate': 0.011133051051823148, 'num_leaves': 73, 'max_depth': 8, 'min_child_samples': 30, 'subsample': 0.8051022246597619, 'colsample_bytree': 0.9677482249854342, 'reg_alpha': 1.77564938323851, 'reg_lambda': 8.023620282530125, 'min_split_gain': 0.04406443320283632}. Best is trial 0 with value: 0.2811786715867379.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1122
1           Price_lag_2         907
14      pct_chg_Load_DA         800
3         Load_DA_lag_0         501
16  roll2d_median_Price         440
11           NGas_lag_2         351
15       lag168_Load_DA         337
5       WindOn_DA_lag_0         331
2           Price_lag_7         292
13            EUA_lag_2         273
10           Coal_lag_2         223
12            Oil_lag_2         203
9                  WD_7          26
7                  WD_1          13
8                  WD_6          13
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          94
3         Load_DA_lag_0           3
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 96.44% done
********************* END NS.. 704 ****************************************************
********************* START NS ... 705 ****************************************************
[I 2025-06-15 20:49:59,996] A new study created in memory with name: no-name-7f01aac5-cb82-45b7-9d0b-fed44fdec812
[I 2025-06-15 20:50:00,525] Trial 0 finished with value: 0.033457793627865655 and parameters: {'n_estimators': 82, 'learning_rate': 0.09021000887843053, 'num_leaves': 96, 'max_depth': 8, 'min_child_samples': 30, 'subsample': 0.7055771103680728, 'colsample_bytree': 0.9121785849569255, 'reg_alpha': 3.631064348883002, 'reg_lambda': 9.36913158592348, 'min_split_gain': 0.027426514758839983}. Best is trial 0 with value: 0.033457793627865655.
[Trial 0] New best: 0.03346
[I 2025-06-15 20:50:01,644] Trial 1 finished with value: 0.032661512130900454 and parameters: {'n_estimators': 214, 'learning_rate': 0.07445480588137361, 'num_leaves': 237, 'max_depth': 14, 'min_child_samples': 30, 'subsample': 0.630050420813351, 'colsample_bytree': 0.6736120710773604, 'reg_alpha': 4.261747356681339, 'reg_lambda': 1.9365979451276427, 'min_split_gain': 0.010733286364438372}. Best is trial 1 with value: 0.032661512130900454.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:50:01,948] Trial 2 finished with value: 0.06173736895544172 and parameters: {'n_estimators': 55, 'learning_rate': 0.01827381920017971, 'num_leaves': 139, 'max_depth': 7, 'min_child_samples': 22, 'subsample': 0.9332910179616952, 'colsample_bytree': 0.8421427260151878, 'reg_alpha': 9.860676342370724, 'reg_lambda': 6.796159935906915, 'min_split_gain': 0.029101551322906455}. Best is trial 1 with value: 0.032661512130900454.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:50:02,289] Trial 3 finished with value: 0.03529500159647648 and parameters: {'n_estimators': 223, 'learning_rate': 0.10241722777019367, 'num_leaves': 142, 'max_depth': 5, 'min_child_samples': 26, 'subsample': 0.7271303657564517, 'colsample_bytree': 0.746682963675497, 'reg_alpha': 7.147574913993693, 'reg_lambda': 2.424118152972008, 'min_split_gain': 0.04306446411708802}. Best is trial 1 with value: 0.032661512130900454.
[I 2025-06-15 20:50:02,292] A new study created in memory with name: no-name-54d5821e-f6dc-4d2d-bd2b-1996c1bd6880
[I 2025-06-15 20:50:02,353] Trial 0 finished with value: 0.26743500377136314 and parameters: {'n_estimators': 162, 'learning_rate': 0.02286974981628965, 'num_leaves': 249, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.9794098423738293, 'colsample_bytree': 0.8468384743495216, 'reg_alpha': 5.237054652429524, 'reg_lambda': 0.0852519597582524, 'min_split_gain': 0.037876650215033295}. Best is trial 0 with value: 0.26743500377136314.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         843
1           Price_lag_2         669
14      pct_chg_Load_DA         503
3         Load_DA_lag_0         377
15       lag168_Load_DA         303
11           NGas_lag_2         266
5       WindOn_DA_lag_0         243
2           Price_lag_7         239
13            EUA_lag_2         233
16  roll2d_median_Price         227
12            Oil_lag_2         160
10           Coal_lag_2         156
9                  WD_7          31
8                  WD_6          21
7                  WD_1           9
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          99
3         Load_DA_lag_0          46
11           NGas_lag_2          28
5       WindOn_DA_lag_0          27
1           Price_lag_2           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 96.58% done
********************* END NS.. 705 ****************************************************
********************* START NS ... 706 ****************************************************
[I 2025-06-15 20:50:04,558] A new study created in memory with name: no-name-d21b129d-1431-4836-956e-26688dd88656
[I 2025-06-15 20:50:05,561] Trial 0 finished with value: 0.01176557886439626 and parameters: {'n_estimators': 275, 'learning_rate': 0.013209015745816038, 'num_leaves': 119, 'max_depth': 5, 'min_child_samples': 16, 'subsample': 0.8995390924456567, 'colsample_bytree': 0.7495993970193495, 'reg_alpha': 8.865807893113525, 'reg_lambda': 5.402783411618448, 'min_split_gain': 0.005584259955374416}. Best is trial 0 with value: 0.01176557886439626.
[Trial 0] New best: 0.01177
[I 2025-06-15 20:50:06,067] Trial 1 finished with value: 0.008258048066460291 and parameters: {'n_estimators': 219, 'learning_rate': 0.2677963869899748, 'num_leaves': 43, 'max_depth': 13, 'min_child_samples': 28, 'subsample': 0.8313101414691597, 'colsample_bytree': 0.6394125003524689, 'reg_alpha': 9.753624594770056, 'reg_lambda': 2.7157872851040796, 'min_split_gain': 0.003212539995955227}. Best is trial 1 with value: 0.008258048066460291.
[Trial 1] New best: 0.00826
[I 2025-06-15 20:50:07,742] Trial 2 finished with value: 0.009869554037696175 and parameters: {'n_estimators': 254, 'learning_rate': 0.015791303475110617, 'num_leaves': 143, 'max_depth': 10, 'min_child_samples': 8, 'subsample': 0.9785410829001147, 'colsample_bytree': 0.9301945218430443, 'reg_alpha': 8.279494137933266, 'reg_lambda': 3.858378586116291, 'min_split_gain': 0.03474119205727934}. Best is trial 1 with value: 0.008258048066460291.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:50:07,980] Trial 3 finished with value: 0.00839943126371437 and parameters: {'n_estimators': 70, 'learning_rate': 0.24674793235106224, 'num_leaves': 151, 'max_depth': 6, 'min_child_samples': 16, 'subsample': 0.660370304262911, 'colsample_bytree': 0.9091933382087327, 'reg_alpha': 4.085901753408537, 'reg_lambda': 7.139553043135084, 'min_split_gain': 0.020455410450752484}. Best is trial 1 with value: 0.008258048066460291.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:50:08,388] Trial 4 finished with value: 0.009676870342287426 and parameters: {'n_estimators': 167, 'learning_rate': 0.26097292124380245, 'num_leaves': 224, 'max_depth': 11, 'min_child_samples': 1, 'subsample': 0.5907466272671764, 'colsample_bytree': 0.8969291296990148, 'reg_alpha': 7.94762168343377, 'reg_lambda': 0.8391540767793426, 'min_split_gain': 0.035305952672932815}. Best is trial 1 with value: 0.008258048066460291.
[I 2025-06-15 20:50:08,390] A new study created in memory with name: no-name-f07ac552-cdce-4b92-bdab-2427c9adbae1
[I 2025-06-15 20:50:08,495] Trial 0 finished with value: 0.044762695706907024 and parameters: {'n_estimators': 191, 'learning_rate': 0.03988464725055328, 'num_leaves': 46, 'max_depth': 13, 'min_child_samples': 11, 'subsample': 0.5256721437603635, 'colsample_bytree': 0.8701545637089763, 'reg_alpha': 1.0094288647460914, 'reg_lambda': 2.8379099624357664, 'min_split_gain': 0.014463997708700266}. Best is trial 0 with value: 0.044762695706907024.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         206
1           Price_lag_2         158
3         Load_DA_lag_0         103
14      pct_chg_Load_DA          82
11           NGas_lag_2          80
16  roll2d_median_Price          70
2           Price_lag_7          70
10           Coal_lag_2          67
5       WindOn_DA_lag_0          55
15       lag168_Load_DA          54
13            EUA_lag_2          51
12            Oil_lag_2          21
9                  WD_7           7
7                  WD_1           5
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         115
3         Load_DA_lag_0          99
5       WindOn_DA_lag_0          56
16  roll2d_median_Price          29
2           Price_lag_7          27
15       lag168_Load_DA          22
14      pct_chg_Load_DA          11
1           Price_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
8                  WD_6           0
-> 96.72% done
********************* END NS.. 706 ****************************************************
********************* START NS ... 707 ****************************************************
[I 2025-06-15 20:50:09,960] A new study created in memory with name: no-name-004e6d42-db8b-41fe-b832-8a7d50908930
[I 2025-06-15 20:50:10,583] Trial 0 finished with value: 0.007832337284444294 and parameters: {'n_estimators': 262, 'learning_rate': 0.01082735005868552, 'num_leaves': 32, 'max_depth': 4, 'min_child_samples': 1, 'subsample': 0.629428619813763, 'colsample_bytree': 0.987452121766446, 'reg_alpha': 3.6898848515123928, 'reg_lambda': 2.7730734560022627, 'min_split_gain': 0.009097865745903034}. Best is trial 0 with value: 0.007832337284444294.
[Trial 0] New best: 0.00783
[I 2025-06-15 20:50:11,984] Trial 1 finished with value: 0.0096261452718097 and parameters: {'n_estimators': 158, 'learning_rate': 0.016267229393353667, 'num_leaves': 55, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.5078456351052478, 'colsample_bytree': 0.5793522117722711, 'reg_alpha': 4.463280456472719, 'reg_lambda': 3.0275142200086567, 'min_split_gain': 0.025628314263040044}. Best is trial 0 with value: 0.007832337284444294.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:50:12,911] Trial 2 finished with value: 0.0059118342112419115 and parameters: {'n_estimators': 106, 'learning_rate': 0.07294521569409984, 'num_leaves': 82, 'max_depth': 11, 'min_child_samples': 1, 'subsample': 0.9106400707672777, 'colsample_bytree': 0.8791836841899834, 'reg_alpha': 3.0179598058354786, 'reg_lambda': 8.854648872648536, 'min_split_gain': 0.03607650939013097}. Best is trial 2 with value: 0.0059118342112419115.
[Trial 2] New best: 0.00591
[I 2025-06-15 20:50:14,036] Trial 3 finished with value: 0.019158838945878626 and parameters: {'n_estimators': 115, 'learning_rate': 0.011980819495203033, 'num_leaves': 48, 'max_depth': 9, 'min_child_samples': 8, 'subsample': 0.6630441013231198, 'colsample_bytree': 0.8702440052166409, 'reg_alpha': 2.6372798398358546, 'reg_lambda': 3.645008299745225, 'min_split_gain': 0.009216214063735989}. Best is trial 2 with value: 0.0059118342112419115.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:50:16,527] Trial 4 finished with value: 0.007007631380391492 and parameters: {'n_estimators': 129, 'learning_rate': 0.023166785700122493, 'num_leaves': 89, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.9176002603275609, 'colsample_bytree': 0.7909447818090329, 'reg_alpha': 0.07686756118221338, 'reg_lambda': 0.48013049642654404, 'min_split_gain': 0.016843833040689045}. Best is trial 2 with value: 0.0059118342112419115.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:50:17,184] Trial 5 finished with value: 0.005897495354673144 and parameters: {'n_estimators': 166, 'learning_rate': 0.10695290500742656, 'num_leaves': 246, 'max_depth': 12, 'min_child_samples': 3, 'subsample': 0.7407087480028931, 'colsample_bytree': 0.8375391849993443, 'reg_alpha': 3.5798475157206298, 'reg_lambda': 4.427028711593556, 'min_split_gain': 0.04182065383586329}. Best is trial 5 with value: 0.005897495354673144.
[I 2025-06-15 20:50:17,187] A new study created in memory with name: no-name-1d5fd4dc-0f74-412a-b898-0af1b8f116b2
[I 2025-06-15 20:50:17,221] Trial 0 finished with value: 0.02175463627799527 and parameters: {'n_estimators': 50, 'learning_rate': 0.16528793492255286, 'num_leaves': 146, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.7883459626429661, 'colsample_bytree': 0.7103611725026656, 'reg_alpha': 7.962077689449556, 'reg_lambda': 4.554990413575732, 'min_split_gain': 0.01844789563660076}. Best is trial 0 with value: 0.02175463627799527.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         499
1           Price_lag_2         369
14      pct_chg_Load_DA         330
3         Load_DA_lag_0         195
16  roll2d_median_Price         181
11           NGas_lag_2         147
2           Price_lag_7         141
13            EUA_lag_2          96
10           Coal_lag_2          84
15       lag168_Load_DA          83
5       WindOn_DA_lag_0          82
12            Oil_lag_2          66
9                  WD_7           9
8                  WD_6           7
7                  WD_1           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          19
3         Load_DA_lag_0           6
5       WindOn_DA_lag_0           4
1           Price_lag_2           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 96.85% done
********************* END NS.. 707 ****************************************************
********************* START NS ... 708 ****************************************************
[I 2025-06-15 20:50:19,070] A new study created in memory with name: no-name-7801ac80-d04f-4700-9911-91869c74bf54
[I 2025-06-15 20:50:20,981] Trial 0 finished with value: 0.0005776475482882417 and parameters: {'n_estimators': 216, 'learning_rate': 0.022945145165386746, 'num_leaves': 158, 'max_depth': 10, 'min_child_samples': 26, 'subsample': 0.6296554508011363, 'colsample_bytree': 0.9620332851491415, 'reg_alpha': 7.259169574040898, 'reg_lambda': 2.859109915478981, 'min_split_gain': 0.0012126029834055197}. Best is trial 0 with value: 0.0005776475482882417.
[Trial 0] New best: 0.00058
[I 2025-06-15 20:50:21,442] Trial 1 finished with value: 0.0005914360443260472 and parameters: {'n_estimators': 95, 'learning_rate': 0.09062842140236796, 'num_leaves': 46, 'max_depth': 11, 'min_child_samples': 28, 'subsample': 0.7839551468966838, 'colsample_bytree': 0.6639136875994538, 'reg_alpha': 5.717180075044633, 'reg_lambda': 7.568035012466159, 'min_split_gain': 0.040839316482185396}. Best is trial 0 with value: 0.0005776475482882417.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:50:22,467] Trial 2 finished with value: 0.0005861898075076302 and parameters: {'n_estimators': 254, 'learning_rate': 0.025770331692763736, 'num_leaves': 148, 'max_depth': 5, 'min_child_samples': 6, 'subsample': 0.5371950899633058, 'colsample_bytree': 0.9599343423916447, 'reg_alpha': 6.514905914818058, 'reg_lambda': 6.95827221095247, 'min_split_gain': 0.04208947714274913}. Best is trial 0 with value: 0.0005776475482882417.
[I 2025-06-15 20:50:22,658] Trial 3 finished with value: 0.0008029634627144851 and parameters: {'n_estimators': 81, 'learning_rate': 0.265033746646587, 'num_leaves': 179, 'max_depth': 4, 'min_child_samples': 3, 'subsample': 0.9083998026866748, 'colsample_bytree': 0.8161800875079581, 'reg_alpha': 2.2975982152933994, 'reg_lambda': 0.8438597598806008, 'min_split_gain': 0.015104574252687887}. Best is trial 0 with value: 0.0005776475482882417.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:50:22,661] A new study created in memory with name: no-name-a24bf18b-3321-49f6-848b-d0cc28f50a59
[I 2025-06-15 20:50:22,722] Trial 0 finished with value: 0.018853223714256833 and parameters: {'n_estimators': 201, 'learning_rate': 0.038990680378107496, 'num_leaves': 177, 'max_depth': 15, 'min_child_samples': 26, 'subsample': 0.557485527385566, 'colsample_bytree': 0.7508283418800654, 'reg_alpha': 8.665312126883649, 'reg_lambda': 0.2474725119088672, 'min_split_gain': 0.028985350160992785}. Best is trial 0 with value: 0.018853223714256833.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2113
14      pct_chg_Load_DA        1261
1           Price_lag_2        1170
16  roll2d_median_Price         562
13            EUA_lag_2         541
2           Price_lag_7         527
3         Load_DA_lag_0         514
11           NGas_lag_2         395
10           Coal_lag_2         356
15       lag168_Load_DA         323
12            Oil_lag_2         304
5       WindOn_DA_lag_0         274
8                  WD_6          35
9                  WD_7          25
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1          70
3         Load_DA_lag_0          10
5       WindOn_DA_lag_0           2
1           Price_lag_2           2
10           Coal_lag_2           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 96.99% done
********************* END NS.. 708 ****************************************************
********************* START NS ... 709 ****************************************************
[I 2025-06-15 20:50:25,571] A new study created in memory with name: no-name-8bfba503-99e0-4af3-a1d8-89d2219533d5
[I 2025-06-15 20:50:27,001] Trial 0 finished with value: 0.0013418199813110979 and parameters: {'n_estimators': 288, 'learning_rate': 0.08123031037005453, 'num_leaves': 39, 'max_depth': 13, 'min_child_samples': 11, 'subsample': 0.80736656830371, 'colsample_bytree': 0.5059396301878833, 'reg_alpha': 4.1527782980034935, 'reg_lambda': 6.145852642351324, 'min_split_gain': 0.004716745958995284}. Best is trial 0 with value: 0.0013418199813110979.
[Trial 0] New best: 0.00134
[I 2025-06-15 20:50:27,767] Trial 1 finished with value: 0.0014627437492382167 and parameters: {'n_estimators': 98, 'learning_rate': 0.052992933308506136, 'num_leaves': 150, 'max_depth': 15, 'min_child_samples': 7, 'subsample': 0.8791622289018666, 'colsample_bytree': 0.8422692057871102, 'reg_alpha': 9.470736933738209, 'reg_lambda': 9.691387457312398, 'min_split_gain': 0.009814051148213071}. Best is trial 0 with value: 0.0013418199813110979.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:50:29,162] Trial 2 finished with value: 0.0016013607683992767 and parameters: {'n_estimators': 115, 'learning_rate': 0.04419555757499818, 'num_leaves': 219, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.6368440529589529, 'colsample_bytree': 0.542446710463001, 'reg_alpha': 3.272541525671472, 'reg_lambda': 1.4874413688498012, 'min_split_gain': 0.014480455314661862}. Best is trial 0 with value: 0.0013418199813110979.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:50:31,877] Trial 3 finished with value: 0.0017234694273112066 and parameters: {'n_estimators': 277, 'learning_rate': 0.0181329213956567, 'num_leaves': 150, 'max_depth': 13, 'min_child_samples': 9, 'subsample': 0.7771519315179632, 'colsample_bytree': 0.5066305731154813, 'reg_alpha': 5.964877763866399, 'reg_lambda': 0.6123037756823313, 'min_split_gain': 0.03427218683093414}. Best is trial 0 with value: 0.0013418199813110979.
[I 2025-06-15 20:50:31,880] A new study created in memory with name: no-name-02520fe1-09e0-472d-b380-825bfa83355f
[I 2025-06-15 20:50:31,921] Trial 0 finished with value: 0.0071519970055627475 and parameters: {'n_estimators': 99, 'learning_rate': 0.0880039316625937, 'num_leaves': 55, 'max_depth': 8, 'min_child_samples': 26, 'subsample': 0.9772055429619457, 'colsample_bytree': 0.6478432320600024, 'reg_alpha': 4.1734043233014875, 'reg_lambda': 9.838818459883786, 'min_split_gain': 0.04535525335979909}. Best is trial 0 with value: 0.0071519970055627475.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1223
1           Price_lag_2        1187
14      pct_chg_Load_DA         548
3         Load_DA_lag_0         446
13            EUA_lag_2         389
5       WindOn_DA_lag_0         377
2           Price_lag_7         320
16  roll2d_median_Price         310
11           NGas_lag_2         282
15       lag168_Load_DA         268
12            Oil_lag_2         257
10           Coal_lag_2         228
8                  WD_6          34
9                  WD_7          28
7                  WD_1          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          43
3         Load_DA_lag_0          14
10           Coal_lag_2           4
5       WindOn_DA_lag_0           1
1           Price_lag_2           1
11           NGas_lag_2           1
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 97.13% done
********************* END NS.. 709 ****************************************************
********************* START NS ... 710 ****************************************************
[I 2025-06-15 20:50:34,651] A new study created in memory with name: no-name-aec4ed9a-680d-4f5e-8322-8750725c06e3
[I 2025-06-15 20:50:35,412] Trial 0 finished with value: 0.0651701907847846 and parameters: {'n_estimators': 85, 'learning_rate': 0.010324714416865062, 'num_leaves': 95, 'max_depth': 16, 'min_child_samples': 22, 'subsample': 0.8806688114719106, 'colsample_bytree': 0.6276104094352408, 'reg_alpha': 3.559803762537875, 'reg_lambda': 5.3409202841995, 'min_split_gain': 0.010419251860356743}. Best is trial 0 with value: 0.0651701907847846.
[Trial 0] New best: 0.06517
[I 2025-06-15 20:50:36,314] Trial 1 finished with value: 0.021395660216340665 and parameters: {'n_estimators': 128, 'learning_rate': 0.042269732778913424, 'num_leaves': 50, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.5425363826326406, 'colsample_bytree': 0.5181265408681343, 'reg_alpha': 6.156469136508139, 'reg_lambda': 8.914280439326252, 'min_split_gain': 0.02929357289129509}. Best is trial 1 with value: 0.021395660216340665.
[Trial 1] New best: 0.02140
[I 2025-06-15 20:50:38,189] Trial 2 finished with value: 0.012171215051839929 and parameters: {'n_estimators': 186, 'learning_rate': 0.03996087501604906, 'num_leaves': 145, 'max_depth': 9, 'min_child_samples': 14, 'subsample': 0.8180183748940958, 'colsample_bytree': 0.8054293387821849, 'reg_alpha': 4.935108580586526, 'reg_lambda': 7.718334667450489, 'min_split_gain': 0.003301973907753497}. Best is trial 2 with value: 0.012171215051839929.
[Trial 2] New best: 0.01217
[I 2025-06-15 20:50:39,474] Trial 3 finished with value: 0.013849261321251994 and parameters: {'n_estimators': 144, 'learning_rate': 0.04692444872178385, 'num_leaves': 109, 'max_depth': 6, 'min_child_samples': 2, 'subsample': 0.8968609055955523, 'colsample_bytree': 0.5873018126668008, 'reg_alpha': 1.3053200603958692, 'reg_lambda': 6.048662272595459, 'min_split_gain': 0.031324843196295866}. Best is trial 2 with value: 0.012171215051839929.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:50:42,670] Trial 4 finished with value: 0.014436840083684824 and parameters: {'n_estimators': 299, 'learning_rate': 0.017281760018201815, 'num_leaves': 238, 'max_depth': 12, 'min_child_samples': 28, 'subsample': 0.5434472507487056, 'colsample_bytree': 0.8282783947268366, 'reg_alpha': 2.2250199163755124, 'reg_lambda': 0.10171439813962069, 'min_split_gain': 0.0240514892084763}. Best is trial 2 with value: 0.012171215051839929.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:50:46,053] Trial 5 finished with value: 0.017076630418833724 and parameters: {'n_estimators': 264, 'learning_rate': 0.011566341285186264, 'num_leaves': 145, 'max_depth': 16, 'min_child_samples': 9, 'subsample': 0.7169455163387929, 'colsample_bytree': 0.8187728145724134, 'reg_alpha': 2.044453907733664, 'reg_lambda': 0.2570147808479839, 'min_split_gain': 0.025309875906510543}. Best is trial 2 with value: 0.012171215051839929.
[I 2025-06-15 20:50:46,055] A new study created in memory with name: no-name-33c371fd-0349-4b3e-977a-91be9c37dd0e
[I 2025-06-15 20:50:46,127] Trial 0 finished with value: 0.11140572692274957 and parameters: {'n_estimators': 191, 'learning_rate': 0.06521400949815617, 'num_leaves': 126, 'max_depth': 12, 'min_child_samples': 3, 'subsample': 0.6708360795341793, 'colsample_bytree': 0.7516371513648523, 'reg_alpha': 4.170717022245698, 'reg_lambda': 4.60156677909021, 'min_split_gain': 0.010759811725406167}. Best is trial 0 with value: 0.11140572692274957.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1651
1           Price_lag_2        1346
14      pct_chg_Load_DA        1156
3         Load_DA_lag_0         834
13            EUA_lag_2         659
2           Price_lag_7         589
5       WindOn_DA_lag_0         587
16  roll2d_median_Price         576
15       lag168_Load_DA         541
11           NGas_lag_2         527
12            Oil_lag_2         476
10           Coal_lag_2         326
9                  WD_7          52
8                  WD_6          49
7                  WD_1          19
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          85
5       WindOn_DA_lag_0          42
3         Load_DA_lag_0          39
2           Price_lag_7          16
1           Price_lag_2          15
15       lag168_Load_DA          12
14      pct_chg_Load_DA           7
12            Oil_lag_2           7
16  roll2d_median_Price           5
13            EUA_lag_2           3
11           NGas_lag_2           1
9                  WD_7           0
10           Coal_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 97.26% done
********************* END NS.. 710 ****************************************************
********************* START NS ... 711 ****************************************************
[I 2025-06-15 20:50:49,172] A new study created in memory with name: no-name-8590966d-3bdd-4e8a-8526-76261c1fd780
[I 2025-06-15 20:50:50,298] Trial 0 finished with value: 0.14997427664149102 and parameters: {'n_estimators': 123, 'learning_rate': 0.026555261731995815, 'num_leaves': 124, 'max_depth': 10, 'min_child_samples': 26, 'subsample': 0.8281429235951971, 'colsample_bytree': 0.6793605757393413, 'reg_alpha': 2.4376445375751574, 'reg_lambda': 3.1008386157201597, 'min_split_gain': 0.03722941238434375}. Best is trial 0 with value: 0.14997427664149102.
[Trial 0] New best: 0.14997
[I 2025-06-15 20:50:51,138] Trial 1 finished with value: 0.13840814387840025 and parameters: {'n_estimators': 224, 'learning_rate': 0.03412292622407726, 'num_leaves': 143, 'max_depth': 5, 'min_child_samples': 8, 'subsample': 0.7211606162134434, 'colsample_bytree': 0.8787733469520411, 'reg_alpha': 5.841290075006906, 'reg_lambda': 4.40692869103637, 'min_split_gain': 0.01393276616162722}. Best is trial 1 with value: 0.13840814387840025.
[Trial 1] New best: 0.13841
[I 2025-06-15 20:50:51,591] Trial 2 finished with value: 0.2657531726828462 and parameters: {'n_estimators': 72, 'learning_rate': 0.018423404794291824, 'num_leaves': 127, 'max_depth': 7, 'min_child_samples': 12, 'subsample': 0.6859776867258276, 'colsample_bytree': 0.6694312955202228, 'reg_alpha': 6.050774664939805, 'reg_lambda': 4.861119593526558, 'min_split_gain': 0.04639054340178399}. Best is trial 1 with value: 0.13840814387840025.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:50:51,896] Trial 3 finished with value: 0.10564671629172741 and parameters: {'n_estimators': 86, 'learning_rate': 0.18587607351200525, 'num_leaves': 241, 'max_depth': 15, 'min_child_samples': 30, 'subsample': 0.8015309512219365, 'colsample_bytree': 0.97723766488618, 'reg_alpha': 3.2725103685746704, 'reg_lambda': 5.3826271603244225, 'min_split_gain': 0.03191995399775236}. Best is trial 3 with value: 0.10564671629172741.
[Trial 3] New best: 0.10565
[I 2025-06-15 20:50:52,142] Trial 4 finished with value: 0.20035597301853939 and parameters: {'n_estimators': 83, 'learning_rate': 0.01619542993010835, 'num_leaves': 183, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.5251542034002712, 'colsample_bytree': 0.8314629168100667, 'reg_alpha': 8.914292630503027, 'reg_lambda': 5.190245337801756, 'min_split_gain': 0.030850480315996926}. Best is trial 3 with value: 0.10564671629172741.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:50:52,492] Trial 5 finished with value: 0.1651582000163838 and parameters: {'n_estimators': 190, 'learning_rate': 0.21590228958085136, 'num_leaves': 49, 'max_depth': 12, 'min_child_samples': 4, 'subsample': 0.791305979725299, 'colsample_bytree': 0.6319909272082462, 'reg_alpha': 7.854591964005956, 'reg_lambda': 7.008035668785061, 'min_split_gain': 0.04963700889290264}. Best is trial 3 with value: 0.10564671629172741.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:50:53,653] Trial 6 finished with value: 0.16337296972240647 and parameters: {'n_estimators': 219, 'learning_rate': 0.014116251902207015, 'num_leaves': 120, 'max_depth': 6, 'min_child_samples': 25, 'subsample': 0.7284188988784135, 'colsample_bytree': 0.6571749735877784, 'reg_alpha': 9.433943383775794, 'reg_lambda': 2.3060826122880163, 'min_split_gain': 0.04834460727338361}. Best is trial 3 with value: 0.10564671629172741.
[I 2025-06-15 20:50:53,654] A new study created in memory with name: no-name-7344bef1-4820-49b5-b761-94ee63945944
[I 2025-06-15 20:50:53,714] Trial 0 finished with value: 1.3997106086127082 and parameters: {'n_estimators': 200, 'learning_rate': 0.03867199606678181, 'num_leaves': 193, 'max_depth': 4, 'min_child_samples': 11, 'subsample': 0.831979884822776, 'colsample_bytree': 0.9247602117197192, 'reg_alpha': 3.655793244991491, 'reg_lambda': 0.5944855381483094, 'min_split_gain': 0.04850138998325106}. Best is trial 0 with value: 1.3997106086127082.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         292
14      pct_chg_Load_DA         211
1           Price_lag_2         173
3         Load_DA_lag_0         120
16  roll2d_median_Price          86
2           Price_lag_7          81
11           NGas_lag_2          74
5       WindOn_DA_lag_0          64
13            EUA_lag_2          60
15       lag168_Load_DA          55
10           Coal_lag_2          34
12            Oil_lag_2          20
9                  WD_7           6
7                  WD_1           5
8                  WD_6           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         120
3         Load_DA_lag_0          63
10           Coal_lag_2          27
15       lag168_Load_DA          21
5       WindOn_DA_lag_0          10
1           Price_lag_2           3
12            Oil_lag_2           1
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 97.40% done
********************* END NS.. 711 ****************************************************
********************* START NS ... 712 ****************************************************
[I 2025-06-15 20:50:55,170] A new study created in memory with name: no-name-fad9c1a2-f0c2-4271-8f05-f84b47022222
[I 2025-06-15 20:50:55,664] Trial 0 finished with value: 0.4672349509239153 and parameters: {'n_estimators': 118, 'learning_rate': 0.10456848896608333, 'num_leaves': 96, 'max_depth': 15, 'min_child_samples': 30, 'subsample': 0.6069534480535423, 'colsample_bytree': 0.5733706004407613, 'reg_alpha': 9.690977475606044, 'reg_lambda': 3.2236656229842344, 'min_split_gain': 0.009827527788980862}. Best is trial 0 with value: 0.4672349509239153.
[Trial 0] New best: 0.46723
[I 2025-06-15 20:50:56,731] Trial 1 finished with value: 0.46851124421036844 and parameters: {'n_estimators': 240, 'learning_rate': 0.028096424289616805, 'num_leaves': 223, 'max_depth': 6, 'min_child_samples': 27, 'subsample': 0.5726800304788817, 'colsample_bytree': 0.7605478980356083, 'reg_alpha': 9.083789670758694, 'reg_lambda': 9.44526445342355, 'min_split_gain': 0.04927760529540117}. Best is trial 0 with value: 0.4672349509239153.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:50:57,311] Trial 2 finished with value: 0.4392960132328239 and parameters: {'n_estimators': 275, 'learning_rate': 0.0928569090808146, 'num_leaves': 100, 'max_depth': 9, 'min_child_samples': 11, 'subsample': 0.9030693042156388, 'colsample_bytree': 0.6364126531564978, 'reg_alpha': 5.733605840358359, 'reg_lambda': 3.773383702671719, 'min_split_gain': 0.035504151404955275}. Best is trial 2 with value: 0.4392960132328239.
[Trial 2] New best: 0.43930
[I 2025-06-15 20:50:58,223] Trial 3 finished with value: 0.45639402076032526 and parameters: {'n_estimators': 187, 'learning_rate': 0.011202702937995222, 'num_leaves': 144, 'max_depth': 5, 'min_child_samples': 30, 'subsample': 0.8462664741651695, 'colsample_bytree': 0.9675777080949848, 'reg_alpha': 4.481949479938857, 'reg_lambda': 3.4359517656149308, 'min_split_gain': 0.047202762620910825}. Best is trial 2 with value: 0.4392960132328239.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:50:58,746] Trial 4 finished with value: 0.624649992380719 and parameters: {'n_estimators': 80, 'learning_rate': 0.028357978392506738, 'num_leaves': 110, 'max_depth': 9, 'min_child_samples': 29, 'subsample': 0.868471646153685, 'colsample_bytree': 0.6147417031970888, 'reg_alpha': 8.56236827931459, 'reg_lambda': 0.6492140219239106, 'min_split_gain': 0.01604552720550405}. Best is trial 2 with value: 0.4392960132328239.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:50:59,527] Trial 5 finished with value: 0.6439328311770228 and parameters: {'n_estimators': 126, 'learning_rate': 0.0130292880356738, 'num_leaves': 75, 'max_depth': 5, 'min_child_samples': 26, 'subsample': 0.8604157374860328, 'colsample_bytree': 0.6497313626633026, 'reg_alpha': 0.20875114216060364, 'reg_lambda': 3.4279995395236718, 'min_split_gain': 0.02953509412247911}. Best is trial 2 with value: 0.4392960132328239.
[I 2025-06-15 20:50:59,528] A new study created in memory with name: no-name-3a53eccb-3a79-4b9f-a2ea-7d57f46cff7c
[I 2025-06-15 20:50:59,616] Trial 0 finished with value: 3.2985079747757955 and parameters: {'n_estimators': 82, 'learning_rate': 0.013128405051862526, 'num_leaves': 131, 'max_depth': 11, 'min_child_samples': 2, 'subsample': 0.6628983622606887, 'colsample_bytree': 0.5062168193165535, 'reg_alpha': 5.253673056795196, 'reg_lambda': 2.1595780208730355, 'min_split_gain': 0.03638742628032835}. Best is trial 0 with value: 3.2985079747757955.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         484
1           Price_lag_2         287
14      pct_chg_Load_DA         276
3         Load_DA_lag_0         183
11           NGas_lag_2         148
16  roll2d_median_Price         135
2           Price_lag_7         122
5       WindOn_DA_lag_0          88
13            EUA_lag_2          81
15       lag168_Load_DA          79
10           Coal_lag_2          68
12            Oil_lag_2          60
9                  WD_7          16
8                  WD_6          11
7                  WD_1           4
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
15       lag168_Load_DA          84
0           Price_lag_1          80
3         Load_DA_lag_0          65
14      pct_chg_Load_DA          37
1           Price_lag_2          32
10           Coal_lag_2          26
5       WindOn_DA_lag_0          21
16  roll2d_median_Price          17
2           Price_lag_7          16
12            Oil_lag_2           7
11           NGas_lag_2           5
13            EUA_lag_2           1
7                  WD_1           0
9                  WD_7           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
8                  WD_6           0
-> 97.54% done
********************* END NS.. 712 ****************************************************
********************* START NS ... 713 ****************************************************
[I 2025-06-15 20:51:01,751] A new study created in memory with name: no-name-7d98afa4-72bc-4ea2-b6b7-e5216feea124
[I 2025-06-15 20:51:03,603] Trial 0 finished with value: 0.2880870356574537 and parameters: {'n_estimators': 248, 'learning_rate': 0.025191627379760672, 'num_leaves': 38, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.9500068022556145, 'colsample_bytree': 0.9928148918222601, 'reg_alpha': 0.11355524597739364, 'reg_lambda': 6.549841163514181, 'min_split_gain': 0.041773747286513445}. Best is trial 0 with value: 0.2880870356574537.
[Trial 0] New best: 0.28809
[I 2025-06-15 20:51:03,977] Trial 1 finished with value: 0.32598449545847247 and parameters: {'n_estimators': 269, 'learning_rate': 0.2997832457491496, 'num_leaves': 107, 'max_depth': 16, 'min_child_samples': 12, 'subsample': 0.5164025255127761, 'colsample_bytree': 0.7349418943522716, 'reg_alpha': 7.915289403470286, 'reg_lambda': 6.466648040374073, 'min_split_gain': 0.03684393042463296}. Best is trial 0 with value: 0.2880870356574537.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:51:04,502] Trial 2 finished with value: 0.2839325410345892 and parameters: {'n_estimators': 141, 'learning_rate': 0.15495689033789625, 'num_leaves': 46, 'max_depth': 9, 'min_child_samples': 21, 'subsample': 0.9236500645663388, 'colsample_bytree': 0.9828472451741332, 'reg_alpha': 0.21166719221087482, 'reg_lambda': 5.596411899855517, 'min_split_gain': 0.02985091447280415}. Best is trial 2 with value: 0.2839325410345892.
[Trial 2] New best: 0.28393
[I 2025-06-15 20:51:05,794] Trial 3 finished with value: 0.3478375949048634 and parameters: {'n_estimators': 263, 'learning_rate': 0.010760059590066944, 'num_leaves': 192, 'max_depth': 5, 'min_child_samples': 12, 'subsample': 0.684104764930713, 'colsample_bytree': 0.9584257609265197, 'reg_alpha': 1.7066651472020333, 'reg_lambda': 2.1667277038059307, 'min_split_gain': 0.03292611080380164}. Best is trial 2 with value: 0.2839325410345892.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:51:07,407] Trial 4 finished with value: 0.331926265307025 and parameters: {'n_estimators': 222, 'learning_rate': 0.06909777869826463, 'num_leaves': 142, 'max_depth': 14, 'min_child_samples': 13, 'subsample': 0.5558531199052668, 'colsample_bytree': 0.6467049010464476, 'reg_alpha': 3.2165066831844067, 'reg_lambda': 0.5236143733776755, 'min_split_gain': 0.029478763462913917}. Best is trial 2 with value: 0.2839325410345892.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:51:09,071] Trial 5 finished with value: 0.49027210298676865 and parameters: {'n_estimators': 134, 'learning_rate': 0.027288333487450325, 'num_leaves': 103, 'max_depth': 13, 'min_child_samples': 4, 'subsample': 0.5151143145114938, 'colsample_bytree': 0.5484406102991418, 'reg_alpha': 5.04051450777093, 'reg_lambda': 4.637454744878955, 'min_split_gain': 0.004663568880702978}. Best is trial 2 with value: 0.2839325410345892.
[I 2025-06-15 20:51:09,074] A new study created in memory with name: no-name-b07af19e-6fd8-468e-bffd-19cb5e744479
[I 2025-06-15 20:51:09,123] Trial 0 finished with value: 1.2388936385557132 and parameters: {'n_estimators': 76, 'learning_rate': 0.06803328633166285, 'num_leaves': 183, 'max_depth': 12, 'min_child_samples': 28, 'subsample': 0.9857348971458946, 'colsample_bytree': 0.960863326791723, 'reg_alpha': 3.5985163779269733, 'reg_lambda': 6.905394627012597, 'min_split_gain': 0.038582091387800665}. Best is trial 0 with value: 1.2388936385557132.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         492
14      pct_chg_Load_DA         315
1           Price_lag_2         246
3         Load_DA_lag_0         207
16  roll2d_median_Price         191
5       WindOn_DA_lag_0         142
15       lag168_Load_DA         128
2           Price_lag_7         111
11           NGas_lag_2         104
13            EUA_lag_2          85
10           Coal_lag_2          78
12            Oil_lag_2          58
9                  WD_7          10
7                  WD_1           3
8                  WD_6           3
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
15       lag168_Load_DA          40
3         Load_DA_lag_0          38
0           Price_lag_1          34
10           Coal_lag_2          11
5       WindOn_DA_lag_0           9
11           NGas_lag_2           8
14      pct_chg_Load_DA           5
16  roll2d_median_Price           1
1           Price_lag_2           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
12            Oil_lag_2           0
13            EUA_lag_2           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
8                  WD_6           0
-> 97.67% done
********************* END NS.. 713 ****************************************************
********************* START NS ... 714 ****************************************************
[I 2025-06-15 20:51:11,026] A new study created in memory with name: no-name-b8769289-1a82-4a91-8045-710885e7f9b1
[I 2025-06-15 20:51:12,919] Trial 0 finished with value: 0.02728623992464596 and parameters: {'n_estimators': 214, 'learning_rate': 0.11973895813531023, 'num_leaves': 125, 'max_depth': 13, 'min_child_samples': 8, 'subsample': 0.5238147715767993, 'colsample_bytree': 0.5496938467335317, 'reg_alpha': 1.243487614437666, 'reg_lambda': 7.066857387724118, 'min_split_gain': 0.015405799821310762}. Best is trial 0 with value: 0.02728623992464596.
[Trial 0] New best: 0.02729
[I 2025-06-15 20:51:13,778] Trial 1 finished with value: 0.02455516176798303 and parameters: {'n_estimators': 157, 'learning_rate': 0.11213375585897181, 'num_leaves': 164, 'max_depth': 11, 'min_child_samples': 22, 'subsample': 0.8339313350277138, 'colsample_bytree': 0.8389824444854148, 'reg_alpha': 8.03490669902475, 'reg_lambda': 8.748528027106344, 'min_split_gain': 0.0033000092925560666}. Best is trial 1 with value: 0.02455516176798303.
[Trial 1] New best: 0.02456
[I 2025-06-15 20:51:16,680] Trial 2 finished with value: 0.027396103764795555 and parameters: {'n_estimators': 117, 'learning_rate': 0.032456193365759736, 'num_leaves': 237, 'max_depth': 13, 'min_child_samples': 1, 'subsample': 0.9139782306113247, 'colsample_bytree': 0.6116694374065176, 'reg_alpha': 2.0005298561548535, 'reg_lambda': 8.288230512507408, 'min_split_gain': 0.007863930976281163}. Best is trial 1 with value: 0.02455516176798303.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:51:17,662] Trial 3 finished with value: 0.024387588655739034 and parameters: {'n_estimators': 154, 'learning_rate': 0.05626001600558971, 'num_leaves': 79, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.5180836433844418, 'colsample_bytree': 0.6491405917283266, 'reg_alpha': 5.384039187880317, 'reg_lambda': 6.920594217254364, 'min_split_gain': 0.01792844556959347}. Best is trial 3 with value: 0.024387588655739034.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:51:17,939] Trial 4 finished with value: 0.03875098045802273 and parameters: {'n_estimators': 90, 'learning_rate': 0.06269580122731097, 'num_leaves': 180, 'max_depth': 4, 'min_child_samples': 7, 'subsample': 0.6012833193472606, 'colsample_bytree': 0.5525609608480371, 'reg_alpha': 9.994651225479082, 'reg_lambda': 6.024295518679171, 'min_split_gain': 0.015013805511194123}. Best is trial 3 with value: 0.024387588655739034.
[I 2025-06-15 20:51:17,941] A new study created in memory with name: no-name-49a6a63f-4671-4185-b5a6-b06a89fa3c3c
[I 2025-06-15 20:51:18,003] Trial 0 finished with value: 0.14405851843909778 and parameters: {'n_estimators': 182, 'learning_rate': 0.1781367712521045, 'num_leaves': 234, 'max_depth': 12, 'min_child_samples': 24, 'subsample': 0.6155522731658604, 'colsample_bytree': 0.7491713852300991, 'reg_alpha': 2.481204763690683, 'reg_lambda': 9.225878135007164, 'min_split_gain': 0.036742678151093396}. Best is trial 0 with value: 0.14405851843909778.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         882
1           Price_lag_2         666
14      pct_chg_Load_DA         541
3         Load_DA_lag_0         366
16  roll2d_median_Price         270
11           NGas_lag_2         262
2           Price_lag_7         229
5       WindOn_DA_lag_0         221
13            EUA_lag_2         212
15       lag168_Load_DA         189
12            Oil_lag_2         140
10           Coal_lag_2         113
9                  WD_7          28
8                  WD_6          27
7                  WD_1          12
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
3         Load_DA_lag_0          27
0           Price_lag_1          19
15       lag168_Load_DA          16
5       WindOn_DA_lag_0          13
16  roll2d_median_Price           9
12            Oil_lag_2           7
10           Coal_lag_2           5
14      pct_chg_Load_DA           4
11           NGas_lag_2           2
1           Price_lag_2           1
13            EUA_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
2           Price_lag_7           0
8                  WD_6           0
-> 97.81% done
********************* END NS.. 714 ****************************************************
********************* START NS ... 715 ****************************************************
[I 2025-06-15 20:51:20,145] A new study created in memory with name: no-name-a387c440-7546-4851-84ad-9c2d3bc65d5b
[I 2025-06-15 20:51:20,729] Trial 0 finished with value: 0.030506505149698604 and parameters: {'n_estimators': 97, 'learning_rate': 0.013913251007092849, 'num_leaves': 145, 'max_depth': 14, 'min_child_samples': 24, 'subsample': 0.8165972224965603, 'colsample_bytree': 0.9566965081779004, 'reg_alpha': 9.775359724606108, 'reg_lambda': 4.285833804133138, 'min_split_gain': 0.03801658587091116}. Best is trial 0 with value: 0.030506505149698604.
[Trial 0] New best: 0.03051
[I 2025-06-15 20:51:21,756] Trial 1 finished with value: 0.004957024157288102 and parameters: {'n_estimators': 137, 'learning_rate': 0.02592002688374197, 'num_leaves': 255, 'max_depth': 11, 'min_child_samples': 21, 'subsample': 0.6128836500989774, 'colsample_bytree': 0.6419943693511561, 'reg_alpha': 4.7459879393292725, 'reg_lambda': 9.018845478966263, 'min_split_gain': 0.0370808619969369}. Best is trial 1 with value: 0.004957024157288102.
[Trial 1] New best: 0.00496
[I 2025-06-15 20:51:22,915] Trial 2 finished with value: 0.0055047070028752644 and parameters: {'n_estimators': 123, 'learning_rate': 0.020724687588135716, 'num_leaves': 252, 'max_depth': 7, 'min_child_samples': 17, 'subsample': 0.5862663493232737, 'colsample_bytree': 0.9388976185403637, 'reg_alpha': 2.478393394034184, 'reg_lambda': 0.683745395357297, 'min_split_gain': 0.032464886385458885}. Best is trial 1 with value: 0.004957024157288102.
[I 2025-06-15 20:51:23,099] Trial 3 finished with value: 0.001415655623471849 and parameters: {'n_estimators': 59, 'learning_rate': 0.10603739380777319, 'num_leaves': 133, 'max_depth': 4, 'min_child_samples': 18, 'subsample': 0.6669232088793228, 'colsample_bytree': 0.7131268250020805, 'reg_alpha': 6.0693833555038434, 'reg_lambda': 4.29273011872597, 'min_split_gain': 0.026825751423888988}. Best is trial 3 with value: 0.001415655623471849.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[Trial 3] New best: 0.00142
[I 2025-06-15 20:51:23,957] Trial 4 finished with value: 0.0017188506662073333 and parameters: {'n_estimators': 203, 'learning_rate': 0.06594714521262236, 'num_leaves': 137, 'max_depth': 12, 'min_child_samples': 30, 'subsample': 0.7361797106155894, 'colsample_bytree': 0.8973200261501189, 'reg_alpha': 3.036183545584649, 'reg_lambda': 6.790886566011764, 'min_split_gain': 0.031966967756272203}. Best is trial 3 with value: 0.001415655623471849.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:51:24,507] Trial 5 finished with value: 0.0025219043387762177 and parameters: {'n_estimators': 270, 'learning_rate': 0.1250030748554179, 'num_leaves': 177, 'max_depth': 13, 'min_child_samples': 11, 'subsample': 0.831694033839619, 'colsample_bytree': 0.6661968146222287, 'reg_alpha': 9.977125438600558, 'reg_lambda': 3.4198305357428795, 'min_split_gain': 0.04016327304059029}. Best is trial 3 with value: 0.001415655623471849.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:51:25,568] Trial 6 finished with value: 0.0038210898640755942 and parameters: {'n_estimators': 219, 'learning_rate': 0.01312997669378222, 'num_leaves': 101, 'max_depth': 6, 'min_child_samples': 23, 'subsample': 0.8772965327046247, 'colsample_bytree': 0.9921724395891285, 'reg_alpha': 8.206734193653269, 'reg_lambda': 3.443736594081226, 'min_split_gain': 0.04251281576029554}. Best is trial 3 with value: 0.001415655623471849.
[I 2025-06-15 20:51:25,571] A new study created in memory with name: no-name-80a551d2-5af6-4197-8a12-ff3ebdfb5caa
[I 2025-06-15 20:51:25,650] Trial 0 finished with value: 0.1457657362632028 and parameters: {'n_estimators': 270, 'learning_rate': 0.08606884154419335, 'num_leaves': 168, 'max_depth': 8, 'min_child_samples': 17, 'subsample': 0.7057988312625204, 'colsample_bytree': 0.8024827734396236, 'reg_alpha': 6.506307155701231, 'reg_lambda': 1.4525265160505396, 'min_split_gain': 0.03628990001746168}. Best is trial 0 with value: 0.1457657362632028.
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         268
14      pct_chg_Load_DA         123
1           Price_lag_2          76
3         Load_DA_lag_0          68
16  roll2d_median_Price          43
11           NGas_lag_2          34
2           Price_lag_7          33
13            EUA_lag_2          26
10           Coal_lag_2          24
5       WindOn_DA_lag_0          18
12            Oil_lag_2          16
15       lag168_Load_DA          16
9                  WD_7           5
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1          41
3         Load_DA_lag_0          32
15       lag168_Load_DA          31
5       WindOn_DA_lag_0          11
16  roll2d_median_Price           8
12            Oil_lag_2           5
11           NGas_lag_2           4
13            EUA_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
1           Price_lag_2           0
9                  WD_7           0
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
2           Price_lag_7           0
8                  WD_6           0
-> 97.95% done
********************* END NS.. 715 ****************************************************
********************* START NS ... 716 ****************************************************
[I 2025-06-15 20:51:27,206] A new study created in memory with name: no-name-d0207750-b255-4236-9b39-09127abf66d0
[I 2025-06-15 20:51:29,048] Trial 0 finished with value: 0.006624435990643096 and parameters: {'n_estimators': 216, 'learning_rate': 0.015863352267840587, 'num_leaves': 237, 'max_depth': 10, 'min_child_samples': 7, 'subsample': 0.6722256767377901, 'colsample_bytree': 0.735353891355538, 'reg_alpha': 5.459229123650216, 'reg_lambda': 0.6890478912897846, 'min_split_gain': 0.04221817775944698}. Best is trial 0 with value: 0.006624435990643096.
[Trial 0] New best: 0.00662
[I 2025-06-15 20:51:30,158] Trial 1 finished with value: 0.027418939268351428 and parameters: {'n_estimators': 99, 'learning_rate': 0.018038940729390857, 'num_leaves': 67, 'max_depth': 15, 'min_child_samples': 19, 'subsample': 0.6635357197356, 'colsample_bytree': 0.5309791240832837, 'reg_alpha': 3.7549079368047353, 'reg_lambda': 2.0325920006226337, 'min_split_gain': 0.043051047005425024}. Best is trial 0 with value: 0.006624435990643096.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:51:31,211] Trial 2 finished with value: 0.00681928589280275 and parameters: {'n_estimators': 131, 'learning_rate': 0.032040413386152995, 'num_leaves': 53, 'max_depth': 16, 'min_child_samples': 19, 'subsample': 0.8558570008758453, 'colsample_bytree': 0.628935131511424, 'reg_alpha': 5.002987105190075, 'reg_lambda': 1.6712231202139038, 'min_split_gain': 0.04499092257287535}. Best is trial 0 with value: 0.006624435990643096.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:51:31,931] Trial 3 finished with value: 0.0020784551810075773 and parameters: {'n_estimators': 190, 'learning_rate': 0.02021247806086841, 'num_leaves': 122, 'max_depth': 4, 'min_child_samples': 1, 'subsample': 0.8231057331769928, 'colsample_bytree': 0.8083869959400242, 'reg_alpha': 9.37906133915058, 'reg_lambda': 7.958829538167472, 'min_split_gain': 0.01368272691174265}. Best is trial 3 with value: 0.0020784551810075773.
[Trial 3] New best: 0.00208
[I 2025-06-15 20:51:33,022] Trial 4 finished with value: 0.003403557729403648 and parameters: {'n_estimators': 241, 'learning_rate': 0.12612688261735475, 'num_leaves': 151, 'max_depth': 13, 'min_child_samples': 5, 'subsample': 0.6197069147582315, 'colsample_bytree': 0.8448818721969484, 'reg_alpha': 7.636287492715784, 'reg_lambda': 4.311054176602322, 'min_split_gain': 0.025549831948323876}. Best is trial 3 with value: 0.0020784551810075773.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:51:34,146] Trial 5 finished with value: 0.005132680862236605 and parameters: {'n_estimators': 96, 'learning_rate': 0.03381952541811541, 'num_leaves': 146, 'max_depth': 13, 'min_child_samples': 5, 'subsample': 0.9622660863453023, 'colsample_bytree': 0.8675113018213022, 'reg_alpha': 7.6312546450921666, 'reg_lambda': 5.25875359746292, 'min_split_gain': 0.024411236385743718}. Best is trial 3 with value: 0.0020784551810075773.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:51:34,725] Trial 6 finished with value: 0.002075559157568018 and parameters: {'n_estimators': 136, 'learning_rate': 0.17955759992838932, 'num_leaves': 104, 'max_depth': 15, 'min_child_samples': 25, 'subsample': 0.67564911790479, 'colsample_bytree': 0.6435390761216849, 'reg_alpha': 0.2730742288264365, 'reg_lambda': 4.420128891425162, 'min_split_gain': 0.039347636612893366}. Best is trial 6 with value: 0.002075559157568018.
[I 2025-06-15 20:51:34,727] A new study created in memory with name: no-name-91a0150c-3a5c-43d0-9895-85eb6b45ee28
[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[I 2025-06-15 20:51:35,012] Trial 0 finished with value: 0.6280087479043918 and parameters: {'n_estimators': 164, 'learning_rate': 0.015019887138047947, 'num_leaves': 126, 'max_depth': 4, 'min_child_samples': 3, 'subsample': 0.5231717850111481, 'colsample_bytree': 0.6097037309051716, 'reg_alpha': 0.17104539245371808, 'reg_lambda': 1.4750668272721001, 'min_split_gain': 0.029854889699020528}. Best is trial 0 with value: 0.6280087479043918.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         520
1           Price_lag_2         407
14      pct_chg_Load_DA         318
3         Load_DA_lag_0         257
16  roll2d_median_Price         205
15       lag168_Load_DA         167
2           Price_lag_7         161
5       WindOn_DA_lag_0         143
11           NGas_lag_2         137
10           Coal_lag_2         122
13            EUA_lag_2         109
12            Oil_lag_2          58
9                  WD_7           9
8                  WD_6           3
7                  WD_1           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         356
3         Load_DA_lag_0         234
15       lag168_Load_DA         181
1           Price_lag_2         181
5       WindOn_DA_lag_0         149
16  roll2d_median_Price         118
2           Price_lag_7         114
10           Coal_lag_2          48
14      pct_chg_Load_DA          33
11           NGas_lag_2          14
12            Oil_lag_2          14
13            EUA_lag_2           8
7                  WD_1           1
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
9                  WD_7           0
8                  WD_6           0
-> 98.08% done
********************* END NS.. 716 ****************************************************
********************* START NS ... 717 ****************************************************
[I 2025-06-15 20:51:37,189] A new study created in memory with name: no-name-4c9378c8-0792-45a7-8ec8-354325b60022
[I 2025-06-15 20:51:37,769] Trial 0 finished with value: 0.005745935927960206 and parameters: {'n_estimators': 123, 'learning_rate': 0.13905382622451828, 'num_leaves': 244, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.8405763260955404, 'colsample_bytree': 0.989386915140211, 'reg_alpha': 2.213498699359624, 'reg_lambda': 2.860795632126071, 'min_split_gain': 0.01986504621377466}. Best is trial 0 with value: 0.005745935927960206.
[Trial 0] New best: 0.00575
[I 2025-06-15 20:51:38,151] Trial 1 finished with value: 0.1319803559301369 and parameters: {'n_estimators': 52, 'learning_rate': 0.011744631962816209, 'num_leaves': 198, 'max_depth': 5, 'min_child_samples': 17, 'subsample': 0.8648868393073721, 'colsample_bytree': 0.8082064405537956, 'reg_alpha': 9.562843733462827, 'reg_lambda': 9.408394253789659, 'min_split_gain': 0.011396199178317535}. Best is trial 0 with value: 0.005745935927960206.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:51:39,221] Trial 2 finished with value: 0.004541376814932243 and parameters: {'n_estimators': 237, 'learning_rate': 0.05958676635273189, 'num_leaves': 131, 'max_depth': 5, 'min_child_samples': 12, 'subsample': 0.9000807374871163, 'colsample_bytree': 0.5075399758003862, 'reg_alpha': 1.8797313348456834, 'reg_lambda': 6.170838040759516, 'min_split_gain': 0.01220890343399605}. Best is trial 2 with value: 0.004541376814932243.
[Trial 2] New best: 0.00454
[I 2025-06-15 20:51:40,554] Trial 3 finished with value: 0.014747655999215027 and parameters: {'n_estimators': 169, 'learning_rate': 0.011959808065381477, 'num_leaves': 201, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.5177928884007195, 'colsample_bytree': 0.646556885980988, 'reg_alpha': 5.17599087749921, 'reg_lambda': 1.401938684249503, 'min_split_gain': 0.032216105776255756}. Best is trial 2 with value: 0.004541376814932243.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:51:41,693] Trial 4 finished with value: 0.005367283779244375 and parameters: {'n_estimators': 117, 'learning_rate': 0.05452539591226577, 'num_leaves': 233, 'max_depth': 7, 'min_child_samples': 1, 'subsample': 0.7475879464405579, 'colsample_bytree': 0.8216817918888903, 'reg_alpha': 6.5248590281238865, 'reg_lambda': 0.07408043981677159, 'min_split_gain': 0.032633330936718545}. Best is trial 2 with value: 0.004541376814932243.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:51:42,232] Trial 5 finished with value: 0.004274319339948697 and parameters: {'n_estimators': 177, 'learning_rate': 0.028458109055349998, 'num_leaves': 57, 'max_depth': 4, 'min_child_samples': 18, 'subsample': 0.5939072804409886, 'colsample_bytree': 0.7292912927416513, 'reg_alpha': 4.660698666387763, 'reg_lambda': 3.266965297922304, 'min_split_gain': 0.02455609095899061}. Best is trial 5 with value: 0.004274319339948697.
[I 2025-06-15 20:51:42,235] A new study created in memory with name: no-name-cb9e7fe9-e84b-4b6b-88a1-3b06f04499f6
[I 2025-06-15 20:51:42,341] Trial 0 finished with value: 0.9729498687454319 and parameters: {'n_estimators': 151, 'learning_rate': 0.025294788725527734, 'num_leaves': 96, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.8841980642439846, 'colsample_bytree': 0.6906762851426403, 'reg_alpha': 7.8171734001092315, 'reg_lambda': 8.69099205640117, 'min_split_gain': 0.04273712056429148}. Best is trial 0 with value: 0.9729498687454319.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         994
1           Price_lag_2         398
14      pct_chg_Load_DA         381
3         Load_DA_lag_0         153
16  roll2d_median_Price         121
2           Price_lag_7          84
11           NGas_lag_2          74
13            EUA_lag_2          71
12            Oil_lag_2          36
10           Coal_lag_2          32
5       WindOn_DA_lag_0          27
15       lag168_Load_DA          26
9                  WD_7          13
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
8                  WD_6           0
short model ************************
                feature  importance
0           Price_lag_1         134
3         Load_DA_lag_0          81
5       WindOn_DA_lag_0          27
1           Price_lag_2          24
16  roll2d_median_Price          11
11           NGas_lag_2          10
15       lag168_Load_DA          10
10           Coal_lag_2           7
12            Oil_lag_2           4
13            EUA_lag_2           4
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
2           Price_lag_7           0
9                  WD_7           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 98.22% done
********************* END NS.. 717 ****************************************************
********************* START NS ... 718 ****************************************************
[I 2025-06-15 20:51:44,056] A new study created in memory with name: no-name-a161b1fe-542f-4359-a83e-4d0ada37fe31
[I 2025-06-15 20:51:47,311] Trial 0 finished with value: 0.0067836563389973075 and parameters: {'n_estimators': 258, 'learning_rate': 0.01003026655527916, 'num_leaves': 146, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.607812304847282, 'colsample_bytree': 0.8236457629598487, 'reg_alpha': 1.023113779336956, 'reg_lambda': 8.749865355473611, 'min_split_gain': 0.03679034072003361}. Best is trial 0 with value: 0.0067836563389973075.
[Trial 0] New best: 0.00678
[I 2025-06-15 20:51:47,913] Trial 1 finished with value: 0.012011923932563756 and parameters: {'n_estimators': 89, 'learning_rate': 0.02212616958735405, 'num_leaves': 128, 'max_depth': 11, 'min_child_samples': 27, 'subsample': 0.6721319162122588, 'colsample_bytree': 0.7770741180951877, 'reg_alpha': 6.853390071292659, 'reg_lambda': 8.66430594454046, 'min_split_gain': 0.017040600838404623}. Best is trial 0 with value: 0.0067836563389973075.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:51:49,765] Trial 2 finished with value: 0.004809164981378137 and parameters: {'n_estimators': 211, 'learning_rate': 0.12702483585191945, 'num_leaves': 251, 'max_depth': 8, 'min_child_samples': 8, 'subsample': 0.8726932491336838, 'colsample_bytree': 0.5826895192268664, 'reg_alpha': 4.673629652045387, 'reg_lambda': 9.452623777015798, 'min_split_gain': 0.0003211424382190298}. Best is trial 2 with value: 0.004809164981378137.
[Trial 2] New best: 0.00481
[I 2025-06-15 20:51:51,327] Trial 3 finished with value: 0.005229860556384638 and parameters: {'n_estimators': 295, 'learning_rate': 0.011085322246589974, 'num_leaves': 150, 'max_depth': 5, 'min_child_samples': 4, 'subsample': 0.7805710706926496, 'colsample_bytree': 0.6164960399481202, 'reg_alpha': 4.7692718044056654, 'reg_lambda': 1.640255191997415, 'min_split_gain': 0.006372853612422242}. Best is trial 2 with value: 0.004809164981378137.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:51:54,009] Trial 4 finished with value: 0.004279445085188619 and parameters: {'n_estimators': 229, 'learning_rate': 0.025714219108960507, 'num_leaves': 215, 'max_depth': 16, 'min_child_samples': 1, 'subsample': 0.5630887808876894, 'colsample_bytree': 0.9232739034581176, 'reg_alpha': 6.772099713110236, 'reg_lambda': 8.409575919881497, 'min_split_gain': 0.015589671368168419}. Best is trial 4 with value: 0.004279445085188619.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:51:54,385] Trial 5 finished with value: 0.004349518454498475 and parameters: {'n_estimators': 175, 'learning_rate': 0.20748616475827056, 'num_leaves': 187, 'max_depth': 7, 'min_child_samples': 7, 'subsample': 0.7551352959772857, 'colsample_bytree': 0.8610211515554553, 'reg_alpha': 2.7418758810117105, 'reg_lambda': 0.39817612230503396, 'min_split_gain': 0.031597364748048105}. Best is trial 4 with value: 0.004279445085188619.
[I 2025-06-15 20:51:54,388] A new study created in memory with name: no-name-291a01bf-0b30-491d-83e0-f68c35a5f7f3
[I 2025-06-15 20:51:54,481] Trial 0 finished with value: 0.08893584918660317 and parameters: {'n_estimators': 169, 'learning_rate': 0.018146335562343126, 'num_leaves': 133, 'max_depth': 13, 'min_child_samples': 21, 'subsample': 0.5250335555417079, 'colsample_bytree': 0.9600239019695262, 'reg_alpha': 3.604548614027181, 'reg_lambda': 3.447604791644313, 'min_split_gain': 0.04313572864175497}. Best is trial 0 with value: 0.08893584918660317.
[Trial 5] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        2006
1           Price_lag_2        1506
14      pct_chg_Load_DA        1258
2           Price_lag_7         771
11           NGas_lag_2         769
3         Load_DA_lag_0         711
16  roll2d_median_Price         654
13            EUA_lag_2         604
10           Coal_lag_2         527
5       WindOn_DA_lag_0         469
12            Oil_lag_2         424
15       lag168_Load_DA         320
8                  WD_6          59
9                  WD_7          57
7                  WD_1           0
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         114
3         Load_DA_lag_0          54
5       WindOn_DA_lag_0          45
15       lag168_Load_DA          24
16  roll2d_median_Price          19
10           Coal_lag_2          12
1           Price_lag_2           7
2           Price_lag_7           7
11           NGas_lag_2           1
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 98.36% done
********************* END NS.. 718 ****************************************************
********************* START NS ... 719 ****************************************************
[I 2025-06-15 20:51:57,857] A new study created in memory with name: no-name-d85150b7-e537-4651-a93e-a382b8e34fb1
[I 2025-06-15 20:51:58,356] Trial 0 finished with value: 0.001124044065686638 and parameters: {'n_estimators': 166, 'learning_rate': 0.09938928390428167, 'num_leaves': 215, 'max_depth': 9, 'min_child_samples': 15, 'subsample': 0.8254895250945751, 'colsample_bytree': 0.6306947507318497, 'reg_alpha': 8.524477134245705, 'reg_lambda': 2.233183072911168, 'min_split_gain': 0.03718528408218718}. Best is trial 0 with value: 0.001124044065686638.
[Trial 0] New best: 0.00112
[I 2025-06-15 20:51:59,471] Trial 1 finished with value: 0.0025888374812344747 and parameters: {'n_estimators': 188, 'learning_rate': 0.01613523282025966, 'num_leaves': 56, 'max_depth': 10, 'min_child_samples': 20, 'subsample': 0.7873998435698266, 'colsample_bytree': 0.9021323861473187, 'reg_alpha': 9.1675150676271, 'reg_lambda': 2.0991485987693057, 'min_split_gain': 0.042651400632806494}. Best is trial 0 with value: 0.001124044065686638.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:52:00,358] Trial 2 finished with value: 0.0007963853266750726 and parameters: {'n_estimators': 257, 'learning_rate': 0.0901166496138234, 'num_leaves': 224, 'max_depth': 11, 'min_child_samples': 6, 'subsample': 0.9113457551371258, 'colsample_bytree': 0.8603991841688141, 'reg_alpha': 1.8530569265075458, 'reg_lambda': 6.0902320450166725, 'min_split_gain': 0.04115306125389047}. Best is trial 2 with value: 0.0007963853266750726.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:52:01,401] Trial 3 finished with value: 0.0012421960266850052 and parameters: {'n_estimators': 234, 'learning_rate': 0.11768519534657251, 'num_leaves': 137, 'max_depth': 9, 'min_child_samples': 10, 'subsample': 0.9017899349519884, 'colsample_bytree': 0.6145364429897742, 'reg_alpha': 4.911782569568616, 'reg_lambda': 0.6403341481673475, 'min_split_gain': 0.00872391738374595}. Best is trial 2 with value: 0.0007963853266750726.
[I 2025-06-15 20:52:01,403] A new study created in memory with name: no-name-1189a95e-e8fb-4b9e-933e-83c335a32c63
[I 2025-06-15 20:52:01,444] Trial 0 finished with value: 0.0030693606374071294 and parameters: {'n_estimators': 106, 'learning_rate': 0.20920400566822708, 'num_leaves': 124, 'max_depth': 16, 'min_child_samples': 28, 'subsample': 0.9635819828088918, 'colsample_bytree': 0.6632168197202681, 'reg_alpha': 1.0932304968734374, 'reg_lambda': 4.086419417307853, 'min_split_gain': 0.041279730380902}. Best is trial 0 with value: 0.0030693606374071294.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         715
14      pct_chg_Load_DA         579
1           Price_lag_2         567
3         Load_DA_lag_0         346
16  roll2d_median_Price         277
5       WindOn_DA_lag_0         261
2           Price_lag_7         229
11           NGas_lag_2         211
15       lag168_Load_DA         179
13            EUA_lag_2         144
10           Coal_lag_2         107
12            Oil_lag_2          94
9                  WD_7          12
8                  WD_6           4
7                  WD_1           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          10
16  roll2d_median_Price          10
3         Load_DA_lag_0           8
15       lag168_Load_DA           8
5       WindOn_DA_lag_0           3
1           Price_lag_2           1
2           Price_lag_7           0
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
9                  WD_7           0
10           Coal_lag_2           0
11           NGas_lag_2           0
12            Oil_lag_2           0
13            EUA_lag_2           0
14      pct_chg_Load_DA           0
8                  WD_6           0
-> 98.50% done
********************* END NS.. 719 ****************************************************
********************* START NS ... 720 ****************************************************
[I 2025-06-15 20:52:03,255] A new study created in memory with name: no-name-86c2f858-0bdf-4be5-bcee-c34dc9ee086f
[I 2025-06-15 20:52:03,750] Trial 0 finished with value: 0.0010320874863725838 and parameters: {'n_estimators': 201, 'learning_rate': 0.15253895575069681, 'num_leaves': 61, 'max_depth': 13, 'min_child_samples': 25, 'subsample': 0.7259544952042902, 'colsample_bytree': 0.6635864468908542, 'reg_alpha': 7.3921584559477616, 'reg_lambda': 3.3972829499248736, 'min_split_gain': 0.03050276117074173}. Best is trial 0 with value: 0.0010320874863725838.
[Trial 0] New best: 0.00103
[I 2025-06-15 20:52:04,026] Trial 1 finished with value: 0.001145632790240151 and parameters: {'n_estimators': 127, 'learning_rate': 0.19148288471578348, 'num_leaves': 221, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.5423485034814401, 'colsample_bytree': 0.8068953657361404, 'reg_alpha': 6.581370135112903, 'reg_lambda': 5.602039107660003, 'min_split_gain': 0.028316607396567207}. Best is trial 0 with value: 0.0010320874863725838.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:52:04,354] Trial 2 finished with value: 0.0009150960117929571 and parameters: {'n_estimators': 125, 'learning_rate': 0.16490650998053072, 'num_leaves': 196, 'max_depth': 5, 'min_child_samples': 16, 'subsample': 0.644659360470569, 'colsample_bytree': 0.8127811454001184, 'reg_alpha': 6.603367214152839, 'reg_lambda': 6.046769890974341, 'min_split_gain': 0.014362614094024584}. Best is trial 2 with value: 0.0009150960117929571.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:52:05,407] Trial 3 finished with value: 0.000772691650719267 and parameters: {'n_estimators': 221, 'learning_rate': 0.13354682038649213, 'num_leaves': 178, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.5092011044907452, 'colsample_bytree': 0.7086594848587998, 'reg_alpha': 1.4613536516991354, 'reg_lambda': 8.497642360416343, 'min_split_gain': 0.010433059958686947}. Best is trial 3 with value: 0.000772691650719267.
[I 2025-06-15 20:52:05,410] A new study created in memory with name: no-name-d185ba76-8fd4-48a2-932c-3afe39e2e5b6
[I 2025-06-15 20:52:05,452] Trial 0 finished with value: 0.00893697536957691 and parameters: {'n_estimators': 148, 'learning_rate': 0.025658661582316142, 'num_leaves': 248, 'max_depth': 6, 'min_child_samples': 21, 'subsample': 0.5128478206561087, 'colsample_bytree': 0.991667318501547, 'reg_alpha': 7.522685413042404, 'reg_lambda': 7.674530942644275, 'min_split_gain': 0.02064178507920697}. Best is trial 0 with value: 0.00893697536957691.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         701
1           Price_lag_2         628
14      pct_chg_Load_DA         524
3         Load_DA_lag_0         473
15       lag168_Load_DA         341
5       WindOn_DA_lag_0         325
13            EUA_lag_2         291
16  roll2d_median_Price         284
2           Price_lag_7         255
11           NGas_lag_2         228
10           Coal_lag_2         167
12            Oil_lag_2         147
9                  WD_7          21
8                  WD_6          12
7                  WD_1           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 98.63% done
********************* END NS.. 720 ****************************************************
********************* START NS ... 721 ****************************************************
[I 2025-06-15 20:52:08,168] A new study created in memory with name: no-name-75371ffc-fd05-464b-8038-6836122574da
[I 2025-06-15 20:52:08,417] Trial 0 finished with value: 0.0033970498776743807 and parameters: {'n_estimators': 95, 'learning_rate': 0.23955900341998615, 'num_leaves': 126, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.6429109813096017, 'colsample_bytree': 0.6981500859147158, 'reg_alpha': 6.909841182906614, 'reg_lambda': 1.0722473592330317, 'min_split_gain': 0.02083991908981759}. Best is trial 0 with value: 0.0033970498776743807.
[Trial 0] New best: 0.00340
[I 2025-06-15 20:52:10,186] Trial 1 finished with value: 0.0009220596376400746 and parameters: {'n_estimators': 168, 'learning_rate': 0.0366845535747948, 'num_leaves': 44, 'max_depth': 9, 'min_child_samples': 23, 'subsample': 0.948244551295935, 'colsample_bytree': 0.5071130257963443, 'reg_alpha': 3.0214464882714975, 'reg_lambda': 5.099215274786627, 'min_split_gain': 0.012975366228194063}. Best is trial 1 with value: 0.0009220596376400746.
[Trial 1] New best: 0.00092
[I 2025-06-15 20:52:11,088] Trial 2 finished with value: 0.0010296262266950526 and parameters: {'n_estimators': 211, 'learning_rate': 0.10550074109497334, 'num_leaves': 154, 'max_depth': 6, 'min_child_samples': 7, 'subsample': 0.6597034762730791, 'colsample_bytree': 0.6917201562528237, 'reg_alpha': 3.618360148356714, 'reg_lambda': 7.2519471418445285, 'min_split_gain': 0.010442119311567728}. Best is trial 1 with value: 0.0009220596376400746.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:52:11,455] Trial 3 finished with value: 0.0007340512508837599 and parameters: {'n_estimators': 211, 'learning_rate': 0.2437397032636462, 'num_leaves': 166, 'max_depth': 11, 'min_child_samples': 16, 'subsample': 0.885210759671309, 'colsample_bytree': 0.5594768664237191, 'reg_alpha': 8.569827531073416, 'reg_lambda': 5.812670197719924, 'min_split_gain': 0.04960202618955471}. Best is trial 3 with value: 0.0007340512508837599.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:52:12,500] Trial 4 finished with value: 0.0007667428335666684 and parameters: {'n_estimators': 123, 'learning_rate': 0.16363154438173627, 'num_leaves': 180, 'max_depth': 15, 'min_child_samples': 12, 'subsample': 0.5268369761779819, 'colsample_bytree': 0.970873170533584, 'reg_alpha': 1.4812757704532598, 'reg_lambda': 0.46385962826766214, 'min_split_gain': 0.01124251127497028}. Best is trial 3 with value: 0.0007340512508837599.
[I 2025-06-15 20:52:12,502] A new study created in memory with name: no-name-cc0c0f31-ed5a-4a34-8de5-26af22fb759a
[I 2025-06-15 20:52:12,567] Trial 0 finished with value: 0.005245492261770329 and parameters: {'n_estimators': 276, 'learning_rate': 0.012041452602403482, 'num_leaves': 77, 'max_depth': 15, 'min_child_samples': 3, 'subsample': 0.7420987555301874, 'colsample_bytree': 0.9962724459252343, 'reg_alpha': 5.289708116742096, 'reg_lambda': 6.386388929385558, 'min_split_gain': 0.01783801493090442}. Best is trial 0 with value: 0.005245492261770329.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         212
1           Price_lag_2         125
16  roll2d_median_Price         108
14      pct_chg_Load_DA          69
2           Price_lag_7          51
3         Load_DA_lag_0          50
5       WindOn_DA_lag_0          41
11           NGas_lag_2          39
13            EUA_lag_2          30
10           Coal_lag_2          28
12            Oil_lag_2          28
15       lag168_Load_DA          17
9                  WD_7          13
8                  WD_6          10
7                  WD_1           6
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          11
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 98.77% done
********************* END NS.. 721 ****************************************************
********************* START NS ... 722 ****************************************************
[I 2025-06-15 20:52:14,113] A new study created in memory with name: no-name-bbfffa59-7bd4-456e-943b-5f30c83a48f3
[I 2025-06-15 20:52:15,163] Trial 0 finished with value: 0.0010303850548217853 and parameters: {'n_estimators': 247, 'learning_rate': 0.1296907003717611, 'num_leaves': 78, 'max_depth': 14, 'min_child_samples': 24, 'subsample': 0.9478243412041569, 'colsample_bytree': 0.9556919219430837, 'reg_alpha': 0.917595113949865, 'reg_lambda': 4.475827095583331, 'min_split_gain': 0.006635608474558114}. Best is trial 0 with value: 0.0010303850548217853.
[Trial 0] New best: 0.00103
[I 2025-06-15 20:52:15,619] Trial 1 finished with value: 0.0012955122713960117 and parameters: {'n_estimators': 66, 'learning_rate': 0.23618658975704585, 'num_leaves': 117, 'max_depth': 5, 'min_child_samples': 18, 'subsample': 0.9375212492511468, 'colsample_bytree': 0.6182878219208556, 'reg_alpha': 0.04600938239153751, 'reg_lambda': 8.904569018989537, 'min_split_gain': 0.0019861069255629527}. Best is trial 0 with value: 0.0010303850548217853.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:52:18,294] Trial 2 finished with value: 0.02972048574917834 and parameters: {'n_estimators': 128, 'learning_rate': 0.011424071619369354, 'num_leaves': 105, 'max_depth': 15, 'min_child_samples': 1, 'subsample': 0.7856829700030938, 'colsample_bytree': 0.8502948321313082, 'reg_alpha': 1.928768607170127, 'reg_lambda': 0.8685213542953063, 'min_split_gain': 0.026301552090576538}. Best is trial 0 with value: 0.0010303850548217853.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:52:19,540] Trial 3 finished with value: 0.005367565602367484 and parameters: {'n_estimators': 197, 'learning_rate': 0.014365894140569957, 'num_leaves': 157, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.6449424607493073, 'colsample_bytree': 0.5267331607307423, 'reg_alpha': 8.828879293390568, 'reg_lambda': 3.632285994067023, 'min_split_gain': 0.046583532440900174}. Best is trial 0 with value: 0.0010303850548217853.
[I 2025-06-15 20:52:19,542] A new study created in memory with name: no-name-8f3ad2ca-88c9-471f-9803-b2f9e495c525
[I 2025-06-15 20:52:19,608] Trial 0 finished with value: 0.003987339422266991 and parameters: {'n_estimators': 211, 'learning_rate': 0.23821556173167593, 'num_leaves': 250, 'max_depth': 6, 'min_child_samples': 21, 'subsample': 0.5496311500889555, 'colsample_bytree': 0.6253993477543536, 'reg_alpha': 0.28857797557752085, 'reg_lambda': 8.325855378606418, 'min_split_gain': 0.028530335128530565}. Best is trial 0 with value: 0.003987339422266991.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         849
14      pct_chg_Load_DA         789
1           Price_lag_2         742
5       WindOn_DA_lag_0         532
3         Load_DA_lag_0         525
15       lag168_Load_DA         496
16  roll2d_median_Price         447
2           Price_lag_7         344
11           NGas_lag_2         306
13            EUA_lag_2         298
10           Coal_lag_2         210
12            Oil_lag_2         193
9                  WD_7          26
8                  WD_6          10
7                  WD_1           7
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          15
3         Load_DA_lag_0           5
2           Price_lag_7           2
15       lag168_Load_DA           2
1           Price_lag_2           2
10           Coal_lag_2           1
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
16  roll2d_median_Price           0
-> 98.91% done
********************* END NS.. 722 ****************************************************
********************* START NS ... 723 ****************************************************
[I 2025-06-15 20:52:22,107] A new study created in memory with name: no-name-fd2fcf92-d92b-4053-95a7-f6dcc4931eff
[I 2025-06-15 20:52:23,645] Trial 0 finished with value: 0.0029677019275621518 and parameters: {'n_estimators': 174, 'learning_rate': 0.030324221773118512, 'num_leaves': 192, 'max_depth': 9, 'min_child_samples': 11, 'subsample': 0.5060623451732835, 'colsample_bytree': 0.8100314838308507, 'reg_alpha': 6.684232038272362, 'reg_lambda': 8.42231113766992, 'min_split_gain': 0.017637294973611713}. Best is trial 0 with value: 0.0029677019275621518.
[Trial 0] New best: 0.00297
[I 2025-06-15 20:52:24,880] Trial 1 finished with value: 0.0031112297183097833 and parameters: {'n_estimators': 265, 'learning_rate': 0.10668900641414353, 'num_leaves': 89, 'max_depth': 15, 'min_child_samples': 1, 'subsample': 0.7779437726827778, 'colsample_bytree': 0.5737186169653345, 'reg_alpha': 2.2604752283409493, 'reg_lambda': 8.131005808695736, 'min_split_gain': 0.036258984632258}. Best is trial 0 with value: 0.0029677019275621518.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:52:26,777] Trial 2 finished with value: 0.003404285852774889 and parameters: {'n_estimators': 210, 'learning_rate': 0.02021774354065626, 'num_leaves': 187, 'max_depth': 9, 'min_child_samples': 23, 'subsample': 0.5435223012398616, 'colsample_bytree': 0.6670831044207235, 'reg_alpha': 3.8053178346115333, 'reg_lambda': 0.07919568253209164, 'min_split_gain': 0.04463396992995552}. Best is trial 0 with value: 0.0029677019275621518.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:52:27,482] Trial 3 finished with value: 0.004008781016359486 and parameters: {'n_estimators': 140, 'learning_rate': 0.22754049088991538, 'num_leaves': 44, 'max_depth': 11, 'min_child_samples': 9, 'subsample': 0.8381455213370324, 'colsample_bytree': 0.5192606158707422, 'reg_alpha': 0.017105380811262005, 'reg_lambda': 7.605017835301434, 'min_split_gain': 0.03327868434261046}. Best is trial 0 with value: 0.0029677019275621518.
[I 2025-06-15 20:52:27,485] A new study created in memory with name: no-name-d6efd4eb-e68e-4c99-a595-d99fda4daaac
[I 2025-06-15 20:52:27,540] Trial 0 finished with value: 0.04645603240059288 and parameters: {'n_estimators': 170, 'learning_rate': 0.010519361412006855, 'num_leaves': 168, 'max_depth': 16, 'min_child_samples': 9, 'subsample': 0.7071053716641763, 'colsample_bytree': 0.5352888567225116, 'reg_alpha': 4.775527387341281, 'reg_lambda': 9.408551219892352, 'min_split_gain': 0.031041746653722863}. Best is trial 0 with value: 0.04645603240059288.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1489
1           Price_lag_2        1034
14      pct_chg_Load_DA         858
3         Load_DA_lag_0         398
13            EUA_lag_2         383
16  roll2d_median_Price         358
2           Price_lag_7         328
11           NGas_lag_2         327
12            Oil_lag_2         186
10           Coal_lag_2         178
5       WindOn_DA_lag_0         169
15       lag168_Load_DA         148
9                  WD_7          36
8                  WD_6          28
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1         106
3         Load_DA_lag_0          52
1           Price_lag_2          31
10           Coal_lag_2          11
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 99.04% done
********************* END NS.. 723 ****************************************************
********************* START NS ... 724 ****************************************************
[I 2025-06-15 20:52:30,272] A new study created in memory with name: no-name-45fa654f-08eb-4435-a3d8-a4e75db2df1a
[I 2025-06-15 20:52:31,296] Trial 0 finished with value: 0.0025762555232560617 and parameters: {'n_estimators': 220, 'learning_rate': 0.05610426170698857, 'num_leaves': 109, 'max_depth': 11, 'min_child_samples': 9, 'subsample': 0.7696255272497077, 'colsample_bytree': 0.8801472601331375, 'reg_alpha': 6.696524850492751, 'reg_lambda': 5.587116069920427, 'min_split_gain': 0.025008430748286327}. Best is trial 0 with value: 0.0025762555232560617.
[Trial 0] New best: 0.00258
[I 2025-06-15 20:52:32,242] Trial 1 finished with value: 0.002354940717385704 and parameters: {'n_estimators': 226, 'learning_rate': 0.08288051217444076, 'num_leaves': 249, 'max_depth': 11, 'min_child_samples': 14, 'subsample': 0.5627505750081605, 'colsample_bytree': 0.9710045447415847, 'reg_alpha': 1.4388469317694397, 'reg_lambda': 3.4026546616683317, 'min_split_gain': 0.03820687764896629}. Best is trial 1 with value: 0.002354940717385704.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:52:33,047] Trial 2 finished with value: 0.003512521169785645 and parameters: {'n_estimators': 167, 'learning_rate': 0.020107989955124313, 'num_leaves': 83, 'max_depth': 6, 'min_child_samples': 22, 'subsample': 0.6623643933935923, 'colsample_bytree': 0.7733855370839442, 'reg_alpha': 9.414046382477261, 'reg_lambda': 3.819269074998086, 'min_split_gain': 0.024754537177311516}. Best is trial 1 with value: 0.002354940717385704.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:52:33,368] Trial 3 finished with value: 0.0025139949097741903 and parameters: {'n_estimators': 101, 'learning_rate': 0.22863880112471802, 'num_leaves': 80, 'max_depth': 4, 'min_child_samples': 25, 'subsample': 0.5396585188897897, 'colsample_bytree': 0.5876932323125542, 'reg_alpha': 3.971129459978171, 'reg_lambda': 2.615911791947201, 'min_split_gain': 0.00024503852237250934}. Best is trial 1 with value: 0.002354940717385704.
[I 2025-06-15 20:52:33,369] A new study created in memory with name: no-name-65fbab36-2350-463e-8c3d-561b236433cf
[I 2025-06-15 20:52:33,409] Trial 0 finished with value: 0.045987109198493935 and parameters: {'n_estimators': 88, 'learning_rate': 0.028429005938632887, 'num_leaves': 103, 'max_depth': 13, 'min_child_samples': 18, 'subsample': 0.6972948461565782, 'colsample_bytree': 0.5265245405272091, 'reg_alpha': 4.5670829267153366, 'reg_lambda': 8.053126099975785, 'min_split_gain': 0.04413316272898804}. Best is trial 0 with value: 0.045987109198493935.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         885
14      pct_chg_Load_DA         701
1           Price_lag_2         599
3         Load_DA_lag_0         444
16  roll2d_median_Price         371
2           Price_lag_7         280
11           NGas_lag_2         251
5       WindOn_DA_lag_0         243
15       lag168_Load_DA         203
13            EUA_lag_2         167
10           Coal_lag_2         121
12            Oil_lag_2          96
9                  WD_7          15
7                  WD_1           2
8                  WD_6           2
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          16
3         Load_DA_lag_0           3
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 99.18% done
********************* END NS.. 724 ****************************************************
********************* START NS ... 725 ****************************************************
[I 2025-06-15 20:52:35,640] A new study created in memory with name: no-name-67e52862-e1f1-462e-8e6e-bc5202120ee4
[I 2025-06-15 20:52:36,383] Trial 0 finished with value: 0.0010565252682082423 and parameters: {'n_estimators': 128, 'learning_rate': 0.246314564251433, 'num_leaves': 155, 'max_depth': 16, 'min_child_samples': 17, 'subsample': 0.9843497644987993, 'colsample_bytree': 0.7987365251007349, 'reg_alpha': 2.166732350132899, 'reg_lambda': 4.007880343958444, 'min_split_gain': 0.003532525507506884}. Best is trial 0 with value: 0.0010565252682082423.
[Trial 0] New best: 0.00106
[I 2025-06-15 20:52:38,384] Trial 1 finished with value: 0.0021868619930955186 and parameters: {'n_estimators': 133, 'learning_rate': 0.023659718084961104, 'num_leaves': 74, 'max_depth': 11, 'min_child_samples': 4, 'subsample': 0.9584727697177263, 'colsample_bytree': 0.6995292834389375, 'reg_alpha': 3.097771256291212, 'reg_lambda': 6.909264641095118, 'min_split_gain': 0.0023608666753858255}. Best is trial 0 with value: 0.0010565252682082423.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:52:39,004] Trial 2 finished with value: 0.0008068399455770245 and parameters: {'n_estimators': 56, 'learning_rate': 0.18209038969280486, 'num_leaves': 219, 'max_depth': 6, 'min_child_samples': 20, 'subsample': 0.8071468994883899, 'colsample_bytree': 0.99838108651287, 'reg_alpha': 0.13280162966975317, 'reg_lambda': 5.863455808890955, 'min_split_gain': 0.015609528314777866}. Best is trial 2 with value: 0.0008068399455770245.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:52:40,004] Trial 3 finished with value: 0.0007808141142571988 and parameters: {'n_estimators': 123, 'learning_rate': 0.09295682244110932, 'num_leaves': 41, 'max_depth': 13, 'min_child_samples': 10, 'subsample': 0.5816515548589359, 'colsample_bytree': 0.998063024893227, 'reg_alpha': 5.678601804373509, 'reg_lambda': 3.016125364623389, 'min_split_gain': 0.007813221618062643}. Best is trial 3 with value: 0.0007808141142571988.
[I 2025-06-15 20:52:40,005] A new study created in memory with name: no-name-d8a99515-b636-4459-a68f-f07da22646bb
[I 2025-06-15 20:52:40,036] Trial 0 finished with value: 0.0033613386937267984 and parameters: {'n_estimators': 95, 'learning_rate': 0.059478145762106714, 'num_leaves': 36, 'max_depth': 9, 'min_child_samples': 13, 'subsample': 0.7128175384592034, 'colsample_bytree': 0.7028337681128507, 'reg_alpha': 7.354567335385461, 'reg_lambda': 1.1077575813149387, 'min_split_gain': 0.01776383236673721}. Best is trial 0 with value: 0.0033613386937267984.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         647
14      pct_chg_Load_DA         478
1           Price_lag_2         353
3         Load_DA_lag_0         348
5       WindOn_DA_lag_0         266
16  roll2d_median_Price         263
11           NGas_lag_2         230
2           Price_lag_7         228
13            EUA_lag_2         194
15       lag168_Load_DA         192
12            Oil_lag_2         165
10           Coal_lag_2         162
9                  WD_7          28
8                  WD_6          15
7                  WD_1          11
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 99.32% done
********************* END NS.. 725 ****************************************************
********************* START NS ... 726 ****************************************************
[I 2025-06-15 20:52:41,953] A new study created in memory with name: no-name-fac2d8c4-6d16-4958-8520-f6bd4eeeb498
[I 2025-06-15 20:52:43,264] Trial 0 finished with value: 0.0037303950891431144 and parameters: {'n_estimators': 182, 'learning_rate': 0.014116369206742026, 'num_leaves': 238, 'max_depth': 6, 'min_child_samples': 18, 'subsample': 0.5345707430924258, 'colsample_bytree': 0.9820579468359564, 'reg_alpha': 0.5526198117308978, 'reg_lambda': 3.2068996910922074, 'min_split_gain': 0.02001931151531416}. Best is trial 0 with value: 0.0037303950891431144.
[Trial 0] New best: 0.00373
[I 2025-06-15 20:52:45,305] Trial 1 finished with value: 0.001283628840747059 and parameters: {'n_estimators': 251, 'learning_rate': 0.04753215347073146, 'num_leaves': 204, 'max_depth': 10, 'min_child_samples': 28, 'subsample': 0.5579908213607018, 'colsample_bytree': 0.6647732403369615, 'reg_alpha': 6.49350766565283, 'reg_lambda': 5.509840837474619, 'min_split_gain': 0.004450266773611944}. Best is trial 1 with value: 0.001283628840747059.
[Trial 1] New best: 0.00128
[I 2025-06-15 20:52:48,534] Trial 2 finished with value: 0.001718844301731405 and parameters: {'n_estimators': 210, 'learning_rate': 0.01589139903355265, 'num_leaves': 151, 'max_depth': 15, 'min_child_samples': 4, 'subsample': 0.9982278694865048, 'colsample_bytree': 0.9514418164005733, 'reg_alpha': 1.8691319543242613, 'reg_lambda': 3.8144250459783704, 'min_split_gain': 0.02940447806451842}. Best is trial 1 with value: 0.001283628840747059.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:52:49,167] Trial 3 finished with value: 0.0015651147439447444 and parameters: {'n_estimators': 114, 'learning_rate': 0.11123619004561856, 'num_leaves': 36, 'max_depth': 9, 'min_child_samples': 27, 'subsample': 0.685963018787222, 'colsample_bytree': 0.5724567099841156, 'reg_alpha': 2.35694796262903, 'reg_lambda': 6.577932122809841, 'min_split_gain': 0.02101973107432661}. Best is trial 1 with value: 0.001283628840747059.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:52:50,525] Trial 4 finished with value: 0.001148207899829077 and parameters: {'n_estimators': 213, 'learning_rate': 0.024940935230888903, 'num_leaves': 117, 'max_depth': 8, 'min_child_samples': 2, 'subsample': 0.8588901488868512, 'colsample_bytree': 0.8313766388191326, 'reg_alpha': 9.942829898082273, 'reg_lambda': 3.756748725905885, 'min_split_gain': 0.03186748608647625}. Best is trial 4 with value: 0.001148207899829077.
[I 2025-06-15 20:52:50,527] A new study created in memory with name: no-name-84128e66-219a-48e5-bf9a-d18e94268713
[I 2025-06-15 20:52:50,726] Trial 0 finished with value: 0.004638081902186789 and parameters: {'n_estimators': 225, 'learning_rate': 0.10306713569163319, 'num_leaves': 109, 'max_depth': 16, 'min_child_samples': 29, 'subsample': 0.87361623813483, 'colsample_bytree': 0.9647060608329797, 'reg_alpha': 4.756745775324775, 'reg_lambda': 8.857581544416025, 'min_split_gain': 0.007634718169327388}. Best is trial 0 with value: 0.004638081902186789.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1502
1           Price_lag_2        1026
14      pct_chg_Load_DA         804
2           Price_lag_7         428
11           NGas_lag_2         395
16  roll2d_median_Price         367
3         Load_DA_lag_0         348
13            EUA_lag_2         285
10           Coal_lag_2         254
5       WindOn_DA_lag_0         212
12            Oil_lag_2         182
15       lag168_Load_DA         127
9                  WD_7          33
8                  WD_6          18
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          18
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 99.45% done
********************* END NS.. 726 ****************************************************
********************* START NS ... 727 ****************************************************
[I 2025-06-15 20:52:53,475] A new study created in memory with name: no-name-290abce7-8b76-4913-b9c6-de79c8d0b0c7
[I 2025-06-15 20:52:54,841] Trial 0 finished with value: 0.004490735616341582 and parameters: {'n_estimators': 200, 'learning_rate': 0.011801739104613561, 'num_leaves': 98, 'max_depth': 9, 'min_child_samples': 23, 'subsample': 0.9084641189752738, 'colsample_bytree': 0.97596499447583, 'reg_alpha': 7.444200044083269, 'reg_lambda': 7.026962209639974, 'min_split_gain': 0.007282715742400081}. Best is trial 0 with value: 0.004490735616341582.
[Trial 0] New best: 0.00449
[I 2025-06-15 20:52:56,427] Trial 1 finished with value: 0.0010283873458533535 and parameters: {'n_estimators': 143, 'learning_rate': 0.034345932634553836, 'num_leaves': 75, 'max_depth': 15, 'min_child_samples': 6, 'subsample': 0.5719372692614388, 'colsample_bytree': 0.9918309407556518, 'reg_alpha': 3.9282157572396317, 'reg_lambda': 1.1374464272452212, 'min_split_gain': 0.032465021960469576}. Best is trial 1 with value: 0.0010283873458533535.
[Trial 1] New best: 0.00103
[I 2025-06-15 20:52:57,054] Trial 2 finished with value: 0.0017512324241724796 and parameters: {'n_estimators': 154, 'learning_rate': 0.25020981961949645, 'num_leaves': 33, 'max_depth': 10, 'min_child_samples': 30, 'subsample': 0.7094765297713835, 'colsample_bytree': 0.6078360955148845, 'reg_alpha': 9.287273617010989, 'reg_lambda': 0.5538526315520487, 'min_split_gain': 0.023614221846291417}. Best is trial 1 with value: 0.0010283873458533535.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:52:58,177] Trial 3 finished with value: 0.006760555934011561 and parameters: {'n_estimators': 208, 'learning_rate': 0.010285188569133708, 'num_leaves': 228, 'max_depth': 5, 'min_child_samples': 3, 'subsample': 0.9841786536321356, 'colsample_bytree': 0.7731729201303171, 'reg_alpha': 7.478066563749062, 'reg_lambda': 9.15072954834747, 'min_split_gain': 0.008237198925781375}. Best is trial 1 with value: 0.0010283873458533535.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:52:59,130] Trial 4 finished with value: 0.001083721444123234 and parameters: {'n_estimators': 151, 'learning_rate': 0.02881238477416502, 'num_leaves': 251, 'max_depth': 6, 'min_child_samples': 11, 'subsample': 0.5286287089576038, 'colsample_bytree': 0.7100773498879172, 'reg_alpha': 7.978094918357247, 'reg_lambda': 7.6396126098499675, 'min_split_gain': 0.036125548638943544}. Best is trial 1 with value: 0.0010283873458533535.
[I 2025-06-15 20:52:59,131] A new study created in memory with name: no-name-e7323c52-e169-4431-8754-d39550a3feac
[I 2025-06-15 20:52:59,201] Trial 0 finished with value: 0.004699748883258384 and parameters: {'n_estimators': 289, 'learning_rate': 0.013919077586183406, 'num_leaves': 205, 'max_depth': 13, 'min_child_samples': 5, 'subsample': 0.9767629474568692, 'colsample_bytree': 0.6346964811236235, 'reg_alpha': 5.727378869678633, 'reg_lambda': 6.278126950849824, 'min_split_gain': 0.0294447777715317}. Best is trial 0 with value: 0.004699748883258384.
[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1557
14      pct_chg_Load_DA         947
1           Price_lag_2         813
16  roll2d_median_Price         542
3         Load_DA_lag_0         444
2           Price_lag_7         411
11           NGas_lag_2         352
13            EUA_lag_2         262
10           Coal_lag_2         206
5       WindOn_DA_lag_0         177
12            Oil_lag_2         167
15       lag168_Load_DA         134
9                  WD_7          18
8                  WD_6          15
4        Solar_DA_lag_0           0
6      WindOff_DA_lag_0           0
7                  WD_1           0
short model ************************
                feature  importance
0           Price_lag_1          13
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 99.59% done
********************* END NS.. 727 ****************************************************
********************* START NS ... 728 ****************************************************
[I 2025-06-15 20:53:02,095] A new study created in memory with name: no-name-c73c4515-1622-4cd2-95a7-bd6616634cf5
[I 2025-06-15 20:53:04,408] Trial 0 finished with value: 0.00033387759223343065 and parameters: {'n_estimators': 141, 'learning_rate': 0.04095835338122608, 'num_leaves': 241, 'max_depth': 13, 'min_child_samples': 17, 'subsample': 0.9708219220603446, 'colsample_bytree': 0.8278489236152321, 'reg_alpha': 0.7100865372519605, 'reg_lambda': 5.6784973779185846, 'min_split_gain': 0.027253375460515185}. Best is trial 0 with value: 0.00033387759223343065.
[Trial 0] New best: 0.00033
[I 2025-06-15 20:53:05,637] Trial 1 finished with value: 0.0003373397786495534 and parameters: {'n_estimators': 209, 'learning_rate': 0.037999449734394404, 'num_leaves': 45, 'max_depth': 6, 'min_child_samples': 19, 'subsample': 0.5447065362709784, 'colsample_bytree': 0.6206397763005677, 'reg_alpha': 1.5332085020699593, 'reg_lambda': 6.446030230126113, 'min_split_gain': 0.03687307048125973}. Best is trial 0 with value: 0.00033387759223343065.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:53:06,480] Trial 2 finished with value: 0.01320155642418148 and parameters: {'n_estimators': 116, 'learning_rate': 0.015390166437859301, 'num_leaves': 51, 'max_depth': 15, 'min_child_samples': 18, 'subsample': 0.9623255484867457, 'colsample_bytree': 0.5863576443342363, 'reg_alpha': 9.905505385485965, 'reg_lambda': 1.8507055538004946, 'min_split_gain': 0.0009732216212199941}. Best is trial 0 with value: 0.00033387759223343065.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:53:07,765] Trial 3 finished with value: 0.00040979250553827186 and parameters: {'n_estimators': 174, 'learning_rate': 0.026563199736126236, 'num_leaves': 194, 'max_depth': 14, 'min_child_samples': 6, 'subsample': 0.571263106508597, 'colsample_bytree': 0.957037187917213, 'reg_alpha': 8.652445389644765, 'reg_lambda': 7.345009342042982, 'min_split_gain': 0.013686933539774938}. Best is trial 0 with value: 0.00033387759223343065.
[I 2025-06-15 20:53:07,767] A new study created in memory with name: no-name-ff6fae0e-a30b-48bd-90ed-1de8a4b75ee6
[I 2025-06-15 20:53:07,807] Trial 0 finished with value: 0.00042387074778565086 and parameters: {'n_estimators': 85, 'learning_rate': 0.1441717253534991, 'num_leaves': 176, 'max_depth': 8, 'min_child_samples': 11, 'subsample': 0.6018984483942457, 'colsample_bytree': 0.5623854091358597, 'reg_alpha': 3.01219743789382, 'reg_lambda': 8.082496021698246, 'min_split_gain': 0.00889507707095783}. Best is trial 0 with value: 0.00042387074778565086.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1922
1           Price_lag_2        1832
14      pct_chg_Load_DA        1597
3         Load_DA_lag_0         989
16  roll2d_median_Price         842
11           NGas_lag_2         617
2           Price_lag_7         609
15       lag168_Load_DA         583
5       WindOn_DA_lag_0         550
13            EUA_lag_2         429
10           Coal_lag_2         362
12            Oil_lag_2         281
9                  WD_7          32
8                  WD_6          16
7                  WD_1          10
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1          18
3         Load_DA_lag_0           8
1           Price_lag_2           3
15       lag168_Load_DA           1
10           Coal_lag_2           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
8                  WD_6           0
9                  WD_7           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 99.73% done
********************* END NS.. 728 ****************************************************
********************* START NS ... 729 ****************************************************
[I 2025-06-15 20:53:11,473] A new study created in memory with name: no-name-c0c4df21-9118-4579-bf6e-6d2b97ffe23d
[I 2025-06-15 20:53:11,926] Trial 0 finished with value: 0.0004996881107968086 and parameters: {'n_estimators': 172, 'learning_rate': 0.07592157662609027, 'num_leaves': 249, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.7001899467651134, 'colsample_bytree': 0.8105799916336626, 'reg_alpha': 8.584669771677039, 'reg_lambda': 0.44477416371002554, 'min_split_gain': 0.02705110036618369}. Best is trial 0 with value: 0.0004996881107968086.
[Trial 0] New best: 0.00050
[I 2025-06-15 20:53:12,470] Trial 1 finished with value: 0.0005194381062098394 and parameters: {'n_estimators': 186, 'learning_rate': 0.23423402742545585, 'num_leaves': 229, 'max_depth': 7, 'min_child_samples': 9, 'subsample': 0.7513848251670574, 'colsample_bytree': 0.8226342055873048, 'reg_alpha': 0.17168411766712577, 'reg_lambda': 1.1278836316575513, 'min_split_gain': 0.03260183147944305}. Best is trial 0 with value: 0.0004996881107968086.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:53:13,785] Trial 2 finished with value: 0.008341066513015381 and parameters: {'n_estimators': 175, 'learning_rate': 0.011643289963819107, 'num_leaves': 95, 'max_depth': 14, 'min_child_samples': 29, 'subsample': 0.7697927835382173, 'colsample_bytree': 0.71627784017936, 'reg_alpha': 5.065527029349503, 'reg_lambda': 0.3598897171682336, 'min_split_gain': 0.037556737397164044}. Best is trial 0 with value: 0.0004996881107968086.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:53:16,386] Trial 3 finished with value: 0.0005038470353936939 and parameters: {'n_estimators': 254, 'learning_rate': 0.032223035263946105, 'num_leaves': 66, 'max_depth': 14, 'min_child_samples': 23, 'subsample': 0.631849672230929, 'colsample_bytree': 0.5217466654936862, 'reg_alpha': 1.6832254803887814, 'reg_lambda': 2.068400004274098, 'min_split_gain': 0.046797290607970804}. Best is trial 0 with value: 0.0004996881107968086.
[I 2025-06-15 20:53:16,388] A new study created in memory with name: no-name-d853fd73-b256-4d39-a34c-bc06cfe5bf0d
[I 2025-06-15 20:53:16,413] Trial 0 finished with value: 0.005139490228415407 and parameters: {'n_estimators': 64, 'learning_rate': 0.011327207729178735, 'num_leaves': 89, 'max_depth': 13, 'min_child_samples': 14, 'subsample': 0.5894472590981407, 'colsample_bytree': 0.7206676405368968, 'reg_alpha': 9.41699098514246, 'reg_lambda': 9.331287069887855, 'min_split_gain': 0.04778302218126752}. Best is trial 0 with value: 0.005139490228415407.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1         561
14      pct_chg_Load_DA         345
1           Price_lag_2         284
3         Load_DA_lag_0         199
11           NGas_lag_2         131
16  roll2d_median_Price         125
2           Price_lag_7         116
13            EUA_lag_2          99
5       WindOn_DA_lag_0          65
10           Coal_lag_2          63
12            Oil_lag_2          49
15       lag168_Load_DA          46
9                  WD_7           8
8                  WD_6           7
6      WindOff_DA_lag_0           0
7                  WD_1           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 99.86% done
********************* END NS.. 729 ****************************************************
********************* START NS ... 730 ****************************************************
[I 2025-06-15 20:53:18,432] A new study created in memory with name: no-name-c6b51e16-68cc-44dd-a97f-1e26a8bf8cc4
[I 2025-06-15 20:53:18,690] Trial 0 finished with value: 0.0005208578741275112 and parameters: {'n_estimators': 191, 'learning_rate': 0.16309030492817997, 'num_leaves': 176, 'max_depth': 7, 'min_child_samples': 12, 'subsample': 0.5651856638658846, 'colsample_bytree': 0.6356670746628785, 'reg_alpha': 9.82035937955145, 'reg_lambda': 3.271551405445611, 'min_split_gain': 0.0490153336825159}. Best is trial 0 with value: 0.0005208578741275112.
[Trial 0] New best: 0.00052
[I 2025-06-15 20:53:20,345] Trial 1 finished with value: 0.00048722515620461493 and parameters: {'n_estimators': 262, 'learning_rate': 0.05142214353508437, 'num_leaves': 152, 'max_depth': 13, 'min_child_samples': 17, 'subsample': 0.7160665792927892, 'colsample_bytree': 0.7298566606379514, 'reg_alpha': 1.3539061649440276, 'reg_lambda': 1.0033261087968304, 'min_split_gain': 0.03736468558823319}. Best is trial 1 with value: 0.00048722515620461493.
[Trial 1] No significant gain (Δ ≤ 0.001). Counter: 1/3
[I 2025-06-15 20:53:22,162] Trial 2 finished with value: 0.0005070183117213531 and parameters: {'n_estimators': 220, 'learning_rate': 0.03452705023712565, 'num_leaves': 255, 'max_depth': 15, 'min_child_samples': 25, 'subsample': 0.8739291036028545, 'colsample_bytree': 0.7714380336433886, 'reg_alpha': 6.7359022328400115, 'reg_lambda': 9.474920340275785, 'min_split_gain': 0.012517138137262152}. Best is trial 1 with value: 0.00048722515620461493.
[Trial 2] No significant gain (Δ ≤ 0.001). Counter: 2/3
[I 2025-06-15 20:53:22,415] Trial 3 finished with value: 0.05427373349684441 and parameters: {'n_estimators': 74, 'learning_rate': 0.014084685181999084, 'num_leaves': 173, 'max_depth': 4, 'min_child_samples': 21, 'subsample': 0.944831201148882, 'colsample_bytree': 0.5505115593336589, 'reg_alpha': 1.7501304368801296, 'reg_lambda': 9.941666306646482, 'min_split_gain': 0.0002351350875622771}. Best is trial 1 with value: 0.00048722515620461493.
[I 2025-06-15 20:53:22,418] A new study created in memory with name: no-name-cd4f1b8c-694a-42c1-89f3-e3f318e160c5
[I 2025-06-15 20:53:22,605] Trial 0 finished with value: 0.002967419023413019 and parameters: {'n_estimators': 219, 'learning_rate': 0.04816942769977133, 'num_leaves': 149, 'max_depth': 15, 'min_child_samples': 27, 'subsample': 0.8203305363533083, 'colsample_bytree': 0.511176047187284, 'reg_alpha': 9.80821797058889, 'reg_lambda': 4.300365057123337, 'min_split_gain': 0.023599644476122946}. Best is trial 0 with value: 0.002967419023413019.
[Trial 3] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
                feature  importance
0           Price_lag_1        1296
1           Price_lag_2        1166
14      pct_chg_Load_DA        1046
3         Load_DA_lag_0         659
5       WindOn_DA_lag_0         462
16  roll2d_median_Price         455
11           NGas_lag_2         415
2           Price_lag_7         371
15       lag168_Load_DA         351
13            EUA_lag_2         313
10           Coal_lag_2         209
12            Oil_lag_2         172
9                  WD_7          33
8                  WD_6          12
7                  WD_1           8
6      WindOff_DA_lag_0           0
4        Solar_DA_lag_0           0
short model ************************
                feature  importance
0           Price_lag_1           0
9                  WD_7           0
15       lag168_Load_DA           0
14      pct_chg_Load_DA           0
13            EUA_lag_2           0
12            Oil_lag_2           0
11           NGas_lag_2           0
10           Coal_lag_2           0
8                  WD_6           0
1           Price_lag_2           0
7                  WD_1           0
6      WindOff_DA_lag_0           0
5       WindOn_DA_lag_0           0
4        Solar_DA_lag_0           0
3         Load_DA_lag_0           0
2           Price_lag_7           0
16  roll2d_median_Price           0
-> 100.00% done
********************* END NS.. 730 ****************************************************
Zone NO1 LBGM-24h RMSE: 9.0451